# World Data League 2022 - Stage 2

## 🎯 Challenge
*Challenge 2 - Predicting the flow of people for public transportation improvements*

## Team: Data Crocodiles
## 👥 Authors
* Diogo Pessoa
* Fábio Lopes
* Francisco Valente
* Júlio Medeiros

## 💻 Development

Development section is divided in the following subsections:
- Requirements
- Import Packages
- Custom Functions
- Data Loading and Data Preprocessing
- Exploratory Data Analysis
- Prepare data to develop forecasting models
- Development of forecasting models
- Evaluating and understanding the predictions obtained from the forecasting models

### Requirements

In [1]:
!pip install openpyxl
!pip install holidays
!pip install suntime
!pip install folium
!pip install geopandas
!pip install plotly
!pip install shapely
!pip install scikit-learn
!pip install shap

### Import Packages

In [2]:
# Import Libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import requests
import zipfile
import io
import os
import datetime
import shap
import holidays
from suntime import Sun
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
import folium
import folium.plugins as plugins
import geopandas as gpd
import json
import requests
import time
from folium.plugins import HeatMapWithTime
import plotly.graph_objects as go
import plotly.express as px
import plotly
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from shapely.geometry import Polygon
from IPython.core.display import display, HTML

### Custom Functions

In [3]:
def split_dataset(dataset,split_datetime):
    '''Function to split the dataset in training and test sets.
    Input: 
        dataset: Data that will be splited.
        split_datetime: Datetime when the datat will be split.
    Output:
        train_dataset: Training set.
        test_dataset: Test set.
    '''
    split_month = split_datetime.month
    split_day = split_datetime.day
    
    train_dataset_first = dataset.loc[(dataset['Mês']<split_month)]
    train_dataset_second = dataset.loc[(dataset['Mês']==split_month)]
    train_dataset_second = train_dataset_second.loc[(train_dataset_second['Dia']<split_day)]
    train_dataset = pd.concat([train_dataset_first,train_dataset_second])
    
    test_dataset_first = dataset.loc[(dataset['Mês']>split_month)]
    test_dataset_second = dataset.loc[(dataset['Mês']==split_month)]
    test_dataset_second = test_dataset_second.loc[(test_dataset_second['Dia']>=split_day)]
    test_dataset = pd.concat([test_dataset_first,test_dataset_second])
    
    return train_dataset,test_dataset

def one_hot_encoding_zones(zones,all_zones):
    '''Function to convert the name of the zones to binary.
    Input:
        zones: List of zones contained in the dataset.
        all_zones: All zones available.
    Output:
        all_zones_dataframe: Dataframe with a binary vector for each zone according to the dataset.
    '''
    
    # Create datasets to train the models
    number_samples = zones.shape[0]
    all_zones_dict = {zone:np.zeros((number_samples,)) for zone in all_zones}
    
    for index,zone in enumerate(zones):
        if zone in all_zones_dict.keys():
            all_zones_dict[zone][index] = 1
    
    all_zones_dataframe = pd.DataFrame(all_zones_dict)
    
    return all_zones_dataframe

def split_covid_periods(dataset,covid_first_datetime,covid_last_datetime):
    ''' Split the dataset according to the COVID periods
    Inputs:
        dataset: Dataset
        covid_first_datetime: First datetime of the COVID lockdown
        covid_last_datetime: Last datetime of the COVID lockdown
    Outputs:
        pre_covid_set: Data before COVID lockdown
        covid_set: Data during COVID lockdown
        post_covid_set: Data after COVID lockdown
    '''
    
    dataset_columns = dataset.columns.to_list()
    dataset = dataset.values
    
    pre_covid_set = []
    covid_set = []
    post_covid_set = []
    for index,element in enumerate(dataset):
        year = int(element[0])
        month = int(element[1])
        day = int(element[2])
        hour = int(element[3])
        
        element_datetime = datetime.datetime(year,month,day,hour)
        
        if element_datetime < covid_first_datetime:
            pre_covid_set.append(element)
        else:
            if element_datetime <= covid_last_datetime:
                covid_set.append(element)
            else:
                post_covid_set.append(element)
    
    pre_covid_set = pd.DataFrame(pre_covid_set,columns=dataset_columns)
    covid_set = pd.DataFrame(covid_set,columns=dataset_columns)
    post_covid_set = pd.DataFrame(post_covid_set,columns=dataset_columns)
    
    return pre_covid_set,covid_set,post_covid_set

'''Verify whether data is available and if not, download it.'''
def download_data(url,folder_name):
    if os.path.isdir(folder_name)==False:
        print("Downloading Data...")
        r = requests.get(url, stream=True)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall()
        print("Data successfully downloaded!")
    else:
        print("Data was already downloaded.")
    
    return None
    
'''Listdir method returning full path filenames. Imported from 
https://stackoverflow.com/questions/120656/directory-tree-listing-in-python'''
def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

'''Get folders which are inside the dataset folder'''
def get_dataset_subfolders(dataset_folder_path):
    dataset_subfolder_paths = listdir_fullpath(dataset_folder_path)
    dataset_subfolder_paths = sorted(dataset_subfolder_paths)
    filtered_dataset_subfolder_paths = []
    
    for dataset_subfolder_path in dataset_subfolder_paths:
        if 'DS_Store' not in dataset_subfolder_path:
            filtered_dataset_subfolder_paths.append(dataset_subfolder_path)
    
    return filtered_dataset_subfolder_paths

'''Merge data inside the subfolders'''
def merging_data(subfolder_paths,subset_names):
    # Dictionary with the subsets
    subsets_data = {subset_name:[] for subset_name in subset_names}
    
    for subfolder_index,subfolder_path in enumerate(subfolder_paths):
        # Get subset filenames
        subsets_filepaths = listdir_fullpath(subfolder_path)
        print(f"Part {subfolder_index+1} from {len(subfolder_paths)}")
        # Get the data and save that at the proper place
        for subset_filepath in subsets_filepaths:
            for subset_name in subset_names:
                if subset_name in subset_filepath:
                    # Rodov Dataset is splited into two pages
                    if subset_name=='Rodov':
                        first_data = pd.read_excel(subset_filepath,sheet_name='1-15')
                        second_data = pd.read_excel(subset_filepath,sheet_name='16-30')
                        data = pd.concat([first_data,second_data], ignore_index=True)
                    else:
                        data = pd.read_excel(subset_filepath)
                        
                    # Save the data into the dictionary
                    if isinstance(subsets_data[subset_name],list):
                        subsets_data[subset_name] = data
                    else:
                        subsets_data[subset_name] = pd.concat([subsets_data[subset_name],data], ignore_index=True)
    
    return subsets_data[subset_names[0]],subsets_data[subset_names[1]],subsets_data[subset_names[2]]

# Extract features from the dates
def get_date_features(dataframe):
    # Get number of samples
    number_samples = dataframe.shape[0]
    
    # Get school time
    first_delta = datetime.datetime(2020,2,21) - datetime.datetime(2020,1,4)
    second_delta = datetime.datetime(2020,3,15) - datetime.datetime(2020,2,27)
    third_delta = datetime.datetime(2020,12,18) - datetime.datetime(2020,9,14)
    
    all_school_datetimes = []
    all_school_datetimes.append(datetime.datetime(2020,1,4))
    for i in range(first_delta.days + 1):
        day = datetime.datetime(2020,2,21) + datetime.timedelta(days=i)
        all_school_datetimes.append(day)
    
    all_school_datetimes.append(datetime.datetime(2020,2,27))
    for i in range(second_delta.days + 1):
        day = datetime.datetime(2020,2,27) + datetime.timedelta(days=i)
        all_school_datetimes.append(day)
    
    all_school_datetimes.append(datetime.datetime(2020,9,14))
    for i in range(third_delta.days + 1):
        day = datetime.datetime(2020,9,14) + datetime.timedelta(days=i)
        all_school_datetimes.append(day)
        
    school_time_binary = np.zeros((number_samples,))
    
    # Get Portugal (PT) holidays
    dataframe_holidays = np.zeros((number_samples,))
    portugal_holidays = holidays.Portugal()
    
    # Get weekday name for each index returned by weekday function provided by pandas library
    dataframe_weekdays = np.zeros((number_samples,))
    dataframe_weekdays_binary = np.zeros((number_samples,7))
    
    # Get day stages (morning, afternoon, night)
    dataframe_day_stage = np.zeros((number_samples,))
    dataframe_day_stage_binary = np.zeros((number_samples,3))
    sun = Sun(41.14961,-8.61099)
    
    # Verify if date is a holiday in Portugal
    # Get the day of the week
    # Get sunrise and sunset times
    for index,year in enumerate(dataframe['Ano'].values):
        
        # Get month, day, and hour
        month = dataframe['Mês'].values[index]
        day = dataframe['Dia'].values[index]
        hour = dataframe['Hora'].values[index]
        
        date = datetime.date(year,month,day)
        
        # Verify if the date is a holiday in Portugal
        if date in portugal_holidays:
            dataframe_holidays[index] = 1
        else:
            dataframe_holidays[index] = 0
        
        # Get the day of the week
        date_weekday = date.weekday()
        dataframe_weekdays[index] = date_weekday
        dataframe_weekdays_binary[index,date_weekday] = 1
        
        # Get sunrise and sunset times (These values come with UTC time zone. No correction needed.)
        sunrise_time = sun.get_sunrise_time(date)
        sunrise_hour = sunrise_time.hour
        sunrise_minute = sunrise_time.minute
        sunset_time = sun.get_sunset_time(date)
        sunset_hour = sunset_time.hour
        sunset_minute = sunset_time.minute
        
        # 11 UTC is 12 CET
        if hour<11 and hour>sunrise_hour:
            day_stage = 0 # Morning
                
        elif hour>=11 and hour<=sunset_hour:
            day_stage = 1 # Afternoon
                
        else:
            day_stage = 2 # Night
            
        dataframe_day_stage[index] = day_stage
        dataframe_day_stage_binary[index,day_stage] = 1
        
        if date in all_school_datetimes:
            school_time_binary[index] = 1
        
    # Store both information in the main dataset
    dataframe['Holiday'] = dataframe_holidays
    dataframe['Weekday'] = dataframe_weekdays
    dataframe['Monday'] = dataframe_weekdays_binary[:,0]
    dataframe['Tuesday'] = dataframe_weekdays_binary[:,1]
    dataframe['Wednesday'] = dataframe_weekdays_binary[:,2]
    dataframe['Thursday'] = dataframe_weekdays_binary[:,3]
    dataframe['Friday'] = dataframe_weekdays_binary[:,4]
    dataframe['Saturday'] = dataframe_weekdays_binary[:,5]
    dataframe['Sunday'] = dataframe_weekdays_binary[:,6]
    dataframe['Day Stage'] = dataframe_day_stage
    dataframe['Morning'] = dataframe_day_stage_binary[:,0]
    dataframe['Afternoon'] = dataframe_day_stage_binary[:,1]
    dataframe['Night'] = dataframe_day_stage_binary[:,2]
    dataframe['School'] = school_time_binary
    
    return dataframe                                                                       

def plot_feature_importances(model, dataframe, max_display):
    '''Function to plot feature imporances using Shapley Values
    Input:
        model: Prediction Model
        dataframe: Data
        max_display: Number of shown features importances
    '''
    explainer = shap.Explainer(model.predict, dataframe)
    shap_values = explainer(dataframe)
    shap.plots.beeswarm(shap_values, max_display=max_display)

def rrmse_metric(y_true,y_pred):
    '''Function to compute the Relative Root Mean Squared Error for a set of values.
    Input:
        y_true: Expected values
        y_pred: Predicted values
    Output:
        rrmse_value: RRMSE value
    '''
    rrmse_value = np.round(np.sqrt(mean_squared_error(y_true,y_pred))/np.mean(y_true)*100,2)
    return rrmse_value


def get_entry_exit_sets(mod_set):
    
    # update zones names
    mod_set= mod_set.replace('C1', 'PRT1')
    mod_set= mod_set.replace('C2', 'PRT2')
    mod_set= mod_set.replace('C6', 'PRT3')

    # PRT to non-PRT

    # select where entry is Porto
    PRT_entry_all = mod_set[mod_set['ZonaEntrada'].str.contains('PRT')]
    # remove where exit is Porto
    PRT_entry =  PRT_entry_all[PRT_entry_all['ZonaSaida'].str.contains('PRT')==False]
    
    #-----
    
    # non PRT to PRT

    # selecionar todas as saidas no porto
    PRT_exit_all = mod_set[mod_set['ZonaSaida'].str.contains('PRT')]
    # remove where entry is Porto
    PRT_exit =  PRT_exit_all[PRT_exit_all['ZonaEntrada'].str.contains('PRT')==False]
    
    #-----
    
    # PRT to PRT

    # select where exit is Porto
    PRT_in_all = mod_set[mod_set['ZonaSaida'].str.contains('PRT')]
    # select where entry is Porto
    PRT_in =  PRT_in_all[PRT_in_all['ZonaEntrada'].str.contains('PRT')]

    return PRT_entry, PRT_exit, PRT_in


def plot_heat_maps(PRT_entry, PRT_exit, metric, period):

    weekdays_dict = { 0: "Mon", 1: "Tue", 2: "Wed", 3: "Thurs", 4: "Frid", 5: "Sat", 6: "Sund"}
    num_elements = len(weekdays_dict)
    current_dict = weekdays_dict
    weekdays_names = ["Mon", "Tue", "Wed", "Thurs","Frid", "Sat", "Sund"]

    def get_hour_averages(num_elements, data, metric):
    
        all_items = []
        for i in range(0, num_elements):
            validations = data.loc[(data['Weekday'] == i)].dropna()
            if (metric == 'mean'):
                data_item = validations.groupby(['Hora']).mean()
            elif (metric == 'median'):
                data_item = validations.groupby(['Hora']).median()

            item_val = data_item['Validações'].values
            all_items.append(item_val)

        data_to_heatmap = np.array(all_items)
    
        return data_to_heatmap
    
    PRT_exit_averages = get_hour_averages(num_elements, PRT_exit, metric)
    PRT_entry_averages = get_hour_averages(num_elements, PRT_entry, metric)

    # PLOT HEAT MAPS
    
    fig, axes = plt.subplots(1,2,figsize=(10,4))
    fig.suptitle('Mean validations per Hour - Period: ' + period)
    
    sns.heatmap(PRT_exit_averages, linewidth=0.5,  ax=axes[0], yticklabels=weekdays_names)
    sns.heatmap(PRT_entry_averages, linewidth=0.5,  ax=axes[1], yticklabels=weekdays_names)
    
    axes[0].set_xlabel('Hours')
    axes[0].set_ylabel('Days of week')
    axes[0].set_title('From outside Porto to Porto')
    axes[1].set_xlabel('Hours')
    axes[1].set_ylabel('Days of week')
    axes[1].set_title('From Porto to outside Porto')
    plt.show()
    
    
def get_max_zones(PRT_entry, PRT_exit):
    
    all_exit_zones = PRT_exit['ZonaEntrada'].unique()
    all_entry_zones = PRT_entry['ZonaSaida'].unique()

    exit_zones_valid = []
    for zone in all_exit_zones:
        PRT_exit_zone = PRT_exit[PRT_exit['ZonaEntrada'] == zone]
        numb_validations = PRT_exit_zone['Validações'].sum()
        exit_zones_valid.append(numb_validations)

    entry_zones_valid = []
    for zone in all_entry_zones:
        PRT_entry_zone = PRT_entry[PRT_entry['ZonaSaida'] == zone]
        numb_validations = PRT_entry_zone['Validações'].sum()
        entry_zones_valid.append(numb_validations)

    exit_zones_valid = np.array(exit_zones_valid)
    max_idx_exit = exit_zones_valid.argsort()[-10:][::-1]
    max_exit_zones = all_exit_zones[max_idx_exit]
    
    entry_zones_valid = np.array(entry_zones_valid)
    max_idx_entry = entry_zones_valid.argsort()[-10:][::-1]
    max_entry_zones = all_entry_zones[max_idx_entry]
    
    return max_entry_zones, max_exit_zones

def plot_trasnportsVSvalidations_rates(PRT_entry, PRT_exit, period):
    
    weekday_exit= PRT_exit[PRT_exit['Weekday'].isin([0,1,2,3,4])]
    weekend_exit = PRT_exit[PRT_exit['Weekday'].isin([5,6])]
    weekday_entry= PRT_entry[PRT_entry['Weekday'].isin([0,1,2,3,4])]
    weekend_entry = PRT_entry[PRT_entry['Weekday'].isin([5,6])]


    def get_hour_rates(data):
        
        data_item = data.groupby(['Hora']).count()
        number_transports_per_hour = data_item['Operador'].values
        transports_per_hour = number_transports_per_hour/np.sum(number_transports_per_hour)

        data_item = data.groupby(['Hora']).sum()
        number_validations_per_hour = data_item['Validações'].values
        validations_per_hour = number_validations_per_hour/np.sum(number_validations_per_hour)
        
        return transports_per_hour, validations_per_hour
        
    weekday_exit_transports, weekday_exit_validations =  get_hour_rates(weekday_exit)   
    weekend_exit_transports, weekend_exit_validations =  get_hour_rates(weekend_exit)   
    weekday_entry_transports, weekday_entry_validations =  get_hour_rates(weekday_entry)   
    weekend_entry_transports, weekend_entry_validations =  get_hour_rates(weekend_entry)   
    
    
    # PLOT RATES
    
    fig, axes = plt.subplots(1,4,figsize=(20,4))
    fig.suptitle('Validations versus Transports - Period: ' + period)
    
    axes[0].plot(weekday_exit_transports, label = 'rates of transports')
    axes[0].plot(weekday_exit_validations, label = 'rates of validations')
    axes[1].plot(weekend_exit_transports, label = 'rates of transports')
    axes[1].plot(weekend_exit_validations, label = 'rates of validations')
    axes[2].plot(weekday_entry_transports, label = 'rates of transports')
    axes[2].plot(weekday_entry_validations, label = 'rates of validations')
    axes[3].plot(weekend_entry_transports, label = 'rates of transports')
    axes[3].plot(weekend_entry_validations, label = 'rates of validations')
    
    axes[0].set_xlabel('Hours')
    axes[0].set_title('From outside Porto to Porto - Weekdays')
    axes[1].set_xlabel('Hours')
    axes[1].set_title('From outside Porto to Porto - Weekends')
    axes[2].set_xlabel('Hours')
    axes[2].set_title('From Porto to outside Porto - Weekdays')
    axes[3].set_xlabel('Hours')
    axes[3].set_title('From Porto to outside Porto - Weekends')
    
    # Defining A custom 'xlim' and 'ylim' to all subplots.
    custom_ylim = (0, 0.18)
    custom_xlim = (0, 23)
    plt.setp(axes, xlim=custom_xlim, ylim=custom_ylim)
    plt.legend(title="Legend", bbox_to_anchor=(1.05, 1.0), loc='upper left')
    
    #plt.legend()
    plt.show()
    
    
def plot_trasnportsVSvalidations_rates_zone(PRT_entry, PRT_exit, zone):
    
    weekday_exit= PRT_exit[ (PRT_exit['Weekday'].isin([0,1,2,3,4])) & (PRT_exit['ZonaEntrada'] == zone) ]
    weekend_exit = PRT_exit[ (PRT_exit['Weekday'].isin([5,6])) & (PRT_exit['ZonaEntrada'] == zone) ]
    weekday_entry= PRT_entry[ (PRT_entry['Weekday'].isin([0,1,2,3,4])) & (PRT_entry['ZonaSaida'] == zone) ]
    weekend_entry = PRT_entry[ (PRT_entry['Weekday'].isin([5,6])) & (PRT_entry['ZonaSaida'] == zone) ]
    
    
    def get_hour_rates(data):
        
        data_item = data.groupby(['Hora']).count()
        number_transports_per_hour = data_item['Operador'].values
        transports_per_hour = number_transports_per_hour/np.sum(number_transports_per_hour)

        data_item = data.groupby(['Hora']).sum()
        number_validations_per_hour = data_item['Validações'].values
        validations_per_hour = number_validations_per_hour/np.sum(number_validations_per_hour)
        
        return transports_per_hour, validations_per_hour
        
    weekday_exit_transports, weekday_exit_validations =  get_hour_rates(weekday_exit)   
    weekend_exit_transports, weekend_exit_validations =  get_hour_rates(weekend_exit)   
    weekday_entry_transports, weekday_entry_validations =  get_hour_rates(weekday_entry)   
    weekend_entry_transports, weekend_entry_validations =  get_hour_rates(weekend_entry)   
    
    
    # PLOT RATES
    
    fig, axes = plt.subplots(1,4,figsize=(20,4))
    fig.suptitle('Validations versus Transports - Zone: ' + zone)
    
    axes[0].plot(weekday_exit_transports, label = 'rates of transports')
    axes[0].plot(weekday_exit_validations, label = 'rates of validations')
    axes[1].plot(weekend_exit_transports, label = 'rates of transports')
    axes[1].plot(weekend_exit_validations, label = 'rates of validations')
    axes[2].plot(weekday_entry_transports, label = 'rates of transports')
    axes[2].plot(weekday_entry_validations, label = 'rates of validations')
    axes[3].plot(weekend_entry_transports, label = 'rates of transports')
    axes[3].plot(weekend_entry_validations, label = 'rates of validations')
    
    axes[0].set_xlabel('Hours')
    axes[0].set_title('From outside Porto to Porto - Weekdays')
    axes[1].set_xlabel('Hours')
    axes[1].set_title('From outside Porto to Porto - Weekends')
    axes[2].set_xlabel('Hours')
    axes[2].set_title('From Porto to outside Porto - Weekdays')
    axes[3].set_xlabel('Hours')
    axes[3].set_title('From Porto to outside Porto - Weekends')
    
    # Defining A custom 'xlim' and 'ylim' to all subplots.
    custom_ylim = (0, 0.27)
    custom_xlim = (0, 23)
    plt.setp(axes, xlim=custom_xlim, ylim=custom_ylim)
    plt.legend(title="Legend", bbox_to_anchor=(1.05, 1.0), loc='upper left')
    
    #plt.legend()
    plt.show()
    

def get_operadores(PRT_entry, PRT_exit, zone):

    zone_exit = PRT_exit[PRT_exit['ZonaEntrada'] == zone]
    operadores_exit = zone_exit['Operador'].unique()
    zone_entry = PRT_entry[PRT_entry['ZonaSaida'] == zone]
    operadores_entry = zone_entry['Operador'].unique()
    operadores = np.unique(np.hstack([operadores_exit, operadores_entry]))
    
    return operadores


def plot_heat_maps_in(PRT_in, metric, period):

    weekdays_dict = { 0: "Mon", 1: "Tue", 2: "Wed", 3: "Thurs", 4: "Frid", 5: "Sat", 6: "Sund"}
    weekdays_names = ["Mon", "Tue", "Wed", "Thurs","Frid", "Sat", "Sund"]

    num_elements = len(weekdays_dict)
    current_dict = weekdays_dict
    
    def get_hour_averages(num_elements, data, metric):
    
        all_items = []
        for i in range(0, num_elements):
            validations = data.loc[(data['Weekday'] == i)].dropna()
            if (metric == 'mean'):
                data_item = validations.groupby(['Hora']).mean()
            elif (metric == 'median'):
                data_item = validations.groupby(['Hora']).median()

            item_val = data_item['Validações'].values
            all_items.append(item_val)

        data_to_heatmap = np.array(all_items)
    
        return data_to_heatmap
    
    PRT_in_averages = get_hour_averages(num_elements, PRT_in, metric)

    # PLOT HEAT MAPS
    
    fig = plt.figure(figsize=(10,4))
    plt.title('Mean validations per Hour - From Porto to Porto - Period: ' + period)
    
    sns.heatmap(PRT_in_averages, linewidth=0.5, yticklabels=weekdays_names)
    
    plt.xlabel('Hours')
    plt.ylabel('Days of week')
    # plt.title('From Porto to Porto')
    plt.show()
    
    
def plot_trasnportsVSvalidations_rates_in(PRT_in, period):
    
    weekday_in = PRT_in[PRT_in['Weekday'].isin([0,1,2,3,4])]
    weekend_in = PRT_in[PRT_in['Weekday'].isin([5,6])]

    def get_hour_rates(data):
        
        data_item = data.groupby(['Hora']).count()
        number_transports_per_hour = data_item['Operador'].values
        transports_per_hour = number_transports_per_hour/np.sum(number_transports_per_hour)

        data_item = data.groupby(['Hora']).sum()
        number_validations_per_hour = data_item['Validações'].values
        validations_per_hour = number_validations_per_hour/np.sum(number_validations_per_hour)
        
        return transports_per_hour, validations_per_hour
        
    weekday_in_transports, weekday_in_validations =  get_hour_rates(weekday_in)   
    weekend_in_transports, weekend_in_validations =  get_hour_rates(weekend_in)   
 
    # PLOT RATES
    
    fig, axes = plt.subplots(1,2,figsize=(10,4))
    fig.suptitle('Validations versus Transports - Period: ' + period)
    
    axes[0].plot(weekday_in_transports, label = 'rates of transports')
    axes[0].plot(weekday_in_validations, label = 'rates of validations')
    axes[1].plot(weekend_in_transports, label = 'rates of transports')
    axes[1].plot(weekend_in_validations, label = 'rates of validations')
    
    axes[0].set_xlabel('Hours')
    axes[0].set_title('From Porto to Porto - Weekdays')
    axes[1].set_xlabel('Hours')
    axes[1].set_title('From Porto to Porto - Weekends')
    
    # Defining A custom 'xlim' and 'ylim' to all subplots.
    custom_ylim = (0, 0.18)
    custom_xlim = (0, 23)
    plt.setp(axes, xlim=custom_xlim, ylim=custom_ylim)
    plt.legend(title="Legend", bbox_to_anchor=(1.05, 1.0), loc='upper left')
    
    #plt.legend()
    plt.show()
    
# create dataframes for prediction output maps
def create_Dataframe(test, predicted, data_zones):
    test = test.iloc[:,16:].reset_index(drop=True)

    new_dataframe = pd.DataFrame()
    zones_ids = []
    for i in range(test.shape[0]):
        try:
          zones_ids.append(test.columns[np.argwhere(np.asarray(test.iloc[[i]]) == 1)[0][1]])
        except:
          zones_ids.append('UNKNOWN')
        
    new_dataframe['zone_id'] = zones_ids
    new_dataframe['Validações'] = predicted

    new_dataframe_merged = pd.merge(new_dataframe, data_zones, on = "zone_id")
    #new_dataframe_merged = new_dataframe_merged.drop(['points', 'coordinates', 'geometry'], axis=1)
    new_dataframe_merged['ValidaçõesNormalized'] = new_dataframe_merged.apply(lambda l: (l['Validações'] - new_dataframe_merged['Validações'].min()) / ( new_dataframe_merged['Validações'].max() - new_dataframe_merged['Validações'].min() ), axis=1)

    return new_dataframe_merged

### Data Loading and Data Preprocessing

#### TIP Dataset

In [4]:
if os.path.isfile('Ferrov_Dataset.csv'):
    print("Loading Datasets...")
    mod_set = pd.read_csv('MOD_Dataset.csv')
    rodov_set = pd.read_csv('Rodov_Dataset.csv')
    ferrov_set = pd.read_csv('Ferrov_Dataset.csv')
else:
    # Downloading Data
    zip_file_url = "https://wdl-data.fra1.digitaloceanspaces.com/porto/InterMunicipality.zip"
    folder_name = "InterMunicipality"
    download_data(zip_file_url,folder_name)

    # Accessing Data
    dataset_name = "TIP"
    dataset_folder_path = f"{folder_name}/{dataset_name}/"
    dataset_subfolder_paths = get_dataset_subfolders(dataset_folder_path)
    
    # Merging Data
    print("Merging Datasets...")
    subset_names = ['MOD','Rodov','Ferrov']
    mod_set,rodov_set,ferrov_set = merging_data(dataset_subfolder_paths,subset_names)

    # Get date info (Day of the week, day stage, holiday, etc.)
    print("Getting Date Infomation...")
    mod_set = get_date_features(mod_set)
    rodov_set = get_date_features(rodov_set)
    ferrov_set = get_date_features(ferrov_set)
    
    # MOD Set
    mod_set_obj = mod_set.select_dtypes(['object'])
    mod_set[mod_set_obj.columns] = mod_set_obj.apply(lambda x: x.str.strip())

    mod_set['Operador'][mod_set['Operador']==''] = 'Unknown'
    mod_set['ZonaEntrada'][mod_set['ZonaEntrada']==''] = 'Unknown'
    mod_set['ZonaSaida'][mod_set['ZonaSaida']==''] = 'Unknown'

    mod_set['Operador'] = mod_set['Operador'].fillna('Unknown')
    mod_set['ZonaEntrada'] = mod_set['ZonaEntrada'].fillna('Unknown')
    mod_set['ZonaSaida'] = mod_set['ZonaSaida'].fillna('Unknown')

    # Rodov Set
    rodov_set_obj = rodov_set.select_dtypes(['object'])
    rodov_set[['Operador','Paragem','Zona']] = rodov_set_obj[['Operador','Paragem','Zona']].apply(lambda x: x.str.strip())

    rodov_set['Operador'][rodov_set['Operador']==''] = 'Unknown'
    rodov_set['Linha_ao_Publico'][rodov_set['Linha_ao_Publico']==''] = 'Unknown'
    rodov_set['Paragem'][rodov_set['Paragem']==''] = 'Unknown'
    rodov_set['Zona'][rodov_set['Zona']==''] = 'Unknown'

    rodov_set['Operador'] = rodov_set['Operador'].fillna('Unknown')
    rodov_set['Linha_ao_Publico'] = rodov_set['Linha_ao_Publico'].fillna('Unknown')
    rodov_set['Paragem'] = rodov_set['Paragem'].fillna('Unknown')
    rodov_set['Zona'] = rodov_set['Zona'].fillna('Unknown')
    
    # There are some samples with errors
    rodov_set = rodov_set[rodov_set.Validações.apply(lambda x: str(x).isnumeric())]

    # Ferrov Set
    ferrov_set_obj = ferrov_set.select_dtypes(['object'])
    ferrov_set[ferrov_set_obj.columns] = ferrov_set_obj.apply(lambda x: x.str.strip())

    ferrov_set['Operador'][ferrov_set['Operador']==''] = 'Unknown'
    ferrov_set['Est/Op'][ferrov_set['Est/Op']==''] = 'Unknown'
    ferrov_set['Zona'][ferrov_set['Zona']==''] = 'Unknown'

    ferrov_set['Operador'] = ferrov_set['Operador'].fillna('Unknown')
    ferrov_set['Est/Op'] = ferrov_set['Est/Op'].fillna('Unknown')
    ferrov_set['Zona'] = ferrov_set['Zona'].fillna('Unknown')

    # Saving Data
    mod_set.to_csv('MOD_Dataset.csv',index=False)
    rodov_set.to_csv('Rodov_Dataset.csv',index=False)
    ferrov_set.to_csv('Ferrov_Dataset.csv',index=False)

Data was already downloaded.
Merging Datasets...
Part 1 from 12
Part 2 from 12
Part 3 from 12
Part 4 from 12
Part 5 from 12
Part 6 from 12
Part 7 from 12
Part 8 from 12
Part 9 from 12
Part 10 from 12


MemoryError: Unable to allocate 166. MiB for an array with shape (2, 10911242) and data type object

As features we extracted the Portuguese holidays, weekdays (monday - sunday), day stage (morning, afternoon, and night), and school time.

#### MOD Set

In [ ]:
mod_set.head()

#### Rodov Set

In [ ]:
rodov_set.head()

#### Ferrov Set

In [ ]:
ferrov_set.head()

#### GTFS Dataset

In [ ]:
if os.path.isdir('gtfs_bus')==False:
    # Downloading Data
    zip_file_url = "https://wdl-data.fra1.digitaloceanspaces.com/porto/InterMunicipality.zip"
    folder_name = "InterMunicipality"
    download_data(zip_file_url,folder_name)

    # Accessing Data
    dataset_name = "GTFS"
    dataset_folder_path = f"{folder_name}/{dataset_name}/"
    gtfs_bus_file_path = f"{folder_name}/{dataset_name}/gtfs_bus.zip"
    gtfs_metro_file_path = f"{folder_name}/{dataset_name}/gtfs_metro.zip"
    
    with zipfile.ZipFile(gtfs_bus_file_path, 'r') as zip_ref:
        zip_ref.extractall("gtfs_bus")
    
    with zipfile.ZipFile(gtfs_metro_file_path, 'r') as zip_ref:
        zip_ref.extractall("gtfs_metro")

# GTFS Bus
agency_gtfs_bus = pd.read_csv('gtfs_bus/agency.txt')
calendar_dates_gtfs_bus = pd.read_csv('gtfs_bus/calendar_dates.txt')
calendar_gtfs_bus = pd.read_csv('gtfs_bus/calendar.txt')
routes_bus = pd.read_csv('gtfs_bus/routes.txt')
shapes_gtfs_bus = pd.read_csv('gtfs_bus/shapes.txt')
stop_times_gtfs_bus = pd.read_csv('gtfs_bus/stop_times.txt')
stops_gtfs_bus = pd.read_csv('gtfs_bus/stops.txt')
transfers_gtfs_bus = pd.read_csv('gtfs_bus/transfers.txt')
trips_gtfs_bus = pd.read_csv('gtfs_bus/trips.txt')

gtfs_bus_set = pd.concat([agency_gtfs_bus,calendar_dates_gtfs_bus,calendar_gtfs_bus,
                          routes_bus,shapes_gtfs_bus,stop_times_gtfs_bus,stops_gtfs_bus,
                          transfers_gtfs_bus,trips_gtfs_bus],axis=1)

# GTFS Metro
agency_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/agency.txt')
calendar_dates_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/calendar_dates.txt')
calendar_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/calendar.txt')
fare_attributes_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/fare_attributes.txt')
fare_rules_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/fare_rules.txt')
frequencies_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/frequencies.txt')
routes_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/routes.txt')
shapes_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/shapes.txt')
stop_times_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/stop_times.txt')
stops_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/stops.txt')
transfers_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/transfers.txt')
trips_gtfs_metro = pd.read_csv('gtfs_metro/google_transit_dez2021/trips.txt')

gtfs_metro_set = pd.concat([agency_gtfs_metro,calendar_dates_gtfs_metro,calendar_gtfs_metro,
                            fare_attributes_gtfs_metro,fare_rules_gtfs_metro,frequencies_gtfs_metro,
                            routes_metro,shapes_gtfs_metro,stop_times_gtfs_metro,stops_gtfs_metro,
                            transfers_gtfs_metro,trips_gtfs_metro],axis=1)

#### GTFS Bus Set

In [ ]:
gtfs_bus_set

### GTFS Metro Set

In [ ]:
gtfs_metro_set

### Exploratory Data Analysis

#### Connections per Zone (Bus)

In [ ]:
# Get bus routes names
bus_routes = gtfs_bus_set[['route_id','route_long_name']].iloc[:,[0,2]]
bus_routes = bus_routes.dropna()
display(bus_routes)

# Filter Bus Set (Only keep bus routes that appear in routes.txt)
rodov_set = rodov_set[rodov_set['Linha_ao_Publico'].isin(bus_routes['route_id'])]
display(rodov_set)

#### Connections per Zone (Train)

In [ ]:
# Get train routes names
train_routes = gtfs_metro_set[['route_id','route_long_name']].iloc[:,[1,3]]
train_routes = train_routes.dropna()
display(train_routes)

# Get trip head sign per route
train_headsign_zones = gtfs_metro_set[['route_id','trip_headsign']].iloc[:,2:4]
train_headsign_zones = train_headsign_zones.dropna()
train_headsign_zones = train_headsign_zones.sort_values('route_id')
train_headsign_zones = train_headsign_zones.groupby('route_id').apply(lambda x: x['trip_headsign'].unique())
display(train_headsign_zones)

# Get stops per zone
stops_zones = gtfs_metro_set[['stop_name','zone_id']]
stops_zones = stops_zones.dropna()
stops_zones = stops_zones.groupby('zone_id').apply(lambda x: x['stop_name'].unique())
display(stops_zones)

#### Average Validations in MOD Set

In [ ]:
# Lists to save average validations and dates
averages_validation_weeks = []
days_dates = []

# Sort MOD dataset
mod_set = mod_set.sort_values(['Ano', 'Mês', 'Dia', 'Hora'],
                              ascending = [True, True, True, True])
mod_set_values = mod_set.values

# Get 7-days average with no overlap
validations_sum = 0
number_samples = 0
number_days = 0
old_day = 1

for index,element in enumerate(mod_set_values):
    year = element[0]
    month = element[1]
    day = element[2]
    hour = element[3]
    
    if day!=old_day:
        diff_days = 1
        number_days = number_days + diff_days
    
    # If there are 7 days of samples, save number of validations
    if number_days==7:
        number_days = 0
        averages_validation_weeks.append(validations_sum/number_samples)
        days_dates.append(datetime.datetime(year,month,day,hour))
        validations_sum = 0
        number_samples = 0

    validations_sum += element[7]
    number_samples += 1
    
    old_day = day

# Save last week information
averages_validation_weeks.append(validations_sum/number_samples)
days_dates.append(datetime.datetime(year,month,day,hour))
month_ticks = [datetime.datetime(2020,i,1) for i in range(1,13)]
month_ticks.append(datetime.datetime(2021,1,1))

plt.figure()
plt.plot(days_dates,averages_validation_weeks)
plt.xticks(month_ticks,rotation=90)
plt.xlabel('Datetime')
plt.ylabel('Average Validations')

The plot presents a clear drop in the number of the validations between the middle of march and the middle of june. This was a consequence of the COVID-19 lockdown. Furthermore, it is seen that after the COVID lockdown the number of validations did not come back to the previous values. Therefore, we can clearly define three periods: pre-COVID, COVID, and post-COVID. According to https://www.portugal.gov.pt/pt/gc22/governo/comunicado-de-conselho-de-ministros?i=330, the first COVID lockdown was between 16th March and 1st June. Therefore, we considered data until 15th March as pre-COVID, data between 16th March and 31st May as COVID, and data from 1st June as post-COVID.

#### Split data according to COVID lockdown

In [ ]:
covid_first_datetime = datetime.datetime(2020,3,16,0)
covid_last_datetime = datetime.datetime(2020,5,31,23)

pre_covid_mod_set,covid_mod_set,post_covid_mod_set = split_covid_periods(mod_set, covid_first_datetime,
                                                                         covid_last_datetime)

pre_covid_rodov_set,covid_rodov_set,post_covid_rodov_set = split_covid_periods(rodov_set, covid_first_datetime,
                                                                               covid_last_datetime)

pre_covid_ferrov_set,covid_ferrov_set,post_covid_ferrov_set = split_covid_periods(ferrov_set, covid_first_datetime,
                                                                                  covid_last_datetime)

#### Analysis of in/outflow of people per hours and week days

We will now get some insights about inter-municipal mobility and intra-municipal mobility, in relation to the municipality of Porto (PRT).

In [ ]:
# get entry and exit datasets for each period
PRT_entry_preCovid, PRT_exit_preCovid, PRT_in_preCovid = get_entry_exit_sets(pre_covid_mod_set)
PRT_entry_Covid, PRT_exit_Covid, PRT_in_Covid = get_entry_exit_sets(covid_mod_set)
PRT_entry_postCovid, PRT_exit_postCovid, PRT_in_postCovid = get_entry_exit_sets(post_covid_mod_set)

##### Inter-Municipal Mobility (to and from Porto)

In [ ]:
plot_heat_maps(PRT_entry_preCovid, PRT_exit_preCovid, 'mean', 'Pre-COVID')
plot_heat_maps(PRT_entry_Covid, PRT_exit_Covid, 'mean', 'COVID')
plot_heat_maps(PRT_entry_postCovid, PRT_exit_postCovid, 'mean', 'Post-COVID')

As we can observe, for the three analysed periods, in the working weekdays (Monday to Friday), there are two periods where the number of validations are significantly higher:
- 7 a.m. to 9 a.m. : larger number of validations from outside Porto to Porto.
- 5 p.m. to 7 p.m. : larger number of validations from Porto to outside Porto.
It was already expected, as they correspond to the rush hours. People enter Porto in the morning to work, and leave the city at the end of the working day. 

We can also observe that the number of validations within the working period is higher than outside of the working period (8 p.m. to 6 a.m. of the next day).

Furthermore, we can observe that, at the weekends, there is a significant decrease of validation in relation to the respective hours on the weekdays. It informs us that people uses the public transports mainly for working purposes.

In [ ]:
plot_trasnportsVSvalidations_rates(PRT_entry_preCovid, PRT_exit_preCovid, 'Pre-Covid')
plot_trasnportsVSvalidations_rates(PRT_entry_Covid, PRT_exit_Covid, 'Covid')
plot_trasnportsVSvalidations_rates(PRT_entry_postCovid, PRT_exit_postCovid, 'Post-Covid')

In the above plots, we can compare the mean rates of transports with the mean rates of validations, and see how they are related over the hours for both weekdays (Monday to Friday) and weekend (Saturday and Sunday), for both directions (to and from Porto), for each period. 

First of all, we can observe that, for the three periods, the relations were pretty the same. Thus, while the average number of validations are different for those periods, the average number of transports are also different. In other words, the number of validations decreased for the COVID period, but the number of public transport trips decreased on the same rate.

Comparing the curves over the hours for a given period, we can observe that they keep more less the same rate except during the periods above mentioned: 7-9 a.m. from outside Porto to Porto and 5-7 p.m. from Porto to outside Porto. For those ones, the rate of validations increases significantly, but the rates of transports does not follow the same variation. In fact, in those scenarios, the validations rate is about the double of the transports rate. It suggest a significant lack of public transports trips on those periods. Ideally, both curves would have the same rates for all hours, so that the number of transports trips would be adjusted to the number of validations.

We will now get the ten zones with the most validations from and to Porto.

In [ ]:
max_entry_preCovid, max_exit_preCovid = get_max_zones(PRT_entry_preCovid, PRT_exit_preCovid)
max_entry_Covid, max_exit_Covid = get_max_zones(PRT_entry_Covid, PRT_exit_Covid)
max_entry_postCovid, max_exit_postCovid = get_max_zones(PRT_entry_postCovid, PRT_exit_postCovid)
              
print('10 zones with maximum number of validation for each direction and period (by descendent order of validations)')
print('\nFrom outside Porto to Porto:')
print('Pre-Covid: ' + ", ".join(max_entry_preCovid))                                                        
print('Covid: ' + ", ".join(max_entry_Covid))                                                        
print('Post-Covid: ' + ", ".join(max_entry_postCovid))                                                        
print('\nFrom Porto to outside Porto:')
print('Pre-Covid: ' + ", ".join(max_exit_preCovid))                                                        
print('Covid: ' + ", ".join(max_exit_Covid))                                                        
print('Post-Covid: ' + ", ".join(max_exit_postCovid)) 

In [ ]:
plot_trasnportsVSvalidations_rates_zone(PRT_entry_preCovid, PRT_exit_preCovid, 'VNG1')
plot_trasnportsVSvalidations_rates_zone(PRT_entry_preCovid, PRT_exit_preCovid, 'MAI4')
plot_trasnportsVSvalidations_rates_zone(PRT_entry_preCovid, PRT_exit_preCovid, 'GDM1')
plot_trasnportsVSvalidations_rates_zone(PRT_entry_preCovid, PRT_exit_preCovid, 'PV_VC')

The above plots are related to the rates of validations and transports, for the pre-COVID period, for some of the zones with the most validations from and to Porto. We can observe a very interesting pattern. The rates of transport are pretty the same between 7 a.m. and 11 p.m., independently of the number of validations. As previously discussed, and more evident in these zones with more validations, the transports trips seem to not be adjusted to the validations. Therefore, there is room for optimization of public transports trips. For some of the hours, there is a lack of transport (when the rate of validations is significantly higher than the rate of transport), and for other hours, there are more transports that the needed ones (when the rate of validations is significantly higher than the rate of transports). 

##### Intra-Municipal Mobility (from Porto to Porto)

In [ ]:
plot_heat_maps_in(PRT_in_preCovid, 'mean', 'Pre-Covid')
plot_heat_maps_in(PRT_in_Covid, 'mean', 'Covid')
plot_heat_maps_in(PRT_in_postCovid, 'mean', 'Post-Covid')

As observed from inter-municipal mobility, the number of validations within the working period is higher than outside of the working period (8 p.m. to 6 a.m. of the next day) and that at the weekends there is a significant decrease of validation in relation to the respective hours on the working weekdays. However, while for inter-municipal mobility the most validations occurs on the peak hours, for the intra-municipal mobility it occurs for all the working hours (7 a.m. to 7 p.m).

In [ ]:
plot_trasnportsVSvalidations_rates_in(PRT_in_preCovid, 'Pre-Covid')
plot_trasnportsVSvalidations_rates_in(PRT_in_Covid, 'Covid')
plot_trasnportsVSvalidations_rates_in(PRT_in_postCovid, 'Post-Covid')

Observing the above plots, we can also conclude that there is also room for the optimization of the intra-municipal transport trips, especially during the peak hours on working weekdays.

#### Geographical-based Data Analysis

To have a better geographical understanding of how people moved, we have performed a visual analysis.
Since no data regarding the metropolitan zones was provided with the dataset, we have obtained the geographic coordinates for each zone from <https://www.linhandante.com/ocasionais-zonas.asp>. However, the designation of the zones presented in the dataset did not perfectly match every zone presented in the website. Therefore, we have replaced some of the old zones designations present in the dataset to match the new ones. We made the matching using the information regarding the old zones available at <https://www.linhandante.com/uploadfiles/guia_intermodal_andante.pdf>. Even though we made this match, there were still some unmatched areas between the dataset and the obtained metropolitanian areas, which is the reason some areas might be lacking in the plots.

##### Prepare Data

In [ ]:
############# Rodoviario
database_Rodoviario = rodov_set.copy(deep = True)
database_Rodoviario["Paragem"] = database_Rodoviario["Paragem"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()
database_Rodoviario['DateTime'] = pd.to_datetime(dict(year=database_Rodoviario.Ano,month=database_Rodoviario.Mês,
                                                      day=database_Rodoviario.Dia,hour=database_Rodoviario.Hora))

stops_Rodoviario = pd.read_table('gtfs_bus/stops.txt', delimiter= ',', header = 0)  #InterMunicipality/GTFS/
stops_Rodoviario['stop_name'] = stops_Rodoviario['stop_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()

shapes_Rodoviario = pd.read_table('gtfs_bus/shapes.txt', delimiter= ',', header = 0)  #InterMunicipality/GTFS/
stopTimes_Rodoviario = pd.read_table('gtfs_bus/stop_times.txt', delimiter= ',', header = 0)  #InterMunicipality/GTFS/

routes_stops = pd.concat([stopTimes_Rodoviario["trip_id"].str.split('_',expand=True)[0],stopTimes_Rodoviario["stop_id"]], axis=1).drop_duplicates()
routes_stops.columns = ['line','stop_id']
routes_stops = routes_stops.join(stops_Rodoviario.set_index('stop_id'), on='stop_id')
routes_stops = routes_stops.drop_duplicates(subset=['line', 'stop_name'], keep='last')
routes_stops.reset_index()

database_Rodoviario.rename(columns={'Zona': 'zone_id', 'Paragem': 'stop_name', 'Linha_ao_Publico': 'line'}, inplace=True)# rename columns
# replace zones
database_Rodoviario['zone_id'] = database_Rodoviario['zone_id'].replace({"C1": "PRT1",
                                                                         "C2": "PRT2",
                                                                         "C3": "PRT3",
                                                                         "C4": "MTS2",
                                                                         "C8": "GDM1",
                                                                         "C9": "MAI4",
                                                                         "C11": "GDM2",
                                                                         "S2": "VNG4",
                                                                         "S3": "VNG8",
                                                                         "S9": "VNG2",
                                                                         "S8": "VNG1",
                                                                         "N1": "MTS3",
                                                                         "N10": "VCD8",
                                                                         "N15": "MAI3",
                                                                         "AB": "",
                                                                         "B": "",
                                                                         "A": "",
                                                                         "C": "",
                                                                         "AC": "",
                                                                         "ABC": ""
                                                                         })
database_Rodoviario_merged = pd.merge(database_Rodoviario, routes_stops, on=['stop_name', 'line', 'zone_id'])

############# Ferrovia
#database_Ferrovia = ferrov_set
database_Ferrovia = ferrov_set.copy(deep = True)
database_Ferrovia["Est/Op"] = database_Ferrovia["Est/Op"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()
database_Ferrovia['DateTime'] = pd.to_datetime(dict(year=database_Ferrovia.Ano,month=database_Ferrovia.Mês,
                                                    day=database_Ferrovia.Dia,hour=database_Ferrovia.Hora))

stops_Ferrovia = pd.read_table('gtfs_metro/google_transit_dez2021/stops.txt', delimiter= ',', header = 0)  
stops_Ferrovia['stop_name'] = stops_Ferrovia['stop_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()

shapes_Ferrovia = pd.read_table('gtfs_metro/google_transit_dez2021/shapes.txt', delimiter= ',', header = 0)  
stopTimes_Ferrovia = pd.read_table('gtfs_metro/google_transit_dez2021/stop_times.txt', delimiter= ',', header = 0)  

stop_names_ferrovia = [i.pop() for i in list(database_Ferrovia["Est/Op"].str.split('/'))]
stop_names_ferrovia = [i.split('- Linha')[0].strip() for i in stop_names_ferrovia]
database_Ferrovia["stop_name"] = stop_names_ferrovia

database_Ferrovia.rename(columns={'Zona': 'zone_id'}, inplace=True)# rename columns
database_Ferrovia_merged = pd.merge(database_Ferrovia, stops_Ferrovia, on=['stop_name', 'zone_id'])

# replace zones
database_Ferrovia['zone_id'] = database_Ferrovia['zone_id'].replace({"C1": "PRT1",
                                                                         "C2": "PRT2",
                                                                         "C3": "PRT3",
                                                                         "C4": "MTS2",
                                                                         "C8": "GDM1",
                                                                         "C9": "MAI4",
                                                                         "C11": "GDM2",
                                                                         "S1": "VNG5", 
                                                                         "S2": "VNG4",
                                                                         "S3": "VNG8",
                                                                         "S9": "VNG2",
                                                                         "S7": "VNG3",
                                                                         "S8": "VNG1",
                                                                         "N1": "MTS3",
                                                                         "N10": "VCD8",
                                                                         "N14": "",
                                                                         "N15": "MAI3",
                                                                         "N16": "VLG2",
                                                                         "VLG": "",
                                                                         "VNG": "",
                                                                         "STR": "",
                                                                         "AB": "",
                                                                         "B": "",
                                                                         "A": "",
                                                                         "C": "",
                                                                         "AC": "",
                                                                         "ABC": ""
                                                                         })

###### Merged Dataset ---> Ferrovia + Rodoviario
database_Rodoviario_Ferrovia_merged = pd.concat([database_Rodoviario, database_Ferrovia], ignore_index=True)

########## Zones - Crete the polygons with the coordinates of each zone (from https://www.linhandante.com/ocasionais-zonas.asp)
# Opening JSON file
#f = open("ZonesPorto.json")
## returns JSON object as  a dictionary
#data_zones = pd.DataFrame(json.load(f))
file_zones_json = [{"zone_id":"esp1","points":[{"lat":40.9733397919129,"lng":-8.6104066182491},{"lat":40.9731674424777,"lng":-8.6109811903907},{"lat":40.9730212396605,"lng":-8.61127328297247},{"lat":40.9729104141571,"lng":-8.61149469725038},{"lat":40.9727846627275,"lng":-8.61173722777206},{"lat":40.9726950259443,"lng":-8.61189839115855},{"lat":40.9726115163895,"lng":-8.61203661213043},{"lat":40.9725423364922,"lng":-8.61212602947614},{"lat":40.9723677531525,"lng":-8.61234165153725},{"lat":40.972260651407,"lng":-8.61243710578808},{"lat":40.9721856173362,"lng":-8.61250819906603},{"lat":40.9719919574654,"lng":-8.61271731221102},{"lat":40.9718848310819,"lng":-8.61288309667442},{"lat":40.9716745812992,"lng":-8.61320432264407},{"lat":40.9714566333394,"lng":-8.61349383743338},{"lat":40.9713176203678,"lng":-8.61365704155916},{"lat":40.9710556732495,"lng":-8.61396446442002},{"lat":40.9709102571177,"lng":-8.61413483619779},{"lat":40.9700407237873,"lng":-8.61672084396175},{"lat":40.9695879661248,"lng":-8.61920411751301},{"lat":40.9681430841431,"lng":-8.62125102441324},{"lat":40.9678850886339,"lng":-8.62279422639178},{"lat":40.9677813446548,"lng":-8.62426908391915},{"lat":40.9682110511553,"lng":-8.62604893863248},{"lat":40.9674242230118,"lng":-8.62856284025215},{"lat":40.9675818100385,"lng":-8.62864444846155},{"lat":40.9675242260747,"lng":-8.6288426421952},{"lat":40.967436604225,"lng":-8.62894890039563},{"lat":40.967341816393,"lng":-8.62900449710401},{"lat":40.9669541402435,"lng":-8.63006375120108},{"lat":40.9667881201572,"lng":-8.63061005675993},{"lat":40.9656079093944,"lng":-8.64383371292558},{"lat":40.9650554855311,"lng":-8.65001735002219},{"lat":40.9648737004845,"lng":-8.65306601115238},{"lat":40.9649826667617,"lng":-8.6531036898257},{"lat":40.9650859844457,"lng":-8.65309524448628},{"lat":40.9651954056048,"lng":-8.65305262746965},{"lat":40.9652827047053,"lng":-8.65302266756074},{"lat":40.9653760516837,"lng":-8.65298943000828},{"lat":40.9654762088006,"lng":-8.65292345281186},{"lat":40.9655661250086,"lng":-8.65289220641276},{"lat":40.9656454702524,"lng":-8.65276976200509},{"lat":40.9657332272847,"lng":-8.65265855467891},{"lat":40.965819854918,"lng":-8.65259793388079},{"lat":40.9658969650449,"lng":-8.6525314221197},{"lat":40.9659732536663,"lng":-8.65246716323647},{"lat":40.9660506057864,"lng":-8.65240588359887},{"lat":40.9661356802775,"lng":-8.65233040183979},{"lat":40.9662161913408,"lng":-8.65226807987274},{"lat":40.9663094956731,"lng":-8.65222426413322},{"lat":40.9663946950591,"lng":-8.65218121632771},{"lat":40.9664951994397,"lng":-8.65215860711815},{"lat":40.9665723458479,"lng":-8.65208377757621},{"lat":40.9666545205664,"lng":-8.65203191721439},{"lat":40.96675121062,"lng":-8.65199608789756},{"lat":40.9668728378346,"lng":-8.65198112553209},{"lat":40.9669625842404,"lng":-8.65196734013496},{"lat":40.9670560617183,"lng":-8.65196545882619},{"lat":40.9671535784938,"lng":-8.65192560015306},{"lat":40.9672682867326,"lng":-8.65186876925296},{"lat":40.9673855498303,"lng":-8.65182288512232},{"lat":40.9674920244193,"lng":-8.65179402197655},{"lat":40.9675988635,"lng":-8.65182384398855},{"lat":40.9677049193244,"lng":-8.65182741399362},{"lat":40.9678135753004,"lng":-8.65181365750893},{"lat":40.9679228502172,"lng":-8.65180323098571},{"lat":40.9680316977779,"lng":-8.6518069333249},{"lat":40.9681401780678,"lng":-8.65183202365241},{"lat":40.9682450626931,"lng":-8.65185553676934},{"lat":40.9683537796476,"lng":-8.65182942681399},{"lat":40.9684617155651,"lng":-8.65179558396929},{"lat":40.9685688526504,"lng":-8.65175936352836},{"lat":40.9686734308728,"lng":-8.65171147346597},{"lat":40.9687724797691,"lng":-8.65165107313593},{"lat":40.9688648558021,"lng":-8.65157434244941},{"lat":40.9689691445227,"lng":-8.65153013954595},{"lat":40.9690731514218,"lng":-8.65148973233958},{"lat":40.9692034332648,"lng":-8.65143242456666},{"lat":40.9693337254595,"lng":-8.65137380865729},{"lat":40.9694180195466,"lng":-8.65133075550111},{"lat":40.9695048325695,"lng":-8.65128902518921},{"lat":40.9695929844588,"lng":-8.65122948627106},{"lat":40.9696962265644,"lng":-8.65115866149548},{"lat":40.9698052721737,"lng":-8.65113872653039},{"lat":40.9699095653762,"lng":-8.65109368600001},{"lat":40.9700141390136,"lng":-8.65104675025736},{"lat":40.9701168658841,"lng":-8.65099077097529},{"lat":40.9702253127743,"lng":-8.6509629899878},{"lat":40.9703533907777,"lng":-8.65089568475921},{"lat":40.970459369832,"lng":-8.65085719674495},{"lat":40.9705642061771,"lng":-8.65081109035716},{"lat":40.9706699413863,"lng":-8.65076653885359},{"lat":40.970763928096,"lng":-8.65071178999414},{"lat":40.9708579228658,"lng":-8.65069435241938},{"lat":40.9709554327675,"lng":-8.65073634283354},{"lat":40.9710646920907,"lng":-8.65070809579048},{"lat":40.9711747806039,"lng":-8.65069672529046},{"lat":40.97128205565,"lng":-8.65067000347502},{"lat":40.9713631874733,"lng":-8.65058913659434},{"lat":40.9714718747151,"lng":-8.65056836697526},{"lat":40.9715814037188,"lng":-8.65056067475233},{"lat":40.9716907897789,"lng":-8.65056556994887},{"lat":40.971800577922,"lng":-8.65056072866892},{"lat":40.9719399138223,"lng":-8.65055208123117},{"lat":40.9720494351224,"lng":-8.65052704327565},{"lat":40.9721573519679,"lng":-8.6504973509672},{"lat":40.9722650389643,"lng":-8.65045827704552},{"lat":40.9723742849556,"lng":-8.65043418473932},{"lat":40.9724843923637,"lng":-8.65041758863038},{"lat":40.9725934511523,"lng":-8.650394676843},{"lat":40.9727046143828,"lng":-8.65038379158308},{"lat":40.9728409205298,"lng":-8.65036965014121},{"lat":40.9729468923962,"lng":-8.65033151149355},{"lat":40.9730528438254,"lng":-8.65029812604504},{"lat":40.9731611287117,"lng":-8.6503074129711},{"lat":40.9732562236752,"lng":-8.65024542876556},{"lat":40.9733940081876,"lng":-8.65026183685622},{"lat":40.973488800174,"lng":-8.65026839512552},{"lat":40.9735037529472,"lng":-8.65038708355032},{"lat":40.9735193763961,"lng":-8.65052296203196},{"lat":40.973547480932,"lng":-8.65066112330072},{"lat":40.9735528244618,"lng":-8.65080757372889},{"lat":40.9735989653927,"lng":-8.65093802471406},{"lat":40.9736066298194,"lng":-8.6510833637122},{"lat":40.9736152862766,"lng":-8.65122030363257},{"lat":40.9736279913563,"lng":-8.65133867063341},{"lat":40.9736511888603,"lng":-8.65145773840717},{"lat":40.9736654860601,"lng":-8.65159175113937},{"lat":40.9736902858364,"lng":-8.65174235601847},{"lat":40.9737164563022,"lng":-8.6518694554419},{"lat":40.9737203551079,"lng":-8.65198958757556},{"lat":40.9737557777337,"lng":-8.65210618116723},{"lat":40.9737682507375,"lng":-8.65225562127001},{"lat":40.9737988678081,"lng":-8.65239367927113},{"lat":40.9737949778838,"lng":-8.65254682630809},{"lat":40.9738497309961,"lng":-8.65265557130218},{"lat":40.9738287777139,"lng":-8.65279259755579},{"lat":40.9739197516457,"lng":-8.65280225786837},{"lat":40.973956050667,"lng":-8.65268088599007},{"lat":40.974007047314,"lng":-8.65256936961072},{"lat":40.973991133816,"lng":-8.65244901330373},{"lat":40.973970609607,"lng":-8.65232233586821},{"lat":40.9739406243048,"lng":-8.65219902075119},{"lat":40.9739312385241,"lng":-8.65205771680261},{"lat":40.9739208247656,"lng":-8.65192724759832},{"lat":40.9738896889281,"lng":-8.65178348386044},{"lat":40.9738759457528,"lng":-8.65165684609308},{"lat":40.9738539577347,"lng":-8.65154083661825},{"lat":40.9738463042012,"lng":-8.65141043856382},{"lat":40.9739029302347,"lng":-8.65128447656608},{"lat":40.9740000343871,"lng":-8.65121573668983},{"lat":40.9740882170213,"lng":-8.65113029592161},{"lat":40.9741839934191,"lng":-8.6510570304912},{"lat":40.9742794240454,"lng":-8.65098114714965},{"lat":40.9743851448626,"lng":-8.65093980004083},{"lat":40.9745101019914,"lng":-8.65086462477036},{"lat":40.9746111289365,"lng":-8.65080637079732},{"lat":40.9747102048748,"lng":-8.65074002230069},{"lat":40.9748150408163,"lng":-8.65069498212895},{"lat":40.9749519775036,"lng":-8.65066053160053},{"lat":40.9750571708504,"lng":-8.65061550106413},{"lat":40.9751564969353,"lng":-8.65055378148666},{"lat":40.9752656380685,"lng":-8.65053278136157},{"lat":40.9753774528037,"lng":-8.65051797074925},{"lat":40.9754859000054,"lng":-8.65049007400193},{"lat":40.9755957717945,"lng":-8.65046657720422},{"lat":40.975705754338,"lng":-8.65045853032452},{"lat":40.9758133670081,"lng":-8.65043620387349},{"lat":40.9759004367129,"lng":-8.65047704505754},{"lat":40.9759908973725,"lng":-8.65044448669252},{"lat":40.9760943725994,"lng":-8.65040205082928},{"lat":40.9761950468079,"lng":-8.65036054345873},{"lat":40.976303880047,"lng":-8.65036781544204},{"lat":40.9764115302521,"lng":-8.65033812014431},{"lat":40.9765188122178,"lng":-8.65030961595127},{"lat":40.9766251303559,"lng":-8.65027516226756},{"lat":40.9767597418782,"lng":-8.65023629688213},{"lat":40.9768644056607,"lng":-8.65018923574417},{"lat":40.9769999049522,"lng":-8.65015299208466},{"lat":40.9771099862328,"lng":-8.6501433983433},{"lat":40.9772139809106,"lng":-8.65010417217593},{"lat":40.9773224143898,"lng":-8.65008031004462},{"lat":40.9774306066567,"lng":-8.65004979468283},{"lat":40.9775378963741,"lng":-8.65001974268602},{"lat":40.9776469542185,"lng":-8.64999599924204},{"lat":40.9777546208068,"lng":-8.64996262316534},{"lat":40.977872662486,"lng":-8.64992362717435},{"lat":40.9779863981625,"lng":-8.64990230341914},{"lat":40.9781030061393,"lng":-8.64988242305328},{"lat":40.9782101089541,"lng":-8.64985367720273},{"lat":40.9783122999267,"lng":-8.64983593931963},{"lat":40.9784144579723,"lng":-8.64982474318482},{"lat":40.9785052512548,"lng":-8.6497984861921},{"lat":40.9785974960018,"lng":-8.64981001720318},{"lat":40.9786876197813,"lng":-8.64979290264659},{"lat":40.9787665275895,"lng":-8.64970642515214},{"lat":40.9788436201668,"lng":-8.64964322393115},{"lat":40.9789366378163,"lng":-8.64960343983239},{"lat":40.9790273772325,"lng":-8.64956909397522},{"lat":40.9791405271584,"lng":-8.64949679287104},{"lat":40.9792621031861,"lng":-8.64947302682633},{"lat":40.9793665803651,"lng":-8.64946755017715},{"lat":40.9794704345329,"lng":-8.64946064164005},{"lat":40.9796089124139,"lng":-8.64944366584203},{"lat":40.9797104342258,"lng":-8.64945408191148},{"lat":40.9797995869917,"lng":-8.64947260721749},{"lat":40.9799110939394,"lng":-8.64946575120389},{"lat":40.9800075827133,"lng":-8.64947423530808},{"lat":40.9800980255055,"lng":-8.6494455897764},{"lat":40.9801980778992,"lng":-8.64940229254212},{"lat":40.9803078053597,"lng":-8.64941075388235},{"lat":40.9804029200105,"lng":-8.6494049664151},{"lat":40.980499528354,"lng":-8.64938695205457},{"lat":40.9806035260227,"lng":-8.64934819448781},{"lat":40.9807126447664,"lng":-8.64937234017862},{"lat":40.9808014041933,"lng":-8.64939786592235},{"lat":40.9808910031958,"lng":-8.64937646504111},{"lat":40.9809841904313,"lng":-8.64935830382774},{"lat":40.9810753216052,"lng":-8.64937660597488},{"lat":40.9811732031467,"lng":-8.64943701543089},{"lat":40.9812640733915,"lng":-8.64945305689988},{"lat":40.9813431320091,"lng":-8.64951439041352},{"lat":40.9813662816764,"lng":-8.64964347340094},{"lat":40.9813879115111,"lng":-8.64977934240686},{"lat":40.981407379137,"lng":-8.64989582597544},{"lat":40.9814222244793,"lng":-8.65001856797364},{"lat":40.9814442875049,"lng":-8.65017772673331},{"lat":40.9814626432168,"lng":-8.6503019239928},{"lat":40.9814833191314,"lng":-8.65043671040689},{"lat":40.9815005731725,"lng":-8.65055674907196},{"lat":40.9815157720417,"lng":-8.65068128030109},{"lat":40.9815301242926,"lng":-8.65081388570057},{"lat":40.9815430248407,"lng":-8.6509481397257},{"lat":40.9815509167732,"lng":-8.65107498413521},{"lat":40.9815728953305,"lng":-8.65119339376544},{"lat":40.981590853393,"lng":-8.65132519111758},{"lat":40.9816054758577,"lng":-8.65145684942387},{"lat":40.9816059794222,"lng":-8.65158506346102},{"lat":40.9816233114159,"lng":-8.651720485329},{"lat":40.9816571101476,"lng":-8.65183398337816},{"lat":40.9816993707412,"lng":-8.65194660559252},{"lat":40.9817272723527,"lng":-8.65206885530539},{"lat":40.9818211143644,"lng":-8.65208515790504},{"lat":40.9819186647452,"lng":-8.65205919451213},{"lat":40.9819829799455,"lng":-8.65195442002426},{"lat":40.9819878067228,"lng":-8.65182268756005},{"lat":40.9819768521459,"lng":-8.65169569727402},{"lat":40.9819552295443,"lng":-8.65157414643656},{"lat":40.981931660537,"lng":-8.65144683730457},{"lat":40.9818787045482,"lng":-8.65133125955231},{"lat":40.9818562634967,"lng":-8.6511617147807},{"lat":40.981834487091,"lng":-8.65103277205097},{"lat":40.9818110068723,"lng":-8.65091326181101},{"lat":40.9817873394156,"lng":-8.65079009614014},{"lat":40.9817687479284,"lng":-8.65066066787108},{"lat":40.9817474766841,"lng":-8.65053801118396},{"lat":40.9817282518793,"lng":-8.65041522096223},{"lat":40.9817130833199,"lng":-8.65029676500321},{"lat":40.9816876576368,"lng":-8.65011083986805},{"lat":40.9816681417611,"lng":-8.64997855269628},{"lat":40.9816404526585,"lng":-8.64985559895605},{"lat":40.981633480665,"lng":-8.64970404681697},{"lat":40.981777681455,"lng":-8.64949521782581},{"lat":40.9818791855948,"lng":-8.64945026501696},{"lat":40.9819759493205,"lng":-8.6493975527781},{"lat":40.9820677210768,"lng":-8.64931402618639},{"lat":40.982217590884,"lng":-8.64934585483297},{"lat":40.9823237826793,"lng":-8.64932042636321},{"lat":40.9824076063352,"lng":-8.64920142650697},{"lat":40.9825365892436,"lng":-8.64911272678692},{"lat":40.9826278503238,"lng":-8.64904285993386},{"lat":40.9827309560582,"lng":-8.64900196195048},{"lat":40.9828600353942,"lng":-8.6489713675313},{"lat":40.9830842662579,"lng":-8.64890740678673},{"lat":40.9832034270184,"lng":-8.64886009918362},{"lat":40.9833091352919,"lng":-8.64882135257576},{"lat":40.9834007604122,"lng":-8.64876990773494},{"lat":40.9835174759723,"lng":-8.64872590774886},{"lat":40.9836314693071,"lng":-8.64866642948863},{"lat":40.9837869961439,"lng":-8.64862190317988},{"lat":40.9839121694814,"lng":-8.64856002213316},{"lat":40.9841168759677,"lng":-8.64853238575323},{"lat":40.9842546378533,"lng":-8.6484928265225},{"lat":40.9843471225502,"lng":-8.64845089458903},{"lat":40.9844769558654,"lng":-8.64843348938073},{"lat":40.9845747285056,"lng":-8.64839729618811},{"lat":40.9846654740146,"lng":-8.64836128703911},{"lat":40.9848051901195,"lng":-8.64832851318631},{"lat":40.9849078392077,"lng":-8.64828938927773},{"lat":40.9850215331075,"lng":-8.64829645913429},{"lat":40.9851259013422,"lng":-8.64835561996921},{"lat":40.9852262613357,"lng":-8.64830423394915},{"lat":40.9853282903911,"lng":-8.6483419886877},{"lat":40.9854587921643,"lng":-8.64829690094665},{"lat":40.9856333163673,"lng":-8.64821235474379},{"lat":40.9857685141234,"lng":-8.64818275456226},{"lat":40.9858824327101,"lng":-8.64814074204537},{"lat":40.9859801353151,"lng":-8.64811999865582},{"lat":40.9860824911893,"lng":-8.64814527887579},{"lat":40.9861778977414,"lng":-8.64813377924468},{"lat":40.9863664708341,"lng":-8.64804922335924},{"lat":40.986489824026,"lng":-8.64803188648413},{"lat":40.9866203890736,"lng":-8.64797159097242},{"lat":40.9867180769508,"lng":-8.64795393424898},{"lat":40.9868087688985,"lng":-8.64795024523548},{"lat":40.986901622202,"lng":-8.64794609650692},{"lat":40.9869931172483,"lng":-8.64794384327396},{"lat":40.9871212305984,"lng":-8.64790848171668},{"lat":40.9872290338436,"lng":-8.64788365226926},{"lat":40.9873298298407,"lng":-8.64787552844303},{"lat":40.987428433515,"lng":-8.64785443117354},{"lat":40.9875176830007,"lng":-8.64783136214674},{"lat":40.9876084216229,"lng":-8.64781721595542},{"lat":40.9876998823219,"lng":-8.64780248473855},{"lat":40.9877908297494,"lng":-8.64780247774408},{"lat":40.9879188918518,"lng":-8.64779754016396},{"lat":40.9880100487348,"lng":-8.64779135898497},{"lat":40.9881070533296,"lng":-8.64780471327401},{"lat":40.9881977169724,"lng":-8.64780755967383},{"lat":40.9882888846339,"lng":-8.64779876183192},{"lat":40.9883845828084,"lng":-8.64778263619161},{"lat":40.9884932224547,"lng":-8.64779251478493},{"lat":40.9885835404381,"lng":-8.64779251002362},{"lat":40.9886739054229,"lng":-8.64778096909638},{"lat":40.9887782374902,"lng":-8.64778736800977},{"lat":40.9888849387734,"lng":-8.64778749064133},{"lat":40.9889868475322,"lng":-8.64783285010205},{"lat":40.9890448275386,"lng":-8.64794901337521},{"lat":40.9890684789605,"lng":-8.64807659269968},{"lat":40.989095780277,"lng":-8.64819362207321},{"lat":40.9891039201584,"lng":-8.64832643264685},{"lat":40.9891303668862,"lng":-8.6484533235354},{"lat":40.9891436459416,"lng":-8.64858343846697},{"lat":40.9891551131064,"lng":-8.64871698577786},{"lat":40.9891793129575,"lng":-8.64884326166919},{"lat":40.9891973186742,"lng":-8.64896390433979},{"lat":40.989225091167,"lng":-8.64910101183952},{"lat":40.9892293811282,"lng":-8.6492239387809},{"lat":40.9892603167649,"lng":-8.64935773295915},{"lat":40.9892954751398,"lng":-8.64948459882697},{"lat":40.989317719447,"lng":-8.64960468409392},{"lat":40.9893306057767,"lng":-8.6497424001324},{"lat":40.989351397468,"lng":-8.64986718374405},{"lat":40.9893692639696,"lng":-8.64998457504355},{"lat":40.9894754667481,"lng":-8.64999044019068},{"lat":40.9895724704441,"lng":-8.64994508781847},{"lat":40.9895846584247,"lng":-8.64981557435995},{"lat":40.9896025969818,"lng":-8.64969280161611},{"lat":40.9895978704355,"lng":-8.64956064792512},{"lat":40.9895347997827,"lng":-8.64947229541801},{"lat":40.9895182793041,"lng":-8.64935137821894},{"lat":40.9894846374872,"lng":-8.649229893941},{"lat":40.9894519003359,"lng":-8.64910374260892},{"lat":40.9894398773207,"lng":-8.64897459104342},{"lat":40.9894141863606,"lng":-8.64886006876654},{"lat":40.9893924676627,"lng":-8.64874403740218},{"lat":40.9893688568379,"lng":-8.64860790205524},{"lat":40.9893536790026,"lng":-8.64847813555789},{"lat":40.9893597123378,"lng":-8.64833807662323},{"lat":40.9894021548884,"lng":-8.64821006821274},{"lat":40.9894969260328,"lng":-8.6480999322284},{"lat":40.9895715217641,"lng":-8.64803123547145},{"lat":40.9896811166899,"lng":-8.64794818909291},{"lat":40.9897601704143,"lng":-8.64788939370652},{"lat":40.9898427811239,"lng":-8.64784083873042},{"lat":40.9899254874233,"lng":-8.64779002884047},{"lat":40.9900178592945,"lng":-8.64771338735412},{"lat":40.9901009327705,"lng":-8.64764142926114},{"lat":40.9901788276553,"lng":-8.64757976132435},{"lat":40.9902595810555,"lng":-8.64752323025508},{"lat":40.9903391771207,"lng":-8.64746372028521},{"lat":40.9904241066448,"lng":-8.64741993829423},{"lat":40.9905109087717,"lng":-8.64737925935814},{"lat":40.9905976391917,"lng":-8.64731528854808},{"lat":40.9906780371921,"lng":-8.64725745275628},{"lat":40.9907657155932,"lng":-8.64722319281761},{"lat":40.9908519766963,"lng":-8.64718286692173},{"lat":40.9909579578064,"lng":-8.6471437636377},{"lat":40.9910444911802,"lng":-8.64710355847777},{"lat":40.9911343499033,"lng":-8.64708476900179},{"lat":40.9912244767228,"lng":-8.64706562860348},{"lat":40.991315880075,"lng":-8.64706372631125},{"lat":40.9914264825348,"lng":-8.64705769367162},{"lat":40.9915188137141,"lng":-8.64704985417245},{"lat":40.9916097675568,"lng":-8.64704819317515},{"lat":40.9917558668803,"lng":-8.64703815549067},{"lat":40.9918489674172,"lng":-8.64703911970516},{"lat":40.9919786894912,"lng":-8.64704678454816},{"lat":40.9920949783717,"lng":-8.64701691123225},{"lat":40.9922228160465,"lng":-8.64700270169119},{"lat":40.9923135359842,"lng":-8.64697250844084},{"lat":40.9924348263736,"lng":-8.6469122465457},{"lat":40.9925512469068,"lng":-8.6468732300925},{"lat":40.9926489345344,"lng":-8.64685557025424},{"lat":40.9927466785022,"lng":-8.64682567119978},{"lat":40.9928352386258,"lng":-8.6467955807417},{"lat":40.9929376967076,"lng":-8.64677795774477},{"lat":40.9930514358561,"lng":-8.64677586949062},{"lat":40.9931795956162,"lng":-8.64674918205092},{"lat":40.9932701778006,"lng":-8.64674988747869},{"lat":40.9933678378862,"lng":-8.64673840218498},{"lat":40.9934960708178,"lng":-8.64669650071298},{"lat":40.9936588527871,"lng":-8.64666081415431},{"lat":40.9937544306587,"lng":-8.64663101046497},{"lat":40.9938963169122,"lng":-8.64661666761258},{"lat":40.9940032504495,"lng":-8.64660526229967},{"lat":40.9941196841422,"lng":-8.64656326150014},{"lat":40.9942684015622,"lng":-8.64659211348958},{"lat":40.9944332815363,"lng":-8.6465901899702},{"lat":40.9945286696943,"lng":-8.64660317243573},{"lat":40.9946285412893,"lng":-8.64660098219485},{"lat":40.9947541939363,"lng":-8.64657129844888},{"lat":40.9948798160428,"lng":-8.64660911558298},{"lat":40.9949933339107,"lng":-8.64661606239573},{"lat":40.9951004541895,"lng":-8.6465831470782},{"lat":40.9952007444586,"lng":-8.64654720273188},{"lat":40.9953276943074,"lng":-8.64653012889038},{"lat":40.9955220698481,"lng":-8.64643597061383},{"lat":40.995633914864,"lng":-8.64639393536367},{"lat":40.9957803855726,"lng":-8.6463613257806},{"lat":40.9959291796329,"lng":-8.64637175361383},{"lat":40.9960567162418,"lng":-8.64642491975881},{"lat":40.9961331379054,"lng":-8.64651132492832},{"lat":40.9961389620939,"lng":-8.64663243728477},{"lat":40.9961126598771,"lng":-8.64679481961757},{"lat":40.9961098619332,"lng":-8.64691757018374},{"lat":40.9961657704304,"lng":-8.64702725581826},{"lat":40.9963353923874,"lng":-8.64703154560397},{"lat":40.996423518744,"lng":-8.647078349493},{"lat":40.9964978964682,"lng":-8.64699740129488},{"lat":40.9966161208252,"lng":-8.64699832300699},{"lat":40.9967077271664,"lng":-8.64701186783402},{"lat":40.996809563682,"lng":-8.64703287154598},{"lat":40.996916348736,"lng":-8.6469937708438},{"lat":40.997045653891,"lng":-8.64701474310163},{"lat":40.9971534986091,"lng":-8.64706051094427},{"lat":40.9972600157976,"lng":-8.64710269597243},{"lat":40.9973679040408,"lng":-8.64713990599265},{"lat":40.9974659895869,"lng":-8.64717394984907},{"lat":40.997557482593,"lng":-8.64721269782605},{"lat":40.9976425722814,"lng":-8.64725329421326},{"lat":40.99773909944,"lng":-8.64731370538589},{"lat":40.997830595747,"lng":-8.6473708769575},{"lat":40.9979334127111,"lng":-8.64741470165986},{"lat":40.9980472006224,"lng":-8.64750176105692},{"lat":40.9981385687569,"lng":-8.64758863811054},{"lat":40.9982248814174,"lng":-8.6476773824833},{"lat":40.9983073125273,"lng":-8.64778902950992},{"lat":40.9983831551002,"lng":-8.64794377267059},{"lat":40.9984502755818,"lng":-8.64811686574137},{"lat":40.9985010421124,"lng":-8.64828139802174},{"lat":40.9985368188486,"lng":-8.64844431805293},{"lat":40.9985097970179,"lng":-8.64856688344419},{"lat":40.9984353389616,"lng":-8.64866400529749},{"lat":40.9984032378995,"lng":-8.64879485256437},{"lat":40.9983448893422,"lng":-8.64895520861765},{"lat":40.998302684296,"lng":-8.64906921302205},{"lat":40.9982419988288,"lng":-8.64916834142379},{"lat":40.9981897994956,"lng":-8.64934193513528},{"lat":40.9981114671486,"lng":-8.64942095355303},{"lat":40.9980217100808,"lng":-8.64953637094871},{"lat":40.9979832759498,"lng":-8.64969343466385},{"lat":40.9979889105499,"lng":-8.64982124918941},{"lat":40.9980263731225,"lng":-8.64995407919778},{"lat":40.9980533290001,"lng":-8.65007372900083},{"lat":40.9980716978938,"lng":-8.65019498882946},{"lat":40.998106146208,"lng":-8.65035606518996},{"lat":40.9981444938875,"lng":-8.6504788981426},{"lat":40.9981764512107,"lng":-8.65062011050777},{"lat":40.9982146751146,"lng":-8.6507383302355},{"lat":40.9982308576029,"lng":-8.65085597922458},{"lat":40.9982665688033,"lng":-8.65097264819116},{"lat":40.9983077216086,"lng":-8.65108228297781},{"lat":40.9984000259563,"lng":-8.6512006721967},{"lat":40.9984905395365,"lng":-8.65117654542446},{"lat":40.9985713572947,"lng":-8.65108602146429},{"lat":40.998629988254,"lng":-8.65094385781624},{"lat":40.9986238554386,"lng":-8.65082252782838},{"lat":40.9985782995843,"lng":-8.65067498468422},{"lat":40.9985346548121,"lng":-8.65054853733981},{"lat":40.9984890048241,"lng":-8.65043858027132},{"lat":40.9984718849503,"lng":-8.65032066793881},{"lat":40.9984469694943,"lng":-8.6502045902704},{"lat":40.9984137678532,"lng":-8.65008975686885},{"lat":40.998397173573,"lng":-8.64991539142047},{"lat":40.998376302857,"lng":-8.64979102729377},{"lat":40.9983941339665,"lng":-8.64966677662288},{"lat":40.9984138503364,"lng":-8.6495110715233},{"lat":40.9984674184634,"lng":-8.64937219519009},{"lat":40.9985320387512,"lng":-8.64919870451651},{"lat":40.9985980336535,"lng":-8.64910128666527},{"lat":40.9986525110799,"lng":-8.64900056621502},{"lat":40.998729692464,"lng":-8.64885700598529},{"lat":40.9988374674245,"lng":-8.648619071632},{"lat":40.9988582751018,"lng":-8.64849646153974},{"lat":40.998873802159,"lng":-8.64834730469764},{"lat":40.9989316771683,"lng":-8.6481773417257},{"lat":40.9989639899488,"lng":-8.64812579945397},{"lat":40.998991840164,"lng":-8.64805969363469},{"lat":41.0011141790844,"lng":-8.63513826941891},{"lat":41.0022135545583,"lng":-8.63441944721206},{"lat":41.0022137354879,"lng":-8.63411573552935},{"lat":41.0033722127487,"lng":-8.63370905249525},{"lat":41.0038119863156,"lng":-8.63084906704574},{"lat":41.0071232681718,"lng":-8.63088779228106},{"lat":41.0100911602574,"lng":-8.63163855882472},{"lat":41.0161960857146,"lng":-8.63148206731223},{"lat":41.0184858812136,"lng":-8.6304238840555},{"lat":41.020482839506,"lng":-8.61834560349968},{"lat":41.021033621126,"lng":-8.61585294756277},{"lat":41.0223357913007,"lng":-8.6143339743552},{"lat":41.0224907255652,"lng":-8.61194013875301},{"lat":41.0243049061779,"lng":-8.60874311904242},{"lat":41.0256943882391,"lng":-8.60605883617437},{"lat":41.0249306891692,"lng":-8.60533514708132},{"lat":41.0239996224356,"lng":-8.60403205815044},{"lat":41.0223023052995,"lng":-8.60206920266402},{"lat":41.0218533808824,"lng":-8.60153400195919},{"lat":41.0209180365045,"lng":-8.60027735834144},{"lat":41.020541788863,"lng":-8.60017522065482},{"lat":41.0200281722722,"lng":-8.59934421899915},{"lat":41.0199343698143,"lng":-8.59922692128701},{"lat":41.0197943330656,"lng":-8.59905377696412},{"lat":41.0195536140453,"lng":-8.5988831294032},{"lat":41.0194131909172,"lng":-8.59882733513319},{"lat":41.0193215159317,"lng":-8.5988084967666},{"lat":41.0192326441201,"lng":-8.5987856323581},{"lat":41.0191435764215,"lng":-8.59876645341745},{"lat":41.0190564552,"lng":-8.59873397612048},{"lat":41.0189656444652,"lng":-8.59872275778305},{"lat":41.0182127484238,"lng":-8.59848643053778},{"lat":41.0181445128397,"lng":-8.59859366315799},{"lat":41.0180407990321,"lng":-8.59863382580618},{"lat":41.017937426948,"lng":-8.59858982039356},{"lat":41.0178796404782,"lng":-8.59847266327111},{"lat":41.0178178534685,"lng":-8.59836546834725},{"lat":41.0177136694378,"lng":-8.5983218168336},{"lat":41.0176103016068,"lng":-8.59827650321031},{"lat":41.0175291155849,"lng":-8.59817986875686},{"lat":41.0174537550139,"lng":-8.59808981192224},{"lat":41.0173741900071,"lng":-8.59799152728469},{"lat":41.0173093783019,"lng":-8.59787574894959},{"lat":41.0172590198982,"lng":-8.59774830855778},{"lat":41.0171959234233,"lng":-8.59763004609256},{"lat":41.017110375971,"lng":-8.59754301340722},{"lat":41.0170192856558,"lng":-8.59746794716945},{"lat":41.0169437013872,"lng":-8.59736577338872},{"lat":41.0168527965992,"lng":-8.59728820990332},{"lat":41.016769017249,"lng":-8.59720903699611},{"lat":41.0167956851129,"lng":-8.59709319502612},{"lat":41.0154289377522,"lng":-8.59601409912793},{"lat":41.0145608892187,"lng":-8.59494532019822},{"lat":41.0142373740368,"lng":-8.59402316383134},{"lat":41.0141061653569,"lng":-8.5936229239907},{"lat":41.0111395308336,"lng":-8.60073023631636},{"lat":41.0102447893234,"lng":-8.60287348172461},{"lat":41.0085421658054,"lng":-8.60146924527322},{"lat":41.0082883093815,"lng":-8.60154042598339},{"lat":41.0080333709373,"lng":-8.60161195510289},{"lat":41.0063781588468,"lng":-8.60192349697455},{"lat":41.0061140941027,"lng":-8.60204678264952},{"lat":41.005966497025,"lng":-8.60211633825503},{"lat":41.0056938330268,"lng":-8.60227272503765},{"lat":41.0053401570563,"lng":-8.60246609367049},{"lat":41.0048122863509,"lng":-8.60221822863212},{"lat":41.0047050333097,"lng":-8.602145036441},{"lat":41.0042539206608,"lng":-8.60192169219949},{"lat":41.0037320984364,"lng":-8.6016601683899},{"lat":41.0035328518238,"lng":-8.60220598651342},{"lat":41.0032005143612,"lng":-8.60156225077716},{"lat":41.0031302645701,"lng":-8.60148507051449},{"lat":41.0028115969741,"lng":-8.60176501016289},{"lat":41.0025454214619,"lng":-8.60199504484612},{"lat":41.0023819212027,"lng":-8.60213536101665},{"lat":41.0019121220251,"lng":-8.60252722696788},{"lat":41.0014450231517,"lng":-8.60289724464794},{"lat":41.0005812411011,"lng":-8.6026551133872},{"lat":41.0002465081836,"lng":-8.60281540595865},{"lat":41.0000411770317,"lng":-8.60279825702812},{"lat":40.9997743542304,"lng":-8.60283858643202},{"lat":40.9994566646807,"lng":-8.60286349240394},{"lat":40.9991116991519,"lng":-8.6028950585872},{"lat":40.9987113715699,"lng":-8.60273888756241},{"lat":40.9984059518456,"lng":-8.60254568752981},{"lat":40.9976761889144,"lng":-8.60193407671393},{"lat":40.9970831182338,"lng":-8.6021729073629},{"lat":40.9947640085964,"lng":-8.60246332804797},{"lat":40.9936555661762,"lng":-8.60260213062562},{"lat":40.9925138149352,"lng":-8.60231127396604},{"lat":40.9924337461157,"lng":-8.60249217474789},{"lat":40.9913899711882,"lng":-8.60262643919769},{"lat":40.9893031272939,"lng":-8.60289483474736},{"lat":40.9892452673859,"lng":-8.6031643102388},{"lat":40.9892174670602,"lng":-8.60337143827351},{"lat":40.9891649222613,"lng":-8.60359509380953},{"lat":40.9890466140855,"lng":-8.60387790866511},{"lat":40.988994038661,"lng":-8.60591317047123},{"lat":40.988773522345,"lng":-8.60591671750904},{"lat":40.9884493213144,"lng":-8.60594360612615},{"lat":40.9882013008233,"lng":-8.60597438791468},{"lat":40.9880699638384,"lng":-8.60601507307684},{"lat":40.9879314785954,"lng":-8.60608372089888},{"lat":40.9877008601797,"lng":-8.60619370837052},{"lat":40.9875998225939,"lng":-8.60617742400742},{"lat":40.9874253772086,"lng":-8.60614966107442},{"lat":40.9871210594847,"lng":-8.60610319354898},{"lat":40.9868443931522,"lng":-8.60605598184653},{"lat":40.9867153402315,"lng":-8.60602739182722},{"lat":40.9865803071616,"lng":-8.60605151926503},{"lat":40.9863224728709,"lng":-8.60613887532659},{"lat":40.9860380615175,"lng":-8.6062504046593},{"lat":40.9858085015609,"lng":-8.60636032626354},{"lat":40.9855931486719,"lng":-8.60648052047919},{"lat":40.985288781657,"lng":-8.60662637104825},{"lat":40.9848550602467,"lng":-8.60685051158719},{"lat":40.9845756736215,"lng":-8.60704691050586},{"lat":40.9847477931682,"lng":-8.6074064479022},{"lat":40.9848743417957,"lng":-8.60765665666493},{"lat":40.9850285784716,"lng":-8.60801020831219},{"lat":40.9852333007293,"lng":-8.60850820601676},{"lat":40.9854876259008,"lng":-8.60913214827149},{"lat":40.9858162113575,"lng":-8.60994566548625},{"lat":40.9859079617596,"lng":-8.61027186756894},{"lat":40.9859494070974,"lng":-8.6104415878696},{"lat":40.9859538228065,"lng":-8.61073054776722},{"lat":40.9858919261004,"lng":-8.61115259572706},{"lat":40.9858177927228,"lng":-8.61155424753122},{"lat":40.9857456307639,"lng":-8.61196166749795},{"lat":40.9856769538719,"lng":-8.61234630258731},{"lat":40.9856287827367,"lng":-8.61273171587297},{"lat":40.9849969874872,"lng":-8.61251706227039},{"lat":40.984411605334,"lng":-8.6123853671861},{"lat":40.9840708813443,"lng":-8.61233011215465},{"lat":40.9828985375087,"lng":-8.61223511472722},{"lat":40.9820738496476,"lng":-8.61213428010213},{"lat":40.9817894320852,"lng":-8.61208621467158},{"lat":40.981775592684,"lng":-8.61192867641461},{"lat":40.9817429372698,"lng":-8.61174830462064},{"lat":40.9816963534269,"lng":-8.61160454980459},{"lat":40.9815882764073,"lng":-8.61150169880792},{"lat":40.9813786296303,"lng":-8.61165690056523},{"lat":40.981110645132,"lng":-8.61185528815423},{"lat":40.980408629792,"lng":-8.6114389425863},{"lat":40.9797763679532,"lng":-8.61115009053317},{"lat":40.9793553653416,"lng":-8.61094216646498},{"lat":40.9788635341848,"lng":-8.61075857090709},{"lat":40.978029510328,"lng":-8.61050640641883},{"lat":40.9773795440724,"lng":-8.61050692703816},{"lat":40.9767303782522,"lng":-8.6104974296615},{"lat":40.9760236932997,"lng":-8.61056982644847},{"lat":40.9746611399157,"lng":-8.61056864471949},{"lat":40.9743179995735,"lng":-8.61053754888288},{"lat":40.9740561645837,"lng":-8.61058315480281},{"lat":40.9736090388876,"lng":-8.61048532096262},{"lat":40.9733397919129,"lng":-8.6104066182491}]},{"zone_id":"smf5","points":[{"lat":40.9630516874593,"lng":-8.5566086405807},{"lat":40.960392332284,"lng":-8.55542086635219},{"lat":40.9577178839103,"lng":-8.55345480539983},{"lat":40.957342294188,"lng":-8.56105077641671},{"lat":40.9571231608013,"lng":-8.56380464700042},{"lat":40.956842307983,"lng":-8.56733291775678},{"lat":40.9553805829636,"lng":-8.57580840109876},{"lat":40.9536824514557,"lng":-8.58365339315088},{"lat":40.9473248517512,"lng":-8.60387299545747},{"lat":40.947279009122,"lng":-8.60400016725394},{"lat":40.9481041893467,"lng":-8.60417722070992},{"lat":40.9482751179164,"lng":-8.60432506909448},{"lat":40.9489071503321,"lng":-8.6044044722415},{"lat":40.9493565957202,"lng":-8.60435150924385},{"lat":40.9495315241351,"lng":-8.60413586943492},{"lat":40.9496906154316,"lng":-8.60409555441127},{"lat":40.9503626627387,"lng":-8.60392525010645},{"lat":40.9505079124889,"lng":-8.60388844155631},{"lat":40.9515351173093,"lng":-8.60384390882592},{"lat":40.9519529822203,"lng":-8.603822263054},{"lat":40.952362576374,"lng":-8.60374999683639},{"lat":40.9525952496099,"lng":-8.60374525052996},{"lat":40.9529737403974,"lng":-8.60375994585741},{"lat":40.953201390698,"lng":-8.60375748339031},{"lat":40.9533710406185,"lng":-8.60374341116132},{"lat":40.9537920071762,"lng":-8.60368548248425},{"lat":40.9542281902276,"lng":-8.60365379338705},{"lat":40.9546219395373,"lng":-8.60365406717107},{"lat":40.9550606071685,"lng":-8.60360862252151},{"lat":40.9551852637477,"lng":-8.60358408580437},{"lat":40.9554971314499,"lng":-8.60358302314892},{"lat":40.9556479885318,"lng":-8.60355973546844},{"lat":40.9579249370022,"lng":-8.60410477640124},{"lat":40.9599853951455,"lng":-8.60523674233452},{"lat":40.9632491847672,"lng":-8.60715989419907},{"lat":40.9650127909937,"lng":-8.60799273120286},{"lat":40.9656988861652,"lng":-8.60807650301706},{"lat":40.9683129530665,"lng":-8.60889441695011},{"lat":40.9694685329035,"lng":-8.60925684012166},{"lat":40.9709577829282,"lng":-8.6096927527008},{"lat":40.9717675666018,"lng":-8.60998188521863},{"lat":40.9725232298744,"lng":-8.61024943952886},{"lat":40.9730082108184,"lng":-8.61037167378274},{"lat":40.9733397919129,"lng":-8.6104066182491},{"lat":40.9736090388876,"lng":-8.61048532096262},{"lat":40.9740561645837,"lng":-8.61058315480281},{"lat":40.9743179995735,"lng":-8.61053754888288},{"lat":40.9746611399157,"lng":-8.61056864471949},{"lat":40.9760236932997,"lng":-8.61056982644847},{"lat":40.9767303782522,"lng":-8.6104974296615},{"lat":40.9773795440724,"lng":-8.61050692703816},{"lat":40.978029510328,"lng":-8.61050640641883},{"lat":40.9788635341848,"lng":-8.61075857090709},{"lat":40.9793553653416,"lng":-8.61094216646498},{"lat":40.9797763679532,"lng":-8.61115009053317},{"lat":40.980408629792,"lng":-8.6114389425863},{"lat":40.981110645132,"lng":-8.61185528815423},{"lat":40.9813786296303,"lng":-8.61165690056523},{"lat":40.9815882764073,"lng":-8.61150169880792},{"lat":40.9816963534269,"lng":-8.61160454980459},{"lat":40.9817429372698,"lng":-8.61174830462064},{"lat":40.981775592684,"lng":-8.61192867641461},{"lat":40.9817894320852,"lng":-8.61208621467158},{"lat":40.9820738496476,"lng":-8.61213428010213},{"lat":40.9828985375087,"lng":-8.61223511472722},{"lat":40.9840708813443,"lng":-8.61233011215465},{"lat":40.984411605334,"lng":-8.6123853671861},{"lat":40.9849969874872,"lng":-8.61251706227039},{"lat":40.9856287827367,"lng":-8.61273171587297},{"lat":40.9856769538719,"lng":-8.61234630258731},{"lat":40.9857456307639,"lng":-8.61196166749795},{"lat":40.9858177927228,"lng":-8.61155424753122},{"lat":40.9858919261004,"lng":-8.61115259572706},{"lat":40.9859538228065,"lng":-8.61073054776722},{"lat":40.9859494070974,"lng":-8.6104415878696},{"lat":40.9859079617596,"lng":-8.61027186756894},{"lat":40.9858162113575,"lng":-8.60994566548625},{"lat":40.9854876259008,"lng":-8.60913214827149},{"lat":40.9852333007293,"lng":-8.60850820601676},{"lat":40.9850285784716,"lng":-8.60801020831219},{"lat":40.9848743417957,"lng":-8.60765665666493},{"lat":40.9847477931682,"lng":-8.6074064479022},{"lat":40.9845756736215,"lng":-8.60704691050586},{"lat":40.9848550602467,"lng":-8.60685051158719},{"lat":40.985288781657,"lng":-8.60662637104825},{"lat":40.9855931486719,"lng":-8.60648052047919},{"lat":40.9858085015609,"lng":-8.60636032626354},{"lat":40.9860380615175,"lng":-8.6062504046593},{"lat":40.9863224728709,"lng":-8.60613887532659},{"lat":40.9865803071616,"lng":-8.60605151926503},{"lat":40.9867153402315,"lng":-8.60602739182722},{"lat":40.9868443931522,"lng":-8.60605598184653},{"lat":40.9871210594847,"lng":-8.60610319354898},{"lat":40.9874253772086,"lng":-8.60614966107442},{"lat":40.9875998225939,"lng":-8.60617742400742},{"lat":40.9877008601797,"lng":-8.60619370837052},{"lat":40.9879314785954,"lng":-8.60608372089888},{"lat":40.9880699638384,"lng":-8.60601507307684},{"lat":40.9882013008233,"lng":-8.60597438791468},{"lat":40.9884493213144,"lng":-8.60594360612615},{"lat":40.988773522345,"lng":-8.60591671750904},{"lat":40.988994038661,"lng":-8.60591317047123},{"lat":40.9890466140855,"lng":-8.60387790866511},{"lat":40.9891649222613,"lng":-8.60359509380953},{"lat":40.9892174670602,"lng":-8.60337143827351},{"lat":40.9892452673859,"lng":-8.6031643102388},{"lat":40.9893031272939,"lng":-8.60289483474736},{"lat":40.9913899711882,"lng":-8.60262643919769},{"lat":40.9924337461157,"lng":-8.60249217474789},{"lat":40.9925138149352,"lng":-8.60231127396604},{"lat":40.9936555661762,"lng":-8.60260213062562},{"lat":40.9947640085964,"lng":-8.60246332804797},{"lat":40.9970831182338,"lng":-8.6021729073629},{"lat":40.9976761889144,"lng":-8.60193407671393},{"lat":40.9984059518456,"lng":-8.60254568752981},{"lat":40.9987113715699,"lng":-8.60273888756241},{"lat":40.9991116991519,"lng":-8.6028950585872},{"lat":40.9994566646807,"lng":-8.60286349240394},{"lat":40.9997743542304,"lng":-8.60283858643202},{"lat":41.0000411770317,"lng":-8.60279825702812},{"lat":41.0002465081836,"lng":-8.60281540595865},{"lat":41.0005812411011,"lng":-8.6026551133872},{"lat":41.0014450231517,"lng":-8.60289724464794},{"lat":41.0019121220251,"lng":-8.60252722696788},{"lat":41.0023819212027,"lng":-8.60213536101665},{"lat":41.0025454214619,"lng":-8.60199504484612},{"lat":41.0028115969741,"lng":-8.60176501016289},{"lat":41.0031302645701,"lng":-8.60148507051449},{"lat":41.0032005143612,"lng":-8.60156225077716},{"lat":41.0035328518238,"lng":-8.60220598651342},{"lat":41.0037320984364,"lng":-8.6016601683899},{"lat":41.0042539206608,"lng":-8.60192169219949},{"lat":41.0047050333097,"lng":-8.602145036441},{"lat":41.0048122863509,"lng":-8.60221822863212},{"lat":41.0053401570563,"lng":-8.60246609367049},{"lat":41.0056938330268,"lng":-8.60227272503765},{"lat":41.005966497025,"lng":-8.60211633825503},{"lat":41.0061140941027,"lng":-8.60204678264952},{"lat":41.0063781588468,"lng":-8.60192349697455},{"lat":41.0080333709373,"lng":-8.60161195510289},{"lat":41.0082883093815,"lng":-8.60154042598339},{"lat":41.0085421658054,"lng":-8.60146924527322},{"lat":41.0102447893234,"lng":-8.60287348172461},{"lat":41.0111395308336,"lng":-8.60073023631636},{"lat":41.0141061653569,"lng":-8.5936229239907},{"lat":41.0142719930551,"lng":-8.58878872731236},{"lat":41.0142531046113,"lng":-8.5859600151318},{"lat":41.0139476306899,"lng":-8.58358576830958},{"lat":41.0135271104059,"lng":-8.57349709604347},{"lat":41.0139527516552,"lng":-8.57307447239083},{"lat":41.0140595263932,"lng":-8.5725388007083},{"lat":41.0140754583723,"lng":-8.57171649635713},{"lat":41.013967452399,"lng":-8.56941284083917},{"lat":41.0140069669259,"lng":-8.56920351786439},{"lat":41.0141335143544,"lng":-8.56898121940691},{"lat":41.0141643418625,"lng":-8.56880655057063},{"lat":41.0140555259426,"lng":-8.5686753009863},{"lat":41.0139208415555,"lng":-8.56852711811271},{"lat":41.0138075516494,"lng":-8.56607971290502},{"lat":41.0147468403312,"lng":-8.5653783106167},{"lat":41.0148112932049,"lng":-8.5651922100807},{"lat":41.0148627751128,"lng":-8.56503098742801},{"lat":41.0149140862219,"lng":-8.56491505889283},{"lat":41.014790279471,"lng":-8.56469180795541},{"lat":41.0146871807351,"lng":-8.56439073659539},{"lat":41.0144345953301,"lng":-8.56364595560271},{"lat":41.0130157821258,"lng":-8.56365426107199},{"lat":41.0121078727299,"lng":-8.5642124068748},{"lat":41.0089837061203,"lng":-8.56625395557585},{"lat":41.0065498757587,"lng":-8.56753671204608},{"lat":41.00317280816,"lng":-8.56846191647153},{"lat":40.9990021783538,"lng":-8.56944599436281},{"lat":40.9953013347889,"lng":-8.57426722172144},{"lat":40.9924872370392,"lng":-8.57314686249403},{"lat":40.9889275674466,"lng":-8.5722241979788},{"lat":40.9890752181941,"lng":-8.56761715899663},{"lat":40.9901597490253,"lng":-8.56402986436183},{"lat":40.9858552026253,"lng":-8.56277133075019},{"lat":40.9829114237997,"lng":-8.56272007323348},{"lat":40.9803753454449,"lng":-8.56295713284177},{"lat":40.9730962116834,"lng":-8.56363891989827},{"lat":40.9698479737872,"lng":-8.56094630975853},{"lat":40.966169586604,"lng":-8.55754000247438},{"lat":40.9630516874593,"lng":-8.5566086405807}]},{"zone_id":"vng8","points":[{"lat":41.0315456007987,"lng":-8.64678282473931},{"lat":41.0315989771323,"lng":-8.64672663996691},{"lat":41.0316511051941,"lng":-8.64672906707576},{"lat":41.0318717065187,"lng":-8.64673138772025},{"lat":41.032086975983,"lng":-8.64671867565435},{"lat":41.032332834248,"lng":-8.64673165941631},{"lat":41.0325630891309,"lng":-8.64679077090229},{"lat":41.0326557122628,"lng":-8.64683941835372},{"lat":41.0327058010406,"lng":-8.64689392023812},{"lat":41.0328016748727,"lng":-8.64698076257357},{"lat":41.0329661070205,"lng":-8.6471000173255},{"lat":41.0331402483331,"lng":-8.64722196008909},{"lat":41.0333156129142,"lng":-8.6472729233128},{"lat":41.033539285679,"lng":-8.64735457960342},{"lat":41.0336443446541,"lng":-8.6474397212883},{"lat":41.0337391795302,"lng":-8.64757757240346},{"lat":41.0338071978462,"lng":-8.64765076930578},{"lat":41.0338699044198,"lng":-8.64772355931463},{"lat":41.0339819472226,"lng":-8.64783788914289},{"lat":41.0341247556657,"lng":-8.64793948876707},{"lat":41.0342656414702,"lng":-8.64802883330537},{"lat":41.0344599586978,"lng":-8.64810931987798},{"lat":41.0345827193961,"lng":-8.648223496106},{"lat":41.0347117444381,"lng":-8.64836685462858},{"lat":41.0348092930873,"lng":-8.64852090515503},{"lat":41.0349266767473,"lng":-8.64865026493749},{"lat":41.0350579478594,"lng":-8.64867424010494},{"lat":41.0352234584502,"lng":-8.64871312339574},{"lat":41.03532385252,"lng":-8.64879477729717},{"lat":41.0354630053183,"lng":-8.64888779322934},{"lat":41.035613073261,"lng":-8.64899730917415},{"lat":41.0357726569051,"lng":-8.64911224979322},{"lat":41.0359019794084,"lng":-8.64918914131696},{"lat":41.035966894109,"lng":-8.64915100117285},{"lat":41.0360302091843,"lng":-8.64908810762896},{"lat":41.0361398271909,"lng":-8.64918185036771},{"lat":41.0362412831606,"lng":-8.64920738070139},{"lat":41.0362945882174,"lng":-8.64916843822455},{"lat":41.0362341137527,"lng":-8.64903916240906},{"lat":41.0362934645122,"lng":-8.64903772741919},{"lat":41.0363580886426,"lng":-8.64906427778612},{"lat":41.0364148294693,"lng":-8.64910171137152},{"lat":41.0364705755085,"lng":-8.64907943466345},{"lat":41.0365372269209,"lng":-8.64909601295713},{"lat":41.0365112608175,"lng":-8.64924327143885},{"lat":41.0366341362575,"lng":-8.6493719612711},{"lat":41.0367604892639,"lng":-8.64930790073494},{"lat":41.0368543233963,"lng":-8.64928723218882},{"lat":41.0369443481986,"lng":-8.64929126815969},{"lat":41.0370643134724,"lng":-8.64920646298546},{"lat":41.0370251850368,"lng":-8.64933673973794},{"lat":41.037076969625,"lng":-8.64939542054524},{"lat":41.0373304924408,"lng":-8.64952667944015},{"lat":41.0374537344088,"lng":-8.64953347533457},{"lat":41.0378388935506,"lng":-8.64960738152759},{"lat":41.0379578225973,"lng":-8.64957180476651},{"lat":41.0380699176633,"lng":-8.64967519999685},{"lat":41.038194102885,"lng":-8.64971280604495},{"lat":41.0382741695231,"lng":-8.64968893928932},{"lat":41.0383786713816,"lng":-8.64971853572758},{"lat":41.0384551590595,"lng":-8.64990977980314},{"lat":41.0386941556818,"lng":-8.65002737879034},{"lat":41.0388947628664,"lng":-8.65009068069845},{"lat":41.0389781464153,"lng":-8.65013033973555},{"lat":41.0390499171731,"lng":-8.65019001742485},{"lat":41.0392334496241,"lng":-8.65024592878819},{"lat":41.0394769536422,"lng":-8.65034132259127},{"lat":41.0396910672179,"lng":-8.65042554033484},{"lat":41.0397880677661,"lng":-8.65044033927589},{"lat":41.0398754791514,"lng":-8.65044590494508},{"lat":41.0399724451644,"lng":-8.65048959955436},{"lat":41.0400819261297,"lng":-8.65059226785977},{"lat":41.0402650012648,"lng":-8.65072952903912},{"lat":41.0404166794064,"lng":-8.65080065039001},{"lat":41.0405794270609,"lng":-8.65089303953637},{"lat":41.040737115784,"lng":-8.65104948197606},{"lat":41.0408669379186,"lng":-8.6512149898919},{"lat":41.0409959887084,"lng":-8.65137073622029},{"lat":41.0411197515312,"lng":-8.65144271643875},{"lat":41.0411849035538,"lng":-8.65141123661942},{"lat":41.041244493591,"lng":-8.65135675792343},{"lat":41.0412926433412,"lng":-8.65148142268462},{"lat":41.0414015370241,"lng":-8.65149548657875},{"lat":41.0415055519212,"lng":-8.65157266175611},{"lat":41.041657809299,"lng":-8.65157552374946},{"lat":41.0417591244631,"lng":-8.65163210881113},{"lat":41.041874387281,"lng":-8.65165085553182},{"lat":41.04194022188,"lng":-8.65164875618551},{"lat":41.0420384864692,"lng":-8.65158412125274},{"lat":41.0421543613049,"lng":-8.65164675635642},{"lat":41.0423325200669,"lng":-8.65151626439233},{"lat":41.0425287959177,"lng":-8.65148212925821},{"lat":41.0426628215305,"lng":-8.65153444687944},{"lat":41.0427668884103,"lng":-8.65160009310145},{"lat":41.0428091145629,"lng":-8.65166143698437},{"lat":41.0428507623651,"lng":-8.65179069260292},{"lat":41.0429388630268,"lng":-8.65184193767142},{"lat":41.0431880021503,"lng":-8.65188695666118},{"lat":41.0432765717564,"lng":-8.65201372563747},{"lat":41.043350437512,"lng":-8.6521874866062},{"lat":41.0435091422094,"lng":-8.65237701133369},{"lat":41.0435516799184,"lng":-8.65248843755789},{"lat":41.0436347161055,"lng":-8.65248564411935},{"lat":41.0437090359456,"lng":-8.65253880229924},{"lat":41.0438698399857,"lng":-8.65262225924869},{"lat":41.0441179457609,"lng":-8.65271544534844},{"lat":41.044301138141,"lng":-8.6528261967857},{"lat":41.0443939747311,"lng":-8.65282705024126},{"lat":41.0444412850449,"lng":-8.65269980324241},{"lat":41.0444931961986,"lng":-8.6527890603849},{"lat":41.0445574900143,"lng":-8.65288840486224},{"lat":41.0446404298454,"lng":-8.65290690572999},{"lat":41.0447384144237,"lng":-8.65288317560399},{"lat":41.0448309178904,"lng":-8.65295764727367},{"lat":41.0448919128244,"lng":-8.65302984952373},{"lat":41.0450525875167,"lng":-8.6531017768504},{"lat":41.0452260557274,"lng":-8.65319306673904},{"lat":41.0453324901268,"lng":-8.65317345128446},{"lat":41.0454818524052,"lng":-8.6531789129849},{"lat":41.0456966361129,"lng":-8.65329241701215},{"lat":41.0458744256433,"lng":-8.6533638764849},{"lat":41.0461522012671,"lng":-8.65348739802363},{"lat":41.0463774797829,"lng":-8.6536300056174},{"lat":41.0465905324776,"lng":-8.6537489756481},{"lat":41.0467057193281,"lng":-8.65372443187827},{"lat":41.0467877761138,"lng":-8.65369903415277},{"lat":41.0469353742226,"lng":-8.65371626079718},{"lat":41.0470491376627,"lng":-8.65388808292599},{"lat":41.0472158999483,"lng":-8.65404771717505},{"lat":41.0475080587461,"lng":-8.65433466367817},{"lat":41.0474823486834,"lng":-8.65442402860063},{"lat":41.0475189129308,"lng":-8.65454183605558},{"lat":41.0475826952902,"lng":-8.65457504960252},{"lat":41.047658731679,"lng":-8.65448704082172},{"lat":41.0478254514168,"lng":-8.65445743649962},{"lat":41.047915544007,"lng":-8.6544475629424},{"lat":41.0479422074945,"lng":-8.654525087462},{"lat":41.0479847030824,"lng":-8.65464532232834},{"lat":41.0480703145689,"lng":-8.65463054150163},{"lat":41.048148155006,"lng":-8.65454194825497},{"lat":41.0482358390763,"lng":-8.65450731696692},{"lat":41.0482694151394,"lng":-8.65460833269507},{"lat":41.0482140857851,"lng":-8.65469710462472},{"lat":41.0482524375395,"lng":-8.65481789876302},{"lat":41.0484435966524,"lng":-8.65491897570894},{"lat":41.0485514162031,"lng":-8.65497109709133},{"lat":41.0486700623496,"lng":-8.65505791374727},{"lat":41.0487762427595,"lng":-8.65527250699313},{"lat":41.0489117036373,"lng":-8.65536398445908},{"lat":41.0490122203871,"lng":-8.65541866107509},{"lat":41.0490585531816,"lng":-8.65532886674163},{"lat":41.0491397701798,"lng":-8.65529014346854},{"lat":41.0491862856232,"lng":-8.6553771062877},{"lat":41.0493238726604,"lng":-8.65561621188713},{"lat":41.0494754045365,"lng":-8.65571875911859},{"lat":41.0497292273747,"lng":-8.65576134215579},{"lat":41.0499974072899,"lng":-8.6557770337708},{"lat":41.0501073542609,"lng":-8.65579610687953},{"lat":41.0501966366341,"lng":-8.65576636772543},{"lat":41.0502990699134,"lng":-8.65577467310588},{"lat":41.0504188101307,"lng":-8.65579894223277},{"lat":41.050540610308,"lng":-8.65574757342262},{"lat":41.0506361086322,"lng":-8.65571633471329},{"lat":41.0508359361125,"lng":-8.65577216186051},{"lat":41.051230257156,"lng":-8.65594908736535},{"lat":41.0513291336003,"lng":-8.65598710323242},{"lat":41.0515870881716,"lng":-8.65611238958141},{"lat":41.0517468011167,"lng":-8.65621714694983},{"lat":41.0518258201512,"lng":-8.65628546193237},{"lat":41.0519591891172,"lng":-8.65628366520853},{"lat":41.0520821924478,"lng":-8.6563426924572},{"lat":41.0522542610299,"lng":-8.65630540304862},{"lat":41.0524666135385,"lng":-8.65633992963186},{"lat":41.0527006657709,"lng":-8.65635392358188},{"lat":41.052779485309,"lng":-8.65626938207283},{"lat":41.0529327889496,"lng":-8.65623907508303},{"lat":41.0531683222415,"lng":-8.65628472747529},{"lat":41.0533535905113,"lng":-8.65633438048899},{"lat":41.0536604997885,"lng":-8.65638642465818},{"lat":41.0538649713784,"lng":-8.65635272269716},{"lat":41.0539181095325,"lng":-8.65634981546619},{"lat":41.0540306336237,"lng":-8.65635761101061},{"lat":41.0541430491888,"lng":-8.656388125881},{"lat":41.0543166177465,"lng":-8.65641698733034},{"lat":41.0544986156138,"lng":-8.65641355726262},{"lat":41.0546607600143,"lng":-8.65642079750384},{"lat":41.0547758236853,"lng":-8.65642338208658},{"lat":41.0549485492183,"lng":-8.65649922025906},{"lat":41.0550788638029,"lng":-8.65655485891833},{"lat":41.0552525296626,"lng":-8.65654137193427},{"lat":41.0553831472093,"lng":-8.65653052006875},{"lat":41.0554652772256,"lng":-8.65652891403012},{"lat":41.0555718552578,"lng":-8.65661588589231},{"lat":41.055717027785,"lng":-8.6566120500251},{"lat":41.0558295035908,"lng":-8.65663042850456},{"lat":41.0559517642881,"lng":-8.65665472140113},{"lat":41.056072700868,"lng":-8.65661404541394},{"lat":41.0562785517491,"lng":-8.65665197334114},{"lat":41.0564521841976,"lng":-8.65670652934941},{"lat":41.0565594941281,"lng":-8.6566726446637},{"lat":41.0567798175235,"lng":-8.65673566957943},{"lat":41.0570062991499,"lng":-8.65681205893147},{"lat":41.0571410699476,"lng":-8.65687689576942},{"lat":41.0572298593944,"lng":-8.65697515572893},{"lat":41.0573870765988,"lng":-8.65705433131757},{"lat":41.0574928119497,"lng":-8.65718805206927},{"lat":41.0576175020199,"lng":-8.6572522181341},{"lat":41.057788366043,"lng":-8.65730140348521},{"lat":41.057956711502,"lng":-8.65733022737005},{"lat":41.0580885773264,"lng":-8.65740087299648},{"lat":41.0582156069872,"lng":-8.65738677635546},{"lat":41.0583544415031,"lng":-8.65735053085202},{"lat":41.0584941495297,"lng":-8.65732023516344},{"lat":41.058672817883,"lng":-8.65725801557139},{"lat":41.0588727394992,"lng":-8.65723343814031},{"lat":41.0591315297823,"lng":-8.65719505075967},{"lat":41.0593870946156,"lng":-8.65719197674019},{"lat":41.0596283629016,"lng":-8.65722340359002},{"lat":41.0598316217752,"lng":-8.6572580973787},{"lat":41.0599819406281,"lng":-8.65730962014468},{"lat":41.0602029890498,"lng":-8.65739228072424},{"lat":41.0603464309078,"lng":-8.65735309271076},{"lat":41.0604437216657,"lng":-8.65730402331957},{"lat":41.0605595385534,"lng":-8.65720215802987},{"lat":41.0606699298834,"lng":-8.65706527469547},{"lat":41.0607793632779,"lng":-8.65695966818133},{"lat":41.0608608553258,"lng":-8.65686181608591},{"lat":41.061052975278,"lng":-8.65681016409134},{"lat":41.0611863320917,"lng":-8.65681098755745},{"lat":41.0612756655789,"lng":-8.65696648320915},{"lat":41.061251940653,"lng":-8.65723350078683},{"lat":41.0612689609086,"lng":-8.6574500398604},{"lat":41.0613474221525,"lng":-8.65758081919399},{"lat":41.0614586073472,"lng":-8.65770436120047},{"lat":41.0615721516215,"lng":-8.65786219268332},{"lat":41.0617238257246,"lng":-8.65797273645605},{"lat":41.0618934018158,"lng":-8.65794910777334},{"lat":41.0620498960714,"lng":-8.65796939093845},{"lat":41.0621542169752,"lng":-8.65797855415289},{"lat":41.0622677300212,"lng":-8.65796696919971},{"lat":41.0623566336916,"lng":-8.65804000994597},{"lat":41.0625794714356,"lng":-8.65810424861182},{"lat":41.0626567843761,"lng":-8.65817125390298},{"lat":41.062735525551,"lng":-8.65824172111673},{"lat":41.0628913796027,"lng":-8.65824463532732},{"lat":41.0628533708124,"lng":-8.65806944035738},{"lat":41.0627584499581,"lng":-8.65797421950101},{"lat":41.0626660149012,"lng":-8.65788556047051},{"lat":41.06259113433,"lng":-8.65775968641011},{"lat":41.062643918155,"lng":-8.65759746957926},{"lat":41.0627253991895,"lng":-8.65769829312781},{"lat":41.0628016948289,"lng":-8.65775232621253},{"lat":41.0628735729861,"lng":-8.65778644636393},{"lat":41.0629158542146,"lng":-8.65783496934938},{"lat":41.0629849463682,"lng":-8.65784896324068},{"lat":41.0630391010703,"lng":-8.65782072216364},{"lat":41.0630168871801,"lng":-8.65775546660834},{"lat":41.0629604849942,"lng":-8.65772385027164},{"lat":41.0628849529344,"lng":-8.65769980625125},{"lat":41.0628688890485,"lng":-8.65762876949657},{"lat":41.0629169441697,"lng":-8.65759631816178},{"lat":41.0633355546827,"lng":-8.65779667496706},{"lat":41.0636241251036,"lng":-8.65807844667635},{"lat":41.063721618709,"lng":-8.65804567843008},{"lat":41.0638155400459,"lng":-8.65800514308472},{"lat":41.0640616169319,"lng":-8.6581471388289},{"lat":41.0641989641851,"lng":-8.65820082306825},{"lat":41.0641195200607,"lng":-8.6580484963061},{"lat":41.0640233032703,"lng":-8.65786129916802},{"lat":41.0643180790045,"lng":-8.65794788499893},{"lat":41.064401879155,"lng":-8.65795521549019},{"lat":41.0645943378047,"lng":-8.65783075402114},{"lat":41.0650546778454,"lng":-8.65758897919164},{"lat":41.0656111058452,"lng":-8.65736426903029},{"lat":41.0662311533738,"lng":-8.65746618081326},{"lat":41.0666033427261,"lng":-8.65759788359412},{"lat":41.0671164880454,"lng":-8.65780067504218},{"lat":41.0673246729962,"lng":-8.65813715019132},{"lat":41.0675064027612,"lng":-8.65821224758607},{"lat":41.0676304717648,"lng":-8.65801810985703},{"lat":41.0678237134983,"lng":-8.65772220180766},{"lat":41.0678347031584,"lng":-8.6574840906212},{"lat":41.0678971781691,"lng":-8.65760618506881},{"lat":41.0681845003331,"lng":-8.65839780177762},{"lat":41.0681796825298,"lng":-8.65866344879844},{"lat":41.0683277533915,"lng":-8.65875422957539},{"lat":41.0684997357816,"lng":-8.65893277139202},{"lat":41.0686325819478,"lng":-8.65904519914737},{"lat":41.0686800145482,"lng":-8.65908947980176},{"lat":41.0687868446611,"lng":-8.65915946755436},{"lat":41.0688467049513,"lng":-8.65930165668865},{"lat":41.0688837425884,"lng":-8.65945341697837},{"lat":41.0689676930268,"lng":-8.65942791321663},{"lat":41.0690779809335,"lng":-8.65948995105827},{"lat":41.0690479261611,"lng":-8.65936811219351},{"lat":41.0689905273218,"lng":-8.65912135819028},{"lat":41.0690586075009,"lng":-8.65900136851514},{"lat":41.0692697157219,"lng":-8.65891405748075},{"lat":41.0695356087167,"lng":-8.6588389656307},{"lat":41.0710609327927,"lng":-8.64916012510524},{"lat":41.0701019105036,"lng":-8.6442593414467},{"lat":41.0701073662969,"lng":-8.64245980108528},{"lat":41.0697378247396,"lng":-8.63948392846325},{"lat":41.0699865241568,"lng":-8.63564704241811},{"lat":41.0704040606612,"lng":-8.63398002881338},{"lat":41.0703850724356,"lng":-8.63179009344031},{"lat":41.0703828817532,"lng":-8.63140727122455},{"lat":41.0702527176611,"lng":-8.62691453952559},{"lat":41.0698294531824,"lng":-8.625414316996},{"lat":41.0694105922413,"lng":-8.62457589451279},{"lat":41.0689576896548,"lng":-8.6231097571957},{"lat":41.0683063132082,"lng":-8.62002305901483},{"lat":41.0652003196112,"lng":-8.6221701181265},{"lat":41.0645331708597,"lng":-8.62221706975845},{"lat":41.0629616349948,"lng":-8.62215483436784},{"lat":41.0614660855257,"lng":-8.62281523227822},{"lat":41.0541519140919,"lng":-8.62545509223732},{"lat":41.0510620440154,"lng":-8.62636108243025},{"lat":41.049680139701,"lng":-8.62616655924091},{"lat":41.0463165622378,"lng":-8.62514844756572},{"lat":41.0448917951489,"lng":-8.62398580557813},{"lat":41.04341859331,"lng":-8.62289588982128},{"lat":41.0427221197396,"lng":-8.62250716141593},{"lat":41.0414330927232,"lng":-8.61978637342328},{"lat":41.0404503017884,"lng":-8.61709656518184},{"lat":41.0382857602213,"lng":-8.6153111257162},{"lat":41.0369705338497,"lng":-8.61282645888687},{"lat":41.0361276452474,"lng":-8.60988078082951},{"lat":41.0357907350377,"lng":-8.60969847899555},{"lat":41.0340222807846,"lng":-8.60726972860741},{"lat":41.0331892049524,"lng":-8.60753743973004},{"lat":41.0328812567676,"lng":-8.6076363979517},{"lat":41.0286990129931,"lng":-8.60891853958328},{"lat":41.0243049061779,"lng":-8.60874311904242},{"lat":41.0224907255652,"lng":-8.61194013875301},{"lat":41.0223357913007,"lng":-8.6143339743552},{"lat":41.021033621126,"lng":-8.61585294756277},{"lat":41.020482839506,"lng":-8.61834560349968},{"lat":41.0184858812136,"lng":-8.6304238840555},{"lat":41.0161960857146,"lng":-8.63148206731223},{"lat":41.0100911602574,"lng":-8.63163855882472},{"lat":41.0071232681718,"lng":-8.63088779228106},{"lat":41.0038119863156,"lng":-8.63084906704574},{"lat":41.0033722127487,"lng":-8.63370905249525},{"lat":41.0022137354879,"lng":-8.63411573552935},{"lat":41.0022135545583,"lng":-8.63441944721206},{"lat":41.0011141790844,"lng":-8.63513826941891},{"lat":40.998991840164,"lng":-8.64805969363469},{"lat":40.999017463595,"lng":-8.64798857581832},{"lat":40.9990767829952,"lng":-8.64795266982976},{"lat":40.9991474566667,"lng":-8.64793527655014},{"lat":40.9992292312719,"lng":-8.64793341857502},{"lat":40.999290609156,"lng":-8.64794008182938},{"lat":40.9993459938214,"lng":-8.64793896730683},{"lat":40.9994184130735,"lng":-8.64791493260378},{"lat":40.9994734087703,"lng":-8.64789955526837},{"lat":40.9995275096701,"lng":-8.64788286249526},{"lat":40.9995830604954,"lng":-8.6478648726102},{"lat":40.9996372492341,"lng":-8.64782856745212},{"lat":40.9996798670702,"lng":-8.64778278703466},{"lat":40.9997359594723,"lng":-8.64776408395039},{"lat":40.999787049562,"lng":-8.64773655471808},{"lat":40.9998519337275,"lng":-8.64770520796546},{"lat":40.999899336209,"lng":-8.64767752363153},{"lat":40.9999526066585,"lng":-8.64764513625202},{"lat":40.9999986172976,"lng":-8.64762600219019},{"lat":41.0000439982044,"lng":-8.64760591340922},{"lat":41.0000871424498,"lng":-8.6475841466837},{"lat":41.0001646642537,"lng":-8.64754540644407},{"lat":41.0002212920299,"lng":-8.64752849384744},{"lat":41.0002892636629,"lng":-8.64751107871758},{"lat":41.0003455328745,"lng":-8.64749333303515},{"lat":41.0004075526162,"lng":-8.647478010007},{"lat":41.0004813911228,"lng":-8.6474583823884},{"lat":41.0005400553346,"lng":-8.64742829489689},{"lat":41.0005953049799,"lng":-8.64739663313988},{"lat":41.0006545024822,"lng":-8.64736880792899},{"lat":41.0006965588164,"lng":-8.64734726481967},{"lat":41.0007402977148,"lng":-8.64733203128366},{"lat":41.0007997195663,"lng":-8.64731478816257},{"lat":41.0008681650368,"lng":-8.64729155146417},{"lat":41.0010257642101,"lng":-8.64722764531014},{"lat":41.0011439599211,"lng":-8.64717603088681},{"lat":41.0012826436614,"lng":-8.64711233520667},{"lat":41.00142106176,"lng":-8.64704839808002},{"lat":41.0015011854298,"lng":-8.64701289143699},{"lat":41.0015939231079,"lng":-8.64697414890285},{"lat":41.001687199624,"lng":-8.64693755581},{"lat":41.0017437962275,"lng":-8.64692646705179},{"lat":41.0017945235356,"lng":-8.64691925786477},{"lat":41.0019530251996,"lng":-8.64689541464365},{"lat":41.0020901823015,"lng":-8.64687152432656},{"lat":41.002147416588,"lng":-8.64685961017577},{"lat":41.002227073256,"lng":-8.64684751228089},{"lat":41.0022777623122,"lng":-8.64684885757094},{"lat":41.0023311432379,"lng":-8.646852482145},{"lat":41.0024171955548,"lng":-8.64685945681368},{"lat":41.0024886822886,"lng":-8.64686274445041},{"lat":41.0025773731096,"lng":-8.64686367557266},{"lat":41.0026336589298,"lng":-8.64686244759528},{"lat":41.0026899500349,"lng":-8.64686003737569},{"lat":41.0027459758175,"lng":-8.646856197044},{"lat":41.0028138238316,"lng":-8.64684614564702},{"lat":41.0029630503766,"lng":-8.64682151884286},{"lat":41.003095600579,"lng":-8.64680163046805},{"lat":41.003185720867,"lng":-8.64678557558752},{"lat":41.0032313244522,"lng":-8.64677641989499},{"lat":41.0032997100181,"lng":-8.64676768103347},{"lat":41.0033472965532,"lng":-8.64675842786716},{"lat":41.003391851296,"lng":-8.64674189121762},{"lat":41.0034389015856,"lng":-8.64673215560557},{"lat":41.003514957713,"lng":-8.64671955765987},{"lat":41.0035732549295,"lng":-8.64671074001107},{"lat":41.0036376923201,"lng":-8.64669935324418},{"lat":41.0036833843272,"lng":-8.64669067635594},{"lat":41.0037667253767,"lng":-8.64667944364321},{"lat":41.003820510542,"lng":-8.6466736793322},{"lat":41.0038745031281,"lng":-8.64666090915959},{"lat":41.0039282983837,"lng":-8.64665288657009},{"lat":41.0039786139828,"lng":-8.64667812070513},{"lat":41.0039741114722,"lng":-8.64673870859935},{"lat":41.0039968700307,"lng":-8.64680402584594},{"lat":41.0039974220837,"lng":-8.64688200916433},{"lat":41.0040159553772,"lng":-8.64694586544277},{"lat":41.0040365189332,"lng":-8.64699903707016},{"lat":41.0040474971057,"lng":-8.64706117390205},{"lat":41.0040753829196,"lng":-8.64712760074992},{"lat":41.0041092243823,"lng":-8.64717196235234},{"lat":41.0041548503035,"lng":-8.64715781870229},{"lat":41.0041661814531,"lng":-8.64708028663685},{"lat":41.0041754580551,"lng":-8.64701997497214},{"lat":41.0041682644653,"lng":-8.64693742292753},{"lat":41.004161186315,"lng":-8.6468684284929},{"lat":41.0041592597488,"lng":-8.64679625945},{"lat":41.0041598571413,"lng":-8.64672339938684},{"lat":41.0041717574642,"lng":-8.64665918261246},{"lat":41.0042139974311,"lng":-8.64663704820435},{"lat":41.0042589321907,"lng":-8.64659627022915},{"lat":41.0043097388663,"lng":-8.64657241580977},{"lat":41.0043632026137,"lng":-8.64655750255095},{"lat":41.0044140154075,"lng":-8.64653115064297},{"lat":41.0044672214958,"lng":-8.64651421608649},{"lat":41.0045215888587,"lng":-8.64651891761764},{"lat":41.0045737309698,"lng":-8.64649686021681},{"lat":41.00462666591,"lng":-8.6464797972765},{"lat":41.0046813453343,"lng":-8.64647546778795},{"lat":41.0047356645902,"lng":-8.64647066388544},{"lat":41.0047900003432,"lng":-8.64646216701628},{"lat":41.0048593539388,"lng":-8.6464383439925},{"lat":41.0049090092066,"lng":-8.6464086550039},{"lat":41.0049594795549,"lng":-8.64637897896749},{"lat":41.0050126776968,"lng":-8.64636381757627},{"lat":41.0050850355931,"lng":-8.64637258485988},{"lat":41.0051390921842,"lng":-8.64636575290393},{"lat":41.0051936560523,"lng":-8.64636701519478},{"lat":41.0052487599317,"lng":-8.64636791637199},{"lat":41.0053043959531,"lng":-8.64633078791248},{"lat":41.0053551927318,"lng":-8.646308008895},{"lat":41.0054097980387,"lng":-8.64629999216472},{"lat":41.0054635460051,"lng":-8.64630254994208},{"lat":41.0055152340117,"lng":-8.64632244804199},{"lat":41.0055625764088,"lng":-8.64630700835596},{"lat":41.0056180937212,"lng":-8.64627617567267},{"lat":41.0056709511658,"lng":-8.64629561130964},{"lat":41.00572604097,"lng":-8.64630078999415},{"lat":41.0057786333367,"lng":-8.64631986493056},{"lat":41.0058322299578,"lng":-8.6463349025358},{"lat":41.0058866190606,"lng":-8.64633473535172},{"lat":41.0059769939875,"lng":-8.64632248061171},{"lat":41.0060269919508,"lng":-8.64633772933787},{"lat":41.006073010475,"lng":-8.64637648099367},{"lat":41.006135827821,"lng":-8.64634476182551},{"lat":41.0061885577102,"lng":-8.64633304389399},{"lat":41.0062431337911,"lng":-8.64633156961719},{"lat":41.0062977324584,"lng":-8.64632391144383},{"lat":41.006352408125,"lng":-8.64632041864424},{"lat":41.0064067390781,"lng":-8.64631299084729},{"lat":41.0064616054327,"lng":-8.64630736066344},{"lat":41.0065167464405,"lng":-8.64629994580122},{"lat":41.0065711642221,"lng":-8.64629335559251},{"lat":41.0066236697497,"lng":-8.64631159315117},{"lat":41.0066710561023,"lng":-8.64634714751128},{"lat":41.0067239225714,"lng":-8.64636455765019},{"lat":41.0067745223206,"lng":-8.64636673894863},{"lat":41.006819910423,"lng":-8.6463449800901},{"lat":41.0068789911591,"lng":-8.64636350815005},{"lat":41.006932442668,"lng":-8.64635132411884},{"lat":41.0069852064901,"lng":-8.64633200734223},{"lat":41.0070541724246,"lng":-8.64635441199722},{"lat":41.0071070438902,"lng":-8.6463718289276},{"lat":41.0071618670259,"lng":-8.64637474727753},{"lat":41.0072155692826,"lng":-8.64638753427404},{"lat":41.0072698290842,"lng":-8.6463960345879},{"lat":41.007324152023,"lng":-8.64639040017009},{"lat":41.0073851190571,"lng":-8.64640906268926},{"lat":41.0074372882504,"lng":-8.64640233539412},{"lat":41.0074891939093,"lng":-8.64637349974135},{"lat":41.0075480573218,"lng":-8.64635957096509},{"lat":41.0076006010415,"lng":-8.64636925357723},{"lat":41.0076534974807,"lng":-8.64638107785062},{"lat":41.0077074727846,"lng":-8.64639243893491},{"lat":41.0077664167343,"lng":-8.6464213086686},{"lat":41.0078024066555,"lng":-8.64648827382368},{"lat":41.0078611296055,"lng":-8.64650466060421},{"lat":41.0079122584538,"lng":-8.64652919174923},{"lat":41.0079621873335,"lng":-8.64655989115765},{"lat":41.0080318381754,"lng":-8.64659003368674},{"lat":41.0080829627582,"lng":-8.64661551481763},{"lat":41.0081346310283,"lng":-8.64663981120755},{"lat":41.0081854695246,"lng":-8.64666850519724},{"lat":41.0082379584959,"lng":-8.64669043002223},{"lat":41.0082887186712,"lng":-8.6467163800933},{"lat":41.0083375587861,"lng":-8.64674861905232},{"lat":41.00838717413,"lng":-8.64678978512838},{"lat":41.0084429740273,"lng":-8.64683645975551},{"lat":41.0085074518664,"lng":-8.64689592280232},{"lat":41.0085710970388,"lng":-8.64694016785608},{"lat":41.0086319955831,"lng":-8.64699437542951},{"lat":41.0086831957007,"lng":-8.64704232309387},{"lat":41.0087267146176,"lng":-8.64707737728002},{"lat":41.008780545228,"lng":-8.64712106775379},{"lat":41.0088575991136,"lng":-8.64718799629511},{"lat":41.0089142168906,"lng":-8.64723289775},{"lat":41.0089723023944,"lng":-8.64727139393178},{"lat":41.0090477594105,"lng":-8.64733308258785},{"lat":41.0090967566519,"lng":-8.64739088462929},{"lat":41.0091423089849,"lng":-8.64745388763189},{"lat":41.0091971095964,"lng":-8.64754275654538},{"lat":41.0092507228472,"lng":-8.64763495095813},{"lat":41.0092900514556,"lng":-8.64770004457435},{"lat":41.0092898775402,"lng":-8.64779954417022},{"lat":41.0092419336671,"lng":-8.64788856002548},{"lat":41.0092198479341,"lng":-8.64799442457409},{"lat":41.00920227867,"lng":-8.64807714804611},{"lat":41.0091802189095,"lng":-8.64815697334299},{"lat":41.0091549629084,"lng":-8.64826732991801},{"lat":41.0091376931482,"lng":-8.64836288924043},{"lat":41.009100938366,"lng":-8.64846673881823},{"lat":41.0090779300525,"lng":-8.64851791321894},{"lat":41.009036426907,"lng":-8.64859557323593},{"lat":41.0089812060513,"lng":-8.648681322772},{"lat":41.0089259859432,"lng":-8.64876577020942},{"lat":41.008877667541,"lng":-8.64885846861268},{"lat":41.0088143012655,"lng":-8.64893274182751},{"lat":41.0087520947852,"lng":-8.6490086912925},{"lat":41.0087017796634,"lng":-8.64906416821188},{"lat":41.0086497676462,"lng":-8.64911654292221},{"lat":41.0086038277948,"lng":-8.64918037718481},{"lat":41.008552107748,"lng":-8.64922823038977},{"lat":41.0084834780697,"lng":-8.64929128873623},{"lat":41.0084378977108,"lng":-8.64935572337146},{"lat":41.0084179218395,"lng":-8.64941167019754},{"lat":41.0083699381299,"lng":-8.64944992726033},{"lat":41.0083800706748,"lng":-8.64951966773152},{"lat":41.0083712324995,"lng":-8.64960304943735},{"lat":41.0083717246582,"lng":-8.64967378374762},{"lat":41.0084105597361,"lng":-8.64974766864331},{"lat":41.0084820868468,"lng":-8.64980125132965},{"lat":41.0085639019805,"lng":-8.64980937970451},{"lat":41.0086411459652,"lng":-8.64977325886848},{"lat":41.0087104112711,"lng":-8.64970877769558},{"lat":41.0087512149258,"lng":-8.64964585329666},{"lat":41.008790864564,"lng":-8.64957995050348},{"lat":41.0088242702871,"lng":-8.6495396791354},{"lat":41.00891540036,"lng":-8.64941878687506},{"lat":41.0089767599706,"lng":-8.64934995854951},{"lat":41.0090168803024,"lng":-8.64931925190609},{"lat":41.0090578449354,"lng":-8.64928093269958},{"lat":41.0091098409739,"lng":-8.64923213143434},{"lat":41.0091516512194,"lng":-8.64920606785649},{"lat":41.0091949252517,"lng":-8.64915433509304},{"lat":41.0092280130771,"lng":-8.649105265851},{"lat":41.0092690273853,"lng":-8.64907491801311},{"lat":41.009303552323,"lng":-8.64902740104526},{"lat":41.0093502488272,"lng":-8.64897545572897},{"lat":41.0093966576957,"lng":-8.64890687474639},{"lat":41.0094322848998,"lng":-8.64881442829544},{"lat":41.0094593553175,"lng":-8.64874165022565},{"lat":41.0094699727261,"lng":-8.64868325092935},{"lat":41.0094770083204,"lng":-8.64862018030655},{"lat":41.0094839278555,"lng":-8.64852240056927},{"lat":41.0094834419404,"lng":-8.64845011737014},{"lat":41.0094803003245,"lng":-8.64838768446064},{"lat":41.0094817892941,"lng":-8.64831708404421},{"lat":41.0094710712856,"lng":-8.64825768071596},{"lat":41.0094896795965,"lng":-8.64818340797915},{"lat":41.0094973041168,"lng":-8.64811012539132},{"lat":41.0094987545769,"lng":-8.6480480804693},{"lat":41.0095148009153,"lng":-8.64798294127153},{"lat":41.009529106869,"lng":-8.64792552055864},{"lat":41.0095454975107,"lng":-8.64784528508506},{"lat":41.0095803810267,"lng":-8.64777601515128},{"lat":41.0095981051734,"lng":-8.64771933182963},{"lat":41.0096256416632,"lng":-8.64766367508671},{"lat":41.0096598775688,"lng":-8.64761877210286},{"lat":41.0096977981356,"lng":-8.64757579769209},{"lat":41.009748199823,"lng":-8.64750082303343},{"lat":41.0097856905201,"lng":-8.64743359247073},{"lat":41.0098259646415,"lng":-8.64736840961525},{"lat":41.0098625423616,"lng":-8.64730390858189},{"lat":41.0098990663399,"lng":-8.64725141052143},{"lat":41.0099401477997,"lng":-8.64718682495401},{"lat":41.0099761027886,"lng":-8.6471400284603},{"lat":41.0099779259577,"lng":-8.64705528764395},{"lat":41.0099860690438,"lng":-8.64698627972995},{"lat":41.0100066122047,"lng":-8.64692248351814},{"lat":41.0100570359052,"lng":-8.64690326098637},{"lat":41.0102144800137,"lng":-8.64687393967874},{"lat":41.0103006913512,"lng":-8.64686557833391},{"lat":41.0103642651925,"lng":-8.64686500501589},{"lat":41.0104290326576,"lng":-8.64685968476021},{"lat":41.0105139972614,"lng":-8.64684881594198},{"lat":41.0106413253146,"lng":-8.64682758275472},{"lat":41.0107747542154,"lng":-8.64681257494563},{"lat":41.0108738009817,"lng":-8.64681334790823},{"lat":41.0109589289586,"lng":-8.64678619193334},{"lat":41.0110792987979,"lng":-8.64673102552096},{"lat":41.0112351801164,"lng":-8.64664830889446},{"lat":41.0113343947562,"lng":-8.64661151731818},{"lat":41.0114901997417,"lng":-8.64656613270174},{"lat":41.0115823891912,"lng":-8.64652952521317},{"lat":41.0118018962566,"lng":-8.64647512436632},{"lat":41.0120002029802,"lng":-8.64643007056903},{"lat":41.0123752525422,"lng":-8.64636760831344},{"lat":41.0125877498068,"lng":-8.64635048546912},{"lat":41.012658474929,"lng":-8.64634176327079},{"lat":41.0129343573821,"lng":-8.64630658707202},{"lat":41.0130831381262,"lng":-8.64626114591181},{"lat":41.0131963660969,"lng":-8.64625251576618},{"lat":41.0133804925352,"lng":-8.64621662362822},{"lat":41.0135292732362,"lng":-8.64617118196815},{"lat":41.0136426668168,"lng":-8.64612545764268},{"lat":41.0138125985581,"lng":-8.64606140025713},{"lat":41.0139614216547,"lng":-8.64600644527469},{"lat":41.0140605097755,"lng":-8.64599794356169},{"lat":41.0142233396288,"lng":-8.64595237127111},{"lat":41.0146406893464,"lng":-8.64595562325902},{"lat":41.01476095479,"lng":-8.64598462145763},{"lat":41.0148882712179,"lng":-8.64596706559757},{"lat":41.0150863992832,"lng":-8.64594055490977},{"lat":41.0153693480649,"lng":-8.64589591768432},{"lat":41.0156244497928,"lng":-8.64579518605191},{"lat":41.0157306944435,"lng":-8.64577734589127},{"lat":41.0158226678639,"lng":-8.64576878821341},{"lat":41.0159078379599,"lng":-8.64573212259654},{"lat":41.0159575258587,"lng":-8.64569506095233},{"lat":41.016014075812,"lng":-8.64569550139192},{"lat":41.0159777774542,"lng":-8.64501137673095},{"lat":41.0160128007234,"lng":-8.64497134379868},{"lat":41.0160851821154,"lng":-8.64489463094107},{"lat":41.0162233789596,"lng":-8.64481902091114},{"lat":41.0163850602117,"lng":-8.64472731013883},{"lat":41.0165689632732,"lng":-8.64464088044006},{"lat":41.01675391656,"lng":-8.64456040431283},{"lat":41.0170018961942,"lng":-8.64448136749431},{"lat":41.0172914943624,"lng":-8.64439872712708},{"lat":41.017538376209,"lng":-8.64432324812443},{"lat":41.0177123414429,"lng":-8.64430462902144},{"lat":41.0177682941941,"lng":-8.64425608075987},{"lat":41.0178280837119,"lng":-8.64421599953141},{"lat":41.0178973267473,"lng":-8.64421689606765},{"lat":41.0179548550723,"lng":-8.64423945931186},{"lat":41.0180943595714,"lng":-8.64427585351255},{"lat":41.0182024310204,"lng":-8.64427145783342},{"lat":41.0183518548694,"lng":-8.64426334403928},{"lat":41.0185379592366,"lng":-8.64430973377721},{"lat":41.0186361565499,"lng":-8.64429896325725},{"lat":41.0187830230643,"lng":-8.64429903452738},{"lat":41.0189879917937,"lng":-8.64433403782232},{"lat":41.0191221826895,"lng":-8.64432972552532},{"lat":41.0192528264839,"lng":-8.64431290878862},{"lat":41.019395090812,"lng":-8.64433517421922},{"lat":41.0195677429972,"lng":-8.64436849848212},{"lat":41.0196820281679,"lng":-8.64442526695055},{"lat":41.0198224010901,"lng":-8.64444740513561},{"lat":41.0199536475527,"lng":-8.64447719919923},{"lat":41.0201020808504,"lng":-8.64450961837578},{"lat":41.0201757257534,"lng":-8.64453218832959},{"lat":41.0202588578446,"lng":-8.64448753701445},{"lat":41.0204551817206,"lng":-8.64446266686057},{"lat":41.0206784083791,"lng":-8.64450268343663},{"lat":41.0209205751527,"lng":-8.64453690788635},{"lat":41.021183840806,"lng":-8.64462408233623},{"lat":41.0217439965632,"lng":-8.64464983713229},{"lat":41.0221414333044,"lng":-8.64473710635182},{"lat":41.0224764144995,"lng":-8.64477610044972},{"lat":41.0227317035082,"lng":-8.64483527105534},{"lat":41.0229933293051,"lng":-8.64494824199249},{"lat":41.023215007126,"lng":-8.64509300728045},{"lat":41.023438765322,"lng":-8.64519545340376},{"lat":41.0236886296458,"lng":-8.64528062935482},{"lat":41.0240044291602,"lng":-8.64537952130217},{"lat":41.0242128179947,"lng":-8.64547567085098},{"lat":41.0243869813087,"lng":-8.64559426435759},{"lat":41.0245305745868,"lng":-8.64570203605594},{"lat":41.0246443731602,"lng":-8.64580791498243},{"lat":41.0247307683602,"lng":-8.64581917252905},{"lat":41.0248753663478,"lng":-8.64570175239728},{"lat":41.0250409376307,"lng":-8.64554573238548},{"lat":41.0252100484037,"lng":-8.64536322175951},{"lat":41.025262145791,"lng":-8.64523033601584},{"lat":41.0252330178576,"lng":-8.64505853349291},{"lat":41.0253259747218,"lng":-8.64484950925007},{"lat":41.0253739775112,"lng":-8.64494892944379},{"lat":41.0253577935298,"lng":-8.64516615896588},{"lat":41.0252909220321,"lng":-8.6454008336671},{"lat":41.0251451908959,"lng":-8.64557044392141},{"lat":41.0250392084707,"lng":-8.64573144651851},{"lat":41.0251106242889,"lng":-8.64589038287767},{"lat":41.0252578490513,"lng":-8.64599164673148},{"lat":41.0253642959304,"lng":-8.64590817350094},{"lat":41.0254939813049,"lng":-8.64574307614864},{"lat":41.0255544072303,"lng":-8.64568112347532},{"lat":41.0256630675473,"lng":-8.64574772248713},{"lat":41.0259218645431,"lng":-8.64588885889582},{"lat":41.0261094009845,"lng":-8.64603692906431},{"lat":41.0262531525486,"lng":-8.64631082253969},{"lat":41.02635519898,"lng":-8.64646679222288},{"lat":41.0264709186937,"lng":-8.64656484334988},{"lat":41.0265999967392,"lng":-8.64665633472854},{"lat":41.0267332115588,"lng":-8.64668912845618},{"lat":41.0268664155118,"lng":-8.64672322454919},{"lat":41.0269480367646,"lng":-8.64679496429274},{"lat":41.02701794991,"lng":-8.64682713792323},{"lat":41.0270837372932,"lng":-8.64683561838506},{"lat":41.0271323038904,"lng":-8.64684908064441},{"lat":41.027182613067,"lng":-8.64687456313832},{"lat":41.0273185762209,"lng":-8.64685648212094},{"lat":41.0274132128206,"lng":-8.64681607830306},{"lat":41.0274874034548,"lng":-8.64677836522775},{"lat":41.0275658030101,"lng":-8.64674461849532},{"lat":41.0276172388288,"lng":-8.6467004285794},{"lat":41.0276885301358,"lng":-8.6466651909764},{"lat":41.0278081506542,"lng":-8.64671678292627},{"lat":41.027877139952,"lng":-8.64677439838007},{"lat":41.0279778487868,"lng":-8.64684676676876},{"lat":41.028065792738,"lng":-8.64687397209896},{"lat":41.0282164420228,"lng":-8.64685397894668},{"lat":41.0283254764404,"lng":-8.6468367571139},{"lat":41.0284531814597,"lng":-8.64683240548288},{"lat":41.0285693726498,"lng":-8.64684520011011},{"lat":41.0286055566329,"lng":-8.64680790724995},{"lat":41.0286597190719,"lng":-8.6467784824549},{"lat":41.0287533583952,"lng":-8.64677993126431},{"lat":41.0288594779965,"lng":-8.64679003586233},{"lat":41.0289894036289,"lng":-8.64673242442293},{"lat":41.029102086334,"lng":-8.64670500450792},{"lat":41.0292135887307,"lng":-8.64675926524365},{"lat":41.0293667098597,"lng":-8.64675023465831},{"lat":41.029520459794,"lng":-8.64678175509996},{"lat":41.0296331507487,"lng":-8.64675136492219},{"lat":41.0297530898445,"lng":-8.64675194263855},{"lat":41.0298230098513,"lng":-8.64678257608942},{"lat":41.0298552826721,"lng":-8.64685477098428},{"lat":41.0298784462351,"lng":-8.64691048221914},{"lat":41.0300579998469,"lng":-8.64701153655151},{"lat":41.0301148016688,"lng":-8.64697475618881},{"lat":41.0301825039642,"lng":-8.64693758913014},{"lat":41.0301519622316,"lng":-8.64716233669237},{"lat":41.0302117254062,"lng":-8.64724866120856},{"lat":41.0303006872617,"lng":-8.64731035590296},{"lat":41.0303726028692,"lng":-8.64733802892689},{"lat":41.0304791230863,"lng":-8.64729795608344},{"lat":41.0305614013671,"lng":-8.64722392456958},{"lat":41.0306902858295,"lng":-8.64715726740222},{"lat":41.030813096545,"lng":-8.6471199394533},{"lat":41.0308952036097,"lng":-8.64706278398041},{"lat":41.0311031269456,"lng":-8.64708153247139},{"lat":41.0311941622601,"lng":-8.6470419358374},{"lat":41.0313139426502,"lng":-8.64705773637911},{"lat":41.0314954295262,"lng":-8.64692680069935},{"lat":41.0315546992691,"lng":-8.64694236098499},{"lat":41.0317669407494,"lng":-8.64700252783869},{"lat":41.0319299232597,"lng":-8.64704316601819},{"lat":41.0320391634996,"lng":-8.64697992203202},{"lat":41.0321050062008,"lng":-8.64691526899179},{"lat":41.0320261050001,"lng":-8.64688004557509},{"lat":41.0319009297306,"lng":-8.64686218296253},{"lat":41.0317396438643,"lng":-8.64686520256577},{"lat":41.0316344680319,"lng":-8.64684547620456},{"lat":41.0315456007987,"lng":-8.64678282473931}]},{"zone_id":"vng10","points":[{"lat":41.0243049061779,"lng":-8.60874311904242},{"lat":41.0286990129931,"lng":-8.60891853958328},{"lat":41.0328812567676,"lng":-8.6076363979517},{"lat":41.0331892049524,"lng":-8.60753743973004},{"lat":41.0355879517099,"lng":-8.60325031501586},{"lat":41.0351724499922,"lng":-8.59907649171214},{"lat":41.0345231707031,"lng":-8.59470989040889},{"lat":41.038393305312,"lng":-8.59142775510329},{"lat":41.0398638174721,"lng":-8.59122317571952},{"lat":41.0418938279843,"lng":-8.5893663679455},{"lat":41.0416756260913,"lng":-8.58662445604142},{"lat":41.0423743586374,"lng":-8.5855425327634},{"lat":41.0420213480354,"lng":-8.58479271063373},{"lat":41.0418034722089,"lng":-8.58294779996386},{"lat":41.0425509080166,"lng":-8.58249892265991},{"lat":41.0426445505852,"lng":-8.58114467034214},{"lat":41.0420585402223,"lng":-8.57988392849237},{"lat":41.0428096556898,"lng":-8.57630062531123},{"lat":41.0438193569828,"lng":-8.57540296528201},{"lat":41.0440866099258,"lng":-8.5729268457125},{"lat":41.0450447421252,"lng":-8.56951289456525},{"lat":41.0465440090778,"lng":-8.56975942657329},{"lat":41.0492863109803,"lng":-8.57049001357153},{"lat":41.0526871042344,"lng":-8.57238704756344},{"lat":41.0535081350042,"lng":-8.56962756413254},{"lat":41.0543155993931,"lng":-8.56476618922353},{"lat":41.0407403965346,"lng":-8.5497586017523},{"lat":41.0386801085497,"lng":-8.54608513161834},{"lat":41.0358189212989,"lng":-8.54218600702728},{"lat":41.0355524904001,"lng":-8.5428110932661},{"lat":41.0353287386692,"lng":-8.54316061119419},{"lat":41.0351183422794,"lng":-8.54348084440019},{"lat":41.0350103671336,"lng":-8.54360455542075},{"lat":41.0348468501921,"lng":-8.54376426326167},{"lat":41.0347038131519,"lng":-8.54392202659993},{"lat":41.034469471304,"lng":-8.54419830727072},{"lat":41.034228002081,"lng":-8.54472518311814},{"lat":41.0340265662484,"lng":-8.54500360184153},{"lat":41.0338842751051,"lng":-8.54516734896363},{"lat":41.0337572115837,"lng":-8.54536875206445},{"lat":41.0337163270503,"lng":-8.54549391555007},{"lat":41.0309540698777,"lng":-8.54785706041216},{"lat":41.0308167203777,"lng":-8.5494318707316},{"lat":41.0304983683998,"lng":-8.54986577663449},{"lat":41.0287837415946,"lng":-8.55201555733383},{"lat":41.0280448543804,"lng":-8.55270157739192},{"lat":41.0275854173237,"lng":-8.55320220298383},{"lat":41.0274373751696,"lng":-8.55335238467379},{"lat":41.0273349350286,"lng":-8.55361426973435},{"lat":41.0268264426633,"lng":-8.55442464159934},{"lat":41.0265521545893,"lng":-8.55475503206893},{"lat":41.0264199668419,"lng":-8.55467125831613},{"lat":41.025826233836,"lng":-8.55430850832225},{"lat":41.0254004101587,"lng":-8.55406113957289},{"lat":41.0247784786747,"lng":-8.55396787390071},{"lat":41.0238970013056,"lng":-8.55401348563629},{"lat":41.0231750125307,"lng":-8.55402462610707},{"lat":41.0230623104995,"lng":-8.55405117862271},{"lat":41.0227698623533,"lng":-8.55418357873114},{"lat":41.0214820965347,"lng":-8.55488976367121},{"lat":41.0212436782774,"lng":-8.55500293399092},{"lat":41.0210761967717,"lng":-8.55508615964072},{"lat":41.0208848440701,"lng":-8.55519234114339},{"lat":41.0207333997843,"lng":-8.55529230784128},{"lat":41.0204401353887,"lng":-8.55540777029594},{"lat":41.0201786080661,"lng":-8.55569328083992},{"lat":41.0200692394639,"lng":-8.55581078489836},{"lat":41.0199625952047,"lng":-8.55597072363775},{"lat":41.0198802526726,"lng":-8.55604005603181},{"lat":41.0197084270191,"lng":-8.55619495975368},{"lat":41.0195885888892,"lng":-8.55630124474929},{"lat":41.0194669309968,"lng":-8.55641286776898},{"lat":41.0193772386886,"lng":-8.55650584264009},{"lat":41.0192671484417,"lng":-8.55663609391451},{"lat":41.0190270353862,"lng":-8.55671995951289},{"lat":41.0187900649397,"lng":-8.55696791491036},{"lat":41.0184923827858,"lng":-8.55717578350338},{"lat":41.0179566597039,"lng":-8.55742438310342},{"lat":41.0161928786023,"lng":-8.55699403799249},{"lat":41.0156012861329,"lng":-8.55686476537534},{"lat":41.0155539036781,"lng":-8.55727332843291},{"lat":41.015509870781,"lng":-8.5575899853134},{"lat":41.0154777194957,"lng":-8.55793204032449},{"lat":41.015453310949,"lng":-8.55827414478322},{"lat":41.0154460440236,"lng":-8.55850996081375},{"lat":41.015503147161,"lng":-8.55933073769544},{"lat":41.0151263586705,"lng":-8.56023809975096},{"lat":41.0144345953301,"lng":-8.56364595560271},{"lat":41.0146871807351,"lng":-8.56439073659539},{"lat":41.014790279471,"lng":-8.56469180795541},{"lat":41.0149140862219,"lng":-8.56491505889283},{"lat":41.0148627751128,"lng":-8.56503098742801},{"lat":41.0148112932049,"lng":-8.5651922100807},{"lat":41.0147468403312,"lng":-8.5653783106167},{"lat":41.0138075516494,"lng":-8.56607971290502},{"lat":41.0139208415555,"lng":-8.56852711811271},{"lat":41.0140555259426,"lng":-8.5686753009863},{"lat":41.0141643418625,"lng":-8.56880655057063},{"lat":41.0141335143544,"lng":-8.56898121940691},{"lat":41.0140069669259,"lng":-8.56920351786439},{"lat":41.013967452399,"lng":-8.56941284083917},{"lat":41.0140754583723,"lng":-8.57171649635713},{"lat":41.0140595263932,"lng":-8.5725388007083},{"lat":41.0139527516552,"lng":-8.57307447239083},{"lat":41.0135271104059,"lng":-8.57349709604347},{"lat":41.0139476306899,"lng":-8.58358576830958},{"lat":41.0142531046113,"lng":-8.5859600151318},{"lat":41.0142719930551,"lng":-8.58878872731236},{"lat":41.0141061653569,"lng":-8.5936229239907},{"lat":41.0142373740368,"lng":-8.59402316383134},{"lat":41.0145608892187,"lng":-8.59494532019822},{"lat":41.0154289377522,"lng":-8.59601409912793},{"lat":41.0167956851129,"lng":-8.59709319502612},{"lat":41.016769017249,"lng":-8.59720903699611},{"lat":41.0168527965992,"lng":-8.59728820990332},{"lat":41.0169437013872,"lng":-8.59736577338872},{"lat":41.0170192856558,"lng":-8.59746794716945},{"lat":41.017110375971,"lng":-8.59754301340722},{"lat":41.0171959234233,"lng":-8.59763004609256},{"lat":41.0172590198982,"lng":-8.59774830855778},{"lat":41.0173093783019,"lng":-8.59787574894959},{"lat":41.0173741900071,"lng":-8.59799152728469},{"lat":41.0174537550139,"lng":-8.59808981192224},{"lat":41.0175291155849,"lng":-8.59817986875686},{"lat":41.0176103016068,"lng":-8.59827650321031},{"lat":41.0177136694378,"lng":-8.5983218168336},{"lat":41.0178178534685,"lng":-8.59836546834725},{"lat":41.0178796404782,"lng":-8.59847266327111},{"lat":41.017937426948,"lng":-8.59858982039356},{"lat":41.0180407990321,"lng":-8.59863382580618},{"lat":41.0181445128397,"lng":-8.59859366315799},{"lat":41.0182127484238,"lng":-8.59848643053778},{"lat":41.0189656444652,"lng":-8.59872275778305},{"lat":41.0190564552,"lng":-8.59873397612048},{"lat":41.0191435764215,"lng":-8.59876645341745},{"lat":41.0192326441201,"lng":-8.5987856323581},{"lat":41.0193215159317,"lng":-8.5988084967666},{"lat":41.0194131909172,"lng":-8.59882733513319},{"lat":41.0195536140453,"lng":-8.5988831294032},{"lat":41.0197943330656,"lng":-8.59905377696412},{"lat":41.0199343698143,"lng":-8.59922692128701},{"lat":41.0200281722722,"lng":-8.59934421899915},{"lat":41.020541788863,"lng":-8.60017522065482},{"lat":41.0209180365045,"lng":-8.60027735834144},{"lat":41.0218533808824,"lng":-8.60153400195919},{"lat":41.0223023052995,"lng":-8.60206920266402},{"lat":41.0239996224356,"lng":-8.60403205815044},{"lat":41.0249306891692,"lng":-8.60533514708132},{"lat":41.0256943882391,"lng":-8.60605883617437},{"lat":41.0243049061779,"lng":-8.60874311904242}]},{"zone_id":"smf6","points":[{"lat":40.9901597490253,"lng":-8.56402986436183},{"lat":40.9890752181941,"lng":-8.56761715899663},{"lat":40.9889275674466,"lng":-8.5722241979788},{"lat":40.9924872370392,"lng":-8.57314686249403},{"lat":40.9953013347889,"lng":-8.57426722172144},{"lat":40.9990021783538,"lng":-8.56944599436281},{"lat":41.00317280816,"lng":-8.56846191647153},{"lat":41.0065498757587,"lng":-8.56753671204608},{"lat":41.0089837061203,"lng":-8.56625395557585},{"lat":41.0121078727299,"lng":-8.5642124068748},{"lat":41.0130157821258,"lng":-8.56365426107199},{"lat":41.0144345953301,"lng":-8.56364595560271},{"lat":41.0151263586705,"lng":-8.56023809975096},{"lat":41.015503147161,"lng":-8.55933073769544},{"lat":41.0154460440236,"lng":-8.55850996081375},{"lat":41.015453310949,"lng":-8.55827414478322},{"lat":41.0154777194957,"lng":-8.55793204032449},{"lat":41.015509870781,"lng":-8.5575899853134},{"lat":41.0155539036781,"lng":-8.55727332843291},{"lat":41.0156012861329,"lng":-8.55686476537534},{"lat":41.0161928786023,"lng":-8.55699403799249},{"lat":41.0179566597039,"lng":-8.55742438310342},{"lat":41.0184923827858,"lng":-8.55717578350338},{"lat":41.0187900649397,"lng":-8.55696791491036},{"lat":41.0190270353862,"lng":-8.55671995951289},{"lat":41.0192671484417,"lng":-8.55663609391451},{"lat":41.0193772386886,"lng":-8.55650584264009},{"lat":41.0194669309968,"lng":-8.55641286776898},{"lat":41.0195885888892,"lng":-8.55630124474929},{"lat":41.0197084270191,"lng":-8.55619495975368},{"lat":41.0198802526726,"lng":-8.55604005603181},{"lat":41.0199625952047,"lng":-8.55597072363775},{"lat":41.0200692394639,"lng":-8.55581078489836},{"lat":41.0201786080661,"lng":-8.55569328083992},{"lat":41.0204401353887,"lng":-8.55540777029594},{"lat":41.0207333997843,"lng":-8.55529230784128},{"lat":41.0208848440701,"lng":-8.55519234114339},{"lat":41.0210761967717,"lng":-8.55508615964072},{"lat":41.0212436782774,"lng":-8.55500293399092},{"lat":41.0214820965347,"lng":-8.55488976367121},{"lat":41.0227698623533,"lng":-8.55418357873114},{"lat":41.0230623104995,"lng":-8.55405117862271},{"lat":41.0231750125307,"lng":-8.55402462610707},{"lat":41.0238970013056,"lng":-8.55401348563629},{"lat":41.0247784786747,"lng":-8.55396787390071},{"lat":41.0254004101587,"lng":-8.55406113957289},{"lat":41.025826233836,"lng":-8.55430850832225},{"lat":41.0264199668419,"lng":-8.55467125831613},{"lat":41.0265521545893,"lng":-8.55475503206893},{"lat":41.0268264426633,"lng":-8.55442464159934},{"lat":41.0273349350286,"lng":-8.55361426973435},{"lat":41.0274373751696,"lng":-8.55335238467379},{"lat":41.0275854173237,"lng":-8.55320220298383},{"lat":41.0280448543804,"lng":-8.55270157739192},{"lat":41.0287837415946,"lng":-8.55201555733383},{"lat":41.0304983683998,"lng":-8.54986577663449},{"lat":41.0308167203777,"lng":-8.5494318707316},{"lat":41.0309540698777,"lng":-8.54785706041216},{"lat":41.0337163270503,"lng":-8.54549391555007},{"lat":41.0337572115837,"lng":-8.54536875206445},{"lat":41.0338842751051,"lng":-8.54516734896363},{"lat":41.0340265662484,"lng":-8.54500360184153},{"lat":41.034228002081,"lng":-8.54472518311814},{"lat":41.034469471304,"lng":-8.54419830727072},{"lat":41.0347038131519,"lng":-8.54392202659993},{"lat":41.0348468501921,"lng":-8.54376426326167},{"lat":41.0350103671336,"lng":-8.54360455542075},{"lat":41.0351183422794,"lng":-8.54348084440019},{"lat":41.0353287386692,"lng":-8.54316061119419},{"lat":41.0355524904001,"lng":-8.5428110932661},{"lat":41.0358189212989,"lng":-8.54218600702728},{"lat":41.0359171338993,"lng":-8.54157883322247},{"lat":41.0360127431451,"lng":-8.54143746994607},{"lat":41.0361142643584,"lng":-8.54122356883392},{"lat":41.0363213579842,"lng":-8.54049135724812},{"lat":41.0362172492654,"lng":-8.53817516224932},{"lat":41.0361812854353,"lng":-8.53722269631545},{"lat":41.0361645717985,"lng":-8.53705967359022},{"lat":41.0361319864083,"lng":-8.53682650955471},{"lat":41.0361030285613,"lng":-8.53661132613446},{"lat":41.036090045795,"lng":-8.53643691013002},{"lat":41.0360911616749,"lng":-8.53634986809156},{"lat":41.0363701225671,"lng":-8.53599835560872},{"lat":41.0365046574192,"lng":-8.53581999654424},{"lat":41.0367524840585,"lng":-8.53559470088384},{"lat":41.0369224217862,"lng":-8.53544450377691},{"lat":41.0370994397861,"lng":-8.5353412432659},{"lat":41.037248134538,"lng":-8.53527553203161},{"lat":41.0374180709862,"lng":-8.53519104615896},{"lat":41.0375596843578,"lng":-8.5350032997753},{"lat":41.0377083780701,"lng":-8.53476861679572},{"lat":41.0377933457549,"lng":-8.5345527084545},{"lat":41.0378499908172,"lng":-8.53433680011329},{"lat":41.0378995552067,"lng":-8.5342147649639},{"lat":41.0380128450997,"lng":-8.53407395517615},{"lat":41.0381402959963,"lng":-8.53386743415412},{"lat":41.0382323437125,"lng":-8.53343561747169},{"lat":41.0383626230934,"lng":-8.53323681660654},{"lat":41.0384958339608,"lng":-8.53302488563727},{"lat":41.0386379252556,"lng":-8.532812954668},{"lat":41.0388954649457,"lng":-8.53256570187053},{"lat":41.0392329292212,"lng":-8.53213006598925},{"lat":41.0393394965276,"lng":-8.53181216953535},{"lat":41.0394460636616,"lng":-8.53134121182587},{"lat":41.0396147946041,"lng":-8.53094089777281},{"lat":41.0397924056552,"lng":-8.53054058371975},{"lat":41.0399877772579,"lng":-8.53014026966669},{"lat":41.0402186702226,"lng":-8.52971640772816},{"lat":41.040422921016,"lng":-8.52915125847678},{"lat":41.0405561277138,"lng":-8.52875094442372},{"lat":41.0406493722418,"lng":-8.52848603071214},{"lat":41.0403632754568,"lng":-8.52842079257556},{"lat":41.0400244173225,"lng":-8.52837487117454},{"lat":41.039705486112,"lng":-8.52832038468361},{"lat":41.0394695756208,"lng":-8.52827526294059},{"lat":41.0391440764702,"lng":-8.52822937290649},{"lat":41.038921630985,"lng":-8.52818859250289},{"lat":41.0386457654883,"lng":-8.52815177298534},{"lat":41.0380711323444,"lng":-8.52809995048978},{"lat":41.037888391759,"lng":-8.52808565155355},{"lat":41.0372538638386,"lng":-8.52802915787246},{"lat":41.0363668872244,"lng":-8.52787509461206},{"lat":41.0361159360484,"lng":-8.52783265078903},{"lat":41.0358845847106,"lng":-8.52774860202082},{"lat":41.0354896211719,"lng":-8.52751783857049},{"lat":41.035088819242,"lng":-8.52722203539709},{"lat":41.0348338386763,"lng":-8.52706297122285},{"lat":41.0345023224068,"lng":-8.52689855360172},{"lat":41.034183615551,"lng":-8.52670833284546},{"lat":41.0337923911987,"lng":-8.52656545962668},{"lat":41.0333011603188,"lng":-8.52622862059975},{"lat":41.0329668825069,"lng":-8.52597266001451},{"lat":41.0326518365194,"lng":-8.5257557127777},{"lat":41.0324696562314,"lng":-8.52561444577262},{"lat":41.0323402898305,"lng":-8.52552568803917},{"lat":41.0322442345965,"lng":-8.52545940553159},{"lat":41.0320817055836,"lng":-8.52531831713178},{"lat":41.0319162051446,"lng":-8.52521671895138},{"lat":41.0317466543774,"lng":-8.5251232556908},{"lat":41.0315047195795,"lng":-8.52500802363589},{"lat":41.0312955300414,"lng":-8.52491457181059},{"lat":41.0310135215316,"lng":-8.52478148498628},{"lat":41.0307237823207,"lng":-8.52465661052271},{"lat":41.0301109882452,"lng":-8.52431491595436},{"lat":41.0298789867011,"lng":-8.52416481672216},{"lat":41.029727294269,"lng":-8.52409804281857},{"lat":41.0294111643825,"lng":-8.52397656754288},{"lat":41.0290558838678,"lng":-8.52383402607508},{"lat":41.0288300662351,"lng":-8.52371426410188},{"lat":41.0286045534002,"lng":-8.52357233849989},{"lat":41.0283460031795,"lng":-8.52331206531183},{"lat":41.0281039507014,"lng":-8.5231130377921},{"lat":41.0275403189467,"lng":-8.52265803655238},{"lat":41.0273346401716,"lng":-8.52252928623354},{"lat":41.0272413394031,"lng":-8.52242943041638},{"lat":41.0266532510309,"lng":-8.52195390644021},{"lat":41.0265076348109,"lng":-8.52171916407463},{"lat":41.0263920948445,"lng":-8.52164321658678},{"lat":41.0262002779407,"lng":-8.52138371041291},{"lat":41.0260912206587,"lng":-8.5212814055054},{"lat":41.0253887674834,"lng":-8.520327246125},{"lat":41.0210100293945,"lng":-8.51852426439112},{"lat":41.0159307330526,"lng":-8.51536680573285},{"lat":41.0159257612925,"lng":-8.51512517275073},{"lat":41.0155227616824,"lng":-8.51518425836384},{"lat":41.0152525241509,"lng":-8.51498460456328},{"lat":41.0149236144684,"lng":-8.51453755057558},{"lat":41.0147391477657,"lng":-8.51428295100061},{"lat":41.0145886696309,"lng":-8.5140883239848},{"lat":41.0144920906361,"lng":-8.51385134459484},{"lat":41.0143895224092,"lng":-8.51360649228277},{"lat":41.0143277587692,"lng":-8.51342351419574},{"lat":41.0141955735577,"lng":-8.5131627275631},{"lat":41.0140348341936,"lng":-8.51299598534634},{"lat":41.0138270570778,"lng":-8.51276256799097},{"lat":41.0137281305528,"lng":-8.5117973282942},{"lat":41.013511479341,"lng":-8.5112339442724},{"lat":41.0121158749054,"lng":-8.50948358588121},{"lat":41.0101904800719,"lng":-8.5068497346675},{"lat":41.0099222971547,"lng":-8.5071071004392},{"lat":41.005909918452,"lng":-8.51089995437931},{"lat":41.0032354002961,"lng":-8.51277161534497},{"lat":41.0005637621721,"lng":-8.51376260847822},{"lat":40.9914241159369,"lng":-8.51544913626225},{"lat":40.9921508145086,"lng":-8.5196649431697},{"lat":40.9927618899989,"lng":-8.52391344255855},{"lat":40.992827572168,"lng":-8.53083150056372},{"lat":40.9926908463786,"lng":-8.53590100566014},{"lat":40.9935011331963,"lng":-8.54172653363691},{"lat":40.9946750779174,"lng":-8.54657628525363},{"lat":40.994696443785,"lng":-8.54898475499451},{"lat":40.9946918993421,"lng":-8.55024251958851},{"lat":40.994399623694,"lng":-8.55200157984578},{"lat":40.9907594282854,"lng":-8.56232891770115},{"lat":40.9901597490253,"lng":-8.56402986436183}]},{"zone_id":"vng9","points":[{"lat":41.0358189212989,"lng":-8.54218600702728},{"lat":41.0386801085497,"lng":-8.54608513161834},{"lat":41.0407403965346,"lng":-8.5497586017523},{"lat":41.0543155993931,"lng":-8.56476618922353},{"lat":41.0574845042287,"lng":-8.56601252970078},{"lat":41.0628821998062,"lng":-8.56761378591302},{"lat":41.0637275316545,"lng":-8.5667835620139},{"lat":41.0676992034877,"lng":-8.56193324024034},{"lat":41.0733093876719,"lng":-8.55530118529578},{"lat":41.0774071203912,"lng":-8.54925651046801},{"lat":41.0825161309998,"lng":-8.53107924159995},{"lat":41.0864211131852,"lng":-8.52175669597924},{"lat":41.084736612369,"lng":-8.52161379231004},{"lat":41.0840868100739,"lng":-8.52149486144295},{"lat":41.0830533793473,"lng":-8.52114863098054},{"lat":41.0822074740624,"lng":-8.52073379030195},{"lat":41.0814506956898,"lng":-8.52034910914222},{"lat":41.0809618080535,"lng":-8.52014398531769},{"lat":41.0807788485899,"lng":-8.52006722055188},{"lat":41.0805876744886,"lng":-8.51998701026852},{"lat":41.079594135427,"lng":-8.51956086164604},{"lat":41.0788363753481,"lng":-8.51924196652187},{"lat":41.0787154730557,"lng":-8.51919108610125},{"lat":41.0779980845901,"lng":-8.51887283672118},{"lat":41.0776980915458,"lng":-8.51869470901622},{"lat":41.077126311845,"lng":-8.51835520679096},{"lat":41.0762711569136,"lng":-8.51776831802474},{"lat":41.0755573752022,"lng":-8.51715157105217},{"lat":41.0749738919482,"lng":-8.51661366094295},{"lat":41.0745769874272,"lng":-8.51623583853119},{"lat":41.0744372296102,"lng":-8.51610280127217},{"lat":41.0737229913418,"lng":-8.51537745149865},{"lat":41.0726341181346,"lng":-8.51385392816643},{"lat":41.0725051199555,"lng":-8.51355037384059},{"lat":41.0721691027027,"lng":-8.51269668259723},{"lat":41.0693926034379,"lng":-8.51460509303299},{"lat":41.0682468611478,"lng":-8.51525954782477},{"lat":41.0660890398846,"lng":-8.51598966240562},{"lat":41.0644670854053,"lng":-8.5163371380578},{"lat":41.062490112753,"lng":-8.51513595294745},{"lat":41.0607789089353,"lng":-8.51524498438111},{"lat":41.0590681002732,"lng":-8.5152350516702},{"lat":41.0567060795228,"lng":-8.51628879834966},{"lat":41.0552088092365,"lng":-8.51695672071067},{"lat":41.0529884466341,"lng":-8.51836008125647},{"lat":41.052721861637,"lng":-8.51852921327321},{"lat":41.0504815441207,"lng":-8.5199504988199},{"lat":41.0479021919679,"lng":-8.52182360175736},{"lat":41.0451058301994,"lng":-8.5244505279023},{"lat":41.0436709253826,"lng":-8.5260469029025},{"lat":41.0407292960178,"lng":-8.52829764762834},{"lat":41.0406493722418,"lng":-8.52848603071214},{"lat":41.0405561277138,"lng":-8.52875094442372},{"lat":41.040422921016,"lng":-8.52915125847678},{"lat":41.0402186702226,"lng":-8.52971640772816},{"lat":41.0399877772579,"lng":-8.53014026966669},{"lat":41.0397924056552,"lng":-8.53054058371975},{"lat":41.0396147946041,"lng":-8.53094089777281},{"lat":41.0394460636616,"lng":-8.53134121182587},{"lat":41.0393394965276,"lng":-8.53181216953535},{"lat":41.0392329292212,"lng":-8.53213006598925},{"lat":41.0388954649457,"lng":-8.53256570187053},{"lat":41.0386379252556,"lng":-8.532812954668},{"lat":41.0384958339608,"lng":-8.53302488563727},{"lat":41.0383626230934,"lng":-8.53323681660654},{"lat":41.0382323437125,"lng":-8.53343561747169},{"lat":41.0381402959963,"lng":-8.53386743415412},{"lat":41.0380128450997,"lng":-8.53407395517615},{"lat":41.0378995552067,"lng":-8.5342147649639},{"lat":41.0378499908172,"lng":-8.53433680011329},{"lat":41.0377933457549,"lng":-8.5345527084545},{"lat":41.0377083780701,"lng":-8.53476861679572},{"lat":41.0375596843578,"lng":-8.5350032997753},{"lat":41.0374180709862,"lng":-8.53519104615896},{"lat":41.037248134538,"lng":-8.53527553203161},{"lat":41.0370994397861,"lng":-8.5353412432659},{"lat":41.0369224217862,"lng":-8.53544450377691},{"lat":41.0367524840585,"lng":-8.53559470088384},{"lat":41.0365046574192,"lng":-8.53581999654424},{"lat":41.0363701225671,"lng":-8.53599835560872},{"lat":41.0360911616749,"lng":-8.53634986809156},{"lat":41.036090045795,"lng":-8.53643691013002},{"lat":41.0361030285613,"lng":-8.53661132613446},{"lat":41.0361319864083,"lng":-8.53682650955471},{"lat":41.0361645717985,"lng":-8.53705967359022},{"lat":41.0361812854353,"lng":-8.53722269631545},{"lat":41.0362172492654,"lng":-8.53817516224932},{"lat":41.0363213579842,"lng":-8.54049135724812},{"lat":41.0361142643584,"lng":-8.54122356883392},{"lat":41.0360127431451,"lng":-8.54143746994607},{"lat":41.0359171338993,"lng":-8.54157883322247},{"lat":41.0358189212989,"lng":-8.54218600702728}]},{"zone_id":"vng12","points":[{"lat":41.0567060795228,"lng":-8.51628879834966},{"lat":41.05396742164,"lng":-8.51316892096676},{"lat":41.0531369479469,"lng":-8.51214127995742},{"lat":41.0525743566728,"lng":-8.51115453866563},{"lat":41.0511962530436,"lng":-8.50764648175955},{"lat":41.0451389823862,"lng":-8.49173710690729},{"lat":41.0450524959666,"lng":-8.4913896595901},{"lat":41.0448639413557,"lng":-8.49146920193676},{"lat":41.0446299166415,"lng":-8.49158424373543},{"lat":41.0445038732073,"lng":-8.49163624128396},{"lat":41.0442430682295,"lng":-8.4914747337642},{"lat":41.0436462151495,"lng":-8.49105533105058},{"lat":41.043028309862,"lng":-8.49060073666764},{"lat":41.0425097949739,"lng":-8.49058865252724},{"lat":41.042039144073,"lng":-8.49054155675628},{"lat":41.0414441654758,"lng":-8.49003527721859},{"lat":41.0403607577981,"lng":-8.4896231892228},{"lat":41.0397924056552,"lng":-8.48944658008174},{"lat":41.0393661383272,"lng":-8.48934061459711},{"lat":41.0391441230006,"lng":-8.4893052927689},{"lat":41.0388666027891,"lng":-8.4891728359131},{"lat":41.0387444935254,"lng":-8.48916694894174},{"lat":41.0385468980552,"lng":-8.48916694894173},{"lat":41.0384372902112,"lng":-8.48916020618729},{"lat":41.0383330867904,"lng":-8.48887767243345},{"lat":41.0382295686098,"lng":-8.48860513293717},{"lat":41.0381813952447,"lng":-8.48848480978084},{"lat":41.038094752818,"lng":-8.48817532912679},{"lat":41.0380071869635,"lng":-8.48789907535975},{"lat":41.0378294132667,"lng":-8.48729612849624},{"lat":41.0377868572496,"lng":-8.48713832493748},{"lat":41.0377045771534,"lng":-8.48692869946702},{"lat":41.0376304433488,"lng":-8.48673219734129},{"lat":41.0376027169155,"lng":-8.48661407433119},{"lat":41.0374300418654,"lng":-8.48607823340894},{"lat":41.0373055291378,"lng":-8.48551632467875},{"lat":41.0371557279201,"lng":-8.4849772147013},{"lat":41.0370536250087,"lng":-8.48471028187534},{"lat":41.0368598289127,"lng":-8.48424807117672},{"lat":41.0367314412713,"lng":-8.48400704741276},{"lat":41.036562765691,"lng":-8.48401518378873},{"lat":41.0363472545109,"lng":-8.48402604800395},{"lat":41.0363490936126,"lng":-8.48389547971435},{"lat":41.0365483149985,"lng":-8.48326352877674},{"lat":41.0364514130633,"lng":-8.48315039536889},{"lat":41.0363689300801,"lng":-8.48309180466182},{"lat":41.0362721937807,"lng":-8.4830130373639},{"lat":41.0363348297054,"lng":-8.48287709110814},{"lat":41.0364080317312,"lng":-8.48278971690259},{"lat":41.0360570719342,"lng":-8.48242419373277},{"lat":41.0359035773997,"lng":-8.48220670663203},{"lat":41.0357633245578,"lng":-8.48201807036498},{"lat":41.0356217483945,"lng":-8.48202706713892},{"lat":41.0355006215861,"lng":-8.48203249128649},{"lat":41.0352408209886,"lng":-8.48198021126961},{"lat":41.0350029083584,"lng":-8.48189678103535},{"lat":41.0349360423789,"lng":-8.48179617357669},{"lat":41.03488965694,"lng":-8.48168165145827},{"lat":41.0348379449837,"lng":-8.48145400381481},{"lat":41.0348009456453,"lng":-8.48121276790703},{"lat":41.0348457592511,"lng":-8.48104437371597},{"lat":41.0348793890258,"lng":-8.48085297506796},{"lat":41.0348562793843,"lng":-8.4806933858571},{"lat":41.0347755997132,"lng":-8.48060460738106},{"lat":41.0347933328826,"lng":-8.48045807163368},{"lat":41.0348524243575,"lng":-8.48033197523329},{"lat":41.0349011888807,"lng":-8.48022603936879},{"lat":41.0349987629194,"lng":-8.48002844006067},{"lat":41.0350554848672,"lng":-8.47991160613618},{"lat":41.035123047027,"lng":-8.47972300526367},{"lat":41.034143052659,"lng":-8.47857445621262},{"lat":41.0341277602425,"lng":-8.47799830691951},{"lat":41.0341083207951,"lng":-8.47724812132067},{"lat":41.0340524855123,"lng":-8.47577838539771},{"lat":41.0337674530593,"lng":-8.47579211685845},{"lat":41.0334723216238,"lng":-8.47587167661389},{"lat":41.0331974909007,"lng":-8.47599724602188},{"lat":41.0329098950887,"lng":-8.47614462377891},{"lat":41.03191014871,"lng":-8.47607971500853},{"lat":41.0318147592877,"lng":-8.47602903621746},{"lat":41.031692063746,"lng":-8.4759579947517},{"lat":41.0315593098831,"lng":-8.47590760053023},{"lat":41.0315865952785,"lng":-8.47572628227224},{"lat":41.0316814197271,"lng":-8.47560203531168},{"lat":41.0318291381023,"lng":-8.47543704644539},{"lat":41.0319675035178,"lng":-8.47520612438811},{"lat":41.0320098387252,"lng":-8.4750512837768},{"lat":41.0319885688951,"lng":-8.47481572965149},{"lat":41.0319903975901,"lng":-8.47462559276479},{"lat":41.0319571912351,"lng":-8.4742928175854},{"lat":41.0319449715496,"lng":-8.47400594843948},{"lat":41.0319110163088,"lng":-8.47367483726219},{"lat":41.0318662362409,"lng":-8.47335056885841},{"lat":41.0318408203981,"lng":-8.47311546773179},{"lat":41.0318506140905,"lng":-8.47288054661279},{"lat":41.03188086874,"lng":-8.47272801977845},{"lat":41.0316457673604,"lng":-8.47269695735778},{"lat":41.031480138586,"lng":-8.47204007121395},{"lat":41.0314331963273,"lng":-8.47189784650661},{"lat":41.0312583022199,"lng":-8.47196746255259},{"lat":41.0311634402035,"lng":-8.47177041214335},{"lat":41.0314384792789,"lng":-8.47163246430574},{"lat":41.031601905757,"lng":-8.47154234020327},{"lat":41.0317103418952,"lng":-8.47135169042805},{"lat":41.0317945150713,"lng":-8.47120550123876},{"lat":41.0319369309385,"lng":-8.47109600535075},{"lat":41.0320995615404,"lng":-8.47109137298321},{"lat":41.0324016618379,"lng":-8.47087792923385},{"lat":41.0323307985656,"lng":-8.47026564066029},{"lat":41.0323270421344,"lng":-8.47013469937499},{"lat":41.0323119102086,"lng":-8.46998276884844},{"lat":41.0323060968426,"lng":-8.4697288581172},{"lat":41.0322945724383,"lng":-8.46960251919409},{"lat":41.0322180145469,"lng":-8.46945574272317},{"lat":41.0321050946505,"lng":-8.46930295959599},{"lat":41.0320401201436,"lng":-8.46919869734584},{"lat":41.0319534073803,"lng":-8.4690450922202},{"lat":41.031877006228,"lng":-8.46887548658531},{"lat":41.0317881422012,"lng":-8.46865588402533},{"lat":41.0317494141596,"lng":-8.4685368907987},{"lat":41.031620390629,"lng":-8.46850186656111},{"lat":41.0313539275687,"lng":-8.4680753026682},{"lat":41.0310726372155,"lng":-8.46773019882931},{"lat":41.0306389798249,"lng":-8.46718278615259},{"lat":41.0302528988207,"lng":-8.46661576271443},{"lat":41.0301946946149,"lng":-8.4671215418972},{"lat":41.0301045440523,"lng":-8.46796082282052},{"lat":41.0300581935144,"lng":-8.46860007694031},{"lat":41.0300444288814,"lng":-8.46874923682277},{"lat":41.0300295028365,"lng":-8.46895760675142},{"lat":41.0300146359397,"lng":-8.46923696735091},{"lat":41.0300120530661,"lng":-8.46953399140685},{"lat":41.0299576408601,"lng":-8.46978758114472},{"lat":41.0298896646272,"lng":-8.47003004389676},{"lat":41.0298313161352,"lng":-8.47024460792214},{"lat":41.0297667612764,"lng":-8.47045640972674},{"lat":41.0296075002404,"lng":-8.4704151665074},{"lat":41.0294332389286,"lng":-8.4703934621688},{"lat":41.0294646486071,"lng":-8.47055129700126},{"lat":41.0294646284321,"lng":-8.47068149513804},{"lat":41.02961707924,"lng":-8.4707095135126},{"lat":41.0294249874442,"lng":-8.47166609061486},{"lat":41.0293914380121,"lng":-8.47183594895565},{"lat":41.0293119815863,"lng":-8.47212234844533},{"lat":41.0292995447232,"lng":-8.47234867394025},{"lat":41.0291587448599,"lng":-8.47303820598629},{"lat":41.0291011241045,"lng":-8.47325843290045},{"lat":41.0289516016668,"lng":-8.4735538654228},{"lat":41.0288277677188,"lng":-8.4738263229927},{"lat":41.028771432995,"lng":-8.47394778285556},{"lat":41.0286610647977,"lng":-8.47407887770294},{"lat":41.0285555824869,"lng":-8.47425300762239},{"lat":41.0285516683473,"lng":-8.47464889972636},{"lat":41.0285386751599,"lng":-8.47509574525289},{"lat":41.0282996233564,"lng":-8.47502793666891},{"lat":41.0281692269943,"lng":-8.47557457293075},{"lat":41.0268224352972,"lng":-8.47590405256978},{"lat":41.0247987180174,"lng":-8.47748353762295},{"lat":41.0238599174846,"lng":-8.48656275369795},{"lat":41.023793495701,"lng":-8.48803886467323},{"lat":41.0235429378047,"lng":-8.48881153214167},{"lat":41.0234289424852,"lng":-8.48918805650698},{"lat":41.0233608270003,"lng":-8.48934641369896},{"lat":41.0232398530888,"lng":-8.48953432894728},{"lat":41.0230734566622,"lng":-8.4898509975039},{"lat":41.0229445444993,"lng":-8.49018772514009},{"lat":41.0226954890536,"lng":-8.49047409949286},{"lat":41.0225672231527,"lng":-8.49063212712446},{"lat":41.0223615137399,"lng":-8.49145471210879},{"lat":41.0223006826293,"lng":-8.49167267204416},{"lat":41.0221499599551,"lng":-8.4919326608894},{"lat":41.0134011311728,"lng":-8.48916627999709},{"lat":41.0117054085721,"lng":-8.49093293586295},{"lat":41.0107936164168,"lng":-8.4951390450283},{"lat":41.0113755766474,"lng":-8.49682873073496},{"lat":41.0117036256735,"lng":-8.49765591353532},{"lat":41.0108693716955,"lng":-8.4994331023985},{"lat":41.0103937780968,"lng":-8.50083232747995},{"lat":41.0100505901408,"lng":-8.5008406318883},{"lat":41.0099255016085,"lng":-8.50081758456242},{"lat":41.0097727574624,"lng":-8.50079842528396},{"lat":41.0096050356556,"lng":-8.50081662553389},{"lat":41.0095166344421,"lng":-8.50086558290004},{"lat":41.0094109827355,"lng":-8.5011832110843},{"lat":41.0094054158368,"lng":-8.50140261708514},{"lat":41.0093989030666,"lng":-8.50166397871588},{"lat":41.0093941331828,"lng":-8.50205527687096},{"lat":41.0093722273631,"lng":-8.50281783443193},{"lat":41.0093779999867,"lng":-8.50298238433122},{"lat":41.0094157308634,"lng":-8.50345582480363},{"lat":41.0094752286263,"lng":-8.50388076878699},{"lat":41.0094725564453,"lng":-8.5040107996886},{"lat":41.0095233020671,"lng":-8.50430137117397},{"lat":41.0095235392726,"lng":-8.50447849104388},{"lat":41.0094973605193,"lng":-8.50466532898623},{"lat":41.0094324273039,"lng":-8.50508529177552},{"lat":41.0094063770672,"lng":-8.5053435713575},{"lat":41.0093622180944,"lng":-8.50549369455889},{"lat":41.0093041228087,"lng":-8.5056373187502},{"lat":41.0101904800719,"lng":-8.5068497346675},{"lat":41.0121158749054,"lng":-8.50948358588121},{"lat":41.013511479341,"lng":-8.5112339442724},{"lat":41.0137281305528,"lng":-8.5117973282942},{"lat":41.0138270570778,"lng":-8.51276256799097},{"lat":41.0140348341936,"lng":-8.51299598534634},{"lat":41.0141955735577,"lng":-8.5131627275631},{"lat":41.0143277587692,"lng":-8.51342351419574},{"lat":41.0143895224092,"lng":-8.51360649228277},{"lat":41.0144920906361,"lng":-8.51385134459484},{"lat":41.0145886696309,"lng":-8.5140883239848},{"lat":41.0147391477657,"lng":-8.51428295100061},{"lat":41.0149236144684,"lng":-8.51453755057558},{"lat":41.0152525241509,"lng":-8.51498460456328},{"lat":41.0155227616824,"lng":-8.51518425836384},{"lat":41.0159257612925,"lng":-8.51512517275073},{"lat":41.0159307330526,"lng":-8.51536680573285},{"lat":41.0210100293945,"lng":-8.51852426439112},{"lat":41.0253887674834,"lng":-8.520327246125},{"lat":41.0260912206587,"lng":-8.5212814055054},{"lat":41.0262002779407,"lng":-8.52138371041291},{"lat":41.0263920948445,"lng":-8.52164321658678},{"lat":41.0265076348109,"lng":-8.52171916407463},{"lat":41.0266532510309,"lng":-8.52195390644021},{"lat":41.0272413394031,"lng":-8.52242943041638},{"lat":41.0273346401716,"lng":-8.52252928623354},{"lat":41.0275403189467,"lng":-8.52265803655238},{"lat":41.0281039507014,"lng":-8.5231130377921},{"lat":41.0283460031795,"lng":-8.52331206531183},{"lat":41.0286045534002,"lng":-8.52357233849989},{"lat":41.0288300662351,"lng":-8.52371426410188},{"lat":41.0290558838678,"lng":-8.52383402607508},{"lat":41.0294111643825,"lng":-8.52397656754288},{"lat":41.029727294269,"lng":-8.52409804281857},{"lat":41.0298789867011,"lng":-8.52416481672216},{"lat":41.0301109882452,"lng":-8.52431491595436},{"lat":41.0307237823207,"lng":-8.52465661052271},{"lat":41.0310135215316,"lng":-8.52478148498628},{"lat":41.0312955300414,"lng":-8.52491457181059},{"lat":41.0315047195795,"lng":-8.52500802363589},{"lat":41.0317466543774,"lng":-8.5251232556908},{"lat":41.0319162051446,"lng":-8.52521671895138},{"lat":41.0320817055836,"lng":-8.52531831713178},{"lat":41.0322442345965,"lng":-8.52545940553159},{"lat":41.0323402898305,"lng":-8.52552568803917},{"lat":41.0324696562314,"lng":-8.52561444577262},{"lat":41.0326518365194,"lng":-8.5257557127777},{"lat":41.0329668825069,"lng":-8.52597266001451},{"lat":41.0333011603188,"lng":-8.52622862059975},{"lat":41.0337923911987,"lng":-8.52656545962668},{"lat":41.034183615551,"lng":-8.52670833284546},{"lat":41.0345023224068,"lng":-8.52689855360172},{"lat":41.0348338386763,"lng":-8.52706297122285},{"lat":41.035088819242,"lng":-8.52722203539709},{"lat":41.0354896211719,"lng":-8.52751783857049},{"lat":41.0358845847106,"lng":-8.52774860202082},{"lat":41.0361159360484,"lng":-8.52783265078903},{"lat":41.0363668872244,"lng":-8.52787509461206},{"lat":41.0372538638386,"lng":-8.52802915787246},{"lat":41.037888391759,"lng":-8.52808565155355},{"lat":41.0380711323444,"lng":-8.52809995048978},{"lat":41.0386457654883,"lng":-8.52815177298534},{"lat":41.038921630985,"lng":-8.52818859250289},{"lat":41.0391440764702,"lng":-8.52822937290649},{"lat":41.0394695756208,"lng":-8.52827526294059},{"lat":41.039705486112,"lng":-8.52832038468361},{"lat":41.0400244173225,"lng":-8.52837487117454},{"lat":41.0403632754568,"lng":-8.52842079257556},{"lat":41.0406493722418,"lng":-8.52848603071214},{"lat":41.0407292960178,"lng":-8.52829764762834},{"lat":41.0436709253826,"lng":-8.5260469029025},{"lat":41.0451058301994,"lng":-8.5244505279023},{"lat":41.0479021919679,"lng":-8.52182360175736},{"lat":41.0504815441207,"lng":-8.5199504988199},{"lat":41.052721861637,"lng":-8.51852921327321},{"lat":41.0529884466341,"lng":-8.51836008125647},{"lat":41.0552088092365,"lng":-8.51695672071067},{"lat":41.0567060795228,"lng":-8.51628879834966}]},{"zone_id":"vng7","points":[{"lat":41.0331892049524,"lng":-8.60753743973004},{"lat":41.0340222807846,"lng":-8.60726972860741},{"lat":41.0357907350377,"lng":-8.60969847899555},{"lat":41.0361276452474,"lng":-8.60988078082951},{"lat":41.0369705338497,"lng":-8.61282645888687},{"lat":41.0382857602213,"lng":-8.6153111257162},{"lat":41.0404503017884,"lng":-8.61709656518184},{"lat":41.0414330927232,"lng":-8.61978637342328},{"lat":41.0427221197396,"lng":-8.62250716141593},{"lat":41.04341859331,"lng":-8.62289588982128},{"lat":41.0448917951489,"lng":-8.62398580557813},{"lat":41.0463165622378,"lng":-8.62514844756572},{"lat":41.049680139701,"lng":-8.62616655924091},{"lat":41.0510620440154,"lng":-8.62636108243025},{"lat":41.0541519140919,"lng":-8.62545509223732},{"lat":41.0614660855257,"lng":-8.62281523227822},{"lat":41.0629616349948,"lng":-8.62215483436784},{"lat":41.0645331708597,"lng":-8.62221706975845},{"lat":41.0652003196112,"lng":-8.6221701181265},{"lat":41.0683063132082,"lng":-8.62002305901483},{"lat":41.0689217057791,"lng":-8.61988852565262},{"lat":41.0690438203498,"lng":-8.61986010068272},{"lat":41.069026926492,"lng":-8.61974838734558},{"lat":41.0688530561634,"lng":-8.61856534742216},{"lat":41.0686794950661,"lng":-8.61701735076271},{"lat":41.0672543017341,"lng":-8.61331863078315},{"lat":41.0667249582235,"lng":-8.61069732636571},{"lat":41.0673710023352,"lng":-8.60689478395609},{"lat":41.0674356336921,"lng":-8.60354035847213},{"lat":41.0675555836308,"lng":-8.60189610385084},{"lat":41.0679942937216,"lng":-8.59924939718115},{"lat":41.0689439238299,"lng":-8.59135651258979},{"lat":41.0718120784802,"lng":-8.5887448848438},{"lat":41.0714111141716,"lng":-8.58154557727523},{"lat":41.0708406856745,"lng":-8.58048160574828},{"lat":41.0682150856866,"lng":-8.58108760465294},{"lat":41.0657421561639,"lng":-8.58045790790753},{"lat":41.0625526609228,"lng":-8.57885146619834},{"lat":41.0586190561121,"lng":-8.5771910099482},{"lat":41.0576114586968,"lng":-8.57671275174511},{"lat":41.0563540827257,"lng":-8.57587158904485},{"lat":41.0546442579223,"lng":-8.57431370926751},{"lat":41.0526871042344,"lng":-8.57238704756344},{"lat":41.0492863109803,"lng":-8.57049001357153},{"lat":41.0465440090778,"lng":-8.56975942657329},{"lat":41.0450447421252,"lng":-8.56951289456525},{"lat":41.0440866099258,"lng":-8.5729268457125},{"lat":41.0438193569828,"lng":-8.57540296528201},{"lat":41.0428096556898,"lng":-8.57630062531123},{"lat":41.0420585402223,"lng":-8.57988392849237},{"lat":41.0426445505852,"lng":-8.58114467034214},{"lat":41.0425509080166,"lng":-8.58249892265991},{"lat":41.0418034722089,"lng":-8.58294779996386},{"lat":41.0420213480354,"lng":-8.58479271063373},{"lat":41.0423743586374,"lng":-8.5855425327634},{"lat":41.0416756260913,"lng":-8.58662445604142},{"lat":41.0418938279843,"lng":-8.5893663679455},{"lat":41.0398638174721,"lng":-8.59122317571952},{"lat":41.038393305312,"lng":-8.59142775510329},{"lat":41.0345231707031,"lng":-8.59470989040889},{"lat":41.0351724499922,"lng":-8.59907649171214},{"lat":41.0355879517099,"lng":-8.60325031501586},{"lat":41.0331892049524,"lng":-8.60753743973004}]},{"zone_id":"gdm8","points":[{"lat":41.0036530261031,"lng":-8.38418583769302},{"lat":41.0027783998644,"lng":-8.3860350397359},{"lat":41.0025159890289,"lng":-8.38650195438444},{"lat":41.0024370976057,"lng":-8.38664232662971},{"lat":41.0022102224785,"lng":-8.3870460040957},{"lat":41.0020793402232,"lng":-8.38731078872559},{"lat":41.001961859332,"lng":-8.3875423502703},{"lat":41.0017153558711,"lng":-8.38821722535855},{"lat":41.0016698252987,"lng":-8.38835332487464},{"lat":41.0013801148086,"lng":-8.38921930883302},{"lat":41.0015453703978,"lng":-8.389450181893},{"lat":41.001697999438,"lng":-8.38977157192384},{"lat":41.0018616633018,"lng":-8.39010846160362},{"lat":41.0019499789664,"lng":-8.39023599046359},{"lat":41.0021009567214,"lng":-8.39040951964598},{"lat":41.0045300821739,"lng":-8.38885242856032},{"lat":41.0047669699725,"lng":-8.38914337431199},{"lat":41.0049495951414,"lng":-8.38929825507118},{"lat":41.0051231540197,"lng":-8.38944109139771},{"lat":41.0050957181281,"lng":-8.38975015260744},{"lat":41.0050920502167,"lng":-8.3898999027349},{"lat":41.0050860898979,"lng":-8.3901076561546},{"lat":41.0054188102117,"lng":-8.39010693994556},{"lat":41.0061236111346,"lng":-8.39121729077673},{"lat":41.0062359317453,"lng":-8.39132233351786},{"lat":41.0063528453229,"lng":-8.39142585284011},{"lat":41.006456907355,"lng":-8.39152016356098},{"lat":41.0065317539402,"lng":-8.39159154252376},{"lat":41.0066457633851,"lng":-8.39170384648566},{"lat":41.0067659561319,"lng":-8.39183222224239},{"lat":41.0068384454319,"lng":-8.39191107900596},{"lat":41.0069277181331,"lng":-8.39201329631008},{"lat":41.0070148305408,"lng":-8.39211551137519},{"lat":41.0071214613749,"lng":-8.39226653944118},{"lat":41.0071956285291,"lng":-8.3923595443494},{"lat":41.0073097199498,"lng":-8.39247589621089},{"lat":41.0073855576778,"lng":-8.39254632416819},{"lat":41.0075330586824,"lng":-8.39266269010391},{"lat":41.0076294475058,"lng":-8.39272392732783},{"lat":41.0077045452454,"lng":-8.39280339425937},{"lat":41.0078103020489,"lng":-8.39294229087647},{"lat":41.0079189392328,"lng":-8.39308239628412},{"lat":41.0080817642452,"lng":-8.39335013748885},{"lat":41.0082106814068,"lng":-8.39355986227428},{"lat":41.0085544654608,"lng":-8.39368069018672},{"lat":41.0091646497174,"lng":-8.39390147613424},{"lat":41.0093828304197,"lng":-8.39397949569575},{"lat":41.0097210281905,"lng":-8.39410280345381},{"lat":41.0103728198116,"lng":-8.39435823764216},{"lat":41.0100416082392,"lng":-8.39468524566778},{"lat":41.0108929958826,"lng":-8.39526933867819},{"lat":41.0113616311754,"lng":-8.39599658948946},{"lat":41.0119196535077,"lng":-8.39642273812568},{"lat":41.0124478751007,"lng":-8.3967659108546},{"lat":41.0123258524348,"lng":-8.39743564959836},{"lat":41.0122161679508,"lng":-8.39833308926347},{"lat":41.0121894432386,"lng":-8.39855955891938},{"lat":41.0122129201598,"lng":-8.39934388605513},{"lat":41.0122842771327,"lng":-8.40036996405314},{"lat":41.0121777056511,"lng":-8.40069762844657},{"lat":41.012138444333,"lng":-8.40092963350657},{"lat":41.012160400268,"lng":-8.40112836756767},{"lat":41.0124816058153,"lng":-8.40196716891252},{"lat":41.012584528524,"lng":-8.40219595581768},{"lat":41.0127826889246,"lng":-8.40255340184407},{"lat":41.0130636666982,"lng":-8.40303410822844},{"lat":41.0131756239632,"lng":-8.40321193270355},{"lat":41.0131921591399,"lng":-8.40333908518304},{"lat":41.013204658787,"lng":-8.40346027723429},{"lat":41.0132240544097,"lng":-8.40386513193404},{"lat":41.0132505095476,"lng":-8.40425219570906},{"lat":41.0132597469877,"lng":-8.40445860684646},{"lat":41.0132820626005,"lng":-8.40482823047156},{"lat":41.0132939571379,"lng":-8.40510509323032},{"lat":41.0132955302979,"lng":-8.40527711555353},{"lat":41.0132683423152,"lng":-8.40557800542325},{"lat":41.0132808133894,"lng":-8.40597439682649},{"lat":41.0132781777687,"lng":-8.40621202114497},{"lat":41.0132827644798,"lng":-8.4065163705509},{"lat":41.0132967592054,"lng":-8.40710274120411},{"lat":41.013306591815,"lng":-8.40747012013203},{"lat":41.0133262959048,"lng":-8.40807767320748},{"lat":41.013330746578,"lng":-8.40828608496188},{"lat":41.0134050677999,"lng":-8.40857110999462},{"lat":41.0135028021975,"lng":-8.408892969716},{"lat":41.0135658943671,"lng":-8.40912861863422},{"lat":41.0136395476159,"lng":-8.40942885480354},{"lat":41.0137224121693,"lng":-8.40975482031212},{"lat":41.0137929640299,"lng":-8.41003459738046},{"lat":41.0138386850751,"lng":-8.4101907666679},{"lat":41.0138831629087,"lng":-8.41034109746423},{"lat":41.0139739912122,"lng":-8.41064950074767},{"lat":41.0140359933435,"lng":-8.41085209347648},{"lat":41.0140630491289,"lng":-8.41098262239576},{"lat":41.014127656768,"lng":-8.41118701345482},{"lat":41.0141739163877,"lng":-8.41138240563815},{"lat":41.0142117634864,"lng":-8.41155435367878},{"lat":41.014259012722,"lng":-8.41175022844152},{"lat":41.0143063202809,"lng":-8.4119587063946},{"lat":41.0143494584957,"lng":-8.41213959238894},{"lat":41.0144010367157,"lng":-8.41229471210088},{"lat":41.0144541879622,"lng":-8.41246909760185},{"lat":41.0144977597979,"lng":-8.41262085527376},{"lat":41.0145270775036,"lng":-8.41274723094645},{"lat":41.0145515394305,"lng":-8.41290640126644},{"lat":41.0146083487072,"lng":-8.41309471357077},{"lat":41.0146471826662,"lng":-8.41326784959138},{"lat":41.0146706993493,"lng":-8.41340811169006},{"lat":41.0147019048567,"lng":-8.41353580256619},{"lat":41.0147339483992,"lng":-8.41365208676472},{"lat":41.0147718361457,"lng":-8.41380632064401},{"lat":41.0148030136944,"lng":-8.41394530564444},{"lat":41.0148341508627,"lng":-8.41409820107999},{"lat":41.0148462344171,"lng":-8.41423841407573},{"lat":41.014859036702,"lng":-8.4143792281602},{"lat":41.0150690314722,"lng":-8.41521586203463},{"lat":41.0151289632879,"lng":-8.41534059092384},{"lat":41.0152605898413,"lng":-8.41560412313364},{"lat":41.0152908006297,"lng":-8.41573217096053},{"lat":41.015380108483,"lng":-8.415811376162},{"lat":41.0155201383064,"lng":-8.41591552398099},{"lat":41.0156082563748,"lng":-8.41596690333998},{"lat":41.0157486356252,"lng":-8.41603979149424},{"lat":41.0158454640575,"lng":-8.41610096057776},{"lat":41.0159656891001,"lng":-8.41623985464839},{"lat":41.0160401103325,"lng":-8.41633219738802},{"lat":41.0162347564103,"lng":-8.41656295645364},{"lat":41.0163122472925,"lng":-8.41672437365352},{"lat":41.0166963693461,"lng":-8.41709137018111},{"lat":41.0170644312649,"lng":-8.41736022031137},{"lat":41.0173309540096,"lng":-8.41776475295592},{"lat":41.017425032773,"lng":-8.41791543169366},{"lat":41.0174811197305,"lng":-8.41802908961656},{"lat":41.0175768781271,"lng":-8.41819225833637},{"lat":41.0183372154058,"lng":-8.41888047797732},{"lat":41.0184270303916,"lng":-8.41900700645817},{"lat":41.018565285465,"lng":-8.41935333848036},{"lat":41.0185850553503,"lng":-8.41951416152385},{"lat":41.0185696684141,"lng":-8.41968873642013},{"lat":41.0185570373081,"lng":-8.41984240220472},{"lat":41.0186188721598,"lng":-8.41996061239585},{"lat":41.0186710350731,"lng":-8.42005892387556},{"lat":41.0187521751054,"lng":-8.42016140118649},{"lat":41.0188454605097,"lng":-8.42027035512295},{"lat":41.0189474323878,"lng":-8.42039634959261},{"lat":41.0190313593516,"lng":-8.42053914606132},{"lat":41.019108175166,"lng":-8.42067799095647},{"lat":41.0191765627082,"lng":-8.42084128858408},{"lat":41.0192101711706,"lng":-8.42097446991343},{"lat":41.0191882129051,"lng":-8.42111275704102},{"lat":41.0191735929946,"lng":-8.42123420191584},{"lat":41.0191577824446,"lng":-8.42136253006507},{"lat":41.0191245564784,"lng":-8.42150505262062},{"lat":41.0190768570975,"lng":-8.42167461650783},{"lat":41.0190322254087,"lng":-8.42180460544403},{"lat":41.0190215772483,"lng":-8.42195673260581},{"lat":41.019073597722,"lng":-8.42210902306895},{"lat":41.0191443605044,"lng":-8.42218542349699},{"lat":41.019252076514,"lng":-8.42228362142887},{"lat":41.0193783199042,"lng":-8.42239129788137},{"lat":41.0194666099782,"lng":-8.42244328648052},{"lat":41.0195455785452,"lng":-8.42251425583217},{"lat":41.0196305450248,"lng":-8.42263685048409},{"lat":41.0197423275648,"lng":-8.42282429095729},{"lat":41.0198865729406,"lng":-8.42301533798219},{"lat":41.0199736137935,"lng":-8.42313532619822},{"lat":41.0200688752372,"lng":-8.42324369699315},{"lat":41.0201770413568,"lng":-8.42337626228355},{"lat":41.0202706690637,"lng":-8.42352719368699},{"lat":41.0203289335053,"lng":-8.42387925418713},{"lat":41.0203947461199,"lng":-8.4243094587199},{"lat":41.0204515661339,"lng":-8.42455521801897},{"lat":41.0204824836477,"lng":-8.42468471219842},{"lat":41.0205065233117,"lng":-8.42482737089614},{"lat":41.0204747130449,"lng":-8.42501293771657},{"lat":41.0204432950937,"lng":-8.42525783363024},{"lat":41.020497239354,"lng":-8.42553568230347},{"lat":41.0205701083509,"lng":-8.42570269509231},{"lat":41.0206597176334,"lng":-8.42590937145102},{"lat":41.0206745760527,"lng":-8.42604901915138},{"lat":41.020694698616,"lng":-8.42634930798336},{"lat":41.0207058927822,"lng":-8.42647930685804},{"lat":41.0207248391186,"lng":-8.42667461383809},{"lat":41.0207955527827,"lng":-8.42676944934042},{"lat":41.0209324655549,"lng":-8.42706871854046},{"lat":41.0212154431521,"lng":-8.42743165897977},{"lat":41.0214258984455,"lng":-8.42791221680457},{"lat":41.0215170658203,"lng":-8.42814114386862},{"lat":41.0216423694302,"lng":-8.42840053035631},{"lat":41.0219039442905,"lng":-8.42885683654349},{"lat":41.0220679026113,"lng":-8.4291752583344},{"lat":41.0224077252144,"lng":-8.42983761461929},{"lat":41.0224565048818,"lng":-8.42998621755125},{"lat":41.0225534375827,"lng":-8.42986656840811},{"lat":41.0226752647539,"lng":-8.42969317865656},{"lat":41.0227687391695,"lng":-8.42951584455005},{"lat":41.0228757576164,"lng":-8.42942799086149},{"lat":41.0229363233707,"lng":-8.42923387350071},{"lat":41.0229910424088,"lng":-8.42900001073339},{"lat":41.0230801717956,"lng":-8.42879781408947},{"lat":41.0231841502891,"lng":-8.42866761167676},{"lat":41.0233461529999,"lng":-8.42862708307688},{"lat":41.0234790671832,"lng":-8.42865859435741},{"lat":41.0235666219164,"lng":-8.42878585402658},{"lat":41.0237258680859,"lng":-8.42904719236017},{"lat":41.0237956307081,"lng":-8.42919529954216},{"lat":41.0238588703331,"lng":-8.42932304132181},{"lat":41.0239921944761,"lng":-8.42947416872435},{"lat":41.024110362603,"lng":-8.42953260489496},{"lat":41.0242198477537,"lng":-8.42957066930452},{"lat":41.0244628815534,"lng":-8.42960351148399},{"lat":41.0246381049036,"lng":-8.42960716050625},{"lat":41.0247673376887,"lng":-8.42963378175975},{"lat":41.0248432588054,"lng":-8.42969856534235},{"lat":41.0249782534112,"lng":-8.42983056019947},{"lat":41.0250782412444,"lng":-8.42995502414213},{"lat":41.0251743230324,"lng":-8.43012655704398},{"lat":41.0252750951085,"lng":-8.43029597130348},{"lat":41.0253406588609,"lng":-8.43039423984232},{"lat":41.0255406639273,"lng":-8.43066647848658},{"lat":41.0256958197167,"lng":-8.43080189796703},{"lat":41.0258956084156,"lng":-8.43101777371978},{"lat":41.0261055462492,"lng":-8.43124286041067},{"lat":41.0262330209026,"lng":-8.43132274382533},{"lat":41.0263535689313,"lng":-8.43136514288366},{"lat":41.0264859464311,"lng":-8.43143078408508},{"lat":41.0265847533342,"lng":-8.43148830625987},{"lat":41.0267754412002,"lng":-8.43155368691805},{"lat":41.0270216178111,"lng":-8.43163809344457},{"lat":41.027285471276,"lng":-8.43173489078877},{"lat":41.0274969678549,"lng":-8.43180933110825},{"lat":41.0276109559305,"lng":-8.43188226695173},{"lat":41.0277306157891,"lng":-8.43199633533219},{"lat":41.0278696613404,"lng":-8.43208700614225},{"lat":41.0280618913381,"lng":-8.43219620427601},{"lat":41.0281609342675,"lng":-8.43226740432282},{"lat":41.0282370354322,"lng":-8.43233196203542},{"lat":41.0283634573762,"lng":-8.43243479564245},{"lat":41.0284492160529,"lng":-8.43259285740251},{"lat":41.0285459988467,"lng":-8.43270102486337},{"lat":41.0286503580792,"lng":-8.43277106121857},{"lat":41.0288170990403,"lng":-8.43284875773074},{"lat":41.0289934103593,"lng":-8.4329166250992},{"lat":41.0292615914092,"lng":-8.4330091705928},{"lat":41.029451344675,"lng":-8.43296282944599},{"lat":41.0296954208797,"lng":-8.43283921259351},{"lat":41.0298603356697,"lng":-8.43282272330596},{"lat":41.0300786902227,"lng":-8.43254904046764},{"lat":41.0302810683888,"lng":-8.43225447297266},{"lat":41.0303783766128,"lng":-8.43209367284208},{"lat":41.0305361550842,"lng":-8.43191401048456},{"lat":41.0306737517725,"lng":-8.43180749828177},{"lat":41.0307225572565,"lng":-8.43166716683682},{"lat":41.0308038898736,"lng":-8.43131318513741},{"lat":41.0308521829361,"lng":-8.43116192598043},{"lat":41.0308979435338,"lng":-8.4310185980027},{"lat":41.031018636517,"lng":-8.43072852259377},{"lat":41.0312046006271,"lng":-8.43096279302407},{"lat":41.0312953871378,"lng":-8.43102898367322},{"lat":41.0314334795817,"lng":-8.4311026520192},{"lat":41.0316030621068,"lng":-8.43113023927624},{"lat":41.0318537707341,"lng":-8.43105194638072},{"lat":41.0322139182457,"lng":-8.4307529708096},{"lat":41.0324114300597,"lng":-8.43035407733057},{"lat":41.0324627970806,"lng":-8.43009436607548},{"lat":41.0326738143621,"lng":-8.42963040184448},{"lat":41.0327753944823,"lng":-8.4294838506912},{"lat":41.0330232009609,"lng":-8.42934239186163},{"lat":41.0331631876712,"lng":-8.42921732898892},{"lat":41.0333207454196,"lng":-8.42918950699947},{"lat":41.0341097313565,"lng":-8.42900981155566},{"lat":41.0342700979907,"lng":-8.42901074313204},{"lat":41.0344768384055,"lng":-8.42901194410574},{"lat":41.0346237546147,"lng":-8.42895635595309},{"lat":41.0346706934283,"lng":-8.4287592916394},{"lat":41.0347449212869,"lng":-8.42865438141914},{"lat":41.0348737607065,"lng":-8.42862546458434},{"lat":41.0353284291326,"lng":-8.42857661691429},{"lat":41.0355023368472,"lng":-8.4285434959067},{"lat":41.0356108627833,"lng":-8.428522826877},{"lat":41.0365938958964,"lng":-8.42837538811168},{"lat":41.0364155017129,"lng":-8.42728343339763},{"lat":41.0363664791499,"lng":-8.42698336521346},{"lat":41.0362824998429,"lng":-8.42460458512837},{"lat":41.0365574474297,"lng":-8.42270307095107},{"lat":41.0380908870768,"lng":-8.42163950772444},{"lat":41.0417832781905,"lng":-8.42141783930099},{"lat":41.0452049382971,"lng":-8.4214328343641},{"lat":41.0503401091116,"lng":-8.42038483344612},{"lat":41.0534054507367,"lng":-8.41885185878262},{"lat":41.05580567627,"lng":-8.41832023655247},{"lat":41.060252597198,"lng":-8.41733508993289},{"lat":41.0634050007361,"lng":-8.41699181165235},{"lat":41.0661685263469,"lng":-8.41637454201407},{"lat":41.0677291401337,"lng":-8.4161776483811},{"lat":41.0692607567816,"lng":-8.41582730576844},{"lat":41.0703421554671,"lng":-8.41547501275963},{"lat":41.0707032064834,"lng":-8.41511961718754},{"lat":41.071877560474,"lng":-8.41357785953933},{"lat":41.0721529053581,"lng":-8.4114373067371},{"lat":41.0709895188562,"lng":-8.4084578095556},{"lat":41.0690150989973,"lng":-8.40571301435013},{"lat":41.0660428172512,"lng":-8.40605760701008},{"lat":41.0628004025381,"lng":-8.40640104621036},{"lat":41.0583013348411,"lng":-8.40507378283147},{"lat":41.0564129519035,"lng":-8.40399536210009},{"lat":41.0523732241908,"lng":-8.39874495097786},{"lat":41.0518501631898,"lng":-8.39889782009339},{"lat":41.0514259545721,"lng":-8.39897948280355},{"lat":41.0508969726429,"lng":-8.39908143619917},{"lat":41.050743561509,"lng":-8.39911100360628},{"lat":41.0505891429678,"lng":-8.39910483106296},{"lat":41.0504796284348,"lng":-8.39910045347789},{"lat":41.0498095289416,"lng":-8.3990736681709},{"lat":41.0492664198351,"lng":-8.39883762266351},{"lat":41.0487714742217,"lng":-8.39873039997543},{"lat":41.0479527208506,"lng":-8.39881882020063},{"lat":41.0476836976714,"lng":-8.39880004556227},{"lat":41.0474146733923,"lng":-8.39863107381697},{"lat":41.0470890109512,"lng":-8.39834945424147},{"lat":41.0467633468985,"lng":-8.39808660930434},{"lat":41.0465084782546,"lng":-8.39788008828231},{"lat":41.0463102464047,"lng":-8.39771111653702},{"lat":41.0460553760059,"lng":-8.39748582087662},{"lat":41.0458146641673,"lng":-8.39731684913132},{"lat":41.0457155472719,"lng":-8.39716665202439},{"lat":41.0455376197995,"lng":-8.39702398126978},{"lat":41.0444505499182,"lng":-8.39838534319454},{"lat":41.043907175074,"lng":-8.39906579414418},{"lat":41.043705994609,"lng":-8.39882686173215},{"lat":41.0435602826443,"lng":-8.39856283335137},{"lat":41.0434004179608,"lng":-8.39834560940213},{"lat":41.0431549204744,"lng":-8.39827955258162},{"lat":41.0430557020727,"lng":-8.3982802284208},{"lat":41.0428281285322,"lng":-8.39848779531241},{"lat":41.0426534512913,"lng":-8.39867607557457},{"lat":41.0423855485432,"lng":-8.39880153319188},{"lat":41.0421582672724,"lng":-8.39892179773429},{"lat":41.0420416559267,"lng":-8.39896224226535},{"lat":41.0418137417271,"lng":-8.39896725979649},{"lat":41.0415751222872,"lng":-8.39896903021836},{"lat":41.0413423569324,"lng":-8.39896796747895},{"lat":41.0410337574476,"lng":-8.39882067016853},{"lat":41.0408080324958,"lng":-8.3987365057254},{"lat":41.0406594370107,"lng":-8.39870473021874},{"lat":41.0403428730678,"lng":-8.39861581152518},{"lat":41.0401202429775,"lng":-8.39851632292423},{"lat":41.0400239231434,"lng":-8.39842709549862},{"lat":41.0398283271498,"lng":-8.39824077557572},{"lat":41.0397372103216,"lng":-8.39819914443363},{"lat":41.0396490409595,"lng":-8.39824468835279},{"lat":41.0396324250533,"lng":-8.39838234201272},{"lat":41.0396337008382,"lng":-8.39868465114259},{"lat":41.0396173784782,"lng":-8.39881231355979},{"lat":41.0395308551318,"lng":-8.39900260241009},{"lat":41.0393676495305,"lng":-8.39921434319968},{"lat":41.0391280074676,"lng":-8.39942255670052},{"lat":41.0389334456931,"lng":-8.39960574532704},{"lat":41.0387660553416,"lng":-8.39976743012567},{"lat":41.0385898906448,"lng":-8.39980714730825},{"lat":41.0385079332066,"lng":-8.39985958549985},{"lat":41.0383022517377,"lng":-8.39998373582424},{"lat":41.0380625212007,"lng":-8.40007600086973},{"lat":41.0378210322968,"lng":-8.4001086700023},{"lat":41.0375696863207,"lng":-8.40012180152301},{"lat":41.0373246372952,"lng":-8.40013840003545},{"lat":41.0370825191665,"lng":-8.40017368472733},{"lat":41.0368483881078,"lng":-8.4002182762099},{"lat":41.0366298681704,"lng":-8.40028814553249},{"lat":41.0363890933478,"lng":-8.40047839314415},{"lat":41.0362713606044,"lng":-8.40053499546266},{"lat":41.0360673794735,"lng":-8.40054771979917},{"lat":41.0358267779191,"lng":-8.40054946931403},{"lat":41.0355942646336,"lng":-8.40055648879956},{"lat":41.0353620500404,"lng":-8.40059038887718},{"lat":41.0351351289799,"lng":-8.40063370002074},{"lat":41.0349092052313,"lng":-8.40063527674597},{"lat":41.0346708091755,"lng":-8.40065511641976},{"lat":41.0344455460484,"lng":-8.40072042870371},{"lat":41.0342179853945,"lng":-8.40080452974668},{"lat":41.0339947410573,"lng":-8.40085237080056},{"lat":41.0337687166322,"lng":-8.40089687416482},{"lat":41.0335419368329,"lng":-8.40091806891938},{"lat":41.033283047626,"lng":-8.4009226005099},{"lat":41.0330493139568,"lng":-8.4007587342125},{"lat":41.0329453800371,"lng":-8.40069029367926},{"lat":41.0326310533439,"lng":-8.40068199431933},{"lat":41.0324037220644,"lng":-8.40067000297843},{"lat":41.0322572808505,"lng":-8.40060554963785},{"lat":41.0320567163972,"lng":-8.40046477274619},{"lat":41.0317944462313,"lng":-8.40033599291415},{"lat":41.0315774906849,"lng":-8.40023783657725},{"lat":41.031487634081,"lng":-8.4001566127053},{"lat":41.0313945649751,"lng":-8.39998773749314},{"lat":41.0312670980904,"lng":-8.39973988385387},{"lat":41.0311273409926,"lng":-8.39947343015527},{"lat":41.0309836012937,"lng":-8.39921445537494},{"lat":41.0308421108761,"lng":-8.39895525294631},{"lat":41.0306977185994,"lng":-8.39870542801819},{"lat":41.0305516529,"lng":-8.39847938562654},{"lat":41.0304645864275,"lng":-8.39839817423312},{"lat":41.0303234561458,"lng":-8.39837382323739},{"lat":41.0298583639362,"lng":-8.39837717079605},{"lat":41.0296090603284,"lng":-8.39836474778389},{"lat":41.0293759561263,"lng":-8.39831624393143},{"lat":41.0291450372863,"lng":-8.3982583542278},{"lat":41.0289046437365,"lng":-8.39817034079994},{"lat":41.02868408594,"lng":-8.39803116544129},{"lat":41.0284844642077,"lng":-8.39783214438321},{"lat":41.0283113756171,"lng":-8.39760814132806},{"lat":41.0281625493029,"lng":-8.39736651696507},{"lat":41.0280807767848,"lng":-8.39725284970854},{"lat":41.0279975787875,"lng":-8.39713720142686},{"lat":41.0278064594217,"lng":-8.39692204546747},{"lat":41.0276260985734,"lng":-8.39668945331384},{"lat":41.0274414541137,"lng":-8.39632189357786},{"lat":41.0272916076083,"lng":-8.39610397692789},{"lat":41.027174051306,"lng":-8.39599406947513},{"lat":41.027004734277,"lng":-8.39596854144055},{"lat":41.0267672297522,"lng":-8.39595617502498},{"lat":41.0265484790514,"lng":-8.39589049780573},{"lat":41.0264264319452,"lng":-8.39582948904237},{"lat":41.0263309987876,"lng":-8.39578178419824},{"lat":41.0262448477527,"lng":-8.39569392305354},{"lat":41.0262135537558,"lng":-8.39547977520894},{"lat":41.0262122700268,"lng":-8.39517526118764},{"lat":41.0262489546242,"lng":-8.3950021680151},{"lat":41.0263552936409,"lng":-8.39472923841114},{"lat":41.0264587288253,"lng":-8.39442537910135},{"lat":41.026489749974,"lng":-8.39412955735624},{"lat":41.0264885300214,"lng":-8.39379566834573},{"lat":41.0264113905652,"lng":-8.39346577036842},{"lat":41.0262053524071,"lng":-8.39323677752243},{"lat":41.0260111272857,"lng":-8.39303756066125},{"lat":41.0258706847698,"lng":-8.39270907009609},{"lat":41.0258549348326,"lng":-8.39258511406774},{"lat":41.0258517283945,"lng":-8.39226338887076},{"lat":41.0258277686284,"lng":-8.39213686712262},{"lat":41.0256655353919,"lng":-8.39184718110022},{"lat":41.0255056769294,"lng":-8.39161968797266},{"lat":41.0254259694838,"lng":-8.39138823221341},{"lat":41.0253739946853,"lng":-8.39107757659487},{"lat":41.02536712523,"lng":-8.39076995448259},{"lat":41.0253978882396,"lng":-8.39062501535574},{"lat":41.0255579951263,"lng":-8.39038106813015},{"lat":41.0257651081749,"lng":-8.39021541905089},{"lat":41.0259881324792,"lng":-8.39010249881651},{"lat":41.0261855108573,"lng":-8.39006344073293},{"lat":41.0263131388813,"lng":-8.39000698565738},{"lat":41.0263267448064,"lng":-8.38984331317928},{"lat":41.0262604286824,"lng":-8.38966136887819},{"lat":41.0261816214134,"lng":-8.38939458371253},{"lat":41.0261217145151,"lng":-8.38928058187919},{"lat":41.0260070712264,"lng":-8.38920748672242},{"lat":41.0257895945632,"lng":-8.38905456070015},{"lat":41.0256033211082,"lng":-8.38883921506488},{"lat":41.0255432237107,"lng":-8.38865444772178},{"lat":41.0254521745309,"lng":-8.38838026342505},{"lat":41.0251937997641,"lng":-8.38827450754741},{"lat":41.0251006750914,"lng":-8.38824204116957},{"lat":41.0249334100496,"lng":-8.38806316421443},{"lat":41.0248540825446,"lng":-8.38765812096707},{"lat":41.0248536962009,"lng":-8.38734767055397},{"lat":41.0247897875629,"lng":-8.38721473511334},{"lat":41.0245625036502,"lng":-8.3871784220237},{"lat":41.024465508749,"lng":-8.3871466536544},{"lat":41.0244137432475,"lng":-8.3870199450662},{"lat":41.0244287434179,"lng":-8.3868333331152},{"lat":41.0245180697953,"lng":-8.38642763756713},{"lat":41.0245245362988,"lng":-8.3861495594001},{"lat":41.0244776187941,"lng":-8.38586639620812},{"lat":41.0243769997987,"lng":-8.38555699041126},{"lat":41.0243548409681,"lng":-8.38539758223156},{"lat":41.0243582232005,"lng":-8.38476196546501},{"lat":41.0243571725717,"lng":-8.3844618603476},{"lat":41.024306015937,"lng":-8.3843023462839},{"lat":41.0241728542691,"lng":-8.3841608491669},{"lat":41.0239436338408,"lng":-8.38403473700182},{"lat":41.0237890929029,"lng":-8.38380039272118},{"lat":41.0237285509458,"lng":-8.38369089630244},{"lat":41.0235388404336,"lng":-8.38348043671125},{"lat":41.0233988803696,"lng":-8.3834111846542},{"lat":41.0231510582036,"lng":-8.38333723478737},{"lat":41.0230239405564,"lng":-8.38332473785385},{"lat":41.022797403754,"lng":-8.38331864540926},{"lat":41.0225555516926,"lng":-8.38327325556363},{"lat":41.0224503165356,"lng":-8.38326096175606},{"lat":41.0223542031225,"lng":-8.38331790493783},{"lat":41.0222599413048,"lng":-8.38340932401001},{"lat":41.0221443078308,"lng":-8.38353075251155},{"lat":41.0219799874559,"lng":-8.38356056724687},{"lat":41.0218559603857,"lng":-8.38353595805383},{"lat":41.0216377861883,"lng":-8.38336938358948},{"lat":41.0214177553607,"lng":-8.38322753924157},{"lat":41.0211837889767,"lng":-8.38315828278417},{"lat":41.0209564382053,"lng":-8.38315420685051},{"lat":41.0208598633789,"lng":-8.38317357082175},{"lat":41.0207413886903,"lng":-8.38332768302539},{"lat":41.0206969140856,"lng":-8.3834496158283},{"lat":41.0205994829375,"lng":-8.38373826796483},{"lat":41.0204608163255,"lng":-8.38398349047768},{"lat":41.0203134196258,"lng":-8.38422605484286},{"lat":41.0201808461249,"lng":-8.38448497344765},{"lat":41.0200797593515,"lng":-8.38475541618571},{"lat":41.01999532054,"lng":-8.38503079967184},{"lat":41.0199181612978,"lng":-8.38531358390177},{"lat":41.0198077433525,"lng":-8.38560931499221},{"lat":41.0196348692878,"lng":-8.38580790737721},{"lat":41.01941979327,"lng":-8.3859505785796},{"lat":41.0192016317306,"lng":-8.38602571308664},{"lat":41.0189863220993,"lng":-8.38603058909199},{"lat":41.0187375364481,"lng":-8.38602832132408},{"lat":41.018512908918,"lng":-8.3860122347311},{"lat":41.018353484537,"lng":-8.38595182370492},{"lat":41.0181844245759,"lng":-8.38576808006863},{"lat":41.018036406542,"lng":-8.38563887463826},{"lat":41.0178308914171,"lng":-8.38544834414196},{"lat":41.0176179984801,"lng":-8.38525658949813},{"lat":41.0174824425265,"lng":-8.38523265172237},{"lat":41.017384097468,"lng":-8.3852818456574},{"lat":41.0173725753861,"lng":-8.38556641220032},{"lat":41.0173263183073,"lng":-8.38571746263208},{"lat":41.0172426024749,"lng":-8.38578775822366},{"lat":41.0170979743826,"lng":-8.38579385727692},{"lat":41.016844958525,"lng":-8.38578860455234},{"lat":41.0163745938309,"lng":-8.38541050441944},{"lat":41.0155885429583,"lng":-8.38543198722393},{"lat":41.015325122147,"lng":-8.38569358789675},{"lat":41.0151539251894,"lng":-8.38590786449034},{"lat":41.0149708848325,"lng":-8.3861004650752},{"lat":41.014731603259,"lng":-8.38627869747884},{"lat":41.014665054759,"lng":-8.38636272803922},{"lat":41.0143938091128,"lng":-8.38686430826504},{"lat":41.0142302776548,"lng":-8.38711237542964},{"lat":41.0141232489329,"lng":-8.3871781706647},{"lat":41.0139003527035,"lng":-8.38731413939149},{"lat":41.0138241920923,"lng":-8.38738707496192},{"lat":41.013646155611,"lng":-8.38759810860257},{"lat":41.0135040437498,"lng":-8.38772832902215},{"lat":41.0133823120743,"lng":-8.38780547842136},{"lat":41.0131625954928,"lng":-8.38780986056316},{"lat":41.0129161468974,"lng":-8.38752395277386},{"lat":41.0127407832588,"lng":-8.38730227958961},{"lat":41.0125905153634,"lng":-8.3870497966579},{"lat":41.0124633552127,"lng":-8.3867732699762},{"lat":41.0123223597117,"lng":-8.38652296226903},{"lat":41.0121763104117,"lng":-8.38627751385078},{"lat":41.0120129961729,"lng":-8.38606278864246},{"lat":41.0118552034181,"lng":-8.38583274683562},{"lat":41.011683394918,"lng":-8.38563130201667},{"lat":41.0114203565325,"lng":-8.38547563553937},{"lat":41.0113036515733,"lng":-8.38543691079809},{"lat":41.0111415269882,"lng":-8.38533417047538},{"lat":41.0109614251315,"lng":-8.38509406406162},{"lat":41.0108595068669,"lng":-8.38500820175724},{"lat":41.0106315351012,"lng":-8.38499805408735},{"lat":41.010524835989,"lng":-8.38503592550399},{"lat":41.0103473770355,"lng":-8.38523102797957},{"lat":41.0102214997748,"lng":-8.38531090614173},{"lat":41.0100649722621,"lng":-8.38532385500059},{"lat":41.0098238546181,"lng":-8.38531496386342},{"lat":41.0096651913396,"lng":-8.38527393352956},{"lat":41.0094992513807,"lng":-8.38526996886365},{"lat":41.0092346954514,"lng":-8.38527061853913},{"lat":41.0091184036878,"lng":-8.3854528728009},{"lat":41.0088321545588,"lng":-8.38545236913791},{"lat":41.0086003843655,"lng":-8.38544850242324},{"lat":41.0083725443395,"lng":-8.38541969609821},{"lat":41.0081378868995,"lng":-8.38542010262333},{"lat":41.0079088300569,"lng":-8.38541363484231},{"lat":41.0076874828011,"lng":-8.38537925969213},{"lat":41.0074381861118,"lng":-8.38523910555248},{"lat":41.0072003670811,"lng":-8.38512134834657},{"lat":41.0069798542749,"lng":-8.38507592532011},{"lat":41.0067526117436,"lng":-8.38502251398237},{"lat":41.0065402085821,"lng":-8.38501658306447},{"lat":41.0063201582874,"lng":-8.38500754202068},{"lat":41.0061098510066,"lng":-8.38495026975617},{"lat":41.0060109566355,"lng":-8.38487785629715},{"lat":41.0058385892381,"lng":-8.38468285445036},{"lat":41.0056699662497,"lng":-8.38446361554672},{"lat":41.0054911070681,"lng":-8.38426602579595},{"lat":41.0052833759211,"lng":-8.38406631442956},{"lat":41.0050531625968,"lng":-8.38392874009017},{"lat":41.004754789573,"lng":-8.38391286243762},{"lat":41.0045364042098,"lng":-8.3839248658828},{"lat":41.0043195735804,"lng":-8.38396682529367},{"lat":41.0041014593813,"lng":-8.38397669235078},{"lat":41.0036530261031,"lng":-8.38418583769302}]},{"zone_id":"arc21","points":[{"lat":41.0013801148086,"lng":-8.38921930883302},{"lat":41.0016698252987,"lng":-8.38835332487464},{"lat":41.0017153558711,"lng":-8.38821722535855},{"lat":41.001961859332,"lng":-8.3875423502703},{"lat":41.0020793402232,"lng":-8.38731078872559},{"lat":41.0022102224785,"lng":-8.3870460040957},{"lat":41.0024370976057,"lng":-8.38664232662971},{"lat":41.0025159890289,"lng":-8.38650195438444},{"lat":41.0027783998644,"lng":-8.3860350397359},{"lat":41.0036530261031,"lng":-8.38418583769302},{"lat":41.0035922099932,"lng":-8.38358641916917},{"lat":41.0035764727532,"lng":-8.38345454452396},{"lat":41.0035368381102,"lng":-8.3832960708344},{"lat":41.0035047269457,"lng":-8.38307474633117},{"lat":41.003460869295,"lng":-8.38278753149651},{"lat":41.0034409733021,"lng":-8.38249731813903},{"lat":41.0034435941106,"lng":-8.38220433466452},{"lat":41.0034770215044,"lng":-8.38194688921701},{"lat":41.0034965029876,"lng":-8.38164220033224},{"lat":41.0034772963589,"lng":-8.38132239419021},{"lat":41.0034266970455,"lng":-8.38102849907502},{"lat":41.0033994636364,"lng":-8.38088100659141},{"lat":41.0033035776388,"lng":-8.38058479491023},{"lat":41.0032462543759,"lng":-8.38027625716722},{"lat":41.0031756960717,"lng":-8.37996861909012},{"lat":41.0031003576755,"lng":-8.37970458255142},{"lat":41.0030312073074,"lng":-8.37941073895179},{"lat":41.0030051572509,"lng":-8.37908841297311},{"lat":41.0030069703492,"lng":-8.37895812004551},{"lat":41.0030175267804,"lng":-8.37882078619725},{"lat":41.003056315243,"lng":-8.37870408804674},{"lat":41.0030953827142,"lng":-8.37858359197326},{"lat":41.0032189244274,"lng":-8.37833349648366},{"lat":41.0033478085019,"lng":-8.37806665670723},{"lat":41.003395736991,"lng":-8.37792688227185},{"lat":41.0034452150629,"lng":-8.37778258803776},{"lat":41.0035547040548,"lng":-8.37749095239579},{"lat":41.0036054711818,"lng":-8.37733162980529},{"lat":41.0036791608849,"lng":-8.37718986674571},{"lat":41.0038212076301,"lng":-8.37695766292229},{"lat":41.0039657708699,"lng":-8.37672665481427},{"lat":41.0040347817686,"lng":-8.3766290418892},{"lat":41.0041182270656,"lng":-8.37651101170484},{"lat":41.0042004857066,"lng":-8.37640159454769},{"lat":41.0042739046427,"lng":-8.37630393526581},{"lat":41.0044384643253,"lng":-8.37606979063829},{"lat":41.0045449361258,"lng":-8.37588202416199},{"lat":41.0040700430111,"lng":-8.37502945407756},{"lat":41.0040004024065,"lng":-8.37483319370179},{"lat":41.0039256243921,"lng":-8.37463905333236},{"lat":41.0037668176503,"lng":-8.37436877348353},{"lat":41.0036873409695,"lng":-8.37414026585},{"lat":41.0036010982761,"lng":-8.37395892380174},{"lat":41.0035749117905,"lng":-8.37356563450931},{"lat":41.0035699527084,"lng":-8.37335629706171},{"lat":41.0035168942309,"lng":-8.37314952609095},{"lat":41.0034513215134,"lng":-8.37290300647923},{"lat":41.0034243692427,"lng":-8.37274565390563},{"lat":41.0035022024786,"lng":-8.37251380335606},{"lat":41.0035281539382,"lng":-8.37237495053805},{"lat":41.0035261688478,"lng":-8.37199137398462},{"lat":41.0034662457945,"lng":-8.37157953823685},{"lat":41.0033808521092,"lng":-8.37150779419742},{"lat":41.0032301745003,"lng":-8.37126370417709},{"lat":41.0030900337458,"lng":-8.37101953192316},{"lat":41.0029579870208,"lng":-8.37078001943781},{"lat":41.0028219985726,"lng":-8.37052909140178},{"lat":41.0027588244074,"lng":-8.37033642848481},{"lat":41.0028162601816,"lng":-8.37004055699905},{"lat":41.0028205834971,"lng":-8.36973403404251},{"lat":41.0028220188195,"lng":-8.36943852308564},{"lat":41.0028647907002,"lng":-8.36904040801559},{"lat":41.002904739239,"lng":-8.36892489713704},{"lat":41.0030552656885,"lng":-8.36767453232621},{"lat":41.0031194719052,"lng":-8.36707487002887},{"lat":41.003138703809,"lng":-8.36689524527057},{"lat":41.0030929026487,"lng":-8.36425042189656},{"lat":41.003070951371,"lng":-8.36298407040863},{"lat":41.0030671186371,"lng":-8.36276304543643},{"lat":41.0036830594235,"lng":-8.36026791735396},{"lat":41.0037559564865,"lng":-8.35997291684013},{"lat":41.0036389365215,"lng":-8.35981751734005},{"lat":41.0035219599965,"lng":-8.35964036431826},{"lat":41.0033005217261,"lng":-8.35932961004995},{"lat":41.0029991122368,"lng":-8.35906910095912},{"lat":41.0027730869935,"lng":-8.35894102387971},{"lat":41.0025429698837,"lng":-8.35878535920753},{"lat":41.0024635219198,"lng":-8.35870770873545},{"lat":41.0024180145333,"lng":-8.35835905203978},{"lat":41.0023916496517,"lng":-8.35789327275875},{"lat":41.002358947424,"lng":-8.35748939389781},{"lat":41.002288283325,"lng":-8.35720163706407},{"lat":41.0021588830015,"lng":-8.35693019062632},{"lat":41.0020124283514,"lng":-8.35681368559138},{"lat":41.0019033576257,"lng":-8.35678003133901},{"lat":41.0017986699499,"lng":-8.35680737166208},{"lat":41.0017104657465,"lng":-8.35687898091588},{"lat":41.0014963522528,"lng":-8.35714912427283},{"lat":41.0012706738731,"lng":-8.35752064521687},{"lat":41.0009833403807,"lng":-8.3578940302899},{"lat":41.0007440295528,"lng":-8.35810881649754},{"lat":41.0005133653545,"lng":-8.3582296134089},{"lat":41.0003205903282,"lng":-8.35822336527578},{"lat":41.0002369106986,"lng":-8.35814570489427},{"lat":41.0001408078564,"lng":-8.35792977002816},{"lat":41.0000785156916,"lng":-8.35769184922805},{"lat":41.0000834866419,"lng":-8.35731556924536},{"lat":41.0001640185834,"lng":-8.35684054092447},{"lat":41.0002488227638,"lng":-8.35638751790875},{"lat":41.0003399996517,"lng":-8.35566875133867},{"lat":41.0003723387456,"lng":-8.3553869339785},{"lat":41.0003811485622,"lng":-8.35502208122337},{"lat":41.0003815013534,"lng":-8.35484039406435},{"lat":41.0003820499134,"lng":-8.35455759267546},{"lat":41.0003743893322,"lng":-8.35433079047548},{"lat":41.0003158138645,"lng":-8.35421459159426},{"lat":41.0001401938999,"lng":-8.35404225473572},{"lat":40.9998933297606,"lng":-8.35383710977819},{"lat":40.9996924311165,"lng":-8.35365411141893},{"lat":40.999437008091,"lng":-8.35344858780222},{"lat":40.9992026512163,"lng":-8.35329293482872},{"lat":40.9990520096095,"lng":-8.35319841248045},{"lat":40.9988976217261,"lng":-8.35319150571135},{"lat":40.9987878390841,"lng":-8.35318659444408},{"lat":40.9986149896379,"lng":-8.35321358170868},{"lat":40.9984889960222,"lng":-8.35331822552015},{"lat":40.9983922355836,"lng":-8.35343949163636},{"lat":40.9983503768218,"lng":-8.35357222424275},{"lat":40.9983034133399,"lng":-8.35397010464084},{"lat":40.9983152955759,"lng":-8.35425205693029},{"lat":40.998404054893,"lng":-8.35479581737837},{"lat":40.9984014826964,"lng":-8.35522004294054},{"lat":40.9983714001289,"lng":-8.35564020004759},{"lat":40.9982363585568,"lng":-8.35608757583055},{"lat":40.9981018577155,"lng":-8.35639672461688},{"lat":40.9979877702099,"lng":-8.35676679722203},{"lat":40.9978573978873,"lng":-8.3570815482557},{"lat":40.9977396571928,"lng":-8.35729127200026},{"lat":40.997558929854,"lng":-8.35747867496098},{"lat":40.9970555283871,"lng":-8.35773129236088},{"lat":40.9966823603547,"lng":-8.3577409538999},{"lat":40.9964025035755,"lng":-8.35780461945967},{"lat":40.9959947240523,"lng":-8.35786029605831},{"lat":40.9956762368279,"lng":-8.35785920484447},{"lat":40.9954960355859,"lng":-8.35791396772293},{"lat":40.9952722895916,"lng":-8.35804309986002},{"lat":40.99505428114,"lng":-8.35823036698251},{"lat":40.9949281190302,"lng":-8.35841794796074},{"lat":40.994836892654,"lng":-8.35863080916682},{"lat":40.9946873473495,"lng":-8.35905931666338},{"lat":40.9946500037793,"lng":-8.35917451145438},{"lat":40.9944859385214,"lng":-8.35945026389176},{"lat":40.9942971679789,"lng":-8.35960434737602},{"lat":40.9940116791014,"lng":-8.35980800907198},{"lat":40.9936886743144,"lng":-8.35995057534555},{"lat":40.993281835301,"lng":-8.36004876152143},{"lat":40.9930427507564,"lng":-8.36014740672264},{"lat":40.9928747429717,"lng":-8.36027410451419},{"lat":40.9927783472848,"lng":-8.36038975869088},{"lat":40.9926308981008,"lng":-8.3606381047414},{"lat":40.9924869220428,"lng":-8.36083258357151},{"lat":40.992315926507,"lng":-8.36099650351763},{"lat":40.9921479974732,"lng":-8.36112843065871},{"lat":40.9919384365947,"lng":-8.36123287764074},{"lat":40.9915849688944,"lng":-8.36138779746768},{"lat":40.9912368679659,"lng":-8.36142532811026},{"lat":40.9910313970313,"lng":-8.36141914790101},{"lat":40.9908642257349,"lng":-8.3613078065983},{"lat":40.9907429919645,"lng":-8.36114160976279},{"lat":40.9906373029312,"lng":-8.36079082988484},{"lat":40.9904807160754,"lng":-8.36017882357323},{"lat":40.9904143289435,"lng":-8.35987436918627},{"lat":40.9903349780665,"lng":-8.35970273407423},{"lat":40.9902471873213,"lng":-8.35956434058132},{"lat":40.9900190748069,"lng":-8.35953586187747},{"lat":40.9898491591188,"lng":-8.3595352752442},{"lat":40.9895640868532,"lng":-8.35952870753269},{"lat":40.9893253483596,"lng":-8.35945040315988},{"lat":40.9890700528509,"lng":-8.35932260317549},{"lat":40.9888857573236,"lng":-8.35917259853033},{"lat":40.9887813979497,"lng":-8.35903391098821},{"lat":40.9887311839486,"lng":-8.35892892626869},{"lat":40.9887442280227,"lng":-8.35875190514092},{"lat":40.9888748334898,"lng":-8.35845942501781},{"lat":40.9890345571106,"lng":-8.35823858827271},{"lat":40.9893117482075,"lng":-8.3579025223904},{"lat":40.9895299543363,"lng":-8.35770421438834},{"lat":40.9897355912718,"lng":-8.3575832318014},{"lat":40.9899914588273,"lng":-8.35755653504747},{"lat":40.9902177088575,"lng":-8.35757395158275},{"lat":40.990410048195,"lng":-8.35775690645358},{"lat":40.990598021191,"lng":-8.35805048577877},{"lat":40.9908234871442,"lng":-8.35860421093468},{"lat":40.9909695551174,"lng":-8.35887007947888},{"lat":40.9910742182744,"lng":-8.35899213596437},{"lat":40.9911576155062,"lng":-8.35907537114126},{"lat":40.9912960626074,"lng":-8.35909794817492},{"lat":40.9914360837143,"lng":-8.35911258921349},{"lat":40.9916481989474,"lng":-8.35902167798993},{"lat":40.9919170402235,"lng":-8.35885682458471},{"lat":40.9921601482828,"lng":-8.35863649694785},{"lat":40.992349340794,"lng":-8.35844926122616},{"lat":40.9924963578852,"lng":-8.35823394578014},{"lat":40.9926015713245,"lng":-8.3579854573001},{"lat":40.9926692358821,"lng":-8.35778127956226},{"lat":40.9926779136329,"lng":-8.3576264577057},{"lat":40.9926992610148,"lng":-8.35739419662659},{"lat":40.9926954999885,"lng":-8.35725171180262},{"lat":40.9926913832152,"lng":-8.35709575732541},{"lat":40.992671101554,"lng":-8.35683031572534},{"lat":40.9926422602538,"lng":-8.35661463584748},{"lat":40.9925880165198,"lng":-8.35644866562552},{"lat":40.9924584038104,"lng":-8.35623799555131},{"lat":40.9922828455692,"lng":-8.35608266854658},{"lat":40.9920231775032,"lng":-8.35588819551947},{"lat":40.9917551721785,"lng":-8.35581003458619},{"lat":40.9916235170922,"lng":-8.35575855396235},{"lat":40.9913362318685,"lng":-8.35569785828073},{"lat":40.9910261486975,"lng":-8.35568028476576},{"lat":40.9907034620154,"lng":-8.35566255376741},{"lat":40.9904394618218,"lng":-8.35565619351558},{"lat":40.9902256669909,"lng":-8.3556223127188},{"lat":40.9901042254887,"lng":-8.35556093319541},{"lat":40.9899620060183,"lng":-8.35548867337422},{"lat":40.9898489066446,"lng":-8.35535008005247},{"lat":40.9896692707225,"lng":-8.35511714807221},{"lat":40.9895355255741,"lng":-8.35485145434636},{"lat":40.9894854416586,"lng":-8.35472413299141},{"lat":40.9894692210805,"lng":-8.35449757716407},{"lat":40.9895033700491,"lng":-8.3541603168707},{"lat":40.989629486362,"lng":-8.3539949620081},{"lat":40.9898051395581,"lng":-8.35381932110507},{"lat":40.9899947172743,"lng":-8.35371003146636},{"lat":40.9902484555403,"lng":-8.35362104462837},{"lat":40.990524407828,"lng":-8.35364194088658},{"lat":40.9907779339696,"lng":-8.35366287430125},{"lat":40.9910614278609,"lng":-8.35373869425672},{"lat":40.9914281885501,"lng":-8.35378485178392},{"lat":40.9917538150094,"lng":-8.35377085298594},{"lat":40.9919694674744,"lng":-8.35368173592544},{"lat":40.9921021563326,"lng":-8.35360731361561},{"lat":40.9922650997465,"lng":-8.35348818940691},{"lat":40.9924168568924,"lng":-8.35328405017558},{"lat":40.9925421057848,"lng":-8.35296003428212},{"lat":40.9925577439518,"lng":-8.35274047059124},{"lat":40.9925507082076,"lng":-8.35246592266956},{"lat":40.9924983975544,"lng":-8.35213631839516},{"lat":40.9924043350754,"lng":-8.35188168280091},{"lat":40.9922082512934,"lng":-8.35158142981544},{"lat":40.9920500110203,"lng":-8.351437446568},{"lat":40.9919551259271,"lng":-8.35135111077996},{"lat":40.9917588258938,"lng":-8.35116066422205},{"lat":40.9916001352762,"lng":-8.35099067393852},{"lat":40.991441484778,"lng":-8.35079548915591},{"lat":40.9913360028253,"lng":-8.35062555540916},{"lat":40.9912718899742,"lng":-8.35048558346988},{"lat":40.9912571775825,"lng":-8.35036075736116},{"lat":40.9913028908346,"lng":-8.35014117590594},{"lat":40.9914355555261,"lng":-8.34993720708349},{"lat":40.991594854651,"lng":-8.34978787388436},{"lat":40.9917276671988,"lng":-8.34964867842303},{"lat":40.9918640697946,"lng":-8.34951424305314},{"lat":40.9920042684468,"lng":-8.34932515822245},{"lat":40.9921256764031,"lng":-8.34912091118224},{"lat":40.9922208984334,"lng":-8.34890660119888},{"lat":40.9922665975677,"lng":-8.34869212215059},{"lat":40.9923199829872,"lng":-8.34845770704477},{"lat":40.9923468113912,"lng":-8.34822332289455},{"lat":40.9924727991078,"lng":-8.34768835673146},{"lat":40.9925718242038,"lng":-8.34741427672406},{"lat":40.9926250376087,"lng":-8.34722453677613},{"lat":40.9926818626406,"lng":-8.34707545609487},{"lat":40.9927237788475,"lng":-8.34695580009181},{"lat":40.9927205061168,"lng":-8.34673117596982},{"lat":40.9926300512019,"lng":-8.34651613828743},{"lat":40.9924642338311,"lng":-8.34625117624017},{"lat":40.9922603611849,"lng":-8.34601604138002},{"lat":40.992033492125,"lng":-8.34589551176687},{"lat":40.9918975076855,"lng":-8.34580522906688},{"lat":40.9916932351914,"lng":-8.34578447889734},{"lat":40.9914208055943,"lng":-8.34580866937271},{"lat":40.991110446002,"lng":-8.34579269451841},{"lat":40.9909487999083,"lng":-8.34575866123418},{"lat":40.9907928517558,"lng":-8.3457466206259},{"lat":40.9905812862769,"lng":-8.34558134729205},{"lat":40.990396007379,"lng":-8.34543097261565},{"lat":40.9902224799648,"lng":-8.34532575236431},{"lat":40.9901204967646,"lng":-8.34525542356612},{"lat":40.9900299019837,"lng":-8.3450654707977},{"lat":40.9899170208011,"lng":-8.34480567976369},{"lat":40.9898946713108,"lng":-8.34457114432371},{"lat":40.9898840713665,"lng":-8.34421673775153},{"lat":40.9898850024056,"lng":-8.34371263918527},{"lat":40.9898665091944,"lng":-8.34338827018142},{"lat":40.9898518354958,"lng":-8.34323849243706},{"lat":40.9897801142689,"lng":-8.34306856426034},{"lat":40.9896631258279,"lng":-8.34288363707672},{"lat":40.9895311706346,"lng":-8.34270852428359},{"lat":40.9893573909551,"lng":-8.34250832168428},{"lat":40.9891989263415,"lng":-8.3423530924292},{"lat":40.9890402076722,"lng":-8.3421878844117},{"lat":40.9888779138562,"lng":-8.34200780305367},{"lat":40.988669397913,"lng":-8.34169543846097},{"lat":40.9884353069066,"lng":-8.34138513083768},{"lat":40.988182559112,"lng":-8.34113477728343},{"lat":40.9879369145655,"lng":-8.34093435727005},{"lat":40.9877365618157,"lng":-8.34078910223872},{"lat":40.9875362106573,"lng":-8.3406388558842},{"lat":40.9874193191179,"lng":-8.34054852996547},{"lat":40.9872588344715,"lng":-8.34053882387224},{"lat":40.9870448134791,"lng":-8.34055233781024},{"lat":40.9867796222645,"lng":-8.34060628020038},{"lat":40.9865677070501,"lng":-8.34062569023149},{"lat":40.9855845868298,"lng":-8.3404925656757},{"lat":40.9847988797924,"lng":-8.34020714230303},{"lat":40.9843713034209,"lng":-8.33949840435715},{"lat":40.9841252874475,"lng":-8.33860135515574},{"lat":40.984100993693,"lng":-8.3383385564979},{"lat":40.9839142326465,"lng":-8.33814273825304},{"lat":40.98355560048,"lng":-8.33808362941141},{"lat":40.9829840069815,"lng":-8.33812497438441},{"lat":40.9824854464616,"lng":-8.33834495404331},{"lat":40.9823785980206,"lng":-8.3383920982828},{"lat":40.9819207635801,"lng":-8.33841835273906},{"lat":40.98154840085,"lng":-8.33827947373964},{"lat":40.9812756704785,"lng":-8.33802077638162},{"lat":40.9812992030808,"lng":-8.33790381127882},{"lat":40.981425306603,"lng":-8.33727705656802},{"lat":40.9816093295536,"lng":-8.33709179168498},{"lat":40.9819346555907,"lng":-8.33694392087777},{"lat":40.9822254031287,"lng":-8.33681244629529},{"lat":40.9823033118592,"lng":-8.33673843087164},{"lat":40.9823827394228,"lng":-8.33607552459153},{"lat":40.9824100449986,"lng":-8.33591187150285},{"lat":40.9825158644509,"lng":-8.33575190807899},{"lat":40.9826384171091,"lng":-8.3355980518351},{"lat":40.9829190712171,"lng":-8.33540381410819},{"lat":40.9830621385801,"lng":-8.33531216007406},{"lat":40.983143005385,"lng":-8.33510589704368},{"lat":40.9831147229072,"lng":-8.33485532809948},{"lat":40.9829844398441,"lng":-8.33469427757145},{"lat":40.9828554830939,"lng":-8.33460012568816},{"lat":40.9826746865145,"lng":-8.33453944635983},{"lat":40.9824608123645,"lng":-8.33454876859954},{"lat":40.9823639710531,"lng":-8.33454844282938},{"lat":40.9820502961234,"lng":-8.33454738765044},{"lat":40.9818515388504,"lng":-8.33446193569917},{"lat":40.9815162036229,"lng":-8.33426128957255},{"lat":40.9813039108943,"lng":-8.33413884659755},{"lat":40.9811427355267,"lng":-8.33398186220371},{"lat":40.9811101760249,"lng":-8.33375564336128},{"lat":40.9811821562186,"lng":-8.33347936760893},{"lat":40.9813092424457,"lng":-8.33335867666911},{"lat":40.9815073972341,"lng":-8.33336353832155},{"lat":40.9817696857049,"lng":-8.33333584834818},{"lat":40.9818146285017,"lng":-8.33322714067848},{"lat":40.9817663644337,"lng":-8.33297176192484},{"lat":40.9816157387048,"lng":-8.33285557339885},{"lat":40.9815439476476,"lng":-8.3327696923195},{"lat":40.9814528253789,"lng":-8.33266068638267},{"lat":40.9814530376018,"lng":-8.33253960526446},{"lat":40.9815686847586,"lng":-8.33242173053541},{"lat":40.9817547829225,"lng":-8.3323322297124},{"lat":40.9819915425603,"lng":-8.33226331847832},{"lat":40.9821126881082,"lng":-8.33217318233316},{"lat":40.9824076709602,"lng":-8.33193841084977},{"lat":40.9828258626922,"lng":-8.33140711393291},{"lat":40.9832512598428,"lng":-8.33082508831197},{"lat":40.9833963760071,"lng":-8.33037838381325},{"lat":40.9835220856311,"lng":-8.3299046483861},{"lat":40.9835563543059,"lng":-8.32914272286786},{"lat":40.9836319251787,"lng":-8.32807636992112},{"lat":40.9837128439676,"lng":-8.32746347400826},{"lat":40.983852103612,"lng":-8.32700842276794},{"lat":40.9838972557012,"lng":-8.32677589942755},{"lat":40.9838463155868,"lng":-8.32660047548138},{"lat":40.9837680587573,"lng":-8.32644541386265},{"lat":40.9836843212884,"lng":-8.32638931946297},{"lat":40.9833330547625,"lng":-8.3261218782033},{"lat":40.9832637730722,"lng":-8.32598514610983},{"lat":40.983192142397,"lng":-8.32574360287722},{"lat":40.9831012074607,"lng":-8.32557840793203},{"lat":40.9828674377778,"lng":-8.32553137449053},{"lat":40.9822184987853,"lng":-8.32557533673466},{"lat":40.9819782955955,"lng":-8.32549787464849},{"lat":40.9818869264617,"lng":-8.32532067824292},{"lat":40.9819169692312,"lng":-8.32502038015159},{"lat":40.98198323587,"lng":-8.32475702916123},{"lat":40.9821329355887,"lng":-8.32441717416934},{"lat":40.9821380918951,"lng":-8.32417750734132},{"lat":40.9819824396681,"lng":-8.32404895884721},{"lat":40.9818126965628,"lng":-8.32400789690896},{"lat":40.9814378345248,"lng":-8.32410975717145},{"lat":40.9810569542813,"lng":-8.32415198500865},{"lat":40.9807691118686,"lng":-8.32408083290582},{"lat":40.9806466168075,"lng":-8.32383344969506},{"lat":40.9806124549768,"lng":-8.32320372244745},{"lat":40.9805080025042,"lng":-8.32271720957219},{"lat":40.9805027539452,"lng":-8.32255038899415},{"lat":40.9804725063706,"lng":-8.3215891479202},{"lat":40.9803767767931,"lng":-8.32126675401412},{"lat":40.9802719392287,"lng":-8.32120265419298},{"lat":40.979981237996,"lng":-8.32119563054822},{"lat":40.9795465347002,"lng":-8.3213361366849},{"lat":40.9793727620592,"lng":-8.32110643567213},{"lat":40.9793090382113,"lng":-8.32081776078226},{"lat":40.9789743949471,"lng":-8.3208367605289},{"lat":40.978717097847,"lng":-8.32091396973297},{"lat":40.9786547817154,"lng":-8.32102630999215},{"lat":40.9786911005576,"lng":-8.32117244037476},{"lat":40.9790168770946,"lng":-8.32156036132655},{"lat":40.9790968740893,"lng":-8.32170103629717},{"lat":40.9791903570161,"lng":-8.32212762471828},{"lat":40.9791895825154,"lng":-8.32243060709222},{"lat":40.9791857516697,"lng":-8.32273156197262},{"lat":40.9791745299538,"lng":-8.32303522488662},{"lat":40.9791380044979,"lng":-8.32333988317983},{"lat":40.9791298169472,"lng":-8.32349729334382},{"lat":40.9790772686008,"lng":-8.32363782356251},{"lat":40.9789626619385,"lng":-8.32367967077363},{"lat":40.9788493402572,"lng":-8.32365177402701},{"lat":40.978677181064,"lng":-8.3236452117596},{"lat":40.9785503652276,"lng":-8.3237195831907},{"lat":40.9784930822445,"lng":-8.32383490189825},{"lat":40.9784428245404,"lng":-8.3239502465953},{"lat":40.9783914493656,"lng":-8.32419485890182},{"lat":40.9782054454525,"lng":-8.32445430497096},{"lat":40.9733320157078,"lng":-8.34970305859324},{"lat":40.972467896656,"lng":-8.35309514339191},{"lat":40.971315949941,"lng":-8.35742918319059},{"lat":40.9665398242665,"lng":-8.37300300671714},{"lat":40.9724339815285,"lng":-8.38091503462413},{"lat":40.9756618384265,"lng":-8.38658556907383},{"lat":40.9775661694168,"lng":-8.38970810600835},{"lat":40.9864350715973,"lng":-8.4028792995486},{"lat":40.9865174053576,"lng":-8.40278932400369},{"lat":40.9867185360553,"lng":-8.40238125216504},{"lat":40.9868523721611,"lng":-8.40210195427883},{"lat":40.987060758776,"lng":-8.40201653315645},{"lat":40.9871500257405,"lng":-8.4019776993734},{"lat":40.9872592356697,"lng":-8.40189328465073},{"lat":40.9873459614008,"lng":-8.40171672151383},{"lat":40.9874510447488,"lng":-8.40161106552452},{"lat":40.9875463583313,"lng":-8.40155147146501},{"lat":40.9876383320796,"lng":-8.40146846324294},{"lat":40.9876905264283,"lng":-8.40136755064132},{"lat":40.9877302743381,"lng":-8.40112149099816},{"lat":40.9877890290663,"lng":-8.40102488256787},{"lat":40.9878728407111,"lng":-8.400956660307},{"lat":40.9879612421725,"lng":-8.40085113764485},{"lat":40.9879779584815,"lng":-8.40071926001928},{"lat":40.9879896024696,"lng":-8.40058340633509},{"lat":40.9880777037875,"lng":-8.4004263475051},{"lat":40.9881499225969,"lng":-8.40033009107404},{"lat":40.988219509472,"lng":-8.40022783750652},{"lat":40.9883102125135,"lng":-8.40017722682596},{"lat":40.9883843297701,"lng":-8.40009517420471},{"lat":40.9884948848604,"lng":-8.39976393179731},{"lat":40.9885631106613,"lng":-8.39955306611516},{"lat":40.988613009726,"lng":-8.39945368623155},{"lat":40.9886986581413,"lng":-8.39929194470332},{"lat":40.9887865335873,"lng":-8.39918035813043},{"lat":40.9888718026122,"lng":-8.39910905761464},{"lat":40.9889707347764,"lng":-8.39903900076818},{"lat":40.9890743037625,"lng":-8.39897724024218},{"lat":40.9892334342446,"lng":-8.39890694160555},{"lat":40.9894018935104,"lng":-8.3987578940186},{"lat":40.9894939822177,"lng":-8.39853699215218},{"lat":40.9895384423183,"lng":-8.39839159942228},{"lat":40.9896534444042,"lng":-8.39831492812157},{"lat":40.9897199328813,"lng":-8.39822969146818},{"lat":40.989759607271,"lng":-8.39811274769741},{"lat":40.9897670490269,"lng":-8.39781473828123},{"lat":40.9897927182597,"lng":-8.39761793022683},{"lat":40.9898513723307,"lng":-8.39748696992904},{"lat":40.990046370398,"lng":-8.39738685869355},{"lat":40.9902015803794,"lng":-8.39735908195982},{"lat":40.9903297156473,"lng":-8.39732050508663},{"lat":40.9905792603725,"lng":-8.39722929032881},{"lat":40.9907754624941,"lng":-8.39712313824103},{"lat":40.9911644344036,"lng":-8.39696095253318},{"lat":40.9912658966717,"lng":-8.39692277093052},{"lat":40.9913519566074,"lng":-8.39687088567471},{"lat":40.9914477917995,"lng":-8.39685879288783},{"lat":40.9918324206084,"lng":-8.3968007968037},{"lat":40.9919628535329,"lng":-8.39681974171009},{"lat":40.9920561109219,"lng":-8.39679337269043},{"lat":40.9922356141567,"lng":-8.39684590550316},{"lat":40.9923617979921,"lng":-8.39684272258401},{"lat":40.9925085454911,"lng":-8.39688848584679},{"lat":40.9926449858175,"lng":-8.39699694024807},{"lat":40.9927662266746,"lng":-8.39694157214758},{"lat":40.9928762599402,"lng":-8.39690172409039},{"lat":40.9929691650683,"lng":-8.39675199692901},{"lat":40.9930526226325,"lng":-8.3966209051954},{"lat":40.9931597657033,"lng":-8.396531839018},{"lat":40.9932810528702,"lng":-8.39646809791772},{"lat":40.9934326617766,"lng":-8.39638842121521},{"lat":40.9935996165844,"lng":-8.39626620597191},{"lat":40.9937478197384,"lng":-8.39623328340572},{"lat":40.9938563417553,"lng":-8.39622539691925},{"lat":40.9939756185506,"lng":-8.39620871859553},{"lat":40.9941149418603,"lng":-8.39618923711747},{"lat":40.9942607740093,"lng":-8.39600992749077},{"lat":40.9943396179806,"lng":-8.39583122608895},{"lat":40.9944932386098,"lng":-8.39576813520445},{"lat":40.9946293326618,"lng":-8.395578773252},{"lat":40.9947811842636,"lng":-8.39532433351506},{"lat":40.9948230074847,"lng":-8.39519187007714},{"lat":40.994840673569,"lng":-8.39502709820989},{"lat":40.9949333107577,"lng":-8.39470411893657},{"lat":40.9949629380218,"lng":-8.39456040243352},{"lat":40.9950024971503,"lng":-8.39443054941273},{"lat":40.9950640152866,"lng":-8.39418561274612},{"lat":40.9952961713451,"lng":-8.39374528257893},{"lat":40.9954761013462,"lng":-8.39365469593076},{"lat":40.9955753841598,"lng":-8.39355407179299},{"lat":40.9956496205244,"lng":-8.39345702835982},{"lat":40.995752103678,"lng":-8.39328240611922},{"lat":40.9958171078576,"lng":-8.39317923680381},{"lat":40.9959028128455,"lng":-8.39302839795573},{"lat":40.9959489916069,"lng":-8.392902964773},{"lat":40.9960085774737,"lng":-8.39280752012128},{"lat":40.9961487017662,"lng":-8.39256253819729},{"lat":40.9962269249929,"lng":-8.39245528739196},{"lat":40.9963361839438,"lng":-8.39234428236242},{"lat":40.99642952981,"lng":-8.39225558905287},{"lat":40.9965269502646,"lng":-8.392259068995},{"lat":40.9966003393357,"lng":-8.3921779453086},{"lat":40.9966925606777,"lng":-8.39217093646348},{"lat":40.9967965589782,"lng":-8.39213355869641},{"lat":40.9968971282218,"lng":-8.39214031313833},{"lat":40.9969748081533,"lng":-8.39207269771276},{"lat":40.9970156336493,"lng":-8.39193784178253},{"lat":40.9970379144083,"lng":-8.39180380333225},{"lat":40.9971250874518,"lng":-8.39175536773149},{"lat":40.9972734757743,"lng":-8.39168175127289},{"lat":40.9973672705885,"lng":-8.3916132208158},{"lat":40.9974042775772,"lng":-8.39148247889364},{"lat":40.9975048585817,"lng":-8.39141397934384},{"lat":40.9976129716977,"lng":-8.39141250461396},{"lat":40.9977075028986,"lng":-8.39140021197625},{"lat":40.9977810369495,"lng":-8.39133151151295},{"lat":40.9978319507899,"lng":-8.39122737638652},{"lat":40.9978941132602,"lng":-8.39113977680584},{"lat":40.998043355422,"lng":-8.3909159131965},{"lat":40.9981444261448,"lng":-8.39082639312649},{"lat":40.9981635799424,"lng":-8.39059708882186},{"lat":40.9981737229765,"lng":-8.39041148245901},{"lat":40.9982832809077,"lng":-8.39034309374272},{"lat":40.9983770477391,"lng":-8.39029069099358},{"lat":40.9985073945037,"lng":-8.39018720796068},{"lat":40.9986449501825,"lng":-8.39012213752102},{"lat":40.9987664599194,"lng":-8.39006580206978},{"lat":40.9988760756233,"lng":-8.39001179850394},{"lat":40.9990382509909,"lng":-8.38992923601597},{"lat":40.9993792023542,"lng":-8.38975121664779},{"lat":40.9995142738359,"lng":-8.38970860084832},{"lat":40.9996642131504,"lng":-8.38965310642389},{"lat":40.9997662652277,"lng":-8.38960879914621},{"lat":40.9998612007647,"lng":-8.38955627867776},{"lat":41.000010752876,"lng":-8.38948150945315},{"lat":41.0002493099916,"lng":-8.38958785376098},{"lat":41.0005627207212,"lng":-8.38972684550483},{"lat":41.0006104102167,"lng":-8.3896216040789},{"lat":41.0007047411119,"lng":-8.38950897551008},{"lat":41.0007956109148,"lng":-8.38943096850702},{"lat":41.0010071174067,"lng":-8.3893541796058},{"lat":41.0012770352642,"lng":-8.38928048038447},{"lat":41.0013801148086,"lng":-8.38921930883302}]},{"zone_id":"gdm7","points":[{"lat":41.0530898310399,"lng":-8.44808118891258},{"lat":41.0534882650347,"lng":-8.44785972459499},{"lat":41.0661018059686,"lng":-8.44151061562332},{"lat":41.0687814886178,"lng":-8.44533226559641},{"lat":41.0770770809805,"lng":-8.4481453933645},{"lat":41.0851172556084,"lng":-8.44583920832847},{"lat":41.091041222472,"lng":-8.44299462124276},{"lat":41.0916068203783,"lng":-8.44253985528578},{"lat":41.0910087793615,"lng":-8.44186207287497},{"lat":41.0903638713627,"lng":-8.44099901557843},{"lat":41.0899926632106,"lng":-8.44057423779824},{"lat":41.0898768833243,"lng":-8.44044175089089},{"lat":41.0894706822759,"lng":-8.43964798900703},{"lat":41.0894046465211,"lng":-8.43951894976447},{"lat":41.0891333465233,"lng":-8.43920648164747},{"lat":41.0889432435749,"lng":-8.43898753386581},{"lat":41.0884742874124,"lng":-8.43872053789439},{"lat":41.086924243506,"lng":-8.437413401222},{"lat":41.0867554061967,"lng":-8.43693093685578},{"lat":41.0867135199405,"lng":-8.43681124524297},{"lat":41.0876003121245,"lng":-8.43390652232281},{"lat":41.0884084224349,"lng":-8.43125924812388},{"lat":41.0900642896495,"lng":-8.42845929001946},{"lat":41.0833307777556,"lng":-8.42585937954339},{"lat":41.0784073835573,"lng":-8.42253581541105},{"lat":41.0740377563277,"lng":-8.42057747904183},{"lat":41.0707032064834,"lng":-8.41511961718754},{"lat":41.0703421554671,"lng":-8.41547501275963},{"lat":41.0692607567816,"lng":-8.41582730576844},{"lat":41.0677291401337,"lng":-8.4161776483811},{"lat":41.0661685263469,"lng":-8.41637454201407},{"lat":41.0634050007361,"lng":-8.41699181165235},{"lat":41.060252597198,"lng":-8.41733508993289},{"lat":41.05580567627,"lng":-8.41832023655247},{"lat":41.0534054507367,"lng":-8.41885185878262},{"lat":41.0503401091116,"lng":-8.42038483344612},{"lat":41.0452049382971,"lng":-8.4214328343641},{"lat":41.0417832781905,"lng":-8.42141783930099},{"lat":41.0380908870768,"lng":-8.42163950772444},{"lat":41.0365574474297,"lng":-8.42270307095107},{"lat":41.0362824998429,"lng":-8.42460458512837},{"lat":41.0363664791499,"lng":-8.42698336521346},{"lat":41.0364155017129,"lng":-8.42728343339763},{"lat":41.0365938958964,"lng":-8.42837538811168},{"lat":41.0366198354486,"lng":-8.4285373919409},{"lat":41.0367176945014,"lng":-8.42914857842214},{"lat":41.0370956651887,"lng":-8.43044747712375},{"lat":41.0372183468822,"lng":-8.43074367422276},{"lat":41.0375933915373,"lng":-8.43157021724379},{"lat":41.0377053557287,"lng":-8.4317629087784},{"lat":41.0378952942591,"lng":-8.43202516873566},{"lat":41.038061179319,"lng":-8.43222285823967},{"lat":41.0389125906502,"lng":-8.43310190389207},{"lat":41.0400231524314,"lng":-8.43411583348953},{"lat":41.0409833055236,"lng":-8.43506964836328},{"lat":41.0424254422126,"lng":-8.43640331492436},{"lat":41.0433342952597,"lng":-8.43717141980218},{"lat":41.0443048414693,"lng":-8.43816984998439},{"lat":41.0445279511711,"lng":-8.43834620174403},{"lat":41.0452628937201,"lng":-8.43878432737856},{"lat":41.0453849299881,"lng":-8.43887516939125},{"lat":41.0457780009643,"lng":-8.43923607111373},{"lat":41.046658434261,"lng":-8.44041945593919},{"lat":41.0470499595232,"lng":-8.44085054405228},{"lat":41.0482531714258,"lng":-8.44187242412746},{"lat":41.0487399860858,"lng":-8.44239449239723},{"lat":41.0494255877381,"lng":-8.44305964127588},{"lat":41.0504504936185,"lng":-8.44427987248234},{"lat":41.0510784099889,"lng":-8.44509311281099},{"lat":41.0514503693866,"lng":-8.44562585781939},{"lat":41.0522429687451,"lng":-8.44653756977101},{"lat":41.0523978949572,"lng":-8.44678263261346},{"lat":41.0530898310399,"lng":-8.44808118891258}]},{"zone_id":"gdm9","points":[{"lat":41.0707032064834,"lng":-8.41511961718754},{"lat":41.0740377563277,"lng":-8.42057747904183},{"lat":41.0784073835573,"lng":-8.42253581541105},{"lat":41.0833307777556,"lng":-8.42585937954339},{"lat":41.0900642896495,"lng":-8.42845929001946},{"lat":41.0907407091611,"lng":-8.42712883706054},{"lat":41.091092840753,"lng":-8.42643620487979},{"lat":41.0911145274455,"lng":-8.42600390978208},{"lat":41.0911330653386,"lng":-8.42563435592378},{"lat":41.0912052629145,"lng":-8.42419486631066},{"lat":41.0913538377459,"lng":-8.42423841213387},{"lat":41.0914402849074,"lng":-8.42426374904147},{"lat":41.0916044785059,"lng":-8.42431803816278},{"lat":41.0918512472068,"lng":-8.42433366033522},{"lat":41.0919285469723,"lng":-8.42435149548625},{"lat":41.0921313108295,"lng":-8.42439274653838},{"lat":41.0921707119469,"lng":-8.42413849434314},{"lat":41.0922253384165,"lng":-8.42381793714995},{"lat":41.0922803591314,"lng":-8.42367523511666},{"lat":41.0924134626872,"lng":-8.42341826634998},{"lat":41.092437035374,"lng":-8.42335469542662},{"lat":41.0924452087705,"lng":-8.42326810505632},{"lat":41.092447464504,"lng":-8.42311288980971},{"lat":41.0924560513703,"lng":-8.42274408009832},{"lat":41.0925182948812,"lng":-8.42259212460345},{"lat":41.0925631657217,"lng":-8.42243914756158},{"lat":41.0925793949209,"lng":-8.42235863793032},{"lat":41.0926080335529,"lng":-8.42229927338725},{"lat":41.0928592689115,"lng":-8.42177848687076},{"lat":41.093667711372,"lng":-8.42185639423058},{"lat":41.0941877712243,"lng":-8.42162676706104},{"lat":41.094577973289,"lng":-8.42092502001199},{"lat":41.09489796298,"lng":-8.42076429408938},{"lat":41.0954140945543,"lng":-8.42001860858322},{"lat":41.0957589305231,"lng":-8.41896085369588},{"lat":41.09658636636,"lng":-8.41628449603349},{"lat":41.0975895268362,"lng":-8.41330983572568},{"lat":41.0977466132133,"lng":-8.41284400046491},{"lat":41.0977033910762,"lng":-8.41279794248109},{"lat":41.0976537499214,"lng":-8.41274504442092},{"lat":41.0974644857334,"lng":-8.41255843094732},{"lat":41.0972970404978,"lng":-8.41235822235245},{"lat":41.0971975786353,"lng":-8.41221488521459},{"lat":41.0971405552029,"lng":-8.41213270754098},{"lat":41.0969973514649,"lng":-8.41188915965583},{"lat":41.0969283860892,"lng":-8.41177062881169},{"lat":41.0968586334305,"lng":-8.41165074533148},{"lat":41.0967199902057,"lng":-8.41138115114564},{"lat":41.0965957618409,"lng":-8.41110959178663},{"lat":41.0965589843793,"lng":-8.41105194879836},{"lat":41.0964942782856,"lng":-8.41092574002536},{"lat":41.0964539065939,"lng":-8.41075321472585},{"lat":41.0964691469537,"lng":-8.41052093855583},{"lat":41.0965035578705,"lng":-8.4104770386842},{"lat":41.0965292694627,"lng":-8.41042120394297},{"lat":41.0966004348547,"lng":-8.41033461483684},{"lat":41.0965311638196,"lng":-8.41023302908895},{"lat":41.0964145032327,"lng":-8.40995829183016},{"lat":41.0963504927175,"lng":-8.40976840765245},{"lat":41.0963146417044,"lng":-8.40969815237681},{"lat":41.0962838939499,"lng":-8.40964195930191},{"lat":41.0962518674186,"lng":-8.40959219327031},{"lat":41.0961922684511,"lng":-8.4094777918288},{"lat":41.0961234461632,"lng":-8.40930288747878},{"lat":41.0960848287996,"lng":-8.40922356894572},{"lat":41.0960539622646,"lng":-8.40915896602871},{"lat":41.0959694333343,"lng":-8.40898204970699},{"lat":41.0958533095684,"lng":-8.40870862975378},{"lat":41.0957346611229,"lng":-8.40843448724138},{"lat":41.0957069906277,"lng":-8.40837069265397},{"lat":41.0956736337465,"lng":-8.4083129412674},{"lat":41.0955859800987,"lng":-8.40814308037218},{"lat":41.0954849534472,"lng":-8.40791781493305},{"lat":41.0954550089527,"lng":-8.40786198501769},{"lat":41.0953592759455,"lng":-8.40764483758068},{"lat":41.0953165646568,"lng":-8.40750777780302},{"lat":41.0952821302781,"lng":-8.40741074594393},{"lat":41.0952564076101,"lng":-8.4073606425566},{"lat":41.0952312037518,"lng":-8.40730385300252},{"lat":41.0950197951158,"lng":-8.40684816394566},{"lat":41.0949783659788,"lng":-8.40678982448878},{"lat":41.0948072067002,"lng":-8.40655866977242},{"lat":41.0946693266842,"lng":-8.40643166133868},{"lat":41.0946157487925,"lng":-8.40638230814349},{"lat":41.094469192523,"lng":-8.40625410346072},{"lat":41.0943301975384,"lng":-8.40612688048193},{"lat":41.0941505162562,"lng":-8.40595759501085},{"lat":41.094059984198,"lng":-8.4058218836477},{"lat":41.0939159590991,"lng":-8.40557538422024},{"lat":41.0937643290848,"lng":-8.40534836624374},{"lat":41.0937289730968,"lng":-8.40529763945168},{"lat":41.0934900641114,"lng":-8.40492898193189},{"lat":41.0933099267978,"lng":-8.40472244068746},{"lat":41.0932644384969,"lng":-8.40469166414259},{"lat":41.0930269111202,"lng":-8.40452189046888},{"lat":41.0928566286162,"lng":-8.40438848782737},{"lat":41.0926720771153,"lng":-8.40422026207995},{"lat":41.0924907909145,"lng":-8.40404300377717},{"lat":41.0924356110642,"lng":-8.4039875449947},{"lat":41.0922917282666,"lng":-8.4038429354418},{"lat":41.0920932394222,"lng":-8.40366524918259},{"lat":41.0918960950471,"lng":-8.40349162100387},{"lat":41.0917159028239,"lng":-8.40330698958154},{"lat":41.0915381442415,"lng":-8.40312272857766},{"lat":41.0914056612852,"lng":-8.4030035205107},{"lat":41.0913088275567,"lng":-8.40291992781507},{"lat":41.0911396884341,"lng":-8.40278378192066},{"lat":41.0909412254415,"lng":-8.40259432505674},{"lat":41.0908183178104,"lng":-8.40247486657793},{"lat":41.0907475352953,"lng":-8.40240607068866},{"lat":41.0906856765986,"lng":-8.40234071426246},{"lat":41.0905672702568,"lng":-8.40221561322356},{"lat":41.0904216776125,"lng":-8.40206672837381},{"lat":41.0903788978352,"lng":-8.4020229814048},{"lat":41.0902562740812,"lng":-8.40190250704026},{"lat":41.0901704440453,"lng":-8.40179373018701},{"lat":41.0900856299422,"lng":-8.40163806333269},{"lat":41.0900639993062,"lng":-8.40149432253734},{"lat":41.0900587805488,"lng":-8.40118593320065},{"lat":41.0900704155856,"lng":-8.40086939293197},{"lat":41.0901045695254,"lng":-8.40081787642084},{"lat":41.0901342307172,"lng":-8.40076420167182},{"lat":41.0902572549082,"lng":-8.40052238964761},{"lat":41.0902858225245,"lng":-8.4004374059056},{"lat":41.0903257493962,"lng":-8.40033727160928},{"lat":41.0903914289457,"lng":-8.40017255034071},{"lat":41.0904472723162,"lng":-8.40005187530648},{"lat":41.0905173159681,"lng":-8.39990051342185},{"lat":41.0905696164567,"lng":-8.39979170218452},{"lat":41.0906165105345,"lng":-8.3996840662565},{"lat":41.0906257554326,"lng":-8.39961828236318},{"lat":41.0906337390545,"lng":-8.39955393110926},{"lat":41.0906350015705,"lng":-8.39912832777976},{"lat":41.0906122170009,"lng":-8.39901230631101},{"lat":41.0906111985324,"lng":-8.39889923710657},{"lat":41.0906102762914,"lng":-8.39879684197429},{"lat":41.0906094971298,"lng":-8.39871034932366},{"lat":41.0906335695713,"lng":-8.39842587219933},{"lat":41.0906348985565,"lng":-8.39808703421723},{"lat":41.0906269630661,"lng":-8.39797417319423},{"lat":41.0906170579063,"lng":-8.39789748497384},{"lat":41.090601291138,"lng":-8.39778340238299},{"lat":41.0905784060374,"lng":-8.39771214060191},{"lat":41.0905278040528,"lng":-8.39755542620209},{"lat":41.0905012047095,"lng":-8.39737571852088},{"lat":41.0904928753529,"lng":-8.39720025576054},{"lat":41.0904726062961,"lng":-8.39708746473941},{"lat":41.0904426684506,"lng":-8.39702878518054},{"lat":41.0903621952097,"lng":-8.39690146646021},{"lat":41.0903271295978,"lng":-8.39682808867408},{"lat":41.0902405893974,"lng":-8.39664699667332},{"lat":41.090086209236,"lng":-8.3963941786958},{"lat":41.0899982727056,"lng":-8.39620038923723},{"lat":41.0899405080429,"lng":-8.39608450586445},{"lat":41.0897916882053,"lng":-8.3958398057176},{"lat":41.0896671524633,"lng":-8.39568541866898},{"lat":41.0895755621428,"lng":-8.39561363761517},{"lat":41.0893439289164,"lng":-8.39539840532994},{"lat":41.0893043896258,"lng":-8.39536166583433},{"lat":41.089134734589,"lng":-8.39513081305444},{"lat":41.0890574023049,"lng":-8.39502925162264},{"lat":41.0889610235958,"lng":-8.39490267678993},{"lat":41.0887766232583,"lng":-8.39470176114305},{"lat":41.0885997588285,"lng":-8.39451217464605},{"lat":41.0884661621709,"lng":-8.39438346673009},{"lat":41.0883634878721,"lng":-8.39425500796806},{"lat":41.0882930520124,"lng":-8.39416688397969},{"lat":41.088101537878,"lng":-8.39396510524432},{"lat":41.0879740850483,"lng":-8.39386474202231},{"lat":41.0877377609903,"lng":-8.39356912877514},{"lat":41.0876237016711,"lng":-8.39345864609493},{"lat":41.0875499836461,"lng":-8.3933872399582},{"lat":41.0873549676936,"lng":-8.39326221562927},{"lat":41.0870507233851,"lng":-8.39284398162261},{"lat":41.0869813957044,"lng":-8.39273334457045},{"lat":41.08690626377,"lng":-8.39247674417235},{"lat":41.0868469014383,"lng":-8.39206684308202},{"lat":41.086855891234,"lng":-8.3919015611365},{"lat":41.0868488967636,"lng":-8.39172413211698},{"lat":41.086804684445,"lng":-8.39160387283286},{"lat":41.0867200588163,"lng":-8.39131898160678},{"lat":41.0867194749454,"lng":-8.39114956500588},{"lat":41.0867190250431,"lng":-8.3910190701518},{"lat":41.086729613013,"lng":-8.39095568155898},{"lat":41.0867000093225,"lng":-8.39084732141784},{"lat":41.0866686938053,"lng":-8.39077787364492},{"lat":41.0866157491138,"lng":-8.39065830006023},{"lat":41.0865459492694,"lng":-8.39054496376104},{"lat":41.0863928304492,"lng":-8.39032490501259},{"lat":41.0862194089642,"lng":-8.39008215602112},{"lat":41.0860815708003,"lng":-8.38995379868916},{"lat":41.0859847211067,"lng":-8.38985809379033},{"lat":41.0854127118953,"lng":-8.38669414696124},{"lat":41.0843441943863,"lng":-8.38432030202746},{"lat":41.0842857401171,"lng":-8.38419635142378},{"lat":41.0837175274101,"lng":-8.38367744776839},{"lat":41.0828233754449,"lng":-8.3828608602444},{"lat":41.0816616095213,"lng":-8.38179994081615},{"lat":41.0811947171414,"lng":-8.38109419450099},{"lat":41.0809744909807,"lng":-8.38076131027503},{"lat":41.0807224347668,"lng":-8.38029633073501},{"lat":41.0804476711742,"lng":-8.37978946989249},{"lat":41.0802274728742,"lng":-8.37935060300766},{"lat":41.0801212676235,"lng":-8.37921916454543},{"lat":41.0799992798117,"lng":-8.37906819451345},{"lat":41.0796995460673,"lng":-8.3789732958843},{"lat":41.0789810434173,"lng":-8.37890800508005},{"lat":41.0786575861282,"lng":-8.37883384493045},{"lat":41.0783930446027,"lng":-8.37874039733843},{"lat":41.0782394708124,"lng":-8.37868614874555},{"lat":41.0780181892546,"lng":-8.37874802803821},{"lat":41.0779739928297,"lng":-8.37870739367876},{"lat":41.0779156515105,"lng":-8.37865375455445},{"lat":41.0777820054391,"lng":-8.37841335613031},{"lat":41.0776903745149,"lng":-8.37814254741376},{"lat":41.0776695056669,"lng":-8.37808087313838},{"lat":41.0775700341818,"lng":-8.37778689408134},{"lat":41.0774003829344,"lng":-8.37741056951241},{"lat":41.0773345147732,"lng":-8.37726446039897},{"lat":41.0768395877616,"lng":-8.37701157242588},{"lat":41.0765277437089,"lng":-8.37685494416768},{"lat":41.0759068379722,"lng":-8.37660230298139},{"lat":41.0755901731263,"lng":-8.37645254243059},{"lat":41.0751731792956,"lng":-8.37625533557361},{"lat":41.0746349394727,"lng":-8.37593801201264},{"lat":41.0744879305297,"lng":-8.37575398689143},{"lat":41.074318391959,"lng":-8.37553489081952},{"lat":41.0741349100686,"lng":-8.37530909079922},{"lat":41.0739222412511,"lng":-8.37513267739544},{"lat":41.0737312290032,"lng":-8.37497466283854},{"lat":41.0735659057284,"lng":-8.37484841539639},{"lat":41.073516102988,"lng":-8.37481038418469},{"lat":41.0733195081275,"lng":-8.37465378253245},{"lat":41.0732374815182,"lng":-8.3745661451099},{"lat":41.0731469205124,"lng":-8.37444716648976},{"lat":41.0730626501683,"lng":-8.37433645295215},{"lat":41.0729143666746,"lng":-8.37407259058629},{"lat":41.0728771959705,"lng":-8.37400256650726},{"lat":41.0727692540094,"lng":-8.37379922080994},{"lat":41.0726076967513,"lng":-8.37354910899414},{"lat":41.072542568764,"lng":-8.3734339328935},{"lat":41.0724868677333,"lng":-8.37345812231482},{"lat":41.0724625348805,"lng":-8.37351335770567},{"lat":41.0723953397012,"lng":-8.37360782259613},{"lat":41.0723209448913,"lng":-8.37354528523086},{"lat":41.0722142209489,"lng":-8.37345557182251},{"lat":41.0721692737473,"lng":-8.37341864125228},{"lat":41.0720217721475,"lng":-8.37329744796603},{"lat":41.0717846032711,"lng":-8.37312238782654},{"lat":41.0716432892905,"lng":-8.37305357286807},{"lat":41.0714132358096,"lng":-8.37296182143389},{"lat":41.0712564150121,"lng":-8.37290211267104},{"lat":41.0711557800878,"lng":-8.37284392068304},{"lat":41.0709292614563,"lng":-8.37257074147663},{"lat":41.0706938868728,"lng":-8.37261861143888},{"lat":41.0674139816052,"lng":-8.3733669136961},{"lat":41.0656789814262,"lng":-8.37273689100575},{"lat":41.0655369349597,"lng":-8.3728447548281},{"lat":41.0653489868478,"lng":-8.37294400783991},{"lat":41.065181615756,"lng":-8.37306403051744},{"lat":41.0649795238859,"lng":-8.37324936271558},{"lat":41.0645931509929,"lng":-8.37363258071121},{"lat":41.0644520543588,"lng":-8.37384572871859},{"lat":41.0643220175342,"lng":-8.37397921570101},{"lat":41.0640865648268,"lng":-8.3741005319888},{"lat":41.0638740031518,"lng":-8.37425547849899},{"lat":41.0637443657493,"lng":-8.37437171062277},{"lat":41.0636259636186,"lng":-8.3744541957014},{"lat":41.0634929128924,"lng":-8.37448035627824},{"lat":41.0633294419018,"lng":-8.37449890682638},{"lat":41.0631202712107,"lng":-8.37460326937386},{"lat":41.0630729079728,"lng":-8.37463178592224},{"lat":41.0628087096935,"lng":-8.37471612499489},{"lat":41.0627213325343,"lng":-8.37477492806928},{"lat":41.0625407528309,"lng":-8.37487978395484},{"lat":41.062293213518,"lng":-8.37496964544752},{"lat":41.0621693366596,"lng":-8.37499999958504},{"lat":41.0618885826133,"lng":-8.37521143860953},{"lat":41.0616724570195,"lng":-8.37534804120834},{"lat":41.0615060183325,"lng":-8.37540666886719},{"lat":41.0612639405049,"lng":-8.37546965706717},{"lat":41.0611543633677,"lng":-8.37550910492513},{"lat":41.0610432317799,"lng":-8.3755157135141},{"lat":41.0609088584673,"lng":-8.37557160754021},{"lat":41.0608117186629,"lng":-8.37560491330208},{"lat":41.0606095134094,"lng":-8.37582332032114},{"lat":41.06043488439,"lng":-8.37603772452895},{"lat":41.0602674320151,"lng":-8.37624613452787},{"lat":41.0600924854688,"lng":-8.37644505870434},{"lat":41.0598954173307,"lng":-8.37666298703821},{"lat":41.0597849002696,"lng":-8.37677777756316},{"lat":41.0595796727042,"lng":-8.3769763525487},{"lat":41.0593195801359,"lng":-8.37710106554475},{"lat":41.0590939305261,"lng":-8.37721137276299},{"lat":41.0588513994385,"lng":-8.3773248463465},{"lat":41.0587255512194,"lng":-8.37734947824258},{"lat":41.0586086732244,"lng":-8.37743311983213},{"lat":41.0554664378152,"lng":-8.37968168020944},{"lat":41.0535352420467,"lng":-8.38025919581838},{"lat":41.0533797524573,"lng":-8.38030569281119},{"lat":41.0533444254647,"lng":-8.38011226691674},{"lat":41.0532193037025,"lng":-8.37942485399967},{"lat":41.0528197876898,"lng":-8.37849827686174},{"lat":41.05265714585,"lng":-8.37821647258952},{"lat":41.0525470392073,"lng":-8.37815720715758},{"lat":41.0516717530036,"lng":-8.37768608013328},{"lat":41.0518467289828,"lng":-8.37679271836449},{"lat":41.0513382179365,"lng":-8.37587505139519},{"lat":41.0503313113526,"lng":-8.37529574449038},{"lat":41.0501226445063,"lng":-8.37522717753146},{"lat":41.0496302012218,"lng":-8.37843125443571},{"lat":41.0495004792291,"lng":-8.37967148079728},{"lat":41.0494837370321,"lng":-8.37983154705646},{"lat":41.0494162157513,"lng":-8.38047703743232},{"lat":41.0493793867538,"lng":-8.38082909161282},{"lat":41.0493187056423,"lng":-8.38140915190191},{"lat":41.0493571047137,"lng":-8.38200757798725},{"lat":41.0494074919643,"lng":-8.38279283252634},{"lat":41.0495693052119,"lng":-8.38531526922985},{"lat":41.0496617382535,"lng":-8.3863932600923},{"lat":41.0497057524289,"lng":-8.38690559217815},{"lat":41.0497430476068,"lng":-8.38733978721955},{"lat":41.0499410112574,"lng":-8.38964450435886},{"lat":41.0506162330671,"lng":-8.3935948341958},{"lat":41.0507535237999,"lng":-8.39439804237148},{"lat":41.0509116716405,"lng":-8.39481625576341},{"lat":41.0509769550655,"lng":-8.39498889424211},{"lat":41.0523732241908,"lng":-8.39874495097786},{"lat":41.0564129519035,"lng":-8.40399536210009},{"lat":41.0583013348411,"lng":-8.40507378283147},{"lat":41.0628004025381,"lng":-8.40640104621036},{"lat":41.0660428172512,"lng":-8.40605760701008},{"lat":41.0690150989973,"lng":-8.40571301435013},{"lat":41.0709895188562,"lng":-8.4084578095556},{"lat":41.0721529053581,"lng":-8.4114373067371},{"lat":41.071877560474,"lng":-8.41357785953933},{"lat":41.0707032064834,"lng":-8.41511961718754}]},{"zone_id":"smf10","points":[{"lat":41.0534611593344,"lng":-8.44874065529944},{"lat":41.0530898310399,"lng":-8.44808118891258},{"lat":41.0523978949572,"lng":-8.44678263261346},{"lat":41.0522429687451,"lng":-8.44653756977101},{"lat":41.0514503693866,"lng":-8.44562585781939},{"lat":41.0510784099889,"lng":-8.44509311281099},{"lat":41.0504504936185,"lng":-8.44427987248234},{"lat":41.0494255877381,"lng":-8.44305964127588},{"lat":41.0487399860858,"lng":-8.44239449239723},{"lat":41.0482531714258,"lng":-8.44187242412746},{"lat":41.0470499595232,"lng":-8.44085054405228},{"lat":41.046658434261,"lng":-8.44041945593919},{"lat":41.0457780009643,"lng":-8.43923607111373},{"lat":41.0453849299881,"lng":-8.43887516939125},{"lat":41.0452628937201,"lng":-8.43878432737856},{"lat":41.0445279511711,"lng":-8.43834620174403},{"lat":41.0443048414693,"lng":-8.43816984998439},{"lat":41.0433342952597,"lng":-8.43717141980218},{"lat":41.0424254422126,"lng":-8.43640331492436},{"lat":41.0409833055236,"lng":-8.43506964836328},{"lat":41.0400231524314,"lng":-8.43411583348953},{"lat":41.0389125906502,"lng":-8.43310190389207},{"lat":41.038061179319,"lng":-8.43222285823967},{"lat":41.0378952942591,"lng":-8.43202516873566},{"lat":41.0377053557287,"lng":-8.4317629087784},{"lat":41.0375933915373,"lng":-8.43157021724379},{"lat":41.0372183468822,"lng":-8.43074367422276},{"lat":41.0370956651887,"lng":-8.43044747712375},{"lat":41.0367176945014,"lng":-8.42914857842214},{"lat":41.0366198354486,"lng":-8.4285373919409},{"lat":41.0365938958964,"lng":-8.42837538811168},{"lat":41.0356108627833,"lng":-8.428522826877},{"lat":41.0355023368472,"lng":-8.4285434959067},{"lat":41.0353284291326,"lng":-8.42857661691429},{"lat":41.0348737607065,"lng":-8.42862546458434},{"lat":41.0347449212869,"lng":-8.42865438141914},{"lat":41.0346706934283,"lng":-8.4287592916394},{"lat":41.0346237546147,"lng":-8.42895635595309},{"lat":41.0344768384055,"lng":-8.42901194410574},{"lat":41.0342700979907,"lng":-8.42901074313204},{"lat":41.0341097313565,"lng":-8.42900981155566},{"lat":41.0333207454196,"lng":-8.42918950699947},{"lat":41.0331631876712,"lng":-8.42921732898892},{"lat":41.0330232009609,"lng":-8.42934239186163},{"lat":41.0327753944823,"lng":-8.4294838506912},{"lat":41.0326738143621,"lng":-8.42963040184448},{"lat":41.0324627970806,"lng":-8.43009436607548},{"lat":41.0324114300597,"lng":-8.43035407733057},{"lat":41.0322139182457,"lng":-8.4307529708096},{"lat":41.0318537707341,"lng":-8.43105194638072},{"lat":41.0316030621068,"lng":-8.43113023927624},{"lat":41.0314334795817,"lng":-8.4311026520192},{"lat":41.0312953871378,"lng":-8.43102898367322},{"lat":41.0312046006271,"lng":-8.43096279302407},{"lat":41.031018636517,"lng":-8.43072852259377},{"lat":41.0308979435338,"lng":-8.4310185980027},{"lat":41.0308521829361,"lng":-8.43116192598043},{"lat":41.0308038898736,"lng":-8.43131318513741},{"lat":41.0307225572565,"lng":-8.43166716683682},{"lat":41.0306737517725,"lng":-8.43180749828177},{"lat":41.0305361550842,"lng":-8.43191401048456},{"lat":41.0303783766128,"lng":-8.43209367284208},{"lat":41.0302810683888,"lng":-8.43225447297266},{"lat":41.0300786902227,"lng":-8.43254904046764},{"lat":41.0298603356697,"lng":-8.43282272330596},{"lat":41.0296954208797,"lng":-8.43283921259351},{"lat":41.029451344675,"lng":-8.43296282944599},{"lat":41.0292615914092,"lng":-8.4330091705928},{"lat":41.0289934103593,"lng":-8.4329166250992},{"lat":41.0288170990403,"lng":-8.43284875773074},{"lat":41.0286503580792,"lng":-8.43277106121857},{"lat":41.0285459988467,"lng":-8.43270102486337},{"lat":41.0284492160529,"lng":-8.43259285740251},{"lat":41.0283634573762,"lng":-8.43243479564245},{"lat":41.0282370354322,"lng":-8.43233196203542},{"lat":41.0281609342675,"lng":-8.43226740432282},{"lat":41.0280618913381,"lng":-8.43219620427601},{"lat":41.0278696613404,"lng":-8.43208700614225},{"lat":41.0277306157891,"lng":-8.43199633533219},{"lat":41.0276109559305,"lng":-8.43188226695173},{"lat":41.0274969678549,"lng":-8.43180933110825},{"lat":41.027285471276,"lng":-8.43173489078877},{"lat":41.0270216178111,"lng":-8.43163809344457},{"lat":41.0267754412002,"lng":-8.43155368691805},{"lat":41.0265847533342,"lng":-8.43148830625987},{"lat":41.0264859464311,"lng":-8.43143078408508},{"lat":41.0263535689313,"lng":-8.43136514288366},{"lat":41.0262330209026,"lng":-8.43132274382533},{"lat":41.0261055462492,"lng":-8.43124286041067},{"lat":41.0258956084156,"lng":-8.43101777371978},{"lat":41.0256958197167,"lng":-8.43080189796703},{"lat":41.0255406639273,"lng":-8.43066647848658},{"lat":41.0253406588609,"lng":-8.43039423984232},{"lat":41.0252750951085,"lng":-8.43029597130348},{"lat":41.0251743230324,"lng":-8.43012655704398},{"lat":41.0250782412444,"lng":-8.42995502414213},{"lat":41.0249782534112,"lng":-8.42983056019947},{"lat":41.0248432588054,"lng":-8.42969856534235},{"lat":41.0247673376887,"lng":-8.42963378175975},{"lat":41.0246381049036,"lng":-8.42960716050625},{"lat":41.0244628815534,"lng":-8.42960351148399},{"lat":41.0242198477537,"lng":-8.42957066930452},{"lat":41.024110362603,"lng":-8.42953260489496},{"lat":41.0239921944761,"lng":-8.42947416872435},{"lat":41.0238588703331,"lng":-8.42932304132181},{"lat":41.0237956307081,"lng":-8.42919529954216},{"lat":41.0237258680859,"lng":-8.42904719236017},{"lat":41.0235666219164,"lng":-8.42878585402658},{"lat":41.0234790671832,"lng":-8.42865859435741},{"lat":41.0233461529999,"lng":-8.42862708307688},{"lat":41.0231841502891,"lng":-8.42866761167676},{"lat":41.0230801717956,"lng":-8.42879781408947},{"lat":41.0229910424088,"lng":-8.42900001073339},{"lat":41.0229363233707,"lng":-8.42923387350071},{"lat":41.0228757576164,"lng":-8.42942799086149},{"lat":41.0227687391695,"lng":-8.42951584455005},{"lat":41.0226752647539,"lng":-8.42969317865656},{"lat":41.0225534375827,"lng":-8.42986656840811},{"lat":41.0224565048818,"lng":-8.42998621755125},{"lat":41.0224077252144,"lng":-8.42983761461929},{"lat":41.0220679026113,"lng":-8.4291752583344},{"lat":41.0219039442905,"lng":-8.42885683654349},{"lat":41.0216423694302,"lng":-8.42840053035631},{"lat":41.0215170658203,"lng":-8.42814114386862},{"lat":41.0214258984455,"lng":-8.42791221680457},{"lat":41.0212154431521,"lng":-8.42743165897977},{"lat":41.0209324655549,"lng":-8.42706871854046},{"lat":41.0207955527827,"lng":-8.42676944934042},{"lat":41.0207248391186,"lng":-8.42667461383809},{"lat":41.0207058927822,"lng":-8.42647930685804},{"lat":41.020694698616,"lng":-8.42634930798336},{"lat":41.0206745760527,"lng":-8.42604901915138},{"lat":41.0206597176334,"lng":-8.42590937145102},{"lat":41.0205701083509,"lng":-8.42570269509231},{"lat":41.020497239354,"lng":-8.42553568230347},{"lat":41.0204432950937,"lng":-8.42525783363024},{"lat":41.0204747130449,"lng":-8.42501293771657},{"lat":41.0205065233117,"lng":-8.42482737089614},{"lat":41.0204824836477,"lng":-8.42468471219842},{"lat":41.0204515661339,"lng":-8.42455521801897},{"lat":41.0203947461199,"lng":-8.4243094587199},{"lat":41.0203289335053,"lng":-8.42387925418713},{"lat":41.0202706690637,"lng":-8.42352719368699},{"lat":41.0201770413568,"lng":-8.42337626228355},{"lat":41.0200688752372,"lng":-8.42324369699315},{"lat":41.0199736137935,"lng":-8.42313532619822},{"lat":41.0198865729406,"lng":-8.42301533798219},{"lat":41.0197423275648,"lng":-8.42282429095729},{"lat":41.0052637131587,"lng":-8.43547959534181},{"lat":41.0026270302074,"lng":-8.43989064196004},{"lat":40.9966973834513,"lng":-8.45014198107079},{"lat":41.0020288550275,"lng":-8.45205443003603},{"lat":41.005170191828,"lng":-8.45332756965765},{"lat":41.007359237654,"lng":-8.45434459548284},{"lat":41.0105579361965,"lng":-8.45636850445364},{"lat":41.0127549683661,"lng":-8.4581303034357},{"lat":41.0244403133584,"lng":-8.46369355387916},{"lat":41.0302528988207,"lng":-8.46661576271443},{"lat":41.0306389798249,"lng":-8.46718278615259},{"lat":41.0310726372155,"lng":-8.46773019882931},{"lat":41.0313539275687,"lng":-8.4680753026682},{"lat":41.031620390629,"lng":-8.46850186656111},{"lat":41.0317494141596,"lng":-8.4685368907987},{"lat":41.0317881422012,"lng":-8.46865588402533},{"lat":41.031877006228,"lng":-8.46887548658531},{"lat":41.0319534073803,"lng":-8.4690450922202},{"lat":41.0320401201436,"lng":-8.46919869734584},{"lat":41.0321050946505,"lng":-8.46930295959599},{"lat":41.0322180145469,"lng":-8.46945574272317},{"lat":41.0322945724383,"lng":-8.46960251919409},{"lat":41.0323060968426,"lng":-8.4697288581172},{"lat":41.0323119102086,"lng":-8.46998276884844},{"lat":41.0323270421344,"lng":-8.47013469937499},{"lat":41.0323307985656,"lng":-8.47026564066029},{"lat":41.0324016618379,"lng":-8.47087792923385},{"lat":41.0320995615404,"lng":-8.47109137298321},{"lat":41.0319369309385,"lng":-8.47109600535075},{"lat":41.0317945150713,"lng":-8.47120550123876},{"lat":41.0317103418952,"lng":-8.47135169042805},{"lat":41.031601905757,"lng":-8.47154234020327},{"lat":41.0314384792789,"lng":-8.47163246430574},{"lat":41.0311634402035,"lng":-8.47177041214335},{"lat":41.0312583022199,"lng":-8.47196746255259},{"lat":41.0314331963273,"lng":-8.47189784650661},{"lat":41.031480138586,"lng":-8.47204007121395},{"lat":41.0316457673604,"lng":-8.47269695735778},{"lat":41.03188086874,"lng":-8.47272801977845},{"lat":41.0318506140905,"lng":-8.47288054661279},{"lat":41.0318408203981,"lng":-8.47311546773179},{"lat":41.0318662362409,"lng":-8.47335056885841},{"lat":41.0319110163088,"lng":-8.47367483726219},{"lat":41.0319449715496,"lng":-8.47400594843948},{"lat":41.0319571912351,"lng":-8.4742928175854},{"lat":41.0319903975901,"lng":-8.47462559276479},{"lat":41.0319885688951,"lng":-8.47481572965149},{"lat":41.0320098387252,"lng":-8.4750512837768},{"lat":41.0319675035178,"lng":-8.47520612438811},{"lat":41.0318291381023,"lng":-8.47543704644539},{"lat":41.0316814197271,"lng":-8.47560203531168},{"lat":41.0315865952785,"lng":-8.47572628227224},{"lat":41.0315593098831,"lng":-8.47590760053023},{"lat":41.031692063746,"lng":-8.4759579947517},{"lat":41.0318147592877,"lng":-8.47602903621746},{"lat":41.03191014871,"lng":-8.47607971500853},{"lat":41.0329098950887,"lng":-8.47614462377891},{"lat":41.0331974909007,"lng":-8.47599724602188},{"lat":41.0334723216238,"lng":-8.47587167661389},{"lat":41.0337674530593,"lng":-8.47579211685845},{"lat":41.0340524855123,"lng":-8.47577838539771},{"lat":41.0341083207951,"lng":-8.47724812132067},{"lat":41.0341277602425,"lng":-8.47799830691951},{"lat":41.034143052659,"lng":-8.47857445621262},{"lat":41.035123047027,"lng":-8.47972300526367},{"lat":41.0350554848672,"lng":-8.47991160613618},{"lat":41.0349987629194,"lng":-8.48002844006067},{"lat":41.0349011888807,"lng":-8.48022603936879},{"lat":41.0348524243575,"lng":-8.48033197523329},{"lat":41.0347933328826,"lng":-8.48045807163368},{"lat":41.0347755997132,"lng":-8.48060460738106},{"lat":41.0348562793843,"lng":-8.4806933858571},{"lat":41.0348793890258,"lng":-8.48085297506796},{"lat":41.0348457592511,"lng":-8.48104437371597},{"lat":41.0348009456453,"lng":-8.48121276790703},{"lat":41.0348379449837,"lng":-8.48145400381481},{"lat":41.03488965694,"lng":-8.48168165145827},{"lat":41.0349360423789,"lng":-8.48179617357669},{"lat":41.0350029083584,"lng":-8.48189678103535},{"lat":41.0352408209886,"lng":-8.48198021126961},{"lat":41.0355006215861,"lng":-8.48203249128649},{"lat":41.0356217483945,"lng":-8.48202706713892},{"lat":41.0357633245578,"lng":-8.48201807036498},{"lat":41.0359035773997,"lng":-8.48220670663203},{"lat":41.0360570719342,"lng":-8.48242419373277},{"lat":41.0364080317312,"lng":-8.48278971690259},{"lat":41.0363348297054,"lng":-8.48287709110814},{"lat":41.0362721937807,"lng":-8.4830130373639},{"lat":41.0363689300801,"lng":-8.48309180466182},{"lat":41.0364514130633,"lng":-8.48315039536889},{"lat":41.0365483149985,"lng":-8.48326352877674},{"lat":41.0363490936126,"lng":-8.48389547971435},{"lat":41.0363472545109,"lng":-8.48402604800395},{"lat":41.036562765691,"lng":-8.48401518378873},{"lat":41.0367314412713,"lng":-8.48400704741276},{"lat":41.0368598289127,"lng":-8.48424807117672},{"lat":41.0370536250087,"lng":-8.48471028187534},{"lat":41.0371557279201,"lng":-8.4849772147013},{"lat":41.0373055291378,"lng":-8.48551632467875},{"lat":41.0374300418654,"lng":-8.48607823340894},{"lat":41.0376027169155,"lng":-8.48661407433119},{"lat":41.0376304433488,"lng":-8.48673219734129},{"lat":41.0377045771534,"lng":-8.48692869946702},{"lat":41.0377868572496,"lng":-8.48713832493748},{"lat":41.0378294132667,"lng":-8.48729612849624},{"lat":41.0380071869635,"lng":-8.48789907535975},{"lat":41.038094752818,"lng":-8.48817532912679},{"lat":41.0381813952447,"lng":-8.48848480978084},{"lat":41.0382295686098,"lng":-8.48860513293717},{"lat":41.0383330867904,"lng":-8.48887767243345},{"lat":41.0384372902112,"lng":-8.48916020618729},{"lat":41.0385468980552,"lng":-8.48916694894173},{"lat":41.0387444935254,"lng":-8.48916694894174},{"lat":41.0388666027891,"lng":-8.4891728359131},{"lat":41.0391441230006,"lng":-8.4893052927689},{"lat":41.0393661383272,"lng":-8.48934061459711},{"lat":41.0397924056552,"lng":-8.48944658008174},{"lat":41.0403607577981,"lng":-8.4896231892228},{"lat":41.0414441654758,"lng":-8.49003527721859},{"lat":41.042039144073,"lng":-8.49054155675628},{"lat":41.0425097949739,"lng":-8.49058865252724},{"lat":41.043028309862,"lng":-8.49060073666764},{"lat":41.0436462151495,"lng":-8.49105533105058},{"lat":41.0442430682295,"lng":-8.4914747337642},{"lat":41.0445038732073,"lng":-8.49163624128396},{"lat":41.0446299166415,"lng":-8.49158424373543},{"lat":41.0448639413557,"lng":-8.49146920193676},{"lat":41.0450524959666,"lng":-8.4913896595901},{"lat":41.0453243160216,"lng":-8.49127499105685},{"lat":41.0468540927363,"lng":-8.48999986316712},{"lat":41.047300474429,"lng":-8.48964968059398},{"lat":41.0476911724783,"lng":-8.48932001019734},{"lat":41.0479930739275,"lng":-8.48910807922807},{"lat":41.0482416976103,"lng":-8.48896679191523},{"lat":41.0484903203537,"lng":-8.48875486094597},{"lat":41.0488099767862,"lng":-8.4885429299767},{"lat":41.0491473902247,"lng":-8.48826035535101},{"lat":41.0495380773076,"lng":-8.48800132861079},{"lat":41.0502839280241,"lng":-8.48745972724489},{"lat":41.0510830444094,"lng":-8.48703586530635},{"lat":41.0519397494387,"lng":-8.48687572680663},{"lat":41.0525964235068,"lng":-8.48626953089616},{"lat":41.052703223117,"lng":-8.48620765997876},{"lat":41.0527142780279,"lng":-8.48580566642206},{"lat":41.0527714624238,"lng":-8.4853342206904},{"lat":41.0529196108812,"lng":-8.48400634478879},{"lat":41.052887571687,"lng":-8.48388246891267},{"lat":41.0528492551746,"lng":-8.48366244020709},{"lat":41.0528579101813,"lng":-8.48348275578453},{"lat":41.0528527370839,"lng":-8.48331833792075},{"lat":41.0528300371587,"lng":-8.48314478809025},{"lat":41.0527806771721,"lng":-8.4829433851333},{"lat":41.0526569680608,"lng":-8.48276311061321},{"lat":41.0525851752813,"lng":-8.48268434204556},{"lat":41.0525089751886,"lng":-8.48260245285497},{"lat":41.0523875371706,"lng":-8.48241600869588},{"lat":41.0522407742167,"lng":-8.48220469093552},{"lat":41.0521773600113,"lng":-8.48206458604447},{"lat":41.0521042027803,"lng":-8.4819136563679},{"lat":41.0520008871496,"lng":-8.48172404423139},{"lat":41.0519039418749,"lng":-8.48163551708761},{"lat":41.0517968868964,"lng":-8.4815444294029},{"lat":41.0518682028369,"lng":-8.48139576572268},{"lat":41.051912951002,"lng":-8.48127860221334},{"lat":41.0519778818856,"lng":-8.48103855110126},{"lat":41.0520114979952,"lng":-8.48091014607689},{"lat":41.0520362172008,"lng":-8.48077586555245},{"lat":41.0520024823233,"lng":-8.48058727302822},{"lat":41.0520265061363,"lng":-8.48032595174337},{"lat":41.05207806359,"lng":-8.48010046102012},{"lat":41.0521912976013,"lng":-8.47960837375455},{"lat":41.0522568524379,"lng":-8.47919001834702},{"lat":41.0523949043397,"lng":-8.47846968183936},{"lat":41.0524372449469,"lng":-8.47822427195827},{"lat":41.0524851855508,"lng":-8.47812246300391},{"lat":41.0525270519508,"lng":-8.47800278792631},{"lat":41.0525752567341,"lng":-8.47778309762025},{"lat":41.0527209586484,"lng":-8.47730057137674},{"lat":41.0528498946438,"lng":-8.47685221485105},{"lat":41.0522419615629,"lng":-8.47719719700871},{"lat":41.0519153242183,"lng":-8.47736355405434},{"lat":41.0517006788525,"lng":-8.47717806180499},{"lat":41.051694369789,"lng":-8.47699842185866},{"lat":41.0516081304643,"lng":-8.4765715375426},{"lat":41.0515617546734,"lng":-8.47630093290842},{"lat":41.0515085815983,"lng":-8.47598615268974},{"lat":41.051463238577,"lng":-8.47573327519409},{"lat":41.0512789224482,"lng":-8.47464561346572},{"lat":41.0511319169957,"lng":-8.47384683815119},{"lat":41.050930847441,"lng":-8.47324118985977},{"lat":41.0508236935564,"lng":-8.472907225898},{"lat":41.0507172342646,"lng":-8.47258290369758},{"lat":41.0506197090261,"lng":-8.47231036683808},{"lat":41.0504683962404,"lng":-8.47189791475605},{"lat":41.0503760442283,"lng":-8.47161221407626},{"lat":41.0503207565537,"lng":-8.47146098514923},{"lat":41.0502617061156,"lng":-8.47127155896456},{"lat":41.050162158958,"lng":-8.47098212639524},{"lat":41.0499912549964,"lng":-8.47049392809921},{"lat":41.0497042207258,"lng":-8.46965759087296},{"lat":41.0493978835512,"lng":-8.46898445868691},{"lat":41.0490251281547,"lng":-8.46802125585025},{"lat":41.0489411905635,"lng":-8.46774939983929},{"lat":41.0488387267826,"lng":-8.46746996083455},{"lat":41.0486760283282,"lng":-8.46709636676592},{"lat":41.0483216650156,"lng":-8.46618085384371},{"lat":41.0482833006937,"lng":-8.46605886404588},{"lat":41.047892941661,"lng":-8.46525831247627},{"lat":41.0471276093223,"lng":-8.46371833732658},{"lat":41.0465625732974,"lng":-8.462595573238},{"lat":41.0463914365055,"lng":-8.46226881823404},{"lat":41.0459449766839,"lng":-8.46142854788306},{"lat":41.0456358177647,"lng":-8.46084409188621},{"lat":41.0451510394458,"lng":-8.4599552460809},{"lat":41.0449392011772,"lng":-8.46028911231618},{"lat":41.0444996091855,"lng":-8.45968749240802},{"lat":41.044387581016,"lng":-8.45953362544925},{"lat":41.0443021919453,"lng":-8.45941783346439},{"lat":41.0442311489742,"lng":-8.45932138613411},{"lat":41.0439738569963,"lng":-8.45895521899378},{"lat":41.0436699578701,"lng":-8.45857478722854},{"lat":41.0434094791349,"lng":-8.45821895770472},{"lat":41.0432606586015,"lng":-8.45802055366174},{"lat":41.0431762626781,"lng":-8.45790167822128},{"lat":41.042926012046,"lng":-8.45755827331032},{"lat":41.0426786409144,"lng":-8.45721583540562},{"lat":41.0423782072674,"lng":-8.45678537275762},{"lat":41.0426302086344,"lng":-8.45660607585673},{"lat":41.0428736631477,"lng":-8.45642305121516},{"lat":41.0432089909621,"lng":-8.45616768713718},{"lat":41.0433627726799,"lng":-8.45601240600909},{"lat":41.0434910217585,"lng":-8.45587578508955},{"lat":41.0438076630479,"lng":-8.45566802108323},{"lat":41.0439354424879,"lng":-8.45563344616056},{"lat":41.044081376491,"lng":-8.45561156093197},{"lat":41.0442947651481,"lng":-8.45555337982808},{"lat":41.04439225442,"lng":-8.45553126142868},{"lat":41.044686103745,"lng":-8.45545515979788},{"lat":41.0448287382087,"lng":-8.45545395876969},{"lat":41.0449662288486,"lng":-8.45542489901462},{"lat":41.0449855008603,"lng":-8.45529558961579},{"lat":41.0450115170571,"lng":-8.45516952907878},{"lat":41.0450661468852,"lng":-8.45498639794508},{"lat":41.0451746771981,"lng":-8.45481697010402},{"lat":41.0452766799625,"lng":-8.45463169243524},{"lat":41.0453249650647,"lng":-8.45443021286851},{"lat":41.0453724409305,"lng":-8.45419780646648},{"lat":41.0454197346617,"lng":-8.45399798422388},{"lat":41.045461177974,"lng":-8.45383013232833},{"lat":41.0454993152071,"lng":-8.45365072125486},{"lat":41.045564054753,"lng":-8.45342708462207},{"lat":41.0456018461623,"lng":-8.45330678121243},{"lat":41.0456575664747,"lng":-8.45315207960019},{"lat":41.0457577171125,"lng":-8.45288662219653},{"lat":41.045831879018,"lng":-8.45270596275358},{"lat":41.0459651845295,"lng":-8.45256257646751},{"lat":41.0461482170826,"lng":-8.45247521954591},{"lat":41.0462714746417,"lng":-8.45244703454351},{"lat":41.0463942886329,"lng":-8.45241741764017},{"lat":41.0464837323452,"lng":-8.4523749187325},{"lat":41.0465820178699,"lng":-8.45235767887973},{"lat":41.0466880788407,"lng":-8.45232905117319},{"lat":41.0467700832917,"lng":-8.45227342783315},{"lat":41.0468626582065,"lng":-8.45217290191165},{"lat":41.0469601921854,"lng":-8.45203873591486},{"lat":41.0470585055743,"lng":-8.4518831663341},{"lat":41.047123081101,"lng":-8.45178047640464},{"lat":41.0472121239376,"lng":-8.45165423739967},{"lat":41.047312072161,"lng":-8.4515256718027},{"lat":41.0474004524022,"lng":-8.45144427097327},{"lat":41.047495817889,"lng":-8.45140964759431},{"lat":41.0477166353374,"lng":-8.45143117565394},{"lat":41.0479656334662,"lng":-8.45142072643775},{"lat":41.0480752499079,"lng":-8.45144171400583},{"lat":41.0481763061534,"lng":-8.4513996278757},{"lat":41.0482835260987,"lng":-8.45137576244671},{"lat":41.0484211485052,"lng":-8.45139640922904},{"lat":41.0486084576196,"lng":-8.45151888044247},{"lat":41.0486921361079,"lng":-8.45160646834908},{"lat":41.0487890707018,"lng":-8.45171827196348},{"lat":41.0489660339764,"lng":-8.45187173332866},{"lat":41.0490808255845,"lng":-8.45190951810611},{"lat":41.0492183754473,"lng":-8.45185868971766},{"lat":41.0493080620133,"lng":-8.45179275374484},{"lat":41.0494096366308,"lng":-8.45162898443712},{"lat":41.0495005187452,"lng":-8.45161978816804},{"lat":41.0495856686221,"lng":-8.45166302504218},{"lat":41.0497281711984,"lng":-8.45164432898653},{"lat":41.0498186520083,"lng":-8.45161645272035},{"lat":41.0499886251213,"lng":-8.45156209217263},{"lat":41.0501067890318,"lng":-8.45148939343426},{"lat":41.0501983980802,"lng":-8.45134674969413},{"lat":41.0502714059841,"lng":-8.45119116261148},{"lat":41.0503801226877,"lng":-8.45108333051544},{"lat":41.0504858994719,"lng":-8.45102603753377},{"lat":41.0506034500259,"lng":-8.45097807890792},{"lat":41.0508615258109,"lng":-8.45090938578839},{"lat":41.0509878481423,"lng":-8.45084850357958},{"lat":41.0510905361212,"lng":-8.45077132815149},{"lat":41.0511884039424,"lng":-8.45067747894278},{"lat":41.0512775058092,"lng":-8.45062806685182},{"lat":41.0513667567649,"lng":-8.45058793927359},{"lat":41.0514524745695,"lng":-8.45052483870901},{"lat":41.0516061929629,"lng":-8.45045480885794},{"lat":41.0517589734768,"lng":-8.4503672838894},{"lat":41.0518816587927,"lng":-8.45022026774714},{"lat":41.0519427740297,"lng":-8.45009994401381},{"lat":41.0520111139053,"lng":-8.44997062287587},{"lat":41.0534611593344,"lng":-8.44874065529944}]},{"zone_id":"smf11","points":[{"lat":40.9909001312093,"lng":-8.44884157651032},{"lat":40.9928882309973,"lng":-8.44911752409347},{"lat":40.9966973834513,"lng":-8.45014198107079},{"lat":41.0026270302074,"lng":-8.43989064196004},{"lat":41.0052637131587,"lng":-8.43547959534181},{"lat":41.0197423275648,"lng":-8.42282429095729},{"lat":41.0196305450248,"lng":-8.42263685048409},{"lat":41.0195455785452,"lng":-8.42251425583217},{"lat":41.0194666099782,"lng":-8.42244328648052},{"lat":41.0193783199042,"lng":-8.42239129788137},{"lat":41.019252076514,"lng":-8.42228362142887},{"lat":41.0191443605044,"lng":-8.42218542349699},{"lat":41.019073597722,"lng":-8.42210902306895},{"lat":41.0190215772483,"lng":-8.42195673260581},{"lat":41.0190322254087,"lng":-8.42180460544403},{"lat":41.0190768570975,"lng":-8.42167461650783},{"lat":41.0191245564784,"lng":-8.42150505262062},{"lat":41.0191577824446,"lng":-8.42136253006507},{"lat":41.0191735929946,"lng":-8.42123420191584},{"lat":41.0191882129051,"lng":-8.42111275704102},{"lat":41.0192101711706,"lng":-8.42097446991343},{"lat":41.0191765627082,"lng":-8.42084128858408},{"lat":41.019108175166,"lng":-8.42067799095647},{"lat":41.0190313593516,"lng":-8.42053914606132},{"lat":41.0189474323878,"lng":-8.42039634959261},{"lat":41.0188454605097,"lng":-8.42027035512295},{"lat":41.0187521751054,"lng":-8.42016140118649},{"lat":41.0186710350731,"lng":-8.42005892387556},{"lat":41.0186188721598,"lng":-8.41996061239585},{"lat":41.0185570373081,"lng":-8.41984240220472},{"lat":41.0185696684141,"lng":-8.41968873642013},{"lat":41.0185850553503,"lng":-8.41951416152385},{"lat":41.018565285465,"lng":-8.41935333848036},{"lat":41.0184270303916,"lng":-8.41900700645817},{"lat":41.0183372154058,"lng":-8.41888047797732},{"lat":41.0175768781271,"lng":-8.41819225833637},{"lat":41.0174811197305,"lng":-8.41802908961656},{"lat":41.017425032773,"lng":-8.41791543169366},{"lat":41.0173309540096,"lng":-8.41776475295592},{"lat":41.0170644312649,"lng":-8.41736022031137},{"lat":41.0166963693461,"lng":-8.41709137018111},{"lat":41.0163122472925,"lng":-8.41672437365352},{"lat":41.0162347564103,"lng":-8.41656295645364},{"lat":41.0160401103325,"lng":-8.41633219738802},{"lat":41.0159656891001,"lng":-8.41623985464839},{"lat":41.0158454640575,"lng":-8.41610096057776},{"lat":41.0157486356252,"lng":-8.41603979149424},{"lat":41.0156082563748,"lng":-8.41596690333998},{"lat":41.0155201383064,"lng":-8.41591552398099},{"lat":41.015380108483,"lng":-8.415811376162},{"lat":41.0152908006297,"lng":-8.41573217096053},{"lat":41.0152605898413,"lng":-8.41560412313364},{"lat":41.0151289632879,"lng":-8.41534059092384},{"lat":41.0150690314722,"lng":-8.41521586203463},{"lat":41.014859036702,"lng":-8.4143792281602},{"lat":41.0148462344171,"lng":-8.41423841407573},{"lat":41.0148341508627,"lng":-8.41409820107999},{"lat":41.0148030136944,"lng":-8.41394530564444},{"lat":41.0147718361457,"lng":-8.41380632064401},{"lat":41.0147339483992,"lng":-8.41365208676472},{"lat":41.0147019048567,"lng":-8.41353580256619},{"lat":41.0146706993493,"lng":-8.41340811169006},{"lat":41.0146471826662,"lng":-8.41326784959138},{"lat":41.0146083487072,"lng":-8.41309471357077},{"lat":41.0145515394305,"lng":-8.41290640126644},{"lat":41.0145270775036,"lng":-8.41274723094645},{"lat":41.0144977597979,"lng":-8.41262085527376},{"lat":41.0144541879622,"lng":-8.41246909760185},{"lat":41.0144010367157,"lng":-8.41229471210088},{"lat":41.0143494584957,"lng":-8.41213959238894},{"lat":41.0143063202809,"lng":-8.4119587063946},{"lat":41.014259012722,"lng":-8.41175022844152},{"lat":41.0142117634864,"lng":-8.41155435367878},{"lat":41.0141739163877,"lng":-8.41138240563815},{"lat":41.014127656768,"lng":-8.41118701345482},{"lat":41.0140630491289,"lng":-8.41098262239576},{"lat":41.0140359933435,"lng":-8.41085209347648},{"lat":41.0139739912122,"lng":-8.41064950074767},{"lat":41.0138831629087,"lng":-8.41034109746423},{"lat":41.0138386850751,"lng":-8.4101907666679},{"lat":41.0137929640299,"lng":-8.41003459738046},{"lat":41.0137224121693,"lng":-8.40975482031212},{"lat":41.0136395476159,"lng":-8.40942885480354},{"lat":41.0135658943671,"lng":-8.40912861863422},{"lat":41.0135028021975,"lng":-8.408892969716},{"lat":41.0134050677999,"lng":-8.40857110999462},{"lat":41.013330746578,"lng":-8.40828608496188},{"lat":41.0133262959048,"lng":-8.40807767320748},{"lat":41.013306591815,"lng":-8.40747012013203},{"lat":41.0132967592054,"lng":-8.40710274120411},{"lat":41.0132827644798,"lng":-8.4065163705509},{"lat":41.0132781777687,"lng":-8.40621202114497},{"lat":41.0132808133894,"lng":-8.40597439682649},{"lat":41.0132683423152,"lng":-8.40557800542325},{"lat":41.0132955302979,"lng":-8.40527711555353},{"lat":41.0132939571379,"lng":-8.40510509323032},{"lat":41.0132820626005,"lng":-8.40482823047156},{"lat":41.0132597469877,"lng":-8.40445860684646},{"lat":41.0132505095476,"lng":-8.40425219570906},{"lat":41.0132240544097,"lng":-8.40386513193404},{"lat":41.013204658787,"lng":-8.40346027723429},{"lat":41.0131921591399,"lng":-8.40333908518304},{"lat":41.0131756239632,"lng":-8.40321193270355},{"lat":41.0130636666982,"lng":-8.40303410822844},{"lat":41.0127826889246,"lng":-8.40255340184407},{"lat":41.012584528524,"lng":-8.40219595581768},{"lat":41.0124816058153,"lng":-8.40196716891252},{"lat":41.012160400268,"lng":-8.40112836756767},{"lat":41.012138444333,"lng":-8.40092963350657},{"lat":41.0121777056511,"lng":-8.40069762844657},{"lat":41.0122842771327,"lng":-8.40036996405314},{"lat":41.0122129201598,"lng":-8.39934388605513},{"lat":41.0121894432386,"lng":-8.39855955891938},{"lat":41.0122161679508,"lng":-8.39833308926347},{"lat":41.0123258524348,"lng":-8.39743564959836},{"lat":41.0124478751007,"lng":-8.3967659108546},{"lat":41.0119196535077,"lng":-8.39642273812568},{"lat":41.0113616311754,"lng":-8.39599658948946},{"lat":41.0108929958826,"lng":-8.39526933867819},{"lat":41.0100416082392,"lng":-8.39468524566778},{"lat":41.0103728198116,"lng":-8.39435823764216},{"lat":41.0097210281905,"lng":-8.39410280345381},{"lat":41.0093828304197,"lng":-8.39397949569575},{"lat":41.0091646497174,"lng":-8.39390147613424},{"lat":41.0085544654608,"lng":-8.39368069018672},{"lat":41.0082106814068,"lng":-8.39355986227428},{"lat":41.0080817642452,"lng":-8.39335013748885},{"lat":41.0079189392328,"lng":-8.39308239628412},{"lat":41.0078103020489,"lng":-8.39294229087647},{"lat":41.0077045452454,"lng":-8.39280339425937},{"lat":41.0076294475058,"lng":-8.39272392732783},{"lat":41.0075330586824,"lng":-8.39266269010391},{"lat":41.0073855576778,"lng":-8.39254632416819},{"lat":41.0073097199498,"lng":-8.39247589621089},{"lat":41.0071956285291,"lng":-8.3923595443494},{"lat":41.0071214613749,"lng":-8.39226653944118},{"lat":41.0070148305408,"lng":-8.39211551137519},{"lat":41.0069277181331,"lng":-8.39201329631008},{"lat":41.0068384454319,"lng":-8.39191107900596},{"lat":41.0067659561319,"lng":-8.39183222224239},{"lat":41.0066457633851,"lng":-8.39170384648566},{"lat":41.0065317539402,"lng":-8.39159154252376},{"lat":41.006456907355,"lng":-8.39152016356098},{"lat":41.0063528453229,"lng":-8.39142585284011},{"lat":41.0062359317453,"lng":-8.39132233351786},{"lat":41.0061236111346,"lng":-8.39121729077673},{"lat":41.0054188102117,"lng":-8.39010693994556},{"lat":41.0050860898979,"lng":-8.3901076561546},{"lat":41.0050920502167,"lng":-8.3898999027349},{"lat":41.0050957181281,"lng":-8.38975015260744},{"lat":41.0051231540197,"lng":-8.38944109139771},{"lat":41.0049495951414,"lng":-8.38929825507118},{"lat":41.0047669699725,"lng":-8.38914337431199},{"lat":41.0045300821739,"lng":-8.38885242856032},{"lat":41.0021009567214,"lng":-8.39040951964598},{"lat":41.0019499789664,"lng":-8.39023599046359},{"lat":41.0018616633018,"lng":-8.39010846160362},{"lat":41.001697999438,"lng":-8.38977157192384},{"lat":41.0015453703978,"lng":-8.389450181893},{"lat":41.0013801148086,"lng":-8.38921930883302},{"lat":41.0012770352642,"lng":-8.38928048038447},{"lat":41.0010071174067,"lng":-8.3893541796058},{"lat":41.0007956109148,"lng":-8.38943096850702},{"lat":41.0007047411119,"lng":-8.38950897551008},{"lat":41.0006104102167,"lng":-8.3896216040789},{"lat":41.0005627207212,"lng":-8.38972684550483},{"lat":41.0002493099916,"lng":-8.38958785376098},{"lat":41.000010752876,"lng":-8.38948150945315},{"lat":40.9998612007647,"lng":-8.38955627867776},{"lat":40.9997662652277,"lng":-8.38960879914621},{"lat":40.9996642131504,"lng":-8.38965310642389},{"lat":40.9995142738359,"lng":-8.38970860084832},{"lat":40.9993792023542,"lng":-8.38975121664779},{"lat":40.9990382509909,"lng":-8.38992923601597},{"lat":40.9988760756233,"lng":-8.39001179850394},{"lat":40.9987664599194,"lng":-8.39006580206978},{"lat":40.9986449501825,"lng":-8.39012213752102},{"lat":40.9985073945037,"lng":-8.39018720796068},{"lat":40.9983770477391,"lng":-8.39029069099358},{"lat":40.9982832809077,"lng":-8.39034309374272},{"lat":40.9981737229765,"lng":-8.39041148245901},{"lat":40.9981635799424,"lng":-8.39059708882186},{"lat":40.9981444261448,"lng":-8.39082639312649},{"lat":40.998043355422,"lng":-8.3909159131965},{"lat":40.9978941132602,"lng":-8.39113977680584},{"lat":40.9978319507899,"lng":-8.39122737638652},{"lat":40.9977810369495,"lng":-8.39133151151295},{"lat":40.9977075028986,"lng":-8.39140021197625},{"lat":40.9976129716977,"lng":-8.39141250461396},{"lat":40.9975048585817,"lng":-8.39141397934384},{"lat":40.9974042775772,"lng":-8.39148247889364},{"lat":40.9973672705885,"lng":-8.3916132208158},{"lat":40.9972734757743,"lng":-8.39168175127289},{"lat":40.9971250874518,"lng":-8.39175536773149},{"lat":40.9970379144083,"lng":-8.39180380333225},{"lat":40.9970156336493,"lng":-8.39193784178253},{"lat":40.9969748081533,"lng":-8.39207269771276},{"lat":40.9968971282218,"lng":-8.39214031313833},{"lat":40.9967965589782,"lng":-8.39213355869641},{"lat":40.9966925606777,"lng":-8.39217093646348},{"lat":40.9966003393357,"lng":-8.3921779453086},{"lat":40.9965269502646,"lng":-8.392259068995},{"lat":40.99642952981,"lng":-8.39225558905287},{"lat":40.9963361839438,"lng":-8.39234428236242},{"lat":40.9962269249929,"lng":-8.39245528739196},{"lat":40.9961487017662,"lng":-8.39256253819729},{"lat":40.9960085774737,"lng":-8.39280752012128},{"lat":40.9959489916069,"lng":-8.392902964773},{"lat":40.9959028128455,"lng":-8.39302839795573},{"lat":40.9958171078576,"lng":-8.39317923680381},{"lat":40.995752103678,"lng":-8.39328240611922},{"lat":40.9956496205244,"lng":-8.39345702835982},{"lat":40.9955753841598,"lng":-8.39355407179299},{"lat":40.9954761013462,"lng":-8.39365469593076},{"lat":40.9952961713451,"lng":-8.39374528257893},{"lat":40.9950640152866,"lng":-8.39418561274612},{"lat":40.9950024971503,"lng":-8.39443054941273},{"lat":40.9949629380218,"lng":-8.39456040243352},{"lat":40.9949333107577,"lng":-8.39470411893657},{"lat":40.994840673569,"lng":-8.39502709820989},{"lat":40.9948230074847,"lng":-8.39519187007714},{"lat":40.9947811842636,"lng":-8.39532433351506},{"lat":40.9946293326618,"lng":-8.395578773252},{"lat":40.9944932386098,"lng":-8.39576813520445},{"lat":40.9943396179806,"lng":-8.39583122608895},{"lat":40.9942607740093,"lng":-8.39600992749077},{"lat":40.9941149418603,"lng":-8.39618923711747},{"lat":40.9939756185506,"lng":-8.39620871859553},{"lat":40.9938563417553,"lng":-8.39622539691925},{"lat":40.9937478197384,"lng":-8.39623328340572},{"lat":40.9935996165844,"lng":-8.39626620597191},{"lat":40.9934326617766,"lng":-8.39638842121521},{"lat":40.9932810528702,"lng":-8.39646809791772},{"lat":40.9931597657033,"lng":-8.396531839018},{"lat":40.9930526226325,"lng":-8.3966209051954},{"lat":40.9929691650683,"lng":-8.39675199692901},{"lat":40.9928762599402,"lng":-8.39690172409039},{"lat":40.9927662266746,"lng":-8.39694157214758},{"lat":40.9926449858175,"lng":-8.39699694024807},{"lat":40.9925085454911,"lng":-8.39688848584679},{"lat":40.9923617979921,"lng":-8.39684272258401},{"lat":40.9922356141567,"lng":-8.39684590550316},{"lat":40.9920561109219,"lng":-8.39679337269043},{"lat":40.9919628535329,"lng":-8.39681974171009},{"lat":40.9918324206084,"lng":-8.3968007968037},{"lat":40.9914477917995,"lng":-8.39685879288783},{"lat":40.9913519566074,"lng":-8.39687088567471},{"lat":40.9912658966717,"lng":-8.39692277093052},{"lat":40.9911644344036,"lng":-8.39696095253318},{"lat":40.9907754624941,"lng":-8.39712313824103},{"lat":40.9905792603725,"lng":-8.39722929032881},{"lat":40.9903297156473,"lng":-8.39732050508663},{"lat":40.9902015803794,"lng":-8.39735908195982},{"lat":40.990046370398,"lng":-8.39738685869355},{"lat":40.9898513723307,"lng":-8.39748696992904},{"lat":40.9897927182597,"lng":-8.39761793022683},{"lat":40.9897670490269,"lng":-8.39781473828123},{"lat":40.989759607271,"lng":-8.39811274769741},{"lat":40.9897199328813,"lng":-8.39822969146818},{"lat":40.9896534444042,"lng":-8.39831492812157},{"lat":40.9895384423183,"lng":-8.39839159942228},{"lat":40.9894939822177,"lng":-8.39853699215218},{"lat":40.9894018935104,"lng":-8.3987578940186},{"lat":40.9892334342446,"lng":-8.39890694160555},{"lat":40.9890743037625,"lng":-8.39897724024218},{"lat":40.9889707347764,"lng":-8.39903900076818},{"lat":40.9888718026122,"lng":-8.39910905761464},{"lat":40.9887865335873,"lng":-8.39918035813043},{"lat":40.9886986581413,"lng":-8.39929194470332},{"lat":40.988613009726,"lng":-8.39945368623155},{"lat":40.9885631106613,"lng":-8.39955306611516},{"lat":40.9884948848604,"lng":-8.39976393179731},{"lat":40.9883843297701,"lng":-8.40009517420471},{"lat":40.9883102125135,"lng":-8.40017722682596},{"lat":40.988219509472,"lng":-8.40022783750652},{"lat":40.9881499225969,"lng":-8.40033009107404},{"lat":40.9880777037875,"lng":-8.4004263475051},{"lat":40.9879896024696,"lng":-8.40058340633509},{"lat":40.9879779584815,"lng":-8.40071926001928},{"lat":40.9879612421725,"lng":-8.40085113764485},{"lat":40.9878728407111,"lng":-8.400956660307},{"lat":40.9877890290663,"lng":-8.40102488256787},{"lat":40.9877302743381,"lng":-8.40112149099816},{"lat":40.9876905264283,"lng":-8.40136755064132},{"lat":40.9876383320796,"lng":-8.40146846324294},{"lat":40.9875463583313,"lng":-8.40155147146501},{"lat":40.9874510447488,"lng":-8.40161106552452},{"lat":40.9873459614008,"lng":-8.40171672151383},{"lat":40.9872592356697,"lng":-8.40189328465073},{"lat":40.9871500257405,"lng":-8.4019776993734},{"lat":40.987060758776,"lng":-8.40201653315645},{"lat":40.9868523721611,"lng":-8.40210195427883},{"lat":40.9867185360553,"lng":-8.40238125216504},{"lat":40.9865174053576,"lng":-8.40278932400369},{"lat":40.9864350715973,"lng":-8.4028792995486},{"lat":40.9862834597663,"lng":-8.40294652615679},{"lat":40.9856882971043,"lng":-8.40300646469373},{"lat":40.9856384602529,"lng":-8.40306365894144},{"lat":40.9855664082677,"lng":-8.40327757644619},{"lat":40.9855432635567,"lng":-8.40333752248808},{"lat":40.9854972100235,"lng":-8.40335373304528},{"lat":40.9854486799596,"lng":-8.40335353373821},{"lat":40.985356670801,"lng":-8.40334495529716},{"lat":40.98529669777,"lng":-8.40336562409521},{"lat":40.9851413904023,"lng":-8.40341334573692},{"lat":40.984952402001,"lng":-8.40348315142747},{"lat":40.9849125354489,"lng":-8.40351091580688},{"lat":40.9848464985845,"lng":-8.40356482612219},{"lat":40.9847891549396,"lng":-8.40363363377588},{"lat":40.9847371927227,"lng":-8.40371267426132},{"lat":40.9847021772988,"lng":-8.40378323300475},{"lat":40.9846759109785,"lng":-8.40384717424079},{"lat":40.9846558126222,"lng":-8.40393074262262},{"lat":40.9846407492312,"lng":-8.40401945069276},{"lat":40.9846284778671,"lng":-8.40410776155164},{"lat":40.98460899174,"lng":-8.40427372425684},{"lat":40.984596087794,"lng":-8.40435958558565},{"lat":40.9845747163571,"lng":-8.40444980763911},{"lat":40.9845292619471,"lng":-8.40453729153669},{"lat":40.984482287748,"lng":-8.4046201790943},{"lat":40.9844423462476,"lng":-8.4047151907356},{"lat":40.9844296509412,"lng":-8.40477880735422},{"lat":40.9837955622559,"lng":-8.40507915777274},{"lat":40.9834786367002,"lng":-8.40523351892107},{"lat":40.9833578214405,"lng":-8.40529236255255},{"lat":40.9832623079681,"lng":-8.40566103538861},{"lat":40.9832348408019,"lng":-8.40577463526693},{"lat":40.9831618658631,"lng":-8.40594187686947},{"lat":40.983125478216,"lng":-8.40598367005782},{"lat":40.9829348268918,"lng":-8.40610859686914},{"lat":40.9828849335308,"lng":-8.40615021292006},{"lat":40.9828548870678,"lng":-8.40620140067795},{"lat":40.9828392057161,"lng":-8.40629368177716},{"lat":40.9828434000018,"lng":-8.40646124162526},{"lat":40.982815946429,"lng":-8.40656877804249},{"lat":40.9827749738275,"lng":-8.4066464405074},{"lat":40.9827891048297,"lng":-8.40688207132095},{"lat":40.9828464293186,"lng":-8.4070775952542},{"lat":40.9829321383831,"lng":-8.40723360896737},{"lat":40.9829786305187,"lng":-8.40734103618948},{"lat":40.9830382813878,"lng":-8.40755749379039},{"lat":40.9830777390267,"lng":-8.40771790854642},{"lat":40.9831074448714,"lng":-8.40783867817713},{"lat":40.9831117233643,"lng":-8.40797035731614},{"lat":40.9830840809158,"lng":-8.40815572601175},{"lat":40.9830610625821,"lng":-8.40829334702216},{"lat":40.9830366602096,"lng":-8.4084115822792},{"lat":40.9829322137884,"lng":-8.40843908325279},{"lat":40.9828302925465,"lng":-8.40847239982725},{"lat":40.9828147082558,"lng":-8.40866269073729},{"lat":40.9828290120291,"lng":-8.40878062735709},{"lat":40.9828264701784,"lng":-8.4088641450216},{"lat":40.9828353464536,"lng":-8.40897505492429},{"lat":40.9828416816662,"lng":-8.40905421484165},{"lat":40.9828503694482,"lng":-8.40922190530205},{"lat":40.9828394143678,"lng":-8.40935137999326},{"lat":40.9828316453299,"lng":-8.40944319877102},{"lat":40.9827878858247,"lng":-8.4095907122421},{"lat":40.9827391250845,"lng":-8.40979655149362},{"lat":40.9826890754884,"lng":-8.40998869328295},{"lat":40.9826336027383,"lng":-8.41020165293555},{"lat":40.9825361562545,"lng":-8.4103211285935},{"lat":40.9824549718244,"lng":-8.41038079747593},{"lat":40.982389443302,"lng":-8.41040808319531},{"lat":40.9822466214888,"lng":-8.41042350159702},{"lat":40.9821458060879,"lng":-8.41043438513567},{"lat":40.9819054336109,"lng":-8.41045071937757},{"lat":40.9817688713348,"lng":-8.41047224727499},{"lat":40.9816495856816,"lng":-8.41050434323548},{"lat":40.9815794348205,"lng":-8.41054048035218},{"lat":40.9814930584351,"lng":-8.41065737715105},{"lat":40.981360533451,"lng":-8.41083672788559},{"lat":40.98124930271,"lng":-8.41095888608467},{"lat":40.981054308564,"lng":-8.41116290759832},{"lat":40.9809515299825,"lng":-8.41128960830316},{"lat":40.9808130891091,"lng":-8.41145478051055},{"lat":40.9807424397477,"lng":-8.41154977244694},{"lat":40.9807024465766,"lng":-8.41166640696831},{"lat":40.9806741912784,"lng":-8.41184320489596},{"lat":40.9806450294735,"lng":-8.41194942845597},{"lat":40.9806167729994,"lng":-8.41201489407749},{"lat":40.9805874716371,"lng":-8.41214551532909},{"lat":40.9805180756689,"lng":-8.41233927346683},{"lat":40.9804739502742,"lng":-8.41246247380628},{"lat":40.98038700853,"lng":-8.41270521800798},{"lat":40.9803690878463,"lng":-8.41277856888781},{"lat":40.9802755825484,"lng":-8.41283057029543},{"lat":40.9802347948525,"lng":-8.41285763755689},{"lat":40.9801161231468,"lng":-8.41293307420734},{"lat":40.9853436319597,"lng":-8.43514929351131},{"lat":40.9868091201826,"lng":-8.44138157714521},{"lat":40.9872382859285,"lng":-8.44237516272507},{"lat":40.9878064224684,"lng":-8.44373609474281},{"lat":40.9909001312093,"lng":-8.44884157651032}]},{"zone_id":"vng6","points":[{"lat":41.0712423491077,"lng":-8.57776784079373},{"lat":41.0740761005192,"lng":-8.57803693863322},{"lat":41.0753954442825,"lng":-8.57782097126857},{"lat":41.0773092361523,"lng":-8.57679417967297},{"lat":41.0791748486481,"lng":-8.57543233657418},{"lat":41.0820435977557,"lng":-8.57465460583274},{"lat":41.084162413682,"lng":-8.57467102960342},{"lat":41.0871923498252,"lng":-8.57492719975503},{"lat":41.0890325522997,"lng":-8.5743717297037},{"lat":41.0899844062621,"lng":-8.57363680336761},{"lat":41.0908873512209,"lng":-8.57336664500674},{"lat":41.0915852296876,"lng":-8.57374038053339},{"lat":41.0953024827548,"lng":-8.57244610904099},{"lat":41.0981252542169,"lng":-8.57178899658966},{"lat":41.0993246936079,"lng":-8.57092993526233},{"lat":41.1002397724453,"lng":-8.56945311242161},{"lat":41.1001384873509,"lng":-8.56798523866135},{"lat":41.1001439747319,"lng":-8.56734292687339},{"lat":41.1006070237514,"lng":-8.56661203660699},{"lat":41.1015773535307,"lng":-8.56605496514331},{"lat":41.102285325421,"lng":-8.56595961754982},{"lat":41.1029863581502,"lng":-8.56643957564383},{"lat":41.1051176298744,"lng":-8.56571194516766},{"lat":41.1073847579031,"lng":-8.56269933381333},{"lat":41.1086528893335,"lng":-8.55944076669029},{"lat":41.1091285285777,"lng":-8.55649990211834},{"lat":41.1094454667943,"lng":-8.55117484851976},{"lat":41.1086010054484,"lng":-8.53821342737145},{"lat":41.1085868513151,"lng":-8.5380433659677},{"lat":41.1080948558609,"lng":-8.53778118510196},{"lat":41.107859227996,"lng":-8.53763779771951},{"lat":41.1075096588873,"lng":-8.53742507330555},{"lat":41.1071352487874,"lng":-8.53714740698212},{"lat":41.1067453620689,"lng":-8.53681500590013},{"lat":41.1063286436525,"lng":-8.53643018630746},{"lat":41.1062338820519,"lng":-8.53630175455824},{"lat":41.1060577111429,"lng":-8.5360629876119},{"lat":41.105990344609,"lng":-8.53597168478737},{"lat":41.1058207677104,"lng":-8.53565485883221},{"lat":41.1057694743728,"lng":-8.5355590258455},{"lat":41.1054903708918,"lng":-8.53506327875905},{"lat":41.1051313448932,"lng":-8.53441765214285},{"lat":41.1049512400983,"lng":-8.53401965606255},{"lat":41.1045165291857,"lng":-8.5332158412586},{"lat":41.1040968564501,"lng":-8.53251331150504},{"lat":41.1036492699542,"lng":-8.53183502800149},{"lat":41.102954947084,"lng":-8.53092776751118},{"lat":41.1024446317507,"lng":-8.53023793887171},{"lat":41.1021470717553,"lng":-8.52989032946705},{"lat":41.1020271698295,"lng":-8.5297505694468},{"lat":41.1016640513261,"lng":-8.52929983967474},{"lat":41.1013661249178,"lng":-8.52887509592027},{"lat":41.1009713844164,"lng":-8.52845489646016},{"lat":41.1005443279876,"lng":-8.528004513519},{"lat":41.1001463255884,"lng":-8.52764215428962},{"lat":41.0998034007321,"lng":-8.52736524163694},{"lat":41.0994371430569,"lng":-8.52717068491282},{"lat":41.0990518166506,"lng":-8.52696970406146},{"lat":41.0985130775685,"lng":-8.52671042957936},{"lat":41.097990118937,"lng":-8.52649708098898},{"lat":41.097332939723,"lng":-8.52622032265078},{"lat":41.0967383529749,"lng":-8.52588741060534},{"lat":41.0962480861359,"lng":-8.52567165038123},{"lat":41.0956342217023,"lng":-8.52539326452236},{"lat":41.0950301219365,"lng":-8.52512994173154},{"lat":41.0945433506756,"lng":-8.52491873540143},{"lat":41.093854277421,"lng":-8.52464610781826},{"lat":41.093358519058,"lng":-8.52448199027956},{"lat":41.0929667432469,"lng":-8.5243233791873},{"lat":41.0923991212625,"lng":-8.5241081319306},{"lat":41.0918279241414,"lng":-8.52385966743192},{"lat":41.0913163915588,"lng":-8.5236197765372},{"lat":41.0908486310603,"lng":-8.52340430462348},{"lat":41.0904911564597,"lng":-8.52327138022469},{"lat":41.090377475371,"lng":-8.52324105907094},{"lat":41.0902214223654,"lng":-8.52319924146932},{"lat":41.0899541782887,"lng":-8.52312762794096},{"lat":41.0895178991518,"lng":-8.5230010214366},{"lat":41.0890950135677,"lng":-8.5228268868528},{"lat":41.0885174062281,"lng":-8.52247677036953},{"lat":41.0883727398084,"lng":-8.52239889625177},{"lat":41.0881987224512,"lng":-8.52230454967007},{"lat":41.0879222146204,"lng":-8.52215512137951},{"lat":41.0876476482153,"lng":-8.5220067444075},{"lat":41.0872159124211,"lng":-8.52186780048207},{"lat":41.0868678060103,"lng":-8.52178635446396},{"lat":41.0867525388677,"lng":-8.52177870117988},{"lat":41.0866416401939,"lng":-8.52177133799804},{"lat":41.0864211131852,"lng":-8.52175669597924},{"lat":41.0825161309998,"lng":-8.53107924159995},{"lat":41.0774071203912,"lng":-8.54925651046801},{"lat":41.0733093876719,"lng":-8.55530118529578},{"lat":41.0676992034877,"lng":-8.56193324024034},{"lat":41.0637275316545,"lng":-8.5667835620139},{"lat":41.0628821998062,"lng":-8.56761378591302},{"lat":41.0574845042287,"lng":-8.56601252970078},{"lat":41.0543155993931,"lng":-8.56476618922353},{"lat":41.0535081350042,"lng":-8.56962756413254},{"lat":41.0526871042344,"lng":-8.57238704756344},{"lat":41.0546442579223,"lng":-8.57431370926751},{"lat":41.0563540827257,"lng":-8.57587158904485},{"lat":41.0576114586968,"lng":-8.57671275174511},{"lat":41.0586190561121,"lng":-8.5771910099482},{"lat":41.0625526609228,"lng":-8.57885146619834},{"lat":41.0657421561639,"lng":-8.58045790790753},{"lat":41.0682150856866,"lng":-8.58108760465294},{"lat":41.0708406856745,"lng":-8.58048160574828},{"lat":41.0712423491077,"lng":-8.57776784079373}]},{"zone_id":"smf7","points":[{"lat":40.9966973834513,"lng":-8.45014198107079},{"lat":40.9928882309973,"lng":-8.44911752409347},{"lat":40.9909001312093,"lng":-8.44884157651032},{"lat":40.9893052440854,"lng":-8.45200079157313},{"lat":40.9884269288176,"lng":-8.45374045469553},{"lat":40.9868666752995,"lng":-8.45789885047271},{"lat":40.9848241588363,"lng":-8.46401966534918},{"lat":40.9835029216451,"lng":-8.46778580778187},{"lat":40.9824795982894,"lng":-8.47147472514984},{"lat":40.9821498807099,"lng":-8.47298608326142},{"lat":40.9816720302293,"lng":-8.47517643895761},{"lat":40.9813314226969,"lng":-8.47704919455157},{"lat":40.9807838185838,"lng":-8.48089753652779},{"lat":40.9804699586402,"lng":-8.48495991825015},{"lat":40.9804891329965,"lng":-8.48841821281277},{"lat":40.9806918895894,"lng":-8.49026629575356},{"lat":40.981074928352,"lng":-8.49206319231294},{"lat":40.9812162457886,"lng":-8.49573029030672},{"lat":40.9795685514252,"lng":-8.49900380606854},{"lat":40.9792795532291,"lng":-8.50286134981157},{"lat":40.9835986055494,"lng":-8.5084579465327},{"lat":40.9914241159369,"lng":-8.51544913626225},{"lat":41.0005637621721,"lng":-8.51376260847822},{"lat":41.0032354002961,"lng":-8.51277161534497},{"lat":41.005909918452,"lng":-8.51089995437931},{"lat":41.0099222971547,"lng":-8.5071071004392},{"lat":41.0101904800719,"lng":-8.5068497346675},{"lat":41.0093041228087,"lng":-8.5056373187502},{"lat":41.0093622180944,"lng":-8.50549369455889},{"lat":41.0094063770672,"lng":-8.5053435713575},{"lat":41.0094324273039,"lng":-8.50508529177552},{"lat":41.0094973605193,"lng":-8.50466532898623},{"lat":41.0095235392726,"lng":-8.50447849104388},{"lat":41.0095233020671,"lng":-8.50430137117397},{"lat":41.0094725564453,"lng":-8.5040107996886},{"lat":41.0094752286263,"lng":-8.50388076878699},{"lat":41.0094157308634,"lng":-8.50345582480363},{"lat":41.0093779999867,"lng":-8.50298238433122},{"lat":41.0093722273631,"lng":-8.50281783443193},{"lat":41.0093941331828,"lng":-8.50205527687096},{"lat":41.0093989030666,"lng":-8.50166397871588},{"lat":41.0094054158368,"lng":-8.50140261708514},{"lat":41.0094109827355,"lng":-8.5011832110843},{"lat":41.0095166344421,"lng":-8.50086558290004},{"lat":41.0096050356556,"lng":-8.50081662553389},{"lat":41.0097727574624,"lng":-8.50079842528396},{"lat":41.0099255016085,"lng":-8.50081758456242},{"lat":41.0100505901408,"lng":-8.5008406318883},{"lat":41.0103937780968,"lng":-8.50083232747995},{"lat":41.0108693716955,"lng":-8.4994331023985},{"lat":41.0117036256735,"lng":-8.49765591353532},{"lat":41.0113755766474,"lng":-8.49682873073496},{"lat":41.0107936164168,"lng":-8.4951390450283},{"lat":41.0117054085721,"lng":-8.49093293586295},{"lat":41.0134011311728,"lng":-8.48916627999709},{"lat":41.0221499599551,"lng":-8.4919326608894},{"lat":41.0223006826293,"lng":-8.49167267204416},{"lat":41.0223615137399,"lng":-8.49145471210879},{"lat":41.0225672231527,"lng":-8.49063212712446},{"lat":41.0226954890536,"lng":-8.49047409949286},{"lat":41.0229445444993,"lng":-8.49018772514009},{"lat":41.0230734566622,"lng":-8.4898509975039},{"lat":41.0232398530888,"lng":-8.48953432894728},{"lat":41.0233608270003,"lng":-8.48934641369896},{"lat":41.0234289424852,"lng":-8.48918805650698},{"lat":41.0235429378047,"lng":-8.48881153214167},{"lat":41.023793495701,"lng":-8.48803886467323},{"lat":41.0238599174846,"lng":-8.48656275369795},{"lat":41.0247987180174,"lng":-8.47748353762295},{"lat":41.0268224352972,"lng":-8.47590405256978},{"lat":41.0281692269943,"lng":-8.47557457293075},{"lat":41.0282996233564,"lng":-8.47502793666891},{"lat":41.0285386751599,"lng":-8.47509574525289},{"lat":41.0285516683473,"lng":-8.47464889972636},{"lat":41.0285555824869,"lng":-8.47425300762239},{"lat":41.0286610647977,"lng":-8.47407887770294},{"lat":41.028771432995,"lng":-8.47394778285556},{"lat":41.0288277677188,"lng":-8.4738263229927},{"lat":41.0289516016668,"lng":-8.4735538654228},{"lat":41.0291011241045,"lng":-8.47325843290045},{"lat":41.0291587448599,"lng":-8.47303820598629},{"lat":41.0292995447232,"lng":-8.47234867394025},{"lat":41.0293119815863,"lng":-8.47212234844533},{"lat":41.0293914380121,"lng":-8.47183594895565},{"lat":41.0294249874442,"lng":-8.47166609061486},{"lat":41.02961707924,"lng":-8.4707095135126},{"lat":41.0294646284321,"lng":-8.47068149513804},{"lat":41.0294646486071,"lng":-8.47055129700126},{"lat":41.0294332389286,"lng":-8.4703934621688},{"lat":41.0296075002404,"lng":-8.4704151665074},{"lat":41.0297667612764,"lng":-8.47045640972674},{"lat":41.0298313161352,"lng":-8.47024460792214},{"lat":41.0298896646272,"lng":-8.47003004389676},{"lat":41.0299576408601,"lng":-8.46978758114472},{"lat":41.0300120530661,"lng":-8.46953399140685},{"lat":41.0300146359397,"lng":-8.46923696735091},{"lat":41.0300295028365,"lng":-8.46895760675142},{"lat":41.0300444288814,"lng":-8.46874923682277},{"lat":41.0300581935144,"lng":-8.46860007694031},{"lat":41.0301045440523,"lng":-8.46796082282052},{"lat":41.0301946946149,"lng":-8.4671215418972},{"lat":41.0302528988207,"lng":-8.46661576271443},{"lat":41.0244403133584,"lng":-8.46369355387916},{"lat":41.0127549683661,"lng":-8.4581303034357},{"lat":41.0105579361965,"lng":-8.45636850445364},{"lat":41.007359237654,"lng":-8.45434459548284},{"lat":41.005170191828,"lng":-8.45332756965765},{"lat":41.0020288550275,"lng":-8.45205443003603},{"lat":40.9966973834513,"lng":-8.45014198107079}]},{"zone_id":"prd8","points":[{"lat":41.09658636636,"lng":-8.41628449603349},{"lat":41.0967551007626,"lng":-8.41623949670393},{"lat":41.099225670582,"lng":-8.41564887410075},{"lat":41.101492834107,"lng":-8.41480725797219},{"lat":41.1220570798218,"lng":-8.42345552682227},{"lat":41.1233426305325,"lng":-8.42372014616764},{"lat":41.1245929926164,"lng":-8.42388319641974},{"lat":41.1263204303485,"lng":-8.42381205286397},{"lat":41.1328138024781,"lng":-8.423308971294},{"lat":41.1328804713578,"lng":-8.4204732945368},{"lat":41.1332452590641,"lng":-8.41748134521624},{"lat":41.1350735545196,"lng":-8.40951146256644},{"lat":41.135511844126,"lng":-8.40554278423306},{"lat":41.1354501338379,"lng":-8.40138294223236},{"lat":41.1349570525881,"lng":-8.39826126658247},{"lat":41.1340348393863,"lng":-8.39523246404444},{"lat":41.1328260197784,"lng":-8.3924862155971},{"lat":41.1313306137319,"lng":-8.39002254172045},{"lat":41.1300134892641,"lng":-8.38827632619309},{"lat":41.1284449538798,"lng":-8.39046820902464},{"lat":41.1267514451228,"lng":-8.39256895471998},{"lat":41.125402277481,"lng":-8.39248170036548},{"lat":41.1241698377252,"lng":-8.39240199839292},{"lat":41.1170224009972,"lng":-8.38839518711597},{"lat":41.1154659011718,"lng":-8.38742994228931},{"lat":41.1082121734289,"lng":-8.38066957932414},{"lat":41.1080585184713,"lng":-8.38056280528918},{"lat":41.107865422575,"lng":-8.38045790522101},{"lat":41.1076717293578,"lng":-8.38034159592935},{"lat":41.1074402479621,"lng":-8.38020296116982},{"lat":41.1072175230135,"lng":-8.3800596194352},{"lat":41.1070004639647,"lng":-8.37995808367811},{"lat":41.1069109573417,"lng":-8.37991670656067},{"lat":41.106778728175,"lng":-8.37985557988834},{"lat":41.1065577101148,"lng":-8.37975593901333},{"lat":41.1063404780977,"lng":-8.37965262130124},{"lat":41.1061344584111,"lng":-8.37952779505034},{"lat":41.1059198006568,"lng":-8.3793974682892},{"lat":41.1058142745443,"lng":-8.37934531840618},{"lat":41.105703301932,"lng":-8.37929046268229},{"lat":41.1054841089562,"lng":-8.37917713708572},{"lat":41.1052628685637,"lng":-8.37905142979783},{"lat":41.1050380668676,"lng":-8.37890963418912},{"lat":41.1048161627004,"lng":-8.37875808866123},{"lat":41.1016624059933,"lng":-8.3768998361672},{"lat":41.1014646094386,"lng":-8.37678331038482},{"lat":41.101245763201,"lng":-8.37665438473086},{"lat":41.1010792282906,"lng":-8.37647540395951},{"lat":41.1005615539165,"lng":-8.37591904931842},{"lat":41.1005123096122,"lng":-8.37585799883556},{"lat":41.1003432289967,"lng":-8.37564838264084},{"lat":41.0997083251317,"lng":-8.37537544264121},{"lat":41.0992332780731,"lng":-8.37517122744139},{"lat":41.0990469738073,"lng":-8.37509113904636},{"lat":41.0989651092342,"lng":-8.37497672839046},{"lat":41.0987807191634,"lng":-8.37475031835961},{"lat":41.0986811075811,"lng":-8.37465461781037},{"lat":41.0986063346242,"lng":-8.37458278089369},{"lat":41.0983827614468,"lng":-8.37439056111751},{"lat":41.098179350996,"lng":-8.37422340504991},{"lat":41.0980906033626,"lng":-8.37416165837646},{"lat":41.0979351770884,"lng":-8.37413119550585},{"lat":41.0978056348533,"lng":-8.37411191924365},{"lat":41.0975833543315,"lng":-8.37396624136052},{"lat":41.0973935856329,"lng":-8.37380240991078},{"lat":41.0970805150809,"lng":-8.37368383655452},{"lat":41.0969619536697,"lng":-8.3736271035991},{"lat":41.0967564965252,"lng":-8.37348743612615},{"lat":41.0965328805549,"lng":-8.37337853809528},{"lat":41.0963250839715,"lng":-8.37328195728743},{"lat":41.0961100780932,"lng":-8.37318678038473},{"lat":41.0959164330054,"lng":-8.37304604727481},{"lat":41.0938944884729,"lng":-8.37141161264508},{"lat":41.0938178047555,"lng":-8.37148072242253},{"lat":41.0937790909323,"lng":-8.3715655675646},{"lat":41.0937178860787,"lng":-8.37164020698847},{"lat":41.0935758355845,"lng":-8.37188035395875},{"lat":41.0934771493544,"lng":-8.37209614277545},{"lat":41.0934446042018,"lng":-8.37219957253548},{"lat":41.0933889432978,"lng":-8.37229185027463},{"lat":41.0931842832094,"lng":-8.3725034372467},{"lat":41.0930917817222,"lng":-8.3726023637451},{"lat":41.093021603916,"lng":-8.37268207825805},{"lat":41.0929874390307,"lng":-8.37272088584438},{"lat":41.0928894441394,"lng":-8.37277848944767},{"lat":41.0928448733804,"lng":-8.37283322590284},{"lat":41.0926792764054,"lng":-8.37303659129723},{"lat":41.0926158394281,"lng":-8.37323001085675},{"lat":41.0924297545225,"lng":-8.37351378427662},{"lat":41.0923557186734,"lng":-8.37359063798234},{"lat":41.0922949804817,"lng":-8.3736321915703},{"lat":41.0920787395049,"lng":-8.37373923325909},{"lat":41.0918729440628,"lng":-8.37388915659008},{"lat":41.0916714613358,"lng":-8.37404349614946},{"lat":41.0915510969937,"lng":-8.37411910372087},{"lat":41.0912888636514,"lng":-8.37425870036154},{"lat":41.0912289686615,"lng":-8.37426466565693},{"lat":41.0909818857212,"lng":-8.37426851583574},{"lat":41.0908237768591,"lng":-8.37426424093892},{"lat":41.090637740731,"lng":-8.3741940001224},{"lat":41.090529115125,"lng":-8.37412455421867},{"lat":41.0903909231778,"lng":-8.37401461678233},{"lat":41.0902283546829,"lng":-8.37382001253693},{"lat":41.0900578833938,"lng":-8.37357230999686},{"lat":41.0899946618217,"lng":-8.37347330397253},{"lat":41.0898992060287,"lng":-8.37332381924229},{"lat":41.0897055323825,"lng":-8.37310506984779},{"lat":41.0895510455804,"lng":-8.37296215333586},{"lat":41.089476485419,"lng":-8.37283596546369},{"lat":41.0894735553229,"lng":-8.37273122198195},{"lat":41.0894819009762,"lng":-8.37265745919552},{"lat":41.0895171496147,"lng":-8.3724809049648},{"lat":41.0895207435878,"lng":-8.37212500586415},{"lat":41.089514122843,"lng":-8.37193467668203},{"lat":41.0894634097136,"lng":-8.37181381072625},{"lat":41.089327757208,"lng":-8.37170596284483},{"lat":41.0892440652331,"lng":-8.37160775351502},{"lat":41.0892050558018,"lng":-8.37156197755966},{"lat":41.0891432924651,"lng":-8.3714730868272},{"lat":41.0890809195088,"lng":-8.3714590542974},{"lat":41.0888482884706,"lng":-8.37139643590371},{"lat":41.088621360103,"lng":-8.3713482867227},{"lat":41.0884095359315,"lng":-8.37133772186674},{"lat":41.0881218467564,"lng":-8.37133952163904},{"lat":41.087895730774,"lng":-8.37130371112857},{"lat":41.0877292963243,"lng":-8.37121661810097},{"lat":41.0876756325972,"lng":-8.37118853665819},{"lat":41.0875016434382,"lng":-8.37106467650835},{"lat":41.087242963269,"lng":-8.37089073784905},{"lat":41.0870217372626,"lng":-8.37075330881374},{"lat":41.0868002602023,"lng":-8.37064169872815},{"lat":41.0867578808367,"lng":-8.37062151330185},{"lat":41.0865466931389,"lng":-8.37052427130454},{"lat":41.0863284129543,"lng":-8.37035972329219},{"lat":41.0861386371787,"lng":-8.37016802167261},{"lat":41.0859472110638,"lng":-8.36994715993525},{"lat":41.0857703137383,"lng":-8.36975325275603},{"lat":41.0855661701624,"lng":-8.36958661297774},{"lat":41.0852980750733,"lng":-8.36951721412402},{"lat":41.0850713964933,"lng":-8.36949283313037},{"lat":41.0848408470949,"lng":-8.3695109186766},{"lat":41.0846234720496,"lng":-8.36960058139907},{"lat":41.0843933844376,"lng":-8.36970162034158},{"lat":41.0841838006533,"lng":-8.36981284185988},{"lat":41.0839786675162,"lng":-8.36991651934692},{"lat":41.0837446558755,"lng":-8.37003022226526},{"lat":41.083566952979,"lng":-8.37018508126929},{"lat":41.0834096546425,"lng":-8.37048254042772},{"lat":41.0831967731385,"lng":-8.37070751921649},{"lat":41.0830804066287,"lng":-8.37079178927682},{"lat":41.0829902617358,"lng":-8.37085706998409},{"lat":41.0827927679835,"lng":-8.37100189807376},{"lat":41.0826614299593,"lng":-8.37104845292196},{"lat":41.0825714564921,"lng":-8.37108034533544},{"lat":41.0823326994938,"lng":-8.37108407046009},{"lat":41.0822241619158,"lng":-8.37107694098178},{"lat":41.0821708975545,"lng":-8.37103405501252},{"lat":41.0820792539276,"lng":-8.37096026804987},{"lat":41.081818375722,"lng":-8.37084198531294},{"lat":41.0816668215201,"lng":-8.37075837257091},{"lat":41.0815170706011,"lng":-8.37059859062369},{"lat":41.0814576695171,"lng":-8.37052927779457},{"lat":41.0813241341649,"lng":-8.37035965998679},{"lat":41.0810317196966,"lng":-8.37024971797763},{"lat":41.0809383316366,"lng":-8.370236467108},{"lat":41.0807855257557,"lng":-8.37021478550293},{"lat":41.0806675145375,"lng":-8.3701792561676},{"lat":41.0805574948033,"lng":-8.37014613292696},{"lat":41.0803404478544,"lng":-8.37003657805147},{"lat":41.0800928225497,"lng":-8.36990811565357},{"lat":41.0798629194342,"lng":-8.36978638179387},{"lat":41.0796877206837,"lng":-8.36972981753392},{"lat":41.0794511725158,"lng":-8.36973110403215},{"lat":41.0792262075416,"lng":-8.36973474421652},{"lat":41.0789695122062,"lng":-8.36969497064917},{"lat":41.0787626552157,"lng":-8.36957749071685},{"lat":41.0785632575877,"lng":-8.36938089769578},{"lat":41.0785012923058,"lng":-8.36934031436511},{"lat":41.0783822383124,"lng":-8.36930884872844},{"lat":41.0782091879375,"lng":-8.3693009632838},{"lat":41.078129618408,"lng":-8.36928663206425},{"lat":41.0778843027462,"lng":-8.36913046368936},{"lat":41.0776642625356,"lng":-8.36898615672444},{"lat":41.0774733045068,"lng":-8.36884340983608},{"lat":41.0772551942627,"lng":-8.36868065675732},{"lat":41.0770300748851,"lng":-8.36850992245991},{"lat":41.0768620147729,"lng":-8.36844160905468},{"lat":41.076624755897,"lng":-8.3684363541434},{"lat":41.0765052817985,"lng":-8.36838678000364},{"lat":41.0762426855306,"lng":-8.36841435444303},{"lat":41.0761041353068,"lng":-8.36850528366928},{"lat":41.0758601097745,"lng":-8.36869752925197},{"lat":41.0749588362081,"lng":-8.36936157798457},{"lat":41.0749061879832,"lng":-8.36939220697577},{"lat":41.0746685331886,"lng":-8.36944743074239},{"lat":41.0743519140468,"lng":-8.36950573784058},{"lat":41.0741811304741,"lng":-8.36951780564366},{"lat":41.0740090662926,"lng":-8.36952996387484},{"lat":41.073959795644,"lng":-8.36948510908657},{"lat":41.0738647242977,"lng":-8.36939855870394},{"lat":41.0737008391833,"lng":-8.36931395625547},{"lat":41.0734941054069,"lng":-8.36913604246559},{"lat":41.0732890986168,"lng":-8.36895098954011},{"lat":41.0730905446572,"lng":-8.3687784592819},{"lat":41.0728375510452,"lng":-8.36863452477045},{"lat":41.0727398498175,"lng":-8.3685915810789},{"lat":41.0725203405777,"lng":-8.36858186786542},{"lat":41.0724109827169,"lng":-8.36856065026933},{"lat":41.0723608639387,"lng":-8.36854285789815},{"lat":41.0723116686171,"lng":-8.36851424347153},{"lat":41.072032543347,"lng":-8.36851061943335},{"lat":41.0718238392785,"lng":-8.3685163490146},{"lat":41.0717680677809,"lng":-8.36851788011148},{"lat":41.0716114087259,"lng":-8.36859561110648},{"lat":41.0715395702651,"lng":-8.36863152262797},{"lat":41.0713915918968,"lng":-8.36865002494726},{"lat":41.0712932447485,"lng":-8.36870369172271},{"lat":41.0712105966765,"lng":-8.36882749141632},{"lat":41.0711516163922,"lng":-8.36887141915541},{"lat":41.0710846891987,"lng":-8.3688847418824},{"lat":41.0709751429759,"lng":-8.36889002428178},{"lat":41.0682927714938,"lng":-8.36901936496767},{"lat":41.0685034316893,"lng":-8.3692512136441},{"lat":41.0687920137864,"lng":-8.36956882496824},{"lat":41.0690751770354,"lng":-8.37005980112071},{"lat":41.0691143447028,"lng":-8.37032074357242},{"lat":41.0691923198246,"lng":-8.37040752992999},{"lat":41.0692399109742,"lng":-8.3704282859031},{"lat":41.0692738530568,"lng":-8.37047528045509},{"lat":41.0692906879362,"lng":-8.37053075928434},{"lat":41.0693777679365,"lng":-8.37081773062034},{"lat":41.0694233341458,"lng":-8.37094615162387},{"lat":41.069476908767,"lng":-8.3710464813954},{"lat":41.069552048503,"lng":-8.37118719694309},{"lat":41.0696981265889,"lng":-8.37143306635558},{"lat":41.0698033891363,"lng":-8.37156028344053},{"lat":41.0698917543757,"lng":-8.37163235119985},{"lat":41.0699462829054,"lng":-8.37165111184667},{"lat":41.0701205329774,"lng":-8.37173715519108},{"lat":41.0702622369594,"lng":-8.37182764134126},{"lat":41.0704717415392,"lng":-8.37201354144746},{"lat":41.0706461932543,"lng":-8.37221069187311},{"lat":41.070818725278,"lng":-8.3724226769618},{"lat":41.0709292614563,"lng":-8.37257074147663},{"lat":41.0711557800878,"lng":-8.37284392068304},{"lat":41.0712564150121,"lng":-8.37290211267104},{"lat":41.0714132358096,"lng":-8.37296182143389},{"lat":41.0716432892905,"lng":-8.37305357286807},{"lat":41.0717846032711,"lng":-8.37312238782654},{"lat":41.0720217721475,"lng":-8.37329744796603},{"lat":41.0721692737473,"lng":-8.37341864125228},{"lat":41.0722142209489,"lng":-8.37345557182251},{"lat":41.0723209448913,"lng":-8.37354528523086},{"lat":41.0723953397012,"lng":-8.37360782259613},{"lat":41.0724625348805,"lng":-8.37351335770567},{"lat":41.0724868677333,"lng":-8.37345812231482},{"lat":41.072542568764,"lng":-8.3734339328935},{"lat":41.0726076967513,"lng":-8.37354910899414},{"lat":41.0727692540094,"lng":-8.37379922080994},{"lat":41.0728771959705,"lng":-8.37400256650726},{"lat":41.0729143666746,"lng":-8.37407259058629},{"lat":41.0730626501683,"lng":-8.37433645295215},{"lat":41.0731469205124,"lng":-8.37444716648976},{"lat":41.0732374815182,"lng":-8.3745661451099},{"lat":41.0733195081275,"lng":-8.37465378253245},{"lat":41.073516102988,"lng":-8.37481038418469},{"lat":41.0735659057284,"lng":-8.37484841539639},{"lat":41.0737312290032,"lng":-8.37497466283854},{"lat":41.0739222412511,"lng":-8.37513267739544},{"lat":41.0741349100686,"lng":-8.37530909079922},{"lat":41.074318391959,"lng":-8.37553489081952},{"lat":41.0744879305297,"lng":-8.37575398689143},{"lat":41.0746349394727,"lng":-8.37593801201264},{"lat":41.0751731792956,"lng":-8.37625533557361},{"lat":41.0755901731263,"lng":-8.37645254243059},{"lat":41.0759068379722,"lng":-8.37660230298139},{"lat":41.0765277437089,"lng":-8.37685494416768},{"lat":41.0768395877616,"lng":-8.37701157242588},{"lat":41.0773345147732,"lng":-8.37726446039897},{"lat":41.0774003829344,"lng":-8.37741056951241},{"lat":41.0775700341818,"lng":-8.37778689408134},{"lat":41.0776695056669,"lng":-8.37808087313838},{"lat":41.0776903745149,"lng":-8.37814254741376},{"lat":41.0777820054391,"lng":-8.37841335613031},{"lat":41.0779156515105,"lng":-8.37865375455445},{"lat":41.0779739928297,"lng":-8.37870739367876},{"lat":41.0780181892546,"lng":-8.37874802803821},{"lat":41.0782394708124,"lng":-8.37868614874555},{"lat":41.0783930446027,"lng":-8.37874039733843},{"lat":41.0786575861282,"lng":-8.37883384493045},{"lat":41.0789810434173,"lng":-8.37890800508005},{"lat":41.0796995460673,"lng":-8.3789732958843},{"lat":41.0799992798117,"lng":-8.37906819451345},{"lat":41.0801212676235,"lng":-8.37921916454543},{"lat":41.0802274728742,"lng":-8.37935060300766},{"lat":41.0804476711742,"lng":-8.37978946989249},{"lat":41.0807224347668,"lng":-8.38029633073501},{"lat":41.0809744909807,"lng":-8.38076131027503},{"lat":41.0811947171414,"lng":-8.38109419450099},{"lat":41.0816616095213,"lng":-8.38179994081615},{"lat":41.0828233754449,"lng":-8.3828608602444},{"lat":41.0837175274101,"lng":-8.38367744776839},{"lat":41.0842857401171,"lng":-8.38419635142378},{"lat":41.0843441943863,"lng":-8.38432030202746},{"lat":41.0854127118953,"lng":-8.38669414696124},{"lat":41.0859847211067,"lng":-8.38985809379033},{"lat":41.0860815708003,"lng":-8.38995379868916},{"lat":41.0862194089642,"lng":-8.39008215602112},{"lat":41.0863928304492,"lng":-8.39032490501259},{"lat":41.0865459492694,"lng":-8.39054496376104},{"lat":41.0866157491138,"lng":-8.39065830006023},{"lat":41.0866686938053,"lng":-8.39077787364492},{"lat":41.0867000093225,"lng":-8.39084732141784},{"lat":41.086729613013,"lng":-8.39095568155898},{"lat":41.0867190250431,"lng":-8.3910190701518},{"lat":41.0867194749454,"lng":-8.39114956500588},{"lat":41.0867200588163,"lng":-8.39131898160678},{"lat":41.086804684445,"lng":-8.39160387283286},{"lat":41.0868488967636,"lng":-8.39172413211698},{"lat":41.086855891234,"lng":-8.3919015611365},{"lat":41.0868469014383,"lng":-8.39206684308202},{"lat":41.08690626377,"lng":-8.39247674417235},{"lat":41.0869813957044,"lng":-8.39273334457045},{"lat":41.0870507233851,"lng":-8.39284398162261},{"lat":41.0873549676936,"lng":-8.39326221562927},{"lat":41.0875499836461,"lng":-8.3933872399582},{"lat":41.0876237016711,"lng":-8.39345864609493},{"lat":41.0877377609903,"lng":-8.39356912877514},{"lat":41.0879740850483,"lng":-8.39386474202231},{"lat":41.088101537878,"lng":-8.39396510524432},{"lat":41.0882930520124,"lng":-8.39416688397969},{"lat":41.0883634878721,"lng":-8.39425500796806},{"lat":41.0884661621709,"lng":-8.39438346673009},{"lat":41.0885997588285,"lng":-8.39451217464605},{"lat":41.0887766232583,"lng":-8.39470176114305},{"lat":41.0889610235958,"lng":-8.39490267678993},{"lat":41.0890574023049,"lng":-8.39502925162264},{"lat":41.089134734589,"lng":-8.39513081305444},{"lat":41.0893043896258,"lng":-8.39536166583433},{"lat":41.0893439289164,"lng":-8.39539840532994},{"lat":41.0895755621428,"lng":-8.39561363761517},{"lat":41.0896671524633,"lng":-8.39568541866898},{"lat":41.0897916882053,"lng":-8.3958398057176},{"lat":41.0899405080429,"lng":-8.39608450586445},{"lat":41.0899982727056,"lng":-8.39620038923723},{"lat":41.090086209236,"lng":-8.3963941786958},{"lat":41.0902405893974,"lng":-8.39664699667332},{"lat":41.0903271295978,"lng":-8.39682808867408},{"lat":41.0903621952097,"lng":-8.39690146646021},{"lat":41.0904426684506,"lng":-8.39702878518054},{"lat":41.0904726062961,"lng":-8.39708746473941},{"lat":41.0904928753529,"lng":-8.39720025576054},{"lat":41.0905012047095,"lng":-8.39737571852088},{"lat":41.0905278040528,"lng":-8.39755542620209},{"lat":41.0905784060374,"lng":-8.39771214060191},{"lat":41.090601291138,"lng":-8.39778340238299},{"lat":41.0906170579063,"lng":-8.39789748497384},{"lat":41.0906269630661,"lng":-8.39797417319423},{"lat":41.0906348985565,"lng":-8.39808703421723},{"lat":41.0906335695713,"lng":-8.39842587219933},{"lat":41.0906094971298,"lng":-8.39871034932366},{"lat":41.0906102762914,"lng":-8.39879684197429},{"lat":41.0906111985324,"lng":-8.39889923710657},{"lat":41.0906122170009,"lng":-8.39901230631101},{"lat":41.0906350015705,"lng":-8.39912832777976},{"lat":41.0906337390545,"lng":-8.39955393110926},{"lat":41.0906257554326,"lng":-8.39961828236318},{"lat":41.0906165105345,"lng":-8.3996840662565},{"lat":41.0905696164567,"lng":-8.39979170218452},{"lat":41.0905173159681,"lng":-8.39990051342185},{"lat":41.0904472723162,"lng":-8.40005187530648},{"lat":41.0903914289457,"lng":-8.40017255034071},{"lat":41.0903257493962,"lng":-8.40033727160928},{"lat":41.0902858225245,"lng":-8.4004374059056},{"lat":41.0902572549082,"lng":-8.40052238964761},{"lat":41.0901342307172,"lng":-8.40076420167182},{"lat":41.0901045695254,"lng":-8.40081787642084},{"lat":41.0900704155856,"lng":-8.40086939293197},{"lat":41.0900587805488,"lng":-8.40118593320065},{"lat":41.0900639993062,"lng":-8.40149432253734},{"lat":41.0900856299422,"lng":-8.40163806333269},{"lat":41.0901704440453,"lng":-8.40179373018701},{"lat":41.0902562740812,"lng":-8.40190250704026},{"lat":41.0903788978352,"lng":-8.4020229814048},{"lat":41.0904216776125,"lng":-8.40206672837381},{"lat":41.0905672702568,"lng":-8.40221561322356},{"lat":41.0906856765986,"lng":-8.40234071426246},{"lat":41.0907475352953,"lng":-8.40240607068866},{"lat":41.0908183178104,"lng":-8.40247486657793},{"lat":41.0909412254415,"lng":-8.40259432505674},{"lat":41.0911396884341,"lng":-8.40278378192066},{"lat":41.0913088275567,"lng":-8.40291992781507},{"lat":41.0914056612852,"lng":-8.4030035205107},{"lat":41.0915381442415,"lng":-8.40312272857766},{"lat":41.0917159028239,"lng":-8.40330698958154},{"lat":41.0918960950471,"lng":-8.40349162100387},{"lat":41.0920932394222,"lng":-8.40366524918259},{"lat":41.0922917282666,"lng":-8.4038429354418},{"lat":41.0924356110642,"lng":-8.4039875449947},{"lat":41.0924907909145,"lng":-8.40404300377717},{"lat":41.0926720771153,"lng":-8.40422026207995},{"lat":41.0928566286162,"lng":-8.40438848782737},{"lat":41.0930269111202,"lng":-8.40452189046888},{"lat":41.0932644384969,"lng":-8.40469166414259},{"lat":41.0933099267978,"lng":-8.40472244068746},{"lat":41.0934900641114,"lng":-8.40492898193189},{"lat":41.0937289730968,"lng":-8.40529763945168},{"lat":41.0937643290848,"lng":-8.40534836624374},{"lat":41.0939159590991,"lng":-8.40557538422024},{"lat":41.094059984198,"lng":-8.4058218836477},{"lat":41.0941505162562,"lng":-8.40595759501085},{"lat":41.0943301975384,"lng":-8.40612688048193},{"lat":41.094469192523,"lng":-8.40625410346072},{"lat":41.0946157487925,"lng":-8.40638230814349},{"lat":41.0946693266842,"lng":-8.40643166133868},{"lat":41.0948072067002,"lng":-8.40655866977242},{"lat":41.0949783659788,"lng":-8.40678982448878},{"lat":41.0950197951158,"lng":-8.40684816394566},{"lat":41.0952312037518,"lng":-8.40730385300252},{"lat":41.0952564076101,"lng":-8.4073606425566},{"lat":41.0952821302781,"lng":-8.40741074594393},{"lat":41.0953165646568,"lng":-8.40750777780302},{"lat":41.0953592759455,"lng":-8.40764483758068},{"lat":41.0954550089527,"lng":-8.40786198501769},{"lat":41.0954849534472,"lng":-8.40791781493305},{"lat":41.0955859800987,"lng":-8.40814308037218},{"lat":41.0956736337465,"lng":-8.4083129412674},{"lat":41.0957069906277,"lng":-8.40837069265397},{"lat":41.0957346611229,"lng":-8.40843448724138},{"lat":41.0958533095684,"lng":-8.40870862975378},{"lat":41.0959694333343,"lng":-8.40898204970699},{"lat":41.0960539622646,"lng":-8.40915896602871},{"lat":41.0960848287996,"lng":-8.40922356894572},{"lat":41.0961234461632,"lng":-8.40930288747878},{"lat":41.0961922684511,"lng":-8.4094777918288},{"lat":41.0962518674186,"lng":-8.40959219327031},{"lat":41.0962838939499,"lng":-8.40964195930191},{"lat":41.0963146417044,"lng":-8.40969815237681},{"lat":41.0963504927175,"lng":-8.40976840765245},{"lat":41.0964145032327,"lng":-8.40995829183016},{"lat":41.0965311638196,"lng":-8.41023302908895},{"lat":41.0966004348547,"lng":-8.41033461483684},{"lat":41.0965292694627,"lng":-8.41042120394297},{"lat":41.0965035578705,"lng":-8.4104770386842},{"lat":41.0964691469537,"lng":-8.41052093855583},{"lat":41.0964539065939,"lng":-8.41075321472585},{"lat":41.0964942782856,"lng":-8.41092574002536},{"lat":41.0965589843793,"lng":-8.41105194879836},{"lat":41.0965957618409,"lng":-8.41110959178663},{"lat":41.0967199902057,"lng":-8.41138115114564},{"lat":41.0968586334305,"lng":-8.41165074533148},{"lat":41.0969283860892,"lng":-8.41177062881169},{"lat":41.0969973514649,"lng":-8.41188915965583},{"lat":41.0971405552029,"lng":-8.41213270754098},{"lat":41.0971975786353,"lng":-8.41221488521459},{"lat":41.0972970404978,"lng":-8.41235822235245},{"lat":41.0974644857334,"lng":-8.41255843094732},{"lat":41.0976537499214,"lng":-8.41274504442092},{"lat":41.0977033910762,"lng":-8.41279794248109},{"lat":41.0977466132133,"lng":-8.41284400046491},{"lat":41.0975895268362,"lng":-8.41330983572568},{"lat":41.09658636636,"lng":-8.41628449603349}]},{"zone_id":"prt3","points":[{"lat":41.139948914651,"lng":-8.5756582921291},{"lat":41.1403394973793,"lng":-8.57597639923897},{"lat":41.1415031242075,"lng":-8.57777089569125},{"lat":41.1421264793441,"lng":-8.57956178190636},{"lat":41.1423263415948,"lng":-8.58036874605719},{"lat":41.1457268793388,"lng":-8.57923052983796},{"lat":41.1460491835906,"lng":-8.57953840563627},{"lat":41.1467373838581,"lng":-8.58082697831668},{"lat":41.1469898872302,"lng":-8.58125666625919},{"lat":41.1474509444263,"lng":-8.58153493662397},{"lat":41.1480046657703,"lng":-8.58175270539899},{"lat":41.1484029987078,"lng":-8.58173675742495},{"lat":41.1487365202937,"lng":-8.58164373851915},{"lat":41.1489171590819,"lng":-8.58150202493903},{"lat":41.1491793916882,"lng":-8.58121791930717},{"lat":41.149556976296,"lng":-8.5806933588394},{"lat":41.1498129335994,"lng":-8.58023654969232},{"lat":41.1500399716234,"lng":-8.57988959116522},{"lat":41.1505270697682,"lng":-8.57966656702252},{"lat":41.1510944631693,"lng":-8.57963468914298},{"lat":41.1517312508672,"lng":-8.58012733299472},{"lat":41.1524426690656,"lng":-8.58138561068154},{"lat":41.1525558142985,"lng":-8.58199781491123},{"lat":41.152507091094,"lng":-8.5826394842097},{"lat":41.1525744956918,"lng":-8.58312909115328},{"lat":41.1528963094903,"lng":-8.58355930191325},{"lat":41.1533301252313,"lng":-8.58358123022646},{"lat":41.1538167165231,"lng":-8.58348926434303},{"lat":41.1542142943361,"lng":-8.58313459417344},{"lat":41.1545850009014,"lng":-8.58308173194048},{"lat":41.155508422363,"lng":-8.58330207395416},{"lat":41.156448242708,"lng":-8.58337074250812},{"lat":41.1571028900387,"lng":-8.5832824385722},{"lat":41.1575671748825,"lng":-8.58273529339255},{"lat":41.1621333936855,"lng":-8.5808690317075},{"lat":41.1626585999988,"lng":-8.58072157326528},{"lat":41.1634663077999,"lng":-8.58068303593998},{"lat":41.1639119315604,"lng":-8.58052104515807},{"lat":41.1642237407458,"lng":-8.58064314399528},{"lat":41.1643702853024,"lng":-8.58086812335464},{"lat":41.1644941129044,"lng":-8.58105822831818},{"lat":41.1646430139813,"lng":-8.58143838774917},{"lat":41.1649324995256,"lng":-8.58180675394962},{"lat":41.1658864616144,"lng":-8.5825476850618},{"lat":41.1663008219487,"lng":-8.58294803731663},{"lat":41.1665984497941,"lng":-8.58365337444105},{"lat":41.1667779673253,"lng":-8.58473404687678},{"lat":41.1668904192221,"lng":-8.58579585593893},{"lat":41.1671043257911,"lng":-8.5866043735008},{"lat":41.1677828737511,"lng":-8.58800367702316},{"lat":41.1715376806776,"lng":-8.59382470043685},{"lat":41.1719958801181,"lng":-8.59480649798753},{"lat":41.1721315666511,"lng":-8.59554138898274},{"lat":41.1721056270616,"lng":-8.59624456107593},{"lat":41.1715398222715,"lng":-8.59902338132957},{"lat":41.1713732349904,"lng":-8.60021483010537},{"lat":41.1710251468455,"lng":-8.60057931317141},{"lat":41.1711785367835,"lng":-8.60262928078039},{"lat":41.1711289469366,"lng":-8.60345459103232},{"lat":41.1710569971132,"lng":-8.60409625847338},{"lat":41.171049025268,"lng":-8.60463731360903},{"lat":41.1712370785264,"lng":-8.60525960062148},{"lat":41.1724775191063,"lng":-8.607042216504},{"lat":41.1723134149919,"lng":-8.60897833628369},{"lat":41.1723458393663,"lng":-8.61025119389457},{"lat":41.1725082818488,"lng":-8.61077330006321},{"lat":41.1745170063708,"lng":-8.61115490900746},{"lat":41.1746483783905,"lng":-8.61289903041225},{"lat":41.1745277321604,"lng":-8.61412141860032},{"lat":41.174721702463,"lng":-8.61422669860832},{"lat":41.1761419905197,"lng":-8.61455084853734},{"lat":41.1776006106532,"lng":-8.61485199635611},{"lat":41.1794964151,"lng":-8.61458813556561},{"lat":41.1808935256339,"lng":-8.61446797112279},{"lat":41.183441922869,"lng":-8.61413804399977},{"lat":41.183695246575,"lng":-8.61284882675119},{"lat":41.1838429592593,"lng":-8.61229811407103},{"lat":41.1843279745594,"lng":-8.61105155503136},{"lat":41.1847333935301,"lng":-8.61000952932108},{"lat":41.1848781994225,"lng":-8.60963733385406},{"lat":41.1849883501274,"lng":-8.60828120658883},{"lat":41.1849676768963,"lng":-8.60743703494867},{"lat":41.1846781754553,"lng":-8.6055985266939},{"lat":41.1842435104375,"lng":-8.60361182613143},{"lat":41.1839600469016,"lng":-8.6023163130343},{"lat":41.183808101084,"lng":-8.60162190732838},{"lat":41.1832961097391,"lng":-8.59950746089051},{"lat":41.1828813449457,"lng":-8.59788352167647},{"lat":41.1825280771195,"lng":-8.59650045024091},{"lat":41.1824399196317,"lng":-8.59618252878179},{"lat":41.182184535331,"lng":-8.59526156170956},{"lat":41.1816199808261,"lng":-8.5936437371898},{"lat":41.1807642701122,"lng":-8.59163498927675},{"lat":41.1803650420823,"lng":-8.59052701072525},{"lat":41.1801906176347,"lng":-8.58983826059556},{"lat":41.1800600993701,"lng":-8.58876704744848},{"lat":41.1800485873193,"lng":-8.58770044660429},{"lat":41.1802769201686,"lng":-8.58661471127986},{"lat":41.1808933859287,"lng":-8.58563953878859},{"lat":41.1815449902676,"lng":-8.58471656437406},{"lat":41.181798785185,"lng":-8.58412061845497},{"lat":41.1816872017519,"lng":-8.58355202331571},{"lat":41.1813714579669,"lng":-8.58304682066402},{"lat":41.1810757005337,"lng":-8.58231505074296},{"lat":41.1809054455686,"lng":-8.5819706601608},{"lat":41.1808118130613,"lng":-8.58128631747989},{"lat":41.1809151939545,"lng":-8.580314670871},{"lat":41.1810415705297,"lng":-8.5791267858716},{"lat":41.1812213052581,"lng":-8.57743715126074},{"lat":41.1813043005894,"lng":-8.57639535930634},{"lat":41.1812241831973,"lng":-8.57577358814504},{"lat":41.1809517988848,"lng":-8.57517797616376},{"lat":41.1804063938591,"lng":-8.57406416392483},{"lat":41.1803386231366,"lng":-8.57347271592017},{"lat":41.1803611661882,"lng":-8.57245704559744},{"lat":41.1802990341563,"lng":-8.57125067764485},{"lat":41.1800678304774,"lng":-8.57088154716688},{"lat":41.1794518201679,"lng":-8.56996191293345},{"lat":41.1786503354669,"lng":-8.56905023158889},{"lat":41.1779789885479,"lng":-8.56818539924072},{"lat":41.1770806427874,"lng":-8.56691684431927},{"lat":41.1764274415825,"lng":-8.56596099289179},{"lat":41.1760262069621,"lng":-8.56556437525337},{"lat":41.1756056734561,"lng":-8.56564271258198},{"lat":41.1741763143807,"lng":-8.56720322786655},{"lat":41.1736907109558,"lng":-8.56733335788768},{"lat":41.1728675529925,"lng":-8.56700969130684},{"lat":41.1717999577083,"lng":-8.56665558969618},{"lat":41.1713032376627,"lng":-8.56646323412942},{"lat":41.1694329918366,"lng":-8.56563238923089},{"lat":41.1681380984773,"lng":-8.56502402108388},{"lat":41.1670605380314,"lng":-8.56449677846598},{"lat":41.1661251422662,"lng":-8.56460755162539},{"lat":41.1654533229185,"lng":-8.56484815464578},{"lat":41.1650835003012,"lng":-8.56502452673401},{"lat":41.1644814557249,"lng":-8.56531603723422},{"lat":41.1631154749122,"lng":-8.56514093851909},{"lat":41.1625229551166,"lng":-8.56448029959937},{"lat":41.1617150923648,"lng":-8.5628347916191},{"lat":41.1609241543947,"lng":-8.56160559866198},{"lat":41.1599087396692,"lng":-8.56103592982601},{"lat":41.1589667763421,"lng":-8.56061363424224},{"lat":41.1582629609195,"lng":-8.56040658252332},{"lat":41.1577583794594,"lng":-8.56049860788897},{"lat":41.1568987530266,"lng":-8.55866689862197},{"lat":41.1570377975383,"lng":-8.55856305812414},{"lat":41.1576266975128,"lng":-8.55823857027273},{"lat":41.1578802612941,"lng":-8.55804783073111},{"lat":41.1581422969175,"lng":-8.55770240503255},{"lat":41.1584494394287,"lng":-8.55712411174038},{"lat":41.1585386270649,"lng":-8.55693753438026},{"lat":41.1589765784315,"lng":-8.55602654520412},{"lat":41.1593912093061,"lng":-8.55522413217822},{"lat":41.1593509402534,"lng":-8.55500012490378},{"lat":41.1597733861871,"lng":-8.55370867116856},{"lat":41.1587928161945,"lng":-8.55346410241563},{"lat":41.1581821812489,"lng":-8.55300446002426},{"lat":41.1573560855234,"lng":-8.55238265009386},{"lat":41.156006345541,"lng":-8.55213575272067},{"lat":41.1545066910641,"lng":-8.55266661034053},{"lat":41.1540228413798,"lng":-8.55283788134322},{"lat":41.1523938164052,"lng":-8.55509092403935},{"lat":41.1518496287918,"lng":-8.5561595822423},{"lat":41.1502183322174,"lng":-8.55900808218663},{"lat":41.1494922550588,"lng":-8.56055198680719},{"lat":41.1497579382616,"lng":-8.56174496529766},{"lat":41.1495760780706,"lng":-8.56222027552182},{"lat":41.1485856310976,"lng":-8.56221380002422},{"lat":41.1477739298572,"lng":-8.56256585706412},{"lat":41.1463288201317,"lng":-8.56374758816913},{"lat":41.1473913475462,"lng":-8.56851937998361},{"lat":41.1463086015994,"lng":-8.56910779076095},{"lat":41.1448652373805,"lng":-8.56981289993442},{"lat":41.1431481148212,"lng":-8.57146905713986},{"lat":41.139948914651,"lng":-8.5756582921291}]},{"zone_id":"vng1","points":[{"lat":41.1468643752968,"lng":-8.6444842755015},{"lat":41.1470882899835,"lng":-8.6422525775009},{"lat":41.1472762286378,"lng":-8.6409798385859},{"lat":41.1474595034018,"lng":-8.63975392586331},{"lat":41.1474099871313,"lng":-8.63801387932725},{"lat":41.1470319333398,"lng":-8.63646001195468},{"lat":41.1465779310471,"lng":-8.63545884434667},{"lat":41.1454172846944,"lng":-8.63318674648564},{"lat":41.1445047298682,"lng":-8.63081446301544},{"lat":41.1440391458299,"lng":-8.62933249702638},{"lat":41.143821304433,"lng":-8.6274571253132},{"lat":41.1434704291305,"lng":-8.62514907318385},{"lat":41.1425796187829,"lng":-8.62304064632388},{"lat":41.1404287898021,"lng":-8.62064244523597},{"lat":41.1395354513104,"lng":-8.61896832719624},{"lat":41.1392145604282,"lng":-8.61771464244233},{"lat":41.1390062591768,"lng":-8.61634409733114},{"lat":41.1390147556855,"lng":-8.61431937056088},{"lat":41.1394778823971,"lng":-8.61122600543434},{"lat":41.1406681290033,"lng":-8.60647034050454},{"lat":41.1411300703752,"lng":-8.6036150518287},{"lat":41.1412332337315,"lng":-8.60039983996631},{"lat":41.1405216100516,"lng":-8.59825083175782},{"lat":41.1392692237691,"lng":-8.59621716315491},{"lat":41.138640294295,"lng":-8.5948394047351},{"lat":41.1384574669531,"lng":-8.59329286596379},{"lat":41.1388381245256,"lng":-8.59144992371173},{"lat":41.1400219052563,"lng":-8.58812318652882},{"lat":41.1412904725998,"lng":-8.58610711677605},{"lat":41.1419816368873,"lng":-8.58485497934995},{"lat":41.1425595971982,"lng":-8.58311273905078},{"lat":41.1424810784184,"lng":-8.58099353239148},{"lat":41.1423263415948,"lng":-8.58036874605719},{"lat":41.1421264793441,"lng":-8.57956178190636},{"lat":41.1415031242075,"lng":-8.57777089569125},{"lat":41.1403394973793,"lng":-8.57597639923897},{"lat":41.139948914651,"lng":-8.5756582921291},{"lat":41.1382749782067,"lng":-8.57429501883738},{"lat":41.1378250780675,"lng":-8.57417280665866},{"lat":41.1360037197412,"lng":-8.57952015466846},{"lat":41.1344492700454,"lng":-8.58558353891755},{"lat":41.1325324382073,"lng":-8.59212048105904},{"lat":41.1307826077738,"lng":-8.59632595533559},{"lat":41.1306422898928,"lng":-8.59646132746471},{"lat":41.1270861419965,"lng":-8.59683381266752},{"lat":41.1241755324404,"lng":-8.59720477815564},{"lat":41.1235135759413,"lng":-8.59574886495238},{"lat":41.1165075774475,"lng":-8.59141321798148},{"lat":41.1149768838268,"lng":-8.59140254404973},{"lat":41.1147048610959,"lng":-8.59187689267228},{"lat":41.1121832424762,"lng":-8.59197834937529},{"lat":41.1105957396223,"lng":-8.59157612494239},{"lat":41.1084180291464,"lng":-8.59442510128477},{"lat":41.1072432068762,"lng":-8.59477487163601},{"lat":41.1048775301907,"lng":-8.5950224908652},{"lat":41.1031643585695,"lng":-8.59560565813672},{"lat":41.1006532368245,"lng":-8.59686919631465},{"lat":41.1027886752761,"lng":-8.60024725948748},{"lat":41.1025446249284,"lng":-8.60095673460691},{"lat":41.1031173111626,"lng":-8.60715224620546},{"lat":41.1037713312092,"lng":-8.6105926895489},{"lat":41.1047033728083,"lng":-8.61549670755224},{"lat":41.1059413332893,"lng":-8.62086283528139},{"lat":41.1075406236919,"lng":-8.62499922020721},{"lat":41.108008617987,"lng":-8.62504222299954},{"lat":41.1088964260793,"lng":-8.62582163451438},{"lat":41.1089558444383,"lng":-8.62691923592165},{"lat":41.1127269520576,"lng":-8.62679052187585},{"lat":41.1141540625106,"lng":-8.62730225952186},{"lat":41.1159163660219,"lng":-8.62929566066596},{"lat":41.117299849872,"lng":-8.63186741830258},{"lat":41.1188396435201,"lng":-8.6340223390564},{"lat":41.1201951873702,"lng":-8.63549528960567},{"lat":41.1210533819702,"lng":-8.63590571364513},{"lat":41.1219971835426,"lng":-8.63611331398377},{"lat":41.1248067979274,"lng":-8.63607193270617},{"lat":41.1252918082193,"lng":-8.63604991623116},{"lat":41.1282972638259,"lng":-8.63590788366947},{"lat":41.132449306687,"lng":-8.63571366016574},{"lat":41.1351026494871,"lng":-8.63563644941913},{"lat":41.1358335697332,"lng":-8.63841198639874},{"lat":41.136803631782,"lng":-8.64016465181845},{"lat":41.1382711098384,"lng":-8.64255311896228},{"lat":41.1393111683793,"lng":-8.64234079316612},{"lat":41.140739921642,"lng":-8.64581443020365},{"lat":41.1443115400709,"lng":-8.64440945113626},{"lat":41.1468643752968,"lng":-8.6444842755015}]},{"zone_id":"vng2","points":[{"lat":41.1147778565923,"lng":-8.54224237476533},{"lat":41.112371048217,"lng":-8.54030091550241},{"lat":41.1085868513151,"lng":-8.5380433659677},{"lat":41.1086010054484,"lng":-8.53821342737145},{"lat":41.1094454667943,"lng":-8.55117484851976},{"lat":41.1091285285777,"lng":-8.55649990211834},{"lat":41.1086528893335,"lng":-8.55944076669029},{"lat":41.1073847579031,"lng":-8.56269933381333},{"lat":41.1051176298744,"lng":-8.56571194516766},{"lat":41.1029863581502,"lng":-8.56643957564383},{"lat":41.102285325421,"lng":-8.56595961754982},{"lat":41.1015773535307,"lng":-8.56605496514331},{"lat":41.1006070237514,"lng":-8.56661203660699},{"lat":41.1001439747319,"lng":-8.56734292687339},{"lat":41.1001384873509,"lng":-8.56798523866135},{"lat":41.1002397724453,"lng":-8.56945311242161},{"lat":41.0993246936079,"lng":-8.57092993526233},{"lat":41.0981252542169,"lng":-8.57178899658966},{"lat":41.0953024827548,"lng":-8.57244610904099},{"lat":41.0915852296876,"lng":-8.57374038053339},{"lat":41.0922083303142,"lng":-8.57559105929429},{"lat":41.0926268199928,"lng":-8.57766853789374},{"lat":41.0941881720552,"lng":-8.58265761244937},{"lat":41.0946769836994,"lng":-8.58606491879814},{"lat":41.0946897069734,"lng":-8.58862761398624},{"lat":41.0954014556572,"lng":-8.58999890922809},{"lat":41.0981306650272,"lng":-8.59342769096835},{"lat":41.1006532368245,"lng":-8.59686919631465},{"lat":41.1031643585695,"lng":-8.59560565813672},{"lat":41.1048775301907,"lng":-8.5950224908652},{"lat":41.1072432068762,"lng":-8.59477487163601},{"lat":41.1084180291464,"lng":-8.59442510128477},{"lat":41.1105957396223,"lng":-8.59157612494239},{"lat":41.1121832424762,"lng":-8.59197834937529},{"lat":41.1147048610959,"lng":-8.59187689267228},{"lat":41.1149768838268,"lng":-8.59140254404973},{"lat":41.1165075774475,"lng":-8.59141321798148},{"lat":41.1235135759413,"lng":-8.59574886495238},{"lat":41.1241755324404,"lng":-8.59720477815564},{"lat":41.1270861419965,"lng":-8.59683381266752},{"lat":41.1306422898928,"lng":-8.59646132746471},{"lat":41.1307826077738,"lng":-8.59632595533559},{"lat":41.1325324382073,"lng":-8.59212048105904},{"lat":41.1344492700454,"lng":-8.58558353891755},{"lat":41.1360037197412,"lng":-8.57952015466846},{"lat":41.1378250780675,"lng":-8.57417280665866},{"lat":41.1377257259315,"lng":-8.57414635975049},{"lat":41.1361215539414,"lng":-8.5732655946634},{"lat":41.1340503666224,"lng":-8.5723610952149},{"lat":41.1315145905407,"lng":-8.5717551911651},{"lat":41.1291927104296,"lng":-8.57113773274333},{"lat":41.126049883309,"lng":-8.56885421922917},{"lat":41.1244386030943,"lng":-8.56634287237586},{"lat":41.123999139043,"lng":-8.56348209708882},{"lat":41.1241059836523,"lng":-8.56033708225287},{"lat":41.1240125984161,"lng":-8.55622269133534},{"lat":41.1227051700073,"lng":-8.55017470410143},{"lat":41.1219071675575,"lng":-8.54875747530509},{"lat":41.1187402323614,"lng":-8.54571496644131},{"lat":41.1159967544012,"lng":-8.54319163227059},{"lat":41.1147778565923,"lng":-8.54224237476533}]},{"zone_id":"gdm4","points":[{"lat":41.1085868513151,"lng":-8.5380433659677},{"lat":41.112371048217,"lng":-8.54030091550241},{"lat":41.1147778565923,"lng":-8.54224237476533},{"lat":41.1168798201536,"lng":-8.5354752117508},{"lat":41.1192744345378,"lng":-8.53046522593166},{"lat":41.1207931511166,"lng":-8.52611642802048},{"lat":41.1247630709264,"lng":-8.52111514037209},{"lat":41.1290366805172,"lng":-8.51551329523038},{"lat":41.1295812910624,"lng":-8.51480163183858},{"lat":41.1296728864799,"lng":-8.51207302985341},{"lat":41.1284205302097,"lng":-8.50956498873679},{"lat":41.1266317405145,"lng":-8.50586315700584},{"lat":41.1259221818122,"lng":-8.50252483807563},{"lat":41.1269406511456,"lng":-8.49371837747378},{"lat":41.1275869107343,"lng":-8.48860127907919},{"lat":41.1282455872257,"lng":-8.47931613438678},{"lat":41.1293294478724,"lng":-8.47055369385253},{"lat":41.1290067395651,"lng":-8.47041550523638},{"lat":41.1282975731755,"lng":-8.47044365999588},{"lat":41.1282395707579,"lng":-8.4704459627617},{"lat":41.1274004948889,"lng":-8.47055679973695},{"lat":41.1271492198711,"lng":-8.47048817982652},{"lat":41.1263858044227,"lng":-8.47027970442407},{"lat":41.1262502618194,"lng":-8.47022522065518},{"lat":41.1261523298021,"lng":-8.47018585528075},{"lat":41.1260756498866,"lng":-8.47016274094053},{"lat":41.1259967815153,"lng":-8.47013896697224},{"lat":41.1258140379101,"lng":-8.47010025177023},{"lat":41.1256080404817,"lng":-8.47002586095049},{"lat":41.1253741558167,"lng":-8.46988649757367},{"lat":41.1251464251635,"lng":-8.46976782220398},{"lat":41.1249129089153,"lng":-8.4696575468301},{"lat":41.1246923205504,"lng":-8.46955948331835},{"lat":41.1245527321694,"lng":-8.46936091763714},{"lat":41.1244964974664,"lng":-8.46926599015875},{"lat":41.124460782557,"lng":-8.46920570144965},{"lat":41.1243825403439,"lng":-8.46911921998074},{"lat":41.1242899817557,"lng":-8.46901691455819},{"lat":41.1242138585389,"lng":-8.46895697000431},{"lat":41.1240749168144,"lng":-8.46884755834633},{"lat":41.1238501807347,"lng":-8.46874590493564},{"lat":41.1238048319062,"lng":-8.46872523632842},{"lat":41.1236915699266,"lng":-8.46867352118977},{"lat":41.1234714829005,"lng":-8.46858975717878},{"lat":41.1232379897712,"lng":-8.4685002025821},{"lat":41.1230046605506,"lng":-8.4684180358499},{"lat":41.1229044036963,"lng":-8.46833845449488},{"lat":41.1227226310893,"lng":-8.46823798334501},{"lat":41.1224880052573,"lng":-8.46804518332362},{"lat":41.1224061968797,"lng":-8.46797784720262},{"lat":41.1222796982325,"lng":-8.46787372711107},{"lat":41.1221235970612,"lng":-8.46776944958113},{"lat":41.1219941860008,"lng":-8.46774509369544},{"lat":41.1217345693174,"lng":-8.46762921488376},{"lat":41.1215327558352,"lng":-8.46748375089771},{"lat":41.1213253429613,"lng":-8.46740695953361},{"lat":41.1212478401275,"lng":-8.46742835332701},{"lat":41.1210157995946,"lng":-8.46749075305377},{"lat":41.1204797807934,"lng":-8.4730771422205},{"lat":41.1192967722692,"lng":-8.48512399381639},{"lat":41.1161250051814,"lng":-8.49165540035887},{"lat":41.1137222308774,"lng":-8.49370295057617},{"lat":41.1125343765221,"lng":-8.49504384953051},{"lat":41.1115047209677,"lng":-8.49569828051128},{"lat":41.1102622193721,"lng":-8.49483770680996},{"lat":41.1084610202837,"lng":-8.49494680535024},{"lat":41.1058490733771,"lng":-8.49517046158004},{"lat":41.1049524205743,"lng":-8.49397507591314},{"lat":41.103874175375,"lng":-8.49325488902302},{"lat":41.1024312658877,"lng":-8.49396118378991},{"lat":41.1004496071132,"lng":-8.49418833505015},{"lat":41.1000398409341,"lng":-8.49412902912289},{"lat":41.0983311937642,"lng":-8.49388174292775},{"lat":41.0981936830805,"lng":-8.49572329713333},{"lat":41.0958461813283,"lng":-8.49773375120779},{"lat":41.0953895119947,"lng":-8.49975464396632},{"lat":41.0942159213895,"lng":-8.50070026381402},{"lat":41.0939412025599,"lng":-8.50212699777373},{"lat":41.0933940293197,"lng":-8.50426631115052},{"lat":41.0917701952004,"lng":-8.50520926562882},{"lat":41.090860853146,"lng":-8.50794150166592},{"lat":41.0908514667539,"lng":-8.51079786429244},{"lat":41.0903029480967,"lng":-8.51329404665976},{"lat":41.0884926044116,"lng":-8.51613985991302},{"lat":41.087851121637,"lng":-8.51946844518349},{"lat":41.0883727398084,"lng":-8.52239889625177},{"lat":41.0885174062281,"lng":-8.52247677036953},{"lat":41.0890950135677,"lng":-8.5228268868528},{"lat":41.0895178991518,"lng":-8.5230010214366},{"lat":41.0899541782887,"lng":-8.52312762794096},{"lat":41.0902214223654,"lng":-8.52319924146932},{"lat":41.090377475371,"lng":-8.52324105907094},{"lat":41.0904911564597,"lng":-8.52327138022469},{"lat":41.0908486310603,"lng":-8.52340430462348},{"lat":41.0913163915588,"lng":-8.5236197765372},{"lat":41.0918279241414,"lng":-8.52385966743192},{"lat":41.0923991212625,"lng":-8.5241081319306},{"lat":41.0929667432469,"lng":-8.5243233791873},{"lat":41.093358519058,"lng":-8.52448199027956},{"lat":41.093854277421,"lng":-8.52464610781826},{"lat":41.0945433506756,"lng":-8.52491873540143},{"lat":41.0950301219365,"lng":-8.52512994173154},{"lat":41.0956342217023,"lng":-8.52539326452236},{"lat":41.0962480861359,"lng":-8.52567165038123},{"lat":41.0967383529749,"lng":-8.52588741060534},{"lat":41.097332939723,"lng":-8.52622032265078},{"lat":41.097990118937,"lng":-8.52649708098898},{"lat":41.0985130775685,"lng":-8.52671042957936},{"lat":41.0990518166506,"lng":-8.52696970406146},{"lat":41.0994371430569,"lng":-8.52717068491282},{"lat":41.0998034007321,"lng":-8.52736524163694},{"lat":41.1001463255884,"lng":-8.52764215428962},{"lat":41.1005443279876,"lng":-8.528004513519},{"lat":41.1009713844164,"lng":-8.52845489646016},{"lat":41.1013661249178,"lng":-8.52887509592027},{"lat":41.1016640513261,"lng":-8.52929983967474},{"lat":41.1020271698295,"lng":-8.5297505694468},{"lat":41.1021470717553,"lng":-8.52989032946705},{"lat":41.1024446317507,"lng":-8.53023793887171},{"lat":41.102954947084,"lng":-8.53092776751118},{"lat":41.1036492699542,"lng":-8.53183502800149},{"lat":41.1040968564501,"lng":-8.53251331150504},{"lat":41.1045165291857,"lng":-8.5332158412586},{"lat":41.1049512400983,"lng":-8.53401965606255},{"lat":41.1051313448932,"lng":-8.53441765214285},{"lat":41.1054903708918,"lng":-8.53506327875905},{"lat":41.1057694743728,"lng":-8.5355590258455},{"lat":41.1058207677104,"lng":-8.53565485883221},{"lat":41.105990344609,"lng":-8.53597168478737},{"lat":41.1060577111429,"lng":-8.5360629876119},{"lat":41.1062338820519,"lng":-8.53630175455824},{"lat":41.1063286436525,"lng":-8.53643018630746},{"lat":41.1067453620689,"lng":-8.53681500590013},{"lat":41.1071352487874,"lng":-8.53714740698212},{"lat":41.1075096588873,"lng":-8.53742507330555},{"lat":41.107859227996,"lng":-8.53763779771951},{"lat":41.1080948558609,"lng":-8.53778118510196},{"lat":41.1085868513151,"lng":-8.5380433659677}]},{"zone_id":"vng5","points":[{"lat":41.1239059504138,"lng":-8.66912696144234},{"lat":41.1240767163121,"lng":-8.66919611486678},{"lat":41.1241260915872,"lng":-8.66922759974483},{"lat":41.1241923391165,"lng":-8.66915644756137},{"lat":41.1241088771244,"lng":-8.66905740358781},{"lat":41.1241032930212,"lng":-8.66892339498414},{"lat":41.1240749621433,"lng":-8.66879979445743},{"lat":41.1240456491299,"lng":-8.66875192445603},{"lat":41.1240416371213,"lng":-8.66862816302882},{"lat":41.1241227515318,"lng":-8.66861238830992},{"lat":41.1241483989275,"lng":-8.668499947775},{"lat":41.1243169815701,"lng":-8.66830233288023},{"lat":41.1246154311177,"lng":-8.66819794981262},{"lat":41.1250049106417,"lng":-8.66805655719593},{"lat":41.1255118838564,"lng":-8.66789825252812},{"lat":41.1257567745339,"lng":-8.66788702399202},{"lat":41.1258998047005,"lng":-8.66791760761191},{"lat":41.1260302613887,"lng":-8.66794153437903},{"lat":41.126095811816,"lng":-8.66796159221408},{"lat":41.1261868729042,"lng":-8.66797519672012},{"lat":41.1262731009035,"lng":-8.66800233699238},{"lat":41.1263463783694,"lng":-8.66810380200598},{"lat":41.1264156305189,"lng":-8.66815985764467},{"lat":41.1264156305189,"lng":-8.66815985764467},{"lat":41.1264063556875,"lng":-8.66821849444795},{"lat":41.1263770417399,"lng":-8.66828637072511},{"lat":41.1264700872864,"lng":-8.66843563001648},{"lat":41.1266102126916,"lng":-8.66831637238905},{"lat":41.1267968650233,"lng":-8.66858941701176},{"lat":41.1268635958824,"lng":-8.6685882907327},{"lat":41.12691874636,"lng":-8.66834687524296},{"lat":41.126930040515,"lng":-8.66810748446154},{"lat":41.1269889291187,"lng":-8.66801078808562},{"lat":41.1272113556105,"lng":-8.66796984396154},{"lat":41.1275144252696,"lng":-8.66803268883909},{"lat":41.1280405878794,"lng":-8.6682215667417},{"lat":41.1281307943666,"lng":-8.66830304239306},{"lat":41.1282456013416,"lng":-8.66855394652008},{"lat":41.1283141382927,"lng":-8.66862952984994},{"lat":41.1287882119926,"lng":-8.66865209485112},{"lat":41.129063795395,"lng":-8.66875973955667},{"lat":41.1292716004575,"lng":-8.66884062587493},{"lat":41.1295233609392,"lng":-8.66905561905053},{"lat":41.1296354771814,"lng":-8.66920777739701},{"lat":41.1297627396569,"lng":-8.66954942243775},{"lat":41.1297452409498,"lng":-8.6697484045642},{"lat":41.1297013678592,"lng":-8.66984891289474},{"lat":41.1295969650523,"lng":-8.67008898303916},{"lat":41.1297079696211,"lng":-8.67007393236889},{"lat":41.129863991445,"lng":-8.66996445182571},{"lat":41.1299178337134,"lng":-8.66981197743917},{"lat":41.1299760509741,"lng":-8.66939598741649},{"lat":41.1300279505693,"lng":-8.66931244716988},{"lat":41.1301115307532,"lng":-8.66930848444547},{"lat":41.1301583228581,"lng":-8.66935329420781},{"lat":41.1301625288771,"lng":-8.66966808946198},{"lat":41.1302583152133,"lng":-8.6699421935688},{"lat":41.1303158543366,"lng":-8.66998172125662},{"lat":41.1304899791845,"lng":-8.66998659848828},{"lat":41.1306642428151,"lng":-8.67007805709004},{"lat":41.1309404769845,"lng":-8.67029481041726},{"lat":41.1310890885857,"lng":-8.67051861370033},{"lat":41.1315249297793,"lng":-8.67034127099712},{"lat":41.1315539780337,"lng":-8.67044559925259},{"lat":41.1315368538479,"lng":-8.67056419738093},{"lat":41.131363893221,"lng":-8.67079120706147},{"lat":41.1313407788631,"lng":-8.67086295134115},{"lat":41.1313844102518,"lng":-8.67096799125378},{"lat":41.1315158749272,"lng":-8.67096775685658},{"lat":41.1319314885349,"lng":-8.67084217955405},{"lat":41.1321037933301,"lng":-8.67075427303846},{"lat":41.1322431837747,"lng":-8.67079364079854},{"lat":41.1323236456176,"lng":-8.67091732505679},{"lat":41.1324223967075,"lng":-8.67098101752278},{"lat":41.1331150012331,"lng":-8.67083388796421},{"lat":41.1333321078121,"lng":-8.67090950469598},{"lat":41.1336416971918,"lng":-8.67079163418669},{"lat":41.1337763675602,"lng":-8.67079988449832},{"lat":41.1340051539703,"lng":-8.67091930911437},{"lat":41.1343311551722,"lng":-8.67089315497589},{"lat":41.1344149985243,"lng":-8.67096732912865},{"lat":41.134534807878,"lng":-8.6712651095658},{"lat":41.1345918516059,"lng":-8.67129320315252},{"lat":41.1348833572937,"lng":-8.67105715196127},{"lat":41.1350968637248,"lng":-8.67076674473939},{"lat":41.1351858841023,"lng":-8.67054320154569},{"lat":41.1352719111672,"lng":-8.67007405044943},{"lat":41.1353738485834,"lng":-8.66993838899867},{"lat":41.1356066598603,"lng":-8.66979427290316},{"lat":41.1356342881518,"lng":-8.66985584150552},{"lat":41.1355393943323,"lng":-8.66998738827421},{"lat":41.1355092709529,"lng":-8.67011303524685},{"lat":41.1355671200442,"lng":-8.67014327788523},{"lat":41.135848386422,"lng":-8.67009401197811},{"lat":41.1359737511646,"lng":-8.67001225690302},{"lat":41.1360135020958,"lng":-8.66994540861081},{"lat":41.1360115134111,"lng":-8.66981211725995},{"lat":41.1359300647115,"lng":-8.66961075714683},{"lat":41.1360209537744,"lng":-8.66948763541156},{"lat":41.1362736423452,"lng":-8.66954127266927},{"lat":41.13661552753,"lng":-8.66948724769249},{"lat":41.1367896065982,"lng":-8.66950189418768},{"lat":41.1368478711857,"lng":-8.66944317595263},{"lat":41.1370492547031,"lng":-8.66910691911617},{"lat":41.1370784713646,"lng":-8.66900174717624},{"lat":41.1369634345845,"lng":-8.66874103640674},{"lat":41.1369717460471,"lng":-8.6682910107566},{"lat":41.1370378170989,"lng":-8.66787074638469},{"lat":41.1371471237177,"lng":-8.66773716794649},{"lat":41.1373813815281,"lng":-8.6675713749494},{"lat":41.1375395414406,"lng":-8.66736946730655},{"lat":41.1377667042235,"lng":-8.66723934163249},{"lat":41.1385998562015,"lng":-8.66705495460219},{"lat":41.139267183641,"lng":-8.66697759731582},{"lat":41.1405288412177,"lng":-8.66703121314852},{"lat":41.1415518985105,"lng":-8.6672334490798},{"lat":41.142158822038,"lng":-8.66748245778792},{"lat":41.1427741304375,"lng":-8.66780657377418},{"lat":41.1430196873605,"lng":-8.66782501449796},{"lat":41.1427622044757,"lng":-8.6630636282963},{"lat":41.1429581578777,"lng":-8.65961095347071},{"lat":41.144060358041,"lng":-8.6548552179485},{"lat":41.145250257943,"lng":-8.65057647904109},{"lat":41.1460601295934,"lng":-8.64878775057835},{"lat":41.1466484271405,"lng":-8.64663671869836},{"lat":41.1468643752968,"lng":-8.6444842755015},{"lat":41.1443115400709,"lng":-8.64440945113626},{"lat":41.140739921642,"lng":-8.64581443020365},{"lat":41.1393111683793,"lng":-8.64234079316612},{"lat":41.1382711098384,"lng":-8.64255311896228},{"lat":41.136803631782,"lng":-8.64016465181845},{"lat":41.1358335697332,"lng":-8.63841198639874},{"lat":41.1351026494871,"lng":-8.63563644941913},{"lat":41.132449306687,"lng":-8.63571366016574},{"lat":41.1282972638259,"lng":-8.63590788366947},{"lat":41.1252918082193,"lng":-8.63604991623116},{"lat":41.1248067979274,"lng":-8.63607193270617},{"lat":41.1228585965168,"lng":-8.6440891191383},{"lat":41.1217770576677,"lng":-8.64431885870801},{"lat":41.118797355511,"lng":-8.64525028143546},{"lat":41.1169833843809,"lng":-8.64641635148598},{"lat":41.1153624027148,"lng":-8.64915053289446},{"lat":41.1139083615726,"lng":-8.65211561414995},{"lat":41.1119904912326,"lng":-8.65805328985726},{"lat":41.1118556343202,"lng":-8.66083561093689},{"lat":41.1116985458066,"lng":-8.66424863147071},{"lat":41.1126809641636,"lng":-8.66414386894151},{"lat":41.1138140011181,"lng":-8.66404980081543},{"lat":41.1147058610416,"lng":-8.66396247047734},{"lat":41.1153113555131,"lng":-8.66409392890407},{"lat":41.1160687487929,"lng":-8.66370128964059},{"lat":41.1169160522856,"lng":-8.66349404855089},{"lat":41.1169887307254,"lng":-8.66353106990966},{"lat":41.1172031076317,"lng":-8.66373045065574},{"lat":41.1173681459926,"lng":-8.66387038376793},{"lat":41.1175545342707,"lng":-8.66402763636241},{"lat":41.1177280259599,"lng":-8.66413155685498},{"lat":41.1177946510992,"lng":-8.66432856284617},{"lat":41.1179152194164,"lng":-8.66452374330937},{"lat":41.1179597002255,"lng":-8.66464114685421},{"lat":41.1180072390505,"lng":-8.66466201129008},{"lat":41.1181173555048,"lng":-8.66462456393593},{"lat":41.1181930042268,"lng":-8.66464267499188},{"lat":41.1182557587994,"lng":-8.66446886379128},{"lat":41.1181907403471,"lng":-8.66435319671498},{"lat":41.1183219764,"lng":-8.66420874910612},{"lat":41.1185477281164,"lng":-8.66442133583406},{"lat":41.1186588886416,"lng":-8.66454761685417},{"lat":41.118911634957,"lng":-8.66478411814348},{"lat":41.1189755143921,"lng":-8.66497074625743},{"lat":41.1190002324203,"lng":-8.6650778716603},{"lat":41.1191746901579,"lng":-8.66512834224609},{"lat":41.1192625110528,"lng":-8.66525824921516},{"lat":41.1193705445533,"lng":-8.66526316973876},{"lat":41.1193523727004,"lng":-8.66510394453162},{"lat":41.1194683407968,"lng":-8.66510678654829},{"lat":41.1195916820105,"lng":-8.66513100699524},{"lat":41.1196643356825,"lng":-8.66519172561874},{"lat":41.1197015565149,"lng":-8.66530228704922},{"lat":41.1196712584179,"lng":-8.66544719402575},{"lat":41.119791508645,"lng":-8.66555533449214},{"lat":41.1199325440385,"lng":-8.6656082774422},{"lat":41.1201237913928,"lng":-8.6657468832192},{"lat":41.1204198432647,"lng":-8.6659844575926},{"lat":41.120588306794,"lng":-8.66612300041368},{"lat":41.120681801272,"lng":-8.66635071649492},{"lat":41.1207592615222,"lng":-8.66657901498825},{"lat":41.1207819523055,"lng":-8.66671495211495},{"lat":41.1208066395262,"lng":-8.66680885954179},{"lat":41.1207923000513,"lng":-8.66698736038372},{"lat":41.120724616301,"lng":-8.66701920152889},{"lat":41.1208658721802,"lng":-8.66725837975611},{"lat":41.1208392156432,"lng":-8.66735544978953},{"lat":41.1208183796872,"lng":-8.66742136692944},{"lat":41.1209801502787,"lng":-8.66748829689142},{"lat":41.1210447403885,"lng":-8.66752120692092},{"lat":41.1211620224178,"lng":-8.66764719509874},{"lat":41.121171448066,"lng":-8.66775074720063},{"lat":41.1211200394732,"lng":-8.66796145152252},{"lat":41.12107532258,"lng":-8.66808897600881},{"lat":41.121125447894,"lng":-8.66819285951719},{"lat":41.1211565006095,"lng":-8.66829183024961},{"lat":41.1212208314133,"lng":-8.66838035270629},{"lat":41.1213272760265,"lng":-8.66835906894135},{"lat":41.121326974222,"lng":-8.66842384938784},{"lat":41.121355027329,"lng":-8.66856827588613},{"lat":41.121431286143,"lng":-8.66862807778766},{"lat":41.1214780744868,"lng":-8.66861655521783},{"lat":41.121558089588,"lng":-8.66864340413924},{"lat":41.1217151550277,"lng":-8.66858073958011},{"lat":41.1220048563506,"lng":-8.6684015058896},{"lat":41.1223124069525,"lng":-8.66829529123132},{"lat":41.1225683890334,"lng":-8.66824164867112},{"lat":41.1232029008809,"lng":-8.66830933298349},{"lat":41.1234126420827,"lng":-8.66836248508412},{"lat":41.1235409293445,"lng":-8.66842652544335},{"lat":41.123620113725,"lng":-8.66863163386051},{"lat":41.1237619396138,"lng":-8.6689399012984},{"lat":41.1237656441176,"lng":-8.6691111707215},{"lat":41.123849607207,"lng":-8.6692570173948},{"lat":41.1240030779701,"lng":-8.66936520335015},{"lat":41.12401702,"lng":-8.66927267035454},{"lat":41.1239059504138,"lng":-8.66912696144234}]},{"zone_id":"vng4","points":[{"lat":41.1248067979274,"lng":-8.63607193270617},{"lat":41.1219971835426,"lng":-8.63611331398377},{"lat":41.1210533819702,"lng":-8.63590571364513},{"lat":41.1201951873702,"lng":-8.63549528960567},{"lat":41.1188396435201,"lng":-8.6340223390564},{"lat":41.117299849872,"lng":-8.63186741830258},{"lat":41.1159163660219,"lng":-8.62929566066596},{"lat":41.1141540625106,"lng":-8.62730225952186},{"lat":41.1127269520576,"lng":-8.62679052187585},{"lat":41.1089558444383,"lng":-8.62691923592165},{"lat":41.1088964260793,"lng":-8.62582163451438},{"lat":41.108008617987,"lng":-8.62504222299954},{"lat":41.1075406236919,"lng":-8.62499922020721},{"lat":41.1059413332893,"lng":-8.62086283528139},{"lat":41.1047033728083,"lng":-8.61549670755224},{"lat":41.1037713312092,"lng":-8.6105926895489},{"lat":41.1033681912756,"lng":-8.61179664740916},{"lat":41.1000257473406,"lng":-8.6143910835357},{"lat":41.0999171803177,"lng":-8.61879456877584},{"lat":41.0999076370169,"lng":-8.61913074783428},{"lat":41.0978942072679,"lng":-8.6191772644016},{"lat":41.0975741090386,"lng":-8.61925338817492},{"lat":41.0956812389177,"lng":-8.61971550616127},{"lat":41.0943281066493,"lng":-8.62030061898357},{"lat":41.0938713342393,"lng":-8.62184454370742},{"lat":41.0924286569092,"lng":-8.62230991003828},{"lat":41.090716362044,"lng":-8.62265423165813},{"lat":41.0888122856744,"lng":-8.62573444194846},{"lat":41.0860025816293,"lng":-8.62999774696424},{"lat":41.0830338060184,"lng":-8.62938031105836},{"lat":41.0802358515799,"lng":-8.63090620112969},{"lat":41.0763610006143,"lng":-8.63159084995506},{"lat":41.072848888833,"lng":-8.6316832423816},{"lat":41.0703850724356,"lng":-8.63179009344031},{"lat":41.0704040606612,"lng":-8.63398002881338},{"lat":41.0699865241568,"lng":-8.63564704241811},{"lat":41.0697378247396,"lng":-8.63948392846325},{"lat":41.0701073662969,"lng":-8.64245980108528},{"lat":41.0701019105036,"lng":-8.6442593414467},{"lat":41.0710609327927,"lng":-8.64916012510524},{"lat":41.0695356087167,"lng":-8.6588389656307},{"lat":41.0697615626421,"lng":-8.65885231501366},{"lat":41.0700893439599,"lng":-8.65867205437373},{"lat":41.0703490197619,"lng":-8.65861737608147},{"lat":41.0705467634653,"lng":-8.65845963702383},{"lat":41.0706954773562,"lng":-8.65842964901274},{"lat":41.0709134203913,"lng":-8.65844198246181},{"lat":41.0711439074896,"lng":-8.65844881786656},{"lat":41.0713610485278,"lng":-8.65840105860291},{"lat":41.0715131648601,"lng":-8.65843356422979},{"lat":41.0718443222637,"lng":-8.65869655864865},{"lat":41.0719161065471,"lng":-8.65873247205384},{"lat":41.0720295115441,"lng":-8.65874456254886},{"lat":41.0720997430346,"lng":-8.65876463903819},{"lat":41.0721690366005,"lng":-8.65875436858988},{"lat":41.0722472864079,"lng":-8.65875404355038},{"lat":41.0723348447059,"lng":-8.65872736967929},{"lat":41.072432042764,"lng":-8.6587185124785},{"lat":41.0726303392647,"lng":-8.65867642711822},{"lat":41.0727249313909,"lng":-8.65866635878646},{"lat":41.0728167587831,"lng":-8.65869089010845},{"lat":41.0729883251147,"lng":-8.65876353374187},{"lat":41.0731132691531,"lng":-8.65883140977829},{"lat":41.0732346550865,"lng":-8.65886974213405},{"lat":41.073304491349,"lng":-8.65891789526669},{"lat":41.0734377212699,"lng":-8.65894537815584},{"lat":41.0735630212784,"lng":-8.65891568142742},{"lat":41.0736595175989,"lng":-8.65886445688348},{"lat":41.0737957640663,"lng":-8.65880343007334},{"lat":41.0739582794667,"lng":-8.65870977270087},{"lat":41.0741016613177,"lng":-8.65866403570874},{"lat":41.0742142852597,"lng":-8.65864887175384},{"lat":41.0742836316571,"lng":-8.6586864358813},{"lat":41.0744097102748,"lng":-8.65868340135108},{"lat":41.0745089025145,"lng":-8.65865218504479},{"lat":41.0746206445785,"lng":-8.65859441208372},{"lat":41.0747450494797,"lng":-8.65858327181578},{"lat":41.0748403144992,"lng":-8.65860319008314},{"lat":41.0749512813837,"lng":-8.65859717535519},{"lat":41.0751176125689,"lng":-8.6586328955247},{"lat":41.0753524568177,"lng":-8.65867165725769},{"lat":41.0754524253519,"lng":-8.65864782749069},{"lat":41.075567962203,"lng":-8.65860745883958},{"lat":41.0756817028181,"lng":-8.65854506626499},{"lat":41.0757772428003,"lng":-8.65850596705478},{"lat":41.0759086625598,"lng":-8.65851558209626},{"lat":41.0760728057104,"lng":-8.65849832919271},{"lat":41.0762465007312,"lng":-8.65847960989927},{"lat":41.0763709108052,"lng":-8.65844752347509},{"lat":41.0765572542593,"lng":-8.65844008946948},{"lat":41.0768088508577,"lng":-8.65855657818773},{"lat":41.0768628336292,"lng":-8.65856593342004},{"lat":41.0770801040502,"lng":-8.65854815430996},{"lat":41.0771292565132,"lng":-8.65851237392499},{"lat":41.0772065788269,"lng":-8.65845848594383},{"lat":41.077259595297,"lng":-8.65844368233462},{"lat":41.077561913764,"lng":-8.65853439191274},{"lat":41.0777093548052,"lng":-8.65854723410777},{"lat":41.0777951134678,"lng":-8.65849925148476},{"lat":41.0779464542592,"lng":-8.65850545528248},{"lat":41.0780637527479,"lng":-8.65845284376004},{"lat":41.0782032984953,"lng":-8.65845919922616},{"lat":41.0783052243686,"lng":-8.65844084980771},{"lat":41.078388983817,"lng":-8.65839761178874},{"lat":41.0784908435068,"lng":-8.6584135406671},{"lat":41.0786103662024,"lng":-8.65842580053536},{"lat":41.0786898719717,"lng":-8.6583884731104},{"lat":41.0787987117674,"lng":-8.65839315338362},{"lat":41.0789155070285,"lng":-8.65845168001705},{"lat":41.07899946114,"lng":-8.65844509635483},{"lat":41.0790769076354,"lng":-8.65842345890025},{"lat":41.0791397226899,"lng":-8.65847096791521},{"lat":41.0792208236839,"lng":-8.65845781146658},{"lat":41.0792975162788,"lng":-8.65842498307551},{"lat":41.0795468617458,"lng":-8.65842305303016},{"lat":41.0797776977475,"lng":-8.65839312571813},{"lat":41.0799795765328,"lng":-8.65829489269782},{"lat":41.080175114635,"lng":-8.65830466804901},{"lat":41.0802576072559,"lng":-8.65834150246276},{"lat":41.0804524297113,"lng":-8.65827118147193},{"lat":41.0806742798681,"lng":-8.65823665952928},{"lat":41.0809230452875,"lng":-8.6581843844237},{"lat":41.0813891021358,"lng":-8.65803149584869},{"lat":41.0815333211192,"lng":-8.65799956443781},{"lat":41.0818471121048,"lng":-8.65792507087993},{"lat":41.0823597378834,"lng":-8.65788656526105},{"lat":41.0825657398972,"lng":-8.65789237352169},{"lat":41.0827952661034,"lng":-8.65791229499806},{"lat":41.0835515471336,"lng":-8.65791571565848},{"lat":41.0840488188043,"lng":-8.65790743140036},{"lat":41.0843188600177,"lng":-8.65792851499557},{"lat":41.0846023886979,"lng":-8.65793363282134},{"lat":41.0849077851531,"lng":-8.65792333709266},{"lat":41.0851409034002,"lng":-8.65790592000437},{"lat":41.0853076170021,"lng":-8.65787773750815},{"lat":41.085681505441,"lng":-8.65783431102404},{"lat":41.0859193444139,"lng":-8.6578089573226},{"lat":41.0860654737706,"lng":-8.65779167616969},{"lat":41.0862285876215,"lng":-8.65776393645035},{"lat":41.0864220959177,"lng":-8.65776310097685},{"lat":41.0866872244384,"lng":-8.65769750319321},{"lat":41.0870078769037,"lng":-8.65765745389028},{"lat":41.0872998453892,"lng":-8.65761039164818},{"lat":41.0875698446632,"lng":-8.65764063935422},{"lat":41.0878824413923,"lng":-8.65759064883406},{"lat":41.0880936303056,"lng":-8.65768255060449},{"lat":41.0882522654603,"lng":-8.65782568205264},{"lat":41.0884226628965,"lng":-8.65797688894971},{"lat":41.088589074104,"lng":-8.65801487705399},{"lat":41.0887224269262,"lng":-8.6580165345336},{"lat":41.0888746920967,"lng":-8.65799656876539},{"lat":41.0890542922148,"lng":-8.65792754617485},{"lat":41.0892302854584,"lng":-8.65783849565038},{"lat":41.0893818271624,"lng":-8.65774115540868},{"lat":41.0895498564131,"lng":-8.65762205206617},{"lat":41.0896862705177,"lng":-8.65756374987738},{"lat":41.0898042710275,"lng":-8.6575357677199},{"lat":41.0899303329789,"lng":-8.65753630195512},{"lat":41.0901131123608,"lng":-8.65751919213106},{"lat":41.0903831286916,"lng":-8.65752694119039},{"lat":41.0907171209251,"lng":-8.65758257533221},{"lat":41.0908588008828,"lng":-8.657673566853},{"lat":41.0909912396453,"lng":-8.65769687846235},{"lat":41.0911186965283,"lng":-8.65776657381225},{"lat":41.0914233641743,"lng":-8.65791576729056},{"lat":41.0915650660107,"lng":-8.65800080806907},{"lat":41.0917601141492,"lng":-8.65811663068726},{"lat":41.091978107793,"lng":-8.65825644139584},{"lat":41.0922690232855,"lng":-8.6584386113786},{"lat":41.0927315087214,"lng":-8.65853515285986},{"lat":41.0930105079395,"lng":-8.65856678266996},{"lat":41.0933623593651,"lng":-8.65879348846621},{"lat":41.0934805560856,"lng":-8.65895797981859},{"lat":41.0936163107154,"lng":-8.65918235985648},{"lat":41.0938048873373,"lng":-8.65929587635005},{"lat":41.0940066937423,"lng":-8.6592926111175},{"lat":41.094344378401,"lng":-8.65927020136186},{"lat":41.0951124059868,"lng":-8.65930349165988},{"lat":41.0954766318855,"lng":-8.6595811375499},{"lat":41.095627277659,"lng":-8.65964007243159},{"lat":41.0955637693221,"lng":-8.66029363882492},{"lat":41.0958277933306,"lng":-8.66027159362974},{"lat":41.0960919361709,"lng":-8.66016551517499},{"lat":41.0963517599385,"lng":-8.66007737343999},{"lat":41.0964569219838,"lng":-8.65998203717875},{"lat":41.0965386866982,"lng":-8.6600007851233},{"lat":41.0967426014309,"lng":-8.66014669183086},{"lat":41.0968470936523,"lng":-8.66017764836562},{"lat":41.0970243800929,"lng":-8.66023918092919},{"lat":41.0971519389103,"lng":-8.66022817882688},{"lat":41.097625979298,"lng":-8.66000356198495},{"lat":41.0977672277355,"lng":-8.65993255876428},{"lat":41.0978631283876,"lng":-8.65987297813857},{"lat":41.0979634558008,"lng":-8.65984985705506},{"lat":41.0981768409195,"lng":-8.6598935869576},{"lat":41.098276140549,"lng":-8.66007484250042},{"lat":41.0984072320755,"lng":-8.66027432655369},{"lat":41.0985024191812,"lng":-8.66037138678723},{"lat":41.0985837220059,"lng":-8.66051048079059},{"lat":41.09864651741,"lng":-8.66067930579604},{"lat":41.0987638241275,"lng":-8.66086058878823},{"lat":41.0988997762776,"lng":-8.66098202396145},{"lat":41.0991028270772,"lng":-8.66137469763408},{"lat":41.0992938937671,"lng":-8.66137623463332},{"lat":41.0995346716511,"lng":-8.66145589743397},{"lat":41.0996481922144,"lng":-8.66148097668767},{"lat":41.099761199546,"lng":-8.66163842249952},{"lat":41.099828048333,"lng":-8.6619637011905},{"lat":41.1002851492492,"lng":-8.66248460301874},{"lat":41.10073447416,"lng":-8.66275880401864},{"lat":41.1012705121255,"lng":-8.6628775276309},{"lat":41.1014432663015,"lng":-8.66294498858785},{"lat":41.1015392815561,"lng":-8.66278338878058},{"lat":41.1016758972859,"lng":-8.66276044384697},{"lat":41.1019034668221,"lng":-8.66277442597519},{"lat":41.1020759818161,"lng":-8.66283593183745},{"lat":41.1022700309618,"lng":-8.66316831656129},{"lat":41.1023790484386,"lng":-8.66319324374676},{"lat":41.1024528659453,"lng":-8.66298325541866},{"lat":41.1026037849058,"lng":-8.66284614338196},{"lat":41.102814532609,"lng":-8.66285593919451},{"lat":41.1030941133562,"lng":-8.66305461868297},{"lat":41.1032398133664,"lng":-8.66301377304235},{"lat":41.1034770556511,"lng":-8.66288330962306},{"lat":41.1036232550083,"lng":-8.66273413005827},{"lat":41.1038019027124,"lng":-8.66250105458325},{"lat":41.1042234602995,"lng":-8.66254754666658},{"lat":41.1044559344855,"lng":-8.66272882602632},{"lat":41.1045989041078,"lng":-8.66331926435091},{"lat":41.1045440314398,"lng":-8.66338512440946},{"lat":41.1045068950886,"lng":-8.66351101796269},{"lat":41.10458799294,"lng":-8.66367405507192},{"lat":41.1047463582976,"lng":-8.66387390559183},{"lat":41.1049955272574,"lng":-8.66408639236206},{"lat":41.1050863975101,"lng":-8.66414129709126},{"lat":41.1052358992038,"lng":-8.66423274519843},{"lat":41.1053119206493,"lng":-8.66454014639143},{"lat":41.1054476949615,"lng":-8.66467958114851},{"lat":41.1055182658501,"lng":-8.66476193668615},{"lat":41.105483031072,"lng":-8.66496260596735},{"lat":41.1055729943544,"lng":-8.66515571464526},{"lat":41.1056637087734,"lng":-8.66518657170678},{"lat":41.1056815146631,"lng":-8.66528897643209},{"lat":41.1057313295179,"lng":-8.66536152534231},{"lat":41.1058092362837,"lng":-8.66524180173832},{"lat":41.1058452450766,"lng":-8.66530221130839},{"lat":41.1058311504058,"lng":-8.66542840614547},{"lat":41.1059077322429,"lng":-8.66555558239211},{"lat":41.1060032451147,"lng":-8.6655802856781},{"lat":41.1060446208037,"lng":-8.66545430508407},{"lat":41.1060274771476,"lng":-8.66526761898741},{"lat":41.105959941278,"lng":-8.66509277844034},{"lat":41.1057876789097,"lng":-8.66497698149898},{"lat":41.1057932399035,"lng":-8.6647484502782},{"lat":41.1058396064381,"lng":-8.66457453951302},{"lat":41.1058817879333,"lng":-8.66427403931926},{"lat":41.1060879422987,"lng":-8.66399296578745},{"lat":41.1062202771594,"lng":-8.66392189014073},{"lat":41.1063705092039,"lng":-8.66387488776007},{"lat":41.1067345486922,"lng":-8.66387783071612},{"lat":41.1068886707692,"lng":-8.66390586631665},{"lat":41.1070657090012,"lng":-8.66396182169811},{"lat":41.1072022358823,"lng":-8.66405614093832},{"lat":41.1072971166459,"lng":-8.66423727206047},{"lat":41.1074005845235,"lng":-8.6644307405074},{"lat":41.1073225890689,"lng":-8.66462856811992},{"lat":41.107399360478,"lng":-8.66469538257201},{"lat":41.1074307834978,"lng":-8.66485195217193},{"lat":41.1075160761532,"lng":-8.66508122518588},{"lat":41.1075882063151,"lng":-8.66517775971468},{"lat":41.1076747741801,"lng":-8.66520870391578},{"lat":41.1077607620774,"lng":-8.66528773508511},{"lat":41.1078333645883,"lng":-8.66536047102767},{"lat":41.1079099639492,"lng":-8.66546324279409},{"lat":41.1079734975068,"lng":-8.66552995089368},{"lat":41.1081492850096,"lng":-8.6656822099674},{"lat":41.108357322041,"lng":-8.66565616077344},{"lat":41.1089443939183,"lng":-8.6652544636243},{"lat":41.1094546975633,"lng":-8.6652913336721},{"lat":41.1103521636831,"lng":-8.66477153538936},{"lat":41.1114864550309,"lng":-8.66427042264364},{"lat":41.1116985458066,"lng":-8.66424863147071},{"lat":41.1118556343202,"lng":-8.66083561093689},{"lat":41.1119904912326,"lng":-8.65805328985726},{"lat":41.1139083615726,"lng":-8.65211561414995},{"lat":41.1153624027148,"lng":-8.64915053289446},{"lat":41.1169833843809,"lng":-8.64641635148598},{"lat":41.118797355511,"lng":-8.64525028143546},{"lat":41.1217770576677,"lng":-8.64431885870801},{"lat":41.1228585965168,"lng":-8.6440891191383},{"lat":41.1248067979274,"lng":-8.63607193270617}]},{"zone_id":"gdm5","points":[{"lat":41.0883727398084,"lng":-8.52239889625177},{"lat":41.087851121637,"lng":-8.51946844518349},{"lat":41.0884926044116,"lng":-8.51613985991302},{"lat":41.0903029480967,"lng":-8.51329404665976},{"lat":41.0908514667539,"lng":-8.51079786429244},{"lat":41.090860853146,"lng":-8.50794150166592},{"lat":41.0917701952004,"lng":-8.50520926562882},{"lat":41.0933940293197,"lng":-8.50426631115052},{"lat":41.0939412025599,"lng":-8.50212699777373},{"lat":41.0942159213895,"lng":-8.50070026381402},{"lat":41.0953895119947,"lng":-8.49975464396632},{"lat":41.0958461813283,"lng":-8.49773375120779},{"lat":41.0981936830805,"lng":-8.49572329713333},{"lat":41.0983311937642,"lng":-8.49388174292775},{"lat":41.1000398409341,"lng":-8.49412902912289},{"lat":41.1004496071132,"lng":-8.49418833505015},{"lat":41.1024312658877,"lng":-8.49396118378991},{"lat":41.103874175375,"lng":-8.49325488902302},{"lat":41.1049524205743,"lng":-8.49397507591314},{"lat":41.1058490733771,"lng":-8.49517046158004},{"lat":41.1084610202837,"lng":-8.49494680535024},{"lat":41.1102622193721,"lng":-8.49483770680996},{"lat":41.1115047209677,"lng":-8.49569828051128},{"lat":41.1125343765221,"lng":-8.49504384953051},{"lat":41.1137222308774,"lng":-8.49370295057617},{"lat":41.1161250051814,"lng":-8.49165540035887},{"lat":41.1192967722692,"lng":-8.48512399381639},{"lat":41.1204797807934,"lng":-8.4730771422205},{"lat":41.1210157995946,"lng":-8.46749075305377},{"lat":41.1208311852707,"lng":-8.46749981704607},{"lat":41.1205498038037,"lng":-8.46740562125517},{"lat":41.1203367451561,"lng":-8.46727444524021},{"lat":41.1201464854966,"lng":-8.46713279308463},{"lat":41.1199401379392,"lng":-8.46697530920622},{"lat":41.1197557974723,"lng":-8.46682157561981},{"lat":41.1194440105348,"lng":-8.46657486323146},{"lat":41.1193585407878,"lng":-8.46650723277129},{"lat":41.119163168729,"lng":-8.46634549025659},{"lat":41.1189678010196,"lng":-8.46618185074424},{"lat":41.1187530503615,"lng":-8.46602561210758},{"lat":41.1186172661814,"lng":-8.46594298435187},{"lat":41.1185448818698,"lng":-8.46589893701089},{"lat":41.1184851972492,"lng":-8.46583148032418},{"lat":41.1184425608536,"lng":-8.46578423138377},{"lat":41.1183133387591,"lng":-8.46569487114847},{"lat":41.1181742049179,"lng":-8.46554639826868},{"lat":41.1180964170297,"lng":-8.46551159132066},{"lat":41.118003907714,"lng":-8.46549397634288},{"lat":41.1178576294243,"lng":-8.46547882875599},{"lat":41.1177440750962,"lng":-8.46542026853168},{"lat":41.1175540168866,"lng":-8.46525629773113},{"lat":41.1174148870351,"lng":-8.46515851766073},{"lat":41.1173459938063,"lng":-8.46511009986853},{"lat":41.1171380649499,"lng":-8.46499176749333},{"lat":41.1170294667717,"lng":-8.46492360617478},{"lat":41.1169299956854,"lng":-8.46486117366972},{"lat":41.1167151352728,"lng":-8.46471113153723},{"lat":41.116543445403,"lng":-8.46451761745556},{"lat":41.1164842718419,"lng":-8.46442778315647},{"lat":41.1163922017816,"lng":-8.46428800759653},{"lat":41.116340192963,"lng":-8.46421158738275},{"lat":41.1162333212742,"lng":-8.46405455391279},{"lat":41.1160377163921,"lng":-8.46387794804282},{"lat":41.1158690723625,"lng":-8.46369405448799},{"lat":41.1158277582925,"lng":-8.46362220984022},{"lat":41.1158345951831,"lng":-8.46349889423374},{"lat":41.1158187426294,"lng":-8.46343868825024},{"lat":41.115732077668,"lng":-8.46314082810159},{"lat":41.1157307160497,"lng":-8.46283268693142},{"lat":41.1157630537439,"lng":-8.46254591012917},{"lat":41.1157901723272,"lng":-8.46232101703592},{"lat":41.1158025296852,"lng":-8.46212664611476},{"lat":41.1157955402388,"lng":-8.46183240712607},{"lat":41.1157229783329,"lng":-8.46151402726537},{"lat":41.1156473156621,"lng":-8.46120943937681},{"lat":41.1160089784334,"lng":-8.45907976929636},{"lat":41.1156179296135,"lng":-8.45736296249215},{"lat":41.1149895802487,"lng":-8.45740521336502},{"lat":41.1117999974208,"lng":-8.45930063472908},{"lat":41.1082034558566,"lng":-8.46089744532851},{"lat":41.1081086261819,"lng":-8.46256365049923},{"lat":41.106935683925,"lng":-8.46339107059894},{"lat":41.10675008571,"lng":-8.46529487194514},{"lat":41.107917855096,"lng":-8.4662531853943},{"lat":41.1084542826492,"lng":-8.46756545737663},{"lat":41.1079066887621,"lng":-8.47006587411162},{"lat":41.1079035376245,"lng":-8.47113408888033},{"lat":41.1070899945846,"lng":-8.47220130789282},{"lat":41.1047808339727,"lng":-8.47205770033056},{"lat":41.101593224837,"lng":-8.47360127655114},{"lat":41.0995258065827,"lng":-8.47240019400531},{"lat":41.0984470716022,"lng":-8.47179944821971},{"lat":41.0984509400063,"lng":-8.4704901257834},{"lat":41.097824152877,"lng":-8.46929659589679},{"lat":41.0969233873026,"lng":-8.46941099902474},{"lat":41.0970067591123,"lng":-8.47167296925019},{"lat":41.0958309219682,"lng":-8.47345228540161},{"lat":41.095646581557,"lng":-8.47487964005375},{"lat":41.0893895427159,"lng":-8.47578571528597},{"lat":41.0853782754782,"lng":-8.47601613709279},{"lat":41.0844523067526,"lng":-8.47632566892803},{"lat":41.081295717111,"lng":-8.477380783295},{"lat":41.0787094654035,"lng":-8.4778851629775},{"lat":41.0781656236106,"lng":-8.4790722467082},{"lat":41.0759523550414,"lng":-8.47855696181367},{"lat":41.0754344341579,"lng":-8.47883855522302},{"lat":41.0753141627757,"lng":-8.47924712763169},{"lat":41.0751395746734,"lng":-8.47984044115353},{"lat":41.0750991336407,"lng":-8.48002518262286},{"lat":41.0748739567454,"lng":-8.4810537998604},{"lat":41.0746206426995,"lng":-8.48200782210294},{"lat":41.0742610895932,"lng":-8.48298517999162},{"lat":41.0739196008571,"lng":-8.48367622730294},{"lat":41.0737538745795,"lng":-8.48399163353195},{"lat":41.0736245199936,"lng":-8.48423781579764},{"lat":41.0733859960617,"lng":-8.48476674216929},{"lat":41.0732273450757,"lng":-8.48520400155551},{"lat":41.0731680417821,"lng":-8.4853680036026},{"lat":41.0730477716477,"lng":-8.48585567675502},{"lat":41.0727636416713,"lng":-8.48683364452841},{"lat":41.0727111303431,"lng":-8.48706574170327},{"lat":41.0722432709698,"lng":-8.48913354688683},{"lat":41.0721594611095,"lng":-8.48942334587794},{"lat":41.0721128954076,"lng":-8.48958435847465},{"lat":41.0719665615329,"lng":-8.49009034455942},{"lat":41.0715318256071,"lng":-8.49225205068698},{"lat":41.0713703797608,"lng":-8.49305597224931},{"lat":41.0708969433457,"lng":-8.49613442037183},{"lat":41.0704989269645,"lng":-8.49928272975751},{"lat":41.0704639449818,"lng":-8.50008590043311},{"lat":41.0703412026067,"lng":-8.50291839307926},{"lat":41.0703163896574,"lng":-8.50335253196221},{"lat":41.0702953202297,"lng":-8.50404881354366},{"lat":41.0703160132202,"lng":-8.50461016568127},{"lat":41.070341312073,"lng":-8.50566590482622},{"lat":41.0704282733873,"lng":-8.5066134922892},{"lat":41.0705806856765,"lng":-8.5073957126415},{"lat":41.0707874234961,"lng":-8.50830532195596},{"lat":41.0709809012184,"lng":-8.50919343718583},{"lat":41.0711956872348,"lng":-8.50999867012373},{"lat":41.0712574385173,"lng":-8.51023017572831},{"lat":41.0714888647867,"lng":-8.51091160927238},{"lat":41.071617009319,"lng":-8.51128893023545},{"lat":41.0721691027027,"lng":-8.51269668259723},{"lat":41.0725051199555,"lng":-8.51355037384059},{"lat":41.0726341181346,"lng":-8.51385392816643},{"lat":41.0737229913418,"lng":-8.51537745149865},{"lat":41.0744372296102,"lng":-8.51610280127217},{"lat":41.0745769874272,"lng":-8.51623583853119},{"lat":41.0749738919482,"lng":-8.51661366094295},{"lat":41.0755573752022,"lng":-8.51715157105217},{"lat":41.0762711569136,"lng":-8.51776831802474},{"lat":41.077126311845,"lng":-8.51835520679096},{"lat":41.0776980915458,"lng":-8.51869470901622},{"lat":41.0779980845901,"lng":-8.51887283672118},{"lat":41.0787154730557,"lng":-8.51919108610125},{"lat":41.0788363753481,"lng":-8.51924196652187},{"lat":41.079594135427,"lng":-8.51956086164604},{"lat":41.0805876744886,"lng":-8.51998701026852},{"lat":41.0807788485899,"lng":-8.52006722055188},{"lat":41.0809618080535,"lng":-8.52014398531769},{"lat":41.0814506956898,"lng":-8.52034910914222},{"lat":41.0822074740624,"lng":-8.52073379030195},{"lat":41.0830533793473,"lng":-8.52114863098054},{"lat":41.0840868100739,"lng":-8.52149486144295},{"lat":41.084736612369,"lng":-8.52161379231004},{"lat":41.0864211131852,"lng":-8.52175669597924},{"lat":41.0866416401939,"lng":-8.52177133799804},{"lat":41.0867525388677,"lng":-8.52177870117988},{"lat":41.0868678060103,"lng":-8.52178635446396},{"lat":41.0872159124211,"lng":-8.52186780048207},{"lat":41.0876476482153,"lng":-8.5220067444075},{"lat":41.0879222146204,"lng":-8.52215512137951},{"lat":41.0881987224512,"lng":-8.52230454967007},{"lat":41.0883727398084,"lng":-8.52239889625177}]},{"zone_id":"gdm6","points":[{"lat":41.0530898310399,"lng":-8.44808118891258},{"lat":41.0534611593344,"lng":-8.44874065529944},{"lat":41.0555276311582,"lng":-8.45079090491084},{"lat":41.0558213500257,"lng":-8.45110094941703},{"lat":41.0565290039323,"lng":-8.4520276307025},{"lat":41.0599365242088,"lng":-8.45649023711613},{"lat":41.0603030018829,"lng":-8.45695950017871},{"lat":41.0603533752995,"lng":-8.45703244788701},{"lat":41.0606094290958,"lng":-8.45740324937115},{"lat":41.0611605684416,"lng":-8.45804493543802},{"lat":41.0619171578988,"lng":-8.45857046485796},{"lat":41.0625252957675,"lng":-8.45914809062115},{"lat":41.0631461504865,"lng":-8.45958969698471},{"lat":41.0636395368366,"lng":-8.45995750877112},{"lat":41.0642682254635,"lng":-8.4604613865488},{"lat":41.0652031879505,"lng":-8.46091702369175},{"lat":41.0653742249008,"lng":-8.46100037547162},{"lat":41.0673782392175,"lng":-8.46210139755534},{"lat":41.0687644630576,"lng":-8.46319700448342},{"lat":41.0693706122289,"lng":-8.46367573467671},{"lat":41.0716115913487,"lng":-8.46558398246862},{"lat":41.0732218779327,"lng":-8.46678152964417},{"lat":41.0735698096681,"lng":-8.46721236842023},{"lat":41.0740072634393,"lng":-8.46794577809141},{"lat":41.0742510450322,"lng":-8.46837046032127},{"lat":41.0745411155495,"lng":-8.46887578833335},{"lat":41.074908988992,"lng":-8.46959624843408},{"lat":41.0751633364401,"lng":-8.47009600128628},{"lat":41.0753108588269,"lng":-8.47045301192907},{"lat":41.0757834730339,"lng":-8.47256480480652},{"lat":41.0759342199764,"lng":-8.47371217983533},{"lat":41.0759397830733,"lng":-8.47450540691209},{"lat":41.0758647748796,"lng":-8.47553322188738},{"lat":41.0757257033128,"lng":-8.47656974210578},{"lat":41.0756138913524,"lng":-8.47755213746439},{"lat":41.0754344341579,"lng":-8.47883855522302},{"lat":41.0759523550414,"lng":-8.47855696181367},{"lat":41.0781656236106,"lng":-8.4790722467082},{"lat":41.0787094654035,"lng":-8.4778851629775},{"lat":41.081295717111,"lng":-8.477380783295},{"lat":41.0844523067526,"lng":-8.47632566892803},{"lat":41.0853782754782,"lng":-8.47601613709279},{"lat":41.0893895427159,"lng":-8.47578571528597},{"lat":41.095646581557,"lng":-8.47487964005375},{"lat":41.0958309219682,"lng":-8.47345228540161},{"lat":41.0970067591123,"lng":-8.47167296925019},{"lat":41.0969233873026,"lng":-8.46941099902474},{"lat":41.097824152877,"lng":-8.46929659589679},{"lat":41.0984509400063,"lng":-8.4704901257834},{"lat":41.0984470716022,"lng":-8.47179944821971},{"lat":41.0995258065827,"lng":-8.47240019400531},{"lat":41.101593224837,"lng":-8.47360127655114},{"lat":41.1047808339727,"lng":-8.47205770033056},{"lat":41.1070899945846,"lng":-8.47220130789282},{"lat":41.1079035376245,"lng":-8.47113408888033},{"lat":41.1079066887621,"lng":-8.47006587411162},{"lat":41.1084542826492,"lng":-8.46756545737663},{"lat":41.107917855096,"lng":-8.4662531853943},{"lat":41.10675008571,"lng":-8.46529487194514},{"lat":41.106935683925,"lng":-8.46339107059894},{"lat":41.1081086261819,"lng":-8.46256365049923},{"lat":41.1082034558566,"lng":-8.46089744532851},{"lat":41.1117999974208,"lng":-8.45930063472908},{"lat":41.1149895802487,"lng":-8.45740521336502},{"lat":41.1140220531197,"lng":-8.45686941261561},{"lat":41.1131078453765,"lng":-8.45645378247352},{"lat":41.1123279844887,"lng":-8.45619345128271},{"lat":41.1120805565966,"lng":-8.45611085552902},{"lat":41.1116348652117,"lng":-8.45596207595722},{"lat":41.1106975776955,"lng":-8.45564715074686},{"lat":41.1097219994482,"lng":-8.45531936996234},{"lat":41.1096091833012,"lng":-8.45528146595503},{"lat":41.1093177960656,"lng":-8.45498839673064},{"lat":41.1089242709409,"lng":-8.45459260538916},{"lat":41.1078594820762,"lng":-8.45370248545588},{"lat":41.1066822464314,"lng":-8.45277220472813},{"lat":41.1063756270372,"lng":-8.45259883829287},{"lat":41.1063199230744,"lng":-8.45256734276764},{"lat":41.1059128141213,"lng":-8.45233716146139},{"lat":41.1054408832251,"lng":-8.45207033345326},{"lat":41.1052231106465,"lng":-8.45188792282077},{"lat":41.1048178886295,"lng":-8.45154850470687},{"lat":41.1046848220745,"lng":-8.4514289369545},{"lat":41.10417091933,"lng":-8.45096717218468},{"lat":41.1035790217699,"lng":-8.45072216878136},{"lat":41.103517259549,"lng":-8.45068547358634},{"lat":41.103137634655,"lng":-8.45045992622933},{"lat":41.1029973774841,"lng":-8.45036218890862},{"lat":41.1028434618939,"lng":-8.45025493435569},{"lat":41.1023600827608,"lng":-8.44991810014578},{"lat":41.1017894469442,"lng":-8.44978805164653},{"lat":41.1016273909311,"lng":-8.4497511193365},{"lat":41.1011176692397,"lng":-8.44955162628608},{"lat":41.1007663574264,"lng":-8.44941413112655},{"lat":41.1002674698632,"lng":-8.44910532892657},{"lat":41.1000668316859,"lng":-8.44886317059736},{"lat":41.0999379270296,"lng":-8.44870759146522},{"lat":41.0994255015709,"lng":-8.44832443227617},{"lat":41.099348958913,"lng":-8.44826719914038},{"lat":41.0992502382677,"lng":-8.44819338311859},{"lat":41.0990121605107,"lng":-8.44792082104328},{"lat":41.0984480433646,"lng":-8.44727500536515},{"lat":41.0982382550887,"lng":-8.4470268241117},{"lat":41.0980145235083,"lng":-8.44676215026978},{"lat":41.0978621155645,"lng":-8.44660586156344},{"lat":41.097585181563,"lng":-8.44651026042026},{"lat":41.0974626874293,"lng":-8.44646657001772},{"lat":41.0962736438681,"lng":-8.44599812810471},{"lat":41.0960271142186,"lng":-8.44590100621424},{"lat":41.0957357141632,"lng":-8.44564615689754},{"lat":41.0952254755466,"lng":-8.44519992345288},{"lat":41.0948986537819,"lng":-8.44495630353457},{"lat":41.094397047047,"lng":-8.44458240066632},{"lat":41.0940202869392,"lng":-8.44429031459991},{"lat":41.0937515695699,"lng":-8.44412800659106},{"lat":41.0935308932098,"lng":-8.44399471694195},{"lat":41.0929415064067,"lng":-8.4436160457183},{"lat":41.0924297754525,"lng":-8.44328893330922},{"lat":41.0919654654757,"lng":-8.44287277777646},{"lat":41.09186023136,"lng":-8.44277509074985},{"lat":41.0916068203783,"lng":-8.44253985528578},{"lat":41.091041222472,"lng":-8.44299462124276},{"lat":41.0851172556084,"lng":-8.44583920832847},{"lat":41.0770770809805,"lng":-8.4481453933645},{"lat":41.0687814886178,"lng":-8.44533226559641},{"lat":41.0661018059686,"lng":-8.44151061562332},{"lat":41.0534882650347,"lng":-8.44785972459499},{"lat":41.0530898310399,"lng":-8.44808118891258}]},{"zone_id":"vng11","points":[{"lat":41.0450524959666,"lng":-8.4913896595901},{"lat":41.0451389823862,"lng":-8.49173710690729},{"lat":41.0511962530436,"lng":-8.50764648175955},{"lat":41.0525743566728,"lng":-8.51115453866563},{"lat":41.0531369479469,"lng":-8.51214127995742},{"lat":41.05396742164,"lng":-8.51316892096676},{"lat":41.0567060795228,"lng":-8.51628879834966},{"lat":41.0590681002732,"lng":-8.5152350516702},{"lat":41.0607789089353,"lng":-8.51524498438111},{"lat":41.062490112753,"lng":-8.51513595294745},{"lat":41.0644670854053,"lng":-8.5163371380578},{"lat":41.0660890398846,"lng":-8.51598966240562},{"lat":41.0682468611478,"lng":-8.51525954782477},{"lat":41.0693926034379,"lng":-8.51460509303299},{"lat":41.0721691027027,"lng":-8.51269668259723},{"lat":41.071617009319,"lng":-8.51128893023545},{"lat":41.0714888647867,"lng":-8.51091160927238},{"lat":41.0712574385173,"lng":-8.51023017572831},{"lat":41.0711956872348,"lng":-8.50999867012373},{"lat":41.0709809012184,"lng":-8.50919343718583},{"lat":41.0707874234961,"lng":-8.50830532195596},{"lat":41.0705806856765,"lng":-8.5073957126415},{"lat":41.0704282733873,"lng":-8.5066134922892},{"lat":41.070341312073,"lng":-8.50566590482622},{"lat":41.0703160132202,"lng":-8.50461016568127},{"lat":41.0702953202297,"lng":-8.50404881354366},{"lat":41.0703163896574,"lng":-8.50335253196221},{"lat":41.0703412026067,"lng":-8.50291839307926},{"lat":41.0704639449818,"lng":-8.50008590043311},{"lat":41.0704989269645,"lng":-8.49928272975751},{"lat":41.0708969433457,"lng":-8.49613442037183},{"lat":41.0713703797608,"lng":-8.49305597224931},{"lat":41.0715318256071,"lng":-8.49225205068698},{"lat":41.0719665615329,"lng":-8.49009034455942},{"lat":41.0721128954076,"lng":-8.48958435847465},{"lat":41.0721594611095,"lng":-8.48942334587794},{"lat":41.0722432709698,"lng":-8.48913354688683},{"lat":41.0727111303431,"lng":-8.48706574170327},{"lat":41.0727636416713,"lng":-8.48683364452841},{"lat":41.0730477716477,"lng":-8.48585567675502},{"lat":41.0731680417821,"lng":-8.4853680036026},{"lat":41.0732273450757,"lng":-8.48520400155551},{"lat":41.0733859960617,"lng":-8.48476674216929},{"lat":41.0736245199936,"lng":-8.48423781579764},{"lat":41.0737538745795,"lng":-8.48399163353195},{"lat":41.0739196008571,"lng":-8.48367622730294},{"lat":41.0742610895932,"lng":-8.48298517999162},{"lat":41.0746206426995,"lng":-8.48200782210294},{"lat":41.0748739567454,"lng":-8.4810537998604},{"lat":41.0750991336407,"lng":-8.48002518262286},{"lat":41.0751395746734,"lng":-8.47984044115353},{"lat":41.0753141627757,"lng":-8.47924712763169},{"lat":41.0754344341579,"lng":-8.47883855522302},{"lat":41.0756138913524,"lng":-8.47755213746439},{"lat":41.0757257033128,"lng":-8.47656974210578},{"lat":41.0758647748796,"lng":-8.47553322188738},{"lat":41.0759397830733,"lng":-8.47450540691209},{"lat":41.0759342199764,"lng":-8.47371217983533},{"lat":41.0757834730339,"lng":-8.47256480480652},{"lat":41.0753108588269,"lng":-8.47045301192907},{"lat":41.0751633364401,"lng":-8.47009600128628},{"lat":41.074908988992,"lng":-8.46959624843408},{"lat":41.0745411155495,"lng":-8.46887578833335},{"lat":41.0742510450322,"lng":-8.46837046032127},{"lat":41.0740072634393,"lng":-8.46794577809141},{"lat":41.0735698096681,"lng":-8.46721236842023},{"lat":41.0732218779327,"lng":-8.46678152964417},{"lat":41.0716115913487,"lng":-8.46558398246862},{"lat":41.0693706122289,"lng":-8.46367573467671},{"lat":41.0687644630576,"lng":-8.46319700448342},{"lat":41.0673782392175,"lng":-8.46210139755534},{"lat":41.0653742249008,"lng":-8.46100037547162},{"lat":41.0652031879505,"lng":-8.46091702369175},{"lat":41.0642682254635,"lng":-8.4604613865488},{"lat":41.0636395368366,"lng":-8.45995750877112},{"lat":41.0631461504865,"lng":-8.45958969698471},{"lat":41.0625252957675,"lng":-8.45914809062115},{"lat":41.0619171578988,"lng":-8.45857046485796},{"lat":41.0611605684416,"lng":-8.45804493543802},{"lat":41.0606094290958,"lng":-8.45740324937115},{"lat":41.0603533752995,"lng":-8.45703244788701},{"lat":41.0603030018829,"lng":-8.45695950017871},{"lat":41.0599365242088,"lng":-8.45649023711613},{"lat":41.0565290039323,"lng":-8.4520276307025},{"lat":41.0558213500257,"lng":-8.45110094941703},{"lat":41.0555276311582,"lng":-8.45079090491084},{"lat":41.0534611593344,"lng":-8.44874065529944},{"lat":41.0520111139053,"lng":-8.44997062287587},{"lat":41.0519427740297,"lng":-8.45009994401381},{"lat":41.0518816587927,"lng":-8.45022026774714},{"lat":41.0517589734768,"lng":-8.4503672838894},{"lat":41.0516061929629,"lng":-8.45045480885794},{"lat":41.0514524745695,"lng":-8.45052483870901},{"lat":41.0513667567649,"lng":-8.45058793927359},{"lat":41.0512775058092,"lng":-8.45062806685182},{"lat":41.0511884039424,"lng":-8.45067747894278},{"lat":41.0510905361212,"lng":-8.45077132815149},{"lat":41.0509878481423,"lng":-8.45084850357958},{"lat":41.0508615258109,"lng":-8.45090938578839},{"lat":41.0506034500259,"lng":-8.45097807890792},{"lat":41.0504858994719,"lng":-8.45102603753377},{"lat":41.0503801226877,"lng":-8.45108333051544},{"lat":41.0502714059841,"lng":-8.45119116261148},{"lat":41.0501983980802,"lng":-8.45134674969413},{"lat":41.0501067890318,"lng":-8.45148939343426},{"lat":41.0499886251213,"lng":-8.45156209217263},{"lat":41.0498186520083,"lng":-8.45161645272035},{"lat":41.0497281711984,"lng":-8.45164432898653},{"lat":41.0495856686221,"lng":-8.45166302504218},{"lat":41.0495005187452,"lng":-8.45161978816804},{"lat":41.0494096366308,"lng":-8.45162898443712},{"lat":41.0493080620133,"lng":-8.45179275374484},{"lat":41.0492183754473,"lng":-8.45185868971766},{"lat":41.0490808255845,"lng":-8.45190951810611},{"lat":41.0489660339764,"lng":-8.45187173332866},{"lat":41.0487890707018,"lng":-8.45171827196348},{"lat":41.0486921361079,"lng":-8.45160646834908},{"lat":41.0486084576196,"lng":-8.45151888044247},{"lat":41.0484211485052,"lng":-8.45139640922904},{"lat":41.0482835260987,"lng":-8.45137576244671},{"lat":41.0481763061534,"lng":-8.4513996278757},{"lat":41.0480752499079,"lng":-8.45144171400583},{"lat":41.0479656334662,"lng":-8.45142072643775},{"lat":41.0477166353374,"lng":-8.45143117565394},{"lat":41.047495817889,"lng":-8.45140964759431},{"lat":41.0474004524022,"lng":-8.45144427097327},{"lat":41.047312072161,"lng":-8.4515256718027},{"lat":41.0472121239376,"lng":-8.45165423739967},{"lat":41.047123081101,"lng":-8.45178047640464},{"lat":41.0470585055743,"lng":-8.4518831663341},{"lat":41.0469601921854,"lng":-8.45203873591486},{"lat":41.0468626582065,"lng":-8.45217290191165},{"lat":41.0467700832917,"lng":-8.45227342783315},{"lat":41.0466880788407,"lng":-8.45232905117319},{"lat":41.0465820178699,"lng":-8.45235767887973},{"lat":41.0464837323452,"lng":-8.4523749187325},{"lat":41.0463942886329,"lng":-8.45241741764017},{"lat":41.0462714746417,"lng":-8.45244703454351},{"lat":41.0461482170826,"lng":-8.45247521954591},{"lat":41.0459651845295,"lng":-8.45256257646751},{"lat":41.045831879018,"lng":-8.45270596275358},{"lat":41.0457577171125,"lng":-8.45288662219653},{"lat":41.0456575664747,"lng":-8.45315207960019},{"lat":41.0456018461623,"lng":-8.45330678121243},{"lat":41.045564054753,"lng":-8.45342708462207},{"lat":41.0454993152071,"lng":-8.45365072125486},{"lat":41.045461177974,"lng":-8.45383013232833},{"lat":41.0454197346617,"lng":-8.45399798422388},{"lat":41.0453724409305,"lng":-8.45419780646648},{"lat":41.0453249650647,"lng":-8.45443021286851},{"lat":41.0452766799625,"lng":-8.45463169243524},{"lat":41.0451746771981,"lng":-8.45481697010402},{"lat":41.0450661468852,"lng":-8.45498639794508},{"lat":41.0450115170571,"lng":-8.45516952907878},{"lat":41.0449855008603,"lng":-8.45529558961579},{"lat":41.0449662288486,"lng":-8.45542489901462},{"lat":41.0448287382087,"lng":-8.45545395876969},{"lat":41.044686103745,"lng":-8.45545515979788},{"lat":41.04439225442,"lng":-8.45553126142868},{"lat":41.0442947651481,"lng":-8.45555337982808},{"lat":41.044081376491,"lng":-8.45561156093197},{"lat":41.0439354424879,"lng":-8.45563344616056},{"lat":41.0438076630479,"lng":-8.45566802108323},{"lat":41.0434910217585,"lng":-8.45587578508955},{"lat":41.0433627726799,"lng":-8.45601240600909},{"lat":41.0432089909621,"lng":-8.45616768713718},{"lat":41.0428736631477,"lng":-8.45642305121516},{"lat":41.0426302086344,"lng":-8.45660607585673},{"lat":41.0423782072674,"lng":-8.45678537275762},{"lat":41.0426786409144,"lng":-8.45721583540562},{"lat":41.042926012046,"lng":-8.45755827331032},{"lat":41.0431762626781,"lng":-8.45790167822128},{"lat":41.0432606586015,"lng":-8.45802055366174},{"lat":41.0434094791349,"lng":-8.45821895770472},{"lat":41.0436699578701,"lng":-8.45857478722854},{"lat":41.0439738569963,"lng":-8.45895521899378},{"lat":41.0442311489742,"lng":-8.45932138613411},{"lat":41.0443021919453,"lng":-8.45941783346439},{"lat":41.044387581016,"lng":-8.45953362544925},{"lat":41.0444996091855,"lng":-8.45968749240802},{"lat":41.0449392011772,"lng":-8.46028911231618},{"lat":41.0451510394458,"lng":-8.4599552460809},{"lat":41.0456358177647,"lng":-8.46084409188621},{"lat":41.0459449766839,"lng":-8.46142854788306},{"lat":41.0463914365055,"lng":-8.46226881823404},{"lat":41.0465625732974,"lng":-8.462595573238},{"lat":41.0471276093223,"lng":-8.46371833732658},{"lat":41.047892941661,"lng":-8.46525831247627},{"lat":41.0482833006937,"lng":-8.46605886404588},{"lat":41.0483216650156,"lng":-8.46618085384371},{"lat":41.0486760283282,"lng":-8.46709636676592},{"lat":41.0488387267826,"lng":-8.46746996083455},{"lat":41.0489411905635,"lng":-8.46774939983929},{"lat":41.0490251281547,"lng":-8.46802125585025},{"lat":41.0493978835512,"lng":-8.46898445868691},{"lat":41.0497042207258,"lng":-8.46965759087296},{"lat":41.0499912549964,"lng":-8.47049392809921},{"lat":41.050162158958,"lng":-8.47098212639524},{"lat":41.0502617061156,"lng":-8.47127155896456},{"lat":41.0503207565537,"lng":-8.47146098514923},{"lat":41.0503760442283,"lng":-8.47161221407626},{"lat":41.0504683962404,"lng":-8.47189791475605},{"lat":41.0506197090261,"lng":-8.47231036683808},{"lat":41.0507172342646,"lng":-8.47258290369758},{"lat":41.0508236935564,"lng":-8.472907225898},{"lat":41.050930847441,"lng":-8.47324118985977},{"lat":41.0511319169957,"lng":-8.47384683815119},{"lat":41.0512789224482,"lng":-8.47464561346572},{"lat":41.051463238577,"lng":-8.47573327519409},{"lat":41.0515085815983,"lng":-8.47598615268974},{"lat":41.0515617546734,"lng":-8.47630093290842},{"lat":41.0516081304643,"lng":-8.4765715375426},{"lat":41.051694369789,"lng":-8.47699842185866},{"lat":41.0517006788525,"lng":-8.47717806180499},{"lat":41.0519153242183,"lng":-8.47736355405434},{"lat":41.0522419615629,"lng":-8.47719719700871},{"lat":41.0528498946438,"lng":-8.47685221485105},{"lat":41.0527209586484,"lng":-8.47730057137674},{"lat":41.0525752567341,"lng":-8.47778309762025},{"lat":41.0525270519508,"lng":-8.47800278792631},{"lat":41.0524851855508,"lng":-8.47812246300391},{"lat":41.0524372449469,"lng":-8.47822427195827},{"lat":41.0523949043397,"lng":-8.47846968183936},{"lat":41.0522568524379,"lng":-8.47919001834702},{"lat":41.0521912976013,"lng":-8.47960837375455},{"lat":41.05207806359,"lng":-8.48010046102012},{"lat":41.0520265061363,"lng":-8.48032595174337},{"lat":41.0520024823233,"lng":-8.48058727302822},{"lat":41.0520362172008,"lng":-8.48077586555245},{"lat":41.0520114979952,"lng":-8.48091014607689},{"lat":41.0519778818856,"lng":-8.48103855110126},{"lat":41.051912951002,"lng":-8.48127860221334},{"lat":41.0518682028369,"lng":-8.48139576572268},{"lat":41.0517968868964,"lng":-8.4815444294029},{"lat":41.0519039418749,"lng":-8.48163551708761},{"lat":41.0520008871496,"lng":-8.48172404423139},{"lat":41.0521042027803,"lng":-8.4819136563679},{"lat":41.0521773600113,"lng":-8.48206458604447},{"lat":41.0522407742167,"lng":-8.48220469093552},{"lat":41.0523875371706,"lng":-8.48241600869588},{"lat":41.0525089751886,"lng":-8.48260245285497},{"lat":41.0525851752813,"lng":-8.48268434204556},{"lat":41.0526569680608,"lng":-8.48276311061321},{"lat":41.0527806771721,"lng":-8.4829433851333},{"lat":41.0528300371587,"lng":-8.48314478809025},{"lat":41.0528527370839,"lng":-8.48331833792075},{"lat":41.0528579101813,"lng":-8.48348275578453},{"lat":41.0528492551746,"lng":-8.48366244020709},{"lat":41.052887571687,"lng":-8.48388246891267},{"lat":41.0529196108812,"lng":-8.48400634478879},{"lat":41.0527714624238,"lng":-8.4853342206904},{"lat":41.0527142780279,"lng":-8.48580566642206},{"lat":41.052703223117,"lng":-8.48620765997876},{"lat":41.0525964235068,"lng":-8.48626953089616},{"lat":41.0519397494387,"lng":-8.48687572680663},{"lat":41.0510830444094,"lng":-8.48703586530635},{"lat":41.0502839280241,"lng":-8.48745972724489},{"lat":41.0495380773076,"lng":-8.48800132861079},{"lat":41.0491473902247,"lng":-8.48826035535101},{"lat":41.0488099767862,"lng":-8.4885429299767},{"lat":41.0484903203537,"lng":-8.48875486094597},{"lat":41.0482416976103,"lng":-8.48896679191523},{"lat":41.0479930739275,"lng":-8.48910807922807},{"lat":41.0476911724783,"lng":-8.48932001019734},{"lat":41.047300474429,"lng":-8.48964968059398},{"lat":41.0468540927363,"lng":-8.48999986316712},{"lat":41.0453243160216,"lng":-8.49127499105685},{"lat":41.0450524959666,"lng":-8.4913896595901}]},{"zone_id":"gdm3","points":[{"lat":41.1378250780675,"lng":-8.57417280665866},{"lat":41.1382749782067,"lng":-8.57429501883738},{"lat":41.139948914651,"lng":-8.5756582921291},{"lat":41.1431481148212,"lng":-8.57146905713986},{"lat":41.1448652373805,"lng":-8.56981289993442},{"lat":41.1463086015994,"lng":-8.56910779076095},{"lat":41.1473913475462,"lng":-8.56851937998361},{"lat":41.1463288201317,"lng":-8.56374758816913},{"lat":41.1477739298572,"lng":-8.56256585706412},{"lat":41.1485856310976,"lng":-8.56221380002422},{"lat":41.1495760780706,"lng":-8.56222027552182},{"lat":41.1497579382616,"lng":-8.56174496529766},{"lat":41.1494922550588,"lng":-8.56055198680719},{"lat":41.1502183322174,"lng":-8.55900808218663},{"lat":41.1518496287918,"lng":-8.5561595822423},{"lat":41.1523938164052,"lng":-8.55509092403935},{"lat":41.1540228413798,"lng":-8.55283788134322},{"lat":41.1545066910641,"lng":-8.55266661034053},{"lat":41.1543391752682,"lng":-8.5521903373883},{"lat":41.1541878054638,"lng":-8.55178102836478},{"lat":41.1539610372493,"lng":-8.55132301600919},{"lat":41.1536778756831,"lng":-8.55083076929608},{"lat":41.153340614583,"lng":-8.55029608225462},{"lat":41.1529134056018,"lng":-8.54986167064868},{"lat":41.1524646745661,"lng":-8.5494925872188},{"lat":41.1521686200381,"lng":-8.54926057976933},{"lat":41.1518850388017,"lng":-8.5490867762999},{"lat":41.1512168042971,"lng":-8.54871676417158},{"lat":41.1485051521785,"lng":-8.54823092796893},{"lat":41.1482346399217,"lng":-8.54839255075372},{"lat":41.1479393333708,"lng":-8.54832387043707},{"lat":41.1477889899205,"lng":-8.54811391165965},{"lat":41.1476840786401,"lng":-8.54796650274336},{"lat":41.1475149139727,"lng":-8.54789041151849},{"lat":41.1468483425928,"lng":-8.54775121477747},{"lat":41.1454065825134,"lng":-8.54740066054179},{"lat":41.1451060545813,"lng":-8.54725895808523},{"lat":41.143956601932,"lng":-8.54694457176556},{"lat":41.1429656586583,"lng":-8.54626619105896},{"lat":41.1423846375259,"lng":-8.54560870423386},{"lat":41.1416017024925,"lng":-8.54399516545206},{"lat":41.1410410891901,"lng":-8.54224729763043},{"lat":41.1401646420216,"lng":-8.54123953347362},{"lat":41.1396365298803,"lng":-8.54099319568611},{"lat":41.1368335267908,"lng":-8.53904315943061},{"lat":41.135645679264,"lng":-8.53810161367275},{"lat":41.1354326468515,"lng":-8.53829400854998},{"lat":41.1351627335168,"lng":-8.53823279356715},{"lat":41.1349114582067,"lng":-8.53799304708284},{"lat":41.1348408938565,"lng":-8.53757576937187},{"lat":41.1349478631815,"lng":-8.53723068963138},{"lat":41.1344383837356,"lng":-8.53656274248081},{"lat":41.1333651883804,"lng":-8.53523171272217},{"lat":41.1321091684327,"lng":-8.53322944879149},{"lat":41.1315151686297,"lng":-8.53153631425419},{"lat":41.131132829695,"lng":-8.52912972974605},{"lat":41.1307570363833,"lng":-8.52352024885259},{"lat":41.1311209260001,"lng":-8.52216971504425},{"lat":41.1314973309597,"lng":-8.52131765409652},{"lat":41.1320126527303,"lng":-8.52049569604474},{"lat":41.1325476500303,"lng":-8.51917676769916},{"lat":41.1325830638505,"lng":-8.51854070490039},{"lat":41.1329468759126,"lng":-8.5177789246073},{"lat":41.13386516227,"lng":-8.51779079288893},{"lat":41.1325485781445,"lng":-8.51704843105883},{"lat":41.1290366805172,"lng":-8.51551329523038},{"lat":41.1247630709264,"lng":-8.52111514037209},{"lat":41.1207931511166,"lng":-8.52611642802048},{"lat":41.1192744345378,"lng":-8.53046522593166},{"lat":41.1168798201536,"lng":-8.5354752117508},{"lat":41.1147778565923,"lng":-8.54224237476533},{"lat":41.1159967544012,"lng":-8.54319163227059},{"lat":41.1187402323614,"lng":-8.54571496644131},{"lat":41.1219071675575,"lng":-8.54875747530509},{"lat":41.1227051700073,"lng":-8.55017470410143},{"lat":41.1240125984161,"lng":-8.55622269133534},{"lat":41.1241059836523,"lng":-8.56033708225287},{"lat":41.123999139043,"lng":-8.56348209708882},{"lat":41.1244386030943,"lng":-8.56634287237586},{"lat":41.126049883309,"lng":-8.56885421922917},{"lat":41.1291927104296,"lng":-8.57113773274333},{"lat":41.1315145905407,"lng":-8.5717551911651},{"lat":41.1340503666224,"lng":-8.5723610952149},{"lat":41.1361215539414,"lng":-8.5732655946634},{"lat":41.1377257259315,"lng":-8.57414635975049},{"lat":41.1378250780675,"lng":-8.57417280665866}]},{"zone_id":"prd9","points":[{"lat":41.1407644390299,"lng":-8.47096848909352},{"lat":41.1431822987603,"lng":-8.46918247827937},{"lat":41.1431242895478,"lng":-8.46904485113521},{"lat":41.140610623898,"lng":-8.46305538088768},{"lat":41.1389719230249,"lng":-8.45910415264843},{"lat":41.1376142313675,"lng":-8.45476438807595},{"lat":41.1366126477936,"lng":-8.45082042552461},{"lat":41.1356704883199,"lng":-8.44687686601394},{"lat":41.1347896576943,"lng":-8.44222469478379},{"lat":41.1340864847991,"lng":-8.43788858276601},{"lat":41.1338554497922,"lng":-8.4356145838218},{"lat":41.1336220037088,"lng":-8.43331731866588},{"lat":41.1328138024781,"lng":-8.423308971294},{"lat":41.1263204303485,"lng":-8.42381205286397},{"lat":41.1245929926164,"lng":-8.42388319641974},{"lat":41.1233426305325,"lng":-8.42372014616764},{"lat":41.1220570798218,"lng":-8.42345552682227},{"lat":41.101492834107,"lng":-8.41480725797219},{"lat":41.099225670582,"lng":-8.41564887410075},{"lat":41.0967551007626,"lng":-8.41623949670393},{"lat":41.09658636636,"lng":-8.41628449603349},{"lat":41.0957589305231,"lng":-8.41896085369588},{"lat":41.0954140945543,"lng":-8.42001860858322},{"lat":41.09489796298,"lng":-8.42076429408938},{"lat":41.094577973289,"lng":-8.42092502001199},{"lat":41.0941877712243,"lng":-8.42162676706104},{"lat":41.093667711372,"lng":-8.42185639423058},{"lat":41.0928592689115,"lng":-8.42177848687076},{"lat":41.0926080335529,"lng":-8.42229927338725},{"lat":41.0925793949209,"lng":-8.42235863793032},{"lat":41.0925631657217,"lng":-8.42243914756158},{"lat":41.0925182948812,"lng":-8.42259212460345},{"lat":41.0924560513703,"lng":-8.42274408009832},{"lat":41.092447464504,"lng":-8.42311288980971},{"lat":41.0924452087705,"lng":-8.42326810505632},{"lat":41.092437035374,"lng":-8.42335469542662},{"lat":41.0924134626872,"lng":-8.42341826634998},{"lat":41.0922803591314,"lng":-8.42367523511666},{"lat":41.0922253384165,"lng":-8.42381793714995},{"lat":41.0921707119469,"lng":-8.42413849434314},{"lat":41.0921313108295,"lng":-8.42439274653838},{"lat":41.0919285469723,"lng":-8.42435149548625},{"lat":41.0918512472068,"lng":-8.42433366033522},{"lat":41.0916044785059,"lng":-8.42431803816278},{"lat":41.0914402849074,"lng":-8.42426374904147},{"lat":41.0913538377459,"lng":-8.42423841213387},{"lat":41.0912052629145,"lng":-8.42419486631066},{"lat":41.0911330653386,"lng":-8.42563435592378},{"lat":41.0911145274455,"lng":-8.42600390978208},{"lat":41.091092840753,"lng":-8.42643620487979},{"lat":41.0907407091611,"lng":-8.42712883706054},{"lat":41.0900642896495,"lng":-8.42845929001946},{"lat":41.0884084224349,"lng":-8.43125924812388},{"lat":41.0876003121245,"lng":-8.43390652232281},{"lat":41.0867135199405,"lng":-8.43681124524297},{"lat":41.0867554061967,"lng":-8.43693093685578},{"lat":41.086924243506,"lng":-8.437413401222},{"lat":41.0884742874124,"lng":-8.43872053789439},{"lat":41.0889432435749,"lng":-8.43898753386581},{"lat":41.0891333465233,"lng":-8.43920648164747},{"lat":41.0894046465211,"lng":-8.43951894976447},{"lat":41.0894706822759,"lng":-8.43964798900703},{"lat":41.0898768833243,"lng":-8.44044175089089},{"lat":41.0899926632106,"lng":-8.44057423779824},{"lat":41.0903638713627,"lng":-8.44099901557843},{"lat":41.0910087793615,"lng":-8.44186207287497},{"lat":41.0916068203783,"lng":-8.44253985528578},{"lat":41.09186023136,"lng":-8.44277509074985},{"lat":41.0919654654757,"lng":-8.44287277777646},{"lat":41.0924297754525,"lng":-8.44328893330922},{"lat":41.0929415064067,"lng":-8.4436160457183},{"lat":41.0935308932098,"lng":-8.44399471694195},{"lat":41.0937515695699,"lng":-8.44412800659106},{"lat":41.0940202869392,"lng":-8.44429031459991},{"lat":41.094397047047,"lng":-8.44458240066632},{"lat":41.0948986537819,"lng":-8.44495630353457},{"lat":41.0952254755466,"lng":-8.44519992345288},{"lat":41.0957357141632,"lng":-8.44564615689754},{"lat":41.0960271142186,"lng":-8.44590100621424},{"lat":41.0962736438681,"lng":-8.44599812810471},{"lat":41.0974626874293,"lng":-8.44646657001772},{"lat":41.097585181563,"lng":-8.44651026042026},{"lat":41.0978621155645,"lng":-8.44660586156344},{"lat":41.0980145235083,"lng":-8.44676215026978},{"lat":41.0982382550887,"lng":-8.4470268241117},{"lat":41.0984480433646,"lng":-8.44727500536515},{"lat":41.0990121605107,"lng":-8.44792082104328},{"lat":41.0992502382677,"lng":-8.44819338311859},{"lat":41.099348958913,"lng":-8.44826719914038},{"lat":41.0994255015709,"lng":-8.44832443227617},{"lat":41.0999379270296,"lng":-8.44870759146522},{"lat":41.1000668316859,"lng":-8.44886317059736},{"lat":41.1002674698632,"lng":-8.44910532892657},{"lat":41.1007663574264,"lng":-8.44941413112655},{"lat":41.1011176692397,"lng":-8.44955162628608},{"lat":41.1016273909311,"lng":-8.4497511193365},{"lat":41.1017894469442,"lng":-8.44978805164653},{"lat":41.1023600827608,"lng":-8.44991810014578},{"lat":41.1028434618939,"lng":-8.45025493435569},{"lat":41.1029973774841,"lng":-8.45036218890862},{"lat":41.103137634655,"lng":-8.45045992622933},{"lat":41.103517259549,"lng":-8.45068547358634},{"lat":41.1035790217699,"lng":-8.45072216878136},{"lat":41.10417091933,"lng":-8.45096717218468},{"lat":41.1046848220745,"lng":-8.4514289369545},{"lat":41.1048178886295,"lng":-8.45154850470687},{"lat":41.1052231106465,"lng":-8.45188792282077},{"lat":41.1054408832251,"lng":-8.45207033345326},{"lat":41.1059128141213,"lng":-8.45233716146139},{"lat":41.1063199230744,"lng":-8.45256734276764},{"lat":41.1063756270372,"lng":-8.45259883829287},{"lat":41.1066822464314,"lng":-8.45277220472813},{"lat":41.1078594820762,"lng":-8.45370248545588},{"lat":41.1089242709409,"lng":-8.45459260538916},{"lat":41.1093177960656,"lng":-8.45498839673064},{"lat":41.1096091833012,"lng":-8.45528146595503},{"lat":41.1097219994482,"lng":-8.45531936996234},{"lat":41.1106975776955,"lng":-8.45564715074686},{"lat":41.1116348652117,"lng":-8.45596207595722},{"lat":41.1120805565966,"lng":-8.45611085552902},{"lat":41.1123279844887,"lng":-8.45619345128271},{"lat":41.1131078453765,"lng":-8.45645378247352},{"lat":41.1140220531197,"lng":-8.45686941261561},{"lat":41.1149895802487,"lng":-8.45740521336502},{"lat":41.1156179296135,"lng":-8.45736296249215},{"lat":41.1160089784334,"lng":-8.45907976929636},{"lat":41.1156473156621,"lng":-8.46120943937681},{"lat":41.1157229783329,"lng":-8.46151402726537},{"lat":41.1157955402388,"lng":-8.46183240712607},{"lat":41.1158025296852,"lng":-8.46212664611476},{"lat":41.1157901723272,"lng":-8.46232101703592},{"lat":41.1157630537439,"lng":-8.46254591012917},{"lat":41.1157307160497,"lng":-8.46283268693142},{"lat":41.115732077668,"lng":-8.46314082810159},{"lat":41.1158187426294,"lng":-8.46343868825024},{"lat":41.1158345951831,"lng":-8.46349889423374},{"lat":41.1158277582925,"lng":-8.46362220984022},{"lat":41.1158690723625,"lng":-8.46369405448799},{"lat":41.1160377163921,"lng":-8.46387794804282},{"lat":41.1162333212742,"lng":-8.46405455391279},{"lat":41.116340192963,"lng":-8.46421158738275},{"lat":41.1163922017816,"lng":-8.46428800759653},{"lat":41.1164842718419,"lng":-8.46442778315647},{"lat":41.116543445403,"lng":-8.46451761745556},{"lat":41.1167151352728,"lng":-8.46471113153723},{"lat":41.1169299956854,"lng":-8.46486117366972},{"lat":41.1170294667717,"lng":-8.46492360617478},{"lat":41.1171380649499,"lng":-8.46499176749333},{"lat":41.1173459938063,"lng":-8.46511009986853},{"lat":41.1174148870351,"lng":-8.46515851766073},{"lat":41.1175540168866,"lng":-8.46525629773113},{"lat":41.1177440750962,"lng":-8.46542026853168},{"lat":41.1178576294243,"lng":-8.46547882875599},{"lat":41.118003907714,"lng":-8.46549397634288},{"lat":41.1180964170297,"lng":-8.46551159132066},{"lat":41.1181742049179,"lng":-8.46554639826868},{"lat":41.1183133387591,"lng":-8.46569487114847},{"lat":41.1184425608536,"lng":-8.46578423138377},{"lat":41.1184851972492,"lng":-8.46583148032418},{"lat":41.1185448818698,"lng":-8.46589893701089},{"lat":41.1186172661814,"lng":-8.46594298435187},{"lat":41.1187530503615,"lng":-8.46602561210758},{"lat":41.1189678010196,"lng":-8.46618185074424},{"lat":41.119163168729,"lng":-8.46634549025659},{"lat":41.1193585407878,"lng":-8.46650723277129},{"lat":41.1194440105348,"lng":-8.46657486323146},{"lat":41.1197557974723,"lng":-8.46682157561981},{"lat":41.1199401379392,"lng":-8.46697530920622},{"lat":41.1201464854966,"lng":-8.46713279308463},{"lat":41.1203367451561,"lng":-8.46727444524021},{"lat":41.1205498038037,"lng":-8.46740562125517},{"lat":41.1208311852707,"lng":-8.46749981704607},{"lat":41.1210157995946,"lng":-8.46749075305377},{"lat":41.1212478401275,"lng":-8.46742835332701},{"lat":41.1213253429613,"lng":-8.46740695953361},{"lat":41.1215327558352,"lng":-8.46748375089771},{"lat":41.1217345693174,"lng":-8.46762921488376},{"lat":41.1219941860008,"lng":-8.46774509369544},{"lat":41.1221235970612,"lng":-8.46776944958113},{"lat":41.1222796982325,"lng":-8.46787372711107},{"lat":41.1224061968797,"lng":-8.46797784720262},{"lat":41.1224880052573,"lng":-8.46804518332362},{"lat":41.1227226310893,"lng":-8.46823798334501},{"lat":41.1229044036963,"lng":-8.46833845449488},{"lat":41.1230046605506,"lng":-8.4684180358499},{"lat":41.1232379897712,"lng":-8.4685002025821},{"lat":41.1234714829005,"lng":-8.46858975717878},{"lat":41.1236915699266,"lng":-8.46867352118977},{"lat":41.1238048319062,"lng":-8.46872523632842},{"lat":41.1238501807347,"lng":-8.46874590493564},{"lat":41.1240749168144,"lng":-8.46884755834633},{"lat":41.1242138585389,"lng":-8.46895697000431},{"lat":41.1242899817557,"lng":-8.46901691455819},{"lat":41.1243825403439,"lng":-8.46911921998074},{"lat":41.124460782557,"lng":-8.46920570144965},{"lat":41.1244964974664,"lng":-8.46926599015875},{"lat":41.1245527321694,"lng":-8.46936091763714},{"lat":41.1246923205504,"lng":-8.46955948331835},{"lat":41.1249129089153,"lng":-8.4696575468301},{"lat":41.1251464251635,"lng":-8.46976782220398},{"lat":41.1253741558167,"lng":-8.46988649757367},{"lat":41.1256080404817,"lng":-8.47002586095049},{"lat":41.1258140379101,"lng":-8.47010025177023},{"lat":41.1259967815153,"lng":-8.47013896697224},{"lat":41.1260756498866,"lng":-8.47016274094053},{"lat":41.1261523298021,"lng":-8.47018585528075},{"lat":41.1262502618194,"lng":-8.47022522065518},{"lat":41.1263858044227,"lng":-8.47027970442407},{"lat":41.1271492198711,"lng":-8.47048817982652},{"lat":41.1274004948889,"lng":-8.47055679973695},{"lat":41.1282395707579,"lng":-8.4704459627617},{"lat":41.1282975731755,"lng":-8.47044365999588},{"lat":41.1290067395651,"lng":-8.47041550523638},{"lat":41.1293294478724,"lng":-8.47055369385253},{"lat":41.1296645736466,"lng":-8.47069720134662},{"lat":41.1297089960356,"lng":-8.47071624285743},{"lat":41.1303362894891,"lng":-8.47117296959361},{"lat":41.1305049770987,"lng":-8.47129579125576},{"lat":41.1309535406416,"lng":-8.47152871619746},{"lat":41.1310599674879,"lng":-8.47159005500555},{"lat":41.1311467561511,"lng":-8.47162904800146},{"lat":41.1313924269853,"lng":-8.4717566194224},{"lat":41.1317488844167,"lng":-8.47189957928298},{"lat":41.1345474745823,"lng":-8.47302203139463},{"lat":41.1355780401482,"lng":-8.473673782616},{"lat":41.1360496290998,"lng":-8.47397203299926},{"lat":41.1382170770619,"lng":-8.47260847129301},{"lat":41.1384514171687,"lng":-8.47244204697365},{"lat":41.1395892932011,"lng":-8.47171715420569},{"lat":41.1407644390299,"lng":-8.47096848909352}]},{"zone_id":"gdm2","points":[{"lat":41.1783773345259,"lng":-8.50633221426163},{"lat":41.1771061330435,"lng":-8.50538669926103},{"lat":41.1739275910958,"lng":-8.50351378371791},{"lat":41.1736812560833,"lng":-8.50336864234435},{"lat":41.1696261110766,"lng":-8.5006842289575},{"lat":41.1650415545832,"lng":-8.49864550788538},{"lat":41.1631771714303,"lng":-8.49764686715909},{"lat":41.1594222066732,"lng":-8.49572288793818},{"lat":41.1569324706516,"lng":-8.49559924996029},{"lat":41.1510727279868,"lng":-8.49487156235589},{"lat":41.1489503861111,"lng":-8.49485566395874},{"lat":41.148229004964,"lng":-8.49467120703117},{"lat":41.1475255370645,"lng":-8.49460287382722},{"lat":41.1471329271716,"lng":-8.49449755253558},{"lat":41.1467591597947,"lng":-8.49421186342183},{"lat":41.1464570827834,"lng":-8.49437779505071},{"lat":41.1461279092887,"lng":-8.49442755757027},{"lat":41.1457470984974,"lng":-8.49437649985008},{"lat":41.1453404743286,"lng":-8.49208340261674},{"lat":41.1454113420437,"lng":-8.49061293666646},{"lat":41.1454164721903,"lng":-8.49012247751771},{"lat":41.1454207032259,"lng":-8.4897178260317},{"lat":41.1456424090634,"lng":-8.48895847026293},{"lat":41.1457553877296,"lng":-8.48786320096069},{"lat":41.1459985036905,"lng":-8.48608561389235},{"lat":41.1462322064327,"lng":-8.4843722821066},{"lat":41.1467104354388,"lng":-8.48336924823919},{"lat":41.1474532124313,"lng":-8.48239119447641},{"lat":41.1471973051327,"lng":-8.48209857976963},{"lat":41.1469485120604,"lng":-8.48174477445915},{"lat":41.146690705118,"lng":-8.4811875925364},{"lat":41.1464906481845,"lng":-8.48061714376094},{"lat":41.1463096910707,"lng":-8.48007394891936},{"lat":41.1460997929071,"lng":-8.47957123639661},{"lat":41.145899068867,"lng":-8.47919054253754},{"lat":41.1456035860877,"lng":-8.47840268969724},{"lat":41.1451525003721,"lng":-8.47818339930095},{"lat":41.1450378576577,"lng":-8.47797946227899},{"lat":41.1445864096827,"lng":-8.47788214913405},{"lat":41.1438679126734,"lng":-8.4771192442932},{"lat":41.1434754753692,"lng":-8.47658843727626},{"lat":41.1429779407176,"lng":-8.47585375940911},{"lat":41.1420892465186,"lng":-8.47415463619615},{"lat":41.1407644390299,"lng":-8.47096848909352},{"lat":41.1395892932011,"lng":-8.47171715420569},{"lat":41.1384514171687,"lng":-8.47244204697365},{"lat":41.1382170770619,"lng":-8.47260847129301},{"lat":41.1360496290998,"lng":-8.47397203299926},{"lat":41.1355780401482,"lng":-8.473673782616},{"lat":41.1345474745823,"lng":-8.47302203139463},{"lat":41.1317488844167,"lng":-8.47189957928298},{"lat":41.1313924269853,"lng":-8.4717566194224},{"lat":41.1311467561511,"lng":-8.47162904800146},{"lat":41.1310599674879,"lng":-8.47159005500555},{"lat":41.1309535406416,"lng":-8.47152871619746},{"lat":41.1305049770987,"lng":-8.47129579125576},{"lat":41.1303362894891,"lng":-8.47117296959361},{"lat":41.1297089960356,"lng":-8.47071624285743},{"lat":41.1296645736466,"lng":-8.47069720134662},{"lat":41.1293294478724,"lng":-8.47055369385253},{"lat":41.1282455872257,"lng":-8.47931613438678},{"lat":41.1275869107343,"lng":-8.48860127907919},{"lat":41.1269406511456,"lng":-8.49371837747378},{"lat":41.1259221818122,"lng":-8.50252483807563},{"lat":41.1266317405145,"lng":-8.50586315700584},{"lat":41.1284205302097,"lng":-8.50956498873679},{"lat":41.1296728864799,"lng":-8.51207302985341},{"lat":41.1295812910624,"lng":-8.51480163183858},{"lat":41.1290366805172,"lng":-8.51551329523038},{"lat":41.1325485781445,"lng":-8.51704843105883},{"lat":41.13386516227,"lng":-8.51779079288893},{"lat":41.1341392393277,"lng":-8.51794526125708},{"lat":41.1360577701451,"lng":-8.51922817763039},{"lat":41.1406389342112,"lng":-8.51986140281359},{"lat":41.1423288485158,"lng":-8.52052972787895},{"lat":41.1465155610017,"lng":-8.51907919311788},{"lat":41.1486810723456,"lng":-8.51909193131546},{"lat":41.1506411581717,"lng":-8.51780971434024},{"lat":41.1533303188316,"lng":-8.51803483033264},{"lat":41.155166975435,"lng":-8.5189307124127},{"lat":41.1568839768701,"lng":-8.51987799717687},{"lat":41.1598414468008,"lng":-8.5195168879186},{"lat":41.1625704768565,"lng":-8.51964947497279},{"lat":41.1670086815799,"lng":-8.51991540783315},{"lat":41.1702228402581,"lng":-8.51981622619824},{"lat":41.1736536553493,"lng":-8.52030238463571},{"lat":41.1749243974809,"lng":-8.52056233668983},{"lat":41.1761633074395,"lng":-8.51852170614269},{"lat":41.1774795560007,"lng":-8.51747186452168},{"lat":41.1782268182667,"lng":-8.51588985699067},{"lat":41.1787465147368,"lng":-8.51407986109821},{"lat":41.1812014513659,"lng":-8.51386749432572},{"lat":41.1797712143244,"lng":-8.51048711088827},{"lat":41.1788147984777,"lng":-8.50764039590738},{"lat":41.1783773345259,"lng":-8.50633221426163}]},{"zone_id":"vng3","points":[{"lat":41.0708406856745,"lng":-8.58048160574828},{"lat":41.0714111141716,"lng":-8.58154557727523},{"lat":41.0718120784802,"lng":-8.5887448848438},{"lat":41.0689439238299,"lng":-8.59135651258979},{"lat":41.0679942937216,"lng":-8.59924939718115},{"lat":41.0675555836308,"lng":-8.60189610385084},{"lat":41.0674356336921,"lng":-8.60354035847213},{"lat":41.0673710023352,"lng":-8.60689478395609},{"lat":41.0667249582235,"lng":-8.61069732636571},{"lat":41.0672543017341,"lng":-8.61331863078315},{"lat":41.0686794950661,"lng":-8.61701735076271},{"lat":41.0688530561634,"lng":-8.61856534742216},{"lat":41.069026926492,"lng":-8.61974838734558},{"lat":41.0690438203498,"lng":-8.61986010068272},{"lat":41.0689217057791,"lng":-8.61988852565262},{"lat":41.0683063132082,"lng":-8.62002305901483},{"lat":41.0689576896548,"lng":-8.6231097571957},{"lat":41.0694105922413,"lng":-8.62457589451279},{"lat":41.0698294531824,"lng":-8.625414316996},{"lat":41.0702527176611,"lng":-8.62691453952559},{"lat":41.0703828817532,"lng":-8.63140727122455},{"lat":41.0703850724356,"lng":-8.63179009344031},{"lat":41.072848888833,"lng":-8.6316832423816},{"lat":41.0763610006143,"lng":-8.63159084995506},{"lat":41.0802358515799,"lng":-8.63090620112969},{"lat":41.0830338060184,"lng":-8.62938031105836},{"lat":41.0860025816293,"lng":-8.62999774696424},{"lat":41.0888122856744,"lng":-8.62573444194846},{"lat":41.090716362044,"lng":-8.62265423165813},{"lat":41.0924286569092,"lng":-8.62230991003828},{"lat":41.0938713342393,"lng":-8.62184454370742},{"lat":41.0943281066493,"lng":-8.62030061898357},{"lat":41.0956812389177,"lng":-8.61971550616127},{"lat":41.0975741090386,"lng":-8.61925338817492},{"lat":41.0978942072679,"lng":-8.6191772644016},{"lat":41.0999076370169,"lng":-8.61913074783428},{"lat":41.0999171803177,"lng":-8.61879456877584},{"lat":41.1000257473406,"lng":-8.6143910835357},{"lat":41.1033681912756,"lng":-8.61179664740916},{"lat":41.1037713312092,"lng":-8.6105926895489},{"lat":41.1031173111626,"lng":-8.60715224620546},{"lat":41.1025446249284,"lng":-8.60095673460691},{"lat":41.1027886752761,"lng":-8.60024725948748},{"lat":41.1006532368245,"lng":-8.59686919631465},{"lat":41.0981306650272,"lng":-8.59342769096835},{"lat":41.0954014556572,"lng":-8.58999890922809},{"lat":41.0946897069734,"lng":-8.58862761398624},{"lat":41.0946769836994,"lng":-8.58606491879814},{"lat":41.0941881720552,"lng":-8.58265761244937},{"lat":41.0926268199928,"lng":-8.57766853789374},{"lat":41.0922083303142,"lng":-8.57559105929429},{"lat":41.0915852296876,"lng":-8.57374038053339},{"lat":41.0908873512209,"lng":-8.57336664500674},{"lat":41.0899844062621,"lng":-8.57363680336761},{"lat":41.0890325522997,"lng":-8.5743717297037},{"lat":41.0871923498252,"lng":-8.57492719975503},{"lat":41.084162413682,"lng":-8.57467102960342},{"lat":41.0820435977557,"lng":-8.57465460583274},{"lat":41.0791748486481,"lng":-8.57543233657418},{"lat":41.0773092361523,"lng":-8.57679417967297},{"lat":41.0753954442825,"lng":-8.57782097126857},{"lat":41.0740761005192,"lng":-8.57803693863322},{"lat":41.0712423491077,"lng":-8.57776784079373},{"lat":41.0708406856745,"lng":-8.58048160574828}]},{"zone_id":"str8","points":[{"lat":41.240638658065,"lng":-8.48789961475316},{"lat":41.241442362907,"lng":-8.49400743514632},{"lat":41.2419245495261,"lng":-8.49562091468441},{"lat":41.2438893536701,"lng":-8.50398346984213},{"lat":41.245287006165,"lng":-8.50656156581844},{"lat":41.2481525735713,"lng":-8.51135007989393},{"lat":41.2493195231369,"lng":-8.51243059253501},{"lat":41.2530103194505,"lng":-8.51269069788892},{"lat":41.2556116545514,"lng":-8.51306668366757},{"lat":41.2566067095177,"lng":-8.51426277860746},{"lat":41.2593971107697,"lng":-8.51451774806095},{"lat":41.2611066495252,"lng":-8.51488573394216},{"lat":41.2648712522219,"lng":-8.517416491209},{"lat":41.2681136231039,"lng":-8.5197003483682},{"lat":41.2696051005244,"lng":-8.5185370835038},{"lat":41.2736264581339,"lng":-8.51542119035921},{"lat":41.2741310671343,"lng":-8.51542561083302},{"lat":41.2720271440986,"lng":-8.487668343119},{"lat":41.2690906809479,"lng":-8.47049009434989},{"lat":41.2685465318063,"lng":-8.46919268103078},{"lat":41.266672417694,"lng":-8.46449403567753},{"lat":41.2661871197115,"lng":-8.46270757214721},{"lat":41.2628740991168,"lng":-8.44954265003576},{"lat":41.2623973783933,"lng":-8.45001933800554},{"lat":41.2620275565402,"lng":-8.45033404510556},{"lat":41.2617683893403,"lng":-8.45021572637651},{"lat":41.2608721346321,"lng":-8.4497548200724},{"lat":41.2599025599313,"lng":-8.44927173257875},{"lat":41.2597546527275,"lng":-8.44919679625457},{"lat":41.2588641348497,"lng":-8.44936453362267},{"lat":41.2580281093305,"lng":-8.44955842967861},{"lat":41.2574075299021,"lng":-8.44969371372036},{"lat":41.2570082444488,"lng":-8.4497742149319},{"lat":41.2565653305591,"lng":-8.44987312787557},{"lat":41.2562315246364,"lng":-8.44994571992324},{"lat":41.2559633250874,"lng":-8.45000002175145},{"lat":41.2556638403786,"lng":-8.45003566802656},{"lat":41.2553053511138,"lng":-8.45008141536052},{"lat":41.2550621307701,"lng":-8.45012139689053},{"lat":41.2549030056456,"lng":-8.45016190526705},{"lat":41.2543819313373,"lng":-8.45032641025164},{"lat":41.2535175970537,"lng":-8.4505963812727},{"lat":41.2531323466115,"lng":-8.45071405167622},{"lat":41.2527642149331,"lng":-8.45082679727855},{"lat":41.2518158455233,"lng":-8.45110720621133},{"lat":41.2511744735577,"lng":-8.45130415540759},{"lat":41.2503124647591,"lng":-8.45158007851913},{"lat":41.2501486388997,"lng":-8.45162640615767},{"lat":41.2499063765063,"lng":-8.4516781950665},{"lat":41.2487986707341,"lng":-8.4519434800304},{"lat":41.2478050323773,"lng":-8.45218116133785},{"lat":41.2473383791021,"lng":-8.45230139881682},{"lat":41.247179962627,"lng":-8.4523451230722},{"lat":41.2446897120868,"lng":-8.45392899638593},{"lat":41.2445021406361,"lng":-8.45404963829215},{"lat":41.2446286381755,"lng":-8.45600720561026},{"lat":41.2446409747197,"lng":-8.45642087348008},{"lat":41.2447436355314,"lng":-8.4571166449975},{"lat":41.2443706772297,"lng":-8.4637968201587},{"lat":41.2438412697375,"lng":-8.46605265180454},{"lat":41.2429313083021,"lng":-8.47037261316188},{"lat":41.2413473507551,"lng":-8.47983846027266},{"lat":41.240638658065,"lng":-8.48789961475316}]},{"zone_id":"vlg2","points":[{"lat":41.2419245495261,"lng":-8.49562091468441},{"lat":41.2392199232918,"lng":-8.49704777684847},{"lat":41.2275246368861,"lng":-8.50749471432146},{"lat":41.2236809428928,"lng":-8.51094731969271},{"lat":41.2213181272246,"lng":-8.51749268407556},{"lat":41.2181060228281,"lng":-8.52511429524343},{"lat":41.2171369790096,"lng":-8.52903509383841},{"lat":41.219026994725,"lng":-8.52928502675193},{"lat":41.2205539643812,"lng":-8.53036754839963},{"lat":41.2219883986535,"lng":-8.53216509261329},{"lat":41.222613694511,"lng":-8.53359998897791},{"lat":41.2234144521225,"lng":-8.53634783302296},{"lat":41.2244011147528,"lng":-8.53742725410795},{"lat":41.2236643248,"lng":-8.54207379896299},{"lat":41.2244738321755,"lng":-8.54231737784373},{"lat":41.2268106065251,"lng":-8.54352463469765},{"lat":41.2304070653838,"lng":-8.5449784441709},{"lat":41.2308590074405,"lng":-8.54478204748722},{"lat":41.2309481581198,"lng":-8.54474330568627},{"lat":41.2323836504335,"lng":-8.54618363276479},{"lat":41.2314685949942,"lng":-8.55023313988714},{"lat":41.2338000579978,"lng":-8.55287210258264},{"lat":41.2338905397955,"lng":-8.55275341227435},{"lat":41.2349688250038,"lng":-8.55335671821265},{"lat":41.2386560664165,"lng":-8.5545732638303},{"lat":41.2409044193506,"lng":-8.55530349006146},{"lat":41.2438717615297,"lng":-8.55639630242218},{"lat":41.2454911471423,"lng":-8.55676467772099},{"lat":41.2486416369314,"lng":-8.55702369401675},{"lat":41.2504419786655,"lng":-8.55715467154919},{"lat":41.2530522303709,"lng":-8.55741022266693},{"lat":41.2573727798899,"lng":-8.55779622095984},{"lat":41.2658346710554,"lng":-8.55832853987904},{"lat":41.2681869057266,"lng":-8.55848686005698},{"lat":41.2651240655649,"lng":-8.55299187900883},{"lat":41.2637092930651,"lng":-8.54894139148496},{"lat":41.2630562731391,"lng":-8.54502214426784},{"lat":41.2632629892282,"lng":-8.54047683794398},{"lat":41.2637496871855,"lng":-8.5364442201266},{"lat":41.2644073998571,"lng":-8.53228046387806},{"lat":41.2654934588819,"lng":-8.52821392157359},{"lat":41.2670793447741,"lng":-8.52424494377153},{"lat":41.2683052005105,"lng":-8.52103147914557},{"lat":41.2696051005244,"lng":-8.5185370835038},{"lat":41.2681136231039,"lng":-8.5197003483682},{"lat":41.2648712522219,"lng":-8.517416491209},{"lat":41.2611066495252,"lng":-8.51488573394216},{"lat":41.2593971107697,"lng":-8.51451774806095},{"lat":41.2566067095177,"lng":-8.51426277860746},{"lat":41.2556116545514,"lng":-8.51306668366757},{"lat":41.2530103194505,"lng":-8.51269069788892},{"lat":41.2493195231369,"lng":-8.51243059253501},{"lat":41.2481525735713,"lng":-8.51135007989393},{"lat":41.245287006165,"lng":-8.50656156581844},{"lat":41.2438893536701,"lng":-8.50398346984213},{"lat":41.2419245495261,"lng":-8.49562091468441}]},{"zone_id":"str1","points":[{"lat":41.3363025161735,"lng":-8.45611482990106},{"lat":41.3329255766662,"lng":-8.45451883706366},{"lat":41.3309583515926,"lng":-8.46013905901476},{"lat":41.3303906330692,"lng":-8.46274876115689},{"lat":41.3303023812913,"lng":-8.46459218359731},{"lat":41.3305373914098,"lng":-8.46673861996373},{"lat":41.3310302259746,"lng":-8.4693012250542},{"lat":41.3312828062952,"lng":-8.47151890692807},{"lat":41.3312309687968,"lng":-8.47394836172209},{"lat":41.3310812695274,"lng":-8.47622321924446},{"lat":41.3303560677953,"lng":-8.47972765011159},{"lat":41.3297000795821,"lng":-8.48295977704236},{"lat":41.3281603164877,"lng":-8.48788186774985},{"lat":41.3267183973163,"lng":-8.49191935371303},{"lat":41.3243202250854,"lng":-8.49696264664516},{"lat":41.3227931210822,"lng":-8.49925501243983},{"lat":41.3217168570356,"lng":-8.50057625310322},{"lat":41.3216113566752,"lng":-8.5006872404336},{"lat":41.318774202614,"lng":-8.50550923893158},{"lat":41.3164801116427,"lng":-8.50889619537125},{"lat":41.3156518715792,"lng":-8.50976748243221},{"lat":41.3164369270609,"lng":-8.51077409149424},{"lat":41.3177838688323,"lng":-8.51199715462107},{"lat":41.3205189550679,"lng":-8.51292305792012},{"lat":41.3258707254747,"lng":-8.51297224871337},{"lat":41.329641489484,"lng":-8.51164770619862},{"lat":41.3317706437614,"lng":-8.51016311743101},{"lat":41.3346106679946,"lng":-8.50768659573061},{"lat":41.3369539279711,"lng":-8.50562831738263},{"lat":41.3413729474165,"lng":-8.50613311111435},{"lat":41.3480927630021,"lng":-8.51252896836003},{"lat":41.3480747683761,"lng":-8.51515892901894},{"lat":41.3481824548559,"lng":-8.51488696103725},{"lat":41.3494454779173,"lng":-8.51419860459431},{"lat":41.352471428871,"lng":-8.51568396719357},{"lat":41.3526360205673,"lng":-8.51576487548165},{"lat":41.354016015781,"lng":-8.51733493132303},{"lat":41.3559245994771,"lng":-8.5163622566097},{"lat":41.3560841670888,"lng":-8.51627692332643},{"lat":41.3562583215068,"lng":-8.51604832247538},{"lat":41.3563284221487,"lng":-8.51595000470953},{"lat":41.356474203231,"lng":-8.51572621182894},{"lat":41.356697269997,"lng":-8.51540940296835},{"lat":41.3568565474733,"lng":-8.51516667226525},{"lat":41.3569380502648,"lng":-8.51502319211842},{"lat":41.3570911869016,"lng":-8.51477462090745},{"lat":41.357244976571,"lng":-8.51455070118036},{"lat":41.3573559135662,"lng":-8.5143927928522},{"lat":41.3580112299947,"lng":-8.51335013938043},{"lat":41.3584323750425,"lng":-8.51266466992568},{"lat":41.3588274844204,"lng":-8.51198120789072},{"lat":41.3590183401704,"lng":-8.51165925227463},{"lat":41.3592282492103,"lng":-8.51139226566492},{"lat":41.3594486189134,"lng":-8.5112139115932},{"lat":41.3596431282611,"lng":-8.51104058273866},{"lat":41.3598653568355,"lng":-8.51089244153041},{"lat":41.3600836977812,"lng":-8.51077525228258},{"lat":41.3599311226268,"lng":-8.51044077154526},{"lat":41.3599524229201,"lng":-8.51031625076321},{"lat":41.3601099537842,"lng":-8.50997143211036},{"lat":41.3603510476073,"lng":-8.50957152161702},{"lat":41.361701827943,"lng":-8.50891240248096},{"lat":41.3619161648125,"lng":-8.50877527373274},{"lat":41.3620672907465,"lng":-8.50864083551257},{"lat":41.3622837474757,"lng":-8.50847160603182},{"lat":41.3625798212335,"lng":-8.50826395190865},{"lat":41.3626549516327,"lng":-8.50818716884873},{"lat":41.3627540131585,"lng":-8.50802040075261},{"lat":41.3628571820441,"lng":-8.50768219186953},{"lat":41.362896307544,"lng":-8.50755271887235},{"lat":41.36308814273,"lng":-8.50784392588121},{"lat":41.3632868625506,"lng":-8.50765122752884},{"lat":41.363766313325,"lng":-8.5076539922798},{"lat":41.3637630696127,"lng":-8.50751688580565},{"lat":41.3636965222386,"lng":-8.50729910944844},{"lat":41.3636312566292,"lng":-8.50721292240335},{"lat":41.3634194852206,"lng":-8.50718469913595},{"lat":41.3631763307778,"lng":-8.5070392115864},{"lat":41.3630212397856,"lng":-8.50690054144562},{"lat":41.3628493950877,"lng":-8.50649810739582},{"lat":41.3628818735003,"lng":-8.506381100717},{"lat":41.3629718651932,"lng":-8.50598089507597},{"lat":41.3635070118225,"lng":-8.50285940236185},{"lat":41.3636127783,"lng":-8.50286765244183},{"lat":41.3637268117568,"lng":-8.50282814436398},{"lat":41.3639470373672,"lng":-8.50274705284965},{"lat":41.3641770902895,"lng":-8.50268967942289},{"lat":41.3644235421347,"lng":-8.50262953236222},{"lat":41.3646517439533,"lng":-8.50261696657332},{"lat":41.3647589181967,"lng":-8.50263657733538},{"lat":41.3649960787377,"lng":-8.50272111302088},{"lat":41.3651098336053,"lng":-8.50279382848154},{"lat":41.3651939260044,"lng":-8.50287844282411},{"lat":41.3653398714154,"lng":-8.50290903393543},{"lat":41.365568132703,"lng":-8.50270978330029},{"lat":41.3657827761941,"lng":-8.50257355920753},{"lat":41.3660126917141,"lng":-8.50247591068674},{"lat":41.3662486229276,"lng":-8.50241103323769},{"lat":41.3664810574846,"lng":-8.50239789510212},{"lat":41.3667320638819,"lng":-8.50246075290882},{"lat":41.3669563754973,"lng":-8.50245306571714},{"lat":41.3670886608722,"lng":-8.50241999756867},{"lat":41.3671825238564,"lng":-8.5023783419302},{"lat":41.3672899122145,"lng":-8.50233150128082},{"lat":41.367449060461,"lng":-8.50226045577802},{"lat":41.3676686869014,"lng":-8.50214254145279},{"lat":41.3678820327092,"lng":-8.50201682186307},{"lat":41.3680095975963,"lng":-8.50191224959107},{"lat":41.368190590374,"lng":-8.50171187796371},{"lat":41.3683349447156,"lng":-8.5014540513898},{"lat":41.3683564173944,"lng":-8.50124333587985},{"lat":41.3683233674767,"lng":-8.50093800634644},{"lat":41.3683265901548,"lng":-8.50071929820217},{"lat":41.3683663708295,"lng":-8.50050079928027},{"lat":41.368482477782,"lng":-8.50020456161134},{"lat":41.3684991231158,"lng":-8.50001270410904},{"lat":41.3684926293951,"lng":-8.49981892132553},{"lat":41.3685172921125,"lng":-8.49959626785401},{"lat":41.3685937745518,"lng":-8.49919020306088},{"lat":41.3684955925075,"lng":-8.49880825341752},{"lat":41.3683839176116,"lng":-8.49853546838771},{"lat":41.3682810633742,"lng":-8.49823428814233},{"lat":41.3682638812093,"lng":-8.49797530719728},{"lat":41.3683054913953,"lng":-8.49771797177239},{"lat":41.3683527290275,"lng":-8.49738823591545},{"lat":41.368367890902,"lng":-8.49720700384446},{"lat":41.3683377592344,"lng":-8.49677321032289},{"lat":41.3682957290068,"lng":-8.49645432815818},{"lat":41.3682274098549,"lng":-8.49616565570445},{"lat":41.3682167331153,"lng":-8.49598344345931},{"lat":41.3682625223239,"lng":-8.49585318151717},{"lat":41.3683347901067,"lng":-8.49577960360482},{"lat":41.3685361432212,"lng":-8.49560001065068},{"lat":41.3687063132458,"lng":-8.49534577619784},{"lat":41.368772959479,"lng":-8.49508474893444},{"lat":41.3687747527515,"lng":-8.49488671002826},{"lat":41.3687630715398,"lng":-8.4945664458687},{"lat":41.368860497563,"lng":-8.49427976925197},{"lat":41.3688854143739,"lng":-8.49414520726376},{"lat":41.3689562021049,"lng":-8.49373921706626},{"lat":41.3690278480302,"lng":-8.49340578966913},{"lat":41.3690734952924,"lng":-8.49329117893573},{"lat":41.3691638425692,"lng":-8.49319367355838},{"lat":41.369317818695,"lng":-8.49310476234433},{"lat":41.3694744089544,"lng":-8.4930439573129},{"lat":41.3697001331545,"lng":-8.49258671010322},{"lat":41.3698517008263,"lng":-8.49205996204172},{"lat":41.3701117022721,"lng":-8.49109216698917},{"lat":41.369987791344,"lng":-8.49003212653027},{"lat":41.3696608952057,"lng":-8.48892090481262},{"lat":41.3694168378891,"lng":-8.48769168920963},{"lat":41.3692120081585,"lng":-8.48687386400245},{"lat":41.3689742782374,"lng":-8.48612470538796},{"lat":41.3687296366587,"lng":-8.4858043717265},{"lat":41.3686522912669,"lng":-8.48563064230589},{"lat":41.3683933710222,"lng":-8.48480142012521},{"lat":41.3683513320459,"lng":-8.48447453599357},{"lat":41.3683468010628,"lng":-8.48413292296216},{"lat":41.3683910018134,"lng":-8.48384235740654},{"lat":41.3684625836537,"lng":-8.48325373638736},{"lat":41.3685824428514,"lng":-8.48280713316199},{"lat":41.368728757204,"lng":-8.48262970897895},{"lat":41.3688713950663,"lng":-8.48253617081585},{"lat":41.3691059105143,"lng":-8.48240094322693},{"lat":41.3693790953329,"lng":-8.48231085117831},{"lat":41.3695871545023,"lng":-8.4822585450674},{"lat":41.3698043939857,"lng":-8.48208962614763},{"lat":41.3698971749213,"lng":-8.48180959978467},{"lat":41.3699965682148,"lng":-8.48148777824997},{"lat":41.3701730630632,"lng":-8.48053454664011},{"lat":41.3704091358113,"lng":-8.47952844210677},{"lat":41.3704789784926,"lng":-8.4791150536287},{"lat":41.3702936486296,"lng":-8.47892051841629},{"lat":41.363552539008,"lng":-8.4718891275505},{"lat":41.359182438485,"lng":-8.46731323116258},{"lat":41.3560888932922,"lng":-8.46554244461326},{"lat":41.3513785026285,"lng":-8.46324213595566},{"lat":41.3363025161735,"lng":-8.45611482990106}]},{"zone_id":"trf1","points":[{"lat":41.3064846051867,"lng":-8.57870652746103},{"lat":41.308826065246,"lng":-8.58458926251087},{"lat":41.3157779894238,"lng":-8.58683041301259},{"lat":41.3189246510498,"lng":-8.58736080472233},{"lat":41.325402695834,"lng":-8.57786732340612},{"lat":41.34643285398,"lng":-8.58566647040204},{"lat":41.3465026603406,"lng":-8.58406525770813},{"lat":41.3465124901428,"lng":-8.58155618934603},{"lat":41.3470491922355,"lng":-8.58024651084859},{"lat":41.3468838065322,"lng":-8.5786911430371},{"lat":41.3459922230104,"lng":-8.57641489529494},{"lat":41.3457308558336,"lng":-8.57414297364699},{"lat":41.3464633925771,"lng":-8.5686527726801},{"lat":41.3460355003197,"lng":-8.56506441507479},{"lat":41.3448735334801,"lng":-8.56278659993786},{"lat":41.3444234571453,"lng":-8.56039519023772},{"lat":41.3445223435938,"lng":-8.55800627236376},{"lat":41.3464289247155,"lng":-8.55371732938586},{"lat":41.3478769243515,"lng":-8.55169541721288},{"lat":41.3475366417489,"lng":-8.54619698414131},{"lat":41.3461097126159,"lng":-8.54236456506011},{"lat":41.3464927337868,"lng":-8.54126773236153},{"lat":41.3468559780278,"lng":-8.54042419898564},{"lat":41.3472748985822,"lng":-8.53950745404765},{"lat":41.3484348846738,"lng":-8.53668300127254},{"lat":41.3496361126126,"lng":-8.53393225229747},{"lat":41.349875304793,"lng":-8.53288561201161},{"lat":41.3503012209032,"lng":-8.52992775457512},{"lat":41.3510047413618,"lng":-8.52680603811767},{"lat":41.3514554632078,"lng":-8.52465732304668},{"lat":41.3521683453104,"lng":-8.52282274497854},{"lat":41.3530483340663,"lng":-8.52073167641319},{"lat":41.3537187850737,"lng":-8.51913581643896},{"lat":41.3539574123831,"lng":-8.51821777615253},{"lat":41.354016015781,"lng":-8.51733493132303},{"lat":41.3526360205673,"lng":-8.51576487548165},{"lat":41.352471428871,"lng":-8.51568396719357},{"lat":41.3494454779173,"lng":-8.51419860459431},{"lat":41.3481824548559,"lng":-8.51488696103725},{"lat":41.3480747683761,"lng":-8.51515892901894},{"lat":41.3480927630021,"lng":-8.51252896836003},{"lat":41.3413729474165,"lng":-8.50613311111435},{"lat":41.3369539279711,"lng":-8.50562831738263},{"lat":41.3346106679946,"lng":-8.50768659573061},{"lat":41.3317706437614,"lng":-8.51016311743101},{"lat":41.329641489484,"lng":-8.51164770619862},{"lat":41.3258707254747,"lng":-8.51297224871337},{"lat":41.3241562547735,"lng":-8.51593680874795},{"lat":41.3232890372931,"lng":-8.51750660498422},{"lat":41.3224416788664,"lng":-8.51878553698343},{"lat":41.321565081364,"lng":-8.52020948202781},{"lat":41.3206048592385,"lng":-8.52163887060228},{"lat":41.3201725938285,"lng":-8.52232226779216},{"lat":41.3198024957622,"lng":-8.52291148891012},{"lat":41.3191808638061,"lng":-8.52425558936462},{"lat":41.31846002815,"lng":-8.5257142012269},{"lat":41.3176488503582,"lng":-8.52718155289406},{"lat":41.3162600597135,"lng":-8.52995254662554},{"lat":41.3148246900517,"lng":-8.53313340524043},{"lat":41.3135861678325,"lng":-8.53651589767424},{"lat":41.3114584196734,"lng":-8.54375702889872},{"lat":41.3102423274777,"lng":-8.55070060074066},{"lat":41.3099920474731,"lng":-8.55401660430127},{"lat":41.3097686528984,"lng":-8.55667750212345},{"lat":41.3096787666598,"lng":-8.55793185629059},{"lat":41.3096249063807,"lng":-8.55923107395548},{"lat":41.3094636553007,"lng":-8.56190527710775},{"lat":41.3093638297132,"lng":-8.56543419795695},{"lat":41.3093107232325,"lng":-8.56896942150508},{"lat":41.3064846051867,"lng":-8.57870652746103}]},{"zone_id":"mai5","points":[{"lat":41.2970449660833,"lng":-8.57565339955001},{"lat":41.2959944994314,"lng":-8.57471087444866},{"lat":41.2934328044021,"lng":-8.57175594638693},{"lat":41.2914374129397,"lng":-8.5702264312707},{"lat":41.2890689519601,"lng":-8.56929957209123},{"lat":41.2850108922809,"lng":-8.56866762095591},{"lat":41.2818701605222,"lng":-8.5676045694375},{"lat":41.2790444561016,"lng":-8.56641877771437},{"lat":41.2740490911437,"lng":-8.56242672666349},{"lat":41.2716229909445,"lng":-8.56136881384518},{"lat":41.2695365638248,"lng":-8.56090856942134},{"lat":41.2681869057266,"lng":-8.55848686005698},{"lat":41.2658346710554,"lng":-8.55832853987904},{"lat":41.2573727798899,"lng":-8.55779622095984},{"lat":41.2530522303709,"lng":-8.55741022266693},{"lat":41.2504419786655,"lng":-8.55715467154919},{"lat":41.2486416369314,"lng":-8.55702369401675},{"lat":41.2454911471423,"lng":-8.55676467772099},{"lat":41.2438717615297,"lng":-8.55639630242218},{"lat":41.2409044193506,"lng":-8.55530349006146},{"lat":41.2386560664165,"lng":-8.5545732638303},{"lat":41.2349688250038,"lng":-8.55335671821265},{"lat":41.2338905397955,"lng":-8.55275341227435},{"lat":41.2338000579978,"lng":-8.55287210258264},{"lat":41.2359234911098,"lng":-8.5630246211629},{"lat":41.2365019210912,"lng":-8.56399524105109},{"lat":41.2382018925538,"lng":-8.56409062775697},{"lat":41.2396564085132,"lng":-8.56390469972956},{"lat":41.2414113066001,"lng":-8.56395989065801},{"lat":41.2430955289186,"lng":-8.56395418198784},{"lat":41.2441685948053,"lng":-8.5643320498358},{"lat":41.2449261605123,"lng":-8.56454088855897},{"lat":41.2464592437067,"lng":-8.56494996676261},{"lat":41.2489886487854,"lng":-8.5667528595308},{"lat":41.251847432641,"lng":-8.56747365383889},{"lat":41.2558203070857,"lng":-8.56625537430909},{"lat":41.2563342026343,"lng":-8.56870549459682},{"lat":41.2566908463909,"lng":-8.5720313505798},{"lat":41.2564727190857,"lng":-8.57274352003328},{"lat":41.255092348175,"lng":-8.57560911217306},{"lat":41.2548716854909,"lng":-8.57697368066762},{"lat":41.2542041018023,"lng":-8.57851870373728},{"lat":41.2536031083331,"lng":-8.58271465735882},{"lat":41.2533967092641,"lng":-8.58434431887659},{"lat":41.2527751110396,"lng":-8.58980409629096},{"lat":41.2527845840682,"lng":-8.59129250568032},{"lat":41.2533037287713,"lng":-8.59229516603806},{"lat":41.2567195388561,"lng":-8.59613192217397},{"lat":41.257665020454,"lng":-8.59722880083864},{"lat":41.2619982208333,"lng":-8.59577273893076},{"lat":41.2657798342743,"lng":-8.59579950011513},{"lat":41.2728875646206,"lng":-8.5971626443078},{"lat":41.2751414122872,"lng":-8.59646253288331},{"lat":41.277311936979,"lng":-8.594090794946},{"lat":41.2802261551953,"lng":-8.59274197472092},{"lat":41.2800921560369,"lng":-8.59198449978698},{"lat":41.2783303250111,"lng":-8.58976711646242},{"lat":41.290732746542,"lng":-8.58800052798689},{"lat":41.2922737817896,"lng":-8.58538487318678},{"lat":41.2964351891066,"lng":-8.5803993837915},{"lat":41.2970449660833,"lng":-8.57565339955001}]},{"zone_id":"mai3","points":[{"lat":41.2523250157297,"lng":-8.61712103986343},{"lat":41.2538343819081,"lng":-8.61676518648254},{"lat":41.2547489268775,"lng":-8.61705716676095},{"lat":41.2570824448937,"lng":-8.61640174388646},{"lat":41.259159547785,"lng":-8.61641705563112},{"lat":41.2602892770763,"lng":-8.61660877359886},{"lat":41.2605309931709,"lng":-8.61636247557236},{"lat":41.2605814184884,"lng":-8.61595838082256},{"lat":41.2603339739776,"lng":-8.61582156645299},{"lat":41.2601495480457,"lng":-8.61516014027386},{"lat":41.2600106577551,"lng":-8.61157039022158},{"lat":41.2600388816158,"lng":-8.60847123800673},{"lat":41.2596801134048,"lng":-8.60591983606939},{"lat":41.2593813327349,"lng":-8.6037359178732},{"lat":41.2594531837694,"lng":-8.6012080320994},{"lat":41.2591682963303,"lng":-8.59937087570314},{"lat":41.2584347991939,"lng":-8.59812186791587},{"lat":41.257665020454,"lng":-8.59722880083864},{"lat":41.2567195388561,"lng":-8.59613192217397},{"lat":41.2533037287713,"lng":-8.59229516603806},{"lat":41.2527845840682,"lng":-8.59129250568032},{"lat":41.2527751110396,"lng":-8.58980409629096},{"lat":41.2533967092641,"lng":-8.58434431887659},{"lat":41.2536031083331,"lng":-8.58271465735882},{"lat":41.2542041018023,"lng":-8.57851870373728},{"lat":41.2548716854909,"lng":-8.57697368066762},{"lat":41.255092348175,"lng":-8.57560911217306},{"lat":41.2564727190857,"lng":-8.57274352003328},{"lat":41.2566908463909,"lng":-8.5720313505798},{"lat":41.2563342026343,"lng":-8.56870549459682},{"lat":41.2558203070857,"lng":-8.56625537430909},{"lat":41.251847432641,"lng":-8.56747365383889},{"lat":41.2489886487854,"lng":-8.5667528595308},{"lat":41.2464592437067,"lng":-8.56494996676261},{"lat":41.2449261605123,"lng":-8.56454088855897},{"lat":41.2441685948053,"lng":-8.5643320498358},{"lat":41.2430955289186,"lng":-8.56395418198784},{"lat":41.2414113066001,"lng":-8.56395989065801},{"lat":41.2396564085132,"lng":-8.56390469972956},{"lat":41.2382018925538,"lng":-8.56409062775697},{"lat":41.2365019210912,"lng":-8.56399524105109},{"lat":41.2362012237805,"lng":-8.56430056843316},{"lat":41.2343846199319,"lng":-8.56531556986083},{"lat":41.2320440455752,"lng":-8.56717283349841},{"lat":41.2293790309508,"lng":-8.56987280992224},{"lat":41.2315067051532,"lng":-8.57332042054282},{"lat":41.2317167243351,"lng":-8.57426479747019},{"lat":41.2316348498216,"lng":-8.57480900377063},{"lat":41.2314467920378,"lng":-8.57504395175096},{"lat":41.2310516693925,"lng":-8.57551975433162},{"lat":41.2305499829535,"lng":-8.57618090739513},{"lat":41.2300381576265,"lng":-8.57685897696883},{"lat":41.2294101480142,"lng":-8.57810464522552},{"lat":41.2287214594936,"lng":-8.58085607773525},{"lat":41.2256530853616,"lng":-8.58274887902271},{"lat":41.2248936270042,"lng":-8.5842680719996},{"lat":41.2241541012694,"lng":-8.58500810595071},{"lat":41.2235766010745,"lng":-8.58534513123734},{"lat":41.2208491648014,"lng":-8.58666935004245},{"lat":41.221010595055,"lng":-8.58905093765231},{"lat":41.2212343294214,"lng":-8.58984576661489},{"lat":41.2214832065759,"lng":-8.59069574565509},{"lat":41.2209146759673,"lng":-8.5911523275391},{"lat":41.2203036549239,"lng":-8.59128180508628},{"lat":41.219910065643,"lng":-8.59139412605573},{"lat":41.2195236139249,"lng":-8.59159953337537},{"lat":41.2188666301445,"lng":-8.59193777419842},{"lat":41.2183715675704,"lng":-8.59216346807495},{"lat":41.2178732591904,"lng":-8.59218687927592},{"lat":41.2173169675239,"lng":-8.59241635126351},{"lat":41.2169416317128,"lng":-8.59270451229514},{"lat":41.2166995261591,"lng":-8.59280287337789},{"lat":41.2163950566184,"lng":-8.59304176420771},{"lat":41.2161456787975,"lng":-8.59333194111348},{"lat":41.2156997610733,"lng":-8.59346998649089},{"lat":41.2154508182981,"lng":-8.59362256984344},{"lat":41.2150122144373,"lng":-8.59402701133823},{"lat":41.2145513119999,"lng":-8.59423762288668},{"lat":41.2144143291007,"lng":-8.59441234391657},{"lat":41.2144496032373,"lng":-8.59488566479966},{"lat":41.2150965436854,"lng":-8.59569544608964},{"lat":41.2154186419896,"lng":-8.59610103287204},{"lat":41.2158082705263,"lng":-8.59662166568911},{"lat":41.2160111681279,"lng":-8.59677853612965},{"lat":41.2162304423618,"lng":-8.59691820673976},{"lat":41.2164208862928,"lng":-8.5970116954006},{"lat":41.2167316394507,"lng":-8.59718367410625},{"lat":41.2171427346286,"lng":-8.59742564380718},{"lat":41.2173994934418,"lng":-8.5975255905857},{"lat":41.2177039160514,"lng":-8.59756677954963},{"lat":41.2186190324265,"lng":-8.5976565014051},{"lat":41.2190791470548,"lng":-8.59761242238939},{"lat":41.2194489201403,"lng":-8.59778008657249},{"lat":41.2203417676533,"lng":-8.59824499401258},{"lat":41.2209782619658,"lng":-8.59854110761638},{"lat":41.2213080934231,"lng":-8.59871775257746},{"lat":41.2217267689038,"lng":-8.59867439021488},{"lat":41.2221488010399,"lng":-8.59864920677798},{"lat":41.2230421538205,"lng":-8.5985404718378},{"lat":41.2233794587416,"lng":-8.59854950990611},{"lat":41.2237330640708,"lng":-8.59858562552766},{"lat":41.2243000317496,"lng":-8.59869755962256},{"lat":41.2247742008339,"lng":-8.59890137116994},{"lat":41.2255206630245,"lng":-8.59950483622896},{"lat":41.2260228108348,"lng":-8.59988454044856},{"lat":41.2263049595565,"lng":-8.60012710665071},{"lat":41.2266362934624,"lng":-8.60025532873359},{"lat":41.2269345547209,"lng":-8.60029147211553},{"lat":41.2276179929891,"lng":-8.60022781975941},{"lat":41.2289941919472,"lng":-8.60004853333958},{"lat":41.2303617769281,"lng":-8.59956918592401},{"lat":41.2308968377575,"lng":-8.59879201074011},{"lat":41.2311369841093,"lng":-8.5984183016311},{"lat":41.2319933675686,"lng":-8.59776822296904},{"lat":41.2328849788626,"lng":-8.59722862976535},{"lat":41.2332039624009,"lng":-8.5966973404149},{"lat":41.2335622416973,"lng":-8.59610842667436},{"lat":41.2339159709406,"lng":-8.59555317950123},{"lat":41.2342045073374,"lng":-8.59514364162544},{"lat":41.2358172451339,"lng":-8.59798829020682},{"lat":41.2362500950908,"lng":-8.59892796629664},{"lat":41.2371032326317,"lng":-8.59850305146582},{"lat":41.2375783557898,"lng":-8.59814727510128},{"lat":41.238317847589,"lng":-8.59777929559913},{"lat":41.2389007084801,"lng":-8.59954420050839},{"lat":41.239370945823,"lng":-8.60089096304777},{"lat":41.2399919816999,"lng":-8.60244508389925},{"lat":41.2400720296784,"lng":-8.60356679360359},{"lat":41.2398447920316,"lng":-8.6061411004252},{"lat":41.239453374898,"lng":-8.61052935919304},{"lat":41.2391076682933,"lng":-8.61340753672409},{"lat":41.2390805126593,"lng":-8.61635064268622},{"lat":41.2392771915943,"lng":-8.6171470696314},{"lat":41.2400044631389,"lng":-8.61982277657332},{"lat":41.2396931681127,"lng":-8.62400694210306},{"lat":41.2398750743707,"lng":-8.62476505491994},{"lat":41.2413112213497,"lng":-8.6237055623247},{"lat":41.2448135426691,"lng":-8.62144855001981},{"lat":41.2465349456705,"lng":-8.62047666341925},{"lat":41.247898045516,"lng":-8.62005610963543},{"lat":41.2482915251622,"lng":-8.6198084573423},{"lat":41.2485152872682,"lng":-8.619435838283},{"lat":41.2502242369394,"lng":-8.61811002048088},{"lat":41.2523250157297,"lng":-8.61712103986343}]},{"zone_id":"mai2","points":[{"lat":41.2538343819081,"lng":-8.61676518648254},{"lat":41.2523250157297,"lng":-8.61712103986343},{"lat":41.2502242369394,"lng":-8.61811002048088},{"lat":41.2485152872682,"lng":-8.619435838283},{"lat":41.2482915251622,"lng":-8.6198084573423},{"lat":41.247898045516,"lng":-8.62005610963543},{"lat":41.2465349456705,"lng":-8.62047666341925},{"lat":41.2448135426691,"lng":-8.62144855001981},{"lat":41.2413112213497,"lng":-8.6237055623247},{"lat":41.2398750743707,"lng":-8.62476505491994},{"lat":41.2407013825324,"lng":-8.62500849730599},{"lat":41.2411461919024,"lng":-8.62577505836657},{"lat":41.2412991947153,"lng":-8.62703041514883},{"lat":41.2410961289333,"lng":-8.62814530697287},{"lat":41.2412963013748,"lng":-8.6288080283107},{"lat":41.2411050443284,"lng":-8.62994519786481},{"lat":41.2399144113857,"lng":-8.63313451033851},{"lat":41.2393638684555,"lng":-8.63630734042799},{"lat":41.2391853255939,"lng":-8.63843982810748},{"lat":41.239440660754,"lng":-8.6401309752147},{"lat":41.2399354795701,"lng":-8.64266167228321},{"lat":41.2397433043014,"lng":-8.64386895628147},{"lat":41.2392928274821,"lng":-8.64409609508339},{"lat":41.2390091352769,"lng":-8.64453400194629},{"lat":41.2397865282249,"lng":-8.64525729958867},{"lat":41.2398921612921,"lng":-8.64568217240789},{"lat":41.2398603449832,"lng":-8.64661935263393},{"lat":41.2403290565107,"lng":-8.64726023444095},{"lat":41.2404022131547,"lng":-8.64758916003502},{"lat":41.2403551936062,"lng":-8.64794113713279},{"lat":41.2401085847947,"lng":-8.6485391325314},{"lat":41.2411267579689,"lng":-8.64932932456459},{"lat":41.241687594799,"lng":-8.64899677596336},{"lat":41.2423815041305,"lng":-8.64876439271487},{"lat":41.2431902401402,"lng":-8.64897273801171},{"lat":41.2434065982136,"lng":-8.64909072407472},{"lat":41.2437357487935,"lng":-8.64904965536083},{"lat":41.2438950062294,"lng":-8.64880059288921},{"lat":41.2443629613316,"lng":-8.64853282104172},{"lat":41.2456319871716,"lng":-8.64876730173073},{"lat":41.2476075395584,"lng":-8.64907198335042},{"lat":41.2483541075042,"lng":-8.64927954057845},{"lat":41.250061231686,"lng":-8.65009215052111},{"lat":41.2543445522172,"lng":-8.65393045402095},{"lat":41.2544320782325,"lng":-8.65412865493957},{"lat":41.2545970904449,"lng":-8.65423490118786},{"lat":41.2548949930745,"lng":-8.65421836292547},{"lat":41.2560370531603,"lng":-8.65482132423756},{"lat":41.256580388404,"lng":-8.65529708368746},{"lat":41.2571677675868,"lng":-8.65650137330594},{"lat":41.2573900982786,"lng":-8.65637102757905},{"lat":41.257558310614,"lng":-8.65593964901095},{"lat":41.258744232946,"lng":-8.6525939550607},{"lat":41.2595278213435,"lng":-8.65291386557544},{"lat":41.2604260430186,"lng":-8.65062666413641},{"lat":41.2617423379303,"lng":-8.65130231250987},{"lat":41.2630906348341,"lng":-8.65282154633536},{"lat":41.2640771042822,"lng":-8.65388922355539},{"lat":41.2652312502821,"lng":-8.65317525334722},{"lat":41.2671735908766,"lng":-8.65373839002898},{"lat":41.2683238044333,"lng":-8.65357803140413},{"lat":41.272918960859,"lng":-8.65289852183213},{"lat":41.2749046413781,"lng":-8.65184014316546},{"lat":41.2759004374516,"lng":-8.65065450827348},{"lat":41.2784609458326,"lng":-8.64184226408683},{"lat":41.2790227425596,"lng":-8.63695290211475},{"lat":41.2791645035134,"lng":-8.63620399947648},{"lat":41.2801287025197,"lng":-8.63111271890451},{"lat":41.2839272983724,"lng":-8.62720251154353},{"lat":41.2846552891112,"lng":-8.6254174710419},{"lat":41.2882644583326,"lng":-8.62365402198006},{"lat":41.2894130977635,"lng":-8.62342845812394},{"lat":41.2928563774278,"lng":-8.623688503375},{"lat":41.2953655446573,"lng":-8.62181972078403},{"lat":41.2945699312371,"lng":-8.61835154098201},{"lat":41.293503097849,"lng":-8.6151202093758},{"lat":41.2917163766938,"lng":-8.61176431204814},{"lat":41.2882218229185,"lng":-8.60767989105436},{"lat":41.2854434902991,"lng":-8.60455603836642},{"lat":41.2822995456512,"lng":-8.60274283196159},{"lat":41.2805912843494,"lng":-8.60213371745315},{"lat":41.2810560770551,"lng":-8.59855620040702},{"lat":41.2802261551953,"lng":-8.59274197472092},{"lat":41.277311936979,"lng":-8.594090794946},{"lat":41.2751414122872,"lng":-8.59646253288331},{"lat":41.2728875646206,"lng":-8.5971626443078},{"lat":41.2657798342743,"lng":-8.59579950011513},{"lat":41.2619982208333,"lng":-8.59577273893076},{"lat":41.257665020454,"lng":-8.59722880083864},{"lat":41.2584347991939,"lng":-8.59812186791587},{"lat":41.2591682963303,"lng":-8.59937087570314},{"lat":41.2594531837694,"lng":-8.6012080320994},{"lat":41.2593813327349,"lng":-8.6037359178732},{"lat":41.2596801134048,"lng":-8.60591983606939},{"lat":41.2600388816158,"lng":-8.60847123800673},{"lat":41.2600106577551,"lng":-8.61157039022158},{"lat":41.2601495480457,"lng":-8.61516014027386},{"lat":41.2603339739776,"lng":-8.61582156645299},{"lat":41.2605814184884,"lng":-8.61595838082256},{"lat":41.2605309931709,"lng":-8.61636247557236},{"lat":41.2602892770763,"lng":-8.61660877359886},{"lat":41.259159547785,"lng":-8.61641705563112},{"lat":41.2570824448937,"lng":-8.61640174388646},{"lat":41.2547489268775,"lng":-8.61705716676095},{"lat":41.2538343819081,"lng":-8.61676518648254}]},{"zone_id":"vcd11","points":[{"lat":41.3525300251361,"lng":-8.61339679709659},{"lat":41.3513145841937,"lng":-8.61281272830336},{"lat":41.3469047965402,"lng":-8.61118082019064},{"lat":41.3434570943184,"lng":-8.61096672359442},{"lat":41.3411116449136,"lng":-8.61062357901041},{"lat":41.3412639619689,"lng":-8.61114894186849},{"lat":41.3414404226948,"lng":-8.61195142576451},{"lat":41.341475537041,"lng":-8.61210281827785},{"lat":41.3414814304836,"lng":-8.6123082370198},{"lat":41.3414797927256,"lng":-8.61295827526895},{"lat":41.3414145119984,"lng":-8.61323796120863},{"lat":41.3413654,"lng":-8.61339793146182},{"lat":41.3412354330755,"lng":-8.61360126651168},{"lat":41.3411413295993,"lng":-8.61375421469981},{"lat":41.3410439831658,"lng":-8.61388634896965},{"lat":41.3409326173325,"lng":-8.61401156971855},{"lat":41.3408379580398,"lng":-8.61410477464051},{"lat":41.3407144748121,"lng":-8.61422082667869},{"lat":41.3403680154258,"lng":-8.61451825679403},{"lat":41.3402314037533,"lng":-8.61463110424239},{"lat":41.3400763294784,"lng":-8.61476603220575},{"lat":41.3399509249863,"lng":-8.61488875671909},{"lat":41.3398140529283,"lng":-8.61506348580185},{"lat":41.3397223711771,"lng":-8.61519756757488},{"lat":41.3396043168226,"lng":-8.61535045273306},{"lat":41.3394663179576,"lng":-8.61551393776802},{"lat":41.3393377644419,"lng":-8.61565801812556},{"lat":41.3392308054084,"lng":-8.61578422569032},{"lat":41.3391066744393,"lng":-8.61592570812429},{"lat":41.3389837259892,"lng":-8.61606493480268},{"lat":41.3388762547824,"lng":-8.6161845616214},{"lat":41.3387676865639,"lng":-8.61630787991304},{"lat":41.338631703573,"lng":-8.61646313714808},{"lat":41.3385265895022,"lng":-8.61657883642808},{"lat":41.3384050122243,"lng":-8.61671233628645},{"lat":41.3383265722473,"lng":-8.61679358737922},{"lat":41.3381656822495,"lng":-8.61692683134058},{"lat":41.3379857634232,"lng":-8.61705973786172},{"lat":41.3377626167683,"lng":-8.61725901764022},{"lat":41.3375294265404,"lng":-8.61744783397469},{"lat":41.337404354475,"lng":-8.61755549462957},{"lat":41.3372633878291,"lng":-8.61769685096333},{"lat":41.3371609701106,"lng":-8.61781340689186},{"lat":41.3368847822239,"lng":-8.61811945053384},{"lat":41.3367025434364,"lng":-8.61831173835544},{"lat":41.3365741547866,"lng":-8.61845879861221},{"lat":41.3364670777544,"lng":-8.61863337411733},{"lat":41.3363823177372,"lng":-8.61876809518682},{"lat":41.3363053247051,"lng":-8.61887694730726},{"lat":41.3362356131151,"lng":-8.61900927080281},{"lat":41.3359803533993,"lng":-8.61960348839079},{"lat":41.3359076833173,"lng":-8.61975323142516},{"lat":41.3358871489568,"lng":-8.61996691492089},{"lat":41.3358394961203,"lng":-8.62050210743331},{"lat":41.3358374835041,"lng":-8.62078486662053},{"lat":41.3358724317095,"lng":-8.62109931118321},{"lat":41.3360569449549,"lng":-8.62138907711746},{"lat":41.3362256486842,"lng":-8.62154421263157},{"lat":41.3365260391196,"lng":-8.62189852985104},{"lat":41.3367650389685,"lng":-8.62214366870055},{"lat":41.3369656093837,"lng":-8.62227897476578},{"lat":41.3371862073885,"lng":-8.62240559341153},{"lat":41.3373939746433,"lng":-8.62252136841659},{"lat":41.3377417716817,"lng":-8.6227170377564},{"lat":41.337887593605,"lng":-8.62276926448544},{"lat":41.3380622095101,"lng":-8.62278384166452},{"lat":41.33826056427,"lng":-8.62278389851058},{"lat":41.3384582679558,"lng":-8.6227257702774},{"lat":41.3386501202411,"lng":-8.62260404496423},{"lat":41.3389295449653,"lng":-8.62246983279286},{"lat":41.3391258763729,"lng":-8.62235470068874},{"lat":41.3392497407979,"lng":-8.62227630884166},{"lat":41.3394907945002,"lng":-8.62210369461596},{"lat":41.3396265405562,"lng":-8.62200579028799},{"lat":41.3397563338536,"lng":-8.62191023883724},{"lat":41.339895547773,"lng":-8.62180244588355},{"lat":41.3400494001081,"lng":-8.62168114447092},{"lat":41.3401995359801,"lng":-8.62156650396931},{"lat":41.340336742672,"lng":-8.62146335399158},{"lat":41.3404634552727,"lng":-8.62137267102922},{"lat":41.3406657759883,"lng":-8.62122377160401},{"lat":41.3408333821399,"lng":-8.62110866561936},{"lat":41.3409780526577,"lng":-8.62102826938545},{"lat":41.3411465260917,"lng":-8.62098508256985},{"lat":41.341282769708,"lng":-8.62098156416502},{"lat":41.3414586113948,"lng":-8.620982287523},{"lat":41.3416397510297,"lng":-8.62098662989596},{"lat":41.3418141744789,"lng":-8.62106787013399},{"lat":41.3419804986723,"lng":-8.62118882610473},{"lat":41.3421530831781,"lng":-8.62136371301698},{"lat":41.3422916647156,"lng":-8.62159307157467},{"lat":41.3424959881799,"lng":-8.62198839263688},{"lat":41.3299299578624,"lng":-8.63159318676851},{"lat":41.3226720697455,"lng":-8.63361773877563},{"lat":41.3198487103598,"lng":-8.64396507086816},{"lat":41.3175997345218,"lng":-8.65143658483418},{"lat":41.3157038186259,"lng":-8.65925261625676},{"lat":41.3143227473202,"lng":-8.66752368794406},{"lat":41.318830610479,"lng":-8.66647745060084},{"lat":41.3197408915732,"lng":-8.66651868648839},{"lat":41.3231941101497,"lng":-8.6665870175356},{"lat":41.3288479695557,"lng":-8.66341597541044},{"lat":41.3326549796443,"lng":-8.66274420553036},{"lat":41.3390050670303,"lng":-8.66987571269762},{"lat":41.340977060347,"lng":-8.67007087849195},{"lat":41.3425739241193,"lng":-8.66971973881738},{"lat":41.343926255584,"lng":-8.66942236342979},{"lat":41.3466081835486,"lng":-8.66901062529472},{"lat":41.3495619580174,"lng":-8.6685085978663},{"lat":41.3520637804532,"lng":-8.64944803421536},{"lat":41.351996845243,"lng":-8.63078589914483},{"lat":41.3525300251361,"lng":-8.61339679709659}]},{"zone_id":"vcd12","points":[{"lat":41.2997711789279,"lng":-8.67304830868219},{"lat":41.3008234160226,"lng":-8.67356664464292},{"lat":41.3028042547918,"lng":-8.67450831703959},{"lat":41.3090135401923,"lng":-8.67052317966686},{"lat":41.3119981256059,"lng":-8.66768175909055},{"lat":41.3143227473202,"lng":-8.66752368794406},{"lat":41.3157038186259,"lng":-8.65925261625676},{"lat":41.3175997345218,"lng":-8.65143658483418},{"lat":41.3198487103598,"lng":-8.64396507086816},{"lat":41.3226720697455,"lng":-8.63361773877563},{"lat":41.3220780570362,"lng":-8.63378342570109},{"lat":41.3195511621037,"lng":-8.634488251797},{"lat":41.3180423538253,"lng":-8.63385261868511},{"lat":41.314881891216,"lng":-8.63227819745704},{"lat":41.312619864641,"lng":-8.63115169687821},{"lat":41.3120772491286,"lng":-8.63093116216056},{"lat":41.3113001380214,"lng":-8.63061509782523},{"lat":41.3112835178409,"lng":-8.63081368766189},{"lat":41.3110580313634,"lng":-8.63108960740684},{"lat":41.3109744631668,"lng":-8.63115500408853},{"lat":41.3108470833215,"lng":-8.63114806489551},{"lat":41.3105611164887,"lng":-8.63096138048585},{"lat":41.3103470962171,"lng":-8.63081561041223},{"lat":41.3101352478811,"lng":-8.6305426796257},{"lat":41.3099924679327,"lng":-8.63030879407127},{"lat":41.3098673877212,"lng":-8.63008339198429},{"lat":41.3097700345132,"lng":-8.62975604828879},{"lat":41.3096852025908,"lng":-8.62946749623226},{"lat":41.3096119648254,"lng":-8.62916445924361},{"lat":41.3095327330809,"lng":-8.62887188614808},{"lat":41.3094594323001,"lng":-8.62858186878877},{"lat":41.3094089814516,"lng":-8.62846469803443},{"lat":41.3093057803352,"lng":-8.62807152030506},{"lat":41.3032491908871,"lng":-8.62572247484382},{"lat":41.3019407355483,"lng":-8.62513349460263},{"lat":41.298963577142,"lng":-8.62410117873415},{"lat":41.2988877288258,"lng":-8.62387805791768},{"lat":41.2989302780899,"lng":-8.62365618415408},{"lat":41.2989020781897,"lng":-8.62353359115068},{"lat":41.2988217187723,"lng":-8.6233131799626},{"lat":41.2986282644975,"lng":-8.62309275267983},{"lat":41.2984512286181,"lng":-8.62290707793968},{"lat":41.298263976026,"lng":-8.62273015949552},{"lat":41.2980626417287,"lng":-8.62258370556078},{"lat":41.297877079244,"lng":-8.62241301385019},{"lat":41.297608302987,"lng":-8.62224551964926},{"lat":41.2965793688054,"lng":-8.6219615271607},{"lat":41.2961477702463,"lng":-8.62182158796058},{"lat":41.2953655446573,"lng":-8.62181972078403},{"lat":41.2928563774278,"lng":-8.623688503375},{"lat":41.2894130977635,"lng":-8.62342845812394},{"lat":41.2882644583326,"lng":-8.62365402198006},{"lat":41.2846552891112,"lng":-8.6254174710419},{"lat":41.2839272983724,"lng":-8.62720251154353},{"lat":41.2801287025197,"lng":-8.63111271890451},{"lat":41.2791645035134,"lng":-8.63620399947648},{"lat":41.2813384102976,"lng":-8.63818657606839},{"lat":41.2803385164967,"lng":-8.64032732252234},{"lat":41.2796081625281,"lng":-8.6425894684515},{"lat":41.2807738579846,"lng":-8.64367282940963},{"lat":41.2813977237646,"lng":-8.64511006523761},{"lat":41.2819352753201,"lng":-8.64571110395779},{"lat":41.2842719505856,"lng":-8.64668441708768},{"lat":41.2847157047398,"lng":-8.64812033590472},{"lat":41.2854290122838,"lng":-8.64967777922671},{"lat":41.2865973330673,"lng":-8.65016452419293},{"lat":41.2871299991999,"lng":-8.65183997073172},{"lat":41.2856780423981,"lng":-8.65433519616096},{"lat":41.2851318445211,"lng":-8.65564389272575},{"lat":41.2855733023847,"lng":-8.65755735303952},{"lat":41.2861036678653,"lng":-8.65971029904598},{"lat":41.2868217634602,"lng":-8.6601935883973},{"lat":41.2933318465574,"lng":-8.66162871006596},{"lat":41.2941882063007,"lng":-8.66383472437267},{"lat":41.295493524758,"lng":-8.67058126544242},{"lat":41.2981102252528,"lng":-8.67234356066137},{"lat":41.2997711789279,"lng":-8.67304830868219}]},{"zone_id":"trf2","points":[{"lat":41.3226720697455,"lng":-8.63361773877563},{"lat":41.3299299578624,"lng":-8.63159318676851},{"lat":41.3424959881799,"lng":-8.62198839263688},{"lat":41.3422916647156,"lng":-8.62159307157467},{"lat":41.3421530831781,"lng":-8.62136371301698},{"lat":41.3419804986723,"lng":-8.62118882610473},{"lat":41.3418141744789,"lng":-8.62106787013399},{"lat":41.3416397510297,"lng":-8.62098662989596},{"lat":41.3414586113948,"lng":-8.620982287523},{"lat":41.341282769708,"lng":-8.62098156416502},{"lat":41.3411465260917,"lng":-8.62098508256985},{"lat":41.3409780526577,"lng":-8.62102826938545},{"lat":41.3408333821399,"lng":-8.62110866561936},{"lat":41.3406657759883,"lng":-8.62122377160401},{"lat":41.3404634552727,"lng":-8.62137267102922},{"lat":41.340336742672,"lng":-8.62146335399158},{"lat":41.3401995359801,"lng":-8.62156650396931},{"lat":41.3400494001081,"lng":-8.62168114447092},{"lat":41.339895547773,"lng":-8.62180244588355},{"lat":41.3397563338536,"lng":-8.62191023883724},{"lat":41.3396265405562,"lng":-8.62200579028799},{"lat":41.3394907945002,"lng":-8.62210369461596},{"lat":41.3392497407979,"lng":-8.62227630884166},{"lat":41.3391258763729,"lng":-8.62235470068874},{"lat":41.3389295449653,"lng":-8.62246983279286},{"lat":41.3386501202411,"lng":-8.62260404496423},{"lat":41.3384582679558,"lng":-8.6227257702774},{"lat":41.33826056427,"lng":-8.62278389851058},{"lat":41.3380622095101,"lng":-8.62278384166452},{"lat":41.337887593605,"lng":-8.62276926448544},{"lat":41.3377417716817,"lng":-8.6227170377564},{"lat":41.3373939746433,"lng":-8.62252136841659},{"lat":41.3371862073885,"lng":-8.62240559341153},{"lat":41.3369656093837,"lng":-8.62227897476578},{"lat":41.3367650389685,"lng":-8.62214366870055},{"lat":41.3365260391196,"lng":-8.62189852985104},{"lat":41.3362256486842,"lng":-8.62154421263157},{"lat":41.3360569449549,"lng":-8.62138907711746},{"lat":41.3358724317095,"lng":-8.62109931118321},{"lat":41.3358374835041,"lng":-8.62078486662053},{"lat":41.3358394961203,"lng":-8.62050210743331},{"lat":41.3358871489568,"lng":-8.61996691492089},{"lat":41.3359076833173,"lng":-8.61975323142516},{"lat":41.3359803533993,"lng":-8.61960348839079},{"lat":41.3362356131151,"lng":-8.61900927080281},{"lat":41.3363053247051,"lng":-8.61887694730726},{"lat":41.3363823177372,"lng":-8.61876809518682},{"lat":41.3364670777544,"lng":-8.61863337411733},{"lat":41.3365741547866,"lng":-8.61845879861221},{"lat":41.3367025434364,"lng":-8.61831173835544},{"lat":41.3368847822239,"lng":-8.61811945053384},{"lat":41.3371609701106,"lng":-8.61781340689186},{"lat":41.3372633878291,"lng":-8.61769685096333},{"lat":41.337404354475,"lng":-8.61755549462957},{"lat":41.3375294265404,"lng":-8.61744783397469},{"lat":41.3377626167683,"lng":-8.61725901764022},{"lat":41.3379857634232,"lng":-8.61705973786172},{"lat":41.3381656822495,"lng":-8.61692683134058},{"lat":41.3383265722473,"lng":-8.61679358737922},{"lat":41.3384050122243,"lng":-8.61671233628645},{"lat":41.3385265895022,"lng":-8.61657883642808},{"lat":41.338631703573,"lng":-8.61646313714808},{"lat":41.3387676865639,"lng":-8.61630787991304},{"lat":41.3388762547824,"lng":-8.6161845616214},{"lat":41.3389837259892,"lng":-8.61606493480268},{"lat":41.3391066744393,"lng":-8.61592570812429},{"lat":41.3392308054084,"lng":-8.61578422569032},{"lat":41.3393377644419,"lng":-8.61565801812556},{"lat":41.3394663179576,"lng":-8.61551393776802},{"lat":41.3396043168226,"lng":-8.61535045273306},{"lat":41.3397223711771,"lng":-8.61519756757488},{"lat":41.3398140529283,"lng":-8.61506348580185},{"lat":41.3399509249863,"lng":-8.61488875671909},{"lat":41.3400763294784,"lng":-8.61476603220575},{"lat":41.3402314037533,"lng":-8.61463110424239},{"lat":41.3403680154258,"lng":-8.61451825679403},{"lat":41.3407144748121,"lng":-8.61422082667869},{"lat":41.3408379580398,"lng":-8.61410477464051},{"lat":41.3409326173325,"lng":-8.61401156971855},{"lat":41.3410439831658,"lng":-8.61388634896965},{"lat":41.3411413295993,"lng":-8.61375421469981},{"lat":41.3412354330755,"lng":-8.61360126651168},{"lat":41.3413654,"lng":-8.61339793146182},{"lat":41.3414145119984,"lng":-8.61323796120863},{"lat":41.3414797927256,"lng":-8.61295827526895},{"lat":41.3414814304836,"lng":-8.6123082370198},{"lat":41.341475537041,"lng":-8.61210281827785},{"lat":41.3414404226948,"lng":-8.61195142576451},{"lat":41.3412639619689,"lng":-8.61114894186849},{"lat":41.3411116449136,"lng":-8.61062357901041},{"lat":41.3410806064963,"lng":-8.61050997281899},{"lat":41.3410111541762,"lng":-8.61030098400753},{"lat":41.3408894001365,"lng":-8.60993845141083},{"lat":41.3405521832261,"lng":-8.60903337543198},{"lat":41.3404635877129,"lng":-8.60881624590036},{"lat":41.3403293119493,"lng":-8.60849699416599},{"lat":41.3401694797716,"lng":-8.60815270303307},{"lat":41.3399052761862,"lng":-8.60761555360054},{"lat":41.3395975291386,"lng":-8.60678729652835},{"lat":41.3393673448621,"lng":-8.60629651444681},{"lat":41.3391493544962,"lng":-8.6058180063808},{"lat":41.3389733063222,"lng":-8.6055168637665},{"lat":41.3388747055093,"lng":-8.60534375156288},{"lat":41.338708004041,"lng":-8.60502571549344},{"lat":41.3386363916354,"lng":-8.60485734648483},{"lat":41.3385442659972,"lng":-8.60462061057369},{"lat":41.3385373136114,"lng":-8.60438568639876},{"lat":41.3385375257799,"lng":-8.60407077192762},{"lat":41.3385470483778,"lng":-8.60385686759567},{"lat":41.3385797066545,"lng":-8.60366583604539},{"lat":41.3387087342506,"lng":-8.60340274790536},{"lat":41.3388475032943,"lng":-8.60326672272837},{"lat":41.3390160104189,"lng":-8.60312373541047},{"lat":41.3393256184267,"lng":-8.60297938285051},{"lat":41.3395140671804,"lng":-8.60291431742692},{"lat":41.3396535236086,"lng":-8.60287446763655},{"lat":41.3399151947728,"lng":-8.60280419663481},{"lat":41.3400639515436,"lng":-8.60275748381457},{"lat":41.340560558424,"lng":-8.60263777587411},{"lat":41.3409042316825,"lng":-8.60257334632738},{"lat":41.34112009481,"lng":-8.60256391573143},{"lat":41.3415360231757,"lng":-8.60255460522909},{"lat":41.3417637954582,"lng":-8.60251814156841},{"lat":41.3419436179266,"lng":-8.60251513056501},{"lat":41.3421064958891,"lng":-8.60251559289596},{"lat":41.3422669413894,"lng":-8.60251638902479},{"lat":41.3424480701787,"lng":-8.60252379138679},{"lat":41.3426280476024,"lng":-8.60254790748055},{"lat":41.3428625870061,"lng":-8.60257420773635},{"lat":41.3433103314937,"lng":-8.60260121874472},{"lat":41.3434847365408,"lng":-8.60262350080236},{"lat":41.3436351448826,"lng":-8.60265637203455},{"lat":41.3438058062733,"lng":-8.60269117154534},{"lat":41.3439531300104,"lng":-8.60270920553259},{"lat":41.3441184888917,"lng":-8.6027415764771},{"lat":41.344267519318,"lng":-8.60275950345426},{"lat":41.3444868985544,"lng":-8.60277100139366},{"lat":41.3447339651641,"lng":-8.60277170974911},{"lat":41.3450235262122,"lng":-8.60277224435148},{"lat":41.3452589918069,"lng":-8.60270392736742},{"lat":41.3453495023372,"lng":-8.60265428066795},{"lat":41.3454704445415,"lng":-8.60254045051056},{"lat":41.34556634396,"lng":-8.60240600987911},{"lat":41.3456217368477,"lng":-8.60227020275154},{"lat":41.34568763812,"lng":-8.60211762499895},{"lat":41.3457888134055,"lng":-8.60192682543519},{"lat":41.3458535843573,"lng":-8.6017872576622},{"lat":41.3459022545931,"lng":-8.60164292153693},{"lat":41.3459528120722,"lng":-8.60152177240042},{"lat":41.3460272657816,"lng":-8.60128143208516},{"lat":41.3460927657954,"lng":-8.60102836072494},{"lat":41.3461279659926,"lng":-8.60074353210315},{"lat":41.3461738172155,"lng":-8.6004733538894},{"lat":41.346231735819,"lng":-8.60013575588454},{"lat":41.3463101096087,"lng":-8.59981706462433},{"lat":41.3464412255457,"lng":-8.59948082402765},{"lat":41.346556185612,"lng":-8.59915343175535},{"lat":41.3468637817218,"lng":-8.59830646849698},{"lat":41.346921129845,"lng":-8.59784257296704},{"lat":41.3469176379205,"lng":-8.59730379601797},{"lat":41.3468440511392,"lng":-8.59673178551905},{"lat":41.3465459437114,"lng":-8.59570366187992},{"lat":41.3463364520295,"lng":-8.59498193519815},{"lat":41.3461934909213,"lng":-8.59421647497173},{"lat":41.3460679869518,"lng":-8.59309413307765},{"lat":41.3460393357607,"lng":-8.59217750695375},{"lat":41.346075243168,"lng":-8.5914400819163},{"lat":41.3461000940974,"lng":-8.5909225433426},{"lat":41.3461187878049,"lng":-8.59061608668958},{"lat":41.3461720614239,"lng":-8.59001403619757},{"lat":41.3462020032837,"lng":-8.58984350517516},{"lat":41.3463179214354,"lng":-8.58847696941985},{"lat":41.3463507075714,"lng":-8.58808888243586},{"lat":41.3465140431261,"lng":-8.58652848117762},{"lat":41.346550951414,"lng":-8.58577005133827},{"lat":41.34643285398,"lng":-8.58566647040204},{"lat":41.325402695834,"lng":-8.57786732340612},{"lat":41.3189246510498,"lng":-8.58736080472233},{"lat":41.3157779894238,"lng":-8.58683041301259},{"lat":41.308826065246,"lng":-8.58458926251087},{"lat":41.3064846051867,"lng":-8.57870652746103},{"lat":41.3024244706131,"lng":-8.57580448416316},{"lat":41.2970449660833,"lng":-8.57565339955001},{"lat":41.2964351891066,"lng":-8.5803993837915},{"lat":41.2922737817896,"lng":-8.58538487318678},{"lat":41.290732746542,"lng":-8.58800052798689},{"lat":41.2783303250111,"lng":-8.58976711646242},{"lat":41.2800921560369,"lng":-8.59198449978698},{"lat":41.2802261551953,"lng":-8.59274197472092},{"lat":41.2810560770551,"lng":-8.59855620040702},{"lat":41.2805912843494,"lng":-8.60213371745315},{"lat":41.2822995456512,"lng":-8.60274283196159},{"lat":41.2854434902991,"lng":-8.60455603836642},{"lat":41.2882218229185,"lng":-8.60767989105436},{"lat":41.2917163766938,"lng":-8.61176431204814},{"lat":41.293503097849,"lng":-8.6151202093758},{"lat":41.2945699312371,"lng":-8.61835154098201},{"lat":41.2953655446573,"lng":-8.62181972078403},{"lat":41.2961477702463,"lng":-8.62182158796058},{"lat":41.2965793688054,"lng":-8.6219615271607},{"lat":41.297608302987,"lng":-8.62224551964926},{"lat":41.297877079244,"lng":-8.62241301385019},{"lat":41.2980626417287,"lng":-8.62258370556078},{"lat":41.298263976026,"lng":-8.62273015949552},{"lat":41.2984512286181,"lng":-8.62290707793968},{"lat":41.2986282644975,"lng":-8.62309275267983},{"lat":41.2988217187723,"lng":-8.6233131799626},{"lat":41.2989020781897,"lng":-8.62353359115068},{"lat":41.2989302780899,"lng":-8.62365618415408},{"lat":41.2988877288258,"lng":-8.62387805791768},{"lat":41.298963577142,"lng":-8.62410117873415},{"lat":41.3019407355483,"lng":-8.62513349460263},{"lat":41.3032491908871,"lng":-8.62572247484382},{"lat":41.3093057803352,"lng":-8.62807152030506},{"lat":41.3094089814516,"lng":-8.62846469803443},{"lat":41.3094594323001,"lng":-8.62858186878877},{"lat":41.3095327330809,"lng":-8.62887188614808},{"lat":41.3096119648254,"lng":-8.62916445924361},{"lat":41.3096852025908,"lng":-8.62946749623226},{"lat":41.3097700345132,"lng":-8.62975604828879},{"lat":41.3098673877212,"lng":-8.63008339198429},{"lat":41.3099924679327,"lng":-8.63030879407127},{"lat":41.3101352478811,"lng":-8.6305426796257},{"lat":41.3103470962171,"lng":-8.63081561041223},{"lat":41.3105611164887,"lng":-8.63096138048585},{"lat":41.3108470833215,"lng":-8.63114806489551},{"lat":41.3109744631668,"lng":-8.63115500408853},{"lat":41.3110580313634,"lng":-8.63108960740684},{"lat":41.3112835178409,"lng":-8.63081368766189},{"lat":41.3113001380214,"lng":-8.63061509782523},{"lat":41.3120772491286,"lng":-8.63093116216056},{"lat":41.312619864641,"lng":-8.63115169687821},{"lat":41.314881891216,"lng":-8.63227819745704},{"lat":41.3180423538253,"lng":-8.63385261868511},{"lat":41.3195511621037,"lng":-8.634488251797},{"lat":41.3220780570362,"lng":-8.63378342570109},{"lat":41.3226720697455,"lng":-8.63361773877563}]},{"zone_id":"vcd3","points":[{"lat":41.3053902717472,"lng":-8.73956419819331},{"lat":41.3058547947976,"lng":-8.73978811240424},{"lat":41.3061665880026,"lng":-8.73987806307058},{"lat":41.3067570104176,"lng":-8.73991053778605},{"lat":41.3071901168291,"lng":-8.73991146251565},{"lat":41.3075800418499,"lng":-8.7399210607805},{"lat":41.3078379358119,"lng":-8.74000454102082},{"lat":41.308183311537,"lng":-8.74024825783503},{"lat":41.3084398235362,"lng":-8.74047311981204},{"lat":41.3086902085049,"lng":-8.74058065615717},{"lat":41.308963800325,"lng":-8.74060708590189},{"lat":41.3092566898539,"lng":-8.74073018719603},{"lat":41.3094224972625,"lng":-8.74087610875978},{"lat":41.3095629089735,"lng":-8.74109069353791},{"lat":41.3097102142676,"lng":-8.74132983359015},{"lat":41.3098524756554,"lng":-8.74167090545189},{"lat":41.3099509558652,"lng":-8.74193048569169},{"lat":41.3100459026932,"lng":-8.74217785535031},{"lat":41.3100935378185,"lng":-8.74253321491489},{"lat":41.3102006369297,"lng":-8.74254317055966},{"lat":41.3103707027842,"lng":-8.74241244450931},{"lat":41.3104924652615,"lng":-8.74222107516658},{"lat":41.3106722541915,"lng":-8.74190389728552},{"lat":41.3107229205429,"lng":-8.74168773948569},{"lat":41.3108338423355,"lng":-8.74152039684512},{"lat":41.3109232158646,"lng":-8.74144288720935},{"lat":41.3111538470849,"lng":-8.74140300315025},{"lat":41.311551043131,"lng":-8.74124715598067},{"lat":41.3119052871211,"lng":-8.74114214407664},{"lat":41.3123937197801,"lng":-8.74110131089972},{"lat":41.3129526888864,"lng":-8.74112144848329},{"lat":41.3133812860842,"lng":-8.74112245990975},{"lat":41.3137352228799,"lng":-8.74111058980971},{"lat":41.3141566307123,"lng":-8.74107534382742},{"lat":41.3144990929276,"lng":-8.74101834675963},{"lat":41.3147283440527,"lng":-8.7409330594272},{"lat":41.3152099144332,"lng":-8.74079315623647},{"lat":41.3156716186618,"lng":-8.74064685950529},{"lat":41.3161334819391,"lng":-8.74052181797988},{"lat":41.3166008951701,"lng":-8.74045379354965},{"lat":41.3171185053263,"lng":-8.74047951529048},{"lat":41.3173625307939,"lng":-8.74051176633821},{"lat":41.3177338939198,"lng":-8.74057553619371},{"lat":41.3185083406325,"lng":-8.74040789797612},{"lat":41.3190877645707,"lng":-8.74032299355835},{"lat":41.3195283654892,"lng":-8.74023381885345},{"lat":41.3199693709794,"lng":-8.74010235967588},{"lat":41.3204607671151,"lng":-8.73997758125685},{"lat":41.3210583045203,"lng":-8.73988974239306},{"lat":41.3215564236782,"lng":-8.73982187426614},{"lat":41.3222205054567,"lng":-8.73976785636793},{"lat":41.3225048407666,"lng":-8.73963493180008},{"lat":41.3229482211835,"lng":-8.73947947711339},{"lat":41.3232811043756,"lng":-8.73942966401758},{"lat":41.3247403157687,"lng":-8.73946890829931},{"lat":41.3257128846482,"lng":-8.73956753937576},{"lat":41.3261994252599,"lng":-8.73956071729322},{"lat":41.3274185143674,"lng":-8.73935632408248},{"lat":41.3281484214922,"lng":-8.73935212357729},{"lat":41.3293635064177,"lng":-8.73948000788464},{"lat":41.3300890348894,"lng":-8.73962364897783},{"lat":41.3317942038281,"lng":-8.73977187649666},{"lat":41.3322799092001,"lng":-8.73987160672648},{"lat":41.332372162106,"lng":-8.739879274192},{"lat":41.3327662011582,"lng":-8.73991077802512},{"lat":41.3342456857565,"lng":-8.74028711854196},{"lat":41.3361849616637,"lng":-8.74118398437329},{"lat":41.336615540434,"lng":-8.74142322931458},{"lat":41.3372044945301,"lng":-8.74193515595715},{"lat":41.3382610507938,"lng":-8.74322599499615},{"lat":41.3385739607195,"lng":-8.74369749196061},{"lat":41.3389694935796,"lng":-8.744446462967},{"lat":41.339459374793,"lng":-8.74550170651534},{"lat":41.3397002168983,"lng":-8.74609703482618},{"lat":41.3397105060553,"lng":-8.74627801411636},{"lat":41.3396368491327,"lng":-8.74641280601647},{"lat":41.3388376399422,"lng":-8.74727062514867},{"lat":41.3387230250545,"lng":-8.7475041868967},{"lat":41.3387226823916,"lng":-8.74756821994399},{"lat":41.3387945016281,"lng":-8.74755802174483},{"lat":41.3393017050713,"lng":-8.74700390936251},{"lat":41.3397485535808,"lng":-8.74642042639738},{"lat":41.3399105221681,"lng":-8.74776437802329},{"lat":41.3415141264579,"lng":-8.74723134744931},{"lat":41.3421564843002,"lng":-8.74496738209927},{"lat":41.3450401474946,"lng":-8.74451651019889},{"lat":41.3473829627786,"lng":-8.74418001808647},{"lat":41.3505494042138,"lng":-8.7413419323322},{"lat":41.3515631139768,"lng":-8.73693019963699},{"lat":41.3511342237127,"lng":-8.73286354298739},{"lat":41.3507884004523,"lng":-8.73011208963671},{"lat":41.3514229904142,"lng":-8.72928145968479},{"lat":41.3534043822353,"lng":-8.7291800786115},{"lat":41.3550219108187,"lng":-8.72979235383448},{"lat":41.356287363422,"lng":-8.728847932154},{"lat":41.3578340323536,"lng":-8.72575498493128},{"lat":41.3600028791235,"lng":-8.7242211089019},{"lat":41.3611896927116,"lng":-8.71989412117795},{"lat":41.3554850484019,"lng":-8.7133864316704},{"lat":41.3500639118809,"lng":-8.71247577599762},{"lat":41.3472945479363,"lng":-8.71238013034321},{"lat":41.3450454414262,"lng":-8.71288651980519},{"lat":41.3432974266152,"lng":-8.71309686924639},{"lat":41.3414760202416,"lng":-8.71301891513109},{"lat":41.3405385697991,"lng":-8.71297879525305},{"lat":41.3376896560598,"lng":-8.71266999543925},{"lat":41.3352038359431,"lng":-8.71231163290414},{"lat":41.3323279769947,"lng":-8.71104302553868},{"lat":41.3294224991663,"lng":-8.70994452771581},{"lat":41.3277635106332,"lng":-8.70931734684161},{"lat":41.3245743788817,"lng":-8.71000232428722},{"lat":41.324444034155,"lng":-8.70868723654834},{"lat":41.3239158574319,"lng":-8.70811756817111},{"lat":41.3222782888797,"lng":-8.70765882945573},{"lat":41.3205631559183,"lng":-8.70670515834223},{"lat":41.3166439412249,"lng":-8.70385378356545},{"lat":41.312545494597,"lng":-8.700952449038},{"lat":41.3081902791125,"lng":-8.69775193333545},{"lat":41.3069393822434,"lng":-8.6972671802686},{"lat":41.3063945035385,"lng":-8.69734182196011},{"lat":41.3040750649156,"lng":-8.6977598100356},{"lat":41.3030865465659,"lng":-8.69889270613165},{"lat":41.3034161222834,"lng":-8.70023067350728},{"lat":41.3027309858801,"lng":-8.70203673490695},{"lat":41.302178342965,"lng":-8.70292445908969},{"lat":41.301273154141,"lng":-8.70408997270375},{"lat":41.3001692641531,"lng":-8.70556796391459},{"lat":41.2992707328297,"lng":-8.70988811262374},{"lat":41.3025547795812,"lng":-8.7128315720339},{"lat":41.3040697961526,"lng":-8.71474865219419},{"lat":41.3043949980675,"lng":-8.71522143703518},{"lat":41.3048705467836,"lng":-8.716824780081},{"lat":41.3051428994178,"lng":-8.71853348002236},{"lat":41.3055001926063,"lng":-8.72077534788809},{"lat":41.3058864778977,"lng":-8.72319946585067},{"lat":41.3061756206944,"lng":-8.72501419888233},{"lat":41.3062171386977,"lng":-8.725894636952},{"lat":41.306278785919,"lng":-8.72721356490086},{"lat":41.3061376006448,"lng":-8.73142983011214},{"lat":41.3061084227159,"lng":-8.73330185176799},{"lat":41.3058257165635,"lng":-8.73398697090801},{"lat":41.3055647726318,"lng":-8.73433084857173},{"lat":41.3056699780784,"lng":-8.73819035334109},{"lat":41.3055859907209,"lng":-8.73908393919838},{"lat":41.3053902717472,"lng":-8.73956419819331}]},{"zone_id":"vcd7","points":[{"lat":41.3425739241193,"lng":-8.66971973881738},{"lat":41.340977060347,"lng":-8.67007087849195},{"lat":41.3390050670303,"lng":-8.66987571269762},{"lat":41.3326549796443,"lng":-8.66274420553036},{"lat":41.3288479695557,"lng":-8.66341597541044},{"lat":41.3231941101497,"lng":-8.6665870175356},{"lat":41.3197408915732,"lng":-8.66651868648839},{"lat":41.318830610479,"lng":-8.66647745060084},{"lat":41.3143227473202,"lng":-8.66752368794406},{"lat":41.3119981256059,"lng":-8.66768175909055},{"lat":41.3090135401923,"lng":-8.67052317966686},{"lat":41.3028042547918,"lng":-8.67450831703959},{"lat":41.3008234160226,"lng":-8.67356664464292},{"lat":41.3011221590224,"lng":-8.67411076702621},{"lat":41.3006706232219,"lng":-8.67460244072456},{"lat":41.3002582177665,"lng":-8.67562553061599},{"lat":41.2996307292126,"lng":-8.67699615622634},{"lat":41.2994088897118,"lng":-8.67796887054463},{"lat":41.2989403630756,"lng":-8.67862815318874},{"lat":41.2988120624751,"lng":-8.67992530622421},{"lat":41.2992891645087,"lng":-8.68095013549028},{"lat":41.2998730515981,"lng":-8.6825881591857},{"lat":41.2999920131044,"lng":-8.68359694227361},{"lat":41.2996833595733,"lng":-8.68464371117124},{"lat":41.2993336934435,"lng":-8.68539088839966},{"lat":41.2989554696767,"lng":-8.6856949415597},{"lat":41.2988336345977,"lng":-8.68621998622059},{"lat":41.2988823153388,"lng":-8.68697631941396},{"lat":41.2993397568517,"lng":-8.68786410172418},{"lat":41.3001648018387,"lng":-8.68912963512745},{"lat":41.3010394701163,"lng":-8.69037119033135},{"lat":41.301590802911,"lng":-8.69196428917485},{"lat":41.302478382545,"lng":-8.69349407414621},{"lat":41.3036787517775,"lng":-8.69452193295891},{"lat":41.3063684441344,"lng":-8.69644656712644},{"lat":41.3063945035385,"lng":-8.69734182196011},{"lat":41.3069393822434,"lng":-8.6972671802686},{"lat":41.3081902791125,"lng":-8.69775193333545},{"lat":41.312545494597,"lng":-8.700952449038},{"lat":41.3166439412249,"lng":-8.70385378356545},{"lat":41.3205631559183,"lng":-8.70670515834223},{"lat":41.3222782888797,"lng":-8.70765882945573},{"lat":41.3239158574319,"lng":-8.70811756817111},{"lat":41.324444034155,"lng":-8.70868723654834},{"lat":41.3245743788817,"lng":-8.71000232428722},{"lat":41.3277635106332,"lng":-8.70931734684161},{"lat":41.3294224991663,"lng":-8.70994452771581},{"lat":41.3323279769947,"lng":-8.71104302553868},{"lat":41.3352038359431,"lng":-8.71231163290414},{"lat":41.3376896560598,"lng":-8.71266999543925},{"lat":41.3405385697991,"lng":-8.71297879525305},{"lat":41.3414760202416,"lng":-8.71301891513109},{"lat":41.341479530465,"lng":-8.71206895584246},{"lat":41.3418700250422,"lng":-8.70836392136954},{"lat":41.3417210238564,"lng":-8.70404739046187},{"lat":41.3416837735067,"lng":-8.70087201140336},{"lat":41.3417582741848,"lng":-8.69655548049569},{"lat":41.3413485194004,"lng":-8.69333048613939},{"lat":41.3404917510655,"lng":-8.69089933654772},{"lat":41.3395604683521,"lng":-8.68931164701846},{"lat":41.3390016923319,"lng":-8.6877239574892},{"lat":41.3389644404267,"lng":-8.68529280789753},{"lat":41.3391506997393,"lng":-8.68221665943459},{"lat":41.340044737025,"lng":-8.67839628150482},{"lat":41.3411250157041,"lng":-8.67467513417063},{"lat":41.3425739241193,"lng":-8.66971973881738}]},{"zone_id":"pvz4","points":[{"lat":41.4416856061153,"lng":-8.78019053507796},{"lat":41.442408389142,"lng":-8.78001931489081},{"lat":41.4430058837423,"lng":-8.77993899334157},{"lat":41.4433995245171,"lng":-8.77979585965113},{"lat":41.4440291710678,"lng":-8.7796970636105},{"lat":41.4442644016014,"lng":-8.77960798749416},{"lat":41.4448971297899,"lng":-8.77952155394476},{"lat":41.4456809677312,"lng":-8.77941447796811},{"lat":41.4463062256353,"lng":-8.77939174282645},{"lat":41.4470146677155,"lng":-8.7791953611297},{"lat":41.4481373547021,"lng":-8.77877776627607},{"lat":41.4486881941933,"lng":-8.77869876067447},{"lat":41.4494226126846,"lng":-8.77868548198077},{"lat":41.4498423953886,"lng":-8.77858482230994},{"lat":41.4506791989832,"lng":-8.77848927104132},{"lat":41.4525673604402,"lng":-8.77858703061526},{"lat":41.4532755756258,"lng":-8.77852621505554},{"lat":41.4543244004909,"lng":-8.77864136633843},{"lat":41.454560491551,"lng":-8.77862336370413},{"lat":41.4549803562988,"lng":-8.77847494733193},{"lat":41.4552927143674,"lng":-8.77845615122996},{"lat":41.4558431456015,"lng":-8.77859387465189},{"lat":41.4565513937304,"lng":-8.77868709276092},{"lat":41.4582831544745,"lng":-8.77808171372216},{"lat":41.4588341327745,"lng":-8.77796211653941},{"lat":41.4597787581537,"lng":-8.77758621877332},{"lat":41.4604871451757,"lng":-8.77739803688413},{"lat":41.461431751093,"lng":-8.7772494252623},{"lat":41.462140043806,"lng":-8.77707763607711},{"lat":41.4624549055084,"lng":-8.77696501720384},{"lat":41.4631370640908,"lng":-8.77689506320821},{"lat":41.4639768355643,"lng":-8.77665454709112},{"lat":41.4641867274155,"lng":-8.77655607878052},{"lat":41.4647685290758,"lng":-8.77646321688516},{"lat":41.4651176521222,"lng":-8.7765183907168},{"lat":41.4655854043587,"lng":-8.77648436686861},{"lat":41.4665160889811,"lng":-8.77624821021832},{"lat":41.4674716879506,"lng":-8.77617819966205},{"lat":41.4681493387863,"lng":-8.77604402552902},{"lat":41.4702536110378,"lng":-8.77600825795386},{"lat":41.4704892400296,"lng":-8.77608852481085},{"lat":41.4709586412474,"lng":-8.7761622510639},{"lat":41.4712205340501,"lng":-8.77626300671286},{"lat":41.4716372140195,"lng":-8.77631227258197},{"lat":41.4719514200731,"lng":-8.77642923556661},{"lat":41.4714968118953,"lng":-8.77561260375269},{"lat":41.4705262956989,"lng":-8.77394047043597},{"lat":41.47015334442,"lng":-8.77275071116159},{"lat":41.4687469127,"lng":-8.76879325556902},{"lat":41.4681431394644,"lng":-8.7649625247195},{"lat":41.4680141607945,"lng":-8.76407162205016},{"lat":41.467904258202,"lng":-8.75968255152316},{"lat":41.4675660630648,"lng":-8.75917461509355},{"lat":41.4669360421567,"lng":-8.75819020207724},{"lat":41.4661400855198,"lng":-8.75258817793821},{"lat":41.4661375259367,"lng":-8.75217805549214},{"lat":41.4661992074971,"lng":-8.75139328120376},{"lat":41.4662309575925,"lng":-8.75081495198978},{"lat":41.4663615660065,"lng":-8.75018560605349},{"lat":41.4665851304683,"lng":-8.74958275704073},{"lat":41.4669250475081,"lng":-8.74905893481978},{"lat":41.4674337532668,"lng":-8.74818753893624},{"lat":41.4676269015439,"lng":-8.74784785898952},{"lat":41.4683099872818,"lng":-8.74676094112464},{"lat":41.4684924439432,"lng":-8.74643288172558},{"lat":41.4685869950143,"lng":-8.74626247531276},{"lat":41.4686311824873,"lng":-8.74579436299416},{"lat":41.4678580382705,"lng":-8.74589443866158},{"lat":41.4677574617196,"lng":-8.74586200300442},{"lat":41.4676643881656,"lng":-8.74584053369488},{"lat":41.4675611994652,"lng":-8.74582519269775},{"lat":41.4674794476723,"lng":-8.74575750895645},{"lat":41.4673675826069,"lng":-8.74562944401166},{"lat":41.4672155705807,"lng":-8.74538273700501},{"lat":41.4671495800596,"lng":-8.74524613526527},{"lat":41.467109124661,"lng":-8.74513382737168},{"lat":41.4670838707074,"lng":-8.74500706219008},{"lat":41.4669947884494,"lng":-8.74467608356195},{"lat":41.4669369556219,"lng":-8.7444957478873},{"lat":41.466847445427,"lng":-8.7442957133893},{"lat":41.4667831589731,"lng":-8.74411028790944},{"lat":41.4667301140445,"lng":-8.74399427804787},{"lat":41.4652680303041,"lng":-8.74216107112928},{"lat":41.4636600063523,"lng":-8.74020138340273},{"lat":41.4623681289735,"lng":-8.73986539858913},{"lat":41.4618029834003,"lng":-8.73989649734876},{"lat":41.4611652102714,"lng":-8.73983571439297},{"lat":41.4605567928936,"lng":-8.73948519845981},{"lat":41.4597399240896,"lng":-8.73811477384364},{"lat":41.4585203449512,"lng":-8.73606812052785},{"lat":41.4556048207402,"lng":-8.73394635038225},{"lat":41.4555969107358,"lng":-8.73378913662621},{"lat":41.455590437527,"lng":-8.73346114911425},{"lat":41.4555529859407,"lng":-8.73315070419502},{"lat":41.4555191356048,"lng":-8.73299504928613},{"lat":41.4554882059503,"lng":-8.73278101003968},{"lat":41.4554369713516,"lng":-8.73247140375525},{"lat":41.4554114812016,"lng":-8.73233508462595},{"lat":41.4553731255867,"lng":-8.73202475496158},{"lat":41.4553282683273,"lng":-8.73178761543557},{"lat":41.4553030081622,"lng":-8.73165919918247},{"lat":41.4552689913592,"lng":-8.73150019299361},{"lat":41.4551581272209,"lng":-8.73128146534953},{"lat":41.4550935838944,"lng":-8.73114120525014},{"lat":41.4550749178196,"lng":-8.7309907137948},{"lat":41.4550831751131,"lng":-8.73082371764447},{"lat":41.455122490706,"lng":-8.73067376086717},{"lat":41.4551539813059,"lng":-8.73055700268032},{"lat":41.4551628209522,"lng":-8.73043344803523},{"lat":41.4550735785752,"lng":-8.73038379882311},{"lat":41.4549116575795,"lng":-8.73030403646417},{"lat":41.4546898029086,"lng":-8.73016639607408},{"lat":41.4544958177202,"lng":-8.72998485328902},{"lat":41.4542899637063,"lng":-8.72985166885315},{"lat":41.4541192465931,"lng":-8.72968004025418},{"lat":41.45401732911,"lng":-8.72957569884198},{"lat":41.4539318784798,"lng":-8.72950717573282},{"lat":41.4538139366909,"lng":-8.72947126248116},{"lat":41.4537208415662,"lng":-8.7295055897252},{"lat":41.4536059906121,"lng":-8.72951590968177},{"lat":41.4535815535772,"lng":-8.72934944589258},{"lat":41.4467969146246,"lng":-8.73098746350942},{"lat":41.4388129330334,"lng":-8.7347582261414},{"lat":41.4352917346292,"lng":-8.73700745297451},{"lat":41.4332086822672,"lng":-8.73780129773914},{"lat":41.431667398168,"lng":-8.73865813865875},{"lat":41.43227394401,"lng":-8.74186076315726},{"lat":41.4312752633227,"lng":-8.74540352824349},{"lat":41.4362389927352,"lng":-8.74995006666577},{"lat":41.4365696776986,"lng":-8.75468976779504},{"lat":41.4391067635655,"lng":-8.76312178656172},{"lat":41.4407788283502,"lng":-8.76680908702707},{"lat":41.4389005251841,"lng":-8.76821176697472},{"lat":41.4383496732436,"lng":-8.77116680350263},{"lat":41.4385132786586,"lng":-8.77377359701192},{"lat":41.4418939027053,"lng":-8.77451751200056},{"lat":41.4416856061153,"lng":-8.78019053507796}]},{"zone_id":"pvz3","points":[{"lat":41.41617324572,"lng":-8.78904411580385},{"lat":41.416314326925,"lng":-8.78900628789553},{"lat":41.4166840524834,"lng":-8.78890726938987},{"lat":41.416987473366,"lng":-8.78864025359437},{"lat":41.4174749682666,"lng":-8.78809926210592},{"lat":41.4176300923838,"lng":-8.78783313032076},{"lat":41.4182169442905,"lng":-8.78717303984253},{"lat":41.4184324173104,"lng":-8.78698095409591},{"lat":41.4190915745952,"lng":-8.78669788148882},{"lat":41.4203056151988,"lng":-8.78605482827863},{"lat":41.4209559401053,"lng":-8.78609019112889},{"lat":41.4218604563269,"lng":-8.7857581281541},{"lat":41.4223116878667,"lng":-8.78566983619002},{"lat":41.4229899559722,"lng":-8.78543010687508},{"lat":41.4238915115387,"lng":-8.78536534871713},{"lat":41.4243427476203,"lng":-8.78527633495062},{"lat":41.4254145976988,"lng":-8.78476145506052},{"lat":41.4257468453519,"lng":-8.78460209437185},{"lat":41.4266507685446,"lng":-8.78432572649707},{"lat":41.4276256350867,"lng":-8.78444759055192},{"lat":41.4286739749569,"lng":-8.78467978111168},{"lat":41.4295752032367,"lng":-8.78467242970269},{"lat":41.4300251731175,"lng":-8.78471127645248},{"lat":41.4304774380236,"lng":-8.7845515526753},{"lat":41.4309283402675,"lng":-8.78452114413233},{"lat":41.4313794617711,"lng":-8.78443593924512},{"lat":41.4316282372213,"lng":-8.78443484394917},{"lat":41.4320367431677,"lng":-8.78454589745042},{"lat":41.4323622797232,"lng":-8.78453599998072},{"lat":41.4329380276441,"lng":-8.78467098948287},{"lat":41.4333066929432,"lng":-8.78451939114494},{"lat":41.4346510991577,"lng":-8.78379551427251},{"lat":41.4351045970063,"lng":-8.78351315113756},{"lat":41.4357832446719,"lng":-8.78322131021379},{"lat":41.4368816205986,"lng":-8.78261636132575},{"lat":41.437359886703,"lng":-8.78247553253068},{"lat":41.4380699902653,"lng":-8.78219308625829},{"lat":41.4400855179277,"lng":-8.78111923852047},{"lat":41.4410828840382,"lng":-8.78073277752201},{"lat":41.4412929232873,"lng":-8.78060887534533},{"lat":41.4415354208088,"lng":-8.78041242477376},{"lat":41.4416856061153,"lng":-8.78019053507796},{"lat":41.4418939027053,"lng":-8.77451751200056},{"lat":41.4385132786586,"lng":-8.77377359701192},{"lat":41.4383496732436,"lng":-8.77116680350263},{"lat":41.4389005251841,"lng":-8.76821176697472},{"lat":41.4407788283502,"lng":-8.76680908702707},{"lat":41.4391067635655,"lng":-8.76312178656172},{"lat":41.4365696776986,"lng":-8.75468976779504},{"lat":41.4362389927352,"lng":-8.74995006666577},{"lat":41.4312752633227,"lng":-8.74540352824349},{"lat":41.43227394401,"lng":-8.74186076315726},{"lat":41.431667398168,"lng":-8.73865813865875},{"lat":41.430865150228,"lng":-8.73876905673636},{"lat":41.4296245061748,"lng":-8.73557363069337},{"lat":41.4290196291875,"lng":-8.73401582594853},{"lat":41.4218816524861,"lng":-8.71915133919041},{"lat":41.4191765703282,"lng":-8.70992036194058},{"lat":41.4190957077856,"lng":-8.70963495142478},{"lat":41.4185812221837,"lng":-8.7102409520862},{"lat":41.41617594985,"lng":-8.71106342731},{"lat":41.4151198912153,"lng":-8.7127040190178},{"lat":41.4147408223956,"lng":-8.71329353042077},{"lat":41.4131131685788,"lng":-8.71582213205772},{"lat":41.4102200259199,"lng":-8.71225798292633},{"lat":41.4089000955695,"lng":-8.71150376313699},{"lat":41.4077670185899,"lng":-8.71099269884406},{"lat":41.4063573322698,"lng":-8.71748363893222},{"lat":41.4066291404945,"lng":-8.72263500270647},{"lat":41.4061647913621,"lng":-8.72538136838132},{"lat":41.4051564491461,"lng":-8.72884024357876},{"lat":41.4037991033684,"lng":-8.73014324363221},{"lat":41.4087432573521,"lng":-8.74002416164786},{"lat":41.4121814552083,"lng":-8.74552927447771},{"lat":41.4146882880672,"lng":-8.74818424356248},{"lat":41.4175609850966,"lng":-8.74976639650062},{"lat":41.4207095580283,"lng":-8.75027476357195},{"lat":41.4208618128425,"lng":-8.75541975487498},{"lat":41.4187649857759,"lng":-8.76018419968062},{"lat":41.4172455456015,"lng":-8.76651107873037},{"lat":41.4162610554094,"lng":-8.77210366638946},{"lat":41.4153445980991,"lng":-8.78030657102441},{"lat":41.4159287027964,"lng":-8.78455869090254},{"lat":41.41617324572,"lng":-8.78904411580385}]},{"zone_id":"pvz2","points":[{"lat":41.4077670185899,"lng":-8.71099269884406},{"lat":41.4067456553324,"lng":-8.71112940118999},{"lat":41.4037909660102,"lng":-8.71009213693133},{"lat":41.4036914353311,"lng":-8.71001064924883},{"lat":41.4034628802076,"lng":-8.71003648690511},{"lat":41.4033091697359,"lng":-8.71005760140037},{"lat":41.4030429082723,"lng":-8.71013273382474},{"lat":41.4029200847336,"lng":-8.71015280576297},{"lat":41.4027721050586,"lng":-8.71019765343615},{"lat":41.4025301377302,"lng":-8.71025828579095},{"lat":41.4022632495364,"lng":-8.71035098528535},{"lat":41.4021586717512,"lng":-8.71043041641507},{"lat":41.4019820263208,"lng":-8.71064386213363},{"lat":41.4018176815607,"lng":-8.71087338584299},{"lat":41.4017497944799,"lng":-8.71096993664287},{"lat":41.4016627352399,"lng":-8.71109328451354},{"lat":41.4014893299635,"lng":-8.71128845917102},{"lat":41.4012916019531,"lng":-8.71143187173616},{"lat":41.4010759276589,"lng":-8.71152717134868},{"lat":41.4007696894507,"lng":-8.71160193838707},{"lat":41.4006325988124,"lng":-8.71166625030155},{"lat":41.4004162129646,"lng":-8.71183209179212},{"lat":41.4002112242692,"lng":-8.71198643987383},{"lat":41.3999557730809,"lng":-8.71214930528145},{"lat":41.3998319774189,"lng":-8.71225510715643},{"lat":41.3998054230182,"lng":-8.71237851583744},{"lat":41.3986314153435,"lng":-8.71287721505642},{"lat":41.3982473594273,"lng":-8.71113777382662},{"lat":41.3981215148413,"lng":-8.71109635743208},{"lat":41.3978579081888,"lng":-8.71087722078913},{"lat":41.3976797744795,"lng":-8.71065167754244},{"lat":41.3966646525811,"lng":-8.70979930015408},{"lat":41.3946815624872,"lng":-8.71254952249762},{"lat":41.3937888090261,"lng":-8.71440396208113},{"lat":41.393994151002,"lng":-8.71462413423148},{"lat":41.3941224476896,"lng":-8.71474951037982},{"lat":41.3942667577545,"lng":-8.71491353402916},{"lat":41.3937923251298,"lng":-8.7168661472917},{"lat":41.3939785496972,"lng":-8.71714450403666},{"lat":41.3941438731591,"lng":-8.7173894345491},{"lat":41.3942292040676,"lng":-8.71755317492045},{"lat":41.394253693604,"lng":-8.71776574618078},{"lat":41.3942831436391,"lng":-8.71806445791698},{"lat":41.3943463123041,"lng":-8.7183009355224},{"lat":41.3944690422711,"lng":-8.71856366061094},{"lat":41.394596334228,"lng":-8.71883251981039},{"lat":41.3947395492742,"lng":-8.71915773136485},{"lat":41.3948463189541,"lng":-8.71935430772572},{"lat":41.3949088481348,"lng":-8.71950373679624},{"lat":41.3950469131015,"lng":-8.71977891666694},{"lat":41.3951690774378,"lng":-8.72004630680053},{"lat":41.395292492261,"lng":-8.72033283724744},{"lat":41.3951910619932,"lng":-8.72053495260946},{"lat":41.3950348207872,"lng":-8.72073334643199},{"lat":41.3948210199215,"lng":-8.72088374470249},{"lat":41.394729616223,"lng":-8.7209919683111},{"lat":41.3945039218114,"lng":-8.72098621966262},{"lat":41.3934080984688,"lng":-8.72141895373806},{"lat":41.3893409893032,"lng":-8.72344827732661},{"lat":41.3882623779305,"lng":-8.72560961080653},{"lat":41.3875707992968,"lng":-8.72686615688738},{"lat":41.3820889687419,"lng":-8.73151274838522},{"lat":41.3834857445398,"lng":-8.73332108070582},{"lat":41.3860799076198,"lng":-8.73715298279372},{"lat":41.3897480689829,"lng":-8.74112028762795},{"lat":41.3945021717392,"lng":-8.74563652218009},{"lat":41.3977007969936,"lng":-8.74871974231019},{"lat":41.3980741426484,"lng":-8.74999434893626},{"lat":41.3979751778048,"lng":-8.75099189649802},{"lat":41.3982948196329,"lng":-8.75122429815666},{"lat":41.3975295121639,"lng":-8.75323267119252},{"lat":41.3966505219968,"lng":-8.75485920563657},{"lat":41.3942177171109,"lng":-8.75764879534031},{"lat":41.3917019578554,"lng":-8.75994138987314},{"lat":41.3907058818502,"lng":-8.76100219783632},{"lat":41.3900248535031,"lng":-8.76222653998304},{"lat":41.3889829570042,"lng":-8.76464567540669},{"lat":41.3887660765109,"lng":-8.76538328744079},{"lat":41.388771958321,"lng":-8.76566391264009},{"lat":41.3886843846684,"lng":-8.76609723499869},{"lat":41.3878596202154,"lng":-8.76839543836315},{"lat":41.3885145156499,"lng":-8.76990755589558},{"lat":41.3898262653628,"lng":-8.77088159647023},{"lat":41.3914978455229,"lng":-8.77134901787018},{"lat":41.3935477516148,"lng":-8.77196026833932},{"lat":41.3937081769869,"lng":-8.77272315381631},{"lat":41.393254179489,"lng":-8.77697264800248},{"lat":41.3940507079045,"lng":-8.77766936843741},{"lat":41.3941386064081,"lng":-8.77785784766005},{"lat":41.3941665008249,"lng":-8.77810120782914},{"lat":41.3940506201492,"lng":-8.77838929096663},{"lat":41.3940021026488,"lng":-8.7787067448175},{"lat":41.3940516542106,"lng":-8.77886171878796},{"lat":41.3941916931918,"lng":-8.7788726763994},{"lat":41.3943536538238,"lng":-8.7787477781708},{"lat":41.3944026557409,"lng":-8.77843965516643},{"lat":41.3946375978115,"lng":-8.77833844205632},{"lat":41.3949206972596,"lng":-8.77851116180415},{"lat":41.3951068223909,"lng":-8.77873134192551},{"lat":41.3952547362631,"lng":-8.77899048322734},{"lat":41.3952931138789,"lng":-8.77913112300841},{"lat":41.3951331829223,"lng":-8.77940742069934},{"lat":41.3951544028958,"lng":-8.77982720549138},{"lat":41.3952230274422,"lng":-8.77998464284116},{"lat":41.3951554949209,"lng":-8.78012865342731},{"lat":41.3951493578363,"lng":-8.78032301240506},{"lat":41.3951909420754,"lng":-8.7806758036232},{"lat":41.3955113537834,"lng":-8.78060282420471},{"lat":41.3956515974656,"lng":-8.78081622061163},{"lat":41.3957781346912,"lng":-8.78079392802724},{"lat":41.3958442447899,"lng":-8.78064714886803},{"lat":41.3959629058605,"lng":-8.78055291863758},{"lat":41.3966473588305,"lng":-8.78038000645237},{"lat":41.3976892901897,"lng":-8.77980386311557},{"lat":41.3979177250982,"lng":-8.77970652794807},{"lat":41.398298165251,"lng":-8.77965374899817},{"lat":41.3987544165821,"lng":-8.77937692309489},{"lat":41.3991803043651,"lng":-8.77940710182514},{"lat":41.3995477373381,"lng":-8.77950545696758},{"lat":41.4005013603302,"lng":-8.77892468731324},{"lat":41.4007481570417,"lng":-8.7788759601042},{"lat":41.4009962891535,"lng":-8.7788785456946},{"lat":41.4014510010408,"lng":-8.77897131336807},{"lat":41.4019039963147,"lng":-8.77922418453789},{"lat":41.4028355295858,"lng":-8.77943243369257},{"lat":41.403517627194,"lng":-8.77975671350566},{"lat":41.4038022668478,"lng":-8.77997516018718},{"lat":41.4039381785562,"lng":-8.78007923458763},{"lat":41.4047946293491,"lng":-8.78058859224637},{"lat":41.4052715431023,"lng":-8.78068745917412},{"lat":41.4057381724328,"lng":-8.78086933813926},{"lat":41.4059607202034,"lng":-8.78100119117327},{"lat":41.4060559423818,"lng":-8.7811356057497},{"lat":41.4060797526199,"lng":-8.78154292999083},{"lat":41.4061419479606,"lng":-8.78168992837318},{"lat":41.4063566297119,"lng":-8.7818914282508},{"lat":41.4069170575582,"lng":-8.78225137329556},{"lat":41.406989512581,"lng":-8.78236714634221},{"lat":41.4070078462258,"lng":-8.78261237233238},{"lat":41.4070745672959,"lng":-8.78275427875862},{"lat":41.4072764891369,"lng":-8.78282398788258},{"lat":41.4075267106969,"lng":-8.78301389330555},{"lat":41.4075545933558,"lng":-8.78319331861091},{"lat":41.4075216462127,"lng":-8.78344664761982},{"lat":41.4076444326913,"lng":-8.78353027557634},{"lat":41.4078764048839,"lng":-8.78311114958323},{"lat":41.4086033295464,"lng":-8.783086720983},{"lat":41.4093744484152,"lng":-8.78377924530377},{"lat":41.410021860792,"lng":-8.78448657080456},{"lat":41.4101472145681,"lng":-8.7847523753815},{"lat":41.4103015139596,"lng":-8.78497900949793},{"lat":41.4107217545153,"lng":-8.78530554325679},{"lat":41.4108952594391,"lng":-8.78552639279123},{"lat":41.4110049007115,"lng":-8.78578379197891},{"lat":41.4111221064013,"lng":-8.78589919063395},{"lat":41.4114561085143,"lng":-8.78609593713204},{"lat":41.4116256523398,"lng":-8.78641123640126},{"lat":41.4118272865555,"lng":-8.78660964335953},{"lat":41.4117768541167,"lng":-8.78743307328063},{"lat":41.4118179918527,"lng":-8.78781345726279},{"lat":41.4119472526592,"lng":-8.787976941735},{"lat":41.4121827465485,"lng":-8.78807989822172},{"lat":41.4124338078381,"lng":-8.78807345817737},{"lat":41.412581100507,"lng":-8.78799636607235},{"lat":41.4129226262738,"lng":-8.78759996904038},{"lat":41.4131024522427,"lng":-8.78759651983528},{"lat":41.4132549203787,"lng":-8.78770043738305},{"lat":41.4133204707429,"lng":-8.78804723178749},{"lat":41.4134116591333,"lng":-8.78814538607383},{"lat":41.4139616765864,"lng":-8.78821228442165},{"lat":41.4142616241792,"lng":-8.78815957309568},{"lat":41.4148164933955,"lng":-8.78813370517723},{"lat":41.4150163685118,"lng":-8.78826179282986},{"lat":41.4152713226233,"lng":-8.78850393562721},{"lat":41.4153955484715,"lng":-8.78877609500105},{"lat":41.4159175732331,"lng":-8.78906268409404},{"lat":41.41617324572,"lng":-8.78904411580385},{"lat":41.4159287027964,"lng":-8.78455869090254},{"lat":41.4153445980991,"lng":-8.78030657102441},{"lat":41.4162610554094,"lng":-8.77210366638946},{"lat":41.4172455456015,"lng":-8.76651107873037},{"lat":41.4187649857759,"lng":-8.76018419968062},{"lat":41.4208618128425,"lng":-8.75541975487498},{"lat":41.4207095580283,"lng":-8.75027476357195},{"lat":41.4175609850966,"lng":-8.74976639650062},{"lat":41.4146882880672,"lng":-8.74818424356248},{"lat":41.4121814552083,"lng":-8.74552927447771},{"lat":41.4087432573521,"lng":-8.74002416164786},{"lat":41.4037991033684,"lng":-8.73014324363221},{"lat":41.4051564491461,"lng":-8.72884024357876},{"lat":41.4061647913621,"lng":-8.72538136838132},{"lat":41.4066291404945,"lng":-8.72263500270647},{"lat":41.4063573322698,"lng":-8.71748363893222},{"lat":41.4077670185899,"lng":-8.71099269884406}]},{"zone_id":"vcd6","points":[{"lat":41.3697757323153,"lng":-8.69567303828525},{"lat":41.3705703955351,"lng":-8.69235311591936},{"lat":41.3666082560361,"lng":-8.68718045368996},{"lat":41.3637364585741,"lng":-8.68081514624987},{"lat":41.3495619580174,"lng":-8.6685085978663},{"lat":41.3466081835486,"lng":-8.66901062529472},{"lat":41.343926255584,"lng":-8.66942236342979},{"lat":41.3425739241193,"lng":-8.66971973881738},{"lat":41.3411250157041,"lng":-8.67467513417063},{"lat":41.340044737025,"lng":-8.67839628150482},{"lat":41.3391506997393,"lng":-8.68221665943459},{"lat":41.3389644404267,"lng":-8.68529280789753},{"lat":41.3390016923319,"lng":-8.6877239574892},{"lat":41.3395604683521,"lng":-8.68931164701846},{"lat":41.3404917510655,"lng":-8.69089933654772},{"lat":41.3413485194004,"lng":-8.69333048613939},{"lat":41.3417582741848,"lng":-8.69655548049569},{"lat":41.3416837735067,"lng":-8.70087201140336},{"lat":41.3417210238564,"lng":-8.70404739046187},{"lat":41.3418700250422,"lng":-8.70836392136954},{"lat":41.341479530465,"lng":-8.71206895584246},{"lat":41.3414760202416,"lng":-8.71301891513109},{"lat":41.3432974266152,"lng":-8.71309686924639},{"lat":41.3450454414262,"lng":-8.71288651980519},{"lat":41.3472945479363,"lng":-8.71238013034321},{"lat":41.3500639118809,"lng":-8.71247577599762},{"lat":41.3554850484019,"lng":-8.7133864316704},{"lat":41.3611896927116,"lng":-8.71989412117795},{"lat":41.3629081282364,"lng":-8.71530699742866},{"lat":41.363367996537,"lng":-8.71339892000288},{"lat":41.365265364986,"lng":-8.71210117503849},{"lat":41.3673439918013,"lng":-8.71056592648024},{"lat":41.3688593632432,"lng":-8.70917520003271},{"lat":41.3704249625138,"lng":-8.70664895731251},{"lat":41.3697757323153,"lng":-8.69567303828525}]},{"zone_id":"pvz6","points":[{"lat":41.4322450560538,"lng":-8.65462035811193},{"lat":41.4317498652118,"lng":-8.65431955768759},{"lat":41.4314299435336,"lng":-8.65412556755473},{"lat":41.4311758290495,"lng":-8.65419112325687},{"lat":41.4303780761464,"lng":-8.6537985368254},{"lat":41.4300039076933,"lng":-8.65365328837092},{"lat":41.4296651768338,"lng":-8.65347734127308},{"lat":41.4292423684215,"lng":-8.653257294894},{"lat":41.4282588613679,"lng":-8.65272614321797},{"lat":41.4274441874126,"lng":-8.65221143088725},{"lat":41.4273980722421,"lng":-8.65189679312033},{"lat":41.427111830646,"lng":-8.65174078111428},{"lat":41.4270147771668,"lng":-8.65165973388399},{"lat":41.4267222184402,"lng":-8.65146647207824},{"lat":41.4264548878044,"lng":-8.65134829626189},{"lat":41.4262274443035,"lng":-8.65125053511554},{"lat":41.4259926651156,"lng":-8.65114159800749},{"lat":41.4257669543366,"lng":-8.65103871161036},{"lat":41.4255028255094,"lng":-8.65095023139606},{"lat":41.4252781027383,"lng":-8.65086804873969},{"lat":41.4251438966346,"lng":-8.65081841495185},{"lat":41.4250137855627,"lng":-8.65078017132734},{"lat":41.4247735167028,"lng":-8.65071114742722},{"lat":41.424458220148,"lng":-8.6507073168618},{"lat":41.4242879874862,"lng":-8.65076074658096},{"lat":41.4240414031096,"lng":-8.65085364058679},{"lat":41.4237632263449,"lng":-8.65096459033881},{"lat":41.4236155754127,"lng":-8.65106221664019},{"lat":41.4213161214709,"lng":-8.65061754986255},{"lat":41.4196860770837,"lng":-8.65013710033011},{"lat":41.4196148592846,"lng":-8.64993736866101},{"lat":41.4193998758588,"lng":-8.64933172076711},{"lat":41.4185477321176,"lng":-8.64693326489983},{"lat":41.4183580366973,"lng":-8.64606551844355},{"lat":41.4181386264552,"lng":-8.64461275254011},{"lat":41.4178682841135,"lng":-8.64282456881822},{"lat":41.4176406384719,"lng":-8.64029488915767},{"lat":41.4182432381078,"lng":-8.63858947896195},{"lat":41.4181923152548,"lng":-8.63819280688505},{"lat":41.4180843875987,"lng":-8.63734773557822},{"lat":41.4186511234536,"lng":-8.63469311322668},{"lat":41.418405334481,"lng":-8.63448415953551},{"lat":41.4182045136672,"lng":-8.63426503298259},{"lat":41.4179721440512,"lng":-8.63411671523053},{"lat":41.4178215697742,"lng":-8.6340000010783},{"lat":41.4175641189253,"lng":-8.63378163922722},{"lat":41.4173281408474,"lng":-8.63349047498408},{"lat":41.4172403157786,"lng":-8.6333750906125},{"lat":41.4170701410214,"lng":-8.633106928346},{"lat":41.4168946154115,"lng":-8.63297185410272},{"lat":41.4162040600938,"lng":-8.6315578908344},{"lat":41.4144904064897,"lng":-8.62972372283801},{"lat":41.4140466289762,"lng":-8.62934500280116},{"lat":41.4143406370037,"lng":-8.62444841759585},{"lat":41.4148475856507,"lng":-8.62280011163342},{"lat":41.4146061447361,"lng":-8.62281072917939},{"lat":41.4112291503758,"lng":-8.6228370490371},{"lat":41.4098976421669,"lng":-8.62254726606658},{"lat":41.4097483560964,"lng":-8.62237930547962},{"lat":41.409774513242,"lng":-8.62226288975963},{"lat":41.4097914765892,"lng":-8.62210658670434},{"lat":41.4097956826656,"lng":-8.62190138770718},{"lat":41.4097951499338,"lng":-8.62177413212278},{"lat":41.4098210471926,"lng":-8.62163332058244},{"lat":41.4098506455773,"lng":-8.62142807240434},{"lat":41.4098900818853,"lng":-8.62121632208106},{"lat":41.4099567414348,"lng":-8.62108287391831},{"lat":41.4100630864734,"lng":-8.6208906387223},{"lat":41.4100961530918,"lng":-8.62073672743308},{"lat":41.410067210313,"lng":-8.62053390602903},{"lat":41.4100360472433,"lng":-8.62030440099158},{"lat":41.4100744833634,"lng":-8.62013701116825},{"lat":41.410203450849,"lng":-8.61991779727688},{"lat":41.4101894195945,"lng":-8.61970205432111},{"lat":41.4101489540034,"lng":-8.61956301606828},{"lat":41.4100404892018,"lng":-8.61932157282062},{"lat":41.4099959877388,"lng":-8.61913645784922},{"lat":41.4099823866264,"lng":-8.61890421893947},{"lat":41.4099871793292,"lng":-8.61830398838435},{"lat":41.409961697016,"lng":-8.61807082531432},{"lat":41.4099461937155,"lng":-8.61775484789484},{"lat":41.4100075338381,"lng":-8.61747329173831},{"lat":41.4101670795661,"lng":-8.61721758371763},{"lat":41.4102297270353,"lng":-8.61709414515926},{"lat":41.410178656319,"lng":-8.61673640513812},{"lat":41.4101353469989,"lng":-8.61652307949322},{"lat":41.4101739412488,"lng":-8.61631801544322},{"lat":41.4102434492526,"lng":-8.61612561737323},{"lat":41.4104122161353,"lng":-8.61592450775034},{"lat":41.410513783313,"lng":-8.6157751665556},{"lat":41.40937259174,"lng":-8.61430891528842},{"lat":41.4081716042799,"lng":-8.6134636842714},{"lat":41.4088840537608,"lng":-8.61185535824776},{"lat":41.409628731688,"lng":-8.61052493896677},{"lat":41.4105333108189,"lng":-8.60888865297115},{"lat":41.4115162508242,"lng":-8.60730515785217},{"lat":41.4126637145856,"lng":-8.60523076972758},{"lat":41.4117727203062,"lng":-8.60445979171444},{"lat":41.410268321787,"lng":-8.60335166892751},{"lat":41.4085788909741,"lng":-8.60221941860916},{"lat":41.4064858663089,"lng":-8.60063166365769},{"lat":41.4064056763848,"lng":-8.60055825043525},{"lat":41.4062214829767,"lng":-8.60033065533439},{"lat":41.405985632394,"lng":-8.60014168244314},{"lat":41.4057682608295,"lng":-8.59999230505849},{"lat":41.4055572556939,"lng":-8.59984871584145},{"lat":41.4053180880536,"lng":-8.59969906371295},{"lat":41.4051022798875,"lng":-8.59956345350364},{"lat":41.404850451142,"lng":-8.59942830181689},{"lat":41.4046250785159,"lng":-8.59927516257024},{"lat":41.4044050751192,"lng":-8.59899697304311},{"lat":41.4042291973316,"lng":-8.59873955855968},{"lat":41.4040627059695,"lng":-8.59856508054284},{"lat":41.403968424028,"lng":-8.59850221787747},{"lat":41.4037048782613,"lng":-8.59834785613511},{"lat":41.4034564904236,"lng":-8.59825208228051},{"lat":41.4033503175721,"lng":-8.59821174036855},{"lat":41.4029849094467,"lng":-8.59806430399275},{"lat":41.4028242576015,"lng":-8.59800311904674},{"lat":41.4026631203709,"lng":-8.59790761796861},{"lat":41.4024813312716,"lng":-8.59764071435655},{"lat":41.4023182775176,"lng":-8.59739488149648},{"lat":41.4021218793051,"lng":-8.59711209390036},{"lat":41.4019053053885,"lng":-8.59683059960405},{"lat":41.4016792351882,"lng":-8.59651579514501},{"lat":41.4016092195865,"lng":-8.59641760066964},{"lat":41.4014017057409,"lng":-8.59612218385469},{"lat":41.4012953643175,"lng":-8.59596800266621},{"lat":41.4011651358598,"lng":-8.59582058721137},{"lat":41.4009658126252,"lng":-8.59579273941754},{"lat":41.4008254791448,"lng":-8.59582797059054},{"lat":41.4007225729821,"lng":-8.59582568007938},{"lat":41.4007155302507,"lng":-8.5956744795816},{"lat":41.4006893656681,"lng":-8.59533121170551},{"lat":41.4006484405379,"lng":-8.59487448457288},{"lat":41.4006303319176,"lng":-8.59469761291604},{"lat":41.4006020028115,"lng":-8.59446590372426},{"lat":41.4005631270206,"lng":-8.59403573241267},{"lat":41.4005638394332,"lng":-8.59385875498777},{"lat":41.4005748126337,"lng":-8.59366104858244},{"lat":41.4005831222159,"lng":-8.59354140369203},{"lat":41.4005924139281,"lng":-8.59326870570144},{"lat":41.4005645617165,"lng":-8.59298521890099},{"lat":41.4005281116827,"lng":-8.59282460395644},{"lat":41.4004931781438,"lng":-8.59268935019323},{"lat":41.4004051394471,"lng":-8.59239228264816},{"lat":41.4003619924163,"lng":-8.59228399478531},{"lat":41.4002432973994,"lng":-8.59199807782467},{"lat":41.400089298959,"lng":-8.59166922059588},{"lat":41.4000190142996,"lng":-8.5914361412281},{"lat":41.399996411699,"lng":-8.59125660866789},{"lat":41.4000160476854,"lng":-8.59091702440489},{"lat":41.4000457079471,"lng":-8.59074946048977},{"lat":41.3999232638679,"lng":-8.59074764126424},{"lat":41.399811929526,"lng":-8.59080497079056},{"lat":41.3997014762503,"lng":-8.59084293415609},{"lat":41.3995892948159,"lng":-8.59088662634982},{"lat":41.3995030545285,"lng":-8.59092787445517},{"lat":41.399403932563,"lng":-8.59094822381754},{"lat":41.3992952204094,"lng":-8.59097854219537},{"lat":41.399156888194,"lng":-8.59103185829501},{"lat":41.3990067914475,"lng":-8.59107791320029},{"lat":41.398895458789,"lng":-8.59111192812694},{"lat":41.3986756701163,"lng":-8.59120304946335},{"lat":41.3982264654764,"lng":-8.5913843892238},{"lat":41.3980899004242,"lng":-8.59140088473947},{"lat":41.3978333046951,"lng":-8.59139715580674},{"lat":41.397718402404,"lng":-8.59142289247188},{"lat":41.3976175053706,"lng":-8.59145913186659},{"lat":41.397509128713,"lng":-8.5914736751338},{"lat":41.3973549423524,"lng":-8.59148441919959},{"lat":41.3972507384722,"lng":-8.59149073970251},{"lat":41.3971037909625,"lng":-8.59151445119551},{"lat":41.3968392553038,"lng":-8.59160477954383},{"lat":41.3966680703076,"lng":-8.59163382555893},{"lat":41.3964103888138,"lng":-8.59169896468454},{"lat":41.3961837569093,"lng":-8.59174459653621},{"lat":41.3960862191635,"lng":-8.59179663966652},{"lat":41.3959873674233,"lng":-8.59195150376407},{"lat":41.3959202466503,"lng":-8.59211939463363},{"lat":41.3956077805297,"lng":-8.59217134767378},{"lat":41.3954761240885,"lng":-8.59222219184413},{"lat":41.3953182431703,"lng":-8.59229975162321},{"lat":41.3951774679204,"lng":-8.59233355417316},{"lat":41.3950432268838,"lng":-8.59233260454799},{"lat":41.3947767256958,"lng":-8.59212625209816},{"lat":41.3946768077721,"lng":-8.59200860659428},{"lat":41.3945298974323,"lng":-8.59186444363038},{"lat":41.3943934974088,"lng":-8.59177332773162},{"lat":41.394293849897,"lng":-8.59172216459039},{"lat":41.3941906458225,"lng":-8.59170397822522},{"lat":41.3940975604499,"lng":-8.5917006946094},{"lat":41.3939373571698,"lng":-8.59173005076543},{"lat":41.3938923080791,"lng":-8.59187153997369},{"lat":41.3938442262164,"lng":-8.59227666404069},{"lat":41.3938169503142,"lng":-8.59256677821303},{"lat":41.3937800586732,"lng":-8.59294220715512},{"lat":41.3937544693088,"lng":-8.59308144096066},{"lat":41.3936692865779,"lng":-8.59341909831637},{"lat":41.3935517340936,"lng":-8.59367856669586},{"lat":41.3935418174813,"lng":-8.59397239107297},{"lat":41.3935735378423,"lng":-8.5941231536595},{"lat":41.3936287260666,"lng":-8.59425794389103},{"lat":41.3936846187055,"lng":-8.59448671786738},{"lat":41.3936962061341,"lng":-8.59462789138771},{"lat":41.3937119769857,"lng":-8.59502676471073},{"lat":41.3936789628784,"lng":-8.59526470749819},{"lat":41.3936501084386,"lng":-8.59538693928448},{"lat":41.3935851713717,"lng":-8.59579385043998},{"lat":41.3935841695431,"lng":-8.59610782922816},{"lat":41.3935650088324,"lng":-8.59646041732344},{"lat":41.393558086286,"lng":-8.59663553447465},{"lat":41.3935318756474,"lng":-8.59685990284004},{"lat":41.3935018709513,"lng":-8.59697758181284},{"lat":41.3934278838725,"lng":-8.59717063312048},{"lat":41.3933935423596,"lng":-8.59731290865981},{"lat":41.3932180486013,"lng":-8.59749340065922},{"lat":41.3931358842917,"lng":-8.59757244175509},{"lat":41.3930570972024,"lng":-8.59763991448436},{"lat":41.3928769689539,"lng":-8.59787381646708},{"lat":41.3927551283196,"lng":-8.59807788172733},{"lat":41.3925514950817,"lng":-8.59826904523175},{"lat":41.3923833854114,"lng":-8.59858230400176},{"lat":41.3922505075555,"lng":-8.59886484467274},{"lat":41.3921801214077,"lng":-8.59901391387173},{"lat":41.3921383222737,"lng":-8.59913018874719},{"lat":41.3919679936678,"lng":-8.59950093798162},{"lat":41.3919179461573,"lng":-8.59962970772044},{"lat":41.3918974451047,"lng":-8.59977734098821},{"lat":41.3919337972804,"lng":-8.59989334173883},{"lat":41.3919842241821,"lng":-8.60000250844083},{"lat":41.392046034958,"lng":-8.60010207543495},{"lat":41.392106835817,"lng":-8.60029513456995},{"lat":41.3921011539329,"lng":-8.60051760489417},{"lat":41.3920480724027,"lng":-8.60070673018214},{"lat":41.3919984442977,"lng":-8.60082103574224},{"lat":41.3919356209409,"lng":-8.60103604741179},{"lat":41.3918556670509,"lng":-8.601609021491},{"lat":41.3917719420884,"lng":-8.60191378990593},{"lat":41.3916123701564,"lng":-8.60253521334202},{"lat":41.3915633231539,"lng":-8.60283783175776},{"lat":41.3915679826003,"lng":-8.60308416815571},{"lat":41.3914997504268,"lng":-8.60338784932385},{"lat":41.3914474920814,"lng":-8.60370347730472},{"lat":41.3913846304419,"lng":-8.60414433451433},{"lat":41.391358823208,"lng":-8.60426837714225},{"lat":41.3913134695405,"lng":-8.60454639279527},{"lat":41.3912753544167,"lng":-8.60479409428779},{"lat":41.3912531367053,"lng":-8.60491959240852},{"lat":41.3912265190353,"lng":-8.60510842839237},{"lat":41.3911986942174,"lng":-8.60528351501317},{"lat":41.3911472554496,"lng":-8.60561755902576},{"lat":41.3911127605568,"lng":-8.60586038210866},{"lat":41.3910998422368,"lng":-8.60600113413432},{"lat":41.3910505183206,"lng":-8.60636843155595},{"lat":41.3910075787684,"lng":-8.60667121433326},{"lat":41.390954057565,"lng":-8.60709408279593},{"lat":41.3909038573261,"lng":-8.60745360166438},{"lat":41.3908509057626,"lng":-8.60782565427849},{"lat":41.3908038823175,"lng":-8.60822273729844},{"lat":41.390746285882,"lng":-8.6086930346747},{"lat":41.3907541647511,"lng":-8.60889898495394},{"lat":41.3909246126535,"lng":-8.6091141283934},{"lat":41.3910267943282,"lng":-8.60922643290472},{"lat":41.3911431326713,"lng":-8.60939730021654},{"lat":41.3911985151942,"lng":-8.60961197068672},{"lat":41.3912772946802,"lng":-8.61021885876975},{"lat":41.38877595833,"lng":-8.61010436762497},{"lat":41.3885347033419,"lng":-8.61162553925343},{"lat":41.3879727532476,"lng":-8.61272755225702},{"lat":41.3878090059581,"lng":-8.61306504727734},{"lat":41.3876667818939,"lng":-8.61342148053727},{"lat":41.3875429603328,"lng":-8.61396144147056},{"lat":41.3875045227227,"lng":-8.61428120631832},{"lat":41.3874912061289,"lng":-8.61457905090829},{"lat":41.3872512843404,"lng":-8.61688077118414},{"lat":41.3872065617572,"lng":-8.61702354726788},{"lat":41.3871275204725,"lng":-8.61714873030507},{"lat":41.3868511111116,"lng":-8.61750809202794},{"lat":41.386666258442,"lng":-8.61782879762796},{"lat":41.3865745778655,"lng":-8.618025020013},{"lat":41.3863987943641,"lng":-8.61826628977892},{"lat":41.3861866846557,"lng":-8.61851588537806},{"lat":41.3857272774712,"lng":-8.61911728196894},{"lat":41.3855498775375,"lng":-8.61948418253435},{"lat":41.3848847768714,"lng":-8.62085800373192},{"lat":41.3847050916739,"lng":-8.6212306091499},{"lat":41.3845751977261,"lng":-8.62147627038108},{"lat":41.3844538447896,"lng":-8.62176849716645},{"lat":41.3843272620161,"lng":-8.62208244949375},{"lat":41.3842553923084,"lng":-8.62231861724521},{"lat":41.3841786761041,"lng":-8.6225505661819},{"lat":41.3841263481616,"lng":-8.62268060802381},{"lat":41.3840693850422,"lng":-8.62277928257458},{"lat":41.3839799508869,"lng":-8.62284830583085},{"lat":41.3838646835274,"lng":-8.62289549611552},{"lat":41.3837077929099,"lng":-8.62292587811569},{"lat":41.3836145931101,"lng":-8.62294944650228},{"lat":41.3835030909746,"lng":-8.62297849399006},{"lat":41.383244797152,"lng":-8.62303536140974},{"lat":41.3829585511291,"lng":-8.62312250511398},{"lat":41.382735664118,"lng":-8.62332130768808},{"lat":41.3829294938818,"lng":-8.62360024335329},{"lat":41.3837932145963,"lng":-8.62473998465581},{"lat":41.3849778680368,"lng":-8.62568001560039},{"lat":41.3855341094538,"lng":-8.6268558585489},{"lat":41.3860670295606,"lng":-8.62869543136144},{"lat":41.3863991954722,"lng":-8.62929657883412},{"lat":41.3892203806416,"lng":-8.63078978639005},{"lat":41.3897559046499,"lng":-8.63156266440427},{"lat":41.3901060521463,"lng":-8.63224816603728},{"lat":41.3905470568612,"lng":-8.63311575238315},{"lat":41.3905910183097,"lng":-8.6333584474142},{"lat":41.3906609675334,"lng":-8.63346468105835},{"lat":41.3908170231621,"lng":-8.63374817369668},{"lat":41.3909071272472,"lng":-8.63402242720287},{"lat":41.3910254257593,"lng":-8.63431279898512},{"lat":41.3912356243892,"lng":-8.63475418036606},{"lat":41.3913953851101,"lng":-8.63515810152839},{"lat":41.391506569431,"lng":-8.63546970912158},{"lat":41.3916035214415,"lng":-8.63576147851663},{"lat":41.3916553389989,"lng":-8.63587511295665},{"lat":41.391715137557,"lng":-8.63603530966395},{"lat":41.3917492990556,"lng":-8.63621193831634},{"lat":41.3917765233975,"lng":-8.63653270013637},{"lat":41.391888208896,"lng":-8.63685196563951},{"lat":41.3919608463676,"lng":-8.63722580940266},{"lat":41.392091016288,"lng":-8.63751748629557},{"lat":41.3922716811747,"lng":-8.63777332597841},{"lat":41.3924636643628,"lng":-8.63807492597503},{"lat":41.3925446901233,"lng":-8.63819906670434},{"lat":41.3927426838006,"lng":-8.63846568704357},{"lat":41.392909809137,"lng":-8.63866618391466},{"lat":41.3932070521454,"lng":-8.63902792021747},{"lat":41.3936516066391,"lng":-8.63970251250336},{"lat":41.3943462220949,"lng":-8.64075727689189},{"lat":41.3967555829168,"lng":-8.64195056050681},{"lat":41.3989845693382,"lng":-8.64466175236073},{"lat":41.4018302792702,"lng":-8.64596118932573},{"lat":41.404451159055,"lng":-8.64715755529768},{"lat":41.4051136181428,"lng":-8.64738738501052},{"lat":41.4053521403329,"lng":-8.6476453181403},{"lat":41.40547206726,"lng":-8.64784307776098},{"lat":41.4057099249478,"lng":-8.64795400962441},{"lat":41.4059682500358,"lng":-8.64804567356029},{"lat":41.406096161674,"lng":-8.64807977930047},{"lat":41.4061305606211,"lng":-8.6482780180377},{"lat":41.4061940007275,"lng":-8.6485664989546},{"lat":41.4062928478824,"lng":-8.64893036131866},{"lat":41.4064296798772,"lng":-8.64921631853073},{"lat":41.4065746697998,"lng":-8.64945007552998},{"lat":41.4071319422994,"lng":-8.65022277321044},{"lat":41.4075639434249,"lng":-8.65076187380872},{"lat":41.4079471228117,"lng":-8.65120574911276},{"lat":41.4077636967616,"lng":-8.65138834628468},{"lat":41.4077000995525,"lng":-8.65157452377383},{"lat":41.407625701638,"lng":-8.65185820498066},{"lat":41.4075289432066,"lng":-8.65203096584544},{"lat":41.4072407626289,"lng":-8.65248060025669},{"lat":41.4071646386581,"lng":-8.65264814419313},{"lat":41.4069992063622,"lng":-8.65301313083734},{"lat":41.4069471823265,"lng":-8.65324723936168},{"lat":41.4069718938773,"lng":-8.65345588701829},{"lat":41.4070365854074,"lng":-8.65364560162652},{"lat":41.4071787973785,"lng":-8.65373475962075},{"lat":41.4073218177151,"lng":-8.65358713354138},{"lat":41.4075774451169,"lng":-8.65350654154738},{"lat":41.4079751669336,"lng":-8.65352120961029},{"lat":41.4095660959596,"lng":-8.65468029653706},{"lat":41.4107727970743,"lng":-8.65525151225539},{"lat":41.4111455204622,"lng":-8.65563435512276},{"lat":41.4112033978081,"lng":-8.65583575045361},{"lat":41.4101109986472,"lng":-8.65887899461778},{"lat":41.4095871525179,"lng":-8.6604284476062},{"lat":41.4092926632734,"lng":-8.66132196144019},{"lat":41.4089749491748,"lng":-8.66285678813031},{"lat":41.4088002386322,"lng":-8.66370186697389},{"lat":41.4082089370002,"lng":-8.66578361246918},{"lat":41.4089413955866,"lng":-8.66655802520109},{"lat":41.4094968651842,"lng":-8.66734511696929},{"lat":41.4101250576757,"lng":-8.66796393751725},{"lat":41.4100503724689,"lng":-8.66830370301285},{"lat":41.4096472495811,"lng":-8.66805562652391},{"lat":41.4095170093442,"lng":-8.66798947279352},{"lat":41.4094239804438,"lng":-8.66794812671203},{"lat":41.4093309514101,"lng":-8.66798120357722},{"lat":41.4092379222433,"lng":-8.66799774200982},{"lat":41.4090146516992,"lng":-8.66820447241727},{"lat":41.4088658042436,"lng":-8.66841120282473},{"lat":41.4087479664328,"lng":-8.66866754852997},{"lat":41.4086611384354,"lng":-8.66889081737003},{"lat":41.4084874820924,"lng":-8.66925466288715},{"lat":41.4083758456269,"lng":-8.66956062389018},{"lat":41.4082580069273,"lng":-8.66986658489322},{"lat":41.4081215618503,"lng":-8.67020562276144},{"lat":41.4079851164868,"lng":-8.67051985298077},{"lat":41.4078982874699,"lng":-8.67081754476751},{"lat":41.4077866499919,"lng":-8.67107389047275},{"lat":41.407833165631,"lng":-8.67123514019057},{"lat":41.4079758133834,"lng":-8.67132610156985},{"lat":41.4092782348986,"lng":-8.67144187059802},{"lat":41.4103139514662,"lng":-8.6720207157389},{"lat":41.4112132129382,"lng":-8.67313705993916},{"lat":41.4115233002147,"lng":-8.67368282821484},{"lat":41.4121186636365,"lng":-8.67520436401371},{"lat":41.4126271989051,"lng":-8.67622974683469},{"lat":41.4129496826225,"lng":-8.67695743786893},{"lat":41.4131481333451,"lng":-8.67740397554903},{"lat":41.4132845678653,"lng":-8.67768512890317},{"lat":41.4134667191586,"lng":-8.6780445517148},{"lat":41.4259114694384,"lng":-8.66189572594811},{"lat":41.4322450560538,"lng":-8.65462035811193}]},{"zone_id":"vcd10","points":[{"lat":41.382735664118,"lng":-8.62332130768808},{"lat":41.3826361524431,"lng":-8.62317830083712},{"lat":41.3812541397045,"lng":-8.62223243535376},{"lat":41.3811617722952,"lng":-8.62214471054463},{"lat":41.3808978600482,"lng":-8.62193590935586},{"lat":41.3808099690752,"lng":-8.62189675722843},{"lat":41.3805558262221,"lng":-8.62178282890191},{"lat":41.380428397758,"lng":-8.6217038068026},{"lat":41.3802127074447,"lng":-8.62154654074845},{"lat":41.3801212877634,"lng":-8.62146875244798},{"lat":41.3797811918181,"lng":-8.62119829679935},{"lat":41.3796952530919,"lng":-8.62114481441544},{"lat":41.3796159621867,"lng":-8.62107345022157},{"lat":41.3795060023187,"lng":-8.62099432212651},{"lat":41.3794214565804,"lng":-8.62088897335944},{"lat":41.3793469655686,"lng":-8.62081059255311},{"lat":41.3792747287553,"lng":-8.62073186893768},{"lat":41.3789696948896,"lng":-8.62079114325434},{"lat":41.378738067784,"lng":-8.6208871916831},{"lat":41.3785991069888,"lng":-8.62093695655279},{"lat":41.3784585629066,"lng":-8.62097809875798},{"lat":41.378270905997,"lng":-8.6210454257767},{"lat":41.3780066408713,"lng":-8.62112939133026},{"lat":41.3778809618431,"lng":-8.62116873362191},{"lat":41.3776543806714,"lng":-8.62126361549158},{"lat":41.3775540667749,"lng":-8.62128724930014},{"lat":41.3774003101363,"lng":-8.62132243383693},{"lat":41.3772968688575,"lng":-8.62136098778107},{"lat":41.3770060691059,"lng":-8.62139753186676},{"lat":41.3768428571151,"lng":-8.62141196963508},{"lat":41.3765974974141,"lng":-8.62141275243145},{"lat":41.3763693594127,"lng":-8.62138749350326},{"lat":41.3762763848986,"lng":-8.62135822615217},{"lat":41.3760144767941,"lng":-8.62120659676666},{"lat":41.3758092898474,"lng":-8.62103400489618},{"lat":41.3756206264324,"lng":-8.6208306922323},{"lat":41.3754489621674,"lng":-8.62063121570637},{"lat":41.3753636118347,"lng":-8.62052334817054},{"lat":41.3752049618101,"lng":-8.62037609173796},{"lat":41.3750972879099,"lng":-8.62028814401715},{"lat":41.37488441187,"lng":-8.62012386284391},{"lat":41.3746906698985,"lng":-8.61997024691641},{"lat":41.3744799853435,"lng":-8.61982056501709},{"lat":41.3743969777786,"lng":-8.61975575490242},{"lat":41.3741989650467,"lng":-8.6196114271115},{"lat":41.3739871637685,"lng":-8.61944847450602},{"lat":41.3737841750821,"lng":-8.61928857453163},{"lat":41.3735870054669,"lng":-8.61913613351953},{"lat":41.3733816358695,"lng":-8.61898578281261},{"lat":41.3731825731373,"lng":-8.61883356864814},{"lat":41.3729694622035,"lng":-8.61866128071481},{"lat":41.3727483904213,"lng":-8.61851762952326},{"lat":41.372653829797,"lng":-8.61845883304088},{"lat":41.3724896683122,"lng":-8.61842198774638},{"lat":41.3724409568369,"lng":-8.61857008341761},{"lat":41.3723494942134,"lng":-8.61863024232035},{"lat":41.3720810700059,"lng":-8.61863456963183},{"lat":41.371828839346,"lng":-8.61845088539751},{"lat":41.3716439439747,"lng":-8.61820590362434},{"lat":41.3715506971248,"lng":-8.61815644286101},{"lat":41.371283312816,"lng":-8.61810651841606},{"lat":41.3711136593755,"lng":-8.61804644425663},{"lat":41.3709676322492,"lng":-8.61804367925975},{"lat":41.3706911913704,"lng":-8.61794145836208},{"lat":41.3704476684149,"lng":-8.61791334591351},{"lat":41.3701653067727,"lng":-8.61770027853305},{"lat":41.3700715556081,"lng":-8.61764137207471},{"lat":41.3698471693789,"lng":-8.61753654991558},{"lat":41.3697504469871,"lng":-8.61752016731255},{"lat":41.3696539423598,"lng":-8.61751622186691},{"lat":41.3695329255661,"lng":-8.61747432683479},{"lat":41.3694470441479,"lng":-8.6174069897331},{"lat":41.36799404294,"lng":-8.61688438000294},{"lat":41.364549090907,"lng":-8.61542516935184},{"lat":41.3624149304293,"lng":-8.6160365340003},{"lat":41.3597342570579,"lng":-8.61565883584895},{"lat":41.3564703834695,"lng":-8.61519904481026},{"lat":41.3547589267432,"lng":-8.61626722033777},{"lat":41.3545692829976,"lng":-8.61592990419601},{"lat":41.3543304697021,"lng":-8.61484572628374},{"lat":41.3532674731596,"lng":-8.61375117617232},{"lat":41.3525300251361,"lng":-8.61339679709659},{"lat":41.351996845243,"lng":-8.63078589914483},{"lat":41.3520637804532,"lng":-8.64944803421536},{"lat":41.3495619580174,"lng":-8.6685085978663},{"lat":41.3637364585741,"lng":-8.68081514624987},{"lat":41.3667451417392,"lng":-8.67144393254349},{"lat":41.3742732691082,"lng":-8.66780577505211},{"lat":41.3775219118745,"lng":-8.66486693214357},{"lat":41.3815990718253,"lng":-8.65732090946911},{"lat":41.3821023572139,"lng":-8.65409847696349},{"lat":41.3829944811298,"lng":-8.65287053761713},{"lat":41.3838399227862,"lng":-8.65168329741912},{"lat":41.3850888854534,"lng":-8.64977095445624},{"lat":41.3860479970894,"lng":-8.64807355142726},{"lat":41.3890762819237,"lng":-8.64328025802728},{"lat":41.3936516066391,"lng":-8.63970251250336},{"lat":41.3932070521454,"lng":-8.63902792021747},{"lat":41.392909809137,"lng":-8.63866618391466},{"lat":41.3927426838006,"lng":-8.63846568704357},{"lat":41.3925446901233,"lng":-8.63819906670434},{"lat":41.3924636643628,"lng":-8.63807492597503},{"lat":41.3922716811747,"lng":-8.63777332597841},{"lat":41.392091016288,"lng":-8.63751748629557},{"lat":41.3919608463676,"lng":-8.63722580940266},{"lat":41.391888208896,"lng":-8.63685196563951},{"lat":41.3917765233975,"lng":-8.63653270013637},{"lat":41.3917492990556,"lng":-8.63621193831634},{"lat":41.391715137557,"lng":-8.63603530966395},{"lat":41.3916553389989,"lng":-8.63587511295665},{"lat":41.3916035214415,"lng":-8.63576147851663},{"lat":41.391506569431,"lng":-8.63546970912158},{"lat":41.3913953851101,"lng":-8.63515810152839},{"lat":41.3912356243892,"lng":-8.63475418036606},{"lat":41.3910254257593,"lng":-8.63431279898512},{"lat":41.3909071272472,"lng":-8.63402242720287},{"lat":41.3908170231621,"lng":-8.63374817369668},{"lat":41.3906609675334,"lng":-8.63346468105835},{"lat":41.3905910183097,"lng":-8.6333584474142},{"lat":41.3905470568612,"lng":-8.63311575238315},{"lat":41.3901060521463,"lng":-8.63224816603728},{"lat":41.3897559046499,"lng":-8.63156266440427},{"lat":41.3892203806416,"lng":-8.63078978639005},{"lat":41.3863991954722,"lng":-8.62929657883412},{"lat":41.3860670295606,"lng":-8.62869543136144},{"lat":41.3855341094538,"lng":-8.6268558585489},{"lat":41.3849778680368,"lng":-8.62568001560039},{"lat":41.3837932145963,"lng":-8.62473998465581},{"lat":41.3829294938818,"lng":-8.62360024335329},{"lat":41.382735664118,"lng":-8.62332130768808}]},{"zone_id":"vcd4","points":[{"lat":41.4018176815607,"lng":-8.71087338584299},{"lat":41.4019820263208,"lng":-8.71064386213363},{"lat":41.4021586717512,"lng":-8.71043041641507},{"lat":41.4022632495364,"lng":-8.71035098528535},{"lat":41.4025301377302,"lng":-8.71025828579095},{"lat":41.4027721050586,"lng":-8.71019765343615},{"lat":41.4029200847336,"lng":-8.71015280576297},{"lat":41.4030429082723,"lng":-8.71013273382474},{"lat":41.4033091697359,"lng":-8.71005760140037},{"lat":41.4034628802076,"lng":-8.71003648690511},{"lat":41.4036914353311,"lng":-8.71001064924883},{"lat":41.4037909660102,"lng":-8.71009213693133},{"lat":41.4067456553324,"lng":-8.71112940118999},{"lat":41.4077670185899,"lng":-8.71099269884406},{"lat":41.4089000955695,"lng":-8.71150376313699},{"lat":41.4102200259199,"lng":-8.71225798292633},{"lat":41.4131131685788,"lng":-8.71582213205772},{"lat":41.4147408223956,"lng":-8.71329353042077},{"lat":41.4151198912153,"lng":-8.7127040190178},{"lat":41.41617594985,"lng":-8.71106342731},{"lat":41.4185812221837,"lng":-8.7102409520862},{"lat":41.4190957077856,"lng":-8.70963495142478},{"lat":41.4255249533691,"lng":-8.70206097057779},{"lat":41.4256378906583,"lng":-8.70192797856333},{"lat":41.425646312107,"lng":-8.70170100195828},{"lat":41.4256542853892,"lng":-8.70150942783634},{"lat":41.425709030539,"lng":-8.70125485946773},{"lat":41.4257070270101,"lng":-8.70115040742103},{"lat":41.4256905544671,"lng":-8.70108243106932},{"lat":41.4256578500265,"lng":-8.70095366840526},{"lat":41.4256080401007,"lng":-8.70063574078556},{"lat":41.4255611757997,"lng":-8.70031210387727},{"lat":41.4255239470152,"lng":-8.69986508908855},{"lat":41.4255376996476,"lng":-8.69966470582381},{"lat":41.4237891653778,"lng":-8.69705163347358},{"lat":41.422685456386,"lng":-8.69491817566864},{"lat":41.4217925544861,"lng":-8.69273510256591},{"lat":41.4205771960609,"lng":-8.69035356827203},{"lat":41.4153558567449,"lng":-8.68072820050091},{"lat":41.4143016161673,"lng":-8.67990127887109},{"lat":41.4140783630261,"lng":-8.67970281767994},{"lat":41.4139543331716,"lng":-8.67947127962359},{"lat":41.4138179000582,"lng":-8.67912397253906},{"lat":41.4136566605547,"lng":-8.67859474269597},{"lat":41.4134667191586,"lng":-8.6780445517148},{"lat":41.4132845678653,"lng":-8.67768512890317},{"lat":41.4131481333451,"lng":-8.67740397554903},{"lat":41.4129496826225,"lng":-8.67695743786893},{"lat":41.4126271989051,"lng":-8.67622974683469},{"lat":41.4121186636365,"lng":-8.67520436401371},{"lat":41.4115233002147,"lng":-8.67368282821484},{"lat":41.4112132129382,"lng":-8.67313705993916},{"lat":41.4103139514662,"lng":-8.6720207157389},{"lat":41.4092782348986,"lng":-8.67144187059802},{"lat":41.4079758133834,"lng":-8.67132610156985},{"lat":41.407833165631,"lng":-8.67123514019057},{"lat":41.4077866499919,"lng":-8.67107389047275},{"lat":41.4078982874699,"lng":-8.67081754476751},{"lat":41.4079851164868,"lng":-8.67051985298077},{"lat":41.4081215618503,"lng":-8.67020562276144},{"lat":41.4082580069273,"lng":-8.66986658489322},{"lat":41.4083758456269,"lng":-8.66956062389018},{"lat":41.4084874820924,"lng":-8.66925466288715},{"lat":41.4086611384354,"lng":-8.66889081737003},{"lat":41.4087479664328,"lng":-8.66866754852997},{"lat":41.4088658042436,"lng":-8.66841120282473},{"lat":41.4090146516992,"lng":-8.66820447241727},{"lat":41.4092379222433,"lng":-8.66799774200982},{"lat":41.4093309514101,"lng":-8.66798120357722},{"lat":41.4094239804438,"lng":-8.66794812671203},{"lat":41.4095170093442,"lng":-8.66798947279352},{"lat":41.4096472495811,"lng":-8.66805562652391},{"lat":41.4100503724689,"lng":-8.66830370301285},{"lat":41.4101250576757,"lng":-8.66796393751725},{"lat":41.4094968651842,"lng":-8.66734511696929},{"lat":41.4089413955866,"lng":-8.66655802520109},{"lat":41.4082089370002,"lng":-8.66578361246918},{"lat":41.4088002386322,"lng":-8.66370186697389},{"lat":41.4089749491748,"lng":-8.66285678813031},{"lat":41.4092926632734,"lng":-8.66132196144019},{"lat":41.4095871525179,"lng":-8.6604284476062},{"lat":41.4101109986472,"lng":-8.65887899461778},{"lat":41.4112033978081,"lng":-8.65583575045361},{"lat":41.4111455204622,"lng":-8.65563435512276},{"lat":41.4107727970743,"lng":-8.65525151225539},{"lat":41.4095660959596,"lng":-8.65468029653706},{"lat":41.4079751669336,"lng":-8.65352120961029},{"lat":41.4075774451169,"lng":-8.65350654154738},{"lat":41.4073218177151,"lng":-8.65358713354138},{"lat":41.4071787973785,"lng":-8.65373475962075},{"lat":41.4070365854074,"lng":-8.65364560162652},{"lat":41.4069718938773,"lng":-8.65345588701829},{"lat":41.4069471823265,"lng":-8.65324723936168},{"lat":41.4069992063622,"lng":-8.65301313083734},{"lat":41.4071646386581,"lng":-8.65264814419313},{"lat":41.4072407626289,"lng":-8.65248060025669},{"lat":41.4075289432066,"lng":-8.65203096584544},{"lat":41.407625701638,"lng":-8.65185820498066},{"lat":41.4077000995525,"lng":-8.65157452377383},{"lat":41.4077636967616,"lng":-8.65138834628468},{"lat":41.4079471228117,"lng":-8.65120574911276},{"lat":41.4075639434249,"lng":-8.65076187380872},{"lat":41.4071319422994,"lng":-8.65022277321044},{"lat":41.4065746697998,"lng":-8.64945007552998},{"lat":41.4064296798772,"lng":-8.64921631853073},{"lat":41.4062928478824,"lng":-8.64893036131866},{"lat":41.4061940007275,"lng":-8.6485664989546},{"lat":41.4061305606211,"lng":-8.6482780180377},{"lat":41.406096161674,"lng":-8.64807977930047},{"lat":41.4059682500358,"lng":-8.64804567356029},{"lat":41.4057099249478,"lng":-8.64795400962441},{"lat":41.3936229211043,"lng":-8.67641521698991},{"lat":41.3928977808334,"lng":-8.67862369166141},{"lat":41.3918763275559,"lng":-8.68051327602236},{"lat":41.3858940592871,"lng":-8.69196082100039},{"lat":41.3892066024246,"lng":-8.6958152094767},{"lat":41.3915304102867,"lng":-8.69930267915656},{"lat":41.396299801057,"lng":-8.70209430746349},{"lat":41.3988787171824,"lng":-8.70630219165428},{"lat":41.3996776347247,"lng":-8.70858123346583},{"lat":41.401738184476,"lng":-8.71063238659623},{"lat":41.4018176815607,"lng":-8.71087338584299}]},{"zone_id":"vcd5","points":[{"lat":41.3667451417392,"lng":-8.67144393254349},{"lat":41.3637364585741,"lng":-8.68081514624987},{"lat":41.3666082560361,"lng":-8.68718045368996},{"lat":41.3705703955351,"lng":-8.69235311591936},{"lat":41.3697757323153,"lng":-8.69567303828525},{"lat":41.3704249625138,"lng":-8.70664895731251},{"lat":41.3722256479313,"lng":-8.70666483862611},{"lat":41.3728582794749,"lng":-8.70619230070652},{"lat":41.3734974709137,"lng":-8.70440497455479},{"lat":41.3730544406705,"lng":-8.70296672438494},{"lat":41.3727085319208,"lng":-8.70009499236543},{"lat":41.373250511574,"lng":-8.6997411276695},{"lat":41.3741496727054,"lng":-8.69998802705882},{"lat":41.3745109923236,"lng":-8.69975211219879},{"lat":41.3751465469365,"lng":-8.69868184807776},{"lat":41.3764046667057,"lng":-8.69917093622002},{"lat":41.3771273019796,"lng":-8.6986990788101},{"lat":41.3772202845649,"lng":-8.69810220040721},{"lat":41.3773179699079,"lng":-8.69654905997152},{"lat":41.3768724960405,"lng":-8.69558890460839},{"lat":41.3776845648796,"lng":-8.69523731564497},{"lat":41.3782230112315,"lng":-8.6956005893844},{"lat":41.3784940226911,"lng":-8.69655920408847},{"lat":41.3786490309836,"lng":-8.69824919837461},{"lat":41.3795617655302,"lng":-8.69800300825774},{"lat":41.3806474753507,"lng":-8.69693654923696},{"lat":41.381907955078,"lng":-8.69694748246999},{"lat":41.3820021037097,"lng":-8.69611147556921},{"lat":41.3821880394055,"lng":-8.69491762037096},{"lat":41.3835368566148,"lng":-8.69383541530972},{"lat":41.3858940592871,"lng":-8.69196082100039},{"lat":41.3918763275559,"lng":-8.68051327602236},{"lat":41.3928977808334,"lng":-8.67862369166141},{"lat":41.3936229211043,"lng":-8.67641521698991},{"lat":41.4057099249478,"lng":-8.64795400962441},{"lat":41.40547206726,"lng":-8.64784307776098},{"lat":41.4053521403329,"lng":-8.6476453181403},{"lat":41.4051136181428,"lng":-8.64738738501052},{"lat":41.404451159055,"lng":-8.64715755529768},{"lat":41.4018302792702,"lng":-8.64596118932573},{"lat":41.3989845693382,"lng":-8.64466175236073},{"lat":41.3967555829168,"lng":-8.64195056050681},{"lat":41.3943462220949,"lng":-8.64075727689189},{"lat":41.3936516066391,"lng":-8.63970251250336},{"lat":41.3890762819237,"lng":-8.64328025802728},{"lat":41.3860479970894,"lng":-8.64807355142726},{"lat":41.3850888854534,"lng":-8.64977095445624},{"lat":41.3838399227862,"lng":-8.65168329741912},{"lat":41.3829944811298,"lng":-8.65287053761713},{"lat":41.3821023572139,"lng":-8.65409847696349},{"lat":41.3815990718253,"lng":-8.65732090946911},{"lat":41.3775219118745,"lng":-8.66486693214357},{"lat":41.3742732691082,"lng":-8.66780577505211},{"lat":41.3667451417392,"lng":-8.67144393254349}]},{"zone_id":"pvz5","points":[{"lat":41.4539318784798,"lng":-8.72950717573282},{"lat":41.4546771835469,"lng":-8.72702770522605},{"lat":41.4553430475914,"lng":-8.72460917494803},{"lat":41.4552875591821,"lng":-8.72246211235429},{"lat":41.455272932864,"lng":-8.71979850808462},{"lat":41.4547573441465,"lng":-8.71363257493504},{"lat":41.454716290145,"lng":-8.71345448250331},{"lat":41.4546771784777,"lng":-8.71312345546268},{"lat":41.4546442922249,"lng":-8.71289708488792},{"lat":41.454607360808,"lng":-8.71257888388763},{"lat":41.4545484293006,"lng":-8.71226767083745},{"lat":41.4543077791673,"lng":-8.71165791151744},{"lat":41.4537880638958,"lng":-8.71034064133376},{"lat":41.4536850119681,"lng":-8.70995615458101},{"lat":41.4536301898334,"lng":-8.70977555065779},{"lat":41.4535992836312,"lng":-8.7096013857317},{"lat":41.4535252005793,"lng":-8.70934676829427},{"lat":41.4533568290804,"lng":-8.70913176777383},{"lat":41.4531744655643,"lng":-8.70892478160997},{"lat":41.4530184947226,"lng":-8.70876817413503},{"lat":41.452912586504,"lng":-8.70868310615102},{"lat":41.4528492569548,"lng":-8.70858213132551},{"lat":41.4527562102314,"lng":-8.7085540223126},{"lat":41.4526614860179,"lng":-8.70853726558347},{"lat":41.4524950149184,"lng":-8.70851771946827},{"lat":41.4522793246012,"lng":-8.70843777774743},{"lat":41.4521900499343,"lng":-8.70846594244839},{"lat":41.4520673481807,"lng":-8.70856921060249},{"lat":41.4518576879342,"lng":-8.70863413125628},{"lat":41.4517590234358,"lng":-8.70861255277315},{"lat":41.4516562259016,"lng":-8.70853612730045},{"lat":41.4515682390587,"lng":-8.70850411069897},{"lat":41.4514768113494,"lng":-8.70851215282373},{"lat":41.4513876298669,"lng":-8.70855696241679},{"lat":41.4512750317627,"lng":-8.70863889413115},{"lat":41.451162816544,"lng":-8.70869773639335},{"lat":41.4509845829828,"lng":-8.70872619455426},{"lat":41.4506995567842,"lng":-8.70882670275933},{"lat":41.4506036600071,"lng":-8.70875692130272},{"lat":41.4503658820839,"lng":-8.70864698595961},{"lat":41.4502413846378,"lng":-8.70856977554788},{"lat":41.450055862767,"lng":-8.70836432402485},{"lat":41.4498700709614,"lng":-8.70815732033219},{"lat":41.4498100206251,"lng":-8.70801246576408},{"lat":41.4497137995295,"lng":-8.70779141972568},{"lat":41.4495475802863,"lng":-8.70756005739478},{"lat":41.4493620511134,"lng":-8.70735461926847},{"lat":41.4492713339289,"lng":-8.70725711963732},{"lat":41.4491654110941,"lng":-8.70699482398562},{"lat":41.4490936101305,"lng":-8.70689546100034},{"lat":41.4488741799444,"lng":-8.70666279801553},{"lat":41.4486813681856,"lng":-8.70650983235905},{"lat":41.4486084839683,"lng":-8.70641058198311},{"lat":41.448492647798,"lng":-8.70611302243509},{"lat":41.4484152976785,"lng":-8.70593486598486},{"lat":41.4483400136661,"lng":-8.70584768463534},{"lat":41.4482485229909,"lng":-8.7058139635627},{"lat":41.4481887719599,"lng":-8.70604152172895},{"lat":41.4480278798383,"lng":-8.70595058979881},{"lat":41.4478077295168,"lng":-8.7058616490121},{"lat":41.4476613511782,"lng":-8.70587567686726},{"lat":41.4473813390441,"lng":-8.70587452333954},{"lat":41.4471114015187,"lng":-8.70562299350074},{"lat":41.446968948455,"lng":-8.70551787087821},{"lat":41.4467549125821,"lng":-8.70534090688558},{"lat":41.4465771182961,"lng":-8.70515445569626},{"lat":41.4463549900979,"lng":-8.70501117246838},{"lat":41.4462595708776,"lng":-8.70497108639112},{"lat":41.4460457645098,"lng":-8.70474794308657},{"lat":41.4458597561109,"lng":-8.70456489259923},{"lat":41.4455467169903,"lng":-8.70421774393705},{"lat":41.4454078065659,"lng":-8.70408812313825},{"lat":41.445182552183,"lng":-8.70393966960452},{"lat":41.4449628195981,"lng":-8.70373024404494},{"lat":41.444771636051,"lng":-8.70348397062639},{"lat":41.4446892197563,"lng":-8.70336358899693},{"lat":41.4444444394219,"lng":-8.70306814458899},{"lat":41.4442918537723,"lng":-8.70286338506614},{"lat":41.4440757310103,"lng":-8.70258004268011},{"lat":41.4438321429419,"lng":-8.70245956013061},{"lat":41.4435302135838,"lng":-8.70230350904489},{"lat":41.4432767441587,"lng":-8.70217887615148},{"lat":41.4430606116844,"lng":-8.70206138623565},{"lat":41.4428188641318,"lng":-8.70191471926806},{"lat":41.4425969547147,"lng":-8.7017811522267},{"lat":41.4424478861899,"lng":-8.70170158757376},{"lat":41.4423240450941,"lng":-8.70131029811567},{"lat":41.4422370164686,"lng":-8.70099423995556},{"lat":41.4421888878987,"lng":-8.70082259567427},{"lat":41.4421870718455,"lng":-8.70067934458078},{"lat":41.4465368702825,"lng":-8.68977495185937},{"lat":41.4466394140902,"lng":-8.6895174718977},{"lat":41.4491624693016,"lng":-8.68318964672187},{"lat":41.4478698372475,"lng":-8.6789818156026},{"lat":41.4457359439597,"lng":-8.67203843505192},{"lat":41.4456857488958,"lng":-8.67187527717428},{"lat":41.445622910447,"lng":-8.67158707847125},{"lat":41.4455713277102,"lng":-8.67137436717774},{"lat":41.44551460137,"lng":-8.67124023487236},{"lat":41.4454460384638,"lng":-8.67109606924534},{"lat":41.4453937282571,"lng":-8.67078953686977},{"lat":41.44535969271,"lng":-8.67048255744304},{"lat":41.445335433809,"lng":-8.67014526650462},{"lat":41.4452956485051,"lng":-8.66993109529672},{"lat":41.445095241064,"lng":-8.66963961773755},{"lat":41.4449851106782,"lng":-8.66948828914403},{"lat":41.4448825092413,"lng":-8.66915262695027},{"lat":41.4448325949905,"lng":-8.66881249295592},{"lat":41.4447435801073,"lng":-8.6684780212636},{"lat":41.444638651711,"lng":-8.66817800130355},{"lat":41.4445704042826,"lng":-8.6680224766107},{"lat":41.4444310334902,"lng":-8.66792439770554},{"lat":41.4441667180896,"lng":-8.66772466319917},{"lat":41.4440717436301,"lng":-8.66764359014879},{"lat":41.4438847510199,"lng":-8.66737006002957},{"lat":41.4437515030759,"lng":-8.66711743547606},{"lat":41.4436070976277,"lng":-8.66688290668987},{"lat":41.4434674971189,"lng":-8.6666404081769},{"lat":41.4433352922331,"lng":-8.6662791466152},{"lat":41.4433177853396,"lng":-8.66611315071187},{"lat":41.4432567166642,"lng":-8.66563365545349},{"lat":41.4431553518676,"lng":-8.66533905958898},{"lat":41.443099731647,"lng":-8.66519824077546},{"lat":41.4430128336733,"lng":-8.66500715107164},{"lat":41.4428178413774,"lng":-8.66465171897109},{"lat":41.4427499702742,"lng":-8.66453102885433},{"lat":41.4426123095467,"lng":-8.66431643033569},{"lat":41.44245658485,"lng":-8.66409343375127},{"lat":41.4422920480408,"lng":-8.66382955308382},{"lat":41.4421402370771,"lng":-8.6635790690973},{"lat":41.441995627293,"lng":-8.66332732267107},{"lat":41.4418303968591,"lng":-8.66303771722313},{"lat":41.4417345514924,"lng":-8.66289203505332},{"lat":41.4416515688252,"lng":-8.66276846898704},{"lat":41.4414891168579,"lng":-8.66253943466769},{"lat":41.4410087346959,"lng":-8.66194178839969},{"lat":41.4407997501194,"lng":-8.66174815688304},{"lat":41.4405527585682,"lng":-8.66151832508524},{"lat":41.4401537603668,"lng":-8.66115622808313},{"lat":41.4400005923229,"lng":-8.6610024240153},{"lat":41.4397760086942,"lng":-8.66081310537709},{"lat":41.4395669564848,"lng":-8.66063360102362},{"lat":41.4394122889811,"lng":-8.66049306423877},{"lat":41.4393219568529,"lng":-8.66032266516079},{"lat":41.4362957713899,"lng":-8.65740191372243},{"lat":41.435157567372,"lng":-8.65662244302572},{"lat":41.4342106019949,"lng":-8.65597962327392},{"lat":41.4337092901316,"lng":-8.65562060989627},{"lat":41.4328224466141,"lng":-8.65498104890366},{"lat":41.4322450560538,"lng":-8.65462035811193},{"lat":41.4259114694384,"lng":-8.66189572594811},{"lat":41.4134667191586,"lng":-8.6780445517148},{"lat":41.4136566605547,"lng":-8.67859474269597},{"lat":41.4138179000582,"lng":-8.67912397253906},{"lat":41.4139543331716,"lng":-8.67947127962359},{"lat":41.4140783630261,"lng":-8.67970281767994},{"lat":41.4143016161673,"lng":-8.67990127887109},{"lat":41.4153558567449,"lng":-8.68072820050091},{"lat":41.4205771960609,"lng":-8.69035356827203},{"lat":41.4217925544861,"lng":-8.69273510256591},{"lat":41.422685456386,"lng":-8.69491817566864},{"lat":41.4237891653778,"lng":-8.69705163347358},{"lat":41.4255376996476,"lng":-8.69966470582381},{"lat":41.4255239470152,"lng":-8.69986508908855},{"lat":41.4255611757997,"lng":-8.70031210387727},{"lat":41.4256080401007,"lng":-8.70063574078556},{"lat":41.4256578500265,"lng":-8.70095366840526},{"lat":41.4256905544671,"lng":-8.70108243106932},{"lat":41.4257070270101,"lng":-8.70115040742103},{"lat":41.425709030539,"lng":-8.70125485946773},{"lat":41.4256542853892,"lng":-8.70150942783634},{"lat":41.425646312107,"lng":-8.70170100195828},{"lat":41.4256378906583,"lng":-8.70192797856333},{"lat":41.4255249533691,"lng":-8.70206097057779},{"lat":41.4190957077856,"lng":-8.70963495142478},{"lat":41.4191765703282,"lng":-8.70992036194058},{"lat":41.4218816524861,"lng":-8.71915133919041},{"lat":41.4290196291875,"lng":-8.73401582594853},{"lat":41.4296245061748,"lng":-8.73557363069337},{"lat":41.430865150228,"lng":-8.73876905673636},{"lat":41.431667398168,"lng":-8.73865813865875},{"lat":41.4332086822672,"lng":-8.73780129773914},{"lat":41.4352917346292,"lng":-8.73700745297451},{"lat":41.4388129330334,"lng":-8.7347582261414},{"lat":41.4467969146246,"lng":-8.73098746350942},{"lat":41.4535815535772,"lng":-8.72934944589258},{"lat":41.4536059906121,"lng":-8.72951590968177},{"lat":41.4537208415662,"lng":-8.7295055897252},{"lat":41.4538139366909,"lng":-8.72947126248116},{"lat":41.4539318784798,"lng":-8.72950717573282}]},{"zone_id":"vcd2","points":[{"lat":41.4018176815607,"lng":-8.71087338584299},{"lat":41.401738184476,"lng":-8.71063238659623},{"lat":41.3996776347247,"lng":-8.70858123346583},{"lat":41.3988787171824,"lng":-8.70630219165428},{"lat":41.396299801057,"lng":-8.70209430746349},{"lat":41.3915304102867,"lng":-8.69930267915656},{"lat":41.3892066024246,"lng":-8.6958152094767},{"lat":41.3858940592871,"lng":-8.69196082100039},{"lat":41.3835368566148,"lng":-8.69383541530972},{"lat":41.3821880394055,"lng":-8.69491762037096},{"lat":41.3820021037097,"lng":-8.69611147556921},{"lat":41.381907955078,"lng":-8.69694748246999},{"lat":41.3806474753507,"lng":-8.69693654923696},{"lat":41.3795617655302,"lng":-8.69800300825774},{"lat":41.3786490309836,"lng":-8.69824919837461},{"lat":41.3784940226911,"lng":-8.69655920408847},{"lat":41.3782230112315,"lng":-8.6956005893844},{"lat":41.3776845648796,"lng":-8.69523731564497},{"lat":41.3768724960405,"lng":-8.69558890460839},{"lat":41.3773179699079,"lng":-8.69654905997152},{"lat":41.3772202845649,"lng":-8.69810220040721},{"lat":41.3771273019796,"lng":-8.6986990788101},{"lat":41.3764046667057,"lng":-8.69917093622002},{"lat":41.3751465469365,"lng":-8.69868184807776},{"lat":41.3745109923236,"lng":-8.69975211219879},{"lat":41.3741496727054,"lng":-8.69998802705882},{"lat":41.373250511574,"lng":-8.6997411276695},{"lat":41.3727085319208,"lng":-8.70009499236543},{"lat":41.3730544406705,"lng":-8.70296672438494},{"lat":41.3734974709137,"lng":-8.70440497455479},{"lat":41.3728582794749,"lng":-8.70619230070652},{"lat":41.3722256479313,"lng":-8.70666483862611},{"lat":41.3704249625138,"lng":-8.70664895731251},{"lat":41.3688593632432,"lng":-8.70917520003271},{"lat":41.3673439918013,"lng":-8.71056592648024},{"lat":41.365265364986,"lng":-8.71210117503849},{"lat":41.363367996537,"lng":-8.71339892000288},{"lat":41.3629081282364,"lng":-8.71530699742866},{"lat":41.3611896927116,"lng":-8.71989412117795},{"lat":41.3600028791235,"lng":-8.7242211089019},{"lat":41.3615334745678,"lng":-8.72423498708583},{"lat":41.3630609925034,"lng":-8.72484640145503},{"lat":41.3641444893716,"lng":-8.72425866410789},{"lat":41.3646323301667,"lng":-8.72327147388323},{"lat":41.364987779075,"lng":-8.7228613277294},{"lat":41.3664348796517,"lng":-8.72349658121242},{"lat":41.3681600384253,"lng":-8.72396879812804},{"lat":41.3708438625384,"lng":-8.7247398209607},{"lat":41.3719573389955,"lng":-8.72506714300571},{"lat":41.3732325689259,"lng":-8.72566922591909},{"lat":41.3736654144621,"lng":-8.72627083773776},{"lat":41.3740756482506,"lng":-8.72764257951594},{"lat":41.374534962606,"lng":-8.7282906921502},{"lat":41.3750856436027,"lng":-8.72853616244095},{"lat":41.3763111605616,"lng":-8.72827862165596},{"lat":41.3769113113583,"lng":-8.72764435977358},{"lat":41.3809430132192,"lng":-8.73042674200666},{"lat":41.3820889687419,"lng":-8.73151274838522},{"lat":41.3875707992968,"lng":-8.72686615688738},{"lat":41.3882623779305,"lng":-8.72560961080653},{"lat":41.3893409893032,"lng":-8.72344827732661},{"lat":41.3934080984688,"lng":-8.72141895373806},{"lat":41.3945039218114,"lng":-8.72098621966262},{"lat":41.394729616223,"lng":-8.7209919683111},{"lat":41.3948210199215,"lng":-8.72088374470249},{"lat":41.3950348207872,"lng":-8.72073334643199},{"lat":41.3951910619932,"lng":-8.72053495260946},{"lat":41.395292492261,"lng":-8.72033283724744},{"lat":41.3951690774378,"lng":-8.72004630680053},{"lat":41.3950469131015,"lng":-8.71977891666694},{"lat":41.3949088481348,"lng":-8.71950373679624},{"lat":41.3948463189541,"lng":-8.71935430772572},{"lat":41.3947395492742,"lng":-8.71915773136485},{"lat":41.394596334228,"lng":-8.71883251981039},{"lat":41.3944690422711,"lng":-8.71856366061094},{"lat":41.3943463123041,"lng":-8.7183009355224},{"lat":41.3942831436391,"lng":-8.71806445791698},{"lat":41.394253693604,"lng":-8.71776574618078},{"lat":41.3942292040676,"lng":-8.71755317492045},{"lat":41.3941438731591,"lng":-8.7173894345491},{"lat":41.3939785496972,"lng":-8.71714450403666},{"lat":41.3937923251298,"lng":-8.7168661472917},{"lat":41.3942667577545,"lng":-8.71491353402916},{"lat":41.3941224476896,"lng":-8.71474951037982},{"lat":41.393994151002,"lng":-8.71462413423148},{"lat":41.3937888090261,"lng":-8.71440396208113},{"lat":41.3946815624872,"lng":-8.71254952249762},{"lat":41.3966646525811,"lng":-8.70979930015408},{"lat":41.3976797744795,"lng":-8.71065167754244},{"lat":41.3978579081888,"lng":-8.71087722078913},{"lat":41.3981215148413,"lng":-8.71109635743208},{"lat":41.3982473594273,"lng":-8.71113777382662},{"lat":41.3986314153435,"lng":-8.71287721505642},{"lat":41.3998054230182,"lng":-8.71237851583744},{"lat":41.3998319774189,"lng":-8.71225510715643},{"lat":41.3999557730809,"lng":-8.71214930528145},{"lat":41.4002112242692,"lng":-8.71198643987383},{"lat":41.4004162129646,"lng":-8.71183209179212},{"lat":41.4006325988124,"lng":-8.71166625030155},{"lat":41.4007696894507,"lng":-8.71160193838707},{"lat":41.4010759276589,"lng":-8.71152717134868},{"lat":41.4012916019531,"lng":-8.71143187173616},{"lat":41.4014893299635,"lng":-8.71128845917102},{"lat":41.4016627352399,"lng":-8.71109328451354},{"lat":41.4017497944799,"lng":-8.71096993664287},{"lat":41.4018176815607,"lng":-8.71087338584299}]},{"zone_id":"pv_vc","points":[{"lat":41.3424058339534,"lng":-8.75438621716039},{"lat":41.3423666036778,"lng":-8.75444749289449},{"lat":41.3423824880617,"lng":-8.75452375507523},{"lat":41.3423982325449,"lng":-8.75459331954657},{"lat":41.3424169591199,"lng":-8.75465945412303},{"lat":41.3423749116567,"lng":-8.75474292182924},{"lat":41.3423970903174,"lng":-8.75480430906339},{"lat":41.3424282323478,"lng":-8.75487271193628},{"lat":41.3424585713302,"lng":-8.75492401565614},{"lat":41.342476186745,"lng":-8.75499646900446},{"lat":41.3425214133244,"lng":-8.75512498052019},{"lat":41.3425534932099,"lng":-8.75521991128424},{"lat":41.3426057710395,"lng":-8.75517620581493},{"lat":41.3426554193292,"lng":-8.75515349919306},{"lat":41.3427101645106,"lng":-8.75518652248172},{"lat":41.3427152999452,"lng":-8.75526949257774},{"lat":41.342733847464,"lng":-8.75533609998496},{"lat":41.342767295372,"lng":-8.75544502464952},{"lat":41.3428027200158,"lng":-8.75538789224923},{"lat":41.3427978725269,"lng":-8.75530194042051},{"lat":41.3428478742953,"lng":-8.75521400181626},{"lat":41.3428779619894,"lng":-8.75532743450362},{"lat":41.3429615912549,"lng":-8.75531437261973},{"lat":41.3429925495755,"lng":-8.75525134878003},{"lat":41.3430595843789,"lng":-8.75519380341714},{"lat":41.3431077344908,"lng":-8.75513093682866},{"lat":41.3431963301419,"lng":-8.75501564528845},{"lat":41.3432202081475,"lng":-8.75512925957912},{"lat":41.3432908101055,"lng":-8.75519433467868},{"lat":41.3433834394461,"lng":-8.75523165908132},{"lat":41.3434318686347,"lng":-8.7552677270844},{"lat":41.3434738564812,"lng":-8.75531233297},{"lat":41.3434278292223,"lng":-8.75534834050068},{"lat":41.3433834751695,"lng":-8.75537455620227},{"lat":41.3434031198652,"lng":-8.75543831058691},{"lat":41.3434414229517,"lng":-8.75548228716679},{"lat":41.3435232858152,"lng":-8.7554962082671},{"lat":41.343530043972,"lng":-8.75542829269699},{"lat":41.3435699669533,"lng":-8.75538876845411},{"lat":41.3436205262576,"lng":-8.7553641542084},{"lat":41.343639702507,"lng":-8.75529802620321},{"lat":41.3435507903197,"lng":-8.75523863114619},{"lat":41.3434934514575,"lng":-8.75518563325613},{"lat":41.3436046008332,"lng":-8.75512934299044},{"lat":41.3437028648809,"lng":-8.75510734033165},{"lat":41.3437933145872,"lng":-8.75504918308764},{"lat":41.3438520032801,"lng":-8.75500254675348},{"lat":41.3438989342446,"lng":-8.75496595316026},{"lat":41.3439575549198,"lng":-8.7549485926664},{"lat":41.344031671539,"lng":-8.75492958400132},{"lat":41.3441189846181,"lng":-8.75491823235997},{"lat":41.3442443792097,"lng":-8.75489146728576},{"lat":41.3443561010189,"lng":-8.75487903314736},{"lat":41.3444756870396,"lng":-8.75487682909834},{"lat":41.3446083963207,"lng":-8.75484655492866},{"lat":41.3447128325161,"lng":-8.75483225513633},{"lat":41.3447895681052,"lng":-8.75481183563965},{"lat":41.3449318352564,"lng":-8.75477914195847},{"lat":41.3450653566266,"lng":-8.75474755983584},{"lat":41.3451166572166,"lng":-8.7547356239179},{"lat":41.3451763828826,"lng":-8.75471396699722},{"lat":41.3452268942597,"lng":-8.75468051794525},{"lat":41.3452837070929,"lng":-8.75464855775037},{"lat":41.3454733637466,"lng":-8.75454378646807},{"lat":41.3455673167293,"lng":-8.75448625452895},{"lat":41.3456522028454,"lng":-8.75445755283637},{"lat":41.3457315127327,"lng":-8.75444396742207},{"lat":41.3457964089911,"lng":-8.7544151952032},{"lat":41.3458748982393,"lng":-8.75437101574756},{"lat":41.3459445867265,"lng":-8.7543381094361},{"lat":41.3460384147081,"lng":-8.75430459189103},{"lat":41.3461384280733,"lng":-8.75429215834362},{"lat":41.3462418924638,"lng":-8.7542909946692},{"lat":41.3463336199002,"lng":-8.75429604870598},{"lat":41.3464200375397,"lng":-8.75430045790852},{"lat":41.3464867720066,"lng":-8.75428066304104},{"lat":41.3465732251872,"lng":-8.75427850259412},{"lat":41.3467290389252,"lng":-8.75428823371992},{"lat":41.3467741181226,"lng":-8.75427766669188},{"lat":41.3468894540546,"lng":-8.75427888628423},{"lat":41.3469874747211,"lng":-8.7542683498137},{"lat":41.3470930695141,"lng":-8.75425657016805},{"lat":41.3472027274458,"lng":-8.75425917114108},{"lat":41.3472973002044,"lng":-8.75425445735805},{"lat":41.3473595484254,"lng":-8.75424919507821},{"lat":41.3474397497983,"lng":-8.75423729348076},{"lat":41.3475143781883,"lng":-8.75423991442105},{"lat":41.3475793185603,"lng":-8.7542531995288},{"lat":41.3476394421763,"lng":-8.75425747832405},{"lat":41.3477437962354,"lng":-8.75427507857248},{"lat":41.3478049944944,"lng":-8.75429741540142},{"lat":41.347853573262,"lng":-8.75430588414807},{"lat":41.3479039661509,"lng":-8.7543119798555},{"lat":41.3479448979082,"lng":-8.75438537583243},{"lat":41.3479916860641,"lng":-8.75442441472051},{"lat":41.3480263806418,"lng":-8.75446955459493},{"lat":41.3480899463531,"lng":-8.75457041535474},{"lat":41.3481127074487,"lng":-8.75467386501619},{"lat":41.3480943306849,"lng":-8.75474108701063},{"lat":41.3480536910683,"lng":-8.75478012677354},{"lat":41.3480848147886,"lng":-8.7548528353794},{"lat":41.3480914430836,"lng":-8.75492590455519},{"lat":41.3481290826152,"lng":-8.75497537283712},{"lat":41.3481412367163,"lng":-8.75507514260194},{"lat":41.3481281033153,"lng":-8.75515615576569},{"lat":41.3481555417123,"lng":-8.7552431649058},{"lat":41.3482157214026,"lng":-8.75525472977378},{"lat":41.3482637711892,"lng":-8.75521037930854},{"lat":41.3482688535046,"lng":-8.75510348927274},{"lat":41.3483006717304,"lng":-8.75504703193043},{"lat":41.3483083663412,"lng":-8.75497338475965},{"lat":41.3482836135494,"lng":-8.75487169843454},{"lat":41.3482609141253,"lng":-8.75475689147527},{"lat":41.348301439759,"lng":-8.75468964689861},{"lat":41.3483706824119,"lng":-8.75457273223227},{"lat":41.3484086400924,"lng":-8.75453091565364},{"lat":41.3484775819369,"lng":-8.75446955711246},{"lat":41.3485297869182,"lng":-8.75444018198418},{"lat":41.3485921418268,"lng":-8.75441520358579},{"lat":41.3486462432584,"lng":-8.75440078294246},{"lat":41.3487039457674,"lng":-8.75438663698329},{"lat":41.3487626127793,"lng":-8.75437835595394},{"lat":41.3488444542894,"lng":-8.7543794968269},{"lat":41.34890209804,"lng":-8.75439294648587},{"lat":41.3489739106774,"lng":-8.75441752837785},{"lat":41.3490323773704,"lng":-8.75444533494941},{"lat":41.3491244544929,"lng":-8.75453451721106},{"lat":41.3491962341343,"lng":-8.754581921282},{"lat":41.3492931514875,"lng":-8.75449262484355},{"lat":41.3493609758619,"lng":-8.75443878655003},{"lat":41.3494053044939,"lng":-8.75441722156095},{"lat":41.3494774069698,"lng":-8.7544040474028},{"lat":41.3495663264634,"lng":-8.75441194629054},{"lat":41.3496149168396,"lng":-8.75441826544477},{"lat":41.3496652612172,"lng":-8.75443332166007},{"lat":41.3497300089273,"lng":-8.75446453368669},{"lat":41.3497953951616,"lng":-8.75451163677871},{"lat":41.3498878478857,"lng":-8.75458254173887},{"lat":41.3499389713617,"lng":-8.75460238641927},{"lat":41.3499875364958,"lng":-8.75461336617045},{"lat":41.3500776120659,"lng":-8.75460753482947},{"lat":41.350149004777,"lng":-8.75457583150173},{"lat":41.3502014620563,"lng":-8.75455004377583},{"lat":41.3502590617183,"lng":-8.75452143386259},{"lat":41.3503655557307,"lng":-8.75449317667812},{"lat":41.3504727666914,"lng":-8.75449885981236},{"lat":41.3505669883922,"lng":-8.75455807053395},{"lat":41.3506423587394,"lng":-8.75460766001358},{"lat":41.3506497695174,"lng":-8.75470165256512},{"lat":41.3506280082098,"lng":-8.75476298893538},{"lat":41.3506096856222,"lng":-8.75482112630116},{"lat":41.3506488236114,"lng":-8.7548763464837},{"lat":41.350694912708,"lng":-8.75487905655583},{"lat":41.3507415349622,"lng":-8.75489945303963},{"lat":41.350790962468,"lng":-8.75491749254967},{"lat":41.3507681484739,"lng":-8.75497343707857},{"lat":41.3508067124899,"lng":-8.75501849592331},{"lat":41.3508629747784,"lng":-8.75502130320636},{"lat":41.3509191575994,"lng":-8.75498862013319},{"lat":41.3509678781522,"lng":-8.75496996815475},{"lat":41.3510032441673,"lng":-8.75492453974038},{"lat":41.3510770578526,"lng":-8.7549614483416},{"lat":41.3511468135197,"lng":-8.75501648716508},{"lat":41.3512103865862,"lng":-8.75508245789723},{"lat":41.3512533609586,"lng":-8.75514464639829},{"lat":41.3512917397441,"lng":-8.7552238782924},{"lat":41.3512696157549,"lng":-8.7553186648446},{"lat":41.3513270735957,"lng":-8.75534968204609},{"lat":41.3512883994813,"lng":-8.75547442511039},{"lat":41.351233055365,"lng":-8.75556769897051},{"lat":41.3511457675578,"lng":-8.75557439704075},{"lat":41.351094230067,"lng":-8.75558083568455},{"lat":41.3510821884188,"lng":-8.75567571816665},{"lat":41.3511358110042,"lng":-8.75575055528945},{"lat":41.3511915759133,"lng":-8.75574487164273},{"lat":41.3512563499885,"lng":-8.75573867980062},{"lat":41.351337382709,"lng":-8.75573957434828},{"lat":41.3513958393354,"lng":-8.75575340076695},{"lat":41.351435876625,"lng":-8.75567610627296},{"lat":41.3514209790743,"lng":-8.75561657597516},{"lat":41.3513778538676,"lng":-8.75558318475933},{"lat":41.3514404999776,"lng":-8.75550360216419},{"lat":41.3514712824303,"lng":-8.75532357545839},{"lat":41.3515245947272,"lng":-8.75525609615441},{"lat":41.3516439563527,"lng":-8.75517920466442},{"lat":41.3517404340609,"lng":-8.75513770467594},{"lat":41.3518622843579,"lng":-8.75508437413577},{"lat":41.3520284019755,"lng":-8.75501988072147},{"lat":41.3522093341443,"lng":-8.75501264673602},{"lat":41.3523226855046,"lng":-8.75503189123014},{"lat":41.3524297373607,"lng":-8.75504928611336},{"lat":41.352537082905,"lng":-8.75503011163562},{"lat":41.3525911738584,"lng":-8.75501856186417},{"lat":41.3526993800558,"lng":-8.75498911893366},{"lat":41.3530225357324,"lng":-8.75500738139478},{"lat":41.3531208370899,"lng":-8.75499517845253},{"lat":41.3532720429292,"lng":-8.75497403180422},{"lat":41.3534460289044,"lng":-8.75497007672906},{"lat":41.3536285027563,"lng":-8.75497732074417},{"lat":41.3538094833035,"lng":-8.75501130749088},{"lat":41.3539677449383,"lng":-8.75503444686498},{"lat":41.3541164106069,"lng":-8.7550335956867},{"lat":41.3542325428052,"lng":-8.75503733528801},{"lat":41.3543863053217,"lng":-8.75505971743174},{"lat":41.3545716063231,"lng":-8.75509399314795},{"lat":41.3547497374792,"lng":-8.75513847082627},{"lat":41.3549159736943,"lng":-8.75520159940117},{"lat":41.3551180485373,"lng":-8.75529829871495},{"lat":41.3552986198156,"lng":-8.75537435278528},{"lat":41.3555202559928,"lng":-8.75548330673858},{"lat":41.355692892557,"lng":-8.75556119582127},{"lat":41.3558345409717,"lng":-8.75564261554949},{"lat":41.355959588823,"lng":-8.75567978366899},{"lat":41.3560395749244,"lng":-8.75572428751688},{"lat":41.3561380942202,"lng":-8.75580457457836},{"lat":41.356242015026,"lng":-8.75588528080096},{"lat":41.3563551005517,"lng":-8.75593679271583},{"lat":41.3565041333914,"lng":-8.7560175701845},{"lat":41.3567817323293,"lng":-8.7561333942047},{"lat":41.3570244208845,"lng":-8.75621243742149},{"lat":41.3572607487022,"lng":-8.75631759108263},{"lat":41.3573756040518,"lng":-8.75637485783146},{"lat":41.3574535567838,"lng":-8.75647873749129},{"lat":41.3574944383449,"lng":-8.75656110692023},{"lat":41.3575218211279,"lng":-8.75664239897581},{"lat":41.3575613226449,"lng":-8.75668054079246},{"lat":41.3576304108136,"lng":-8.75670916268021},{"lat":41.3577034166422,"lng":-8.75672886700868},{"lat":41.3577866531436,"lng":-8.75678810265381},{"lat":41.3578205897367,"lng":-8.75687255649875},{"lat":41.3578571061599,"lng":-8.75693039006819},{"lat":41.3579137079737,"lng":-8.75696977118193},{"lat":41.3579774866296,"lng":-8.7569810200154},{"lat":41.3580556768909,"lng":-8.756991091337},{"lat":41.3581403563458,"lng":-8.75700050361248},{"lat":41.3582219197411,"lng":-8.75705279221671},{"lat":41.3582911489129,"lng":-8.75710544361521},{"lat":41.3584222205115,"lng":-8.75716083791229},{"lat":41.3585299867154,"lng":-8.75721242303972},{"lat":41.3587032649746,"lng":-8.75728865701331},{"lat":41.3588605496174,"lng":-8.75734238663812},{"lat":41.3589853585431,"lng":-8.75739006880847},{"lat":41.3591101958353,"lng":-8.75746583948433},{"lat":41.3592088654326,"lng":-8.75753490855461},{"lat":41.3593118260741,"lng":-8.75764297239677},{"lat":41.3595528171072,"lng":-8.75786733770837},{"lat":41.3596748927736,"lng":-8.75793830192191},{"lat":41.3597687560007,"lng":-8.7580630153591},{"lat":41.3598061914598,"lng":-8.7581164325283},{"lat":41.3598664252346,"lng":-8.75816696986618},{"lat":41.359973165904,"lng":-8.7582423289744},{"lat":41.3600960651941,"lng":-8.7583098362789},{"lat":41.3601750290045,"lng":-8.75837740291272},{"lat":41.3602571095969,"lng":-8.75849996796016},{"lat":41.360309616238,"lng":-8.75862966880416},{"lat":41.3604105329451,"lng":-8.75874882914309},{"lat":41.3605271490851,"lng":-8.75882990778201},{"lat":41.360606892861,"lng":-8.75890201751686},{"lat":41.3606605804286,"lng":-8.75896360591395},{"lat":41.3607509448547,"lng":-8.75907000402541},{"lat":41.3608259900057,"lng":-8.7591779247511},{"lat":41.3608849041634,"lng":-8.75927290843633},{"lat":41.3608405630532,"lng":-8.75934550233885},{"lat":41.3608112835959,"lng":-8.75939828631726},{"lat":41.3608693402206,"lng":-8.75948441322473},{"lat":41.360962180066,"lng":-8.75950036555644},{"lat":41.3609796762964,"lng":-8.75959411657191},{"lat":41.3610734001114,"lng":-8.75957900350379},{"lat":41.3611320956785,"lng":-8.75954801309655},{"lat":41.3611896538869,"lng":-8.75952813107003},{"lat":41.361529515722,"lng":-8.76009800988151},{"lat":41.361904116475,"lng":-8.76093308037006},{"lat":41.3619024725172,"lng":-8.7610367997283},{"lat":41.361698648071,"lng":-8.76137676646204},{"lat":41.3620780197406,"lng":-8.76104267950049},{"lat":41.362158022601,"lng":-8.76108372156496},{"lat":41.3622101471891,"lng":-8.76111900527193},{"lat":41.3622782322226,"lng":-8.76116615019691},{"lat":41.3623465897059,"lng":-8.76117912436945},{"lat":41.3624398591304,"lng":-8.76124802355598},{"lat":41.3625009917473,"lng":-8.76128207877728},{"lat":41.3625485492224,"lng":-8.76129592773221},{"lat":41.3625817956758,"lng":-8.76134130118187},{"lat":41.3626446078504,"lng":-8.76136449357045},{"lat":41.3627211961922,"lng":-8.76135460024002},{"lat":41.3627900066703,"lng":-8.76128570107925},{"lat":41.3628279924282,"lng":-8.76123826296016},{"lat":41.3628993375594,"lng":-8.76124958984089},{"lat":41.362904068593,"lng":-8.76132289836952},{"lat":41.3628072110643,"lng":-8.76143251281024},{"lat":41.3627410254553,"lng":-8.76149892605788},{"lat":41.3626499796676,"lng":-8.7615183634875},{"lat":41.3625995797949,"lng":-8.76153018558666},{"lat":41.3625363408408,"lng":-8.76156842390813},{"lat":41.3624872980312,"lng":-8.76162830370578},{"lat":41.3624888890707,"lng":-8.7616997916731},{"lat":41.362566207284,"lng":-8.76172109413059},{"lat":41.3626327980157,"lng":-8.76172866906194},{"lat":41.3626967792593,"lng":-8.76171936239933},{"lat":41.3627473106643,"lng":-8.7616835192725},{"lat":41.3628007386389,"lng":-8.76162774210563},{"lat":41.3628630223036,"lng":-8.76161603472559},{"lat":41.3629323564073,"lng":-8.76159949335466},{"lat":41.3629918553209,"lng":-8.76156982731726},{"lat":41.3630405952182,"lng":-8.76156528207704},{"lat":41.3631028282294,"lng":-8.7615462811887},{"lat":41.3631632708879,"lng":-8.76150873681962},{"lat":41.3632069182666,"lng":-8.76146385762462},{"lat":41.3632415477695,"lng":-8.76135579279022},{"lat":41.3632546250962,"lng":-8.76126927190057},{"lat":41.3632838896973,"lng":-8.76121935842133},{"lat":41.3633501411297,"lng":-8.76125584766753},{"lat":41.363412949055,"lng":-8.7612963709999},{"lat":41.3634430056554,"lng":-8.76139872892137},{"lat":41.3635545281014,"lng":-8.76143841342836},{"lat":41.363649142704,"lng":-8.76142605593742},{"lat":41.3637230002094,"lng":-8.76140489617798},{"lat":41.3637967822306,"lng":-8.76143154751904},{"lat":41.3638348909819,"lng":-8.76152620994294},{"lat":41.3638819640771,"lng":-8.76151364092198},{"lat":41.3639773202253,"lng":-8.76151288466676},{"lat":41.3640466405602,"lng":-8.76149885393765},{"lat":41.3640769376829,"lng":-8.76140831065569},{"lat":41.364024833918,"lng":-8.76137016077559},{"lat":41.3639937764726,"lng":-8.76128703331414},{"lat":41.3639210325975,"lng":-8.7612517967312},{"lat":41.36384490149,"lng":-8.7613427325724},{"lat":41.3637562698342,"lng":-8.76128235825217},{"lat":41.3636656630474,"lng":-8.7611875370944},{"lat":41.363577403633,"lng":-8.76122373852665},{"lat":41.3634974145862,"lng":-8.76118018437978},{"lat":41.3634950269375,"lng":-8.76110617650843},{"lat":41.3637768126245,"lng":-8.76108200570267},{"lat":41.3640123174371,"lng":-8.76118957114641},{"lat":41.36431893339,"lng":-8.76110288765306},{"lat":41.3645666390091,"lng":-8.7611194950387},{"lat":41.3648381560201,"lng":-8.76122764971292},{"lat":41.3652634161986,"lng":-8.76145979612403},{"lat":41.365457799989,"lng":-8.76162768503704},{"lat":41.3656426460821,"lng":-8.7618429354557},{"lat":41.3657697856259,"lng":-8.76210591729527},{"lat":41.3657917138965,"lng":-8.76221226146872},{"lat":41.3657825598375,"lng":-8.76253571946615},{"lat":41.3657402321683,"lng":-8.76281606571832},{"lat":41.3656424942585,"lng":-8.76284009921373},{"lat":41.365411516044,"lng":-8.76277810443412},{"lat":41.3657375053708,"lng":-8.76300129499709},{"lat":41.366185653021,"lng":-8.7632292563003},{"lat":41.3663001339402,"lng":-8.76333757249296},{"lat":41.3667495016472,"lng":-8.76360619085383},{"lat":41.3668869056698,"lng":-8.76363847780331},{"lat":41.3669816621911,"lng":-8.76360031037823},{"lat":41.366981619355,"lng":-8.76349357455294},{"lat":41.366690777337,"lng":-8.76342872589122},{"lat":41.3664740066875,"lng":-8.7632882221196},{"lat":41.3662878847258,"lng":-8.76305932770596},{"lat":41.366036246937,"lng":-8.7629221955753},{"lat":41.3660352758587,"lng":-8.76263963296883},{"lat":41.366068005415,"lng":-8.76251672547707},{"lat":41.3662942905127,"lng":-8.76198906801614},{"lat":41.3664253000024,"lng":-8.76180913629259},{"lat":41.3665533929964,"lng":-8.76175013636913},{"lat":41.3670372839016,"lng":-8.76174895016747},{"lat":41.3673707498878,"lng":-8.76169228595942},{"lat":41.3679095798954,"lng":-8.76173884348588},{"lat":41.3681970529951,"lng":-8.76197433570571},{"lat":41.3682636996437,"lng":-8.76208649902454},{"lat":41.3682619593906,"lng":-8.76228931754149},{"lat":41.3681110694354,"lng":-8.76246654986473},{"lat":41.3682962397183,"lng":-8.76250765894791},{"lat":41.3687376258799,"lng":-8.76421374657533},{"lat":41.369046099309,"lng":-8.76496892373189},{"lat":41.3694907187409,"lng":-8.76578734643011},{"lat":41.3695518940479,"lng":-8.76600966210796},{"lat":41.3698498800427,"lng":-8.76642744281273},{"lat":41.3702048338048,"lng":-8.76685020343129},{"lat":41.3706006533663,"lng":-8.7672107352104},{"lat":41.3710141945559,"lng":-8.76749171318894},{"lat":41.3716988888573,"lng":-8.76787980987722},{"lat":41.3720400975987,"lng":-8.76800303174934},{"lat":41.3721551753188,"lng":-8.76800259756077},{"lat":41.372170629923,"lng":-8.76784556136362},{"lat":41.3713215593786,"lng":-8.76741689350347},{"lat":41.3705228785071,"lng":-8.7667820579241},{"lat":41.3701449626666,"lng":-8.76637628462906},{"lat":41.3697417322312,"lng":-8.76587368913316},{"lat":41.3692966283368,"lng":-8.76511048771261},{"lat":41.369254846629,"lng":-8.76493042704557},{"lat":41.3692672596947,"lng":-8.76455630927158},{"lat":41.3692045267052,"lng":-8.76430564798624},{"lat":41.3692650052074,"lng":-8.76419674996228},{"lat":41.3693998874156,"lng":-8.76413123906896},{"lat":41.3694093761097,"lng":-8.76404348025901},{"lat":41.3693823163713,"lng":-8.76395381110153},{"lat":41.3694151799041,"lng":-8.76390476393626},{"lat":41.369504689405,"lng":-8.76401846196013},{"lat":41.369590774632,"lng":-8.76421412717401},{"lat":41.3696638485001,"lng":-8.76423766480936},{"lat":41.3698696152888,"lng":-8.76420129393656},{"lat":41.3699580087549,"lng":-8.76404198345931},{"lat":41.3699647674767,"lng":-8.7639428361204},{"lat":41.3699015211041,"lng":-8.76367089453329},{"lat":41.3701138308231,"lng":-8.76354159040214},{"lat":41.3701786457294,"lng":-8.76357747754622},{"lat":41.3704064014158,"lng":-8.76386512032308},{"lat":41.3705957955743,"lng":-8.76395695807256},{"lat":41.3705962413875,"lng":-8.76408916506387},{"lat":41.3706578971982,"lng":-8.76422375538032},{"lat":41.3709044755218,"lng":-8.76452760201383},{"lat":41.3710961306136,"lng":-8.76451930132226},{"lat":41.3711206683227,"lng":-8.76439594517839},{"lat":41.3712661045376,"lng":-8.76431069090163},{"lat":41.3715757292712,"lng":-8.76423444180437},{"lat":41.3717456088132,"lng":-8.76414178227336},{"lat":41.3719223547711,"lng":-8.76414277387157},{"lat":41.3720818615971,"lng":-8.76420074080579},{"lat":41.3723415212138,"lng":-8.76363129028099},{"lat":41.3724357171536,"lng":-8.76364570833542},{"lat":41.372544769806,"lng":-8.76377394810119},{"lat":41.3726912797627,"lng":-8.7637889933849},{"lat":41.3727985561905,"lng":-8.76371664498264},{"lat":41.3728425432877,"lng":-8.76364212042119},{"lat":41.3728573558195,"lng":-8.76351938119551},{"lat":41.372719830096,"lng":-8.76365587402031},{"lat":41.372535946242,"lng":-8.76362707476615},{"lat":41.3724263817672,"lng":-8.76349213749708},{"lat":41.3724299891623,"lng":-8.76340969455455},{"lat":41.3724264720681,"lng":-8.76332813780324},{"lat":41.3725851261132,"lng":-8.76291811809817},{"lat":41.3729201901577,"lng":-8.76222495440211},{"lat":41.3734855482517,"lng":-8.76226090289181},{"lat":41.3749659743002,"lng":-8.76269693973084},{"lat":41.3749806947292,"lng":-8.76293483978206},{"lat":41.3751902957405,"lng":-8.76359409325448},{"lat":41.3755099241224,"lng":-8.76381068073608},{"lat":41.3757481870051,"lng":-8.76372704014442},{"lat":41.3758769798059,"lng":-8.76378626842671},{"lat":41.3758663164743,"lng":-8.76398985904199},{"lat":41.3760988344503,"lng":-8.76411571584984},{"lat":41.3761932179114,"lng":-8.76425983970563},{"lat":41.3764921872709,"lng":-8.76489319745544},{"lat":41.3771255429858,"lng":-8.76623367388002},{"lat":41.3772620709895,"lng":-8.76652310106968},{"lat":41.3772432276647,"lng":-8.76691883798019},{"lat":41.3771481962992,"lng":-8.76700661558864},{"lat":41.377186515466,"lng":-8.76716072080555},{"lat":41.3771632574527,"lng":-8.76724679904205},{"lat":41.376833834744,"lng":-8.76738632626709},{"lat":41.3766241795597,"lng":-8.76739324798083},{"lat":41.3759598157475,"lng":-8.76642089427822},{"lat":41.3758666351934,"lng":-8.76653258925213},{"lat":41.3761656515282,"lng":-8.76700756911818},{"lat":41.3760877261342,"lng":-8.76725951482354},{"lat":41.3754271782207,"lng":-8.76748785402667},{"lat":41.3753583194973,"lng":-8.76732078378752},{"lat":41.37525819381,"lng":-8.76665170744554},{"lat":41.3752226400331,"lng":-8.76656804510248},{"lat":41.375101435181,"lng":-8.76660200780903},{"lat":41.375101372782,"lng":-8.76672728543382},{"lat":41.3752438548272,"lng":-8.76751990057359},{"lat":41.3745995608822,"lng":-8.76777229928908},{"lat":41.3745103574164,"lng":-8.76766946464392},{"lat":41.3740183623643,"lng":-8.76667562569506},{"lat":41.3739264044803,"lng":-8.76656571300941},{"lat":41.3737818856884,"lng":-8.76669687004331},{"lat":41.3737524361921,"lng":-8.76677894943552},{"lat":41.3739215305666,"lng":-8.76720434925499},{"lat":41.3740008973507,"lng":-8.76749033497785},{"lat":41.3740508234325,"lng":-8.76782468873166},{"lat":41.3740085011317,"lng":-8.76891540417333},{"lat":41.3739007547308,"lng":-8.76907201727819},{"lat":41.373700916884,"lng":-8.76924040127392},{"lat":41.3723969811609,"lng":-8.7700587786076},{"lat":41.3722068931671,"lng":-8.77023980180681},{"lat":41.3717845903925,"lng":-8.77053247464849},{"lat":41.3711497069615,"lng":-8.77085580377679},{"lat":41.3709262646815,"lng":-8.7710593243041},{"lat":41.370392312463,"lng":-8.77125394933867},{"lat":41.3703583525502,"lng":-8.77133764802595},{"lat":41.3703624029936,"lng":-8.77141908760985},{"lat":41.3704776103974,"lng":-8.77170206796893},{"lat":41.370615615306,"lng":-8.77182127701765},{"lat":41.3708730229891,"lng":-8.77177670624087},{"lat":41.3714087564924,"lng":-8.77113147281789},{"lat":41.371807513782,"lng":-8.77076626385632},{"lat":41.3728604289609,"lng":-8.7701400530655},{"lat":41.3737834003346,"lng":-8.76943521584258},{"lat":41.3740051417746,"lng":-8.76942459157833},{"lat":41.374174581361,"lng":-8.76931400309255},{"lat":41.3746151203514,"lng":-8.7691257329613},{"lat":41.3757867160474,"lng":-8.76877365754241},{"lat":41.376007319111,"lng":-8.76877414157154},{"lat":41.3785912925088,"lng":-8.76925423978797},{"lat":41.3792953969427,"lng":-8.76948598773156},{"lat":41.3797631266769,"lng":-8.76968781318491},{"lat":41.3800541947611,"lng":-8.7697591586579},{"lat":41.3801425714608,"lng":-8.76986462887704},{"lat":41.3808929532595,"lng":-8.77076176725105},{"lat":41.3810942315553,"lng":-8.77095035416672},{"lat":41.3812793370044,"lng":-8.77106765798212},{"lat":41.3813214191122,"lng":-8.77109413050076},{"lat":41.3817760112874,"lng":-8.77130995412264},{"lat":41.3824595513191,"lng":-8.77140165175988},{"lat":41.3830695467209,"lng":-8.77155385055107},{"lat":41.3844332107273,"lng":-8.77218819590106},{"lat":41.3846348126812,"lng":-8.7723670001639},{"lat":41.3847885951243,"lng":-8.77259459264028},{"lat":41.3853518464084,"lng":-8.77365365784177},{"lat":41.3853809900978,"lng":-8.77388421761246},{"lat":41.3854675835202,"lng":-8.77459834401022},{"lat":41.3861190945853,"lng":-8.77513560585896},{"lat":41.386702080717,"lng":-8.77507727362116},{"lat":41.387156645174,"lng":-8.77529661035847},{"lat":41.387383717792,"lng":-8.77545989601916},{"lat":41.3874595662346,"lng":-8.77546997130846},{"lat":41.3878654606162,"lng":-8.77534401157787},{"lat":41.3890055365882,"lng":-8.77531712672589},{"lat":41.3898428681848,"lng":-8.77505373789051},{"lat":41.3905267178226,"lng":-8.77504075410286},{"lat":41.3907543573488,"lng":-8.77508652186988},{"lat":41.3911898410682,"lng":-8.77511174424213},{"lat":41.3914351203625,"lng":-8.7751261172131},{"lat":41.3918661329172,"lng":-8.7753704672712},{"lat":41.3921121915436,"lng":-8.77558453130136},{"lat":41.3926975397609,"lng":-8.77658394024959},{"lat":41.3929242151764,"lng":-8.77680153480485},{"lat":41.3930013342507,"lng":-8.7768476294183},{"lat":41.393254179489,"lng":-8.77697264800248},{"lat":41.3937081769869,"lng":-8.77272315381631},{"lat":41.3935477516148,"lng":-8.77196026833932},{"lat":41.3914978455229,"lng":-8.77134901787018},{"lat":41.3898262653628,"lng":-8.77088159647023},{"lat":41.3885145156499,"lng":-8.76990755589558},{"lat":41.3878596202154,"lng":-8.76839543836315},{"lat":41.3886843846684,"lng":-8.76609723499869},{"lat":41.388771958321,"lng":-8.76566391264009},{"lat":41.3887660765109,"lng":-8.76538328744079},{"lat":41.3889829570042,"lng":-8.76464567540669},{"lat":41.3900248535031,"lng":-8.76222653998304},{"lat":41.3907058818502,"lng":-8.76100219783632},{"lat":41.3917019578554,"lng":-8.75994138987314},{"lat":41.3942177171109,"lng":-8.75764879534031},{"lat":41.3966505219968,"lng":-8.75485920563657},{"lat":41.3975295121639,"lng":-8.75323267119252},{"lat":41.3982948196329,"lng":-8.75122429815666},{"lat":41.3979751778048,"lng":-8.75099189649802},{"lat":41.3980741426484,"lng":-8.74999434893626},{"lat":41.3977007969936,"lng":-8.74871974231019},{"lat":41.3945021717392,"lng":-8.74563652218009},{"lat":41.3897480689829,"lng":-8.74112028762795},{"lat":41.3860799076198,"lng":-8.73715298279372},{"lat":41.3834857445398,"lng":-8.73332108070582},{"lat":41.3820889687419,"lng":-8.73151274838522},{"lat":41.3809430132192,"lng":-8.73042674200666},{"lat":41.3769113113583,"lng":-8.72764435977358},{"lat":41.3763111605616,"lng":-8.72827862165596},{"lat":41.3750856436027,"lng":-8.72853616244095},{"lat":41.374534962606,"lng":-8.7282906921502},{"lat":41.3740756482506,"lng":-8.72764257951594},{"lat":41.3736654144621,"lng":-8.72627083773776},{"lat":41.3732325689259,"lng":-8.72566922591909},{"lat":41.3719573389955,"lng":-8.72506714300571},{"lat":41.3708438625384,"lng":-8.7247398209607},{"lat":41.3681600384253,"lng":-8.72396879812804},{"lat":41.3664348796517,"lng":-8.72349658121242},{"lat":41.364987779075,"lng":-8.7228613277294},{"lat":41.3646323301667,"lng":-8.72327147388323},{"lat":41.3641444893716,"lng":-8.72425866410789},{"lat":41.3630609925034,"lng":-8.72484640145503},{"lat":41.3615334745678,"lng":-8.72423498708583},{"lat":41.3600028791235,"lng":-8.7242211089019},{"lat":41.3578340323536,"lng":-8.72575498493128},{"lat":41.356287363422,"lng":-8.728847932154},{"lat":41.3550219108187,"lng":-8.72979235383448},{"lat":41.3534043822353,"lng":-8.7291800786115},{"lat":41.3514229904142,"lng":-8.72928145968479},{"lat":41.3507884004523,"lng":-8.73011208963671},{"lat":41.3511342237127,"lng":-8.73286354298739},{"lat":41.3515631139768,"lng":-8.73693019963699},{"lat":41.3505494042138,"lng":-8.7413419323322},{"lat":41.3473829627786,"lng":-8.74418001808647},{"lat":41.3450401474946,"lng":-8.74451651019889},{"lat":41.3421564843002,"lng":-8.74496738209927},{"lat":41.3415141264579,"lng":-8.74723134744931},{"lat":41.3399105221681,"lng":-8.74776437802329},{"lat":41.3387179293054,"lng":-8.74816077306526},{"lat":41.3385265083726,"lng":-8.74842034977953},{"lat":41.3386182245895,"lng":-8.74926635685813},{"lat":41.3382513710462,"lng":-8.74925321562896},{"lat":41.3374849898125,"lng":-8.74929244238159},{"lat":41.3367186876258,"lng":-8.74963559839747},{"lat":41.3365975792196,"lng":-8.74965237747989},{"lat":41.3365812432897,"lng":-8.74986009371802},{"lat":41.3367528125081,"lng":-8.74987055574578},{"lat":41.3370402448671,"lng":-8.7496998103441},{"lat":41.337486653747,"lng":-8.74951957162801},{"lat":41.338402411281,"lng":-8.74951497301847},{"lat":41.339355166205,"lng":-8.74963019264372},{"lat":41.3395989156568,"lng":-8.74977968612335},{"lat":41.3396719600831,"lng":-8.74989363732586},{"lat":41.3396834414431,"lng":-8.74995181474651},{"lat":41.3396746992403,"lng":-8.75007120092508},{"lat":41.3397413156534,"lng":-8.75009011067777},{"lat":41.3398239974774,"lng":-8.75008659390551},{"lat":41.3398963870528,"lng":-8.75018752119557},{"lat":41.3399432100198,"lng":-8.75020444785685},{"lat":41.3399899442849,"lng":-8.75020417597181},{"lat":41.3400485874456,"lng":-8.75018250630019},{"lat":41.3400979228195,"lng":-8.75021690823827},{"lat":41.3401357333832,"lng":-8.75025179528503},{"lat":41.3402253234539,"lng":-8.7503194252078},{"lat":41.3403165868326,"lng":-8.75041085168629},{"lat":41.3403648768855,"lng":-8.7504899276862},{"lat":41.3404049922135,"lng":-8.75054813006002},{"lat":41.3404478793127,"lng":-8.75059334020511},{"lat":41.3405194385309,"lng":-8.75068111573731},{"lat":41.340569755141,"lng":-8.75073511556336},{"lat":41.3406289326582,"lng":-8.75078155519322},{"lat":41.3407246602635,"lng":-8.75086238999273},{"lat":41.3407871835928,"lng":-8.75095653644448},{"lat":41.3408374333677,"lng":-8.75098927229792},{"lat":41.3408795314009,"lng":-8.75104737402738},{"lat":41.3409334019117,"lng":-8.75124311120097},{"lat":41.3409328873495,"lng":-8.75133868328698},{"lat":41.3409190000076,"lng":-8.75140976025836},{"lat":41.340863678514,"lng":-8.75148354854109},{"lat":41.3407688752123,"lng":-8.75154788241794},{"lat":41.3407116774702,"lng":-8.75160194443498},{"lat":41.3406646845009,"lng":-8.75165024149234},{"lat":41.340616529295,"lng":-8.75171346878943},{"lat":41.3405911479333,"lng":-8.75177834086656},{"lat":41.3405782027941,"lng":-8.75184165522324},{"lat":41.3405865677022,"lng":-8.75194388714393},{"lat":41.340633586938,"lng":-8.75205783569763},{"lat":41.3406540951942,"lng":-8.75211107957599},{"lat":41.3407075032063,"lng":-8.75222616392321},{"lat":41.340779112716,"lng":-8.75230438172506},{"lat":41.340836661598,"lng":-8.75231962280942},{"lat":41.3408724537607,"lng":-8.75236106170188},{"lat":41.3408582468037,"lng":-8.75249127336333},{"lat":41.340869037162,"lng":-8.75264370738071},{"lat":41.3409335984108,"lng":-8.7527105159095},{"lat":41.3409946830281,"lng":-8.75272041662826},{"lat":41.3409836208521,"lng":-8.75278434995039},{"lat":41.3409832558862,"lng":-8.75285197010435},{"lat":41.3410230502417,"lng":-8.75298711512751},{"lat":41.3410235339027,"lng":-8.75304757310761},{"lat":41.3411031039003,"lng":-8.75316936369163},{"lat":41.3411324788922,"lng":-8.75324837621062},{"lat":41.341224749344,"lng":-8.75330289790285},{"lat":41.3413146481861,"lng":-8.7533297974704},{"lat":41.3413606932199,"lng":-8.75330622105975},{"lat":41.3414366560322,"lng":-8.75324624977893},{"lat":41.3415040508115,"lng":-8.75327186034873},{"lat":41.3415697105584,"lng":-8.753301039635},{"lat":41.3417665475553,"lng":-8.75338416344983},{"lat":41.3419731825697,"lng":-8.7534697644853},{"lat":41.342087900796,"lng":-8.75353597086053},{"lat":41.3422090424245,"lng":-8.75361275389676},{"lat":41.342312081555,"lng":-8.75370704389321},{"lat":41.3423746851857,"lng":-8.75378590054103},{"lat":41.3424326869546,"lng":-8.75388299345745},{"lat":41.3424798995723,"lng":-8.75399431797957},{"lat":41.342498399381,"lng":-8.75406988405176},{"lat":41.3424665998932,"lng":-8.75412203080098},{"lat":41.3424227492196,"lng":-8.75422173873228},{"lat":41.34235055878,"lng":-8.75425116123301},{"lat":41.3423626495343,"lng":-8.75434602096403},{"lat":41.3424058339534,"lng":-8.75438621716039}]},{"zone_id":"vcd8","points":[{"lat":41.2997711789279,"lng":-8.67304830868219},{"lat":41.2981102252528,"lng":-8.67234356066137},{"lat":41.295493524758,"lng":-8.67058126544242},{"lat":41.2941882063007,"lng":-8.66383472437267},{"lat":41.2933318465574,"lng":-8.66162871006596},{"lat":41.2868217634602,"lng":-8.6601935883973},{"lat":41.2861036678653,"lng":-8.65971029904598},{"lat":41.2855733023847,"lng":-8.65755735303952},{"lat":41.2851318445211,"lng":-8.65564389272575},{"lat":41.2856780423981,"lng":-8.65433519616096},{"lat":41.2871299991999,"lng":-8.65183997073172},{"lat":41.2865973330673,"lng":-8.65016452419293},{"lat":41.2854290122838,"lng":-8.64967777922671},{"lat":41.2847157047398,"lng":-8.64812033590472},{"lat":41.2842719505856,"lng":-8.64668441708768},{"lat":41.2819352753201,"lng":-8.64571110395779},{"lat":41.2813977237646,"lng":-8.64511006523761},{"lat":41.2807738579846,"lng":-8.64367282940963},{"lat":41.2796081625281,"lng":-8.6425894684515},{"lat":41.2803385164967,"lng":-8.64032732252234},{"lat":41.2813384102976,"lng":-8.63818657606839},{"lat":41.2791645035134,"lng":-8.63620399947648},{"lat":41.2790227425596,"lng":-8.63695290211475},{"lat":41.2784609458326,"lng":-8.64184226408683},{"lat":41.2759004374516,"lng":-8.65065450827348},{"lat":41.2749046413781,"lng":-8.65184014316546},{"lat":41.272918960859,"lng":-8.65289852183213},{"lat":41.2683238044333,"lng":-8.65357803140413},{"lat":41.2671735908766,"lng":-8.65373839002898},{"lat":41.2652312502821,"lng":-8.65317525334722},{"lat":41.2640771042822,"lng":-8.65388922355539},{"lat":41.2630906348341,"lng":-8.65282154633536},{"lat":41.2617423379303,"lng":-8.65130231250987},{"lat":41.2604260430186,"lng":-8.65062666413641},{"lat":41.2595278213435,"lng":-8.65291386557544},{"lat":41.258744232946,"lng":-8.6525939550607},{"lat":41.257558310614,"lng":-8.65593964901095},{"lat":41.2573900982786,"lng":-8.65637102757905},{"lat":41.2571677675868,"lng":-8.65650137330594},{"lat":41.256580388404,"lng":-8.65529708368746},{"lat":41.2560370531603,"lng":-8.65482132423756},{"lat":41.2548949930745,"lng":-8.65421836292547},{"lat":41.2545970904449,"lng":-8.65423490118786},{"lat":41.2544320782325,"lng":-8.65412865493957},{"lat":41.2543445522172,"lng":-8.65393045402095},{"lat":41.250061231686,"lng":-8.65009215052111},{"lat":41.2483541075042,"lng":-8.64927954057845},{"lat":41.2476075395584,"lng":-8.64907198335042},{"lat":41.2456319871716,"lng":-8.64876730173073},{"lat":41.2443629613316,"lng":-8.64853282104172},{"lat":41.2438950062294,"lng":-8.64880059288921},{"lat":41.2437357487935,"lng":-8.64904965536083},{"lat":41.2434065982136,"lng":-8.64909072407472},{"lat":41.2431902401402,"lng":-8.64897273801171},{"lat":41.2423815041305,"lng":-8.64876439271487},{"lat":41.241687594799,"lng":-8.64899677596336},{"lat":41.2411267579689,"lng":-8.64932932456459},{"lat":41.2401085847947,"lng":-8.6485391325314},{"lat":41.2394474326728,"lng":-8.64962331565954},{"lat":41.236252162947,"lng":-8.65319706294858},{"lat":41.2343474646539,"lng":-8.65353698966911},{"lat":41.2314583905856,"lng":-8.65305348212308},{"lat":41.2300254117585,"lng":-8.65371400841759},{"lat":41.2291314217971,"lng":-8.65422410067458},{"lat":41.2282734859569,"lng":-8.65453723000805},{"lat":41.2282903121318,"lng":-8.65534811306683},{"lat":41.2280270675281,"lng":-8.6559687468274},{"lat":41.2267337041273,"lng":-8.6574995780653},{"lat":41.2257297171842,"lng":-8.66051627003189},{"lat":41.2248975453497,"lng":-8.66082641229288},{"lat":41.2229099746521,"lng":-8.66065533312798},{"lat":41.2224017179571,"lng":-8.66185329869589},{"lat":41.2230430607429,"lng":-8.66301271442247},{"lat":41.2224633047771,"lng":-8.66428814970679},{"lat":41.2227047131627,"lng":-8.66555928469945},{"lat":41.2224141059198,"lng":-8.66615516893182},{"lat":41.2226317748646,"lng":-8.66702273771205},{"lat":41.2231935414476,"lng":-8.66834614900684},{"lat":41.2238887077696,"lng":-8.66872640668575},{"lat":41.2259570883627,"lng":-8.67103017488194},{"lat":41.2305738851475,"lng":-8.68052952068546},{"lat":41.2317695402543,"lng":-8.68571197967668},{"lat":41.2321262438731,"lng":-8.69337088458373},{"lat":41.2323392749301,"lng":-8.69550253780104},{"lat":41.232529811234,"lng":-8.69680506796807},{"lat":41.2329500384464,"lng":-8.69721951937033},{"lat":41.2336201072017,"lng":-8.69687593735683},{"lat":41.2343325930763,"lng":-8.69615792833876},{"lat":41.2350552749517,"lng":-8.69605556820955},{"lat":41.2372372855627,"lng":-8.69632232935868},{"lat":41.2381778072448,"lng":-8.69628272298335},{"lat":41.2385980350808,"lng":-8.69622357641352},{"lat":41.241211834275,"lng":-8.69523766832658},{"lat":41.2438885668989,"lng":-8.69477409860546},{"lat":41.2467038853153,"lng":-8.69532036704913},{"lat":41.2501278542641,"lng":-8.69736861934156},{"lat":41.2532566882522,"lng":-8.70026951684232},{"lat":41.256563304367,"lng":-8.70186942527064},{"lat":41.2584503813702,"lng":-8.70247081204899},{"lat":41.2596060046108,"lng":-8.70334568090795},{"lat":41.2600736618944,"lng":-8.70332440955585},{"lat":41.2609545276462,"lng":-8.70284707334607},{"lat":41.2629426013592,"lng":-8.70176577241272},{"lat":41.2644355120669,"lng":-8.70186141433356},{"lat":41.267498908216,"lng":-8.70223022374388},{"lat":41.2689384969202,"lng":-8.70244186716038},{"lat":41.2700871822963,"lng":-8.70254697453498},{"lat":41.2704624058112,"lng":-8.70253900070172},{"lat":41.2720505788493,"lng":-8.70236739004768},{"lat":41.2752077546158,"lng":-8.70120142165723},{"lat":41.2782398823016,"lng":-8.69972106772148},{"lat":41.2800250241131,"lng":-8.69946240539884},{"lat":41.2821694652382,"lng":-8.69992304024636},{"lat":41.2843976302481,"lng":-8.70198180328046},{"lat":41.2857254618174,"lng":-8.70232797536657},{"lat":41.2871193322764,"lng":-8.70345390943664},{"lat":41.2873837404274,"lng":-8.70497083756817},{"lat":41.2888183545477,"lng":-8.70590402358679},{"lat":41.289756913783,"lng":-8.70548395155211},{"lat":41.2911743688634,"lng":-8.70626238280033},{"lat":41.2933144317462,"lng":-8.70736496561945},{"lat":41.2956602514745,"lng":-8.70769672979681},{"lat":41.2965655579339,"lng":-8.70768562944843},{"lat":41.2979037890066,"lng":-8.70830163373427},{"lat":41.2992707328297,"lng":-8.70988811262374},{"lat":41.3001692641531,"lng":-8.70556796391459},{"lat":41.301273154141,"lng":-8.70408997270375},{"lat":41.302178342965,"lng":-8.70292445908969},{"lat":41.3027309858801,"lng":-8.70203673490695},{"lat":41.3034161222834,"lng":-8.70023067350728},{"lat":41.3030865465659,"lng":-8.69889270613165},{"lat":41.3040750649156,"lng":-8.6977598100356},{"lat":41.3063945035385,"lng":-8.69734182196011},{"lat":41.3063684441344,"lng":-8.69644656712644},{"lat":41.3036787517775,"lng":-8.69452193295891},{"lat":41.302478382545,"lng":-8.69349407414621},{"lat":41.301590802911,"lng":-8.69196428917485},{"lat":41.3010394701163,"lng":-8.69037119033135},{"lat":41.3001648018387,"lng":-8.68912963512745},{"lat":41.2993397568517,"lng":-8.68786410172418},{"lat":41.2988823153388,"lng":-8.68697631941396},{"lat":41.2988336345977,"lng":-8.68621998622059},{"lat":41.2989554696767,"lng":-8.6856949415597},{"lat":41.2993336934435,"lng":-8.68539088839966},{"lat":41.2996833595733,"lng":-8.68464371117124},{"lat":41.2999920131044,"lng":-8.68359694227361},{"lat":41.2998730515981,"lng":-8.6825881591857},{"lat":41.2992891645087,"lng":-8.68095013549028},{"lat":41.2988120624751,"lng":-8.67992530622421},{"lat":41.2989403630756,"lng":-8.67862815318874},{"lat":41.2994088897118,"lng":-8.67796887054463},{"lat":41.2996307292126,"lng":-8.67699615622634},{"lat":41.3002582177665,"lng":-8.67562553061599},{"lat":41.3006706232219,"lng":-8.67460244072456},{"lat":41.3011221590224,"lng":-8.67411076702621},{"lat":41.3008234160226,"lng":-8.67356664464292},{"lat":41.2997711789279,"lng":-8.67304830868219}]},{"zone_id":"vcd9","points":[{"lat":41.2703409611146,"lng":-8.72862453397451},{"lat":41.2703392067987,"lng":-8.7288586730381},{"lat":41.271052751401,"lng":-8.72929696595392},{"lat":41.2711958242222,"lng":-8.72933180980244},{"lat":41.2713012718269,"lng":-8.72929494004173},{"lat":41.2715012787301,"lng":-8.72927206329871},{"lat":41.2717181930551,"lng":-8.72928824257026},{"lat":41.2719282529827,"lng":-8.72942776713456},{"lat":41.2721394368321,"lng":-8.7295069081074},{"lat":41.2722945919354,"lng":-8.72958960100298},{"lat":41.272367960528,"lng":-8.72969696848096},{"lat":41.2725291638642,"lng":-8.72986385903526},{"lat":41.2727227571837,"lng":-8.72988006686789},{"lat":41.2730010529464,"lng":-8.72986578507513},{"lat":41.2732062859671,"lng":-8.72984307051906},{"lat":41.2734165195795,"lng":-8.72979331248503},{"lat":41.2737012313717,"lng":-8.72973981990717},{"lat":41.2738743546754,"lng":-8.7296754035868},{"lat":41.2742120231457,"lng":-8.7296546195445},{"lat":41.2743031776108,"lng":-8.72964781570894},{"lat":41.2744145289427,"lng":-8.7296707878338},{"lat":41.2745656121279,"lng":-8.72972360893804},{"lat":41.2747272213682,"lng":-8.72996832551477},{"lat":41.2747544649204,"lng":-8.73009723208562},{"lat":41.2747428884454,"lng":-8.73026231042554},{"lat":41.2747403330297,"lng":-8.73042460290493},{"lat":41.2747582425923,"lng":-8.73061644747234},{"lat":41.2748240793834,"lng":-8.73076886198593},{"lat":41.2749226410916,"lng":-8.73086118644994},{"lat":41.2750586885193,"lng":-8.73084500564039},{"lat":41.2751570700679,"lng":-8.73079912146117},{"lat":41.2752163484483,"lng":-8.73065453213492},{"lat":41.2752965033006,"lng":-8.73044389529374},{"lat":41.2753969104424,"lng":-8.73018176237176},{"lat":41.2754727812046,"lng":-8.72998014787984},{"lat":41.2755455028259,"lng":-8.72986539987257},{"lat":41.2756847458587,"lng":-8.72975328758367},{"lat":41.2758786386345,"lng":-8.7296946681876},{"lat":41.2760868115552,"lng":-8.72965992890132},{"lat":41.2763416271173,"lng":-8.72962477976621},{"lat":41.27649006129,"lng":-8.72965048690514},{"lat":41.2766103602077,"lng":-8.7297184191242},{"lat":41.2767654978418,"lng":-8.72984038749914},{"lat":41.2768956721858,"lng":-8.72991426235745},{"lat":41.277152961338,"lng":-8.72995707756922},{"lat":41.2773402809647,"lng":-8.72996762004206},{"lat":41.2774929663233,"lng":-8.73004134642617},{"lat":41.2776040052764,"lng":-8.7301243568573},{"lat":41.2777820526134,"lng":-8.73018590420712},{"lat":41.2779099025777,"lng":-8.73016988537015},{"lat":41.2780759895503,"lng":-8.73009048224636},{"lat":41.2782792787104,"lng":-8.72995674771258},{"lat":41.2784303139413,"lng":-8.72986228541275},{"lat":41.2786352040411,"lng":-8.72974969306876},{"lat":41.2789207698491,"lng":-8.72970527545453},{"lat":41.2791186661898,"lng":-8.72974265299834},{"lat":41.2792803538863,"lng":-8.72981622299116},{"lat":41.2794326674146,"lng":-8.7299441401602},{"lat":41.2795082989226,"lng":-8.73011754477752},{"lat":41.279467185596,"lng":-8.73023186894193},{"lat":41.2793846045594,"lng":-8.7303557314281},{"lat":41.279318566142,"lng":-8.73046720440086},{"lat":41.2792931028357,"lng":-8.73065364289331},{"lat":41.2794681622716,"lng":-8.73073617600332},{"lat":41.2796255494604,"lng":-8.73077079780923},{"lat":41.2797327612828,"lng":-8.73077607531445},{"lat":41.2799705961086,"lng":-8.7307679871654},{"lat":41.2801505006972,"lng":-8.73080245668294},{"lat":41.2802871764483,"lng":-8.73097546879832},{"lat":41.2803893772206,"lng":-8.7310617518617},{"lat":41.2805261421527,"lng":-8.73111456861888},{"lat":41.2806536645239,"lng":-8.73116145242206},{"lat":41.2808196694865,"lng":-8.73127075202573},{"lat":41.2810458047676,"lng":-8.73122722702838},{"lat":41.2812461438689,"lng":-8.73133052663921},{"lat":41.2812873476329,"lng":-8.73152820747674},{"lat":41.2814078131227,"lng":-8.73158135688785},{"lat":41.281334631863,"lng":-8.73180114732605},{"lat":41.2814236316882,"lng":-8.73184541064807},{"lat":41.2814968664615,"lng":-8.73176981070517},{"lat":41.2815716927323,"lng":-8.73192507447662},{"lat":41.2816542276067,"lng":-8.73181040772029},{"lat":41.2817611996753,"lng":-8.73201669005251},{"lat":41.2818888467273,"lng":-8.73203970576101},{"lat":41.2820208174925,"lng":-8.73197550355889},{"lat":41.2822442817842,"lng":-8.73189244922149},{"lat":41.282399655786,"lng":-8.73200225659543},{"lat":41.2823466099184,"lng":-8.73212579321427},{"lat":41.2822597716413,"lng":-8.73221952994856},{"lat":41.2823510261583,"lng":-8.73231478213512},{"lat":41.2824379419167,"lng":-8.73225721057013},{"lat":41.2825362738308,"lng":-8.73218720976407},{"lat":41.2826293731655,"lng":-8.73213542402505},{"lat":41.2827230367493,"lng":-8.73213162764133},{"lat":41.2828552020185,"lng":-8.73215170346062},{"lat":41.2830442011986,"lng":-8.73216799832674},{"lat":41.2832909647058,"lng":-8.73217491882265},{"lat":41.2835662677438,"lng":-8.73219964876954},{"lat":41.2838926983373,"lng":-8.73229610876174},{"lat":41.2841065372774,"lng":-8.73236634785914},{"lat":41.284407429063,"lng":-8.73250781707585},{"lat":41.284662763936,"lng":-8.73258071242063},{"lat":41.2849975031681,"lng":-8.73263809592471},{"lat":41.2852011116957,"lng":-8.73261525926905},{"lat":41.2854560063356,"lng":-8.73256519161903},{"lat":41.285675094596,"lng":-8.73252733477902},{"lat":41.2859214100349,"lng":-8.7324473506979},{"lat":41.2860583260052,"lng":-8.73241912439453},{"lat":41.2862456447871,"lng":-8.73242955507218},{"lat":41.2863703310527,"lng":-8.73248561147931},{"lat":41.2864682582317,"lng":-8.73252674021463},{"lat":41.2876354272364,"lng":-8.73282324558643},{"lat":41.287836844065,"lng":-8.73268650121799},{"lat":41.2879278189005,"lng":-8.73271383208927},{"lat":41.2880164169541,"lng":-8.73278293384761},{"lat":41.2881827116952,"lng":-8.7328362720952},{"lat":41.2883169142159,"lng":-8.73282854655561},{"lat":41.2884680708277,"lng":-8.73286622985193},{"lat":41.2886108929615,"lng":-8.73294871824711},{"lat":41.2889176476557,"lng":-8.73315817530226},{"lat":41.2891815393202,"lng":-8.73333298572388},{"lat":41.2894545612335,"lng":-8.73346681122298},{"lat":41.2897827655213,"lng":-8.73355076554433},{"lat":41.2901244637902,"lng":-8.73362064415979},{"lat":41.290532200249,"lng":-8.7336140100895},{"lat":41.290908556218,"lng":-8.73364970635897},{"lat":41.2911704800677,"lng":-8.73380528077842},{"lat":41.2913220873407,"lng":-8.73392975503031},{"lat":41.2915041154327,"lng":-8.73409068686052},{"lat":41.2919816895304,"lng":-8.73434770288113},{"lat":41.2924996851309,"lng":-8.73462299216518},{"lat":41.2925776286604,"lng":-8.7347319950922},{"lat":41.2926790798737,"lng":-8.73494316443893},{"lat":41.2926389736268,"lng":-8.73515636735689},{"lat":41.2925972985514,"lng":-8.73527334082978},{"lat":41.2927196240976,"lng":-8.73541856608286},{"lat":41.292823860692,"lng":-8.73545856629604},{"lat":41.2930035349931,"lng":-8.73553531512417},{"lat":41.2934290910274,"lng":-8.73577061213813},{"lat":41.2937776301748,"lng":-8.7359599506008},{"lat":41.2943507881448,"lng":-8.73617250108457},{"lat":41.2949060795877,"lng":-8.73640912793354},{"lat":41.2951544860909,"lng":-8.73658274452357},{"lat":41.2954760277707,"lng":-8.73688932468326},{"lat":41.2958446319911,"lng":-8.73716625558071},{"lat":41.2960885698836,"lng":-8.73721352624985},{"lat":41.2964056428257,"lng":-8.73717120945353},{"lat":41.2966841986637,"lng":-8.73712567269492},{"lat":41.2967979310246,"lng":-8.73720791850187},{"lat":41.2970150962031,"lng":-8.7375525779248},{"lat":41.297132143512,"lng":-8.73770684008598},{"lat":41.297305529018,"lng":-8.73779859294518},{"lat":41.2975605427527,"lng":-8.73789706646317},{"lat":41.2978930074728,"lng":-8.73802646080956},{"lat":41.2982343403458,"lng":-8.73816490025717},{"lat":41.2986345238718,"lng":-8.73825863172945},{"lat":41.2990845554856,"lng":-8.7382899126598},{"lat":41.2996115812664,"lng":-8.73823450190751},{"lat":41.3001358797917,"lng":-8.73821811565643},{"lat":41.3004031977871,"lng":-8.73820256687952},{"lat":41.3007544967345,"lng":-8.73815794666037},{"lat":41.3013019354813,"lng":-8.73819298810881},{"lat":41.3014954175428,"lng":-8.73821281688529},{"lat":41.3017091101653,"lng":-8.73829276744432},{"lat":41.3018632767417,"lng":-8.73845968374459},{"lat":41.3020225041395,"lng":-8.73862354438648},{"lat":41.3022286805622,"lng":-8.7387967966254},{"lat":41.302454549086,"lng":-8.73885585374251},{"lat":41.3026283589438,"lng":-8.73881543341508},{"lat":41.3028251999964,"lng":-8.73872699288269},{"lat":41.3029965183789,"lng":-8.73869837948941},{"lat":41.3033519709328,"lng":-8.73871982711338},{"lat":41.3035821660592,"lng":-8.73876088893218},{"lat":41.303864561197,"lng":-8.73882356767136},{"lat":41.3041169205722,"lng":-8.73894627140158},{"lat":41.3044509887624,"lng":-8.73909660379825},{"lat":41.3048811991324,"lng":-8.73927214656949},{"lat":41.3053902717472,"lng":-8.73956419819331},{"lat":41.3055859907209,"lng":-8.73908393919838},{"lat":41.3056699780784,"lng":-8.73819035334109},{"lat":41.3055647726318,"lng":-8.73433084857173},{"lat":41.3058257165635,"lng":-8.73398697090801},{"lat":41.3061084227159,"lng":-8.73330185176799},{"lat":41.3061376006448,"lng":-8.73142983011214},{"lat":41.306278785919,"lng":-8.72721356490086},{"lat":41.3062171386977,"lng":-8.725894636952},{"lat":41.3061756206944,"lng":-8.72501419888233},{"lat":41.3058864778977,"lng":-8.72319946585067},{"lat":41.3055001926063,"lng":-8.72077534788809},{"lat":41.3051428994178,"lng":-8.71853348002236},{"lat":41.3048705467836,"lng":-8.716824780081},{"lat":41.3043949980675,"lng":-8.71522143703518},{"lat":41.3040697961526,"lng":-8.71474865219419},{"lat":41.3025547795812,"lng":-8.7128315720339},{"lat":41.2992707328297,"lng":-8.70988811262374},{"lat":41.2979037890066,"lng":-8.70830163373427},{"lat":41.2965655579339,"lng":-8.70768562944843},{"lat":41.2956602514745,"lng":-8.70769672979681},{"lat":41.2933144317462,"lng":-8.70736496561945},{"lat":41.2911743688634,"lng":-8.70626238280033},{"lat":41.289756913783,"lng":-8.70548395155211},{"lat":41.2888183545477,"lng":-8.70590402358679},{"lat":41.2873837404274,"lng":-8.70497083756817},{"lat":41.2871193322764,"lng":-8.70345390943664},{"lat":41.2857254618174,"lng":-8.70232797536657},{"lat":41.2843976302481,"lng":-8.70198180328046},{"lat":41.2821694652382,"lng":-8.69992304024636},{"lat":41.2800250241131,"lng":-8.69946240539884},{"lat":41.2782398823016,"lng":-8.69972106772148},{"lat":41.2752077546158,"lng":-8.70120142165723},{"lat":41.2720505788493,"lng":-8.70236739004768},{"lat":41.2704624058112,"lng":-8.70253900070172},{"lat":41.270630450848,"lng":-8.70273237490359},{"lat":41.2707026104676,"lng":-8.70283384404311},{"lat":41.2708429072341,"lng":-8.70303115309702},{"lat":41.2710723950702,"lng":-8.70334393271346},{"lat":41.2712508308862,"lng":-8.70352939908165},{"lat":41.2713780021218,"lng":-8.70363111747592},{"lat":41.2715000538695,"lng":-8.70369388468513},{"lat":41.2715947864572,"lng":-8.70372896446428},{"lat":41.271694532792,"lng":-8.70378688208926},{"lat":41.2718079222896,"lng":-8.70387129564243},{"lat":41.2718989793724,"lng":-8.70393832773136},{"lat":41.2719831817188,"lng":-8.704007687239},{"lat":41.2721331014524,"lng":-8.70402499074415},{"lat":41.2722100644914,"lng":-8.70415587464507},{"lat":41.2722679157886,"lng":-8.70430950112142},{"lat":41.2723376661269,"lng":-8.70449736399913},{"lat":41.2724297142289,"lng":-8.70472671748595},{"lat":41.2724912605794,"lng":-8.70482548529641},{"lat":41.2725633288674,"lng":-8.70490834524152},{"lat":41.2726379344484,"lng":-8.7049881268342},{"lat":41.2727072976506,"lng":-8.70507191713076},{"lat":41.2727882154643,"lng":-8.70516810022737},{"lat":41.272923566757,"lng":-8.70530796924074},{"lat":41.2730253266764,"lng":-8.70543214577838},{"lat":41.2730852413518,"lng":-8.70553280691204},{"lat":41.2731147010559,"lng":-8.70569179680732},{"lat":41.273088066181,"lng":-8.70590424014039},{"lat":41.2729859132904,"lng":-8.70614812669876},{"lat":41.2728540608108,"lng":-8.7062811172445},{"lat":41.2727519098284,"lng":-8.70645196321864},{"lat":41.2726557517598,"lng":-8.70668516234434},{"lat":41.2725406431789,"lng":-8.7068552910094},{"lat":41.2723837467173,"lng":-8.70713437845681},{"lat":41.2722199951275,"lng":-8.70741591079714},{"lat":41.2723348887398,"lng":-8.7075953444251},{"lat":41.2724330200077,"lng":-8.70774121295787},{"lat":41.2724919598765,"lng":-8.70792851130492},{"lat":41.2725515861382,"lng":-8.7081587779051},{"lat":41.2727712173982,"lng":-8.70862019635054},{"lat":41.2727917634702,"lng":-8.70886826700813},{"lat":41.2725873350225,"lng":-8.70912640368921},{"lat":41.2724834210961,"lng":-8.70927192477391},{"lat":41.2723180084694,"lng":-8.70952431543971},{"lat":41.2722330323106,"lng":-8.70964577429073},{"lat":41.2721056837618,"lng":-8.70979574209546},{"lat":41.2720157915473,"lng":-8.70996310693348},{"lat":41.2719400834658,"lng":-8.71015649221543},{"lat":41.2718670681313,"lng":-8.71033426746989},{"lat":41.2716900343167,"lng":-8.71037316308545},{"lat":41.2714879136616,"lng":-8.71026109915857},{"lat":41.2713507847553,"lng":-8.710225755968},{"lat":41.2712032581223,"lng":-8.71023292791283},{"lat":41.271067768734,"lng":-8.71030070701579},{"lat":41.2710288415239,"lng":-8.71044966688618},{"lat":41.2710913526999,"lng":-8.71057051945303},{"lat":41.2711854414012,"lng":-8.71071396589499},{"lat":41.2712690438168,"lng":-8.71083083141015},{"lat":41.2713814542683,"lng":-8.71089399951686},{"lat":41.2715011828458,"lng":-8.71095258501681},{"lat":41.2715805065992,"lng":-8.71102453809096},{"lat":41.2715216466421,"lng":-8.71121604179808},{"lat":41.2715362718071,"lng":-8.71138695592061},{"lat":41.2716223740152,"lng":-8.711471733528},{"lat":41.2717838365941,"lng":-8.71136002586845},{"lat":41.2719015613352,"lng":-8.71142384260606},{"lat":41.271893820019,"lng":-8.71160100149223},{"lat":41.2718104009061,"lng":-8.71184217941263},{"lat":41.2716541928924,"lng":-8.71207113337827},{"lat":41.2715357972016,"lng":-8.71226593046576},{"lat":41.2715229870143,"lng":-8.71243146682266},{"lat":41.2715356165753,"lng":-8.71262265011551},{"lat":41.2715100864442,"lng":-8.7127572824501},{"lat":41.2714273590259,"lng":-8.71296755406209},{"lat":41.2713241708356,"lng":-8.71314553032744},{"lat":41.2711844504389,"lng":-8.71330158929671},{"lat":41.2711071067594,"lng":-8.71344411600825},{"lat":41.2710430595953,"lng":-8.71359176910719},{"lat":41.2709457563168,"lng":-8.71369246551522},{"lat":41.2707424068062,"lng":-8.71368278117155},{"lat":41.2706502238385,"lng":-8.71369712011541},{"lat":41.2705219709893,"lng":-8.71377522523355},{"lat":41.2703439533548,"lng":-8.7138830772299},{"lat":41.2702097031239,"lng":-8.71397198640834},{"lat":41.2700908564695,"lng":-8.71407726256728},{"lat":41.2699461782499,"lng":-8.71421632585987},{"lat":41.2698328528208,"lng":-8.7143344222731},{"lat":41.2697938858112,"lng":-8.71449005146329},{"lat":41.2698009046055,"lng":-8.71463284379464},{"lat":41.2698703796268,"lng":-8.71472893997348},{"lat":41.2699598919569,"lng":-8.7147798599959},{"lat":41.2701059924848,"lng":-8.71480479525509},{"lat":41.2702322239145,"lng":-8.71484088360999},{"lat":41.2703354335218,"lng":-8.71490708361572},{"lat":41.2703964546272,"lng":-8.71512531228589},{"lat":41.270428520013,"lng":-8.71537215715168},{"lat":41.2704613420579,"lng":-8.71555814914867},{"lat":41.2704828105943,"lng":-8.71572756962088},{"lat":41.2704713532597,"lng":-8.71589132687694},{"lat":41.2704607800717,"lng":-8.71605807317089},{"lat":41.2704563562891,"lng":-8.71627320873174},{"lat":41.2704444694995,"lng":-8.71650415186126},{"lat":41.2704461742824,"lng":-8.71668246440918},{"lat":41.2704794882854,"lng":-8.71685927194437},{"lat":41.2704721940858,"lng":-8.7170181717575},{"lat":41.2704512125875,"lng":-8.71716071747364},{"lat":41.2703645745091,"lng":-8.71725303286173},{"lat":41.2702744283408,"lng":-8.7173091506153},{"lat":41.270168867514,"lng":-8.71726225849748},{"lat":41.270085220877,"lng":-8.71713871146767},{"lat":41.2700137745736,"lng":-8.71697063287711},{"lat":41.269933423648,"lng":-8.71685391130267},{"lat":41.2698207258225,"lng":-8.71684682211643},{"lat":41.269727988618,"lng":-8.71695244558315},{"lat":41.2694991935776,"lng":-8.71709420478006},{"lat":41.2693487245648,"lng":-8.71720313299866},{"lat":41.2691609711453,"lng":-8.71734788550404},{"lat":41.2689888157953,"lng":-8.7174891960131},{"lat":41.2688148974002,"lng":-8.71762320663551},{"lat":41.2689148483497,"lng":-8.71781623734032},{"lat":41.2690687764915,"lng":-8.71794805524953},{"lat":41.269156365672,"lng":-8.71805803989796},{"lat":41.2692271504542,"lng":-8.7182319533666},{"lat":41.26930877382,"lng":-8.71836360454734},{"lat":41.2694393562263,"lng":-8.71855333239238},{"lat":41.2695381913462,"lng":-8.71871724266114},{"lat":41.2695901293446,"lng":-8.71889683720728},{"lat":41.2696039819843,"lng":-8.71909363494397},{"lat":41.2696104191318,"lng":-8.71922628154932},{"lat":41.269585802397,"lng":-8.71935578734129},{"lat":41.2695706962822,"lng":-8.71949313423945},{"lat":41.2696981538124,"lng":-8.7196767482295},{"lat":41.269866697313,"lng":-8.71976789101019},{"lat":41.2700289816678,"lng":-8.71983152793078},{"lat":41.2701422204823,"lng":-8.71989125787565},{"lat":41.2702705055709,"lng":-8.71998359278483},{"lat":41.2703889546179,"lng":-8.72009779339103},{"lat":41.2704589686716,"lng":-8.7201927004428},{"lat":41.2705221876497,"lng":-8.72034913231438},{"lat":41.2705493793428,"lng":-8.72049032225762},{"lat":41.2705435114417,"lng":-8.72063288197932},{"lat":41.2705115146701,"lng":-8.72078082253441},{"lat":41.2704745364421,"lng":-8.72098540222931},{"lat":41.2704470102416,"lng":-8.72115616904781},{"lat":41.2704262043506,"lng":-8.72135075278564},{"lat":41.2704415644583,"lng":-8.72153348202234},{"lat":41.2704305325501,"lng":-8.72170082349145},{"lat":41.2703860093211,"lng":-8.72188421540771},{"lat":41.2703445293537,"lng":-8.72207180193407},{"lat":41.2702884159764,"lng":-8.72226845214996},{"lat":41.2702314579794,"lng":-8.72238465519979},{"lat":41.2701490061622,"lng":-8.72253678645019},{"lat":41.2701123495961,"lng":-8.72271451927196},{"lat":41.2700631830705,"lng":-8.72290741056162},{"lat":41.2700909369762,"lng":-8.72304335091081},{"lat":41.2701422025036,"lng":-8.72324777268023},{"lat":41.2701976619833,"lng":-8.72342406254266},{"lat":41.2703201869883,"lng":-8.72363700379739},{"lat":41.2703864815893,"lng":-8.72380695110953},{"lat":41.2704628873698,"lng":-8.72405921842327},{"lat":41.2705323891139,"lng":-8.7242716822163},{"lat":41.2706046272508,"lng":-8.72447617504221},{"lat":41.2706380916239,"lng":-8.72469165607521},{"lat":41.2706681487224,"lng":-8.72493860768613},{"lat":41.2706754581643,"lng":-8.72505790062101},{"lat":41.2706862601168,"lng":-8.72528522528456},{"lat":41.2706886606001,"lng":-8.72555317433134},{"lat":41.270669201635,"lng":-8.7257639992851},{"lat":41.2706215660943,"lng":-8.72595654452891},{"lat":41.2705487541752,"lng":-8.72610792518264},{"lat":41.2704400711705,"lng":-8.72630075502702},{"lat":41.2703873497194,"lng":-8.72646664290183},{"lat":41.2703823160142,"lng":-8.72662078688411},{"lat":41.2704002544418,"lng":-8.72675675988562},{"lat":41.2704370285307,"lng":-8.72687142517563},{"lat":41.2705273781557,"lng":-8.72698502940651},{"lat":41.2706484733194,"lng":-8.7270734849834},{"lat":41.2707302662051,"lng":-8.72713628940171},{"lat":41.2707536530736,"lng":-8.7272972605884},{"lat":41.2707163003372,"lng":-8.72752068807707},{"lat":41.2707245474577,"lng":-8.72770216155268},{"lat":41.2707441670798,"lng":-8.72785999933964},{"lat":41.270724491681,"lng":-8.72800804148515},{"lat":41.2706663523251,"lng":-8.72815884202329},{"lat":41.2703409611146,"lng":-8.72862453397451}]},{"zone_id":"mts3","points":[{"lat":41.2319683079194,"lng":-8.72259229187527},{"lat":41.2320845635845,"lng":-8.7227069387696},{"lat":41.2322056476776,"lng":-8.72278079045959},{"lat":41.2322283328624,"lng":-8.72290456333664},{"lat":41.232326658499,"lng":-8.72306193627631},{"lat":41.2324215139563,"lng":-8.72315916722202},{"lat":41.2325240609185,"lng":-8.72302089461642},{"lat":41.2326505115162,"lng":-8.72311948336307},{"lat":41.2327524109453,"lng":-8.72326484648061},{"lat":41.2327642001019,"lng":-8.7233888755997},{"lat":41.2326668372131,"lng":-8.72344656186817},{"lat":41.232623487466,"lng":-8.72361160866286},{"lat":41.2326131512654,"lng":-8.72387093578685},{"lat":41.2327344222209,"lng":-8.72392594510472},{"lat":41.2327651061729,"lng":-8.7237545842628},{"lat":41.2328259904925,"lng":-8.72362965252299},{"lat":41.2329568589708,"lng":-8.72356881602271},{"lat":41.2330755555348,"lng":-8.72379257361477},{"lat":41.2331721886979,"lng":-8.72392858652158},{"lat":41.2333395951976,"lng":-8.72392449257098},{"lat":41.2334819067611,"lng":-8.72389762564824},{"lat":41.233602778466,"lng":-8.7238553016311},{"lat":41.2337074243581,"lng":-8.72381653019701},{"lat":41.2338933624095,"lng":-8.72376346136556},{"lat":41.2340908296332,"lng":-8.72367471245917},{"lat":41.2343159498007,"lng":-8.72367090109611},{"lat":41.2344957016402,"lng":-8.72373550312677},{"lat":41.2346261381871,"lng":-8.72375958246695},{"lat":41.2349870478922,"lng":-8.72396394595651},{"lat":41.2350776531853,"lng":-8.7240292972663},{"lat":41.2352145738088,"lng":-8.72399999455158},{"lat":41.2352564531203,"lng":-8.72412072704135},{"lat":41.2353205542219,"lng":-8.72422699074179},{"lat":41.2354586347834,"lng":-8.72435813052746},{"lat":41.2354495942954,"lng":-8.72452194231465},{"lat":41.2355339137568,"lng":-8.72460142788109},{"lat":41.2356134338367,"lng":-8.72435380471151},{"lat":41.2356870536782,"lng":-8.72425618499882},{"lat":41.2357842829501,"lng":-8.72422354406666},{"lat":41.2359431217903,"lng":-8.72415281431274},{"lat":41.2361006911649,"lng":-8.72415257212116},{"lat":41.2362013174863,"lng":-8.72421276309854},{"lat":41.2361571348457,"lng":-8.72438198323543},{"lat":41.2362913631612,"lng":-8.7243868965836},{"lat":41.2363980698995,"lng":-8.72431557527536},{"lat":41.2365187638025,"lng":-8.72427264980889},{"lat":41.2365825881575,"lng":-8.7245027278196},{"lat":41.2367942904228,"lng":-8.72446468444502},{"lat":41.2369612098865,"lng":-8.72441549572345},{"lat":41.2371655679528,"lng":-8.72438681031315},{"lat":41.2373967931281,"lng":-8.72440416494689},{"lat":41.2376710899666,"lng":-8.72450302768059},{"lat":41.2378356803765,"lng":-8.72450356298722},{"lat":41.2380417579761,"lng":-8.72454347392092},{"lat":41.2381702415133,"lng":-8.7246143046909},{"lat":41.2383169949259,"lng":-8.72475638192653},{"lat":41.2384209013943,"lng":-8.72489641284369},{"lat":41.238546032827,"lng":-8.72507611599323},{"lat":41.2386319740449,"lng":-8.72499804647496},{"lat":41.2387181457562,"lng":-8.72505011847215},{"lat":41.238766935662,"lng":-8.72519131735338},{"lat":41.2389024061548,"lng":-8.72526947904142},{"lat":41.2389305131779,"lng":-8.72538914171207},{"lat":41.239027938375,"lng":-8.72552815496706},{"lat":41.2390731911198,"lng":-8.72565775203001},{"lat":41.2391986449021,"lng":-8.7257745978715},{"lat":41.2393572202856,"lng":-8.72578916046587},{"lat":41.239410502522,"lng":-8.72593242857565},{"lat":41.2394873413572,"lng":-8.7260485951982},{"lat":41.2394350007259,"lng":-8.72620998908886},{"lat":41.2396080532909,"lng":-8.72621155836403},{"lat":41.2397249387238,"lng":-8.72624351445249},{"lat":41.2398215705058,"lng":-8.72632765178631},{"lat":41.2398778244921,"lng":-8.72643612060643},{"lat":41.2398489569295,"lng":-8.7266379302633},{"lat":41.2399697683475,"lng":-8.72660729712462},{"lat":41.2400413883971,"lng":-8.72668751580543},{"lat":41.2401332036426,"lng":-8.72677948491607},{"lat":41.2402839925704,"lng":-8.72676844812113},{"lat":41.2403673078098,"lng":-8.72681501178443},{"lat":41.2404569815415,"lng":-8.7268853706485},{"lat":41.2404733395675,"lng":-8.72704858350641},{"lat":41.2404554369834,"lng":-8.7271839361332},{"lat":41.2404735819688,"lng":-8.72733213469449},{"lat":41.2406142261773,"lng":-8.7275057891882},{"lat":41.2406705729619,"lng":-8.72736983263859},{"lat":41.2407526777993,"lng":-8.72754616979033},{"lat":41.2408458363773,"lng":-8.72760523618725},{"lat":41.2409445465715,"lng":-8.72749554801421},{"lat":41.2410896806654,"lng":-8.72767186563805},{"lat":41.2411831878557,"lng":-8.7276634095977},{"lat":41.241141846413,"lng":-8.72749102160198},{"lat":41.2412447500619,"lng":-8.72744232990697},{"lat":41.2413617480662,"lng":-8.72745257894733},{"lat":41.2414512740919,"lng":-8.72755216732259},{"lat":41.2415148468777,"lng":-8.72777736664679},{"lat":41.2415792094461,"lng":-8.72769373464514},{"lat":41.2416551653067,"lng":-8.72780512630934},{"lat":41.2416148788129,"lng":-8.72796913974105},{"lat":41.241751213883,"lng":-8.72805281188161},{"lat":41.2417834505316,"lng":-8.72824397699132},{"lat":41.2419394923033,"lng":-8.72836468423349},{"lat":41.2419565963678,"lng":-8.72853888643528},{"lat":41.2421543341142,"lng":-8.7286072540273},{"lat":41.2422843738975,"lng":-8.72855129777015},{"lat":41.242391809776,"lng":-8.72866870352914},{"lat":41.2423829008873,"lng":-8.72880676102022},{"lat":41.2423526350283,"lng":-8.72894773088653},{"lat":41.2425166546675,"lng":-8.72890210324816},{"lat":41.2425829939642,"lng":-8.72913175132753},{"lat":41.2426681661039,"lng":-8.72904639607661},{"lat":41.2427864898761,"lng":-8.72895824366815},{"lat":41.2428775673949,"lng":-8.72889644615347},{"lat":41.242960227813,"lng":-8.72858345804483},{"lat":41.2430387650378,"lng":-8.72837171299034},{"lat":41.2431176904454,"lng":-8.72811892753714},{"lat":41.243265446407,"lng":-8.72789326102443},{"lat":41.2434300270645,"lng":-8.72758268593683},{"lat":41.2435751347445,"lng":-8.72738144281749},{"lat":41.2438446853953,"lng":-8.72716117195708},{"lat":41.244128299013,"lng":-8.72699052879964},{"lat":41.2444631010296,"lng":-8.72684540830763},{"lat":41.2446647135343,"lng":-8.72670348310886},{"lat":41.2449288631028,"lng":-8.72660531596951},{"lat":41.2453887106893,"lng":-8.72646395233672},{"lat":41.2456754940092,"lng":-8.72641083857597},{"lat":41.2459533140204,"lng":-8.72628022589812},{"lat":41.2462807582469,"lng":-8.72618202982166},{"lat":41.2466144161803,"lng":-8.72608365513123},{"lat":41.2468266183588,"lng":-8.72596639606497},{"lat":41.2471340517286,"lng":-8.72587302819655},{"lat":41.2475380700528,"lng":-8.72578423424045},{"lat":41.2482028963692,"lng":-8.72566917229964},{"lat":41.2486666768424,"lng":-8.72565547646374},{"lat":41.2490688941728,"lng":-8.7257234282912},{"lat":41.2492137580732,"lng":-8.7257429959125},{"lat":41.249331860263,"lng":-8.72587160746972},{"lat":41.2494106761336,"lng":-8.72597014868584},{"lat":41.2494584732204,"lng":-8.72614774966024},{"lat":41.2495588447469,"lng":-8.726047373135},{"lat":41.2496606745249,"lng":-8.72583915493109},{"lat":41.2497361488742,"lng":-8.72571277995047},{"lat":41.2498042642868,"lng":-8.72559981631676},{"lat":41.2499400857627,"lng":-8.72545287206544},{"lat":41.2500684119086,"lng":-8.72534462732251},{"lat":41.2502106587701,"lng":-8.72520847409298},{"lat":41.2504433641613,"lng":-8.72507827423507},{"lat":41.2509212762258,"lng":-8.72494289167126},{"lat":41.251063210246,"lng":-8.72486698900231},{"lat":41.2513038961923,"lng":-8.72477873243833},{"lat":41.2515196377082,"lng":-8.72469084198476},{"lat":41.2516630886658,"lng":-8.72463511166458},{"lat":41.2518053901581,"lng":-8.72461015170975},{"lat":41.2521622705438,"lng":-8.7245831934245},{"lat":41.252402621404,"lng":-8.72457642133069},{"lat":41.2525719888432,"lng":-8.72457628817566},{"lat":41.2526950199212,"lng":-8.7246215434695},{"lat":41.2528947565952,"lng":-8.72463421222382},{"lat":41.2530532098569,"lng":-8.72465497389757},{"lat":41.2531827488977,"lng":-8.72464313974605},{"lat":41.2534471421316,"lng":-8.724707096715},{"lat":41.2537189942617,"lng":-8.72475478024201},{"lat":41.2539051890531,"lng":-8.72477472136808},{"lat":41.2540827376243,"lng":-8.72472395138525},{"lat":41.2542975939571,"lng":-8.72463306865422},{"lat":41.254560947712,"lng":-8.72453164454035},{"lat":41.2548132671847,"lng":-8.72449061819723},{"lat":41.2552307813325,"lng":-8.72454379425673},{"lat":41.2555002019663,"lng":-8.72436522948519},{"lat":41.2558511738868,"lng":-8.7242098362063},{"lat":41.2562152847943,"lng":-8.72410753126665},{"lat":41.256542204735,"lng":-8.72411048884643},{"lat":41.256758132171,"lng":-8.72414394518915},{"lat":41.2569002132756,"lng":-8.72419677581674},{"lat":41.2571165000479,"lng":-8.72415935479609},{"lat":41.257642843806,"lng":-8.72423571172481},{"lat":41.2578388205443,"lng":-8.72426146834489},{"lat":41.2580141161676,"lng":-8.72431674863068},{"lat":41.2581870669465,"lng":-8.72431962675768},{"lat":41.2583577449029,"lng":-8.72439705509814},{"lat":41.2585347146636,"lng":-8.72445963119527},{"lat":41.2587443966203,"lng":-8.72449887432275},{"lat":41.2589531678749,"lng":-8.72450505801314},{"lat":41.2591149936223,"lng":-8.72455270218335},{"lat":41.2592287234059,"lng":-8.72465611436789},{"lat":41.2593530844879,"lng":-8.72465330118649},{"lat":41.259499794477,"lng":-8.72447194598922},{"lat":41.259615097633,"lng":-8.72447907165422},{"lat":41.2596557083697,"lng":-8.72461821416887},{"lat":41.2599001948586,"lng":-8.72473510024125},{"lat":41.2600247011034,"lng":-8.72482703548332},{"lat":41.26066531361,"lng":-8.72495395124418},{"lat":41.2607190036425,"lng":-8.72517530894991},{"lat":41.2607745828993,"lng":-8.72541458195013},{"lat":41.2609817054777,"lng":-8.72556669255795},{"lat":41.2611469829883,"lng":-8.72564336633008},{"lat":41.2612830912978,"lng":-8.72577132409744},{"lat":41.2614259633979,"lng":-8.72582775020579},{"lat":41.2614419972882,"lng":-8.72605294350184},{"lat":41.261517605346,"lng":-8.72623262356695},{"lat":41.2616715967931,"lng":-8.7262278129872},{"lat":41.261761506349,"lng":-8.72625344688581},{"lat":41.2618655007475,"lng":-8.72632336482064},{"lat":41.2617832987813,"lng":-8.7265489818515},{"lat":41.261884697537,"lng":-8.72663331201404},{"lat":41.2620034385969,"lng":-8.72658570793897},{"lat":41.2621108988238,"lng":-8.72657713567131},{"lat":41.2622816153027,"lng":-8.72664694623792},{"lat":41.2623990011067,"lng":-8.72675707434844},{"lat":41.2625626831915,"lng":-8.72677646579452},{"lat":41.2628475958182,"lng":-8.72691067520589},{"lat":41.2629178926097,"lng":-8.72707240758661},{"lat":41.2629148741412,"lng":-8.72727261515904},{"lat":41.2630699694066,"lng":-8.72743738781813},{"lat":41.2631848129951,"lng":-8.72748150773728},{"lat":41.2633454775668,"lng":-8.72757939057636},{"lat":41.2634778739114,"lng":-8.72760744695605},{"lat":41.2636194957328,"lng":-8.72767877823061},{"lat":41.2637440040606,"lng":-8.72766488246672},{"lat":41.2639560503873,"lng":-8.72773423248286},{"lat":41.2640402945015,"lng":-8.72781017769251},{"lat":41.2641329158261,"lng":-8.72781626750036},{"lat":41.2642596162931,"lng":-8.72789988000254},{"lat":41.2643672336234,"lng":-8.72798307858574},{"lat":41.2644492849155,"lng":-8.7281172443258},{"lat":41.2645116703443,"lng":-8.72824108011173},{"lat":41.2645841249952,"lng":-8.72831632409036},{"lat":41.2646223866467,"lng":-8.72843886840033},{"lat":41.2647141804633,"lng":-8.72858720040687},{"lat":41.2647799998418,"lng":-8.7287435296697},{"lat":41.2648660407548,"lng":-8.72881925241684},{"lat":41.2650640169489,"lng":-8.72884134940072},{"lat":41.265169537408,"lng":-8.72882536766901},{"lat":41.2652602834469,"lng":-8.72870625936483},{"lat":41.2653731991637,"lng":-8.72863628563053},{"lat":41.2654916792565,"lng":-8.72851672216963},{"lat":41.2655732341163,"lng":-8.72841770070549},{"lat":41.2657195721275,"lng":-8.72830924665443},{"lat":41.2658549242805,"lng":-8.728252485519},{"lat":41.2659434346996,"lng":-8.72821772933802},{"lat":41.2660409507271,"lng":-8.72818234119519},{"lat":41.2661389815938,"lng":-8.72816998485235},{"lat":41.2662768408701,"lng":-8.72815059475429},{"lat":41.2663696624776,"lng":-8.72815311415061},{"lat":41.266483867194,"lng":-8.7281634640933},{"lat":41.2665699749339,"lng":-8.72820995335987},{"lat":41.2666618360467,"lng":-8.72824133909859},{"lat":41.2667707539124,"lng":-8.72822896702895},{"lat":41.2668646322627,"lng":-8.72820141579139},{"lat":41.2670327960707,"lng":-8.72805115255067},{"lat":41.2671700510338,"lng":-8.72795824384741},{"lat":41.2673106594036,"lng":-8.72792992994155},{"lat":41.2674254184,"lng":-8.72788562855844},{"lat":41.2675824333794,"lng":-8.72781963267808},{"lat":41.2676779796627,"lng":-8.72779913669773},{"lat":41.2678005031056,"lng":-8.72776838950073},{"lat":41.2679050947386,"lng":-8.7277238759273},{"lat":41.2679996370662,"lng":-8.72772342333618},{"lat":41.2681807757348,"lng":-8.72769249636456},{"lat":41.268288967763,"lng":-8.72766495907614},{"lat":41.268440548739,"lng":-8.7276222997846},{"lat":41.2685351594482,"lng":-8.72760860318992},{"lat":41.2687585160592,"lng":-8.72761564524633},{"lat":41.2688644348249,"lng":-8.72762675729183},{"lat":41.2690083130643,"lng":-8.72766267049906},{"lat":41.2691980566292,"lng":-8.72770939232909},{"lat":41.2692890595244,"lng":-8.72773158031306},{"lat":41.2693905616972,"lng":-8.72774444271283},{"lat":41.2695292207554,"lng":-8.72776228952448},{"lat":41.2696541269701,"lng":-8.72781044771676},{"lat":41.2697383918723,"lng":-8.72789976994029},{"lat":41.2698547585076,"lng":-8.72803174241006},{"lat":41.270019521174,"lng":-8.72799875970528},{"lat":41.2701308502615,"lng":-8.72792208329079},{"lat":41.270250426348,"lng":-8.72797364768394},{"lat":41.2703042782295,"lng":-8.72811114864924},{"lat":41.2703437468585,"lng":-8.72826116376356},{"lat":41.2703409611146,"lng":-8.72862453397451},{"lat":41.2706663523251,"lng":-8.72815884202329},{"lat":41.270724491681,"lng":-8.72800804148515},{"lat":41.2707441670798,"lng":-8.72785999933964},{"lat":41.2707245474577,"lng":-8.72770216155268},{"lat":41.2707163003372,"lng":-8.72752068807707},{"lat":41.2707536530736,"lng":-8.7272972605884},{"lat":41.2707302662051,"lng":-8.72713628940171},{"lat":41.2706484733194,"lng":-8.7270734849834},{"lat":41.2705273781557,"lng":-8.72698502940651},{"lat":41.2704370285307,"lng":-8.72687142517563},{"lat":41.2704002544418,"lng":-8.72675675988562},{"lat":41.2703823160142,"lng":-8.72662078688411},{"lat":41.2703873497194,"lng":-8.72646664290183},{"lat":41.2704400711705,"lng":-8.72630075502702},{"lat":41.2705487541752,"lng":-8.72610792518264},{"lat":41.2706215660943,"lng":-8.72595654452891},{"lat":41.270669201635,"lng":-8.7257639992851},{"lat":41.2706886606001,"lng":-8.72555317433134},{"lat":41.2706862601168,"lng":-8.72528522528456},{"lat":41.2706754581643,"lng":-8.72505790062101},{"lat":41.2706681487224,"lng":-8.72493860768613},{"lat":41.2706380916239,"lng":-8.72469165607521},{"lat":41.2706046272508,"lng":-8.72447617504221},{"lat":41.2705323891139,"lng":-8.7242716822163},{"lat":41.2704628873698,"lng":-8.72405921842327},{"lat":41.2703864815893,"lng":-8.72380695110953},{"lat":41.2703201869883,"lng":-8.72363700379739},{"lat":41.2701976619833,"lng":-8.72342406254266},{"lat":41.2701422025036,"lng":-8.72324777268023},{"lat":41.2700909369762,"lng":-8.72304335091081},{"lat":41.2700631830705,"lng":-8.72290741056162},{"lat":41.2701123495961,"lng":-8.72271451927196},{"lat":41.2701490061622,"lng":-8.72253678645019},{"lat":41.2702314579794,"lng":-8.72238465519979},{"lat":41.2702884159764,"lng":-8.72226845214996},{"lat":41.2703445293537,"lng":-8.72207180193407},{"lat":41.2703860093211,"lng":-8.72188421540771},{"lat":41.2704305325501,"lng":-8.72170082349145},{"lat":41.2704415644583,"lng":-8.72153348202234},{"lat":41.2704262043506,"lng":-8.72135075278564},{"lat":41.2704470102416,"lng":-8.72115616904781},{"lat":41.2704745364421,"lng":-8.72098540222931},{"lat":41.2705115146701,"lng":-8.72078082253441},{"lat":41.2705435114417,"lng":-8.72063288197932},{"lat":41.2705493793428,"lng":-8.72049032225762},{"lat":41.2705221876497,"lng":-8.72034913231438},{"lat":41.2704589686716,"lng":-8.7201927004428},{"lat":41.2703889546179,"lng":-8.72009779339103},{"lat":41.2702705055709,"lng":-8.71998359278483},{"lat":41.2701422204823,"lng":-8.71989125787565},{"lat":41.2700289816678,"lng":-8.71983152793078},{"lat":41.269866697313,"lng":-8.71976789101019},{"lat":41.2696981538124,"lng":-8.7196767482295},{"lat":41.2695706962822,"lng":-8.71949313423945},{"lat":41.269585802397,"lng":-8.71935578734129},{"lat":41.2696104191318,"lng":-8.71922628154932},{"lat":41.2696039819843,"lng":-8.71909363494397},{"lat":41.2695901293446,"lng":-8.71889683720728},{"lat":41.2695381913462,"lng":-8.71871724266114},{"lat":41.2694393562263,"lng":-8.71855333239238},{"lat":41.26930877382,"lng":-8.71836360454734},{"lat":41.2692271504542,"lng":-8.7182319533666},{"lat":41.269156365672,"lng":-8.71805803989796},{"lat":41.2690687764915,"lng":-8.71794805524953},{"lat":41.2689148483497,"lng":-8.71781623734032},{"lat":41.2688148974002,"lng":-8.71762320663551},{"lat":41.2689888157953,"lng":-8.7174891960131},{"lat":41.2691609711453,"lng":-8.71734788550404},{"lat":41.2693487245648,"lng":-8.71720313299866},{"lat":41.2694991935776,"lng":-8.71709420478006},{"lat":41.269727988618,"lng":-8.71695244558315},{"lat":41.2698207258225,"lng":-8.71684682211643},{"lat":41.269933423648,"lng":-8.71685391130267},{"lat":41.2700137745736,"lng":-8.71697063287711},{"lat":41.270085220877,"lng":-8.71713871146767},{"lat":41.270168867514,"lng":-8.71726225849748},{"lat":41.2702744283408,"lng":-8.7173091506153},{"lat":41.2703645745091,"lng":-8.71725303286173},{"lat":41.2704512125875,"lng":-8.71716071747364},{"lat":41.2704721940858,"lng":-8.7170181717575},{"lat":41.2704794882854,"lng":-8.71685927194437},{"lat":41.2704461742824,"lng":-8.71668246440918},{"lat":41.2704444694995,"lng":-8.71650415186126},{"lat":41.2704563562891,"lng":-8.71627320873174},{"lat":41.2704607800717,"lng":-8.71605807317089},{"lat":41.2704713532597,"lng":-8.71589132687694},{"lat":41.2704828105943,"lng":-8.71572756962088},{"lat":41.2704613420579,"lng":-8.71555814914867},{"lat":41.270428520013,"lng":-8.71537215715168},{"lat":41.2703964546272,"lng":-8.71512531228589},{"lat":41.2703354335218,"lng":-8.71490708361572},{"lat":41.2702322239145,"lng":-8.71484088360999},{"lat":41.2701059924848,"lng":-8.71480479525509},{"lat":41.2699598919569,"lng":-8.7147798599959},{"lat":41.2698703796268,"lng":-8.71472893997348},{"lat":41.2698009046055,"lng":-8.71463284379464},{"lat":41.2697938858112,"lng":-8.71449005146329},{"lat":41.2698328528208,"lng":-8.7143344222731},{"lat":41.2699461782499,"lng":-8.71421632585987},{"lat":41.2700908564695,"lng":-8.71407726256728},{"lat":41.2702097031239,"lng":-8.71397198640834},{"lat":41.2703439533548,"lng":-8.7138830772299},{"lat":41.2705219709893,"lng":-8.71377522523355},{"lat":41.2706502238385,"lng":-8.71369712011541},{"lat":41.2707424068062,"lng":-8.71368278117155},{"lat":41.2709457563168,"lng":-8.71369246551522},{"lat":41.2710430595953,"lng":-8.71359176910719},{"lat":41.2711071067594,"lng":-8.71344411600825},{"lat":41.2711844504389,"lng":-8.71330158929671},{"lat":41.2713241708356,"lng":-8.71314553032744},{"lat":41.2714273590259,"lng":-8.71296755406209},{"lat":41.2715100864442,"lng":-8.7127572824501},{"lat":41.2715356165753,"lng":-8.71262265011551},{"lat":41.2715229870143,"lng":-8.71243146682266},{"lat":41.2715357972016,"lng":-8.71226593046576},{"lat":41.2716541928924,"lng":-8.71207113337827},{"lat":41.2718104009061,"lng":-8.71184217941263},{"lat":41.271893820019,"lng":-8.71160100149223},{"lat":41.2719015613352,"lng":-8.71142384260606},{"lat":41.2717838365941,"lng":-8.71136002586845},{"lat":41.2716223740152,"lng":-8.711471733528},{"lat":41.2715362718071,"lng":-8.71138695592061},{"lat":41.2715216466421,"lng":-8.71121604179808},{"lat":41.2715805065992,"lng":-8.71102453809096},{"lat":41.2715011828458,"lng":-8.71095258501681},{"lat":41.2713814542683,"lng":-8.71089399951686},{"lat":41.2712690438168,"lng":-8.71083083141015},{"lat":41.2711854414012,"lng":-8.71071396589499},{"lat":41.2710913526999,"lng":-8.71057051945303},{"lat":41.2710288415239,"lng":-8.71044966688618},{"lat":41.271067768734,"lng":-8.71030070701579},{"lat":41.2712032581223,"lng":-8.71023292791283},{"lat":41.2713507847553,"lng":-8.710225755968},{"lat":41.2714879136616,"lng":-8.71026109915857},{"lat":41.2716900343167,"lng":-8.71037316308545},{"lat":41.2718670681313,"lng":-8.71033426746989},{"lat":41.2719400834658,"lng":-8.71015649221543},{"lat":41.2720157915473,"lng":-8.70996310693348},{"lat":41.2721056837618,"lng":-8.70979574209546},{"lat":41.2722330323106,"lng":-8.70964577429073},{"lat":41.2723180084694,"lng":-8.70952431543971},{"lat":41.2724834210961,"lng":-8.70927192477391},{"lat":41.2725873350225,"lng":-8.70912640368921},{"lat":41.2727917634702,"lng":-8.70886826700813},{"lat":41.2727712173982,"lng":-8.70862019635054},{"lat":41.2725515861382,"lng":-8.7081587779051},{"lat":41.2724919598765,"lng":-8.70792851130492},{"lat":41.2724330200077,"lng":-8.70774121295787},{"lat":41.2723348887398,"lng":-8.7075953444251},{"lat":41.2722199951275,"lng":-8.70741591079714},{"lat":41.2723837467173,"lng":-8.70713437845681},{"lat":41.2725406431789,"lng":-8.7068552910094},{"lat":41.2726557517598,"lng":-8.70668516234434},{"lat":41.2727519098284,"lng":-8.70645196321864},{"lat":41.2728540608108,"lng":-8.7062811172445},{"lat":41.2729859132904,"lng":-8.70614812669876},{"lat":41.273088066181,"lng":-8.70590424014039},{"lat":41.2731147010559,"lng":-8.70569179680732},{"lat":41.2730852413518,"lng":-8.70553280691204},{"lat":41.2730253266764,"lng":-8.70543214577838},{"lat":41.272923566757,"lng":-8.70530796924074},{"lat":41.2727882154643,"lng":-8.70516810022737},{"lat":41.2727072976506,"lng":-8.70507191713076},{"lat":41.2726379344484,"lng":-8.7049881268342},{"lat":41.2725633288674,"lng":-8.70490834524152},{"lat":41.2724912605794,"lng":-8.70482548529641},{"lat":41.2724297142289,"lng":-8.70472671748595},{"lat":41.2723376661269,"lng":-8.70449736399913},{"lat":41.2722679157886,"lng":-8.70430950112142},{"lat":41.2722100644914,"lng":-8.70415587464507},{"lat":41.2721331014524,"lng":-8.70402499074415},{"lat":41.2719831817188,"lng":-8.704007687239},{"lat":41.2718989793724,"lng":-8.70393832773136},{"lat":41.2718079222896,"lng":-8.70387129564243},{"lat":41.271694532792,"lng":-8.70378688208926},{"lat":41.2715947864572,"lng":-8.70372896446428},{"lat":41.2715000538695,"lng":-8.70369388468513},{"lat":41.2713780021218,"lng":-8.70363111747592},{"lat":41.2712508308862,"lng":-8.70352939908165},{"lat":41.2710723950702,"lng":-8.70334393271346},{"lat":41.2708429072341,"lng":-8.70303115309702},{"lat":41.2707026104676,"lng":-8.70283384404311},{"lat":41.270630450848,"lng":-8.70273237490359},{"lat":41.2704624058112,"lng":-8.70253900070172},{"lat":41.2700871822963,"lng":-8.70254697453498},{"lat":41.2689384969202,"lng":-8.70244186716038},{"lat":41.267498908216,"lng":-8.70223022374388},{"lat":41.2644355120669,"lng":-8.70186141433356},{"lat":41.2629426013592,"lng":-8.70176577241272},{"lat":41.2609545276462,"lng":-8.70284707334607},{"lat":41.2600736618944,"lng":-8.70332440955585},{"lat":41.2596060046108,"lng":-8.70334568090795},{"lat":41.2584503813702,"lng":-8.70247081204899},{"lat":41.256563304367,"lng":-8.70186942527064},{"lat":41.2532566882522,"lng":-8.70026951684232},{"lat":41.2501278542641,"lng":-8.69736861934156},{"lat":41.2467038853153,"lng":-8.69532036704913},{"lat":41.2438885668989,"lng":-8.69477409860546},{"lat":41.241211834275,"lng":-8.69523766832658},{"lat":41.2385980350808,"lng":-8.69622357641352},{"lat":41.2386602101758,"lng":-8.70010214090155},{"lat":41.2377433083129,"lng":-8.70343415481335},{"lat":41.2370212333012,"lng":-8.70378571330317},{"lat":41.2362445112095,"lng":-8.70847399366566},{"lat":41.2335032668454,"lng":-8.71752981594723},{"lat":41.2319683079194,"lng":-8.72259229187527}]},{"zone_id":"mts1","points":[{"lat":41.1743934209365,"lng":-8.69088975628196},{"lat":41.1747195982227,"lng":-8.69111538201201},{"lat":41.1750997580383,"lng":-8.6916358325256},{"lat":41.1754938059116,"lng":-8.69207811273903},{"lat":41.1759403634739,"lng":-8.69264728969309},{"lat":41.1763952100933,"lng":-8.69314015399513},{"lat":41.1768722158697,"lng":-8.69376049878672},{"lat":41.1771413464346,"lng":-8.69413225484247},{"lat":41.1774972711418,"lng":-8.69465348672848},{"lat":41.1775572881604,"lng":-8.69469893346399},{"lat":41.1777034345609,"lng":-8.69502995125928},{"lat":41.1781019308656,"lng":-8.69564619190368},{"lat":41.1783602006824,"lng":-8.69623072031037},{"lat":41.1785536148504,"lng":-8.69667156459995},{"lat":41.1787816351051,"lng":-8.69697814903014},{"lat":41.1789738988072,"lng":-8.69743340684169},{"lat":41.1769249206641,"lng":-8.70136030337563},{"lat":41.1766693974809,"lng":-8.70204645769888},{"lat":41.1766078026579,"lng":-8.70266528598171},{"lat":41.176632667912,"lng":-8.7034424164237},{"lat":41.1768873016912,"lng":-8.70422512563635},{"lat":41.1772222243903,"lng":-8.7047775734814},{"lat":41.1774001806351,"lng":-8.705043219071},{"lat":41.1775075742033,"lng":-8.70517608992096},{"lat":41.1778481906297,"lng":-8.7056694864799},{"lat":41.1778962612628,"lng":-8.70577991066459},{"lat":41.1779708517755,"lng":-8.70577364964472},{"lat":41.1780047138757,"lng":-8.70570016966944},{"lat":41.1780077287866,"lng":-8.70560104016379},{"lat":41.1779288919732,"lng":-8.70555673076819},{"lat":41.1777208826424,"lng":-8.70517795167123},{"lat":41.1769532764319,"lng":-8.70397721215554},{"lat":41.1767754207356,"lng":-8.70336452317076},{"lat":41.1767236478996,"lng":-8.70274626039926},{"lat":41.1768261303893,"lng":-8.70205329897642},{"lat":41.1769773165423,"lng":-8.70168956712561},{"lat":41.177177492859,"lng":-8.70161610355734},{"lat":41.1794334802643,"lng":-8.69752687216116},{"lat":41.1796251233996,"lng":-8.69774150421538},{"lat":41.1797840452316,"lng":-8.69746779489711},{"lat":41.1803094310907,"lng":-8.6964469869221},{"lat":41.180430354677,"lng":-8.69622788948192},{"lat":41.1806723480655,"lng":-8.69610875796224},{"lat":41.1819711389048,"lng":-8.69726947175935},{"lat":41.1820845317783,"lng":-8.69709953687771},{"lat":41.1824441000157,"lng":-8.69744374300899},{"lat":41.1810615220717,"lng":-8.70006404114089},{"lat":41.1808950735192,"lng":-8.70025484766923},{"lat":41.1807559823217,"lng":-8.70050464414993},{"lat":41.1809755772927,"lng":-8.7007108267319},{"lat":41.1811126541768,"lng":-8.70046756584769},{"lat":41.1811890427534,"lng":-8.70016943321541},{"lat":41.1824639998024,"lng":-8.69775511530966},{"lat":41.183023410707,"lng":-8.69825850339733},{"lat":41.1818681794488,"lng":-8.70045837586021},{"lat":41.1816744868323,"lng":-8.70064143523101},{"lat":41.1815492192076,"lng":-8.70088158333571},{"lat":41.1817722413239,"lng":-8.70108648736222},{"lat":41.1818994593458,"lng":-8.70085291590171},{"lat":41.1819750278108,"lng":-8.70053867566668},{"lat":41.1831346296976,"lng":-8.69836505993976},{"lat":41.1836920464464,"lng":-8.69888762935572},{"lat":41.1826714550314,"lng":-8.70082706282347},{"lat":41.1824695520483,"lng":-8.70101279613338},{"lat":41.1823422140068,"lng":-8.70127057109554},{"lat":41.1825628131066,"lng":-8.70145555078606},{"lat":41.1826964306765,"lng":-8.70122024115383},{"lat":41.1827708521234,"lng":-8.70092041350311},{"lat":41.1843427612838,"lng":-8.69803709791528},{"lat":41.1850944446234,"lng":-8.69899032643182},{"lat":41.1854858839892,"lng":-8.69959412308604},{"lat":41.1883251885845,"lng":-8.69533274916373},{"lat":41.1913424701989,"lng":-8.69016139881032},{"lat":41.1937035150028,"lng":-8.68611417172874},{"lat":41.1950949386008,"lng":-8.68372878813696},{"lat":41.1956963738347,"lng":-8.68269766173494},{"lat":41.1961755514294,"lng":-8.6823854690479},{"lat":41.1975033154816,"lng":-8.68196363723181},{"lat":41.198064015806,"lng":-8.68178549703856},{"lat":41.1985691251797,"lng":-8.68130707146928},{"lat":41.1988861562047,"lng":-8.68072720748703},{"lat":41.1991210735375,"lng":-8.68070326352301},{"lat":41.1999075092396,"lng":-8.68004609039097},{"lat":41.2003746584161,"lng":-8.67990089406379},{"lat":41.2009507520181,"lng":-8.68009534605321},{"lat":41.2011163811829,"lng":-8.67963070026864},{"lat":41.2014666572145,"lng":-8.67925735553503},{"lat":41.2016911351273,"lng":-8.67888181931526},{"lat":41.2018566378339,"lng":-8.67860494163719},{"lat":41.2021973746264,"lng":-8.6783248270925},{"lat":41.202594219772,"lng":-8.67786586087087},{"lat":41.2028546523195,"lng":-8.67756465675437},{"lat":41.2027954264712,"lng":-8.67679852798762},{"lat":41.2030610742266,"lng":-8.67599226754132},{"lat":41.2032164550115,"lng":-8.6754871138492},{"lat":41.2032955654997,"lng":-8.67470548138972},{"lat":41.2030595550043,"lng":-8.67410433477647},{"lat":41.2029997583457,"lng":-8.6734547122949},{"lat":41.2031246510491,"lng":-8.67269834386249},{"lat":41.203182228206,"lng":-8.6720747331553},{"lat":41.2034793219793,"lng":-8.67137689886667},{"lat":41.2035769723314,"lng":-8.67099608799599},{"lat":41.2036077936385,"lng":-8.66978129339178},{"lat":41.2036568267092,"lng":-8.66906575892375},{"lat":41.2037329320824,"lng":-8.66854238379003},{"lat":41.2045961054376,"lng":-8.66810826743622},{"lat":41.2049591961184,"lng":-8.6678583378552},{"lat":41.2051013906168,"lng":-8.66757975636654},{"lat":41.205450266841,"lng":-8.66824303072857},{"lat":41.2060357693186,"lng":-8.66870170809021},{"lat":41.2066444200521,"lng":-8.66960641637364},{"lat":41.2070110395994,"lng":-8.67050826654524},{"lat":41.2074102467268,"lng":-8.67116071270154},{"lat":41.2075547229229,"lng":-8.67121793785795},{"lat":41.2076447422874,"lng":-8.67074650021533},{"lat":41.2078326422731,"lng":-8.66955100227317},{"lat":41.2082833902174,"lng":-8.66943546139825},{"lat":41.2085512780688,"lng":-8.66991457840169},{"lat":41.208838797938,"lng":-8.67039386028204},{"lat":41.2091928950212,"lng":-8.67032640821632},{"lat":41.2097618181853,"lng":-8.66963193835214},{"lat":41.2101899026907,"lng":-8.66948562797553},{"lat":41.2105431097928,"lng":-8.66922217925785},{"lat":41.2117102576764,"lng":-8.66918513897956},{"lat":41.2120831082222,"lng":-8.6692014860986},{"lat":41.2128775657975,"lng":-8.66899610932684},{"lat":41.2133379473958,"lng":-8.66917438123922},{"lat":41.2136195233684,"lng":-8.66912913430464},{"lat":41.2140652090747,"lng":-8.66869214188756},{"lat":41.214467168832,"lng":-8.66811740220817},{"lat":41.2148566044245,"lng":-8.66749285637332},{"lat":41.2154486395827,"lng":-8.66723132554782},{"lat":41.2159256084277,"lng":-8.66738504265862},{"lat":41.2165509387365,"lng":-8.66749310140608},{"lat":41.217031548523,"lng":-8.66744400569586},{"lat":41.217199219284,"lng":-8.66788081988233},{"lat":41.2179208154833,"lng":-8.66879886607164},{"lat":41.218407596576,"lng":-8.6688869782377},{"lat":41.2186861570053,"lng":-8.66845640047944},{"lat":41.2188145716748,"lng":-8.66787475792763},{"lat":41.2187805776164,"lng":-8.66707536037566},{"lat":41.2183083992698,"lng":-8.6658894883112},{"lat":41.2181356573138,"lng":-8.66518886019604},{"lat":41.2176711860609,"lng":-8.66438465967642},{"lat":41.2173431513933,"lng":-8.6639504734781},{"lat":41.217140120391,"lng":-8.66326718833839},{"lat":41.2171610513019,"lng":-8.66255057217179},{"lat":41.2173231332197,"lng":-8.66175840118238},{"lat":41.2175959896424,"lng":-8.66116436519654},{"lat":41.2171615303379,"lng":-8.66087228586437},{"lat":41.2165665660562,"lng":-8.66061462579858},{"lat":41.2163421368696,"lng":-8.66021327340642},{"lat":41.2162661605047,"lng":-8.65929975797336},{"lat":41.2165270114758,"lng":-8.65865158996939},{"lat":41.2171687019972,"lng":-8.65818031679319},{"lat":41.2179725281544,"lng":-8.65758995230108},{"lat":41.2182285156377,"lng":-8.65704642610051},{"lat":41.2186999005472,"lng":-8.65604553135574},{"lat":41.2190955350042,"lng":-8.65581226768047},{"lat":41.219821910956,"lng":-8.65567197806073},{"lat":41.2201421410413,"lng":-8.65569928884951},{"lat":41.2205355057277,"lng":-8.65469451229657},{"lat":41.2205664330149,"lng":-8.65416579088788},{"lat":41.2205418311528,"lng":-8.65333968680584},{"lat":41.2204160611788,"lng":-8.65256825169482},{"lat":41.2193431591707,"lng":-8.65318864815172},{"lat":41.2182663538025,"lng":-8.65416394552076},{"lat":41.2166713577329,"lng":-8.6549618453456},{"lat":41.214892373616,"lng":-8.65583929543874},{"lat":41.2134782188613,"lng":-8.65588461084605},{"lat":41.2122605784305,"lng":-8.65581829769806},{"lat":41.210794616797,"lng":-8.65515819150042},{"lat":41.209340245349,"lng":-8.65501696789106},{"lat":41.2047743818434,"lng":-8.65559250539438},{"lat":41.2031289928289,"lng":-8.65568738985823},{"lat":41.2011814374622,"lng":-8.65597305219219},{"lat":41.1995886404169,"lng":-8.65628449697664},{"lat":41.198361593201,"lng":-8.65692299214244},{"lat":41.1965832123609,"lng":-8.6571650770664},{"lat":41.1962186897418,"lng":-8.65706793628513},{"lat":41.194585405844,"lng":-8.65673491225217},{"lat":41.1923993062019,"lng":-8.65646194923535},{"lat":41.1865091761395,"lng":-8.65558125423425},{"lat":41.1833821568553,"lng":-8.65493546095186},{"lat":41.1817300650383,"lng":-8.65482964492732},{"lat":41.1790389250655,"lng":-8.65420131241252},{"lat":41.178801654245,"lng":-8.65487607070049},{"lat":41.1784475674539,"lng":-8.65588300403144},{"lat":41.1781171645281,"lng":-8.65639101865561},{"lat":41.1777341809566,"lng":-8.65772724474836},{"lat":41.1774514406321,"lng":-8.65923624615938},{"lat":41.1773817205892,"lng":-8.6608940832116},{"lat":41.1774691398101,"lng":-8.66170196458943},{"lat":41.1777974552519,"lng":-8.66229677726399},{"lat":41.1785400684249,"lng":-8.66278490228239},{"lat":41.1792182093856,"lng":-8.66340083749553},{"lat":41.1795927561219,"lng":-8.66427171590295},{"lat":41.1796184238318,"lng":-8.66487175646687},{"lat":41.1796660981644,"lng":-8.66598644675204},{"lat":41.1796078091619,"lng":-8.66652708308533},{"lat":41.1794334432893,"lng":-8.66690734678309},{"lat":41.1790863997006,"lng":-8.66736547594221},{"lat":41.1784149904479,"lng":-8.66786182715305},{"lat":41.1776747369188,"lng":-8.66823174753957},{"lat":41.1770170480807,"lng":-8.66882304634026},{"lat":41.1768422825511,"lng":-8.66949692206131},{"lat":41.1765911780414,"lng":-8.67013988256681},{"lat":41.1763276799659,"lng":-8.67062843887275},{"lat":41.1760406861615,"lng":-8.6710785510021},{"lat":41.1753214009467,"lng":-8.67151733541752},{"lat":41.1748099694954,"lng":-8.67218735384555},{"lat":41.1737444047562,"lng":-8.67363883088937},{"lat":41.1727110879752,"lng":-8.67523396107685},{"lat":41.1723018647874,"lng":-8.67648659535406},{"lat":41.1719579186703,"lng":-8.67798871879124},{"lat":41.1719593544496,"lng":-8.67935463758057},{"lat":41.1721168506283,"lng":-8.67973510579506},{"lat":41.1730330041512,"lng":-8.68168353722123},{"lat":41.1738659589977,"lng":-8.68352069965248},{"lat":41.1740131775164,"lng":-8.68436998244056},{"lat":41.1739279671983,"lng":-8.68596858052286},{"lat":41.1738361103289,"lng":-8.68769140159211},{"lat":41.1741234376645,"lng":-8.68803607902793},{"lat":41.1742010363027,"lng":-8.68848541843597},{"lat":41.1746529310121,"lng":-8.68851317349185},{"lat":41.1743934209365,"lng":-8.69088975628196}]},{"zone_id":"mts2","points":[{"lat":41.1914040645143,"lng":-8.70728764831564},{"lat":41.1914542506716,"lng":-8.70731776972129},{"lat":41.1914822842433,"lng":-8.7074552194228},{"lat":41.1915296349297,"lng":-8.70754824963421},{"lat":41.191566440818,"lng":-8.70739039256629},{"lat":41.1915599182946,"lng":-8.70723667845982},{"lat":41.1915984655148,"lng":-8.7071269957665},{"lat":41.1916783272115,"lng":-8.70714628651363},{"lat":41.1918362061456,"lng":-8.70708676106336},{"lat":41.1920863129896,"lng":-8.70714854918559},{"lat":41.192300724792,"lng":-8.70730706413952},{"lat":41.1923233220922,"lng":-8.7074157452007},{"lat":41.1922870698129,"lng":-8.70751722257164},{"lat":41.1924090761486,"lng":-8.70757182116409},{"lat":41.1925169039192,"lng":-8.70761722980758},{"lat":41.1925218975114,"lng":-8.70769928555469},{"lat":41.192446838477,"lng":-8.70778111868258},{"lat":41.1924529050334,"lng":-8.70799061124054},{"lat":41.1924909410566,"lng":-8.70814508372331},{"lat":41.1926411323232,"lng":-8.7081639208885},{"lat":41.1927056553524,"lng":-8.70826199997201},{"lat":41.1927450502665,"lng":-8.70830680910012},{"lat":41.1928066475519,"lng":-8.70819732075411},{"lat":41.1928316890893,"lng":-8.70826787589005},{"lat":41.1928108282858,"lng":-8.70844161586451},{"lat":41.1928258167231,"lng":-8.70852316032419},{"lat":41.192924523933,"lng":-8.70842949851036},{"lat":41.1930337677627,"lng":-8.7081920471961},{"lat":41.1930729183058,"lng":-8.70824949065093},{"lat":41.1931692475737,"lng":-8.70811014464152},{"lat":41.1934266887005,"lng":-8.70812695243855},{"lat":41.1934961367268,"lng":-8.70806843021868},{"lat":41.1935673698125,"lng":-8.70822927170234},{"lat":41.193544316637,"lng":-8.70833742775784},{"lat":41.1933762878756,"lng":-8.70846434079201},{"lat":41.1932521703776,"lng":-8.70863240555745},{"lat":41.1931598763179,"lng":-8.70868630985913},{"lat":41.1931226348927,"lng":-8.7087690744684},{"lat":41.1931366207889,"lng":-8.70883475691769},{"lat":41.1931489793134,"lng":-8.70890186391118},{"lat":41.1932561986817,"lng":-8.70888885772381},{"lat":41.1932601823144,"lng":-8.70901059978688},{"lat":41.1933689277118,"lng":-8.70903444374908},{"lat":41.1934586624959,"lng":-8.70909662258932},{"lat":41.1935795514937,"lng":-8.70906884230345},{"lat":41.1936605065001,"lng":-8.70908576756071},{"lat":41.1937641874379,"lng":-8.70922150317489},{"lat":41.1938670390894,"lng":-8.70919951955549},{"lat":41.1938251901118,"lng":-8.70906921010374},{"lat":41.1937596734379,"lng":-8.70906207938813},{"lat":41.1938268156935,"lng":-8.7089607455838},{"lat":41.1939439007139,"lng":-8.70911662968424},{"lat":41.1939769517797,"lng":-8.7089905556836},{"lat":41.1939746941029,"lng":-8.7088838486662},{"lat":41.1940985714863,"lng":-8.70881698555834},{"lat":41.1942343306822,"lng":-8.70884106693928},{"lat":41.1943405933161,"lng":-8.70885630938647},{"lat":41.1944140188056,"lng":-8.70892085021921},{"lat":41.1945231419491,"lng":-8.70904900109303},{"lat":41.1947147798637,"lng":-8.7090987277323},{"lat":41.1948458006738,"lng":-8.70916866514295},{"lat":41.1948909919744,"lng":-8.70931474026451},{"lat":41.1948231969693,"lng":-8.7093665955756},{"lat":41.1947610059809,"lng":-8.70941468312027},{"lat":41.1947432439239,"lng":-8.7095270639488},{"lat":41.194872009075,"lng":-8.70950900557773},{"lat":41.194783622925,"lng":-8.70957319618505},{"lat":41.1948324154257,"lng":-8.70970141735546},{"lat":41.1948891938463,"lng":-8.70988860227887},{"lat":41.1948369185548,"lng":-8.70988086777893},{"lat":41.1948803296277,"lng":-8.71002155861803},{"lat":41.1949789605297,"lng":-8.71010563465925},{"lat":41.1950390113696,"lng":-8.7100889971896},{"lat":41.1950730298962,"lng":-8.71020028237581},{"lat":41.1951723736616,"lng":-8.71030368329834},{"lat":41.195259997825,"lng":-8.71022899485556},{"lat":41.1953494443999,"lng":-8.71034839624972},{"lat":41.1954693864427,"lng":-8.71049107536075},{"lat":41.1955991669843,"lng":-8.71048625728599},{"lat":41.1955861682635,"lng":-8.71038552817449},{"lat":41.1955983548722,"lng":-8.71030743679757},{"lat":41.1955699572883,"lng":-8.71018904578839},{"lat":41.1957150893965,"lng":-8.71015718358635},{"lat":41.1959907037233,"lng":-8.71017713794568},{"lat":41.1962877384583,"lng":-8.71036119458321},{"lat":41.1964486725377,"lng":-8.7105891184139},{"lat":41.1965141092381,"lng":-8.7108452854187},{"lat":41.1964520530532,"lng":-8.71083043576462},{"lat":41.1964568578248,"lng":-8.71093156697916},{"lat":41.1965694149804,"lng":-8.71109278346247},{"lat":41.1964526682099,"lng":-8.71103024068362},{"lat":41.1965205399182,"lng":-8.71114230046069},{"lat":41.1965991983519,"lng":-8.71127555947309},{"lat":41.1966527073747,"lng":-8.71120009667746},{"lat":41.1967151028014,"lng":-8.71116452168213},{"lat":41.196812026964,"lng":-8.71119363542571},{"lat":41.196730690806,"lng":-8.71125251863318},{"lat":41.1968157021152,"lng":-8.71135818209638},{"lat":41.1968239213492,"lng":-8.71151358926039},{"lat":41.1967518529888,"lng":-8.71152058037613},{"lat":41.1966573693193,"lng":-8.71134653222296},{"lat":41.1966177415163,"lng":-8.71150914400158},{"lat":41.1966630977446,"lng":-8.71165784412151},{"lat":41.1965571710461,"lng":-8.71173582789798},{"lat":41.1964755199075,"lng":-8.71169636311711},{"lat":41.1964002303394,"lng":-8.71178844660344},{"lat":41.1963933396828,"lng":-8.71192273549309},{"lat":41.1964214147167,"lng":-8.71205185761274},{"lat":41.1963607726676,"lng":-8.71213083363949},{"lat":41.1965044974919,"lng":-8.71219957885999},{"lat":41.1965406004368,"lng":-8.71200474803488},{"lat":41.1965876952966,"lng":-8.71211185564181},{"lat":41.19660807256,"lng":-8.71219489215817},{"lat":41.1966831625394,"lng":-8.71230415941676},{"lat":41.196724902641,"lng":-8.71240144759837},{"lat":41.1968477428814,"lng":-8.71232611901129},{"lat":41.1968527212137,"lng":-8.71244645114225},{"lat":41.1969638069144,"lng":-8.71252289611421},{"lat":41.1970859474052,"lng":-8.71223011330436},{"lat":41.1970155785948,"lng":-8.71207761347194},{"lat":41.1970592075671,"lng":-8.71201481383918},{"lat":41.1972338416769,"lng":-8.71236505955936},{"lat":41.1973498579552,"lng":-8.71241067180675},{"lat":41.197396912073,"lng":-8.71234718957301},{"lat":41.1974376393571,"lng":-8.71230606105294},{"lat":41.1973333409741,"lng":-8.71211487608633},{"lat":41.1974564280985,"lng":-8.71197254569739},{"lat":41.1975585905203,"lng":-8.71208741604707},{"lat":41.1976405100691,"lng":-8.71207359775221},{"lat":41.1977486900967,"lng":-8.71188798061918},{"lat":41.1978270949444,"lng":-8.71183872423054},{"lat":41.1978850720698,"lng":-8.71177032808775},{"lat":41.1979117438973,"lng":-8.71167662065502},{"lat":41.1980149010047,"lng":-8.71157608096351},{"lat":41.1982338307437,"lng":-8.7115346201972},{"lat":41.1983896691559,"lng":-8.71154100615022},{"lat":41.1984824852953,"lng":-8.71172339170047},{"lat":41.1987038149326,"lng":-8.71174191490048},{"lat":41.1988279320525,"lng":-8.71178759876412},{"lat":41.1988928691149,"lng":-8.7119107293862},{"lat":41.1990098385815,"lng":-8.71192797793917},{"lat":41.1991388513272,"lng":-8.71207551808012},{"lat":41.199219869488,"lng":-8.71207981169876},{"lat":41.1993086784268,"lng":-8.71214664215256},{"lat":41.1994805132273,"lng":-8.71217582139446},{"lat":41.1996608113744,"lng":-8.71229579687322},{"lat":41.1997789966485,"lng":-8.71246506107046},{"lat":41.1998383098288,"lng":-8.71264858763298},{"lat":41.1998587338178,"lng":-8.71272328152417},{"lat":41.1999669118936,"lng":-8.71287731592601},{"lat":41.2001116026907,"lng":-8.71291507744589},{"lat":41.200223725312,"lng":-8.71284084371375},{"lat":41.2002464578887,"lng":-8.71295656933301},{"lat":41.2002590006896,"lng":-8.71314814405379},{"lat":41.2002599311183,"lng":-8.71330266227593},{"lat":41.2002935006914,"lng":-8.71344840951838},{"lat":41.2003928526849,"lng":-8.71338824660533},{"lat":41.2004873078629,"lng":-8.71340697086473},{"lat":41.2006380139571,"lng":-8.71351989412058},{"lat":41.2007183518958,"lng":-8.71365878204285},{"lat":41.200834790783,"lng":-8.71378094240246},{"lat":41.2009399831144,"lng":-8.71379475274881},{"lat":41.2010505873585,"lng":-8.71384282717817},{"lat":41.2011358837248,"lng":-8.71389162999596},{"lat":41.2013087668414,"lng":-8.71403479796363},{"lat":41.2014568592855,"lng":-8.71396994423638},{"lat":41.2015407413034,"lng":-8.7139591229323},{"lat":41.2015972207694,"lng":-8.71397321448712},{"lat":41.2016595832102,"lng":-8.71410550997007},{"lat":41.2017001974505,"lng":-8.71426490720812},{"lat":41.2017910734712,"lng":-8.71431376058629},{"lat":41.2019022103939,"lng":-8.71420279719538},{"lat":41.2019765713938,"lng":-8.71413550364951},{"lat":41.2020249299323,"lng":-8.71415154867802},{"lat":41.2020534556214,"lng":-8.71424479810057},{"lat":41.2019514856459,"lng":-8.71443047181895},{"lat":41.2018688133703,"lng":-8.71457507134677},{"lat":41.2019210901736,"lng":-8.7147603352718},{"lat":41.2019998662273,"lng":-8.71486952724809},{"lat":41.2020485614329,"lng":-8.7150336544031},{"lat":41.2021677720543,"lng":-8.71510600504109},{"lat":41.2022485490326,"lng":-8.71515786806378},{"lat":41.2023194246764,"lng":-8.71522741110755},{"lat":41.2022234945453,"lng":-8.71533994101678},{"lat":41.2022335039166,"lng":-8.71546175365615},{"lat":41.2022994331366,"lng":-8.71561864683355},{"lat":41.2023999044409,"lng":-8.71567533596611},{"lat":41.2025181563361,"lng":-8.71565289719992},{"lat":41.2024725994431,"lng":-8.71575812694071},{"lat":41.2025237691016,"lng":-8.71577146015631},{"lat":41.2026104056145,"lng":-8.71571559598194},{"lat":41.2026990716092,"lng":-8.71565045598841},{"lat":41.2027084607011,"lng":-8.71575164461876},{"lat":41.2026486687434,"lng":-8.715823237228},{"lat":41.2026678225789,"lng":-8.71593464586638},{"lat":41.2027209576638,"lng":-8.71611097347248},{"lat":41.2027753711653,"lng":-8.71621184359966},{"lat":41.202839929067,"lng":-8.71628383706239},{"lat":41.2029380596595,"lng":-8.71630497735889},{"lat":41.2029897001221,"lng":-8.71622555748945},{"lat":41.2029940277724,"lng":-8.71609957385263},{"lat":41.2030528255579,"lng":-8.71620704570404},{"lat":41.2030954878414,"lng":-8.7163351135052},{"lat":41.2031806491087,"lng":-8.71640931628155},{"lat":41.2032890362773,"lng":-8.71632599121109},{"lat":41.2033779732275,"lng":-8.71633143355104},{"lat":41.2034603452944,"lng":-8.71628208707674},{"lat":41.2035804691202,"lng":-8.71617538149046},{"lat":41.2036353037877,"lng":-8.71619327157816},{"lat":41.2037239767002,"lng":-8.71612681882362},{"lat":41.2037692604627,"lng":-8.71607547608459},{"lat":41.2038589286917,"lng":-8.71598936482426},{"lat":41.2039184824295,"lng":-8.71578720797051},{"lat":41.2038991107595,"lng":-8.7156661416504},{"lat":41.2039610355902,"lng":-8.71570675295773},{"lat":41.2040401355954,"lng":-8.71559215884639},{"lat":41.2040789944748,"lng":-8.71536895735797},{"lat":41.2041012219052,"lng":-8.71526518961924},{"lat":41.2041713988664,"lng":-8.71524124954725},{"lat":41.2041935535486,"lng":-8.71531167403591},{"lat":41.20420464014,"lng":-8.71545126832268},{"lat":41.2042295315336,"lng":-8.71553268344621},{"lat":41.2042029370721,"lng":-8.71578711418323},{"lat":41.2041506120738,"lng":-8.71578927353112},{"lat":41.204173751215,"lng":-8.71587782845174},{"lat":41.2042257913995,"lng":-8.71587924430757},{"lat":41.204231869037,"lng":-8.71595869999204},{"lat":41.2042159134781,"lng":-8.71605095774165},{"lat":41.2042896638826,"lng":-8.71610419341445},{"lat":41.204311393792,"lng":-8.7162581935114},{"lat":41.2043616341085,"lng":-8.71627616954232},{"lat":41.2043275828322,"lng":-8.71640343668027},{"lat":41.2043601896303,"lng":-8.7165606318659},{"lat":41.2043910354384,"lng":-8.71669300761486},{"lat":41.2044376327967,"lng":-8.71671989240154},{"lat":41.2045093364826,"lng":-8.71662526647749},{"lat":41.2045744966191,"lng":-8.71654298736051},{"lat":41.2046648332077,"lng":-8.71648584297018},{"lat":41.2047134772271,"lng":-8.71648127290332},{"lat":41.204739309376,"lng":-8.71637729680734},{"lat":41.2047560335294,"lng":-8.7162585756191},{"lat":41.2048155855073,"lng":-8.71626899961275},{"lat":41.2048664542539,"lng":-8.716341586338},{"lat":41.2049285147897,"lng":-8.71637325916362},{"lat":41.2049227677816,"lng":-8.71644081129913},{"lat":41.2048791110065,"lng":-8.71650861861697},{"lat":41.2048679667404,"lng":-8.71659365134484},{"lat":41.2048325910628,"lng":-8.71680424728111},{"lat":41.2049102197146,"lng":-8.71692595499329},{"lat":41.2050026912504,"lng":-8.71699724770427},{"lat":41.2050402320117,"lng":-8.71685772520193},{"lat":41.2050741654225,"lng":-8.71693099443798},{"lat":41.2051248216145,"lng":-8.7170453134406},{"lat":41.205127842306,"lng":-8.71717660996427},{"lat":41.2051499036408,"lng":-8.7172651570771},{"lat":41.2052099137756,"lng":-8.71725769752754},{"lat":41.2052411336957,"lng":-8.7171916848871},{"lat":41.2053132418559,"lng":-8.71717790392521},{"lat":41.2054065057961,"lng":-8.71711113200536},{"lat":41.2057749770851,"lng":-8.71707376392262},{"lat":41.2059610484893,"lng":-8.7171022466222},{"lat":41.2060986252966,"lng":-8.7171573667437},{"lat":41.2062660889492,"lng":-8.71716124567873},{"lat":41.2064182676191,"lng":-8.7171609374676},{"lat":41.2065287216626,"lng":-8.71718481526571},{"lat":41.2066209134423,"lng":-8.71715177181973},{"lat":41.2067261349274,"lng":-8.71715974606791},{"lat":41.2068403141385,"lng":-8.71717734107696},{"lat":41.2069414715351,"lng":-8.71715236589853},{"lat":41.2070926623965,"lng":-8.71713356677543},{"lat":41.2072480663588,"lng":-8.71706496200421},{"lat":41.2073544991181,"lng":-8.71699401468402},{"lat":41.2075098301918,"lng":-8.71692290350686},{"lat":41.2077004405535,"lng":-8.7168017966598},{"lat":41.2079242847853,"lng":-8.71671436545979},{"lat":41.208100065242,"lng":-8.71667574945795},{"lat":41.2082317904625,"lng":-8.71660705148025},{"lat":41.2083394854665,"lng":-8.71651835075075},{"lat":41.2086057741188,"lng":-8.71637191732017},{"lat":41.2088755168178,"lng":-8.71623588745179},{"lat":41.2091112601499,"lng":-8.71612769081676},{"lat":41.2093367638689,"lng":-8.71605124397453},{"lat":41.2095089948326,"lng":-8.71600209980246},{"lat":41.2096353181293,"lng":-8.71595076103111},{"lat":41.2097911189562,"lng":-8.71591089144832},{"lat":41.209931713717,"lng":-8.71592168430301},{"lat":41.2101862693951,"lng":-8.71593956832774},{"lat":41.2103439364329,"lng":-8.71592260889309},{"lat":41.2105270611054,"lng":-8.71587440016613},{"lat":41.2109683048527,"lng":-8.71588750963726},{"lat":41.2112215952371,"lng":-8.71595939981636},{"lat":41.2113686893162,"lng":-8.7159139681326},{"lat":41.2115202547624,"lng":-8.7158391253058},{"lat":41.2116092437233,"lng":-8.71586900841769},{"lat":41.2117239857035,"lng":-8.7158292517604},{"lat":41.2118933566971,"lng":-8.71577650175091},{"lat":41.2120855506626,"lng":-8.71573278323248},{"lat":41.2122365011075,"lng":-8.7157613122967},{"lat":41.2123553778435,"lng":-8.71579420688819},{"lat":41.2124793496552,"lng":-8.71581558636889},{"lat":41.2126038321209,"lng":-8.71578986183241},{"lat":41.2129541228017,"lng":-8.71578630326879},{"lat":41.2133041120732,"lng":-8.71586000900609},{"lat":41.2135394217604,"lng":-8.71596215281479},{"lat":41.2135640942292,"lng":-8.7160866225354},{"lat":41.2136606754903,"lng":-8.71616415706202},{"lat":41.2136241455394,"lng":-8.71626530675394},{"lat":41.213572668632,"lng":-8.71634819905144},{"lat":41.2136711683468,"lng":-8.71642145991371},{"lat":41.2138070684228,"lng":-8.71643423840387},{"lat":41.2139276508252,"lng":-8.71644926591219},{"lat":41.2140293468417,"lng":-8.71644301577171},{"lat":41.2140462859823,"lng":-8.71631665024466},{"lat":41.2140361459878,"lng":-8.71620291697723},{"lat":41.2140998190551,"lng":-8.71607648782057},{"lat":41.2141730843792,"lng":-8.71599426842642},{"lat":41.2143210118874,"lng":-8.71596207568842},{"lat":41.2144781195467,"lng":-8.71605433625557},{"lat":41.2146299764796,"lng":-8.71608168915452},{"lat":41.2147139052947,"lng":-8.71606263970636},{"lat":41.2149680306796,"lng":-8.71600563851035},{"lat":41.2151020214198,"lng":-8.71604069525196},{"lat":41.2150925734065,"lng":-8.71612766212693},{"lat":41.2150101306352,"lng":-8.71624450384317},{"lat":41.2149951955357,"lng":-8.71636564498887},{"lat":41.2150313878385,"lng":-8.71649153299004},{"lat":41.2151287085997,"lng":-8.71665422178226},{"lat":41.2152012198549,"lng":-8.71673834111183},{"lat":41.2152782742334,"lng":-8.71663635777385},{"lat":41.215338074236,"lng":-8.71654542833188},{"lat":41.2154425064288,"lng":-8.71653145301291},{"lat":41.2155763328481,"lng":-8.7164571619791},{"lat":41.2155964209527,"lng":-8.7163844758697},{"lat":41.215706542544,"lng":-8.71634896789883},{"lat":41.2157720999727,"lng":-8.71636577100023},{"lat":41.2158596268012,"lng":-8.71636547811091},{"lat":41.2159093471348,"lng":-8.7163272861277},{"lat":41.2159944661946,"lng":-8.71623336262129},{"lat":41.2160789555469,"lng":-8.71622779163779},{"lat":41.2161362818235,"lng":-8.71628709470031},{"lat":41.2161593036714,"lng":-8.71636397712272},{"lat":41.2161993353797,"lng":-8.71646008890606},{"lat":41.2162393442664,"lng":-8.71654284686168},{"lat":41.2162800487538,"lng":-8.71650648021131},{"lat":41.2163219767026,"lng":-8.71644221746404},{"lat":41.2164533616762,"lng":-8.71645829478357},{"lat":41.2165044229783,"lng":-8.71647508893883},{"lat":41.2166663138207,"lng":-8.71635263575276},{"lat":41.2167146338643,"lng":-8.71625229948903},{"lat":41.2167619704452,"lng":-8.71629314260629},{"lat":41.2167726308707,"lng":-8.71635787448999},{"lat":41.2168138117529,"lng":-8.7163872147943},{"lat":41.2169253633901,"lng":-8.71642481877522},{"lat":41.2169564631953,"lng":-8.71629416253751},{"lat":41.2170237658976,"lng":-8.71633984162254},{"lat":41.2170721392046,"lng":-8.71638856398146},{"lat":41.2171210930608,"lng":-8.71634178173153},{"lat":41.2171125058018,"lng":-8.71625918278849},{"lat":41.2171815446359,"lng":-8.71629986636082},{"lat":41.2172440847462,"lng":-8.71621921375797},{"lat":41.2173460376865,"lng":-8.7162148711003},{"lat":41.217374263395,"lng":-8.71629549961717},{"lat":41.2173610166899,"lng":-8.7164220243985},{"lat":41.2173901284946,"lng":-8.71655799595751},{"lat":41.217450784167,"lng":-8.71668887797659},{"lat":41.2175457037914,"lng":-8.71677546582744},{"lat":41.2176619735023,"lng":-8.71677077856797},{"lat":41.217713816009,"lng":-8.71681106977199},{"lat":41.217756792903,"lng":-8.71687608425206},{"lat":41.2178994406759,"lng":-8.71678481758827},{"lat":41.2180134383647,"lng":-8.71671333289038},{"lat":41.218153656252,"lng":-8.71658531333497},{"lat":41.2182370970932,"lng":-8.71645034334109},{"lat":41.218353763396,"lng":-8.71636755309401},{"lat":41.218538562649,"lng":-8.7163086195666},{"lat":41.2185917108311,"lng":-8.71632268727592},{"lat":41.2187045714025,"lng":-8.71626203955998},{"lat":41.2188406189235,"lng":-8.71622890650038},{"lat":41.2189352819687,"lng":-8.71620554027044},{"lat":41.2190143828614,"lng":-8.71619789523577},{"lat":41.2192185572297,"lng":-8.71610061623007},{"lat":41.2193347355522,"lng":-8.71606038852202},{"lat":41.2194763446707,"lng":-8.71604972154427},{"lat":41.2196354885551,"lng":-8.71605996871946},{"lat":41.2197922163931,"lng":-8.7160684018375},{"lat":41.2199493067868,"lng":-8.71598655543658},{"lat":41.2200492421662,"lng":-8.71598911814273},{"lat":41.2200864328481,"lng":-8.71609582576113},{"lat":41.2201727289202,"lng":-8.71615956208907},{"lat":41.220264952118,"lng":-8.71622728312861},{"lat":41.220312427911,"lng":-8.71631106558345},{"lat":41.2203688731625,"lng":-8.71615439235998},{"lat":41.2204387321004,"lng":-8.71608703557723},{"lat":41.2205403196768,"lng":-8.71608328906474},{"lat":41.2205778961648,"lng":-8.716166510431},{"lat":41.2206395777008,"lng":-8.71621929232605},{"lat":41.2208232216917,"lng":-8.71622832566841},{"lat":41.2208381634539,"lng":-8.71633578935794},{"lat":41.2208682387339,"lng":-8.71640689585147},{"lat":41.2209228139868,"lng":-8.71629945182846},{"lat":41.2209154483317,"lng":-8.71618930993401},{"lat":41.2209597071815,"lng":-8.71612674257991},{"lat":41.2210235649532,"lng":-8.71617703846321},{"lat":41.2211271869537,"lng":-8.7161630542356},{"lat":41.221202503521,"lng":-8.71619020152238},{"lat":41.2211397934091,"lng":-8.71634002354053},{"lat":41.2211957386383,"lng":-8.71645822918943},{"lat":41.2212313523351,"lng":-8.71637900454063},{"lat":41.2212670701964,"lng":-8.71626042283065},{"lat":41.2213384174868,"lng":-8.71623589510577},{"lat":41.2214180807626,"lng":-8.71627703586121},{"lat":41.2214754620467,"lng":-8.71632548855636},{"lat":41.2215226785905,"lng":-8.71628392564505},{"lat":41.221581946726,"lng":-8.71629769534417},{"lat":41.2215922518664,"lng":-8.71637888245531},{"lat":41.2217020636486,"lng":-8.71642219820708},{"lat":41.2217971169523,"lng":-8.71650032246337},{"lat":41.2218523586066,"lng":-8.71642019914789},{"lat":41.2218680951019,"lng":-8.71653065051284},{"lat":41.2218395007372,"lng":-8.71662962421818},{"lat":41.2217544279957,"lng":-8.71675003238017},{"lat":41.2217980454958,"lng":-8.71681374373802},{"lat":41.2218956779687,"lng":-8.7167555838183},{"lat":41.2219357868392,"lng":-8.71683643746448},{"lat":41.2220174841916,"lng":-8.71686638840918},{"lat":41.2220233835447,"lng":-8.71698176381693},{"lat":41.2221091650631,"lng":-8.71709415350973},{"lat":41.2222226762117,"lng":-8.71708276448768},{"lat":41.2223114792105,"lng":-8.71699077184472},{"lat":41.2223668895355,"lng":-8.71705471575797},{"lat":41.2224228214418,"lng":-8.71701704915015},{"lat":41.222555149697,"lng":-8.71688215390459},{"lat":41.2226256921761,"lng":-8.71696352655},{"lat":41.2227561254529,"lng":-8.71681239320553},{"lat":41.2228095167065,"lng":-8.71693701702449},{"lat":41.2228674525691,"lng":-8.71689412320593},{"lat":41.2228857928639,"lng":-8.71682977052324},{"lat":41.2229622364761,"lng":-8.71684797932379},{"lat":41.2230311282789,"lng":-8.71674030674331},{"lat":41.2230556692309,"lng":-8.71688948461315},{"lat":41.2231608702246,"lng":-8.71707861477042},{"lat":41.223203159322,"lng":-8.7172798326758},{"lat":41.2233007035483,"lng":-8.71741713419958},{"lat":41.2233961494941,"lng":-8.7173991465639},{"lat":41.2234671979365,"lng":-8.71741659492349},{"lat":41.2234909974319,"lng":-8.71749969774558},{"lat":41.223557875855,"lng":-8.71746844637888},{"lat":41.223583416794,"lng":-8.71759890799781},{"lat":41.2237575155617,"lng":-8.71771472260011},{"lat":41.2238245102722,"lng":-8.7178197909009},{"lat":41.2238958897181,"lng":-8.71794911674514},{"lat":41.2240226939757,"lng":-8.71796253242487},{"lat":41.2241475349126,"lng":-8.71802554666931},{"lat":41.2241218979413,"lng":-8.71791404296144},{"lat":41.2240565676431,"lng":-8.71778251154599},{"lat":41.2240485346953,"lng":-8.71746758320379},{"lat":41.2240363646327,"lng":-8.71736443057384},{"lat":41.2240999211412,"lng":-8.71743822583067},{"lat":41.2241691039723,"lng":-8.71748512085357},{"lat":41.2242729701423,"lng":-8.71744096404215},{"lat":41.2243814564039,"lng":-8.71751528086805},{"lat":41.2243959759334,"lng":-8.71765208347443},{"lat":41.2244405653594,"lng":-8.71773727931807},{"lat":41.2244874128074,"lng":-8.71769715178008},{"lat":41.2244603032979,"lng":-8.71760996611221},{"lat":41.224578028865,"lng":-8.71765633554907},{"lat":41.2246154409885,"lng":-8.71770044149904},{"lat":41.2245770964784,"lng":-8.71783951797048},{"lat":41.2246181882459,"lng":-8.71788627601477},{"lat":41.2246482490769,"lng":-8.7178355020431},{"lat":41.2246719855765,"lng":-8.71777178923142},{"lat":41.2247180345461,"lng":-8.71772938188275},{"lat":41.2247429186037,"lng":-8.71781189630139},{"lat":41.2247773404166,"lng":-8.7179126232165},{"lat":41.2247833528447,"lng":-8.71800558550234},{"lat":41.2248801967805,"lng":-8.71804903013834},{"lat":41.2248763890069,"lng":-8.71791243546333},{"lat":41.224869618404,"lng":-8.71784583072306},{"lat":41.2249119975445,"lng":-8.71781662605428},{"lat":41.224942203109,"lng":-8.71791433655157},{"lat":41.2250287472927,"lng":-8.71771760946747},{"lat":41.2251095611014,"lng":-8.71760276336008},{"lat":41.2251095611014,"lng":-8.71760276336008},{"lat":41.225143347266,"lng":-8.71765244204899},{"lat":41.2251413276455,"lng":-8.71771313045696},{"lat":41.2251166651746,"lng":-8.71779961454184},{"lat":41.225087383871,"lng":-8.71785540751615},{"lat":41.2250724556779,"lng":-8.71797584716389},{"lat":41.2251155874122,"lng":-8.71801130099383},{"lat":41.2251856596581,"lng":-8.71800798866597},{"lat":41.2253954849162,"lng":-8.71803836820748},{"lat":41.2254482421788,"lng":-8.71811135601012},{"lat":41.2255395792858,"lng":-8.71819375259794},{"lat":41.2255743292561,"lng":-8.71810651856707},{"lat":41.2256456482978,"lng":-8.71803523958671},{"lat":41.2256942612531,"lng":-8.71803674713975},{"lat":41.2257326050262,"lng":-8.71812808709089},{"lat":41.2257934004805,"lng":-8.71801974583052},{"lat":41.2258686627218,"lng":-8.71793394645128},{"lat":41.2258653874897,"lng":-8.71786963787527},{"lat":41.2258146135457,"lng":-8.71777866},{"lat":41.2258264640517,"lng":-8.7177151930642},{"lat":41.2258378105406,"lng":-8.71764397743289},{"lat":41.2258870426619,"lng":-8.71770178545901},{"lat":41.2259251914994,"lng":-8.71777809523492},{"lat":41.2259626418367,"lng":-8.71767430991084},{"lat":41.2259314419422,"lng":-8.71761177782206},{"lat":41.2258796385259,"lng":-8.71754607605609},{"lat":41.2259571648009,"lng":-8.71752947406306},{"lat":41.2260012209885,"lng":-8.71756027783259},{"lat":41.2260430512847,"lng":-8.71762111883069},{"lat":41.2260852818517,"lng":-8.71767349951288},{"lat":41.2261128185107,"lng":-8.71774792858472},{"lat":41.2261464443415,"lng":-8.71782909001179},{"lat":41.2261272042256,"lng":-8.71789415732914},{"lat":41.2260802965893,"lng":-8.71794609474657},{"lat":41.2261041508595,"lng":-8.7180005773485},{"lat":41.226056252651,"lng":-8.71807016018291},{"lat":41.2260115728324,"lng":-8.71816183130442},{"lat":41.2259487349033,"lng":-8.71830069139718},{"lat":41.2258847548869,"lng":-8.71848689244082},{"lat":41.2258810907644,"lng":-8.71865788462963},{"lat":41.225830155489,"lng":-8.7187585695261},{"lat":41.2259715528105,"lng":-8.71894732169131},{"lat":41.2261297413183,"lng":-8.71900372052761},{"lat":41.2261740735547,"lng":-8.71913912798186},{"lat":41.2262634095451,"lng":-8.71927827772597},{"lat":41.2263664724432,"lng":-8.71937389790898},{"lat":41.2264768038445,"lng":-8.7194738755215},{"lat":41.2264854818632,"lng":-8.71953823986163},{"lat":41.2264295230322,"lng":-8.71952879446794},{"lat":41.2263620591776,"lng":-8.71951495399724},{"lat":41.2262875715225,"lng":-8.71950104394485},{"lat":41.2261896271703,"lng":-8.71962002817508},{"lat":41.2262206395613,"lng":-8.71971906155818},{"lat":41.2262694135987,"lng":-8.71986535941836},{"lat":41.2263010877481,"lng":-8.71999338316451},{"lat":41.2263880688238,"lng":-8.72009887711054},{"lat":41.2263900825965,"lng":-8.72019907704727},{"lat":41.2263751645597,"lng":-8.72031606638511},{"lat":41.2264070351973,"lng":-8.72040556440806},{"lat":41.2264587605585,"lng":-8.72052064613626},{"lat":41.2265440696926,"lng":-8.72058402581308},{"lat":41.2266341088445,"lng":-8.72056765955725},{"lat":41.2267398950553,"lng":-8.72046508795956},{"lat":41.2267799566854,"lng":-8.72055466009339},{"lat":41.2267509817451,"lng":-8.72060341816251},{"lat":41.226813601941,"lng":-8.72068376920824},{"lat":41.226876140038,"lng":-8.72076245351477},{"lat":41.2269352436738,"lng":-8.72082595842027},{"lat":41.2269971604671,"lng":-8.72088542994915},{"lat":41.227056030779,"lng":-8.72097779754822},{"lat":41.2270823250681,"lng":-8.72111757810449},{"lat":41.2271706397785,"lng":-8.72112159079608},{"lat":41.227219730904,"lng":-8.7212065931483},{"lat":41.2272812133237,"lng":-8.72129779185727},{"lat":41.2273595462482,"lng":-8.72133403873273},{"lat":41.2273869627141,"lng":-8.72123791401453},{"lat":41.2274289646625,"lng":-8.72128253836087},{"lat":41.2274726763834,"lng":-8.72137942258876},{"lat":41.2274633032249,"lng":-8.72145197002516},{"lat":41.2274815811772,"lng":-8.721557449721},{"lat":41.2275434089696,"lng":-8.72163421651162},{"lat":41.2276169628324,"lng":-8.72167221375132},{"lat":41.2276827662859,"lng":-8.72167519164328},{"lat":41.2277405304675,"lng":-8.72166712044641},{"lat":41.227780562292,"lng":-8.72174465127697},{"lat":41.2278052054535,"lng":-8.72183825899603},{"lat":41.2278441923281,"lng":-8.72196158056681},{"lat":41.2278526327801,"lng":-8.72207102461573},{"lat":41.2279178715195,"lng":-8.72202712480218},{"lat":41.2279742632968,"lng":-8.72205709050371},{"lat":41.2280462635304,"lng":-8.7220644166754},{"lat":41.2281379518298,"lng":-8.722129770126},{"lat":41.2281723368849,"lng":-8.72223790207355},{"lat":41.2282087451219,"lng":-8.72237240480786},{"lat":41.2283348963799,"lng":-8.72251296796385},{"lat":41.2284430159127,"lng":-8.72251788083801},{"lat":41.2285009274776,"lng":-8.72246246497374},{"lat":41.2284896827773,"lng":-8.72235537386702},{"lat":41.2284515332939,"lng":-8.72222681320198},{"lat":41.2284846893771,"lng":-8.72211738270516},{"lat":41.2285227825965,"lng":-8.72220396135387},{"lat":41.2286780436741,"lng":-8.72249817086514},{"lat":41.2287526770225,"lng":-8.72253557963618},{"lat":41.2288411336718,"lng":-8.72245896462452},{"lat":41.2288238293563,"lng":-8.72232176671347},{"lat":41.2288075869541,"lng":-8.72220509274775},{"lat":41.2288868704762,"lng":-8.7223198284661},{"lat":41.2289719313204,"lng":-8.72241374890137},{"lat":41.2290171747957,"lng":-8.72254630686353},{"lat":41.229072621007,"lng":-8.72263793482192},{"lat":41.2292103450845,"lng":-8.72262843870737},{"lat":41.2292756801319,"lng":-8.72256581114968},{"lat":41.2293499139329,"lng":-8.72262910311639},{"lat":41.2294714838694,"lng":-8.7226269793864},{"lat":41.2296030841165,"lng":-8.72261754799103},{"lat":41.2296568375906,"lng":-8.72267134490777},{"lat":41.2297416887669,"lng":-8.72259529639474},{"lat":41.2297883320863,"lng":-8.72266585159298},{"lat":41.2298144148611,"lng":-8.72284642606281},{"lat":41.2298633125507,"lng":-8.72279343411406},{"lat":41.2299150866148,"lng":-8.72284662036922},{"lat":41.2299773560405,"lng":-8.72299496332646},{"lat":41.2300643354071,"lng":-8.72288959495802},{"lat":41.2301762724136,"lng":-8.72283501829221},{"lat":41.2302454323767,"lng":-8.72269382457428},{"lat":41.2302497467785,"lng":-8.72255514553324},{"lat":41.2301799009367,"lng":-8.72251432654901},{"lat":41.230148734375,"lng":-8.72244581662466},{"lat":41.2302407438445,"lng":-8.72239655157279},{"lat":41.2304078834338,"lng":-8.72244576498289},{"lat":41.2304811415885,"lng":-8.72238344741826},{"lat":41.2305131352998,"lng":-8.72228975253709},{"lat":41.2305101207807,"lng":-8.72221040532251},{"lat":41.2308012340777,"lng":-8.72212237571495},{"lat":41.2310734064975,"lng":-8.72211122946712},{"lat":41.2313187656055,"lng":-8.7221667590799},{"lat":41.2314778713387,"lng":-8.72225490450897},{"lat":41.2317004983217,"lng":-8.72235244710785},{"lat":41.2318491686281,"lng":-8.7224572023138},{"lat":41.2319683079194,"lng":-8.72259229187527},{"lat":41.2335032668454,"lng":-8.71752981594723},{"lat":41.2362445112095,"lng":-8.70847399366566},{"lat":41.2370212333012,"lng":-8.70378571330317},{"lat":41.2377433083129,"lng":-8.70343415481335},{"lat":41.2386602101758,"lng":-8.70010214090155},{"lat":41.2385980350808,"lng":-8.69622357641352},{"lat":41.2381778072448,"lng":-8.69628272298335},{"lat":41.2372372855627,"lng":-8.69632232935868},{"lat":41.2350552749517,"lng":-8.69605556820955},{"lat":41.2343325930763,"lng":-8.69615792833876},{"lat":41.2336201072017,"lng":-8.69687593735683},{"lat":41.2329500384464,"lng":-8.69721951937033},{"lat":41.232529811234,"lng":-8.69680506796807},{"lat":41.2323392749301,"lng":-8.69550253780104},{"lat":41.2321262438731,"lng":-8.69337088458373},{"lat":41.2317695402543,"lng":-8.68571197967668},{"lat":41.2305738851475,"lng":-8.68052952068546},{"lat":41.2259570883627,"lng":-8.67103017488194},{"lat":41.2238887077696,"lng":-8.66872640668575},{"lat":41.2231935414476,"lng":-8.66834614900684},{"lat":41.2226317748646,"lng":-8.66702273771205},{"lat":41.2224141059198,"lng":-8.66615516893182},{"lat":41.2227047131627,"lng":-8.66555928469945},{"lat":41.2224633047771,"lng":-8.66428814970679},{"lat":41.2230430607429,"lng":-8.66301271442247},{"lat":41.2224017179571,"lng":-8.66185329869589},{"lat":41.2229099746521,"lng":-8.66065533312798},{"lat":41.2248975453497,"lng":-8.66082641229288},{"lat":41.2257297171842,"lng":-8.66051627003189},{"lat":41.2267337041273,"lng":-8.6574995780653},{"lat":41.2280270675281,"lng":-8.6559687468274},{"lat":41.2282903121318,"lng":-8.65534811306683},{"lat":41.2282734859569,"lng":-8.65453723000805},{"lat":41.2274444431411,"lng":-8.65373004666163},{"lat":41.2251918658646,"lng":-8.65406992295463},{"lat":41.222048084554,"lng":-8.65237533660137},{"lat":41.2217208037336,"lng":-8.6521677366658},{"lat":41.2208432319432,"lng":-8.65230596368609},{"lat":41.2204160611788,"lng":-8.65256825169482},{"lat":41.2205418311528,"lng":-8.65333968680584},{"lat":41.2205664330149,"lng":-8.65416579088788},{"lat":41.2205355057277,"lng":-8.65469451229657},{"lat":41.2201421410413,"lng":-8.65569928884951},{"lat":41.219821910956,"lng":-8.65567197806073},{"lat":41.2190955350042,"lng":-8.65581226768047},{"lat":41.2186999005472,"lng":-8.65604553135574},{"lat":41.2182285156377,"lng":-8.65704642610051},{"lat":41.2179725281544,"lng":-8.65758995230108},{"lat":41.2171687019972,"lng":-8.65818031679319},{"lat":41.2165270114758,"lng":-8.65865158996939},{"lat":41.2162661605047,"lng":-8.65929975797336},{"lat":41.2163421368696,"lng":-8.66021327340642},{"lat":41.2165665660562,"lng":-8.66061462579858},{"lat":41.2171615303379,"lng":-8.66087228586437},{"lat":41.2175959896424,"lng":-8.66116436519654},{"lat":41.2173231332197,"lng":-8.66175840118238},{"lat":41.2171610513019,"lng":-8.66255057217179},{"lat":41.217140120391,"lng":-8.66326718833839},{"lat":41.2173431513933,"lng":-8.6639504734781},{"lat":41.2176711860609,"lng":-8.66438465967642},{"lat":41.2181356573138,"lng":-8.66518886019604},{"lat":41.2183083992698,"lng":-8.6658894883112},{"lat":41.2187805776164,"lng":-8.66707536037566},{"lat":41.2188145716748,"lng":-8.66787475792763},{"lat":41.2186861570053,"lng":-8.66845640047944},{"lat":41.218407596576,"lng":-8.6688869782377},{"lat":41.2179208154833,"lng":-8.66879886607164},{"lat":41.217199219284,"lng":-8.66788081988233},{"lat":41.217031548523,"lng":-8.66744400569586},{"lat":41.2165509387365,"lng":-8.66749310140608},{"lat":41.2159256084277,"lng":-8.66738504265862},{"lat":41.2154486395827,"lng":-8.66723132554782},{"lat":41.2148566044245,"lng":-8.66749285637332},{"lat":41.214467168832,"lng":-8.66811740220817},{"lat":41.2140652090747,"lng":-8.66869214188756},{"lat":41.2136195233684,"lng":-8.66912913430464},{"lat":41.2133379473958,"lng":-8.66917438123922},{"lat":41.2128775657975,"lng":-8.66899610932684},{"lat":41.2120831082222,"lng":-8.6692014860986},{"lat":41.2117102576764,"lng":-8.66918513897956},{"lat":41.2105431097928,"lng":-8.66922217925785},{"lat":41.2101899026907,"lng":-8.66948562797553},{"lat":41.2097618181853,"lng":-8.66963193835214},{"lat":41.2091928950212,"lng":-8.67032640821632},{"lat":41.208838797938,"lng":-8.67039386028204},{"lat":41.2085512780688,"lng":-8.66991457840169},{"lat":41.2082833902174,"lng":-8.66943546139825},{"lat":41.2078326422731,"lng":-8.66955100227317},{"lat":41.2076447422874,"lng":-8.67074650021533},{"lat":41.2075547229229,"lng":-8.67121793785795},{"lat":41.2074102467268,"lng":-8.67116071270154},{"lat":41.2070110395994,"lng":-8.67050826654524},{"lat":41.2066444200521,"lng":-8.66960641637364},{"lat":41.2060357693186,"lng":-8.66870170809021},{"lat":41.205450266841,"lng":-8.66824303072857},{"lat":41.2051013906168,"lng":-8.66757975636654},{"lat":41.2049591961184,"lng":-8.6678583378552},{"lat":41.2045961054376,"lng":-8.66810826743622},{"lat":41.2037329320824,"lng":-8.66854238379003},{"lat":41.2036568267092,"lng":-8.66906575892375},{"lat":41.2036077936385,"lng":-8.66978129339178},{"lat":41.2035769723314,"lng":-8.67099608799599},{"lat":41.2034793219793,"lng":-8.67137689886667},{"lat":41.203182228206,"lng":-8.6720747331553},{"lat":41.2031246510491,"lng":-8.67269834386249},{"lat":41.2029997583457,"lng":-8.6734547122949},{"lat":41.2030595550043,"lng":-8.67410433477647},{"lat":41.2032955654997,"lng":-8.67470548138972},{"lat":41.2032164550115,"lng":-8.6754871138492},{"lat":41.2030610742266,"lng":-8.67599226754132},{"lat":41.2027954264712,"lng":-8.67679852798762},{"lat":41.2028546523195,"lng":-8.67756465675437},{"lat":41.202594219772,"lng":-8.67786586087087},{"lat":41.2021973746264,"lng":-8.6783248270925},{"lat":41.2018566378339,"lng":-8.67860494163719},{"lat":41.2016911351273,"lng":-8.67888181931526},{"lat":41.2014666572145,"lng":-8.67925735553503},{"lat":41.2011163811829,"lng":-8.67963070026864},{"lat":41.2009507520181,"lng":-8.68009534605321},{"lat":41.2003746584161,"lng":-8.67990089406379},{"lat":41.1999075092396,"lng":-8.68004609039097},{"lat":41.1991210735375,"lng":-8.68070326352301},{"lat":41.1988861562047,"lng":-8.68072720748703},{"lat":41.1985691251797,"lng":-8.68130707146928},{"lat":41.198064015806,"lng":-8.68178549703856},{"lat":41.1975033154816,"lng":-8.68196363723181},{"lat":41.1961755514294,"lng":-8.6823854690479},{"lat":41.1956963738347,"lng":-8.68269766173494},{"lat":41.1950949386008,"lng":-8.68372878813696},{"lat":41.1937035150028,"lng":-8.68611417172874},{"lat":41.1913424701989,"lng":-8.69016139881032},{"lat":41.1883251885845,"lng":-8.69533274916373},{"lat":41.1854858839892,"lng":-8.69959412308604},{"lat":41.1831683311654,"lng":-8.70307197772478},{"lat":41.1848383941784,"lng":-8.70449442373651},{"lat":41.1850679766867,"lng":-8.70468200399828},{"lat":41.1853154989564,"lng":-8.70491968853471},{"lat":41.1855088866329,"lng":-8.705124719068},{"lat":41.1856189446071,"lng":-8.70524689986945},{"lat":41.1861247397175,"lng":-8.70584120946867},{"lat":41.1860766579902,"lng":-8.70594901546652},{"lat":41.1862951355485,"lng":-8.70619563260685},{"lat":41.186257492222,"lng":-8.70630484205482},{"lat":41.1861865013096,"lng":-8.70631173408575},{"lat":41.1856760224505,"lng":-8.70734269497781},{"lat":41.1858203251573,"lng":-8.70751286220443},{"lat":41.1857889364197,"lng":-8.70764893995376},{"lat":41.1857528763895,"lng":-8.70783766746189},{"lat":41.1856047243617,"lng":-8.70809501954981},{"lat":41.185416374893,"lng":-8.70850386186853},{"lat":41.1852353695748,"lng":-8.70884757436714},{"lat":41.1848443573444,"lng":-8.70928646025987},{"lat":41.184541034654,"lng":-8.70961920091458},{"lat":41.1844080049381,"lng":-8.70973400003525},{"lat":41.1843056021087,"lng":-8.7098810177393},{"lat":41.1841043315777,"lng":-8.70995695849572},{"lat":41.1838807448196,"lng":-8.71004807422047},{"lat":41.1838163764342,"lng":-8.70975811264333},{"lat":41.1834683805989,"lng":-8.70984146687573},{"lat":41.1834919449271,"lng":-8.70997254102766},{"lat":41.1838003778524,"lng":-8.7099130426205},{"lat":41.1838222667225,"lng":-8.71003706947667},{"lat":41.1834093352782,"lng":-8.71012378001797},{"lat":41.1833005102946,"lng":-8.71011578864666},{"lat":41.1833107972575,"lng":-8.71018346034249},{"lat":41.1827816663462,"lng":-8.71031384225356},{"lat":41.1826746676297,"lng":-8.71028286260825},{"lat":41.1826771205418,"lng":-8.71035046468173},{"lat":41.1819990570953,"lng":-8.71051588265381},{"lat":41.1819681516462,"lng":-8.71057413240833},{"lat":41.181568055025,"lng":-8.71066893575532},{"lat":41.1814319154057,"lng":-8.71066749624933},{"lat":41.180950450584,"lng":-8.71082653579414},{"lat":41.1809316465391,"lng":-8.71076964044725},{"lat":41.1808620709581,"lng":-8.7106751084666},{"lat":41.1804750774295,"lng":-8.71061663389498},{"lat":41.1801346013289,"lng":-8.71045810550527},{"lat":41.1800237397179,"lng":-8.71028264021951},{"lat":41.1798448630678,"lng":-8.71004139007677},{"lat":41.179797246266,"lng":-8.70984038951775},{"lat":41.1797774980409,"lng":-8.70968480149326},{"lat":41.1797238547075,"lng":-8.70958969320067},{"lat":41.1795973066954,"lng":-8.70952481890414},{"lat":41.1795148360904,"lng":-8.70941480590967},{"lat":41.1793572904698,"lng":-8.70942664700194},{"lat":41.1792082011582,"lng":-8.70936765559416},{"lat":41.1790252326707,"lng":-8.70920550641686},{"lat":41.1787504572455,"lng":-8.70889155501572},{"lat":41.1786088418423,"lng":-8.70874276461183},{"lat":41.1784235025238,"lng":-8.70856856374855},{"lat":41.1783259779546,"lng":-8.70842505001304},{"lat":41.1782134837912,"lng":-8.70841321395635},{"lat":41.1781975855919,"lng":-8.70854846590904},{"lat":41.1782782406514,"lng":-8.70862532765896},{"lat":41.1785815591614,"lng":-8.70897588189751},{"lat":41.1787112544612,"lng":-8.7091497151675},{"lat":41.1785611183307,"lng":-8.70985596498294},{"lat":41.1786451290414,"lng":-8.70998184730447},{"lat":41.1785806216091,"lng":-8.71027565287257},{"lat":41.175021030559,"lng":-8.70884175466369},{"lat":41.1750477612383,"lng":-8.7087187644305},{"lat":41.1748924008923,"lng":-8.70867103778601},{"lat":41.1749421234546,"lng":-8.70841596801541},{"lat":41.1751551934611,"lng":-8.70850161303003},{"lat":41.1751934743146,"lng":-8.70839111688776},{"lat":41.1753462692598,"lng":-8.70844775749916},{"lat":41.1753342205428,"lng":-8.70855181142704},{"lat":41.1754258129195,"lng":-8.70858479374263},{"lat":41.175460868829,"lng":-8.70843505826589},{"lat":41.1749386758121,"lng":-8.70820511573112},{"lat":41.1748987351552,"lng":-8.70830546811081},{"lat":41.1747610844908,"lng":-8.70826409778389},{"lat":41.1748759117523,"lng":-8.70836771926407},{"lat":41.1748319386216,"lng":-8.70866133033184},{"lat":41.1734904325739,"lng":-8.70814212016126},{"lat":41.1733911749625,"lng":-8.70796773728869},{"lat":41.1733618707309,"lng":-8.70808331711576},{"lat":41.1731869121201,"lng":-8.70795808160907},{"lat":41.1731119340591,"lng":-8.70798745702029},{"lat":41.1729612592532,"lng":-8.70788614842502},{"lat":41.1727314301053,"lng":-8.70776592036532},{"lat":41.172644327878,"lng":-8.70807845551101},{"lat":41.1725952542921,"lng":-8.70814976384541},{"lat":41.1726018235701,"lng":-8.70825873809834},{"lat":41.1726810385007,"lng":-8.70828160132119},{"lat":41.1727130240651,"lng":-8.70834909260081},{"lat":41.1728596928641,"lng":-8.70840626981757},{"lat":41.1731031814948,"lng":-8.70849552190095},{"lat":41.173348596068,"lng":-8.70859504006936},{"lat":41.1736325416688,"lng":-8.70869584956899},{"lat":41.1738705962211,"lng":-8.70880912262104},{"lat":41.1740684794478,"lng":-8.70886985493046},{"lat":41.1743287556059,"lng":-8.70902289414287},{"lat":41.1745678636145,"lng":-8.70908755914665},{"lat":41.1746719007051,"lng":-8.70915234958786},{"lat":41.1749839326951,"lng":-8.70927319540907},{"lat":41.1752461904093,"lng":-8.70938918963543},{"lat":41.1754616138564,"lng":-8.70952610974031},{"lat":41.1756288572533,"lng":-8.70955606276249},{"lat":41.1759381058472,"lng":-8.70967498321782},{"lat":41.1760729141159,"lng":-8.70972717388405},{"lat":41.1761160062551,"lng":-8.70977010036185},{"lat":41.1764922286871,"lng":-8.70994764307806},{"lat":41.1766554714355,"lng":-8.70993156506328},{"lat":41.1767540374099,"lng":-8.71002765059194},{"lat":41.1768448549732,"lng":-8.71003548240318},{"lat":41.1769399312589,"lng":-8.71009102086934},{"lat":41.1770569897246,"lng":-8.7100916916531},{"lat":41.1771287194575,"lng":-8.7101521527332},{"lat":41.178052848151,"lng":-8.71047623670528},{"lat":41.1782026853777,"lng":-8.71056551109665},{"lat":41.1782509358549,"lng":-8.71063922612829},{"lat":41.1797156895785,"lng":-8.71121455653392},{"lat":41.1811334926097,"lng":-8.7118152458764},{"lat":41.181162644799,"lng":-8.71194434078353},{"lat":41.1812848485714,"lng":-8.71197736144608},{"lat":41.1814517426177,"lng":-8.71209444679663},{"lat":41.1814970060257,"lng":-8.71204681457848},{"lat":41.1815937897086,"lng":-8.71215672090516},{"lat":41.1816791946951,"lng":-8.71218417558861},{"lat":41.1818079662288,"lng":-8.71220092454696},{"lat":41.1819562072389,"lng":-8.71226719006799},{"lat":41.1820180780449,"lng":-8.71228287517663},{"lat":41.1821487921477,"lng":-8.71225364035395},{"lat":41.182273850319,"lng":-8.71225641050956},{"lat":41.182336619021,"lng":-8.7123085743745},{"lat":41.1824135049949,"lng":-8.71223928936026},{"lat":41.1825114999519,"lng":-8.7122885477954},{"lat":41.1825955154567,"lng":-8.71225126758065},{"lat":41.1827152080665,"lng":-8.71222944104464},{"lat":41.1830460759203,"lng":-8.71218540392339},{"lat":41.1831476874334,"lng":-8.71217701081428},{"lat":41.183313001445,"lng":-8.71208943123557},{"lat":41.1834758564364,"lng":-8.71193663923209},{"lat":41.1836093208274,"lng":-8.71187882168046},{"lat":41.1836331954575,"lng":-8.71182301325918},{"lat":41.1837774720457,"lng":-8.71178233440417},{"lat":41.1839970902426,"lng":-8.71158629698466},{"lat":41.184168848013,"lng":-8.71145300881006},{"lat":41.1843198917325,"lng":-8.71135541626115},{"lat":41.1844073527827,"lng":-8.71113532521096},{"lat":41.1845277659354,"lng":-8.71098822828834},{"lat":41.1845426416287,"lng":-8.71093079422205},{"lat":41.1846085257494,"lng":-8.71086486502078},{"lat":41.1846729683116,"lng":-8.71076340336088},{"lat":41.1847386955463,"lng":-8.71060355486291},{"lat":41.1848287227663,"lng":-8.71042722927039},{"lat":41.1849409034664,"lng":-8.71028709095988},{"lat":41.1849828926294,"lng":-8.71020998729881},{"lat":41.1850648216062,"lng":-8.71003299644396},{"lat":41.1853566060792,"lng":-8.7094870402789},{"lat":41.1856656620601,"lng":-8.70889081148265},{"lat":41.1858927959621,"lng":-8.70848719141459},{"lat":41.1860149030929,"lng":-8.70821614300514},{"lat":41.1862482449128,"lng":-8.70781328559528},{"lat":41.1863791457881,"lng":-8.70756638933862},{"lat":41.186419395665,"lng":-8.70745839549531},{"lat":41.1865085117409,"lng":-8.70724796452665},{"lat":41.1866348042141,"lng":-8.70720210763295},{"lat":41.1866840564742,"lng":-8.70709490468685},{"lat":41.1867618827467,"lng":-8.70699892177768},{"lat":41.1868734694689,"lng":-8.70686806648427},{"lat":41.186960873216,"lng":-8.7068199631854},{"lat":41.1870352585417,"lng":-8.70690917362427},{"lat":41.1871182708997,"lng":-8.70701896779201},{"lat":41.1873109469836,"lng":-8.70702327305101},{"lat":41.1874683511225,"lng":-8.70705778838925},{"lat":41.187571508109,"lng":-8.70713676801364},{"lat":41.1876833121476,"lng":-8.70712559182584},{"lat":41.1878699541219,"lng":-8.70705761629427},{"lat":41.1879385476647,"lng":-8.70706226684639},{"lat":41.1882934994566,"lng":-8.70699111590691},{"lat":41.1885908577691,"lng":-8.7069493798799},{"lat":41.1887413987718,"lng":-8.70695165085584},{"lat":41.1889141396533,"lng":-8.70694481784135},{"lat":41.1890212422447,"lng":-8.70697329157113},{"lat":41.1892327418401,"lng":-8.7069946935307},{"lat":41.1894608473068,"lng":-8.70695437530595},{"lat":41.1896429395555,"lng":-8.70689565621007},{"lat":41.1898033536265,"lng":-8.70686952507572},{"lat":41.1899339338627,"lng":-8.70686577954583},{"lat":41.1900542461928,"lng":-8.70689997110597},{"lat":41.1900825745627,"lng":-8.70701477324504},{"lat":41.1901613275028,"lng":-8.70709485438198},{"lat":41.1902301661615,"lng":-8.70722967477395},{"lat":41.1903153023074,"lng":-8.70729300680126},{"lat":41.1903865273678,"lng":-8.70729363102253},{"lat":41.1904466881219,"lng":-8.70730929190992},{"lat":41.1905098855578,"lng":-8.70727611481332},{"lat":41.1905210937245,"lng":-8.70717691844582},{"lat":41.1905920084239,"lng":-8.70722152204375},{"lat":41.1906860548333,"lng":-8.70715916702923},{"lat":41.1907205810281,"lng":-8.70711608007453},{"lat":41.1907969754636,"lng":-8.70714452563041},{"lat":41.1908604384991,"lng":-8.70722029738441},{"lat":41.1909021253666,"lng":-8.70716702185181},{"lat":41.190997056495,"lng":-8.70709073277894},{"lat":41.1910422705313,"lng":-8.70705154930422},{"lat":41.1911151484553,"lng":-8.70704563333861},{"lat":41.1911645109073,"lng":-8.70707849154155},{"lat":41.1912530433356,"lng":-8.70721837574593},{"lat":41.1912336567925,"lng":-8.70731392456518},{"lat":41.1912828892702,"lng":-8.70739088446743},{"lat":41.1913148283932,"lng":-8.70730569692183},{"lat":41.1913788049056,"lng":-8.70738743555094},{"lat":41.1914040645143,"lng":-8.70728764831564}]},{"zone_id":"mai1","points":[{"lat":41.2167316394507,"lng":-8.59718367410625},{"lat":41.2160366880121,"lng":-8.59832952145624},{"lat":41.2153384753105,"lng":-8.59786901679853},{"lat":41.2146930590305,"lng":-8.59652873195523},{"lat":41.2140179489085,"lng":-8.5966182309243},{"lat":41.2141196717801,"lng":-8.5978355872785},{"lat":41.2129684284014,"lng":-8.59814190857682},{"lat":41.212521986716,"lng":-8.5998492534856},{"lat":41.2117590141938,"lng":-8.60009402788782},{"lat":41.2109731678151,"lng":-8.5998052451016},{"lat":41.2101457873643,"lng":-8.59997391693863},{"lat":41.2089276495702,"lng":-8.6008827625164},{"lat":41.208387700214,"lng":-8.60080811167497},{"lat":41.2074957398707,"lng":-8.60170155376835},{"lat":41.2065096375041,"lng":-8.60146126098633},{"lat":41.2056811729915,"lng":-8.60078990749096},{"lat":41.2043568280856,"lng":-8.60138346286483},{"lat":41.2041369810728,"lng":-8.60209500027284},{"lat":41.2036328342235,"lng":-8.60255542582819},{"lat":41.2027260428034,"lng":-8.60191109858588},{"lat":41.201106307955,"lng":-8.60166107488511},{"lat":41.2006786456382,"lng":-8.60074413063179},{"lat":41.1994909409004,"lng":-8.60033816135654},{"lat":41.1986849452527,"lng":-8.59925949422843},{"lat":41.198511626813,"lng":-8.5975892781287},{"lat":41.1969799922202,"lng":-8.59781684328686},{"lat":41.1961754150158,"lng":-8.59638063202045},{"lat":41.1952735774674,"lng":-8.59673187891375},{"lat":41.1840195925732,"lng":-8.59641391817419},{"lat":41.1824399196317,"lng":-8.59618252878179},{"lat":41.1825280771195,"lng":-8.59650045024091},{"lat":41.1828813449457,"lng":-8.59788352167647},{"lat":41.1832961097391,"lng":-8.59950746089051},{"lat":41.183808101084,"lng":-8.60162190732838},{"lat":41.1839600469016,"lng":-8.6023163130343},{"lat":41.1842435104375,"lng":-8.60361182613143},{"lat":41.1846781754553,"lng":-8.6055985266939},{"lat":41.1849676768963,"lng":-8.60743703494867},{"lat":41.1849883501274,"lng":-8.60828120658883},{"lat":41.1848781994225,"lng":-8.60963733385406},{"lat":41.1847333935301,"lng":-8.61000952932108},{"lat":41.1843279745594,"lng":-8.61105155503136},{"lat":41.1838429592593,"lng":-8.61229811407103},{"lat":41.183695246575,"lng":-8.61284882675119},{"lat":41.183441922869,"lng":-8.61413804399977},{"lat":41.1833802167181,"lng":-8.61475262570444},{"lat":41.1833010717259,"lng":-8.61554084554126},{"lat":41.1831910002236,"lng":-8.61663697591417},{"lat":41.1831213058447,"lng":-8.61733096125999},{"lat":41.1830450139461,"lng":-8.61759574920303},{"lat":41.1827692872696,"lng":-8.6181502080896},{"lat":41.1825371189608,"lng":-8.61852878304335},{"lat":41.182146133973,"lng":-8.61916631475128},{"lat":41.1817616099745,"lng":-8.62010080504737},{"lat":41.1818023783936,"lng":-8.62031627667139},{"lat":41.1819087824544,"lng":-8.62087866093551},{"lat":41.1825656746792,"lng":-8.62209849600043},{"lat":41.1824582273645,"lng":-8.62259562394398},{"lat":41.1815661726074,"lng":-8.62274567987148},{"lat":41.1809511768826,"lng":-8.62314204334637},{"lat":41.1805373100728,"lng":-8.62382439791806},{"lat":41.1803811873993,"lng":-8.62504135241453},{"lat":41.1802067197228,"lng":-8.62640117836105},{"lat":41.179996994485,"lng":-8.62828615048194},{"lat":41.1798438606616,"lng":-8.62977827813638},{"lat":41.1797380124922,"lng":-8.63113164928929},{"lat":41.1797483249034,"lng":-8.63224784417801},{"lat":41.1798118176388,"lng":-8.63269536046027},{"lat":41.1804564822646,"lng":-8.63385533379247},{"lat":41.181505500009,"lng":-8.63574297165251},{"lat":41.1831038144482,"lng":-8.63861925693161},{"lat":41.184143007609,"lng":-8.64048950745177},{"lat":41.1846758732874,"lng":-8.6416106399274},{"lat":41.184708806095,"lng":-8.64246883588769},{"lat":41.1842488833915,"lng":-8.64375973525135},{"lat":41.183330978309,"lng":-8.64577176733752},{"lat":41.1861196888784,"lng":-8.64727391326729},{"lat":41.1868319779367,"lng":-8.64787726049747},{"lat":41.1887722652948,"lng":-8.64795223535506},{"lat":41.1890363387855,"lng":-8.64944788763703},{"lat":41.1892574164416,"lng":-8.65046526552968},{"lat":41.1897482880199,"lng":-8.65104385108705},{"lat":41.1917038050677,"lng":-8.652572368278},{"lat":41.1927559315009,"lng":-8.65357991730701},{"lat":41.1923993062019,"lng":-8.65646194923535},{"lat":41.194585405844,"lng":-8.65673491225217},{"lat":41.1962186897418,"lng":-8.65706793628513},{"lat":41.1965832123609,"lng":-8.6571650770664},{"lat":41.198361593201,"lng":-8.65692299214244},{"lat":41.1995886404169,"lng":-8.65628449697664},{"lat":41.2011814374622,"lng":-8.65597305219219},{"lat":41.2031289928289,"lng":-8.65568738985823},{"lat":41.2047743818434,"lng":-8.65559250539438},{"lat":41.209340245349,"lng":-8.65501696789106},{"lat":41.210794616797,"lng":-8.65515819150042},{"lat":41.2122605784305,"lng":-8.65581829769806},{"lat":41.2134782188613,"lng":-8.65588461084605},{"lat":41.214892373616,"lng":-8.65583929543874},{"lat":41.2166713577329,"lng":-8.6549618453456},{"lat":41.2182663538025,"lng":-8.65416394552076},{"lat":41.2193431591707,"lng":-8.65318864815172},{"lat":41.2204160611788,"lng":-8.65256825169482},{"lat":41.2208432319432,"lng":-8.65230596368609},{"lat":41.2217208037336,"lng":-8.6521677366658},{"lat":41.222048084554,"lng":-8.65237533660137},{"lat":41.2251918658646,"lng":-8.65406992295463},{"lat":41.2274444431411,"lng":-8.65373004666163},{"lat":41.2282734859569,"lng":-8.65453723000805},{"lat":41.2291314217971,"lng":-8.65422410067458},{"lat":41.2300254117585,"lng":-8.65371400841759},{"lat":41.2314583905856,"lng":-8.65305348212308},{"lat":41.2343474646539,"lng":-8.65353698966911},{"lat":41.236252162947,"lng":-8.65319706294858},{"lat":41.2394474326728,"lng":-8.64962331565954},{"lat":41.2401085847947,"lng":-8.6485391325314},{"lat":41.2403551936062,"lng":-8.64794113713279},{"lat":41.2404022131547,"lng":-8.64758916003502},{"lat":41.2403290565107,"lng":-8.64726023444095},{"lat":41.2398603449832,"lng":-8.64661935263393},{"lat":41.2398921612921,"lng":-8.64568217240789},{"lat":41.2397865282249,"lng":-8.64525729958867},{"lat":41.2390091352769,"lng":-8.64453400194629},{"lat":41.2392928274821,"lng":-8.64409609508339},{"lat":41.2397433043014,"lng":-8.64386895628147},{"lat":41.2399354795701,"lng":-8.64266167228321},{"lat":41.239440660754,"lng":-8.6401309752147},{"lat":41.2391853255939,"lng":-8.63843982810748},{"lat":41.2393638684555,"lng":-8.63630734042799},{"lat":41.2399144113857,"lng":-8.63313451033851},{"lat":41.2411050443284,"lng":-8.62994519786481},{"lat":41.2412963013748,"lng":-8.6288080283107},{"lat":41.2410961289333,"lng":-8.62814530697287},{"lat":41.2412991947153,"lng":-8.62703041514883},{"lat":41.2411461919024,"lng":-8.62577505836657},{"lat":41.2407013825324,"lng":-8.62500849730599},{"lat":41.2398750743707,"lng":-8.62476505491994},{"lat":41.2396931681127,"lng":-8.62400694210306},{"lat":41.2400044631389,"lng":-8.61982277657332},{"lat":41.2392771915943,"lng":-8.6171470696314},{"lat":41.2390805126593,"lng":-8.61635064268622},{"lat":41.2391076682933,"lng":-8.61340753672409},{"lat":41.239453374898,"lng":-8.61052935919304},{"lat":41.2398447920316,"lng":-8.6061411004252},{"lat":41.2400720296784,"lng":-8.60356679360359},{"lat":41.2399919816999,"lng":-8.60244508389925},{"lat":41.239370945823,"lng":-8.60089096304777},{"lat":41.2389007084801,"lng":-8.59954420050839},{"lat":41.238317847589,"lng":-8.59777929559913},{"lat":41.2375783557898,"lng":-8.59814727510128},{"lat":41.2371032326317,"lng":-8.59850305146582},{"lat":41.2362500950908,"lng":-8.59892796629664},{"lat":41.2358172451339,"lng":-8.59798829020682},{"lat":41.2342045073374,"lng":-8.59514364162544},{"lat":41.2339159709406,"lng":-8.59555317950123},{"lat":41.2335622416973,"lng":-8.59610842667436},{"lat":41.2332039624009,"lng":-8.5966973404149},{"lat":41.2328849788626,"lng":-8.59722862976535},{"lat":41.2319933675686,"lng":-8.59776822296904},{"lat":41.2311369841093,"lng":-8.5984183016311},{"lat":41.2308968377575,"lng":-8.59879201074011},{"lat":41.2303617769281,"lng":-8.59956918592401},{"lat":41.2289941919472,"lng":-8.60004853333958},{"lat":41.2276179929891,"lng":-8.60022781975941},{"lat":41.2269345547209,"lng":-8.60029147211553},{"lat":41.2266362934624,"lng":-8.60025532873359},{"lat":41.2263049595565,"lng":-8.60012710665071},{"lat":41.2260228108348,"lng":-8.59988454044856},{"lat":41.2255206630245,"lng":-8.59950483622896},{"lat":41.2247742008339,"lng":-8.59890137116994},{"lat":41.2243000317496,"lng":-8.59869755962256},{"lat":41.2237330640708,"lng":-8.59858562552766},{"lat":41.2233794587416,"lng":-8.59854950990611},{"lat":41.2230421538205,"lng":-8.5985404718378},{"lat":41.2221488010399,"lng":-8.59864920677798},{"lat":41.2217267689038,"lng":-8.59867439021488},{"lat":41.2213080934231,"lng":-8.59871775257746},{"lat":41.2209782619658,"lng":-8.59854110761638},{"lat":41.2203417676533,"lng":-8.59824499401258},{"lat":41.2194489201403,"lng":-8.59778008657249},{"lat":41.2190791470548,"lng":-8.59761242238939},{"lat":41.2186190324265,"lng":-8.5976565014051},{"lat":41.2177039160514,"lng":-8.59756677954963},{"lat":41.2173994934418,"lng":-8.5975255905857},{"lat":41.2171427346286,"lng":-8.59742564380718},{"lat":41.2167316394507,"lng":-8.59718367410625}]},{"zone_id":"mai4","points":[{"lat":41.2365019210912,"lng":-8.56399524105109},{"lat":41.2359234911098,"lng":-8.5630246211629},{"lat":41.2338000579978,"lng":-8.55287210258264},{"lat":41.2314685949942,"lng":-8.55023313988714},{"lat":41.2323836504335,"lng":-8.54618363276479},{"lat":41.2309481581198,"lng":-8.54474330568627},{"lat":41.2308590074405,"lng":-8.54478204748722},{"lat":41.2304070653838,"lng":-8.5449784441709},{"lat":41.2268106065251,"lng":-8.54352463469765},{"lat":41.2244738321755,"lng":-8.54231737784373},{"lat":41.2236643248,"lng":-8.54207379896299},{"lat":41.2244011147528,"lng":-8.53742725410795},{"lat":41.2234144521225,"lng":-8.53634783302296},{"lat":41.222613694511,"lng":-8.53359998897791},{"lat":41.2219883986535,"lng":-8.53216509261329},{"lat":41.2205539643812,"lng":-8.53036754839963},{"lat":41.219026994725,"lng":-8.52928502675193},{"lat":41.2171369790096,"lng":-8.52903509383841},{"lat":41.2142569331354,"lng":-8.52865995023448},{"lat":41.2140144137347,"lng":-8.528801104422},{"lat":41.2123356261738,"lng":-8.52964994424377},{"lat":41.2105861233366,"lng":-8.52957715245871},{"lat":41.2086021098336,"lng":-8.52895260181623},{"lat":41.2074672047378,"lng":-8.52863451887113},{"lat":41.2057447236141,"lng":-8.52831290106707},{"lat":41.2050377331398,"lng":-8.52865125695266},{"lat":41.2040178828334,"lng":-8.52819064056844},{"lat":41.2032958751695,"lng":-8.52792039078201},{"lat":41.2029320766186,"lng":-8.52779404268047},{"lat":41.2015561401112,"lng":-8.52760137346792},{"lat":41.2004377869935,"lng":-8.52691487922484},{"lat":41.199740355887,"lng":-8.52589781789248},{"lat":41.196171150614,"lng":-8.52641270036173},{"lat":41.1946725064332,"lng":-8.52698671715016},{"lat":41.1941418752939,"lng":-8.52768314191441},{"lat":41.1925965946071,"lng":-8.52901472802131},{"lat":41.1917111803378,"lng":-8.53046685059066},{"lat":41.1905618364363,"lng":-8.53185903113475},{"lat":41.189062150434,"lng":-8.53272435460172},{"lat":41.1879170476644,"lng":-8.53289225709913},{"lat":41.1865972879147,"lng":-8.53265103067234},{"lat":41.185275701849,"lng":-8.53293443888017},{"lat":41.1843496772768,"lng":-8.53339512999892},{"lat":41.1832485919761,"lng":-8.53356327837907},{"lat":41.1825442866074,"lng":-8.53355897517788},{"lat":41.1823047335713,"lng":-8.53448707351953},{"lat":41.1813722628043,"lng":-8.5367922583753},{"lat":41.1800794804782,"lng":-8.53826212619819},{"lat":41.1787312619508,"lng":-8.54012487774906},{"lat":41.1774679533796,"lng":-8.54341818318093},{"lat":41.1758468407738,"lng":-8.54630760942387},{"lat":41.173692145086,"lng":-8.54790565592652},{"lat":41.1738867350222,"lng":-8.54863778945577},{"lat":41.1744769032746,"lng":-8.552358715617},{"lat":41.1741961782277,"lng":-8.55471633606333},{"lat":41.1742990995023,"lng":-8.55612487688808},{"lat":41.172618697921,"lng":-8.55869395515977},{"lat":41.1717778279261,"lng":-8.56166929858419},{"lat":41.1698142414839,"lng":-8.56124583435462},{"lat":41.1691531403845,"lng":-8.56218428951323},{"lat":41.1681380984773,"lng":-8.56502402108388},{"lat":41.1694329918366,"lng":-8.56563238923089},{"lat":41.1713032376627,"lng":-8.56646323412942},{"lat":41.1717999577083,"lng":-8.56665558969618},{"lat":41.1728675529925,"lng":-8.56700969130684},{"lat":41.1736907109558,"lng":-8.56733335788768},{"lat":41.1741763143807,"lng":-8.56720322786655},{"lat":41.1756056734561,"lng":-8.56564271258198},{"lat":41.1760262069621,"lng":-8.56556437525337},{"lat":41.1764274415825,"lng":-8.56596099289179},{"lat":41.1770806427874,"lng":-8.56691684431927},{"lat":41.1779789885479,"lng":-8.56818539924072},{"lat":41.1786503354669,"lng":-8.56905023158889},{"lat":41.1794518201679,"lng":-8.56996191293345},{"lat":41.1800678304774,"lng":-8.57088154716688},{"lat":41.1802990341563,"lng":-8.57125067764485},{"lat":41.1803611661882,"lng":-8.57245704559744},{"lat":41.1803386231366,"lng":-8.57347271592017},{"lat":41.1804063938591,"lng":-8.57406416392483},{"lat":41.1809517988848,"lng":-8.57517797616376},{"lat":41.1812241831973,"lng":-8.57577358814504},{"lat":41.1813043005894,"lng":-8.57639535930634},{"lat":41.1812213052581,"lng":-8.57743715126074},{"lat":41.1810415705297,"lng":-8.5791267858716},{"lat":41.1809151939545,"lng":-8.580314670871},{"lat":41.1808118130613,"lng":-8.58128631747989},{"lat":41.1809054455686,"lng":-8.5819706601608},{"lat":41.1810757005337,"lng":-8.58231505074296},{"lat":41.1813714579669,"lng":-8.58304682066402},{"lat":41.1816872017519,"lng":-8.58355202331571},{"lat":41.181798785185,"lng":-8.58412061845497},{"lat":41.1815449902676,"lng":-8.58471656437406},{"lat":41.1808933859287,"lng":-8.58563953878859},{"lat":41.1802769201686,"lng":-8.58661471127986},{"lat":41.1800485873193,"lng":-8.58770044660429},{"lat":41.1800600993701,"lng":-8.58876704744848},{"lat":41.1801906176347,"lng":-8.58983826059556},{"lat":41.1803650420823,"lng":-8.59052701072525},{"lat":41.1807642701122,"lng":-8.59163498927675},{"lat":41.1816199808261,"lng":-8.5936437371898},{"lat":41.182184535331,"lng":-8.59526156170956},{"lat":41.1824399196317,"lng":-8.59618252878179},{"lat":41.1840195925732,"lng":-8.59641391817419},{"lat":41.1952735774674,"lng":-8.59673187891375},{"lat":41.1961754150158,"lng":-8.59638063202045},{"lat":41.1969799922202,"lng":-8.59781684328686},{"lat":41.198511626813,"lng":-8.5975892781287},{"lat":41.1986849452527,"lng":-8.59925949422843},{"lat":41.1994909409004,"lng":-8.60033816135654},{"lat":41.2006786456382,"lng":-8.60074413063179},{"lat":41.201106307955,"lng":-8.60166107488511},{"lat":41.2027260428034,"lng":-8.60191109858588},{"lat":41.2036328342235,"lng":-8.60255542582819},{"lat":41.2041369810728,"lng":-8.60209500027284},{"lat":41.2043568280856,"lng":-8.60138346286483},{"lat":41.2056811729915,"lng":-8.60078990749096},{"lat":41.2065096375041,"lng":-8.60146126098633},{"lat":41.2074957398707,"lng":-8.60170155376835},{"lat":41.208387700214,"lng":-8.60080811167497},{"lat":41.2089276495702,"lng":-8.6008827625164},{"lat":41.2101457873643,"lng":-8.59997391693863},{"lat":41.2109731678151,"lng":-8.5998052451016},{"lat":41.2117590141938,"lng":-8.60009402788782},{"lat":41.212521986716,"lng":-8.5998492534856},{"lat":41.2129684284014,"lng":-8.59814190857682},{"lat":41.2141196717801,"lng":-8.5978355872785},{"lat":41.2140179489085,"lng":-8.5966182309243},{"lat":41.2146930590305,"lng":-8.59652873195523},{"lat":41.2153384753105,"lng":-8.59786901679853},{"lat":41.2160366880121,"lng":-8.59832952145624},{"lat":41.2167316394507,"lng":-8.59718367410625},{"lat":41.2164208862928,"lng":-8.5970116954006},{"lat":41.2162304423618,"lng":-8.59691820673976},{"lat":41.2160111681279,"lng":-8.59677853612965},{"lat":41.2158082705263,"lng":-8.59662166568911},{"lat":41.2154186419896,"lng":-8.59610103287204},{"lat":41.2150965436854,"lng":-8.59569544608964},{"lat":41.2144496032373,"lng":-8.59488566479966},{"lat":41.2144143291007,"lng":-8.59441234391657},{"lat":41.2145513119999,"lng":-8.59423762288668},{"lat":41.2150122144373,"lng":-8.59402701133823},{"lat":41.2154508182981,"lng":-8.59362256984344},{"lat":41.2156997610733,"lng":-8.59346998649089},{"lat":41.2161456787975,"lng":-8.59333194111348},{"lat":41.2163950566184,"lng":-8.59304176420771},{"lat":41.2166995261591,"lng":-8.59280287337789},{"lat":41.2169416317128,"lng":-8.59270451229514},{"lat":41.2173169675239,"lng":-8.59241635126351},{"lat":41.2178732591904,"lng":-8.59218687927592},{"lat":41.2183715675704,"lng":-8.59216346807495},{"lat":41.2188666301445,"lng":-8.59193777419842},{"lat":41.2195236139249,"lng":-8.59159953337537},{"lat":41.219910065643,"lng":-8.59139412605573},{"lat":41.2203036549239,"lng":-8.59128180508628},{"lat":41.2209146759673,"lng":-8.5911523275391},{"lat":41.2214832065759,"lng":-8.59069574565509},{"lat":41.2212343294214,"lng":-8.58984576661489},{"lat":41.221010595055,"lng":-8.58905093765231},{"lat":41.2208491648014,"lng":-8.58666935004245},{"lat":41.2235766010745,"lng":-8.58534513123734},{"lat":41.2241541012694,"lng":-8.58500810595071},{"lat":41.2248936270042,"lng":-8.5842680719996},{"lat":41.2256530853616,"lng":-8.58274887902271},{"lat":41.2287214594936,"lng":-8.58085607773525},{"lat":41.2294101480142,"lng":-8.57810464522552},{"lat":41.2300381576265,"lng":-8.57685897696883},{"lat":41.2305499829535,"lng":-8.57618090739513},{"lat":41.2310516693925,"lng":-8.57551975433162},{"lat":41.2314467920378,"lng":-8.57504395175096},{"lat":41.2316348498216,"lng":-8.57480900377063},{"lat":41.2317167243351,"lng":-8.57426479747019},{"lat":41.2315067051532,"lng":-8.57332042054282},{"lat":41.2293790309508,"lng":-8.56987280992224},{"lat":41.2320440455752,"lng":-8.56717283349841},{"lat":41.2343846199319,"lng":-8.56531556986083},{"lat":41.2362012237805,"lng":-8.56430056843316},{"lat":41.2365019210912,"lng":-8.56399524105109}]},{"zone_id":"vlg1","points":[{"lat":41.241442362907,"lng":-8.49400743514632},{"lat":41.240638658065,"lng":-8.48789961475316},{"lat":41.2152806140713,"lng":-8.49110436629046},{"lat":41.1962360944192,"lng":-8.47657592621016},{"lat":41.1925343842822,"lng":-8.47989418040302},{"lat":41.1878540580845,"lng":-8.47927342979088},{"lat":41.1847214904875,"lng":-8.47943329753867},{"lat":41.18172615526,"lng":-8.47919828306483},{"lat":41.1805148661389,"lng":-8.47798339301606},{"lat":41.1774157511783,"lng":-8.47707312717412},{"lat":41.1747184382457,"lng":-8.47574809647406},{"lat":41.1740899272663,"lng":-8.4751489648805},{"lat":41.1736422041544,"lng":-8.47431245028817},{"lat":41.172479431922,"lng":-8.47168474116369},{"lat":41.1711368653416,"lng":-8.46893703958536},{"lat":41.1702406275569,"lng":-8.46750250177278},{"lat":41.1665606643786,"lng":-8.46343244151535},{"lat":41.1643171433877,"lng":-8.46079980451593},{"lat":41.162166297975,"lng":-8.45721462731388},{"lat":41.1611808743471,"lng":-8.45542256929544},{"lat":41.1604996210519,"lng":-8.45390937557413},{"lat":41.1603966758072,"lng":-8.45395080681382},{"lat":41.1603414457206,"lng":-8.45389811354095},{"lat":41.1602762139249,"lng":-8.45388182882725},{"lat":41.1599423747069,"lng":-8.45444576369919},{"lat":41.1459180073474,"lng":-8.46704678434871},{"lat":41.1458707888132,"lng":-8.46708919946238},{"lat":41.1451909000606,"lng":-8.46769991657099},{"lat":41.1431822987603,"lng":-8.46918247827937},{"lat":41.1407644390299,"lng":-8.47096848909352},{"lat":41.1420892465186,"lng":-8.47415463619615},{"lat":41.1429779407176,"lng":-8.47585375940911},{"lat":41.1434754753692,"lng":-8.47658843727626},{"lat":41.1438679126734,"lng":-8.4771192442932},{"lat":41.1445864096827,"lng":-8.47788214913405},{"lat":41.1450378576577,"lng":-8.47797946227899},{"lat":41.1451525003721,"lng":-8.47818339930095},{"lat":41.1456035860877,"lng":-8.47840268969724},{"lat":41.145899068867,"lng":-8.47919054253754},{"lat":41.1460997929071,"lng":-8.47957123639661},{"lat":41.1463096910707,"lng":-8.48007394891936},{"lat":41.1464906481845,"lng":-8.48061714376094},{"lat":41.146690705118,"lng":-8.4811875925364},{"lat":41.1469485120604,"lng":-8.48174477445915},{"lat":41.1471973051327,"lng":-8.48209857976963},{"lat":41.1474532124313,"lng":-8.48239119447641},{"lat":41.1467104354388,"lng":-8.48336924823919},{"lat":41.1462322064327,"lng":-8.4843722821066},{"lat":41.1459985036905,"lng":-8.48608561389235},{"lat":41.1457553877296,"lng":-8.48786320096069},{"lat":41.1456424090634,"lng":-8.48895847026293},{"lat":41.1454207032259,"lng":-8.4897178260317},{"lat":41.1454164721903,"lng":-8.49012247751771},{"lat":41.1454113420437,"lng":-8.49061293666646},{"lat":41.1453404743286,"lng":-8.49208340261674},{"lat":41.1457470984974,"lng":-8.49437649985008},{"lat":41.1461279092887,"lng":-8.49442755757027},{"lat":41.1464570827834,"lng":-8.49437779505071},{"lat":41.1467591597947,"lng":-8.49421186342183},{"lat":41.1471329271716,"lng":-8.49449755253558},{"lat":41.1475255370645,"lng":-8.49460287382722},{"lat":41.148229004964,"lng":-8.49467120703117},{"lat":41.1489503861111,"lng":-8.49485566395874},{"lat":41.1510727279868,"lng":-8.49487156235589},{"lat":41.1569324706516,"lng":-8.49559924996029},{"lat":41.1594222066732,"lng":-8.49572288793818},{"lat":41.1631771714303,"lng":-8.49764686715909},{"lat":41.1650415545832,"lng":-8.49864550788538},{"lat":41.1696261110766,"lng":-8.5006842289575},{"lat":41.1736812560833,"lng":-8.50336864234435},{"lat":41.1739275910958,"lng":-8.50351378371791},{"lat":41.1771061330435,"lng":-8.50538669926103},{"lat":41.1783773345259,"lng":-8.50633221426163},{"lat":41.1792713865404,"lng":-8.50687659826009},{"lat":41.1802825215652,"lng":-8.50746815164142},{"lat":41.1813493914824,"lng":-8.50789246835659},{"lat":41.1819674152267,"lng":-8.50823067698241},{"lat":41.1824272639527,"lng":-8.50861458167313},{"lat":41.1828023047106,"lng":-8.50874688452033},{"lat":41.1832495236543,"lng":-8.50894467743042},{"lat":41.1839782709863,"lng":-8.50964610946221},{"lat":41.1845371053838,"lng":-8.50996541900728},{"lat":41.1852398902148,"lng":-8.5105644582024},{"lat":41.1861583098567,"lng":-8.51171339357158},{"lat":41.186886669088,"lng":-8.51255445947458},{"lat":41.1873388847461,"lng":-8.51322646906199},{"lat":41.1885308042069,"lng":-8.51430030983264},{"lat":41.1887740185309,"lng":-8.51445978364862},{"lat":41.1891653624956,"lng":-8.51461547340286},{"lat":41.1897329472246,"lng":-8.51508579725754},{"lat":41.1929552091212,"lng":-8.51687106684594},{"lat":41.1936608603225,"lng":-8.51726205324221},{"lat":41.1940749159485,"lng":-8.51819904270758},{"lat":41.1952194572533,"lng":-8.51958777045306},{"lat":41.1959764122632,"lng":-8.52159051012298},{"lat":41.1975132412209,"lng":-8.5233511357426},{"lat":41.1987649172728,"lng":-8.52426541640126},{"lat":41.199740355887,"lng":-8.52589781789248},{"lat":41.2004377869935,"lng":-8.52691487922484},{"lat":41.2015561401112,"lng":-8.52760137346792},{"lat":41.2029320766186,"lng":-8.52779404268047},{"lat":41.2032958751695,"lng":-8.52792039078201},{"lat":41.2040178828334,"lng":-8.52819064056844},{"lat":41.2050377331398,"lng":-8.52865125695266},{"lat":41.2057447236141,"lng":-8.52831290106707},{"lat":41.2074672047378,"lng":-8.52863451887113},{"lat":41.2086021098336,"lng":-8.52895260181623},{"lat":41.2105861233366,"lng":-8.52957715245871},{"lat":41.2123356261738,"lng":-8.52964994424377},{"lat":41.2140144137347,"lng":-8.528801104422},{"lat":41.2142569331354,"lng":-8.52865995023448},{"lat":41.2171369790096,"lng":-8.52903509383841},{"lat":41.2181060228281,"lng":-8.52511429524343},{"lat":41.2213181272246,"lng":-8.51749268407556},{"lat":41.2236809428928,"lng":-8.51094731969271},{"lat":41.2275246368861,"lng":-8.50749471432146},{"lat":41.2392199232918,"lng":-8.49704777684847},{"lat":41.2419245495261,"lng":-8.49562091468441},{"lat":41.241442362907,"lng":-8.49400743514632}]},{"zone_id":"prd5","points":[{"lat":41.1580381166742,"lng":-8.37727535920638},{"lat":41.1577186042378,"lng":-8.377558159878},{"lat":41.1576303636107,"lng":-8.37763626112311},{"lat":41.1573288101912,"lng":-8.37791878640835},{"lat":41.1570497928556,"lng":-8.37821286379957},{"lat":41.1567308187292,"lng":-8.37855147366475},{"lat":41.1566860039693,"lng":-8.37859904682965},{"lat":41.1565044662664,"lng":-8.37883035575655},{"lat":41.1560211977484,"lng":-8.37944611030864},{"lat":41.155501570131,"lng":-8.38010844580189},{"lat":41.1551523453102,"lng":-8.38037696483912},{"lat":41.1550395543709,"lng":-8.38040054970977},{"lat":41.1546453556023,"lng":-8.38048297703024},{"lat":41.1542251858827,"lng":-8.38050775459538},{"lat":41.1539600398007,"lng":-8.38052339020166},{"lat":41.1534153077151,"lng":-8.38096830236756},{"lat":41.1530341041012,"lng":-8.3811997544515},{"lat":41.1527677966382,"lng":-8.38131142924739},{"lat":41.1523159046332,"lng":-8.3815009258874},{"lat":41.1521586586098,"lng":-8.38159599302799},{"lat":41.1521697665786,"lng":-8.38149488773843},{"lat":41.1521608012579,"lng":-8.38139740568048},{"lat":41.1521311979962,"lng":-8.38130568857449},{"lat":41.1519984217155,"lng":-8.38099973419068},{"lat":41.1517891457357,"lng":-8.38071613030385},{"lat":41.1516638044538,"lng":-8.38063285762819},{"lat":41.1516222714634,"lng":-8.38060196597687},{"lat":41.1515851875601,"lng":-8.38055345634901},{"lat":41.1515609011516,"lng":-8.38050178791879},{"lat":41.1515382016105,"lng":-8.38029644015892},{"lat":41.151436379079,"lng":-8.38003993335574},{"lat":41.1514145459242,"lng":-8.37997575871818},{"lat":41.1514098180988,"lng":-8.37983016388514},{"lat":41.1514121271267,"lng":-8.37951114686105},{"lat":41.1514613901388,"lng":-8.37925675553769},{"lat":41.1514900459045,"lng":-8.37903659728491},{"lat":41.1511932129939,"lng":-8.37901915549068},{"lat":41.1506466003726,"lng":-8.37908595463482},{"lat":41.1497225749422,"lng":-8.37944749713722},{"lat":41.1494063741028,"lng":-8.37957663379388},{"lat":41.1490128910125,"lng":-8.37965924761595},{"lat":41.1486238728285,"lng":-8.3798432504426},{"lat":41.1484564230766,"lng":-8.37961875357747},{"lat":41.1451717969062,"lng":-8.37729203634661},{"lat":41.14435199385,"lng":-8.3763034114789},{"lat":41.1408396531051,"lng":-8.37179351047397},{"lat":41.1402904230606,"lng":-8.37108839008279},{"lat":41.1401763889405,"lng":-8.37136787784631},{"lat":41.1390452985552,"lng":-8.3741433169299},{"lat":41.1389327930973,"lng":-8.37441982443714},{"lat":41.1388963780963,"lng":-8.37460644413591},{"lat":41.1388728575481,"lng":-8.37466019401925},{"lat":41.1388454567559,"lng":-8.37471404873155},{"lat":41.1388001838727,"lng":-8.37483875808737},{"lat":41.1387404861756,"lng":-8.37500320141821},{"lat":41.1386279121423,"lng":-8.37526851233305},{"lat":41.1385863396762,"lng":-8.37538329197515},{"lat":41.1385570791323,"lng":-8.37550776676362},{"lat":41.1385483983345,"lng":-8.37561028145492},{"lat":41.1385100578933,"lng":-8.37577176215764},{"lat":41.1385043520361,"lng":-8.37583106061606},{"lat":41.1384840226809,"lng":-8.37599046361488},{"lat":41.1384633010456,"lng":-8.37611988746007},{"lat":41.1384172946835,"lng":-8.37624569207277},{"lat":41.1383865776547,"lng":-8.3762948883831},{"lat":41.138273721706,"lng":-8.37643608316484},{"lat":41.1381521613079,"lng":-8.37656521207506},{"lat":41.1380897184827,"lng":-8.37666955667192},{"lat":41.1380327997199,"lng":-8.37684287855485},{"lat":41.1379524238399,"lng":-8.37712437480602},{"lat":41.1379187161423,"lng":-8.37722155851218},{"lat":41.1378726762122,"lng":-8.37727867599904},{"lat":41.137717719136,"lng":-8.37752844579949},{"lat":41.1376487165463,"lng":-8.37766718558265},{"lat":41.13756318837,"lng":-8.37795449851251},{"lat":41.1374785300639,"lng":-8.37817416166409},{"lat":41.1374454481737,"lng":-8.37827515898118},{"lat":41.1373876904248,"lng":-8.37842060078001},{"lat":41.137367844279,"lng":-8.37847745727066},{"lat":41.13734522529,"lng":-8.37853025505373},{"lat":41.1372719363093,"lng":-8.37873614313136},{"lat":41.1372572193623,"lng":-8.37879659167388},{"lat":41.1365165078914,"lng":-8.37989048939212},{"lat":41.1363808758526,"lng":-8.38011293528349},{"lat":41.1363534831307,"lng":-8.3801622538906},{"lat":41.1362716352,"lng":-8.38028759565058},{"lat":41.136080772248,"lng":-8.38060677741417},{"lat":41.1359566933185,"lng":-8.38085558401136},{"lat":41.1358403678852,"lng":-8.38114145810721},{"lat":41.1357196296198,"lng":-8.38142731336074},{"lat":41.135627098147,"lng":-8.38166170942975},{"lat":41.1355582087911,"lng":-8.38178780910631},{"lat":41.1355284067082,"lng":-8.38187130509331},{"lat":41.1354861064221,"lng":-8.38194522216711},{"lat":41.1354580299793,"lng":-8.3820193143959},{"lat":41.1339008512094,"lng":-8.38324555070377},{"lat":41.1337978627843,"lng":-8.38311093798254},{"lat":41.1306501659632,"lng":-8.3873865830367},{"lat":41.1305846632668,"lng":-8.38747812300435},{"lat":41.1300134892641,"lng":-8.38827632619309},{"lat":41.1313306137319,"lng":-8.39002254172045},{"lat":41.1328260197784,"lng":-8.3924862155971},{"lat":41.1340348393863,"lng":-8.39523246404444},{"lat":41.1349570525881,"lng":-8.39826126658247},{"lat":41.1354501338379,"lng":-8.40138294223236},{"lat":41.135511844126,"lng":-8.40554278423306},{"lat":41.1350735545196,"lng":-8.40951146256644},{"lat":41.1332452590641,"lng":-8.41748134521624},{"lat":41.1328804713578,"lng":-8.4204732945368},{"lat":41.1328138024781,"lng":-8.423308971294},{"lat":41.1336220037088,"lng":-8.43331731866588},{"lat":41.1338554497922,"lng":-8.4356145838218},{"lat":41.1340864847991,"lng":-8.43788858276601},{"lat":41.1347896576943,"lng":-8.44222469478379},{"lat":41.1356704883199,"lng":-8.44687686601394},{"lat":41.1366126477936,"lng":-8.45082042552461},{"lat":41.1376142313675,"lng":-8.45476438807595},{"lat":41.1389719230249,"lng":-8.45910415264843},{"lat":41.140610623898,"lng":-8.46305538088768},{"lat":41.1431242895478,"lng":-8.46904485113521},{"lat":41.1431822987603,"lng":-8.46918247827937},{"lat":41.1451909000606,"lng":-8.46769991657099},{"lat":41.1458707888132,"lng":-8.46708919946238},{"lat":41.1459180073474,"lng":-8.46704678434871},{"lat":41.1599423747069,"lng":-8.45444576369919},{"lat":41.1602762139249,"lng":-8.45388182882725},{"lat":41.1603414457206,"lng":-8.45389811354095},{"lat":41.1603966758072,"lng":-8.45395080681382},{"lat":41.1604996210519,"lng":-8.45390937557413},{"lat":41.1606184444124,"lng":-8.45385765593714},{"lat":41.1606976011364,"lng":-8.45382134610484},{"lat":41.1607529480447,"lng":-8.4538320971919},{"lat":41.1608162579499,"lng":-8.4538271681675},{"lat":41.1608756578922,"lng":-8.45380648800157},{"lat":41.1609349903256,"lng":-8.45377544769327},{"lat":41.1610063187322,"lng":-8.45373909943693},{"lat":41.1610578332888,"lng":-8.45370265573784},{"lat":41.1611292780395,"lng":-8.45362448978823},{"lat":41.1611649578715,"lng":-8.45358272871878},{"lat":41.161220524126,"lng":-8.45351496414954},{"lat":41.1612642211079,"lng":-8.45344190397062},{"lat":41.1612762816421,"lng":-8.45337905189799},{"lat":41.1612844992083,"lng":-8.4532743664587},{"lat":41.1612610336286,"lng":-8.45316427842862},{"lat":41.1612375450787,"lng":-8.45309613365286},{"lat":41.1611665978564,"lng":-8.45299629862144},{"lat":41.1611035951233,"lng":-8.45289126473548},{"lat":41.1610683936182,"lng":-8.45276014793239},{"lat":41.1610566849467,"lng":-8.4526973007915},{"lat":41.1610568603679,"lng":-8.45263439343778},{"lat":41.1610533339902,"lng":-8.45247734469285},{"lat":41.161041858553,"lng":-8.45233061837935},{"lat":41.1610658309955,"lng":-8.45225746073143},{"lat":41.1611061692595,"lng":-8.45209669283504},{"lat":41.1611724824912,"lng":-8.45214622318979},{"lat":41.1612354913011,"lng":-8.45225125888609},{"lat":41.161290604095,"lng":-8.4523458954427},{"lat":41.1613615900757,"lng":-8.45243000391995},{"lat":41.1614167617947,"lng":-8.45250366164859},{"lat":41.1614720219044,"lng":-8.45254575541747},{"lat":41.1615590902168,"lng":-8.45254618176465},{"lat":41.1616700764475,"lng":-8.45246296103764},{"lat":41.1619238874157,"lng":-8.45224413960863},{"lat":41.1621063851994,"lng":-8.45205654036554},{"lat":41.1621779558341,"lng":-8.45193118914554},{"lat":41.1622647632579,"lng":-8.45202585414885},{"lat":41.1623674151221,"lng":-8.45208914567783},{"lat":41.1624464770767,"lng":-8.45212099035865},{"lat":41.1625570196529,"lng":-8.45219492000724},{"lat":41.1626442059236,"lng":-8.45215340705947},{"lat":41.1625097957489,"lng":-8.44999988144493},{"lat":41.1623388733427,"lng":-8.44801453414585},{"lat":41.1651217851655,"lng":-8.44098035296278},{"lat":41.1667430158109,"lng":-8.43978643119635},{"lat":41.1720852884861,"lng":-8.4387422924561},{"lat":41.172386288967,"lng":-8.43597961653675},{"lat":41.1723282573885,"lng":-8.43321527676606},{"lat":41.1728468380065,"lng":-8.42947163556495},{"lat":41.1739542088169,"lng":-8.42621313649745},{"lat":41.1750320336279,"lng":-8.4239477107101},{"lat":41.1757522201801,"lng":-8.42177523508581},{"lat":41.1761150090686,"lng":-8.41960115462854},{"lat":41.1759776908824,"lng":-8.41733029336298},{"lat":41.175339333169,"lng":-8.41534107322565},{"lat":41.1740586181459,"lng":-8.41297078797765},{"lat":41.1727764850794,"lng":-8.41116812730928},{"lat":41.1714952301171,"lng":-8.40898718931625},{"lat":41.1693580706859,"lng":-8.40604609624724},{"lat":41.1676481102981,"lng":-8.40376902452034},{"lat":41.1665800246304,"lng":-8.40206217515498},{"lat":41.1660677886265,"lng":-8.4006775762032},{"lat":41.1658037088346,"lng":-8.39949110335717},{"lat":41.1655206157482,"lng":-8.39827934223329},{"lat":41.1655807470167,"lng":-8.39699330536399},{"lat":41.1657559081245,"lng":-8.39540507059208},{"lat":41.1657026805302,"lng":-8.39366459485517},{"lat":41.1653068725132,"lng":-8.39169578442767},{"lat":41.1643406042732,"lng":-8.38911947567435},{"lat":41.1625189431214,"lng":-8.38548070755155},{"lat":41.1609250580212,"lng":-8.38222129296721},{"lat":41.1596150256702,"lng":-8.37987097679534},{"lat":41.1580381166742,"lng":-8.37727535920638}]},{"zone_id":"prt2","points":[{"lat":41.1472762286378,"lng":-8.6409798385859},{"lat":41.1470882899835,"lng":-8.6422525775009},{"lat":41.1468643752968,"lng":-8.6444842755015},{"lat":41.1466484271405,"lng":-8.64663671869836},{"lat":41.1460601295934,"lng":-8.64878775057835},{"lat":41.145250257943,"lng":-8.65057647904109},{"lat":41.144060358041,"lng":-8.6548552179485},{"lat":41.1429581578777,"lng":-8.65961095347071},{"lat":41.1427622044757,"lng":-8.6630636282963},{"lat":41.1430196873605,"lng":-8.66782501449796},{"lat":41.1446094271112,"lng":-8.66867977925776},{"lat":41.1456759119188,"lng":-8.66987957151162},{"lat":41.1465282555794,"lng":-8.67096424129883},{"lat":41.1473784596182,"lng":-8.67255937484305},{"lat":41.1475218417161,"lng":-8.67287992331113},{"lat":41.1473665599223,"lng":-8.67298454593256},{"lat":41.1473407982791,"lng":-8.67325415173084},{"lat":41.1472875137282,"lng":-8.67370960186178},{"lat":41.1472131650781,"lng":-8.67412330073553},{"lat":41.1471481353269,"lng":-8.67454731997937},{"lat":41.1472718726986,"lng":-8.67408638133294},{"lat":41.1473186821947,"lng":-8.67384065358369},{"lat":41.1473707790126,"lng":-8.673408904306},{"lat":41.1474033308147,"lng":-8.67313137306662},{"lat":41.1475003620731,"lng":-8.67308106867358},{"lat":41.1475236720897,"lng":-8.67344744637916},{"lat":41.1475431588394,"lng":-8.6740335803657},{"lat":41.1475999943045,"lng":-8.67469840429436},{"lat":41.1477056969044,"lng":-8.67489225829133},{"lat":41.1477372674032,"lng":-8.67493779051204},{"lat":41.1475313429788,"lng":-8.67558436327399},{"lat":41.1476294753495,"lng":-8.67535859671623},{"lat":41.1477145348248,"lng":-8.67515631258097},{"lat":41.1478139455306,"lng":-8.67502311567132},{"lat":41.1484050206369,"lng":-8.67542052013868},{"lat":41.1486723725817,"lng":-8.67562881408452},{"lat":41.149016128212,"lng":-8.67588444518534},{"lat":41.1493543962181,"lng":-8.67612037609789},{"lat":41.1495111165036,"lng":-8.67624532923577},{"lat":41.1497557034413,"lng":-8.6763546876902},{"lat":41.1498982052018,"lng":-8.6763825476896},{"lat":41.1500694054402,"lng":-8.67647260067499},{"lat":41.1502417515383,"lng":-8.67662567753808},{"lat":41.1504431272477,"lng":-8.67682724478913},{"lat":41.1505945671554,"lng":-8.67700243088058},{"lat":41.150787048928,"lng":-8.6772384694173},{"lat":41.1508946277366,"lng":-8.6773954264288},{"lat":41.1510161024242,"lng":-8.67758453910234},{"lat":41.1511007489007,"lng":-8.67771759486283},{"lat":41.1511402677181,"lng":-8.67775795106372},{"lat":41.1512267454543,"lng":-8.67784539564064},{"lat":41.1514222060415,"lng":-8.67790813566216},{"lat":41.1516552539651,"lng":-8.6779633195041},{"lat":41.1517956457021,"lng":-8.67801797414485},{"lat":41.1519073782774,"lng":-8.67807763680686},{"lat":41.152176806557,"lng":-8.67828418586138},{"lat":41.1522982615938,"lng":-8.67836370160749},{"lat":41.1524615486641,"lng":-8.67847001385167},{"lat":41.1525979357002,"lng":-8.67855311607554},{"lat":41.1526498551819,"lng":-8.67869352663194},{"lat":41.1528563393456,"lng":-8.67888132989031},{"lat":41.153109811721,"lng":-8.67907178997717},{"lat":41.1534197323183,"lng":-8.67935898462624},{"lat":41.1535745840841,"lng":-8.67963059012367},{"lat":41.1538892113524,"lng":-8.67991282910448},{"lat":41.1544463281378,"lng":-8.68019553573581},{"lat":41.154745745756,"lng":-8.68032408313009},{"lat":41.154861590718,"lng":-8.68039057118904},{"lat":41.1551370872456,"lng":-8.68081474053255},{"lat":41.1551958574428,"lng":-8.68082106984766},{"lat":41.1552437529442,"lng":-8.68080443362139},{"lat":41.1552981159623,"lng":-8.68079058756315},{"lat":41.15532310247,"lng":-8.68085620499225},{"lat":41.1553747561057,"lng":-8.68088189452233},{"lat":41.155333963091,"lng":-8.68093910021869},{"lat":41.1553669928092,"lng":-8.68098095149394},{"lat":41.1553926370532,"lng":-8.68104156820328},{"lat":41.1554508538672,"lng":-8.6810320485267},{"lat":41.1554508184329,"lng":-8.68111437356446},{"lat":41.1554829070827,"lng":-8.68116467869012},{"lat":41.1555304405437,"lng":-8.68116698658438},{"lat":41.1555628631235,"lng":-8.68122217446571},{"lat":41.1555861852751,"lng":-8.68127801199418},{"lat":41.155628689685,"lng":-8.68133459576347},{"lat":41.1556878657889,"lng":-8.68131174286751},{"lat":41.1556942842244,"lng":-8.68138280454477},{"lat":41.1556588715792,"lng":-8.68144386323142},{"lat":41.1557105239331,"lng":-8.68146979301225},{"lat":41.1557528861611,"lng":-8.68142332555093},{"lat":41.1558077564025,"lng":-8.68143605394502},{"lat":41.1558500646306,"lng":-8.68147679833149},{"lat":41.1559059672411,"lng":-8.68149882920822},{"lat":41.1559566450201,"lng":-8.68154023619902},{"lat":41.1559892696976,"lng":-8.68160996618031},{"lat":41.1560442875682,"lng":-8.68162865443361},{"lat":41.1560967025088,"lng":-8.68162658961951},{"lat":41.1561567454325,"lng":-8.68164877517276},{"lat":41.1562029711874,"lng":-8.68169943902619},{"lat":41.1562274079859,"lng":-8.68176731664559},{"lat":41.1562130283269,"lng":-8.68183665373928},{"lat":41.1562593666915,"lng":-8.68184371224538},{"lat":41.1563157534131,"lng":-8.681878143855},{"lat":41.1563674125842,"lng":-8.68192158348687},{"lat":41.1564220568253,"lng":-8.68196159293117},{"lat":41.1564784668559,"lng":-8.68199113831995},{"lat":41.1565318258055,"lng":-8.68201791533123},{"lat":41.1565776287757,"lng":-8.6820812054878},{"lat":41.1566339984587,"lng":-8.68211813389479},{"lat":41.1566875761331,"lng":-8.68215599101592},{"lat":41.1567249845044,"lng":-8.68222278622074},{"lat":41.1567466056309,"lng":-8.68229647964407},{"lat":41.1567739114784,"lng":-8.68234829083145},{"lat":41.156789661063,"lng":-8.68240609025082},{"lat":41.1567747010387,"lng":-8.68248412432139},{"lat":41.1568055825712,"lng":-8.68254156454836},{"lat":41.1568634001192,"lng":-8.68257767341776},{"lat":41.1569165709152,"lng":-8.6826248215654},{"lat":41.1569485917188,"lng":-8.6826881168544},{"lat":41.1569969323216,"lng":-8.68270996592672},{"lat":41.1570217137311,"lng":-8.68276248199816},{"lat":41.1570719461591,"lng":-8.68278434700656},{"lat":41.1571283533367,"lng":-8.68275765693634},{"lat":41.1571670773565,"lng":-8.68279443833198},{"lat":41.1572270824132,"lng":-8.68282448759712},{"lat":41.1572534690179,"lng":-8.68289905392873},{"lat":41.1573013867823,"lng":-8.68295246996214},{"lat":41.1573323284936,"lng":-8.68299728859577},{"lat":41.1573485896506,"lng":-8.68308059150329},{"lat":41.157384533382,"lng":-8.68315261514456},{"lat":41.1574436658308,"lng":-8.68314000230627},{"lat":41.1574842799432,"lng":-8.68317608759505},{"lat":41.1575279415446,"lng":-8.68323459467029},{"lat":41.1575433795299,"lng":-8.68330167970906},{"lat":41.1575930364651,"lng":-8.68333021806056},{"lat":41.1576222654398,"lng":-8.68339432312029},{"lat":41.1576095657077,"lng":-8.68347094553141},{"lat":41.1575906318124,"lng":-8.68353178365478},{"lat":41.1575640016596,"lng":-8.6835837491257},{"lat":41.157523459725,"lng":-8.68364393308782},{"lat":41.1574758378394,"lng":-8.68366009075565},{"lat":41.1574722958606,"lng":-8.68374190952868},{"lat":41.1574579766667,"lng":-8.68379838552708},{"lat":41.1574441203208,"lng":-8.68387166250007},{"lat":41.1574106947309,"lng":-8.68391260558941},{"lat":41.1574485023201,"lng":-8.68395259574363},{"lat":41.1574095070603,"lng":-8.68399134826405},{"lat":41.1574268287358,"lng":-8.68407894125518},{"lat":41.157436834405,"lng":-8.68415181945258},{"lat":41.1574123284635,"lng":-8.68422977366727},{"lat":41.1574709101818,"lng":-8.68421870838341},{"lat":41.1574960770483,"lng":-8.68413408873345},{"lat":41.1574933353028,"lng":-8.68404900154197},{"lat":41.1575451930584,"lng":-8.6840886305218},{"lat":41.1575878612804,"lng":-8.68410972080064},{"lat":41.1576269242849,"lng":-8.68407573566705},{"lat":41.157660139056,"lng":-8.68400322033746},{"lat":41.1576630745657,"lng":-8.68391674923516},{"lat":41.157663748967,"lng":-8.68383275574324},{"lat":41.1576655905805,"lng":-8.68374877870756},{"lat":41.1577167332525,"lng":-8.68371191878363},{"lat":41.1577448634609,"lng":-8.68364828836311},{"lat":41.1577934550495,"lng":-8.68361772499364},{"lat":41.1578479303112,"lng":-8.68363617122172},{"lat":41.1579105104139,"lng":-8.68367481823207},{"lat":41.1579519839962,"lng":-8.68370161725094},{"lat":41.1580015540314,"lng":-8.68374826416625},{"lat":41.1580348282048,"lng":-8.68379453463509},{"lat":41.1580919796231,"lng":-8.68385720376044},{"lat":41.1581201079401,"lng":-8.68390628726582},{"lat":41.1581512945203,"lng":-8.68397446405871},{"lat":41.1581887829376,"lng":-8.68402434531949},{"lat":41.1582343630884,"lng":-8.68409609184828},{"lat":41.1582821952867,"lng":-8.68414939565494},{"lat":41.1583337047133,"lng":-8.68420498739646},{"lat":41.1583756579135,"lng":-8.68426288902052},{"lat":41.158430371032,"lng":-8.68428836847502},{"lat":41.1584853034934,"lng":-8.68434363636657},{"lat":41.1585260681664,"lng":-8.68440593562178},{"lat":41.1585704963138,"lng":-8.68447338553371},{"lat":41.1584808540396,"lng":-8.68461441421622},{"lat":41.1584308513651,"lng":-8.68465796111519},{"lat":41.1583825578006,"lng":-8.68470080329356},{"lat":41.1583467993442,"lng":-8.68475911679195},{"lat":41.1583048096546,"lng":-8.68482203152391},{"lat":41.1582357517997,"lng":-8.68491378583485},{"lat":41.1581808373479,"lng":-8.68498576586526},{"lat":41.1581419565123,"lng":-8.68503833381773},{"lat":41.1580817253098,"lng":-8.68511098125935},{"lat":41.1580165861028,"lng":-8.68519348052294},{"lat":41.1578918064413,"lng":-8.68533873836118},{"lat":41.1577552815109,"lng":-8.68547376381521},{"lat":41.1576950299299,"lng":-8.68553175684186},{"lat":41.1576702673744,"lng":-8.68558754547429},{"lat":41.1577086418198,"lng":-8.68562254164971},{"lat":41.1577580703743,"lng":-8.68558650092137},{"lat":41.157807541336,"lng":-8.68554163913489},{"lat":41.1578496742088,"lng":-8.68550553683931},{"lat":41.1579070157826,"lng":-8.68545312487909},{"lat":41.1579561949242,"lng":-8.68541243477881},{"lat":41.1580305288484,"lng":-8.68534633780006},{"lat":41.1580881231703,"lng":-8.68529785570639},{"lat":41.1581392657842,"lng":-8.6852611099336},{"lat":41.1581740282559,"lng":-8.68522268161968},{"lat":41.1582155647258,"lng":-8.68517978307451},{"lat":41.1582899683483,"lng":-8.68509915244637},{"lat":41.1583389668061,"lng":-8.68515258668162},{"lat":41.1583875831908,"lng":-8.68517348830594},{"lat":41.1583995276234,"lng":-8.6852553186776},{"lat":41.1584583995479,"lng":-8.68523937650133},{"lat":41.1585140590478,"lng":-8.68519981235201},{"lat":41.1585379261988,"lng":-8.68527376068676},{"lat":41.1585868745068,"lng":-8.6852810835311},{"lat":41.158643827367,"lng":-8.68525404007899},{"lat":41.1586838411469,"lng":-8.68530287181439},{"lat":41.1587281480618,"lng":-8.68532111422951},{"lat":41.1587851896966,"lng":-8.685348877939},{"lat":41.1588396877755,"lng":-8.68530704027916},{"lat":41.1588921689467,"lng":-8.6852734078787},{"lat":41.1589503074102,"lng":-8.68524256604858},{"lat":41.1590006170455,"lng":-8.68530374828012},{"lat":41.1590253921853,"lng":-8.68537616024827},{"lat":41.1590531283977,"lng":-8.68545014846863},{"lat":41.1590888354492,"lng":-8.68551562204279},{"lat":41.1591239710155,"lng":-8.68558692307514},{"lat":41.1591376196355,"lng":-8.68566901513435},{"lat":41.1591697395174,"lng":-8.68573123632165},{"lat":41.1591814284369,"lng":-8.68580961688817},{"lat":41.1592111651288,"lng":-8.68587980742505},{"lat":41.1592425975848,"lng":-8.68595310818735},{"lat":41.1598845302798,"lng":-8.6862155193828},{"lat":41.1599024310467,"lng":-8.68629430615408},{"lat":41.1599500725997,"lng":-8.68634832369933},{"lat":41.1600139677341,"lng":-8.68633730472582},{"lat":41.1600709248335,"lng":-8.68630942941878},{"lat":41.1601036534262,"lng":-8.68635593106653},{"lat":41.1601151125357,"lng":-8.68642656782038},{"lat":41.1601456926772,"lng":-8.68650855783608},{"lat":41.1601839022036,"lng":-8.68646491836616},{"lat":41.1601890806378,"lng":-8.68638012745829},{"lat":41.1602419865748,"lng":-8.68642656645433},{"lat":41.1602511160364,"lng":-8.68634466503186},{"lat":41.1602694203373,"lng":-8.68628322075176},{"lat":41.1603280601427,"lng":-8.68626012653297},{"lat":41.1603728841823,"lng":-8.68624513157356},{"lat":41.1604191166742,"lng":-8.6862375387382},{"lat":41.1604736785045,"lng":-8.68623895771144},{"lat":41.1605143370305,"lng":-8.68621022560127},{"lat":41.1605611017026,"lng":-8.6862028768997},{"lat":41.1606051041063,"lng":-8.68617250845408},{"lat":41.1606521106676,"lng":-8.68617135362103},{"lat":41.1607217529221,"lng":-8.68620089631684},{"lat":41.1607848052723,"lng":-8.68619666106207},{"lat":41.1608475398916,"lng":-8.68620302921626},{"lat":41.1609083897738,"lng":-8.68618804943063},{"lat":41.1609671675964,"lng":-8.68619271972813},{"lat":41.1609837159822,"lng":-8.68625350659582},{"lat":41.1610255065991,"lng":-8.68628852710201},{"lat":41.1610699614665,"lng":-8.68623838817519},{"lat":41.1611289916565,"lng":-8.68624603011285},{"lat":41.1611948416382,"lng":-8.68624074654488},{"lat":41.1612619080955,"lng":-8.68622630518918},{"lat":41.1613207130328,"lng":-8.68622429781769},{"lat":41.1613836219585,"lng":-8.68624984920353},{"lat":41.1614375060228,"lng":-8.68628021147597},{"lat":41.1614993389041,"lng":-8.68630456210092},{"lat":41.1615454808247,"lng":-8.68629696842217},{"lat":41.1615922407797,"lng":-8.68629164367179},{"lat":41.1616538955734,"lng":-8.68627774253789},{"lat":41.1617006325728,"lng":-8.68627718471814},{"lat":41.1617446350283,"lng":-8.68626457277003},{"lat":41.1617896682153,"lng":-8.68626256909036},{"lat":41.161852390988,"lng":-8.68625153985611},{"lat":41.1619057346132,"lng":-8.68622589696916},{"lat":41.1619602077628,"lng":-8.68618929775841},{"lat":41.1620092089347,"lng":-8.68612918268746},{"lat":41.1620684561744,"lng":-8.68614922378793},{"lat":41.1621250716771,"lng":-8.68613480720062},{"lat":41.162307619633,"lng":-8.68614873752599},{"lat":41.1623678503032,"lng":-8.68613172809055},{"lat":41.1624145677779,"lng":-8.68613522478325},{"lat":41.1624589680703,"lng":-8.68615180508163},{"lat":41.162515602758,"lng":-8.6861908864689},{"lat":41.1625720243635,"lng":-8.68621674311398},{"lat":41.1626364221908,"lng":-8.68623194085951},{"lat":41.1626940321442,"lng":-8.68623671467225},{"lat":41.1627488668762,"lng":-8.68625564690133},{"lat":41.1628032898679,"lng":-8.68628589452069},{"lat":41.1628628674431,"lng":-8.68629270906977},{"lat":41.1629037102157,"lng":-8.68635632601375},{"lat":41.1629489244486,"lng":-8.68637314649119},{"lat":41.1630078593077,"lng":-8.68640057602486},{"lat":41.1630511791713,"lng":-8.68645551186466},{"lat":41.1630407782545,"lng":-8.68652036112728},{"lat":41.1629918262493,"lng":-8.68657023035353},{"lat":41.1629515510081,"lng":-8.6866311387276},{"lat":41.1629882281748,"lng":-8.68668101165663},{"lat":41.1629630835526,"lng":-8.68676063061875},{"lat":41.1630087430345,"lng":-8.6867967638045},{"lat":41.1630701838418,"lng":-8.68679073100157},{"lat":41.1631086360778,"lng":-8.68673433477779},{"lat":41.1631258392869,"lng":-8.68667693347825},{"lat":41.1631737878146,"lng":-8.68664945385652},{"lat":41.1632143381584,"lng":-8.68669853405241},{"lat":41.163173307805,"lng":-8.68674906274138},{"lat":41.1631961731019,"lng":-8.68680620984687},{"lat":41.1632518386525,"lng":-8.68680203250838},{"lat":41.1633102337441,"lng":-8.68683076965945},{"lat":41.1633060706302,"lng":-8.68689066484439},{"lat":41.1633401173709,"lng":-8.68694445096069},{"lat":41.1633897352732,"lng":-8.68698192958591},{"lat":41.163364397463,"lng":-8.68704522139493},{"lat":41.163397963339,"lng":-8.68708602009309},{"lat":41.163416425849,"lng":-8.68716088106501},{"lat":41.163392156617,"lng":-8.68722585306811},{"lat":41.1633600847978,"lng":-8.68728527938358},{"lat":41.1633964518671,"lng":-8.68734409232446},{"lat":41.1634580912182,"lng":-8.68733341425374},{"lat":41.1635039580981,"lng":-8.68727124143745},{"lat":41.1635486967177,"lng":-8.68721764809306},{"lat":41.163593961396,"lng":-8.6872239830671},{"lat":41.1636555243022,"lng":-8.68722915077728},{"lat":41.1636355740429,"lng":-8.68729583070779},{"lat":41.1636853977568,"lng":-8.68728958028325},{"lat":41.1637338363322,"lng":-8.68732847426052},{"lat":41.1637257056976,"lng":-8.68738988762604},{"lat":41.1637457041648,"lng":-8.68744462755033},{"lat":41.1637690418982,"lng":-8.68751536884521},{"lat":41.1637382787123,"lng":-8.68758302268035},{"lat":41.1637883836036,"lng":-8.687631106046},{"lat":41.1638035984346,"lng":-8.68768759670087},{"lat":41.1638569409973,"lng":-8.68771747731281},{"lat":41.1639009625867,"lng":-8.68770093106848},{"lat":41.1639277309586,"lng":-8.68775203983069},{"lat":41.1639583058253,"lng":-8.68781616443044},{"lat":41.1640117657324,"lng":-8.68782173690968},{"lat":41.1640634314733,"lng":-8.68784457907139},{"lat":41.1641090017293,"lng":-8.68784389264223},{"lat":41.1641194209038,"lng":-8.68790462431068},{"lat":41.1641607881116,"lng":-8.68795323350621},{"lat":41.1642061543551,"lng":-8.68795624076983},{"lat":41.1642614300987,"lng":-8.68797756227644},{"lat":41.1643189940768,"lng":-8.68795433259568},{"lat":41.1643803610475,"lng":-8.68794484448637},{"lat":41.164353542976,"lng":-8.6880155150487},{"lat":41.1643082517918,"lng":-8.6880719668859},{"lat":41.1642750678224,"lng":-8.6881375830587},{"lat":41.1643048882506,"lng":-8.68819014300198},{"lat":41.1643211431297,"lng":-8.68825521891247},{"lat":41.1643823000364,"lng":-8.68823297866847},{"lat":41.1644387828426,"lng":-8.68820962682504},{"lat":41.1644539708211,"lng":-8.68814040918471},{"lat":41.1644970467859,"lng":-8.68807797933538},{"lat":41.1645285238355,"lng":-8.68814175273177},{"lat":41.1645371047234,"lng":-8.6882298815393},{"lat":41.1645514216638,"lng":-8.68830447598397},{"lat":41.1645933604998,"lng":-8.68834617260156},{"lat":41.1646334766729,"lng":-8.68831732224084},{"lat":41.1646607808849,"lng":-8.68825750202848},{"lat":41.1647034320719,"lng":-8.68822676210742},{"lat":41.1647555128913,"lng":-8.68827510310598},{"lat":41.1648122812939,"lng":-8.68830406150089},{"lat":41.1648452556692,"lng":-8.68822665118074},{"lat":41.1649056145948,"lng":-8.68820190044359},{"lat":41.1649624794971,"lng":-8.68824837779035},{"lat":41.1650225706986,"lng":-8.68827902898104},{"lat":41.1650855111676,"lng":-8.68827920903566},{"lat":41.1651406155205,"lng":-8.68829849885134},{"lat":41.1651924958618,"lng":-8.68833313559245},{"lat":41.1652547873883,"lng":-8.6883561884128},{"lat":41.1652782132767,"lng":-8.68840857523208},{"lat":41.165286355971,"lng":-8.68847489514862},{"lat":41.1653180687572,"lng":-8.68854606231308},{"lat":41.1653693778143,"lng":-8.68858748606734},{"lat":41.1654143725544,"lng":-8.68859239498735},{"lat":41.1654607425021,"lng":-8.68859385985372},{"lat":41.165524296477,"lng":-8.68861596490253},{"lat":41.16556798733,"lng":-8.68864910780594},{"lat":41.165601159198,"lng":-8.68871611289604},{"lat":41.1655608573336,"lng":-8.68878333430182},{"lat":41.1655904542213,"lng":-8.68884459622687},{"lat":41.1656458024463,"lng":-8.68885089815124},{"lat":41.1656900877982,"lng":-8.68883602573007},{"lat":41.1657370595211,"lng":-8.6887858996826},{"lat":41.1657315689021,"lng":-8.68885996815486},{"lat":41.1657356962151,"lng":-8.68893769373025},{"lat":41.1657888432434,"lng":-8.68895279951696},{"lat":41.1658149425938,"lng":-8.68889975992045},{"lat":41.1658610865849,"lng":-8.68890944627961},{"lat":41.1658459508456,"lng":-8.68900416749871},{"lat":41.1658780809214,"lng":-8.6890638998519},{"lat":41.1658952297339,"lng":-8.68912922482113},{"lat":41.1659431943841,"lng":-8.68909852995726},{"lat":41.1659969356991,"lng":-8.68910208187842},{"lat":41.1660148984278,"lng":-8.68916765358344},{"lat":41.1660118742628,"lng":-8.68927058104869},{"lat":41.1660513915739,"lng":-8.68923659854261},{"lat":41.1660913528247,"lng":-8.68929591822045},{"lat":41.1661503363853,"lng":-8.68931417674418},{"lat":41.1661503363853,"lng":-8.68931417674418},{"lat":41.1661320206556,"lng":-8.68937764972513},{"lat":41.1660986011977,"lng":-8.68943563444113},{"lat":41.1661314938225,"lng":-8.68948643803606},{"lat":41.1660857735301,"lng":-8.68951810391121},{"lat":41.1660882693584,"lng":-8.68957936968129},{"lat":41.1661233751443,"lng":-8.6896562866759},{"lat":41.1661185821619,"lng":-8.68971606569371},{"lat":41.1661107023286,"lng":-8.68978153146122},{"lat":41.1661605506936,"lng":-8.68982628683403},{"lat":41.1662009637687,"lng":-8.68979231219193},{"lat":41.1662259868002,"lng":-8.68973925680261},{"lat":41.1662774312407,"lng":-8.68975161225929},{"lat":41.1663401842281,"lng":-8.68975309721137},{"lat":41.1663345520806,"lng":-8.68968906127613},{"lat":41.1663494719952,"lng":-8.68962031246926},{"lat":41.1664109429288,"lng":-8.68960808322183},{"lat":41.1664562056671,"lng":-8.68961478633746},{"lat":41.1665077169247,"lng":-8.68966943716758},{"lat":41.1664828819063,"lng":-8.68973977323352},{"lat":41.1664667549646,"lng":-8.68981565648027},{"lat":41.1664356867811,"lng":-8.68989094004911},{"lat":41.1664080578647,"lng":-8.68996255729075},{"lat":41.1664729605725,"lng":-8.68998455559097},{"lat":41.1664756602554,"lng":-8.69005976638411},{"lat":41.1665194088961,"lng":-8.69009958304982},{"lat":41.1665741607068,"lng":-8.69006179501804},{"lat":41.1666208968855,"lng":-8.69000535490103},{"lat":41.1666524224852,"lng":-8.69005889088529},{"lat":41.1666719844643,"lng":-8.69014734913197},{"lat":41.1666146600035,"lng":-8.69017713172132},{"lat":41.1666149156713,"lng":-8.69025410639854},{"lat":41.1666007125538,"lng":-8.69032334084272},{"lat":41.1665894685911,"lng":-8.69039557010919},{"lat":41.1666408054233,"lng":-8.69043009146078},{"lat":41.1666994263507,"lng":-8.69046705188506},{"lat":41.1667459473046,"lng":-8.69049235028862},{"lat":41.166763090506,"lng":-8.69055970063852},{"lat":41.1668133225262,"lng":-8.69052533020746},{"lat":41.1668623709721,"lng":-8.69051193074098},{"lat":41.166917704457,"lng":-8.69048391589932},{"lat":41.1669107377596,"lng":-8.69039842080305},{"lat":41.1669355517672,"lng":-8.69033249897385},{"lat":41.1669865143476,"lng":-8.69027716668805},{"lat":41.1670474577961,"lng":-8.69029901199491},{"lat":41.1670855601614,"lng":-8.69033377393586},{"lat":41.1670610132335,"lng":-8.69040065035938},{"lat":41.1670964354215,"lng":-8.69046720422918},{"lat":41.1671540749695,"lng":-8.69042849520793},{"lat":41.1672113204871,"lng":-8.69039632815571},{"lat":41.1672696587194,"lng":-8.6903795450142},{"lat":41.1673297434577,"lng":-8.69033776035293},{"lat":41.1673845575844,"lng":-8.69030581218288},{"lat":41.1674415625624,"lng":-8.69026721065715},{"lat":41.1674839009418,"lng":-8.69024505105016},{"lat":41.1675706303055,"lng":-8.69022362154574},{"lat":41.1676348979598,"lng":-8.69020951204662},{"lat":41.1676944411284,"lng":-8.6902233619155},{"lat":41.1677540598132,"lng":-8.69024031537634},{"lat":41.1677853564405,"lng":-8.69034103357392},{"lat":41.1677611297549,"lng":-8.69039790549326},{"lat":41.1677830474591,"lng":-8.69046434526516},{"lat":41.1677773306753,"lng":-8.69054746971978},{"lat":41.1678116698681,"lng":-8.69059662281391},{"lat":41.1678380584468,"lng":-8.69060335062364},{"lat":41.1678967509691,"lng":-8.69061831439497},{"lat":41.167945483121,"lng":-8.69067008701845},{"lat":41.1679997360984,"lng":-8.69067913143696},{"lat":41.1680548327198,"lng":-8.69068126469129},{"lat":41.1680850138964,"lng":-8.69060430744376},{"lat":41.1681031597969,"lng":-8.69051997652575},{"lat":41.168096249212,"lng":-8.69044055306999},{"lat":41.1681103992025,"lng":-8.69036465129973},{"lat":41.1681354665881,"lng":-8.69030146713065},{"lat":41.1681832860529,"lng":-8.69029972946},{"lat":41.1682409990156,"lng":-8.69030189129998},{"lat":41.1682805565831,"lng":-8.69033332358711},{"lat":41.1682879595105,"lng":-8.6904772208795},{"lat":41.1682670871175,"lng":-8.69054793847918},{"lat":41.1682726551473,"lng":-8.69062508678086},{"lat":41.1683032477898,"lng":-8.6907038728964},{"lat":41.1682611092764,"lng":-8.69074081646432},{"lat":41.1682807233375,"lng":-8.69081843717529},{"lat":41.1682763657221,"lng":-8.69089930983618},{"lat":41.1683256625447,"lng":-8.69094667971832},{"lat":41.1683693344771,"lng":-8.69100222529816},{"lat":41.1684224822515,"lng":-8.69099743051924},{"lat":41.1684768906852,"lng":-8.69097549362564},{"lat":41.168537599885,"lng":-8.69098959400898},{"lat":41.1686018101791,"lng":-8.690988349224},{"lat":41.1686579795572,"lng":-8.69101099411366},{"lat":41.1686312174926,"lng":-8.69106974935267},{"lat":41.1685993627126,"lng":-8.69114002739527},{"lat":41.1686450983346,"lng":-8.69116019311472},{"lat":41.1687019241677,"lng":-8.6911212371651},{"lat":41.1687545798583,"lng":-8.69110690961936},{"lat":41.1687581892574,"lng":-8.69118010655169},{"lat":41.1687513198694,"lng":-8.69125905399032},{"lat":41.1687655662704,"lng":-8.69132911867142},{"lat":41.1688217261804,"lng":-8.69129778138985},{"lat":41.1688530554022,"lng":-8.69122488182568},{"lat":41.1688281802856,"lng":-8.69113756701334},{"lat":41.1688453403097,"lng":-8.69107085926426},{"lat":41.1688825600033,"lng":-8.69112051072383},{"lat":41.1689094574499,"lng":-8.69119962591714},{"lat":41.1689618441836,"lng":-8.6911857688731},{"lat":41.1690032520446,"lng":-8.69113249835568},{"lat":41.1690505967956,"lng":-8.69111765277792},{"lat":41.1690854098167,"lng":-8.69106920689675},{"lat":41.1691223174592,"lng":-8.69099766618375},{"lat":41.1691841662756,"lng":-8.69096256138138},{"lat":41.1692188056769,"lng":-8.69089493654417},{"lat":41.1692318017526,"lng":-8.6908142560737},{"lat":41.1692664826765,"lng":-8.69075568075741},{"lat":41.1692974456295,"lng":-8.69068456854427},{"lat":41.1693079570373,"lng":-8.69061506023768},{"lat":41.1692913601673,"lng":-8.69054711927594},{"lat":41.1692451042081,"lng":-8.69052217500432},{"lat":41.1692006947728,"lng":-8.69050762085137},{"lat":41.1691526910796,"lng":-8.69049029274234},{"lat":41.1690952340561,"lng":-8.69049242112352},{"lat":41.1690485514548,"lng":-8.69048177637091},{"lat":41.1690508552852,"lng":-8.69041554122191},{"lat":41.169105311775,"lng":-8.69038263705305},{"lat":41.1691210795542,"lng":-8.69032485972676},{"lat":41.1690940699653,"lng":-8.69026874307899},{"lat":41.169123039269,"lng":-8.6902175102988},{"lat":41.1691696525314,"lng":-8.69024245734589},{"lat":41.1691900849103,"lng":-8.69018876810703},{"lat":41.1691724613629,"lng":-8.69012808337946},{"lat":41.1691301274865,"lng":-8.69009328448991},{"lat":41.1691545236006,"lng":-8.69003879656314},{"lat":41.1691804313752,"lng":-8.68998801040152},{"lat":41.1691536871281,"lng":-8.68993213581038},{"lat":41.1691540429013,"lng":-8.68985873246349},{"lat":41.1691810854434,"lng":-8.6897970021732},{"lat":41.169214176823,"lng":-8.68975068500696},{"lat":41.1692606984911,"lng":-8.68973952036957},{"lat":41.1693209967846,"lng":-8.68978341440786},{"lat":41.1693781219008,"lng":-8.68983108375476},{"lat":41.1694441593651,"lng":-8.68986084343967},{"lat":41.1694934378856,"lng":-8.68985601481012},{"lat":41.1695329421633,"lng":-8.68993583660172},{"lat":41.1695585815992,"lng":-8.68986669612522},{"lat":41.1695674210513,"lng":-8.68978907608853},{"lat":41.169528267782,"lng":-8.6897489438656},{"lat":41.1695459457276,"lng":-8.68968629466239},{"lat":41.1694845044419,"lng":-8.68959997422811},{"lat":41.1694520254838,"lng":-8.68955727525138},{"lat":41.1694018389513,"lng":-8.68952622542199},{"lat":41.1693993452771,"lng":-8.68946554922095},{"lat":41.1694496698287,"lng":-8.68941200001901},{"lat":41.169490937849,"lng":-8.68936873500071},{"lat":41.1695211125525,"lng":-8.68931059989376},{"lat":41.1695564208948,"lng":-8.68927193205989},{"lat":41.1696156618869,"lng":-8.68923597093177},{"lat":41.1696511808954,"lng":-8.68917144855923},{"lat":41.1696569751816,"lng":-8.68909081933677},{"lat":41.1696974351272,"lng":-8.68902943510313},{"lat":41.1697356709658,"lng":-8.6889805438899},{"lat":41.1697792646066,"lng":-8.68894016143533},{"lat":41.1698220619031,"lng":-8.68891669230286},{"lat":41.1698958858788,"lng":-8.68884510296528},{"lat":41.1699460591416,"lng":-8.68882276833394},{"lat":41.1699911078573,"lng":-8.68881755011826},{"lat":41.1700518812877,"lng":-8.68881842488328},{"lat":41.1700985790515,"lng":-8.6888259657205},{"lat":41.1701598238217,"lng":-8.68884174040588},{"lat":41.1702370232266,"lng":-8.68883584252463},{"lat":41.1703435268582,"lng":-8.68883936235853},{"lat":41.1703969854559,"lng":-8.68884518263539},{"lat":41.1704475874072,"lng":-8.6888465638475},{"lat":41.1704951413296,"lng":-8.68884458310589},{"lat":41.1705443990267,"lng":-8.68884404847342},{"lat":41.1706583378981,"lng":-8.68885621473828},{"lat":41.1707108843578,"lng":-8.68886440456645},{"lat":41.1708160851073,"lng":-8.68887721598239},{"lat":41.1708652776626,"lng":-8.68889014522154},{"lat":41.1709495041607,"lng":-8.68892005221794},{"lat":41.1709951627667,"lng":-8.68891877461619},{"lat":41.1710444227792,"lng":-8.68891776061321},{"lat":41.17109418253,"lng":-8.68892473495672},{"lat":41.1711787031713,"lng":-8.68895000328771},{"lat":41.1712517698422,"lng":-8.6889798158723},{"lat":41.1713061561279,"lng":-8.68896133222497},{"lat":41.1713618482472,"lng":-8.68897038127335},{"lat":41.1714151505296,"lng":-8.68898978480432},{"lat":41.1714719279125,"lng":-8.68901683574816},{"lat":41.1715238477219,"lng":-8.68904325954834},{"lat":41.17156478227,"lng":-8.68906887104532},{"lat":41.1716226563844,"lng":-8.68909283497467},{"lat":41.1717007974191,"lng":-8.68911589875006},{"lat":41.1717594890663,"lng":-8.6891383248087},{"lat":41.1718164159512,"lng":-8.68917086434866},{"lat":41.1718824828315,"lng":-8.68923303436722},{"lat":41.1719383040995,"lng":-8.68927152452878},{"lat":41.1719885461915,"lng":-8.68930877544295},{"lat":41.1720483213038,"lng":-8.68933085137833},{"lat":41.1720910898904,"lng":-8.68938710391186},{"lat":41.1721497762713,"lng":-8.6894293077751},{"lat":41.1721949808596,"lng":-8.68944697175802},{"lat":41.1722401979582,"lng":-8.68946309096278},{"lat":41.1723006302547,"lng":-8.68947933267583},{"lat":41.1723576521132,"lng":-8.68949328137629},{"lat":41.1724183839691,"lng":-8.68950273339961},{"lat":41.1724725019535,"lng":-8.68953965807558},{"lat":41.1725075309117,"lng":-8.68957737967065},{"lat":41.1725588275651,"lng":-8.68962023376103},{"lat":41.172617903548,"lng":-8.68961835135532},{"lat":41.1726398832411,"lng":-8.68967312285885},{"lat":41.1726504720258,"lng":-8.68975472053931},{"lat":41.172679668355,"lng":-8.6898248028247},{"lat":41.1727269344047,"lng":-8.68982615652459},{"lat":41.1727646686929,"lng":-8.68988082228076},{"lat":41.172802771129,"lng":-8.68991558703664},{"lat":41.1728281515435,"lng":-8.68984382388668},{"lat":41.172879683718,"lng":-8.68983806190705},{"lat":41.1729366154184,"lng":-8.68987060265704},{"lat":41.1729748747844,"lng":-8.68992800999115},{"lat":41.1730272566922,"lng":-8.68991510999506},{"lat":41.17308780743,"lng":-8.68988821518997},{"lat":41.1731510634626,"lng":-8.68989697659909},{"lat":41.1731969377593,"lng":-8.68990725517088},{"lat":41.1732534269639,"lng":-8.6899200079202},{"lat":41.1733026482614,"lng":-8.68992697869972},{"lat":41.1733271528002,"lng":-8.68997986113842},{"lat":41.1733321113146,"lng":-8.69005307456097},{"lat":41.1733914640375,"lng":-8.69006883511908},{"lat":41.1734006192814,"lng":-8.69014923627362},{"lat":41.1734367114912,"lng":-8.69018970470221},{"lat":41.1734762771117,"lng":-8.69021947470254},{"lat":41.173507082893,"lng":-8.690421371005},{"lat":41.1736005411634,"lng":-8.69053442690162},{"lat":41.1737413689437,"lng":-8.69079542155225},{"lat":41.1739434161688,"lng":-8.69072384880156},{"lat":41.174292414945,"lng":-8.69081988820439},{"lat":41.1743934209365,"lng":-8.69088975628196},{"lat":41.1746529310121,"lng":-8.68851317349185},{"lat":41.1742010363027,"lng":-8.68848541843597},{"lat":41.1741234376645,"lng":-8.68803607902793},{"lat":41.1738361103289,"lng":-8.68769140159211},{"lat":41.1739279671983,"lng":-8.68596858052286},{"lat":41.1740131775164,"lng":-8.68436998244056},{"lat":41.1738659589977,"lng":-8.68352069965248},{"lat":41.1730330041512,"lng":-8.68168353722123},{"lat":41.1721168506283,"lng":-8.67973510579506},{"lat":41.1719593544496,"lng":-8.67935463758057},{"lat":41.1719579186703,"lng":-8.67798871879124},{"lat":41.1723018647874,"lng":-8.67648659535406},{"lat":41.1727110879752,"lng":-8.67523396107685},{"lat":41.1737444047562,"lng":-8.67363883088937},{"lat":41.1748099694954,"lng":-8.67218735384555},{"lat":41.1753214009467,"lng":-8.67151733541752},{"lat":41.1760406861615,"lng":-8.6710785510021},{"lat":41.1763276799659,"lng":-8.67062843887275},{"lat":41.1765911780414,"lng":-8.67013988256681},{"lat":41.1768422825511,"lng":-8.66949692206131},{"lat":41.1770170480807,"lng":-8.66882304634026},{"lat":41.1776747369188,"lng":-8.66823174753957},{"lat":41.1784149904479,"lng":-8.66786182715305},{"lat":41.1790863997006,"lng":-8.66736547594221},{"lat":41.1794334432893,"lng":-8.66690734678309},{"lat":41.1796078091619,"lng":-8.66652708308533},{"lat":41.1796660981644,"lng":-8.66598644675204},{"lat":41.1796184238318,"lng":-8.66487175646687},{"lat":41.1795927561219,"lng":-8.66427171590295},{"lat":41.1792182093856,"lng":-8.66340083749553},{"lat":41.1785400684249,"lng":-8.66278490228239},{"lat":41.1777974552519,"lng":-8.66229677726399},{"lat":41.1774691398101,"lng":-8.66170196458943},{"lat":41.1773817205892,"lng":-8.6608940832116},{"lat":41.1774514406321,"lng":-8.65923624615938},{"lat":41.1777341809566,"lng":-8.65772724474836},{"lat":41.1781171645281,"lng":-8.65639101865561},{"lat":41.1784475674539,"lng":-8.65588300403144},{"lat":41.178801654245,"lng":-8.65487607070049},{"lat":41.1790389250655,"lng":-8.65420131241252},{"lat":41.1817300650383,"lng":-8.65482964492732},{"lat":41.1833821568553,"lng":-8.65493546095186},{"lat":41.1865091761395,"lng":-8.65558125423425},{"lat":41.1923993062019,"lng":-8.65646194923535},{"lat":41.1927559315009,"lng":-8.65357991730701},{"lat":41.1917038050677,"lng":-8.652572368278},{"lat":41.1897482880199,"lng":-8.65104385108705},{"lat":41.1892574164416,"lng":-8.65046526552968},{"lat":41.1890363387855,"lng":-8.64944788763703},{"lat":41.1887722652948,"lng":-8.64795223535506},{"lat":41.1868319779367,"lng":-8.64787726049747},{"lat":41.1861196888784,"lng":-8.64727391326729},{"lat":41.183330978309,"lng":-8.64577176733752},{"lat":41.1842488833915,"lng":-8.64375973525135},{"lat":41.184708806095,"lng":-8.64246883588769},{"lat":41.1846758732874,"lng":-8.6416106399274},{"lat":41.184143007609,"lng":-8.64048950745177},{"lat":41.1831038144482,"lng":-8.63861925693161},{"lat":41.181505500009,"lng":-8.63574297165251},{"lat":41.1804564822646,"lng":-8.63385533379247},{"lat":41.1798118176388,"lng":-8.63269536046027},{"lat":41.1797483249034,"lng":-8.63224784417801},{"lat":41.1797380124922,"lng":-8.63113164928929},{"lat":41.1798438606616,"lng":-8.62977827813638},{"lat":41.179996994485,"lng":-8.62828615048194},{"lat":41.1802067197228,"lng":-8.62640117836105},{"lat":41.1803811873993,"lng":-8.62504135241453},{"lat":41.1805373100728,"lng":-8.62382439791806},{"lat":41.1809511768826,"lng":-8.62314204334637},{"lat":41.1815661726074,"lng":-8.62274567987148},{"lat":41.1824582273645,"lng":-8.62259562394398},{"lat":41.1825656746792,"lng":-8.62209849600043},{"lat":41.1819087824544,"lng":-8.62087866093551},{"lat":41.1818023783936,"lng":-8.62031627667139},{"lat":41.1817616099745,"lng":-8.62010080504737},{"lat":41.182146133973,"lng":-8.61916631475128},{"lat":41.1825371189608,"lng":-8.61852878304335},{"lat":41.1827692872696,"lng":-8.6181502080896},{"lat":41.1830450139461,"lng":-8.61759574920303},{"lat":41.1831213058447,"lng":-8.61733096125999},{"lat":41.1831910002236,"lng":-8.61663697591417},{"lat":41.1833010717259,"lng":-8.61554084554126},{"lat":41.1833802167181,"lng":-8.61475262570444},{"lat":41.183441922869,"lng":-8.61413804399977},{"lat":41.1808935256339,"lng":-8.61446797112279},{"lat":41.1794964151,"lng":-8.61458813556561},{"lat":41.1776006106532,"lng":-8.61485199635611},{"lat":41.1761419905197,"lng":-8.61455084853734},{"lat":41.174721702463,"lng":-8.61422669860832},{"lat":41.1739880924196,"lng":-8.61881172376012},{"lat":41.1731264102285,"lng":-8.62704017934647},{"lat":41.1726752111672,"lng":-8.62963669984453},{"lat":41.1712805824519,"lng":-8.63585545612138},{"lat":41.1708670142514,"lng":-8.63711416191875},{"lat":41.1702066970498,"lng":-8.63835476621461},{"lat":41.1692873626092,"lng":-8.63959011289681},{"lat":41.1687902512585,"lng":-8.64027909492182},{"lat":41.1686357654205,"lng":-8.64074666254564},{"lat":41.1685306332801,"lng":-8.64128965615944},{"lat":41.1684744112083,"lng":-8.64238654455038},{"lat":41.1683942436923,"lng":-8.64268126259962},{"lat":41.1676826773541,"lng":-8.64386613881409},{"lat":41.1675132614777,"lng":-8.64401048046034},{"lat":41.1673296017001,"lng":-8.64405760382876},{"lat":41.1669193348459,"lng":-8.64390875032216},{"lat":41.1665748918225,"lng":-8.64380896366973},{"lat":41.1663398677919,"lng":-8.64385568717986},{"lat":41.1661735099267,"lng":-8.64407112778748},{"lat":41.1657255626675,"lng":-8.64503557773734},{"lat":41.164776402811,"lng":-8.64725182849186},{"lat":41.1640814348515,"lng":-8.64841160561936},{"lat":41.1632035226709,"lng":-8.64917602360922},{"lat":41.1622091472339,"lng":-8.64953094683466},{"lat":41.1614560885526,"lng":-8.64962022082206},{"lat":41.1608101241125,"lng":-8.64938617130213},{"lat":41.1594375374851,"lng":-8.64844957466514},{"lat":41.1575485592863,"lng":-8.64690174257265},{"lat":41.1568659853527,"lng":-8.64643196738166},{"lat":41.1554395827748,"lng":-8.64574176103912},{"lat":41.1546155756239,"lng":-8.64536514886301},{"lat":41.153350400267,"lng":-8.64502481801633},{"lat":41.1520319748163,"lng":-8.64506659978941},{"lat":41.1511093401895,"lng":-8.64504033050109},{"lat":41.1503990475858,"lng":-8.64468326571614},{"lat":41.1497319563349,"lng":-8.64400552338424},{"lat":41.1492733605763,"lng":-8.64320713490876},{"lat":41.1489170179747,"lng":-8.64204890993824},{"lat":41.1490127339192,"lng":-8.64147790703128},{"lat":41.1472762286378,"lng":-8.6409798385859}]},{"zone_id":"gdm1","points":[{"lat":41.1936608603225,"lng":-8.51726205324221},{"lat":41.1929552091212,"lng":-8.51687106684594},{"lat":41.1897329472246,"lng":-8.51508579725754},{"lat":41.1891653624956,"lng":-8.51461547340286},{"lat":41.1887740185309,"lng":-8.51445978364862},{"lat":41.1885308042069,"lng":-8.51430030983264},{"lat":41.1873388847461,"lng":-8.51322646906199},{"lat":41.186886669088,"lng":-8.51255445947458},{"lat":41.1861583098567,"lng":-8.51171339357158},{"lat":41.1852398902148,"lng":-8.5105644582024},{"lat":41.1845371053838,"lng":-8.50996541900728},{"lat":41.1839782709863,"lng":-8.50964610946221},{"lat":41.1832495236543,"lng":-8.50894467743042},{"lat":41.1828023047106,"lng":-8.50874688452033},{"lat":41.1824272639527,"lng":-8.50861458167313},{"lat":41.1819674152267,"lng":-8.50823067698241},{"lat":41.1813493914824,"lng":-8.50789246835659},{"lat":41.1802825215652,"lng":-8.50746815164142},{"lat":41.1792713865404,"lng":-8.50687659826009},{"lat":41.1783773345259,"lng":-8.50633221426163},{"lat":41.1788147984777,"lng":-8.50764039590738},{"lat":41.1797712143244,"lng":-8.51048711088827},{"lat":41.1812014513659,"lng":-8.51386749432572},{"lat":41.1787465147368,"lng":-8.51407986109821},{"lat":41.1782268182667,"lng":-8.51588985699067},{"lat":41.1774795560007,"lng":-8.51747186452168},{"lat":41.1761633074395,"lng":-8.51852170614269},{"lat":41.1749243974809,"lng":-8.52056233668983},{"lat":41.1736536553493,"lng":-8.52030238463571},{"lat":41.1702228402581,"lng":-8.51981622619824},{"lat":41.1670086815799,"lng":-8.51991540783315},{"lat":41.1625704768565,"lng":-8.51964947497279},{"lat":41.1598414468008,"lng":-8.5195168879186},{"lat":41.1568839768701,"lng":-8.51987799717687},{"lat":41.155166975435,"lng":-8.5189307124127},{"lat":41.1533303188316,"lng":-8.51803483033264},{"lat":41.1506411581717,"lng":-8.51780971434024},{"lat":41.1486810723456,"lng":-8.51909193131546},{"lat":41.1465155610017,"lng":-8.51907919311788},{"lat":41.1423288485158,"lng":-8.52052972787895},{"lat":41.1406389342112,"lng":-8.51986140281359},{"lat":41.1360577701451,"lng":-8.51922817763039},{"lat":41.1341392393277,"lng":-8.51794526125708},{"lat":41.13386516227,"lng":-8.51779079288893},{"lat":41.1329468759126,"lng":-8.5177789246073},{"lat":41.1325830638505,"lng":-8.51854070490039},{"lat":41.1325476500303,"lng":-8.51917676769916},{"lat":41.1320126527303,"lng":-8.52049569604474},{"lat":41.1314973309597,"lng":-8.52131765409652},{"lat":41.1311209260001,"lng":-8.52216971504425},{"lat":41.1307570363833,"lng":-8.52352024885259},{"lat":41.131132829695,"lng":-8.52912972974605},{"lat":41.1315151686297,"lng":-8.53153631425419},{"lat":41.1321091684327,"lng":-8.53322944879149},{"lat":41.1333651883804,"lng":-8.53523171272217},{"lat":41.1344383837356,"lng":-8.53656274248081},{"lat":41.1349478631815,"lng":-8.53723068963138},{"lat":41.1348408938565,"lng":-8.53757576937187},{"lat":41.1349114582067,"lng":-8.53799304708284},{"lat":41.1351627335168,"lng":-8.53823279356715},{"lat":41.1354326468515,"lng":-8.53829400854998},{"lat":41.135645679264,"lng":-8.53810161367275},{"lat":41.1368335267908,"lng":-8.53904315943061},{"lat":41.1396365298803,"lng":-8.54099319568611},{"lat":41.1401646420216,"lng":-8.54123953347362},{"lat":41.1410410891901,"lng":-8.54224729763043},{"lat":41.1416017024925,"lng":-8.54399516545206},{"lat":41.1423846375259,"lng":-8.54560870423386},{"lat":41.1429656586583,"lng":-8.54626619105896},{"lat":41.143956601932,"lng":-8.54694457176556},{"lat":41.1451060545813,"lng":-8.54725895808523},{"lat":41.1454065825134,"lng":-8.54740066054179},{"lat":41.1468483425928,"lng":-8.54775121477747},{"lat":41.1475149139727,"lng":-8.54789041151849},{"lat":41.1476840786401,"lng":-8.54796650274336},{"lat":41.1477889899205,"lng":-8.54811391165965},{"lat":41.1479393333708,"lng":-8.54832387043707},{"lat":41.1482346399217,"lng":-8.54839255075372},{"lat":41.1485051521785,"lng":-8.54823092796893},{"lat":41.1512168042971,"lng":-8.54871676417158},{"lat":41.1518850388017,"lng":-8.5490867762999},{"lat":41.1521686200381,"lng":-8.54926057976933},{"lat":41.1524646745661,"lng":-8.5494925872188},{"lat":41.1529134056018,"lng":-8.54986167064868},{"lat":41.153340614583,"lng":-8.55029608225462},{"lat":41.1536778756831,"lng":-8.55083076929608},{"lat":41.1539610372493,"lng":-8.55132301600919},{"lat":41.1541878054638,"lng":-8.55178102836478},{"lat":41.1543391752682,"lng":-8.5521903373883},{"lat":41.1545066910641,"lng":-8.55266661034053},{"lat":41.156006345541,"lng":-8.55213575272067},{"lat":41.1573560855234,"lng":-8.55238265009386},{"lat":41.1581821812489,"lng":-8.55300446002426},{"lat":41.1587928161945,"lng":-8.55346410241563},{"lat":41.1597733861871,"lng":-8.55370867116856},{"lat":41.1593509402534,"lng":-8.55500012490378},{"lat":41.1593912093061,"lng":-8.55522413217822},{"lat":41.1589765784315,"lng":-8.55602654520412},{"lat":41.1585386270649,"lng":-8.55693753438026},{"lat":41.1584494394287,"lng":-8.55712411174038},{"lat":41.1581422969175,"lng":-8.55770240503255},{"lat":41.1578802612941,"lng":-8.55804783073111},{"lat":41.1576266975128,"lng":-8.55823857027273},{"lat":41.1570377975383,"lng":-8.55856305812414},{"lat":41.1568987530266,"lng":-8.55866689862197},{"lat":41.1577583794594,"lng":-8.56049860788897},{"lat":41.1582629609195,"lng":-8.56040658252332},{"lat":41.1589667763421,"lng":-8.56061363424224},{"lat":41.1599087396692,"lng":-8.56103592982601},{"lat":41.1609241543947,"lng":-8.56160559866198},{"lat":41.1617150923648,"lng":-8.5628347916191},{"lat":41.1625229551166,"lng":-8.56448029959937},{"lat":41.1631154749122,"lng":-8.56514093851909},{"lat":41.1644814557249,"lng":-8.56531603723422},{"lat":41.1650835003012,"lng":-8.56502452673401},{"lat":41.1654533229185,"lng":-8.56484815464578},{"lat":41.1661251422662,"lng":-8.56460755162539},{"lat":41.1670605380314,"lng":-8.56449677846598},{"lat":41.1681380984773,"lng":-8.56502402108388},{"lat":41.1691531403845,"lng":-8.56218428951323},{"lat":41.1698142414839,"lng":-8.56124583435462},{"lat":41.1717778279261,"lng":-8.56166929858419},{"lat":41.172618697921,"lng":-8.55869395515977},{"lat":41.1742990995023,"lng":-8.55612487688808},{"lat":41.1741961782277,"lng":-8.55471633606333},{"lat":41.1744769032746,"lng":-8.552358715617},{"lat":41.1738867350222,"lng":-8.54863778945577},{"lat":41.173692145086,"lng":-8.54790565592652},{"lat":41.1758468407738,"lng":-8.54630760942387},{"lat":41.1774679533796,"lng":-8.54341818318093},{"lat":41.1787312619508,"lng":-8.54012487774906},{"lat":41.1800794804782,"lng":-8.53826212619819},{"lat":41.1813722628043,"lng":-8.5367922583753},{"lat":41.1823047335713,"lng":-8.53448707351953},{"lat":41.1825442866074,"lng":-8.53355897517788},{"lat":41.1832485919761,"lng":-8.53356327837907},{"lat":41.1843496772768,"lng":-8.53339512999892},{"lat":41.185275701849,"lng":-8.53293443888017},{"lat":41.1865972879147,"lng":-8.53265103067234},{"lat":41.1879170476644,"lng":-8.53289225709913},{"lat":41.189062150434,"lng":-8.53272435460172},{"lat":41.1905618364363,"lng":-8.53185903113475},{"lat":41.1917111803378,"lng":-8.53046685059066},{"lat":41.1925965946071,"lng":-8.52901472802131},{"lat":41.1941418752939,"lng":-8.52768314191441},{"lat":41.1946725064332,"lng":-8.52698671715016},{"lat":41.196171150614,"lng":-8.52641270036173},{"lat":41.199740355887,"lng":-8.52589781789248},{"lat":41.1987649172728,"lng":-8.52426541640126},{"lat":41.1975132412209,"lng":-8.5233511357426},{"lat":41.1959764122632,"lng":-8.52159051012298},{"lat":41.1952194572533,"lng":-8.51958777045306},{"lat":41.1940749159485,"lng":-8.51819904270758},{"lat":41.1936608603225,"lng":-8.51726205324221}]},{"zone_id":"prd7","points":[{"lat":41.2444555793559,"lng":-8.44011079692562},{"lat":41.2446382237856,"lng":-8.43986018048848},{"lat":41.2449555426405,"lng":-8.43948421711379},{"lat":41.2458058493417,"lng":-8.43913175138831},{"lat":41.2466608699596,"lng":-8.4388316661636},{"lat":41.2480052997862,"lng":-8.43828214139462},{"lat":41.2482666913898,"lng":-8.43814700766757},{"lat":41.248503204194,"lng":-8.43788635656484},{"lat":41.2499815299986,"lng":-8.43635792331365},{"lat":41.2503378998579,"lng":-8.43591396282898},{"lat":41.251000221559,"lng":-8.43493227066504},{"lat":41.2511565992984,"lng":-8.43463448470182},{"lat":41.2513020479877,"lng":-8.43441682253472},{"lat":41.2515815589212,"lng":-8.43396402366945},{"lat":41.2517430796351,"lng":-8.4337302071555},{"lat":41.2518689913998,"lng":-8.43361350835428},{"lat":41.2519974369252,"lng":-8.43342463236304},{"lat":41.2523844918977,"lng":-8.43285229274702},{"lat":41.2526234788395,"lng":-8.43250154590588},{"lat":41.2528159128374,"lng":-8.43222014141885},{"lat":41.2529839746145,"lng":-8.43199624428202},{"lat":41.2530251597773,"lng":-8.43187938658437},{"lat":41.2529802517775,"lng":-8.43169902218679},{"lat":41.2528744690743,"lng":-8.43134716404835},{"lat":41.2528633540754,"lng":-8.43125857946628},{"lat":41.2528749419959,"lng":-8.43116557374676},{"lat":41.2529092454987,"lng":-8.43106359743008},{"lat":41.2531916915709,"lng":-8.43065350326625},{"lat":41.2533238693156,"lng":-8.43044566604218},{"lat":41.2534718227586,"lng":-8.43026617578083},{"lat":41.2535970465561,"lng":-8.43013765081023},{"lat":41.2535748004362,"lng":-8.42996633720908},{"lat":41.2535503691152,"lng":-8.42973511675042},{"lat":41.2535508358235,"lng":-8.42955495272879},{"lat":41.2535379533373,"lng":-8.42928000102512},{"lat":41.2535070866754,"lng":-8.4289256158141},{"lat":41.2534916614231,"lng":-8.42872653096266},{"lat":41.2534820366195,"lng":-8.4285147056949},{"lat":41.2534507569047,"lng":-8.42831949059265},{"lat":41.2533931952821,"lng":-8.42818930062553},{"lat":41.2532392864439,"lng":-8.42802645201046},{"lat":41.2531907333517,"lng":-8.42796407588709},{"lat":41.253164841364,"lng":-8.42787840548178},{"lat":41.2531242908447,"lng":-8.42775211452448},{"lat":41.2531098293213,"lng":-8.42766864924769},{"lat":41.2531022023097,"lng":-8.42755300118933},{"lat":41.2531128045948,"lng":-8.42738648443483},{"lat":41.2531167471925,"lng":-8.42721923261219},{"lat":41.253121470455,"lng":-8.4271337010641},{"lat":41.2531388802439,"lng":-8.42698058380806},{"lat":41.2531426254262,"lng":-8.42685448472222},{"lat":41.2531507967438,"lng":-8.42675847609904},{"lat":41.2531661664533,"lng":-8.42669829382676},{"lat":41.253195918553,"lng":-8.42650740907783},{"lat":41.2531823946355,"lng":-8.42633864353691},{"lat":41.2531896517718,"lng":-8.4262475166073},{"lat":41.2533291173717,"lng":-8.42625172545782},{"lat":41.2533869069612,"lng":-8.42608172715974},{"lat":41.2533639711198,"lng":-8.42593140692173},{"lat":41.2533331819836,"lng":-8.42585956402032},{"lat":41.2533124280083,"lng":-8.42580614034839},{"lat":41.2533141284952,"lng":-8.42574147611474},{"lat":41.2533839268221,"lng":-8.42573462993954},{"lat":41.2534442462305,"lng":-8.42577284346507},{"lat":41.2535412179002,"lng":-8.42573891606963},{"lat":41.2535663605917,"lng":-8.42558929207473},{"lat":41.2534690276477,"lng":-8.4254474715807},{"lat":41.2534258161886,"lng":-8.42537341980013},{"lat":41.2534321973753,"lng":-8.42530783013806},{"lat":41.2534911633375,"lng":-8.42527731215769},{"lat":41.2535736754429,"lng":-8.42515872793761},{"lat":41.2535740654264,"lng":-8.42500600462795},{"lat":41.2534639243644,"lng":-8.42504846751605},{"lat":41.2534096624883,"lng":-8.42503593060072},{"lat":41.2534146600436,"lng":-8.42491294459054},{"lat":41.2534694503208,"lng":-8.42471834920584},{"lat":41.253505228423,"lng":-8.42466935314489},{"lat":41.2535563935798,"lng":-8.42466134793826},{"lat":41.2536063448541,"lng":-8.42452925606478},{"lat":41.253563409578,"lng":-8.42434651636168},{"lat":41.2535172353999,"lng":-8.42419859996216},{"lat":41.2534820066679,"lng":-8.42403033076322},{"lat":41.2534714696697,"lng":-8.42392564290593},{"lat":41.253474737509,"lng":-8.42380789527975},{"lat":41.2534036260475,"lng":-8.4237296666189},{"lat":41.2533323998128,"lng":-8.42369653955784},{"lat":41.2533108456062,"lng":-8.42363905276539},{"lat":41.2533017549835,"lng":-8.42356516064953},{"lat":41.2533112777885,"lng":-8.42346879703564},{"lat":41.2533301542908,"lng":-8.42337450075198},{"lat":41.2533503982825,"lng":-8.42330992685341},{"lat":41.2533924980617,"lng":-8.42321788802592},{"lat":41.2534796692776,"lng":-8.42307259198642},{"lat":41.2535613927851,"lng":-8.42294230750409},{"lat":41.2536416704001,"lng":-8.42278111260779},{"lat":41.2537032117064,"lng":-8.42262138807331},{"lat":41.2537702786831,"lng":-8.42248530959378},{"lat":41.2538263662672,"lng":-8.4223809237865},{"lat":41.2538748459508,"lng":-8.42222316800156},{"lat":41.2538696321822,"lng":-8.4218652032319},{"lat":41.2538856364609,"lng":-8.42166005284392},{"lat":41.2539062836553,"lng":-8.42146936229594},{"lat":41.2539404847724,"lng":-8.42140484860537},{"lat":41.2539887140458,"lng":-8.42134552468336},{"lat":41.2540710919235,"lng":-8.42127823729784},{"lat":41.2541704714214,"lng":-8.42121711192897},{"lat":41.2542231765821,"lng":-8.42120493824398},{"lat":41.2544450119895,"lng":-8.4211793101727},{"lat":41.2547893598247,"lng":-8.42113060374405},{"lat":41.2550082060326,"lng":-8.42107715998259},{"lat":41.2551292289597,"lng":-8.42103867708155},{"lat":41.2552072271928,"lng":-8.42088630146917},{"lat":41.2552076091221,"lng":-8.42073453479268},{"lat":41.2552109011968,"lng":-8.42064219386036},{"lat":41.2552374864809,"lng":-8.42056237002813},{"lat":41.2553480781266,"lng":-8.4203379491335},{"lat":41.2554540195648,"lng":-8.42013736356983},{"lat":41.2556392180032,"lng":-8.41989215417582},{"lat":41.2557978628254,"lng":-8.41971626125599},{"lat":41.255936261232,"lng":-8.4195701087619},{"lat":41.2560341826157,"lng":-8.4193718774944},{"lat":41.2560923116698,"lng":-8.41927894301097},{"lat":41.2561242080544,"lng":-8.41923553189872},{"lat":41.2561947267018,"lng":-8.41919228926117},{"lat":41.2563106631617,"lng":-8.41913505280001},{"lat":41.256412007378,"lng":-8.41900782992625},{"lat":41.2565451671024,"lng":-8.41886785160956},{"lat":41.2566225586566,"lng":-8.41877810817153},{"lat":41.2566863914669,"lng":-8.4186717139847},{"lat":41.2567560583148,"lng":-8.41850163387503},{"lat":41.2567801231065,"lng":-8.41838313889701},{"lat":41.2567565157576,"lng":-8.41831848027784},{"lat":41.2567374024657,"lng":-8.41821924431598},{"lat":41.2567217018299,"lng":-8.41816214212479},{"lat":41.2566677750177,"lng":-8.4179418799991},{"lat":41.2566543434238,"lng":-8.41787417306567},{"lat":41.2566818486887,"lng":-8.41774960525764},{"lat":41.2567344353142,"lng":-8.41760271037927},{"lat":41.2568177715135,"lng":-8.41743853498807},{"lat":41.2568669121217,"lng":-8.41730355906414},{"lat":41.2568422781728,"lng":-8.41718031707412},{"lat":41.2567629699194,"lng":-8.41710038079432},{"lat":41.2566303239846,"lng":-8.41703441523237},{"lat":41.2565066626473,"lng":-8.41697529603796},{"lat":41.2564034472227,"lng":-8.41698737297336},{"lat":41.2563376232735,"lng":-8.41698863317065},{"lat":41.2562820242428,"lng":-8.41696882166096},{"lat":41.2562094174639,"lng":-8.41694595596035},{"lat":41.2560938289321,"lng":-8.41689975395426},{"lat":41.2560336875835,"lng":-8.41686190695027},{"lat":41.2559801760669,"lng":-8.41680523958269},{"lat":41.25592395572,"lng":-8.41674283031171},{"lat":41.2558683825085,"lng":-8.41671251800457},{"lat":41.2558333398319,"lng":-8.41664709791049},{"lat":41.255830271704,"lng":-8.41650462169973},{"lat":41.2558204306963,"lng":-8.41637081934768},{"lat":41.2558048249801,"lng":-8.41627255387355},{"lat":41.2557645254176,"lng":-8.41600343814976},{"lat":41.2557568741533,"lng":-8.41592918899213},{"lat":41.2557686497261,"lng":-8.41575360879401},{"lat":41.255765559126,"lng":-8.41561984053728},{"lat":41.2557749745736,"lng":-8.41545403064244},{"lat":41.2557900386148,"lng":-8.41533394404208},{"lat":41.2557982676164,"lng":-8.4152108402401},{"lat":41.2558380567804,"lng":-8.41517796134021},{"lat":41.2559116937035,"lng":-8.41525643673845},{"lat":41.2560331320465,"lng":-8.41526663130961},{"lat":41.2561274961867,"lng":-8.41519353793126},{"lat":41.2562025067577,"lng":-8.41511964876473},{"lat":41.2562924262519,"lng":-8.41502231281572},{"lat":41.2563596871086,"lng":-8.41491223848513},{"lat":41.2564234574953,"lng":-8.41483137153961},{"lat":41.2565565961267,"lng":-8.41469831233954},{"lat":41.2565988281074,"lng":-8.41458657504303},{"lat":41.2566409731397,"lng":-8.41456225297404},{"lat":41.2567022496954,"lng":-8.41452688996439},{"lat":41.2568071523899,"lng":-8.41434191978101},{"lat":41.256872252925,"lng":-8.41416047971853},{"lat":41.2568944993618,"lng":-8.41390152997886},{"lat":41.2568984076959,"lng":-8.41370335632138},{"lat":41.2568964628113,"lng":-8.41354036077656},{"lat":41.2569000462569,"lng":-8.41347427485149},{"lat":41.2569138951015,"lng":-8.41340978039388},{"lat":41.2569481966866,"lng":-8.41330027486214},{"lat":41.2570200090528,"lng":-8.4131692130568},{"lat":41.2571123573505,"lng":-8.41299993659064},{"lat":41.257167249686,"lng":-8.41282990443597},{"lat":41.2571596707082,"lng":-8.41268871282998},{"lat":41.2570998045799,"lng":-8.41257438905082},{"lat":41.2570795310886,"lng":-8.41250521340612},{"lat":41.2569755007467,"lng":-8.41233414614675},{"lat":41.256917910142,"lng":-8.41220921077278},{"lat":41.2568342511295,"lng":-8.41210217732704},{"lat":41.2567594363964,"lng":-8.41206223983504},{"lat":41.2567130215618,"lng":-8.41200644029059},{"lat":41.2566993535877,"lng":-8.41194788977569},{"lat":41.2565737084166,"lng":-8.4118669543808},{"lat":41.2564875061828,"lng":-8.41184403523514},{"lat":41.2563536354794,"lng":-8.41179847665279},{"lat":41.2562923684881,"lng":-8.41177709787402},{"lat":41.2562425645715,"lng":-8.4117453810929},{"lat":41.2561337154908,"lng":-8.41166676679954},{"lat":41.2560136315798,"lng":-8.41158153773175},{"lat":41.2559229325812,"lng":-8.41151802926198},{"lat":41.2558209383567,"lng":-8.41147177542137},{"lat":41.2556768902608,"lng":-8.41142760669089},{"lat":41.2555839210429,"lng":-8.41144761190663},{"lat":41.2554883459079,"lng":-8.41153729253515},{"lat":41.2553699728429,"lng":-8.41163593586955},{"lat":41.2552745850045,"lng":-8.41164888829389},{"lat":41.2551088681591,"lng":-8.411666195288},{"lat":41.2549930293671,"lng":-8.41168682112348},{"lat":41.2547796959987,"lng":-8.41166037143058},{"lat":41.2547320860902,"lng":-8.41165110110858},{"lat":41.2546732287839,"lng":-8.41156374758252},{"lat":41.2545598372324,"lng":-8.41150325110021},{"lat":41.2544826565145,"lng":-8.41150888803451},{"lat":41.254409985982,"lng":-8.41151155762316},{"lat":41.2543442764888,"lng":-8.41146772710177},{"lat":41.2542559692154,"lng":-8.41138323456195},{"lat":41.2541403523394,"lng":-8.4113105555654},{"lat":41.2540518822778,"lng":-8.41129526220222},{"lat":41.2539610781697,"lng":-8.41127518517425},{"lat":41.2538703207428,"lng":-8.41123578173482},{"lat":41.2537614349884,"lng":-8.41121133520704},{"lat":41.2537000814428,"lng":-8.41122610747457},{"lat":41.2536423196256,"lng":-8.4112077254567},{"lat":41.2535821516208,"lng":-8.41117751873454},{"lat":41.2535097342606,"lng":-8.4111133783965},{"lat":41.2534327182491,"lng":-8.41105303318816},{"lat":41.2533353226375,"lng":-8.41096707091423},{"lat":41.2531495941302,"lng":-8.41084398367386},{"lat":41.2530316195097,"lng":-8.4108165107743},{"lat":41.252861281667,"lng":-8.41084430647011},{"lat":41.2526961453013,"lng":-8.41088153785207},{"lat":41.2526151050009,"lng":-8.41088322527622},{"lat":41.2525547868948,"lng":-8.4108797411695},{"lat":41.2524596840042,"lng":-8.41084998743333},{"lat":41.2523391801765,"lng":-8.41078850247399},{"lat":41.2522837854269,"lng":-8.4106864945579},{"lat":41.2522320642773,"lng":-8.41044061428402},{"lat":41.2522037363336,"lng":-8.41031450089824},{"lat":41.2521866445277,"lng":-8.41016111263219},{"lat":41.2521729779419,"lng":-8.4099289965355},{"lat":41.2521621688082,"lng":-8.40978196325786},{"lat":41.2521071103588,"lng":-8.4095759190753},{"lat":41.2520835255483,"lng":-8.40950017590663},{"lat":41.2520567860728,"lng":-8.40938671576559},{"lat":41.2519558032419,"lng":-8.40914587614276},{"lat":41.2518594967807,"lng":-8.40890613073247},{"lat":41.251791526997,"lng":-8.4087893969889},{"lat":41.2516871379545,"lng":-8.40865449565701},{"lat":41.251587270926,"lng":-8.40858272329266},{"lat":41.251474600242,"lng":-8.40859584665351},{"lat":41.2514142383159,"lng":-8.40861026590608},{"lat":41.2513443528778,"lng":-8.40861629696237},{"lat":41.2512746145515,"lng":-8.40859810025789},{"lat":41.2511969613141,"lng":-8.40853895765837},{"lat":41.2511724548861,"lng":-8.40846953071496},{"lat":41.2511943014825,"lng":-8.40833373043454},{"lat":41.2511088720923,"lng":-8.40821465952814},{"lat":41.2510454932147,"lng":-8.40817203621592},{"lat":41.250989265254,"lng":-8.40811405080525},{"lat":41.2509156598056,"lng":-8.40802032379045},{"lat":41.2508706604439,"lng":-8.40797348506568},{"lat":41.250829458555,"lng":-8.40792081614823},{"lat":41.2507797601347,"lng":-8.40790144914448},{"lat":41.2507248375745,"lng":-8.40788004637893},{"lat":41.2506568060591,"lng":-8.40778837041703},{"lat":41.2506166077247,"lng":-8.40773093242887},{"lat":41.2505728037715,"lng":-8.40763661169778},{"lat":41.2504456692469,"lng":-8.4072973615006},{"lat":41.2501427073913,"lng":-8.40653407592829},{"lat":41.2495941023574,"lng":-8.40505690867112},{"lat":41.2494589947865,"lng":-8.40469830657206},{"lat":41.2490747285079,"lng":-8.40373534056041},{"lat":41.2482765011148,"lng":-8.40393042710242},{"lat":41.2480722365028,"lng":-8.40399089665859},{"lat":41.2474823862297,"lng":-8.4042154772196},{"lat":41.2472952102059,"lng":-8.40428222197192},{"lat":41.2473595582942,"lng":-8.40448506502388},{"lat":41.2476079146772,"lng":-8.40474176295787},{"lat":41.2477173289974,"lng":-8.40488741020577},{"lat":41.2477634891735,"lng":-8.40497552855764},{"lat":41.2477911321813,"lng":-8.40505140242873},{"lat":41.2477878789115,"lng":-8.40513108764327},{"lat":41.2477290018621,"lng":-8.40520241986746},{"lat":41.2476577556544,"lng":-8.40525127509271},{"lat":41.2475942654128,"lng":-8.4052930029834},{"lat":41.2474764944193,"lng":-8.40540501387233},{"lat":41.2472931214814,"lng":-8.40576644509315},{"lat":41.2472107801254,"lng":-8.40593384165374},{"lat":41.247161132096,"lng":-8.40602561173383},{"lat":41.2471207934789,"lng":-8.40606230846723},{"lat":41.2470448485172,"lng":-8.40611722838264},{"lat":41.246967547896,"lng":-8.40613742428208},{"lat":41.2469102758283,"lng":-8.40613920924355},{"lat":41.2468299174607,"lng":-8.40611835353685},{"lat":41.2467251698206,"lng":-8.40601865814904},{"lat":41.2466186103242,"lng":-8.40595891215357},{"lat":41.2464810266502,"lng":-8.40596037032645},{"lat":41.2463587250475,"lng":-8.40604575384859},{"lat":41.2462888085202,"lng":-8.40617847556311},{"lat":41.2462359225851,"lng":-8.406305069039},{"lat":41.2461938381549,"lng":-8.4064315939858},{"lat":41.2461581326283,"lng":-8.40652759950733},{"lat":41.2461284881632,"lng":-8.40657340087465},{"lat":41.2460231847185,"lng":-8.4067059787248},{"lat":41.2459285560073,"lng":-8.40681605525345},{"lat":41.2458292487514,"lng":-8.40696392423176},{"lat":41.2457393191268,"lng":-8.40706793537351},{"lat":41.2456850649229,"lng":-8.40712699741331},{"lat":41.2455688358055,"lng":-8.40723280222121},{"lat":41.2454726037744,"lng":-8.40737352343404},{"lat":41.2454691623246,"lng":-8.40753301414694},{"lat":41.2454827475953,"lng":-8.40764759887205},{"lat":41.2454779581843,"lng":-8.4077314466368},{"lat":41.2454668755101,"lng":-8.40784592212964},{"lat":41.2454481701999,"lng":-8.40791229258738},{"lat":41.2453830873345,"lng":-8.40797953996951},{"lat":41.2453149095071,"lng":-8.40802423020465},{"lat":41.2451987485606,"lng":-8.40810152175867},{"lat":41.2451383419518,"lng":-8.40813502423297},{"lat":41.2450733869472,"lng":-8.40814907016408},{"lat":41.2449467480262,"lng":-8.40809127209758},{"lat":41.2448556675621,"lng":-8.40803768422225},{"lat":41.2447585981059,"lng":-8.40792799391575},{"lat":41.2446600339595,"lng":-8.40776390118984},{"lat":41.2445352733768,"lng":-8.40759874614154},{"lat":41.2444488291916,"lng":-8.40752465943024},{"lat":41.2443529927419,"lng":-8.4075385682717},{"lat":41.2442709242305,"lng":-8.4075934585528},{"lat":41.2441158476439,"lng":-8.40775552650682},{"lat":41.2440568800896,"lng":-8.40786371726696},{"lat":41.2440009683962,"lng":-8.40797382581373},{"lat":41.2438812957751,"lng":-8.40823923371553},{"lat":41.2438423413701,"lng":-8.40830038416161},{"lat":41.2437380808683,"lng":-8.40844631905202},{"lat":41.2436093985124,"lng":-8.40860181631565},{"lat":41.2435409251052,"lng":-8.40865628099265},{"lat":41.2434560263263,"lng":-8.40868884653512},{"lat":41.2433959591241,"lng":-8.40869407822681},{"lat":41.2433400845493,"lng":-8.40867738302733},{"lat":41.2431767911675,"lng":-8.40862193674666},{"lat":41.2429925871531,"lng":-8.40861280405666},{"lat":41.2428021801515,"lng":-8.40860101968687},{"lat":41.2426189595662,"lng":-8.40863245501102},{"lat":41.2424619722934,"lng":-8.4085015225956},{"lat":41.2429188869255,"lng":-8.40668362682476},{"lat":41.2430179599466,"lng":-8.40629479104346},{"lat":41.2430868771516,"lng":-8.4060542215784},{"lat":41.2431946900025,"lng":-8.40505809279076},{"lat":41.2432872968522,"lng":-8.40435797958119},{"lat":41.2434733333647,"lng":-8.40271713487044},{"lat":41.2436196307318,"lng":-8.40157680590645},{"lat":41.2435902216245,"lng":-8.40146316253468},{"lat":41.2303674164475,"lng":-8.40217681612486},{"lat":41.2242809269719,"lng":-8.40233344810619},{"lat":41.2199611391176,"lng":-8.40442187452394},{"lat":41.2147521334127,"lng":-8.4075725826378},{"lat":41.2159763412535,"lng":-8.41034167555702},{"lat":41.2169833161577,"lng":-8.41263338332874},{"lat":41.2175155077144,"lng":-8.41421321765511},{"lat":41.217927202661,"lng":-8.41634471124446},{"lat":41.2181002308821,"lng":-8.41863295243233},{"lat":41.2180945365381,"lng":-8.42092042215992},{"lat":41.2180093098498,"lng":-8.42235017715168},{"lat":41.2178670015816,"lng":-8.42473685586075},{"lat":41.2175609713641,"lng":-8.42796951396181},{"lat":41.2145721694502,"lng":-8.4468515800786},{"lat":41.2146504358338,"lng":-8.44679811624086},{"lat":41.215197724331,"lng":-8.44651080422016},{"lat":41.2164382461641,"lng":-8.44465665160172},{"lat":41.2172202281214,"lng":-8.44404030469798},{"lat":41.2194442968474,"lng":-8.44369193043697},{"lat":41.220722263414,"lng":-8.44362079741745},{"lat":41.2208002148612,"lng":-8.44356652208149},{"lat":41.2215090955497,"lng":-8.44332460878993},{"lat":41.2215604333138,"lng":-8.44328035079372},{"lat":41.2222180151174,"lng":-8.443032809779},{"lat":41.2230370608672,"lng":-8.44272447985665},{"lat":41.2244566843846,"lng":-8.44137645851112},{"lat":41.224504932082,"lng":-8.4413306430153},{"lat":41.2246796659421,"lng":-8.44116471690741},{"lat":41.2255526494989,"lng":-8.44058314383367},{"lat":41.2266375127275,"lng":-8.44063346451616},{"lat":41.2271213915421,"lng":-8.44150494440122},{"lat":41.2273774274542,"lng":-8.44118783954049},{"lat":41.227677271019,"lng":-8.44138639968295},{"lat":41.2277525128738,"lng":-8.4413966594597},{"lat":41.2277969902824,"lng":-8.44129811473661},{"lat":41.2278359797048,"lng":-8.44119680453829},{"lat":41.2278926393642,"lng":-8.44112228956018},{"lat":41.2280170589119,"lng":-8.44102818056797},{"lat":41.2284315527626,"lng":-8.44081569774178},{"lat":41.2287728536641,"lng":-8.44063172656647},{"lat":41.229025126942,"lng":-8.44057304110702},{"lat":41.2291788099057,"lng":-8.44051857618633},{"lat":41.229250493302,"lng":-8.44049317165067},{"lat":41.2293582359116,"lng":-8.44045498765222},{"lat":41.2294183987799,"lng":-8.4404502600327},{"lat":41.229516255964,"lng":-8.44045573244382},{"lat":41.2296516090764,"lng":-8.44048129635368},{"lat":41.2298003129411,"lng":-8.44049092677889},{"lat":41.2298999035947,"lng":-8.4404973765384},{"lat":41.2301293888955,"lng":-8.44050847621246},{"lat":41.2303284523601,"lng":-8.44064908124579},{"lat":41.2306682960386,"lng":-8.44090568644582},{"lat":41.2307482857471,"lng":-8.44069257001456},{"lat":41.2308318597389,"lng":-8.44051989869467},{"lat":41.2308691575523,"lng":-8.44044337490543},{"lat":41.2309312708014,"lng":-8.44031593695161},{"lat":41.2309865485861,"lng":-8.44035281506224},{"lat":41.2310613734437,"lng":-8.44045288050399},{"lat":41.2311012339934,"lng":-8.44049755507636},{"lat":41.2311288126455,"lng":-8.44055541035846},{"lat":41.2311839755264,"lng":-8.44063427502077},{"lat":41.2312430861752,"lng":-8.44071839921743},{"lat":41.2312981729334,"lng":-8.4407920121614},{"lat":41.2313494436458,"lng":-8.44084473435906},{"lat":41.2314047471915,"lng":-8.44087123522134},{"lat":41.231469352803,"lng":-8.44092139965749},{"lat":41.2315313757599,"lng":-8.44099420706937},{"lat":41.231605158848,"lng":-8.44107923939437},{"lat":41.2316837943372,"lng":-8.44123693658766},{"lat":41.2317547311127,"lng":-8.44134210989119},{"lat":41.2318729421259,"lng":-8.44148150729464},{"lat":41.2319793887997,"lng":-8.44160783815361},{"lat":41.232124816401,"lng":-8.44180553725096},{"lat":41.2322629975907,"lng":-8.44201793674036},{"lat":41.2323261504944,"lng":-8.44207071542554},{"lat":41.2323217799609,"lng":-8.44222264996407},{"lat":41.2323329495191,"lng":-8.44238670935887},{"lat":41.2322931310503,"lng":-8.4425947725599},{"lat":41.232324548916,"lng":-8.44266302608229},{"lat":41.2323791445204,"lng":-8.4429515725372},{"lat":41.232402562668,"lng":-8.44308276390838},{"lat":41.2324362856847,"lng":-8.44323046775137},{"lat":41.2325506915081,"lng":-8.44330985125872},{"lat":41.2327235891409,"lng":-8.44337043331541},{"lat":41.2329564960883,"lng":-8.44348127417935},{"lat":41.2331442384666,"lng":-8.44361182504253},{"lat":41.2333470947619,"lng":-8.44371250709786},{"lat":41.2334913426,"lng":-8.44381207136264},{"lat":41.2335913040206,"lng":-8.44357411707436},{"lat":41.2336143115041,"lng":-8.4434245341101},{"lat":41.2337124967215,"lng":-8.4432754232803},{"lat":41.2338105259016,"lng":-8.44318619747857},{"lat":41.2339388525143,"lng":-8.4430471319328},{"lat":41.2340292324141,"lng":-8.4429876819591},{"lat":41.2342823661847,"lng":-8.44287927027972},{"lat":41.2343953837338,"lng":-8.44283984870544},{"lat":41.2345081483623,"lng":-8.44286030668668},{"lat":41.2345833145382,"lng":-8.44290062270484},{"lat":41.2345980034636,"lng":-8.44303034468656},{"lat":41.234640344616,"lng":-8.44312048545812},{"lat":41.2346914420355,"lng":-8.44297091450611},{"lat":41.234751807961,"lng":-8.44289140461637},{"lat":41.2348271361239,"lng":-8.44287184041156},{"lat":41.235060336909,"lng":-8.44287294966224},{"lat":41.2352257352105,"lng":-8.44294351355671},{"lat":41.2353084008374,"lng":-8.44297384709395},{"lat":41.2354699923776,"lng":-8.44298641908367},{"lat":41.2356018410579,"lng":-8.44297524239943},{"lat":41.2356878918488,"lng":-8.4429183969839},{"lat":41.2357792039421,"lng":-8.44278249097428},{"lat":41.2359682157534,"lng":-8.44251082961083},{"lat":41.2360833801417,"lng":-8.44234354781615},{"lat":41.2361703864593,"lng":-8.44233346336475},{"lat":41.2363166745304,"lng":-8.44234465567932},{"lat":41.2364589434904,"lng":-8.44237682417836},{"lat":41.236748466906,"lng":-8.44239394150374},{"lat":41.2368315750444,"lng":-8.44239433684137},{"lat":41.2369422067325,"lng":-8.442405358942},{"lat":41.2370213362569,"lng":-8.44241098167129},{"lat":41.2370962027348,"lng":-8.44249519405042},{"lat":41.2371634203965,"lng":-8.44251125659274},{"lat":41.2372227462269,"lng":-8.44251678488742},{"lat":41.2373214663552,"lng":-8.44256962370043},{"lat":41.2373274397946,"lng":-8.44279032674387},{"lat":41.2373109436124,"lng":-8.44303143832478},{"lat":41.2373026496483,"lng":-8.44316762256777},{"lat":41.2372865216268,"lng":-8.44330388186611},{"lat":41.2372384686099,"lng":-8.44349235971942},{"lat":41.2371665763005,"lng":-8.4437730532027},{"lat":41.2370707066978,"lng":-8.44412913239011},{"lat":41.2370382959094,"lng":-8.44439104316409},{"lat":41.2369902743775,"lng":-8.44460051012962},{"lat":41.2369583293893,"lng":-8.44472620105151},{"lat":41.2369740065248,"lng":-8.44478913717206},{"lat":41.2370290340196,"lng":-8.44491524486877},{"lat":41.2371315460825,"lng":-8.44503108028918},{"lat":41.2373366514036,"lng":-8.44519977965935},{"lat":41.2374787519725,"lng":-8.44532630394129},{"lat":41.2375811726638,"lng":-8.44544214155965},{"lat":41.237746811705,"lng":-8.44562114538935},{"lat":41.2379023279676,"lng":-8.44583597906058},{"lat":41.2380225616279,"lng":-8.4460000046382},{"lat":41.2380853735517,"lng":-8.44617851684168},{"lat":41.238092695572,"lng":-8.44639875564651},{"lat":41.2381238382279,"lng":-8.44656661751882},{"lat":41.2381232063897,"lng":-8.44679731090297},{"lat":41.2381464612422,"lng":-8.44698613211567},{"lat":41.2382013425972,"lng":-8.44716460763687},{"lat":41.2382956694372,"lng":-8.44737476766402},{"lat":41.2384215152689,"lng":-8.44761656875191},{"lat":41.2385159264496,"lng":-8.44779523634468},{"lat":41.2386022169902,"lng":-8.44807872141279},{"lat":41.2386571870077,"lng":-8.44822582453855},{"lat":41.2386488581544,"lng":-8.44837250803255},{"lat":41.2386957607,"lng":-8.44857194350714},{"lat":41.2386948633992,"lng":-8.44889699638174},{"lat":41.2387024985331,"lng":-8.44900188325969},{"lat":41.2386860703756,"lng":-8.44921151152013},{"lat":41.2386302314558,"lng":-8.44937907503012},{"lat":41.2385998841617,"lng":-8.44944572247149},{"lat":41.2386487098428,"lng":-8.44947089611395},{"lat":41.2387240459143,"lng":-8.44941639070106},{"lat":41.2388484582601,"lng":-8.44932728731938},{"lat":41.2389049096111,"lng":-8.44932756060077},{"lat":41.2391907459526,"lng":-8.44934386009183},{"lat":41.2394200211802,"lng":-8.44939984405719},{"lat":41.2396342241899,"lng":-8.44946565316505},{"lat":41.23993869896,"lng":-8.44954694053861},{"lat":41.2400382106805,"lng":-8.44950876820471},{"lat":41.240071245886,"lng":-8.44958031575877},{"lat":41.2401131053804,"lng":-8.44964632132639},{"lat":41.2401618648722,"lng":-8.44969391119778},{"lat":41.24025036856,"lng":-8.44966439761525},{"lat":41.2403255667269,"lng":-8.44965975742442},{"lat":41.2404008001797,"lng":-8.44967503414901},{"lat":41.2404619463998,"lng":-8.44973520955481},{"lat":41.2405050143736,"lng":-8.44979029236353},{"lat":41.2405171759204,"lng":-8.44988637896412},{"lat":41.2405356479174,"lng":-8.45001112707146},{"lat":41.2403845593136,"lng":-8.45023978818447},{"lat":41.2403351798615,"lng":-8.45041406546069},{"lat":41.240331042533,"lng":-8.450543715364},{"lat":41.2403874665736,"lng":-8.45055388830447},{"lat":41.2404778631374,"lng":-8.45052438613429},{"lat":41.240579522908,"lng":-8.45049005221912},{"lat":41.2406886591364,"lng":-8.45045562791646},{"lat":41.2407941729117,"lng":-8.45039638293313},{"lat":41.2408996089746,"lng":-8.45033199711433},{"lat":41.2409749886633,"lng":-8.45029252179261},{"lat":41.2410579893081,"lng":-8.45020310354844},{"lat":41.2411071955146,"lng":-8.45008870310333},{"lat":41.2411450104513,"lng":-8.45002399458729},{"lat":41.2411753767142,"lng":-8.4499508983883},{"lat":41.2412175273674,"lng":-8.44991555293118},{"lat":41.241334667656,"lng":-8.44972155769753},{"lat":41.2414216841686,"lng":-8.44957727832673},{"lat":41.241467029669,"lng":-8.4494926872554},{"lat":41.2415012639118,"lng":-8.4494199679994},{"lat":41.2415843295416,"lng":-8.44927376150444},{"lat":41.2416561919838,"lng":-8.44913942381846},{"lat":41.2416752295915,"lng":-8.4490597133545},{"lat":41.2417169013863,"lng":-8.44893525528495},{"lat":41.241781347651,"lng":-8.44878084360715},{"lat":41.2418268718919,"lng":-8.44863147380225},{"lat":41.2419250637798,"lng":-8.44851718707047},{"lat":41.2419628357617,"lng":-8.44846750999406},{"lat":41.2420080439154,"lng":-8.44843277790506},{"lat":41.2420757512868,"lng":-8.4484331057453},{"lat":41.2421773904734,"lng":-8.44843860464463},{"lat":41.2423052286536,"lng":-8.44847918504148},{"lat":41.2423691578929,"lng":-8.44847949476952},{"lat":41.2424593054876,"lng":-8.44853969819484},{"lat":41.2425343421286,"lng":-8.44862487975561},{"lat":41.2426092993287,"lng":-8.4487400052665},{"lat":41.2426626363159,"lng":-8.44885907941659},{"lat":41.2427501618874,"lng":-8.44839974550164},{"lat":41.2428159671786,"lng":-8.44807844730935},{"lat":41.242901212151,"lng":-8.4477596276189},{"lat":41.2430414219492,"lng":-8.44732822301082},{"lat":41.2431219079503,"lng":-8.44707081448182},{"lat":41.243153701554,"lng":-8.44696789764822},{"lat":41.2431372194639,"lng":-8.44687202564533},{"lat":41.2430689798383,"lng":-8.44676862560109},{"lat":41.2429620271166,"lng":-8.44656674383031},{"lat":41.2427047495272,"lng":-8.44605016101315},{"lat":41.2424346507581,"lng":-8.44548162578852},{"lat":41.2422724958277,"lng":-8.44511509945182},{"lat":41.2422302133048,"lng":-8.44500192104508},{"lat":41.2422192219432,"lng":-8.44493804893916},{"lat":41.242236339408,"lng":-8.44480321245691},{"lat":41.2423090576907,"lng":-8.44461686396946},{"lat":41.2423836583745,"lng":-8.44446750997433},{"lat":41.2424199442359,"lng":-8.44440135578567},{"lat":41.2424331032654,"lng":-8.44433031883714},{"lat":41.2424296632717,"lng":-8.44423916526739},{"lat":41.2424206718023,"lng":-8.44413378578869},{"lat":41.2423954745406,"lng":-8.44386358540875},{"lat":41.2423791136171,"lng":-8.44375555143706},{"lat":41.2423367519921,"lng":-8.44363748705913},{"lat":41.2422502007425,"lng":-8.4434136373912},{"lat":41.2421565318147,"lng":-8.44309157827238},{"lat":41.242085322097,"lng":-8.44288737369842},{"lat":41.2442153154563,"lng":-8.43999227397344},{"lat":41.2444555793559,"lng":-8.44011079692562}]},{"zone_id":"vlg3","points":[{"lat":41.2443706772297,"lng":-8.4637968201587},{"lat":41.2447436355314,"lng":-8.4571166449975},{"lat":41.2446409747197,"lng":-8.45642087348008},{"lat":41.2446286381755,"lng":-8.45600720561026},{"lat":41.2445021406361,"lng":-8.45404963829215},{"lat":41.2446897120868,"lng":-8.45392899638593},{"lat":41.247179962627,"lng":-8.4523451230722},{"lat":41.246895838091,"lng":-8.45226380040415},{"lat":41.2466928174875,"lng":-8.45241789320336},{"lat":41.2465505140575,"lng":-8.4524642017408},{"lat":41.2464227617383,"lng":-8.45242599734847},{"lat":41.2459615388152,"lng":-8.45217917480564},{"lat":41.2456066245767,"lng":-8.45200815320591},{"lat":41.2453474923971,"lng":-8.45190810375113},{"lat":41.2452159797856,"lng":-8.45189588655919},{"lat":41.2450062071586,"lng":-8.45191836655334},{"lat":41.2447148005338,"lng":-8.45193102026094},{"lat":41.2446488536174,"lng":-8.45194482895453},{"lat":41.2444409148187,"lng":-8.45198836929407},{"lat":41.2441600910071,"lng":-8.45201526566666},{"lat":41.2439146374671,"lng":-8.45207992068635},{"lat":41.2437510796439,"lng":-8.45216131360482},{"lat":41.2435164249446,"lng":-8.4521625510396},{"lat":41.2433368892646,"lng":-8.45216035838942},{"lat":41.2435892863754,"lng":-8.44962788725047},{"lat":41.2437427898159,"lng":-8.4480348508285},{"lat":41.2440298784504,"lng":-8.44443529133892},{"lat":41.2441774017744,"lng":-8.44249684235779},{"lat":41.244285057501,"lng":-8.44112473157883},{"lat":41.2443300484774,"lng":-8.44055124241095},{"lat":41.2444555793559,"lng":-8.44011079692562},{"lat":41.2442153154563,"lng":-8.43999227397344},{"lat":41.242085322097,"lng":-8.44288737369842},{"lat":41.2421565318147,"lng":-8.44309157827238},{"lat":41.2422502007425,"lng":-8.4434136373912},{"lat":41.2423367519921,"lng":-8.44363748705913},{"lat":41.2423791136171,"lng":-8.44375555143706},{"lat":41.2423954745406,"lng":-8.44386358540875},{"lat":41.2424206718023,"lng":-8.44413378578869},{"lat":41.2424296632717,"lng":-8.44423916526739},{"lat":41.2424331032654,"lng":-8.44433031883714},{"lat":41.2424199442359,"lng":-8.44440135578567},{"lat":41.2423836583745,"lng":-8.44446750997433},{"lat":41.2423090576907,"lng":-8.44461686396946},{"lat":41.242236339408,"lng":-8.44480321245691},{"lat":41.2422192219432,"lng":-8.44493804893916},{"lat":41.2422302133048,"lng":-8.44500192104508},{"lat":41.2422724958277,"lng":-8.44511509945182},{"lat":41.2424346507581,"lng":-8.44548162578852},{"lat":41.2427047495272,"lng":-8.44605016101315},{"lat":41.2429620271166,"lng":-8.44656674383031},{"lat":41.2430689798383,"lng":-8.44676862560109},{"lat":41.2431372194639,"lng":-8.44687202564533},{"lat":41.243153701554,"lng":-8.44696789764822},{"lat":41.2431219079503,"lng":-8.44707081448182},{"lat":41.2430414219492,"lng":-8.44732822301082},{"lat":41.242901212151,"lng":-8.4477596276189},{"lat":41.2428159671786,"lng":-8.44807844730935},{"lat":41.2427501618874,"lng":-8.44839974550164},{"lat":41.2426626363159,"lng":-8.44885907941659},{"lat":41.2426092993287,"lng":-8.4487400052665},{"lat":41.2425343421286,"lng":-8.44862487975561},{"lat":41.2424593054876,"lng":-8.44853969819484},{"lat":41.2423691578929,"lng":-8.44847949476952},{"lat":41.2423052286536,"lng":-8.44847918504148},{"lat":41.2421773904734,"lng":-8.44843860464463},{"lat":41.2420757512868,"lng":-8.4484331057453},{"lat":41.2420080439154,"lng":-8.44843277790506},{"lat":41.2419628357617,"lng":-8.44846750999406},{"lat":41.2419250637798,"lng":-8.44851718707047},{"lat":41.2418268718919,"lng":-8.44863147380225},{"lat":41.241781347651,"lng":-8.44878084360715},{"lat":41.2417169013863,"lng":-8.44893525528495},{"lat":41.2416752295915,"lng":-8.4490597133545},{"lat":41.2416561919838,"lng":-8.44913942381846},{"lat":41.2415843295416,"lng":-8.44927376150444},{"lat":41.2415012639118,"lng":-8.4494199679994},{"lat":41.241467029669,"lng":-8.4494926872554},{"lat":41.2414216841686,"lng":-8.44957727832673},{"lat":41.241334667656,"lng":-8.44972155769753},{"lat":41.2412175273674,"lng":-8.44991555293118},{"lat":41.2411753767142,"lng":-8.4499508983883},{"lat":41.2411450104513,"lng":-8.45002399458729},{"lat":41.2411071955146,"lng":-8.45008870310333},{"lat":41.2410579893081,"lng":-8.45020310354844},{"lat":41.2409749886633,"lng":-8.45029252179261},{"lat":41.2408996089746,"lng":-8.45033199711433},{"lat":41.2407941729117,"lng":-8.45039638293313},{"lat":41.2406886591364,"lng":-8.45045562791646},{"lat":41.240579522908,"lng":-8.45049005221912},{"lat":41.2404778631374,"lng":-8.45052438613429},{"lat":41.2403874665736,"lng":-8.45055388830447},{"lat":41.240331042533,"lng":-8.450543715364},{"lat":41.2403351798615,"lng":-8.45041406546069},{"lat":41.2403845593136,"lng":-8.45023978818447},{"lat":41.2405356479174,"lng":-8.45001112707146},{"lat":41.2405171759204,"lng":-8.44988637896412},{"lat":41.2405050143736,"lng":-8.44979029236353},{"lat":41.2404619463998,"lng":-8.44973520955481},{"lat":41.2404008001797,"lng":-8.44967503414901},{"lat":41.2403255667269,"lng":-8.44965975742442},{"lat":41.24025036856,"lng":-8.44966439761525},{"lat":41.2401618648722,"lng":-8.44969391119778},{"lat":41.2401131053804,"lng":-8.44964632132639},{"lat":41.240071245886,"lng":-8.44958031575877},{"lat":41.2400382106805,"lng":-8.44950876820471},{"lat":41.23993869896,"lng":-8.44954694053861},{"lat":41.2396342241899,"lng":-8.44946565316505},{"lat":41.2394200211802,"lng":-8.44939984405719},{"lat":41.2391907459526,"lng":-8.44934386009183},{"lat":41.2389049096111,"lng":-8.44932756060077},{"lat":41.2388484582601,"lng":-8.44932728731938},{"lat":41.2387240459143,"lng":-8.44941639070106},{"lat":41.2386487098428,"lng":-8.44947089611395},{"lat":41.2385998841617,"lng":-8.44944572247149},{"lat":41.2386302314558,"lng":-8.44937907503012},{"lat":41.2386860703756,"lng":-8.44921151152013},{"lat":41.2387024985331,"lng":-8.44900188325969},{"lat":41.2386948633992,"lng":-8.44889699638174},{"lat":41.2386957607,"lng":-8.44857194350714},{"lat":41.2386488581544,"lng":-8.44837250803255},{"lat":41.2386571870077,"lng":-8.44822582453855},{"lat":41.2386022169902,"lng":-8.44807872141279},{"lat":41.2385159264496,"lng":-8.44779523634468},{"lat":41.2384215152689,"lng":-8.44761656875191},{"lat":41.2382956694372,"lng":-8.44737476766402},{"lat":41.2382013425972,"lng":-8.44716460763687},{"lat":41.2381464612422,"lng":-8.44698613211567},{"lat":41.2381232063897,"lng":-8.44679731090297},{"lat":41.2381238382279,"lng":-8.44656661751882},{"lat":41.238092695572,"lng":-8.44639875564651},{"lat":41.2380853735517,"lng":-8.44617851684168},{"lat":41.2380225616279,"lng":-8.4460000046382},{"lat":41.2379023279676,"lng":-8.44583597906058},{"lat":41.237746811705,"lng":-8.44562114538935},{"lat":41.2375811726638,"lng":-8.44544214155965},{"lat":41.2374787519725,"lng":-8.44532630394129},{"lat":41.2373366514036,"lng":-8.44519977965935},{"lat":41.2371315460825,"lng":-8.44503108028918},{"lat":41.2370290340196,"lng":-8.44491524486877},{"lat":41.2369740065248,"lng":-8.44478913717206},{"lat":41.2369583293893,"lng":-8.44472620105151},{"lat":41.2369902743775,"lng":-8.44460051012962},{"lat":41.2370382959094,"lng":-8.44439104316409},{"lat":41.2370707066978,"lng":-8.44412913239011},{"lat":41.2371665763005,"lng":-8.4437730532027},{"lat":41.2372384686099,"lng":-8.44349235971942},{"lat":41.2372865216268,"lng":-8.44330388186611},{"lat":41.2373026496483,"lng":-8.44316762256777},{"lat":41.2373109436124,"lng":-8.44303143832478},{"lat":41.2373274397946,"lng":-8.44279032674387},{"lat":41.2373214663552,"lng":-8.44256962370043},{"lat":41.2372227462269,"lng":-8.44251678488742},{"lat":41.2371634203965,"lng":-8.44251125659274},{"lat":41.2370962027348,"lng":-8.44249519405042},{"lat":41.2370213362569,"lng":-8.44241098167129},{"lat":41.2369422067325,"lng":-8.442405358942},{"lat":41.2368315750444,"lng":-8.44239433684137},{"lat":41.236748466906,"lng":-8.44239394150374},{"lat":41.2364589434904,"lng":-8.44237682417836},{"lat":41.2363166745304,"lng":-8.44234465567932},{"lat":41.2361703864593,"lng":-8.44233346336475},{"lat":41.2360833801417,"lng":-8.44234354781615},{"lat":41.2359682157534,"lng":-8.44251082961083},{"lat":41.2357792039421,"lng":-8.44278249097428},{"lat":41.2356878918488,"lng":-8.4429183969839},{"lat":41.2356018410579,"lng":-8.44297524239943},{"lat":41.2354699923776,"lng":-8.44298641908367},{"lat":41.2353084008374,"lng":-8.44297384709395},{"lat":41.2352257352105,"lng":-8.44294351355671},{"lat":41.235060336909,"lng":-8.44287294966224},{"lat":41.2348271361239,"lng":-8.44287184041156},{"lat":41.234751807961,"lng":-8.44289140461637},{"lat":41.2346914420355,"lng":-8.44297091450611},{"lat":41.234640344616,"lng":-8.44312048545812},{"lat":41.2345980034636,"lng":-8.44303034468656},{"lat":41.2345833145382,"lng":-8.44290062270484},{"lat":41.2345081483623,"lng":-8.44286030668668},{"lat":41.2343953837338,"lng":-8.44283984870544},{"lat":41.2342823661847,"lng":-8.44287927027972},{"lat":41.2340292324141,"lng":-8.4429876819591},{"lat":41.2339388525143,"lng":-8.4430471319328},{"lat":41.2338105259016,"lng":-8.44318619747857},{"lat":41.2337124967215,"lng":-8.4432754232803},{"lat":41.2336143115041,"lng":-8.4434245341101},{"lat":41.2335913040206,"lng":-8.44357411707436},{"lat":41.2334913426,"lng":-8.44381207136264},{"lat":41.2333470947619,"lng":-8.44371250709786},{"lat":41.2331442384666,"lng":-8.44361182504253},{"lat":41.2329564960883,"lng":-8.44348127417935},{"lat":41.2327235891409,"lng":-8.44337043331541},{"lat":41.2325506915081,"lng":-8.44330985125872},{"lat":41.2324362856847,"lng":-8.44323046775137},{"lat":41.232402562668,"lng":-8.44308276390838},{"lat":41.2323791445204,"lng":-8.4429515725372},{"lat":41.232324548916,"lng":-8.44266302608229},{"lat":41.2322931310503,"lng":-8.4425947725599},{"lat":41.2323329495191,"lng":-8.44238670935887},{"lat":41.2323217799609,"lng":-8.44222264996407},{"lat":41.2323261504944,"lng":-8.44207071542554},{"lat":41.2322629975907,"lng":-8.44201793674036},{"lat":41.232124816401,"lng":-8.44180553725096},{"lat":41.2319793887997,"lng":-8.44160783815361},{"lat":41.2318729421259,"lng":-8.44148150729464},{"lat":41.2317547311127,"lng":-8.44134210989119},{"lat":41.2316837943372,"lng":-8.44123693658766},{"lat":41.231605158848,"lng":-8.44107923939437},{"lat":41.2315313757599,"lng":-8.44099420706937},{"lat":41.231469352803,"lng":-8.44092139965749},{"lat":41.2314047471915,"lng":-8.44087123522134},{"lat":41.2313494436458,"lng":-8.44084473435906},{"lat":41.2312981729334,"lng":-8.4407920121614},{"lat":41.2312430861752,"lng":-8.44071839921743},{"lat":41.2311839755264,"lng":-8.44063427502077},{"lat":41.2311288126455,"lng":-8.44055541035846},{"lat":41.2311012339934,"lng":-8.44049755507636},{"lat":41.2310613734437,"lng":-8.44045288050399},{"lat":41.2309865485861,"lng":-8.44035281506224},{"lat":41.2309312708014,"lng":-8.44031593695161},{"lat":41.2308691575523,"lng":-8.44044337490543},{"lat":41.2308318597389,"lng":-8.44051989869467},{"lat":41.2307482857471,"lng":-8.44069257001456},{"lat":41.2306682960386,"lng":-8.44090568644582},{"lat":41.2303284523601,"lng":-8.44064908124579},{"lat":41.2301293888955,"lng":-8.44050847621246},{"lat":41.2298999035947,"lng":-8.4404973765384},{"lat":41.2298003129411,"lng":-8.44049092677889},{"lat":41.2296516090764,"lng":-8.44048129635368},{"lat":41.229516255964,"lng":-8.44045573244382},{"lat":41.2294183987799,"lng":-8.4404502600327},{"lat":41.2293582359116,"lng":-8.44045498765222},{"lat":41.229250493302,"lng":-8.44049317165067},{"lat":41.2291788099057,"lng":-8.44051857618633},{"lat":41.229025126942,"lng":-8.44057304110702},{"lat":41.2287728536641,"lng":-8.44063172656647},{"lat":41.2284315527626,"lng":-8.44081569774178},{"lat":41.2280170589119,"lng":-8.44102818056797},{"lat":41.2278926393642,"lng":-8.44112228956018},{"lat":41.2278359797048,"lng":-8.44119680453829},{"lat":41.2277969902824,"lng":-8.44129811473661},{"lat":41.2277525128738,"lng":-8.4413966594597},{"lat":41.227677271019,"lng":-8.44138639968295},{"lat":41.2273774274542,"lng":-8.44118783954049},{"lat":41.2271213915421,"lng":-8.44150494440122},{"lat":41.2266375127275,"lng":-8.44063346451616},{"lat":41.2255526494989,"lng":-8.44058314383367},{"lat":41.2246796659421,"lng":-8.44116471690741},{"lat":41.224504932082,"lng":-8.4413306430153},{"lat":41.2244566843846,"lng":-8.44137645851112},{"lat":41.2230370608672,"lng":-8.44272447985665},{"lat":41.2222180151174,"lng":-8.443032809779},{"lat":41.2215604333138,"lng":-8.44328035079372},{"lat":41.2215090955497,"lng":-8.44332460878993},{"lat":41.2208002148612,"lng":-8.44356652208149},{"lat":41.220722263414,"lng":-8.44362079741745},{"lat":41.2194442968474,"lng":-8.44369193043697},{"lat":41.2172202281214,"lng":-8.44404030469798},{"lat":41.2164382461641,"lng":-8.44465665160172},{"lat":41.215197724331,"lng":-8.44651080422016},{"lat":41.2146504358338,"lng":-8.44679811624086},{"lat":41.2145721694502,"lng":-8.4468515800786},{"lat":41.2145211084512,"lng":-8.44692447625593},{"lat":41.2142848944081,"lng":-8.44726170000671},{"lat":41.2141165561511,"lng":-8.44737738891443},{"lat":41.2137253112376,"lng":-8.44764546627593},{"lat":41.2129933029594,"lng":-8.44802588672875},{"lat":41.2125427624859,"lng":-8.44824620880833},{"lat":41.2124013983395,"lng":-8.44831525431448},{"lat":41.2120288310784,"lng":-8.44849722323007},{"lat":41.2111225542264,"lng":-8.44889216696834},{"lat":41.2106107870664,"lng":-8.44930415417641},{"lat":41.2102381379526,"lng":-8.44974768653585},{"lat":41.2096910251052,"lng":-8.45040546393868},{"lat":41.2091008726644,"lng":-8.45092191787447},{"lat":41.208470681253,"lng":-8.4511212388367},{"lat":41.207773840883,"lng":-8.45128655084114},{"lat":41.2065877088636,"lng":-8.45142086456992},{"lat":41.2058795112272,"lng":-8.45155571299657},{"lat":41.2050555155893,"lng":-8.45159771572693},{"lat":41.2044929362685,"lng":-8.45158665432052},{"lat":41.2043714079655,"lng":-8.45158426486616},{"lat":41.2038690964732,"lng":-8.45167664175382},{"lat":41.2035873547821,"lng":-8.4514408358203},{"lat":41.2032420932173,"lng":-8.45139929986107},{"lat":41.2030944496959,"lng":-8.45147622407982},{"lat":41.202825613149,"lng":-8.45157703268635},{"lat":41.2025262017415,"lng":-8.45141643315363},{"lat":41.2018796670744,"lng":-8.45026198470934},{"lat":41.2018419966663,"lng":-8.45019145546318},{"lat":41.2013370051744,"lng":-8.44924598979097},{"lat":41.2007882548191,"lng":-8.44824667546025},{"lat":41.2002337825347,"lng":-8.44726154095329},{"lat":41.1999334125935,"lng":-8.44679968472362},{"lat":41.1996555450579,"lng":-8.44656790161913},{"lat":41.1991334840397,"lng":-8.44653463885896},{"lat":41.1987368602049,"lng":-8.44656289738399},{"lat":41.1971151983923,"lng":-8.44659945639452},{"lat":41.196857696587,"lng":-8.44692225723196},{"lat":41.1963902088825,"lng":-8.44750828422759},{"lat":41.1953821242057,"lng":-8.44880612694044},{"lat":41.1952905490665,"lng":-8.44893930908102},{"lat":41.1948023890191,"lng":-8.44954132944726},{"lat":41.1943273691588,"lng":-8.45020920286753},{"lat":41.1941293148818,"lng":-8.45048670167009},{"lat":41.1937714207457,"lng":-8.45069474475897},{"lat":41.1937180627608,"lng":-8.45072576142772},{"lat":41.1927499551242,"lng":-8.45128850686462},{"lat":41.1926742294001,"lng":-8.45133252424703},{"lat":41.1926393846345,"lng":-8.45201121299363},{"lat":41.1926367115629,"lng":-8.45296903794257},{"lat":41.1926362511436,"lng":-8.45313372948505},{"lat":41.1926089259573,"lng":-8.45587501021695},{"lat":41.1926238714911,"lng":-8.45756311070822},{"lat":41.1923341530974,"lng":-8.45731654041544},{"lat":41.19204314822,"lng":-8.45706887790161},{"lat":41.19192468492,"lng":-8.45699486158022},{"lat":41.191672024255,"lng":-8.45683649727765},{"lat":41.1915378307847,"lng":-8.45675203972799},{"lat":41.1914194233389,"lng":-8.45668851337736},{"lat":41.1913403095748,"lng":-8.45667763053134},{"lat":41.1912059686771,"lng":-8.45664561320579},{"lat":41.191142875442,"lng":-8.45657187828243},{"lat":41.1910796640216,"lng":-8.45654021371889},{"lat":41.1909373095512,"lng":-8.45653950835598},{"lat":41.1908425258855,"lng":-8.45649708209158},{"lat":41.1907080959956,"lng":-8.456496417015},{"lat":41.1906606751615,"lng":-8.45648569047158},{"lat":41.1903384276637,"lng":-8.45662174056076},{"lat":41.1900114605635,"lng":-8.45715213598946},{"lat":41.1897190807296,"lng":-8.45750971669013},{"lat":41.189336686945,"lng":-8.45781878231549},{"lat":41.1888255656644,"lng":-8.45768143544646},{"lat":41.1889133161139,"lng":-8.45740950708289},{"lat":41.1889611231332,"lng":-8.45728399356023},{"lat":41.1889932903785,"lng":-8.45711656571339},{"lat":41.1890094050165,"lng":-8.45699089445274},{"lat":41.189025846892,"lng":-8.45678142548539},{"lat":41.1889392488323,"lng":-8.45664487471171},{"lat":41.1888682952951,"lng":-8.45655012260793},{"lat":41.1888211691809,"lng":-8.45643474767509},{"lat":41.1888055864482,"lng":-8.45634038390658},{"lat":41.1887349256449,"lng":-8.45617232776675},{"lat":41.1886955466868,"lng":-8.45611980805276},{"lat":41.1886886840132,"lng":-8.45603264037448},{"lat":41.1887481522031,"lng":-8.45598573064349},{"lat":41.1888034535877,"lng":-8.45601222801554},{"lat":41.1888825674293,"lng":-8.45599164399393},{"lat":41.1889301947201,"lng":-8.45592906468135},{"lat":41.1890018335967,"lng":-8.45578268477175},{"lat":41.1889825129591,"lng":-8.45560975398448},{"lat":41.1889512593841,"lng":-8.45548384764451},{"lat":41.1888761043182,"lng":-8.45539466927455},{"lat":41.1888212308907,"lng":-8.45532611071751},{"lat":41.1887801843508,"lng":-8.45529050683198},{"lat":41.1887624832053,"lng":-8.45486131278819},{"lat":41.1887903644324,"lng":-8.45480900505381},{"lat":41.1888300966927,"lng":-8.45473589342901},{"lat":41.188842143311,"lng":-8.45467826582974},{"lat":41.1888703006036,"lng":-8.45449507862407},{"lat":41.188906458838,"lng":-8.45431705941277},{"lat":41.1889185926626,"lng":-8.45422808087973},{"lat":41.1889190633983,"lng":-8.45406049566001},{"lat":41.1889035079437,"lng":-8.45395564191132},{"lat":41.1888721524019,"lng":-8.45383498425296},{"lat":41.1888054962836,"lng":-8.45365121259946},{"lat":41.1887426356824,"lng":-8.45349380826358},{"lat":41.1887111779127,"lng":-8.45340973685913},{"lat":41.1886483641884,"lng":-8.4532680659666},{"lat":41.1885971433163,"lng":-8.45319963725014},{"lat":41.1885103237538,"lng":-8.45311016760051},{"lat":41.1884708800396,"lng":-8.45307850893145},{"lat":41.1883831774855,"lng":-8.45301514403321},{"lat":41.1884053267564,"lng":-8.45284174242835},{"lat":41.1885318684602,"lng":-8.45171259122735},{"lat":41.1884195625352,"lng":-8.45159261343531},{"lat":41.1883370946436,"lng":-8.45152260095171},{"lat":41.1882396276579,"lng":-8.451472658902},{"lat":41.1880217113951,"lng":-8.45151140628213},{"lat":41.1878942123554,"lng":-8.45144677840056},{"lat":41.187755366254,"lng":-8.45131689498068},{"lat":41.1876891471911,"lng":-8.45120131296054},{"lat":41.1875705342364,"lng":-8.45117976184907},{"lat":41.1874522101424,"lng":-8.45108489929941},{"lat":41.1873812760981,"lng":-8.45097978672764},{"lat":41.1873340280447,"lng":-8.45090625140049},{"lat":41.1872776207982,"lng":-8.45075924833245},{"lat":41.186868253981,"lng":-8.45061053574511},{"lat":41.1867342273315,"lng":-8.45043181260557},{"lat":41.1865128428556,"lng":-8.45042024650684},{"lat":41.1859058080369,"lng":-8.4507342191425},{"lat":41.1858772920265,"lng":-8.45092002257812},{"lat":41.1858611808395,"lng":-8.4510142216117},{"lat":41.185766976381,"lng":-8.45076239139279},{"lat":41.1856513346965,"lng":-8.45062543266359},{"lat":41.1854335879819,"lng":-8.45044707684566},{"lat":41.1853899373532,"lng":-8.4503643281021},{"lat":41.185361268099,"lng":-8.45029235112452},{"lat":41.1853233360724,"lng":-8.45022950040323},{"lat":41.1852796053418,"lng":-8.45017579265862},{"lat":41.1852046518224,"lng":-8.45013874765352},{"lat":41.1851446418794,"lng":-8.4501262304135},{"lat":41.1850903590802,"lng":-8.45013055403796},{"lat":41.1850256168009,"lng":-8.45015928277967},{"lat":41.1849747728077,"lng":-8.45017279378818},{"lat":41.1849263139201,"lng":-8.45015880380635},{"lat":41.184794761122,"lng":-8.45009080699592},{"lat":41.184604621769,"lng":-8.44995055426918},{"lat":41.184506241994,"lng":-8.44987176687952},{"lat":41.1843225650635,"lng":-8.44974025088932},{"lat":41.1841257334135,"lng":-8.44960878552824},{"lat":41.183955095755,"lng":-8.4495469324871},{"lat":41.1838106315623,"lng":-8.44949403159865},{"lat":41.1836399255493,"lng":-8.44942359971078},{"lat":41.1835085420677,"lng":-8.44939686310587},{"lat":41.1834034124633,"lng":-8.44941375024624},{"lat":41.1832326842853,"lng":-8.44935202430537},{"lat":41.1830022404358,"lng":-8.44923184137263},{"lat":41.1823392298177,"lng":-8.44898049049691},{"lat":41.1821094855588,"lng":-8.44893373287179},{"lat":41.1818409384317,"lng":-8.44881718558564},{"lat":41.1813603164968,"lng":-8.44864514364665},{"lat":41.1812061793556,"lng":-8.44860769663147},{"lat":41.1810323044452,"lng":-8.44854404094135},{"lat":41.1808308442476,"lng":-8.44849075084877},{"lat":41.1807083378539,"lng":-8.44844296700211},{"lat":41.180613593719,"lng":-8.44838494287288},{"lat":41.1801935984945,"lng":-8.44745057249488},{"lat":41.1801824411589,"lng":-8.44721692270684},{"lat":41.1801950737634,"lng":-8.44691319476543},{"lat":41.1802236527952,"lng":-8.44660430234464},{"lat":41.1802521846469,"lng":-8.44627967272028},{"lat":41.1802725527698,"lng":-8.44607549637539},{"lat":41.1802648142859,"lng":-8.44600728420004},{"lat":41.1802294912473,"lng":-8.44591808762491},{"lat":41.180130957498,"lng":-8.44579712903094},{"lat":41.1800284872182,"lng":-8.44566577618923},{"lat":41.179902286824,"lng":-8.44551846590873},{"lat":41.1796104396639,"lng":-8.44522901454839},{"lat":41.1794329756555,"lng":-8.44506059677589},{"lat":41.1792537423075,"lng":-8.44491303629187},{"lat":41.1790168277025,"lng":-8.44478617275808},{"lat":41.1788746155019,"lng":-8.44473340907235},{"lat":41.17874022435,"lng":-8.44468533898991},{"lat":41.1786178686076,"lng":-8.44464816779355},{"lat":41.1782146825137,"lng":-8.44457283077001},{"lat":41.1780171808243,"lng":-8.44448810515726},{"lat":41.1778987370202,"lng":-8.44440376509103},{"lat":41.1777883316677,"lng":-8.4443089674676},{"lat":41.1777094144657,"lng":-8.44422481624605},{"lat":41.1775910280165,"lng":-8.44415083972505},{"lat":41.1774805616044,"lng":-8.44404555993182},{"lat":41.1772987602638,"lng":-8.44398189083782},{"lat":41.176313503752,"lng":-8.44355403211219},{"lat":41.174295218585,"lng":-8.44196866721348},{"lat":41.1720852884861,"lng":-8.4387422924561},{"lat":41.1667430158109,"lng":-8.43978643119635},{"lat":41.1651217851655,"lng":-8.44098035296278},{"lat":41.1623388733427,"lng":-8.44801453414585},{"lat":41.1625097957489,"lng":-8.44999988144493},{"lat":41.1626442059236,"lng":-8.45215340705947},{"lat":41.1625570196529,"lng":-8.45219492000724},{"lat":41.1624464770767,"lng":-8.45212099035865},{"lat":41.1623674151221,"lng":-8.45208914567783},{"lat":41.1622647632579,"lng":-8.45202585414885},{"lat":41.1621779558341,"lng":-8.45193118914554},{"lat":41.1621063851994,"lng":-8.45205654036554},{"lat":41.1619238874157,"lng":-8.45224413960863},{"lat":41.1616700764475,"lng":-8.45246296103764},{"lat":41.1615590902168,"lng":-8.45254618176465},{"lat":41.1614720219044,"lng":-8.45254575541747},{"lat":41.1614167617947,"lng":-8.45250366164859},{"lat":41.1613615900757,"lng":-8.45243000391995},{"lat":41.161290604095,"lng":-8.4523458954427},{"lat":41.1612354913011,"lng":-8.45225125888609},{"lat":41.1611724824912,"lng":-8.45214622318979},{"lat":41.1611061692595,"lng":-8.45209669283504},{"lat":41.1610658309955,"lng":-8.45225746073143},{"lat":41.161041858553,"lng":-8.45233061837935},{"lat":41.1610533339902,"lng":-8.45247734469285},{"lat":41.1610568603679,"lng":-8.45263439343778},{"lat":41.1610566849467,"lng":-8.4526973007915},{"lat":41.1610683936182,"lng":-8.45276014793239},{"lat":41.1611035951233,"lng":-8.45289126473548},{"lat":41.1611665978564,"lng":-8.45299629862144},{"lat":41.1612375450787,"lng":-8.45309613365286},{"lat":41.1612610336286,"lng":-8.45316427842862},{"lat":41.1612844992083,"lng":-8.4532743664587},{"lat":41.1612762816421,"lng":-8.45337905189799},{"lat":41.1612642211079,"lng":-8.45344190397062},{"lat":41.161220524126,"lng":-8.45351496414954},{"lat":41.1611649578715,"lng":-8.45358272871878},{"lat":41.1611292780395,"lng":-8.45362448978823},{"lat":41.1610578332888,"lng":-8.45370265573784},{"lat":41.1610063187322,"lng":-8.45373909943693},{"lat":41.1609349903256,"lng":-8.45377544769327},{"lat":41.1608756578922,"lng":-8.45380648800157},{"lat":41.1608162579499,"lng":-8.4538271681675},{"lat":41.1607529480447,"lng":-8.4538320971919},{"lat":41.1606976011364,"lng":-8.45382134610484},{"lat":41.1606184444124,"lng":-8.45385765593714},{"lat":41.1604996210519,"lng":-8.45390937557413},{"lat":41.1611808743471,"lng":-8.45542256929544},{"lat":41.162166297975,"lng":-8.45721462731388},{"lat":41.1643171433877,"lng":-8.46079980451593},{"lat":41.1665606643786,"lng":-8.46343244151535},{"lat":41.1702406275569,"lng":-8.46750250177278},{"lat":41.1711368653416,"lng":-8.46893703958536},{"lat":41.172479431922,"lng":-8.47168474116369},{"lat":41.1736422041544,"lng":-8.47431245028817},{"lat":41.1740899272663,"lng":-8.4751489648805},{"lat":41.1747184382457,"lng":-8.47574809647406},{"lat":41.1774157511783,"lng":-8.47707312717412},{"lat":41.1805148661389,"lng":-8.47798339301606},{"lat":41.18172615526,"lng":-8.47919828306483},{"lat":41.1847214904875,"lng":-8.47943329753867},{"lat":41.1878540580845,"lng":-8.47927342979088},{"lat":41.1925343842822,"lng":-8.47989418040302},{"lat":41.1962360944192,"lng":-8.47657592621016},{"lat":41.2152806140713,"lng":-8.49110436629046},{"lat":41.240638658065,"lng":-8.48789961475316},{"lat":41.2413473507551,"lng":-8.47983846027266},{"lat":41.2429313083021,"lng":-8.47037261316188},{"lat":41.2438412697375,"lng":-8.46605265180454},{"lat":41.2443706772297,"lng":-8.4637968201587}]},{"zone_id":"prd6","points":[{"lat":41.2038690964732,"lng":-8.45167664175382},{"lat":41.2043714079655,"lng":-8.45158426486616},{"lat":41.2044929362685,"lng":-8.45158665432052},{"lat":41.2050555155893,"lng":-8.45159771572693},{"lat":41.2058795112272,"lng":-8.45155571299657},{"lat":41.2065877088636,"lng":-8.45142086456992},{"lat":41.207773840883,"lng":-8.45128655084114},{"lat":41.208470681253,"lng":-8.4511212388367},{"lat":41.2091008726644,"lng":-8.45092191787447},{"lat":41.2096910251052,"lng":-8.45040546393868},{"lat":41.2102381379526,"lng":-8.44974768653585},{"lat":41.2106107870664,"lng":-8.44930415417641},{"lat":41.2111225542264,"lng":-8.44889216696834},{"lat":41.2120288310784,"lng":-8.44849722323007},{"lat":41.2124013983395,"lng":-8.44831525431448},{"lat":41.2125427624859,"lng":-8.44824620880833},{"lat":41.2129933029594,"lng":-8.44802588672875},{"lat":41.2137253112376,"lng":-8.44764546627593},{"lat":41.2141165561511,"lng":-8.44737738891443},{"lat":41.2142848944081,"lng":-8.44726170000671},{"lat":41.2145211084512,"lng":-8.44692447625593},{"lat":41.2145721694502,"lng":-8.4468515800786},{"lat":41.2175609713641,"lng":-8.42796951396181},{"lat":41.2178670015816,"lng":-8.42473685586075},{"lat":41.2180093098498,"lng":-8.42235017715168},{"lat":41.2180945365381,"lng":-8.42092042215992},{"lat":41.2181002308821,"lng":-8.41863295243233},{"lat":41.217927202661,"lng":-8.41634471124446},{"lat":41.2175155077144,"lng":-8.41421321765511},{"lat":41.2169833161577,"lng":-8.41263338332874},{"lat":41.2159763412535,"lng":-8.41034167555702},{"lat":41.2147521334127,"lng":-8.4075725826378},{"lat":41.2135510337712,"lng":-8.40768623638237},{"lat":41.209024537881,"lng":-8.40766731137218},{"lat":41.2044752666686,"lng":-8.40720666396524},{"lat":41.2009037729187,"lng":-8.40632431889824},{"lat":41.197662132449,"lng":-8.40623739829209},{"lat":41.1947970068883,"lng":-8.40559587916679},{"lat":41.1895593540536,"lng":-8.40406034073962},{"lat":41.18460756063,"lng":-8.40252622092094},{"lat":41.1794651513396,"lng":-8.40099155754089},{"lat":41.1749880311084,"lng":-8.40021657423883},{"lat":41.1719368231132,"lng":-8.40096088877244},{"lat":41.1690599229534,"lng":-8.40149631094441},{"lat":41.1665800246304,"lng":-8.40206217515498},{"lat":41.1676481102981,"lng":-8.40376902452034},{"lat":41.1693580706859,"lng":-8.40604609624724},{"lat":41.1714952301171,"lng":-8.40898718931625},{"lat":41.1727764850794,"lng":-8.41116812730928},{"lat":41.1740586181459,"lng":-8.41297078797765},{"lat":41.175339333169,"lng":-8.41534107322565},{"lat":41.1759776908824,"lng":-8.41733029336298},{"lat":41.1761150090686,"lng":-8.41960115462854},{"lat":41.1757522201801,"lng":-8.42177523508581},{"lat":41.1750320336279,"lng":-8.4239477107101},{"lat":41.1739542088169,"lng":-8.42621313649745},{"lat":41.1728468380065,"lng":-8.42947163556495},{"lat":41.1723282573885,"lng":-8.43321527676606},{"lat":41.172386288967,"lng":-8.43597961653675},{"lat":41.1720852884861,"lng":-8.4387422924561},{"lat":41.174295218585,"lng":-8.44196866721348},{"lat":41.176313503752,"lng":-8.44355403211219},{"lat":41.1772987602638,"lng":-8.44398189083782},{"lat":41.1774805616044,"lng":-8.44404555993182},{"lat":41.1775910280165,"lng":-8.44415083972505},{"lat":41.1777094144657,"lng":-8.44422481624605},{"lat":41.1777883316677,"lng":-8.4443089674676},{"lat":41.1778987370202,"lng":-8.44440376509103},{"lat":41.1780171808243,"lng":-8.44448810515726},{"lat":41.1782146825137,"lng":-8.44457283077001},{"lat":41.1786178686076,"lng":-8.44464816779355},{"lat":41.17874022435,"lng":-8.44468533898991},{"lat":41.1788746155019,"lng":-8.44473340907235},{"lat":41.1790168277025,"lng":-8.44478617275808},{"lat":41.1792537423075,"lng":-8.44491303629187},{"lat":41.1794329756555,"lng":-8.44506059677589},{"lat":41.1796104396639,"lng":-8.44522901454839},{"lat":41.179902286824,"lng":-8.44551846590873},{"lat":41.1800284872182,"lng":-8.44566577618923},{"lat":41.180130957498,"lng":-8.44579712903094},{"lat":41.1802294912473,"lng":-8.44591808762491},{"lat":41.1802648142859,"lng":-8.44600728420004},{"lat":41.1802725527698,"lng":-8.44607549637539},{"lat":41.1802521846469,"lng":-8.44627967272028},{"lat":41.1802236527952,"lng":-8.44660430234464},{"lat":41.1801950737634,"lng":-8.44691319476543},{"lat":41.1801824411589,"lng":-8.44721692270684},{"lat":41.1801935984945,"lng":-8.44745057249488},{"lat":41.180613593719,"lng":-8.44838494287288},{"lat":41.1807083378539,"lng":-8.44844296700211},{"lat":41.1808308442476,"lng":-8.44849075084877},{"lat":41.1810323044452,"lng":-8.44854404094135},{"lat":41.1812061793556,"lng":-8.44860769663147},{"lat":41.1813603164968,"lng":-8.44864514364665},{"lat":41.1818409384317,"lng":-8.44881718558564},{"lat":41.1821094855588,"lng":-8.44893373287179},{"lat":41.1823392298177,"lng":-8.44898049049691},{"lat":41.1830022404358,"lng":-8.44923184137263},{"lat":41.1832326842853,"lng":-8.44935202430537},{"lat":41.1834034124633,"lng":-8.44941375024624},{"lat":41.1835085420677,"lng":-8.44939686310587},{"lat":41.1836399255493,"lng":-8.44942359971078},{"lat":41.1838106315623,"lng":-8.44949403159865},{"lat":41.183955095755,"lng":-8.4495469324871},{"lat":41.1841257334135,"lng":-8.44960878552824},{"lat":41.1843225650635,"lng":-8.44974025088932},{"lat":41.184506241994,"lng":-8.44987176687952},{"lat":41.184604621769,"lng":-8.44995055426918},{"lat":41.184794761122,"lng":-8.45009080699592},{"lat":41.1849263139201,"lng":-8.45015880380635},{"lat":41.1849747728077,"lng":-8.45017279378818},{"lat":41.1850256168009,"lng":-8.45015928277967},{"lat":41.1850903590802,"lng":-8.45013055403796},{"lat":41.1851446418794,"lng":-8.4501262304135},{"lat":41.1852046518224,"lng":-8.45013874765352},{"lat":41.1852796053418,"lng":-8.45017579265862},{"lat":41.1853233360724,"lng":-8.45022950040323},{"lat":41.185361268099,"lng":-8.45029235112452},{"lat":41.1853899373532,"lng":-8.4503643281021},{"lat":41.1854335879819,"lng":-8.45044707684566},{"lat":41.1856513346965,"lng":-8.45062543266359},{"lat":41.185766976381,"lng":-8.45076239139279},{"lat":41.1858611808395,"lng":-8.4510142216117},{"lat":41.1858772920265,"lng":-8.45092002257812},{"lat":41.1859058080369,"lng":-8.4507342191425},{"lat":41.1865128428556,"lng":-8.45042024650684},{"lat":41.1867342273315,"lng":-8.45043181260557},{"lat":41.186868253981,"lng":-8.45061053574511},{"lat":41.1872776207982,"lng":-8.45075924833245},{"lat":41.1873340280447,"lng":-8.45090625140049},{"lat":41.1873812760981,"lng":-8.45097978672764},{"lat":41.1874522101424,"lng":-8.45108489929941},{"lat":41.1875705342364,"lng":-8.45117976184907},{"lat":41.1876891471911,"lng":-8.45120131296054},{"lat":41.187755366254,"lng":-8.45131689498068},{"lat":41.1878942123554,"lng":-8.45144677840056},{"lat":41.1880217113951,"lng":-8.45151140628213},{"lat":41.1882396276579,"lng":-8.451472658902},{"lat":41.1883370946436,"lng":-8.45152260095171},{"lat":41.1884195625352,"lng":-8.45159261343531},{"lat":41.1885318684602,"lng":-8.45171259122735},{"lat":41.1884053267564,"lng":-8.45284174242835},{"lat":41.1883831774855,"lng":-8.45301514403321},{"lat":41.1884708800396,"lng":-8.45307850893145},{"lat":41.1885103237538,"lng":-8.45311016760051},{"lat":41.1885971433163,"lng":-8.45319963725014},{"lat":41.1886483641884,"lng":-8.4532680659666},{"lat":41.1887111779127,"lng":-8.45340973685913},{"lat":41.1887426356824,"lng":-8.45349380826358},{"lat":41.1888054962836,"lng":-8.45365121259946},{"lat":41.1888721524019,"lng":-8.45383498425296},{"lat":41.1889035079437,"lng":-8.45395564191132},{"lat":41.1889190633983,"lng":-8.45406049566001},{"lat":41.1889185926626,"lng":-8.45422808087973},{"lat":41.188906458838,"lng":-8.45431705941277},{"lat":41.1888703006036,"lng":-8.45449507862407},{"lat":41.188842143311,"lng":-8.45467826582974},{"lat":41.1888300966927,"lng":-8.45473589342901},{"lat":41.1887903644324,"lng":-8.45480900505381},{"lat":41.1887624832053,"lng":-8.45486131278819},{"lat":41.1887801843508,"lng":-8.45529050683198},{"lat":41.1888212308907,"lng":-8.45532611071751},{"lat":41.1888761043182,"lng":-8.45539466927455},{"lat":41.1889512593841,"lng":-8.45548384764451},{"lat":41.1889825129591,"lng":-8.45560975398448},{"lat":41.1890018335967,"lng":-8.45578268477175},{"lat":41.1889301947201,"lng":-8.45592906468135},{"lat":41.1888825674293,"lng":-8.45599164399393},{"lat":41.1888034535877,"lng":-8.45601222801554},{"lat":41.1887481522031,"lng":-8.45598573064349},{"lat":41.1886886840132,"lng":-8.45603264037448},{"lat":41.1886955466868,"lng":-8.45611980805276},{"lat":41.1887349256449,"lng":-8.45617232776675},{"lat":41.1888055864482,"lng":-8.45634038390658},{"lat":41.1888211691809,"lng":-8.45643474767509},{"lat":41.1888682952951,"lng":-8.45655012260793},{"lat":41.1889392488323,"lng":-8.45664487471171},{"lat":41.189025846892,"lng":-8.45678142548539},{"lat":41.1890094050165,"lng":-8.45699089445274},{"lat":41.1889932903785,"lng":-8.45711656571339},{"lat":41.1889611231332,"lng":-8.45728399356023},{"lat":41.1889133161139,"lng":-8.45740950708289},{"lat":41.1888255656644,"lng":-8.45768143544646},{"lat":41.189336686945,"lng":-8.45781878231549},{"lat":41.1897190807296,"lng":-8.45750971669013},{"lat":41.1900114605635,"lng":-8.45715213598946},{"lat":41.1903384276637,"lng":-8.45662174056076},{"lat":41.1906606751615,"lng":-8.45648569047158},{"lat":41.1907080959956,"lng":-8.456496417015},{"lat":41.1908425258855,"lng":-8.45649708209158},{"lat":41.1909373095512,"lng":-8.45653950835598},{"lat":41.1910796640216,"lng":-8.45654021371889},{"lat":41.191142875442,"lng":-8.45657187828243},{"lat":41.1912059686771,"lng":-8.45664561320579},{"lat":41.1913403095748,"lng":-8.45667763053134},{"lat":41.1914194233389,"lng":-8.45668851337736},{"lat":41.1915378307847,"lng":-8.45675203972799},{"lat":41.191672024255,"lng":-8.45683649727765},{"lat":41.19192468492,"lng":-8.45699486158022},{"lat":41.19204314822,"lng":-8.45706887790161},{"lat":41.1923341530974,"lng":-8.45731654041544},{"lat":41.1926238714911,"lng":-8.45756311070822},{"lat":41.1926089259573,"lng":-8.45587501021695},{"lat":41.1926362511436,"lng":-8.45313372948505},{"lat":41.1926367115629,"lng":-8.45296903794257},{"lat":41.1926393846345,"lng":-8.45201121299363},{"lat":41.1926742294001,"lng":-8.45133252424703},{"lat":41.1927499551242,"lng":-8.45128850686462},{"lat":41.1937180627608,"lng":-8.45072576142772},{"lat":41.1937714207457,"lng":-8.45069474475897},{"lat":41.1941293148818,"lng":-8.45048670167009},{"lat":41.1943273691588,"lng":-8.45020920286753},{"lat":41.1948023890191,"lng":-8.44954132944726},{"lat":41.1952905490665,"lng":-8.44893930908102},{"lat":41.1953821242057,"lng":-8.44880612694044},{"lat":41.1963902088825,"lng":-8.44750828422759},{"lat":41.196857696587,"lng":-8.44692225723196},{"lat":41.1971151983923,"lng":-8.44659945639452},{"lat":41.1987368602049,"lng":-8.44656289738399},{"lat":41.1991334840397,"lng":-8.44653463885896},{"lat":41.1996555450579,"lng":-8.44656790161913},{"lat":41.1999334125935,"lng":-8.44679968472362},{"lat":41.2002337825347,"lng":-8.44726154095329},{"lat":41.2007882548191,"lng":-8.44824667546025},{"lat":41.2013370051744,"lng":-8.44924598979097},{"lat":41.2018419966663,"lng":-8.45019145546318},{"lat":41.2018796670744,"lng":-8.45026198470934},{"lat":41.2025262017415,"lng":-8.45141643315363},{"lat":41.202825613149,"lng":-8.45157703268635},{"lat":41.2030944496959,"lng":-8.45147622407982},{"lat":41.2032420932173,"lng":-8.45139929986107},{"lat":41.2035873547821,"lng":-8.4514408358203},{"lat":41.2038690964732,"lng":-8.45167664175382}]},{"zone_id":"str6","points":[{"lat":41.2984980778133,"lng":-8.5069663481637},{"lat":41.2987261390776,"lng":-8.50642304137655},{"lat":41.2993026319463,"lng":-8.50004647353598},{"lat":41.2995314434518,"lng":-8.49077456180571},{"lat":41.3014820084035,"lng":-8.48347993733135},{"lat":41.3017747864843,"lng":-8.48120688593323},{"lat":41.3014986079898,"lng":-8.47798305468974},{"lat":41.3002212044713,"lng":-8.47494357230869},{"lat":41.2988007810245,"lng":-8.47190346846928},{"lat":41.2985248934989,"lng":-8.46849028193559},{"lat":41.2985838855655,"lng":-8.46457337581717},{"lat":41.3001028358778,"lng":-8.4557532712525},{"lat":41.3004326459981,"lng":-8.4528169070461},{"lat":41.300619833205,"lng":-8.44973766181591},{"lat":41.3004824805157,"lng":-8.44769935779103},{"lat":41.3000239928127,"lng":-8.44546998251331},{"lat":41.2992081134699,"lng":-8.44323894538573},{"lat":41.2979967604838,"lng":-8.44017705177028},{"lat":41.2922220470603,"lng":-8.42942699332266},{"lat":41.2920919614212,"lng":-8.42918372496377},{"lat":41.2918842731167,"lng":-8.429163276038},{"lat":41.2900579806293,"lng":-8.42898974631447},{"lat":41.2891947575772,"lng":-8.4288863710124},{"lat":41.2888360549488,"lng":-8.42895015984451},{"lat":41.2884530894144,"lng":-8.42902804555292},{"lat":41.287884182623,"lng":-8.42911745841129},{"lat":41.2871000408209,"lng":-8.42924069638548},{"lat":41.286889893955,"lng":-8.42927233209333},{"lat":41.2865170382897,"lng":-8.42944420001779},{"lat":41.2845986663552,"lng":-8.43040414791441},{"lat":41.283550240613,"lng":-8.43095217663858},{"lat":41.2811670103803,"lng":-8.43217441057891},{"lat":41.2807632794309,"lng":-8.43237854378486},{"lat":41.2798231897424,"lng":-8.43288389315301},{"lat":41.278368516316,"lng":-8.43360489634985},{"lat":41.2782090950026,"lng":-8.43368391082007},{"lat":41.2779682621037,"lng":-8.43381003544607},{"lat":41.277722657154,"lng":-8.43407952381762},{"lat":41.2769533267601,"lng":-8.43492364882349},{"lat":41.2767478523719,"lng":-8.43514909500779},{"lat":41.2761434385947,"lng":-8.43581224586489},{"lat":41.2752712647352,"lng":-8.43677900306048},{"lat":41.2741008829313,"lng":-8.43807625288981},{"lat":41.2707949421153,"lng":-8.44210308017748},{"lat":41.2703673525031,"lng":-8.4425260035189},{"lat":41.2697668982845,"lng":-8.44311989308275},{"lat":41.2694077361834,"lng":-8.44348116681371},{"lat":41.2693306260712,"lng":-8.44355872961561},{"lat":41.2691110489299,"lng":-8.44377959453812},{"lat":41.2687318085032,"lng":-8.44417471207262},{"lat":41.2685807228347,"lng":-8.4443382058004},{"lat":41.2668386515314,"lng":-8.44590883086154},{"lat":41.265689273754,"lng":-8.44692947212502},{"lat":41.2655158911683,"lng":-8.44708186262386},{"lat":41.2641375473605,"lng":-8.44835684780462},{"lat":41.2637036178137,"lng":-8.44876411638577},{"lat":41.2628740991168,"lng":-8.44954265003576},{"lat":41.2661871197115,"lng":-8.46270757214721},{"lat":41.266672417694,"lng":-8.46449403567753},{"lat":41.2685465318063,"lng":-8.46919268103078},{"lat":41.2690906809479,"lng":-8.47049009434989},{"lat":41.2720271440986,"lng":-8.487668343119},{"lat":41.2741310671343,"lng":-8.51542561083302},{"lat":41.2789326581237,"lng":-8.51546767542149},{"lat":41.2852361300558,"lng":-8.51526595941858},{"lat":41.2864825634647,"lng":-8.51438636767019},{"lat":41.2874889898763,"lng":-8.51353373725208},{"lat":41.2894349675841,"lng":-8.51246243189715},{"lat":41.2923060071606,"lng":-8.50999700515161},{"lat":41.2984980778133,"lng":-8.5069663481637}]},{"zone_id":"trf3","points":[{"lat":41.2890689519601,"lng":-8.56929957209123},{"lat":41.2914374129397,"lng":-8.5702264312707},{"lat":41.2934328044021,"lng":-8.57175594638693},{"lat":41.2959944994314,"lng":-8.57471087444866},{"lat":41.2970449660833,"lng":-8.57565339955001},{"lat":41.3024244706131,"lng":-8.57580448416316},{"lat":41.3064846051867,"lng":-8.57870652746103},{"lat":41.3093107232325,"lng":-8.56896942150508},{"lat":41.3093638297132,"lng":-8.56543419795695},{"lat":41.3094636553007,"lng":-8.56190527710775},{"lat":41.3096249063807,"lng":-8.55923107395548},{"lat":41.3096787666598,"lng":-8.55793185629059},{"lat":41.3097686528984,"lng":-8.55667750212345},{"lat":41.3099920474731,"lng":-8.55401660430127},{"lat":41.3102423274777,"lng":-8.55070060074066},{"lat":41.3114584196734,"lng":-8.54375702889872},{"lat":41.3135861678325,"lng":-8.53651589767424},{"lat":41.3148246900517,"lng":-8.53313340524043},{"lat":41.3162600597135,"lng":-8.52995254662554},{"lat":41.3176488503582,"lng":-8.52718155289406},{"lat":41.31846002815,"lng":-8.5257142012269},{"lat":41.3191808638061,"lng":-8.52425558936462},{"lat":41.3198024957622,"lng":-8.52291148891012},{"lat":41.3201725938285,"lng":-8.52232226779216},{"lat":41.3206048592385,"lng":-8.52163887060228},{"lat":41.321565081364,"lng":-8.52020948202781},{"lat":41.3224416788664,"lng":-8.51878553698343},{"lat":41.3232890372931,"lng":-8.51750660498422},{"lat":41.3241562547735,"lng":-8.51593680874795},{"lat":41.3258707254747,"lng":-8.51297224871337},{"lat":41.3205189550679,"lng":-8.51292305792012},{"lat":41.3177838688323,"lng":-8.51199715462107},{"lat":41.3164369270609,"lng":-8.51077409149424},{"lat":41.3156518715792,"lng":-8.50976748243221},{"lat":41.3134401489741,"lng":-8.50731343066383},{"lat":41.310100307068,"lng":-8.5071607195532},{"lat":41.3054134870104,"lng":-8.51001329774041},{"lat":41.2984980778133,"lng":-8.5069663481637},{"lat":41.2923060071606,"lng":-8.50999700515161},{"lat":41.2894349675841,"lng":-8.51246243189715},{"lat":41.2874889898763,"lng":-8.51353373725208},{"lat":41.2864825634647,"lng":-8.51438636767019},{"lat":41.2852361300558,"lng":-8.51526595941858},{"lat":41.2789326581237,"lng":-8.51546767542149},{"lat":41.2741310671343,"lng":-8.51542561083302},{"lat":41.2736264581339,"lng":-8.51542119035921},{"lat":41.2696051005244,"lng":-8.5185370835038},{"lat":41.2683052005105,"lng":-8.52103147914557},{"lat":41.2670793447741,"lng":-8.52424494377153},{"lat":41.2654934588819,"lng":-8.52821392157359},{"lat":41.2644073998571,"lng":-8.53228046387806},{"lat":41.2637496871855,"lng":-8.5364442201266},{"lat":41.2632629892282,"lng":-8.54047683794398},{"lat":41.2630562731391,"lng":-8.54502214426784},{"lat":41.2637092930651,"lng":-8.54894139148496},{"lat":41.2651240655649,"lng":-8.55299187900883},{"lat":41.2681869057266,"lng":-8.55848686005698},{"lat":41.2695365638248,"lng":-8.56090856942134},{"lat":41.2716229909445,"lng":-8.56136881384518},{"lat":41.2740490911437,"lng":-8.56242672666349},{"lat":41.2790444561016,"lng":-8.56641877771437},{"lat":41.2818701605222,"lng":-8.5676045694375},{"lat":41.2850108922809,"lng":-8.56866762095591},{"lat":41.2890689519601,"lng":-8.56929957209123}]},{"zone_id":"str4","points":[{"lat":41.2995462438514,"lng":-8.42588389176486},{"lat":41.299343448364,"lng":-8.42628452732865},{"lat":41.2985685733925,"lng":-8.42781529455566},{"lat":41.2980408025604,"lng":-8.42966684061887},{"lat":41.2979748584546,"lng":-8.42989817884494},{"lat":41.2962886922948,"lng":-8.4296478800196},{"lat":41.2935916355541,"lng":-8.42933138640998},{"lat":41.2929215057952,"lng":-8.42926540298798},{"lat":41.2920919614212,"lng":-8.42918372496377},{"lat":41.2922220470603,"lng":-8.42942699332266},{"lat":41.2979967604838,"lng":-8.44017705177028},{"lat":41.2992081134699,"lng":-8.44323894538573},{"lat":41.3000239928127,"lng":-8.44546998251331},{"lat":41.3004824805157,"lng":-8.44769935779103},{"lat":41.300619833205,"lng":-8.44973766181591},{"lat":41.3004326459981,"lng":-8.4528169070461},{"lat":41.3001028358778,"lng":-8.4557532712525},{"lat":41.2985838855655,"lng":-8.46457337581717},{"lat":41.2985248934989,"lng":-8.46849028193559},{"lat":41.2988007810245,"lng":-8.47190346846928},{"lat":41.3002212044713,"lng":-8.47494357230869},{"lat":41.3014986079898,"lng":-8.47798305468974},{"lat":41.3017747864843,"lng":-8.48120688593323},{"lat":41.3014820084035,"lng":-8.48347993733135},{"lat":41.2995314434518,"lng":-8.49077456180571},{"lat":41.2993026319463,"lng":-8.50004647353598},{"lat":41.2987261390776,"lng":-8.50642304137655},{"lat":41.2984980778133,"lng":-8.5069663481637},{"lat":41.3054134870104,"lng":-8.51001329774041},{"lat":41.310100307068,"lng":-8.5071607195532},{"lat":41.3134401489741,"lng":-8.50731343066383},{"lat":41.3156518715792,"lng":-8.50976748243221},{"lat":41.3164801116427,"lng":-8.50889619537125},{"lat":41.318774202614,"lng":-8.50550923893158},{"lat":41.3216113566752,"lng":-8.5006872404336},{"lat":41.3217168570356,"lng":-8.50057625310322},{"lat":41.3227931210822,"lng":-8.49925501243983},{"lat":41.3243202250854,"lng":-8.49696264664516},{"lat":41.3267183973163,"lng":-8.49191935371303},{"lat":41.3281603164877,"lng":-8.48788186774985},{"lat":41.3297000795821,"lng":-8.48295977704236},{"lat":41.3303560677953,"lng":-8.47972765011159},{"lat":41.3310812695274,"lng":-8.47622321924446},{"lat":41.3312309687968,"lng":-8.47394836172209},{"lat":41.3312828062952,"lng":-8.47151890692807},{"lat":41.3310302259746,"lng":-8.4693012250542},{"lat":41.3305373914098,"lng":-8.46673861996373},{"lat":41.3303023812913,"lng":-8.46459218359731},{"lat":41.3303906330692,"lng":-8.46274876115689},{"lat":41.3309583515926,"lng":-8.46013905901476},{"lat":41.3329255766662,"lng":-8.45451883706366},{"lat":41.3318596993319,"lng":-8.4541343877302},{"lat":41.3294308036854,"lng":-8.45374320093269},{"lat":41.3267139157658,"lng":-8.45410912912464},{"lat":41.3246402489888,"lng":-8.45447818789195},{"lat":41.3221388239363,"lng":-8.45446589855199},{"lat":41.3195677791567,"lng":-8.45378965775681},{"lat":41.3170700413263,"lng":-8.45245024359157},{"lat":41.3145730717476,"lng":-8.45082655298277},{"lat":41.3121483277425,"lng":-8.44891896325251},{"lat":41.3099379591231,"lng":-8.44701254368226},{"lat":41.3077736045023,"lng":-8.44351132304613},{"lat":41.2995462438514,"lng":-8.42588389176486}]},{"zone_id":"str5","points":[{"lat":41.3348360011718,"lng":-8.37455982788335},{"lat":41.3351168083262,"lng":-8.37482799318894},{"lat":41.3357871626228,"lng":-8.37570175431235},{"lat":41.3358992682274,"lng":-8.37685425919846},{"lat":41.3358301784077,"lng":-8.37745621205149},{"lat":41.3358750108132,"lng":-8.37804880148542},{"lat":41.3365577991878,"lng":-8.38033745506853},{"lat":41.3366370902678,"lng":-8.38060344629344},{"lat":41.3368574153232,"lng":-8.38230009886125},{"lat":41.3370865747434,"lng":-8.38317772459242},{"lat":41.3371958319895,"lng":-8.38359616490877},{"lat":41.3372163745722,"lng":-8.3840842221506},{"lat":41.3372231300292,"lng":-8.38424473127148},{"lat":41.3363560121397,"lng":-8.38674498329843},{"lat":41.3358388207224,"lng":-8.38780356060406},{"lat":41.3357370495027,"lng":-8.3880118596303},{"lat":41.3354864539559,"lng":-8.38830630320413},{"lat":41.3346390959825,"lng":-8.38930190457425},{"lat":41.3345859886684,"lng":-8.3902557095617},{"lat":41.334601678844,"lng":-8.39034196121798},{"lat":41.334667190725,"lng":-8.39070209584833},{"lat":41.3347348311666,"lng":-8.39113027857395},{"lat":41.3347628433597,"lng":-8.39156510655439},{"lat":41.3339428327244,"lng":-8.39308041131466},{"lat":41.3336838365428,"lng":-8.39337503602771},{"lat":41.3335974146531,"lng":-8.39348674123478},{"lat":41.3335369706647,"lng":-8.39361585954126},{"lat":41.3335967755571,"lng":-8.39376710776016},{"lat":41.3336239120746,"lng":-8.39382993751412},{"lat":41.3336924264219,"lng":-8.39398992606973},{"lat":41.3337523813583,"lng":-8.39411452994786},{"lat":41.3335086493349,"lng":-8.39462387380677},{"lat":41.3334221807951,"lng":-8.39479279711978},{"lat":41.3325819870745,"lng":-8.39386218342901},{"lat":41.3324618478555,"lng":-8.39371715605212},{"lat":41.3316096821266,"lng":-8.39467632226069},{"lat":41.331240510453,"lng":-8.39511011399258},{"lat":41.3310420417339,"lng":-8.39531464910676},{"lat":41.3309153003786,"lng":-8.39549463717506},{"lat":41.3307549173403,"lng":-8.39585596944502},{"lat":41.3306845942972,"lng":-8.39601440119715},{"lat":41.3305619619296,"lng":-8.39628864626718},{"lat":41.3304970848666,"lng":-8.39642516098527},{"lat":41.33040349793,"lng":-8.396559761979},{"lat":41.3303724208687,"lng":-8.39661366313875},{"lat":41.3303127317676,"lng":-8.39671718977086},{"lat":41.3301457131441,"lng":-8.39702744987681},{"lat":41.3300057622667,"lng":-8.39728382175228},{"lat":41.3299185320764,"lng":-8.39744361507418},{"lat":41.3298905372913,"lng":-8.39749489736864},{"lat":41.3297887764739,"lng":-8.3976832314769},{"lat":41.3297594155713,"lng":-8.39773757103178},{"lat":41.3296009286459,"lng":-8.39801715741372},{"lat":41.3295125741746,"lng":-8.39837873510875},{"lat":41.3293980932533,"lng":-8.398860019301},{"lat":41.3292489422178,"lng":-8.3994590496066},{"lat":41.3291635068547,"lng":-8.39980080620811},{"lat":41.3328678736236,"lng":-8.40554545914764},{"lat":41.3345720491389,"lng":-8.41019890525794},{"lat":41.3361333285436,"lng":-8.41475715255439},{"lat":41.3465268963547,"lng":-8.41324080038705},{"lat":41.3507554932369,"lng":-8.4132589264777},{"lat":41.3542090675619,"lng":-8.41358990616346},{"lat":41.3571256552359,"lng":-8.41431384757669},{"lat":41.3602189701495,"lng":-8.4158291219728},{"lat":41.3627754637855,"lng":-8.41765841909451},{"lat":41.36712859177,"lng":-8.42174760348679},{"lat":41.3671688454414,"lng":-8.42162921155713},{"lat":41.3672222833798,"lng":-8.42151665519683},{"lat":41.3672610872023,"lng":-8.42143492242218},{"lat":41.3674412121,"lng":-8.42115558847863},{"lat":41.3674748897856,"lng":-8.42109886297531},{"lat":41.3675899174038,"lng":-8.42089507039866},{"lat":41.3676340286063,"lng":-8.42084273256619},{"lat":41.3677504900976,"lng":-8.42070455081933},{"lat":41.3679057491455,"lng":-8.42061966425677},{"lat":41.3680932866339,"lng":-8.42055392001292},{"lat":41.3681886198042,"lng":-8.42050308055143},{"lat":41.3682998238392,"lng":-8.42044020899963},{"lat":41.368428886759,"lng":-8.42036548155508},{"lat":41.3685745957953,"lng":-8.42017704195161},{"lat":41.3687263545846,"lng":-8.41990902939644},{"lat":41.368857454115,"lng":-8.4195848605734},{"lat":41.3689674038363,"lng":-8.41925570202676},{"lat":41.3690397561908,"lng":-8.41901034965363},{"lat":41.3690897270675,"lng":-8.41884089323187},{"lat":41.3691586294526,"lng":-8.41850749020261},{"lat":41.3692517063593,"lng":-8.41808334768602},{"lat":41.3693857858889,"lng":-8.41784480930036},{"lat":41.3694230025749,"lng":-8.41777859745977},{"lat":41.3695402275475,"lng":-8.41767243331872},{"lat":41.3696130524314,"lng":-8.41760647963291},{"lat":41.3697290736547,"lng":-8.41754580702851},{"lat":41.3698142302045,"lng":-8.41750127477267},{"lat":41.3700093089444,"lng":-8.41731991975042},{"lat":41.3701906279611,"lng":-8.41703863135552},{"lat":41.370229144423,"lng":-8.41693684426715},{"lat":41.370648201862,"lng":-8.41624219842203},{"lat":41.3708762254945,"lng":-8.41597055293773},{"lat":41.371063459076,"lng":-8.41580713707467},{"lat":41.3711990772044,"lng":-8.41572312692189},{"lat":41.3714630395407,"lng":-8.41555811025317},{"lat":41.3715816976062,"lng":-8.41552839375059},{"lat":41.3718144084974,"lng":-8.41547011375576},{"lat":41.3721846950446,"lng":-8.41545140777943},{"lat":41.3723411914628,"lng":-8.41544886427139},{"lat":41.3724472733997,"lng":-8.41544227178542},{"lat":41.3725703229782,"lng":-8.41542045376993},{"lat":41.3726920999321,"lng":-8.41540281498688},{"lat":41.3728359810101,"lng":-8.41536567198954},{"lat":41.3729212730855,"lng":-8.41521053498372},{"lat":41.3730058538692,"lng":-8.41494171902994},{"lat":41.3730565963544,"lng":-8.41478613767903},{"lat":41.37314502636,"lng":-8.41451500063878},{"lat":41.3731711850256,"lng":-8.41444344552789},{"lat":41.3732022252324,"lng":-8.41435853708667},{"lat":41.3732506706094,"lng":-8.41427297705691},{"lat":41.3732903959599,"lng":-8.41420281740413},{"lat":41.3733542968565,"lng":-8.41413146787857},{"lat":41.3734135794244,"lng":-8.41407117923503},{"lat":41.3735014247023,"lng":-8.4139921563908},{"lat":41.3735547308497,"lng":-8.41394670182319},{"lat":41.3736082575044,"lng":-8.41390105913708},{"lat":41.3737649816426,"lng":-8.41377455770847},{"lat":41.3738537824709,"lng":-8.41372430495229},{"lat":41.374010461193,"lng":-8.41364532956758},{"lat":41.374303441654,"lng":-8.41337616727466},{"lat":41.3743474252161,"lng":-8.41333575914572},{"lat":41.3745101858859,"lng":-8.41316445232395},{"lat":41.3745846940163,"lng":-8.41309648748314},{"lat":41.3747567985438,"lng":-8.41298299450199},{"lat":41.3748498719441,"lng":-8.41290752455594},{"lat":41.3749211761312,"lng":-8.41284970632614},{"lat":41.3750538653741,"lng":-8.41272741408676},{"lat":41.375253031924,"lng":-8.41256640251725},{"lat":41.3754380579189,"lng":-8.4124208918945},{"lat":41.3755684866732,"lng":-8.41229067967998},{"lat":41.3756498155865,"lng":-8.41219286959978},{"lat":41.3758024774855,"lng":-8.41200276957013},{"lat":41.3759080298233,"lng":-8.41188093916563},{"lat":41.3760286704628,"lng":-8.41155547863512},{"lat":41.3760889593378,"lng":-8.41142161745951},{"lat":41.3761392944464,"lng":-8.41130958794229},{"lat":41.376167179346,"lng":-8.41118666195691},{"lat":41.3761853472687,"lng":-8.4111065712956},{"lat":41.376201609056,"lng":-8.41095585488983},{"lat":41.3762056978942,"lng":-8.41073633266908},{"lat":41.3762020669793,"lng":-8.41048935562586},{"lat":41.3761947450112,"lng":-8.41024116158115},{"lat":41.3761923320067,"lng":-8.41008480227424},{"lat":41.3761998225995,"lng":-8.40989201857755},{"lat":41.3762216788699,"lng":-8.40971531504341},{"lat":41.3763071806347,"lng":-8.40950792991973},{"lat":41.3763642521103,"lng":-8.40933256937115},{"lat":41.3763933582441,"lng":-8.40927232707681},{"lat":41.3764695335756,"lng":-8.40913040566441},{"lat":41.3766159315016,"lng":-8.40888769936676},{"lat":41.3768002994314,"lng":-8.40862389615428},{"lat":41.3769823142492,"lng":-8.40838116634777},{"lat":41.3770693808578,"lng":-8.408265665472},{"lat":41.3772067790307,"lng":-8.40808339477814},{"lat":41.3772670792345,"lng":-8.4080188638395},{"lat":41.377384355649,"lng":-8.40792432169719},{"lat":41.3774270692086,"lng":-8.40788817785379},{"lat":41.3774670022613,"lng":-8.4078543867996},{"lat":41.3775616496445,"lng":-8.40780924598984},{"lat":41.3777231192382,"lng":-8.40772158921608},{"lat":41.3777917784103,"lng":-8.40758918712938},{"lat":41.3778261344403,"lng":-8.40745617081685},{"lat":41.3778923260794,"lng":-8.40730152554562},{"lat":41.3781189356878,"lng":-8.40678152187863},{"lat":41.3782680995809,"lng":-8.40656961306636},{"lat":41.378304995832,"lng":-8.4065150978381},{"lat":41.3783614442688,"lng":-8.40643169352311},{"lat":41.3784616229142,"lng":-8.40629488404299},{"lat":41.3786983717285,"lng":-8.40612888511335},{"lat":41.3787856867031,"lng":-8.40617867020326},{"lat":41.3789264534779,"lng":-8.40631107276192},{"lat":41.3790342999107,"lng":-8.40639496672347},{"lat":41.3791316954604,"lng":-8.40648144835945},{"lat":41.3792178119184,"lng":-8.40657840407128},{"lat":41.3793239843024,"lng":-8.40668464420096},{"lat":41.3794151287754,"lng":-8.40678675716934},{"lat":41.379525493148,"lng":-8.40687221957727},{"lat":41.3796295032089,"lng":-8.406940320442},{"lat":41.3797250175708,"lng":-8.40698435974963},{"lat":41.379815675499,"lng":-8.40698928518842},{"lat":41.3798972441393,"lng":-8.40699333688701},{"lat":41.3800148101745,"lng":-8.40700411462164},{"lat":41.3801349132438,"lng":-8.40700593831432},{"lat":41.3802540521511,"lng":-8.40696304782205},{"lat":41.3803670357793,"lng":-8.40689371287387},{"lat":41.3804594518205,"lng":-8.40684126342569},{"lat":41.3805663069712,"lng":-8.40677393106171},{"lat":41.3806835123317,"lng":-8.40671071242056},{"lat":41.3807962654443,"lng":-8.40665308872226},{"lat":41.3809823640762,"lng":-8.40655500431696},{"lat":41.3810700502935,"lng":-8.40648362690785},{"lat":41.3811278813127,"lng":-8.40643655178702},{"lat":41.3812584651914,"lng":-8.40631205525951},{"lat":41.3813518000072,"lng":-8.40611391123402},{"lat":41.3814110452222,"lng":-8.40598813664686},{"lat":41.3814868639914,"lng":-8.40583567230366},{"lat":41.3815436647328,"lng":-8.40561343517354},{"lat":41.3815692702003,"lng":-8.40548610481545},{"lat":41.3815890075113,"lng":-8.40536604472068},{"lat":41.3816173896302,"lng":-8.40524362843034},{"lat":41.3816715813891,"lng":-8.40505999003707},{"lat":41.3817011114922,"lng":-8.40494773985848},{"lat":41.381773615757,"lng":-8.40486185559673},{"lat":41.381812029384,"lng":-8.40480056509009},{"lat":41.3818924193036,"lng":-8.4047314477787},{"lat":41.3820530046452,"lng":-8.40463552386702},{"lat":41.3824212882105,"lng":-8.40455075279271},{"lat":41.3832649082087,"lng":-8.40425704321405},{"lat":41.3835653051515,"lng":-8.40396546458152},{"lat":41.3836377847669,"lng":-8.40380710391601},{"lat":41.3837179142874,"lng":-8.40363202801338},{"lat":41.3839397880275,"lng":-8.40314724826784},{"lat":41.3841160786645,"lng":-8.4025923756492},{"lat":41.3841937846926,"lng":-8.40234779325246},{"lat":41.3842272376569,"lng":-8.40175435831151},{"lat":41.3841060768883,"lng":-8.40074388501374},{"lat":41.3840958374132,"lng":-8.40065835792808},{"lat":41.3842121063634,"lng":-8.39991415375866},{"lat":41.3842769169527,"lng":-8.39950219394472},{"lat":41.3842886054182,"lng":-8.39942789669722},{"lat":41.383904091862,"lng":-8.39771979888922},{"lat":41.3818140458003,"lng":-8.38843765821005},{"lat":41.3809157561596,"lng":-8.38444882019582},{"lat":41.3808611667593,"lng":-8.38433862798033},{"lat":41.3801299435744,"lng":-8.3849987781982},{"lat":41.3800198667765,"lng":-8.38514263883966},{"lat":41.3790037132928,"lng":-8.38647325499621},{"lat":41.3782918557606,"lng":-8.38773043210985},{"lat":41.3770520529573,"lng":-8.39016521954198},{"lat":41.376555936945,"lng":-8.39056561331238},{"lat":41.3763939447273,"lng":-8.39064960082512},{"lat":41.3762996593023,"lng":-8.39069572345784},{"lat":41.3761438903849,"lng":-8.39077818566488},{"lat":41.3759744863083,"lng":-8.39087564946707},{"lat":41.3759141874022,"lng":-8.39094187210153},{"lat":41.3758084093211,"lng":-8.39105298069458},{"lat":41.3757424554812,"lng":-8.39111451629347},{"lat":41.375670168038,"lng":-8.39118654961836},{"lat":41.375588132631,"lng":-8.39127109207549},{"lat":41.3755182139582,"lng":-8.39133273349475},{"lat":41.3754280344618,"lng":-8.39139393363139},{"lat":41.3753242990074,"lng":-8.39147814937665},{"lat":41.3752765082432,"lng":-8.39170866380666},{"lat":41.3752411882714,"lng":-8.39179913153137},{"lat":41.3751831076755,"lng":-8.39184121377828},{"lat":41.3750716576225,"lng":-8.39183275961575},{"lat":41.3749580557323,"lng":-8.39181904095122},{"lat":41.3747605981352,"lng":-8.39182063733072},{"lat":41.3743497139583,"lng":-8.39211376721115},{"lat":41.374199943493,"lng":-8.39225015758373},{"lat":41.3741243588954,"lng":-8.39234739428732},{"lat":41.3740485290008,"lng":-8.39251228034481},{"lat":41.3740034198732,"lng":-8.39267072211831},{"lat":41.3739593992172,"lng":-8.3928662188667},{"lat":41.3738631501649,"lng":-8.39349588859674},{"lat":41.3737754999165,"lng":-8.39379078685613},{"lat":41.3736902046506,"lng":-8.39396245075394},{"lat":41.3734615453942,"lng":-8.39414070473636},{"lat":41.3732867139984,"lng":-8.39420777512014},{"lat":41.3731531417144,"lng":-8.39426772583688},{"lat":41.3730171535656,"lng":-8.39431857379921},{"lat":41.37283273148,"lng":-8.39448493381219},{"lat":41.3723789794904,"lng":-8.39470315700116},{"lat":41.3722733647209,"lng":-8.3947412169844},{"lat":41.3720771583334,"lng":-8.39478488917254},{"lat":41.3718419886996,"lng":-8.39481788864816},{"lat":41.3716902045743,"lng":-8.39484811433761},{"lat":41.3715325247247,"lng":-8.3948976765116},{"lat":41.3713275746137,"lng":-8.39494562053073},{"lat":41.3710480143388,"lng":-8.3950172796108},{"lat":41.3707900237369,"lng":-8.39510886874425},{"lat":41.3706979374718,"lng":-8.39513897901966},{"lat":41.3705927109749,"lng":-8.39516580443728},{"lat":41.3705039672062,"lng":-8.39518791360435},{"lat":41.3703612649899,"lng":-8.39522224061686},{"lat":41.3702404701376,"lng":-8.39524565769525},{"lat":41.370098974267,"lng":-8.39526504120851},{"lat":41.3700843001303,"lng":-8.39538211900043},{"lat":41.3698540427967,"lng":-8.39543449843197},{"lat":41.369622074021,"lng":-8.39548399959815},{"lat":41.3693973521484,"lng":-8.39555540159832},{"lat":41.3691696508415,"lng":-8.39559164703097},{"lat":41.3689296193644,"lng":-8.39558565984121},{"lat":41.3688225989575,"lng":-8.39556992454477},{"lat":41.3687548952107,"lng":-8.39552948972491},{"lat":41.3687104530827,"lng":-8.39551209838909},{"lat":41.3686614593487,"lng":-8.39551763632574},{"lat":41.3686121505181,"lng":-8.39554289692939},{"lat":41.3683536739966,"lng":-8.39549080953128},{"lat":41.3683033987427,"lng":-8.39546801870763},{"lat":41.3682533383375,"lng":-8.39546710165475},{"lat":41.3680225813671,"lng":-8.395460551078},{"lat":41.3677762257533,"lng":-8.39542596592033},{"lat":41.3675514663312,"lng":-8.39539493391366},{"lat":41.3673262402453,"lng":-8.39533318543664},{"lat":41.3672430585488,"lng":-8.39528875021522},{"lat":41.367161817288,"lng":-8.39526009022426},{"lat":41.367084452435,"lng":-8.39522858690255},{"lat":41.3670093308765,"lng":-8.39524118657297},{"lat":41.3668387971906,"lng":-8.39536001864741},{"lat":41.366788181659,"lng":-8.39536483649291},{"lat":41.3666813542464,"lng":-8.39530428129114},{"lat":41.3666251124031,"lng":-8.39528959467268},{"lat":41.3665751367415,"lng":-8.39525186571382},{"lat":41.366347001158,"lng":-8.39524316807953},{"lat":41.3661157115581,"lng":-8.3952336307737},{"lat":41.3658824009926,"lng":-8.39524128787871},{"lat":41.3656477579496,"lng":-8.39528169380171},{"lat":41.3654060337639,"lng":-8.39530485793014},{"lat":41.365158685281,"lng":-8.39531078868703},{"lat":41.3649318881765,"lng":-8.39530581104814},{"lat":41.36470352402,"lng":-8.39527608227559},{"lat":41.3645350253014,"lng":-8.39525376978007},{"lat":41.3644477995258,"lng":-8.39524385357107},{"lat":41.3643549973802,"lng":-8.39523044902459},{"lat":41.3642992881012,"lng":-8.39522030557821},{"lat":41.3642227807883,"lng":-8.39520720671074},{"lat":41.3639926611417,"lng":-8.39519946373225},{"lat":41.3638727300627,"lng":-8.39519897892189},{"lat":41.3636178019611,"lng":-8.39509337167906},{"lat":41.3634494473512,"lng":-8.39500652604824},{"lat":41.3632230166639,"lng":-8.39495840270822},{"lat":41.3631739192421,"lng":-8.39493035540345},{"lat":41.3631258241044,"lng":-8.39489789251143},{"lat":41.3629122866788,"lng":-8.39476485169678},{"lat":41.3628440370988,"lng":-8.39472513510478},{"lat":41.3626296056486,"lng":-8.39462842279488},{"lat":41.3625613376324,"lng":-8.39459719429223},{"lat":41.3625143676904,"lng":-8.39458265868426},{"lat":41.3621066713217,"lng":-8.39434593461246},{"lat":41.3618886088946,"lng":-8.39422232346543},{"lat":41.3618538013673,"lng":-8.39416541245759},{"lat":41.3618577765724,"lng":-8.39404197464016},{"lat":41.3618959739098,"lng":-8.39395405026806},{"lat":41.3621176430096,"lng":-8.3933662243978},{"lat":41.3621724212837,"lng":-8.39327250919474},{"lat":41.3622224090965,"lng":-8.3931860662418},{"lat":41.362378572308,"lng":-8.39293392845287},{"lat":41.362480568431,"lng":-8.39274323233576},{"lat":41.3626329119675,"lng":-8.39254665354611},{"lat":41.3627928296645,"lng":-8.39226679314801},{"lat":41.3629096625978,"lng":-8.39196477361522},{"lat":41.3630032445629,"lng":-8.3916747343029},{"lat":41.3630466610691,"lng":-8.39150722869787},{"lat":41.3630825757386,"lng":-8.39127324318648},{"lat":41.3631062007569,"lng":-8.3910583359864},{"lat":41.3631284547355,"lng":-8.3908933767049},{"lat":41.363150526641,"lng":-8.39072638604028},{"lat":41.363205776962,"lng":-8.39042268293764},{"lat":41.3632551792225,"lng":-8.39023548439016},{"lat":41.3633457478319,"lng":-8.39000064032717},{"lat":41.363465510807,"lng":-8.3897960293354},{"lat":41.363643366822,"lng":-8.38954407735306},{"lat":41.363785556892,"lng":-8.38925505973343},{"lat":41.3638933851442,"lng":-8.38907083406724},{"lat":41.3639796246734,"lng":-8.38891915473007},{"lat":41.3640109762474,"lng":-8.3888310769952},{"lat":41.3640693362247,"lng":-8.38866195573829},{"lat":41.3640929784121,"lng":-8.38855842761529},{"lat":41.364103621483,"lng":-8.38842927450051},{"lat":41.3640844359728,"lng":-8.38831302849418},{"lat":41.3640395395149,"lng":-8.38813573247461},{"lat":41.3640290875807,"lng":-8.38806015983593},{"lat":41.364014189029,"lng":-8.38791644208237},{"lat":41.3640322091363,"lng":-8.38779102517316},{"lat":41.3640580094326,"lng":-8.38768762352514},{"lat":41.3640929197738,"lng":-8.38753705671951},{"lat":41.3641261485842,"lng":-8.38741432454052},{"lat":41.3642158892004,"lng":-8.38718281428515},{"lat":41.3642609421289,"lng":-8.3870488954965},{"lat":41.364321394393,"lng":-8.38691014095645},{"lat":41.3643695593264,"lng":-8.38679069060095},{"lat":41.3644326015111,"lng":-8.38666389375037},{"lat":41.3644899286121,"lng":-8.38655511916004},{"lat":41.3645365203928,"lng":-8.38645479084561},{"lat":41.3645742256921,"lng":-8.38634354828542},{"lat":41.3646358169928,"lng":-8.38609710579207},{"lat":41.3646987515312,"lng":-8.38577381953828},{"lat":41.3647377962641,"lng":-8.38554343212965},{"lat":41.3647647736629,"lng":-8.38531370300331},{"lat":41.3647990212749,"lng":-8.38509512284473},{"lat":41.3648261576637,"lng":-8.38491619131468},{"lat":41.3648414534327,"lng":-8.38471749838653},{"lat":41.364823043711,"lng":-8.38453253369758},{"lat":41.3647804137715,"lng":-8.38434711978744},{"lat":41.3647398798038,"lng":-8.38423139166927},{"lat":41.364734184712,"lng":-8.38411914406723},{"lat":41.3647811993916,"lng":-8.38398989081998},{"lat":41.3648183848851,"lng":-8.38394915503818},{"lat":41.3649173963124,"lng":-8.38383958487994},{"lat":41.3650104325588,"lng":-8.38374695942955},{"lat":41.3650722365582,"lng":-8.3836881606335},{"lat":41.3651652175264,"lng":-8.38362087360229},{"lat":41.3654311100774,"lng":-8.38345278873902},{"lat":41.3655610054065,"lng":-8.38338564245626},{"lat":41.3658572385739,"lng":-8.38326021478299},{"lat":41.366232519137,"lng":-8.38330253759002},{"lat":41.3663838672516,"lng":-8.38330670895008},{"lat":41.3665451647022,"lng":-8.3832894059002},{"lat":41.3666983456574,"lng":-8.38323824482738},{"lat":41.3668400459248,"lng":-8.38316576595675},{"lat":41.3669727806069,"lng":-8.38307472780928},{"lat":41.3671002129524,"lng":-8.38294004116006},{"lat":41.3672105085451,"lng":-8.38281867373771},{"lat":41.3673186471113,"lng":-8.38265307684019},{"lat":41.3674180087266,"lng":-8.38250680731162},{"lat":41.3675341536883,"lng":-8.38226199764255},{"lat":41.3676242548074,"lng":-8.38210959367879},{"lat":41.3677357656848,"lng":-8.38192464588451},{"lat":41.3678073287423,"lng":-8.38176894447434},{"lat":41.3678434914295,"lng":-8.38170071969456},{"lat":41.367900813836,"lng":-8.38159193386354},{"lat":41.367956357248,"lng":-8.38143126793501},{"lat":41.3679984264464,"lng":-8.38121521596019},{"lat":41.368016513318,"lng":-8.38105333155559},{"lat":41.3680036969194,"lng":-8.38073678598798},{"lat":41.368016798321,"lng":-8.38054859335731},{"lat":41.3680444836874,"lng":-8.38036439774995},{"lat":41.3680999355479,"lng":-8.37995189856706},{"lat":41.3681139797533,"lng":-8.37983027656646},{"lat":41.3681038433724,"lng":-8.37960362504081},{"lat":41.3681029813915,"lng":-8.37945995318049},{"lat":41.3681085489058,"lng":-8.37934200673223},{"lat":41.3681384922166,"lng":-8.37919451263605},{"lat":41.3681627431289,"lng":-8.37909623836137},{"lat":41.3682037614154,"lng":-8.37898990071885},{"lat":41.3682386021035,"lng":-8.37890624564726},{"lat":41.3682701446385,"lng":-8.37876772017309},{"lat":41.3682556158904,"lng":-8.37869929306121},{"lat":41.3681436800134,"lng":-8.37841572045509},{"lat":41.3680661889145,"lng":-8.37827439038086},{"lat":41.3679822155801,"lng":-8.37817367667489},{"lat":41.3678681552254,"lng":-8.37808264649911},{"lat":41.3675893907985,"lng":-8.37795740602516},{"lat":41.3674460243063,"lng":-8.37788395866075},{"lat":41.3673307920072,"lng":-8.3777922118043},{"lat":41.3672168014911,"lng":-8.37770799622448},{"lat":41.367117311937,"lng":-8.37762335475128},{"lat":41.3669536136049,"lng":-8.37750095064209},{"lat":41.366846841471,"lng":-8.37740780135129},{"lat":41.366744445011,"lng":-8.37729184018651},{"lat":41.3666731575912,"lng":-8.37719702775298},{"lat":41.3665869894728,"lng":-8.37707061001254},{"lat":41.3664852500953,"lng":-8.37690146319987},{"lat":41.3664037165224,"lng":-8.37675355100407},{"lat":41.3663456679167,"lng":-8.37665760098827},{"lat":41.3662327810733,"lng":-8.37656287380621},{"lat":41.3661306819709,"lng":-8.37651707562145},{"lat":41.366001352092,"lng":-8.37649077019212},{"lat":41.3658803241977,"lng":-8.37649712703474},{"lat":41.3657051582051,"lng":-8.37655898047023},{"lat":41.3655323454286,"lng":-8.37657123362489},{"lat":41.3654289776486,"lng":-8.37661566426377},{"lat":41.3652134268694,"lng":-8.37665441764126},{"lat":41.3650728844982,"lng":-8.37669309194361},{"lat":41.3649141775716,"lng":-8.37676312673079},{"lat":41.364821509086,"lng":-8.37681357484193},{"lat":41.3647555158798,"lng":-8.3768521694137},{"lat":41.3646516915759,"lng":-8.37685786898773},{"lat":41.3645115934134,"lng":-8.37685555102436},{"lat":41.3643867231711,"lng":-8.37684970558836},{"lat":41.3642415694629,"lng":-8.37681198710294},{"lat":41.3641068884412,"lng":-8.37676104393253},{"lat":41.363993103594,"lng":-8.3767073164938},{"lat":41.3638966827718,"lng":-8.37666009709077},{"lat":41.3637641787951,"lng":-8.37651809763359},{"lat":41.3636594760258,"lng":-8.3763839708017},{"lat":41.3635634075421,"lng":-8.37629863401579},{"lat":41.3634537029949,"lng":-8.37618826926346},{"lat":41.3631787010528,"lng":-8.37607190231051},{"lat":41.3630436506189,"lng":-8.37606697436412},{"lat":41.3629288783584,"lng":-8.37605387538403},{"lat":41.3628051347216,"lng":-8.37602699898208},{"lat":41.3626963578963,"lng":-8.37598847196435},{"lat":41.3625530272223,"lng":-8.37589781749507},{"lat":41.3624847892196,"lng":-8.37584951710525},{"lat":41.3623997639879,"lng":-8.37573590262684},{"lat":41.3623510917303,"lng":-8.37559147070614},{"lat":41.3623278514791,"lng":-8.37542597714645},{"lat":41.3623032322324,"lng":-8.3752319233443},{"lat":41.3622906951023,"lng":-8.37502977918545},{"lat":41.3622975585667,"lng":-8.37485125573932},{"lat":41.3622937537699,"lng":-8.37464998533134},{"lat":41.3623058478985,"lng":-8.37450757170961},{"lat":41.3623303153569,"lng":-8.37434946000193},{"lat":41.3375997872772,"lng":-8.3746157419205},{"lat":41.3348360011718,"lng":-8.37455982788335}]},{"zone_id":"str7","points":[{"lat":41.3348360011718,"lng":-8.37455982788335},{"lat":41.3375997872772,"lng":-8.3746157419205},{"lat":41.3623303153569,"lng":-8.37434946000193},{"lat":41.3623500887738,"lng":-8.3741934180983},{"lat":41.3623871055463,"lng":-8.37401918737172},{"lat":41.3624200002521,"lng":-8.37387625492122},{"lat":41.3624841488657,"lng":-8.37372853621318},{"lat":41.3625669743484,"lng":-8.37356403815138},{"lat":41.362624393669,"lng":-8.37344916507425},{"lat":41.3626649249489,"lng":-8.37327040340006},{"lat":41.3627135318554,"lng":-8.37306263051888},{"lat":41.3627437578161,"lng":-8.37286292039159},{"lat":41.3627684152728,"lng":-8.3726545822677},{"lat":41.3627898997789,"lng":-8.37245554966581},{"lat":41.3628066235546,"lng":-8.37225172253035},{"lat":41.3628315306351,"lng":-8.3720540252186},{"lat":41.3628657262973,"lng":-8.37184835306821},{"lat":41.3628947991125,"lng":-8.37168137500719},{"lat":41.3629370724807,"lng":-8.37152963403513},{"lat":41.3629848754082,"lng":-8.37140467706469},{"lat":41.363037285703,"lng":-8.37122918451971},{"lat":41.3630761472537,"lng":-8.37107288775003},{"lat":41.3631092018601,"lng":-8.37089481084181},{"lat":41.3631413510868,"lng":-8.37071900960256},{"lat":41.3631627409068,"lng":-8.37056599397992},{"lat":41.3631760140453,"lng":-8.37041832573272},{"lat":41.3631999703947,"lng":-8.37024308712839},{"lat":41.3632366065203,"lng":-8.37007458721886},{"lat":41.3632868321098,"lng":-8.36990876690611},{"lat":41.3633358648279,"lng":-8.36983950529532},{"lat":41.3633947562414,"lng":-8.36979335287213},{"lat":41.3634793122887,"lng":-8.36974525799983},{"lat":41.3635525370506,"lng":-8.36969042790811},{"lat":41.3636024415359,"lng":-8.36963766452626},{"lat":41.3636515547111,"lng":-8.36957342471091},{"lat":41.3637010435238,"lng":-8.36945911032024},{"lat":41.3637262710956,"lng":-8.36936394883453},{"lat":41.3637419621135,"lng":-8.3692654135281},{"lat":41.3637385992532,"lng":-8.36914947117765},{"lat":41.3637209248755,"lng":-8.36898889781733},{"lat":41.363673302366,"lng":-8.36872388178968},{"lat":41.3635355668268,"lng":-8.36810011510439},{"lat":41.3634719283197,"lng":-8.36787185240048},{"lat":41.3634183089603,"lng":-8.36767589518664},{"lat":41.3633723885929,"lng":-8.3674136303166},{"lat":41.3633456330311,"lng":-8.36715610273667},{"lat":41.363353310021,"lng":-8.36696957097782},{"lat":41.3633589858669,"lng":-8.36657148932338},{"lat":41.3633590260716,"lng":-8.36611136519584},{"lat":41.3634146182433,"lng":-8.36560817198793},{"lat":41.3634894306885,"lng":-8.36521452450964},{"lat":41.3636357248166,"lng":-8.36482722260082},{"lat":41.3637681379785,"lng":-8.36457779811012},{"lat":41.3638799945519,"lng":-8.3643869678188},{"lat":41.3640355804839,"lng":-8.36412005458263},{"lat":41.3641286636431,"lng":-8.36399705005677},{"lat":41.3642775454078,"lng":-8.36383826541243},{"lat":41.3643528966206,"lng":-8.36375522718623},{"lat":41.3645214018625,"lng":-8.36360344627131},{"lat":41.3647354324614,"lng":-8.36337509608806},{"lat":41.3648409976131,"lng":-8.36322213422776},{"lat":41.3649383067885,"lng":-8.36310068954672},{"lat":41.365116460666,"lng":-8.36280420898912},{"lat":41.3651822679383,"lng":-8.36267548015644},{"lat":41.3652345194029,"lng":-8.36257252300745},{"lat":41.3652887852165,"lng":-8.36245092429492},{"lat":41.3653560632154,"lng":-8.36230870268953},{"lat":41.3654183846551,"lng":-8.36221199168371},{"lat":41.3654663629845,"lng":-8.3621268295064},{"lat":41.3655219449565,"lng":-8.36202412337701},{"lat":41.3655710686277,"lng":-8.36195246222284},{"lat":41.3656229679807,"lng":-8.36188906586865},{"lat":41.3657484348136,"lng":-8.36177859737012},{"lat":41.3658090335254,"lng":-8.36173255550043},{"lat":41.3659529110597,"lng":-8.36164689018885},{"lat":41.3660079372981,"lng":-8.36159485734863},{"lat":41.3660421515972,"lng":-8.36150605305028},{"lat":41.3660764089158,"lng":-8.36139371294102},{"lat":41.3661041742658,"lng":-8.36115369430836},{"lat":41.3661167377751,"lng":-8.36094996012153},{"lat":41.3661086026511,"lng":-8.36064958035302},{"lat":41.3660886242723,"lng":-8.36050800388609},{"lat":41.3660464662567,"lng":-8.36033622056153},{"lat":41.365990621819,"lng":-8.36012245038398},{"lat":41.3659355414754,"lng":-8.3599287577526},{"lat":41.3658841719029,"lng":-8.35972610957276},{"lat":41.3658465613651,"lng":-8.35953200485617},{"lat":41.3658183081624,"lng":-8.35934031718761},{"lat":41.3657857052567,"lng":-8.35916259840311},{"lat":41.3657362478885,"lng":-8.35899425875427},{"lat":41.3656810137648,"lng":-8.35887753767891},{"lat":41.365608046376,"lng":-8.35871426021722},{"lat":41.3655028691443,"lng":-8.35853689181815},{"lat":41.3654178728859,"lng":-8.35840022518169},{"lat":41.3653140458834,"lng":-8.35827066749163},{"lat":41.3652252464821,"lng":-8.35818992310344},{"lat":41.3651441056306,"lng":-8.35810561737983},{"lat":41.3650834852009,"lng":-8.35802533260182},{"lat":41.3650415267461,"lng":-8.35788869729907},{"lat":41.3650189875259,"lng":-8.35776802485739},{"lat":41.3650159166382,"lng":-8.3576335625075},{"lat":41.365035053367,"lng":-8.35747407289298},{"lat":41.365070265157,"lng":-8.35733196849091},{"lat":41.3651090574762,"lng":-8.35720338057888},{"lat":41.3651520565395,"lng":-8.35708533410739},{"lat":41.3652325893705,"lng":-8.35688231184329},{"lat":41.3652936303986,"lng":-8.35674806383214},{"lat":41.3653702113642,"lng":-8.35658507072564},{"lat":41.3654733011339,"lng":-8.3564038767066},{"lat":41.3655556929309,"lng":-8.35626265505813},{"lat":41.365623181623,"lng":-8.35614575928851},{"lat":41.3657363513638,"lng":-8.35596842527778},{"lat":41.3658072416606,"lng":-8.35586074348048},{"lat":41.3658694387811,"lng":-8.35577801738337},{"lat":41.3659611541948,"lng":-8.35565785655055},{"lat":41.3660274445633,"lng":-8.35555362445623},{"lat":41.366098320495,"lng":-8.35545550567088},{"lat":41.366167084271,"lng":-8.3553754246334},{"lat":41.3662336862926,"lng":-8.35529665199144},{"lat":41.3664180223165,"lng":-8.35509900280894},{"lat":41.3664713466058,"lng":-8.35504086035189},{"lat":41.3665528351234,"lng":-8.35494743770292},{"lat":41.3667235422353,"lng":-8.35472021381929},{"lat":41.3668530197524,"lng":-8.35453349343453},{"lat":41.3669204745464,"lng":-8.35443236789309},{"lat":41.3669874234088,"lng":-8.35431475122618},{"lat":41.3670815858358,"lng":-8.35418503163164},{"lat":41.3671696379402,"lng":-8.35405063561209},{"lat":41.3676153435529,"lng":-8.35348681982506},{"lat":41.3686813003955,"lng":-8.35209358156926},{"lat":41.3688060069505,"lng":-8.35190539771844},{"lat":41.368920041059,"lng":-8.35174072149123},{"lat":41.3690348964107,"lng":-8.3515701894983},{"lat":41.3691499798963,"lng":-8.35142213206425},{"lat":41.3692543572692,"lng":-8.35127080636669},{"lat":41.3693589903006,"lng":-8.35112605855334},{"lat":41.3694340957341,"lng":-8.35097726262442},{"lat":41.36946495902,"lng":-8.35084780162064},{"lat":41.369483344936,"lng":-8.35064945182951},{"lat":41.3694847462882,"lng":-8.35052921553353},{"lat":41.3694950828422,"lng":-8.35025314797243},{"lat":41.369512708744,"lng":-8.35002849962183},{"lat":41.3695317516479,"lng":-8.34991226260537},{"lat":41.3695723051565,"lng":-8.34975008641804},{"lat":41.3696091627604,"lng":-8.34959243858663},{"lat":41.3697141977359,"lng":-8.34928202354103},{"lat":41.3698031466701,"lng":-8.3490960964505},{"lat":41.3699003485551,"lng":-8.34888032155752},{"lat":41.3700398264202,"lng":-8.34849496050637},{"lat":41.3700968887293,"lng":-8.34836307102046},{"lat":41.3701821821094,"lng":-8.34820761383805},{"lat":41.3702830970884,"lng":-8.34807348000522},{"lat":41.3703455006205,"lng":-8.34797507859441},{"lat":41.3704205939397,"lng":-8.34783010403865},{"lat":41.3705105247697,"lng":-8.34760306091863},{"lat":41.3705728369958,"lng":-8.347408439417},{"lat":41.3706734948034,"lng":-8.34712406270733},{"lat":41.3707496478266,"lng":-8.34689374536007},{"lat":41.3708033318437,"lng":-8.34673853988593},{"lat":41.3708913525338,"lng":-8.34647120529714},{"lat":41.3709433589941,"lng":-8.34634587707483},{"lat":41.3710261613015,"lng":-8.34617319374168},{"lat":41.3710889283863,"lng":-8.34602195750797},{"lat":41.3711536012429,"lng":-8.34586546431967},{"lat":41.3712243241182,"lng":-8.34569704953261},{"lat":41.371273422861,"lng":-8.34558508713827},{"lat":41.3713231295253,"lng":-8.345486646542},{"lat":41.371414906236,"lng":-8.34532689470968},{"lat":41.3715107864259,"lng":-8.34518736109047},{"lat":41.3716265087964,"lng":-8.3450325963869},{"lat":41.3716964274799,"lng":-8.34495751804457},{"lat":41.3717717055596,"lng":-8.34485784442021},{"lat":41.3718331538271,"lng":-8.34478727803554},{"lat":41.3719816007351,"lng":-8.34470457538999},{"lat":41.372073408358,"lng":-8.3446748730057},{"lat":41.3722859741361,"lng":-8.34458663995932},{"lat":41.372371881369,"lng":-8.34453145676389},{"lat":41.3724429129719,"lng":-8.34448854061501},{"lat":41.3725113415317,"lng":-8.34444118849262},{"lat":41.372646762726,"lng":-8.34439047425057},{"lat":41.3727489982126,"lng":-8.34436988811238},{"lat":41.372924266463,"lng":-8.34434141235466},{"lat":41.3730486156877,"lng":-8.34433907372991},{"lat":41.3731819677238,"lng":-8.34433544218555},{"lat":41.3732934456288,"lng":-8.34433067004686},{"lat":41.3733705307597,"lng":-8.34432542445536},{"lat":41.3734828550129,"lng":-8.34430104315984},{"lat":41.3736975562547,"lng":-8.3442741324229},{"lat":41.3738088219027,"lng":-8.34423732551064},{"lat":41.3739520677071,"lng":-8.34419189226575},{"lat":41.3741379212323,"lng":-8.34413679229603},{"lat":41.374364965163,"lng":-8.34401262023759},{"lat":41.3745046148943,"lng":-8.34391446776395},{"lat":41.3746258185206,"lng":-8.34381158389293},{"lat":41.3747169646292,"lng":-8.34369844215085},{"lat":41.3747964970014,"lng":-8.34358837058997},{"lat":41.3748686783039,"lng":-8.34350493042268},{"lat":41.3749246095638,"lng":-8.34340051770509},{"lat":41.3749610937283,"lng":-8.34329269519139},{"lat":41.3749933220132,"lng":-8.3431959805204},{"lat":41.3750790935636,"lng":-8.34306871230845},{"lat":41.3751382862957,"lng":-8.3430014888181},{"lat":41.3752152206595,"lng":-8.34293025319193},{"lat":41.3753065447253,"lng":-8.34286720049912},{"lat":41.3754412200012,"lng":-8.34278169195542},{"lat":41.3754969359072,"lng":-8.34274373970215},{"lat":41.3755510753763,"lng":-8.34267948411547},{"lat":41.3755894772174,"lng":-8.34245906808169},{"lat":41.3755906290156,"lng":-8.34232231981358},{"lat":41.3755925887592,"lng":-8.34184358603197},{"lat":41.3755275132692,"lng":-8.34114181854348},{"lat":41.3754686871033,"lng":-8.34066515479243},{"lat":41.3753957248288,"lng":-8.34008816625643},{"lat":41.3753238203637,"lng":-8.33942379370761},{"lat":41.3752294570656,"lng":-8.33882581788575},{"lat":41.3752035845815,"lng":-8.33869245297368},{"lat":41.3751430377006,"lng":-8.33841613405809},{"lat":41.3749988142486,"lng":-8.33785434166614},{"lat":41.3748788880207,"lng":-8.33745089231722},{"lat":41.3748277994457,"lng":-8.3372208611334},{"lat":41.3748131025118,"lng":-8.33692855214748},{"lat":41.3746365762369,"lng":-8.33619991190267},{"lat":41.3744869242277,"lng":-8.335849810253},{"lat":41.3744730393317,"lng":-8.33560830335609},{"lat":41.3744977577336,"lng":-8.33548107602295},{"lat":41.3747736453164,"lng":-8.33476700486247},{"lat":41.3748633570191,"lng":-8.33459431923678},{"lat":41.3751024939984,"lng":-8.33414047542605},{"lat":41.375158362125,"lng":-8.33406271316374},{"lat":41.375197924744,"lng":-8.33399374153589},{"lat":41.3752675677125,"lng":-8.33371173420432},{"lat":41.3753011460625,"lng":-8.3335119789103},{"lat":41.3753246718241,"lng":-8.33324034612552},{"lat":41.3753501223206,"lng":-8.33300171123467},{"lat":41.3753870742873,"lng":-8.33282885788326},{"lat":41.3754369065872,"lng":-8.33264959228347},{"lat":41.3754874643767,"lng":-8.33236632647671},{"lat":41.3755254433068,"lng":-8.33206820047064},{"lat":41.375536208676,"lng":-8.33183310461521},{"lat":41.3755275576742,"lng":-8.33163273739385},{"lat":41.3755205530199,"lng":-8.33146536512043},{"lat":41.3755536874258,"lng":-8.33131042706707},{"lat":41.3756442968245,"lng":-8.3311891360819},{"lat":41.3757907313705,"lng":-8.33112109237066},{"lat":41.3759016040626,"lng":-8.33109931529415},{"lat":41.3759670708106,"lng":-8.33108453132444},{"lat":41.3761471069621,"lng":-8.3310162788843},{"lat":41.3762908100908,"lng":-8.33091283775943},{"lat":41.3763983618158,"lng":-8.33083355788484},{"lat":41.3764562076151,"lng":-8.3307721515071},{"lat":41.3764935840094,"lng":-8.33072555386121},{"lat":41.3765467151923,"lng":-8.330645257245},{"lat":41.3765823672224,"lng":-8.33058766870458},{"lat":41.3766476672161,"lng":-8.33049278563876},{"lat":41.3768323385052,"lng":-8.33032350059274},{"lat":41.376900574978,"lng":-8.33022664518196},{"lat":41.3770332773531,"lng":-8.32992365561554},{"lat":41.3770202497644,"lng":-8.32982760129984},{"lat":41.3769960217083,"lng":-8.32964896597744},{"lat":41.3768790119708,"lng":-8.32928497375625},{"lat":41.3767594669368,"lng":-8.32898397506524},{"lat":41.3767100745304,"lng":-8.32874940763899},{"lat":41.3767134073735,"lng":-8.32848786530619},{"lat":41.3767945672537,"lng":-8.32804761790185},{"lat":41.3768243073037,"lng":-8.32768385085749},{"lat":41.3768292653733,"lng":-8.32734303876777},{"lat":41.376811929518,"lng":-8.32710893576431},{"lat":41.3768048707834,"lng":-8.32701193282725},{"lat":41.3767908131624,"lng":-8.32681875281336},{"lat":41.3768431027791,"lng":-8.3263010703826},{"lat":41.3768506743085,"lng":-8.32608532361963},{"lat":41.3765785840923,"lng":-8.32600250398416},{"lat":41.3761601786338,"lng":-8.32591626959512},{"lat":41.3759804961527,"lng":-8.32585769792872},{"lat":41.3758851822389,"lng":-8.32582217490135},{"lat":41.3756944177458,"lng":-8.32574955815708},{"lat":41.3755081533895,"lng":-8.32568098472453},{"lat":41.37525749514,"lng":-8.3255664440219},{"lat":41.3743846179012,"lng":-8.32556682637513},{"lat":41.3737231114703,"lng":-8.32556711616272},{"lat":41.3734861457948,"lng":-8.32557096495879},{"lat":41.3733872702926,"lng":-8.32559588198541},{"lat":41.3731879916665,"lng":-8.32564610077254},{"lat":41.3727583777134,"lng":-8.32561242524305},{"lat":41.3725198090112,"lng":-8.32559318613662},{"lat":41.3723535306297,"lng":-8.3257915915796},{"lat":41.3721782103988,"lng":-8.3258527451535},{"lat":41.3719637540635,"lng":-8.3258945395904},{"lat":41.370918836907,"lng":-8.32608033152625},{"lat":41.3703948876705,"lng":-8.32617349029482},{"lat":41.3703394755067,"lng":-8.32628300969792},{"lat":41.3702901824824,"lng":-8.32638043459461},{"lat":41.3700925785051,"lng":-8.32657455143187},{"lat":41.3699300693789,"lng":-8.32670735936767},{"lat":41.3697355280007,"lng":-8.32677513248913},{"lat":41.3696695596398,"lng":-8.32679273465823},{"lat":41.3695900133874,"lng":-8.32679568842875},{"lat":41.3693925822667,"lng":-8.32680301955808},{"lat":41.3690437815896,"lng":-8.32679552110601},{"lat":41.3686340115524,"lng":-8.32675410894397},{"lat":41.3681679399859,"lng":-8.32670803299901},{"lat":41.3679967348785,"lng":-8.32688678899468},{"lat":41.3678589563979,"lng":-8.32703064321137},{"lat":41.3675249770803,"lng":-8.32737934699693},{"lat":41.3673548771667,"lng":-8.32755694447082},{"lat":41.3671710727323,"lng":-8.32757730999697},{"lat":41.3670946377678,"lng":-8.32762393138412},{"lat":41.3670191667793,"lng":-8.32763613208728},{"lat":41.366846162205,"lng":-8.32765784539022},{"lat":41.366743420702,"lng":-8.32766838290266},{"lat":41.3666033390764,"lng":-8.32770204880734},{"lat":41.3662442124645,"lng":-8.32794359545005},{"lat":41.3660970944895,"lng":-8.32799854205773},{"lat":41.3660020253377,"lng":-8.32803644507598},{"lat":41.365925531972,"lng":-8.3280669297619},{"lat":41.3658535652136,"lng":-8.32807808657286},{"lat":41.3657712501834,"lng":-8.32809084761813},{"lat":41.3649514510975,"lng":-8.32766552440366},{"lat":41.3647494173163,"lng":-8.32755089449745},{"lat":41.3646082944438,"lng":-8.32751760474114},{"lat":41.3644231734748,"lng":-8.32751668683374},{"lat":41.364328741537,"lng":-8.32745329732351},{"lat":41.3642081042409,"lng":-8.32740437464014},{"lat":41.3641605022734,"lng":-8.32737451465319},{"lat":41.3641113336244,"lng":-8.32732608140777},{"lat":41.3636663507942,"lng":-8.32706635129754},{"lat":41.3633474114694,"lng":-8.32644416772825},{"lat":41.3631758944558,"lng":-8.32606622913837},{"lat":41.3631213714942,"lng":-8.32593448252826},{"lat":41.3630317027744,"lng":-8.32571118881156},{"lat":41.3629446105702,"lng":-8.3255178109079},{"lat":41.3628722916847,"lng":-8.32536808093965},{"lat":41.3627245982452,"lng":-8.32517331790834},{"lat":41.3626498343849,"lng":-8.32508238478059},{"lat":41.3625859724998,"lng":-8.32498933264126},{"lat":41.362499232159,"lng":-8.32484924592556},{"lat":41.3624194672076,"lng":-8.32468312853457},{"lat":41.3623577985079,"lng":-8.32453168472357},{"lat":41.3623132044129,"lng":-8.32437268278972},{"lat":41.3622449221059,"lng":-8.32406796738768},{"lat":41.3621527009576,"lng":-8.32391826434717},{"lat":41.3620182473266,"lng":-8.32382648096005},{"lat":41.3618710915928,"lng":-8.32379258190651},{"lat":41.3617254954345,"lng":-8.32374123760799},{"lat":41.361646471124,"lng":-8.32372176438563},{"lat":41.3615531719983,"lng":-8.32367822890317},{"lat":41.3614811361318,"lng":-8.32362805574346},{"lat":41.3613817768391,"lng":-8.32354721767021},{"lat":41.3613002069231,"lng":-8.32349080839105},{"lat":41.3611414470158,"lng":-8.32339460906105},{"lat":41.3606985614056,"lng":-8.32306107122131},{"lat":41.3606225207323,"lng":-8.32298113813393},{"lat":41.3604081296739,"lng":-8.32293870314337},{"lat":41.3603596041713,"lng":-8.32292909840168},{"lat":41.360241584599,"lng":-8.32293269365068},{"lat":41.3600519207292,"lng":-8.3229384713856},{"lat":41.3599887480878,"lng":-8.32294630482519},{"lat":41.3598259670248,"lng":-8.32296648969015},{"lat":41.3596232923072,"lng":-8.32296182783609},{"lat":41.3591785126792,"lng":-8.32295442365524},{"lat":41.3589371047758,"lng":-8.32295981101221},{"lat":41.3585999464276,"lng":-8.32299156878551},{"lat":41.3583255816912,"lng":-8.32305541084951},{"lat":41.3582446947411,"lng":-8.32301764970525},{"lat":41.3580602051278,"lng":-8.32290895394016},{"lat":41.357736559438,"lng":-8.32271823422118},{"lat":41.3573529042061,"lng":-8.32234019671916},{"lat":41.3568924338751,"lng":-8.32120238774788},{"lat":41.3561302368011,"lng":-8.32099391068646},{"lat":41.3554125078527,"lng":-8.32019781962967},{"lat":41.3553351055257,"lng":-8.3201119678854},{"lat":41.3553051466639,"lng":-8.32004244889144},{"lat":41.3552223381062,"lng":-8.3198077551074},{"lat":41.3555829801961,"lng":-8.31958726760441},{"lat":41.3558127753912,"lng":-8.31944677519505},{"lat":41.3559236172268,"lng":-8.31930668673909},{"lat":41.3560572919739,"lng":-8.3191377397807},{"lat":41.3560898445378,"lng":-8.31899610820678},{"lat":41.3560452931753,"lng":-8.31843396644654},{"lat":41.3560463154398,"lng":-8.31797246118135},{"lat":41.3512596584627,"lng":-8.31644970396603},{"lat":41.3507638123508,"lng":-8.31671264321477},{"lat":41.3484561345998,"lng":-8.31793631097611},{"lat":41.3468224009445,"lng":-8.31896301505754},{"lat":41.3464911797513,"lng":-8.31938633806658},{"lat":41.3463059334325,"lng":-8.31962309260341},{"lat":41.3457626637164,"lng":-8.32031740851835},{"lat":41.345654018024,"lng":-8.3204562591252},{"lat":41.3455562320646,"lng":-8.32066469278224},{"lat":41.3454351765284,"lng":-8.32092272424786},{"lat":41.3437288083367,"lng":-8.32455963734668},{"lat":41.343413382698,"lng":-8.32536863339251},{"lat":41.3430745367569,"lng":-8.32623696243486},{"lat":41.3428719664557,"lng":-8.32675605821694},{"lat":41.3417754440414,"lng":-8.32908418986523},{"lat":41.3414397196512,"lng":-8.32963681465308},{"lat":41.3413722019096,"lng":-8.3297479442065},{"lat":41.3411016522112,"lng":-8.33019324629879},{"lat":41.3409374465718,"lng":-8.33048663833937},{"lat":41.3405303838892,"lng":-8.33121393953944},{"lat":41.3396748558089,"lng":-8.33274246004287},{"lat":41.3377479737566,"lng":-8.33521055182174},{"lat":41.3358146770841,"lng":-8.33835632459669},{"lat":41.3356459941047,"lng":-8.33863078241815},{"lat":41.3351698284593,"lng":-8.33942009867571},{"lat":41.335072533045,"lng":-8.3395813779535},{"lat":41.3349692080556,"lng":-8.33975265106645},{"lat":41.3350756996872,"lng":-8.3398812395959},{"lat":41.3351693816469,"lng":-8.33999436095331},{"lat":41.3351344495973,"lng":-8.34041911997042},{"lat":41.3351193567659,"lng":-8.34091346608721},{"lat":41.3350736435685,"lng":-8.34114630730689},{"lat":41.3350406495337,"lng":-8.34131436148752},{"lat":41.3350928240794,"lng":-8.3415040988943},{"lat":41.3351349875469,"lng":-8.34165743117357},{"lat":41.3351606268186,"lng":-8.34177079995326},{"lat":41.3352216485717,"lng":-8.34204258913172},{"lat":41.3352719903049,"lng":-8.34238071144301},{"lat":41.3352573940308,"lng":-8.34297200364222},{"lat":41.335216588043,"lng":-8.34327614032929},{"lat":41.335154304628,"lng":-8.34379929983804},{"lat":41.3352225725345,"lng":-8.34427653896526},{"lat":41.3352247754476,"lng":-8.34444845174306},{"lat":41.3352514238092,"lng":-8.34463640416857},{"lat":41.335256230153,"lng":-8.34482043885205},{"lat":41.3352797441016,"lng":-8.34510212644249},{"lat":41.3353155391508,"lng":-8.34528982618757},{"lat":41.3353531641949,"lng":-8.34546979349923},{"lat":41.3353948191531,"lng":-8.34558580993103},{"lat":41.3354725663328,"lng":-8.34585126788764},{"lat":41.3355004165692,"lng":-8.34598002368764},{"lat":41.3355143444404,"lng":-8.34604410058576},{"lat":41.3355257112052,"lng":-8.34611770485199},{"lat":41.3355458436784,"lng":-8.34624442267446},{"lat":41.3356088384437,"lng":-8.34640710418442},{"lat":41.3357058044289,"lng":-8.34660286312472},{"lat":41.3357860137186,"lng":-8.34675627861548},{"lat":41.3358320130708,"lng":-8.34681103024651},{"lat":41.3358639518422,"lng":-8.3468709804876},{"lat":41.3358936322184,"lng":-8.34698576489909},{"lat":41.33586951071,"lng":-8.34707755317669},{"lat":41.3356934402172,"lng":-8.34733883220901},{"lat":41.3354887980914,"lng":-8.34766297843021},{"lat":41.3354314493132,"lng":-8.34776096208109},{"lat":41.3353891028799,"lng":-8.34783331317005},{"lat":41.3353449014896,"lng":-8.34790883336665},{"lat":41.335308343485,"lng":-8.3479732193739},{"lat":41.3352565490697,"lng":-8.34806443953643},{"lat":41.3351669772482,"lng":-8.34823877169472},{"lat":41.3351145755418,"lng":-8.34834075981115},{"lat":41.3349979279863,"lng":-8.34848877970734},{"lat":41.3347792517989,"lng":-8.34873147825186},{"lat":41.3346167158958,"lng":-8.3489774995884},{"lat":41.3345908121903,"lng":-8.34902843040991},{"lat":41.3345486594592,"lng":-8.34911130922953},{"lat":41.3344716560058,"lng":-8.34935989097044},{"lat":41.3343927004301,"lng":-8.34954315853615},{"lat":41.3339600721809,"lng":-8.35100338693234},{"lat":41.3339632397095,"lng":-8.35108856737986},{"lat":41.3339373514288,"lng":-8.35118310845433},{"lat":41.3338452805582,"lng":-8.35142206412782},{"lat":41.3338208976371,"lng":-8.35150634404437},{"lat":41.3337915409013,"lng":-8.35160262430917},{"lat":41.3337694671818,"lng":-8.3516558253267},{"lat":41.3337253077984,"lng":-8.35177131937645},{"lat":41.333751174537,"lng":-8.3518928970548},{"lat":41.3338345364657,"lng":-8.35204752010673},{"lat":41.3339845196549,"lng":-8.35425170298808},{"lat":41.3337589299481,"lng":-8.35681250110235},{"lat":41.3336937356653,"lng":-8.35731878444235},{"lat":41.3336780083941,"lng":-8.35744051170594},{"lat":41.3336589009056,"lng":-8.35758840004157},{"lat":41.3335773692171,"lng":-8.35887309244985},{"lat":41.3335603153873,"lng":-8.3591641484595},{"lat":41.33344665037,"lng":-8.36110373337393},{"lat":41.3334348918353,"lng":-8.36130434932721},{"lat":41.3334173400092,"lng":-8.36168273554511},{"lat":41.3333101800311,"lng":-8.3639923287621},{"lat":41.3332836500388,"lng":-8.36610282781519},{"lat":41.3328530377194,"lng":-8.36831590808286},{"lat":41.3325824284727,"lng":-8.36925206909656},{"lat":41.3325338529758,"lng":-8.36948901543504},{"lat":41.3324635350337,"lng":-8.3698309400281},{"lat":41.3324391692895,"lng":-8.36994941838296},{"lat":41.3324356814733,"lng":-8.37016649532366},{"lat":41.3324273037995,"lng":-8.37068780620501},{"lat":41.3324224146301,"lng":-8.37099197274175},{"lat":41.3324802779816,"lng":-8.3713092237375},{"lat":41.3325225042163,"lng":-8.37145112032717},{"lat":41.3325741578553,"lng":-8.37162469771728},{"lat":41.3326986862622,"lng":-8.3718646282108},{"lat":41.3327302013375,"lng":-8.37192534901333},{"lat":41.3328836867263,"lng":-8.37215671080652},{"lat":41.334489735596,"lng":-8.37412047306054},{"lat":41.3346915071403,"lng":-8.37438069660635},{"lat":41.3348360011718,"lng":-8.37455982788335}]},{"zone_id":"smf4","points":[{"lat":40.947267200764,"lng":-8.50492894885178},{"lat":40.9302989925647,"lng":-8.49315832873953},{"lat":40.9254088945652,"lng":-8.48976741936876},{"lat":40.9241946535608,"lng":-8.48949104881503},{"lat":40.9225922500529,"lng":-8.49055142006195},{"lat":40.9206082680716,"lng":-8.49075339553756},{"lat":40.9167728141759,"lng":-8.48898499576732},{"lat":40.9159130388278,"lng":-8.49007302295998},{"lat":40.9156722270899,"lng":-8.48972440914533},{"lat":40.9152169739164,"lng":-8.49103245803335},{"lat":40.9138229142112,"lng":-8.49153898829306},{"lat":40.9138152379563,"lng":-8.49154642740431},{"lat":40.9135758977348,"lng":-8.49148913597461},{"lat":40.9134643256653,"lng":-8.49153162299244},{"lat":40.913258873239,"lng":-8.49147752949901},{"lat":40.9132192280616,"lng":-8.49145665916921},{"lat":40.9131633223514,"lng":-8.4914238286616},{"lat":40.9131386158944,"lng":-8.49140648253971},{"lat":40.9123088597661,"lng":-8.49116909572445},{"lat":40.911434899858,"lng":-8.4913321914386},{"lat":40.9112347987498,"lng":-8.4914764990167},{"lat":40.9110022133285,"lng":-8.49173694734051},{"lat":40.910802851948,"lng":-8.49171659287822},{"lat":40.9100657550713,"lng":-8.49303279574714},{"lat":40.9095613161413,"lng":-8.49278588637556},{"lat":40.9093721483181,"lng":-8.49348800333609},{"lat":40.9093103114301,"lng":-8.49384180258679},{"lat":40.9092660950372,"lng":-8.49445178831094},{"lat":40.9091269063971,"lng":-8.49446619162536},{"lat":40.9089019040685,"lng":-8.49448620439316},{"lat":40.908686461234,"lng":-8.49469054937441},{"lat":40.9079051748318,"lng":-8.49836210655259},{"lat":40.9077863452449,"lng":-8.49856910629726},{"lat":40.9075674239387,"lng":-8.49864147488183},{"lat":40.90734568504,"lng":-8.49891339159111},{"lat":40.9066899500424,"lng":-8.49912802151431},{"lat":40.9061034688925,"lng":-8.49907708482125},{"lat":40.9052501192554,"lng":-8.49995214068382},{"lat":40.9048759499688,"lng":-8.50039802184157},{"lat":40.904579106746,"lng":-8.50059942122296},{"lat":40.9044186413181,"lng":-8.50068749824582},{"lat":40.9041840582651,"lng":-8.50080476392478},{"lat":40.9039480433343,"lng":-8.50092369195479},{"lat":40.9035045383038,"lng":-8.50119374460425},{"lat":40.9032311999006,"lng":-8.5014411511636},{"lat":40.9031166952506,"lng":-8.50154404632367},{"lat":40.9029755521643,"lng":-8.50167294180161},{"lat":40.9028691765666,"lng":-8.50176855703376},{"lat":40.902754626337,"lng":-8.5018218905658},{"lat":40.9025361573455,"lng":-8.50188170952224},{"lat":40.9024286113429,"lng":-8.50191267834832},{"lat":40.9019915974248,"lng":-8.50189741315696},{"lat":40.9006813771888,"lng":-8.50245459867903},{"lat":40.8994966468984,"lng":-8.50371699694569},{"lat":40.8988649611283,"lng":-8.50424077540329},{"lat":40.8983706089754,"lng":-8.5046451867915},{"lat":40.8982621103422,"lng":-8.5047366056604},{"lat":40.8978214111301,"lng":-8.5051271699295},{"lat":40.8976510992685,"lng":-8.50527613489536},{"lat":40.8973676760499,"lng":-8.50554598026441},{"lat":40.8965401188666,"lng":-8.50638952655139},{"lat":40.8962523818937,"lng":-8.50668615295151},{"lat":40.8957784732911,"lng":-8.50705209282936},{"lat":40.8956906401839,"lng":-8.50710571738833},{"lat":40.8954642041378,"lng":-8.50730459521612},{"lat":40.8953920121852,"lng":-8.50744659055305},{"lat":40.8952859137104,"lng":-8.50768863200965},{"lat":40.8952602368945,"lng":-8.50776471622773},{"lat":40.8951737868929,"lng":-8.50775993985621},{"lat":40.8950644222329,"lng":-8.50909108837596},{"lat":40.894431955507,"lng":-8.5098921232233},{"lat":40.893588433478,"lng":-8.51061346729984},{"lat":40.8928962108482,"lng":-8.51204474028974},{"lat":40.8928150876798,"lng":-8.51231434468365},{"lat":40.8922359257348,"lng":-8.51419231713343},{"lat":40.8916192871906,"lng":-8.51640237408322},{"lat":40.8914867101279,"lng":-8.51737795170409},{"lat":40.8914300507398,"lng":-8.5177880206622},{"lat":40.8913815941644,"lng":-8.51852789080591},{"lat":40.8912979686081,"lng":-8.51892203864677},{"lat":40.8910618232196,"lng":-8.51981004247981},{"lat":40.8909466398039,"lng":-8.52030670900772},{"lat":40.8907688589972,"lng":-8.52105837860207},{"lat":40.8907040311175,"lng":-8.52163831338079},{"lat":40.890666019307,"lng":-8.52214756816615},{"lat":40.8906373672035,"lng":-8.52278448190126},{"lat":40.8905108225651,"lng":-8.52326564461008},{"lat":40.8903712516211,"lng":-8.52645116006317},{"lat":40.8902537049967,"lng":-8.52791325965459},{"lat":40.8904724253704,"lng":-8.52896192746637},{"lat":40.891030229073,"lng":-8.53053043018235},{"lat":40.8912102081796,"lng":-8.53069013436012},{"lat":40.8913273795414,"lng":-8.53084334392517},{"lat":40.8915152675302,"lng":-8.53106269008129},{"lat":40.8930056952916,"lng":-8.53105206781968},{"lat":40.8950262674417,"lng":-8.53126068154176},{"lat":40.8953909022986,"lng":-8.53154188251856},{"lat":40.8982507089194,"lng":-8.53374744963529},{"lat":40.9010170628333,"lng":-8.53638142265323},{"lat":40.9039983683771,"lng":-8.54012607674781},{"lat":40.9043380040874,"lng":-8.54128720102943},{"lat":40.9057915196363,"lng":-8.54151418790548},{"lat":40.9070780338408,"lng":-8.5416143193101},{"lat":40.9089699210271,"lng":-8.54181458211934},{"lat":40.9104077190876,"lng":-8.54191471352396},{"lat":40.9122995110077,"lng":-8.54171445071472},{"lat":40.9141912487985,"lng":-8.54181458211934},{"lat":40.9158181000056,"lng":-8.54196477922627},{"lat":40.917577324279,"lng":-8.54128889224507},{"lat":40.9212753258773,"lng":-8.54020935053897},{"lat":40.9242827551823,"lng":-8.54005915343204},{"lat":40.9270063466461,"lng":-8.54111053318056},{"lat":40.9296102072293,"lng":-8.54090867410702},{"lat":40.9316092482936,"lng":-8.54111551846903},{"lat":40.9336131709664,"lng":-8.54158318334398},{"lat":40.9320687480561,"lng":-8.53658849047905},{"lat":40.9320301476711,"lng":-8.53341384234223},{"lat":40.9389479963764,"lng":-8.528139659891},{"lat":40.9422157484565,"lng":-8.51687864111915},{"lat":40.947267200764,"lng":-8.50492894885178}]},{"zone_id":"arc5","points":[{"lat":40.8607596876563,"lng":-8.24215090949117},{"lat":40.8857617306792,"lng":-8.26941009786677},{"lat":40.8901883310277,"lng":-8.27288753224977},{"lat":40.8925095663283,"lng":-8.27430526007792},{"lat":40.8951285006973,"lng":-8.27580222088663},{"lat":40.9007248703254,"lng":-8.27785527259696},{"lat":40.9067382369222,"lng":-8.27983108586241},{"lat":40.9113830316793,"lng":-8.28070500253754},{"lat":40.9148278970815,"lng":-8.28087985562267},{"lat":40.9128760325009,"lng":-8.27535694716896},{"lat":40.9119625336645,"lng":-8.27240578311283},{"lat":40.9115142197504,"lng":-8.26860184754054},{"lat":40.9114580334285,"lng":-8.26569672714071},{"lat":40.913139468909,"lng":-8.25321609627012},{"lat":40.9134165636006,"lng":-8.25036154576368},{"lat":40.9137049297554,"lng":-8.24791234563159},{"lat":40.9150968464108,"lng":-8.23656107525625},{"lat":40.9133870557816,"lng":-8.23680880518195},{"lat":40.9093377647481,"lng":-8.23570332229729},{"lat":40.8991376242726,"lng":-8.2331921224664},{"lat":40.8927061157855,"lng":-8.23167539303297},{"lat":40.8848838999189,"lng":-8.22930354212419},{"lat":40.8830212592757,"lng":-8.22859757067294},{"lat":40.8821578952718,"lng":-8.22958709163284},{"lat":40.8793540660289,"lng":-8.2319184583611},{"lat":40.8771228500094,"lng":-8.23327115579432},{"lat":40.8748922120684,"lng":-8.23394577859882},{"lat":40.8728906891634,"lng":-8.23416872994026},{"lat":40.871003856738,"lng":-8.23401520654544},{"lat":40.8690616765454,"lng":-8.23346169672948},{"lat":40.8649560582558,"lng":-8.2310643139813},{"lat":40.86453375579,"lng":-8.2306422752119},{"lat":40.8646825221406,"lng":-8.23104720463261},{"lat":40.8646945418377,"lng":-8.23151799687916},{"lat":40.8641244025881,"lng":-8.23310962107749},{"lat":40.8625015172452,"lng":-8.23706487870031},{"lat":40.8617738292393,"lng":-8.23912262834955},{"lat":40.8609431885232,"lng":-8.24173203864986},{"lat":40.8608207211084,"lng":-8.24201159126415},{"lat":40.8607596876563,"lng":-8.24215090949117}]},{"zone_id":"arc17","points":[{"lat":40.9504506359783,"lng":-8.1315727304949},{"lat":40.9519351230421,"lng":-8.14222501063955},{"lat":40.9528879589536,"lng":-8.14411064791726},{"lat":40.9543173719458,"lng":-8.14499080709898},{"lat":40.9573217754609,"lng":-8.14637252644747},{"lat":40.9622761581482,"lng":-8.14688950466129},{"lat":40.970281151931,"lng":-8.14839997111646},{"lat":40.9816362866363,"lng":-8.14708190613179},{"lat":40.9813108921664,"lng":-8.14665774850874},{"lat":40.9810367366156,"lng":-8.14639045038244},{"lat":40.9809026854592,"lng":-8.14621385155076},{"lat":40.9806899873768,"lng":-8.14607102932999},{"lat":40.9805232969725,"lng":-8.14569939519156},{"lat":40.9803504343844,"lng":-8.14548322562776},{"lat":40.9801387688525,"lng":-8.1452634771294},{"lat":40.9797822958346,"lng":-8.14507269958668},{"lat":40.9791153596301,"lng":-8.14403955975625},{"lat":40.9790794478687,"lng":-8.14390837671039},{"lat":40.9788536672353,"lng":-8.14338933529769},{"lat":40.9785565704243,"lng":-8.14291805524605},{"lat":40.9789364385164,"lng":-8.14235362493656},{"lat":40.9790962325776,"lng":-8.14179912581971},{"lat":40.9790777045263,"lng":-8.14048143745092},{"lat":40.9791629681409,"lng":-8.13941553661595},{"lat":40.9793149963136,"lng":-8.13897877885081},{"lat":40.9793308891434,"lng":-8.13833229553592},{"lat":40.9790892169815,"lng":-8.13848161650816},{"lat":40.9786906403933,"lng":-8.13872788070909},{"lat":40.9787870307111,"lng":-8.13804291592672},{"lat":40.9791717553314,"lng":-8.13710976082635},{"lat":40.9788546762768,"lng":-8.13586167109663},{"lat":40.9787601303293,"lng":-8.13530841598073},{"lat":40.9786958703712,"lng":-8.1349323953544},{"lat":40.978566950763,"lng":-8.13399135794112},{"lat":40.9785868630463,"lng":-8.13342044298546},{"lat":40.9786717847009,"lng":-8.1330796851145},{"lat":40.979219810703,"lng":-8.13247325394381},{"lat":40.979793319736,"lng":-8.13166589229211},{"lat":40.9798034952552,"lng":-8.13130004980972},{"lat":40.9804321033401,"lng":-8.13072662408764},{"lat":40.9809533774728,"lng":-8.13087740726613},{"lat":40.9815998991461,"lng":-8.12970031724715},{"lat":40.9820368762534,"lng":-8.12890470916172},{"lat":40.9832850155634,"lng":-8.12720217544376},{"lat":40.9833735501733,"lng":-8.12708140539469},{"lat":40.9834593442719,"lng":-8.12680475917229},{"lat":40.9836638221401,"lng":-8.1261454012359},{"lat":40.9846690687514,"lng":-8.12395045611904},{"lat":40.9851243994231,"lng":-8.12353130005304},{"lat":40.9862683561822,"lng":-8.12248819168462},{"lat":40.9864302563476,"lng":-8.12233497461059},{"lat":40.9874216984846,"lng":-8.12139668522727},{"lat":40.9876321716433,"lng":-8.12119749129332},{"lat":40.9878375509193,"lng":-8.12103909992107},{"lat":40.9882045519932,"lng":-8.12075606067619},{"lat":40.9884907228096,"lng":-8.12080010243577},{"lat":40.9885884616251,"lng":-8.12078052660183},{"lat":40.9888599886524,"lng":-8.1206371318866},{"lat":40.9890837150913,"lng":-8.12049990143726},{"lat":40.9895572548806,"lng":-8.12020943580866},{"lat":40.9905644433816,"lng":-8.11959059967742},{"lat":40.991261657432,"lng":-8.11926620486681},{"lat":40.9917014827958,"lng":-8.11906156228405},{"lat":40.9920007143256,"lng":-8.11897840683686},{"lat":40.9925654483787,"lng":-8.11882146701421},{"lat":40.992838817031,"lng":-8.11875034554454},{"lat":40.9931658966189,"lng":-8.11908851191777},{"lat":40.9936310988136,"lng":-8.11936236393047},{"lat":40.9940386561901,"lng":-8.11928877346124},{"lat":40.9941384086756,"lng":-8.11927076154487},{"lat":40.9961766182249,"lng":-8.11870801214746},{"lat":40.9970797404695,"lng":-8.11845053508526},{"lat":40.9973431876692,"lng":-8.11742599287592},{"lat":40.9982220567422,"lng":-8.11420872592959},{"lat":40.9993196688848,"lng":-8.11114011413737},{"lat":40.999808416042,"lng":-8.11003746744708},{"lat":41.0004220494231,"lng":-8.10908699850054},{"lat":41.0005566552683,"lng":-8.10887850083444},{"lat":41.0012874838325,"lng":-8.10796098267708},{"lat":41.0019844156786,"lng":-8.10708599258955},{"lat":41.0034518629554,"lng":-8.10572675476895},{"lat":41.0034612530065,"lng":-8.10543649327926},{"lat":41.0027226262888,"lng":-8.10471339140289},{"lat":41.0021050570203,"lng":-8.10377442024445},{"lat":41.002421328017,"lng":-8.10352230605804},{"lat":41.0035181012947,"lng":-8.10287417099185},{"lat":41.005635343096,"lng":-8.10216388437281},{"lat":41.0059435224138,"lng":-8.10220785461756},{"lat":41.0075240164285,"lng":-8.10243336200098},{"lat":41.007723490469,"lng":-8.10216725103591},{"lat":41.0086497591588,"lng":-8.10135293766541},{"lat":41.0100190051648,"lng":-8.10072728417854},{"lat":41.0110154579098,"lng":-8.10023591853497},{"lat":41.0121117724118,"lng":-8.09969529116497},{"lat":41.0122385005747,"lng":-8.09963105231304},{"lat":41.0129942964393,"lng":-8.09913770658147},{"lat":41.0134444102194,"lng":-8.09906782566718},{"lat":41.0136869844419,"lng":-8.09776573468771},{"lat":41.0135856768275,"lng":-8.09731509781355},{"lat":41.0134648737401,"lng":-8.09677775316721},{"lat":41.0134288443582,"lng":-8.09661749328303},{"lat":41.0131645116997,"lng":-8.09643467331929},{"lat":41.013089055356,"lng":-8.09614596074953},{"lat":41.0129583802031,"lng":-8.09621242866399},{"lat":41.0125411252906,"lng":-8.09642466346823},{"lat":41.0107927557122,"lng":-8.09451431478765},{"lat":41.0103842135788,"lng":-8.09374071893319},{"lat":41.0098094864104,"lng":-8.09265247594365},{"lat":41.0089703643622,"lng":-8.09108350551256},{"lat":41.0084748928928,"lng":-8.0909776193221},{"lat":41.007394006166,"lng":-8.09074663083651},{"lat":41.0068911353888,"lng":-8.09058432859138},{"lat":41.0065549060563,"lng":-8.09054759024987},{"lat":41.0048080094494,"lng":-8.09035672036933},{"lat":41.0044591899685,"lng":-8.09031860877639},{"lat":41.002566937305,"lng":-8.09045492050893},{"lat":41.000638046016,"lng":-8.09043626006128},{"lat":41.0002106949948,"lng":-8.09056333834712},{"lat":41.0000626515687,"lng":-8.09060870924971},{"lat":40.9988640906419,"lng":-8.09136627376898},{"lat":40.9984470340626,"lng":-8.09162987206438},{"lat":40.9978172078915,"lng":-8.09202794313739},{"lat":40.9971815266004,"lng":-8.09185086855459},{"lat":40.9964534251634,"lng":-8.09156491717984},{"lat":40.9960633131719,"lng":-8.09153938889957},{"lat":40.995353766743,"lng":-8.09149295813277},{"lat":40.9951699839234,"lng":-8.09145328036927},{"lat":40.9943700352217,"lng":-8.09128057817906},{"lat":40.9939918199365,"lng":-8.09110876071006},{"lat":40.9931590508993,"lng":-8.09073045372937},{"lat":40.9925754140955,"lng":-8.09046532757846},{"lat":40.9923089715063,"lng":-8.09049369168367},{"lat":40.991462673357,"lng":-8.0905837826897},{"lat":40.990678478156,"lng":-8.09034741902898},{"lat":40.9896396614578,"lng":-8.08995650629329},{"lat":40.9879721472459,"lng":-8.08932903785977},{"lat":40.9875552776646,"lng":-8.08917217928996},{"lat":40.9867802942961,"lng":-8.09080089984835},{"lat":40.9864583265384,"lng":-8.09128877347892},{"lat":40.9857791512815,"lng":-8.09231789314627},{"lat":40.9846793127756,"lng":-8.09361993419788},{"lat":40.9842598667756,"lng":-8.09413611567404},{"lat":40.9838037870254,"lng":-8.09469687329531},{"lat":40.9833025023355,"lng":-8.09528331671384},{"lat":40.9825464056557,"lng":-8.09620449268936},{"lat":40.9822598360792,"lng":-8.09665521941048},{"lat":40.9821840342122,"lng":-8.09682139271264},{"lat":40.9820013899905,"lng":-8.09722178272888},{"lat":40.981790752057,"lng":-8.09777001965516},{"lat":40.9815235573253,"lng":-8.0982871475794},{"lat":40.9812907786455,"lng":-8.09856554455017},{"lat":40.9810950037346,"lng":-8.09882502408784},{"lat":40.980805785698,"lng":-8.09918459574905},{"lat":40.9805627630485,"lng":-8.09955923948989},{"lat":40.9803119011975,"lng":-8.09990464969264},{"lat":40.9800229443477,"lng":-8.10023212835123},{"lat":40.9798015587992,"lng":-8.10037944921369},{"lat":40.9795984635681,"lng":-8.10054671942435},{"lat":40.9793255978612,"lng":-8.10078686248718},{"lat":40.978963776363,"lng":-8.10106600965468},{"lat":40.9788648699272,"lng":-8.10111696689344},{"lat":40.9786484769933,"lng":-8.10122845336505},{"lat":40.9783088771677,"lng":-8.10141633527743},{"lat":40.9780985999263,"lng":-8.10135346171476},{"lat":40.9778174186706,"lng":-8.10111381415019},{"lat":40.9771079260904,"lng":-8.10068141523435},{"lat":40.9768498867137,"lng":-8.10047503439099},{"lat":40.9766928226193,"lng":-8.10038254936444},{"lat":40.9765344249921,"lng":-8.10034436828248},{"lat":40.9762476154263,"lng":-8.10029709758525},{"lat":40.9760351966,"lng":-8.10029256198802},{"lat":40.9758429491446,"lng":-8.10029122437217},{"lat":40.9755413146625,"lng":-8.1003657421136},{"lat":40.97517620443,"lng":-8.10046274234867},{"lat":40.9749480577727,"lng":-8.10057523967164},{"lat":40.974767722675,"lng":-8.10067654813894},{"lat":40.9746520293084,"lng":-8.10073279994395},{"lat":40.9744699761915,"lng":-8.10080274595757},{"lat":40.9741925903258,"lng":-8.10096552028835},{"lat":40.9739452760108,"lng":-8.1011339815923},{"lat":40.973758200519,"lng":-8.10128139060237},{"lat":40.9735202660998,"lng":-8.10149653818315},{"lat":40.9732645305067,"lng":-8.10182978458308},{"lat":40.9732478455411,"lng":-8.10229019534845},{"lat":40.9732311071121,"lng":-8.10275203019549},{"lat":40.9737235054961,"lng":-8.10408978741095},{"lat":40.9737600911596,"lng":-8.10419955348482},{"lat":40.9736410836191,"lng":-8.10436189023384},{"lat":40.9735862770204,"lng":-8.10448642339374},{"lat":40.9735634398788,"lng":-8.10464492430606},{"lat":40.9735316900071,"lng":-8.10480105036577},{"lat":40.9734174768801,"lng":-8.10503098791486},{"lat":40.9733314227335,"lng":-8.10517536955016},{"lat":40.9732496003954,"lng":-8.1053039499049},{"lat":40.9730648655979,"lng":-8.10547902924566},{"lat":40.9729227986002,"lng":-8.10558363218661},{"lat":40.9727348236522,"lng":-8.10575942050104},{"lat":40.9726518156386,"lng":-8.10582704778344},{"lat":40.9724436633581,"lng":-8.10598680401804},{"lat":40.9723530899481,"lng":-8.10604493521217},{"lat":40.9722801678995,"lng":-8.10612408900178},{"lat":40.9721816809034,"lng":-8.10623152179709},{"lat":40.9720829190975,"lng":-8.10630379368259},{"lat":40.9719267928462,"lng":-8.10636016256988},{"lat":40.9718291841289,"lng":-8.10636495184537},{"lat":40.9716848611731,"lng":-8.10647193032662},{"lat":40.9715781822854,"lng":-8.10657545266083},{"lat":40.971409377984,"lng":-8.10673696798268},{"lat":40.9712957707514,"lng":-8.10687256475347},{"lat":40.9711930654684,"lng":-8.10705258969279},{"lat":40.9710929763976,"lng":-8.10725150058247},{"lat":40.9710068408253,"lng":-8.10743508327592},{"lat":40.9708622851192,"lng":-8.10772810411265},{"lat":40.9707579544445,"lng":-8.10788935259363},{"lat":40.9706347320922,"lng":-8.10813116132316},{"lat":40.9705807348163,"lng":-8.10830308563804},{"lat":40.9705557260105,"lng":-8.10842427574441},{"lat":40.9705308189045,"lng":-8.10860082337149},{"lat":40.9704691717464,"lng":-8.10876562164488},{"lat":40.970444835826,"lng":-8.1091141958593},{"lat":40.9704437272999,"lng":-8.10944945793399},{"lat":40.9704504467203,"lng":-8.10959469467484},{"lat":40.9677049573569,"lng":-8.11293446251264},{"lat":40.9655038541219,"lng":-8.11561172227331},{"lat":40.9654617138063,"lng":-8.11592127162859},{"lat":40.965375656775,"lng":-8.11611492492649},{"lat":40.9652836625075,"lng":-8.11635728121462},{"lat":40.9651961591556,"lng":-8.11653608336527},{"lat":40.9651112657194,"lng":-8.11670668763109},{"lat":40.964938763287,"lng":-8.11690712769471},{"lat":40.9648129822118,"lng":-8.11699743771175},{"lat":40.9646379431908,"lng":-8.1170718398653},{"lat":40.9645217879739,"lng":-8.11712947648446},{"lat":40.9642167309471,"lng":-8.11725569422253},{"lat":40.9640996800799,"lng":-8.11726049203176},{"lat":40.9639603724098,"lng":-8.11726620210489},{"lat":40.9636981554425,"lng":-8.11727041180904},{"lat":40.963485381033,"lng":-8.11728483517514},{"lat":40.9632445080632,"lng":-8.11729046635056},{"lat":40.9630118309614,"lng":-8.11731439155744},{"lat":40.9628067116176,"lng":-8.11735007376914},{"lat":40.9627076669903,"lng":-8.11738822160035},{"lat":40.96248373331,"lng":-8.11749102009902},{"lat":40.9623602884054,"lng":-8.11756101341735},{"lat":40.9621472545928,"lng":-8.11767615630634},{"lat":40.961908916516,"lng":-8.1177977283585},{"lat":40.9616818350928,"lng":-8.11792629948233},{"lat":40.9614437680527,"lng":-8.11804311110554},{"lat":40.9613174451939,"lng":-8.11815705054288},{"lat":40.9612283017469,"lng":-8.11819449205122},{"lat":40.9610363444635,"lng":-8.1183510850611},{"lat":40.9608019737632,"lng":-8.11854190162726},{"lat":40.9606456030321,"lng":-8.11867455416101},{"lat":40.9604190438282,"lng":-8.11885354066555},{"lat":40.9602575117412,"lng":-8.11895251739744},{"lat":40.9600552788868,"lng":-8.1190708618043},{"lat":40.9598109997585,"lng":-8.11922235511763},{"lat":40.95951899359,"lng":-8.11935769906107},{"lat":40.9591000354275,"lng":-8.11957751900553},{"lat":40.958890149694,"lng":-8.11970358660366},{"lat":40.9587966874245,"lng":-8.11977534651602},{"lat":40.9587275431199,"lng":-8.11988558562782},{"lat":40.9586256168234,"lng":-8.11995390210653},{"lat":40.958407,"lng":-8.12005870265081},{"lat":40.9582370892663,"lng":-8.12016943562137},{"lat":40.958088252895,"lng":-8.12024666647823},{"lat":40.9579930766818,"lng":-8.12027792315593},{"lat":40.9579022276491,"lng":-8.1203699911112},{"lat":40.9578217347926,"lng":-8.12047797344462},{"lat":40.9577336788726,"lng":-8.12057847733732},{"lat":40.9576008708849,"lng":-8.12067518245011},{"lat":40.9574922816055,"lng":-8.12073589112058},{"lat":40.9574080987891,"lng":-8.12081489429005},{"lat":40.957348497165,"lng":-8.12091289749601},{"lat":40.9572801609996,"lng":-8.12102907303554},{"lat":40.957021557772,"lng":-8.12116000418124},{"lat":40.9567945614137,"lng":-8.12126967320034},{"lat":40.956692723887,"lng":-8.12132503510718},{"lat":40.9564701369989,"lng":-8.12139954388444},{"lat":40.9562495324468,"lng":-8.1214607439447},{"lat":40.9559984910161,"lng":-8.12153953034705},{"lat":40.9557891409766,"lng":-8.12165120896653},{"lat":40.9555730383441,"lng":-8.12177358268311},{"lat":40.955378905493,"lng":-8.1218429729162},{"lat":40.955113815555,"lng":-8.12192211078255},{"lat":40.9550252148831,"lng":-8.12195241471352},{"lat":40.9548462101093,"lng":-8.12206242128843},{"lat":40.9547396049215,"lng":-8.12215401349138},{"lat":40.9546690161182,"lng":-8.1222300364079},{"lat":40.9545252285904,"lng":-8.12247710434595},{"lat":40.9543715432616,"lng":-8.12279222696848},{"lat":40.9542708866141,"lng":-8.12297242085277},{"lat":40.9541954363905,"lng":-8.12312980576616},{"lat":40.9540978376379,"lng":-8.12328326931131},{"lat":40.9539510733284,"lng":-8.12345170625506},{"lat":40.95387391229,"lng":-8.12353093353352},{"lat":40.9537763087834,"lng":-8.12363297398859},{"lat":40.9536952757187,"lng":-8.12375804905977},{"lat":40.9536063268905,"lng":-8.12397128857082},{"lat":40.9536215557412,"lng":-8.12424537845311},{"lat":40.9537015216855,"lng":-8.12435844228277},{"lat":40.9537375560834,"lng":-8.12457816699743},{"lat":40.9537010141969,"lng":-8.12490241796131},{"lat":40.9535935193061,"lng":-8.1252605273313},{"lat":40.9533767983513,"lng":-8.12566602630682},{"lat":40.9531416141522,"lng":-8.12605098292795},{"lat":40.9529302108207,"lng":-8.12642442952995},{"lat":40.9528307066374,"lng":-8.12660020333253},{"lat":40.9525259197444,"lng":-8.12713302642613},{"lat":40.9520676087491,"lng":-8.12787310315039},{"lat":40.9519655000817,"lng":-8.12806052933151},{"lat":40.9519168834795,"lng":-8.12824521811305},{"lat":40.9519163463333,"lng":-8.12849689073472},{"lat":40.9519621851202,"lng":-8.12873585070809},{"lat":40.9519233809215,"lng":-8.12902517361664},{"lat":40.9518485610927,"lng":-8.12943623387811},{"lat":40.9518797244426,"lng":-8.12994764627183},{"lat":40.9518254281703,"lng":-8.13012936800069},{"lat":40.9516183251195,"lng":-8.13041881034498},{"lat":40.9513300094842,"lng":-8.13078453433185},{"lat":40.9512555037493,"lng":-8.13087904296081},{"lat":40.9511689946722,"lng":-8.1309887771325},{"lat":40.9510903593172,"lng":-8.13105170276604},{"lat":40.9508546436039,"lng":-8.13124032627877},{"lat":40.9506087263647,"lng":-8.13151883639371},{"lat":40.9504506359783,"lng":-8.1315727304949}]},{"zone_id":"arc17","points":[{"lat":40.9664287540205,"lng":-8.24693176449445},{"lat":40.9636373587151,"lng":-8.24089909780235},{"lat":40.9636031088317,"lng":-8.24055431538824},{"lat":40.9636481437697,"lng":-8.24041525055072},{"lat":40.963747079082,"lng":-8.24041211525196},{"lat":40.9659263612303,"lng":-8.23337934573623},{"lat":40.9672879152292,"lng":-8.22923239232421},{"lat":40.9685015646943,"lng":-8.2254009914919},{"lat":40.9699337148216,"lng":-8.22202391977954},{"lat":40.9713654736253,"lng":-8.2190396248295},{"lat":40.972916166266,"lng":-8.21621252283094},{"lat":40.97452613221,"lng":-8.21369971396993},{"lat":40.9766720497955,"lng":-8.21126600586502},{"lat":40.9789966535392,"lng":-8.20875375074108},{"lat":40.9814399255162,"lng":-8.20687022335596},{"lat":40.9846531785058,"lng":-8.20496642152924},{"lat":40.98622591194,"lng":-8.20449651493161},{"lat":40.9879414759115,"lng":-8.20421538950171},{"lat":40.9900857697032,"lng":-8.20412336486405},{"lat":40.9948095843893,"lng":-8.20490247231036},{"lat":40.9949036052346,"lng":-8.20469949983692},{"lat":40.9952329199288,"lng":-8.20452302132724},{"lat":40.9953019600374,"lng":-8.20442562028701},{"lat":40.9955043808699,"lng":-8.20414004531481},{"lat":40.9957464234817,"lng":-8.20371114965358},{"lat":40.9958107872888,"lng":-8.20332867154357},{"lat":40.9958528143077,"lng":-8.20305962547297},{"lat":40.9960924052178,"lng":-8.20265794345993},{"lat":40.9962409410805,"lng":-8.20240129920226},{"lat":40.9963860352504,"lng":-8.20208778950813},{"lat":40.9963808426617,"lng":-8.20189726901422},{"lat":40.9960610770695,"lng":-8.20087469259184},{"lat":40.9959523388449,"lng":-8.20052696816901},{"lat":40.9960255004149,"lng":-8.20000494535759},{"lat":40.996007135908,"lng":-8.1989342266423},{"lat":40.9961704772786,"lng":-8.19831971522948},{"lat":40.996357811825,"lng":-8.19801389859841},{"lat":40.9966384314679,"lng":-8.19755579180796},{"lat":40.9967725471832,"lng":-8.19733684775716},{"lat":40.9969723800821,"lng":-8.1971556812154},{"lat":40.9970612941485,"lng":-8.1970750721604},{"lat":40.9978062453171,"lng":-8.19626655730645},{"lat":40.9977328065472,"lng":-8.1960197033248},{"lat":40.9976725161052,"lng":-8.19581704731819},{"lat":40.9976868388169,"lng":-8.19547857346888},{"lat":40.9977670358062,"lng":-8.19521361671377},{"lat":40.9978739264576,"lng":-8.19504589967354},{"lat":40.9980131607753,"lng":-8.19483923571111},{"lat":40.9980946394975,"lng":-8.19469229424642},{"lat":40.9981926333306,"lng":-8.19448808175721},{"lat":40.9982191928601,"lng":-8.19432824893539},{"lat":40.9981679677347,"lng":-8.19414908882515},{"lat":40.998070519051,"lng":-8.19402063902546},{"lat":40.9978504155044,"lng":-8.19374826069246},{"lat":40.9976938349198,"lng":-8.1935644870038},{"lat":40.9975534767309,"lng":-8.19335672087108},{"lat":40.9974432141322,"lng":-8.19311309326898},{"lat":40.9973854330372,"lng":-8.192891499052},{"lat":40.9973547249083,"lng":-8.19273078516409},{"lat":40.9972168741046,"lng":-8.19203669041725},{"lat":40.9970600190768,"lng":-8.19185067125673},{"lat":40.9965576159574,"lng":-8.19124680880917},{"lat":40.9965033720119,"lng":-8.19114910007421},{"lat":40.9962329048741,"lng":-8.19111713496086},{"lat":40.9886021512386,"lng":-8.19033639911109},{"lat":40.9850287023523,"lng":-8.18976738345174},{"lat":40.9821701776528,"lng":-8.18882176716919},{"lat":40.9784542151116,"lng":-8.18730965328591},{"lat":40.9729989310967,"lng":-8.18480430980506},{"lat":40.9680208602347,"lng":-8.18296346047996},{"lat":40.9625891758814,"lng":-8.18201648177376},{"lat":40.9564425715199,"lng":-8.18144626278316},{"lat":40.9506401702424,"lng":-8.18164206993797},{"lat":40.9482012440066,"lng":-8.18164028413297},{"lat":40.9482787171396,"lng":-8.18722978452341},{"lat":40.9483307081913,"lng":-8.19099182816484},{"lat":40.948043499544,"lng":-8.19744801418193},{"lat":40.9474367638678,"lng":-8.20299722315581},{"lat":40.9467661788154,"lng":-8.20852682293083},{"lat":40.9448534415178,"lng":-8.21820240424942},{"lat":40.943117749188,"lng":-8.22693246962648},{"lat":40.9428447245263,"lng":-8.22830536596784},{"lat":40.9424132996048,"lng":-8.23047458262167},{"lat":40.9415804723515,"lng":-8.23448925232643},{"lat":40.943512116779,"lng":-8.23597066456277},{"lat":40.9476786456657,"lng":-8.23911939244825},{"lat":40.9510713667694,"lng":-8.24163875756907},{"lat":40.9539285725511,"lng":-8.24352897185942},{"lat":40.9569376132215,"lng":-8.24501469024395},{"lat":40.9597361247228,"lng":-8.24596226689662},{"lat":40.9628327482784,"lng":-8.24659613782358},{"lat":40.9664287540205,"lng":-8.24693176449445}]},{"zone_id":"arc4","points":[{"lat":40.8830212592757,"lng":-8.22859757067294},{"lat":40.8848838999189,"lng":-8.22930354212419},{"lat":40.8927061157855,"lng":-8.23167539303297},{"lat":40.8991376242726,"lng":-8.2331921224664},{"lat":40.9093377647481,"lng":-8.23570332229729},{"lat":40.9133870557816,"lng":-8.23680880518195},{"lat":40.9150968464108,"lng":-8.23656107525625},{"lat":40.9166038705799,"lng":-8.23634271333629},{"lat":40.9186305389968,"lng":-8.23461837867762},{"lat":40.9208966464893,"lng":-8.23148085160518},{"lat":40.9231633138044,"lng":-8.2275578357843},{"lat":40.9261429788262,"lng":-8.22567733256258},{"lat":40.9275748409756,"lng":-8.22269508592275},{"lat":40.9275568405932,"lng":-8.21102693840875},{"lat":40.9274631286673,"lng":-8.20863943640618},{"lat":40.9270836252801,"lng":-8.20600032302676},{"lat":40.9230363971344,"lng":-8.1885078664144},{"lat":40.9218705797431,"lng":-8.18347163171172},{"lat":40.9178783771236,"lng":-8.18257332070475},{"lat":40.9107076564438,"lng":-8.18159159637543},{"lat":40.8978652205561,"lng":-8.18255840961815},{"lat":40.8923835934994,"lng":-8.18663586740959},{"lat":40.882849241645,"lng":-8.19510401165546},{"lat":40.8805140916586,"lng":-8.19731801286224},{"lat":40.882294341839,"lng":-8.19801981086673},{"lat":40.8840088608739,"lng":-8.19960366786255},{"lat":40.8857231346088,"lng":-8.20156432609453},{"lat":40.8870368793922,"lng":-8.20390138812586},{"lat":40.8880069159111,"lng":-8.20714232253329},{"lat":40.8885192645938,"lng":-8.2106088604576},{"lat":40.8886887418643,"lng":-8.2136229501045},{"lat":40.8884577669242,"lng":-8.21678724560897},{"lat":40.8878263842263,"lng":-8.2200263419176},{"lat":40.8862223081809,"lng":-8.22394215699004},{"lat":40.8843900542933,"lng":-8.2270286909484},{"lat":40.8830212592757,"lng":-8.22859757067294}]},{"zone_id":"arc12","points":[{"lat":40.8621546996592,"lng":-8.17221558045516},{"lat":40.8616760882929,"lng":-8.17307573907446},{"lat":40.8615537759349,"lng":-8.17322518604629},{"lat":40.8612008742755,"lng":-8.17365637341064},{"lat":40.8607415888489,"lng":-8.17448833763907},{"lat":40.8600676189055,"lng":-8.17535240775945},{"lat":40.8598434722522,"lng":-8.17580798087416},{"lat":40.8595288871763,"lng":-8.17644735670837},{"lat":40.8589294766118,"lng":-8.17681153044409},{"lat":40.8577111560592,"lng":-8.17784072712973},{"lat":40.8574232956335,"lng":-8.17813289358285},{"lat":40.8561715494928,"lng":-8.17956750997064},{"lat":40.8547468185077,"lng":-8.18100286231336},{"lat":40.8538352429865,"lng":-8.18192119176025},{"lat":40.8516961405566,"lng":-8.18429532061126},{"lat":40.8513757087126,"lng":-8.18464378283465},{"lat":40.8493803264197,"lng":-8.1873301581452},{"lat":40.8482109770551,"lng":-8.18877292508884},{"lat":40.8461229654848,"lng":-8.19051293446127},{"lat":40.846209757395,"lng":-8.19129830811348},{"lat":40.8464128927826,"lng":-8.19313666264207},{"lat":40.8466687748158,"lng":-8.19597441965792},{"lat":40.8468337035666,"lng":-8.19746870347401},{"lat":40.8470001235908,"lng":-8.19884998582485},{"lat":40.8472540335571,"lng":-8.20026758850569},{"lat":40.8472775950451,"lng":-8.20040841621995},{"lat":40.847439828512,"lng":-8.2014081369222},{"lat":40.8475517679109,"lng":-8.20213365182515},{"lat":40.8476892558568,"lng":-8.20302479884024},{"lat":40.8479265842236,"lng":-8.20465722918978},{"lat":40.8480812122587,"lng":-8.20533214347405},{"lat":40.8481285481728,"lng":-8.20543721775958},{"lat":40.8483766384449,"lng":-8.20598792412502},{"lat":40.8485338624817,"lng":-8.20633693240304},{"lat":40.8489982099979,"lng":-8.20720063772139},{"lat":40.8493199857724,"lng":-8.20782856035649},{"lat":40.8497370175873,"lng":-8.20820567311774},{"lat":40.8503348691677,"lng":-8.20843452656609},{"lat":40.8506568915721,"lng":-8.20868618856742},{"lat":40.8512643589457,"lng":-8.20961260409795},{"lat":40.8528842112802,"lng":-8.21223539571382},{"lat":40.8535989504669,"lng":-8.21333491668405},{"lat":40.8544093476674,"lng":-8.21426178190099},{"lat":40.8547788464729,"lng":-8.21459193908141},{"lat":40.8549931306432,"lng":-8.21512565233207},{"lat":40.8550955645968,"lng":-8.21539823724441},{"lat":40.8553900052764,"lng":-8.21618178160852},{"lat":40.8560234730964,"lng":-8.21781147689856},{"lat":40.8570683849807,"lng":-8.220499874353},{"lat":40.8575800365906,"lng":-8.22188189719163},{"lat":40.857889809236,"lng":-8.22232173924477},{"lat":40.8586878236736,"lng":-8.22332727723383},{"lat":40.8596054369729,"lng":-8.2244115175936},{"lat":40.8604153039682,"lng":-8.22543289774922},{"lat":40.8618151828477,"lng":-8.22728872548159},{"lat":40.8624790965309,"lng":-8.22814694765281},{"lat":40.8630064797075,"lng":-8.22882869894138},{"lat":40.8641502903526,"lng":-8.23016440874863},{"lat":40.8644166047168,"lng":-8.23043203868499},{"lat":40.86453375579,"lng":-8.2306422752119},{"lat":40.8649560582558,"lng":-8.2310643139813},{"lat":40.8690616765454,"lng":-8.23346169672948},{"lat":40.871003856738,"lng":-8.23401520654544},{"lat":40.8728906891634,"lng":-8.23416872994026},{"lat":40.8748922120684,"lng":-8.23394577859882},{"lat":40.8771228500094,"lng":-8.23327115579432},{"lat":40.8793540660289,"lng":-8.2319184583611},{"lat":40.8821578952718,"lng":-8.22958709163284},{"lat":40.8830212592757,"lng":-8.22859757067294},{"lat":40.8843900542933,"lng":-8.2270286909484},{"lat":40.8862223081809,"lng":-8.22394215699004},{"lat":40.8878263842263,"lng":-8.2200263419176},{"lat":40.8884577669242,"lng":-8.21678724560897},{"lat":40.8886887418643,"lng":-8.2136229501045},{"lat":40.8885192645938,"lng":-8.2106088604576},{"lat":40.8880069159111,"lng":-8.20714232253329},{"lat":40.8870368793922,"lng":-8.20390138812586},{"lat":40.8857231346088,"lng":-8.20156432609453},{"lat":40.8840088608739,"lng":-8.19960366786255},{"lat":40.882294341839,"lng":-8.19801981086673},{"lat":40.8805140916586,"lng":-8.19731801286224},{"lat":40.8791966089031,"lng":-8.19679866933537},{"lat":40.8762220131489,"lng":-8.19503602378351},{"lat":40.8744115772595,"lng":-8.19304559536833},{"lat":40.8729032200718,"lng":-8.19065779956533},{"lat":40.8621546996592,"lng":-8.17221558045516}]},{"zone_id":"vcb7","points":[{"lat":40.7859010975452,"lng":-8.31382883640988},{"lat":40.7863207079039,"lng":-8.31533752468201},{"lat":40.7865767452922,"lng":-8.31851333685501},{"lat":40.7862810249604,"lng":-8.32100361373383},{"lat":40.7861831626919,"lng":-8.32181644063767},{"lat":40.7861343295608,"lng":-8.32198383666685},{"lat":40.7860680333373,"lng":-8.32211190998374},{"lat":40.7859430697521,"lng":-8.32237843664677},{"lat":40.7862918376047,"lng":-8.32275115587014},{"lat":40.7861553447337,"lng":-8.32301735459495},{"lat":40.7859077085675,"lng":-8.32352862150235},{"lat":40.7858210772268,"lng":-8.32374186579216},{"lat":40.7856373836633,"lng":-8.32422228026727},{"lat":40.7855482267142,"lng":-8.32430962881258},{"lat":40.7853454711972,"lng":-8.32450827065028},{"lat":40.7851465784085,"lng":-8.3247598068087},{"lat":40.7849575537193,"lng":-8.32499886092026},{"lat":40.7847081489603,"lng":-8.32514155293212},{"lat":40.7844497420768,"lng":-8.32534568193642},{"lat":40.7842515933617,"lng":-8.32543583670247},{"lat":40.7841292850479,"lng":-8.32540621335761},{"lat":40.7840446639085,"lng":-8.32545572330473},{"lat":40.7829079817407,"lng":-8.32625282004003},{"lat":40.7826148835923,"lng":-8.32644529395773},{"lat":40.7820964939513,"lng":-8.32685262733425},{"lat":40.7816962059062,"lng":-8.32706321938458},{"lat":40.7814278961738,"lng":-8.3272043759458},{"lat":40.7809091327666,"lng":-8.32752405956833},{"lat":40.780437278033,"lng":-8.3279035920356},{"lat":40.7796626992016,"lng":-8.32852660428587},{"lat":40.778625543926,"lng":-8.32923534709671},{"lat":40.7782974193237,"lng":-8.33121802727191},{"lat":40.7780812183198,"lng":-8.33252429032325},{"lat":40.7780367090468,"lng":-8.33279319839265},{"lat":40.7779066292274,"lng":-8.33357906734758},{"lat":40.7773856544844,"lng":-8.33487957290169},{"lat":40.776477663003,"lng":-8.33708261852667},{"lat":40.7765907506782,"lng":-8.33723768366833},{"lat":40.7766485768267,"lng":-8.33737317557437},{"lat":40.7765514727407,"lng":-8.33786393013226},{"lat":40.776509703706,"lng":-8.33802748459548},{"lat":40.7764347909623,"lng":-8.33816911243626},{"lat":40.776400207621,"lng":-8.33837612309351},{"lat":40.7764590747669,"lng":-8.33850528728158},{"lat":40.7766215723214,"lng":-8.33858923493777},{"lat":40.7767641134521,"lng":-8.33863499951101},{"lat":40.7767854382847,"lng":-8.33877037922819},{"lat":40.7768616413288,"lng":-8.33883390732867},{"lat":40.7769311696854,"lng":-8.3389192396342},{"lat":40.7770006707169,"lng":-8.33901985073051},{"lat":40.7771148995268,"lng":-8.33915770246485},{"lat":40.7772366405083,"lng":-8.33929130510779},{"lat":40.7773452161011,"lng":-8.33932210595409},{"lat":40.777433028582,"lng":-8.33938785432387},{"lat":40.7774538593529,"lng":-8.339834465342},{"lat":40.7788302252689,"lng":-8.34237120946892},{"lat":40.7798961555308,"lng":-8.34433594082195},{"lat":40.7803192589428,"lng":-8.3451158432395},{"lat":40.7804850857928,"lng":-8.34542151563935},{"lat":40.7812686200714,"lng":-8.34674421319283},{"lat":40.7813737543209,"lng":-8.34691089016014},{"lat":40.7818141054956,"lng":-8.34760902079552},{"lat":40.7830083098217,"lng":-8.34957475975322},{"lat":40.783696552946,"lng":-8.35070771021752},{"lat":40.7846966149051,"lng":-8.35235403553251},{"lat":40.7865542388759,"lng":-8.35541233480386},{"lat":40.7872442101254,"lng":-8.35654834724827},{"lat":40.7878488047787,"lng":-8.35754382541455},{"lat":40.7879204104359,"lng":-8.3576617278207},{"lat":40.7881866202589,"lng":-8.35793419329038},{"lat":40.7883043381268,"lng":-8.35804618823722},{"lat":40.7884605255597,"lng":-8.35821194261985},{"lat":40.7885598674979,"lng":-8.35831893351702},{"lat":40.7887585839562,"lng":-8.35853175410347},{"lat":40.7889387948843,"lng":-8.35872520360364},{"lat":40.7891056773987,"lng":-8.35890338914246},{"lat":40.7892841616921,"lng":-8.35909533129743},{"lat":40.7894587484366,"lng":-8.35928537387114},{"lat":40.7896517343719,"lng":-8.35948751160235},{"lat":40.7898236541526,"lng":-8.35967110184324},{"lat":40.7900052624745,"lng":-8.3598705230565},{"lat":40.7901038507814,"lng":-8.35997669128493},{"lat":40.7903097416699,"lng":-8.35973169674035},{"lat":40.7947435942992,"lng":-8.35445522998137},{"lat":40.7977388662633,"lng":-8.35098974635447},{"lat":40.8001254198409,"lng":-8.3488655516785},{"lat":40.8027962105267,"lng":-8.34762003334317},{"lat":40.8063235422472,"lng":-8.34700427424484},{"lat":40.8098490095069,"lng":-8.34739188449814},{"lat":40.8151833332578,"lng":-8.34878888721066},{"lat":40.8198335608492,"lng":-8.35047541690901},{"lat":40.8217514792429,"lng":-8.34406971636769},{"lat":40.8262098678179,"lng":-8.32884035698592},{"lat":40.8274391557034,"lng":-8.32520930101246},{"lat":40.8277501620196,"lng":-8.32342133015547},{"lat":40.8276938156004,"lng":-8.32146080014886},{"lat":40.8272798013537,"lng":-8.319656100513},{"lat":40.8262116064552,"lng":-8.31753522132472},{"lat":40.8237986156912,"lng":-8.31485412542508},{"lat":40.8211155772944,"lng":-8.31198651378968},{"lat":40.8199549235365,"lng":-8.30986130648461},{"lat":40.8175167634881,"lng":-8.30964357975392},{"lat":40.8144067910442,"lng":-8.30879821092909},{"lat":40.8136614977296,"lng":-8.30866691023934},{"lat":40.8027548580379,"lng":-8.30674579736239},{"lat":40.8009966047172,"lng":-8.30674121539012},{"lat":40.8006820373177,"lng":-8.30674039567672},{"lat":40.7993999215632,"lng":-8.30693771135915},{"lat":40.7977144828872,"lng":-8.30752040944507},{"lat":40.7859010975452,"lng":-8.31382883640988}]},{"zone_id":"arc13","points":[{"lat":40.8607596876563,"lng":-8.24215090949117},{"lat":40.8605011029719,"lng":-8.24274116163432},{"lat":40.860440791134,"lng":-8.24287882935373},{"lat":40.8594394765109,"lng":-8.24516291620276},{"lat":40.8590906537061,"lng":-8.24595857129843},{"lat":40.8590411144755,"lng":-8.2461095004452},{"lat":40.8574160753025,"lng":-8.25105998220748},{"lat":40.8559081952014,"lng":-8.25495668590518},{"lat":40.8558421984819,"lng":-8.25508778987782},{"lat":40.8536692102145,"lng":-8.25949623590181},{"lat":40.852538510113,"lng":-8.26192965968732},{"lat":40.8522483146702,"lng":-8.26255417722975},{"lat":40.8511361921303,"lng":-8.26494737411799},{"lat":40.8498891280688,"lng":-8.26801139405421},{"lat":40.847526813802,"lng":-8.27322866099985},{"lat":40.8449121207561,"lng":-8.2797637965397},{"lat":40.8509765716939,"lng":-8.2830290559041},{"lat":40.8527408079256,"lng":-8.28379793215339},{"lat":40.8545953937047,"lng":-8.2846062306785},{"lat":40.8555650883173,"lng":-8.28502887931774},{"lat":40.8578631571456,"lng":-8.28552520272861},{"lat":40.8579918476139,"lng":-8.28563710825731},{"lat":40.8596304347532,"lng":-8.28706202292794},{"lat":40.8600884450516,"lng":-8.28738196115689},{"lat":40.8616015081637,"lng":-8.28843893133169},{"lat":40.861691015444,"lng":-8.28850512135764},{"lat":40.8625786485094,"lng":-8.28916153123169},{"lat":40.8640268179031,"lng":-8.29023250489627},{"lat":40.866048808689,"lng":-8.29140841439602},{"lat":40.8688698018463,"lng":-8.29474451271071},{"lat":40.8698327400608,"lng":-8.29591938092531},{"lat":40.8715952628937,"lng":-8.29796801460199},{"lat":40.8740811535778,"lng":-8.30341083597928},{"lat":40.8750332632183,"lng":-8.30546085855889},{"lat":40.8762268832378,"lng":-8.30862415358441},{"lat":40.8769774959995,"lng":-8.30958233572237},{"lat":40.877608359698,"lng":-8.31038767894404},{"lat":40.877700713312,"lng":-8.31049798000351},{"lat":40.877850444097,"lng":-8.3106768088948},{"lat":40.878572006523,"lng":-8.3115815114907},{"lat":40.8793542699609,"lng":-8.31244054993308},{"lat":40.8795592605433,"lng":-8.31267883230919},{"lat":40.8801584026436,"lng":-8.31337528318447},{"lat":40.8801998843561,"lng":-8.3134532772755},{"lat":40.8813785282769,"lng":-8.30233583931086},{"lat":40.8822928303437,"lng":-8.2927652312679},{"lat":40.8832132951963,"lng":-8.28269081591442},{"lat":40.8843171876877,"lng":-8.27431618283853},{"lat":40.8857617306792,"lng":-8.26941009786677},{"lat":40.8607596876563,"lng":-8.24215090949117}]},{"zone_id":"oaz5","points":[{"lat":40.7880283011191,"lng":-8.4061526876876},{"lat":40.787905548595,"lng":-8.40622790721127},{"lat":40.7877878921167,"lng":-8.4063000017804},{"lat":40.7876590526131,"lng":-8.40633052890057},{"lat":40.7875285881352,"lng":-8.40637494416682},{"lat":40.7874161418927,"lng":-8.40647171909207},{"lat":40.7872942936238,"lng":-8.40652929759478},{"lat":40.7871627292188,"lng":-8.40655143544655},{"lat":40.7870313483569,"lng":-8.40654551657081},{"lat":40.7869146637812,"lng":-8.40662994454989},{"lat":40.7868052579487,"lng":-8.4067393944535},{"lat":40.7867018524687,"lng":-8.40685244776794},{"lat":40.7865797257457,"lng":-8.40693962593698},{"lat":40.7864642830761,"lng":-8.40703791336591},{"lat":40.7863413560254,"lng":-8.4071238033348},{"lat":40.7862291862056,"lng":-8.40720818638995},{"lat":40.7860955546307,"lng":-8.4072229023297},{"lat":40.7859604177079,"lng":-8.40725152320484},{"lat":40.7858244415953,"lng":-8.40724599646127},{"lat":40.7856971794325,"lng":-8.4071840435962},{"lat":40.7856000028536,"lng":-8.4071066720567},{"lat":40.7854580695866,"lng":-8.40707914335446},{"lat":40.7852968151766,"lng":-8.40715003927708},{"lat":40.7851535730828,"lng":-8.40727676196983},{"lat":40.7850487632014,"lng":-8.40735911569129},{"lat":40.7849058325951,"lng":-8.40736872341681},{"lat":40.7847721499413,"lng":-8.40735527547948},{"lat":40.7846416077243,"lng":-8.40737727056134},{"lat":40.7845056281368,"lng":-8.40747433847871},{"lat":40.7843563521609,"lng":-8.40752011380327},{"lat":40.7842331277323,"lng":-8.40755935825875},{"lat":40.7841399319851,"lng":-8.40763083762248},{"lat":40.784047679204,"lng":-8.40770261447713},{"lat":40.7839403201962,"lng":-8.40781161737022},{"lat":40.7838366631716,"lng":-8.40792812010253},{"lat":40.7837316651925,"lng":-8.40802836881793},{"lat":40.7836065974036,"lng":-8.40807360129774},{"lat":40.7834715857639,"lng":-8.40810017277553},{"lat":40.7833579595528,"lng":-8.40811899444526},{"lat":40.7832452175166,"lng":-8.40815599637969},{"lat":40.7831338952059,"lng":-8.40819729735274},{"lat":40.7830304425767,"lng":-8.40824906299936},{"lat":40.7829216568629,"lng":-8.40828850235592},{"lat":40.7828165743326,"lng":-8.40829720610102},{"lat":40.7827096547813,"lng":-8.40831535242409},{"lat":40.7826029001758,"lng":-8.4083047354035},{"lat":40.7825016010114,"lng":-8.40826342940368},{"lat":40.7823972697803,"lng":-8.4082489734966},{"lat":40.7822905577947,"lng":-8.40827448337884},{"lat":40.7822051542271,"lng":-8.40832424700776},{"lat":40.7820981872334,"lng":-8.4084025885125},{"lat":40.7819963874958,"lng":-8.40845070991329},{"lat":40.7818957932579,"lng":-8.40850195181553},{"lat":40.7818112263228,"lng":-8.40859230272306},{"lat":40.7817181203478,"lng":-8.40864777602569},{"lat":40.7816099433655,"lng":-8.40862907338249},{"lat":40.781502062855,"lng":-8.40860801811085},{"lat":40.7813963878308,"lng":-8.40857481745013},{"lat":40.7812904623255,"lng":-8.40853903094571},{"lat":40.7811846325876,"lng":-8.40850197500428},{"lat":40.7810799789441,"lng":-8.40845923297802},{"lat":40.7809814847538,"lng":-8.40839645330999},{"lat":40.7808824815087,"lng":-8.40833641655371},{"lat":40.7807678159499,"lng":-8.4082533202018},{"lat":40.7806448433571,"lng":-8.40818717882867},{"lat":40.7805382113621,"lng":-8.40814625620291},{"lat":40.7804441171335,"lng":-8.40813549064864},{"lat":40.7803503786381,"lng":-8.40815153493664},{"lat":40.7802251631552,"lng":-8.40815101015415},{"lat":40.7801073672215,"lng":-8.40809441057232},{"lat":40.7799914125142,"lng":-8.4080587073816},{"lat":40.7798627833483,"lng":-8.40806229476605},{"lat":40.7797348549515,"lng":-8.40806807184809},{"lat":40.7796137386557,"lng":-8.4081045232299},{"lat":40.7794999081539,"lng":-8.40813837585422},{"lat":40.7793753404231,"lng":-8.40810182238},{"lat":40.7792492724978,"lng":-8.40809274329055},{"lat":40.7791200944572,"lng":-8.40806222474906},{"lat":40.7789955722257,"lng":-8.40801789838425},{"lat":40.7788703241173,"lng":-8.40797737653429},{"lat":40.7787160005374,"lng":-8.40792030804322},{"lat":40.7786244100587,"lng":-8.40788289385987},{"lat":40.7785318864027,"lng":-8.40785012314105},{"lat":40.7784366130416,"lng":-8.40783533175801},{"lat":40.7783417596794,"lng":-8.40781698005094},{"lat":40.7782480705988,"lng":-8.40779146399329},{"lat":40.7781545552982,"lng":-8.40776557320614},{"lat":40.7780604831433,"lng":-8.40774209559173},{"lat":40.7779656096911,"lng":-8.40772271699714},{"lat":40.7778703040624,"lng":-8.40770595936847},{"lat":40.7777740152695,"lng":-8.40770123556029},{"lat":40.7776792373217,"lng":-8.4076841011159},{"lat":40.7775912082983,"lng":-8.40763340313486},{"lat":40.7775061101909,"lng":-8.40757369316885},{"lat":40.7774208752687,"lng":-8.40751432276882},{"lat":40.7773371462095,"lng":-8.40745143220999},{"lat":40.7772604062377,"lng":-8.40737474766206},{"lat":40.7771873268098,"lng":-8.40729196708717},{"lat":40.7771131533382,"lng":-8.40721103132651},{"lat":40.7770302350863,"lng":-8.40715739751445},{"lat":40.7769536517885,"lng":-8.4072323129209},{"lat":40.7768887305497,"lng":-8.40732627314976},{"lat":40.7767934146648,"lng":-8.40734449561638},{"lat":40.7766955016969,"lng":-8.40733583295572},{"lat":40.7765986883349,"lng":-8.40731310569602},{"lat":40.7765061641852,"lng":-8.40726984741728},{"lat":40.776417208449,"lng":-8.40721455805963},{"lat":40.7763311857061,"lng":-8.40715164163672},{"lat":40.7762473283693,"lng":-8.40708915043258},{"lat":40.7761703474125,"lng":-8.40716095592211},{"lat":40.7761063756132,"lng":-8.40724456950583},{"lat":40.776044701125,"lng":-8.4073311289021},{"lat":40.7759838553291,"lng":-8.4074186984229},{"lat":40.7759235536275,"lng":-8.40750691560731},{"lat":40.7758319107452,"lng":-8.40770801867904},{"lat":40.7757931874363,"lng":-8.40781515043438},{"lat":40.775758293882,"lng":-8.40792453428801},{"lat":40.7757214415879,"lng":-8.40803280713156},{"lat":40.7756673969866,"lng":-8.4081554012881},{"lat":40.7755974245518,"lng":-8.40826616573201},{"lat":40.7755299665943,"lng":-8.40835033647456},{"lat":40.7754288174152,"lng":-8.40840089494551},{"lat":40.7753442714075,"lng":-8.40834809414822},{"lat":40.7752596300316,"lng":-8.40827372130739},{"lat":40.7751826174615,"lng":-8.40818619257715},{"lat":40.7751090225455,"lng":-8.40809361066089},{"lat":40.7750276197952,"lng":-8.40801369151262},{"lat":40.7749298361791,"lng":-8.4080081961095},{"lat":40.7748637763908,"lng":-8.40810428495726},{"lat":40.7747992197236,"lng":-8.40825314154088},{"lat":40.7747695121041,"lng":-8.40841785458472},{"lat":40.7747727553084,"lng":-8.40856782933654},{"lat":40.7748018858639,"lng":-8.40870119250144},{"lat":40.774837539685,"lng":-8.40883182632825},{"lat":40.7748470278677,"lng":-8.40896913237361},{"lat":40.7748254366545,"lng":-8.40910358066247},{"lat":40.7747606207385,"lng":-8.40921271874114},{"lat":40.7746777046957,"lng":-8.40929741217403},{"lat":40.7745770753404,"lng":-8.40933791165262},{"lat":40.7744746362368,"lng":-8.40937077630528},{"lat":40.7743757566006,"lng":-8.40941913979602},{"lat":40.7742805212342,"lng":-8.40947691274856},{"lat":40.7742167623922,"lng":-8.40959083352933},{"lat":40.774127574737,"lng":-8.40978045124329},{"lat":40.7740452014952,"lng":-8.40997467255952},{"lat":40.7739061250373,"lng":-8.41010152382467},{"lat":40.7738172957613,"lng":-8.41016831723256},{"lat":40.7737127372399,"lng":-8.41030502894886},{"lat":40.7736725633256,"lng":-8.41044933536082},{"lat":40.7736943991659,"lng":-8.41059181957263},{"lat":40.7737267742843,"lng":-8.41070696402201},{"lat":40.7737479614708,"lng":-8.4108262397999},{"lat":40.7737378925876,"lng":-8.41094800086835},{"lat":40.7737227768274,"lng":-8.41106916400539},{"lat":40.7737088497388,"lng":-8.41120185267871},{"lat":40.7736726700723,"lng":-8.41133127632701},{"lat":40.7736157954424,"lng":-8.41143775345809},{"lat":40.7735365712854,"lng":-8.41151734926916},{"lat":40.7734441179857,"lng":-8.41159428253688},{"lat":40.7733399591536,"lng":-8.41167080661107},{"lat":40.7732370274757,"lng":-8.41174963944763},{"lat":40.7731025278627,"lng":-8.41192764813762},{"lat":40.7728931142062,"lng":-8.41230952413767},{"lat":40.7728331963187,"lng":-8.41240023296955},{"lat":40.7726448606202,"lng":-8.41267326843097},{"lat":40.7726580343698,"lng":-8.41279747921603},{"lat":40.7728321739206,"lng":-8.41340131043969},{"lat":40.772860766401,"lng":-8.41360557161626},{"lat":40.77282634166,"lng":-8.41380318564992},{"lat":40.772772289811,"lng":-8.41409640923963},{"lat":40.7727235251887,"lng":-8.41421102703905},{"lat":40.7726214675463,"lng":-8.41430628792934},{"lat":40.7725143930653,"lng":-8.41447509299435},{"lat":40.772422450018,"lng":-8.41459528450567},{"lat":40.7724286711466,"lng":-8.41474013560304},{"lat":40.7725027118047,"lng":-8.41485452367442},{"lat":40.7725902335293,"lng":-8.41495312628976},{"lat":40.7726218790547,"lng":-8.41506767560622},{"lat":40.7726197545854,"lng":-8.41519009298757},{"lat":40.7725348264125,"lng":-8.41526026358377},{"lat":40.7724416109289,"lng":-8.41525309156799},{"lat":40.7723497055849,"lng":-8.41523048285153},{"lat":40.7722579935067,"lng":-8.41520653772229},{"lat":40.7721663500471,"lng":-8.41518214758085},{"lat":40.7720746974311,"lng":-8.41515781669295},{"lat":40.771982900169,"lng":-8.41513445180377},{"lat":40.7718905367175,"lng":-8.41511578344576},{"lat":40.7717970760098,"lng":-8.41511314369504},{"lat":40.7717035757621,"lng":-8.4151126808663},{"lat":40.7716098379507,"lng":-8.41515526111198},{"lat":40.7715787010163,"lng":-8.41527000046095},{"lat":40.7715821794217,"lng":-8.41538976270479},{"lat":40.7716199328131,"lng":-8.41550275151597},{"lat":40.7716359828008,"lng":-8.41563981197375},{"lat":40.771642414606,"lng":-8.41576024779583},{"lat":40.771573088076,"lng":-8.41583851355827},{"lat":40.771494020327,"lng":-8.41589896947225},{"lat":40.7714030621601,"lng":-8.41593526239562},{"lat":40.7713132366626,"lng":-8.41597675750801},{"lat":40.771223038218,"lng":-8.41599516076904},{"lat":40.7711349121261,"lng":-8.41602151821934},{"lat":40.7710548783939,"lng":-8.41608403395901},{"lat":40.770963457427,"lng":-8.41610852710431},{"lat":40.770871496265,"lng":-8.41610359923708},{"lat":40.7707909583543,"lng":-8.41604156692382},{"lat":40.7707121429454,"lng":-8.41597590812378},{"lat":40.7706291163535,"lng":-8.41592070821445},{"lat":40.7705348280511,"lng":-8.41587765948792},{"lat":40.7704452281049,"lng":-8.41584180622247},{"lat":40.7703494807013,"lng":-8.41583121284464},{"lat":40.7702585743064,"lng":-8.41586769379496},{"lat":40.770164536587,"lng":-8.41587709417467},{"lat":40.770069610459,"lng":-8.41586869670003},{"lat":40.7699846225742,"lng":-8.41582376783257},{"lat":40.7699618351791,"lng":-8.41569115823185},{"lat":40.7699462154256,"lng":-8.41555751424906},{"lat":40.7698858732665,"lng":-8.41546174931723},{"lat":40.7697986769183,"lng":-8.41542245683454},{"lat":40.7697032003503,"lng":-8.41542113361968},{"lat":40.769615535608,"lng":-8.41544851892309},{"lat":40.7695313394142,"lng":-8.41550725214861},{"lat":40.7694806136933,"lng":-8.4156131197773},{"lat":40.769464281203,"lng":-8.4157297716334},{"lat":40.769451931903,"lng":-8.41584956606223},{"lat":40.7694199003899,"lng":-8.41596748373864},{"lat":40.7693292809216,"lng":-8.4159985665345},{"lat":40.7692345952378,"lng":-8.41597803898592},{"lat":40.7691448296176,"lng":-8.41593777355974},{"lat":40.7690574452991,"lng":-8.41589256739231},{"lat":40.7689687723872,"lng":-8.41585285876775},{"lat":40.7688755619063,"lng":-8.41583996080592},{"lat":40.7687893042296,"lng":-8.41588547773574},{"lat":40.7687074340035,"lng":-8.41594596066269},{"lat":40.7686253139546,"lng":-8.4160058525366},{"lat":40.7685420571645,"lng":-8.41606292050429},{"lat":40.7684551153913,"lng":-8.41610901912483},{"lat":40.7683619941602,"lng":-8.41612032499853},{"lat":40.7682777174029,"lng":-8.41606561784159},{"lat":40.7682218748598,"lng":-8.41596678140889},{"lat":40.7681570677209,"lng":-8.41588376890545},{"lat":40.7680740472683,"lng":-8.41600678148737},{"lat":40.7680475311716,"lng":-8.41626349589988},{"lat":40.7679965627323,"lng":-8.41637330960699},{"lat":40.7680052229605,"lng":-8.41651263275274},{"lat":40.7680101592104,"lng":-8.41673780923231},{"lat":40.7680050447942,"lng":-8.41694861152022},{"lat":40.7679887808052,"lng":-8.41718909518388},{"lat":40.7680036147338,"lng":-8.41730795429077},{"lat":40.7680347918869,"lng":-8.41744500589386},{"lat":40.768091371122,"lng":-8.41768757810144},{"lat":40.7681035782179,"lng":-8.41784823630682},{"lat":40.7681049476544,"lng":-8.41805847428801},{"lat":40.7681036231181,"lng":-8.41826514868659},{"lat":40.7681085115983,"lng":-8.41839936316231},{"lat":40.7681199843499,"lng":-8.41852957880138},{"lat":40.7681364295488,"lng":-8.41865140638583},{"lat":40.7681501582366,"lng":-8.41881562465251},{"lat":40.7681972892591,"lng":-8.41894587597686},{"lat":40.7682590447318,"lng":-8.4190613860813},{"lat":40.7683390174716,"lng":-8.41920220156605},{"lat":40.7683992630027,"lng":-8.41930917781561},{"lat":40.768475578694,"lng":-8.41943576626499},{"lat":40.7685672026701,"lng":-8.41963419588851},{"lat":40.7686414498214,"lng":-8.419795479917},{"lat":40.7687251992377,"lng":-8.41997397822644},{"lat":40.7688054714108,"lng":-8.42017425557312},{"lat":40.7688696031618,"lng":-8.42027520957011},{"lat":40.7689039212256,"lng":-8.42045136299811},{"lat":40.7689853126907,"lng":-8.42063577603156},{"lat":40.7690311788919,"lng":-8.42076685392409},{"lat":40.7690981884026,"lng":-8.42090524828237},{"lat":40.7691599330284,"lng":-8.42102431503124},{"lat":40.7692277208555,"lng":-8.42113937981582},{"lat":40.7692934643512,"lng":-8.42124365877691},{"lat":40.769369319903,"lng":-8.42135532517978},{"lat":40.7695667387042,"lng":-8.42239639881653},{"lat":40.769614413073,"lng":-8.42267777048319},{"lat":40.7698059856086,"lng":-8.42380846665858},{"lat":40.7699341287468,"lng":-8.42445996091184},{"lat":40.7699866275213,"lng":-8.42469716255393},{"lat":40.7699260813514,"lng":-8.42496160750382},{"lat":40.7700522554142,"lng":-8.42489505193416},{"lat":40.7701320402052,"lng":-8.42528700447539},{"lat":40.7702427185878,"lng":-8.42546355566777},{"lat":40.7704106234733,"lng":-8.42570397395355},{"lat":40.7706715338341,"lng":-8.42611279241747},{"lat":40.7706022585695,"lng":-8.42620711020144},{"lat":40.770798808821,"lng":-8.4265640096232},{"lat":40.7716376963652,"lng":-8.42798987607469},{"lat":40.7716073279944,"lng":-8.42822202343146},{"lat":40.7716988355355,"lng":-8.42829425063888},{"lat":40.7717807397322,"lng":-8.42835747964458},{"lat":40.7718625678259,"lng":-8.4284191153351},{"lat":40.7719414025069,"lng":-8.42847868875768},{"lat":40.7720338138236,"lng":-8.42854992216023},{"lat":40.7721242074455,"lng":-8.42862663930599},{"lat":40.7721981320123,"lng":-8.42870117053917},{"lat":40.7722707460517,"lng":-8.42877790994703},{"lat":40.7723395237772,"lng":-8.42885892156752},{"lat":40.772409167692,"lng":-8.42893994548876},{"lat":40.7724666813733,"lng":-8.42903379716806},{"lat":40.7725056416124,"lng":-8.42914536374417},{"lat":40.7725482137581,"lng":-8.42925407644807},{"lat":40.7726018500395,"lng":-8.42935724404375},{"lat":40.7726821153597,"lng":-8.42942233352462},{"lat":40.7727707323916,"lng":-8.42946301925154},{"lat":40.7728616551001,"lng":-8.42948636116482},{"lat":40.7729614948481,"lng":-8.42949765725686},{"lat":40.7737649291649,"lng":-8.43098913159586},{"lat":40.774020353364,"lng":-8.43148330243653},{"lat":40.7744771466752,"lng":-8.43157006040363},{"lat":40.7747267507463,"lng":-8.43161746779352},{"lat":40.774764243722,"lng":-8.43180423879633},{"lat":40.7750666063358,"lng":-8.43331051893231},{"lat":40.7751461749751,"lng":-8.43322957232652},{"lat":40.7752219426258,"lng":-8.4331468683299},{"lat":40.7752923537573,"lng":-8.43305631256658},{"lat":40.7753603668588,"lng":-8.4329626127},{"lat":40.7754291768792,"lng":-8.43286993138357},{"lat":40.7755001190722,"lng":-8.43278007394962},{"lat":40.7755733822785,"lng":-8.43269349137672},{"lat":40.7756578339213,"lng":-8.43260072929183},{"lat":40.775746813873,"lng":-8.43251568572624},{"lat":40.7758391988649,"lng":-8.43243711305185},{"lat":40.7759359277003,"lng":-8.43236344003336},{"lat":40.7760361530702,"lng":-8.43229603696664},{"lat":40.7761439317724,"lng":-8.43225329699576},{"lat":40.7762349883629,"lng":-8.43226956438304},{"lat":40.7763014842696,"lng":-8.43235688674953},{"lat":40.7763368959002,"lng":-8.43247543706834},{"lat":40.7763650442864,"lng":-8.43259734749313},{"lat":40.7763896556277,"lng":-8.43272055462987},{"lat":40.7764072646284,"lng":-8.43284582114597},{"lat":40.7764227136608,"lng":-8.43297160396373},{"lat":40.7764372570805,"lng":-8.43309756992753},{"lat":40.7764593571623,"lng":-8.43322117712557},{"lat":40.7765336193071,"lng":-8.43332435443165},{"lat":40.7766259463708,"lng":-8.4333357640667},{"lat":40.7767158288064,"lng":-8.43330278153765},{"lat":40.7768011007638,"lng":-8.4332483042124},{"lat":40.7768820223978,"lng":-8.43319105540411},{"lat":40.7769626227484,"lng":-8.43313302792962},{"lat":40.7770443691837,"lng":-8.43307785556589},{"lat":40.7771241998252,"lng":-8.43301833652072},{"lat":40.7772029078836,"lng":-8.43295694301715},{"lat":40.7772937992698,"lng":-8.43296466057145},{"lat":40.7773801802015,"lng":-8.43300524666246},{"lat":40.7774644378185,"lng":-8.43305342716506},{"lat":40.7775506271729,"lng":-8.43309529789161},{"lat":40.7776372901263,"lng":-8.43313545438865},{"lat":40.7777280136899,"lng":-8.43317411535574},{"lat":40.7778207789065,"lng":-8.43319156009627},{"lat":40.7779158408007,"lng":-8.43319672506159},{"lat":40.7780109333466,"lng":-8.43320083707507},{"lat":40.7781059200859,"lng":-8.43320789116421},{"lat":40.7781994829698,"lng":-8.43322989916441},{"lat":40.7782877465181,"lng":-8.4332740294788},{"lat":40.7783583952974,"lng":-8.43335663346933},{"lat":40.7784191997864,"lng":-8.43345515463549},{"lat":40.7784798595524,"lng":-8.43354420560698},{"lat":40.7785451775941,"lng":-8.43362665697729},{"lat":40.7786221803449,"lng":-8.43369012218999},{"lat":40.7787030705786,"lng":-8.43374457125811},{"lat":40.7787889634759,"lng":-8.4337833019293},{"lat":40.7788773603502,"lng":-8.43381108515815},{"lat":40.7789735982507,"lng":-8.43383110788813},{"lat":40.7790823350861,"lng":-8.43381379113831},{"lat":40.7791899476955,"lng":-8.4337838412532},{"lat":40.7792884653524,"lng":-8.43378877280896},{"lat":40.7793859153425,"lng":-8.43384230570293},{"lat":40.7794540452666,"lng":-8.43393655697057},{"lat":40.7795053399627,"lng":-8.43403405887548},{"lat":40.7795513488739,"lng":-8.43413596755798},{"lat":40.7795939217855,"lng":-8.43424048954408},{"lat":40.7796367472572,"lng":-8.43434481378772},{"lat":40.779674671117,"lng":-8.43445228607092},{"lat":40.7797023602659,"lng":-8.43456510936114},{"lat":40.7797255052878,"lng":-8.43467969860975},{"lat":40.7797558162648,"lng":-8.43480361849135},{"lat":40.7798223207621,"lng":-8.43489516888091},{"lat":40.7798554654668,"lng":-8.43502077219623},{"lat":40.7798843805461,"lng":-8.43513668753846},{"lat":40.7799035593776,"lng":-8.43526841038529},{"lat":40.7798926930887,"lng":-8.43539219395088},{"lat":40.7798630088897,"lng":-8.43551189519824},{"lat":40.7798292809153,"lng":-8.43563038388812},{"lat":40.7797873106054,"lng":-8.43574096438779},{"lat":40.7797295941868,"lng":-8.43584087559507},{"lat":40.7796725227569,"lng":-8.43594135107097},{"lat":40.7796107390427,"lng":-8.43603527058227},{"lat":40.7795448069362,"lng":-8.43612840106361},{"lat":40.7794766844905,"lng":-8.43621820091068},{"lat":40.7794105136153,"lng":-8.43630755692332},{"lat":40.7793553916913,"lng":-8.4364079029714},{"lat":40.779301050505,"lng":-8.43650924037767},{"lat":40.7792401398836,"lng":-8.43660381553881},{"lat":40.7791724860701,"lng":-8.43668785465247},{"lat":40.7791060482282,"lng":-8.43677415108609},{"lat":40.7790603407276,"lng":-8.43687889026475},{"lat":40.7790336797041,"lng":-8.43699484790835},{"lat":40.7790109802126,"lng":-8.43711511063432},{"lat":40.7789859125456,"lng":-8.43723493483235},{"lat":40.7789628558873,"lng":-8.43735531896023},{"lat":40.7789219055639,"lng":-8.43746674148315},{"lat":40.7788876794084,"lng":-8.43758113359082},{"lat":40.7788573913411,"lng":-8.43769710607},{"lat":40.7788261819832,"lng":-8.43781886342225},{"lat":40.7788168922782,"lng":-8.43795463528754},{"lat":40.778805727692,"lng":-8.43808900188532},{"lat":40.7787866021504,"lng":-8.4382187558669},{"lat":40.7787738755665,"lng":-8.43834844311134},{"lat":40.7787572868134,"lng":-8.43847873335469},{"lat":40.778740209038,"lng":-8.43861170916884},{"lat":40.7787235861403,"lng":-8.43874525916518},{"lat":40.7787087631024,"lng":-8.43887849868827},{"lat":40.7786951062092,"lng":-8.43901303827746},{"lat":40.77868679968,"lng":-8.43914766396588},{"lat":40.7786914468281,"lng":-8.43928357398046},{"lat":40.7787048793396,"lng":-8.43941632474393},{"lat":40.7787203308819,"lng":-8.43954897348583},{"lat":40.7787318029054,"lng":-8.43968266658693},{"lat":40.7787396614129,"lng":-8.43981691291019},{"lat":40.7787430390902,"lng":-8.43995176293506},{"lat":40.7787573817941,"lng":-8.44008667292772},{"lat":40.7787724732583,"lng":-8.4402203402181},{"lat":40.7788063593134,"lng":-8.44034773636629},{"lat":40.7788673569378,"lng":-8.44055811519386},{"lat":40.7788669843473,"lng":-8.44083472950902},{"lat":40.7790254691466,"lng":-8.44098254483836},{"lat":40.779087315555,"lng":-8.44112988841753},{"lat":40.7790989511989,"lng":-8.44149121171932},{"lat":40.7789647399958,"lng":-8.4420209443135},{"lat":40.779053010871,"lng":-8.44235658590749},{"lat":40.77915338252,"lng":-8.44277244750467},{"lat":40.7791876857125,"lng":-8.44301442131968},{"lat":40.7791598630779,"lng":-8.44316997065375},{"lat":40.7790788928474,"lng":-8.44331952722916},{"lat":40.7788678355655,"lng":-8.44336651791082},{"lat":40.7786602383669,"lng":-8.44341273793318},{"lat":40.7785270702132,"lng":-8.4435309530335},{"lat":40.7784476934968,"lng":-8.44375340261621},{"lat":40.7784627817759,"lng":-8.44399399238593},{"lat":40.7786743737394,"lng":-8.44484499125249},{"lat":40.7787367372905,"lng":-8.445141315181},{"lat":40.7786448487994,"lng":-8.4451886212961},{"lat":40.7785566785453,"lng":-8.4452262321097},{"lat":40.7784305064792,"lng":-8.44526449178733},{"lat":40.778319018388,"lng":-8.44530057051019},{"lat":40.7782121029571,"lng":-8.44534132599676},{"lat":40.7780925799097,"lng":-8.44537866161389},{"lat":40.7779454300526,"lng":-8.44540019790379},{"lat":40.7778535630209,"lng":-8.44541657349954},{"lat":40.7777892946091,"lng":-8.44550406612751},{"lat":40.7778521119635,"lng":-8.44559799650814},{"lat":40.7779512124625,"lng":-8.44558137904665},{"lat":40.7780626676741,"lng":-8.44557843161954},{"lat":40.778154424378,"lng":-8.44561710749291},{"lat":40.7782620718543,"lng":-8.44571040639906},{"lat":40.7783509601605,"lng":-8.44577278010228},{"lat":40.7784547468782,"lng":-8.44582029784178},{"lat":40.7785144045894,"lng":-8.44596853466395},{"lat":40.7785873549001,"lng":-8.4461295106086},{"lat":40.7786566712205,"lng":-8.44630219626011},{"lat":40.7786808304372,"lng":-8.44642467845931},{"lat":40.7786693403016,"lng":-8.44654332090036},{"lat":40.778683827106,"lng":-8.44671183894736},{"lat":40.7787424824551,"lng":-8.44696301362789},{"lat":40.7787942891314,"lng":-8.44708408877765},{"lat":40.7788568662589,"lng":-8.44728398905839},{"lat":40.7788801246548,"lng":-8.44740587661324},{"lat":40.7788999862431,"lng":-8.44758460837951},{"lat":40.7788740781337,"lng":-8.44773919396354},{"lat":40.7792284350345,"lng":-8.44838559159733},{"lat":40.7793787704538,"lng":-8.44860918466352},{"lat":40.7794341938896,"lng":-8.4498592615535},{"lat":40.7794448974151,"lng":-8.4500735062617},{"lat":40.7794581930788,"lng":-8.45033964710982},{"lat":40.7795139403831,"lng":-8.45061192736639},{"lat":40.7795725543521,"lng":-8.45089821283205},{"lat":40.7799226169228,"lng":-8.45260808993636},{"lat":40.7800459042064,"lng":-8.45268177478463},{"lat":40.7801669860867,"lng":-8.45265760453802},{"lat":40.7802884930845,"lng":-8.4526373040773},{"lat":40.7804081336699,"lng":-8.4526100573624},{"lat":40.7805228965494,"lng":-8.45257164682899},{"lat":40.7806389953838,"lng":-8.45254117611058},{"lat":40.7807168950658,"lng":-8.45262166790042},{"lat":40.7807642795473,"lng":-8.45272786018176},{"lat":40.7808134004292,"lng":-8.45283262870447},{"lat":40.7808423846336,"lng":-8.45294930836042},{"lat":40.7808454548418,"lng":-8.45307194573908},{"lat":40.7808397839461,"lng":-8.45319485746809},{"lat":40.7808326816369,"lng":-8.45331758697443},{"lat":40.7808081230721,"lng":-8.45343610410553},{"lat":40.780774691596,"lng":-8.45355112763905},{"lat":40.7807404902706,"lng":-8.45366575879302},{"lat":40.7807097691283,"lng":-8.45378205321998},{"lat":40.7806878884384,"lng":-8.45390176630228},{"lat":40.7806646823418,"lng":-8.45402106659463},{"lat":40.7809676936262,"lng":-8.45427158447092},{"lat":40.785580199969,"lng":-8.45714295547456},{"lat":40.7903780090518,"lng":-8.46000961158329},{"lat":40.7955181341139,"lng":-8.46280082937337},{"lat":40.7980309393129,"lng":-8.46453591070233},{"lat":40.800486543406,"lng":-8.46634643034995},{"lat":40.8060826970132,"lng":-8.47011834628249},{"lat":40.8084238049512,"lng":-8.47132535938091},{"lat":40.8125450846298,"lng":-8.47283205689006},{"lat":40.8149781087828,"lng":-8.47177714874845},{"lat":40.8160670910481,"lng":-8.47093821062699},{"lat":40.8170418907556,"lng":-8.46998126037913},{"lat":40.8179605056403,"lng":-8.4687063679188},{"lat":40.8188801963111,"lng":-8.46705510537878},{"lat":40.8204196174302,"lng":-8.46431549224465},{"lat":40.8217586756107,"lng":-8.46262854822379},{"lat":40.8208992154169,"lng":-8.45901080975752},{"lat":40.8204859297337,"lng":-8.45422336877818},{"lat":40.8208682328608,"lng":-8.45254268477866},{"lat":40.8211238268032,"lng":-8.45141896991769},{"lat":40.821122787258,"lng":-8.44948701885163},{"lat":40.8207470707269,"lng":-8.44747802816195},{"lat":40.8199914732568,"lng":-8.44496548235735},{"lat":40.8185582521766,"lng":-8.44256311880713},{"lat":40.8175917173072,"lng":-8.43864149979833},{"lat":40.8185711208897,"lng":-8.43733187074599},{"lat":40.818608602459,"lng":-8.43645180265436},{"lat":40.8177746549177,"lng":-8.43496143485419},{"lat":40.818169764974,"lng":-8.43361486615758},{"lat":40.8148898775636,"lng":-8.43026874955951},{"lat":40.8147675724891,"lng":-8.42910235329339},{"lat":40.8147099867822,"lng":-8.42627668691898},{"lat":40.8149776998492,"lng":-8.4258399837196},{"lat":40.8157616565315,"lng":-8.42539843408105},{"lat":40.8167539135934,"lng":-8.42590255752214},{"lat":40.8191776286635,"lng":-8.42652131305937},{"lat":40.8207710188532,"lng":-8.42813715432177},{"lat":40.822102924622,"lng":-8.4281819515207},{"lat":40.8228355500284,"lng":-8.42625823952767},{"lat":40.8221085208281,"lng":-8.42427596442854},{"lat":40.8235121965912,"lng":-8.42287820603018},{"lat":40.8265148871043,"lng":-8.42200409868934},{"lat":40.8259805263765,"lng":-8.41828537981912},{"lat":40.8259699763744,"lng":-8.41519461061005},{"lat":40.8264210185399,"lng":-8.4133881289921},{"lat":40.8264601942349,"lng":-8.41237588704221},{"lat":40.8260290040607,"lng":-8.40978268247786},{"lat":40.8255224201021,"lng":-8.41003479415648},{"lat":40.8245830530711,"lng":-8.41040177631133},{"lat":40.8236124493919,"lng":-8.41087577275896},{"lat":40.8232891158204,"lng":-8.41090238741216},{"lat":40.8228033839849,"lng":-8.41094408988702},{"lat":40.8224921658947,"lng":-8.41095036700076},{"lat":40.821464669322,"lng":-8.4110799928054},{"lat":40.8205981541879,"lng":-8.41088954472774},{"lat":40.8198155204915,"lng":-8.41034242600783},{"lat":40.8194783539463,"lng":-8.40979493685292},{"lat":40.8191759952264,"lng":-8.40948655494383},{"lat":40.8192075111896,"lng":-8.40937538578729},{"lat":40.8180926372827,"lng":-8.40798535402068},{"lat":40.8174725392013,"lng":-8.406753988141},{"lat":40.8173358295162,"lng":-8.40649906203592},{"lat":40.8172112921267,"lng":-8.4049083712741},{"lat":40.8170417724335,"lng":-8.40306445636524},{"lat":40.8172601322093,"lng":-8.39983664024736},{"lat":40.8173327737001,"lng":-8.39848129557104},{"lat":40.8173556717448,"lng":-8.39805400174576},{"lat":40.8168334770061,"lng":-8.39817245237838},{"lat":40.8166115158635,"lng":-8.39831322604299},{"lat":40.8161729050842,"lng":-8.39859140159262},{"lat":40.815465976969,"lng":-8.39931144865754},{"lat":40.814299827163,"lng":-8.40055953640307},{"lat":40.8135653968241,"lng":-8.40129139774182},{"lat":40.8127752629314,"lng":-8.40189063203662},{"lat":40.811445365559,"lng":-8.40260794819101},{"lat":40.8072783478827,"lng":-8.40204913955759},{"lat":40.8055477871887,"lng":-8.40182523529255},{"lat":40.803959015075,"lng":-8.40176425821774},{"lat":40.8020857774917,"lng":-8.40167868951068},{"lat":40.8018377940263,"lng":-8.40194252518637},{"lat":40.8017451058904,"lng":-8.402339716382},{"lat":40.8017720985723,"lng":-8.40261688746028},{"lat":40.8019754862053,"lng":-8.40284464349197},{"lat":40.8025522861101,"lng":-8.4034512321253},{"lat":40.8040928745454,"lng":-8.40507145242485},{"lat":40.8035960895594,"lng":-8.40593780122251},{"lat":40.8035310841009,"lng":-8.40631105551477},{"lat":40.8037410779825,"lng":-8.40661292250474},{"lat":40.8040520660119,"lng":-8.40685512871955},{"lat":40.8049028762109,"lng":-8.4072562441682},{"lat":40.8050674926093,"lng":-8.40736535609002},{"lat":40.8053597833476,"lng":-8.40766757492018},{"lat":40.8055423054846,"lng":-8.40793331575097},{"lat":40.8052753374361,"lng":-8.40848622299006},{"lat":40.8052194532925,"lng":-8.40891961139193},{"lat":40.805529706648,"lng":-8.40942680863089},{"lat":40.8059034891377,"lng":-8.41010292141711},{"lat":40.8061675230126,"lng":-8.4107666089616},{"lat":40.8057915333347,"lng":-8.41100594929794},{"lat":40.8055248386714,"lng":-8.411438332572},{"lat":40.8054051525832,"lng":-8.41170281230583},{"lat":40.8053580870001,"lng":-8.41220863479392},{"lat":40.8053283267854,"lng":-8.41307574554743},{"lat":40.8053170267637,"lng":-8.41405148202606},{"lat":40.8057159873314,"lng":-8.41568028586415},{"lat":40.8061437932557,"lng":-8.41677831406807},{"lat":40.8060785793962,"lng":-8.4171876027206},{"lat":40.8060271095365,"lng":-8.41752551003806},{"lat":40.8058365382614,"lng":-8.4187765778527},{"lat":40.8055666552713,"lng":-8.42047374765519},{"lat":40.8052974437379,"lng":-8.42193021077988},{"lat":40.8048995921315,"lng":-8.42343385889457},{"lat":40.8048560843551,"lng":-8.4236179457461},{"lat":40.8047773873409,"lng":-8.42372283099827},{"lat":40.8046581608683,"lng":-8.42400707476965},{"lat":40.8045828842727,"lng":-8.42418379641831},{"lat":40.8047674090691,"lng":-8.4276325352167},{"lat":40.8030133651404,"lng":-8.43028262557301},{"lat":40.8029130760233,"lng":-8.43055046496057},{"lat":40.8027663822151,"lng":-8.43079487305035},{"lat":40.802666889129,"lng":-8.4309645934407},{"lat":40.8025244190811,"lng":-8.43121197171291},{"lat":40.8023802019152,"lng":-8.4314748780152},{"lat":40.8023031786075,"lng":-8.43161269788708},{"lat":40.8022335496033,"lng":-8.43173728640527},{"lat":40.8020956035855,"lng":-8.43197639128796},{"lat":40.8019480050557,"lng":-8.43222125963756},{"lat":40.8018041812246,"lng":-8.43246946191416},{"lat":40.8016636923428,"lng":-8.43271625372729},{"lat":40.8015265436858,"lng":-8.43295891554071},{"lat":40.8013795260066,"lng":-8.4332213156628},{"lat":40.8012259378741,"lng":-8.43351348123583},{"lat":40.8011741555816,"lng":-8.43362316746797},{"lat":40.8010434566047,"lng":-8.43387912443125},{"lat":40.8009134107752,"lng":-8.43412370613284},{"lat":40.8007754370851,"lng":-8.43437323159162},{"lat":40.8006414109588,"lng":-8.43462538206074},{"lat":40.8005586117976,"lng":-8.43474586870202},{"lat":40.7987249514087,"lng":-8.43341108450053},{"lat":40.7986674815264,"lng":-8.43331330308874},{"lat":40.7985898716414,"lng":-8.43320701436347},{"lat":40.79853980138,"lng":-8.43306992240252},{"lat":40.7985331191026,"lng":-8.4329391923989},{"lat":40.798497189432,"lng":-8.43280157542674},{"lat":40.7984679320218,"lng":-8.432384340634},{"lat":40.7984321343524,"lng":-8.43219589352667},{"lat":40.7983951991511,"lng":-8.43199558838669},{"lat":40.7983373180834,"lng":-8.43184886812302},{"lat":40.798256625114,"lng":-8.43171448792794},{"lat":40.7981457727659,"lng":-8.43161244188054},{"lat":40.7980534623781,"lng":-8.43151248769662},{"lat":40.7978625268491,"lng":-8.43135308424456},{"lat":40.7975937482416,"lng":-8.43113917803369},{"lat":40.7973136243718,"lng":-8.43085495481162},{"lat":40.7972074253156,"lng":-8.43069273290851},{"lat":40.7971345734002,"lng":-8.43055554992181},{"lat":40.7970488819704,"lng":-8.43040373171205},{"lat":40.7968102546086,"lng":-8.43019506393855},{"lat":40.7967240930699,"lng":-8.4300850680594},{"lat":40.7966132206113,"lng":-8.42981098912966},{"lat":40.7965650066733,"lng":-8.42969215478752},{"lat":40.7963739880323,"lng":-8.42945858247095},{"lat":40.796140313324,"lng":-8.42917872629164},{"lat":40.7959886651289,"lng":-8.42893632872712},{"lat":40.7958662519366,"lng":-8.42874240519297},{"lat":40.7957098498324,"lng":-8.42859714761927},{"lat":40.7955355721412,"lng":-8.42840180843714},{"lat":40.7953845978352,"lng":-8.42817660012637},{"lat":40.7952374952711,"lng":-8.4279185905599},{"lat":40.795115216484,"lng":-8.42767028479384},{"lat":40.7949717651947,"lng":-8.42742674570174},{"lat":40.7948127544072,"lng":-8.42721086197069},{"lat":40.7946994164198,"lng":-8.42719744175713},{"lat":40.7945579808351,"lng":-8.42715024793061},{"lat":40.7943198210309,"lng":-8.4269670791172},{"lat":40.7941179528359,"lng":-8.42678489384096},{"lat":40.7939172320584,"lng":-8.42660995314548},{"lat":40.7936961753966,"lng":-8.4265344404132},{"lat":40.7935369722306,"lng":-8.4265327880571},{"lat":40.7934205657618,"lng":-8.42659352858164},{"lat":40.7933012670804,"lng":-8.42669312217295},{"lat":40.7931592201518,"lng":-8.42677886718568},{"lat":40.7930635080454,"lng":-8.42677547619525},{"lat":40.7929185726742,"lng":-8.42658431203371},{"lat":40.7927887300349,"lng":-8.42661596726648},{"lat":40.7926823796174,"lng":-8.4267229582887},{"lat":40.7925818171531,"lng":-8.42674869819959},{"lat":40.7924834294971,"lng":-8.42673511169031},{"lat":40.7923412703981,"lng":-8.42661860470328},{"lat":40.792164128925,"lng":-8.4265895032392},{"lat":40.7920667938576,"lng":-8.426551863404},{"lat":40.7920663716256,"lng":-8.42629392190472},{"lat":40.7921734577691,"lng":-8.42523006008595},{"lat":40.7908831114646,"lng":-8.42407841800249},{"lat":40.7899069381009,"lng":-8.42231648854435},{"lat":40.7883911749291,"lng":-8.41730493322249},{"lat":40.7881500208088,"lng":-8.41259842588982},{"lat":40.7880106515086,"lng":-8.40716422953121},{"lat":40.7880283011191,"lng":-8.4061526876876}]},{"zone_id":"sjm1","points":[{"lat":40.8906254434026,"lng":-8.50940844008076},{"lat":40.890707984079,"lng":-8.50963003894274},{"lat":40.8911022287787,"lng":-8.51029636360749},{"lat":40.8928962108482,"lng":-8.51204474028974},{"lat":40.893588433478,"lng":-8.51061346729984},{"lat":40.894431955507,"lng":-8.5098921232233},{"lat":40.8950644222329,"lng":-8.50909108837596},{"lat":40.8951737868929,"lng":-8.50775993985621},{"lat":40.8952602368945,"lng":-8.50776471622773},{"lat":40.8952859137104,"lng":-8.50768863200965},{"lat":40.8953920121852,"lng":-8.50744659055305},{"lat":40.8954642041378,"lng":-8.50730459521612},{"lat":40.8956906401839,"lng":-8.50710571738833},{"lat":40.8957784732911,"lng":-8.50705209282936},{"lat":40.8962523818937,"lng":-8.50668615295151},{"lat":40.8965401188666,"lng":-8.50638952655139},{"lat":40.8973676760499,"lng":-8.50554598026441},{"lat":40.8976510992685,"lng":-8.50527613489536},{"lat":40.8978214111301,"lng":-8.5051271699295},{"lat":40.8982621103422,"lng":-8.5047366056604},{"lat":40.8983706089754,"lng":-8.5046451867915},{"lat":40.8988649611283,"lng":-8.50424077540329},{"lat":40.8994966468984,"lng":-8.50371699694569},{"lat":40.9006813771888,"lng":-8.50245459867903},{"lat":40.9019915974248,"lng":-8.50189741315696},{"lat":40.9024286113429,"lng":-8.50191267834832},{"lat":40.9025361573455,"lng":-8.50188170952224},{"lat":40.902754626337,"lng":-8.5018218905658},{"lat":40.9028691765666,"lng":-8.50176855703376},{"lat":40.9029755521643,"lng":-8.50167294180161},{"lat":40.9031166952506,"lng":-8.50154404632367},{"lat":40.9032311999006,"lng":-8.5014411511636},{"lat":40.9035045383038,"lng":-8.50119374460425},{"lat":40.9039480433343,"lng":-8.50092369195479},{"lat":40.9041840582651,"lng":-8.50080476392478},{"lat":40.9044186413181,"lng":-8.50068749824582},{"lat":40.904579106746,"lng":-8.50059942122296},{"lat":40.9048759499688,"lng":-8.50039802184157},{"lat":40.9052501192554,"lng":-8.49995214068382},{"lat":40.9061034688925,"lng":-8.49907708482125},{"lat":40.9066899500424,"lng":-8.49912802151431},{"lat":40.90734568504,"lng":-8.49891339159111},{"lat":40.9075674239387,"lng":-8.49864147488183},{"lat":40.9077863452449,"lng":-8.49856910629726},{"lat":40.9079051748318,"lng":-8.49836210655259},{"lat":40.908686461234,"lng":-8.49469054937441},{"lat":40.9089019040685,"lng":-8.49448620439316},{"lat":40.9091269063971,"lng":-8.49446619162536},{"lat":40.9092660950372,"lng":-8.49445178831094},{"lat":40.9093103114301,"lng":-8.49384180258679},{"lat":40.9093721483181,"lng":-8.49348800333609},{"lat":40.9095613161413,"lng":-8.49278588637556},{"lat":40.9100657550713,"lng":-8.49303279574714},{"lat":40.910802851948,"lng":-8.49171659287822},{"lat":40.9110022133285,"lng":-8.49173694734051},{"lat":40.9112347987498,"lng":-8.4914764990167},{"lat":40.911434899858,"lng":-8.4913321914386},{"lat":40.9123088597661,"lng":-8.49116909572445},{"lat":40.9131386158944,"lng":-8.49140648253971},{"lat":40.9131633223514,"lng":-8.4914238286616},{"lat":40.9132192280616,"lng":-8.49145665916921},{"lat":40.913258873239,"lng":-8.49147752949901},{"lat":40.9134643256653,"lng":-8.49153162299244},{"lat":40.9135758977348,"lng":-8.49148913597461},{"lat":40.9138152379563,"lng":-8.49154642740431},{"lat":40.9138229142112,"lng":-8.49153898829306},{"lat":40.9152169739164,"lng":-8.49103245803335},{"lat":40.9156722270899,"lng":-8.48972440914533},{"lat":40.9159130388278,"lng":-8.49007302295998},{"lat":40.9167728141759,"lng":-8.48898499576732},{"lat":40.9169383006894,"lng":-8.48838251738865},{"lat":40.916400222682,"lng":-8.48765581748797},{"lat":40.9157752394077,"lng":-8.4872984086288},{"lat":40.9150596794567,"lng":-8.48360448553362},{"lat":40.9140001874563,"lng":-8.48211323824415},{"lat":40.9138695418615,"lng":-8.48182849705016},{"lat":40.9099630447303,"lng":-8.48023700243726},{"lat":40.9096710052611,"lng":-8.47886578524544},{"lat":40.9081909327997,"lng":-8.47913546462463},{"lat":40.9043645380627,"lng":-8.47808414931957},{"lat":40.9043927499306,"lng":-8.4779266651359},{"lat":40.9044138654633,"lng":-8.47773166304605},{"lat":40.9044293693369,"lng":-8.47757721597109},{"lat":40.904449376152,"lng":-8.47739265098713},{"lat":40.9042459979068,"lng":-8.47731883797216},{"lat":40.9040325362222,"lng":-8.47721660765622},{"lat":40.903857871786,"lng":-8.47711849441383},{"lat":40.9036776954879,"lng":-8.47702640599231},{"lat":40.9035358604971,"lng":-8.47693914884395},{"lat":40.9031711977491,"lng":-8.47671994310527},{"lat":40.9027329376611,"lng":-8.47642060381643},{"lat":40.9024499278987,"lng":-8.47619876952021},{"lat":40.9022767498693,"lng":-8.47606350051234},{"lat":40.90205754124,"lng":-8.47589395577389},{"lat":40.9019666754816,"lng":-8.47606392906832},{"lat":40.9018497125567,"lng":-8.47606462406999},{"lat":40.9015156171205,"lng":-8.47641380363837},{"lat":40.9007372554846,"lng":-8.47617334603455},{"lat":40.8995721161364,"lng":-8.47717986723799},{"lat":40.8993505203525,"lng":-8.47702859907313},{"lat":40.8989459032001,"lng":-8.47754350428631},{"lat":40.893216708802,"lng":-8.47361207833261},{"lat":40.8917563042494,"lng":-8.47604871059997},{"lat":40.8903037276257,"lng":-8.47472696647786},{"lat":40.8887446062885,"lng":-8.47496123676672},{"lat":40.888392302629,"lng":-8.47440268761171},{"lat":40.8877597181182,"lng":-8.47438733658166},{"lat":40.8873543146576,"lng":-8.47538659869086},{"lat":40.8876290776918,"lng":-8.47568680167696},{"lat":40.8877395933352,"lng":-8.47628226674882},{"lat":40.8875000587168,"lng":-8.47654497733595},{"lat":40.8871054529991,"lng":-8.47677233287743},{"lat":40.8863765704544,"lng":-8.47672570244377},{"lat":40.8857878157136,"lng":-8.47719073597639},{"lat":40.8854175878289,"lng":-8.47782536981468},{"lat":40.8849380752703,"lng":-8.47773443766875},{"lat":40.8832366783559,"lng":-8.4781278414719},{"lat":40.8821994199509,"lng":-8.47994556573053},{"lat":40.8818046727528,"lng":-8.48012636940984},{"lat":40.8811074195084,"lng":-8.48020823173781},{"lat":40.8801331760326,"lng":-8.48059505018124},{"lat":40.8799769139612,"lng":-8.48078747549387},{"lat":40.880134845443,"lng":-8.48109905878637},{"lat":40.8814237117128,"lng":-8.48158127956842},{"lat":40.8815500017706,"lng":-8.48223053236049},{"lat":40.8813378377883,"lng":-8.48285597746233},{"lat":40.8789508539873,"lng":-8.4846896061451},{"lat":40.8790118256284,"lng":-8.48509574000062},{"lat":40.8799359924021,"lng":-8.48568627500205},{"lat":40.8810630711415,"lng":-8.49435050420012},{"lat":40.8815273327266,"lng":-8.4978969804741},{"lat":40.8818142283378,"lng":-8.49917952650462},{"lat":40.881967566805,"lng":-8.49966954412052},{"lat":40.8821984646239,"lng":-8.49982143564286},{"lat":40.8848619773503,"lng":-8.50402300100954},{"lat":40.8906254434026,"lng":-8.50940844008076}]},{"zone_id":"oaz7","points":[{"lat":40.7766569189472,"lng":-8.51692661324558},{"lat":40.7774519442563,"lng":-8.5182301784012},{"lat":40.7801962364197,"lng":-8.52098262670855},{"lat":40.7803595653488,"lng":-8.52085512202108},{"lat":40.7803914430857,"lng":-8.52093759221669},{"lat":40.7803639614513,"lng":-8.5211292213263},{"lat":40.7803325748748,"lng":-8.52130724937385},{"lat":40.7803109158416,"lng":-8.52145077314366},{"lat":40.7802684193036,"lng":-8.52190150984678},{"lat":40.7802431994318,"lng":-8.52206282596541},{"lat":40.7802255904436,"lng":-8.52213837036706},{"lat":40.7801948949127,"lng":-8.52224101778145},{"lat":40.7801165962049,"lng":-8.52250285082725},{"lat":40.7800048511631,"lng":-8.52281680142713},{"lat":40.7799241463641,"lng":-8.52311365947532},{"lat":40.7799059524485,"lng":-8.52318249008565},{"lat":40.779802998126,"lng":-8.52352168700612},{"lat":40.7797515349075,"lng":-8.52366829810542},{"lat":40.7796589220164,"lng":-8.52403813160243},{"lat":40.7796148268338,"lng":-8.52431600224955},{"lat":40.7795537459266,"lng":-8.5246312743565},{"lat":40.7795256805054,"lng":-8.52480180866507},{"lat":40.7794337297783,"lng":-8.52509122654719},{"lat":40.7794058509338,"lng":-8.52523368480837},{"lat":40.7793942087388,"lng":-8.52535706897208},{"lat":40.7793824399633,"lng":-8.52543712849516},{"lat":40.779367204566,"lng":-8.52552344369466},{"lat":40.7793747752492,"lng":-8.52571643967021},{"lat":40.7793821583692,"lng":-8.52579869715119},{"lat":40.7794035509161,"lng":-8.52604751719065},{"lat":40.7794141015488,"lng":-8.5262351104151},{"lat":40.7794385804927,"lng":-8.52648512169239},{"lat":40.7794466466081,"lng":-8.5266355543151},{"lat":40.7794722344854,"lng":-8.52681498447744},{"lat":40.7794946649117,"lng":-8.52695484995511},{"lat":40.7794842078006,"lng":-8.52712409901571},{"lat":40.779474857791,"lng":-8.52732111688216},{"lat":40.779473036841,"lng":-8.52747428454538},{"lat":40.7794715366073,"lng":-8.52756992659938},{"lat":40.7794450816085,"lng":-8.52782097806921},{"lat":40.7794220699593,"lng":-8.52807033322163},{"lat":40.7794295231403,"lng":-8.52833236056685},{"lat":40.7794416599043,"lng":-8.52850247035402},{"lat":40.7794808714508,"lng":-8.5286723605511},{"lat":40.7795007849591,"lng":-8.52876772095346},{"lat":40.7795273913132,"lng":-8.52891651316912},{"lat":40.7795354227628,"lng":-8.52898924916999},{"lat":40.7795337794769,"lng":-8.52923981630593},{"lat":40.7795551617118,"lng":-8.52937568313572},{"lat":40.7795661440266,"lng":-8.5294334062864},{"lat":40.7795879289853,"lng":-8.52957832140914},{"lat":40.7796309038054,"lng":-8.52988366931957},{"lat":40.779661268647,"lng":-8.53007398079786},{"lat":40.7796735724371,"lng":-8.53029816342214},{"lat":40.7796732253379,"lng":-8.53042096649741},{"lat":40.7801355374825,"lng":-8.53014566746461},{"lat":40.7805521963186,"lng":-8.52975859653717},{"lat":40.781088520751,"lng":-8.53060049016005},{"lat":40.7825652307185,"lng":-8.53286228242934},{"lat":40.7826806263896,"lng":-8.5330389548366},{"lat":40.7833927866927,"lng":-8.53230794103457},{"lat":40.7833995254163,"lng":-8.53241585919112},{"lat":40.7833997958604,"lng":-8.53248030672471},{"lat":40.7834147303669,"lng":-8.53275607107803},{"lat":40.7834309348384,"lng":-8.53291254608765},{"lat":40.7834470420695,"lng":-8.53307103456282},{"lat":40.783459256694,"lng":-8.53315770870265},{"lat":40.7834662394526,"lng":-8.53322124882583},{"lat":40.7834814547644,"lng":-8.53335118105706},{"lat":40.7834916318264,"lng":-8.53345407332374},{"lat":40.7834920487363,"lng":-8.53359291923456},{"lat":40.7834839768624,"lng":-8.53366110893761},{"lat":40.7834772973777,"lng":-8.53379458107673},{"lat":40.7834713564385,"lng":-8.53389737532818},{"lat":40.7834427499886,"lng":-8.53409528117058},{"lat":40.7834195401749,"lng":-8.5342695255231},{"lat":40.7833940859919,"lng":-8.53446709343567},{"lat":40.7833554304546,"lng":-8.53473187199257},{"lat":40.7833472805133,"lng":-8.5348735092533},{"lat":40.783336980998,"lng":-8.53506346998202},{"lat":40.7833465308229,"lng":-8.53526788391969},{"lat":40.7833547895311,"lng":-8.53542952405509},{"lat":40.7833633715517,"lng":-8.53555029442521},{"lat":40.7833852165746,"lng":-8.53574056768168},{"lat":40.783427210959,"lng":-8.53598830059937},{"lat":40.7834630070809,"lng":-8.53620602486964},{"lat":40.7835024318098,"lng":-8.53639024575132},{"lat":40.7835272971601,"lng":-8.53648943517963},{"lat":40.7835516928012,"lng":-8.53661966195231},{"lat":40.7835693973777,"lng":-8.53673219521235},{"lat":40.7836046532595,"lng":-8.53687255781293},{"lat":40.7836450499691,"lng":-8.53703605204542},{"lat":40.7836912899879,"lng":-8.53720230787343},{"lat":40.7837107983265,"lng":-8.53728843495067},{"lat":40.7837280267099,"lng":-8.53738248452562},{"lat":40.7837478712628,"lng":-8.53750060051745},{"lat":40.7837669488769,"lng":-8.53763221627276},{"lat":40.783728805157,"lng":-8.53774950421762},{"lat":40.783666988038,"lng":-8.53786427892072},{"lat":40.7836672239187,"lng":-8.53802773515022},{"lat":40.7837439599614,"lng":-8.53827832097186},{"lat":40.7838005107393,"lng":-8.53848249045263},{"lat":40.7838691187407,"lng":-8.53871433826358},{"lat":40.783894938033,"lng":-8.53882336908394},{"lat":40.783831882464,"lng":-8.53888055808369},{"lat":40.7837657106136,"lng":-8.5389276592783},{"lat":40.7837829622672,"lng":-8.53909125114719},{"lat":40.7838483524365,"lng":-8.53911606268049},{"lat":40.7838964256601,"lng":-8.53914134752458},{"lat":40.7839502543284,"lng":-8.53919794973156},{"lat":40.7839992763316,"lng":-8.53926447304269},{"lat":40.7840387940618,"lng":-8.5393190927221},{"lat":40.7840765537369,"lng":-8.53938708769221},{"lat":40.7841260277557,"lng":-8.53950396386169},{"lat":40.7841476254254,"lng":-8.53958418139764},{"lat":40.7841882870374,"lng":-8.53977362521439},{"lat":40.784248368653,"lng":-8.53989530658937},{"lat":40.7842782380579,"lng":-8.53995353937038},{"lat":40.784333146933,"lng":-8.54006120878415},{"lat":40.7844353564924,"lng":-8.54023764445647},{"lat":40.784502936218,"lng":-8.54035238272198},{"lat":40.7845304357849,"lng":-8.54041889441723},{"lat":40.7845355542571,"lng":-8.54052448210376},{"lat":40.7845896136388,"lng":-8.54066804393278},{"lat":40.7846991987009,"lng":-8.54089760143016},{"lat":40.784767628813,"lng":-8.54100085311755},{"lat":40.7848323183009,"lng":-8.541092354636},{"lat":40.7848840681773,"lng":-8.54117631342207},{"lat":40.7849324070876,"lng":-8.54125717000915},{"lat":40.7850182394054,"lng":-8.54148065938547},{"lat":40.7850879246071,"lng":-8.54166140048689},{"lat":40.78516551727,"lng":-8.5418465739117},{"lat":40.7852178677045,"lng":-8.54196394653048},{"lat":40.7852705509204,"lng":-8.54208902088564},{"lat":40.785298846884,"lng":-8.54215921159081},{"lat":40.7853298854908,"lng":-8.54224292503316},{"lat":40.7853546744916,"lng":-8.54231143572707},{"lat":40.7853911656613,"lng":-8.54243190924388},{"lat":40.7854391730455,"lng":-8.5425549415436},{"lat":40.7855107853927,"lng":-8.54272503421096},{"lat":40.7855932763477,"lng":-8.54292504925937},{"lat":40.7856700612245,"lng":-8.54310915433488},{"lat":40.7857505456127,"lng":-8.54331614908703},{"lat":40.7857874784221,"lng":-8.54341316890326},{"lat":40.7858509548874,"lng":-8.543591742178},{"lat":40.7859348170137,"lng":-8.54378584528082},{"lat":40.7860080331961,"lng":-8.54396021546721},{"lat":40.7861020629251,"lng":-8.54418115605557},{"lat":40.7861793370831,"lng":-8.54437900996798},{"lat":40.7862548187469,"lng":-8.54454913154935},{"lat":40.7863321370089,"lng":-8.54473419140968},{"lat":40.7863939686275,"lng":-8.54486868601375},{"lat":40.7864217103473,"lng":-8.54491707639164},{"lat":40.786479295931,"lng":-8.54505510052476},{"lat":40.7865260546888,"lng":-8.5451742193472},{"lat":40.7866036471586,"lng":-8.54535786068652},{"lat":40.7866762500861,"lng":-8.54552678071872},{"lat":40.7867437586105,"lng":-8.54568488837868},{"lat":40.7868158428538,"lng":-8.54584788334707},{"lat":40.7868790551844,"lng":-8.54597338440584},{"lat":40.786909090609,"lng":-8.54603494054373},{"lat":40.7869528027969,"lng":-8.54614930293681},{"lat":40.7870167332176,"lng":-8.54630075627196},{"lat":40.7870759953564,"lng":-8.54644791465185},{"lat":40.7871601919994,"lng":-8.54667271098379},{"lat":40.7872127562259,"lng":-8.54680442660598},{"lat":40.7874126261087,"lng":-8.54731025044913},{"lat":40.7877643374568,"lng":-8.54750480989082},{"lat":40.7881560512064,"lng":-8.54770709966663},{"lat":40.7883675097421,"lng":-8.54781630184592},{"lat":40.7888920258187,"lng":-8.54807193521441},{"lat":40.7889573694592,"lng":-8.54810416538789},{"lat":40.789094775302,"lng":-8.54816887272882},{"lat":40.7892484164921,"lng":-8.54824072859522},{"lat":40.7894142485033,"lng":-8.54831508837762},{"lat":40.7895661271172,"lng":-8.54837899580028},{"lat":40.7896988331696,"lng":-8.54843543236512},{"lat":40.7899553033286,"lng":-8.54853778861226},{"lat":40.7903612763784,"lng":-8.54868727705593},{"lat":40.7907824412108,"lng":-8.54883157424058},{"lat":40.7911483667739,"lng":-8.54894642216942},{"lat":40.791485249463,"lng":-8.54904379512983},{"lat":40.7918067043872,"lng":-8.54912629904163},{"lat":40.7918750315106,"lng":-8.54872335161647},{"lat":40.7919074626584,"lng":-8.54851781701205},{"lat":40.7919881804416,"lng":-8.54823672944561},{"lat":40.7920475450457,"lng":-8.54803378610926},{"lat":40.7921003170974,"lng":-8.54785054917912},{"lat":40.7921449223369,"lng":-8.54766128362255},{"lat":40.792159065673,"lng":-8.54755669293363},{"lat":40.7922025184384,"lng":-8.547428447684},{"lat":40.7922505355862,"lng":-8.547262339247},{"lat":40.7923277839883,"lng":-8.54699138486589},{"lat":40.7923484399938,"lng":-8.5468958307296},{"lat":40.792352448903,"lng":-8.54680870766917},{"lat":40.792351870042,"lng":-8.54674184783674},{"lat":40.7923552951078,"lng":-8.5466814157691},{"lat":40.7923650978838,"lng":-8.54661507769443},{"lat":40.7923822046573,"lng":-8.54655121384712},{"lat":40.7923980958751,"lng":-8.54648680687618},{"lat":40.7924126950046,"lng":-8.54642185394049},{"lat":40.79243101876,"lng":-8.54635859826209},{"lat":40.7924525595419,"lng":-8.54629520253739},{"lat":40.7924818598403,"lng":-8.54624186566908},{"lat":40.7925880512802,"lng":-8.54624137695986},{"lat":40.792684525411,"lng":-8.54625726194308},{"lat":40.792759417944,"lng":-8.54626495617229},{"lat":40.7928829240593,"lng":-8.54627603359117},{"lat":40.7929832171264,"lng":-8.5462818710689},{"lat":40.7930310338593,"lng":-8.54628145776327},{"lat":40.7930849797964,"lng":-8.54627966062015},{"lat":40.7931418402796,"lng":-8.54626840382979},{"lat":40.7932131452285,"lng":-8.54624692824818},{"lat":40.7932589324972,"lng":-8.54623512764058},{"lat":40.7933701959317,"lng":-8.54622018063538},{"lat":40.7934524505792,"lng":-8.54620884417278},{"lat":40.7935711097577,"lng":-8.54619062528644},{"lat":40.7936668875999,"lng":-8.54617487124254},{"lat":40.7937132775679,"lng":-8.54617053893864},{"lat":40.7938449978272,"lng":-8.54615121598729},{"lat":40.7939493280274,"lng":-8.54613610726435},{"lat":40.7940878981753,"lng":-8.54611481233462},{"lat":40.7942200737385,"lng":-8.54609407131937},{"lat":40.7943468396807,"lng":-8.54607519133293},{"lat":40.79442865085,"lng":-8.54606195597139},{"lat":40.7945054979704,"lng":-8.5460518890141},{"lat":40.7946082591828,"lng":-8.54604743426752},{"lat":40.7946780563311,"lng":-8.54604466871708},{"lat":40.7948181586744,"lng":-8.5460481482973},{"lat":40.79493999076,"lng":-8.546048906737},{"lat":40.7950325567034,"lng":-8.54604995661698},{"lat":40.7950941387719,"lng":-8.54605294614828},{"lat":40.7951749988154,"lng":-8.54610392621433},{"lat":40.7952147331874,"lng":-8.54614765824288},{"lat":40.7952659459519,"lng":-8.54617961365628},{"lat":40.7953333285476,"lng":-8.54622221615526},{"lat":40.7953949477365,"lng":-8.54626513729664},{"lat":40.7954775657211,"lng":-8.54632785811448},{"lat":40.7955507508274,"lng":-8.54638459725161},{"lat":40.795615389912,"lng":-8.54643891251295},{"lat":40.7956914259122,"lng":-8.54650419972366},{"lat":40.7957573192304,"lng":-8.54656030036089},{"lat":40.7958218890384,"lng":-8.54660904645658},{"lat":40.7958800582026,"lng":-8.54665953029447},{"lat":40.7959238512706,"lng":-8.54670139363532},{"lat":40.7959894484361,"lng":-8.5467647207947},{"lat":40.7960626663894,"lng":-8.54683710073663},{"lat":40.7961414460264,"lng":-8.54691567693289},{"lat":40.7962070222451,"lng":-8.54698481047303},{"lat":40.7962528624232,"lng":-8.54703308550368},{"lat":40.7963113443963,"lng":-8.54709672457948},{"lat":40.796351711259,"lng":-8.54713975092864},{"lat":40.7963882030953,"lng":-8.54718358388159},{"lat":40.7964300475201,"lng":-8.54724119403855},{"lat":40.79647008804,"lng":-8.54729986067045},{"lat":40.7965297448284,"lng":-8.54741220661678},{"lat":40.7965623070432,"lng":-8.54747177369418},{"lat":40.7965921538127,"lng":-8.54753547236035},{"lat":40.7966377538067,"lng":-8.54752544828145},{"lat":40.7966695804094,"lng":-8.54746462375498},{"lat":40.7966930928587,"lng":-8.54741204195961},{"lat":40.7967468359901,"lng":-8.54729139866534},{"lat":40.7967792014772,"lng":-8.5472309327545},{"lat":40.7968148237927,"lng":-8.54719110475852},{"lat":40.7968680084042,"lng":-8.54712567407497},{"lat":40.7969013323916,"lng":-8.54707386390966},{"lat":40.7969441131153,"lng":-8.54699675611825},{"lat":40.796972326504,"lng":-8.54693898944128},{"lat":40.7970001116717,"lng":-8.54687482143052},{"lat":40.7970251877879,"lng":-8.54681300759575},{"lat":40.7970553184112,"lng":-8.54674790609468},{"lat":40.7971004775304,"lng":-8.54668550583986},{"lat":40.7971396259365,"lng":-8.54664167046971},{"lat":40.7972056652792,"lng":-8.54658212496656},{"lat":40.7972669376534,"lng":-8.54654612998932},{"lat":40.7973153872356,"lng":-8.54652012617341},{"lat":40.7973649228922,"lng":-8.54649258982378},{"lat":40.7974137517518,"lng":-8.54646125606238},{"lat":40.7974607672945,"lng":-8.54643311152855},{"lat":40.7975077867058,"lng":-8.54640413633647},{"lat":40.7975564341691,"lng":-8.54637315798611},{"lat":40.7976040966256,"lng":-8.54634063187377},{"lat":40.7976643784033,"lng":-8.54630463031992},{"lat":40.7977053561347,"lng":-8.54627798737715},{"lat":40.797756011329,"lng":-8.54623955614285},{"lat":40.7978163329537,"lng":-8.54619241499302},{"lat":40.7978904835142,"lng":-8.54613090549441},{"lat":40.797959875592,"lng":-8.54606569184751},{"lat":40.7979961206262,"lng":-8.54602788079844},{"lat":40.7980323035797,"lng":-8.54598224984445},{"lat":40.7980770673378,"lng":-8.54592956111575},{"lat":40.7981193063029,"lng":-8.54587756883262},{"lat":40.7981815906643,"lng":-8.54578541227397},{"lat":40.7982118672769,"lng":-8.54572943421848},{"lat":40.7982523683873,"lng":-8.5456600121345},{"lat":40.7983051421342,"lng":-8.54558331921901},{"lat":40.7983451143126,"lng":-8.54551069311322},{"lat":40.7983984194872,"lng":-8.54541125329789},{"lat":40.7984408960607,"lng":-8.54531814391935},{"lat":40.7984882369474,"lng":-8.5451995879248},{"lat":40.7985164308979,"lng":-8.5451217932456},{"lat":40.7985475004337,"lng":-8.54502043679157},{"lat":40.7985903212341,"lng":-8.54493171343162},{"lat":40.7986210924713,"lng":-8.54488841676992},{"lat":40.7986586270105,"lng":-8.54484267344133},{"lat":40.798709977163,"lng":-8.54478599646227},{"lat":40.7987627748608,"lng":-8.54472766939086},{"lat":40.7988067313712,"lng":-8.54467367079956},{"lat":40.7988599132327,"lng":-8.5446083546405},{"lat":40.7989143684487,"lng":-8.54453972833809},{"lat":40.7989493692216,"lng":-8.54449716837696},{"lat":40.7990075777749,"lng":-8.54456210960019},{"lat":40.7990694312934,"lng":-8.54476595046503},{"lat":40.7991336750688,"lng":-8.54498094400297},{"lat":40.7991938199083,"lng":-8.54518394559627},{"lat":40.7992264768406,"lng":-8.54529269083632},{"lat":40.7992555809498,"lng":-8.5453879055321},{"lat":40.799269876946,"lng":-8.54544416161608},{"lat":40.7992827696432,"lng":-8.54551486450516},{"lat":40.7992897234691,"lng":-8.54558458386691},{"lat":40.7992983720282,"lng":-8.54570893974351},{"lat":40.7992992179402,"lng":-8.54579923863704},{"lat":40.7992941461026,"lng":-8.54588239104465},{"lat":40.7992784096865,"lng":-8.54597661609366},{"lat":40.7992610140349,"lng":-8.54605637416918},{"lat":40.79922621397,"lng":-8.54619313870992},{"lat":40.7991937562733,"lng":-8.54635444650759},{"lat":40.7991716834944,"lng":-8.54648263943387},{"lat":40.7991509069261,"lng":-8.54662600792186},{"lat":40.7991572349014,"lng":-8.54689245905763},{"lat":40.7991241521413,"lng":-8.54709193148383},{"lat":40.7991029753498,"lng":-8.5473388112674},{"lat":40.7991154530416,"lng":-8.5475252594719},{"lat":40.7991674707345,"lng":-8.54776591489469},{"lat":40.7992577358128,"lng":-8.54797414867082},{"lat":40.7994250763478,"lng":-8.54823793359865},{"lat":40.7995415861159,"lng":-8.54846097633888},{"lat":40.7996380379137,"lng":-8.54873501825374},{"lat":40.7997035396291,"lng":-8.54907887869989},{"lat":40.8002475248356,"lng":-8.54900639154888},{"lat":40.8007774884136,"lng":-8.54893564893862},{"lat":40.801168534597,"lng":-8.54888124114139},{"lat":40.8016340649976,"lng":-8.54881404663662},{"lat":40.8019898767649,"lng":-8.54876535698107},{"lat":40.8024148053101,"lng":-8.54870976975156},{"lat":40.8028066222787,"lng":-8.54866531372537},{"lat":40.8034131693547,"lng":-8.54857884743945},{"lat":40.8036282052109,"lng":-8.54855246537538},{"lat":40.8038433334226,"lng":-8.54852618919241},{"lat":40.8040604845193,"lng":-8.54849780183726},{"lat":40.8042353033636,"lng":-8.54847496428384},{"lat":40.8044729230903,"lng":-8.54844468604787},{"lat":40.8046857761656,"lng":-8.54841603081441},{"lat":40.8048916529427,"lng":-8.54838817687309},{"lat":40.8050846614259,"lng":-8.54836415093198},{"lat":40.8052889294212,"lng":-8.54833793153532},{"lat":40.8054958068635,"lng":-8.54831125658374},{"lat":40.8056963310314,"lng":-8.54828466512686},{"lat":40.8059211795934,"lng":-8.54825622229623},{"lat":40.806117537881,"lng":-8.54823293585189},{"lat":40.8062883485233,"lng":-8.54821410484986},{"lat":40.8065092012812,"lng":-8.54819024787186},{"lat":40.8067305786135,"lng":-8.54816675191156},{"lat":40.8069386600532,"lng":-8.54814554792063},{"lat":40.8071137182086,"lng":-8.54812923759999},{"lat":40.8072575809599,"lng":-8.54811735361204},{"lat":40.8074954899547,"lng":-8.54810047711248},{"lat":40.8076812407794,"lng":-8.54808659510472},{"lat":40.8078846266204,"lng":-8.54807401392856},{"lat":40.8080904823697,"lng":-8.54806322101791},{"lat":40.8082716959486,"lng":-8.54805500153348},{"lat":40.8084873384379,"lng":-8.54804865210303},{"lat":40.8086734764026,"lng":-8.54804354338793},{"lat":40.8088569035115,"lng":-8.54804279567577},{"lat":40.809108404228,"lng":-8.54804200031676},{"lat":40.8093486068942,"lng":-8.54804409627272},{"lat":40.8095227998757,"lng":-8.54804648740726},{"lat":40.8097012300094,"lng":-8.54805328766234},{"lat":40.8099089698005,"lng":-8.5480616962934},{"lat":40.8101406418103,"lng":-8.54807238381606},{"lat":40.8103332981403,"lng":-8.54808436432788},{"lat":40.8105364278251,"lng":-8.54809866317678},{"lat":40.810713541639,"lng":-8.54811316031724},{"lat":40.8108883135375,"lng":-8.54812658221259},{"lat":40.8112334243958,"lng":-8.54815462408342},{"lat":40.8115176287589,"lng":-8.54817767394698},{"lat":40.8120243903303,"lng":-8.54824643388126},{"lat":40.8125387426608,"lng":-8.5483295346282},{"lat":40.8132939628555,"lng":-8.54848276766769},{"lat":40.8136205334533,"lng":-8.54856053800297},{"lat":40.8138852027555,"lng":-8.54862249830425},{"lat":40.8141684041372,"lng":-8.54869344268227},{"lat":40.8144464190489,"lng":-8.54876602412348},{"lat":40.8146803002601,"lng":-8.54882954445937},{"lat":40.8148674164615,"lng":-8.54888353190136},{"lat":40.815098714744,"lng":-8.54895186684056},{"lat":40.8152485840846,"lng":-8.54855215908025},{"lat":40.8152938277477,"lng":-8.54841673185838},{"lat":40.8153467221228,"lng":-8.54823216516317},{"lat":40.8153831899382,"lng":-8.54810806131305},{"lat":40.8154133621336,"lng":-8.54800679240715},{"lat":40.8154673750657,"lng":-8.54786158262759},{"lat":40.8154902338582,"lng":-8.54779531337975},{"lat":40.8155214291286,"lng":-8.54770487584428},{"lat":40.8155541328758,"lng":-8.54760042328269},{"lat":40.8155943704902,"lng":-8.54747954236178},{"lat":40.8156283832533,"lng":-8.54736182278184},{"lat":40.8156847844818,"lng":-8.54722812272823},{"lat":40.8157087556863,"lng":-8.54714803063175},{"lat":40.8157401345397,"lng":-8.5470615852824},{"lat":40.8157866723986,"lng":-8.54691632638998},{"lat":40.8158156649113,"lng":-8.54679217456275},{"lat":40.8158259486865,"lng":-8.54671199714356},{"lat":40.8158511580962,"lng":-8.54663807694887},{"lat":40.8159232155977,"lng":-8.54640751933344},{"lat":40.8160115608975,"lng":-8.54615501861399},{"lat":40.8160532533957,"lng":-8.54602952245756},{"lat":40.8160986822582,"lng":-8.54589184101751},{"lat":40.816169681214,"lng":-8.54567988513176},{"lat":40.8162775711529,"lng":-8.54535046191414},{"lat":40.8163708262964,"lng":-8.54505887354899},{"lat":40.8164583603817,"lng":-8.54478052528871},{"lat":40.8165137610272,"lng":-8.5445988126264},{"lat":40.8165620539644,"lng":-8.54441504078767},{"lat":40.8165897815198,"lng":-8.54431659917372},{"lat":40.8166306045507,"lng":-8.54418208727974},{"lat":40.816674363878,"lng":-8.54405719411488},{"lat":40.8167224587833,"lng":-8.54390376224379},{"lat":40.8167458447868,"lng":-8.54383563662613},{"lat":40.8167895974132,"lng":-8.54371252090033},{"lat":40.8168433860758,"lng":-8.54357820667823},{"lat":40.8168900140812,"lng":-8.54345700596174},{"lat":40.8169628499685,"lng":-8.54323367634136},{"lat":40.8170095989125,"lng":-8.54307833965031},{"lat":40.8170431759115,"lng":-8.54295527884163},{"lat":40.8170663905283,"lng":-8.54288454268935},{"lat":40.8171039140584,"lng":-8.54284139321534},{"lat":40.8172074439092,"lng":-8.5429999132626},{"lat":40.8173095070765,"lng":-8.54324115579426},{"lat":40.8174115448607,"lng":-8.54348939214364},{"lat":40.8175101711457,"lng":-8.54370951789422},{"lat":40.817589440466,"lng":-8.54387820032802},{"lat":40.8176476510189,"lng":-8.54399294067271},{"lat":40.8177029246532,"lng":-8.54410185239415},{"lat":40.8177887047595,"lng":-8.54395695290114},{"lat":40.8178570118696,"lng":-8.54381668256004},{"lat":40.8179164428126,"lng":-8.54369105496156},{"lat":40.8179599198563,"lng":-8.54364509737104},{"lat":40.818015757983,"lng":-8.54361758932783},{"lat":40.8180596850371,"lng":-8.54359699993285},{"lat":40.8181086407406,"lng":-8.54358011595771},{"lat":40.8182206815426,"lng":-8.54354892535541},{"lat":40.8182760953322,"lng":-8.54343868110259},{"lat":40.8182754591177,"lng":-8.54331422088614},{"lat":40.8182182384187,"lng":-8.5431994849576},{"lat":40.8181534821243,"lng":-8.54320287795783},{"lat":40.8181071317341,"lng":-8.54317082511452},{"lat":40.8180679811105,"lng":-8.54308950911854},{"lat":40.8180500106929,"lng":-8.54302799966241},{"lat":40.8180404019723,"lng":-8.54297009762042},{"lat":40.8180281004298,"lng":-8.54285955266858},{"lat":40.8180239945866,"lng":-8.54279883979413},{"lat":40.8180208741733,"lng":-8.54273955654005},{"lat":40.8180153667147,"lng":-8.54259219046805},{"lat":40.8180037842337,"lng":-8.54245628453042},{"lat":40.817982407858,"lng":-8.54226401650709},{"lat":40.8179777411916,"lng":-8.54218386246352},{"lat":40.8179835652284,"lng":-8.54209073451835},{"lat":40.8179903777614,"lng":-8.54202356960534},{"lat":40.8180090611656,"lng":-8.54185964054428},{"lat":40.8180174770503,"lng":-8.54177221789526},{"lat":40.8180218006108,"lng":-8.54161981659343},{"lat":40.8180205139042,"lng":-8.54152593374127},{"lat":40.8180237907995,"lng":-8.54133891391002},{"lat":40.8180238511265,"lng":-8.54116989216974},{"lat":40.8180290779017,"lng":-8.5410926434977},{"lat":40.8181531108464,"lng":-8.5389984463611},{"lat":40.818168927157,"lng":-8.538811254692},{"lat":40.818314204493,"lng":-8.53723720279367},{"lat":40.8182997873174,"lng":-8.53709819697498},{"lat":40.8182925791127,"lng":-8.53702869527572},{"lat":40.8182853708744,"lng":-8.5369591912206},{"lat":40.8182709533631,"lng":-8.53682018551962},{"lat":40.8182559175265,"lng":-8.53668130058857},{"lat":40.8182365459352,"lng":-8.53654336970222},{"lat":40.8182140069284,"lng":-8.53640627787595},{"lat":40.8181902147881,"lng":-8.53626955548322},{"lat":40.818178300631,"lng":-8.53620119895514},{"lat":40.8181667713438,"lng":-8.53613272743535},{"lat":40.8181558838126,"lng":-8.53606407017347},{"lat":40.8181457569571,"lng":-8.53599521129279},{"lat":40.8181368224247,"lng":-8.53592606094821},{"lat":40.8181291017063,"lng":-8.53585665364001},{"lat":40.8181230496543,"lng":-8.53578695892641},{"lat":40.8181187074791,"lng":-8.53571703750209},{"lat":40.8181149471883,"lng":-8.53564705559772},{"lat":40.8181111877596,"lng":-8.53557707252107},{"lat":40.8181036714637,"lng":-8.53543710996263},{"lat":40.8180999137126,"lng":-8.53536712573425},{"lat":40.8180961568113,"lng":-8.53529714388944},{"lat":40.8180923989711,"lng":-8.53522716086146},{"lat":40.8180886419847,"lng":-8.53515717903175},{"lat":40.8180581927819,"lng":-8.53495326011266},{"lat":40.8180407623413,"lng":-8.53485180539335},{"lat":40.8180272629454,"lng":-8.53474943093428},{"lat":40.8180263141782,"lng":-8.53454181902346},{"lat":40.8180163759216,"lng":-8.53433459151653},{"lat":40.8180119570417,"lng":-8.53423131180057},{"lat":40.8180398525214,"lng":-8.53413413189203},{"lat":40.818068787815,"lng":-8.5340375188122},{"lat":40.8181019973329,"lng":-8.53394332925146},{"lat":40.8181511926444,"lng":-8.53374868143641},{"lat":40.8181586816148,"lng":-8.53364583113705},{"lat":40.818177451834,"lng":-8.53354479224102},{"lat":40.818199382104,"lng":-8.53344495759411},{"lat":40.8182279937015,"lng":-8.53334803978339},{"lat":40.8182552575714,"lng":-8.53325049150343},{"lat":40.8182955068618,"lng":-8.53316112274784},{"lat":40.8183581118007,"lng":-8.53297038167909},{"lat":40.8183955789161,"lng":-8.53287903422246},{"lat":40.8184367926554,"lng":-8.53279035476817},{"lat":40.8185140437244,"lng":-8.53260915131987},{"lat":40.818550830811,"lng":-8.53251724701077},{"lat":40.818601916065,"lng":-8.53243801046517},{"lat":40.8186849177243,"lng":-8.53226339424641},{"lat":40.8187543548987,"lng":-8.53229526763233},{"lat":40.8188127770987,"lng":-8.53232667440942},{"lat":40.8189248995668,"lng":-8.53240060183499},{"lat":40.8190266463586,"lng":-8.53245455385667},{"lat":40.8191716132402,"lng":-8.53253496216208},{"lat":40.8193288874375,"lng":-8.53262409755745},{"lat":40.8194985308204,"lng":-8.53272930958039},{"lat":40.8196406769605,"lng":-8.532818355067},{"lat":40.8197338818066,"lng":-8.53286858249229},{"lat":40.8197988936159,"lng":-8.53289505139171},{"lat":40.8198601881041,"lng":-8.53292908418471},{"lat":40.8199260744877,"lng":-8.5329630260816},{"lat":40.8199976092448,"lng":-8.53300435226282},{"lat":40.8201746879607,"lng":-8.53309491273307},{"lat":40.8202912817895,"lng":-8.53315168352342},{"lat":40.8203629167993,"lng":-8.53319004618728},{"lat":40.8204829015329,"lng":-8.53325572661591},{"lat":40.8206075267239,"lng":-8.53333352694126},{"lat":40.820761387844,"lng":-8.53341968226706},{"lat":40.8208351739341,"lng":-8.53346066620916},{"lat":40.8209213581008,"lng":-8.53350990494341},{"lat":40.8209884515758,"lng":-8.53356284319974},{"lat":40.8211024977433,"lng":-8.53364944782317},{"lat":40.8212180663508,"lng":-8.53369044772095},{"lat":40.8212769030582,"lng":-8.53370609525282},{"lat":40.8213285199589,"lng":-8.5337263211067},{"lat":40.8213725655025,"lng":-8.53374863509067},{"lat":40.8214345598423,"lng":-8.53378824491587},{"lat":40.8215047973936,"lng":-8.53383999517008},{"lat":40.8216520056064,"lng":-8.53394887189107},{"lat":40.8217992384304,"lng":-8.53405063582511},{"lat":40.8219479955456,"lng":-8.53412834746747},{"lat":40.8219968048395,"lng":-8.53415317991113},{"lat":40.8220737456573,"lng":-8.53419347440049},{"lat":40.8222079629488,"lng":-8.53425758575476},{"lat":40.8222610093635,"lng":-8.53428102146537},{"lat":40.8223209612811,"lng":-8.53431232234271},{"lat":40.8225042006617,"lng":-8.53438872098042},{"lat":40.822653185809,"lng":-8.53442824833053},{"lat":40.8227811081621,"lng":-8.53447900966303},{"lat":40.8229421146422,"lng":-8.53455584660053},{"lat":40.8230230415787,"lng":-8.53458817306124},{"lat":40.8231761323486,"lng":-8.53465786346296},{"lat":40.8232824729178,"lng":-8.53471499855889},{"lat":40.823417965124,"lng":-8.53479432853547},{"lat":40.8235429771637,"lng":-8.53486725961334},{"lat":40.8236287165134,"lng":-8.53494888358859},{"lat":40.8237459630832,"lng":-8.5349849803935},{"lat":40.8239114126809,"lng":-8.53501386112037},{"lat":40.8240343810714,"lng":-8.53503168045751},{"lat":40.8254696275392,"lng":-8.5230536167688},{"lat":40.8263399867888,"lng":-8.5192948928439},{"lat":40.8273051079384,"lng":-8.51566207743413},{"lat":40.8295135396835,"lng":-8.51065610003284},{"lat":40.8271574616408,"lng":-8.50704629295659},{"lat":40.8241941985688,"lng":-8.50386904209243},{"lat":40.8215981589263,"lng":-8.50065500167934},{"lat":40.818407645363,"lng":-8.49736776348237},{"lat":40.814468457276,"lng":-8.49875908293634},{"lat":40.7986919513163,"lng":-8.50432958076185},{"lat":40.7905124515058,"lng":-8.50792013216861},{"lat":40.7876739667617,"lng":-8.50881482103287},{"lat":40.7826099787604,"lng":-8.51177586715368},{"lat":40.7766569189472,"lng":-8.51692661324558}]},{"zone_id":"arc3","points":[{"lat":40.9197065308886,"lng":-8.17412610953536},{"lat":40.9213383045184,"lng":-8.18117262190087},{"lat":40.9218705797431,"lng":-8.18347163171172},{"lat":40.9230363971344,"lng":-8.1885078664144},{"lat":40.9270836252801,"lng":-8.20600032302676},{"lat":40.9274631286673,"lng":-8.20863943640618},{"lat":40.9275568405932,"lng":-8.21102693840875},{"lat":40.9275748409756,"lng":-8.22269508592275},{"lat":40.9261429788262,"lng":-8.22567733256258},{"lat":40.9315470712997,"lng":-8.22773708770907},{"lat":40.9336190742415,"lng":-8.22868252305134},{"lat":40.9355479208722,"lng":-8.22986343582352},{"lat":40.9415804723515,"lng":-8.23448925232643},{"lat":40.9424132996048,"lng":-8.23047458262167},{"lat":40.9428447245263,"lng":-8.22830536596784},{"lat":40.943117749188,"lng":-8.22693246962648},{"lat":40.9448534415178,"lng":-8.21820240424942},{"lat":40.9467661788154,"lng":-8.20852682293083},{"lat":40.9474367638678,"lng":-8.20299722315581},{"lat":40.948043499544,"lng":-8.19744801418193},{"lat":40.9483307081913,"lng":-8.19099182816484},{"lat":40.9482787171396,"lng":-8.18722978452341},{"lat":40.9482012440066,"lng":-8.18164028413297},{"lat":40.9477331919919,"lng":-8.17665654125559},{"lat":40.9467756902433,"lng":-8.17022069650979},{"lat":40.9419180928716,"lng":-8.16922329951215},{"lat":40.9352521208278,"lng":-8.1692196690494},{"lat":40.9292209039514,"lng":-8.17012394009496},{"lat":40.923562028839,"lng":-8.17246509904844},{"lat":40.9197065308886,"lng":-8.17412610953536}]},{"zone_id":"smf2","points":[{"lat":40.9272169790499,"lng":-8.58434435565064},{"lat":40.9277707863242,"lng":-8.59173676655298},{"lat":40.9276340568357,"lng":-8.59531645781632},{"lat":40.9287333037935,"lng":-8.59502315802628},{"lat":40.9289753314543,"lng":-8.59495318918016},{"lat":40.9295966059709,"lng":-8.59477357981659},{"lat":40.9309432246909,"lng":-8.59438426302661},{"lat":40.931052070614,"lng":-8.59432550280823},{"lat":40.9313132474898,"lng":-8.59429713103174},{"lat":40.9321240585766,"lng":-8.59305184350355},{"lat":40.9322211842241,"lng":-8.59312767822176},{"lat":40.9323921558645,"lng":-8.593223974051},{"lat":40.9325050936004,"lng":-8.59326394360786},{"lat":40.9328524107537,"lng":-8.59339625822867},{"lat":40.9330030051774,"lng":-8.59344878028089},{"lat":40.9336399311477,"lng":-8.59367092049944},{"lat":40.9342126293667,"lng":-8.59389374580974},{"lat":40.9347207955369,"lng":-8.59410876341797},{"lat":40.9351025183607,"lng":-8.59426115690331},{"lat":40.9352227165519,"lng":-8.59430913476283},{"lat":40.9353518214663,"lng":-8.59435931325347},{"lat":40.9354847068462,"lng":-8.59440963698},{"lat":40.9356159908778,"lng":-8.59445508141702},{"lat":40.9357149167281,"lng":-8.59448130248015},{"lat":40.9358310072285,"lng":-8.59450312959328},{"lat":40.9359374355776,"lng":-8.59454946898069},{"lat":40.9360444198539,"lng":-8.59459189393154},{"lat":40.9362395460372,"lng":-8.59465987055401},{"lat":40.9364905356093,"lng":-8.59474153793253},{"lat":40.9366940621501,"lng":-8.59482548541426},{"lat":40.9368763778243,"lng":-8.59492163492764},{"lat":40.9369965842006,"lng":-8.59496771546981},{"lat":40.9370817409231,"lng":-8.59501913294932},{"lat":40.9371877801986,"lng":-8.59511712460747},{"lat":40.9375344687968,"lng":-8.59544965370729},{"lat":40.9377498075457,"lng":-8.59566371072781},{"lat":40.9378746976841,"lng":-8.59577612703092},{"lat":40.9380093787145,"lng":-8.59589482320908},{"lat":40.9381160113021,"lng":-8.59600184520909},{"lat":40.9383247118508,"lng":-8.59620981127092},{"lat":40.938451829483,"lng":-8.59632765913385},{"lat":40.9385807527058,"lng":-8.59642237120585},{"lat":40.9387104630956,"lng":-8.59647826044381},{"lat":40.9388224405564,"lng":-8.59703790794917},{"lat":40.9388115413204,"lng":-8.59728600674144},{"lat":40.9388003802139,"lng":-8.59757776498809},{"lat":40.9388359044732,"lng":-8.59809815577127},{"lat":40.9387868920992,"lng":-8.59871304247993},{"lat":40.9388729315699,"lng":-8.59884395089666},{"lat":40.9395757271347,"lng":-8.59997819954652},{"lat":40.9410332647656,"lng":-8.60233347434329},{"lat":40.9412663985082,"lng":-8.60271021884821},{"lat":40.9429992164828,"lng":-8.6030819539765},{"lat":40.9436820440416,"lng":-8.60322844390561},{"lat":40.9454126184853,"lng":-8.6035997251128},{"lat":40.947279009122,"lng":-8.60400016725394},{"lat":40.9473248517512,"lng":-8.60387299545747},{"lat":40.9536824514557,"lng":-8.58365339315088},{"lat":40.9553805829636,"lng":-8.57580840109876},{"lat":40.956842307983,"lng":-8.56733291775678},{"lat":40.9571231608013,"lng":-8.56380464700042},{"lat":40.957342294188,"lng":-8.56105077641671},{"lat":40.9577178839103,"lng":-8.55345480539983},{"lat":40.9552619987416,"lng":-8.55119024884096},{"lat":40.949920732943,"lng":-8.54687121805273},{"lat":40.9491077321028,"lng":-8.54622036392269},{"lat":40.9466561273343,"lng":-8.54443725992854},{"lat":40.9432516953147,"lng":-8.54268521341851},{"lat":40.9406799939163,"lng":-8.5422922960675},{"lat":40.9378572350451,"lng":-8.54187898206781},{"lat":40.9365457127924,"lng":-8.54359693741901},{"lat":40.9358313818571,"lng":-8.54455491361522},{"lat":40.9341769294374,"lng":-8.54772494728468},{"lat":40.9290773273199,"lng":-8.5574397844966},{"lat":40.9266314910541,"lng":-8.56140422814039},{"lat":40.9262429968746,"lng":-8.56363957386519},{"lat":40.9257017114828,"lng":-8.56682714687722},{"lat":40.9248913854232,"lng":-8.57742169089595},{"lat":40.9272169790499,"lng":-8.58434435565064}]},{"zone_id":"smf3","points":[{"lat":40.9914241159369,"lng":-8.51544913626225},{"lat":40.9835986055494,"lng":-8.5084579465327},{"lat":40.9792795532291,"lng":-8.50286134981157},{"lat":40.9785126233742,"lng":-8.50510140816833},{"lat":40.9753557024359,"lng":-8.50895811121971},{"lat":40.9735264729577,"lng":-8.51072893480791},{"lat":40.9673548320533,"lng":-8.51079289925682},{"lat":40.9667606050096,"lng":-8.51102630944083},{"lat":40.9625147850796,"lng":-8.51202724556394},{"lat":40.9606440302156,"lng":-8.512247814076},{"lat":40.9589413751449,"lng":-8.51192823982049},{"lat":40.9570967846545,"lng":-8.5113677218551},{"lat":40.9551607349688,"lng":-8.51039582101501},{"lat":40.9521797857876,"lng":-8.50833803841344},{"lat":40.9491554447494,"lng":-8.5062392273982},{"lat":40.947267200764,"lng":-8.50492894885178},{"lat":40.9422157484565,"lng":-8.51687864111915},{"lat":40.9389479963764,"lng":-8.528139659891},{"lat":40.9320301476711,"lng":-8.53341384234223},{"lat":40.9320687480561,"lng":-8.53658849047905},{"lat":40.9336131709664,"lng":-8.54158318334398},{"lat":40.9359746821409,"lng":-8.54137961869191},{"lat":40.9378572350451,"lng":-8.54187898206781},{"lat":40.9406799939163,"lng":-8.5422922960675},{"lat":40.9432516953147,"lng":-8.54268521341851},{"lat":40.9466561273343,"lng":-8.54443725992854},{"lat":40.9491077321028,"lng":-8.54622036392269},{"lat":40.949920732943,"lng":-8.54687121805273},{"lat":40.9552619987416,"lng":-8.55119024884096},{"lat":40.9577178839103,"lng":-8.55345480539983},{"lat":40.960392332284,"lng":-8.55542086635219},{"lat":40.9630516874593,"lng":-8.5566086405807},{"lat":40.966169586604,"lng":-8.55754000247438},{"lat":40.9698479737872,"lng":-8.56094630975853},{"lat":40.9730962116834,"lng":-8.56363891989827},{"lat":40.9803753454449,"lng":-8.56295713284177},{"lat":40.9829114237997,"lng":-8.56272007323348},{"lat":40.9858552026253,"lng":-8.56277133075019},{"lat":40.9901597490253,"lng":-8.56402986436183},{"lat":40.9907594282854,"lng":-8.56232891770115},{"lat":40.994399623694,"lng":-8.55200157984578},{"lat":40.9946918993421,"lng":-8.55024251958851},{"lat":40.994696443785,"lng":-8.54898475499451},{"lat":40.9946750779174,"lng":-8.54657628525363},{"lat":40.9935011331963,"lng":-8.54172653363691},{"lat":40.9926908463786,"lng":-8.53590100566014},{"lat":40.992827572168,"lng":-8.53083150056372},{"lat":40.9927618899989,"lng":-8.52391344255855},{"lat":40.9921508145086,"lng":-8.5196649431697},{"lat":40.9914241159369,"lng":-8.51544913626225}]},{"zone_id":"prt1","points":[{"lat":41.174721702463,"lng":-8.61422669860832},{"lat":41.1745277321604,"lng":-8.61412141860032},{"lat":41.1746483783905,"lng":-8.61289903041225},{"lat":41.1745170063708,"lng":-8.61115490900746},{"lat":41.1725082818488,"lng":-8.61077330006321},{"lat":41.1723458393663,"lng":-8.61025119389457},{"lat":41.1723134149919,"lng":-8.60897833628369},{"lat":41.1724775191063,"lng":-8.607042216504},{"lat":41.1712370785264,"lng":-8.60525960062148},{"lat":41.171049025268,"lng":-8.60463731360903},{"lat":41.1710569971132,"lng":-8.60409625847338},{"lat":41.1711289469366,"lng":-8.60345459103232},{"lat":41.1711785367835,"lng":-8.60262928078039},{"lat":41.1710251468455,"lng":-8.60057931317141},{"lat":41.1713732349904,"lng":-8.60021483010537},{"lat":41.1715398222715,"lng":-8.59902338132957},{"lat":41.1721056270616,"lng":-8.59624456107593},{"lat":41.1721315666511,"lng":-8.59554138898274},{"lat":41.1719958801181,"lng":-8.59480649798753},{"lat":41.1715376806776,"lng":-8.59382470043685},{"lat":41.1677828737511,"lng":-8.58800367702316},{"lat":41.1671043257911,"lng":-8.5866043735008},{"lat":41.1668904192221,"lng":-8.58579585593893},{"lat":41.1667779673253,"lng":-8.58473404687678},{"lat":41.1665984497941,"lng":-8.58365337444105},{"lat":41.1663008219487,"lng":-8.58294803731663},{"lat":41.1658864616144,"lng":-8.5825476850618},{"lat":41.1649324995256,"lng":-8.58180675394962},{"lat":41.1646430139813,"lng":-8.58143838774917},{"lat":41.1644941129044,"lng":-8.58105822831818},{"lat":41.1643702853024,"lng":-8.58086812335464},{"lat":41.1642237407458,"lng":-8.58064314399528},{"lat":41.1639119315604,"lng":-8.58052104515807},{"lat":41.1634663077999,"lng":-8.58068303593998},{"lat":41.1626585999988,"lng":-8.58072157326528},{"lat":41.1621333936855,"lng":-8.5808690317075},{"lat":41.1575671748825,"lng":-8.58273529339255},{"lat":41.1571028900387,"lng":-8.5832824385722},{"lat":41.156448242708,"lng":-8.58337074250812},{"lat":41.155508422363,"lng":-8.58330207395416},{"lat":41.1545850009014,"lng":-8.58308173194048},{"lat":41.1542142943361,"lng":-8.58313459417344},{"lat":41.1538167165231,"lng":-8.58348926434303},{"lat":41.1533301252313,"lng":-8.58358123022646},{"lat":41.1528963094903,"lng":-8.58355930191325},{"lat":41.1525744956918,"lng":-8.58312909115328},{"lat":41.152507091094,"lng":-8.5826394842097},{"lat":41.1525558142985,"lng":-8.58199781491123},{"lat":41.1524426690656,"lng":-8.58138561068154},{"lat":41.1517312508672,"lng":-8.58012733299472},{"lat":41.1510944631693,"lng":-8.57963468914298},{"lat":41.1505270697682,"lng":-8.57966656702252},{"lat":41.1500399716234,"lng":-8.57988959116522},{"lat":41.1498129335994,"lng":-8.58023654969232},{"lat":41.149556976296,"lng":-8.5806933588394},{"lat":41.1491793916882,"lng":-8.58121791930717},{"lat":41.1489171590819,"lng":-8.58150202493903},{"lat":41.1487365202937,"lng":-8.58164373851915},{"lat":41.1484029987078,"lng":-8.58173675742495},{"lat":41.1480046657703,"lng":-8.58175270539899},{"lat":41.1474509444263,"lng":-8.58153493662397},{"lat":41.1469898872302,"lng":-8.58125666625919},{"lat":41.1467373838581,"lng":-8.58082697831668},{"lat":41.1460491835906,"lng":-8.57953840563627},{"lat":41.1457268793388,"lng":-8.57923052983796},{"lat":41.1423263415948,"lng":-8.58036874605719},{"lat":41.1424810784184,"lng":-8.58099353239148},{"lat":41.1425595971982,"lng":-8.58311273905078},{"lat":41.1419816368873,"lng":-8.58485497934995},{"lat":41.1412904725998,"lng":-8.58610711677605},{"lat":41.1400219052563,"lng":-8.58812318652882},{"lat":41.1388381245256,"lng":-8.59144992371173},{"lat":41.1384574669531,"lng":-8.59329286596379},{"lat":41.138640294295,"lng":-8.5948394047351},{"lat":41.1392692237691,"lng":-8.59621716315491},{"lat":41.1405216100516,"lng":-8.59825083175782},{"lat":41.1412332337315,"lng":-8.60039983996631},{"lat":41.1411300703752,"lng":-8.6036150518287},{"lat":41.1406681290033,"lng":-8.60647034050454},{"lat":41.1394778823971,"lng":-8.61122600543434},{"lat":41.1390147556855,"lng":-8.61431937056088},{"lat":41.1390062591768,"lng":-8.61634409733114},{"lat":41.1392145604282,"lng":-8.61771464244233},{"lat":41.1395354513104,"lng":-8.61896832719624},{"lat":41.1404287898021,"lng":-8.62064244523597},{"lat":41.1425796187829,"lng":-8.62304064632388},{"lat":41.1434704291305,"lng":-8.62514907318385},{"lat":41.143821304433,"lng":-8.6274571253132},{"lat":41.1440391458299,"lng":-8.62933249702638},{"lat":41.1445047298682,"lng":-8.63081446301544},{"lat":41.1454172846944,"lng":-8.63318674648564},{"lat":41.1465779310471,"lng":-8.63545884434667},{"lat":41.1470319333398,"lng":-8.63646001195468},{"lat":41.1474099871313,"lng":-8.63801387932725},{"lat":41.1474595034018,"lng":-8.63975392586331},{"lat":41.1472762286378,"lng":-8.6409798385859},{"lat":41.1490127339192,"lng":-8.64147790703128},{"lat":41.1489170179747,"lng":-8.64204890993824},{"lat":41.1492733605763,"lng":-8.64320713490876},{"lat":41.1497319563349,"lng":-8.64400552338424},{"lat":41.1503990475858,"lng":-8.64468326571614},{"lat":41.1511093401895,"lng":-8.64504033050109},{"lat":41.1520319748163,"lng":-8.64506659978941},{"lat":41.153350400267,"lng":-8.64502481801633},{"lat":41.1546155756239,"lng":-8.64536514886301},{"lat":41.1554395827748,"lng":-8.64574176103912},{"lat":41.1568659853527,"lng":-8.64643196738166},{"lat":41.1575485592863,"lng":-8.64690174257265},{"lat":41.1594375374851,"lng":-8.64844957466514},{"lat":41.1608101241125,"lng":-8.64938617130213},{"lat":41.1614560885526,"lng":-8.64962022082206},{"lat":41.1622091472339,"lng":-8.64953094683466},{"lat":41.1632035226709,"lng":-8.64917602360922},{"lat":41.1640814348515,"lng":-8.64841160561936},{"lat":41.164776402811,"lng":-8.64725182849186},{"lat":41.1657255626675,"lng":-8.64503557773734},{"lat":41.1661735099267,"lng":-8.64407112778748},{"lat":41.1663398677919,"lng":-8.64385568717986},{"lat":41.1665748918225,"lng":-8.64380896366973},{"lat":41.1669193348459,"lng":-8.64390875032216},{"lat":41.1673296017001,"lng":-8.64405760382876},{"lat":41.1675132614777,"lng":-8.64401048046034},{"lat":41.1676826773541,"lng":-8.64386613881409},{"lat":41.1683942436923,"lng":-8.64268126259962},{"lat":41.1684744112083,"lng":-8.64238654455038},{"lat":41.1685306332801,"lng":-8.64128965615944},{"lat":41.1686357654205,"lng":-8.64074666254564},{"lat":41.1687902512585,"lng":-8.64027909492182},{"lat":41.1692873626092,"lng":-8.63959011289681},{"lat":41.1702066970498,"lng":-8.63835476621461},{"lat":41.1708670142514,"lng":-8.63711416191875},{"lat":41.1712805824519,"lng":-8.63585545612138},{"lat":41.1726752111672,"lng":-8.62963669984453},{"lat":41.1731264102285,"lng":-8.62704017934647},{"lat":41.1739880924196,"lng":-8.61881172376012},{"lat":41.174721702463,"lng":-8.61422669860832}]},{"zone_id":"oaz1","points":[{"lat":40.8125450846298,"lng":-8.47283205689006},{"lat":40.814468457276,"lng":-8.49875908293634},{"lat":40.818407645363,"lng":-8.49736776348237},{"lat":40.8215981589263,"lng":-8.50065500167934},{"lat":40.8241941985688,"lng":-8.50386904209243},{"lat":40.8271574616408,"lng":-8.50704629295659},{"lat":40.8295135396835,"lng":-8.51065610003284},{"lat":40.8304726832615,"lng":-8.50877951215725},{"lat":40.8320011548525,"lng":-8.507658896244},{"lat":40.836018494418,"lng":-8.50512728996131},{"lat":40.8477672418179,"lng":-8.49772163073552},{"lat":40.8468514563555,"lng":-8.49381182001714},{"lat":40.8466172968103,"lng":-8.4899512544343},{"lat":40.8464668772747,"lng":-8.48382146742838},{"lat":40.8464102023942,"lng":-8.47750123694784},{"lat":40.8459016576655,"lng":-8.46967572789757},{"lat":40.8458821437066,"lng":-8.46421295294657},{"lat":40.8429071817939,"lng":-8.46433667941354},{"lat":40.8385465636364,"lng":-8.46111198295236},{"lat":40.8344043632707,"lng":-8.45866030422937},{"lat":40.8302532394474,"lng":-8.45815374107196},{"lat":40.8277736560988,"lng":-8.45876895100143},{"lat":40.8256267332471,"lng":-8.45969939486745},{"lat":40.8236223970414,"lng":-8.46075593634388},{"lat":40.8217586756107,"lng":-8.46262854822379},{"lat":40.8204196174302,"lng":-8.46431549224465},{"lat":40.8188801963111,"lng":-8.46705510537878},{"lat":40.8179605056403,"lng":-8.4687063679188},{"lat":40.8170418907556,"lng":-8.46998126037913},{"lat":40.8160670910481,"lng":-8.47093821062699},{"lat":40.8149781087828,"lng":-8.47177714874845},{"lat":40.8125450846298,"lng":-8.47283205689006}]},{"zone_id":"prd3","points":[{"lat":41.2090368861198,"lng":-8.37530784271146},{"lat":41.2072638101314,"lng":-8.37603835372978},{"lat":41.2053158999884,"lng":-8.37737623935821},{"lat":41.2029507684916,"lng":-8.38049022604812},{"lat":41.1997464301978,"lng":-8.38629873771804},{"lat":41.1984858905525,"lng":-8.39070979361794},{"lat":41.1979425794121,"lng":-8.39394071620701},{"lat":41.1976346970579,"lng":-8.39835535004097},{"lat":41.1978665449368,"lng":-8.40111620571427},{"lat":41.1983374901027,"lng":-8.4034837891075},{"lat":41.1992959886587,"lng":-8.40502273916952},{"lat":41.2009037729187,"lng":-8.40632431889824},{"lat":41.2044752666686,"lng":-8.40720666396524},{"lat":41.209024537881,"lng":-8.40766731137218},{"lat":41.2135510337712,"lng":-8.40768623638237},{"lat":41.2147521334127,"lng":-8.4075725826378},{"lat":41.2199611391176,"lng":-8.40442187452394},{"lat":41.2242809269719,"lng":-8.40233344810619},{"lat":41.2303674164475,"lng":-8.40217681612486},{"lat":41.2435902216245,"lng":-8.40146316253468},{"lat":41.2433203443349,"lng":-8.40042032134652},{"lat":41.242976817959,"lng":-8.39907852505217},{"lat":41.2427590651304,"lng":-8.39818915447021},{"lat":41.2428054935783,"lng":-8.39811655057833},{"lat":41.2436302994779,"lng":-8.39699674703635},{"lat":41.2438071604925,"lng":-8.39654491496145},{"lat":41.2438828050726,"lng":-8.39635166138345},{"lat":41.2439231285703,"lng":-8.39624864409408},{"lat":41.244138377826,"lng":-8.39575717534535},{"lat":41.2442918697878,"lng":-8.39542537577639},{"lat":41.2447519396606,"lng":-8.39443083303916},{"lat":41.2448489127243,"lng":-8.39422120017223},{"lat":41.2449205619995,"lng":-8.39406631042775},{"lat":41.2450411922023,"lng":-8.39380553306522},{"lat":41.2453776750276,"lng":-8.39313224248457},{"lat":41.2456636668919,"lng":-8.3926538900188},{"lat":41.2458361975284,"lng":-8.39236531015579},{"lat":41.2458851720478,"lng":-8.39229070939356},{"lat":41.2460297058781,"lng":-8.392124024239},{"lat":41.2461573951804,"lng":-8.39184743109563},{"lat":41.2461984346113,"lng":-8.39175853333161},{"lat":41.246308037203,"lng":-8.39152111586658},{"lat":41.2463902319235,"lng":-8.39108885822883},{"lat":41.247005849019,"lng":-8.38951336194289},{"lat":41.2468267683632,"lng":-8.38738700914673},{"lat":41.2468316535556,"lng":-8.38717431436088},{"lat":41.2468747757743,"lng":-8.38677244171872},{"lat":41.2469045031626,"lng":-8.38635893613116},{"lat":41.2469252498908,"lng":-8.38597666116274},{"lat":41.2469594944998,"lng":-8.38551676714911},{"lat":41.2469699392437,"lng":-8.38531220813048},{"lat":41.2470236348372,"lng":-8.38513632737652},{"lat":41.2471497928469,"lng":-8.38476268646796},{"lat":41.2475684752545,"lng":-8.38407175160837},{"lat":41.2477740960372,"lng":-8.38378622721751},{"lat":41.243501197822,"lng":-8.37553575373298},{"lat":41.2415077314181,"lng":-8.37183552493368},{"lat":41.2394419002814,"lng":-8.36851400508938},{"lat":41.2366616445204,"lng":-8.36500080399247},{"lat":41.2338092448124,"lng":-8.36177168296807},{"lat":41.2315104904519,"lng":-8.36018067824596},{"lat":41.2298689042738,"lng":-8.35903262304606},{"lat":41.2272978742477,"lng":-8.3580392228488},{"lat":41.2254674746075,"lng":-8.35841161286455},{"lat":41.2233780173449,"lng":-8.35940715264384},{"lat":41.2209328094542,"lng":-8.36355112479097},{"lat":41.2195542121522,"lng":-8.36549961597103},{"lat":41.2179277394045,"lng":-8.36860142928677},{"lat":41.2154435784982,"lng":-8.37174755885022},{"lat":41.2132481219829,"lng":-8.37350634298991},{"lat":41.2114354073136,"lng":-8.37450925804255},{"lat":41.2090368861198,"lng":-8.37530784271146}]},{"zone_id":"smf8","points":[{"lat":40.9302989925647,"lng":-8.49315832873953},{"lat":40.947267200764,"lng":-8.50492894885178},{"lat":40.9491554447494,"lng":-8.5062392273982},{"lat":40.9521797857876,"lng":-8.50833803841344},{"lat":40.9551607349688,"lng":-8.51039582101501},{"lat":40.9570967846545,"lng":-8.5113677218551},{"lat":40.9589413751449,"lng":-8.51192823982049},{"lat":40.9606440302156,"lng":-8.512247814076},{"lat":40.9625147850796,"lng":-8.51202724556394},{"lat":40.9667606050096,"lng":-8.51102630944083},{"lat":40.9673548320533,"lng":-8.51079289925682},{"lat":40.9735264729577,"lng":-8.51072893480791},{"lat":40.9753557024359,"lng":-8.50895811121971},{"lat":40.9785126233742,"lng":-8.50510140816833},{"lat":40.9792795532291,"lng":-8.50286134981157},{"lat":40.9795685514252,"lng":-8.49900380606854},{"lat":40.9812162457886,"lng":-8.49573029030672},{"lat":40.981074928352,"lng":-8.49206319231294},{"lat":40.9806918895894,"lng":-8.49026629575356},{"lat":40.9804891329965,"lng":-8.48841821281277},{"lat":40.9804699586402,"lng":-8.48495991825015},{"lat":40.9807838185838,"lng":-8.48089753652779},{"lat":40.9813314226969,"lng":-8.47704919455157},{"lat":40.9816720302293,"lng":-8.47517643895761},{"lat":40.9821498807099,"lng":-8.47298608326142},{"lat":40.9788419221188,"lng":-8.47199170203429},{"lat":40.9763472602788,"lng":-8.47119065079732},{"lat":40.9727184928219,"lng":-8.47089025658346},{"lat":40.9694675524778,"lng":-8.47038959956035},{"lat":40.9659811927095,"lng":-8.47001270225091},{"lat":40.963699165782,"lng":-8.46824100818505},{"lat":40.961418563104,"lng":-8.46596657865386},{"lat":40.9608056366624,"lng":-8.46517731235596},{"lat":40.9575378250426,"lng":-8.4693850691727},{"lat":40.9533627777479,"lng":-8.47132796170453},{"lat":40.9452819413368,"lng":-8.47274782593915},{"lat":40.9406112027029,"lng":-8.47310092910109},{"lat":40.9387945836652,"lng":-8.47510238808737},{"lat":40.9372611977982,"lng":-8.47798490022926},{"lat":40.9346722304555,"lng":-8.48324938910821},{"lat":40.933271047189,"lng":-8.48628405897639},{"lat":40.9302989925647,"lng":-8.49315832873953}]},{"zone_id":"oaz6","points":[{"lat":40.814468457276,"lng":-8.49875908293634},{"lat":40.8125450846298,"lng":-8.47283205689006},{"lat":40.8084238049512,"lng":-8.47132535938091},{"lat":40.8060826970132,"lng":-8.47011834628249},{"lat":40.800486543406,"lng":-8.46634643034995},{"lat":40.7980309393129,"lng":-8.46453591070233},{"lat":40.7955181341139,"lng":-8.46280082937337},{"lat":40.7903780090518,"lng":-8.46000961158329},{"lat":40.785580199969,"lng":-8.45714295547456},{"lat":40.7809676936262,"lng":-8.45427158447092},{"lat":40.7806646823418,"lng":-8.45402106659463},{"lat":40.7806412979911,"lng":-8.45414030671349},{"lat":40.7806128356862,"lng":-8.45425725844175},{"lat":40.7805580604024,"lng":-8.45435655298451},{"lat":40.7805115968023,"lng":-8.45447388683332},{"lat":40.7804737282265,"lng":-8.45458220184018},{"lat":40.7804182502554,"lng":-8.45469507557781},{"lat":40.7803441151087,"lng":-8.45478781032689},{"lat":40.7802605338734,"lng":-8.45486562145554},{"lat":40.7801701303595,"lng":-8.4549289267951},{"lat":40.7800728247333,"lng":-8.45496955204123},{"lat":40.7799708197339,"lng":-8.45496518356365},{"lat":40.7798692757758,"lng":-8.45494725807276},{"lat":40.779767793827,"lng":-8.45492872996197},{"lat":40.7796658193218,"lng":-8.45491568434281},{"lat":40.7795637961448,"lng":-8.45490329954914},{"lat":40.7794617411213,"lng":-8.45489167753653},{"lat":40.7793594890742,"lng":-8.45489904168358},{"lat":40.7792571652484,"lng":-8.45490275921091},{"lat":40.7791566369153,"lng":-8.4548793837609},{"lat":40.7790594008382,"lng":-8.45483725761742},{"lat":40.7789577575695,"lng":-8.45482684016648},{"lat":40.7788654915818,"lng":-8.45483888318341},{"lat":40.7787750802777,"lng":-8.45485847515061},{"lat":40.7786883244467,"lng":-8.45489731324678},{"lat":40.7786042281825,"lng":-8.4549453660945},{"lat":40.7785304684426,"lng":-8.45501675211369},{"lat":40.7784644918077,"lng":-8.4551000215868},{"lat":40.7784239018481,"lng":-8.45520815274562},{"lat":40.7783646659513,"lng":-8.45530150580675},{"lat":40.7782872150936,"lng":-8.45538312486714},{"lat":40.7782298421989,"lng":-8.45549871321825},{"lat":40.7782120604058,"lng":-8.45563137811266},{"lat":40.7782308402065,"lng":-8.45576733533523},{"lat":40.7782524781766,"lng":-8.45590259334061},{"lat":40.7782742142168,"lng":-8.4560378246627},{"lat":40.7782813381644,"lng":-8.45617548004937},{"lat":40.7782706078354,"lng":-8.45631280490223},{"lat":40.778237654119,"lng":-8.45644333726109},{"lat":40.7781901613156,"lng":-8.45656663333203},{"lat":40.7781398553963,"lng":-8.45668794322247},{"lat":40.7780797977561,"lng":-8.45680126998304},{"lat":40.7780135741006,"lng":-8.45690857254802},{"lat":40.7779471995195,"lng":-8.45701571187769},{"lat":40.7778828692476,"lng":-8.45712499556342},{"lat":40.7778298869484,"lng":-8.45724388911292},{"lat":40.7777881384384,"lng":-8.45737071581213},{"lat":40.7777475495788,"lng":-8.4574982007028},{"lat":40.777700015422,"lng":-8.45762139067155},{"lat":40.7776482641541,"lng":-8.45774168375421},{"lat":40.7776013217166,"lng":-8.45785290835767},{"lat":40.7775568040584,"lng":-8.45797218322366},{"lat":40.7775133739012,"lng":-8.45809215976679},{"lat":40.7774657909497,"lng":-8.45820911061534},{"lat":40.7773963357089,"lng":-8.45830817283851},{"lat":40.7773236369531,"lng":-8.45840334544801},{"lat":40.7772509731139,"lng":-8.45849856185032},{"lat":40.7771791417915,"lng":-8.45859486481514},{"lat":40.7771077374782,"lng":-8.4586917157485},{"lat":40.7770345872908,"lng":-8.45878623736096},{"lat":40.7769537317337,"lng":-8.4588691845376},{"lat":40.7768705149409,"lng":-8.45894801187123},{"lat":40.7767810864584,"lng":-8.45901377005428},{"lat":40.7766854400088,"lng":-8.45906231309036},{"lat":40.7765880847875,"lng":-8.45908712781079},{"lat":40.7764897423623,"lng":-8.45910661300077},{"lat":40.7763982525606,"lng":-8.45915419993375},{"lat":40.7763281723339,"lng":-8.45924661349516},{"lat":40.776265607467,"lng":-8.45934830322542},{"lat":40.7762109785517,"lng":-8.45945764715485},{"lat":40.7761582530744,"lng":-8.4595686207039},{"lat":40.7761050522475,"lng":-8.45967919847552},{"lat":40.7760502037672,"lng":-8.45978837614159},{"lat":40.7759944623165,"lng":-8.45989676744349},{"lat":40.7759366946478,"lng":-8.46000325097022},{"lat":40.775862103627,"lng":-8.46008914003738},{"lat":40.7757781890165,"lng":-8.46015938377875},{"lat":40.7756965898509,"lng":-8.46023364723162},{"lat":40.7756255503511,"lng":-8.46032520465893},{"lat":40.7755494505497,"lng":-8.46040926135355},{"lat":40.7754535112114,"lng":-8.46048624109943},{"lat":40.7753597150432,"lng":-8.46050639535586},{"lat":40.7752646508876,"lng":-8.46050816983542},{"lat":40.7751713823643,"lng":-8.46053119437845},{"lat":40.7750788189596,"lng":-8.46055611048129},{"lat":40.7758805822949,"lng":-8.46355865179285},{"lat":40.7760649387516,"lng":-8.46419818471249},{"lat":40.7765748211575,"lng":-8.46596705049845},{"lat":40.7767713506986,"lng":-8.46598303285861},{"lat":40.7766685467585,"lng":-8.46680232021528},{"lat":40.7765869278053,"lng":-8.4673800548448},{"lat":40.7767197523876,"lng":-8.46732454821986},{"lat":40.776789623051,"lng":-8.46749312395834},{"lat":40.7769069740465,"lng":-8.46792645706142},{"lat":40.7770261780616,"lng":-8.46837125902123},{"lat":40.7770899047771,"lng":-8.46872583706686},{"lat":40.7770373748707,"lng":-8.46884669575285},{"lat":40.7769646439808,"lng":-8.46895487704902},{"lat":40.7768968476767,"lng":-8.46906987413727},{"lat":40.7768484662142,"lng":-8.46919993355917},{"lat":40.7768223676016,"lng":-8.46934166302149},{"lat":40.7767857667631,"lng":-8.46955058405765},{"lat":40.776759487841,"lng":-8.46976200666644},{"lat":40.7767314009279,"lng":-8.46997291771669},{"lat":40.7766946805933,"lng":-8.47018166338374},{"lat":40.7766847467939,"lng":-8.47039415723771},{"lat":40.7766918598521,"lng":-8.47053491673724},{"lat":40.7767100236928,"lng":-8.47067349928642},{"lat":40.7767231500594,"lng":-8.47081575108129},{"lat":40.7767235571525,"lng":-8.47096309138262},{"lat":40.7767171307419,"lng":-8.47110932145938},{"lat":40.776693107731,"lng":-8.47122567860503},{"lat":40.7766583743574,"lng":-8.47135823128698},{"lat":40.7766324887928,"lng":-8.47147215752821},{"lat":40.7766093603908,"lng":-8.47158947606811},{"lat":40.7765868907107,"lng":-8.47170721128579},{"lat":40.7765592143295,"lng":-8.47182184839138},{"lat":40.7765061499292,"lng":-8.47192827765786},{"lat":40.7764528627711,"lng":-8.47204245269173},{"lat":40.7763976096429,"lng":-8.47215266820481},{"lat":40.7763018287197,"lng":-8.47233210830078},{"lat":40.7762086439906,"lng":-8.47251418727015},{"lat":40.7761138077866,"lng":-8.47269462618111},{"lat":40.7760129740791,"lng":-8.47286922494507},{"lat":40.7759111289801,"lng":-8.47304286208846},{"lat":40.775812820683,"lng":-8.47321958355382},{"lat":40.7757111053786,"lng":-8.47339208313274},{"lat":40.775615505886,"lng":-8.47357011091391},{"lat":40.7755154384304,"lng":-8.47374397772945},{"lat":40.7752213677119,"lng":-8.47446870853013},{"lat":40.7745855321662,"lng":-8.4742971033822},{"lat":40.7745395620022,"lng":-8.47498409046102},{"lat":40.7747111745556,"lng":-8.47527775432003},{"lat":40.7748377903915,"lng":-8.47530148911898},{"lat":40.7749102495631,"lng":-8.47538206043019},{"lat":40.7749302440079,"lng":-8.47553176925906},{"lat":40.7749315064242,"lng":-8.47565165000434},{"lat":40.7748910301425,"lng":-8.47581751096534},{"lat":40.7748465135518,"lng":-8.47606958535466},{"lat":40.7747816527413,"lng":-8.47616769515823},{"lat":40.774722266036,"lng":-8.4762615413624},{"lat":40.7745520357644,"lng":-8.47646724571546},{"lat":40.7743995724253,"lng":-8.47684968129137},{"lat":40.7742204595102,"lng":-8.47727798584078},{"lat":40.7741321504595,"lng":-8.47736374583877},{"lat":40.7739320827121,"lng":-8.47748806758999},{"lat":40.7735807682135,"lng":-8.47773034798063},{"lat":40.7734837774336,"lng":-8.47780816808926},{"lat":40.7733893410843,"lng":-8.47789299572401},{"lat":40.7733048188543,"lng":-8.47799388897229},{"lat":40.7732227092548,"lng":-8.47809860145856},{"lat":40.7731417955903,"lng":-8.47820506347149},{"lat":40.7730546390066,"lng":-8.47830900900474},{"lat":40.7729682302161,"lng":-8.47841369845411},{"lat":40.7728709713315,"lng":-8.47850060771909},{"lat":40.7727656894188,"lng":-8.47856961407615},{"lat":40.7726589299879,"lng":-8.4786352320003},{"lat":40.7725529673254,"lng":-8.47870299896927},{"lat":40.7724665637901,"lng":-8.4787589143379},{"lat":40.7723975157951,"lng":-8.47886462803278},{"lat":40.7723659672823,"lng":-8.47899581662411},{"lat":40.7724198978886,"lng":-8.47911452095805},{"lat":40.7724819063129,"lng":-8.47920213765255},{"lat":40.7724983241392,"lng":-8.47933574049837},{"lat":40.7724645251167,"lng":-8.47945150471369},{"lat":40.7724390909321,"lng":-8.47957108843417},{"lat":40.772426318603,"lng":-8.47970863084557},{"lat":40.7724253103955,"lng":-8.47987787578676},{"lat":40.7724367218633,"lng":-8.48005451853697},{"lat":40.7724186013145,"lng":-8.48022875118826},{"lat":40.7724015019911,"lng":-8.48040317502758},{"lat":40.7723198577227,"lng":-8.4805218910086},{"lat":40.772270264322,"lng":-8.480634756477},{"lat":40.7722316353837,"lng":-8.48079391201514},{"lat":40.7722285256019,"lng":-8.48095958016209},{"lat":40.7722298868549,"lng":-8.48112634121802},{"lat":40.7722138519083,"lng":-8.48129255899204},{"lat":40.772222653059,"lng":-8.48141381516484},{"lat":40.7722358052246,"lng":-8.48156759892843},{"lat":40.7722274936048,"lng":-8.48172205825522},{"lat":40.7722110930857,"lng":-8.48185282213421},{"lat":40.772194067292,"lng":-8.48199019924629},{"lat":40.7721934673779,"lng":-8.48214065018711},{"lat":40.7721850614942,"lng":-8.48228642771433},{"lat":40.7721833811037,"lng":-8.48243343913299},{"lat":40.7721524908504,"lng":-8.48257340020943},{"lat":40.7720953884586,"lng":-8.48269978928467},{"lat":40.7720863262386,"lng":-8.48285420630302},{"lat":40.7721210221785,"lng":-8.4830170625319},{"lat":40.7721475396615,"lng":-8.48318256242937},{"lat":40.7721533354365,"lng":-8.48335137208704},{"lat":40.7721553153881,"lng":-8.48352053484119},{"lat":40.7721591481799,"lng":-8.48370396979524},{"lat":40.7721317159448,"lng":-8.48393907718411},{"lat":40.7721125935374,"lng":-8.48407601851608},{"lat":40.7720212409838,"lng":-8.48410735672968},{"lat":40.7719740354227,"lng":-8.48420998577644},{"lat":40.7719237931251,"lng":-8.48431385424379},{"lat":40.7718691943261,"lng":-8.48442889891884},{"lat":40.7718141682212,"lng":-8.48456505458985},{"lat":40.7717576354198,"lng":-8.48470002945542},{"lat":40.771727034162,"lng":-8.48484826846014},{"lat":40.7717037501705,"lng":-8.48499915890051},{"lat":40.7716907277964,"lng":-8.48514511198753},{"lat":40.7716884721285,"lng":-8.48528439282291},{"lat":40.7717496596314,"lng":-8.48539868687678},{"lat":40.7718019881079,"lng":-8.48551941109418},{"lat":40.7717934052095,"lng":-8.48565895008812},{"lat":40.7717898584917,"lng":-8.48579882601054},{"lat":40.7717436950627,"lng":-8.48592031954048},{"lat":40.7716711798599,"lng":-8.48602192341471},{"lat":40.7716511452369,"lng":-8.48615070203202},{"lat":40.7716356599508,"lng":-8.48628884074134},{"lat":40.7715964262911,"lng":-8.48639678318048},{"lat":40.7715256374831,"lng":-8.48648555807269},{"lat":40.7714661826732,"lng":-8.48658919171027},{"lat":40.7714211115682,"lng":-8.48670423449236},{"lat":40.771426359444,"lng":-8.4868714939584},{"lat":40.7714212649762,"lng":-8.4870202207232},{"lat":40.7714140553349,"lng":-8.48716885450823},{"lat":40.771420309871,"lng":-8.48731656243126},{"lat":40.7714664391223,"lng":-8.48745253794266},{"lat":40.7715142071241,"lng":-8.48758763162296},{"lat":40.7715610663016,"lng":-8.48772303929002},{"lat":40.7716515707987,"lng":-8.48773298091481},{"lat":40.7716790988905,"lng":-8.48785841230437},{"lat":40.7715980464672,"lng":-8.48792711935209},{"lat":40.7715134218921,"lng":-8.48798786766428},{"lat":40.7714311576648,"lng":-8.48804153099356},{"lat":40.7713476883373,"lng":-8.48809145667778},{"lat":40.77126888349,"lng":-8.48815252809871},{"lat":40.7712370635259,"lng":-8.48828275748315},{"lat":40.7712255036016,"lng":-8.4884092143279},{"lat":40.7712046322049,"lng":-8.4885333958852},{"lat":40.7711607893536,"lng":-8.48864577127644},{"lat":40.7710917141401,"lng":-8.48873516944798},{"lat":40.7710156194984,"lng":-8.4888143376344},{"lat":40.7709308864882,"lng":-8.4888765145548},{"lat":40.770837971897,"lng":-8.4889117022314},{"lat":40.770754275156,"lng":-8.4889701308213},{"lat":40.7707015814403,"lng":-8.48907678197875},{"lat":40.7706896902679,"lng":-8.4892017458451},{"lat":40.7707069908245,"lng":-8.48932685371778},{"lat":40.7707395128139,"lng":-8.48944656008269},{"lat":40.7707570069791,"lng":-8.48957187287468},{"lat":40.7707532753961,"lng":-8.48969922561819},{"lat":40.7707290928864,"lng":-8.48982266210663},{"lat":40.770675576423,"lng":-8.48992808578074},{"lat":40.7705996336673,"lng":-8.49000679335535},{"lat":40.770515897592,"lng":-8.49007077172313},{"lat":40.7704271527059,"lng":-8.49012113736833},{"lat":40.7703369954152,"lng":-8.49016028205462},{"lat":40.7702314597211,"lng":-8.49021388114073},{"lat":40.770127862477,"lng":-8.49027543102003},{"lat":40.7700420700437,"lng":-8.49032265534775},{"lat":40.7700127478184,"lng":-8.49044315493032},{"lat":40.7699846125097,"lng":-8.49055807015448},{"lat":40.7699398328067,"lng":-8.49066264937392},{"lat":40.7698621329171,"lng":-8.49072674867048},{"lat":40.7697850903899,"lng":-8.49079345120722},{"lat":40.7697235700867,"lng":-8.49088413543109},{"lat":40.7697117874176,"lng":-8.4910086156528},{"lat":40.7696836906443,"lng":-8.49112486046007},{"lat":40.7696451545069,"lng":-8.49123639306474},{"lat":40.7696551514892,"lng":-8.49135603551178},{"lat":40.7697052031718,"lng":-8.49146239759798},{"lat":40.7697769171203,"lng":-8.49154759290464},{"lat":40.7698474792054,"lng":-8.49163425802786},{"lat":40.7699120924931,"lng":-8.49172847302119},{"lat":40.7699699141877,"lng":-8.4918302096598},{"lat":40.7700339121972,"lng":-8.49192531833745},{"lat":40.7701060026839,"lng":-8.49200978104365},{"lat":40.770185290124,"lng":-8.49208229597684},{"lat":40.7702560961734,"lng":-8.49216769495886},{"lat":40.7703123631408,"lng":-8.49227003531317},{"lat":40.7703721874527,"lng":-8.49236904179758},{"lat":40.7704190042769,"lng":-8.4924714298977},{"lat":40.7704220922723,"lng":-8.49262402388144},{"lat":40.770375487422,"lng":-8.49274124494266},{"lat":40.7703176393831,"lng":-8.49284905035517},{"lat":40.770238361361,"lng":-8.49295016742339},{"lat":40.7702055521925,"lng":-8.49309237622314},{"lat":40.7701899813484,"lng":-8.4933089362013},{"lat":40.7701236312625,"lng":-8.49349342976736},{"lat":40.7700431538387,"lng":-8.49366986812634},{"lat":40.770026953001,"lng":-8.49387242972635},{"lat":40.7699997472323,"lng":-8.49407486416077},{"lat":40.769964903433,"lng":-8.49420985349514},{"lat":40.7699571486923,"lng":-8.49433778910962},{"lat":40.7699629345056,"lng":-8.49449306885662},{"lat":40.769965696931,"lng":-8.49461564612105},{"lat":40.7699704228041,"lng":-8.4947386403069},{"lat":40.7699609829944,"lng":-8.49485985799819},{"lat":40.7699309525203,"lng":-8.49497439853891},{"lat":40.7699368343395,"lng":-8.49509478013743},{"lat":40.7699910682595,"lng":-8.49519390985475},{"lat":40.7700374960209,"lng":-8.4952958021321},{"lat":40.7700567203996,"lng":-8.49541238089148},{"lat":40.7700256861975,"lng":-8.49552434918445},{"lat":40.7699880653772,"lng":-8.4956329651106},{"lat":40.7699634756911,"lng":-8.49574783933123},{"lat":40.7699543879167,"lng":-8.49601309278612},{"lat":40.77002195767,"lng":-8.49618799538707},{"lat":40.7701040297323,"lng":-8.49631108994801},{"lat":40.7702159420051,"lng":-8.49632758286594},{"lat":40.7703192403796,"lng":-8.49633769919133},{"lat":40.7704221213233,"lng":-8.4963482598173},{"lat":40.7705203284203,"lng":-8.49640553386566},{"lat":40.7706186839869,"lng":-8.4964808518175},{"lat":40.7707134341624,"lng":-8.49656352482456},{"lat":40.7707781239735,"lng":-8.49664903093207},{"lat":40.7708424080927,"lng":-8.49673466528552},{"lat":40.7709087704717,"lng":-8.49681744946165},{"lat":40.7709872594336,"lng":-8.49687977545308},{"lat":40.7710672366929,"lng":-8.49693901942521},{"lat":40.7711076705669,"lng":-8.49710671834981},{"lat":40.7710780732259,"lng":-8.49732129079316},{"lat":40.7710898748103,"lng":-8.49752335142586},{"lat":40.7711268594864,"lng":-8.49774730854124},{"lat":40.764111859687,"lng":-8.49718229513525},{"lat":40.7640855696814,"lng":-8.49730015763939},{"lat":40.7637750064732,"lng":-8.4973975822974},{"lat":40.7636712579666,"lng":-8.49741281136234},{"lat":40.7635169087446,"lng":-8.49743515078607},{"lat":40.7631415317621,"lng":-8.4977775516324},{"lat":40.7630296537225,"lng":-8.49798786427103},{"lat":40.7629676022513,"lng":-8.49807528001389},{"lat":40.7628945172584,"lng":-8.49814949039349},{"lat":40.7626903472458,"lng":-8.49821528044914},{"lat":40.762542781341,"lng":-8.49820771877942},{"lat":40.7624083562101,"lng":-8.49825885156714},{"lat":40.7622621923576,"lng":-8.49823507240034},{"lat":40.7621704438399,"lng":-8.49823148874077},{"lat":40.7620594347601,"lng":-8.49819653269956},{"lat":40.7619696430777,"lng":-8.49822908125036},{"lat":40.7619366263541,"lng":-8.4983608302121},{"lat":40.7618736231264,"lng":-8.49846434555662},{"lat":40.7618070685425,"lng":-8.49858027765715},{"lat":40.7617619290448,"lng":-8.49870118251649},{"lat":40.7616694453383,"lng":-8.498710798026},{"lat":40.7615294060791,"lng":-8.49889867218431},{"lat":40.761374554376,"lng":-8.49913164412688},{"lat":40.7612445472067,"lng":-8.49923455217141},{"lat":40.761003449337,"lng":-8.49943810351663},{"lat":40.7606556816944,"lng":-8.49965809274467},{"lat":40.7604550113895,"lng":-8.50012210669268},{"lat":40.7599555413498,"lng":-8.50154921121246},{"lat":40.7598869313732,"lng":-8.50176017289213},{"lat":40.7593758263798,"lng":-8.50315078302836},{"lat":40.7593173352631,"lng":-8.50344251668696},{"lat":40.7592719868544,"lng":-8.50366841986552},{"lat":40.7591138099351,"lng":-8.50461791849565},{"lat":40.7590928391918,"lng":-8.50497635605461},{"lat":40.759036765535,"lng":-8.50514583881942},{"lat":40.7588290595084,"lng":-8.50644301821344},{"lat":40.7587727958041,"lng":-8.50658040476645},{"lat":40.7587837964982,"lng":-8.50671416728052},{"lat":40.758796289048,"lng":-8.50688772790788},{"lat":40.7588052450172,"lng":-8.50704101923753},{"lat":40.7588188108258,"lng":-8.50724419351504},{"lat":40.7588259672275,"lng":-8.50736940804312},{"lat":40.7588317687963,"lng":-8.50749556239949},{"lat":40.7588380413513,"lng":-8.50761556130185},{"lat":40.7588432504649,"lng":-8.50775769980124},{"lat":40.7588486449016,"lng":-8.50798107799193},{"lat":40.7588523313418,"lng":-8.50812083947657},{"lat":40.7588570318787,"lng":-8.50825302745206},{"lat":40.7588673045931,"lng":-8.50844363051231},{"lat":40.7588773286733,"lng":-8.50860012452262},{"lat":40.758895371722,"lng":-8.50872777027636},{"lat":40.7589303653672,"lng":-8.50887623482787},{"lat":40.758972508154,"lng":-8.50904120215153},{"lat":40.7590258241425,"lng":-8.50923157457123},{"lat":40.759071856766,"lng":-8.50939123512956},{"lat":40.7591337606526,"lng":-8.5095985926487},{"lat":40.7591688350971,"lng":-8.50972195352545},{"lat":40.7592214144915,"lng":-8.50991694150441},{"lat":40.7592778437399,"lng":-8.5101459410276},{"lat":40.7593242161958,"lng":-8.51031116958769},{"lat":40.7593706797457,"lng":-8.51047628165033},{"lat":40.7594176738736,"lng":-8.51061676425594},{"lat":40.7594626436337,"lng":-8.51077014290317},{"lat":40.759511565034,"lng":-8.51098169275301},{"lat":40.75954071421,"lng":-8.51109909897458},{"lat":40.75958317141,"lng":-8.51124985978795},{"lat":40.7596340336537,"lng":-8.51147314444115},{"lat":40.7596674799028,"lng":-8.51159839282021},{"lat":40.7597017716784,"lng":-8.51171263242144},{"lat":40.7597400044738,"lng":-8.5118332895631},{"lat":40.7597896920393,"lng":-8.5119485642328},{"lat":40.7598396239407,"lng":-8.51209853961192},{"lat":40.7598845537107,"lng":-8.51226376390434},{"lat":40.759910979115,"lng":-8.51238755093132},{"lat":40.7599628320335,"lng":-8.51258301393732},{"lat":40.7600067163095,"lng":-8.51276445628548},{"lat":40.7600353398576,"lng":-8.5128771252487},{"lat":40.7600699728917,"lng":-8.51302381604045},{"lat":40.7601096544004,"lng":-8.51316899736443},{"lat":40.7601692224233,"lng":-8.51337291389188},{"lat":40.7602472966734,"lng":-8.5136168453056},{"lat":40.7603090178241,"lng":-8.51379566820946},{"lat":40.7603419548544,"lng":-8.5139107313013},{"lat":40.7604127379686,"lng":-8.51415343924145},{"lat":40.7604557087599,"lng":-8.51431107455161},{"lat":40.7604890343543,"lng":-8.51444437832619},{"lat":40.7605270715652,"lng":-8.51462306604286},{"lat":40.7605744707164,"lng":-8.51482987623856},{"lat":40.7606100254753,"lng":-8.51502382747074},{"lat":40.760637658274,"lng":-8.51519002126453},{"lat":40.7606500776868,"lng":-8.51535435161546},{"lat":40.7606479796209,"lng":-8.51552475592759},{"lat":40.7606415191558,"lng":-8.51565309426513},{"lat":40.7606311270601,"lng":-8.5157721726043},{"lat":40.7607089831758,"lng":-8.51605317542951},{"lat":40.7607705400675,"lng":-8.51619895924813},{"lat":40.7608537735703,"lng":-8.51637945026511},{"lat":40.7609133238073,"lng":-8.51653280262866},{"lat":40.7610639907383,"lng":-8.51717164718889},{"lat":40.7611854497526,"lng":-8.51769141795029},{"lat":40.7612757677779,"lng":-8.51772469024498},{"lat":40.7613666277591,"lng":-8.51775232849492},{"lat":40.761459474852,"lng":-8.51778680458204},{"lat":40.761550812129,"lng":-8.51781469800277},{"lat":40.7616401008955,"lng":-8.51776609975858},{"lat":40.7617204820794,"lng":-8.51782227994605},{"lat":40.7617700556477,"lng":-8.51792943158221},{"lat":40.7617904490944,"lng":-8.51804638926071},{"lat":40.7618524104027,"lng":-8.51815899889063},{"lat":40.7619346166747,"lng":-8.51822706058872},{"lat":40.7620238654491,"lng":-8.51824935422535},{"lat":40.7621211604724,"lng":-8.51827500999516},{"lat":40.762210728078,"lng":-8.51830427150759},{"lat":40.7622975610114,"lng":-8.51834589813796},{"lat":40.7623813894549,"lng":-8.5183965497835},{"lat":40.7624642591424,"lng":-8.51844738195788},{"lat":40.7625509453426,"lng":-8.51848735951514},{"lat":40.7626415832639,"lng":-8.51850296299317},{"lat":40.7627336694627,"lng":-8.51849502225748},{"lat":40.7628253021497,"lng":-8.51848428269199},{"lat":40.7629135946132,"lng":-8.51853672534992},{"lat":40.7629159679148,"lng":-8.5186584344579},{"lat":40.7629046978734,"lng":-8.51877926765928},{"lat":40.762981281728,"lng":-8.51884551154965},{"lat":40.7630717508523,"lng":-8.51883993529573},{"lat":40.7631624848066,"lng":-8.51883972202145},{"lat":40.7632551182877,"lng":-8.51886849203865},{"lat":40.763327091027,"lng":-8.51893973721128},{"lat":40.7633658329207,"lng":-8.51905102807289},{"lat":40.7633729572771,"lng":-8.51917310423408},{"lat":40.7633609372122,"lng":-8.51929482563648},{"lat":40.7633301155361,"lng":-8.51941062661036},{"lat":40.7632840700209,"lng":-8.51951725666136},{"lat":40.7632301566678,"lng":-8.51961772983619},{"lat":40.7631723140373,"lng":-8.51971453471181},{"lat":40.7631199780338,"lng":-8.5198311768762},{"lat":40.7631005974742,"lng":-8.51996096767523},{"lat":40.7631102696452,"lng":-8.52011838202667},{"lat":40.7631185791158,"lng":-8.52025486852186},{"lat":40.7631312253548,"lng":-8.52039077858444},{"lat":40.7631666244332,"lng":-8.52051726208623},{"lat":40.7632279566308,"lng":-8.52062234270639},{"lat":40.7632993571696,"lng":-8.52071563395064},{"lat":40.7633873850022,"lng":-8.52077926443527},{"lat":40.7634745521651,"lng":-8.52082834881354},{"lat":40.7635593492789,"lng":-8.52088410868267},{"lat":40.7636416437841,"lng":-8.52094552476028},{"lat":40.7637147981851,"lng":-8.52102355743475},{"lat":40.76375899248,"lng":-8.52113294309552},{"lat":40.7637830140866,"lng":-8.52126685413315},{"lat":40.7637907603311,"lng":-8.52140493299099},{"lat":40.7637985838352,"lng":-8.52153472312484},{"lat":40.7638075236593,"lng":-8.52166348822856},{"lat":40.7638164866553,"lng":-8.52178586516226},{"lat":40.7638268437044,"lng":-8.52190363487196},{"lat":40.7638392467681,"lng":-8.52202107663837},{"lat":40.7638527338236,"lng":-8.52213827724134},{"lat":40.7638655693177,"lng":-8.52225559134103},{"lat":40.7638762675439,"lng":-8.52237325187174},{"lat":40.7638662003369,"lng":-8.52250100738774},{"lat":40.7637978778841,"lng":-8.52259331295531},{"lat":40.7637247126163,"lng":-8.52266263636155},{"lat":40.7636717347873,"lng":-8.52276926220226},{"lat":40.7636965671104,"lng":-8.52289853222215},{"lat":40.7637689680818,"lng":-8.5229824855861},{"lat":40.7638575040918,"lng":-8.52303606030876},{"lat":40.7639479412829,"lng":-8.52308396262027},{"lat":40.764039615284,"lng":-8.52312746889564},{"lat":40.7641313190625,"lng":-8.5231709517787},{"lat":40.7642205136016,"lng":-8.52322282188127},{"lat":40.7643069227598,"lng":-8.52328245109478},{"lat":40.764382856513,"lng":-8.52336095460785},{"lat":40.7643838635296,"lng":-8.52348468735803},{"lat":40.7643287074523,"lng":-8.52358788043983},{"lat":40.764251719162,"lng":-8.52366673033127},{"lat":40.7641724041011,"lng":-8.5237415396347},{"lat":40.7640976284564,"lng":-8.52382351340167},{"lat":40.7640442836122,"lng":-8.52393024098451},{"lat":40.7640360034567,"lng":-8.52405575561429},{"lat":40.764073913067,"lng":-8.52417282890689},{"lat":40.7641416834921,"lng":-8.52426479870141},{"lat":40.7642253161076,"lng":-8.52433020437734},{"lat":40.7643195782047,"lng":-8.52436239467522},{"lat":40.7644155138023,"lng":-8.52438597161083},{"lat":40.7645102205592,"lng":-8.52441688321841},{"lat":40.7645903128496,"lng":-8.5244861178366},{"lat":40.7646629415744,"lng":-8.52457080969101},{"lat":40.7647459734967,"lng":-8.52463803783719},{"lat":40.7648342862358,"lng":-8.52469242671058},{"lat":40.7649269102978,"lng":-8.52473255408608},{"lat":40.7650121106334,"lng":-8.52479497270724},{"lat":40.7650751019271,"lng":-8.52488976713304},{"lat":40.7651009469592,"lng":-8.52501339956007},{"lat":40.7651152235492,"lng":-8.52514036678481},{"lat":40.7651311183625,"lng":-8.5252670190597},{"lat":40.7651502484824,"lng":-8.52539288860289},{"lat":40.7651801009154,"lng":-8.52551498275552},{"lat":40.7652178000682,"lng":-8.52563339479876},{"lat":40.7652545779929,"lng":-8.52575230609442},{"lat":40.7652870705682,"lng":-8.52587326375168},{"lat":40.7653030832953,"lng":-8.52599969455932},{"lat":40.765303415465,"lng":-8.52612799436674},{"lat":40.7653016510718,"lng":-8.52625631261375},{"lat":40.7654529293766,"lng":-8.52624965017597},{"lat":40.7655108582856,"lng":-8.52614365628363},{"lat":40.7655652000834,"lng":-8.52603379305345},{"lat":40.7656221614817,"lng":-8.52592626407241},{"lat":40.7656803938968,"lng":-8.5258199066301},{"lat":40.7657295957891,"lng":-8.52570647251125},{"lat":40.7657585058579,"lng":-8.52558109640385},{"lat":40.7657942624812,"lng":-8.52545910157936},{"lat":40.7658509035158,"lng":-8.52535135192982},{"lat":40.7659034286742,"lng":-8.52524004016559},{"lat":40.7659436385168,"lng":-8.52512051325072},{"lat":40.7659606667733,"lng":-8.52499142946053},{"lat":40.7659936620875,"lng":-8.52486803793157},{"lat":40.7660507951346,"lng":-8.52476108625102},{"lat":40.7661214393793,"lng":-8.52466892727085},{"lat":40.7661945598296,"lng":-8.52457944867433},{"lat":40.7662629111052,"lng":-8.5244780423859},{"lat":40.7663156903197,"lng":-8.52437388442913},{"lat":40.7663609930723,"lng":-8.5242706536675},{"lat":40.7663984234276,"lng":-8.52415843449309},{"lat":40.7664481059981,"lng":-8.52405387655219},{"lat":40.7665007254922,"lng":-8.52395581900913},{"lat":40.7665604146757,"lng":-8.52386628058699},{"lat":40.7666231520057,"lng":-8.52376981576757},{"lat":40.766690534156,"lng":-8.5236892748791},{"lat":40.7667588834274,"lng":-8.52360286608},{"lat":40.7668246478113,"lng":-8.52351311382678},{"lat":40.7668870013033,"lng":-8.52341902431636},{"lat":40.7669542497503,"lng":-8.52333827115177},{"lat":40.7670047554053,"lng":-8.52323721288781},{"lat":40.7670579561665,"lng":-8.52312735948199},{"lat":40.7671130057207,"lng":-8.52302944717033},{"lat":40.7671601754814,"lng":-8.52291804569964},{"lat":40.7671697990312,"lng":-8.52279140890167},{"lat":40.7671835299834,"lng":-8.52266572817603},{"lat":40.7672050462391,"lng":-8.5225411907867},{"lat":40.7672301911077,"lng":-8.52241806379653},{"lat":40.7672693767232,"lng":-8.52231139418276},{"lat":40.7673131100599,"lng":-8.52220783752509},{"lat":40.7673584617913,"lng":-8.52207961428333},{"lat":40.767389902246,"lng":-8.52195387283322},{"lat":40.7674203273432,"lng":-8.52182770369947},{"lat":40.7674579892112,"lng":-8.52170298604911},{"lat":40.7674933822729,"lng":-8.52159093649171},{"lat":40.767531601749,"lng":-8.52148053301514},{"lat":40.7675712158106,"lng":-8.52137098556515},{"lat":40.7676197792862,"lng":-8.52126801852146},{"lat":40.7676866111644,"lng":-8.52118456653145},{"lat":40.7677578599147,"lng":-8.52110781185511},{"lat":40.7678377765426,"lng":-8.52104720596352},{"lat":40.7679180559463,"lng":-8.52098743112699},{"lat":40.7679970873998,"lng":-8.52092492591988},{"lat":40.7680717484138,"lng":-8.52085366363716},{"lat":40.768144256216,"lng":-8.52077867197663},{"lat":40.7682077233221,"lng":-8.52069097683742},{"lat":40.7682602237425,"lng":-8.52059132269168},{"lat":40.7682996382153,"lng":-8.52048169537235},{"lat":40.7683383896324,"lng":-8.52037158675208},{"lat":40.7683867455777,"lng":-8.5202427073439},{"lat":40.7684306680745,"lng":-8.52012864404122},{"lat":40.7684617331609,"lng":-8.52000759845282},{"lat":40.7684906424116,"lng":-8.51988357326768},{"lat":40.7685279123601,"lng":-8.51973678558636},{"lat":40.7685669762257,"lng":-8.51960200400183},{"lat":40.768607027035,"lng":-8.51946771359053},{"lat":40.7686563535727,"lng":-8.51933938227957},{"lat":40.7687212567085,"lng":-8.51922317828727},{"lat":40.7687742617693,"lng":-8.51910769049791},{"lat":40.7688240359877,"lng":-8.51898970245313},{"lat":40.7688739852516,"lng":-8.51887184316317},{"lat":40.768939104197,"lng":-8.51876852502893},{"lat":40.7690209384513,"lng":-8.51868719401909},{"lat":40.7690989299607,"lng":-8.51862157819845},{"lat":40.769178771616,"lng":-8.51855983773366},{"lat":40.7692594506468,"lng":-8.51849433781432},{"lat":40.7693395832949,"lng":-8.51842768212112},{"lat":40.7694361500014,"lng":-8.51831137937426},{"lat":40.7695262537246,"lng":-8.51819480683002},{"lat":40.7696395138222,"lng":-8.51812540548434},{"lat":40.7697312867942,"lng":-8.51808714811509},{"lat":40.7698215836901,"lng":-8.51804434568592},{"lat":40.7699148586023,"lng":-8.51801501567827},{"lat":40.7700079704449,"lng":-8.51798435687829},{"lat":40.770101579405,"lng":-8.51795692257505},{"lat":40.7701814147647,"lng":-8.51788858768352},{"lat":40.7702724946817,"lng":-8.51784939588752},{"lat":40.7703642972055,"lng":-8.51781250837055},{"lat":40.7704560215678,"lng":-8.51777529220692},{"lat":40.7705470503123,"lng":-8.51773523040808},{"lat":40.7706378834902,"lng":-8.51769440102573},{"lat":40.7707273596122,"lng":-8.51764897509401},{"lat":40.7708148605089,"lng":-8.51759718773282},{"lat":40.7709040914952,"lng":-8.51755694573848},{"lat":40.7710029225838,"lng":-8.51752321432209},{"lat":40.7711028707003,"lng":-8.51749557387243},{"lat":40.7712024054992,"lng":-8.51746549449852},{"lat":40.7713023890762,"lng":-8.51743855766434},{"lat":40.7714040735168,"lng":-8.51742843590335},{"lat":40.7715048640007,"lng":-8.51744987886275},{"lat":40.7716061436519,"lng":-8.51746720508514},{"lat":40.7717078607841,"lng":-8.51747890048638},{"lat":40.7718099683961,"lng":-8.5174774480363},{"lat":40.7719120250759,"lng":-8.51748182414291},{"lat":40.7720138602409,"lng":-8.51747267924883},{"lat":40.7721158735836,"lng":-8.5174665439573},{"lat":40.7722179601119,"lng":-8.51746249732758},{"lat":40.7723350504583,"lng":-8.51741391018541},{"lat":40.7724155889131,"lng":-8.51732443201503},{"lat":40.7724968655034,"lng":-8.51723663749546},{"lat":40.7725885306194,"lng":-8.51716791376489},{"lat":40.7726734462513,"lng":-8.51708599594144},{"lat":40.772748858788,"lng":-8.51700627376322},{"lat":40.7728558375241,"lng":-8.51690427015508},{"lat":40.7729685586863,"lng":-8.51681281291632},{"lat":40.7730781987676,"lng":-8.51671524810182},{"lat":40.7731550247096,"lng":-8.5166241796454},{"lat":40.7732378905642,"lng":-8.51654276036259},{"lat":40.7733121280374,"lng":-8.51647482550064},{"lat":40.773387604651,"lng":-8.5164093009837},{"lat":40.7734657216373,"lng":-8.51634942385079},{"lat":40.7735470809082,"lng":-8.51629752277515},{"lat":40.7736300691084,"lng":-8.51625024580583},{"lat":40.7737132462594,"lng":-8.51620355140307},{"lat":40.7737966822574,"lng":-8.5161578142781},{"lat":40.7738855359692,"lng":-8.51613774710052},{"lat":40.7740014448127,"lng":-8.51612562821574},{"lat":40.7740914931568,"lng":-8.51611461045448},{"lat":40.7741429597813,"lng":-8.51610846693395},{"lat":40.7741944462041,"lng":-8.516102598328},{"lat":40.7742459838049,"lng":-8.51609758878928},{"lat":40.774297557767,"lng":-8.51609329016871},{"lat":40.7743490158232,"lng":-8.51609759647843},{"lat":40.7743995396033,"lng":-8.51611180957499},{"lat":40.7744494323172,"lng":-8.51612949448211},{"lat":40.7745024746285,"lng":-8.51614999299957},{"lat":40.7745558014196,"lng":-8.51616915703119},{"lat":40.7746097147365,"lng":-8.51618524467569},{"lat":40.7746639001719,"lng":-8.51619965305522},{"lat":40.7747184861766,"lng":-8.51621112255804},{"lat":40.7747735248298,"lng":-8.51621779967686},{"lat":40.7748287867151,"lng":-8.51621935022708},{"lat":40.7748840509635,"lng":-8.51621747629339},{"lat":40.7749392859596,"lng":-8.51621437737523},{"lat":40.7749944953526,"lng":-8.51621058179551},{"lat":40.7750426466162,"lng":-8.51620734149015},{"lat":40.775089708725,"lng":-8.51620450951368},{"lat":40.775143077239,"lng":-8.51620214967075},{"lat":40.7751941974777,"lng":-8.51620133813863},{"lat":40.7752394023735,"lng":-8.516201607283},{"lat":40.7752883171112,"lng":-8.51620193682827},{"lat":40.7753402416922,"lng":-8.51620160728767},{"lat":40.7753859638155,"lng":-8.51619987396614},{"lat":40.775431684342,"lng":-8.51619807903679},{"lat":40.7754923499809,"lng":-8.51619017002293},{"lat":40.7755375310873,"lng":-8.51618078612812},{"lat":40.7755826235641,"lng":-8.5161707028268},{"lat":40.7756273549902,"lng":-8.51615816304746},{"lat":40.7756714529599,"lng":-8.51614221876346},{"lat":40.7757146228575,"lng":-8.51612237313328},{"lat":40.7757565396948,"lng":-8.51609832696076},{"lat":40.7757951859804,"lng":-8.51606666661343},{"lat":40.7758254354246,"lng":-8.5160216211102},{"lat":40.7758442859537,"lng":-8.51596693180981},{"lat":40.7758879499983,"lng":-8.51591457041535},{"lat":40.7759324728482,"lng":-8.51590137652022},{"lat":40.7759781194322,"lng":-8.51589770934869},{"lat":40.7760247721663,"lng":-8.51589015193413},{"lat":40.7766569189472,"lng":-8.51692661324558},{"lat":40.7826099787604,"lng":-8.51177586715368},{"lat":40.7876739667617,"lng":-8.50881482103287},{"lat":40.7905124515058,"lng":-8.50792013216861},{"lat":40.7986919513163,"lng":-8.50432958076185},{"lat":40.814468457276,"lng":-8.49875908293634}]},{"zone_id":"arc15","points":[{"lat":40.8998298118896,"lng":-8.3734949777164},{"lat":40.8999971494155,"lng":-8.3736928223327},{"lat":40.9002049574158,"lng":-8.3738739732031},{"lat":40.9004077790187,"lng":-8.37402804384594},{"lat":40.9006729683306,"lng":-8.374240735114},{"lat":40.9008437233696,"lng":-8.37439955725751},{"lat":40.9010278057265,"lng":-8.37459901328668},{"lat":40.9012241463671,"lng":-8.3747935376409},{"lat":40.9014646843791,"lng":-8.37508185644501},{"lat":40.9016666476813,"lng":-8.37525836466399},{"lat":40.9018640206479,"lng":-8.3754333119877},{"lat":40.9020496968497,"lng":-8.37560192399657},{"lat":40.9022454491805,"lng":-8.37577651020263},{"lat":40.9024353713066,"lng":-8.37593838077153},{"lat":40.9024570795975,"lng":-8.37606343048603},{"lat":40.9032232721474,"lng":-8.37624961587487},{"lat":40.9065714774975,"lng":-8.37658762604081},{"lat":40.9228164146771,"lng":-8.37822809558975},{"lat":40.9308474253404,"lng":-8.3782366010822},{"lat":40.9356732959254,"lng":-8.37762610071319},{"lat":40.939129894942,"lng":-8.37669629089521},{"lat":40.9413368363393,"lng":-8.37521196850881},{"lat":40.9428293406312,"lng":-8.37356785318657},{"lat":40.9436439040859,"lng":-8.37236259691614},{"lat":40.944637239265,"lng":-8.37023220133922},{"lat":40.9460802130867,"lng":-8.36476096961573},{"lat":40.9470394446498,"lng":-8.36095766322538},{"lat":40.9463838125181,"lng":-8.3597858513771},{"lat":40.9452458662437,"lng":-8.35689118725546},{"lat":40.945191947632,"lng":-8.35670569507855},{"lat":40.9440149481661,"lng":-8.35273947818776},{"lat":40.9434148880855,"lng":-8.35075745330156},{"lat":40.9419680660084,"lng":-8.34597914799799},{"lat":40.9414871362139,"lng":-8.34354149600075},{"lat":40.9412996777743,"lng":-8.34181282411508},{"lat":40.9413504424014,"lng":-8.34008490564894},{"lat":40.9416642046006,"lng":-8.33782366285319},{"lat":40.9422521585335,"lng":-8.33489440357846},{"lat":40.9383591735917,"lng":-8.33404327616669},{"lat":40.935441166895,"lng":-8.33372026628894},{"lat":40.9279943640757,"lng":-8.3345616081975},{"lat":40.9232872817761,"lng":-8.3354896459401},{"lat":40.9213542613938,"lng":-8.33594704905017},{"lat":40.9199237244719,"lng":-8.33653160312904},{"lat":40.9191186599532,"lng":-8.33707879381985},{"lat":40.9180409278756,"lng":-8.33822189362449},{"lat":40.9158978054921,"lng":-8.34136719472603},{"lat":40.9150850426342,"lng":-8.34271639469538},{"lat":40.9015907096687,"lng":-8.3698701153352},{"lat":40.8998298118896,"lng":-8.3734949777164}]},{"zone_id":"vcb6","points":[{"lat":40.8311785754702,"lng":-8.28524810109897},{"lat":40.8261923223182,"lng":-8.28337800643756},{"lat":40.8168200940133,"lng":-8.27920781023714},{"lat":40.8169707262182,"lng":-8.28274839328905},{"lat":40.8170376354243,"lng":-8.28622960266238},{"lat":40.8168893707945,"lng":-8.29018072526163},{"lat":40.8165270661654,"lng":-8.29375498482344},{"lat":40.8159500876181,"lng":-8.29742275717114},{"lat":40.8136614977296,"lng":-8.30866691023934},{"lat":40.8144067910442,"lng":-8.30879821092909},{"lat":40.8175167634881,"lng":-8.30964357975392},{"lat":40.8199549235365,"lng":-8.30986130648461},{"lat":40.8211155772944,"lng":-8.31198651378968},{"lat":40.8237986156912,"lng":-8.31485412542508},{"lat":40.8262116064552,"lng":-8.31753522132472},{"lat":40.8272798013537,"lng":-8.319656100513},{"lat":40.8276938156004,"lng":-8.32146080014886},{"lat":40.8277501620196,"lng":-8.32342133015547},{"lat":40.8274391557034,"lng":-8.32520930101246},{"lat":40.8288774655762,"lng":-8.32607501895451},{"lat":40.8332341872264,"lng":-8.32806386625664},{"lat":40.8362350674455,"lng":-8.32873142560158},{"lat":40.8399510223598,"lng":-8.32921292506073},{"lat":40.8435248217836,"lng":-8.32922345864243},{"lat":40.849458920429,"lng":-8.32829966841203},{"lat":40.8548208820767,"lng":-8.32756231630562},{"lat":40.8593807624543,"lng":-8.32757565052999},{"lat":40.8623258396281,"lng":-8.32758426429802},{"lat":40.8639402152707,"lng":-8.32725561347249},{"lat":40.8648490546131,"lng":-8.326925236189},{"lat":40.8635294988675,"lng":-8.32215593781169},{"lat":40.8628314412674,"lng":-8.31875394243494},{"lat":40.861874093166,"lng":-8.31506185442916},{"lat":40.8605178263345,"lng":-8.3111587899659},{"lat":40.8588423995532,"lng":-8.30820511956128},{"lat":40.8559701407633,"lng":-8.30314168458192},{"lat":40.8526988054573,"lng":-8.29955508480488},{"lat":40.8497065890465,"lng":-8.29549378799852},{"lat":40.8465546416303,"lng":-8.29333127930942},{"lat":40.8426843239259,"lng":-8.29027212067606},{"lat":40.8373010020646,"lng":-8.2871922354957},{"lat":40.8311785754702,"lng":-8.28524810109897}]},{"zone_id":"str2","points":[{"lat":41.3704789784926,"lng":-8.4791150536287},{"lat":41.3706443822878,"lng":-8.47813601451992},{"lat":41.3706895767655,"lng":-8.4778693527338},{"lat":41.3717918517337,"lng":-8.47642687799752},{"lat":41.3721733188599,"lng":-8.47604711727611},{"lat":41.3722315444039,"lng":-8.47586446884228},{"lat":41.3723656477134,"lng":-8.47544379380444},{"lat":41.3724409702385,"lng":-8.47518695941233},{"lat":41.3724588180268,"lng":-8.47493879200744},{"lat":41.3724208828937,"lng":-8.47482730848389},{"lat":41.3723513501465,"lng":-8.47474482781156},{"lat":41.3722272402785,"lng":-8.47457527751572},{"lat":41.3720485985481,"lng":-8.47436611962714},{"lat":41.3721433830312,"lng":-8.47428354099977},{"lat":41.372253886611,"lng":-8.47421587446726},{"lat":41.3723458856079,"lng":-8.47413100972254},{"lat":41.3724010984233,"lng":-8.47382243856302},{"lat":41.372487852372,"lng":-8.47350506046547},{"lat":41.3725695112645,"lng":-8.47320403321978},{"lat":41.3726107250223,"lng":-8.47287984346113},{"lat":41.3726125769401,"lng":-8.47252902976185},{"lat":41.3726034089426,"lng":-8.47240204297686},{"lat":41.3725172794198,"lng":-8.47232832138279},{"lat":41.372519957892,"lng":-8.47218358134067},{"lat":41.3726245925986,"lng":-8.47190729981997},{"lat":41.372689201393,"lng":-8.47173861692919},{"lat":41.3727770428867,"lng":-8.47144658945651},{"lat":41.3728556975175,"lng":-8.47121654368287},{"lat":41.3727488419691,"lng":-8.47117953193023},{"lat":41.3716820503884,"lng":-8.46950879860306},{"lat":41.3713683277772,"lng":-8.4690203506973},{"lat":41.3712275411806,"lng":-8.46897730955063},{"lat":41.3711351712935,"lng":-8.46863785242021},{"lat":41.3711501189119,"lng":-8.46832919394286},{"lat":41.3711717189555,"lng":-8.46817977268454},{"lat":41.3713051952664,"lng":-8.46782749568835},{"lat":41.3713647062018,"lng":-8.46767588592238},{"lat":41.371437402835,"lng":-8.4673561628393},{"lat":41.3714816154449,"lng":-8.46723148299776},{"lat":41.3715454818667,"lng":-8.46712853840966},{"lat":41.3716967719674,"lng":-8.4668741250867},{"lat":41.3717590388649,"lng":-8.46676375717876},{"lat":41.3718993233931,"lng":-8.46651681401837},{"lat":41.3719461558765,"lng":-8.4663577224947},{"lat":41.3720898633871,"lng":-8.46629475433425},{"lat":41.3721441352057,"lng":-8.46611526153693},{"lat":41.3723064274468,"lng":-8.46576192656434},{"lat":41.3723750573522,"lng":-8.4654757655429},{"lat":41.3723948240683,"lng":-8.46514882950365},{"lat":41.372414624223,"lng":-8.46502856444413},{"lat":41.3725729248442,"lng":-8.46468130354553},{"lat":41.3726516563158,"lng":-8.46448304379052},{"lat":41.3730821181329,"lng":-8.46411613664455},{"lat":41.3732794386772,"lng":-8.46391357580947},{"lat":41.3734746840875,"lng":-8.46371172642367},{"lat":41.373587195363,"lng":-8.46363161810319},{"lat":41.373684746992,"lng":-8.46361776785523},{"lat":41.3737724169434,"lng":-8.46365837859524},{"lat":41.3740286875719,"lng":-8.46271704940868},{"lat":41.3742335696506,"lng":-8.46173229852713},{"lat":41.3746259973395,"lng":-8.46046244493134},{"lat":41.3747961517171,"lng":-8.45990304098595},{"lat":41.3748936769346,"lng":-8.45955126738948},{"lat":41.3749721785749,"lng":-8.45914631533424},{"lat":41.3750562909144,"lng":-8.45876016704149},{"lat":41.3751222041273,"lng":-8.45834750237815},{"lat":41.3751708780311,"lng":-8.45794873688196},{"lat":41.3752279385687,"lng":-8.45760636381872},{"lat":41.3752527681081,"lng":-8.45745738020709},{"lat":41.375341125512,"lng":-8.45695422708506},{"lat":41.3754292104813,"lng":-8.45645261152905},{"lat":41.3755051303769,"lng":-8.45596910856185},{"lat":41.3755699534448,"lng":-8.45565206500264},{"lat":41.3756264357554,"lng":-8.45513056965707},{"lat":41.3754254807676,"lng":-8.45468094222786},{"lat":41.3751803080923,"lng":-8.45413238889465},{"lat":41.3750297749243,"lng":-8.45390321625064},{"lat":41.3747426092203,"lng":-8.45327184748025},{"lat":41.3749899156066,"lng":-8.4529420891846},{"lat":41.3752149025502,"lng":-8.45271033095585},{"lat":41.3752876838214,"lng":-8.45263514241877},{"lat":41.3756231649777,"lng":-8.45228309125547},{"lat":41.3755409682029,"lng":-8.45221454847866},{"lat":41.3754340873436,"lng":-8.45218414545502},{"lat":41.3751331050359,"lng":-8.45208058093004},{"lat":41.375022478085,"lng":-8.45200580132975},{"lat":41.3748359860934,"lng":-8.45188379826048},{"lat":41.3745776921101,"lng":-8.45167955894899},{"lat":41.3745775156703,"lng":-8.45154974265108},{"lat":41.374719743374,"lng":-8.45129157584709},{"lat":41.3748062570189,"lng":-8.45114624027733},{"lat":41.3739572444587,"lng":-8.44933976493084},{"lat":41.3737568038743,"lng":-8.44937679998115},{"lat":41.3735100712264,"lng":-8.44938683148411},{"lat":41.373282133962,"lng":-8.44937149940188},{"lat":41.3730442356022,"lng":-8.44934499907124},{"lat":41.3727983794807,"lng":-8.44933028810821},{"lat":41.3725541479594,"lng":-8.44931416270183},{"lat":41.3723183874442,"lng":-8.44929507940073},{"lat":41.372086366995,"lng":-8.44929179839844},{"lat":41.3718589632377,"lng":-8.44931100791159},{"lat":41.3716319356089,"lng":-8.44939058655077},{"lat":41.371548379072,"lng":-8.44950668508671},{"lat":41.3713162127397,"lng":-8.44982926869854},{"lat":41.3711482817159,"lng":-8.44970210715091},{"lat":41.3709778001007,"lng":-8.44958593293997},{"lat":41.3708067907424,"lng":-8.44948721247021},{"lat":41.3706272365954,"lng":-8.44937349866701},{"lat":41.3704741215146,"lng":-8.44929242960462},{"lat":41.3702731722596,"lng":-8.44918781793065},{"lat":41.3697786404946,"lng":-8.4489409780844},{"lat":41.3696900880587,"lng":-8.44882628253322},{"lat":41.3693570818038,"lng":-8.448384212724},{"lat":41.3692497929879,"lng":-8.44825918817853},{"lat":41.3690968068997,"lng":-8.44808091321526},{"lat":41.368990082799,"lng":-8.44792728793743},{"lat":41.3688676955224,"lng":-8.44780477653291},{"lat":41.3684410894112,"lng":-8.44728051140189},{"lat":41.3683809294309,"lng":-8.44693819745446},{"lat":41.3682101419963,"lng":-8.44596644162465},{"lat":41.3686187485429,"lng":-8.44477664781683},{"lat":41.3679534496623,"lng":-8.44275508561414},{"lat":41.3671527287477,"lng":-8.44179031951862},{"lat":41.365950637687,"lng":-8.44251449787},{"lat":41.3655504743465,"lng":-8.4424254610006},{"lat":41.3654590981942,"lng":-8.44236275448381},{"lat":41.3649004538904,"lng":-8.44113388315161},{"lat":41.3650816776449,"lng":-8.42739387510712},{"lat":41.3651758462303,"lng":-8.4273071922667},{"lat":41.3653181701485,"lng":-8.4272122209417},{"lat":41.365455050681,"lng":-8.42695874884005},{"lat":41.3655032338458,"lng":-8.42670905654484},{"lat":41.3655194979822,"lng":-8.42640603702324},{"lat":41.3655907333411,"lng":-8.42615656821557},{"lat":41.3656964189618,"lng":-8.42588286899938},{"lat":41.3657411075112,"lng":-8.42576653684839},{"lat":41.3658538650439,"lng":-8.4254739899301},{"lat":41.3659948096661,"lng":-8.42514428148306},{"lat":41.3661170873995,"lng":-8.42486025601505},{"lat":41.3663234635562,"lng":-8.42436303033296},{"lat":41.3663706153478,"lng":-8.42409097430134},{"lat":41.3663272506632,"lng":-8.42390122663741},{"lat":41.366244374155,"lng":-8.42374105438468},{"lat":41.3661420675934,"lng":-8.42357267593763},{"lat":41.3660343019461,"lng":-8.42346163489063},{"lat":41.3659385806598,"lng":-8.42335854439531},{"lat":41.3658350762753,"lng":-8.4231303939356},{"lat":41.3658710394378,"lng":-8.42272419570102},{"lat":41.3659843640092,"lng":-8.42252426963784},{"lat":41.3661522182284,"lng":-8.42244518243019},{"lat":41.3663134710851,"lng":-8.42244793340786},{"lat":41.3665276077409,"lng":-8.42243873115785},{"lat":41.3666911536381,"lng":-8.42242583504805},{"lat":41.3668393807661,"lng":-8.42231091459512},{"lat":41.3669328040959,"lng":-8.42216884290041},{"lat":41.3670818512889,"lng":-8.42188507343894},{"lat":41.36712859177,"lng":-8.42174760348679},{"lat":41.3627754637855,"lng":-8.41765841909451},{"lat":41.3602189701495,"lng":-8.4158291219728},{"lat":41.3571256552359,"lng":-8.41431384757669},{"lat":41.3542090675619,"lng":-8.41358990616346},{"lat":41.3507554932369,"lng":-8.4132589264777},{"lat":41.3465268963547,"lng":-8.41324080038705},{"lat":41.3361333285436,"lng":-8.41475715255439},{"lat":41.340307965555,"lng":-8.43146551391605},{"lat":41.3429543623801,"lng":-8.44267623688464},{"lat":41.3399187492223,"lng":-8.44877041334471},{"lat":41.3364539962199,"lng":-8.45577732169977},{"lat":41.3363025161735,"lng":-8.45611482990106},{"lat":41.3513785026285,"lng":-8.46324213595566},{"lat":41.3560888932922,"lng":-8.46554244461326},{"lat":41.359182438485,"lng":-8.46731323116258},{"lat":41.363552539008,"lng":-8.4718891275505},{"lat":41.3702936486296,"lng":-8.47892051841629},{"lat":41.3704789784926,"lng":-8.4791150536287}]},{"zone_id":"arc20","points":[{"lat":40.9864350715973,"lng":-8.4028792995486},{"lat":40.9775661694168,"lng":-8.38970810600835},{"lat":40.9756618384265,"lng":-8.38658556907383},{"lat":40.9724339815285,"lng":-8.38091503462413},{"lat":40.9665398242665,"lng":-8.37300300671714},{"lat":40.9641490423429,"lng":-8.36979435109058},{"lat":40.9611338774571,"lng":-8.36964387490246},{"lat":40.9580814154002,"lng":-8.36949155205126},{"lat":40.9556091806655,"lng":-8.36911960670404},{"lat":40.9542326538411,"lng":-8.36866047690932},{"lat":40.9529426006717,"lng":-8.3682302078258},{"lat":40.951639419689,"lng":-8.36714759030812},{"lat":40.9507341394651,"lng":-8.36639555909393},{"lat":40.9499562721864,"lng":-8.36542388666042},{"lat":40.9485679720087,"lng":-8.36368977496424},{"lat":40.9470394446498,"lng":-8.36095766322538},{"lat":40.9460802130867,"lng":-8.36476096961573},{"lat":40.944637239265,"lng":-8.37023220133922},{"lat":40.9436439040859,"lng":-8.37236259691614},{"lat":40.9428293406312,"lng":-8.37356785318657},{"lat":40.9413368363393,"lng":-8.37521196850881},{"lat":40.939129894942,"lng":-8.37669629089521},{"lat":40.9397048870653,"lng":-8.37846243313495},{"lat":40.9452008810426,"lng":-8.39717789601564},{"lat":40.9461475065768,"lng":-8.39994679056564},{"lat":40.9469227745086,"lng":-8.40170686168011},{"lat":40.9471116383657,"lng":-8.40382704484533},{"lat":40.9473791945719,"lng":-8.40730685406982},{"lat":40.9491261525642,"lng":-8.41655856242714},{"lat":40.9497199974891,"lng":-8.41882062927747},{"lat":40.9487701642549,"lng":-8.42132077301543},{"lat":40.9490546072284,"lng":-8.42424647157619},{"lat":40.9482124542511,"lng":-8.42723373688834},{"lat":40.9482587736953,"lng":-8.42727285164965},{"lat":40.9483389340288,"lng":-8.42739619546508},{"lat":40.9483768972385,"lng":-8.42745461015776},{"lat":40.9483995471687,"lng":-8.42751158778505},{"lat":40.9484231517406,"lng":-8.42757096634126},{"lat":40.9484561097502,"lng":-8.4276781722386},{"lat":40.9485474169313,"lng":-8.42791710978769},{"lat":40.9486085677168,"lng":-8.42809621157682},{"lat":40.9487056522972,"lng":-8.42832455211459},{"lat":40.9489345539526,"lng":-8.42856528392145},{"lat":40.9490838979691,"lng":-8.42864229328237},{"lat":40.9494808366431,"lng":-8.42916031256677},{"lat":40.9495460201651,"lng":-8.42942238790401},{"lat":40.9496160979207,"lng":-8.42970414222288},{"lat":40.9497204792254,"lng":-8.42983912150325},{"lat":40.9501989900455,"lng":-8.42999424170517},{"lat":40.9504780583102,"lng":-8.43005822060324},{"lat":40.9506411258797,"lng":-8.43009800286954},{"lat":40.9508249790319,"lng":-8.43012917891517},{"lat":40.9510413286692,"lng":-8.43027243782837},{"lat":40.9511393906266,"lng":-8.43040129565436},{"lat":40.9512342079289,"lng":-8.43056716920837},{"lat":40.9514152524497,"lng":-8.43080549761826},{"lat":40.9518014436426,"lng":-8.43124667392775},{"lat":40.9522136600049,"lng":-8.43175718653969},{"lat":40.9523706569648,"lng":-8.43192120797163},{"lat":40.9526894949491,"lng":-8.43202619245817},{"lat":40.9527996671099,"lng":-8.43213698996031},{"lat":40.9528643441212,"lng":-8.43227263762702},{"lat":40.9529257409641,"lng":-8.43252381322209},{"lat":40.9529428855601,"lng":-8.43270238275959},{"lat":40.953011450164,"lng":-8.4329095638136},{"lat":40.9530582026871,"lng":-8.43314988156361},{"lat":40.9532297002253,"lng":-8.43321291071501},{"lat":40.9533973388782,"lng":-8.43325985599656},{"lat":40.9535944435476,"lng":-8.43324358409562},{"lat":40.9536561331981,"lng":-8.43303376006512},{"lat":40.9537331108418,"lng":-8.43290144008725},{"lat":40.9538249324446,"lng":-8.43273902207874},{"lat":40.9539906562102,"lng":-8.4324826361214},{"lat":40.9540771628927,"lng":-8.43245539053105},{"lat":40.9541690931836,"lng":-8.43242643667597},{"lat":40.9542514484414,"lng":-8.43247562476249},{"lat":40.9544161664447,"lng":-8.4326029870216},{"lat":40.9545005789295,"lng":-8.43263598634985},{"lat":40.9545455982965,"lng":-8.43269301157111},{"lat":40.9546664555762,"lng":-8.43282504396509},{"lat":40.9547185650305,"lng":-8.43290298900538},{"lat":40.9548037702832,"lng":-8.43303369942379},{"lat":40.9549149261534,"lng":-8.43320458315858},{"lat":40.9550120621531,"lng":-8.43325208421564},{"lat":40.9551563685441,"lng":-8.43332265983777},{"lat":40.955284087778,"lng":-8.43338511948068},{"lat":40.9553544175834,"lng":-8.43341950549201},{"lat":40.9555137498243,"lng":-8.43349740124944},{"lat":40.9555928713547,"lng":-8.43353609380674},{"lat":40.955703657525,"lng":-8.43372830508434},{"lat":40.9557701253194,"lng":-8.43386341703192},{"lat":40.9558071227773,"lng":-8.43391897472069},{"lat":40.9559736068536,"lng":-8.4341276709519},{"lat":40.9560788928381,"lng":-8.43417103218804},{"lat":40.9560956183315,"lng":-8.43387249642153},{"lat":40.9561047784216,"lng":-8.43370899072994},{"lat":40.9561191777982,"lng":-8.43357495877943},{"lat":40.9561195963412,"lng":-8.43347334929557},{"lat":40.9561304340811,"lng":-8.43340266858273},{"lat":40.9561644575212,"lng":-8.43326728809662},{"lat":40.9570771731497,"lng":-8.4317507485397},{"lat":40.9572850990787,"lng":-8.43155583862795},{"lat":40.95740179362,"lng":-8.43153676275119},{"lat":40.9577482681032,"lng":-8.43148012470772},{"lat":40.9581798081605,"lng":-8.43168760427069},{"lat":40.958487735274,"lng":-8.43223535754828},{"lat":40.958837091903,"lng":-8.43289784263206},{"lat":40.9589762038948,"lng":-8.43302515741098},{"lat":40.9590595839858,"lng":-8.43297211704553},{"lat":40.9592158379692,"lng":-8.43290958669808},{"lat":40.959384583554,"lng":-8.43282821402997},{"lat":40.9596828301374,"lng":-8.43271544122098},{"lat":40.9598116724288,"lng":-8.43268609738715},{"lat":40.9599418074626,"lng":-8.43261042987617},{"lat":40.9599999656825,"lng":-8.43247706277944},{"lat":40.960031560254,"lng":-8.4323435742923},{"lat":40.9600600033959,"lng":-8.43220959868934},{"lat":40.960105138057,"lng":-8.43203454112642},{"lat":40.9601369610126,"lng":-8.4318768789128},{"lat":40.960099424012,"lng":-8.43173440581512},{"lat":40.960066053521,"lng":-8.43161654240405},{"lat":40.9600290605599,"lng":-8.43147904923603},{"lat":40.9599782918322,"lng":-8.43132678849094},{"lat":40.9600674420526,"lng":-8.43101728696401},{"lat":40.9602877393003,"lng":-8.43018379972461},{"lat":40.9604686972038,"lng":-8.42960035865704},{"lat":40.9605395066831,"lng":-8.42943366120916},{"lat":40.9606575223132,"lng":-8.42926696376128},{"lat":40.960779471576,"lng":-8.42921487080882},{"lat":40.9609210897919,"lng":-8.42919924292308},{"lat":40.9610548400501,"lng":-8.42915235926586},{"lat":40.9611846562181,"lng":-8.42909505701815},{"lat":40.9613026706946,"lng":-8.42904296406569},{"lat":40.9614560891987,"lng":-8.42897524322749},{"lat":40.9615937721679,"lng":-8.42885022014158},{"lat":40.9617353886366,"lng":-8.42869915057944},{"lat":40.9618534021284,"lng":-8.42856370890303},{"lat":40.9619832167256,"lng":-8.42844389511237},{"lat":40.9620618921149,"lng":-8.42835012779794},{"lat":40.962138600529,"lng":-8.42831887202646},{"lat":40.962217275733,"lng":-8.42836836033131},{"lat":40.9623844602301,"lng":-8.4285142205982},{"lat":40.9625457436966,"lng":-8.42870696452231},{"lat":40.9626716232014,"lng":-8.42888408056068},{"lat":40.9627817675711,"lng":-8.42890491774167},{"lat":40.9629194477749,"lng":-8.4289309642179},{"lat":40.9630885973462,"lng":-8.42894659210363},{"lat":40.9632459453955,"lng":-8.42894138280839},{"lat":40.9633501882715,"lng":-8.42892315027503},{"lat":40.9634962263634,"lng":-8.42873301099853},{"lat":40.9636334133675,"lng":-8.42860017396976},{"lat":40.9638005942773,"lng":-8.42848556947434},{"lat":40.964105452493,"lng":-8.42830324414073},{"lat":40.9643375371583,"lng":-8.42818343035006},{"lat":40.9645263511992,"lng":-8.42813003507379},{"lat":40.9646086052501,"lng":-8.42788534564126},{"lat":40.9646701740319,"lng":-8.42790505863534},{"lat":40.9649597855542,"lng":-8.42794738415921},{"lat":40.9651122121607,"lng":-8.427990360845},{"lat":40.9652066181404,"lng":-8.42801119802598},{"lat":40.9653954296948,"lng":-8.42791743071155},{"lat":40.9659421934402,"lng":-8.42748766385374},{"lat":40.9662195071335,"lng":-8.42721157120569},{"lat":40.9663729142078,"lng":-8.42705789699592},{"lat":40.9665676226733,"lng":-8.42685733912895},{"lat":40.9667190621936,"lng":-8.42669845562394},{"lat":40.9668134658746,"lng":-8.42664636267147},{"lat":40.9671281438359,"lng":-8.42663073478574},{"lat":40.9671684317587,"lng":-8.42637719211335},{"lat":40.9675108292957,"lng":-8.42616720253021},{"lat":40.9676449889013,"lng":-8.42613275677609},{"lat":40.9677879406255,"lng":-8.42647937897856},{"lat":40.9678713715566,"lng":-8.42663924104619},{"lat":40.9679638502985,"lng":-8.42685247989021},{"lat":40.9679255275023,"lng":-8.42708209070618},{"lat":40.9682212719228,"lng":-8.42721073267283},{"lat":40.9684553838015,"lng":-8.42731751320785},{"lat":40.9685940832005,"lng":-8.42744679296015},{"lat":40.9689420328676,"lng":-8.42768642222038},{"lat":40.9690581743758,"lng":-8.42784946804107},{"lat":40.9691219246048,"lng":-8.42793896449302},{"lat":40.9694097324143,"lng":-8.42817559553494},{"lat":40.9697231782659,"lng":-8.42832654039687},{"lat":40.96980548166,"lng":-8.42838538901701},{"lat":40.9698967791232,"lng":-8.42838853203968},{"lat":40.9705231401709,"lng":-8.4283646192674},{"lat":40.9705881478061,"lng":-8.42823603177695},{"lat":40.9705824608498,"lng":-8.42811563143663},{"lat":40.9706017636636,"lng":-8.42796270227242},{"lat":40.9708306397005,"lng":-8.4280060177025},{"lat":40.971037759531,"lng":-8.42799970354852},{"lat":40.971373006859,"lng":-8.42796199898462},{"lat":40.9714931869505,"lng":-8.42787905460398},{"lat":40.9716389954457,"lng":-8.42778878661767},{"lat":40.9717177833682,"lng":-8.42771966535475},{"lat":40.9718475970653,"lng":-8.42762719823687},{"lat":40.9721089133197,"lng":-8.42736839199915},{"lat":40.9721943478902,"lng":-8.4272818427575},{"lat":40.9721080060322,"lng":-8.42710276901377},{"lat":40.9726326854129,"lng":-8.42682818280149},{"lat":40.9726502050964,"lng":-8.42607003885424},{"lat":40.9725820959208,"lng":-8.42597671414528},{"lat":40.9724412731195,"lng":-8.42574914676707},{"lat":40.9723539944304,"lng":-8.42566274670053},{"lat":40.9722652189224,"lng":-8.42562672917238},{"lat":40.9721652517834,"lng":-8.42559801058693},{"lat":40.9720623878208,"lng":-8.42548515021777},{"lat":40.9720270931232,"lng":-8.42532747223201},{"lat":40.9719924666618,"lng":-8.42516931005874},{"lat":40.9719572876719,"lng":-8.42501838920374},{"lat":40.9716254462247,"lng":-8.42357439524102},{"lat":40.9715600639174,"lng":-8.42346583566089},{"lat":40.97148022393,"lng":-8.42331749492872},{"lat":40.9714384611652,"lng":-8.42311127260988},{"lat":40.9714111210381,"lng":-8.42297626964045},{"lat":40.9713628961523,"lng":-8.42275931785107},{"lat":40.9713271987565,"lng":-8.42258642291113},{"lat":40.9712664379392,"lng":-8.42227392665953},{"lat":40.9712454193774,"lng":-8.42212985498498},{"lat":40.9712305521652,"lng":-8.42200370682179},{"lat":40.9712364247748,"lng":-8.42182209529112},{"lat":40.9712581638482,"lng":-8.42164554690995},{"lat":40.9713439896245,"lng":-8.42156941297298},{"lat":40.9719650729499,"lng":-8.42116095497032},{"lat":40.9719714998201,"lng":-8.42102975235342},{"lat":40.972160563061,"lng":-8.4210670398251},{"lat":40.9723124482798,"lng":-8.42112862386726},{"lat":40.9724523241545,"lng":-8.42123081574791},{"lat":40.9725687037102,"lng":-8.42109888210992},{"lat":40.9726238907427,"lng":-8.42085708812919},{"lat":40.972619203802,"lng":-8.42060804716263},{"lat":40.9723192149622,"lng":-8.41961664405814},{"lat":40.972274748674,"lng":-8.41946969504182},{"lat":40.9721440141205,"lng":-8.41877910856246},{"lat":40.9720413123511,"lng":-8.41796900903513},{"lat":40.9721351494603,"lng":-8.41793236314296},{"lat":40.9722296761731,"lng":-8.41789544783287},{"lat":40.97234332677,"lng":-8.41785106396885},{"lat":40.9726580144678,"lng":-8.41772337744206},{"lat":40.9730407494044,"lng":-8.41709025036912},{"lat":40.973318529116,"lng":-8.4169688460024},{"lat":40.9737174548044,"lng":-8.41679449258559},{"lat":40.9738266362027,"lng":-8.41677551128283},{"lat":40.9739711972529,"lng":-8.41679278584218},{"lat":40.9741000615487,"lng":-8.4168334437226},{"lat":40.974297893,"lng":-8.41687175535204},{"lat":40.9745028189772,"lng":-8.41676294802218},{"lat":40.9745325181286,"lng":-8.41658759463162},{"lat":40.9744888748263,"lng":-8.41606390470608},{"lat":40.9745480066729,"lng":-8.41593183562352},{"lat":40.974669253317,"lng":-8.41584087279772},{"lat":40.9748317716662,"lng":-8.41584656503287},{"lat":40.9753220841273,"lng":-8.41587860616202},{"lat":40.9754790543721,"lng":-8.4156857445793},{"lat":40.9757647291192,"lng":-8.41562946662042},{"lat":40.9760739313068,"lng":-8.41529231036703},{"lat":40.9762161235948,"lng":-8.4151558024823},{"lat":40.9763355811942,"lng":-8.41505971579186},{"lat":40.9764198061502,"lng":-8.41486344470038},{"lat":40.9765731675393,"lng":-8.41475321575469},{"lat":40.9766691075123,"lng":-8.41470007323313},{"lat":40.9768463759262,"lng":-8.4145982976618},{"lat":40.9771400634502,"lng":-8.41457627024901},{"lat":40.9772129534927,"lng":-8.41444897609571},{"lat":40.9772789655399,"lng":-8.41454531256297},{"lat":40.9773607012109,"lng":-8.41465172773495},{"lat":40.9774798616155,"lng":-8.41477341122322},{"lat":40.9775978818612,"lng":-8.41478675011904},{"lat":40.9776934323699,"lng":-8.41463400383839},{"lat":40.977732041994,"lng":-8.41450583672948},{"lat":40.9778980174763,"lng":-8.41408393602787},{"lat":40.9780769522065,"lng":-8.41420230732901},{"lat":40.9783044417564,"lng":-8.41410052586478},{"lat":40.9785421989269,"lng":-8.41399861115037},{"lat":40.9786483257985,"lng":-8.41395311957239},{"lat":40.9788341132513,"lng":-8.41390697918157},{"lat":40.97891980035,"lng":-8.41381169740917},{"lat":40.979097564553,"lng":-8.41373154149498},{"lat":40.9792250216321,"lng":-8.41368649796147},{"lat":40.9793261835579,"lng":-8.41365978317022},{"lat":40.9795430087749,"lng":-8.41362709910424},{"lat":40.9797210411543,"lng":-8.41340032310813},{"lat":40.9797938678132,"lng":-8.41329856396079},{"lat":40.9798746742934,"lng":-8.41326189840257},{"lat":40.9799209726626,"lng":-8.41324089069387},{"lat":40.9799780321636,"lng":-8.41321617511643},{"lat":40.9800364112488,"lng":-8.41307271014454},{"lat":40.9800751434282,"lng":-8.41297752651357},{"lat":40.9801161231468,"lng":-8.41293307420734},{"lat":40.9802347948525,"lng":-8.41285763755689},{"lat":40.9802755825484,"lng":-8.41283057029543},{"lat":40.9803690878463,"lng":-8.41277856888781},{"lat":40.98038700853,"lng":-8.41270521800798},{"lat":40.9804739502742,"lng":-8.41246247380628},{"lat":40.9805180756689,"lng":-8.41233927346683},{"lat":40.9805874716371,"lng":-8.41214551532909},{"lat":40.9806167729994,"lng":-8.41201489407749},{"lat":40.9806450294735,"lng":-8.41194942845597},{"lat":40.9806741912784,"lng":-8.41184320489596},{"lat":40.9807024465766,"lng":-8.41166640696831},{"lat":40.9807424397477,"lng":-8.41154977244694},{"lat":40.9808130891091,"lng":-8.41145478051055},{"lat":40.9809515299825,"lng":-8.41128960830316},{"lat":40.981054308564,"lng":-8.41116290759832},{"lat":40.98124930271,"lng":-8.41095888608467},{"lat":40.981360533451,"lng":-8.41083672788559},{"lat":40.9814930584351,"lng":-8.41065737715105},{"lat":40.9815794348205,"lng":-8.41054048035218},{"lat":40.9816495856816,"lng":-8.41050434323548},{"lat":40.9817688713348,"lng":-8.41047224727499},{"lat":40.9819054336109,"lng":-8.41045071937757},{"lat":40.9821458060879,"lng":-8.41043438513567},{"lat":40.9822466214888,"lng":-8.41042350159702},{"lat":40.982389443302,"lng":-8.41040808319531},{"lat":40.9824549718244,"lng":-8.41038079747593},{"lat":40.9825361562545,"lng":-8.4103211285935},{"lat":40.9826336027383,"lng":-8.41020165293555},{"lat":40.9826890754884,"lng":-8.40998869328295},{"lat":40.9827391250845,"lng":-8.40979655149362},{"lat":40.9827878858247,"lng":-8.4095907122421},{"lat":40.9828316453299,"lng":-8.40944319877102},{"lat":40.9828394143678,"lng":-8.40935137999326},{"lat":40.9828503694482,"lng":-8.40922190530205},{"lat":40.9828416816662,"lng":-8.40905421484165},{"lat":40.9828353464536,"lng":-8.40897505492429},{"lat":40.9828264701784,"lng":-8.4088641450216},{"lat":40.9828290120291,"lng":-8.40878062735709},{"lat":40.9828147082558,"lng":-8.40866269073729},{"lat":40.9828302925465,"lng":-8.40847239982725},{"lat":40.9829322137884,"lng":-8.40843908325279},{"lat":40.9830366602096,"lng":-8.4084115822792},{"lat":40.9830610625821,"lng":-8.40829334702216},{"lat":40.9830840809158,"lng":-8.40815572601175},{"lat":40.9831117233643,"lng":-8.40797035731614},{"lat":40.9831074448714,"lng":-8.40783867817713},{"lat":40.9830777390267,"lng":-8.40771790854642},{"lat":40.9830382813878,"lng":-8.40755749379039},{"lat":40.9829786305187,"lng":-8.40734103618948},{"lat":40.9829321383831,"lng":-8.40723360896737},{"lat":40.9828464293186,"lng":-8.4070775952542},{"lat":40.9827891048297,"lng":-8.40688207132095},{"lat":40.9827749738275,"lng":-8.4066464405074},{"lat":40.982815946429,"lng":-8.40656877804249},{"lat":40.9828434000018,"lng":-8.40646124162526},{"lat":40.9828392057161,"lng":-8.40629368177716},{"lat":40.9828548870678,"lng":-8.40620140067795},{"lat":40.9828849335308,"lng":-8.40615021292006},{"lat":40.9829348268918,"lng":-8.40610859686914},{"lat":40.983125478216,"lng":-8.40598367005782},{"lat":40.9831618658631,"lng":-8.40594187686947},{"lat":40.9832348408019,"lng":-8.40577463526693},{"lat":40.9832623079681,"lng":-8.40566103538861},{"lat":40.9833578214405,"lng":-8.40529236255255},{"lat":40.9834786367002,"lng":-8.40523351892107},{"lat":40.9837955622559,"lng":-8.40507915777274},{"lat":40.9844296509412,"lng":-8.40477880735422},{"lat":40.9844423462476,"lng":-8.4047151907356},{"lat":40.984482287748,"lng":-8.4046201790943},{"lat":40.9845292619471,"lng":-8.40453729153669},{"lat":40.9845747163571,"lng":-8.40444980763911},{"lat":40.984596087794,"lng":-8.40435958558565},{"lat":40.98460899174,"lng":-8.40427372425684},{"lat":40.9846284778671,"lng":-8.40410776155164},{"lat":40.9846407492312,"lng":-8.40401945069276},{"lat":40.9846558126222,"lng":-8.40393074262262},{"lat":40.9846759109785,"lng":-8.40384717424079},{"lat":40.9847021772988,"lng":-8.40378323300475},{"lat":40.9847371927227,"lng":-8.40371267426132},{"lat":40.9847891549396,"lng":-8.40363363377588},{"lat":40.9848464985845,"lng":-8.40356482612219},{"lat":40.9849125354489,"lng":-8.40351091580688},{"lat":40.984952402001,"lng":-8.40348315142747},{"lat":40.9851413904023,"lng":-8.40341334573692},{"lat":40.98529669777,"lng":-8.40336562409521},{"lat":40.985356670801,"lng":-8.40334495529716},{"lat":40.9854486799596,"lng":-8.40335353373821},{"lat":40.9854972100235,"lng":-8.40335373304528},{"lat":40.9855432635567,"lng":-8.40333752248808},{"lat":40.9855664082677,"lng":-8.40327757644619},{"lat":40.9856384602529,"lng":-8.40306365894144},{"lat":40.9856882971043,"lng":-8.40300646469373},{"lat":40.9862834597663,"lng":-8.40294652615679},{"lat":40.9864350715973,"lng":-8.4028792995486}]},{"zone_id":"arc11","points":[{"lat":40.9218705797431,"lng":-8.18347163171172},{"lat":40.9213383045184,"lng":-8.18117262190087},{"lat":40.9197065308886,"lng":-8.17412610953536},{"lat":40.9144285391282,"lng":-8.15134918593099},{"lat":40.9114703199093,"lng":-8.14078490109757},{"lat":40.9089689291124,"lng":-8.13246247185172},{"lat":40.9064076205279,"lng":-8.12452183156746},{"lat":40.9059884543609,"lng":-8.12470936693814},{"lat":40.9057093133768,"lng":-8.12482273681295},{"lat":40.9053607525817,"lng":-8.12498703207593},{"lat":40.9051324810917,"lng":-8.12498408216127},{"lat":40.9047329424073,"lng":-8.12493332530527},{"lat":40.9043810906101,"lng":-8.12494308363348},{"lat":40.9041832985811,"lng":-8.12494856919141},{"lat":40.9039005457143,"lng":-8.12488687870447},{"lat":40.9035845684662,"lng":-8.12482139145365},{"lat":40.9027696702339,"lng":-8.12532431956748},{"lat":40.9025361143582,"lng":-8.12582707544822},{"lat":40.9023471510246,"lng":-8.12689521049759},{"lat":40.9012339089238,"lng":-8.12719090697518},{"lat":40.900298061656,"lng":-8.12747839450235},{"lat":40.8998169645801,"lng":-8.12776353418062},{"lat":40.8996147943493,"lng":-8.12788335635531},{"lat":40.8988146437366,"lng":-8.12808727217006},{"lat":40.8981501860349,"lng":-8.12796044464418},{"lat":40.8974748155555,"lng":-8.12736948945749},{"lat":40.8970245342616,"lng":-8.13019023867571},{"lat":40.8961107740743,"lng":-8.13457715303758},{"lat":40.8944135378135,"lng":-8.14008207185198},{"lat":40.8923899502929,"lng":-8.14283425367985},{"lat":40.8896483101803,"lng":-8.14567214169493},{"lat":40.885209431337,"lng":-8.14945543526015},{"lat":40.8820561444451,"lng":-8.15116200021597},{"lat":40.8793113364217,"lng":-8.15198995783962},{"lat":40.8769095539252,"lng":-8.15311928464598},{"lat":40.8746792180503,"lng":-8.15470057292364},{"lat":40.8726774757787,"lng":-8.15680913887174},{"lat":40.8709615646518,"lng":-8.15906834323987},{"lat":40.8697602547953,"lng":-8.16125232360274},{"lat":40.8684444018234,"lng":-8.16396344325584},{"lat":40.8673835805707,"lng":-8.16610495301658},{"lat":40.8659831684243,"lng":-8.16827851365994},{"lat":40.8659522403141,"lng":-8.16840974836154},{"lat":40.8655401002496,"lng":-8.16895277002937},{"lat":40.8654305494323,"lng":-8.16910204539771},{"lat":40.8651527200871,"lng":-8.1694813298699},{"lat":40.8632529502497,"lng":-8.1708208625451},{"lat":40.8630510510472,"lng":-8.17096321703238},{"lat":40.8629185659111,"lng":-8.17113221951923},{"lat":40.8623635925748,"lng":-8.17184015134593},{"lat":40.8621546996592,"lng":-8.17221558045516},{"lat":40.8729032200718,"lng":-8.19065779956533},{"lat":40.8744115772595,"lng":-8.19304559536833},{"lat":40.8762220131489,"lng":-8.19503602378351},{"lat":40.8791966089031,"lng":-8.19679866933537},{"lat":40.8805140916586,"lng":-8.19731801286224},{"lat":40.882849241645,"lng":-8.19510401165546},{"lat":40.8923835934994,"lng":-8.18663586740959},{"lat":40.8978652205561,"lng":-8.18255840961815},{"lat":40.9107076564438,"lng":-8.18159159637543},{"lat":40.9178783771236,"lng":-8.18257332070475},{"lat":40.9218705797431,"lng":-8.18347163171172}]},{"zone_id":"prd2","points":[{"lat":41.1908973134801,"lng":-8.33652260721394},{"lat":41.1821062890519,"lng":-8.33984593263653},{"lat":41.1762956560851,"lng":-8.3417983874746},{"lat":41.1731528759471,"lng":-8.34459252066219},{"lat":41.1731285545993,"lng":-8.34479037910074},{"lat":41.1731014230107,"lng":-8.34499979007048},{"lat":41.1730876314368,"lng":-8.34515418803626},{"lat":41.1730658108738,"lng":-8.34541342352625},{"lat":41.1727582406119,"lng":-8.34649115377873},{"lat":41.1725335196345,"lng":-8.34659893647984},{"lat":41.1723416307687,"lng":-8.34674738569853},{"lat":41.1722111175564,"lng":-8.34700816608739},{"lat":41.1721902751383,"lng":-8.34717588289827},{"lat":41.1721803598124,"lng":-8.34737283346782},{"lat":41.1721889603748,"lng":-8.34787538231056},{"lat":41.1721749997,"lng":-8.34811783890521},{"lat":41.1721609950241,"lng":-8.34823898579307},{"lat":41.1720790194301,"lng":-8.34830401858798},{"lat":41.1718189997577,"lng":-8.34829386323108},{"lat":41.171675489443,"lng":-8.34828409393801},{"lat":41.171575400338,"lng":-8.34826350707758},{"lat":41.1714533075656,"lng":-8.34826012015566},{"lat":41.1712003505521,"lng":-8.34822651742644},{"lat":41.1709680628453,"lng":-8.34816986341943},{"lat":41.1706603139936,"lng":-8.34815955151984},{"lat":41.1703586106886,"lng":-8.34819358714823},{"lat":41.1699419492641,"lng":-8.34830636699034},{"lat":41.169750471083,"lng":-8.34838033139931},{"lat":41.1694495007172,"lng":-8.34850052441734},{"lat":41.1692622118447,"lng":-8.34859677988812},{"lat":41.1691433500491,"lng":-8.34879002529176},{"lat":41.1691312883965,"lng":-8.34907227778187},{"lat":41.1691289598973,"lng":-8.34925803671423},{"lat":41.1691375050565,"lng":-8.3494987671699},{"lat":41.1691802852301,"lng":-8.35010972706538},{"lat":41.1692305136237,"lng":-8.35059082514038},{"lat":41.1692560743957,"lng":-8.35078621429061},{"lat":41.1692937713712,"lng":-8.3509442282322},{"lat":41.1693801096504,"lng":-8.35109322335342},{"lat":41.1695031814343,"lng":-8.35119492732319},{"lat":41.1696142575542,"lng":-8.35121340949807},{"lat":41.1697203790257,"lng":-8.35118599896602},{"lat":41.1698974974768,"lng":-8.35113654626036},{"lat":41.1700445177859,"lng":-8.35105243149645},{"lat":41.1703247477268,"lng":-8.35094969980945},{"lat":41.1705131726437,"lng":-8.35096557810292},{"lat":41.1706395450387,"lng":-8.35113414081631},{"lat":41.1707028033539,"lng":-8.35129629943537},{"lat":41.1707648985348,"lng":-8.35164291012604},{"lat":41.170826023788,"lng":-8.35188573318527},{"lat":41.1709072285543,"lng":-8.35226804491455},{"lat":41.1691192213957,"lng":-8.35734264565782},{"lat":41.1665786255264,"lng":-8.36206302581668},{"lat":41.1627001317258,"lng":-8.36559609853187},{"lat":41.1577747692585,"lng":-8.37129218737127},{"lat":41.1574284440102,"lng":-8.3719972663474},{"lat":41.1572582929215,"lng":-8.37249988087324},{"lat":41.1572785388358,"lng":-8.37289144332876},{"lat":41.157594691715,"lng":-8.3732280985027},{"lat":41.1582497211305,"lng":-8.37332844845914},{"lat":41.1585978041454,"lng":-8.37359517182876},{"lat":41.1589138660186,"lng":-8.37397390020037},{"lat":41.1592716605831,"lng":-8.3747162835991},{"lat":41.1592932979897,"lng":-8.37538296140562},{"lat":41.1591584825296,"lng":-8.37564874417807},{"lat":41.1588649056738,"lng":-8.37596242747568},{"lat":41.1585343884706,"lng":-8.37658298998119},{"lat":41.1580381166742,"lng":-8.37727535920638},{"lat":41.1596150256702,"lng":-8.37987097679534},{"lat":41.1609250580212,"lng":-8.38222129296721},{"lat":41.1625189431214,"lng":-8.38548070755155},{"lat":41.1643406042732,"lng":-8.38911947567435},{"lat":41.1653068725132,"lng":-8.39169578442767},{"lat":41.1657026805302,"lng":-8.39366459485517},{"lat":41.1657559081245,"lng":-8.39540507059208},{"lat":41.1655807470167,"lng":-8.39699330536399},{"lat":41.1655206157482,"lng":-8.39827934223329},{"lat":41.1658037088346,"lng":-8.39949110335717},{"lat":41.1660677886265,"lng":-8.4006775762032},{"lat":41.1665800246304,"lng":-8.40206217515498},{"lat":41.1690599229534,"lng":-8.40149631094441},{"lat":41.1719368231132,"lng":-8.40096088877244},{"lat":41.1749880311084,"lng":-8.40021657423883},{"lat":41.1794651513396,"lng":-8.40099155754089},{"lat":41.18460756063,"lng":-8.40252622092094},{"lat":41.1895593540536,"lng":-8.40406034073962},{"lat":41.1947970068883,"lng":-8.40559587916679},{"lat":41.197662132449,"lng":-8.40623739829209},{"lat":41.2009037729187,"lng":-8.40632431889824},{"lat":41.1992959886587,"lng":-8.40502273916952},{"lat":41.1983374901027,"lng":-8.4034837891075},{"lat":41.1978665449368,"lng":-8.40111620571427},{"lat":41.1976346970579,"lng":-8.39835535004097},{"lat":41.1979425794121,"lng":-8.39394071620701},{"lat":41.1984858905525,"lng":-8.39070979361794},{"lat":41.1997464301978,"lng":-8.38629873771804},{"lat":41.2029507684916,"lng":-8.38049022604812},{"lat":41.2053158999884,"lng":-8.37737623935821},{"lat":41.2072638101314,"lng":-8.37603835372978},{"lat":41.2090368861198,"lng":-8.37530784271146},{"lat":41.2076519746752,"lng":-8.37523530715033},{"lat":41.2071011890791,"lng":-8.37502546429098},{"lat":41.2053223688717,"lng":-8.37434777996523},{"lat":41.203681888508,"lng":-8.3727329735627},{"lat":41.2004758128327,"lng":-8.36780053976812},{"lat":41.198810540529,"lng":-8.36425553340985},{"lat":41.1971045509305,"lng":-8.35955614373595},{"lat":41.1957444274516,"lng":-8.35450721971363},{"lat":41.1953770325566,"lng":-8.35314358768865},{"lat":41.1908973134801,"lng":-8.33652260721394}]},{"zone_id":"arc10","points":[{"lat":40.9467756902433,"lng":-8.17022069650979},{"lat":40.940915781023,"lng":-8.13088987587815},{"lat":40.9406968508521,"lng":-8.13073466934383},{"lat":40.9405174038665,"lng":-8.13062007791288},{"lat":40.9403675108721,"lng":-8.13055391380676},{"lat":40.9402357914993,"lng":-8.1305623516},{"lat":40.9400938230725,"lng":-8.13065455361276},{"lat":40.9398772850825,"lng":-8.13081992977341},{"lat":40.9396934084451,"lng":-8.1308794266596},{"lat":40.9395582592055,"lng":-8.13084708539403},{"lat":40.9394665010983,"lng":-8.13080143840315},{"lat":40.939363076769,"lng":-8.13071896083201},{"lat":40.9392496717792,"lng":-8.13064035711449},{"lat":40.9391245480502,"lng":-8.13060492767447},{"lat":40.9388850724212,"lng":-8.13061664468302},{"lat":40.9387350226284,"lng":-8.13059658404221},{"lat":40.9384534708066,"lng":-8.13058058542637},{"lat":40.9382142280232,"lng":-8.13057997020562},{"lat":40.9379690914283,"lng":-8.13049816930066},{"lat":40.9378496121504,"lng":-8.13048373771448},{"lat":40.9376945014287,"lng":-8.13051188756972},{"lat":40.9374402907821,"lng":-8.13052875153982},{"lat":40.9372505145196,"lng":-8.13052025546205},{"lat":40.9369348087508,"lng":-8.13050215865627},{"lat":40.9367060218803,"lng":-8.13048399922233},{"lat":40.9364659011842,"lng":-8.13042529679816},{"lat":40.9363271255907,"lng":-8.13045517675157},{"lat":40.9362354578007,"lng":-8.1304734643087},{"lat":40.9360396081854,"lng":-8.13054399749172},{"lat":40.9357853132863,"lng":-8.13049294268473},{"lat":40.9355322879168,"lng":-8.13039877940069},{"lat":40.9352353992406,"lng":-8.130294650451},{"lat":40.9350152845069,"lng":-8.13019876034293},{"lat":40.9348603577212,"lng":-8.13013126891747},{"lat":40.9347472487273,"lng":-8.13005971761708},{"lat":40.9346625239286,"lng":-8.13000612197406},{"lat":40.9345565458508,"lng":-8.12996215520519},{"lat":40.9345394525389,"lng":-8.12976631770563},{"lat":40.9345166662318,"lng":-8.12963443087851},{"lat":40.9344669535255,"lng":-8.12945061509962},{"lat":40.934227921096,"lng":-8.1289636573177},{"lat":40.9340911415203,"lng":-8.12873275351582},{"lat":40.9339150898105,"lng":-8.12839557313378},{"lat":40.9337840284611,"lng":-8.12817032354736},{"lat":40.9336927610701,"lng":-8.12804437827273},{"lat":40.9335022614944,"lng":-8.1278900283842},{"lat":40.933282813207,"lng":-8.12770050658229},{"lat":40.9330473264717,"lng":-8.12753576876872},{"lat":40.9328288796058,"lng":-8.12733763266937},{"lat":40.932717765676,"lng":-8.12720042263705},{"lat":40.9325934451018,"lng":-8.1270900158932},{"lat":40.9325114678506,"lng":-8.12701590615535},{"lat":40.9324146391411,"lng":-8.12695578340354},{"lat":40.9322100333024,"lng":-8.12700495724203},{"lat":40.9320813831799,"lng":-8.12706013907542},{"lat":40.931991967235,"lng":-8.12711308467216},{"lat":40.9318633023025,"lng":-8.12718356556536},{"lat":40.9316341891999,"lng":-8.12735726733342},{"lat":40.9315409288628,"lng":-8.12745162857763},{"lat":40.9313583211633,"lng":-8.12773912612125},{"lat":40.931204246427,"lng":-8.12794974117032},{"lat":40.9310026297063,"lng":-8.12796670118175},{"lat":40.9308646488543,"lng":-8.12794498131677},{"lat":40.9306084656352,"lng":-8.12766739756228},{"lat":40.9304781925035,"lng":-8.12727058017366},{"lat":40.93018218592,"lng":-8.12725226436532},{"lat":40.9299415907475,"lng":-8.12721061448276},{"lat":40.9296906515938,"lng":-8.12715926067944},{"lat":40.9294758324064,"lng":-8.12713625668369},{"lat":40.9293506028446,"lng":-8.1271472049898},{"lat":40.929149831188,"lng":-8.12721734525394},{"lat":40.9288537484646,"lng":-8.1272566058658},{"lat":40.928677695656,"lng":-8.12722569503527},{"lat":40.928432780009,"lng":-8.12716651657261},{"lat":40.9282850098051,"lng":-8.12707047194145},{"lat":40.9281766798421,"lng":-8.12688158627112},{"lat":40.9280815801218,"lng":-8.12662609625301},{"lat":40.9278355579867,"lng":-8.12622565195595},{"lat":40.9277745346005,"lng":-8.12612040369151},{"lat":40.927584307813,"lng":-8.12579231802944},{"lat":40.927338101866,"lng":-8.12540684051725},{"lat":40.9270989161278,"lng":-8.12497731305643},{"lat":40.9269553642085,"lng":-8.12467648295615},{"lat":40.9267527483397,"lng":-8.1244459376356},{"lat":40.9264845740528,"lng":-8.12424188401629},{"lat":40.9259676915435,"lng":-8.12394405779131},{"lat":40.9256885343445,"lng":-8.12368729592432},{"lat":40.9255458976284,"lng":-8.12357428632043},{"lat":40.9254447705571,"lng":-8.12349416473062},{"lat":40.9253592143359,"lng":-8.12342637993772},{"lat":40.9251647793606,"lng":-8.12308044956993},{"lat":40.9249118264185,"lng":-8.12290121565563},{"lat":40.9179367031923,"lng":-8.12041597022977},{"lat":40.9178591023185,"lng":-8.12061742964436},{"lat":40.9176325764617,"lng":-8.12093559849302},{"lat":40.9171159898549,"lng":-8.12172888494355},{"lat":40.9167801082939,"lng":-8.1220948958255},{"lat":40.9163643227074,"lng":-8.12273145199724},{"lat":40.9157820843195,"lng":-8.12281140801532},{"lat":40.911027280538,"lng":-8.1245295303123},{"lat":40.910625119608,"lng":-8.12430844543344},{"lat":40.909559845527,"lng":-8.1239613763305},{"lat":40.9091563447285,"lng":-8.12396889419589},{"lat":40.9089818294083,"lng":-8.12390078357484},{"lat":40.9084710733406,"lng":-8.1236937233106},{"lat":40.9080816234124,"lng":-8.12375500916874},{"lat":40.9079702394578,"lng":-8.12380784253659},{"lat":40.9077279316059,"lng":-8.12391990889004},{"lat":40.9073635270613,"lng":-8.12417227832501},{"lat":40.9070270317077,"lng":-8.12432222279683},{"lat":40.9064076205279,"lng":-8.12452183156746},{"lat":40.9089689291124,"lng":-8.13246247185172},{"lat":40.9114703199093,"lng":-8.14078490109757},{"lat":40.9144285391282,"lng":-8.15134918593099},{"lat":40.9197065308886,"lng":-8.17412610953536},{"lat":40.923562028839,"lng":-8.17246509904844},{"lat":40.9292209039514,"lng":-8.17012394009496},{"lat":40.9352521208278,"lng":-8.1692196690494},{"lat":40.9419180928716,"lng":-8.16922329951215},{"lat":40.9467756902433,"lng":-8.17022069650979}]},{"zone_id":"arc16","points":[{"lat":40.9665398242665,"lng":-8.37300300671714},{"lat":40.971315949941,"lng":-8.35742918319059},{"lat":40.972467896656,"lng":-8.35309514339191},{"lat":40.9733320157078,"lng":-8.34970305859324},{"lat":40.9782054454525,"lng":-8.32445430497096},{"lat":40.9780108441517,"lng":-8.32461363997284},{"lat":40.9778047054561,"lng":-8.32479102480985},{"lat":40.9776004103585,"lng":-8.32493989685152},{"lat":40.9773614326931,"lng":-8.32499195390178},{"lat":40.9772141221286,"lng":-8.32493709891027},{"lat":40.9770424137297,"lng":-8.32477049688533},{"lat":40.9770215628851,"lng":-8.32463807544781},{"lat":40.9770568656015,"lng":-8.32452697260946},{"lat":40.9770703482641,"lng":-8.32438158549436},{"lat":40.9769530773936,"lng":-8.32418472102166},{"lat":40.9768158365997,"lng":-8.32413786730067},{"lat":40.9765324642187,"lng":-8.32429815462856},{"lat":40.9763039439079,"lng":-8.32434370267703},{"lat":40.9761019821106,"lng":-8.32428382563704},{"lat":40.9759979208413,"lng":-8.32426629374379},{"lat":40.9757514836113,"lng":-8.32436252521284},{"lat":40.9755758830954,"lng":-8.32436771630066},{"lat":40.9753417657809,"lng":-8.32436691418876},{"lat":40.9752144390026,"lng":-8.32431581264641},{"lat":40.9750063808372,"lng":-8.32418486870271},{"lat":40.9748108779252,"lng":-8.32397876034831},{"lat":40.974676478117,"lng":-8.32373967736254},{"lat":40.9746132026717,"lng":-8.32362436053694},{"lat":40.9745337409279,"lng":-8.32347954547896},{"lat":40.9744330429477,"lng":-8.32327965224796},{"lat":40.9743947294211,"lng":-8.32314504780269},{"lat":40.9743497089776,"lng":-8.32303454512681},{"lat":40.9742826432945,"lng":-8.32291577914235},{"lat":40.9742141151571,"lng":-8.32281067829484},{"lat":40.974156911736,"lng":-8.32271510418234},{"lat":40.9739788252089,"lng":-8.3224851742882},{"lat":40.9738774393413,"lng":-8.32237141549569},{"lat":40.9736656514722,"lng":-8.32231864834588},{"lat":40.9735603254091,"lng":-8.32235719551993},{"lat":40.9734768916892,"lng":-8.32249595844521},{"lat":40.9734367079577,"lng":-8.32262058657873},{"lat":40.9733549129052,"lng":-8.32291083552037},{"lat":40.9732941441894,"lng":-8.32322549979968},{"lat":40.9732217253376,"lng":-8.32345921399907},{"lat":40.9730713467434,"lng":-8.32383409288799},{"lat":40.9729078201386,"lng":-8.32389231172465},{"lat":40.9728155738485,"lng":-8.32386222175511},{"lat":40.9727400534272,"lng":-8.32373594374101},{"lat":40.9727084821795,"lng":-8.32360790220925},{"lat":40.9725819505057,"lng":-8.32340223733563},{"lat":40.9723719872801,"lng":-8.32322781754679},{"lat":40.9721589566527,"lng":-8.32310922188601},{"lat":40.9719624504507,"lng":-8.32296323851682},{"lat":40.9717406404269,"lng":-8.32281646357456},{"lat":40.9714663996442,"lng":-8.32279120065213},{"lat":40.97123812605,"lng":-8.32279814087905},{"lat":40.9710498015202,"lng":-8.32287576522148},{"lat":40.9708999047487,"lng":-8.32301932263307},{"lat":40.9708327295364,"lng":-8.32312937527428},{"lat":40.9708278102647,"lng":-8.32337944451421},{"lat":40.9708289879621,"lng":-8.32369843394146},{"lat":40.9708008325801,"lng":-8.32389579775901},{"lat":40.970735192991,"lng":-8.32400996249688},{"lat":40.9706500097696,"lng":-8.32405728780329},{"lat":40.9705433147006,"lng":-8.3241037839771},{"lat":40.9703589891887,"lng":-8.32410693295993},{"lat":40.9701014597843,"lng":-8.32410487398971},{"lat":40.9700011230794,"lng":-8.32406537511074},{"lat":40.9699398603503,"lng":-8.32397692957706},{"lat":40.9699235354284,"lng":-8.32383349058594},{"lat":40.9699769477868,"lng":-8.32371520082034},{"lat":40.9700471739089,"lng":-8.32361359037909},{"lat":40.970141086871,"lng":-8.32345300527597},{"lat":40.9701563981018,"lng":-8.32323600154338},{"lat":40.9701164445777,"lng":-8.32305886978826},{"lat":40.970049622699,"lng":-8.32295389317369},{"lat":40.9698719384969,"lng":-8.32307764381959},{"lat":40.9697553102768,"lng":-8.32314300204501},{"lat":40.969652377545,"lng":-8.32309399680698},{"lat":40.9695087185144,"lng":-8.32284588477644},{"lat":40.9694721722705,"lng":-8.32267720201985},{"lat":40.9694634129918,"lng":-8.32231292938252},{"lat":40.969433527757,"lng":-8.32219523540093},{"lat":40.9693555094063,"lng":-8.32205684446022},{"lat":40.9692415338214,"lng":-8.32193414832089},{"lat":40.9691068362276,"lng":-8.32187270071826},{"lat":40.9690060839448,"lng":-8.32186765800054},{"lat":40.968771973785,"lng":-8.32186175888298},{"lat":40.9686816916274,"lng":-8.32184272526511},{"lat":40.9685860609976,"lng":-8.32178994086532},{"lat":40.9684606121558,"lng":-8.32179884632564},{"lat":40.9682010358652,"lng":-8.32199969837831},{"lat":40.9680055489966,"lng":-8.32216438412985},{"lat":40.9678799461936,"lng":-8.32221058874983},{"lat":40.9677890403689,"lng":-8.32223967481714},{"lat":40.9676862932992,"lng":-8.32224246544999},{"lat":40.9675219458155,"lng":-8.32214386433157},{"lat":40.9674598813617,"lng":-8.32204864537721},{"lat":40.967436705177,"lng":-8.32163219892824},{"lat":40.9673897651144,"lng":-8.32143082557019},{"lat":40.9673194975611,"lng":-8.32134176282675},{"lat":40.9671222928662,"lng":-8.32117867950694},{"lat":40.9668892828009,"lng":-8.32099946277273},{"lat":40.9666286548747,"lng":-8.32085687711295},{"lat":40.9665142467346,"lng":-8.32077837785874},{"lat":40.9663690524032,"lng":-8.32063790260163},{"lat":40.9663030814075,"lng":-8.32050941879011},{"lat":40.9662552228499,"lng":-8.32037005367294},{"lat":40.9661780559108,"lng":-8.32025995203031},{"lat":40.965974635946,"lng":-8.32020983023466},{"lat":40.965731865921,"lng":-8.32032365792872},{"lat":40.9655532928332,"lng":-8.3204386107457},{"lat":40.9653296795993,"lng":-8.32056710275259},{"lat":40.9651166088492,"lng":-8.32058158066689},{"lat":40.9649849604722,"lng":-8.32058096576891},{"lat":40.9649001065876,"lng":-8.32016020228239},{"lat":40.9646609073363,"lng":-8.31897350172426},{"lat":40.9633604580062,"lng":-8.31393778577869},{"lat":40.9623386153013,"lng":-8.30997458974444},{"lat":40.9622587420798,"lng":-8.30957192693551},{"lat":40.9632930069407,"lng":-8.30557187121528},{"lat":40.9633264816092,"lng":-8.30480945438369},{"lat":40.963327527696,"lng":-8.30411346981489},{"lat":40.9633036205986,"lng":-8.30372484707327},{"lat":40.9632545795332,"lng":-8.3016286134032},{"lat":40.9577287270585,"lng":-8.3038515803286},{"lat":40.9552679802988,"lng":-8.30520285235678},{"lat":40.9530925477517,"lng":-8.30693187319359},{"lat":40.95108736191,"lng":-8.30949083131978},{"lat":40.9495960337514,"lng":-8.31250352072855},{"lat":40.9481607308165,"lng":-8.31619496212588},{"lat":40.9467822450458,"lng":-8.32003723024594},{"lat":40.9453034046925,"lng":-8.32496715961515},{"lat":40.943579322208,"lng":-8.3301151591232},{"lat":40.9422521585335,"lng":-8.33489440357846},{"lat":40.9416642046006,"lng":-8.33782366285319},{"lat":40.9413504424014,"lng":-8.34008490564894},{"lat":40.9412996777743,"lng":-8.34181282411508},{"lat":40.9414871362139,"lng":-8.34354149600075},{"lat":40.9419680660084,"lng":-8.34597914799799},{"lat":40.9434148880855,"lng":-8.35075745330156},{"lat":40.9440149481661,"lng":-8.35273947818776},{"lat":40.945191947632,"lng":-8.35670569507855},{"lat":40.9452458662437,"lng":-8.35689118725546},{"lat":40.9463838125181,"lng":-8.3597858513771},{"lat":40.9470394446498,"lng":-8.36095766322538},{"lat":40.9485679720087,"lng":-8.36368977496424},{"lat":40.9499562721864,"lng":-8.36542388666042},{"lat":40.9507341394651,"lng":-8.36639555909393},{"lat":40.951639419689,"lng":-8.36714759030812},{"lat":40.9529426006717,"lng":-8.3682302078258},{"lat":40.9542326538411,"lng":-8.36866047690932},{"lat":40.9556091806655,"lng":-8.36911960670404},{"lat":40.9580814154002,"lng":-8.36949155205126},{"lat":40.9611338774571,"lng":-8.36964387490246},{"lat":40.9641490423429,"lng":-8.36979435109058},{"lat":40.9665398242665,"lng":-8.37300300671714}]},{"zone_id":"prd4","points":[{"lat":41.2391072843969,"lng":-8.3092483025582},{"lat":41.2389791725011,"lng":-8.30917781716958},{"lat":41.2388415162335,"lng":-8.30910956695937},{"lat":41.2387470437678,"lng":-8.30906231177033},{"lat":41.2386605663755,"lng":-8.30903082888648},{"lat":41.2385149171344,"lng":-8.3090043036844},{"lat":41.2384323328921,"lng":-8.30895707955447},{"lat":41.2383142878368,"lng":-8.30889938971765},{"lat":41.2381216351432,"lng":-8.30875799008897},{"lat":41.2379920163087,"lng":-8.30861676229563},{"lat":41.2356532287097,"lng":-8.30834237416213},{"lat":41.2355131481166,"lng":-8.30944698629885},{"lat":41.2341646065566,"lng":-8.32007646319471},{"lat":41.2338912556442,"lng":-8.32669237094115},{"lat":41.2334552419019,"lng":-8.33091999884936},{"lat":41.233022270071,"lng":-8.33331716642602},{"lat":41.2324947301056,"lng":-8.3352721899959},{"lat":41.2315864250131,"lng":-8.33697353250027},{"lat":41.2256618146287,"lng":-8.34597994900027},{"lat":41.2252306476159,"lng":-8.34724075476494},{"lat":41.2250152687733,"lng":-8.34891393972956},{"lat":41.225310246422,"lng":-8.35041377864947},{"lat":41.2265954482994,"lng":-8.35534412278553},{"lat":41.2272978742477,"lng":-8.3580392228488},{"lat":41.2298689042738,"lng":-8.35903262304606},{"lat":41.2315104904519,"lng":-8.36018067824596},{"lat":41.2338092448124,"lng":-8.36177168296807},{"lat":41.2366616445204,"lng":-8.36500080399247},{"lat":41.2394419002814,"lng":-8.36851400508938},{"lat":41.2415077314181,"lng":-8.37183552493368},{"lat":41.243501197822,"lng":-8.37553575373298},{"lat":41.2477740960372,"lng":-8.38378622721751},{"lat":41.2480252042914,"lng":-8.38343930141895},{"lat":41.2483884473028,"lng":-8.38295730404056},{"lat":41.2491112084269,"lng":-8.38198772218332},{"lat":41.2496008385245,"lng":-8.38133340743119},{"lat":41.2496916363518,"lng":-8.38139770114553},{"lat":41.2498503494433,"lng":-8.38137182044851},{"lat":41.2499988434036,"lng":-8.3812808792092},{"lat":41.2500576891417,"lng":-8.38124417734504},{"lat":41.2501579941462,"lng":-8.38118428371718},{"lat":41.2503116903701,"lng":-8.38109476154923},{"lat":41.2506665245936,"lng":-8.38072995725678},{"lat":41.2507347384968,"lng":-8.38066183633281},{"lat":41.2509889377707,"lng":-8.38040798194159},{"lat":41.2513022635905,"lng":-8.38000006448778},{"lat":41.2516646908734,"lng":-8.37950480845891},{"lat":41.2517830620518,"lng":-8.37934305270345},{"lat":41.2522018286338,"lng":-8.37876359528397},{"lat":41.2522948988702,"lng":-8.37861254496218},{"lat":41.2526611399419,"lng":-8.37800436638225},{"lat":41.2527789610558,"lng":-8.37783989472224},{"lat":41.2529371832121,"lng":-8.37767872533116},{"lat":41.2530110271666,"lng":-8.37759083036687},{"lat":41.253042051387,"lng":-8.37753203038859},{"lat":41.2530725945731,"lng":-8.37747414200118},{"lat":41.2530925919718,"lng":-8.37734931121499},{"lat":41.2531064020239,"lng":-8.37720068385361},{"lat":41.2531365108385,"lng":-8.37705940663804},{"lat":41.2531913729123,"lng":-8.37683566442954},{"lat":41.253248303588,"lng":-8.3764441787483},{"lat":41.2532942686101,"lng":-8.3762113288175},{"lat":41.2533039321761,"lng":-8.37581727716454},{"lat":41.2534771289443,"lng":-8.37596396151055},{"lat":41.2538339155091,"lng":-8.37623983532898},{"lat":41.2539399395779,"lng":-8.37631904327628},{"lat":41.2540353074309,"lng":-8.37624740529546},{"lat":41.2542180034024,"lng":-8.37587199372732},{"lat":41.2550429338215,"lng":-8.3746092474748},{"lat":41.2551869110669,"lng":-8.37449475988584},{"lat":41.2552269313085,"lng":-8.37446293655048},{"lat":41.2555618587248,"lng":-8.37421755525025},{"lat":41.2558282847321,"lng":-8.37405065705939},{"lat":41.2559773182701,"lng":-8.37395849980109},{"lat":41.2560929515058,"lng":-8.37386287735126},{"lat":41.2561880531719,"lng":-8.37386406976099},{"lat":41.256302949177,"lng":-8.37385327478765},{"lat":41.2564430354492,"lng":-8.37381907661298},{"lat":41.2565955334174,"lng":-8.37374769238973},{"lat":41.2566883079954,"lng":-8.37369016253635},{"lat":41.2567777871782,"lng":-8.37361401121095},{"lat":41.2568953254389,"lng":-8.37351147337787},{"lat":41.2572455771067,"lng":-8.37320474597849},{"lat":41.2573125357076,"lng":-8.37314610757967},{"lat":41.2574880358339,"lng":-8.37292672672797},{"lat":41.2576933901414,"lng":-8.3726843124674},{"lat":41.2578697427549,"lng":-8.37248712739264},{"lat":41.2579609845351,"lng":-8.37238782971015},{"lat":41.2579623207324,"lng":-8.37222054425733},{"lat":41.2577762782419,"lng":-8.37076581502223},{"lat":41.2575966571536,"lng":-8.369283008186},{"lat":41.2575545990122,"lng":-8.36893583406141},{"lat":41.2574958015962,"lng":-8.36845049876966},{"lat":41.2574735097571,"lng":-8.36826649845482},{"lat":41.2574215774388,"lng":-8.36784699176005},{"lat":41.2572721843406,"lng":-8.36664027432257},{"lat":41.2572153104949,"lng":-8.36618414514145},{"lat":41.2572154780286,"lng":-8.3661018152475},{"lat":41.2572351783082,"lng":-8.36589128432647},{"lat":41.2572477458846,"lng":-8.36573382955269},{"lat":41.2573866620097,"lng":-8.36516138810713},{"lat":41.2574720013073,"lng":-8.36477732593174},{"lat":41.2573496935452,"lng":-8.36465995509047},{"lat":41.2571173583885,"lng":-8.36445306106012},{"lat":41.2572635388901,"lng":-8.36398716156019},{"lat":41.2570907507036,"lng":-8.36371796021848},{"lat":41.2571786225592,"lng":-8.36354454356291},{"lat":41.2572438082221,"lng":-8.36336877131179},{"lat":41.257282527974,"lng":-8.36323241195882},{"lat":41.2573214144245,"lng":-8.36306119922666},{"lat":41.2573478596267,"lng":-8.36293863780291},{"lat":41.2574183937236,"lng":-8.36265407023482},{"lat":41.257550190991,"lng":-8.36239751469629},{"lat":41.2576373025264,"lng":-8.3622431875074},{"lat":41.2578158713366,"lng":-8.36190232395709},{"lat":41.2579662872299,"lng":-8.36162650128003},{"lat":41.2581994141828,"lng":-8.36120670282708},{"lat":41.2582659414217,"lng":-8.36108690424782},{"lat":41.2583355313077,"lng":-8.36096609573175},{"lat":41.2583724690226,"lng":-8.36090197141334},{"lat":41.25840039382,"lng":-8.36080637461624},{"lat":41.2585380799953,"lng":-8.36043731567849},{"lat":41.2584352218565,"lng":-8.36031882518042},{"lat":41.2584446750631,"lng":-8.36022817206371},{"lat":41.2584959869632,"lng":-8.35996167227411},{"lat":41.2585288520403,"lng":-8.35979309235511},{"lat":41.2585862298606,"lng":-8.3594987719945},{"lat":41.2587149413861,"lng":-8.35893124725618},{"lat":41.2587825177643,"lng":-8.35867355640549},{"lat":41.2588670763309,"lng":-8.35831082077933},{"lat":41.2589629231419,"lng":-8.35810746791835},{"lat":41.2590155394076,"lng":-8.35795300624341},{"lat":41.2590586183888,"lng":-8.35775514883043},{"lat":41.2591090153858,"lng":-8.35739740243088},{"lat":41.2591632150913,"lng":-8.35708820485259},{"lat":41.2591774955915,"lng":-8.35700673681929},{"lat":41.2592353323613,"lng":-8.35666187709354},{"lat":41.2592282488,"lng":-8.35650281152616},{"lat":41.2592040106484,"lng":-8.35606417252324},{"lat":41.2591789440361,"lng":-8.35552701936879},{"lat":41.2591744544393,"lng":-8.35543081738511},{"lat":41.2591764355885,"lng":-8.35536420561273},{"lat":41.259187650666,"lng":-8.35498709886993},{"lat":41.2592018611072,"lng":-8.35480995305586},{"lat":41.2594838745263,"lng":-8.35457584400258},{"lat":41.259685774969,"lng":-8.3544238009685},{"lat":41.2596590621476,"lng":-8.35411311007008},{"lat":41.2596615240391,"lng":-8.3539310688685},{"lat":41.2596765156351,"lng":-8.35367131094074},{"lat":41.2597714359935,"lng":-8.35235428616422},{"lat":41.2596189772419,"lng":-8.35161264934941},{"lat":41.2594910499035,"lng":-8.35094352442454},{"lat":41.2594607240555,"lng":-8.35069904885804},{"lat":41.2594243885965,"lng":-8.35039238391085},{"lat":41.2593850362235,"lng":-8.350017692735},{"lat":41.2593336548841,"lng":-8.34957256163155},{"lat":41.2592826583833,"lng":-8.34911899280321},{"lat":41.2592715827168,"lng":-8.34902048672894},{"lat":41.259257860383,"lng":-8.34894407591367},{"lat":41.2592427859025,"lng":-8.34882183272566},{"lat":41.2593384895356,"lng":-8.34863791434269},{"lat":41.2590487815001,"lng":-8.34799380572991},{"lat":41.2589798413798,"lng":-8.34783368015259},{"lat":41.2590022385178,"lng":-8.34774963447215},{"lat":41.2590874434634,"lng":-8.34744862212296},{"lat":41.2591069822309,"lng":-8.34730561682927},{"lat":41.259120584977,"lng":-8.34706295827522},{"lat":41.2591100616973,"lng":-8.34691126575796},{"lat":41.2591035779336,"lng":-8.34683323869117},{"lat":41.2590709637988,"lng":-8.3464407627633},{"lat":41.2590590217814,"lng":-8.34623203104768},{"lat":41.2590690920771,"lng":-8.34580452347528},{"lat":41.2592087303809,"lng":-8.34580902498541},{"lat":41.2593325284286,"lng":-8.34567198454541},{"lat":41.259365492341,"lng":-8.34533011132471},{"lat":41.2593841104889,"lng":-8.34524258566691},{"lat":41.2594228027841,"lng":-8.34511241000251},{"lat":41.2595032132697,"lng":-8.34486722589923},{"lat":41.2596105602163,"lng":-8.34427525198046},{"lat":41.2594877329601,"lng":-8.34408750825874},{"lat":41.2591603338568,"lng":-8.34384720715848},{"lat":41.2586162438629,"lng":-8.34349319514943},{"lat":41.2583319567164,"lng":-8.34331102716714},{"lat":41.2585833462172,"lng":-8.34307933720974},{"lat":41.2586637844947,"lng":-8.34300520182072},{"lat":41.2585215208634,"lng":-8.3428572599078},{"lat":41.2585078563861,"lng":-8.34279628067662},{"lat":41.258400949999,"lng":-8.34231920504131},{"lat":41.2583432761215,"lng":-8.34199195129685},{"lat":41.2583489526619,"lng":-8.34141075799255},{"lat":41.2583710399142,"lng":-8.34110776155082},{"lat":41.258376562607,"lng":-8.3410319986316},{"lat":41.2584118249461,"lng":-8.34055828168192},{"lat":41.2584178975773,"lng":-8.34038719697121},{"lat":41.2583557742194,"lng":-8.33983859030095},{"lat":41.2581945021078,"lng":-8.3400406864225},{"lat":41.2581593420792,"lng":-8.34007946736899},{"lat":41.2579584825569,"lng":-8.34030101111809},{"lat":41.2578635933429,"lng":-8.34039306088899},{"lat":41.2577681659718,"lng":-8.34018573497704},{"lat":41.257721448656,"lng":-8.34007795758244},{"lat":41.2576966463979,"lng":-8.34000187463714},{"lat":41.2576882003104,"lng":-8.33994230622324},{"lat":41.25762787348,"lng":-8.33984272063753},{"lat":41.2574195039471,"lng":-8.33970280567481},{"lat":41.2571382942687,"lng":-8.33956003956249},{"lat":41.2572976077999,"lng":-8.33879534582936},{"lat":41.2573262145142,"lng":-8.33867648097914},{"lat":41.2572865982853,"lng":-8.33850787549307},{"lat":41.2569067669,"lng":-8.33823404530953},{"lat":41.2568296025403,"lng":-8.3381784159931},{"lat":41.2561834666549,"lng":-8.33772941021055},{"lat":41.2556138622059,"lng":-8.33732861879747},{"lat":41.2555419041834,"lng":-8.33760424199854},{"lat":41.2554807712215,"lng":-8.3378383991498},{"lat":41.2552265454128,"lng":-8.33765778645555},{"lat":41.2554424686902,"lng":-8.33680141376633},{"lat":41.2553545644707,"lng":-8.33676450770091},{"lat":41.2550426812953,"lng":-8.33665459152787},{"lat":41.2549070783956,"lng":-8.33660841663858},{"lat":41.2548513665456,"lng":-8.33658944594792},{"lat":41.2546968847331,"lng":-8.33653514083118},{"lat":41.2543749865914,"lng":-8.3364219845773},{"lat":41.2542966013238,"lng":-8.33639895098541},{"lat":41.2541416619624,"lng":-8.33633773296826},{"lat":41.2538450812896,"lng":-8.33622918432897},{"lat":41.25372596516,"lng":-8.33618677076953},{"lat":41.2532751183615,"lng":-8.33602623966955},{"lat":41.2530226191112,"lng":-8.33593930430005},{"lat":41.2528697777145,"lng":-8.33592853010648},{"lat":41.2525479821419,"lng":-8.33590584605706},{"lat":41.2525082071504,"lng":-8.3356312719975},{"lat":41.2524054816214,"lng":-8.33492216130864},{"lat":41.2518714618302,"lng":-8.33496344855947},{"lat":41.251864524059,"lng":-8.334815479658},{"lat":41.2524219246502,"lng":-8.33376739490114},{"lat":41.2525515665722,"lng":-8.33353563792951},{"lat":41.2528416682887,"lng":-8.33301702545186},{"lat":41.2528922213843,"lng":-8.33299332045493},{"lat":41.253023602081,"lng":-8.33298764428195},{"lat":41.2531327193075,"lng":-8.33304501130667},{"lat":41.2532228049868,"lng":-8.33307189062427},{"lat":41.2534796255505,"lng":-8.33305992025237},{"lat":41.2535467225313,"lng":-8.33304998071814},{"lat":41.253743901785,"lng":-8.33310643090633},{"lat":41.2550539482972,"lng":-8.33349561777785},{"lat":41.2552588910262,"lng":-8.3335405157596},{"lat":41.2554556675895,"lng":-8.33353618600158},{"lat":41.2557629652074,"lng":-8.33352942435445},{"lat":41.2559847921033,"lng":-8.3335235798883},{"lat":41.2564925067196,"lng":-8.33351020300906},{"lat":41.2562829636259,"lng":-8.33209994991738},{"lat":41.2561929505138,"lng":-8.33180900748263},{"lat":41.2555845743603,"lng":-8.33002632360087},{"lat":41.2548218913517,"lng":-8.3275838855318},{"lat":41.2548128205098,"lng":-8.32740894117798},{"lat":41.2548132272542,"lng":-8.32717043377072},{"lat":41.2547702447977,"lng":-8.32687416566972},{"lat":41.2547458070029,"lng":-8.32668485458316},{"lat":41.2546283041982,"lng":-8.32647069232786},{"lat":41.2545540201832,"lng":-8.32636344162042},{"lat":41.254399264971,"lng":-8.32619033259946},{"lat":41.2541144946751,"lng":-8.32585218388098},{"lat":41.254058809907,"lng":-8.3257698129623},{"lat":41.2540092774728,"lng":-8.3256708706271},{"lat":41.2540281352608,"lng":-8.32553932217932},{"lat":41.2539600923506,"lng":-8.32541575037889},{"lat":41.2538734493139,"lng":-8.32532505413402},{"lat":41.2537662487938,"lng":-8.32518228117165},{"lat":41.2537310514023,"lng":-8.32513540435305},{"lat":41.253669000624,"lng":-8.32514345417449},{"lat":41.2535573624382,"lng":-8.32513488975304},{"lat":41.2534768264931,"lng":-8.3250524462086},{"lat":41.2533778515965,"lng":-8.32490408262978},{"lat":41.2533192159285,"lng":-8.32480839390043},{"lat":41.2530976901397,"lng":-8.32444688422667},{"lat":41.2530190820181,"lng":-8.32428605505886},{"lat":41.2529200369097,"lng":-8.32417886259458},{"lat":41.2527484875744,"lng":-8.32424600855102},{"lat":41.2523992716558,"lng":-8.32351467373365},{"lat":41.2523256031379,"lng":-8.32303733063725},{"lat":41.2522148056102,"lng":-8.3225270728707},{"lat":41.2522818240487,"lng":-8.32224042441019},{"lat":41.2523147400576,"lng":-8.32209963610131},{"lat":41.2522560828011,"lng":-8.32203350878247},{"lat":41.2521537737455,"lng":-8.32191817095079},{"lat":41.2520678148549,"lng":-8.32186090914304},{"lat":41.2519058150794,"lng":-8.32180231650364},{"lat":41.2516957535571,"lng":-8.32130823703638},{"lat":41.2514051785917,"lng":-8.32071633647203},{"lat":41.251306761551,"lng":-8.32054403878629},{"lat":41.2512758906819,"lng":-8.32048999367842},{"lat":41.2512019628538,"lng":-8.32021835131978},{"lat":41.2511526809548,"lng":-8.31996324580436},{"lat":41.2511468177212,"lng":-8.31974930814633},{"lat":41.2512217683358,"lng":-8.31945339896184},{"lat":41.2514707507644,"lng":-8.31894418013625},{"lat":41.2517197630177,"lng":-8.31841838280022},{"lat":41.251771950463,"lng":-8.318288664561},{"lat":41.2516015948637,"lng":-8.31810604555117},{"lat":41.2514664326606,"lng":-8.31794602751064},{"lat":41.251316224297,"lng":-8.31779097283829},{"lat":41.2513315217398,"lng":-8.31763138307983},{"lat":41.2513617803402,"lng":-8.31751681185318},{"lat":41.2513992215068,"lng":-8.31741574043365},{"lat":41.2514747292694,"lng":-8.31721647439733},{"lat":41.2515162452205,"lng":-8.31715669684942},{"lat":41.2515353279517,"lng":-8.31699210362208},{"lat":41.2514827586568,"lng":-8.31692717159241},{"lat":41.251464037297,"lng":-8.31686722267631},{"lat":41.2513552545629,"lng":-8.31670740188181},{"lat":41.2512500911457,"lng":-8.31659232601125},{"lat":41.2512163228179,"lng":-8.3165373475478},{"lat":41.2512129946732,"lng":-8.31625803496245},{"lat":41.2512210048349,"lng":-8.31597875402226},{"lat":41.2512403173301,"lng":-8.31567051681352},{"lat":41.2512631585838,"lng":-8.31552096861384},{"lat":41.2512221070398,"lng":-8.31529141809406},{"lat":41.2511847036044,"lng":-8.31514671480542},{"lat":41.2511135467315,"lng":-8.31493700733272},{"lat":41.2510873952316,"lng":-8.31484721118251},{"lat":41.2510725402197,"lng":-8.31473239961371},{"lat":41.2510538814435,"lng":-8.31463260348528},{"lat":41.2510239594349,"lng":-8.31453778966301},{"lat":41.2509452905187,"lng":-8.31435299718691},{"lat":41.2508627569651,"lng":-8.3141582929664},{"lat":41.2507917049642,"lng":-8.31393856706777},{"lat":41.2507355298509,"lng":-8.31376385992726},{"lat":41.2506786054014,"lng":-8.3135471612605},{"lat":41.2506524460498,"lng":-8.31346225911626},{"lat":41.2506189341868,"lng":-8.31324764639571},{"lat":41.2506011892925,"lng":-8.31310002110238},{"lat":41.2505442022976,"lng":-8.31290335698586},{"lat":41.25049175704,"lng":-8.3127586141557},{"lat":41.2504169264181,"lng":-8.31253398781688},{"lat":41.2503719328346,"lng":-8.31245905447023},{"lat":41.2502894682985,"lng":-8.31227926930839},{"lat":41.2502256916515,"lng":-8.31212446871782},{"lat":41.2501797612846,"lng":-8.31201137600683},{"lat":41.250128345343,"lng":-8.31184490671491},{"lat":41.2500624579103,"lng":-8.31168207933185},{"lat":41.249998629398,"lng":-8.31153721168097},{"lat":41.2499086839683,"lng":-8.31136242050074},{"lat":41.2498561269874,"lng":-8.31128746798979},{"lat":41.249823896756,"lng":-8.31093888805959},{"lat":41.2496596922825,"lng":-8.31097709011754},{"lat":41.2495317888505,"lng":-8.31094679231056},{"lat":41.2494340656681,"lng":-8.31090667753038},{"lat":41.2493403843034,"lng":-8.31086475329481},{"lat":41.2491783414131,"lng":-8.31085610928151},{"lat":41.2490767635918,"lng":-8.31086083769813},{"lat":41.2489451113744,"lng":-8.3108703833044},{"lat":41.2488658385488,"lng":-8.31106475198602},{"lat":41.2488692277501,"lng":-8.3112592290097},{"lat":41.2488316057658,"lng":-8.31130899198406},{"lat":41.2487562902482,"lng":-8.31133371870683},{"lat":41.2486472451585,"lng":-8.31127853815889},{"lat":41.2485419483541,"lng":-8.31130318219588},{"lat":41.2484627783676,"lng":-8.31137777038996},{"lat":41.2483986458713,"lng":-8.31150728092205},{"lat":41.2483457753566,"lng":-8.31163180745935},{"lat":41.2483071922709,"lng":-8.31171652122236},{"lat":41.248228244562,"lng":-8.31170628341537},{"lat":41.248145600004,"lng":-8.31164127703581},{"lat":41.2480403850296,"lng":-8.31155616145233},{"lat":41.2479852456764,"lng":-8.31160047333254},{"lat":41.2479125576017,"lng":-8.31166878779731},{"lat":41.2478143416267,"lng":-8.3118098947002},{"lat":41.2477614716807,"lng":-8.31193442108017},{"lat":41.2477801016759,"lng":-8.31205425087188},{"lat":41.2477611193453,"lng":-8.31215882283878},{"lat":41.2477384316434,"lng":-8.31221363959039},{"lat":41.24767808267,"lng":-8.3123431582161},{"lat":41.2476401472963,"lng":-8.31252964347799},{"lat":41.2476249876853,"lng":-8.31260941272593},{"lat":41.2471447077981,"lng":-8.3123816480677},{"lat":41.2471054393205,"lng":-8.31232964037812},{"lat":41.2470035080686,"lng":-8.31204745302548},{"lat":41.2469175418847,"lng":-8.31162978228675},{"lat":41.2468082004751,"lng":-8.31107616373621},{"lat":41.2464869335976,"lng":-8.3100941503271},{"lat":41.246458341673,"lng":-8.30976636778564},{"lat":41.2464518060815,"lng":-8.30970651550716},{"lat":41.2464093544051,"lng":-8.30937360463994},{"lat":41.2462676130235,"lng":-8.3093837186758},{"lat":41.2462678393629,"lng":-8.30923757743405},{"lat":41.2462443711145,"lng":-8.30910186636629},{"lat":41.2459372638258,"lng":-8.30914279046736},{"lat":41.2458396910387,"lng":-8.30912164621804},{"lat":41.245734861762,"lng":-8.30907116815035},{"lat":41.2454353961622,"lng":-8.3089269685045},{"lat":41.2450765301175,"lng":-8.30875416802991},{"lat":41.2449627387974,"lng":-8.30869656185537},{"lat":41.2446414280494,"lng":-8.30853371565714},{"lat":41.2445482732911,"lng":-8.30849412731871},{"lat":41.2442975440916,"lng":-8.30842852609744},{"lat":41.2440143828879,"lng":-8.30823986645929},{"lat":41.2439579886453,"lng":-8.30820545150856},{"lat":41.2438392363042,"lng":-8.30813298234134},{"lat":41.2437904380394,"lng":-8.30810334106185},{"lat":41.2435285271044,"lng":-8.30794425068688},{"lat":41.2434027860644,"lng":-8.30787711710939},{"lat":41.2433515605621,"lng":-8.30784976771647},{"lat":41.2433041758572,"lng":-8.30798015015292},{"lat":41.2432290882085,"lng":-8.30815221205679},{"lat":41.2431146580849,"lng":-8.30831891708404},{"lat":41.243061872154,"lng":-8.30839681591772},{"lat":41.2429420079219,"lng":-8.30828695242123},{"lat":41.2427213289494,"lng":-8.30857325676836},{"lat":41.2425083309114,"lng":-8.30883882847746},{"lat":41.242437327365,"lng":-8.30893001304798},{"lat":41.2422577895488,"lng":-8.30921654516034},{"lat":41.2422122505561,"lng":-8.30926186916544},{"lat":41.2421688835423,"lng":-8.30929825428025},{"lat":41.242129444023,"lng":-8.30935826770222},{"lat":41.2420463519031,"lng":-8.30963981388347},{"lat":41.2419243846804,"lng":-8.30961347729664},{"lat":41.241690345847,"lng":-8.30956846460107},{"lat":41.2416415680533,"lng":-8.30932031846862},{"lat":41.241299266317,"lng":-8.30969618743669},{"lat":41.2412864041123,"lng":-8.30975417079831},{"lat":41.2411108276088,"lng":-8.30998344614319},{"lat":41.2413571948612,"lng":-8.30992677204225},{"lat":41.2414475227883,"lng":-8.30990599297917},{"lat":41.2416462564463,"lng":-8.30995353598255},{"lat":41.2417734979811,"lng":-8.31000410383684},{"lat":41.2412835837706,"lng":-8.31029288269352},{"lat":41.2409473154063,"lng":-8.31023305361168},{"lat":41.2408961540159,"lng":-8.31022395099956},{"lat":41.2407505536416,"lng":-8.31016879362038},{"lat":41.240675709954,"lng":-8.31017896876682},{"lat":41.2404078135081,"lng":-8.31019660692628},{"lat":41.2401225287504,"lng":-8.31021670548522},{"lat":41.2398433848926,"lng":-8.31005407216134},{"lat":41.2398125659204,"lng":-8.31000960731472},{"lat":41.2397359678108,"lng":-8.30987639651724},{"lat":41.23965953277,"lng":-8.30969606},{"lat":41.2393921925027,"lng":-8.30946833123338},{"lat":41.2392548032009,"lng":-8.3093452091061},{"lat":41.2391072843969,"lng":-8.3092483025582}]},{"zone_id":"prd1","points":[{"lat":41.2272978742477,"lng":-8.3580392228488},{"lat":41.2265954482994,"lng":-8.35534412278553},{"lat":41.225310246422,"lng":-8.35041377864947},{"lat":41.2250152687733,"lng":-8.34891393972956},{"lat":41.2252306476159,"lng":-8.34724075476494},{"lat":41.2256618146287,"lng":-8.34597994900027},{"lat":41.2315864250131,"lng":-8.33697353250027},{"lat":41.2324947301056,"lng":-8.3352721899959},{"lat":41.233022270071,"lng":-8.33331716642602},{"lat":41.2334552419019,"lng":-8.33091999884936},{"lat":41.2338912556442,"lng":-8.32669237094115},{"lat":41.2341646065566,"lng":-8.32007646319471},{"lat":41.2355131481166,"lng":-8.30944698629885},{"lat":41.2356532287097,"lng":-8.30834237416213},{"lat":41.2340357516825,"lng":-8.3081647267938},{"lat":41.2339838068051,"lng":-8.30798173457729},{"lat":41.2339424990631,"lng":-8.30777441115889},{"lat":41.2338542370622,"lng":-8.30780389877618},{"lat":41.2337240781313,"lng":-8.30795491443157},{"lat":41.2335725429662,"lng":-8.30771740462467},{"lat":41.2335238723342,"lng":-8.30757352315201},{"lat":41.2334257618976,"lng":-8.30737489810704},{"lat":41.2332962795422,"lng":-8.30713981308445},{"lat":41.2331772866066,"lng":-8.30687553614063},{"lat":41.2325814695982,"lng":-8.30522460726687},{"lat":41.2311955161722,"lng":-8.29960364169014},{"lat":41.2309537699543,"lng":-8.29862333805151},{"lat":41.2309232884826,"lng":-8.29791346560775},{"lat":41.2310107609548,"lng":-8.29682235012502},{"lat":41.2308463039895,"lng":-8.29630675170492},{"lat":41.2308108703411,"lng":-8.29619566294034},{"lat":41.2294371679539,"lng":-8.29646700515074},{"lat":41.2274825390791,"lng":-8.29821600237941},{"lat":41.2243349710928,"lng":-8.30158031045049},{"lat":41.2230886765763,"lng":-8.30226852833206},{"lat":41.2222182483761,"lng":-8.30280925317943},{"lat":41.2220724255812,"lng":-8.30289983922736},{"lat":41.2218004281185,"lng":-8.3030687107564},{"lat":41.2206654843632,"lng":-8.30394964436868},{"lat":41.2201043181635,"lng":-8.30569736277002},{"lat":41.2195489699076,"lng":-8.30748205386757},{"lat":41.2192102248165,"lng":-8.30857060120072},{"lat":41.218479331545,"lng":-8.30957354070814},{"lat":41.2167810859575,"lng":-8.3127219186587},{"lat":41.2152821529839,"lng":-8.31409205734663},{"lat":41.2143079888275,"lng":-8.31460100612119},{"lat":41.2132621837745,"lng":-8.31514718428824},{"lat":41.212823657147,"lng":-8.31521857545581},{"lat":41.2123360728829,"lng":-8.31529817987723},{"lat":41.2121822166708,"lng":-8.31532329864288},{"lat":41.2114652006931,"lng":-8.31544035805017},{"lat":41.2106290828333,"lng":-8.31548773450865},{"lat":41.2107734216798,"lng":-8.31604866209658},{"lat":41.2108003787164,"lng":-8.31619085219663},{"lat":41.2108067548798,"lng":-8.31648730852257},{"lat":41.2107449703998,"lng":-8.3167763998458},{"lat":41.2105800563286,"lng":-8.31713983548683},{"lat":41.2104087644388,"lng":-8.31743791589523},{"lat":41.2101415787167,"lng":-8.3176798000186},{"lat":41.2097855821277,"lng":-8.317884106899},{"lat":41.2096898070512,"lng":-8.31797719958866},{"lat":41.2095389523942,"lng":-8.31833148377341},{"lat":41.2093234161255,"lng":-8.3185370260052},{"lat":41.2090170070938,"lng":-8.31875434549203},{"lat":41.2086818537437,"lng":-8.31876757853184},{"lat":41.2085417027682,"lng":-8.3187390350517},{"lat":41.2084239540824,"lng":-8.31866883550065},{"lat":41.2082499442445,"lng":-8.31842271693859},{"lat":41.2080348922219,"lng":-8.31810769078543},{"lat":41.2079052518442,"lng":-8.31804210946472},{"lat":41.2077408333454,"lng":-8.31809755888731},{"lat":41.2075046500103,"lng":-8.31832079990269},{"lat":41.2072677392943,"lng":-8.31849276812867},{"lat":41.2070760936578,"lng":-8.31851558679214},{"lat":41.2068744636262,"lng":-8.31847769511604},{"lat":41.2067481487761,"lng":-8.31846803275794},{"lat":41.206604270853,"lng":-8.31851888938012},{"lat":41.2064674190047,"lng":-8.31862115335676},{"lat":41.2063714920088,"lng":-8.31875167415922},{"lat":41.2062617894541,"lng":-8.31893783304469},{"lat":41.2061622523192,"lng":-8.31923609491974},{"lat":41.2059819129334,"lng":-8.31972309372529},{"lat":41.2058482118236,"lng":-8.31993731585795},{"lat":41.2056705055817,"lng":-8.32002551219058},{"lat":41.2054343003318,"lng":-8.32009470121744},{"lat":41.2051573004816,"lng":-8.3201686535272},{"lat":41.2048306322263,"lng":-8.32033104991453},{"lat":41.2046455435596,"lng":-8.32052628514062},{"lat":41.204412692223,"lng":-8.32080102062102},{"lat":41.2042687423702,"lng":-8.32105705772012},{"lat":41.2041247502747,"lng":-8.3213413462857},{"lat":41.2040148076283,"lng":-8.32161166400811},{"lat":41.203895643615,"lng":-8.32190854137107},{"lat":41.2037175024092,"lng":-8.32209461030651},{"lat":41.2035138478216,"lng":-8.32219201557256},{"lat":41.2034077640828,"lng":-8.32220100846153},{"lat":41.2032777559323,"lng":-8.32219132884811},{"lat":41.2031483508732,"lng":-8.32203691898205},{"lat":41.203063010774,"lng":-8.32191554306807},{"lat":41.2028784880572,"lng":-8.3218776865162},{"lat":41.2026766533502,"lng":-8.3219097698376},{"lat":41.2024426570741,"lng":-8.32189514003001},{"lat":41.2023094555156,"lng":-8.32180605521836},{"lat":41.2021182032834,"lng":-8.32159089936939},{"lat":41.2020323197696,"lng":-8.32147840000888},{"lat":41.2019682471196,"lng":-8.32139447112458},{"lat":41.2018182345327,"lng":-8.32106748985274},{"lat":41.201719635924,"lng":-8.32079669888727},{"lat":41.2016960642832,"lng":-8.32061935195993},{"lat":41.2016926671434,"lng":-8.32044218310209},{"lat":41.2016144115057,"lng":-8.32028327775605},{"lat":41.2013328947667,"lng":-8.32003520552273},{"lat":41.2012029068606,"lng":-8.32001623860379},{"lat":41.2011038634715,"lng":-8.32001595324155},{"lat":41.2009225275071,"lng":-8.32006192440515},{"lat":41.2008747593957,"lng":-8.32020234401969},{"lat":41.2009035231252,"lng":-8.32034119922323},{"lat":41.2009195109487,"lng":-8.32052900758889},{"lat":41.2009219182808,"lng":-8.32065371755401},{"lat":41.2009034176211,"lng":-8.32095230592112},{"lat":41.2006908680179,"lng":-8.32130863033231},{"lat":41.2005999514867,"lng":-8.3215112713192},{"lat":41.2000186998807,"lng":-8.32256381633257},{"lat":41.1997716626831,"lng":-8.32324417478553},{"lat":41.1995328741823,"lng":-8.32383359227527},{"lat":41.199459826547,"lng":-8.3239582732762},{"lat":41.1993186794621,"lng":-8.32419918805132},{"lat":41.198927854352,"lng":-8.324869809838},{"lat":41.1985918302477,"lng":-8.32539586097993},{"lat":41.1982712784035,"lng":-8.32592898817399},{"lat":41.1981647996344,"lng":-8.32617139183955},{"lat":41.1981130078844,"lng":-8.3265024087686},{"lat":41.1981057511458,"lng":-8.3267961268976},{"lat":41.1980266951283,"lng":-8.3270101126308},{"lat":41.1978743674354,"lng":-8.32720992023188},{"lat":41.1975507300613,"lng":-8.32743344803617},{"lat":41.1973941512903,"lng":-8.32753585946591},{"lat":41.1970707188211,"lng":-8.32774696508953},{"lat":41.1967490455182,"lng":-8.3279324481656},{"lat":41.1966737752945,"lng":-8.32803486114815},{"lat":41.1966222518582,"lng":-8.3282027925302},{"lat":41.1965330211516,"lng":-8.32841245824705},{"lat":41.1962278616357,"lng":-8.32878454296379},{"lat":41.1958324607271,"lng":-8.32916601097214},{"lat":41.1957434162717,"lng":-8.32931976410896},{"lat":41.1956369521319,"lng":-8.32949658663144},{"lat":41.1955095224049,"lng":-8.32982139652841},{"lat":41.1954529148101,"lng":-8.33017014968327},{"lat":41.1954388402607,"lng":-8.33049661394769},{"lat":41.1954006375707,"lng":-8.33077174798719},{"lat":41.1952907373268,"lng":-8.33106049156338},{"lat":41.195199787747,"lng":-8.33122937933577},{"lat":41.1951750742628,"lng":-8.33142296788181},{"lat":41.1950536733776,"lng":-8.33237390033147},{"lat":41.1939774826312,"lng":-8.33473614376966},{"lat":41.1908973134801,"lng":-8.33652260721394},{"lat":41.1953770325566,"lng":-8.35314358768865},{"lat":41.1957444274516,"lng":-8.35450721971363},{"lat":41.1971045509305,"lng":-8.35955614373595},{"lat":41.198810540529,"lng":-8.36425553340985},{"lat":41.2004758128327,"lng":-8.36780053976812},{"lat":41.203681888508,"lng":-8.3727329735627},{"lat":41.2053223688717,"lng":-8.37434777996523},{"lat":41.2071011890791,"lng":-8.37502546429098},{"lat":41.2076519746752,"lng":-8.37523530715033},{"lat":41.2090368861198,"lng":-8.37530784271146},{"lat":41.2114354073136,"lng":-8.37450925804255},{"lat":41.2132481219829,"lng":-8.37350634298991},{"lat":41.2154435784982,"lng":-8.37174755885022},{"lat":41.2179277394045,"lng":-8.36860142928677},{"lat":41.2195542121522,"lng":-8.36549961597103},{"lat":41.2209328094542,"lng":-8.36355112479097},{"lat":41.2233780173449,"lng":-8.35940715264384},{"lat":41.2254674746075,"lng":-8.35841161286455},{"lat":41.2272978742477,"lng":-8.3580392228488}]},{"zone_id":"arc6","points":[{"lat":40.9185937347884,"lng":-8.29282597903297},{"lat":40.9148278970815,"lng":-8.28087985562267},{"lat":40.9113830316793,"lng":-8.28070500253754},{"lat":40.9067382369222,"lng":-8.27983108586241},{"lat":40.9007248703254,"lng":-8.27785527259696},{"lat":40.8951285006973,"lng":-8.27580222088663},{"lat":40.8925095663283,"lng":-8.27430526007792},{"lat":40.8901883310277,"lng":-8.27288753224977},{"lat":40.8857617306792,"lng":-8.26941009786677},{"lat":40.8843171876877,"lng":-8.27431618283853},{"lat":40.8832132951963,"lng":-8.28269081591442},{"lat":40.8822928303437,"lng":-8.2927652312679},{"lat":40.8813785282769,"lng":-8.30233583931086},{"lat":40.8801998843561,"lng":-8.3134532772755},{"lat":40.8815704407001,"lng":-8.31603023058854},{"lat":40.8826804288266,"lng":-8.31866522416963},{"lat":40.8828287960967,"lng":-8.31916819520466},{"lat":40.884586257013,"lng":-8.31891832459348},{"lat":40.8875735246312,"lng":-8.31852318991638},{"lat":40.8917729433435,"lng":-8.31837342465663},{"lat":40.893172606933,"lng":-8.3184165750081},{"lat":40.9051397482978,"lng":-8.31637601625597},{"lat":40.9051667652162,"lng":-8.31545003562483},{"lat":40.9054578810878,"lng":-8.31212215770664},{"lat":40.9060817816783,"lng":-8.30923477203283},{"lat":40.9074269364506,"lng":-8.30576164005727},{"lat":40.9084301164394,"lng":-8.3040684319099},{"lat":40.9094737432467,"lng":-8.30214417753041},{"lat":40.9142503757127,"lng":-8.29607120061705},{"lat":40.9176776514841,"lng":-8.29343213792003},{"lat":40.9185937347884,"lng":-8.29282597903297}]},{"zone_id":"smf12","points":[{"lat":40.9909001312093,"lng":-8.44884157651032},{"lat":40.9878064224684,"lng":-8.44373609474281},{"lat":40.9872382859285,"lng":-8.44237516272507},{"lat":40.9868091201826,"lng":-8.44138157714521},{"lat":40.9853436319597,"lng":-8.43514929351131},{"lat":40.9801161231468,"lng":-8.41293307420734},{"lat":40.9800751434282,"lng":-8.41297752651357},{"lat":40.9800364112488,"lng":-8.41307271014454},{"lat":40.9799780321636,"lng":-8.41321617511643},{"lat":40.9799209726626,"lng":-8.41324089069387},{"lat":40.9798746742934,"lng":-8.41326189840257},{"lat":40.9797938678132,"lng":-8.41329856396079},{"lat":40.9797210411543,"lng":-8.41340032310813},{"lat":40.9795430087749,"lng":-8.41362709910424},{"lat":40.9793261835579,"lng":-8.41365978317022},{"lat":40.9792250216321,"lng":-8.41368649796147},{"lat":40.979097564553,"lng":-8.41373154149498},{"lat":40.97891980035,"lng":-8.41381169740917},{"lat":40.9788341132513,"lng":-8.41390697918157},{"lat":40.9786483257985,"lng":-8.41395311957239},{"lat":40.9785421989269,"lng":-8.41399861115037},{"lat":40.9783044417564,"lng":-8.41410052586478},{"lat":40.9780769522065,"lng":-8.41420230732901},{"lat":40.9778980174763,"lng":-8.41408393602787},{"lat":40.977732041994,"lng":-8.41450583672948},{"lat":40.9776934323699,"lng":-8.41463400383839},{"lat":40.9775978818612,"lng":-8.41478675011904},{"lat":40.9774798616155,"lng":-8.41477341122322},{"lat":40.9773607012109,"lng":-8.41465172773495},{"lat":40.9772789655399,"lng":-8.41454531256297},{"lat":40.9772129534927,"lng":-8.41444897609571},{"lat":40.9771400634502,"lng":-8.41457627024901},{"lat":40.9768463759262,"lng":-8.4145982976618},{"lat":40.9766691075123,"lng":-8.41470007323313},{"lat":40.9765731675393,"lng":-8.41475321575469},{"lat":40.9764198061502,"lng":-8.41486344470038},{"lat":40.9763355811942,"lng":-8.41505971579186},{"lat":40.9762161235948,"lng":-8.4151558024823},{"lat":40.9760739313068,"lng":-8.41529231036703},{"lat":40.9757647291192,"lng":-8.41562946662042},{"lat":40.9754790543721,"lng":-8.4156857445793},{"lat":40.9753220841273,"lng":-8.41587860616202},{"lat":40.9748317716662,"lng":-8.41584656503287},{"lat":40.974669253317,"lng":-8.41584087279772},{"lat":40.9745480066729,"lng":-8.41593183562352},{"lat":40.9744888748263,"lng":-8.41606390470608},{"lat":40.9745325181286,"lng":-8.41658759463162},{"lat":40.9745028189772,"lng":-8.41676294802218},{"lat":40.974297893,"lng":-8.41687175535204},{"lat":40.9741000615487,"lng":-8.4168334437226},{"lat":40.9739711972529,"lng":-8.41679278584218},{"lat":40.9738266362027,"lng":-8.41677551128283},{"lat":40.9737174548044,"lng":-8.41679449258559},{"lat":40.973318529116,"lng":-8.4169688460024},{"lat":40.9730407494044,"lng":-8.41709025036912},{"lat":40.9726580144678,"lng":-8.41772337744206},{"lat":40.97234332677,"lng":-8.41785106396885},{"lat":40.9722296761731,"lng":-8.41789544783287},{"lat":40.9721351494603,"lng":-8.41793236314296},{"lat":40.9720413123511,"lng":-8.41796900903513},{"lat":40.9721440141205,"lng":-8.41877910856246},{"lat":40.972274748674,"lng":-8.41946969504182},{"lat":40.9723192149622,"lng":-8.41961664405814},{"lat":40.972619203802,"lng":-8.42060804716263},{"lat":40.9726238907427,"lng":-8.42085708812919},{"lat":40.9725687037102,"lng":-8.42109888210992},{"lat":40.9724523241545,"lng":-8.42123081574791},{"lat":40.9723124482798,"lng":-8.42112862386726},{"lat":40.972160563061,"lng":-8.4210670398251},{"lat":40.9719714998201,"lng":-8.42102975235342},{"lat":40.9719650729499,"lng":-8.42116095497032},{"lat":40.9713439896245,"lng":-8.42156941297298},{"lat":40.9712581638482,"lng":-8.42164554690995},{"lat":40.9712364247748,"lng":-8.42182209529112},{"lat":40.9712305521652,"lng":-8.42200370682179},{"lat":40.9712454193774,"lng":-8.42212985498498},{"lat":40.9712664379392,"lng":-8.42227392665953},{"lat":40.9713271987565,"lng":-8.42258642291113},{"lat":40.9713628961523,"lng":-8.42275931785107},{"lat":40.9714111210381,"lng":-8.42297626964045},{"lat":40.9714384611652,"lng":-8.42311127260988},{"lat":40.97148022393,"lng":-8.42331749492872},{"lat":40.9715600639174,"lng":-8.42346583566089},{"lat":40.9716254462247,"lng":-8.42357439524102},{"lat":40.9719572876719,"lng":-8.42501838920374},{"lat":40.9719924666618,"lng":-8.42516931005874},{"lat":40.9720270931232,"lng":-8.42532747223201},{"lat":40.9720623878208,"lng":-8.42548515021777},{"lat":40.9721652517834,"lng":-8.42559801058693},{"lat":40.9722652189224,"lng":-8.42562672917238},{"lat":40.9723539944304,"lng":-8.42566274670053},{"lat":40.9724412731195,"lng":-8.42574914676707},{"lat":40.9725820959208,"lng":-8.42597671414528},{"lat":40.9726502050964,"lng":-8.42607003885424},{"lat":40.9726326854129,"lng":-8.42682818280149},{"lat":40.9721080060322,"lng":-8.42710276901377},{"lat":40.9721943478902,"lng":-8.4272818427575},{"lat":40.9721089133197,"lng":-8.42736839199915},{"lat":40.9718475970653,"lng":-8.42762719823687},{"lat":40.9717177833682,"lng":-8.42771966535475},{"lat":40.9716389954457,"lng":-8.42778878661767},{"lat":40.9714931869505,"lng":-8.42787905460398},{"lat":40.971373006859,"lng":-8.42796199898462},{"lat":40.971037759531,"lng":-8.42799970354852},{"lat":40.9708306397005,"lng":-8.4280060177025},{"lat":40.9706017636636,"lng":-8.42796270227242},{"lat":40.9705824608498,"lng":-8.42811563143663},{"lat":40.9705881478061,"lng":-8.42823603177695},{"lat":40.9705231401709,"lng":-8.4283646192674},{"lat":40.9698967791232,"lng":-8.42838853203968},{"lat":40.96980548166,"lng":-8.42838538901701},{"lat":40.9697231782659,"lng":-8.42832654039687},{"lat":40.9694097324143,"lng":-8.42817559553494},{"lat":40.9691219246048,"lng":-8.42793896449302},{"lat":40.9690581743758,"lng":-8.42784946804107},{"lat":40.9689420328676,"lng":-8.42768642222038},{"lat":40.9685940832005,"lng":-8.42744679296015},{"lat":40.9684553838015,"lng":-8.42731751320785},{"lat":40.9682212719228,"lng":-8.42721073267283},{"lat":40.9679255275023,"lng":-8.42708209070618},{"lat":40.9679638502985,"lng":-8.42685247989021},{"lat":40.9678713715566,"lng":-8.42663924104619},{"lat":40.9677879406255,"lng":-8.42647937897856},{"lat":40.9676449889013,"lng":-8.42613275677609},{"lat":40.9675108292957,"lng":-8.42616720253021},{"lat":40.9671684317587,"lng":-8.42637719211335},{"lat":40.9671281438359,"lng":-8.42663073478574},{"lat":40.9668134658746,"lng":-8.42664636267147},{"lat":40.9667190621936,"lng":-8.42669845562394},{"lat":40.9665676226733,"lng":-8.42685733912895},{"lat":40.9663729142078,"lng":-8.42705789699592},{"lat":40.9662195071335,"lng":-8.42721157120569},{"lat":40.9659421934402,"lng":-8.42748766385374},{"lat":40.9653954296948,"lng":-8.42791743071155},{"lat":40.9652066181404,"lng":-8.42801119802598},{"lat":40.9651122121607,"lng":-8.427990360845},{"lat":40.9649597855542,"lng":-8.42794738415921},{"lat":40.9646701740319,"lng":-8.42790505863534},{"lat":40.9646086052501,"lng":-8.42788534564126},{"lat":40.9645263511992,"lng":-8.42813003507379},{"lat":40.9643375371583,"lng":-8.42818343035006},{"lat":40.964105452493,"lng":-8.42830324414073},{"lat":40.9638005942773,"lng":-8.42848556947434},{"lat":40.9636334133675,"lng":-8.42860017396976},{"lat":40.9634962263634,"lng":-8.42873301099853},{"lat":40.9633501882715,"lng":-8.42892315027503},{"lat":40.9632459453955,"lng":-8.42894138280839},{"lat":40.9630885973462,"lng":-8.42894659210363},{"lat":40.9629194477749,"lng":-8.4289309642179},{"lat":40.9627817675711,"lng":-8.42890491774167},{"lat":40.9626716232014,"lng":-8.42888408056068},{"lat":40.9625457436966,"lng":-8.42870696452231},{"lat":40.9623844602301,"lng":-8.4285142205982},{"lat":40.962217275733,"lng":-8.42836836033131},{"lat":40.962138600529,"lng":-8.42831887202646},{"lat":40.9620618921149,"lng":-8.42835012779794},{"lat":40.9619832167256,"lng":-8.42844389511237},{"lat":40.9618534021284,"lng":-8.42856370890303},{"lat":40.9617353886366,"lng":-8.42869915057944},{"lat":40.9615937721679,"lng":-8.42885022014158},{"lat":40.9614560891987,"lng":-8.42897524322749},{"lat":40.9613026706946,"lng":-8.42904296406569},{"lat":40.9611846562181,"lng":-8.42909505701815},{"lat":40.9610548400501,"lng":-8.42915235926586},{"lat":40.9609210897919,"lng":-8.42919924292308},{"lat":40.960779471576,"lng":-8.42921487080882},{"lat":40.9606575223132,"lng":-8.42926696376128},{"lat":40.9605395066831,"lng":-8.42943366120916},{"lat":40.9604686972038,"lng":-8.42960035865704},{"lat":40.9602877393003,"lng":-8.43018379972461},{"lat":40.9600674420526,"lng":-8.43101728696401},{"lat":40.9599782918322,"lng":-8.43132678849094},{"lat":40.9600290605599,"lng":-8.43147904923603},{"lat":40.960066053521,"lng":-8.43161654240405},{"lat":40.960099424012,"lng":-8.43173440581512},{"lat":40.9601369610126,"lng":-8.4318768789128},{"lat":40.960105138057,"lng":-8.43203454112642},{"lat":40.9600600033959,"lng":-8.43220959868934},{"lat":40.960031560254,"lng":-8.4323435742923},{"lat":40.9599999656825,"lng":-8.43247706277944},{"lat":40.9599418074626,"lng":-8.43261042987617},{"lat":40.9598116724288,"lng":-8.43268609738715},{"lat":40.9596828301374,"lng":-8.43271544122098},{"lat":40.959384583554,"lng":-8.43282821402997},{"lat":40.9592158379692,"lng":-8.43290958669808},{"lat":40.9590595839858,"lng":-8.43297211704553},{"lat":40.9589762038948,"lng":-8.43302515741098},{"lat":40.958837091903,"lng":-8.43289784263206},{"lat":40.958487735274,"lng":-8.43223535754828},{"lat":40.9581798081605,"lng":-8.43168760427069},{"lat":40.9577482681032,"lng":-8.43148012470772},{"lat":40.95740179362,"lng":-8.43153676275119},{"lat":40.9572850990787,"lng":-8.43155583862795},{"lat":40.9570771731497,"lng":-8.4317507485397},{"lat":40.9561644575212,"lng":-8.43326728809662},{"lat":40.9561304340811,"lng":-8.43340266858273},{"lat":40.9561195963412,"lng":-8.43347334929557},{"lat":40.9561191777982,"lng":-8.43357495877943},{"lat":40.9561047784216,"lng":-8.43370899072994},{"lat":40.9560956183315,"lng":-8.43387249642153},{"lat":40.9560788928381,"lng":-8.43417103218804},{"lat":40.9559736068536,"lng":-8.4341276709519},{"lat":40.9558071227773,"lng":-8.43391897472069},{"lat":40.9557701253194,"lng":-8.43386341703192},{"lat":40.955703657525,"lng":-8.43372830508434},{"lat":40.9555928713547,"lng":-8.43353609380674},{"lat":40.9555137498243,"lng":-8.43349740124944},{"lat":40.9553544175834,"lng":-8.43341950549201},{"lat":40.955284087778,"lng":-8.43338511948068},{"lat":40.9551563685441,"lng":-8.43332265983777},{"lat":40.9550120621531,"lng":-8.43325208421564},{"lat":40.9549149261534,"lng":-8.43320458315858},{"lat":40.9548037702832,"lng":-8.43303369942379},{"lat":40.9547185650305,"lng":-8.43290298900538},{"lat":40.9546664555762,"lng":-8.43282504396509},{"lat":40.9545455982965,"lng":-8.43269301157111},{"lat":40.9545005789295,"lng":-8.43263598634985},{"lat":40.9544161664447,"lng":-8.4326029870216},{"lat":40.9542514484414,"lng":-8.43247562476249},{"lat":40.9541690931836,"lng":-8.43242643667597},{"lat":40.9540771628927,"lng":-8.43245539053105},{"lat":40.9539906562102,"lng":-8.4324826361214},{"lat":40.9538249324446,"lng":-8.43273902207874},{"lat":40.9537331108418,"lng":-8.43290144008725},{"lat":40.9536561331981,"lng":-8.43303376006512},{"lat":40.9535944435476,"lng":-8.43324358409562},{"lat":40.9533973388782,"lng":-8.43325985599656},{"lat":40.9532297002253,"lng":-8.43321291071501},{"lat":40.9530582026871,"lng":-8.43314988156361},{"lat":40.953011450164,"lng":-8.4329095638136},{"lat":40.9529428855601,"lng":-8.43270238275959},{"lat":40.9529257409641,"lng":-8.43252381322209},{"lat":40.9528643441212,"lng":-8.43227263762702},{"lat":40.9527996671099,"lng":-8.43213698996031},{"lat":40.9526894949491,"lng":-8.43202619245817},{"lat":40.9523706569648,"lng":-8.43192120797163},{"lat":40.9522136600049,"lng":-8.43175718653969},{"lat":40.9518014436426,"lng":-8.43124667392775},{"lat":40.9514152524497,"lng":-8.43080549761826},{"lat":40.9512342079289,"lng":-8.43056716920837},{"lat":40.9511393906266,"lng":-8.43040129565436},{"lat":40.9510413286692,"lng":-8.43027243782837},{"lat":40.9508249790319,"lng":-8.43012917891517},{"lat":40.9506411258797,"lng":-8.43009800286954},{"lat":40.9504780583102,"lng":-8.43005822060324},{"lat":40.9501989900455,"lng":-8.42999424170517},{"lat":40.9497204792254,"lng":-8.42983912150325},{"lat":40.9496160979207,"lng":-8.42970414222288},{"lat":40.9495460201651,"lng":-8.42942238790401},{"lat":40.9494808366431,"lng":-8.42916031256677},{"lat":40.9490838979691,"lng":-8.42864229328237},{"lat":40.9489345539526,"lng":-8.42856528392145},{"lat":40.9487056522972,"lng":-8.42832455211459},{"lat":40.9486085677168,"lng":-8.42809621157682},{"lat":40.9485474169313,"lng":-8.42791710978769},{"lat":40.9484561097502,"lng":-8.4276781722386},{"lat":40.9484231517406,"lng":-8.42757096634126},{"lat":40.9483995471687,"lng":-8.42751158778505},{"lat":40.9483768972385,"lng":-8.42745461015776},{"lat":40.9483389340288,"lng":-8.42739619546508},{"lat":40.9482587736953,"lng":-8.42727285164965},{"lat":40.9482124542511,"lng":-8.42723373688834},{"lat":40.9481563083386,"lng":-8.42727001809662},{"lat":40.9481076815799,"lng":-8.42736513574921},{"lat":40.9480305928661,"lng":-8.42751592650412},{"lat":40.9479625906965,"lng":-8.42760609477419},{"lat":40.9478376770503,"lng":-8.42774435253547},{"lat":40.9477474739865,"lng":-8.42780929345917},{"lat":40.947548814931,"lng":-8.4279523165553},{"lat":40.9474092337873,"lng":-8.42801779956562},{"lat":40.947277573138,"lng":-8.42804620286779},{"lat":40.9471178857433,"lng":-8.42808054786465},{"lat":40.9469662675474,"lng":-8.42810789602193},{"lat":40.9468540590968,"lng":-8.42814729672057},{"lat":40.946731933786,"lng":-8.42825152133831},{"lat":40.9465868542971,"lng":-8.4284556369756},{"lat":40.9464746498438,"lng":-8.42858429118744},{"lat":40.9463823050104,"lng":-8.42867697267791},{"lat":40.9461504025198,"lng":-8.42893008564272},{"lat":40.9460497545849,"lng":-8.42903593322114},{"lat":40.9459262990961,"lng":-8.42916576297374},{"lat":40.9458667051602,"lng":-8.42927213273934},{"lat":40.9458380971008,"lng":-8.42932319613754},{"lat":40.9457986088013,"lng":-8.42944640486914},{"lat":40.9458116820235,"lng":-8.42954420361369},{"lat":40.9458987053457,"lng":-8.42971842371364},{"lat":40.945136040731,"lng":-8.43074817256776},{"lat":40.944886312275,"lng":-8.43106495675323},{"lat":40.9438548652311,"lng":-8.43237332408613},{"lat":40.9437148935046,"lng":-8.43255087110701},{"lat":40.9478650537169,"lng":-8.43912640966637},{"lat":40.9497551977389,"lng":-8.44504276414396},{"lat":40.9530703467639,"lng":-8.45247469602237},{"lat":40.9563892777256,"lng":-8.45839890926084},{"lat":40.9583802013173,"lng":-8.46205428862203},{"lat":40.9608056366624,"lng":-8.46517731235596},{"lat":40.961418563104,"lng":-8.46596657865386},{"lat":40.963699165782,"lng":-8.46824100818505},{"lat":40.9659811927095,"lng":-8.47001270225091},{"lat":40.9694675524778,"lng":-8.47038959956035},{"lat":40.9727184928219,"lng":-8.47089025658346},{"lat":40.9763472602788,"lng":-8.47119065079732},{"lat":40.9788419221188,"lng":-8.47199170203429},{"lat":40.9821498807099,"lng":-8.47298608326142},{"lat":40.9824795982894,"lng":-8.47147472514984},{"lat":40.9835029216451,"lng":-8.46778580778187},{"lat":40.9848241588363,"lng":-8.46401966534918},{"lat":40.9868666752995,"lng":-8.45789885047271},{"lat":40.9884269288176,"lng":-8.45374045469553},{"lat":40.9893052440854,"lng":-8.45200079157313},{"lat":40.9909001312093,"lng":-8.44884157651032}]},{"zone_id":"oaz8","points":[{"lat":40.8764152238787,"lng":-8.43401835290799},{"lat":40.8781899296018,"lng":-8.43488892314045},{"lat":40.8787665412871,"lng":-8.43517178704534},{"lat":40.8807452550922,"lng":-8.43665012304198},{"lat":40.8828478595615,"lng":-8.43812911621268},{"lat":40.8856904287262,"lng":-8.44091779504638},{"lat":40.8902050587118,"lng":-8.44395945690879},{"lat":40.8979674549647,"lng":-8.44877238666552},{"lat":40.9030299186591,"lng":-8.44865403676439},{"lat":40.9085124653555,"lng":-8.45121637282024},{"lat":40.913982768633,"lng":-8.45307035248724},{"lat":40.9154514351228,"lng":-8.45406593802366},{"lat":40.916523054763,"lng":-8.4558332705752},{"lat":40.9168597569308,"lng":-8.4554939873115},{"lat":40.9192121345555,"lng":-8.45266250067477},{"lat":40.9193812902543,"lng":-8.45267088130324},{"lat":40.9200098916775,"lng":-8.45293987690152},{"lat":40.9204830063931,"lng":-8.45332300111914},{"lat":40.9207712785047,"lng":-8.45356847435415},{"lat":40.9213631714985,"lng":-8.45388368682266},{"lat":40.9218584420796,"lng":-8.45429613702035},{"lat":40.9220354599584,"lng":-8.45458986825164},{"lat":40.9223005564985,"lng":-8.45516728761662},{"lat":40.922428279781,"lng":-8.45557035703423},{"lat":40.9226772767549,"lng":-8.45607528771197},{"lat":40.9227536223242,"lng":-8.45620779320878},{"lat":40.9228572014688,"lng":-8.45635835807512},{"lat":40.9229441563239,"lng":-8.45652866826712},{"lat":40.9230143650275,"lng":-8.45663407599117},{"lat":40.9231332438401,"lng":-8.45672357923557},{"lat":40.9232434272695,"lng":-8.45683084576805},{"lat":40.9233924026127,"lng":-8.45700846595366},{"lat":40.9235034101712,"lng":-8.45714280650488},{"lat":40.9236460167808,"lng":-8.45731149302896},{"lat":40.9237342701174,"lng":-8.45737211737943},{"lat":40.9240933689808,"lng":-8.45761125433916},{"lat":40.92688470488,"lng":-8.45382364412427},{"lat":40.9293643458148,"lng":-8.455350796362},{"lat":40.9294977707474,"lng":-8.455583324756},{"lat":40.9295972868592,"lng":-8.4556421072984},{"lat":40.9297431007899,"lng":-8.45575976903932},{"lat":40.9298420590139,"lng":-8.45585666057783},{"lat":40.9299776210668,"lng":-8.4559054096331},{"lat":40.9301030602636,"lng":-8.45587171167981},{"lat":40.9302129353358,"lng":-8.4557689552032},{"lat":40.9303174645521,"lng":-8.45574203919465},{"lat":40.9304323623617,"lng":-8.45574260165567},{"lat":40.9304796245408,"lng":-8.45561899826526},{"lat":40.9304643046143,"lng":-8.45549508694955},{"lat":40.9303395936806,"lng":-8.45530177893593},{"lat":40.9302250060315,"lng":-8.45519115634734},{"lat":40.9302139095388,"lng":-8.45503829694804},{"lat":40.9302068177939,"lng":-8.45467768030266},{"lat":40.9303724670087,"lng":-8.45452972032852},{"lat":40.9303766284364,"lng":-8.45436221193248},{"lat":40.9304307190487,"lng":-8.45411480514512},{"lat":40.9305709504698,"lng":-8.45404009404157},{"lat":40.9306378439838,"lng":-8.45385104542671},{"lat":40.9306675495041,"lng":-8.4536613393992},{"lat":40.930715976038,"lng":-8.45347469310945},{"lat":40.9308284337188,"lng":-8.45344603178496},{"lat":40.9309317739729,"lng":-8.45339310496147},{"lat":40.9310441345532,"lng":-8.45323823156833},{"lat":40.9311512281325,"lng":-8.45316347675165},{"lat":40.9313039610586,"lng":-8.45299692508422},{"lat":40.9314331427577,"lng":-8.4529125399806},{"lat":40.9315199958511,"lng":-8.45283044271288},{"lat":40.9315923256659,"lng":-8.45266076872319},{"lat":40.9316682257628,"lng":-8.45253492423039},{"lat":40.9317582456854,"lng":-8.45247943777466},{"lat":40.931885416727,"lng":-8.45247031753068},{"lat":40.9320513450777,"lng":-8.45241531737189},{"lat":40.9322003145296,"lng":-8.45233862444262},{"lat":40.9323761998672,"lng":-8.45203836788936},{"lat":40.9324557887624,"lng":-8.45188083789032},{"lat":40.9325126415583,"lng":-8.45173922633139},{"lat":40.9325939859833,"lng":-8.45163026558633},{"lat":40.9326991403256,"lng":-8.45157259383688},{"lat":40.9327841849811,"lng":-8.45149285928545},{"lat":40.9329005501877,"lng":-8.45138656049754},{"lat":40.932842438991,"lng":-8.45113824286927},{"lat":40.9326532631996,"lng":-8.45113436258388},{"lat":40.932548955901,"lng":-8.45101654974138},{"lat":40.9324145978238,"lng":-8.45079256422354},{"lat":40.9321394583896,"lng":-8.45077758354087},{"lat":40.9310529970727,"lng":-8.44651310406278},{"lat":40.9310251159933,"lng":-8.44636918857651},{"lat":40.9310386353259,"lng":-8.44603502423634},{"lat":40.9310298499503,"lng":-8.44555863313699},{"lat":40.9309582742353,"lng":-8.44512492552346},{"lat":40.9309415170043,"lng":-8.44483039309734},{"lat":40.9310398994589,"lng":-8.44464563884301},{"lat":40.9311983600329,"lng":-8.4445206511549},{"lat":40.9301692223924,"lng":-8.43981302727087},{"lat":40.9317096253472,"lng":-8.43813126098627},{"lat":40.932366392312,"lng":-8.43741419036069},{"lat":40.9323058808982,"lng":-8.43716179976485},{"lat":40.9325244891062,"lng":-8.43683118436246},{"lat":40.9325818160225,"lng":-8.43672136749393},{"lat":40.9326482986688,"lng":-8.43662871990369},{"lat":40.9326614631754,"lng":-8.43648618138454},{"lat":40.9327595861256,"lng":-8.4365305647499},{"lat":40.932852788167,"lng":-8.43659677309599},{"lat":40.9334129922521,"lng":-8.43679051696585},{"lat":40.933831299113,"lng":-8.43667263943295},{"lat":40.9345156440075,"lng":-8.43698047318441},{"lat":40.9344966403854,"lng":-8.43613188821211},{"lat":40.9350207170072,"lng":-8.4360605622676},{"lat":40.9351663807967,"lng":-8.43607227407468},{"lat":40.9354713764506,"lng":-8.43617080539872},{"lat":40.9357324887817,"lng":-8.43624693129724},{"lat":40.9357589619926,"lng":-8.43607689899959},{"lat":40.9357154220998,"lng":-8.43585655581045},{"lat":40.9357003518468,"lng":-8.43542546272475},{"lat":40.9358642395022,"lng":-8.43552643089662},{"lat":40.9359960706532,"lng":-8.43562725214243},{"lat":40.9361402964116,"lng":-8.43556996895872},{"lat":40.9362643229976,"lng":-8.4355915555177},{"lat":40.9363658393575,"lng":-8.43568083995075},{"lat":40.9364421758967,"lng":-8.43555283160286},{"lat":40.9365718259119,"lng":-8.43545665266599},{"lat":40.9367087682265,"lng":-8.43542937729343},{"lat":40.9368340530781,"lng":-8.43548587990451},{"lat":40.9373217194112,"lng":-8.43559688594091},{"lat":40.9374081452706,"lng":-8.43543329851103},{"lat":40.9370168718412,"lng":-8.43513488594397},{"lat":40.9368632279798,"lng":-8.43490548620395},{"lat":40.9365682370339,"lng":-8.43466593940057},{"lat":40.9363823980583,"lng":-8.43448875885677},{"lat":40.9361901756181,"lng":-8.43396482822861},{"lat":40.9360378318161,"lng":-8.43375360620638},{"lat":40.9356162369703,"lng":-8.43365075461933},{"lat":40.9356547400469,"lng":-8.43345833407393},{"lat":40.9357045386303,"lng":-8.4333197551976},{"lat":40.9357845008223,"lng":-8.43314735269655},{"lat":40.9358063542639,"lng":-8.43298608496306},{"lat":40.9357810162129,"lng":-8.43282733368793},{"lat":40.9357297316703,"lng":-8.43270812357864},{"lat":40.9356935729014,"lng":-8.43255502210172},{"lat":40.9356746805931,"lng":-8.43241054924017},{"lat":40.9356751913263,"lng":-8.43221510666053},{"lat":40.9356969252727,"lng":-8.43209907739462},{"lat":40.9357124353603,"lng":-8.43191795122648},{"lat":40.935693330173,"lng":-8.43183067022664},{"lat":40.9347936300796,"lng":-8.4295944262802},{"lat":40.9343348716649,"lng":-8.42845422728013},{"lat":40.9344800347524,"lng":-8.42820280358766},{"lat":40.9345923820948,"lng":-8.42800821563464},{"lat":40.9344966580907,"lng":-8.42692395317739},{"lat":40.9340629817877,"lng":-8.4256115659135},{"lat":40.9337996887288,"lng":-8.4248148213887},{"lat":40.9340215788415,"lng":-8.42270895576157},{"lat":40.9340903257275,"lng":-8.42205643111367},{"lat":40.9332557644854,"lng":-8.42127552974447},{"lat":40.9314253130417,"lng":-8.41971843311033},{"lat":40.9302255038092,"lng":-8.41869735519584},{"lat":40.9297821732361,"lng":-8.41827703276632},{"lat":40.9284112824421,"lng":-8.41512401397534},{"lat":40.9276046536437,"lng":-8.41358188434503},{"lat":40.9277590116668,"lng":-8.41316961421527},{"lat":40.9278278901064,"lng":-8.41273194989534},{"lat":40.9279825490847,"lng":-8.41174918093324},{"lat":40.9279969665355,"lng":-8.41081205344062},{"lat":40.9279998556461,"lng":-8.41062420301177},{"lat":40.9280048942251,"lng":-8.41029654615833},{"lat":40.9278793064308,"lng":-8.40883118174522},{"lat":40.9278699689949,"lng":-8.40870743079146},{"lat":40.9278091901966,"lng":-8.40860021634708},{"lat":40.9277189090946,"lng":-8.40844096021711},{"lat":40.9276692516734,"lng":-8.40834171962552},{"lat":40.927591864628,"lng":-8.40824451667509},{"lat":40.9275205913639,"lng":-8.40815161344386},{"lat":40.9274510934703,"lng":-8.40806803406211},{"lat":40.9273790687961,"lng":-8.40798908210546},{"lat":40.9272954983948,"lng":-8.40790860995239},{"lat":40.9272021251567,"lng":-8.40780265845943},{"lat":40.9271472471858,"lng":-8.40770817203738},{"lat":40.927076447263,"lng":-8.40760362799247},{"lat":40.9270021346317,"lng":-8.40751765833018},{"lat":40.9268873974841,"lng":-8.4073890511788},{"lat":40.9267346562304,"lng":-8.40721753946064},{"lat":40.9265150549089,"lng":-8.40698262153407},{"lat":40.9264393977693,"lng":-8.40691522879144},{"lat":40.92635751972,"lng":-8.40685065872324},{"lat":40.9262549464936,"lng":-8.40677982979147},{"lat":40.9261622512341,"lng":-8.40672008431634},{"lat":40.9260615216404,"lng":-8.40664933461268},{"lat":40.9259722375245,"lng":-8.40658218996879},{"lat":40.9258838712683,"lng":-8.40652183011155},{"lat":40.925801667258,"lng":-8.40647174338787},{"lat":40.9257047378943,"lng":-8.40641354356053},{"lat":40.925562594484,"lng":-8.40630045935192},{"lat":40.9254006984617,"lng":-8.40618889643565},{"lat":40.9252731714369,"lng":-8.40613623966161},{"lat":40.925189368243,"lng":-8.40608542948108},{"lat":40.9250957264515,"lng":-8.40605976274101},{"lat":40.9249972540391,"lng":-8.40605660858068},{"lat":40.9248971507191,"lng":-8.40605680559966},{"lat":40.9247214707012,"lng":-8.40607300974942},{"lat":40.924627676424,"lng":-8.40609796242729},{"lat":40.9244725222804,"lng":-8.40603795208789},{"lat":40.924385192921,"lng":-8.40597291572358},{"lat":40.9243066466863,"lng":-8.40577175420949},{"lat":40.9242583399467,"lng":-8.40565198241525},{"lat":40.9242055638694,"lng":-8.40554279931335},{"lat":40.9241546673809,"lng":-8.4054367738918},{"lat":40.9240691796009,"lng":-8.40530972819083},{"lat":40.9240117434553,"lng":-8.40521790724784},{"lat":40.9238938373073,"lng":-8.40504302204048},{"lat":40.9238302063881,"lng":-8.40495395897497},{"lat":40.9236835384192,"lng":-8.40475663770642},{"lat":40.9236094746493,"lng":-8.4046605987893},{"lat":40.9235321877479,"lng":-8.40455965430892},{"lat":40.9234551526459,"lng":-8.40446495639116},{"lat":40.9233777595152,"lng":-8.40436930629248},{"lat":40.9233055747,"lng":-8.40427937738719},{"lat":40.9232391202933,"lng":-8.40416537341975},{"lat":40.9231738763299,"lng":-8.40403475286042},{"lat":40.9231177284556,"lng":-8.40394109226553},{"lat":40.9230533901258,"lng":-8.40384495680872},{"lat":40.9229364432378,"lng":-8.40368368038173},{"lat":40.9228657337705,"lng":-8.40357915772862},{"lat":40.9228057539456,"lng":-8.4034686232315},{"lat":40.9227578249484,"lng":-8.40336466868096},{"lat":40.9226627617952,"lng":-8.4032390333004},{"lat":40.9225831461981,"lng":-8.40314990618636},{"lat":40.9224767414576,"lng":-8.40304191416064},{"lat":40.9223898859148,"lng":-8.40295599801327},{"lat":40.9208965958612,"lng":-8.40151472277315},{"lat":40.9207923196559,"lng":-8.4014390337958},{"lat":40.9206676967301,"lng":-8.40136160079947},{"lat":40.9204937699967,"lng":-8.40124454720039},{"lat":40.9204058928325,"lng":-8.40118614632535},{"lat":40.9202180450451,"lng":-8.40103687368561},{"lat":40.9201410610233,"lng":-8.40096774260023},{"lat":40.920026730936,"lng":-8.40088631614138},{"lat":40.9199226253454,"lng":-8.40079611163834},{"lat":40.9198442844145,"lng":-8.40072165840658},{"lat":40.9197650205781,"lng":-8.40065218766903},{"lat":40.919637137548,"lng":-8.40058682714147},{"lat":40.919541869144,"lng":-8.40054361024402},{"lat":40.9194516406955,"lng":-8.40049505357875},{"lat":40.9193589014931,"lng":-8.40046368741568},{"lat":40.9192674660003,"lng":-8.4004417111427},{"lat":40.9191745208212,"lng":-8.40037176950238},{"lat":40.9190237201369,"lng":-8.40020579532071},{"lat":40.9189587714012,"lng":-8.4001154990489},{"lat":40.9188826833392,"lng":-8.40002336038181},{"lat":40.9188087163513,"lng":-8.39994510695134},{"lat":40.9187224873609,"lng":-8.39987028820663},{"lat":40.9186156197287,"lng":-8.39982403403876},{"lat":40.9183514646,"lng":-8.39992208661827},{"lat":40.9180687324223,"lng":-8.40005767178642},{"lat":40.9179708209548,"lng":-8.40004554369222},{"lat":40.9178592810959,"lng":-8.39991783667408},{"lat":40.917774076919,"lng":-8.39981053631044},{"lat":40.9176978136373,"lng":-8.39973081254202},{"lat":40.917629463099,"lng":-8.39961705140432},{"lat":40.9175759311275,"lng":-8.39951985164546},{"lat":40.9174872685303,"lng":-8.39949803038342},{"lat":40.9173966195129,"lng":-8.39946467679248},{"lat":40.917309443617,"lng":-8.39941944466806},{"lat":40.9171787550357,"lng":-8.3993182239369},{"lat":40.9170683749428,"lng":-8.39923349643282},{"lat":40.9169696460814,"lng":-8.39917255738523},{"lat":40.9168612815632,"lng":-8.39911146118741},{"lat":40.916767650174,"lng":-8.3990653813107},{"lat":40.9166719715827,"lng":-8.39900932199185},{"lat":40.9165445456092,"lng":-8.39893616061412},{"lat":40.9164223464012,"lng":-8.39888202406569},{"lat":40.9163305253872,"lng":-8.39884971019668},{"lat":40.9162375516233,"lng":-8.39883081736927},{"lat":40.9161044775363,"lng":-8.39882458307137},{"lat":40.9159833623465,"lng":-8.39882551901647},{"lat":40.9158827086231,"lng":-8.39881787139796},{"lat":40.915733526012,"lng":-8.39880729928283},{"lat":40.9156268164331,"lng":-8.39880210641661},{"lat":40.9155261541221,"lng":-8.3987971009334},{"lat":40.9154397032787,"lng":-8.39875389923642},{"lat":40.915358407588,"lng":-8.39868423336974},{"lat":40.9152755549456,"lng":-8.39863590420869},{"lat":40.9151853472266,"lng":-8.39864151479305},{"lat":40.9150494400092,"lng":-8.39861985366657},{"lat":40.9150075812256,"lng":-8.39841815306803},{"lat":40.9148999363301,"lng":-8.39843626920533},{"lat":40.9147833778966,"lng":-8.39829372620604},{"lat":40.9146854974345,"lng":-8.3980421575884},{"lat":40.9144703716897,"lng":-8.39793004375398},{"lat":40.9143777458824,"lng":-8.39766889192839},{"lat":40.9142464018304,"lng":-8.39764369268769},{"lat":40.9141090460144,"lng":-8.39747858888554},{"lat":40.9140104119268,"lng":-8.39720888606972},{"lat":40.9138889865212,"lng":-8.39716995265374},{"lat":40.913907249917,"lng":-8.3970502438142},{"lat":40.9138016346455,"lng":-8.3970074803468},{"lat":40.9136850763901,"lng":-8.39670733251772},{"lat":40.9136223052174,"lng":-8.39662134930127},{"lat":40.913525864238,"lng":-8.39666333381321},{"lat":40.913381825454,"lng":-8.39672401203532},{"lat":40.9132951402688,"lng":-8.3967587388214},{"lat":40.9132017315739,"lng":-8.39672424035729},{"lat":40.9130655567635,"lng":-8.3966577201873},{"lat":40.9130030857595,"lng":-8.39656615587025},{"lat":40.9129409812178,"lng":-8.39647891507199},{"lat":40.9128638359329,"lng":-8.39637773734533},{"lat":40.9127915867045,"lng":-8.39628650090719},{"lat":40.9127234652875,"lng":-8.39619354528796},{"lat":40.9124786839723,"lng":-8.39612583986682},{"lat":40.9123349695929,"lng":-8.39608645100531},{"lat":40.912192699543,"lng":-8.39604520529546},{"lat":40.9120610478826,"lng":-8.39600808577162},{"lat":40.9119174279126,"lng":-8.39596715590101},{"lat":40.9117477113987,"lng":-8.39591864371135},{"lat":40.9115626091158,"lng":-8.39586508555906},{"lat":40.9113858713301,"lng":-8.39581547782736},{"lat":40.9111438045077,"lng":-8.39574626186443},{"lat":40.9109754405556,"lng":-8.39569728256724},{"lat":40.9108491902007,"lng":-8.39566010234671},{"lat":40.9106446365751,"lng":-8.3956069423231},{"lat":40.910544478055,"lng":-8.39557900640363},{"lat":40.9104443201565,"lng":-8.39555118926737},{"lat":40.9103215727475,"lng":-8.39552027397738},{"lat":40.9102002761047,"lng":-8.39559852496024},{"lat":40.9101166559625,"lng":-8.39570181149419},{"lat":40.9100543149944,"lng":-8.39587105768237},{"lat":40.9100104275701,"lng":-8.39600381982059},{"lat":40.9099657033149,"lng":-8.39614809067451},{"lat":40.909731904308,"lng":-8.39585113682766},{"lat":40.9096724255054,"lng":-8.39575452045553},{"lat":40.9095550819213,"lng":-8.39556378837528},{"lat":40.909473354421,"lng":-8.39543052685414},{"lat":40.9093656124888,"lng":-8.39527580629665},{"lat":40.9092916245362,"lng":-8.39516946939825},{"lat":40.909164035089,"lng":-8.39498609630184},{"lat":40.9090705557471,"lng":-8.3949674487453},{"lat":40.9089737709432,"lng":-8.3949610112096},{"lat":40.9088837877475,"lng":-8.39498648814478},{"lat":40.9087970417557,"lng":-8.39502558908062},{"lat":40.9087098737279,"lng":-8.39506607180221},{"lat":40.9085987800094,"lng":-8.39509506551678},{"lat":40.9085092921686,"lng":-8.39512267546131},{"lat":40.9084160716286,"lng":-8.39512350716703},{"lat":40.9083249639972,"lng":-8.39512756629015},{"lat":40.9082358279072,"lng":-8.39517065796542},{"lat":40.9081476756631,"lng":-8.39523961750325},{"lat":40.9080550248689,"lng":-8.39531521054377},{"lat":40.9079742219644,"lng":-8.3953797176489},{"lat":40.9078833467219,"lng":-8.39537600843361},{"lat":40.9077923034278,"lng":-8.39533944562145},{"lat":40.9076855303516,"lng":-8.39532952500261},{"lat":40.9075826042783,"lng":-8.39533101509033},{"lat":40.9074905250273,"lng":-8.39531403113901},{"lat":40.9073966301981,"lng":-8.39523209024063},{"lat":40.907298102141,"lng":-8.39508226443901},{"lat":40.9072066315157,"lng":-8.39495440351755},{"lat":40.9071249945314,"lng":-8.3948174999008},{"lat":40.9070558347156,"lng":-8.39472086448753},{"lat":40.906964557604,"lng":-8.39457377360982},{"lat":40.9068119318968,"lng":-8.39441441228634},{"lat":40.9066993774234,"lng":-8.39433576706513},{"lat":40.9066072151836,"lng":-8.39430996348693},{"lat":40.9064598000088,"lng":-8.39439937058344},{"lat":40.9063795802705,"lng":-8.39455657043658},{"lat":40.9063070892577,"lng":-8.39471523251276},{"lat":40.9061922240933,"lng":-8.39492813918884},{"lat":40.906087353523,"lng":-8.39504769356805},{"lat":40.9060331971043,"lng":-8.39494761016641},{"lat":40.9060044253752,"lng":-8.39482296106297},{"lat":40.9059717311401,"lng":-8.39463515382902},{"lat":40.9059458257287,"lng":-8.39448186701572},{"lat":40.9058990820051,"lng":-8.39437566684891},{"lat":40.9058225075565,"lng":-8.39428010181559},{"lat":40.9057484758647,"lng":-8.39419728123048},{"lat":40.9056257920149,"lng":-8.39407394069859},{"lat":40.9054571567135,"lng":-8.3938999996502},{"lat":40.905367521866,"lng":-8.39380506666819},{"lat":40.9053222803345,"lng":-8.39366315575684},{"lat":40.9053235553471,"lng":-8.39353250204658},{"lat":40.905329443757,"lng":-8.39340350018012},{"lat":40.905339805315,"lng":-8.39320284427466},{"lat":40.9053489621149,"lng":-8.39301678203368},{"lat":40.9053636925344,"lng":-8.39283513294967},{"lat":40.9053782402652,"lng":-8.3926937178245},{"lat":40.905384938108,"lng":-8.39256731606093},{"lat":40.9053913654955,"lng":-8.39242075649167},{"lat":40.9054090786913,"lng":-8.39204595201221},{"lat":40.9054184032612,"lng":-8.39189761554879},{"lat":40.9054164295909,"lng":-8.39145574098101},{"lat":40.9054217173987,"lng":-8.3911073022441},{"lat":40.9054227425463,"lng":-8.39097224198239},{"lat":40.9054473121123,"lng":-8.39081792837507},{"lat":40.9055125430475,"lng":-8.39071239945976},{"lat":40.9055450657391,"lng":-8.3905969941774},{"lat":40.9055836431802,"lng":-8.39046660187504},{"lat":40.905624738562,"lng":-8.39034912700885},{"lat":40.9056624022633,"lng":-8.39023645163607},{"lat":40.9057151473522,"lng":-8.39000974459709},{"lat":40.9057591829303,"lng":-8.38983176008948},{"lat":40.9057698157576,"lng":-8.38962926104351},{"lat":40.9057609076048,"lng":-8.38947737847929},{"lat":40.9057491215687,"lng":-8.38932235869172},{"lat":40.9057361002087,"lng":-8.38920189557274},{"lat":40.9057276509429,"lng":-8.38907711729655},{"lat":40.9057283737202,"lng":-8.3889568825662},{"lat":40.905726576611,"lng":-8.38876015951313},{"lat":40.9057011864052,"lng":-8.38854193139708},{"lat":40.9057182949279,"lng":-8.38833230423556},{"lat":40.9057379422551,"lng":-8.38821150987438},{"lat":40.9057865261977,"lng":-8.38797872878051},{"lat":40.9058321910882,"lng":-8.3877798570099},{"lat":40.9058567864803,"lng":-8.38766494559775},{"lat":40.9058737931591,"lng":-8.38752077228684},{"lat":40.9058622772211,"lng":-8.38740265786531},{"lat":40.9058441965063,"lng":-8.38728094704432},{"lat":40.9058048784236,"lng":-8.38703511485468},{"lat":40.9057726322472,"lng":-8.38685268899997},{"lat":40.9057990486116,"lng":-8.38669959071388},{"lat":40.9057772557959,"lng":-8.38652565547747},{"lat":40.9056855866919,"lng":-8.38644843662203},{"lat":40.9055810963113,"lng":-8.38635697054542},{"lat":40.9054781306088,"lng":-8.38619195325461},{"lat":40.9053636030354,"lng":-8.38600830616704},{"lat":40.9052695099526,"lng":-8.38585685630426},{"lat":40.9051783206814,"lng":-8.38569685832529},{"lat":40.9051285402323,"lng":-8.38558759397049},{"lat":40.9050652546412,"lng":-8.3854444386756},{"lat":40.9050148424643,"lng":-8.3853315057895},{"lat":40.9049520381516,"lng":-8.38519344473161},{"lat":40.904196947553,"lng":-8.38569820675863},{"lat":40.9039002371082,"lng":-8.38592542200049},{"lat":40.903629531348,"lng":-8.38538661732374},{"lat":40.9022577263428,"lng":-8.38566329230541},{"lat":40.9005672296887,"lng":-8.38695039050173},{"lat":40.9004521962991,"lng":-8.38699726684677},{"lat":40.9003168931956,"lng":-8.38697440537373},{"lat":40.9002187505953,"lng":-8.38695288406253},{"lat":40.9001029289937,"lng":-8.38692193767525},{"lat":40.8999572370661,"lng":-8.38686022897186},{"lat":40.8998560210092,"lng":-8.38695916181121},{"lat":40.8997791771858,"lng":-8.38709605460398},{"lat":40.8997189039608,"lng":-8.38719159359471},{"lat":40.8996201699156,"lng":-8.38733267318612},{"lat":40.8995585201444,"lng":-8.38743983554163},{"lat":40.8995012810052,"lng":-8.38754772784286},{"lat":40.89945727603,"lng":-8.38769685126865},{"lat":40.8994332388766,"lng":-8.38785661326311},{"lat":40.8994088968049,"lng":-8.38799085864706},{"lat":40.8993588631616,"lng":-8.38829969371775},{"lat":40.899326466898,"lng":-8.38849241378947},{"lat":40.8992936922384,"lng":-8.38869320335198},{"lat":40.8992469629179,"lng":-8.38897297455929},{"lat":40.8992096965683,"lng":-8.38912841225451},{"lat":40.899150031964,"lng":-8.38923296907733},{"lat":40.8990029900011,"lng":-8.38935142837419},{"lat":40.8988297229469,"lng":-8.38951986432738},{"lat":40.8986943297835,"lng":-8.38966399996074},{"lat":40.8986245372819,"lng":-8.38974739346734},{"lat":40.8985528036978,"lng":-8.38985285154469},{"lat":40.8984299920081,"lng":-8.39000771643655},{"lat":40.8982391612201,"lng":-8.39017809750308},{"lat":40.898157680644,"lng":-8.39025278145692},{"lat":40.8980590591658,"lng":-8.39034211197241},{"lat":40.8980056032592,"lng":-8.39044823389911},{"lat":40.8979845662301,"lng":-8.39071326513478},{"lat":40.8979581294408,"lng":-8.39085699227689},{"lat":40.8978303920455,"lng":-8.39099961146838},{"lat":40.8977636494035,"lng":-8.39112727817341},{"lat":40.897747415071,"lng":-8.39133833190092},{"lat":40.8977278537698,"lng":-8.39154747262284},{"lat":40.8976985346008,"lng":-8.39169071296063},{"lat":40.8976490051865,"lng":-8.39181215730977},{"lat":40.8975789378956,"lng":-8.39193624971071},{"lat":40.8975091705525,"lng":-8.39204740890118},{"lat":40.8974320766537,"lng":-8.39213337954385},{"lat":40.8973559723075,"lng":-8.39221935384853},{"lat":40.8972840851697,"lng":-8.39231199124311},{"lat":40.8972158509342,"lng":-8.39242196762653},{"lat":40.8971813592877,"lng":-8.3925823936947},{"lat":40.8971545166841,"lng":-8.3927053491854},{"lat":40.8971051182038,"lng":-8.39280816082273},{"lat":40.8970310432692,"lng":-8.39291241730845},{"lat":40.896844574645,"lng":-8.39316464640451},{"lat":40.8968033976079,"lng":-8.39329917584392},{"lat":40.8968285948681,"lng":-8.3934344186428},{"lat":40.8968915905313,"lng":-8.39358956147108},{"lat":40.8969637419783,"lng":-8.39368137229633},{"lat":40.8970476441214,"lng":-8.39373035929636},{"lat":40.8971416717414,"lng":-8.39376158538813},{"lat":40.8972272992261,"lng":-8.3938041711065},{"lat":40.8974036606369,"lng":-8.39386111891281},{"lat":40.8975202642583,"lng":-8.39390382658491},{"lat":40.8976136139412,"lng":-8.39391665740198},{"lat":40.897715073665,"lng":-8.39392655210158},{"lat":40.8978115145054,"lng":-8.3939657393649},{"lat":40.8979039081557,"lng":-8.39404253025465},{"lat":40.8979584943396,"lng":-8.39415311094612},{"lat":40.8979992220633,"lng":-8.39429947668188},{"lat":40.8978339118885,"lng":-8.39448941064581},{"lat":40.8949388874681,"lng":-8.39665866490605},{"lat":40.8948637271248,"lng":-8.3967635039205},{"lat":40.8947863987748,"lng":-8.39687129959204},{"lat":40.8946131061917,"lng":-8.39708539209863},{"lat":40.8942766148624,"lng":-8.39749328640464},{"lat":40.8941406358807,"lng":-8.39766816375952},{"lat":40.8940567449566,"lng":-8.39777082840109},{"lat":40.8939003949108,"lng":-8.39797750782665},{"lat":40.8938234540461,"lng":-8.3980733168125},{"lat":40.8936957914786,"lng":-8.39821864288408},{"lat":40.8935860481818,"lng":-8.39832523634739},{"lat":40.8930061049494,"lng":-8.39942943811162},{"lat":40.8927173034759,"lng":-8.39955452779908},{"lat":40.8925940666583,"lng":-8.39961609096899},{"lat":40.8924741214116,"lng":-8.39968188919133},{"lat":40.8923158677484,"lng":-8.39975107187797},{"lat":40.8921749892443,"lng":-8.3998478040051},{"lat":40.8920699817029,"lng":-8.39993032401345},{"lat":40.8918537662575,"lng":-8.40009047197997},{"lat":40.891756913407,"lng":-8.40015154613342},{"lat":40.8916681934152,"lng":-8.40020066794665},{"lat":40.8915533917256,"lng":-8.40027436639481},{"lat":40.8914499595641,"lng":-8.4003379054304},{"lat":40.8913494868205,"lng":-8.40040776607118},{"lat":40.8912423059886,"lng":-8.40049464387556},{"lat":40.8911497949882,"lng":-8.40054719076233},{"lat":40.8910363029927,"lng":-8.40059965300628},{"lat":40.8909231144576,"lng":-8.40067655922118},{"lat":40.8908132289713,"lng":-8.40076582036851},{"lat":40.8907304286057,"lng":-8.40082457602129},{"lat":40.890429374502,"lng":-8.40102875212745},{"lat":40.8903276244081,"lng":-8.40110451765552},{"lat":40.8902427425787,"lng":-8.40116765446199},{"lat":40.8900202717444,"lng":-8.40130641012119},{"lat":40.889901527475,"lng":-8.40137178279617},{"lat":40.8897973077298,"lng":-8.40142522938188},{"lat":40.889704870323,"lng":-8.40148465633396},{"lat":40.8895735922812,"lng":-8.40155733335867},{"lat":40.887779588243,"lng":-8.402127450724},{"lat":40.8876761712072,"lng":-8.40207980604819},{"lat":40.8875748726958,"lng":-8.40085061190796},{"lat":40.886776559373,"lng":-8.40191607048505},{"lat":40.8867165530382,"lng":-8.40204245083416},{"lat":40.8858836884274,"lng":-8.40217825351063},{"lat":40.8857208918771,"lng":-8.40220479789343},{"lat":40.8845134311812,"lng":-8.4017762850824},{"lat":40.8831258267214,"lng":-8.39999688122138},{"lat":40.8830592584607,"lng":-8.39984633945971},{"lat":40.8829968553556,"lng":-8.3997134455958},{"lat":40.8829461677932,"lng":-8.39957295339023},{"lat":40.882941384415,"lng":-8.39941661864222},{"lat":40.8829061832258,"lng":-8.39926976530464},{"lat":40.8828373558834,"lng":-8.39917024668594},{"lat":40.8827623782176,"lng":-8.3990765659605},{"lat":40.882684994836,"lng":-8.39898737962307},{"lat":40.8826070893483,"lng":-8.3989027770689},{"lat":40.8825357552016,"lng":-8.39880267026345},{"lat":40.8824791788861,"lng":-8.3986709110549},{"lat":40.8824157058625,"lng":-8.39854101098224},{"lat":40.8823497133307,"lng":-8.39840985650658},{"lat":40.8822740315955,"lng":-8.39830724171397},{"lat":40.8822263801356,"lng":-8.39817515125295},{"lat":40.8821694945833,"lng":-8.39803853581519},{"lat":40.8821504879411,"lng":-8.39788333039882},{"lat":40.8821628251257,"lng":-8.39772611666911},{"lat":40.8821704202136,"lng":-8.39756652534024},{"lat":40.8820862150747,"lng":-8.39745941444842},{"lat":40.8819972559581,"lng":-8.39732056879167},{"lat":40.881924535729,"lng":-8.39722193028964},{"lat":40.8818433036153,"lng":-8.3971352922952},{"lat":40.8817594714457,"lng":-8.39705311470453},{"lat":40.8816758413672,"lng":-8.39697034964892},{"lat":40.8815720831812,"lng":-8.39687047828438},{"lat":40.8814924009451,"lng":-8.39677262717774},{"lat":40.8814237217602,"lng":-8.39666485387611},{"lat":40.881346201556,"lng":-8.39655392541426},{"lat":40.8812588543015,"lng":-8.39645798208849},{"lat":40.8811714878914,"lng":-8.39636763415919},{"lat":40.8811243151077,"lng":-8.39623670476337},{"lat":40.8810865348501,"lng":-8.39610982998963},{"lat":40.8810299621522,"lng":-8.39599013959336},{"lat":40.8809532328922,"lng":-8.39588826816237},{"lat":40.8808834206854,"lng":-8.39576796586323},{"lat":40.8808206567907,"lng":-8.39564433407865},{"lat":40.8807298984466,"lng":-8.39557657701088},{"lat":40.8805997378496,"lng":-8.39549926211784},{"lat":40.8805018904607,"lng":-8.39535009253932},{"lat":40.8803952079244,"lng":-8.39521929822119},{"lat":40.8802661773216,"lng":-8.3951379586178},{"lat":40.8801806036674,"lng":-8.39499008799261},{"lat":40.8798689839238,"lng":-8.39473145626889},{"lat":40.8798145068508,"lng":-8.39483256941486},{"lat":40.8797476784333,"lng":-8.39491911802453},{"lat":40.8796705494158,"lng":-8.394990169356},{"lat":40.8795928106756,"lng":-8.39506005425574},{"lat":40.8795075009913,"lng":-8.39511194935925},{"lat":40.8794199268784,"lng":-8.39515736832179},{"lat":40.8793324304866,"lng":-8.3952030520391},{"lat":40.8792448000267,"lng":-8.39524828546257},{"lat":40.8791557344102,"lng":-8.3952882680418},{"lat":40.8790653430421,"lng":-8.39532280565828},{"lat":40.8789749864248,"lng":-8.39535749992257},{"lat":40.8788837457509,"lng":-8.39538773807256},{"lat":40.8788039895852,"lng":-8.39544935169165},{"lat":40.8787440765449,"lng":-8.39554493512648},{"lat":40.8786750862128,"lng":-8.39562880846002},{"lat":40.878594374839,"lng":-8.39569225316397},{"lat":40.8785073995719,"lng":-8.39573953217703},{"lat":40.878416746179,"lng":-8.3957720224851},{"lat":40.8783240211828,"lng":-8.39577151240459},{"lat":40.8782339976465,"lng":-8.39573540378221},{"lat":40.8781506388831,"lng":-8.39567897993446},{"lat":40.8780575479078,"lng":-8.39567824075717},{"lat":40.8779680783452,"lng":-8.39571086848468},{"lat":40.8778937944651,"lng":-8.39578542331146},{"lat":40.8778336969583,"lng":-8.39588082289336},{"lat":40.8777817119897,"lng":-8.39598404530507},{"lat":40.8777352273976,"lng":-8.39609186278716},{"lat":40.8776894043408,"lng":-8.39620017033613},{"lat":40.8776447606732,"lng":-8.39630932828917},{"lat":40.8775987945107,"lng":-8.39641751515041},{"lat":40.8775448123017,"lng":-8.39651893744407},{"lat":40.8774880201917,"lng":-8.39661773604906},{"lat":40.877435926396,"lng":-8.39672100555719},{"lat":40.877359125941,"lng":-8.39687707094873},{"lat":40.8773080645214,"lng":-8.39698123392617},{"lat":40.8772575090075,"lng":-8.3970858210987},{"lat":40.8772183534763,"lng":-8.3971963152344},{"lat":40.8771473312662,"lng":-8.39727746958015},{"lat":40.8770172684556,"lng":-8.39729082463556},{"lat":40.8768836457054,"lng":-8.39734402826023},{"lat":40.8767990694305,"lng":-8.39739842090033},{"lat":40.8767131628038,"lng":-8.39744902126942},{"lat":40.8766260900241,"lng":-8.39749608865727},{"lat":40.8765383675492,"lng":-8.39754098587452},{"lat":40.876448073024,"lng":-8.39757578402567},{"lat":40.8763567735235,"lng":-8.39760588133369},{"lat":40.8762645165044,"lng":-8.39763021859883},{"lat":40.8761706219041,"lng":-8.39763631991579},{"lat":40.8760707456611,"lng":-8.3976309219167},{"lat":40.8759784287359,"lng":-8.39761344860442},{"lat":40.8758856136408,"lng":-8.39762904730191},{"lat":40.8757935019595,"lng":-8.397651259028},{"lat":40.8757021544021,"lng":-8.39767834720211},{"lat":40.8756207633102,"lng":-8.39773690331057},{"lat":40.8755494033209,"lng":-8.39781675001088},{"lat":40.8754569716551,"lng":-8.39792734295898},{"lat":40.8753877983911,"lng":-8.39801050546337},{"lat":40.8753191348731,"lng":-8.39809439348805},{"lat":40.8752397489287,"lng":-8.39815843995397},{"lat":40.8751572667775,"lng":-8.3982167463341},{"lat":40.8750750360964,"lng":-8.39827572028776},{"lat":40.8749884659085,"lng":-8.39832257746985},{"lat":40.874897519155,"lng":-8.39835072775603},{"lat":40.8748040314353,"lng":-8.39835647129527},{"lat":40.8747106706897,"lng":-8.39836416919151},{"lat":40.8745975069102,"lng":-8.39839637560804},{"lat":40.8745111474825,"lng":-8.39849955104403},{"lat":40.8744290572435,"lng":-8.39860972590517},{"lat":40.8743445445237,"lng":-8.39871680640307},{"lat":40.8742471746656,"lng":-8.39879963213369},{"lat":40.8741333056519,"lng":-8.39883419552065},{"lat":40.8735988740338,"lng":-8.39913386955474},{"lat":40.8735079978627,"lng":-8.39916489876247},{"lat":40.873417730321,"lng":-8.39919869794869},{"lat":40.8733271548382,"lng":-8.39921921169894},{"lat":40.8729576980166,"lng":-8.39938419570216},{"lat":40.8723314706327,"lng":-8.40052143060049},{"lat":40.8719247291842,"lng":-8.40167484327062},{"lat":40.8714716380911,"lng":-8.40288878431617},{"lat":40.8711357023118,"lng":-8.40344956641468},{"lat":40.8707654163027,"lng":-8.40379763560732},{"lat":40.8710984283663,"lng":-8.40458915520876},{"lat":40.8711781571138,"lng":-8.40503029433396},{"lat":40.8713273242645,"lng":-8.40535000667233},{"lat":40.8717651267265,"lng":-8.40573177017792},{"lat":40.8721571984843,"lng":-8.40580337675513},{"lat":40.8726414499453,"lng":-8.40607857597967},{"lat":40.8729752992657,"lng":-8.40647510761164},{"lat":40.8723972458751,"lng":-8.40686797364694},{"lat":40.8716573095577,"lng":-8.40729041248511},{"lat":40.8715414519709,"lng":-8.40750274507262},{"lat":40.8717136076428,"lng":-8.40788341745156},{"lat":40.8719681869542,"lng":-8.40817831584199},{"lat":40.8725381476942,"lng":-8.40952614712443},{"lat":40.8726180689342,"lng":-8.40970656215974},{"lat":40.872669209931,"lng":-8.40982185377533},{"lat":40.8732071604952,"lng":-8.41102721010403},{"lat":40.8734562880711,"lng":-8.41150314562764},{"lat":40.8740304749834,"lng":-8.41259375255543},{"lat":40.8742424663096,"lng":-8.41324746892971},{"lat":40.8742733993465,"lng":-8.41359589991533},{"lat":40.8741873801046,"lng":-8.4139000539331},{"lat":40.8741013021204,"lng":-8.41411748788401},{"lat":40.8736103131878,"lng":-8.41465196024536},{"lat":40.8734857514736,"lng":-8.41473600597357},{"lat":40.8732876280916,"lng":-8.41485521564961},{"lat":40.8731650477913,"lng":-8.414904382977},{"lat":40.873287115565,"lng":-8.41506496274345},{"lat":40.8734315357052,"lng":-8.41520421728018},{"lat":40.8738638200218,"lng":-8.41570636518386},{"lat":40.8744993141004,"lng":-8.41637973751722},{"lat":40.8759271331714,"lng":-8.41763731227979},{"lat":40.877093172951,"lng":-8.41853851560136},{"lat":40.8775944608771,"lng":-8.41872565687295},{"lat":40.8783252521045,"lng":-8.4196003803618},{"lat":40.87871938177,"lng":-8.42100501033679},{"lat":40.8797516983288,"lng":-8.42240870200296},{"lat":40.880172427977,"lng":-8.42307793729907},{"lat":40.8797023761356,"lng":-8.42429710218324},{"lat":40.8794032364792,"lng":-8.42563305435582},{"lat":40.879254261208,"lng":-8.4268954950013},{"lat":40.8786702790764,"lng":-8.42781948784879},{"lat":40.878085147159,"lng":-8.42892715441165},{"lat":40.8780252429333,"lng":-8.43022172788523},{"lat":40.8778201446954,"lng":-8.43101031525207},{"lat":40.8774458210871,"lng":-8.43150826829543},{"lat":40.8769230062407,"lng":-8.43285173874084},{"lat":40.8764152238787,"lng":-8.43401835290799}]},{"zone_id":"smf9","points":[{"lat":40.9167728141759,"lng":-8.48898499576732},{"lat":40.9206082680716,"lng":-8.49075339553756},{"lat":40.9225922500529,"lng":-8.49055142006195},{"lat":40.9241946535608,"lng":-8.48949104881503},{"lat":40.9254088945652,"lng":-8.48976741936876},{"lat":40.9302989925647,"lng":-8.49315832873953},{"lat":40.933271047189,"lng":-8.48628405897639},{"lat":40.9346722304555,"lng":-8.48324938910821},{"lat":40.9372611977982,"lng":-8.47798490022926},{"lat":40.9387945836652,"lng":-8.47510238808737},{"lat":40.9406112027029,"lng":-8.47310092910109},{"lat":40.9452819413368,"lng":-8.47274782593915},{"lat":40.9533627777479,"lng":-8.47132796170453},{"lat":40.9575378250426,"lng":-8.4693850691727},{"lat":40.9608056366624,"lng":-8.46517731235596},{"lat":40.9583802013173,"lng":-8.46205428862203},{"lat":40.9563892777256,"lng":-8.45839890926084},{"lat":40.9530703467639,"lng":-8.45247469602237},{"lat":40.9497551977389,"lng":-8.44504276414396},{"lat":40.9478650537169,"lng":-8.43912640966637},{"lat":40.9437148935046,"lng":-8.43255087110701},{"lat":40.9436120487576,"lng":-8.4325176563391},{"lat":40.9434162234703,"lng":-8.43245589592586},{"lat":40.9431424282252,"lng":-8.43239273180316},{"lat":40.9429790366562,"lng":-8.43234798336155},{"lat":40.9427395838045,"lng":-8.43224625975037},{"lat":40.9424355932953,"lng":-8.43207114204034},{"lat":40.942346572333,"lng":-8.43200798344835},{"lat":40.9422440633009,"lng":-8.43193525558345},{"lat":40.9420251090974,"lng":-8.43181717437041},{"lat":40.941891538661,"lng":-8.4317935299641},{"lat":40.9417080430722,"lng":-8.43168308850547},{"lat":40.941622980055,"lng":-8.43160482376041},{"lat":40.9415093393505,"lng":-8.43150026566161},{"lat":40.9413960915335,"lng":-8.43139896134432},{"lat":40.9413105314984,"lng":-8.43132242504594},{"lat":40.9412059959002,"lng":-8.4312257359009},{"lat":40.9411123659802,"lng":-8.43113913417785},{"lat":40.9409370290069,"lng":-8.43093872224924},{"lat":40.9408356837578,"lng":-8.43081319374209},{"lat":40.9407687316161,"lng":-8.4307302656176},{"lat":40.9406854714587,"lng":-8.43057072047366},{"lat":40.9406332573478,"lng":-8.43047066694342},{"lat":40.9405993991242,"lng":-8.43030452126453},{"lat":40.940549370771,"lng":-8.43018557757841},{"lat":40.9404435917299,"lng":-8.42989457588682},{"lat":40.9403295044514,"lng":-8.42961607242742},{"lat":40.9401578675359,"lng":-8.42924675239943},{"lat":40.9400824938965,"lng":-8.42913744067577},{"lat":40.9398691040849,"lng":-8.42895322222924},{"lat":40.9396061710121,"lng":-8.42886780420099},{"lat":40.9394165985922,"lng":-8.42886653149313},{"lat":40.9391791576302,"lng":-8.42892567141055},{"lat":40.9389564098666,"lng":-8.42899393002519},{"lat":40.9387134185164,"lng":-8.42901819796846},{"lat":40.9385388861928,"lng":-8.42903091473174},{"lat":40.9383049941047,"lng":-8.4290523969509},{"lat":40.9381680519302,"lng":-8.42904810343174},{"lat":40.9381260099879,"lng":-8.42923631393218},{"lat":40.9379651041052,"lng":-8.42941227234938},{"lat":40.9377937039295,"lng":-8.42960675270524},{"lat":40.9375733316216,"lng":-8.42988458178503},{"lat":40.9373669505395,"lng":-8.43011610601819},{"lat":40.9372060428079,"lng":-8.43029669492006},{"lat":40.9370661227227,"lng":-8.43050969721456},{"lat":40.9369262023411,"lng":-8.43076437387104},{"lat":40.9367687915575,"lng":-8.43103757246617},{"lat":40.9366358666037,"lng":-8.4313107710613},{"lat":40.9365134354886,"lng":-8.43151914287114},{"lat":40.936324541323,"lng":-8.43169047080368},{"lat":40.9360621874191,"lng":-8.43184327679757},{"lat":40.9358243189791,"lng":-8.43192199503684},{"lat":40.9357124353603,"lng":-8.43191795122648},{"lat":40.9356969252727,"lng":-8.43209907739462},{"lat":40.9356751913263,"lng":-8.43221510666053},{"lat":40.9356746805931,"lng":-8.43241054924017},{"lat":40.9356935729014,"lng":-8.43255502210172},{"lat":40.9357297316703,"lng":-8.43270812357864},{"lat":40.9357810162129,"lng":-8.43282733368793},{"lat":40.9358063542639,"lng":-8.43298608496306},{"lat":40.9357845008223,"lng":-8.43314735269655},{"lat":40.9357045386303,"lng":-8.4333197551976},{"lat":40.9356547400469,"lng":-8.43345833407393},{"lat":40.9356162369703,"lng":-8.43365075461933},{"lat":40.9360378318161,"lng":-8.43375360620638},{"lat":40.9361901756181,"lng":-8.43396482822861},{"lat":40.9363823980583,"lng":-8.43448875885677},{"lat":40.9365682370339,"lng":-8.43466593940057},{"lat":40.9368632279798,"lng":-8.43490548620395},{"lat":40.9370168718412,"lng":-8.43513488594397},{"lat":40.9374081452706,"lng":-8.43543329851103},{"lat":40.9373217194112,"lng":-8.43559688594091},{"lat":40.9368340530781,"lng":-8.43548587990451},{"lat":40.9367087682265,"lng":-8.43542937729343},{"lat":40.9365718259119,"lng":-8.43545665266599},{"lat":40.9364421758967,"lng":-8.43555283160286},{"lat":40.9363658393575,"lng":-8.43568083995075},{"lat":40.9362643229976,"lng":-8.4355915555177},{"lat":40.9361402964116,"lng":-8.43556996895872},{"lat":40.9359960706532,"lng":-8.43562725214243},{"lat":40.9358642395022,"lng":-8.43552643089662},{"lat":40.9357003518468,"lng":-8.43542546272475},{"lat":40.9357154220998,"lng":-8.43585655581045},{"lat":40.9357589619926,"lng":-8.43607689899959},{"lat":40.9357324887817,"lng":-8.43624693129724},{"lat":40.9354713764506,"lng":-8.43617080539872},{"lat":40.9351663807967,"lng":-8.43607227407468},{"lat":40.9350207170072,"lng":-8.4360605622676},{"lat":40.9344966403854,"lng":-8.43613188821211},{"lat":40.9345156440075,"lng":-8.43698047318441},{"lat":40.933831299113,"lng":-8.43667263943295},{"lat":40.9334129922521,"lng":-8.43679051696585},{"lat":40.932852788167,"lng":-8.43659677309599},{"lat":40.9327595861256,"lng":-8.4365305647499},{"lat":40.9326614631754,"lng":-8.43648618138454},{"lat":40.9326482986688,"lng":-8.43662871990369},{"lat":40.9325818160225,"lng":-8.43672136749393},{"lat":40.9325244891062,"lng":-8.43683118436246},{"lat":40.9323058808982,"lng":-8.43716179976485},{"lat":40.932366392312,"lng":-8.43741419036069},{"lat":40.9317096253472,"lng":-8.43813126098627},{"lat":40.9301692223924,"lng":-8.43981302727087},{"lat":40.9311983600329,"lng":-8.4445206511549},{"lat":40.9310398994589,"lng":-8.44464563884301},{"lat":40.9309415170043,"lng":-8.44483039309734},{"lat":40.9309582742353,"lng":-8.44512492552346},{"lat":40.9310298499503,"lng":-8.44555863313699},{"lat":40.9310386353259,"lng":-8.44603502423634},{"lat":40.9310251159933,"lng":-8.44636918857651},{"lat":40.9310529970727,"lng":-8.44651310406278},{"lat":40.9321394583896,"lng":-8.45077758354087},{"lat":40.9324145978238,"lng":-8.45079256422354},{"lat":40.932548955901,"lng":-8.45101654974138},{"lat":40.9326532631996,"lng":-8.45113436258388},{"lat":40.932842438991,"lng":-8.45113824286927},{"lat":40.9329005501877,"lng":-8.45138656049754},{"lat":40.9327841849811,"lng":-8.45149285928545},{"lat":40.9326991403256,"lng":-8.45157259383688},{"lat":40.9325939859833,"lng":-8.45163026558633},{"lat":40.9325126415583,"lng":-8.45173922633139},{"lat":40.9324557887624,"lng":-8.45188083789032},{"lat":40.9323761998672,"lng":-8.45203836788936},{"lat":40.9322003145296,"lng":-8.45233862444262},{"lat":40.9320513450777,"lng":-8.45241531737189},{"lat":40.931885416727,"lng":-8.45247031753068},{"lat":40.9317582456854,"lng":-8.45247943777466},{"lat":40.9316682257628,"lng":-8.45253492423039},{"lat":40.9315923256659,"lng":-8.45266076872319},{"lat":40.9315199958511,"lng":-8.45283044271288},{"lat":40.9314331427577,"lng":-8.4529125399806},{"lat":40.9313039610586,"lng":-8.45299692508422},{"lat":40.9311512281325,"lng":-8.45316347675165},{"lat":40.9310441345532,"lng":-8.45323823156833},{"lat":40.9309317739729,"lng":-8.45339310496147},{"lat":40.9308284337188,"lng":-8.45344603178496},{"lat":40.930715976038,"lng":-8.45347469310945},{"lat":40.9306675495041,"lng":-8.4536613393992},{"lat":40.9306378439838,"lng":-8.45385104542671},{"lat":40.9305709504698,"lng":-8.45404009404157},{"lat":40.9304307190487,"lng":-8.45411480514512},{"lat":40.9303766284364,"lng":-8.45436221193248},{"lat":40.9303724670087,"lng":-8.45452972032852},{"lat":40.9302068177939,"lng":-8.45467768030266},{"lat":40.9302139095388,"lng":-8.45503829694804},{"lat":40.9302250060315,"lng":-8.45519115634734},{"lat":40.9303395936806,"lng":-8.45530177893593},{"lat":40.9304643046143,"lng":-8.45549508694955},{"lat":40.9304796245408,"lng":-8.45561899826526},{"lat":40.9304323623617,"lng":-8.45574260165567},{"lat":40.9303174645521,"lng":-8.45574203919465},{"lat":40.9302129353358,"lng":-8.4557689552032},{"lat":40.9301030602636,"lng":-8.45587171167981},{"lat":40.9299776210668,"lng":-8.4559054096331},{"lat":40.9298420590139,"lng":-8.45585666057783},{"lat":40.9297431007899,"lng":-8.45575976903932},{"lat":40.9295972868592,"lng":-8.4556421072984},{"lat":40.9294977707474,"lng":-8.455583324756},{"lat":40.9293643458148,"lng":-8.455350796362},{"lat":40.92688470488,"lng":-8.45382364412427},{"lat":40.9240933689808,"lng":-8.45761125433916},{"lat":40.9237342701174,"lng":-8.45737211737943},{"lat":40.9236460167808,"lng":-8.45731149302896},{"lat":40.9235034101712,"lng":-8.45714280650488},{"lat":40.9233924026127,"lng":-8.45700846595366},{"lat":40.9232434272695,"lng":-8.45683084576805},{"lat":40.9231332438401,"lng":-8.45672357923557},{"lat":40.9230143650275,"lng":-8.45663407599117},{"lat":40.9229441563239,"lng":-8.45652866826712},{"lat":40.9228572014688,"lng":-8.45635835807512},{"lat":40.9227536223242,"lng":-8.45620779320878},{"lat":40.9226772767549,"lng":-8.45607528771197},{"lat":40.922428279781,"lng":-8.45557035703423},{"lat":40.9223005564985,"lng":-8.45516728761662},{"lat":40.9220354599584,"lng":-8.45458986825164},{"lat":40.9218584420796,"lng":-8.45429613702035},{"lat":40.9213631714985,"lng":-8.45388368682266},{"lat":40.9207712785047,"lng":-8.45356847435415},{"lat":40.9204830063931,"lng":-8.45332300111914},{"lat":40.9200098916775,"lng":-8.45293987690152},{"lat":40.9193812902543,"lng":-8.45267088130324},{"lat":40.9192121345555,"lng":-8.45266250067477},{"lat":40.9168597569308,"lng":-8.4554939873115},{"lat":40.916523054763,"lng":-8.4558332705752},{"lat":40.9151358295041,"lng":-8.45723108309794},{"lat":40.9148168318422,"lng":-8.45755250465711},{"lat":40.9137492843028,"lng":-8.45914569797635},{"lat":40.9129978329845,"lng":-8.46026710316891},{"lat":40.9123864556015,"lng":-8.46392542228411},{"lat":40.912511068612,"lng":-8.46393708601191},{"lat":40.9125938909531,"lng":-8.46508887218431},{"lat":40.912047933597,"lng":-8.46524984746451},{"lat":40.9119073477227,"lng":-8.46528566051383},{"lat":40.9119361253733,"lng":-8.46542278108502},{"lat":40.9119481448575,"lng":-8.46559400292105},{"lat":40.9117874672572,"lng":-8.46569907062231},{"lat":40.9117465211646,"lng":-8.46575078744295},{"lat":40.911417216129,"lng":-8.46615774407102},{"lat":40.9113445111235,"lng":-8.46617043473814},{"lat":40.9112725996399,"lng":-8.46617188442982},{"lat":40.911000784189,"lng":-8.46617736403979},{"lat":40.9106934840018,"lng":-8.46613664397403},{"lat":40.9104727177173,"lng":-8.4661579626212},{"lat":40.9102286614387,"lng":-8.46623043937758},{"lat":40.9100611661441,"lng":-8.4662646089773},{"lat":40.9097753364586,"lng":-8.46614827106617},{"lat":40.9096635803461,"lng":-8.4661144381101},{"lat":40.909550858732,"lng":-8.46608340016545},{"lat":40.9093498392758,"lng":-8.46609555961558},{"lat":40.9092657225389,"lng":-8.46606926127562},{"lat":40.9092173634661,"lng":-8.46607079672009},{"lat":40.9091266847076,"lng":-8.46607140771556},{"lat":40.909014247937,"lng":-8.46602953617312},{"lat":40.9089437891514,"lng":-8.46598217803565},{"lat":40.9087689717989,"lng":-8.46577774126887},{"lat":40.9087034924698,"lng":-8.46568981690597},{"lat":40.9085165219363,"lng":-8.4657492402331},{"lat":40.9084159931637,"lng":-8.46528437765044},{"lat":40.9083462860853,"lng":-8.46531437829784},{"lat":40.9083199228568,"lng":-8.46524528645745},{"lat":40.908220078798,"lng":-8.46514591561452},{"lat":40.9079624015286,"lng":-8.46528906424837},{"lat":40.9078928780161,"lng":-8.46526046669491},{"lat":40.907525537087,"lng":-8.46555450931407},{"lat":40.9074347525782,"lng":-8.46565363212646},{"lat":40.9073712465319,"lng":-8.46575479040587},{"lat":40.9072841007957,"lng":-8.46598086859948},{"lat":40.9072067888727,"lng":-8.46623120825346},{"lat":40.9071549838369,"lng":-8.46636641767981},{"lat":40.9071047127726,"lng":-8.46649762289429},{"lat":40.9070671307439,"lng":-8.46675629118625},{"lat":40.9070290766902,"lng":-8.46680606697991},{"lat":40.9069566940315,"lng":-8.46686290898256},{"lat":40.9068316065677,"lng":-8.46686714275931},{"lat":40.9067057192941,"lng":-8.46680609363361},{"lat":40.9066454015312,"lng":-8.46698726194566},{"lat":40.90656172593,"lng":-8.4672421343131},{"lat":40.9064956999455,"lng":-8.4672804920429},{"lat":40.906421552702,"lng":-8.46726565888103},{"lat":40.9062613554191,"lng":-8.46723361151913},{"lat":40.9061688453407,"lng":-8.4672139160799},{"lat":40.9061252340855,"lng":-8.46734733668657},{"lat":40.9060865507935,"lng":-8.46748920758255},{"lat":40.9060007717156,"lng":-8.46766205582238},{"lat":40.9059629382724,"lng":-8.46779052005082},{"lat":40.9059310233751,"lng":-8.46793782882052},{"lat":40.9058752190951,"lng":-8.46810352519381},{"lat":40.9056123465064,"lng":-8.46860541802822},{"lat":40.905505347418,"lng":-8.46882883314192},{"lat":40.9053263453043,"lng":-8.46876639515983},{"lat":40.9050820211776,"lng":-8.46868117268469},{"lat":40.904849354226,"lng":-8.46873814419006},{"lat":40.9047541244154,"lng":-8.46906285467937},{"lat":40.9045968130158,"lng":-8.46958485566648},{"lat":40.9044273975175,"lng":-8.47030812033733},{"lat":40.9047491433924,"lng":-8.47057522021021},{"lat":40.904789747928,"lng":-8.47073019240005},{"lat":40.9048927240812,"lng":-8.47089580896225},{"lat":40.9050241580041,"lng":-8.47093778395738},{"lat":40.9052339002996,"lng":-8.47102146166035},{"lat":40.9054217317567,"lng":-8.47108437687975},{"lat":40.9056593731042,"lng":-8.47123418877185},{"lat":40.9058560364395,"lng":-8.47138640286351},{"lat":40.9054399442002,"lng":-8.47269206266543},{"lat":40.905261353967,"lng":-8.47306607069027},{"lat":40.9050422741836,"lng":-8.47358110056136},{"lat":40.9049664661017,"lng":-8.47360753236767},{"lat":40.9049311660694,"lng":-8.47382050720331},{"lat":40.9048891835771,"lng":-8.4738676991394},{"lat":40.9048069370569,"lng":-8.47396015107372},{"lat":40.9047705295368,"lng":-8.47403034338634},{"lat":40.9045948455179,"lng":-8.47394054132505},{"lat":40.9043916639995,"lng":-8.47386187365436},{"lat":40.9042148912157,"lng":-8.47377479671202},{"lat":40.9040707278409,"lng":-8.47371376185242},{"lat":40.9038874707543,"lng":-8.47356624360587},{"lat":40.9037173090947,"lng":-8.47343612109405},{"lat":40.9035600888167,"lng":-8.47337501932708},{"lat":40.9034521583385,"lng":-8.47333268735552},{"lat":40.9027798890238,"lng":-8.47508260826988},{"lat":40.9024499278987,"lng":-8.47619876952021},{"lat":40.9027329376611,"lng":-8.47642060381643},{"lat":40.9031711977491,"lng":-8.47671994310527},{"lat":40.9035358604971,"lng":-8.47693914884395},{"lat":40.9036776954879,"lng":-8.47702640599231},{"lat":40.903857871786,"lng":-8.47711849441383},{"lat":40.9040325362222,"lng":-8.47721660765622},{"lat":40.9042459979068,"lng":-8.47731883797216},{"lat":40.904449376152,"lng":-8.47739265098713},{"lat":40.9044293693369,"lng":-8.47757721597109},{"lat":40.9044138654633,"lng":-8.47773166304605},{"lat":40.9043927499306,"lng":-8.4779266651359},{"lat":40.9043645380627,"lng":-8.47808414931957},{"lat":40.9081909327997,"lng":-8.47913546462463},{"lat":40.9096710052611,"lng":-8.47886578524544},{"lat":40.9099630447303,"lng":-8.48023700243726},{"lat":40.9138695418615,"lng":-8.48182849705016},{"lat":40.9140001874563,"lng":-8.48211323824415},{"lat":40.9150596794567,"lng":-8.48360448553362},{"lat":40.9157752394077,"lng":-8.4872984086288},{"lat":40.916400222682,"lng":-8.48765581748797},{"lat":40.9169383006894,"lng":-8.48838251738865},{"lat":40.9167728141759,"lng":-8.48898499576732}]},{"zone_id":"arc7","points":[{"lat":40.9632545795332,"lng":-8.3016286134032},{"lat":40.9632662704891,"lng":-8.30006224520206},{"lat":40.9628218265369,"lng":-8.29628066806479},{"lat":40.9627920770984,"lng":-8.29602758275824},{"lat":40.9628507574762,"lng":-8.29517558624685},{"lat":40.9629532494828,"lng":-8.29368725375783},{"lat":40.9633978810915,"lng":-8.28368259639279},{"lat":40.9602911156922,"lng":-8.28256656608524},{"lat":40.9565186146699,"lng":-8.28150211632897},{"lat":40.9519452940998,"lng":-8.28066226563037},{"lat":40.9479424141662,"lng":-8.28087959854383},{"lat":40.9442244603512,"lng":-8.28185164552156},{"lat":40.9389901227832,"lng":-8.2836684597042},{"lat":40.9353073692135,"lng":-8.285026727096},{"lat":40.9315111556105,"lng":-8.28620974094388},{"lat":40.9256019171145,"lng":-8.28748655119905},{"lat":40.9234997846484,"lng":-8.28880906219787},{"lat":40.9185937347884,"lng":-8.29282597903297},{"lat":40.9176776514841,"lng":-8.29343213792003},{"lat":40.9142503757127,"lng":-8.29607120061705},{"lat":40.9094737432467,"lng":-8.30214417753041},{"lat":40.9084301164394,"lng":-8.3040684319099},{"lat":40.9074269364506,"lng":-8.30576164005727},{"lat":40.9060817816783,"lng":-8.30923477203283},{"lat":40.9054578810878,"lng":-8.31212215770664},{"lat":40.9051667652162,"lng":-8.31545003562483},{"lat":40.9051397482978,"lng":-8.31637601625597},{"lat":40.9050659517285,"lng":-8.31890402623316},{"lat":40.9053951204677,"lng":-8.32160556994951},{"lat":40.9061539090159,"lng":-8.32380593066879},{"lat":40.9158978054921,"lng":-8.34136719472603},{"lat":40.9180409278756,"lng":-8.33822189362449},{"lat":40.9191186599532,"lng":-8.33707879381985},{"lat":40.9199237244719,"lng":-8.33653160312904},{"lat":40.9213542613938,"lng":-8.33594704905017},{"lat":40.9232872817761,"lng":-8.3354896459401},{"lat":40.9279943640757,"lng":-8.3345616081975},{"lat":40.935441166895,"lng":-8.33372026628894},{"lat":40.9383591735917,"lng":-8.33404327616669},{"lat":40.9422521585335,"lng":-8.33489440357846},{"lat":40.943579322208,"lng":-8.3301151591232},{"lat":40.9453034046925,"lng":-8.32496715961515},{"lat":40.9467822450458,"lng":-8.32003723024594},{"lat":40.9481607308165,"lng":-8.31619496212588},{"lat":40.9495960337514,"lng":-8.31250352072855},{"lat":40.95108736191,"lng":-8.30949083131978},{"lat":40.9530925477517,"lng":-8.30693187319359},{"lat":40.9552679802988,"lng":-8.30520285235678},{"lat":40.9577287270585,"lng":-8.3038515803286},{"lat":40.9632545795332,"lng":-8.3016286134032}]},{"zone_id":"smf1","points":[{"lat":40.891030229073,"lng":-8.53053043018235},{"lat":40.8908796731589,"lng":-8.53065993121292},{"lat":40.8906211406332,"lng":-8.53089628753967},{"lat":40.8906011408553,"lng":-8.53102894586294},{"lat":40.8905675224671,"lng":-8.53132349085485},{"lat":40.8905626548288,"lng":-8.53150643400791},{"lat":40.8904407618414,"lng":-8.53152385305487},{"lat":40.8904031586314,"lng":-8.53177797669318},{"lat":40.8904323795933,"lng":-8.53194088665928},{"lat":40.890473498559,"lng":-8.53213158240374},{"lat":40.890366413293,"lng":-8.53216321091265},{"lat":40.8902499862887,"lng":-8.53221388615397},{"lat":40.8903159498173,"lng":-8.53261392509945},{"lat":40.8901784199168,"lng":-8.53262223108458},{"lat":40.8894528364047,"lng":-8.53410701382531},{"lat":40.8887469666698,"lng":-8.53472698244226},{"lat":40.887712057609,"lng":-8.53560724251899},{"lat":40.883056392567,"lng":-8.53831963730074},{"lat":40.8820842089699,"lng":-8.53888597982361},{"lat":40.881405769422,"lng":-8.5396687031489},{"lat":40.8813221005096,"lng":-8.53976523138164},{"lat":40.8798785843429,"lng":-8.54143055077102},{"lat":40.8796831905318,"lng":-8.54169577346904},{"lat":40.8782361713743,"lng":-8.54365984698384},{"lat":40.8757784587714,"lng":-8.54699545881706},{"lat":40.8734554535272,"lng":-8.55014789858198},{"lat":40.8733939376092,"lng":-8.55055876825974},{"lat":40.8730927006831,"lng":-8.55257060151547},{"lat":40.8730723462057,"lng":-8.55270652818261},{"lat":40.872679523872,"lng":-8.55332091635762},{"lat":40.8721521111671,"lng":-8.55414578922317},{"lat":40.8722422504011,"lng":-8.55445931718222},{"lat":40.8729048679164,"lng":-8.55676419265289},{"lat":40.8730176390075,"lng":-8.55715648076315},{"lat":40.872950290262,"lng":-8.55725225509596},{"lat":40.8729862226593,"lng":-8.55749495973209},{"lat":40.8730547722169,"lng":-8.5576347860024},{"lat":40.8731651351685,"lng":-8.55776586632608},{"lat":40.8732641892394,"lng":-8.55783815230107},{"lat":40.8734042110794,"lng":-8.55793573348031},{"lat":40.873523677475,"lng":-8.55801574394123},{"lat":40.8736375198749,"lng":-8.55808243221393},{"lat":40.8737745167671,"lng":-8.55818209154419},{"lat":40.8739648037576,"lng":-8.55833555360734},{"lat":40.8741002003003,"lng":-8.558490760469},{"lat":40.8741856506197,"lng":-8.55861326082378},{"lat":40.874337281427,"lng":-8.5588099749121},{"lat":40.8744557800316,"lng":-8.55898073344629},{"lat":40.8746066361148,"lng":-8.55921611801904},{"lat":40.8746854029677,"lng":-8.55934379716292},{"lat":40.8747461123034,"lng":-8.55948464630845},{"lat":40.8747705255469,"lng":-8.55962727831551},{"lat":40.874766375629,"lng":-8.56117798733853},{"lat":40.8747687343217,"lng":-8.56136662440539},{"lat":40.8747471654834,"lng":-8.56152414438652},{"lat":40.8746765268101,"lng":-8.56160865403423},{"lat":40.8747390805471,"lng":-8.56183253073175},{"lat":40.8747808370105,"lng":-8.56195985457672},{"lat":40.8747380677559,"lng":-8.56210371353777},{"lat":40.874672871621,"lng":-8.56220068552519},{"lat":40.8745706661602,"lng":-8.56232197379328},{"lat":40.8744799414043,"lng":-8.56243099941176},{"lat":40.8743685285421,"lng":-8.5625338396911},{"lat":40.8742554193779,"lng":-8.56260879163959},{"lat":40.8741546808737,"lng":-8.56267123938861},{"lat":40.8739777109811,"lng":-8.56277840802497},{"lat":40.8738119632225,"lng":-8.5628671333283},{"lat":40.8737164743684,"lng":-8.56297375237019},{"lat":40.8736724695759,"lng":-8.56313468305183},{"lat":40.8736199896252,"lng":-8.56329816732475},{"lat":40.8734901886854,"lng":-8.56367017076112},{"lat":40.8733748989061,"lng":-8.56401415292368},{"lat":40.8733332782394,"lng":-8.56418731634203},{"lat":40.8732920245607,"lng":-8.564310420504},{"lat":40.8732217649353,"lng":-8.56443653805788},{"lat":40.8731596549526,"lng":-8.56457433386669},{"lat":40.8731052311672,"lng":-8.56467967627647},{"lat":40.8730304510784,"lng":-8.56481027160622},{"lat":40.8729590766061,"lng":-8.5649453966484},{"lat":40.8729326051326,"lng":-8.56520798280692},{"lat":40.8730305235738,"lng":-8.56536556610741},{"lat":40.8731696836552,"lng":-8.56542733011394},{"lat":40.8733120537696,"lng":-8.56549753911031},{"lat":40.8735019046273,"lng":-8.56556070353912},{"lat":40.8736696715435,"lng":-8.56562977472294},{"lat":40.8738276794955,"lng":-8.56568407144138},{"lat":40.8739453707825,"lng":-8.56570797391861},{"lat":40.8740628524754,"lng":-8.56578751155695},{"lat":40.8741518711386,"lng":-8.56594088837365},{"lat":40.8742744649831,"lng":-8.56607360653205},{"lat":40.8743944365669,"lng":-8.56636553309665},{"lat":40.8744904989101,"lng":-8.56659580447398},{"lat":40.8745938017204,"lng":-8.56670989217121},{"lat":40.8747202700182,"lng":-8.56672246539718},{"lat":40.8748695951572,"lng":-8.56676282901546},{"lat":40.8749781650634,"lng":-8.56686532607466},{"lat":40.8750444975427,"lng":-8.56699305124725},{"lat":40.8750893301431,"lng":-8.56723416605439},{"lat":40.8750975827229,"lng":-8.56736151110196},{"lat":40.8751092453211,"lng":-8.56753977168174},{"lat":40.8751757164869,"lng":-8.56763048540404},{"lat":40.8753687621253,"lng":-8.56770578035195},{"lat":40.8755089038559,"lng":-8.56784099166342},{"lat":40.8756016002832,"lng":-8.56797351922626},{"lat":40.8756395279734,"lng":-8.56815895256292},{"lat":40.8756246747816,"lng":-8.5683717993918},{"lat":40.8755534629127,"lng":-8.56860515384124},{"lat":40.8754913040437,"lng":-8.56877842146757},{"lat":40.875439730517,"lng":-8.56891462613245},{"lat":40.8753706879645,"lng":-8.56905071599026},{"lat":40.8753228042976,"lng":-8.5691638122441},{"lat":40.8752607231059,"lng":-8.5693163185756},{"lat":40.8751562442242,"lng":-8.56951243898688},{"lat":40.8750820642653,"lng":-8.56960222736182},{"lat":40.8750182350226,"lng":-8.56974060403524},{"lat":40.8750088403267,"lng":-8.56991421720093},{"lat":40.8751538497448,"lng":-8.5701419975993},{"lat":40.8752167193979,"lng":-8.57025582360593},{"lat":40.8753066192813,"lng":-8.57048371491994},{"lat":40.8752830493475,"lng":-8.57064323676533},{"lat":40.8752632417549,"lng":-8.57080788447281},{"lat":40.8752231038694,"lng":-8.5709672969363},{"lat":40.8751447860487,"lng":-8.57107935936867},{"lat":40.8750268871568,"lng":-8.57120456485057},{"lat":40.8749501023105,"lng":-8.57126882812559},{"lat":40.8748591359869,"lng":-8.57136811273292},{"lat":40.8746728008396,"lng":-8.57146996867855},{"lat":40.8745611339001,"lng":-8.57154301597203},{"lat":40.8751468181158,"lng":-8.57193873531184},{"lat":40.875962072523,"lng":-8.57241748629502},{"lat":40.8776133220144,"lng":-8.5733657888811},{"lat":40.877711143942,"lng":-8.57342801187708},{"lat":40.8778177052343,"lng":-8.57348922524282},{"lat":40.8780765808517,"lng":-8.57362683144223},{"lat":40.8783257936612,"lng":-8.57375930245498},{"lat":40.8806682479914,"lng":-8.57519432365274},{"lat":40.8811535303008,"lng":-8.57556298360292},{"lat":40.8815169048094,"lng":-8.57583401970982},{"lat":40.8817312176734,"lng":-8.57599704565497},{"lat":40.8818322836284,"lng":-8.57608195909881},{"lat":40.8819098142054,"lng":-8.5761983915151},{"lat":40.882005521282,"lng":-8.57626150441868},{"lat":40.882273021626,"lng":-8.57646587493904},{"lat":40.8824750402049,"lng":-8.57661766887842},{"lat":40.8826486323302,"lng":-8.57673878317776},{"lat":40.8830200961382,"lng":-8.57701297073529},{"lat":40.8831562928246,"lng":-8.57711770002778},{"lat":40.8833256793127,"lng":-8.57727829580179},{"lat":40.8834512878351,"lng":-8.57734985278766},{"lat":40.8835673313489,"lng":-8.57738005563874},{"lat":40.8841630237671,"lng":-8.57781344060849},{"lat":40.8850674989552,"lng":-8.57854493879979},{"lat":40.8867708608822,"lng":-8.58158705304661},{"lat":40.8886006045785,"lng":-8.58485521418334},{"lat":40.8893514317631,"lng":-8.58619639021199},{"lat":40.8895963147,"lng":-8.58616175959743},{"lat":40.8896512481686,"lng":-8.58666445446218},{"lat":40.8897317596086,"lng":-8.58673479427538},{"lat":40.8903748224482,"lng":-8.58661687203112},{"lat":40.8910825111673,"lng":-8.58648709784218},{"lat":40.8914273131084,"lng":-8.58627824746819},{"lat":40.8914984103437,"lng":-8.58610584643722},{"lat":40.89155028747,"lng":-8.58598896506065},{"lat":40.8916122433091,"lng":-8.58585103162513},{"lat":40.8916812276657,"lng":-8.58571160322622},{"lat":40.8917535974961,"lng":-8.58555867014722},{"lat":40.8917769492124,"lng":-8.58539116230453},{"lat":40.8917985989334,"lng":-8.5852442899388},{"lat":40.8918699724618,"lng":-8.58495524625756},{"lat":40.8919859119508,"lng":-8.58482824047741},{"lat":40.892099447428,"lng":-8.58471700002175},{"lat":40.8920959750018,"lng":-8.58459261930395},{"lat":40.8919959743662,"lng":-8.58449048086574},{"lat":40.8918271376819,"lng":-8.58444447442344},{"lat":40.8917594622238,"lng":-8.58436557902215},{"lat":40.8917271465861,"lng":-8.58420243675954},{"lat":40.8917125776467,"lng":-8.58408296521558},{"lat":40.8918445114807,"lng":-8.5835971188387},{"lat":40.8919095394467,"lng":-8.58345576218521},{"lat":40.8919715480419,"lng":-8.58332660581578},{"lat":40.8920557916702,"lng":-8.58319890754792},{"lat":40.8921634399871,"lng":-8.5830731462747},{"lat":40.8922977639056,"lng":-8.58303371569308},{"lat":40.8925106090981,"lng":-8.58294794724068},{"lat":40.892703422665,"lng":-8.58285005757012},{"lat":40.8928136691163,"lng":-8.58277355757157},{"lat":40.8928825099617,"lng":-8.58269286111727},{"lat":40.8929816680019,"lng":-8.58248160305935},{"lat":40.8930359957987,"lng":-8.58235880079872},{"lat":40.8930515342748,"lng":-8.58223074975149},{"lat":40.8930396125088,"lng":-8.58207866149596},{"lat":40.8930014097659,"lng":-8.58190028958434},{"lat":40.8929419870673,"lng":-8.58176010182988},{"lat":40.8928576578739,"lng":-8.58167943167546},{"lat":40.8928405100616,"lng":-8.58155116010605},{"lat":40.8929244119271,"lng":-8.58149465400231},{"lat":40.8930218192857,"lng":-8.58144713636464},{"lat":40.8931138422386,"lng":-8.58142521373714},{"lat":40.8932131100612,"lng":-8.58143905796704},{"lat":40.8933179905672,"lng":-8.58142208849875},{"lat":40.8934978323085,"lng":-8.58147436799216},{"lat":40.8936361493173,"lng":-8.58151457656304},{"lat":40.8938531513362,"lng":-8.58132603856324},{"lat":40.8939018975526,"lng":-8.58122574139094},{"lat":40.8939931470657,"lng":-8.58121746072272},{"lat":40.894088644985,"lng":-8.58115889428808},{"lat":40.8941318094372,"lng":-8.58103530042574},{"lat":40.8940868392467,"lng":-8.58090624443888},{"lat":40.8944652682067,"lng":-8.58075166260931},{"lat":40.8959206112026,"lng":-8.58017319069991},{"lat":40.896423755204,"lng":-8.58015914891035},{"lat":40.8967510168762,"lng":-8.58015079987953},{"lat":40.8969187777045,"lng":-8.58012653838683},{"lat":40.8972154244365,"lng":-8.58011786320684},{"lat":40.8975105321007,"lng":-8.58011131488693},{"lat":40.8977716443996,"lng":-8.58011723327686},{"lat":40.8979473772773,"lng":-8.58012684680719},{"lat":40.8980778913353,"lng":-8.58014054688038},{"lat":40.8981762036933,"lng":-8.58014536468388},{"lat":40.8983116542476,"lng":-8.58013999044182},{"lat":40.8984358744422,"lng":-8.58012801470102},{"lat":40.8985278846385,"lng":-8.58010952998569},{"lat":40.8986389980387,"lng":-8.58013365917793},{"lat":40.8988581063469,"lng":-8.58017335461596},{"lat":40.8990063013392,"lng":-8.58020188852797},{"lat":40.8991480300695,"lng":-8.5802262253989},{"lat":40.8992978127208,"lng":-8.58026331606471},{"lat":40.8994266560844,"lng":-8.58031308046753},{"lat":40.899531170215,"lng":-8.58036683483869},{"lat":40.8996277529572,"lng":-8.58042267211564},{"lat":40.8997402882342,"lng":-8.58049784257832},{"lat":40.8998206544882,"lng":-8.58055570711724},{"lat":40.8999411376066,"lng":-8.58064253653889},{"lat":40.9001067973318,"lng":-8.5807699541787},{"lat":40.9002321346478,"lng":-8.58086431911165},{"lat":40.9003415350873,"lng":-8.58093507881422},{"lat":40.9004236451491,"lng":-8.58098441140387},{"lat":40.900599205315,"lng":-8.58101526982005},{"lat":40.9007491037981,"lng":-8.58102269320895},{"lat":40.9008490355127,"lng":-8.58102764333795},{"lat":40.9010443021101,"lng":-8.58099276965347},{"lat":40.901171822743,"lng":-8.58094272145572},{"lat":40.9013301653014,"lng":-8.5808630925106},{"lat":40.9014641569351,"lng":-8.58079269808825},{"lat":40.9016096114162,"lng":-8.58071628114924},{"lat":40.9018551183999,"lng":-8.58059036485671},{"lat":40.9020585139649,"lng":-8.58052587409179},{"lat":40.9021519881943,"lng":-8.58052437132545},{"lat":40.902412891714,"lng":-8.58058357706838},{"lat":40.9027038200539,"lng":-8.5806559243327},{"lat":40.9028450412953,"lng":-8.58068071489702},{"lat":40.9029541580046,"lng":-8.58069986977337},{"lat":40.9032843058522,"lng":-8.5807825706271},{"lat":40.9036160979242,"lng":-8.58085923175673},{"lat":40.9038883966404,"lng":-8.58090570039803},{"lat":40.9042654260805,"lng":-8.58095050581054},{"lat":40.904521714944,"lng":-8.58096921375436},{"lat":40.9046232766074,"lng":-8.58097203761708},{"lat":40.9048133538141,"lng":-8.58095172516417},{"lat":40.904968251869,"lng":-8.5809699842637},{"lat":40.9052438999247,"lng":-8.58098894298905},{"lat":40.9055017929538,"lng":-8.58101193492742},{"lat":40.9057725055564,"lng":-8.58102646809378},{"lat":40.9067520953578,"lng":-8.58109383396685},{"lat":40.9076533741279,"lng":-8.58116397344107},{"lat":40.9084527930433,"lng":-8.58124225318285},{"lat":40.9085380710108,"lng":-8.5812873395666},{"lat":40.9086602248764,"lng":-8.58143581970894},{"lat":40.9090449396484,"lng":-8.58154145303978},{"lat":40.9092896757291,"lng":-8.58161326129549},{"lat":40.9094245681183,"lng":-8.58165824247426},{"lat":40.9096874916044,"lng":-8.58173061843041},{"lat":40.9098695340345,"lng":-8.58176164586042},{"lat":40.9099839383217,"lng":-8.58177298654243},{"lat":40.9102546164169,"lng":-8.58181945267145},{"lat":40.9105929672391,"lng":-8.58185355999087},{"lat":40.9110761417624,"lng":-8.58194906696144},{"lat":40.912927753381,"lng":-8.58397429399938},{"lat":40.9131853714004,"lng":-8.58426921564754},{"lat":40.9132825706878,"lng":-8.58437779896623},{"lat":40.9133566245338,"lng":-8.58446044444466},{"lat":40.913441033048,"lng":-8.58454304184228},{"lat":40.9135790269563,"lng":-8.58464654164832},{"lat":40.9137426492187,"lng":-8.58482772698955},{"lat":40.9138356389893,"lng":-8.58492640844181},{"lat":40.9139646739959,"lng":-8.58508682423501},{"lat":40.9140645521221,"lng":-8.58521973941527},{"lat":40.9141403701832,"lng":-8.58528863060128},{"lat":40.9142798587249,"lng":-8.58544686375096},{"lat":40.9143986817112,"lng":-8.58557076919332},{"lat":40.9145193143037,"lng":-8.58569243326031},{"lat":40.9146760284433,"lng":-8.58586669186826},{"lat":40.9148326607504,"lng":-8.58606148757345},{"lat":40.9149066145256,"lng":-8.58616906269092},{"lat":40.9152942538679,"lng":-8.58658207588236},{"lat":40.9155971700124,"lng":-8.58692388288197},{"lat":40.9158984908011,"lng":-8.58728158975691},{"lat":40.9161517725593,"lng":-8.58755919837936},{"lat":40.9164066757977,"lng":-8.58783670272089},{"lat":40.9166683924343,"lng":-8.58814405123089},{"lat":40.916876642951,"lng":-8.58840070123845},{"lat":40.9169764270617,"lng":-8.58851083607533},{"lat":40.9171297057248,"lng":-8.58868745839447},{"lat":40.9173070706465,"lng":-8.58887552418121},{"lat":40.9174569903848,"lng":-8.58903610077966},{"lat":40.9175964191606,"lng":-8.58920811949441},{"lat":40.9183706842489,"lng":-8.59007920267218},{"lat":40.9184689697676,"lng":-8.5902941504392},{"lat":40.9194529741678,"lng":-8.59118024478286},{"lat":40.9195433861497,"lng":-8.5913288633821},{"lat":40.919912352647,"lng":-8.59165823327463},{"lat":40.9201892468957,"lng":-8.59188361336804},{"lat":40.9209369483837,"lng":-8.59254749626111},{"lat":40.9216891270145,"lng":-8.59326121213797},{"lat":40.9219058089612,"lng":-8.59348156670199},{"lat":40.9220263422865,"lng":-8.59359415313532},{"lat":40.9224962272638,"lng":-8.59400160695378},{"lat":40.9235089038687,"lng":-8.5944636984464},{"lat":40.9243984218877,"lng":-8.5948726446279},{"lat":40.92660460583,"lng":-8.5956736534503},{"lat":40.9276340568357,"lng":-8.59531645781632},{"lat":40.9277707863242,"lng":-8.59173676655298},{"lat":40.9272169790499,"lng":-8.58434435565064},{"lat":40.9248913854232,"lng":-8.57742169089595},{"lat":40.9257017114828,"lng":-8.56682714687722},{"lat":40.9262429968746,"lng":-8.56363957386519},{"lat":40.9266314910541,"lng":-8.56140422814039},{"lat":40.9290773273199,"lng":-8.5574397844966},{"lat":40.9341769294374,"lng":-8.54772494728468},{"lat":40.9358313818571,"lng":-8.54455491361522},{"lat":40.9365457127924,"lng":-8.54359693741901},{"lat":40.9378572350451,"lng":-8.54187898206781},{"lat":40.9359746821409,"lng":-8.54137961869191},{"lat":40.9336131709664,"lng":-8.54158318334398},{"lat":40.9316092482936,"lng":-8.54111551846903},{"lat":40.9296102072293,"lng":-8.54090867410702},{"lat":40.9270063466461,"lng":-8.54111053318056},{"lat":40.9242827551823,"lng":-8.54005915343204},{"lat":40.9212753258773,"lng":-8.54020935053897},{"lat":40.917577324279,"lng":-8.54128889224507},{"lat":40.9158181000056,"lng":-8.54196477922627},{"lat":40.9141912487985,"lng":-8.54181458211934},{"lat":40.9122995110077,"lng":-8.54171445071472},{"lat":40.9104077190876,"lng":-8.54191471352396},{"lat":40.9089699210271,"lng":-8.54181458211934},{"lat":40.9070780338408,"lng":-8.5416143193101},{"lat":40.9057915196363,"lng":-8.54151418790548},{"lat":40.9043380040874,"lng":-8.54128720102943},{"lat":40.9039983683771,"lng":-8.54012607674781},{"lat":40.9010170628333,"lng":-8.53638142265323},{"lat":40.8982507089194,"lng":-8.53374744963529},{"lat":40.8953909022986,"lng":-8.53154188251856},{"lat":40.8950262674417,"lng":-8.53126068154176},{"lat":40.8930056952916,"lng":-8.53105206781968},{"lat":40.8915152675302,"lng":-8.53106269008129},{"lat":40.8913273795414,"lng":-8.53084334392517},{"lat":40.8912102081796,"lng":-8.53069013436012},{"lat":40.891030229073,"lng":-8.53053043018235}]},{"zone_id":"oaz3","points":[{"lat":40.916523054763,"lng":-8.4558332705752},{"lat":40.9154514351228,"lng":-8.45406593802366},{"lat":40.913982768633,"lng":-8.45307035248724},{"lat":40.9085124653555,"lng":-8.45121637282024},{"lat":40.9030299186591,"lng":-8.44865403676439},{"lat":40.8979674549647,"lng":-8.44877238666552},{"lat":40.8902050587118,"lng":-8.44395945690879},{"lat":40.8856904287262,"lng":-8.44091779504638},{"lat":40.8828478595615,"lng":-8.43812911621268},{"lat":40.8807452550922,"lng":-8.43665012304198},{"lat":40.8787665412871,"lng":-8.43517178704534},{"lat":40.8781899296018,"lng":-8.43488892314045},{"lat":40.8764152238787,"lng":-8.43401835290799},{"lat":40.8742469975694,"lng":-8.43392691339236},{"lat":40.871083857484,"lng":-8.43521848756763},{"lat":40.8670775122113,"lng":-8.43907690840037},{"lat":40.8608305233847,"lng":-8.4466680798672},{"lat":40.8550646327603,"lng":-8.45547775350103},{"lat":40.8531067791949,"lng":-8.457463186327},{"lat":40.8498200521348,"lng":-8.46085680487938},{"lat":40.8458821437066,"lng":-8.46421295294657},{"lat":40.8459016576655,"lng":-8.46967572789757},{"lat":40.8464102023942,"lng":-8.47750123694784},{"lat":40.8464668772747,"lng":-8.48382146742838},{"lat":40.8466172968103,"lng":-8.4899512544343},{"lat":40.8468514563555,"lng":-8.49381182001714},{"lat":40.8477672418179,"lng":-8.49772163073552},{"lat":40.8508740742092,"lng":-8.49576279539231},{"lat":40.8541738500614,"lng":-8.49435447757555},{"lat":40.8595592306764,"lng":-8.49664968825132},{"lat":40.8627751475222,"lng":-8.49604224937052},{"lat":40.8727160077467,"lng":-8.50034984658321},{"lat":40.8776721133762,"lng":-8.50348086306376},{"lat":40.8814907842753,"lng":-8.50392158239252},{"lat":40.8848619773503,"lng":-8.50402300100954},{"lat":40.8821984646239,"lng":-8.49982143564286},{"lat":40.881967566805,"lng":-8.49966954412052},{"lat":40.8818142283378,"lng":-8.49917952650462},{"lat":40.8815273327266,"lng":-8.4978969804741},{"lat":40.8810630711415,"lng":-8.49435050420012},{"lat":40.8799359924021,"lng":-8.48568627500205},{"lat":40.8790118256284,"lng":-8.48509574000062},{"lat":40.8789508539873,"lng":-8.4846896061451},{"lat":40.8813378377883,"lng":-8.48285597746233},{"lat":40.8815500017706,"lng":-8.48223053236049},{"lat":40.8814237117128,"lng":-8.48158127956842},{"lat":40.880134845443,"lng":-8.48109905878637},{"lat":40.8799769139612,"lng":-8.48078747549387},{"lat":40.8801331760326,"lng":-8.48059505018124},{"lat":40.8811074195084,"lng":-8.48020823173781},{"lat":40.8818046727528,"lng":-8.48012636940984},{"lat":40.8821994199509,"lng":-8.47994556573053},{"lat":40.8832366783559,"lng":-8.4781278414719},{"lat":40.8849380752703,"lng":-8.47773443766875},{"lat":40.8854175878289,"lng":-8.47782536981468},{"lat":40.8857878157136,"lng":-8.47719073597639},{"lat":40.8863765704544,"lng":-8.47672570244377},{"lat":40.8871054529991,"lng":-8.47677233287743},{"lat":40.8875000587168,"lng":-8.47654497733595},{"lat":40.8877395933352,"lng":-8.47628226674882},{"lat":40.8876290776918,"lng":-8.47568680167696},{"lat":40.8873543146576,"lng":-8.47538659869086},{"lat":40.8877597181182,"lng":-8.47438733658166},{"lat":40.888392302629,"lng":-8.47440268761171},{"lat":40.8887446062885,"lng":-8.47496123676672},{"lat":40.8903037276257,"lng":-8.47472696647786},{"lat":40.8917563042494,"lng":-8.47604871059997},{"lat":40.893216708802,"lng":-8.47361207833261},{"lat":40.8989459032001,"lng":-8.47754350428631},{"lat":40.8993505203525,"lng":-8.47702859907313},{"lat":40.8995721161364,"lng":-8.47717986723799},{"lat":40.9007372554846,"lng":-8.47617334603455},{"lat":40.9015156171205,"lng":-8.47641380363837},{"lat":40.9018497125567,"lng":-8.47606462406999},{"lat":40.9019666754816,"lng":-8.47606392906832},{"lat":40.90205754124,"lng":-8.47589395577389},{"lat":40.9022767498693,"lng":-8.47606350051234},{"lat":40.9024499278987,"lng":-8.47619876952021},{"lat":40.9027798890238,"lng":-8.47508260826988},{"lat":40.9034521583385,"lng":-8.47333268735552},{"lat":40.9035600888167,"lng":-8.47337501932708},{"lat":40.9037173090947,"lng":-8.47343612109405},{"lat":40.9038874707543,"lng":-8.47356624360587},{"lat":40.9040707278409,"lng":-8.47371376185242},{"lat":40.9042148912157,"lng":-8.47377479671202},{"lat":40.9043916639995,"lng":-8.47386187365436},{"lat":40.9045948455179,"lng":-8.47394054132505},{"lat":40.9047705295368,"lng":-8.47403034338634},{"lat":40.9048069370569,"lng":-8.47396015107372},{"lat":40.9048891835771,"lng":-8.4738676991394},{"lat":40.9049311660694,"lng":-8.47382050720331},{"lat":40.9049664661017,"lng":-8.47360753236767},{"lat":40.9050422741836,"lng":-8.47358110056136},{"lat":40.905261353967,"lng":-8.47306607069027},{"lat":40.9054399442002,"lng":-8.47269206266543},{"lat":40.9058560364395,"lng":-8.47138640286351},{"lat":40.9056593731042,"lng":-8.47123418877185},{"lat":40.9054217317567,"lng":-8.47108437687975},{"lat":40.9052339002996,"lng":-8.47102146166035},{"lat":40.9050241580041,"lng":-8.47093778395738},{"lat":40.9048927240812,"lng":-8.47089580896225},{"lat":40.904789747928,"lng":-8.47073019240005},{"lat":40.9047491433924,"lng":-8.47057522021021},{"lat":40.9044273975175,"lng":-8.47030812033733},{"lat":40.9045968130158,"lng":-8.46958485566648},{"lat":40.9047541244154,"lng":-8.46906285467937},{"lat":40.904849354226,"lng":-8.46873814419006},{"lat":40.9050820211776,"lng":-8.46868117268469},{"lat":40.9053263453043,"lng":-8.46876639515983},{"lat":40.905505347418,"lng":-8.46882883314192},{"lat":40.9056123465064,"lng":-8.46860541802822},{"lat":40.9058752190951,"lng":-8.46810352519381},{"lat":40.9059310233751,"lng":-8.46793782882052},{"lat":40.9059629382724,"lng":-8.46779052005082},{"lat":40.9060007717156,"lng":-8.46766205582238},{"lat":40.9060865507935,"lng":-8.46748920758255},{"lat":40.9061252340855,"lng":-8.46734733668657},{"lat":40.9061688453407,"lng":-8.4672139160799},{"lat":40.9062613554191,"lng":-8.46723361151913},{"lat":40.906421552702,"lng":-8.46726565888103},{"lat":40.9064956999455,"lng":-8.4672804920429},{"lat":40.90656172593,"lng":-8.4672421343131},{"lat":40.9066454015312,"lng":-8.46698726194566},{"lat":40.9067057192941,"lng":-8.46680609363361},{"lat":40.9068316065677,"lng":-8.46686714275931},{"lat":40.9069566940315,"lng":-8.46686290898256},{"lat":40.9070290766902,"lng":-8.46680606697991},{"lat":40.9070671307439,"lng":-8.46675629118625},{"lat":40.9071047127726,"lng":-8.46649762289429},{"lat":40.9071549838369,"lng":-8.46636641767981},{"lat":40.9072067888727,"lng":-8.46623120825346},{"lat":40.9072841007957,"lng":-8.46598086859948},{"lat":40.9073712465319,"lng":-8.46575479040587},{"lat":40.9074347525782,"lng":-8.46565363212646},{"lat":40.907525537087,"lng":-8.46555450931407},{"lat":40.9078928780161,"lng":-8.46526046669491},{"lat":40.9079624015286,"lng":-8.46528906424837},{"lat":40.908220078798,"lng":-8.46514591561452},{"lat":40.9083199228568,"lng":-8.46524528645745},{"lat":40.9083462860853,"lng":-8.46531437829784},{"lat":40.9084159931637,"lng":-8.46528437765044},{"lat":40.9085165219363,"lng":-8.4657492402331},{"lat":40.9087034924698,"lng":-8.46568981690597},{"lat":40.9087689717989,"lng":-8.46577774126887},{"lat":40.9089437891514,"lng":-8.46598217803565},{"lat":40.909014247937,"lng":-8.46602953617312},{"lat":40.9091266847076,"lng":-8.46607140771556},{"lat":40.9092173634661,"lng":-8.46607079672009},{"lat":40.9092657225389,"lng":-8.46606926127562},{"lat":40.9093498392758,"lng":-8.46609555961558},{"lat":40.909550858732,"lng":-8.46608340016545},{"lat":40.9096635803461,"lng":-8.4661144381101},{"lat":40.9097753364586,"lng":-8.46614827106617},{"lat":40.9100611661441,"lng":-8.4662646089773},{"lat":40.9102286614387,"lng":-8.46623043937758},{"lat":40.9104727177173,"lng":-8.4661579626212},{"lat":40.9106934840018,"lng":-8.46613664397403},{"lat":40.911000784189,"lng":-8.46617736403979},{"lat":40.9112725996399,"lng":-8.46617188442982},{"lat":40.9113445111235,"lng":-8.46617043473814},{"lat":40.911417216129,"lng":-8.46615774407102},{"lat":40.9117465211646,"lng":-8.46575078744295},{"lat":40.9117874672572,"lng":-8.46569907062231},{"lat":40.9119481448575,"lng":-8.46559400292105},{"lat":40.9119361253733,"lng":-8.46542278108502},{"lat":40.9119073477227,"lng":-8.46528566051383},{"lat":40.912047933597,"lng":-8.46524984746451},{"lat":40.9125938909531,"lng":-8.46508887218431},{"lat":40.912511068612,"lng":-8.46393708601191},{"lat":40.9123864556015,"lng":-8.46392542228411},{"lat":40.9129978329845,"lng":-8.46026710316891},{"lat":40.9137492843028,"lng":-8.45914569797635},{"lat":40.9148168318422,"lng":-8.45755250465711},{"lat":40.9151358295041,"lng":-8.45723108309794},{"lat":40.916523054763,"lng":-8.4558332705752}]},{"zone_id":"oaz2","points":[{"lat":40.8848619773503,"lng":-8.50402300100954},{"lat":40.8814907842753,"lng":-8.50392158239252},{"lat":40.8776721133762,"lng":-8.50348086306376},{"lat":40.8727160077467,"lng":-8.50034984658321},{"lat":40.8627751475222,"lng":-8.49604224937052},{"lat":40.8595592306764,"lng":-8.49664968825132},{"lat":40.8541738500614,"lng":-8.49435447757555},{"lat":40.8508740742092,"lng":-8.49576279539231},{"lat":40.8477672418179,"lng":-8.49772163073552},{"lat":40.836018494418,"lng":-8.50512728996131},{"lat":40.8320011548525,"lng":-8.507658896244},{"lat":40.8304726832615,"lng":-8.50877951215725},{"lat":40.8295135396835,"lng":-8.51065610003284},{"lat":40.8273051079384,"lng":-8.51566207743413},{"lat":40.8263399867888,"lng":-8.5192948928439},{"lat":40.8254696275392,"lng":-8.5230536167688},{"lat":40.8240343810714,"lng":-8.53503168045751},{"lat":40.8241346614982,"lng":-8.53504621213479},{"lat":40.8243415759412,"lng":-8.53508152312589},{"lat":40.8244984540619,"lng":-8.53512297851773},{"lat":40.8246783919458,"lng":-8.53515148747922},{"lat":40.8248233930296,"lng":-8.53516052005561},{"lat":40.8249664036708,"lng":-8.53516138727482},{"lat":40.8250767705147,"lng":-8.53516205735746},{"lat":40.8271558621225,"lng":-8.53829548472313},{"lat":40.8303895318905,"lng":-8.53995852796922},{"lat":40.8340827490458,"lng":-8.5395261233552},{"lat":40.836810201453,"lng":-8.54006409053555},{"lat":40.8383459363971,"lng":-8.54014250254426},{"lat":40.8383838427513,"lng":-8.54054977052327},{"lat":40.8387764154446,"lng":-8.54055953731685},{"lat":40.8389590184822,"lng":-8.54053789636153},{"lat":40.83922563238,"lng":-8.54049175294857},{"lat":40.8391854214378,"lng":-8.54020113565672},{"lat":40.8393866695926,"lng":-8.54007645421325},{"lat":40.8394652805406,"lng":-8.54020344897751},{"lat":40.8395312878812,"lng":-8.54035526517063},{"lat":40.8396230624726,"lng":-8.5405282273208},{"lat":40.8397363004859,"lng":-8.54065273660883},{"lat":40.8403209495795,"lng":-8.54072699440891},{"lat":40.840456891597,"lng":-8.54067415973549},{"lat":40.8406132369396,"lng":-8.54056112287404},{"lat":40.8407548952037,"lng":-8.54076639867621},{"lat":40.8409498014059,"lng":-8.54116735952996},{"lat":40.8410523723279,"lng":-8.54140098606545},{"lat":40.8412704537323,"lng":-8.54178697602098},{"lat":40.841385847645,"lng":-8.54203945773989},{"lat":40.8420088279135,"lng":-8.54283971764337},{"lat":40.8421101367748,"lng":-8.5429858806763},{"lat":40.8423307048356,"lng":-8.54318288661032},{"lat":40.8424420012499,"lng":-8.54325910657916},{"lat":40.842533597654,"lng":-8.5433799066044},{"lat":40.8427040171983,"lng":-8.54356771580415},{"lat":40.8428026367274,"lng":-8.54373895428728},{"lat":40.8428541526285,"lng":-8.5438373335272},{"lat":40.8428988933658,"lng":-8.54394148124855},{"lat":40.8429231013726,"lng":-8.54407135160266},{"lat":40.8429106505747,"lng":-8.54445533377045},{"lat":40.8428998108055,"lng":-8.54464083393064},{"lat":40.8428506003583,"lng":-8.54482905976342},{"lat":40.8428016397265,"lng":-8.54497270399384},{"lat":40.8427607826812,"lng":-8.5451165155839},{"lat":40.8433682375519,"lng":-8.54526839493159},{"lat":40.8432858391034,"lng":-8.54542048637057},{"lat":40.8431566879319,"lng":-8.54555224837008},{"lat":40.8431193655734,"lng":-8.54566430532542},{"lat":40.8431288361765,"lng":-8.54586060481948},{"lat":40.8431690232082,"lng":-8.54602863695586},{"lat":40.8432156945813,"lng":-8.54614631621688},{"lat":40.8432660938658,"lng":-8.54630421586806},{"lat":40.8432616473435,"lng":-8.54648868925592},{"lat":40.8432483659975,"lng":-8.5466261530238},{"lat":40.8432389075447,"lng":-8.5467770383388},{"lat":40.8432293948757,"lng":-8.54694298234588},{"lat":40.8432429291157,"lng":-8.54706045617617},{"lat":40.8433250149227,"lng":-8.54727060847343},{"lat":40.8433704782237,"lng":-8.54739836028551},{"lat":40.8434332386183,"lng":-8.54763229600191},{"lat":40.8434798474851,"lng":-8.54780812672771},{"lat":40.8435877491693,"lng":-8.54800065529208},{"lat":40.8436878685489,"lng":-8.54815388839455},{"lat":40.8437579323026,"lng":-8.54825155924946},{"lat":40.8438633419621,"lng":-8.54836119117951},{"lat":40.8440078822525,"lng":-8.54853154190969},{"lat":40.8441004076191,"lng":-8.54866789142317},{"lat":40.8441700863887,"lng":-8.54877231991427},{"lat":40.8442232814883,"lng":-8.5488783045048},{"lat":40.8443104030203,"lng":-8.5488406706376},{"lat":40.8443406617857,"lng":-8.54871552464776},{"lat":40.8444050796036,"lng":-8.54858181951261},{"lat":40.8444713524864,"lng":-8.54868183974868},{"lat":40.8445694276382,"lng":-8.54882735559233},{"lat":40.8446796952018,"lng":-8.54896286895441},{"lat":40.8447780935266,"lng":-8.54904388237343},{"lat":40.844883057489,"lng":-8.54922643894439},{"lat":40.8449815964064,"lng":-8.54926867871545},{"lat":40.8450602992086,"lng":-8.54934292813857},{"lat":40.8451642581881,"lng":-8.54947970942998},{"lat":40.8452236469395,"lng":-8.5495907152645},{"lat":40.8453000624013,"lng":-8.54967491165951},{"lat":40.8453915953862,"lng":-8.54968722652677},{"lat":40.8454804242794,"lng":-8.54977482240271},{"lat":40.8455882220486,"lng":-8.54987107507465},{"lat":40.8456488894887,"lng":-8.54997699105489},{"lat":40.8457288348961,"lng":-8.55005611162725},{"lat":40.845814838598,"lng":-8.55012803817548},{"lat":40.8481560918456,"lng":-8.55066748201844},{"lat":40.8487151767691,"lng":-8.5507963055855},{"lat":40.8496756878968,"lng":-8.54998455966086},{"lat":40.8504012158893,"lng":-8.54912301586228},{"lat":40.8507577580644,"lng":-8.54883427673906},{"lat":40.8513469067148,"lng":-8.54821558718342},{"lat":40.8519112032869,"lng":-8.54759742670265},{"lat":40.8520551243679,"lng":-8.54738949534315},{"lat":40.8521628809685,"lng":-8.54724798031672},{"lat":40.852260177978,"lng":-8.54711031324453},{"lat":40.8523348390126,"lng":-8.54700713129288},{"lat":40.8524259134742,"lng":-8.54689705668532},{"lat":40.8525548533187,"lng":-8.54674926785491},{"lat":40.8526664055329,"lng":-8.54660374219628},{"lat":40.8527690103481,"lng":-8.54644238765581},{"lat":40.8528564607573,"lng":-8.5463383360771},{"lat":40.8529333688583,"lng":-8.54623611622321},{"lat":40.8529967911515,"lng":-8.54612788274502},{"lat":40.853092547554,"lng":-8.54599257342455},{"lat":40.8531809628085,"lng":-8.5458203368818},{"lat":40.8532691854987,"lng":-8.54567644129423},{"lat":40.8533596504169,"lng":-8.54556019268699},{"lat":40.8534757121103,"lng":-8.54543746121714},{"lat":40.8535675864325,"lng":-8.54525457094029},{"lat":40.8536480070561,"lng":-8.54510173088885},{"lat":40.853723619034,"lng":-8.54495894118816},{"lat":40.8538009977777,"lng":-8.54474998770768},{"lat":40.8538592371505,"lng":-8.54460519227876},{"lat":40.8539364072141,"lng":-8.54445458396286},{"lat":40.854005849587,"lng":-8.54432456297774},{"lat":40.854084528465,"lng":-8.54417977447262},{"lat":40.8541744616705,"lng":-8.544010388612},{"lat":40.8542908384057,"lng":-8.54377392337071},{"lat":40.8544064664383,"lng":-8.54354527997233},{"lat":40.8545379744205,"lng":-8.54328234102128},{"lat":40.8546458466025,"lng":-8.54310713478887},{"lat":40.854778117611,"lng":-8.54285700709295},{"lat":40.8548544505587,"lng":-8.54271326809005},{"lat":40.8549329764279,"lng":-8.54256053013022},{"lat":40.8550608643107,"lng":-8.54227669189948},{"lat":40.8551342013973,"lng":-8.54211419611613},{"lat":40.8552227282023,"lng":-8.54190909852054},{"lat":40.8553045146113,"lng":-8.54170016386452},{"lat":40.855399115772,"lng":-8.54148324278657},{"lat":40.8554573843877,"lng":-8.54135421343969},{"lat":40.8555126822268,"lng":-8.54122516682144},{"lat":40.8555754652242,"lng":-8.54106746533918},{"lat":40.8556231774101,"lng":-8.54094418182447},{"lat":40.8557789797499,"lng":-8.54055816230258},{"lat":40.8560859237623,"lng":-8.54005791191924},{"lat":40.8562213967294,"lng":-8.5398640367486},{"lat":40.8564468178018,"lng":-8.5395406000612},{"lat":40.8567085898969,"lng":-8.53911971560387},{"lat":40.8567831563469,"lng":-8.53899018931212},{"lat":40.8568200558677,"lng":-8.53871562276007},{"lat":40.8568245910324,"lng":-8.53849993579185},{"lat":40.85682174477,"lng":-8.53823732550685},{"lat":40.8568112183784,"lng":-8.53803054462987},{"lat":40.8568141980839,"lng":-8.53782372741637},{"lat":40.8568261101733,"lng":-8.53763771945502},{"lat":40.8568718153581,"lng":-8.53739285542903},{"lat":40.8569219158905,"lng":-8.53720411548094},{"lat":40.8569763457464,"lng":-8.5370390029254},{"lat":40.8570600009558,"lng":-8.53685936232865},{"lat":40.857136819842,"lng":-8.53670328004381},{"lat":40.8572045089552,"lng":-8.53658260350317},{"lat":40.8573689623986,"lng":-8.53642112441603},{"lat":40.8574791461479,"lng":-8.53635395777134},{"lat":40.8575892261559,"lng":-8.53631620213879},{"lat":40.857721727692,"lng":-8.53627858232391},{"lat":40.8577401066719,"lng":-8.53614764172343},{"lat":40.8576325551643,"lng":-8.53610630731184},{"lat":40.8575361561303,"lng":-8.53606883584534},{"lat":40.8574296797646,"lng":-8.53602904882841},{"lat":40.8573370522929,"lng":-8.53599456545472},{"lat":40.8573529269965,"lng":-8.53578177714942},{"lat":40.857407126821,"lng":-8.5356043264952},{"lat":40.8575020515716,"lng":-8.53567724875727},{"lat":40.8577329666128,"lng":-8.53520484652827},{"lat":40.8579123361318,"lng":-8.53533485581223},{"lat":40.8580545442749,"lng":-8.53489084883183},{"lat":40.8582178955754,"lng":-8.53477513436707},{"lat":40.8584688877207,"lng":-8.53459733579099},{"lat":40.8585254273238,"lng":-8.53439415853734},{"lat":40.8585791717656,"lng":-8.53426936050993},{"lat":40.8585637736614,"lng":-8.53385040456461},{"lat":40.8585367241401,"lng":-8.53329894267007},{"lat":40.8588002694502,"lng":-8.53321809922124},{"lat":40.8589026100368,"lng":-8.53295590578221},{"lat":40.8589495723155,"lng":-8.53278718065634},{"lat":40.8590356914443,"lng":-8.53272567217788},{"lat":40.8590942520965,"lng":-8.53243248472033},{"lat":40.8592526033459,"lng":-8.53249606214926},{"lat":40.8592899956615,"lng":-8.5323083028932},{"lat":40.8593596401619,"lng":-8.53198647941803},{"lat":40.8594328510754,"lng":-8.53162280932659},{"lat":40.8594841862883,"lng":-8.53138947224359},{"lat":40.8595122126896,"lng":-8.53126208107104},{"lat":40.8596724477192,"lng":-8.53134280578656},{"lat":40.8597280079162,"lng":-8.53108280569523},{"lat":40.859928820132,"lng":-8.53100288878541},{"lat":40.8599558308824,"lng":-8.53076999350797},{"lat":40.8599693166916,"lng":-8.53056797435895},{"lat":40.8601134870857,"lng":-8.52991604206239},{"lat":40.8610009203446,"lng":-8.52819354101643},{"lat":40.8618910965186,"lng":-8.52646561967082},{"lat":40.8634247308133,"lng":-8.52745066267807},{"lat":40.8638701862069,"lng":-8.52758402742147},{"lat":40.8641784247929,"lng":-8.52768668434886},{"lat":40.8642759183911,"lng":-8.52772083400956},{"lat":40.8646224632396,"lng":-8.52786535419518},{"lat":40.8648189469905,"lng":-8.52794362469304},{"lat":40.8652264806023,"lng":-8.52810618436816},{"lat":40.8654994316988,"lng":-8.52823164747317},{"lat":40.8656093801091,"lng":-8.52829979819778},{"lat":40.8657696234818,"lng":-8.52802780416202},{"lat":40.8659233968788,"lng":-8.52780129479423},{"lat":40.8660010246659,"lng":-8.52768193541175},{"lat":40.8660599637022,"lng":-8.52756999033265},{"lat":40.8661477912484,"lng":-8.52737520400258},{"lat":40.8661776094129,"lng":-8.52724322309942},{"lat":40.8662127294528,"lng":-8.52708037512214},{"lat":40.8662558305791,"lng":-8.52682565202309},{"lat":40.8663315253301,"lng":-8.52650895111135},{"lat":40.8663690154471,"lng":-8.52626979325924},{"lat":40.8664209174977,"lng":-8.52602773209387},{"lat":40.866472825501,"lng":-8.52576447165293},{"lat":40.8666247066009,"lng":-8.52521428897252},{"lat":40.8666865874554,"lng":-8.5250090410369},{"lat":40.8667525891766,"lng":-8.52483569695814},{"lat":40.8668310160958,"lng":-8.52468472601674},{"lat":40.8668854070442,"lng":-8.52457829417047},{"lat":40.8670469534493,"lng":-8.52433350782209},{"lat":40.8671397291139,"lng":-8.52429173267774},{"lat":40.8672436695797,"lng":-8.52428430312276},{"lat":40.8673990189098,"lng":-8.52430175785848},{"lat":40.8675802639939,"lng":-8.52431867096944},{"lat":40.8677326338634,"lng":-8.52431565671639},{"lat":40.8679198566613,"lng":-8.52430653105737},{"lat":40.8680586714075,"lng":-8.52424887851107},{"lat":40.8684523984735,"lng":-8.52400794055015},{"lat":40.8732874859869,"lng":-8.52910050034321},{"lat":40.8734171179963,"lng":-8.52898015819205},{"lat":40.873553873557,"lng":-8.52896163973315},{"lat":40.8736614959964,"lng":-8.52890451166149},{"lat":40.8737642335095,"lng":-8.52882790020454},{"lat":40.873881382681,"lng":-8.52882860130802},{"lat":40.8739939114918,"lng":-8.52875868988246},{"lat":40.8741130618991,"lng":-8.52872725391778},{"lat":40.8742643617908,"lng":-8.52874737751999},{"lat":40.8744355540754,"lng":-8.5286391433763},{"lat":40.8745139324132,"lng":-8.52854969095071},{"lat":40.8746261751591,"lng":-8.52856317339437},{"lat":40.8747239827622,"lng":-8.52850586674256},{"lat":40.874825996071,"lng":-8.52837800022238},{"lat":40.8748994906641,"lng":-8.52829492404859},{"lat":40.8750561602884,"lng":-8.52814152115013},{"lat":40.8752511859459,"lng":-8.52800481868024},{"lat":40.8756864684906,"lng":-8.52767609689901},{"lat":40.8757799275898,"lng":-8.52760354358736},{"lat":40.8759165215748,"lng":-8.52749750376601},{"lat":40.8760579843052,"lng":-8.5275240904459},{"lat":40.8761465948018,"lng":-8.52754893873598},{"lat":40.8763147739263,"lng":-8.5273213355407},{"lat":40.8764002223507,"lng":-8.52740192315616},{"lat":40.8764873048476,"lng":-8.52747836727493},{"lat":40.8765754062828,"lng":-8.52754675165618},{"lat":40.8766609923846,"lng":-8.52761310330469},{"lat":40.8767451629535,"lng":-8.52767232979416},{"lat":40.8768561082994,"lng":-8.52774868029773},{"lat":40.8769393771189,"lng":-8.52780825649893},{"lat":40.8770526433556,"lng":-8.52786421708013},{"lat":40.8771604212655,"lng":-8.52776200353247},{"lat":40.8772146838731,"lng":-8.52758888346494},{"lat":40.8772593771429,"lng":-8.52737062460591},{"lat":40.8772987637392,"lng":-8.52725507174092},{"lat":40.877362796854,"lng":-8.52709482154681},{"lat":40.8774316047417,"lng":-8.52696022481082},{"lat":40.8775349269726,"lng":-8.52687020357299},{"lat":40.8775717687894,"lng":-8.52676163442158},{"lat":40.8776664788225,"lng":-8.52666206954482},{"lat":40.8777762140215,"lng":-8.52656650837261},{"lat":40.8778908410721,"lng":-8.52646207923695},{"lat":40.8779931740268,"lng":-8.52637157501638},{"lat":40.8780680611256,"lng":-8.52627580670866},{"lat":40.8781489158522,"lng":-8.52617283658596},{"lat":40.8782437771743,"lng":-8.5260815755571},{"lat":40.8783200986378,"lng":-8.52601405089211},{"lat":40.8784090291079,"lng":-8.52597175072323},{"lat":40.8784978179139,"lng":-8.52593164560015},{"lat":40.8785914482333,"lng":-8.52587128180567},{"lat":40.8786764280456,"lng":-8.52579941808791},{"lat":40.878725943466,"lng":-8.52569709057276},{"lat":40.8787831179659,"lng":-8.52559326584021},{"lat":40.8788776623092,"lng":-8.52554210123425},{"lat":40.8789816395859,"lng":-8.52549716142537},{"lat":40.8790867431453,"lng":-8.52546551573882},{"lat":40.8791812575358,"lng":-8.52544934802664},{"lat":40.8792797258778,"lng":-8.52543557759621},{"lat":40.8793587700395,"lng":-8.52536142219904},{"lat":40.8794212129879,"lng":-8.52527056023053},{"lat":40.8795024240351,"lng":-8.52519265306248},{"lat":40.8796997336052,"lng":-8.52497869818269},{"lat":40.8798666104407,"lng":-8.52484111704995},{"lat":40.8800355058385,"lng":-8.52474495167745},{"lat":40.8801357275229,"lng":-8.52476877829497},{"lat":40.8802269490757,"lng":-8.52477560136169},{"lat":40.8803665760325,"lng":-8.52475450563998},{"lat":40.8804640640255,"lng":-8.52471118634918},{"lat":40.880539927363,"lng":-8.52461921373746},{"lat":40.8806707263248,"lng":-8.52457787121724},{"lat":40.8807802016692,"lng":-8.52458409597846},{"lat":40.8808763446471,"lng":-8.52459213930188},{"lat":40.8809763243208,"lng":-8.52458407102677},{"lat":40.8810654242681,"lng":-8.52454473435162},{"lat":40.8811493179485,"lng":-8.52447452089021},{"lat":40.8811993214627,"lng":-8.524360683079},{"lat":40.8812859287922,"lng":-8.52428656999221},{"lat":40.8813933915215,"lng":-8.52427581646775},{"lat":40.8814912444881,"lng":-8.52428386949403},{"lat":40.8815911954792,"lng":-8.52431079927183},{"lat":40.8816834618621,"lng":-8.52434646353982},{"lat":40.8817554928968,"lng":-8.52442744826547},{"lat":40.8818061921613,"lng":-8.5245323922839},{"lat":40.8818654790447,"lng":-8.52462730111261},{"lat":40.8819684122644,"lng":-8.52459801281464},{"lat":40.8820430646374,"lng":-8.52451789652422},{"lat":40.8821239340695,"lng":-8.52443639174851},{"lat":40.8822216768597,"lng":-8.52439781816553},{"lat":40.882323768409,"lng":-8.5244039981222},{"lat":40.8824173073914,"lng":-8.52438331506776},{"lat":40.8825111126806,"lng":-8.52433748067727},{"lat":40.8826017737995,"lng":-8.5243157115298},{"lat":40.8826927320073,"lng":-8.52428599598901},{"lat":40.8827958045874,"lng":-8.52426868968898},{"lat":40.8828892353037,"lng":-8.52427992148751},{"lat":40.8829888388659,"lng":-8.52430281582084},{"lat":40.8831841542506,"lng":-8.52430183616286},{"lat":40.8832901941339,"lng":-8.52431207331674},{"lat":40.8833858974112,"lng":-8.52429045340972},{"lat":40.8834647043893,"lng":-8.52423290131721},{"lat":40.8835564457873,"lng":-8.52418491820152},{"lat":40.8836411873021,"lng":-8.5241298935506},{"lat":40.8850434741119,"lng":-8.52513626093958},{"lat":40.8905108225651,"lng":-8.52326564461008},{"lat":40.8906373672035,"lng":-8.52278448190126},{"lat":40.890666019307,"lng":-8.52214756816615},{"lat":40.8907040311175,"lng":-8.52163831338079},{"lat":40.8907688589972,"lng":-8.52105837860207},{"lat":40.8909466398039,"lng":-8.52030670900772},{"lat":40.8910618232196,"lng":-8.51981004247981},{"lat":40.8912979686081,"lng":-8.51892203864677},{"lat":40.8913815941644,"lng":-8.51852789080591},{"lat":40.8914300507398,"lng":-8.5177880206622},{"lat":40.8914867101279,"lng":-8.51737795170409},{"lat":40.8916192871906,"lng":-8.51640237408322},{"lat":40.8922359257348,"lng":-8.51419231713343},{"lat":40.8928150876798,"lng":-8.51231434468365},{"lat":40.8928962108482,"lng":-8.51204474028974},{"lat":40.8911022287787,"lng":-8.51029636360749},{"lat":40.890707984079,"lng":-8.50963003894274},{"lat":40.8906254434026,"lng":-8.50940844008076},{"lat":40.8848619773503,"lng":-8.50402300100954}]},{"zone_id":"oaz4","points":[{"lat":40.8324792764823,"lng":-8.40897774031952},{"lat":40.830065839594,"lng":-8.40885527424734},{"lat":40.8293013676501,"lng":-8.40902825015726},{"lat":40.8284481461749,"lng":-8.40952188315564},{"lat":40.8274486484367,"lng":-8.40962384738912},{"lat":40.8271283401828,"lng":-8.40965968587128},{"lat":40.8263205760422,"lng":-8.40963757396389},{"lat":40.8260290040607,"lng":-8.40978268247786},{"lat":40.8264601942349,"lng":-8.41237588704221},{"lat":40.8264210185399,"lng":-8.4133881289921},{"lat":40.8259699763744,"lng":-8.41519461061005},{"lat":40.8259805263765,"lng":-8.41828537981912},{"lat":40.8265148871043,"lng":-8.42200409868934},{"lat":40.8235121965912,"lng":-8.42287820603018},{"lat":40.8221085208281,"lng":-8.42427596442854},{"lat":40.8228355500284,"lng":-8.42625823952767},{"lat":40.822102924622,"lng":-8.4281819515207},{"lat":40.8207710188532,"lng":-8.42813715432177},{"lat":40.8191776286635,"lng":-8.42652131305937},{"lat":40.8167539135934,"lng":-8.42590255752214},{"lat":40.8157616565315,"lng":-8.42539843408105},{"lat":40.8149776998492,"lng":-8.4258399837196},{"lat":40.8147099867822,"lng":-8.42627668691898},{"lat":40.8147675724891,"lng":-8.42910235329339},{"lat":40.8148898775636,"lng":-8.43026874955951},{"lat":40.818169764974,"lng":-8.43361486615758},{"lat":40.8177746549177,"lng":-8.43496143485419},{"lat":40.818608602459,"lng":-8.43645180265436},{"lat":40.8185711208897,"lng":-8.43733187074599},{"lat":40.8175917173072,"lng":-8.43864149979833},{"lat":40.8185582521766,"lng":-8.44256311880713},{"lat":40.8199914732568,"lng":-8.44496548235735},{"lat":40.8207470707269,"lng":-8.44747802816195},{"lat":40.821122787258,"lng":-8.44948701885163},{"lat":40.8211238268032,"lng":-8.45141896991769},{"lat":40.8208682328608,"lng":-8.45254268477866},{"lat":40.8204859297337,"lng":-8.45422336877818},{"lat":40.8208992154169,"lng":-8.45901080975752},{"lat":40.8217586756107,"lng":-8.46262854822379},{"lat":40.8236223970414,"lng":-8.46075593634388},{"lat":40.8256267332471,"lng":-8.45969939486745},{"lat":40.8277736560988,"lng":-8.45876895100143},{"lat":40.8302532394474,"lng":-8.45815374107196},{"lat":40.8344043632707,"lng":-8.45866030422937},{"lat":40.8385465636364,"lng":-8.46111198295236},{"lat":40.8429071817939,"lng":-8.46433667941354},{"lat":40.8458821437066,"lng":-8.46421295294657},{"lat":40.8498200521348,"lng":-8.46085680487938},{"lat":40.8531067791949,"lng":-8.457463186327},{"lat":40.8550646327603,"lng":-8.45547775350103},{"lat":40.8608305233847,"lng":-8.4466680798672},{"lat":40.8670775122113,"lng":-8.43907690840037},{"lat":40.871083857484,"lng":-8.43521848756763},{"lat":40.8742469975694,"lng":-8.43392691339236},{"lat":40.8764152238787,"lng":-8.43401835290799},{"lat":40.8769230062407,"lng":-8.43285173874084},{"lat":40.8774458210871,"lng":-8.43150826829543},{"lat":40.8778201446954,"lng":-8.43101031525207},{"lat":40.8780252429333,"lng":-8.43022172788523},{"lat":40.878085147159,"lng":-8.42892715441165},{"lat":40.8786702790764,"lng":-8.42781948784879},{"lat":40.879254261208,"lng":-8.4268954950013},{"lat":40.8794032364792,"lng":-8.42563305435582},{"lat":40.8797023761356,"lng":-8.42429710218324},{"lat":40.880172427977,"lng":-8.42307793729907},{"lat":40.8797516983288,"lng":-8.42240870200296},{"lat":40.87871938177,"lng":-8.42100501033679},{"lat":40.8783252521045,"lng":-8.4196003803618},{"lat":40.8775944608771,"lng":-8.41872565687295},{"lat":40.877093172951,"lng":-8.41853851560136},{"lat":40.8759271331714,"lng":-8.41763731227979},{"lat":40.8744993141004,"lng":-8.41637973751722},{"lat":40.8738638200218,"lng":-8.41570636518386},{"lat":40.8734315357052,"lng":-8.41520421728018},{"lat":40.873287115565,"lng":-8.41506496274345},{"lat":40.8731650477913,"lng":-8.414904382977},{"lat":40.8725209239167,"lng":-8.41459942679643},{"lat":40.8722041380753,"lng":-8.41477402571951},{"lat":40.8720788115034,"lng":-8.41479416256877},{"lat":40.8718663703834,"lng":-8.41476026512292},{"lat":40.871637196541,"lng":-8.41468482132694},{"lat":40.8714671660696,"lng":-8.41463924439867},{"lat":40.8712474637334,"lng":-8.41459619098544},{"lat":40.8710186603779,"lng":-8.41457627751901},{"lat":40.8707928144689,"lng":-8.41455807874565},{"lat":40.8705610356417,"lng":-8.41455971218487},{"lat":40.8704641065062,"lng":-8.41451469639245},{"lat":40.8702953064587,"lng":-8.41446394745674},{"lat":40.8700582883571,"lng":-8.4143735840419},{"lat":40.8698318368846,"lng":-8.41430286034394},{"lat":40.8696459340996,"lng":-8.41430692865514},{"lat":40.869421575994,"lng":-8.41437615768695},{"lat":40.8693035646649,"lng":-8.41440748190062},{"lat":40.869103787511,"lng":-8.41444294623821},{"lat":40.8689872932535,"lng":-8.41567858111096},{"lat":40.8688567188073,"lng":-8.41706339023139},{"lat":40.8687931754765,"lng":-8.4177392549137},{"lat":40.8685796173417,"lng":-8.41863070712634},{"lat":40.8683671681962,"lng":-8.41907603638737},{"lat":40.8678017362125,"lng":-8.41980664472671},{"lat":40.8676053245343,"lng":-8.42020334755989},{"lat":40.8675182716025,"lng":-8.42037917124048},{"lat":40.8675171869454,"lng":-8.42081153171265},{"lat":40.8675152290129,"lng":-8.42159036283549},{"lat":40.8672759000439,"lng":-8.42343784511922},{"lat":40.8672550741789,"lng":-8.42449399334714},{"lat":40.8671192924282,"lng":-8.42443792991997},{"lat":40.8670057131068,"lng":-8.42437895110941},{"lat":40.8668570132235,"lng":-8.4243206466007},{"lat":40.866753821831,"lng":-8.42428413244102},{"lat":40.8666335944259,"lng":-8.42425430378745},{"lat":40.8665331953201,"lng":-8.42422582831948},{"lat":40.866424426605,"lng":-8.42419922118973},{"lat":40.8662899772266,"lng":-8.4241721625969},{"lat":40.8661920679436,"lng":-8.42415758357788},{"lat":40.8660071216343,"lng":-8.42411821884104},{"lat":40.8659098616023,"lng":-8.4240981587093},{"lat":40.8657441762473,"lng":-8.42408283868573},{"lat":40.8656435163962,"lng":-8.42408043873636},{"lat":40.8655432343936,"lng":-8.42408005304113},{"lat":40.8654191569212,"lng":-8.42408465706242},{"lat":40.8652734501297,"lng":-8.42410478499692},{"lat":40.8651031595942,"lng":-8.42414483709721},{"lat":40.8649722858508,"lng":-8.42415849432797},{"lat":40.8648667542634,"lng":-8.42419254548499},{"lat":40.8647750047413,"lng":-8.42418905742618},{"lat":40.8646606228602,"lng":-8.4241622207905},{"lat":40.8645453724862,"lng":-8.42415839148791},{"lat":40.8644263427734,"lng":-8.42418882492809},{"lat":40.864308974013,"lng":-8.4242033713655},{"lat":40.8642051634788,"lng":-8.42423339681202},{"lat":40.8641050358938,"lng":-8.42426699667982},{"lat":40.8640043422606,"lng":-8.42431043881798},{"lat":40.8638865971014,"lng":-8.42443766393606},{"lat":40.8637814410463,"lng":-8.4245008943638},{"lat":40.8636817126705,"lng":-8.42455465942079},{"lat":40.861821074907,"lng":-8.42345110869957},{"lat":40.861736020564,"lng":-8.42339961459995},{"lat":40.8616424256401,"lng":-8.42337832850243},{"lat":40.8615256235605,"lng":-8.42334697687995},{"lat":40.86138377524,"lng":-8.42328527101429},{"lat":40.8612781092189,"lng":-8.42323036651163},{"lat":40.8611543344202,"lng":-8.42314324053219},{"lat":40.8610530782563,"lng":-8.42308894871631},{"lat":40.8609466302612,"lng":-8.42295137217534},{"lat":40.8608611044456,"lng":-8.42283689818879},{"lat":40.8608370267366,"lng":-8.42267287958007},{"lat":40.8606868906639,"lng":-8.42275410113099},{"lat":40.8605794894974,"lng":-8.42292271878453},{"lat":40.860462573083,"lng":-8.42290097627528},{"lat":40.8603637478482,"lng":-8.42288263247185},{"lat":40.8602576511109,"lng":-8.4228553627746},{"lat":40.860134133429,"lng":-8.42280880194121},{"lat":40.8600421218877,"lng":-8.42276653091107},{"lat":40.8599641230578,"lng":-8.42270226160609},{"lat":40.8598627510195,"lng":-8.42258712762474},{"lat":40.8597815913249,"lng":-8.42249093898348},{"lat":40.8589411160218,"lng":-8.42199608670872},{"lat":40.8587275862822,"lng":-8.42198484529122},{"lat":40.8585540598892,"lng":-8.42193466485474},{"lat":40.8584438581033,"lng":-8.42188021127177},{"lat":40.8583405260396,"lng":-8.42182757385217},{"lat":40.858252288374,"lng":-8.42178840656568},{"lat":40.8581466209656,"lng":-8.42173338726146},{"lat":40.8580601919275,"lng":-8.42169114441148},{"lat":40.8579536355704,"lng":-8.42163185308441},{"lat":40.8578655030079,"lng":-8.42158663687152},{"lat":40.8577735946607,"lng":-8.42153891376931},{"lat":40.8576123418503,"lng":-8.42145839214253},{"lat":40.8574533170603,"lng":-8.42138748605941},{"lat":40.8573286730953,"lng":-8.42135895237773},{"lat":40.8572364790428,"lng":-8.42135309474696},{"lat":40.8570231729704,"lng":-8.42134670861813},{"lat":40.8568499231778,"lng":-8.42134607153268},{"lat":40.8567417181668,"lng":-8.42133350180001},{"lat":40.8566184082907,"lng":-8.42131161588518},{"lat":40.8564941141074,"lng":-8.42128723520289},{"lat":40.8563924031629,"lng":-8.42127042539646},{"lat":40.8562672055284,"lng":-8.42124722695052},{"lat":40.8561677478222,"lng":-8.42122924104023},{"lat":40.8560259870814,"lng":-8.42120407223731},{"lat":40.8559323818574,"lng":-8.42118634900767},{"lat":40.8558275885419,"lng":-8.42117782801576},{"lat":40.8557043483018,"lng":-8.42120700548956},{"lat":40.8550900099251,"lng":-8.42137240858219},{"lat":40.8542783995743,"lng":-8.42150602955569},{"lat":40.8540870451764,"lng":-8.42154079960013},{"lat":40.8537544981958,"lng":-8.42150648183335},{"lat":40.8518357129201,"lng":-8.42117073800047},{"lat":40.851754166533,"lng":-8.42108464129765},{"lat":40.8516541873399,"lng":-8.42102372482552},{"lat":40.8515375778635,"lng":-8.42102274213058},{"lat":40.8514129414384,"lng":-8.42102717984299},{"lat":40.8513191122081,"lng":-8.4210270084845},{"lat":40.851194536214,"lng":-8.42104294964972},{"lat":40.8510535535191,"lng":-8.42106640827411},{"lat":40.8509568741852,"lng":-8.42108934826925},{"lat":40.8508645511843,"lng":-8.42109902625467},{"lat":40.850766804902,"lng":-8.42111686218983},{"lat":40.8506767221298,"lng":-8.42113093760029},{"lat":40.8505672738025,"lng":-8.42114706252247},{"lat":40.8504714487198,"lng":-8.42115316755257},{"lat":40.8503745306186,"lng":-8.42116389159207},{"lat":40.8502945216174,"lng":-8.42121833047644},{"lat":40.8501649701567,"lng":-8.42124337958465},{"lat":40.8499640210334,"lng":-8.42130013675727},{"lat":40.8498340646186,"lng":-8.42134309052542},{"lat":40.8496914088108,"lng":-8.42138705637306},{"lat":40.8495920548964,"lng":-8.42143559863282},{"lat":40.8494788846201,"lng":-8.42146368392853},{"lat":40.8493555221305,"lng":-8.42146243406923},{"lat":40.8492354401168,"lng":-8.42148171386915},{"lat":40.8491404161486,"lng":-8.42152719119361},{"lat":40.8489914220273,"lng":-8.42154907292209},{"lat":40.848884850688,"lng":-8.42156698802528},{"lat":40.8487681702911,"lng":-8.42159434606985},{"lat":40.8481611528806,"lng":-8.42174051678657},{"lat":40.8480071951027,"lng":-8.42176676261574},{"lat":40.8477657749999,"lng":-8.42180341750492},{"lat":40.847639747394,"lng":-8.42182397485852},{"lat":40.8474823680754,"lng":-8.42185008567541},{"lat":40.8473164303728,"lng":-8.42187805773169},{"lat":40.8471014767832,"lng":-8.42191779131752},{"lat":40.846905535092,"lng":-8.42195286510142},{"lat":40.8467751735671,"lng":-8.42197767204577},{"lat":40.8466060823699,"lng":-8.42200634010578},{"lat":40.8464318628988,"lng":-8.4220332082156},{"lat":40.8463149502219,"lng":-8.42204538506802},{"lat":40.8461166687147,"lng":-8.42207985500749},{"lat":40.8458889359765,"lng":-8.42211644864092},{"lat":40.845658309513,"lng":-8.42215789123301},{"lat":40.8454359688942,"lng":-8.42219877665088},{"lat":40.8452775044161,"lng":-8.42222701545733},{"lat":40.8450921011177,"lng":-8.42226094860077},{"lat":40.8449738177657,"lng":-8.42228106245208},{"lat":40.8448403044637,"lng":-8.42230585435432},{"lat":40.844595256896,"lng":-8.4223525677868},{"lat":40.844381278111,"lng":-8.42239846972844},{"lat":40.8442571793438,"lng":-8.42241417076389},{"lat":40.8441138683981,"lng":-8.42242125124064},{"lat":40.8440006690187,"lng":-8.42242561707835},{"lat":40.8438640530668,"lng":-8.42243082968397},{"lat":40.8436690356871,"lng":-8.42245629963833},{"lat":40.8435738026875,"lng":-8.42247722562472},{"lat":40.8434460861942,"lng":-8.42252445151965},{"lat":40.8427698907548,"lng":-8.42279294553964},{"lat":40.8397596050248,"lng":-8.4175046005324},{"lat":40.8399475021615,"lng":-8.41628375634364},{"lat":40.8400108995498,"lng":-8.4158718172142},{"lat":40.8399789850432,"lng":-8.41574113740029},{"lat":40.8399352130996,"lng":-8.41556214829663},{"lat":40.8399003382708,"lng":-8.41542671343173},{"lat":40.8398789330072,"lng":-8.41530627442059},{"lat":40.8398599968563,"lng":-8.41518910780401},{"lat":40.839844106988,"lng":-8.41500448806864},{"lat":40.839904152246,"lng":-8.41486483377142},{"lat":40.8401196485358,"lng":-8.41463151429427},{"lat":40.8402280155144,"lng":-8.41451418425308},{"lat":40.840044383639,"lng":-8.41450418510446},{"lat":40.8399017319942,"lng":-8.41449301048005},{"lat":40.8398081093105,"lng":-8.41446983921988},{"lat":40.8396560680943,"lng":-8.41440027941389},{"lat":40.8394399628041,"lng":-8.41427594600306},{"lat":40.8392878323375,"lng":-8.4141517208464},{"lat":40.8391445553114,"lng":-8.41402137519791},{"lat":40.8390312146746,"lng":-8.41389976050256},{"lat":40.838851913231,"lng":-8.41364289129672},{"lat":40.8388243970225,"lng":-8.41350072130846},{"lat":40.8385332683799,"lng":-8.41376226806074},{"lat":40.8383232727701,"lng":-8.4139085888659},{"lat":40.8382413248602,"lng":-8.4139709201847},{"lat":40.8379667764596,"lng":-8.41394860399252},{"lat":40.8373650058545,"lng":-8.41331611455118},{"lat":40.8368836300961,"lng":-8.41293858257874},{"lat":40.8361084470487,"lng":-8.412221559898},{"lat":40.8350689454393,"lng":-8.41144001215809},{"lat":40.8334253137489,"lng":-8.41028379769512},{"lat":40.8330531772401,"lng":-8.41008614906699},{"lat":40.8324792764823,"lng":-8.40897774031952}]},{"zone_id":"vcb3","points":[{"lat":40.8274391557034,"lng":-8.32520930101246},{"lat":40.8262098678179,"lng":-8.32884035698592},{"lat":40.8217514792429,"lng":-8.34406971636769},{"lat":40.8198335608492,"lng":-8.35047541690901},{"lat":40.8187430447631,"lng":-8.35411696284596},{"lat":40.8178672960115,"lng":-8.35704097483154},{"lat":40.8175747680195,"lng":-8.36042699266891},{"lat":40.8182846839326,"lng":-8.36287560985136},{"lat":40.8201385437609,"lng":-8.36514009462629},{"lat":40.8254201404937,"lng":-8.36892227991222},{"lat":40.8367869362673,"lng":-8.3765821866671},{"lat":40.837935310392,"lng":-8.37499176228939},{"lat":40.8437371299439,"lng":-8.36908327244067},{"lat":40.8476751782564,"lng":-8.36570870815367},{"lat":40.8505377997013,"lng":-8.36393024875625},{"lat":40.8535881317435,"lng":-8.36349558117573},{"lat":40.8559703405041,"lng":-8.36366072994707},{"lat":40.8578747869112,"lng":-8.36445183575437},{"lat":40.8598977212026,"lng":-8.36555721154767},{"lat":40.8625548481579,"lng":-8.35935876066842},{"lat":40.8649056698675,"lng":-8.3534333286842},{"lat":40.8680643895184,"lng":-8.34609972275247},{"lat":40.8702856677775,"lng":-8.34218746767164},{"lat":40.8672357839342,"lng":-8.3337223270969},{"lat":40.8654311479689,"lng":-8.3287136323494},{"lat":40.8648490546131,"lng":-8.326925236189},{"lat":40.8639402152707,"lng":-8.32725561347249},{"lat":40.8623258396281,"lng":-8.32758426429802},{"lat":40.8593807624543,"lng":-8.32757565052999},{"lat":40.8548208820767,"lng":-8.32756231630562},{"lat":40.849458920429,"lng":-8.32829966841203},{"lat":40.8435248217836,"lng":-8.32922345864243},{"lat":40.8399510223598,"lng":-8.32921292506073},{"lat":40.8362350674455,"lng":-8.32873142560158},{"lat":40.8332341872264,"lng":-8.32806386625664},{"lat":40.8288774655762,"lng":-8.32607501895451},{"lat":40.8274391557034,"lng":-8.32520930101246}]},{"zone_id":"vcb2","points":[{"lat":40.9024570795975,"lng":-8.37606343048603},{"lat":40.9024353713066,"lng":-8.37593838077153},{"lat":40.9022454491805,"lng":-8.37577651020263},{"lat":40.9020496968497,"lng":-8.37560192399657},{"lat":40.9018640206479,"lng":-8.3754333119877},{"lat":40.9016666476813,"lng":-8.37525836466399},{"lat":40.9014646843791,"lng":-8.37508185644501},{"lat":40.9012241463671,"lng":-8.3747935376409},{"lat":40.9010278057265,"lng":-8.37459901328668},{"lat":40.9008437233696,"lng":-8.37439955725751},{"lat":40.9006729683306,"lng":-8.374240735114},{"lat":40.9004077790187,"lng":-8.37402804384594},{"lat":40.9002049574158,"lng":-8.3738739732031},{"lat":40.8999971494155,"lng":-8.3736928223327},{"lat":40.8998298118896,"lng":-8.3734949777164},{"lat":40.8991190192021,"lng":-8.37272694732323},{"lat":40.8990743264741,"lng":-8.37260192560802},{"lat":40.8990274859209,"lng":-8.37242936140294},{"lat":40.8989388845643,"lng":-8.37212724280914},{"lat":40.8988567138005,"lng":-8.37182494016003},{"lat":40.8987855712969,"lng":-8.37152639172135},{"lat":40.8987061645921,"lng":-8.37123023752371},{"lat":40.8986255811345,"lng":-8.37086128618232},{"lat":40.8984784599336,"lng":-8.37037079594584},{"lat":40.8983593379899,"lng":-8.3699799063701},{"lat":40.8982590247741,"lng":-8.36968841618331},{"lat":40.898120223905,"lng":-8.36931416533042},{"lat":40.8980114308792,"lng":-8.36899106631901},{"lat":40.897980434897,"lng":-8.3688229185183},{"lat":40.8979753296471,"lng":-8.36840604827723},{"lat":40.8979959009358,"lng":-8.36809773733892},{"lat":40.8980287709262,"lng":-8.36778991833176},{"lat":40.8980519459292,"lng":-8.36745518466925},{"lat":40.898076385651,"lng":-8.3670595253017},{"lat":40.8980729329762,"lng":-8.36690974610961},{"lat":40.8980639656949,"lng":-8.36674030871847},{"lat":40.8980401567366,"lng":-8.36658637694352},{"lat":40.8979328267559,"lng":-8.36644059762276},{"lat":40.8978566536194,"lng":-8.36636453025537},{"lat":40.8976651252733,"lng":-8.36620757582624},{"lat":40.897565289945,"lng":-8.36614408558833},{"lat":40.8974567430641,"lng":-8.36607415324761},{"lat":40.8972519332101,"lng":-8.36593556422971},{"lat":40.8970490188173,"lng":-8.36578589048138},{"lat":40.8968390270705,"lng":-8.36562374204041},{"lat":40.8966671003303,"lng":-8.36543605173129},{"lat":40.896493149244,"lng":-8.3652158597982},{"lat":40.8963122156338,"lng":-8.36499701815204},{"lat":40.8961300257008,"lng":-8.36479094111374},{"lat":40.8959744006183,"lng":-8.36457686340743},{"lat":40.8958508700276,"lng":-8.36441706167891},{"lat":40.8957585486678,"lng":-8.36418149744799},{"lat":40.8957477021263,"lng":-8.3639087021992},{"lat":40.8957696656425,"lng":-8.36370061658262},{"lat":40.8957467782373,"lng":-8.36300107869763},{"lat":40.8960040081451,"lng":-8.36212026982798},{"lat":40.896035427878,"lng":-8.3611776773102},{"lat":40.8959428155119,"lng":-8.36079298312786},{"lat":40.8960022692129,"lng":-8.36001140646459},{"lat":40.8961715415403,"lng":-8.35954174977797},{"lat":40.8965223530772,"lng":-8.35856839734514},{"lat":40.8968185160239,"lng":-8.35788147701579},{"lat":40.8968743197343,"lng":-8.35777019676155},{"lat":40.8968220055832,"lng":-8.35765928070134},{"lat":40.8963661744502,"lng":-8.35667784422237},{"lat":40.8959626851734,"lng":-8.35589815429525},{"lat":40.8954949398609,"lng":-8.35513712411776},{"lat":40.8951961955848,"lng":-8.35462663347409},{"lat":40.8950457810006,"lng":-8.35438834752294},{"lat":40.8948922220731,"lng":-8.35417732926315},{"lat":40.8946908153471,"lng":-8.35403378841061},{"lat":40.8944022071708,"lng":-8.35383622002834},{"lat":40.8941804633955,"lng":-8.35370037842587},{"lat":40.8940912100865,"lng":-8.35362697125364},{"lat":40.8940033508699,"lng":-8.35347180281281},{"lat":40.8939639308883,"lng":-8.35332768988013},{"lat":40.8939020991987,"lng":-8.35300441775258},{"lat":40.8938784954224,"lng":-8.35288138887671},{"lat":40.8938567793145,"lng":-8.35275505371048},{"lat":40.8938233914471,"lng":-8.35262680208013},{"lat":40.8937441229851,"lng":-8.35233098039048},{"lat":40.8937003420969,"lng":-8.35219931552373},{"lat":40.8936414766852,"lng":-8.3520864757781},{"lat":40.8935587584565,"lng":-8.35195600844505},{"lat":40.8933911934974,"lng":-8.35174931790716},{"lat":40.8932088371348,"lng":-8.35158398623127},{"lat":40.8929987767157,"lng":-8.35142041940624},{"lat":40.8928032363781,"lng":-8.35126581351602},{"lat":40.8926075798299,"lng":-8.35109194495434},{"lat":40.8922118108807,"lng":-8.35076072418241},{"lat":40.8918052390517,"lng":-8.35043793278825},{"lat":40.8916057833755,"lng":-8.35017930639757},{"lat":40.8914568648762,"lng":-8.34989780595916},{"lat":40.8913164420292,"lng":-8.34965202984815},{"lat":40.8911664708918,"lng":-8.34943567883216},{"lat":40.8910100361497,"lng":-8.3492241327892},{"lat":40.8906786763431,"lng":-8.34872823826284},{"lat":40.8905153313566,"lng":-8.34851447797515},{"lat":40.8904315207827,"lng":-8.34839922603642},{"lat":40.8902373649644,"lng":-8.34812104957506},{"lat":40.8901256398304,"lng":-8.34786453138058},{"lat":40.8899492854527,"lng":-8.34762881929109},{"lat":40.8897189599774,"lng":-8.34733282319376},{"lat":40.8895553227127,"lng":-8.34707629677987},{"lat":40.889400098938,"lng":-8.34682324730189},{"lat":40.8892335810806,"lng":-8.3465944115361},{"lat":40.8891287186932,"lng":-8.34640101280533},{"lat":40.8891697357181,"lng":-8.3462892268478},{"lat":40.8892313645435,"lng":-8.34599559288008},{"lat":40.8891935160377,"lng":-8.34571397330459},{"lat":40.889165129644,"lng":-8.34549493585698},{"lat":40.8891414743067,"lng":-8.34523834913264},{"lat":40.8890941636067,"lng":-8.34491918028041},{"lat":40.8889475002217,"lng":-8.34450613823634},{"lat":40.8887393322784,"lng":-8.34399296478766},{"lat":40.8885666470103,"lng":-8.34388031695747},{"lat":40.8883942238958,"lng":-8.34384434500333},{"lat":40.8883937802745,"lng":-8.34371651146164},{"lat":40.8883924410275,"lng":-8.3430745814023},{"lat":40.8883915945508,"lng":-8.34275183835144},{"lat":40.8883970400152,"lng":-8.34267935297585},{"lat":40.8884031296076,"lng":-8.34245096234041},{"lat":40.888400126245,"lng":-8.3423712158346},{"lat":40.8883750219745,"lng":-8.3422613832618},{"lat":40.8883518269035,"lng":-8.34219212890308},{"lat":40.8882011631885,"lng":-8.34195398441964},{"lat":40.8881029232419,"lng":-8.34180488120308},{"lat":40.8879645874151,"lng":-8.34157259367312},{"lat":40.8878308696661,"lng":-8.3413233550673},{"lat":40.8877497925769,"lng":-8.3410466344326},{"lat":40.8877430426555,"lng":-8.34084620596114},{"lat":40.8877381882519,"lng":-8.34044430704133},{"lat":40.8876646145765,"lng":-8.34019917229578},{"lat":40.8875733345288,"lng":-8.33993428981485},{"lat":40.887508865269,"lng":-8.33963543989951},{"lat":40.8875010563188,"lng":-8.33932038499918},{"lat":40.8874981974735,"lng":-8.33900879124779},{"lat":40.8874923924589,"lng":-8.33803071448376},{"lat":40.887417139854,"lng":-8.33776516955693},{"lat":40.8873756855007,"lng":-8.33768234203774},{"lat":40.8871926018483,"lng":-8.33744244683957},{"lat":40.8870815858333,"lng":-8.33723410496234},{"lat":40.8870602772952,"lng":-8.33701394001712},{"lat":40.8853739031025,"lng":-8.33802189912658},{"lat":40.8841678148168,"lng":-8.34040788918821},{"lat":40.8832477270087,"lng":-8.34170062632217},{"lat":40.8824471510241,"lng":-8.3433535920037},{"lat":40.8817564153201,"lng":-8.34467577578936},{"lat":40.881045218864,"lng":-8.34523492269587},{"lat":40.8790565380961,"lng":-8.34612791662026},{"lat":40.8771621399664,"lng":-8.34261604751042},{"lat":40.8752822022203,"lng":-8.34304700339876},{"lat":40.8726370932696,"lng":-8.34332113183672},{"lat":40.8713512191401,"lng":-8.34294042703706},{"lat":40.8702856677775,"lng":-8.34218746767164},{"lat":40.8680643895184,"lng":-8.34609972275247},{"lat":40.8649056698675,"lng":-8.3534333286842},{"lat":40.8625548481579,"lng":-8.35935876066842},{"lat":40.8598977212026,"lng":-8.36555721154767},{"lat":40.8618008822563,"lng":-8.36697605389229},{"lat":40.8640601206769,"lng":-8.36902389867755},{"lat":40.8654844686604,"lng":-8.37153965012923},{"lat":40.866788651044,"lng":-8.37452583881371},{"lat":40.8678541714609,"lng":-8.37766819135556},{"lat":40.8696333478033,"lng":-8.38128408907332},{"lat":40.8723636986711,"lng":-8.38568845776781},{"lat":40.874976914292,"lng":-8.38862028399364},{"lat":40.8777093690179,"lng":-8.39245751349118},{"lat":40.8798689839238,"lng":-8.39473145626889},{"lat":40.8801806036674,"lng":-8.39499008799261},{"lat":40.8802661773216,"lng":-8.3951379586178},{"lat":40.8803952079244,"lng":-8.39521929822119},{"lat":40.8805018904607,"lng":-8.39535009253932},{"lat":40.8805997378496,"lng":-8.39549926211784},{"lat":40.8807298984466,"lng":-8.39557657701088},{"lat":40.8808206567907,"lng":-8.39564433407865},{"lat":40.8808834206854,"lng":-8.39576796586323},{"lat":40.8809532328922,"lng":-8.39588826816237},{"lat":40.8810299621522,"lng":-8.39599013959336},{"lat":40.8810865348501,"lng":-8.39610982998963},{"lat":40.8811243151077,"lng":-8.39623670476337},{"lat":40.8811714878914,"lng":-8.39636763415919},{"lat":40.8812588543015,"lng":-8.39645798208849},{"lat":40.881346201556,"lng":-8.39655392541426},{"lat":40.8814237217602,"lng":-8.39666485387611},{"lat":40.8814924009451,"lng":-8.39677262717774},{"lat":40.8815720831812,"lng":-8.39687047828438},{"lat":40.8816758413672,"lng":-8.39697034964892},{"lat":40.8817594714457,"lng":-8.39705311470453},{"lat":40.8818433036153,"lng":-8.3971352922952},{"lat":40.881924535729,"lng":-8.39722193028964},{"lat":40.8819972559581,"lng":-8.39732056879167},{"lat":40.8820862150747,"lng":-8.39745941444842},{"lat":40.8821704202136,"lng":-8.39756652534024},{"lat":40.8821628251257,"lng":-8.39772611666911},{"lat":40.8821504879411,"lng":-8.39788333039882},{"lat":40.8821694945833,"lng":-8.39803853581519},{"lat":40.8822263801356,"lng":-8.39817515125295},{"lat":40.8822740315955,"lng":-8.39830724171397},{"lat":40.8823497133307,"lng":-8.39840985650658},{"lat":40.8824157058625,"lng":-8.39854101098224},{"lat":40.8824791788861,"lng":-8.3986709110549},{"lat":40.8825357552016,"lng":-8.39880267026345},{"lat":40.8826070893483,"lng":-8.3989027770689},{"lat":40.882684994836,"lng":-8.39898737962307},{"lat":40.8827623782176,"lng":-8.3990765659605},{"lat":40.8828373558834,"lng":-8.39917024668594},{"lat":40.8829061832258,"lng":-8.39926976530464},{"lat":40.882941384415,"lng":-8.39941661864222},{"lat":40.8829461677932,"lng":-8.39957295339023},{"lat":40.8829968553556,"lng":-8.3997134455958},{"lat":40.8830592584607,"lng":-8.39984633945971},{"lat":40.8831258267214,"lng":-8.39999688122138},{"lat":40.8845134311812,"lng":-8.4017762850824},{"lat":40.8857208918771,"lng":-8.40220479789343},{"lat":40.8858836884274,"lng":-8.40217825351063},{"lat":40.8867165530382,"lng":-8.40204245083416},{"lat":40.886776559373,"lng":-8.40191607048505},{"lat":40.8875748726958,"lng":-8.40085061190796},{"lat":40.8876761712072,"lng":-8.40207980604819},{"lat":40.887779588243,"lng":-8.402127450724},{"lat":40.8895735922812,"lng":-8.40155733335867},{"lat":40.889704870323,"lng":-8.40148465633396},{"lat":40.8897973077298,"lng":-8.40142522938188},{"lat":40.889901527475,"lng":-8.40137178279617},{"lat":40.8900202717444,"lng":-8.40130641012119},{"lat":40.8902427425787,"lng":-8.40116765446199},{"lat":40.8903276244081,"lng":-8.40110451765552},{"lat":40.890429374502,"lng":-8.40102875212745},{"lat":40.8907304286057,"lng":-8.40082457602129},{"lat":40.8908132289713,"lng":-8.40076582036851},{"lat":40.8909231144576,"lng":-8.40067655922118},{"lat":40.8910363029927,"lng":-8.40059965300628},{"lat":40.8911497949882,"lng":-8.40054719076233},{"lat":40.8912423059886,"lng":-8.40049464387556},{"lat":40.8913494868205,"lng":-8.40040776607118},{"lat":40.8914499595641,"lng":-8.4003379054304},{"lat":40.8915533917256,"lng":-8.40027436639481},{"lat":40.8916681934152,"lng":-8.40020066794665},{"lat":40.891756913407,"lng":-8.40015154613342},{"lat":40.8918537662575,"lng":-8.40009047197997},{"lat":40.8920699817029,"lng":-8.39993032401345},{"lat":40.8921749892443,"lng":-8.3998478040051},{"lat":40.8923158677484,"lng":-8.39975107187797},{"lat":40.8924741214116,"lng":-8.39968188919133},{"lat":40.8925940666583,"lng":-8.39961609096899},{"lat":40.8927173034759,"lng":-8.39955452779908},{"lat":40.8930061049494,"lng":-8.39942943811162},{"lat":40.8935860481818,"lng":-8.39832523634739},{"lat":40.8936957914786,"lng":-8.39821864288408},{"lat":40.8938234540461,"lng":-8.3980733168125},{"lat":40.8939003949108,"lng":-8.39797750782665},{"lat":40.8940567449566,"lng":-8.39777082840109},{"lat":40.8941406358807,"lng":-8.39766816375952},{"lat":40.8942766148624,"lng":-8.39749328640464},{"lat":40.8946131061917,"lng":-8.39708539209863},{"lat":40.8947863987748,"lng":-8.39687129959204},{"lat":40.8948637271248,"lng":-8.3967635039205},{"lat":40.8949388874681,"lng":-8.39665866490605},{"lat":40.8978339118885,"lng":-8.39448941064581},{"lat":40.8979992220633,"lng":-8.39429947668188},{"lat":40.8979584943396,"lng":-8.39415311094612},{"lat":40.8979039081557,"lng":-8.39404253025465},{"lat":40.8978115145054,"lng":-8.3939657393649},{"lat":40.897715073665,"lng":-8.39392655210158},{"lat":40.8976136139412,"lng":-8.39391665740198},{"lat":40.8975202642583,"lng":-8.39390382658491},{"lat":40.8974036606369,"lng":-8.39386111891281},{"lat":40.8972272992261,"lng":-8.3938041711065},{"lat":40.8971416717414,"lng":-8.39376158538813},{"lat":40.8970476441214,"lng":-8.39373035929636},{"lat":40.8969637419783,"lng":-8.39368137229633},{"lat":40.8968915905313,"lng":-8.39358956147108},{"lat":40.8968285948681,"lng":-8.3934344186428},{"lat":40.8968033976079,"lng":-8.39329917584392},{"lat":40.896844574645,"lng":-8.39316464640451},{"lat":40.8970310432692,"lng":-8.39291241730845},{"lat":40.8971051182038,"lng":-8.39280816082273},{"lat":40.8971545166841,"lng":-8.3927053491854},{"lat":40.8971813592877,"lng":-8.3925823936947},{"lat":40.8972158509342,"lng":-8.39242196762653},{"lat":40.8972840851697,"lng":-8.39231199124311},{"lat":40.8973559723075,"lng":-8.39221935384853},{"lat":40.8974320766537,"lng":-8.39213337954385},{"lat":40.8975091705525,"lng":-8.39204740890118},{"lat":40.8975789378956,"lng":-8.39193624971071},{"lat":40.8976490051865,"lng":-8.39181215730977},{"lat":40.8976985346008,"lng":-8.39169071296063},{"lat":40.8977278537698,"lng":-8.39154747262284},{"lat":40.897747415071,"lng":-8.39133833190092},{"lat":40.8977636494035,"lng":-8.39112727817341},{"lat":40.8978303920455,"lng":-8.39099961146838},{"lat":40.8979581294408,"lng":-8.39085699227689},{"lat":40.8979845662301,"lng":-8.39071326513478},{"lat":40.8980056032592,"lng":-8.39044823389911},{"lat":40.8980590591658,"lng":-8.39034211197241},{"lat":40.898157680644,"lng":-8.39025278145692},{"lat":40.8982391612201,"lng":-8.39017809750308},{"lat":40.8984299920081,"lng":-8.39000771643655},{"lat":40.8985528036978,"lng":-8.38985285154469},{"lat":40.8986245372819,"lng":-8.38974739346734},{"lat":40.8986943297835,"lng":-8.38966399996074},{"lat":40.8988297229469,"lng":-8.38951986432738},{"lat":40.8990029900011,"lng":-8.38935142837419},{"lat":40.899150031964,"lng":-8.38923296907733},{"lat":40.8992096965683,"lng":-8.38912841225451},{"lat":40.8992469629179,"lng":-8.38897297455929},{"lat":40.8992936922384,"lng":-8.38869320335198},{"lat":40.899326466898,"lng":-8.38849241378947},{"lat":40.8993588631616,"lng":-8.38829969371775},{"lat":40.8994088968049,"lng":-8.38799085864706},{"lat":40.8994332388766,"lng":-8.38785661326311},{"lat":40.89945727603,"lng":-8.38769685126865},{"lat":40.8995012810052,"lng":-8.38754772784286},{"lat":40.8995585201444,"lng":-8.38743983554163},{"lat":40.8996201699156,"lng":-8.38733267318612},{"lat":40.8997189039608,"lng":-8.38719159359471},{"lat":40.8997791771858,"lng":-8.38709605460398},{"lat":40.8998560210092,"lng":-8.38695916181121},{"lat":40.8999572370661,"lng":-8.38686022897186},{"lat":40.9001029289937,"lng":-8.38692193767525},{"lat":40.9002187505953,"lng":-8.38695288406253},{"lat":40.9003168931956,"lng":-8.38697440537373},{"lat":40.9004521962991,"lng":-8.38699726684677},{"lat":40.9005672296887,"lng":-8.38695039050173},{"lat":40.9022577263428,"lng":-8.38566329230541},{"lat":40.903629531348,"lng":-8.38538661732374},{"lat":40.9021218357163,"lng":-8.38238591359606},{"lat":40.9016125783681,"lng":-8.38103757486611},{"lat":40.9024570795975,"lng":-8.37606343048603}]},{"zone_id":"vcb8","points":[{"lat":40.8311785754702,"lng":-8.28524810109897},{"lat":40.8449121207561,"lng":-8.2797637965397},{"lat":40.847526813802,"lng":-8.27322866099985},{"lat":40.8498891280688,"lng":-8.26801139405421},{"lat":40.8511361921303,"lng":-8.26494737411799},{"lat":40.8522483146702,"lng":-8.26255417722975},{"lat":40.852538510113,"lng":-8.26192965968732},{"lat":40.8536692102145,"lng":-8.25949623590181},{"lat":40.8558421984819,"lng":-8.25508778987782},{"lat":40.8551792814941,"lng":-8.25456888105196},{"lat":40.854569929191,"lng":-8.25415053318476},{"lat":40.8538567185166,"lng":-8.25375726530736},{"lat":40.853355766935,"lng":-8.25341748221217},{"lat":40.8529324164197,"lng":-8.25313033855725},{"lat":40.8523157923965,"lng":-8.25242525665998},{"lat":40.8517527302895,"lng":-8.25164663006167},{"lat":40.850933300806,"lng":-8.25086341450655},{"lat":40.8495327586901,"lng":-8.24950482226051},{"lat":40.8485143309444,"lng":-8.24895673373504},{"lat":40.8467022501654,"lng":-8.24857682548323},{"lat":40.8446896478958,"lng":-8.24815490213485},{"lat":40.8437692449984,"lng":-8.24763918284003},{"lat":40.8431864306773,"lng":-8.24731262896907},{"lat":40.8430572496942,"lng":-8.24717889887802},{"lat":40.8429423102126,"lng":-8.24707041725599},{"lat":40.8414717774815,"lng":-8.24568254803179},{"lat":40.8401984378215,"lng":-8.24448084966813},{"lat":40.8398463716622,"lng":-8.24414860190026},{"lat":40.8389811229989,"lng":-8.24383734567296},{"lat":40.8382786456429,"lng":-8.24358464945944},{"lat":40.8378526285672,"lng":-8.24324081732393},{"lat":40.837712838751,"lng":-8.24312799615976},{"lat":40.8369549006914,"lng":-8.24251629154901},{"lat":40.8336681052664,"lng":-8.24100818336388},{"lat":40.8326894036266,"lng":-8.24055914817224},{"lat":40.8317603425122,"lng":-8.23983526558556},{"lat":40.8298700065865,"lng":-8.23854021343451},{"lat":40.8287417953124,"lng":-8.23769741036643},{"lat":40.8274915885086,"lng":-8.23676351247143},{"lat":40.8265524004045,"lng":-8.23682898471403},{"lat":40.8262929867759,"lng":-8.23684706851275},{"lat":40.8255418421341,"lng":-8.23689943023338},{"lat":40.8246856715073,"lng":-8.23705353007322},{"lat":40.8234167455541,"lng":-8.23731208019528},{"lat":40.8232102494978,"lng":-8.23735415387384},{"lat":40.8203118553383,"lng":-8.23858979451803},{"lat":40.818173755333,"lng":-8.23950123451276},{"lat":40.815693177643,"lng":-8.24085146936341},{"lat":40.8155385158196,"lng":-8.24137817721319},{"lat":40.8153164181734,"lng":-8.24141680529884},{"lat":40.8150098466491,"lng":-8.24156136894719},{"lat":40.8148459628032,"lng":-8.24184651602251},{"lat":40.8147854473676,"lng":-8.24193932629029},{"lat":40.8146917103925,"lng":-8.24202640275264},{"lat":40.814546943758,"lng":-8.24209016578982},{"lat":40.8143451061672,"lng":-8.24222103723205},{"lat":40.8141476713141,"lng":-8.24236861987726},{"lat":40.8138797220869,"lng":-8.24243808947037},{"lat":40.8137394345769,"lng":-8.24252397783302},{"lat":40.8136281249597,"lng":-8.24261274054226},{"lat":40.8135371564032,"lng":-8.24264781809352},{"lat":40.8133665310691,"lng":-8.24271032153306},{"lat":40.8131435468659,"lng":-8.24282846376973},{"lat":40.813055982032,"lng":-8.24287015425021},{"lat":40.8127777288849,"lng":-8.24287301311251},{"lat":40.8125571172089,"lng":-8.24286825122109},{"lat":40.8123862748478,"lng":-8.24288527017631},{"lat":40.8122420721283,"lng":-8.24301539806832},{"lat":40.812080001432,"lng":-8.24327978533233},{"lat":40.8119574139581,"lng":-8.24350002403481},{"lat":40.8118402587437,"lng":-8.2437400652499},{"lat":40.8116447425822,"lng":-8.24407501779305},{"lat":40.8114526416387,"lng":-8.24428518194152},{"lat":40.8113380714533,"lng":-8.24431651202209},{"lat":40.8110256645262,"lng":-8.24434715366959},{"lat":40.8107616377105,"lng":-8.24444661390868},{"lat":40.8106693286999,"lng":-8.24445166846988},{"lat":40.8105600971388,"lng":-8.24445131312354},{"lat":40.8103320871198,"lng":-8.24455586447789},{"lat":40.8101177856635,"lng":-8.24454127442165},{"lat":40.8098092704894,"lng":-8.24437044329662},{"lat":40.809594035441,"lng":-8.244377157891},{"lat":40.8094272505353,"lng":-8.24437280644665},{"lat":40.8091611939803,"lng":-8.24436543440764},{"lat":40.8086822822181,"lng":-8.24403218657323},{"lat":40.8082272262112,"lng":-8.2435231210046},{"lat":40.8077742631418,"lng":-8.24335975780916},{"lat":40.8072612868964,"lng":-8.24342123060067},{"lat":40.8069964592399,"lng":-8.24369655478006},{"lat":40.8066837793477,"lng":-8.24428004115975},{"lat":40.8064114870555,"lng":-8.24498072279907},{"lat":40.8060829811024,"lng":-8.24582603515218},{"lat":40.8060105523935,"lng":-8.24594785061134},{"lat":40.8059228848037,"lng":-8.24608683093561},{"lat":40.805807679446,"lng":-8.24621438014652},{"lat":40.805629727858,"lng":-8.24632049778657},{"lat":40.8054329894338,"lng":-8.2463365248351},{"lat":40.8050443685844,"lng":-8.24636593916836},{"lat":40.8048335095098,"lng":-8.24651204759872},{"lat":40.8045421448492,"lng":-8.24683861887429},{"lat":40.8043424770059,"lng":-8.24704897676301},{"lat":40.8041694095247,"lng":-8.24731104532224},{"lat":40.8040373006006,"lng":-8.24767817566256},{"lat":40.8038951286048,"lng":-8.24801886632743},{"lat":40.8037867784614,"lng":-8.24830604834337},{"lat":40.8036685944148,"lng":-8.24861371380646},{"lat":40.8035832144141,"lng":-8.24897891496484},{"lat":40.8035121087372,"lng":-8.24929946367532},{"lat":40.803467869819,"lng":-8.24958829223417},{"lat":40.8034137298661,"lng":-8.24995745412096},{"lat":40.8034368325985,"lng":-8.2502588515262},{"lat":40.8035257244028,"lng":-8.25050739215136},{"lat":40.803597867655,"lng":-8.25075389757145},{"lat":40.8036839833293,"lng":-8.25098631662817},{"lat":40.8037844785943,"lng":-8.25124329598367},{"lat":40.8038770888779,"lng":-8.25146661003179},{"lat":40.8039267637101,"lng":-8.25184176799625},{"lat":40.8037971689958,"lng":-8.25202900695178},{"lat":40.8035773240341,"lng":-8.25206059791406},{"lat":40.8033575207856,"lng":-8.25205782827686},{"lat":40.8028937755036,"lng":-8.25205414558837},{"lat":40.8026743162425,"lng":-8.2520635813048},{"lat":40.8024593076443,"lng":-8.25210769239198},{"lat":40.811776657718,"lng":-8.26316907112427},{"lat":40.8133459266124,"lng":-8.26599445972011},{"lat":40.8147722832183,"lng":-8.26872561081024},{"lat":40.8157699409825,"lng":-8.27126782655693},{"lat":40.8163387980952,"lng":-8.27371513081249},{"lat":40.8165872220722,"lng":-8.27614238817461},{"lat":40.8167625380736,"lng":-8.27785564638476},{"lat":40.8168200940133,"lng":-8.27920781023714},{"lat":40.8261923223182,"lng":-8.28337800643756},{"lat":40.8311785754702,"lng":-8.28524810109897}]},{"zone_id":"vcb9","points":[{"lat":40.8168200940133,"lng":-8.27920781023714},{"lat":40.8167625380736,"lng":-8.27785564638476},{"lat":40.8165872220722,"lng":-8.27614238817461},{"lat":40.8163387980952,"lng":-8.27371513081249},{"lat":40.8157699409825,"lng":-8.27126782655693},{"lat":40.8147722832183,"lng":-8.26872561081024},{"lat":40.8133459266124,"lng":-8.26599445972011},{"lat":40.811776657718,"lng":-8.26316907112427},{"lat":40.8024593076443,"lng":-8.25210769239198},{"lat":40.8023565573526,"lng":-8.25212877250338},{"lat":40.8021952157603,"lng":-8.25219164506772},{"lat":40.8020710036865,"lng":-8.25231169719473},{"lat":40.8018914026271,"lng":-8.25260786007343},{"lat":40.801695476589,"lng":-8.25302024231513},{"lat":40.8015324857979,"lng":-8.25336925980687},{"lat":40.8013463717896,"lng":-8.25368497946018},{"lat":40.8011839545951,"lng":-8.25399692721571},{"lat":40.8010731617792,"lng":-8.25419663102484},{"lat":40.8009533124141,"lng":-8.254446801278},{"lat":40.8006731168344,"lng":-8.25491897460465},{"lat":40.8004980310801,"lng":-8.25528172699103},{"lat":40.8003807318301,"lng":-8.2555042850648},{"lat":40.8002374314078,"lng":-8.25578178053009},{"lat":40.7998663637935,"lng":-8.25626016941929},{"lat":40.7996814891932,"lng":-8.25643081125153},{"lat":40.7995369001623,"lng":-8.25652371080858},{"lat":40.7994424254938,"lng":-8.25658441126049},{"lat":40.7991905185755,"lng":-8.25670704904334},{"lat":40.7989858191702,"lng":-8.25680774021303},{"lat":40.7987641688682,"lng":-8.25692511203842},{"lat":40.7983776392974,"lng":-8.25713079836929},{"lat":40.7981572030366,"lng":-8.2572100100085},{"lat":40.7978002435528,"lng":-8.25729761092086},{"lat":40.7975517610456,"lng":-8.25733435156004},{"lat":40.7973556458893,"lng":-8.25732343288885},{"lat":40.7971386219776,"lng":-8.25733356956927},{"lat":40.796949626408,"lng":-8.2573131823994},{"lat":40.7966610605467,"lng":-8.25719805603015},{"lat":40.7965092580475,"lng":-8.25709811870632},{"lat":40.7963268195855,"lng":-8.25693034373803},{"lat":40.796221696443,"lng":-8.25679874185644},{"lat":40.7959923574867,"lng":-8.25653798401043},{"lat":40.7957860064324,"lng":-8.25633817973156},{"lat":40.7956045714626,"lng":-8.25624162124998},{"lat":40.7953918508724,"lng":-8.25626373315217},{"lat":40.7952664099739,"lng":-8.25643329573488},{"lat":40.7951286279045,"lng":-8.25660840266432},{"lat":40.7949724305221,"lng":-8.25665313226581},{"lat":40.7947897981796,"lng":-8.25666842722574},{"lat":40.7946245598832,"lng":-8.25666598451919},{"lat":40.7944332144356,"lng":-8.25666100160435},{"lat":40.7940505140394,"lng":-8.25665400397764},{"lat":40.7938594529372,"lng":-8.25663658281064},{"lat":40.7936693070932,"lng":-8.25660115333024},{"lat":40.7934440025534,"lng":-8.25652169426298},{"lat":40.7932656756012,"lng":-8.25646863741082},{"lat":40.7930567125283,"lng":-8.25634845252223},{"lat":40.792883231794,"lng":-8.25622466797089},{"lat":40.7927035898063,"lng":-8.25605264539819},{"lat":40.7925639983397,"lng":-8.25581766199941},{"lat":40.792471051972,"lng":-8.25558277359079},{"lat":40.7923048744972,"lng":-8.25528459302481},{"lat":40.7921911230949,"lng":-8.25505120969906},{"lat":40.7920190615693,"lng":-8.25494993651765},{"lat":40.7918690705963,"lng":-8.25491826160669},{"lat":40.7917160065045,"lng":-8.2549042329856},{"lat":40.791572791838,"lng":-8.25493560173811},{"lat":40.7913731856673,"lng":-8.25499400234961},{"lat":40.7911796873999,"lng":-8.25514706997256},{"lat":40.791023522569,"lng":-8.25541490485779},{"lat":40.7908843048801,"lng":-8.25558395514731},{"lat":40.7907688048721,"lng":-8.25572272042717},{"lat":40.7906590352191,"lng":-8.25584454442591},{"lat":40.7904859589438,"lng":-8.25605796365377},{"lat":40.7902000271561,"lng":-8.25633313655215},{"lat":40.7900614041371,"lng":-8.25645005341516},{"lat":40.7899037624562,"lng":-8.25650472736784},{"lat":40.7897467229773,"lng":-8.25649981117648},{"lat":40.7895788514435,"lng":-8.25643338368329},{"lat":40.7894466124969,"lng":-8.25639309395194},{"lat":40.7892885432465,"lng":-8.25634563835877},{"lat":40.7888159489966,"lng":-8.25644450852271},{"lat":40.7886355265109,"lng":-8.25649582901385},{"lat":40.7884238332503,"lng":-8.2565682941904},{"lat":40.7882656543076,"lng":-8.25678314655926},{"lat":40.7881275296696,"lng":-8.25710135367516},{"lat":40.7880654507448,"lng":-8.25730027822132},{"lat":40.7880405865389,"lng":-8.25756104973301},{"lat":40.7880442939537,"lng":-8.2578725980305},{"lat":40.7880586600453,"lng":-8.25807438752622},{"lat":40.7880662934763,"lng":-8.25833943625011},{"lat":40.7880209862376,"lng":-8.25859526023065},{"lat":40.7879140732839,"lng":-8.25877786477094},{"lat":40.7877656081936,"lng":-8.25891939484232},{"lat":40.7875870582614,"lng":-8.25906832792365},{"lat":40.7874102189307,"lng":-8.25921821717963},{"lat":40.7872628391015,"lng":-8.25935086272959},{"lat":40.7870983700693,"lng":-8.25946653443057},{"lat":40.7867207157542,"lng":-8.25954092124357},{"lat":40.7865806442859,"lng":-8.2594185007938},{"lat":40.7865108720548,"lng":-8.25916068699445},{"lat":40.7865362812235,"lng":-8.2588215485352},{"lat":40.7865066501235,"lng":-8.25866546011274},{"lat":40.7864147907574,"lng":-8.25850866240215},{"lat":40.7862680446077,"lng":-8.25840116821047},{"lat":40.7861440037451,"lng":-8.25835484033436},{"lat":40.7859877770099,"lng":-8.25834861721353},{"lat":40.7858243457953,"lng":-8.25834617411478},{"lat":40.7856230225408,"lng":-8.25840348529516},{"lat":40.7855111031423,"lng":-8.25863654487419},{"lat":40.7854083444269,"lng":-8.25889402281167},{"lat":40.7853186062287,"lng":-8.25918043665607},{"lat":40.7853472114054,"lng":-8.25945297458266},{"lat":40.7854316149299,"lng":-8.25975204291692},{"lat":40.7855183151893,"lng":-8.26000822438725},{"lat":40.7856178722485,"lng":-8.26028149711837},{"lat":40.785690503785,"lng":-8.26055886339213},{"lat":40.7857037044191,"lng":-8.2608306666302},{"lat":40.7856747472534,"lng":-8.26103461635314},{"lat":40.7855725214055,"lng":-8.26129315969002},{"lat":40.7854562115206,"lng":-8.26142514952253},{"lat":40.7853360061957,"lng":-8.26149540471445},{"lat":40.7852168913987,"lng":-8.26155831527233},{"lat":40.7850884844228,"lng":-8.26163731992911},{"lat":40.7849611384014,"lng":-8.26173066500579},{"lat":40.7848479362382,"lng":-8.26182012509146},{"lat":40.784586648079,"lng":-8.26202740887054},{"lat":40.7842094978969,"lng":-8.2622910920961},{"lat":40.7840476322406,"lng":-8.2624112581903},{"lat":40.7836986369438,"lng":-8.26269904239764},{"lat":40.7834467582522,"lng":-8.26294875071712},{"lat":40.7832672321092,"lng":-8.26315536066311},{"lat":40.7830798025647,"lng":-8.2634252139981},{"lat":40.7828847471028,"lng":-8.26374587145459},{"lat":40.7827163836428,"lng":-8.26403684989294},{"lat":40.7825711529156,"lng":-8.26428902766845},{"lat":40.7824936095327,"lng":-8.26442367269637},{"lat":40.7822717371473,"lng":-8.26480433202772},{"lat":40.7820997299631,"lng":-8.26512787396403},{"lat":40.7819769124153,"lng":-8.26543101829665},{"lat":40.7818730020728,"lng":-8.26573550343781},{"lat":40.781744244727,"lng":-8.26610769848788},{"lat":40.7816355617806,"lng":-8.26641216604687},{"lat":40.7814363652107,"lng":-8.26688005372904},{"lat":40.7812769558152,"lng":-8.26720491589961},{"lat":40.7810983135541,"lng":-8.2674996440243},{"lat":40.780908909241,"lng":-8.26775454463069},{"lat":40.7807305630896,"lng":-8.26794727463315},{"lat":40.7805432220533,"lng":-8.26813417945826},{"lat":40.7803517767171,"lng":-8.2682084228555},{"lat":40.7802091826795,"lng":-8.26817554697049},{"lat":40.7800642660457,"lng":-8.26811898332369},{"lat":40.7799296180617,"lng":-8.26799206767458},{"lat":40.7797019208476,"lng":-8.26780941020938},{"lat":40.7795466881439,"lng":-8.26772486626572},{"lat":40.779371002205,"lng":-8.26757015185774},{"lat":40.7792468273827,"lng":-8.26741530509144},{"lat":40.7791392203684,"lng":-8.26726143992703},{"lat":40.7790429618323,"lng":-8.26703024976617},{"lat":40.7790104587131,"lng":-8.26686745325744},{"lat":40.7789847862778,"lng":-8.26649669975827},{"lat":40.7790110762824,"lng":-8.26633358039317},{"lat":40.7790471012441,"lng":-8.2661100568311},{"lat":40.7790876683588,"lng":-8.2658420625812},{"lat":40.7790531815152,"lng":-8.26552973155353},{"lat":40.7790121747086,"lng":-8.26537869423635},{"lat":40.7788869157866,"lng":-8.26525207993642},{"lat":40.7787625324535,"lng":-8.26519567698151},{"lat":40.7786281724104,"lng":-8.26512516119886},{"lat":40.7784416573529,"lng":-8.26506875894936},{"lat":40.7782036020838,"lng":-8.26504008252769},{"lat":40.7778088404949,"lng":-8.26502780623374},{"lat":40.7775968717171,"lng":-8.26502512908936},{"lat":40.7774032314448,"lng":-8.26505494352287},{"lat":40.7772157488478,"lng":-8.26513192112304},{"lat":40.7770149788624,"lng":-8.26525257822366},{"lat":40.7768375492593,"lng":-8.26536179642112},{"lat":40.7766463716905,"lng":-8.26544113432422},{"lat":40.7764730416283,"lng":-8.26550818294313},{"lat":40.776257204395,"lng":-8.26557680635971},{"lat":40.7760351032192,"lng":-8.26561094967649},{"lat":40.7757688782539,"lng":-8.26564452888762},{"lat":40.7755763099189,"lng":-8.26568169063969},{"lat":40.7753708519773,"lng":-8.26573126314456},{"lat":40.7751845138083,"lng":-8.26575540443202},{"lat":40.7749970892743,"lng":-8.26578298570934},{"lat":40.774818648972,"lng":-8.26583095170173},{"lat":40.7745874900419,"lng":-8.26590771518184},{"lat":40.7743777670445,"lng":-8.26598558691856},{"lat":40.7742372513317,"lng":-8.26609747606795},{"lat":40.7740291870488,"lng":-8.26637032793675},{"lat":40.7739141177905,"lng":-8.26658922796645},{"lat":40.7738157465628,"lng":-8.26677582324019},{"lat":40.7736956294308,"lng":-8.26699921291585},{"lat":40.7736021810143,"lng":-8.26721022033198},{"lat":40.7735175584374,"lng":-8.2674169818516},{"lat":40.7732092124329,"lng":-8.26781481776635},{"lat":40.7729621984391,"lng":-8.26797517028929},{"lat":40.7726332483941,"lng":-8.26820582413635},{"lat":40.7723644273154,"lng":-8.26837773294327},{"lat":40.7720093194678,"lng":-8.26856711288101},{"lat":40.7717598560582,"lng":-8.26866680077274},{"lat":40.7715900648114,"lng":-8.26869098886489},{"lat":40.7712846010195,"lng":-8.26864497861764},{"lat":40.7711415686297,"lng":-8.26852362749163},{"lat":40.7709868237102,"lng":-8.26825668736862},{"lat":40.7708768406108,"lng":-8.26797859008088},{"lat":40.7707576276711,"lng":-8.26766647373239},{"lat":40.7706022210238,"lng":-8.26749749230831},{"lat":40.7704792697655,"lng":-8.26745246594934},{"lat":40.770184097399,"lng":-8.26744665392485},{"lat":40.7699729007352,"lng":-8.26754785124314},{"lat":40.7698491501873,"lng":-8.26771886772926},{"lat":40.7697540889317,"lng":-8.26794735898312},{"lat":40.7694947064922,"lng":-8.26833735958117},{"lat":40.7692622412268,"lng":-8.26860646021083},{"lat":40.7690104061642,"lng":-8.26880386107442},{"lat":40.7688243487726,"lng":-8.26889633394971},{"lat":40.7685073008635,"lng":-8.2690348447265},{"lat":40.7681972043733,"lng":-8.26908265103754},{"lat":40.7680067791554,"lng":-8.26909205511702},{"lat":40.767865714805,"lng":-8.2690990214943},{"lat":40.7675471440602,"lng":-8.2690800030422},{"lat":40.7672794518676,"lng":-8.26913558646452},{"lat":40.7669989996388,"lng":-8.26931704887922},{"lat":40.7667556222958,"lng":-8.2695238156039},{"lat":40.7665674561759,"lng":-8.2697197962794},{"lat":40.7663467141998,"lng":-8.27004480284356},{"lat":40.7661427684589,"lng":-8.27040300582482},{"lat":40.7659685964909,"lng":-8.27064425214498},{"lat":40.7657379267493,"lng":-8.27083788691543},{"lat":40.765547238557,"lng":-8.27095415191944},{"lat":40.7651220702521,"lng":-8.2711454775067},{"lat":40.7648989751212,"lng":-8.27132893943834},{"lat":40.7645831418787,"lng":-8.27164958984007},{"lat":40.76416395233,"lng":-8.27210231178093},{"lat":40.7638371055714,"lng":-8.27244460552562},{"lat":40.7635613099874,"lng":-8.27271830866441},{"lat":40.7633157865339,"lng":-8.27305236402271},{"lat":40.7631734140056,"lng":-8.27336897182393},{"lat":40.7642125800084,"lng":-8.27323625806602},{"lat":40.7661974908386,"lng":-8.27298275005402},{"lat":40.767135241136,"lng":-8.27343038974834},{"lat":40.7674602192395,"lng":-8.27341525315346},{"lat":40.7676535668037,"lng":-8.27338677788565},{"lat":40.7677824337011,"lng":-8.2733934399746},{"lat":40.767877303994,"lng":-8.2734201360793},{"lat":40.7679828875742,"lng":-8.27345937766264},{"lat":40.7681681740408,"lng":-8.2735610403541},{"lat":40.768249182028,"lng":-8.27362839499292},{"lat":40.769101673903,"lng":-8.27441098605486},{"lat":40.7689948977934,"lng":-8.27451342333864},{"lat":40.7689006796699,"lng":-8.27466528973989},{"lat":40.7688554101175,"lng":-8.27481908964692},{"lat":40.7689853728756,"lng":-8.27500671287694},{"lat":40.7690763950135,"lng":-8.27510143180788},{"lat":40.7692344071816,"lng":-8.27519309214377},{"lat":40.7694875476362,"lng":-8.27525582066887},{"lat":40.7695835758339,"lng":-8.27579009392595},{"lat":40.7695900274875,"lng":-8.27598841152162},{"lat":40.7695654492684,"lng":-8.27611127371198},{"lat":40.7693943960397,"lng":-8.27617794655833},{"lat":40.769230824037,"lng":-8.27620692368812},{"lat":40.7691094663673,"lng":-8.27623821089184},{"lat":40.7690096557905,"lng":-8.27632541887677},{"lat":40.7688890674974,"lng":-8.27641804694984},{"lat":40.7686916882113,"lng":-8.27646133081794},{"lat":40.7685599100365,"lng":-8.27644711394518},{"lat":40.7684590834607,"lng":-8.27643623632083},{"lat":40.7684276656043,"lng":-8.27656985510497},{"lat":40.7684047526277,"lng":-8.2766956594568},{"lat":40.7683925434616,"lng":-8.27705507818781},{"lat":40.7714886507301,"lng":-8.27925603497021},{"lat":40.7720161148924,"lng":-8.27963102153804},{"lat":40.7726210277209,"lng":-8.28006107659742},{"lat":40.7727505470898,"lng":-8.2801786302093},{"lat":40.7728478161767,"lng":-8.28033279141474},{"lat":40.7729202137435,"lng":-8.28044753439982},{"lat":40.773124991241,"lng":-8.28072196759579},{"lat":40.7733289175735,"lng":-8.28088778064742},{"lat":40.7739183975457,"lng":-8.28121014030968},{"lat":40.7763985179732,"lng":-8.28324186883987},{"lat":40.7779459584138,"lng":-8.28418562764091},{"lat":40.7794461930245,"lng":-8.28478454746479},{"lat":40.7807804887166,"lng":-8.28506781941942},{"lat":40.7808805928463,"lng":-8.28536297867587},{"lat":40.7811073144807,"lng":-8.28539930130282},{"lat":40.7827314606647,"lng":-8.2856595109767},{"lat":40.7833959391971,"lng":-8.2851635384929},{"lat":40.7835386233238,"lng":-8.28508261992235},{"lat":40.7836633819997,"lng":-8.28503045174368},{"lat":40.7843479091386,"lng":-8.28504962108207},{"lat":40.7846078679784,"lng":-8.28522133841805},{"lat":40.7847621570631,"lng":-8.28523108643278},{"lat":40.7847353903035,"lng":-8.28537171128358},{"lat":40.7840726269505,"lng":-8.28885335605979},{"lat":40.7841061290578,"lng":-8.28959823928813},{"lat":40.7841163229502,"lng":-8.28982491086124},{"lat":40.7838260261014,"lng":-8.29137616838857},{"lat":40.783368478697,"lng":-8.29408543763963},{"lat":40.7830422161748,"lng":-8.29601707879086},{"lat":40.7809518906682,"lng":-8.29883251419164},{"lat":40.7785565556276,"lng":-8.30129286348824},{"lat":40.7784821957379,"lng":-8.30138031306887},{"lat":40.7784085131855,"lng":-8.30156254597561},{"lat":40.7781782941545,"lng":-8.30173770809255},{"lat":40.7771001804735,"lng":-8.30255797190307},{"lat":40.7769244281562,"lng":-8.30325528428779},{"lat":40.7767816373008,"lng":-8.30364558865708},{"lat":40.7767078616934,"lng":-8.30384724475209},{"lat":40.7765313782755,"lng":-8.30432963366022},{"lat":40.7766768389518,"lng":-8.30439844821961},{"lat":40.7768846660621,"lng":-8.30444193005044},{"lat":40.7770349808308,"lng":-8.3044158112635},{"lat":40.7775194027676,"lng":-8.30491856415535},{"lat":40.7777325978466,"lng":-8.30519000369817},{"lat":40.7779945887974,"lng":-8.30552357355629},{"lat":40.7783760911554,"lng":-8.30604182209498},{"lat":40.7784881432677,"lng":-8.30615863758681},{"lat":40.7786408429275,"lng":-8.30639476259922},{"lat":40.7787078229548,"lng":-8.30653735513807},{"lat":40.7787710250332,"lng":-8.3067094041326},{"lat":40.7789309064849,"lng":-8.30713216583169},{"lat":40.7791580943707,"lng":-8.30748144375489},{"lat":40.7792658216628,"lng":-8.3078304074951},{"lat":40.779325595583,"lng":-8.30809350022175},{"lat":40.7793696495886,"lng":-8.30828740431398},{"lat":40.7794402096458,"lng":-8.30852823032411},{"lat":40.7796672193733,"lng":-8.30899046815134},{"lat":40.7799015497039,"lng":-8.30954603710114},{"lat":40.7801192730952,"lng":-8.30977153924139},{"lat":40.7802855578296,"lng":-8.30993490940581},{"lat":40.7804586171621,"lng":-8.31009589186113},{"lat":40.7805937063451,"lng":-8.31021903392454},{"lat":40.7831187930413,"lng":-8.31169202462763},{"lat":40.7846887641106,"lng":-8.31260791835634},{"lat":40.7855235731153,"lng":-8.31309495013593},{"lat":40.785729747295,"lng":-8.31321523527189},{"lat":40.7859010975452,"lng":-8.31382883640988},{"lat":40.7977144828872,"lng":-8.30752040944507},{"lat":40.7993999215632,"lng":-8.30693771135915},{"lat":40.8006820373177,"lng":-8.30674039567672},{"lat":40.8009966047172,"lng":-8.30674121539012},{"lat":40.8027548580379,"lng":-8.30674579736239},{"lat":40.8136614977296,"lng":-8.30866691023934},{"lat":40.8159500876181,"lng":-8.29742275717114},{"lat":40.8165270661654,"lng":-8.29375498482344},{"lat":40.8168893707945,"lng":-8.29018072526163},{"lat":40.8170376354243,"lng":-8.28622960266238},{"lat":40.8169707262182,"lng":-8.28274839328905},{"lat":40.8168200940133,"lng":-8.27920781023714}]},{"zone_id":"str3","points":[{"lat":41.3329255766662,"lng":-8.45451883706366},{"lat":41.3363025161735,"lng":-8.45611482990106},{"lat":41.3364539962199,"lng":-8.45577732169977},{"lat":41.3399187492223,"lng":-8.44877041334471},{"lat":41.3429543623801,"lng":-8.44267623688464},{"lat":41.340307965555,"lng":-8.43146551391605},{"lat":41.3361333285436,"lng":-8.41475715255439},{"lat":41.3345720491389,"lng":-8.41019890525794},{"lat":41.3328678736236,"lng":-8.40554545914764},{"lat":41.3291635068547,"lng":-8.39980080620811},{"lat":41.3285201657559,"lng":-8.39875978591523},{"lat":41.3282572573375,"lng":-8.39832355891111},{"lat":41.328084057502,"lng":-8.39812062331782},{"lat":41.327859698274,"lng":-8.39772349602975},{"lat":41.3277710611225,"lng":-8.3975843342588},{"lat":41.3275764249812,"lng":-8.39745525361722},{"lat":41.3274919633966,"lng":-8.39737893766056},{"lat":41.3274031255249,"lng":-8.39732626113858},{"lat":41.3272269175424,"lng":-8.397210632293},{"lat":41.3271510650314,"lng":-8.39711058390486},{"lat":41.3270783066974,"lng":-8.39691606915972},{"lat":41.3269934468639,"lng":-8.39670095496448},{"lat":41.3269630592482,"lng":-8.39648642977985},{"lat":41.3268706554096,"lng":-8.39614467462184},{"lat":41.3267453745756,"lng":-8.3957292072132},{"lat":41.326590891111,"lng":-8.39552349158526},{"lat":41.3264577696927,"lng":-8.39542321524964},{"lat":41.3263130348234,"lng":-8.39536279291257},{"lat":41.3261279885146,"lng":-8.39533122169897},{"lat":41.3259802591426,"lng":-8.39531927841665},{"lat":41.3258597943522,"lng":-8.3953539139107},{"lat":41.325749320203,"lng":-8.39539145127146},{"lat":41.3256373766842,"lng":-8.39540246560516},{"lat":41.3255198206504,"lng":-8.39534680665546},{"lat":41.3254498194755,"lng":-8.39524583212894},{"lat":41.3253870186162,"lng":-8.39514691604438},{"lat":41.3242446211111,"lng":-8.39434896983443},{"lat":41.3233717935468,"lng":-8.39373999009821},{"lat":41.3229363028277,"lng":-8.39341875738296},{"lat":41.3226456554497,"lng":-8.39305879700695},{"lat":41.321997400235,"lng":-8.39225596385766},{"lat":41.3216622074412,"lng":-8.39204393769893},{"lat":41.321443535163,"lng":-8.39189170087348},{"lat":41.3212162727897,"lng":-8.39186205016908},{"lat":41.3210982092072,"lng":-8.39188057066751},{"lat":41.3201995060609,"lng":-8.39204634500963},{"lat":41.3200586977562,"lng":-8.39207958561748},{"lat":41.3198939759695,"lng":-8.39209696233898},{"lat":41.3197246388107,"lng":-8.39212387671197},{"lat":41.3195389312949,"lng":-8.3922235766092},{"lat":41.319385175359,"lng":-8.39232663444239},{"lat":41.3192208078989,"lng":-8.39246516312671},{"lat":41.3190681849091,"lng":-8.39258692216313},{"lat":41.318927304078,"lng":-8.39269217810471},{"lat":41.3188343085694,"lng":-8.39280275931429},{"lat":41.3184008256314,"lng":-8.39313423814457},{"lat":41.3175530443358,"lng":-8.39390820952259},{"lat":41.317422778217,"lng":-8.39393634891515},{"lat":41.3171806222312,"lng":-8.39434893025252},{"lat":41.3169726268791,"lng":-8.39470330481703},{"lat":41.3168206487891,"lng":-8.39497343396969},{"lat":41.3163589123795,"lng":-8.39521329360322},{"lat":41.3151244832675,"lng":-8.39559620147198},{"lat":41.3140886694491,"lng":-8.39579013751013},{"lat":41.3131504062548,"lng":-8.39597484911203},{"lat":41.3128832064348,"lng":-8.39602745055156},{"lat":41.312248826814,"lng":-8.39651569555674},{"lat":41.3120617799865,"lng":-8.39660987532487},{"lat":41.3118678514045,"lng":-8.39675704969112},{"lat":41.3117359755768,"lng":-8.39685861940467},{"lat":41.3115919981238,"lng":-8.3969393583887},{"lat":41.3114870179526,"lng":-8.3970171947514},{"lat":41.3111738104142,"lng":-8.39719130166869},{"lat":41.3110585729793,"lng":-8.39726224702348},{"lat":41.310926692128,"lng":-8.39736619566801},{"lat":41.3107422027864,"lng":-8.39756105626776},{"lat":41.3105388006496,"lng":-8.39779476133191},{"lat":41.3104125834411,"lng":-8.39797910063225},{"lat":41.3102744537924,"lng":-8.39821725443361},{"lat":41.3099098818034,"lng":-8.39879636639867},{"lat":41.3095556627403,"lng":-8.39941587208203},{"lat":41.3093347661221,"lng":-8.39977559943299},{"lat":41.3091721130062,"lng":-8.40002486822117},{"lat":41.3090233739336,"lng":-8.40044495694977},{"lat":41.308966238976,"lng":-8.40069572547931},{"lat":41.3088749416236,"lng":-8.40100188907467},{"lat":41.3087918081794,"lng":-8.40128264385804},{"lat":41.3086857117624,"lng":-8.4015243720469},{"lat":41.3085302745877,"lng":-8.40184220651333},{"lat":41.3083895563972,"lng":-8.40206684204116},{"lat":41.3082672693747,"lng":-8.40220140736703},{"lat":41.3078576571416,"lng":-8.40265214203704},{"lat":41.3077078270683,"lng":-8.40292091528073},{"lat":41.3075527207304,"lng":-8.4032119921631},{"lat":41.3074405864195,"lng":-8.4034925040237},{"lat":41.3073674791488,"lng":-8.40421296226005},{"lat":41.3066132673514,"lng":-8.4052880859215},{"lat":41.3060692311053,"lng":-8.40657471033817},{"lat":41.3052921170825,"lng":-8.40862762866666},{"lat":41.3055330501536,"lng":-8.40885899561836},{"lat":41.3061422860464,"lng":-8.40944405179235},{"lat":41.3044534756207,"lng":-8.41435806798615},{"lat":41.3029033594806,"lng":-8.41633250255703},{"lat":41.302616452979,"lng":-8.41669792992533},{"lat":41.3025273421957,"lng":-8.41732227808155},{"lat":41.3021861960241,"lng":-8.41971225880913},{"lat":41.3020315769042,"lng":-8.42097341343003},{"lat":41.3014212139226,"lng":-8.42217951930299},{"lat":41.2995462438514,"lng":-8.42588389176486},{"lat":41.3077736045023,"lng":-8.44351132304613},{"lat":41.3099379591231,"lng":-8.44701254368226},{"lat":41.3121483277425,"lng":-8.44891896325251},{"lat":41.3145730717476,"lng":-8.45082655298277},{"lat":41.3170700413263,"lng":-8.45245024359157},{"lat":41.3195677791567,"lng":-8.45378965775681},{"lat":41.3221388239363,"lng":-8.45446589855199},{"lat":41.3246402489888,"lng":-8.45447818789195},{"lat":41.3267139157658,"lng":-8.45410912912464},{"lat":41.3294308036854,"lng":-8.45374320093269},{"lat":41.3318596993319,"lng":-8.4541343877302},{"lat":41.3329255766662,"lng":-8.45451883706366}]},{"zone_id":"vcb4","points":[{"lat":40.8260290040607,"lng":-8.40978268247786},{"lat":40.8263205760422,"lng":-8.40963757396389},{"lat":40.8271283401828,"lng":-8.40965968587128},{"lat":40.8274486484367,"lng":-8.40962384738912},{"lat":40.8284481461749,"lng":-8.40952188315564},{"lat":40.8293013676501,"lng":-8.40902825015726},{"lat":40.830065839594,"lng":-8.40885527424734},{"lat":40.8324792764823,"lng":-8.40897774031952},{"lat":40.831225372789,"lng":-8.40553469273465},{"lat":40.830743525779,"lng":-8.40134689700758},{"lat":40.8305237246853,"lng":-8.3962715257247},{"lat":40.8311286786043,"lng":-8.38962834392478},{"lat":40.8342075994207,"lng":-8.38015409129746},{"lat":40.8367869362673,"lng":-8.3765821866671},{"lat":40.8254201404937,"lng":-8.36892227991222},{"lat":40.8201385437609,"lng":-8.36514009462629},{"lat":40.8182846839326,"lng":-8.36287560985136},{"lat":40.8175747680195,"lng":-8.36042699266891},{"lat":40.8178672960115,"lng":-8.35704097483154},{"lat":40.8187430447631,"lng":-8.35411696284596},{"lat":40.8198335608492,"lng":-8.35047541690901},{"lat":40.8151833332578,"lng":-8.34878888721066},{"lat":40.8098490095069,"lng":-8.34739188449814},{"lat":40.8063235422472,"lng":-8.34700427424484},{"lat":40.8027962105267,"lng":-8.34762003334317},{"lat":40.8001254198409,"lng":-8.3488655516785},{"lat":40.7977388662633,"lng":-8.35098974635447},{"lat":40.7947435942992,"lng":-8.35445522998137},{"lat":40.7903097416699,"lng":-8.35973169674035},{"lat":40.7901038507814,"lng":-8.35997669128493},{"lat":40.7901911544016,"lng":-8.36007070756058},{"lat":40.7903722374237,"lng":-8.36026809958569},{"lat":40.7905691560202,"lng":-8.36048889712384},{"lat":40.7907540855077,"lng":-8.36068021218175},{"lat":40.7909361158155,"lng":-8.36086704686275},{"lat":40.7911154819757,"lng":-8.36104849906956},{"lat":40.7913059993643,"lng":-8.3612364359094},{"lat":40.7914948528651,"lng":-8.36142273423736},{"lat":40.7916728534812,"lng":-8.36159984116469},{"lat":40.7918600494699,"lng":-8.36178932072195},{"lat":40.7920499459181,"lng":-8.36197552504062},{"lat":40.7922327571534,"lng":-8.36214938482727},{"lat":40.7924197992149,"lng":-8.36232431819274},{"lat":40.7926094192773,"lng":-8.36251502710692},{"lat":40.792796988024,"lng":-8.36269470611208},{"lat":40.7929961481061,"lng":-8.36288817760684},{"lat":40.7931323679815,"lng":-8.36302563977313},{"lat":40.793288202792,"lng":-8.36318289665599},{"lat":40.7933777902741,"lng":-8.36328993540414},{"lat":40.7948310981392,"lng":-8.36430641223299},{"lat":40.7952558361763,"lng":-8.36458896123936},{"lat":40.7965076490728,"lng":-8.36542173198572},{"lat":40.7980712888801,"lng":-8.36646199531895},{"lat":40.7987262177811,"lng":-8.36689772512276},{"lat":40.799454213176,"lng":-8.36738207809919},{"lat":40.7997638858238,"lng":-8.3673831746831},{"lat":40.7999224639089,"lng":-8.36738373623163},{"lat":40.799919955865,"lng":-8.36860714157848},{"lat":40.7999196080634,"lng":-8.3687762987031},{"lat":40.7999191330273,"lng":-8.36900714316816},{"lat":40.7998631028343,"lng":-8.36925505722917},{"lat":40.7998275321442,"lng":-8.36941244368686},{"lat":40.7996917130036,"lng":-8.37001338409199},{"lat":40.7995898722712,"lng":-8.37046397322387},{"lat":40.7994851098738,"lng":-8.37092747747531},{"lat":40.7994105003187,"lng":-8.37125757321481},{"lat":40.7993543121569,"lng":-8.3715061616015},{"lat":40.7994463186973,"lng":-8.37354057422067},{"lat":40.7993459787065,"lng":-8.3747930701773},{"lat":40.7993151459072,"lng":-8.37522053039652},{"lat":40.799251817252,"lng":-8.37609843707219},{"lat":40.7992027072833,"lng":-8.37677802429326},{"lat":40.7989583128604,"lng":-8.38015913364536},{"lat":40.7987724485582,"lng":-8.38272956627651},{"lat":40.7986852860038,"lng":-8.38297677155876},{"lat":40.798627138757,"lng":-8.38370660423707},{"lat":40.7969682495208,"lng":-8.38408576785278},{"lat":40.7964996892563,"lng":-8.38475963190723},{"lat":40.7958923068371,"lng":-8.3860700701502},{"lat":40.7961833907827,"lng":-8.3867812994248},{"lat":40.7965999184893,"lng":-8.38890258926032},{"lat":40.7969268238272,"lng":-8.39021663051261},{"lat":40.7969745588104,"lng":-8.39085394679472},{"lat":40.7979955182444,"lng":-8.39149743684177},{"lat":40.7983338937798,"lng":-8.39178776352865},{"lat":40.7986975185142,"lng":-8.39301762579341},{"lat":40.7989705494218,"lng":-8.39372932111372},{"lat":40.7991226253536,"lng":-8.39440795289915},{"lat":40.7991973200297,"lng":-8.39474193170876},{"lat":40.7989500448794,"lng":-8.39470481337547},{"lat":40.7983649426431,"lng":-8.39425683960868},{"lat":40.7976416846097,"lng":-8.39413359470794},{"lat":40.7967627982023,"lng":-8.39390132220457},{"lat":40.7961674536585,"lng":-8.39356163303705},{"lat":40.7958318451585,"lng":-8.39369492936618},{"lat":40.7946544702733,"lng":-8.39416254574131},{"lat":40.7943386261604,"lng":-8.39428798618175},{"lat":40.791609034404,"lng":-8.39624783238988},{"lat":40.788146392192,"lng":-8.39873661935696},{"lat":40.7880458194369,"lng":-8.39880860357169},{"lat":40.788355861192,"lng":-8.40062550721884},{"lat":40.7883170401966,"lng":-8.40151652919736},{"lat":40.7881369879537,"lng":-8.40370922520328},{"lat":40.788053097897,"lng":-8.40473067030623},{"lat":40.7880426947811,"lng":-8.40532737884101},{"lat":40.7880323141991,"lng":-8.40592261835067},{"lat":40.7880283011191,"lng":-8.4061526876876},{"lat":40.7880106515086,"lng":-8.40716422953121},{"lat":40.7881500208088,"lng":-8.41259842588982},{"lat":40.7883911749291,"lng":-8.41730493322249},{"lat":40.7899069381009,"lng":-8.42231648854435},{"lat":40.7908831114646,"lng":-8.42407841800249},{"lat":40.7921734577691,"lng":-8.42523006008595},{"lat":40.7920663716256,"lng":-8.42629392190472},{"lat":40.7920667938576,"lng":-8.426551863404},{"lat":40.792164128925,"lng":-8.4265895032392},{"lat":40.7923412703981,"lng":-8.42661860470328},{"lat":40.7924834294971,"lng":-8.42673511169031},{"lat":40.7925818171531,"lng":-8.42674869819959},{"lat":40.7926823796174,"lng":-8.4267229582887},{"lat":40.7927887300349,"lng":-8.42661596726648},{"lat":40.7929185726742,"lng":-8.42658431203371},{"lat":40.7930635080454,"lng":-8.42677547619525},{"lat":40.7931592201518,"lng":-8.42677886718568},{"lat":40.7933012670804,"lng":-8.42669312217295},{"lat":40.7934205657618,"lng":-8.42659352858164},{"lat":40.7935369722306,"lng":-8.4265327880571},{"lat":40.7936961753966,"lng":-8.4265344404132},{"lat":40.7939172320584,"lng":-8.42660995314548},{"lat":40.7941179528359,"lng":-8.42678489384096},{"lat":40.7943198210309,"lng":-8.4269670791172},{"lat":40.7945579808351,"lng":-8.42715024793061},{"lat":40.7946994164198,"lng":-8.42719744175713},{"lat":40.7948127544072,"lng":-8.42721086197069},{"lat":40.7949717651947,"lng":-8.42742674570174},{"lat":40.795115216484,"lng":-8.42767028479384},{"lat":40.7952374952711,"lng":-8.4279185905599},{"lat":40.7953845978352,"lng":-8.42817660012637},{"lat":40.7955355721412,"lng":-8.42840180843714},{"lat":40.7957098498324,"lng":-8.42859714761927},{"lat":40.7958662519366,"lng":-8.42874240519297},{"lat":40.7959886651289,"lng":-8.42893632872712},{"lat":40.796140313324,"lng":-8.42917872629164},{"lat":40.7963739880323,"lng":-8.42945858247095},{"lat":40.7965650066733,"lng":-8.42969215478752},{"lat":40.7966132206113,"lng":-8.42981098912966},{"lat":40.7967240930699,"lng":-8.4300850680594},{"lat":40.7968102546086,"lng":-8.43019506393855},{"lat":40.7970488819704,"lng":-8.43040373171205},{"lat":40.7971345734002,"lng":-8.43055554992181},{"lat":40.7972074253156,"lng":-8.43069273290851},{"lat":40.7973136243718,"lng":-8.43085495481162},{"lat":40.7975937482416,"lng":-8.43113917803369},{"lat":40.7978625268491,"lng":-8.43135308424456},{"lat":40.7980534623781,"lng":-8.43151248769662},{"lat":40.7981457727659,"lng":-8.43161244188054},{"lat":40.798256625114,"lng":-8.43171448792794},{"lat":40.7983373180834,"lng":-8.43184886812302},{"lat":40.7983951991511,"lng":-8.43199558838669},{"lat":40.7984321343524,"lng":-8.43219589352667},{"lat":40.7984679320218,"lng":-8.432384340634},{"lat":40.798497189432,"lng":-8.43280157542674},{"lat":40.7985331191026,"lng":-8.4329391923989},{"lat":40.79853980138,"lng":-8.43306992240252},{"lat":40.7985898716414,"lng":-8.43320701436347},{"lat":40.7986674815264,"lng":-8.43331330308874},{"lat":40.7987249514087,"lng":-8.43341108450053},{"lat":40.8005586117976,"lng":-8.43474586870202},{"lat":40.8006414109588,"lng":-8.43462538206074},{"lat":40.8007754370851,"lng":-8.43437323159162},{"lat":40.8009134107752,"lng":-8.43412370613284},{"lat":40.8010434566047,"lng":-8.43387912443125},{"lat":40.8011741555816,"lng":-8.43362316746797},{"lat":40.8012259378741,"lng":-8.43351348123583},{"lat":40.8013795260066,"lng":-8.4332213156628},{"lat":40.8015265436858,"lng":-8.43295891554071},{"lat":40.8016636923428,"lng":-8.43271625372729},{"lat":40.8018041812246,"lng":-8.43246946191416},{"lat":40.8019480050557,"lng":-8.43222125963756},{"lat":40.8020956035855,"lng":-8.43197639128796},{"lat":40.8022335496033,"lng":-8.43173728640527},{"lat":40.8023031786075,"lng":-8.43161269788708},{"lat":40.8023802019152,"lng":-8.4314748780152},{"lat":40.8025244190811,"lng":-8.43121197171291},{"lat":40.802666889129,"lng":-8.4309645934407},{"lat":40.8027663822151,"lng":-8.43079487305035},{"lat":40.8029130760233,"lng":-8.43055046496057},{"lat":40.8030133651404,"lng":-8.43028262557301},{"lat":40.8047674090691,"lng":-8.4276325352167},{"lat":40.8045828842727,"lng":-8.42418379641831},{"lat":40.8046581608683,"lng":-8.42400707476965},{"lat":40.8047773873409,"lng":-8.42372283099827},{"lat":40.8048560843551,"lng":-8.4236179457461},{"lat":40.8048995921315,"lng":-8.42343385889457},{"lat":40.8052974437379,"lng":-8.42193021077988},{"lat":40.8055666552713,"lng":-8.42047374765519},{"lat":40.8058365382614,"lng":-8.4187765778527},{"lat":40.8060271095365,"lng":-8.41752551003806},{"lat":40.8060785793962,"lng":-8.4171876027206},{"lat":40.8061437932557,"lng":-8.41677831406807},{"lat":40.8057159873314,"lng":-8.41568028586415},{"lat":40.8053170267637,"lng":-8.41405148202606},{"lat":40.8053283267854,"lng":-8.41307574554743},{"lat":40.8053580870001,"lng":-8.41220863479392},{"lat":40.8054051525832,"lng":-8.41170281230583},{"lat":40.8055248386714,"lng":-8.411438332572},{"lat":40.8057915333347,"lng":-8.41100594929794},{"lat":40.8061675230126,"lng":-8.4107666089616},{"lat":40.8059034891377,"lng":-8.41010292141711},{"lat":40.805529706648,"lng":-8.40942680863089},{"lat":40.8052194532925,"lng":-8.40891961139193},{"lat":40.8052753374361,"lng":-8.40848622299006},{"lat":40.8055423054846,"lng":-8.40793331575097},{"lat":40.8053597833476,"lng":-8.40766757492018},{"lat":40.8050674926093,"lng":-8.40736535609002},{"lat":40.8049028762109,"lng":-8.4072562441682},{"lat":40.8040520660119,"lng":-8.40685512871955},{"lat":40.8037410779825,"lng":-8.40661292250474},{"lat":40.8035310841009,"lng":-8.40631105551477},{"lat":40.8035960895594,"lng":-8.40593780122251},{"lat":40.8040928745454,"lng":-8.40507145242485},{"lat":40.8025522861101,"lng":-8.4034512321253},{"lat":40.8019754862053,"lng":-8.40284464349197},{"lat":40.8017720985723,"lng":-8.40261688746028},{"lat":40.8017451058904,"lng":-8.402339716382},{"lat":40.8018377940263,"lng":-8.40194252518637},{"lat":40.8020857774917,"lng":-8.40167868951068},{"lat":40.803959015075,"lng":-8.40176425821774},{"lat":40.8055477871887,"lng":-8.40182523529255},{"lat":40.8072783478827,"lng":-8.40204913955759},{"lat":40.811445365559,"lng":-8.40260794819101},{"lat":40.8127752629314,"lng":-8.40189063203662},{"lat":40.8135653968241,"lng":-8.40129139774182},{"lat":40.814299827163,"lng":-8.40055953640307},{"lat":40.815465976969,"lng":-8.39931144865754},{"lat":40.8161729050842,"lng":-8.39859140159262},{"lat":40.8166115158635,"lng":-8.39831322604299},{"lat":40.8168334770061,"lng":-8.39817245237838},{"lat":40.8173556717448,"lng":-8.39805400174576},{"lat":40.8173327737001,"lng":-8.39848129557104},{"lat":40.8172601322093,"lng":-8.39983664024736},{"lat":40.8170417724335,"lng":-8.40306445636524},{"lat":40.8172112921267,"lng":-8.4049083712741},{"lat":40.8173358295162,"lng":-8.40649906203592},{"lat":40.8174725392013,"lng":-8.406753988141},{"lat":40.8180926372827,"lng":-8.40798535402068},{"lat":40.8192075111896,"lng":-8.40937538578729},{"lat":40.8191759952264,"lng":-8.40948655494383},{"lat":40.8194783539463,"lng":-8.40979493685292},{"lat":40.8198155204915,"lng":-8.41034242600783},{"lat":40.8205981541879,"lng":-8.41088954472774},{"lat":40.821464669322,"lng":-8.4110799928054},{"lat":40.8224921658947,"lng":-8.41095036700076},{"lat":40.8228033839849,"lng":-8.41094408988702},{"lat":40.8232891158204,"lng":-8.41090238741216},{"lat":40.8236124493919,"lng":-8.41087577275896},{"lat":40.8245830530711,"lng":-8.41040177631133},{"lat":40.8255224201021,"lng":-8.41003479415648},{"lat":40.8260290040607,"lng":-8.40978268247786}]},{"zone_id":"vcb1","points":[{"lat":40.8367869362673,"lng":-8.3765821866671},{"lat":40.8342075994207,"lng":-8.38015409129746},{"lat":40.8311286786043,"lng":-8.38962834392478},{"lat":40.8305237246853,"lng":-8.3962715257247},{"lat":40.830743525779,"lng":-8.40134689700758},{"lat":40.831225372789,"lng":-8.40553469273465},{"lat":40.8324792764823,"lng":-8.40897774031952},{"lat":40.8330531772401,"lng":-8.41008614906699},{"lat":40.8334253137489,"lng":-8.41028379769512},{"lat":40.8350689454393,"lng":-8.41144001215809},{"lat":40.8361084470487,"lng":-8.412221559898},{"lat":40.8368836300961,"lng":-8.41293858257874},{"lat":40.8373650058545,"lng":-8.41331611455118},{"lat":40.8379667764596,"lng":-8.41394860399252},{"lat":40.8382413248602,"lng":-8.4139709201847},{"lat":40.8383232727701,"lng":-8.4139085888659},{"lat":40.8385332683799,"lng":-8.41376226806074},{"lat":40.8388243970225,"lng":-8.41350072130846},{"lat":40.838851913231,"lng":-8.41364289129672},{"lat":40.8390312146746,"lng":-8.41389976050256},{"lat":40.8391445553114,"lng":-8.41402137519791},{"lat":40.8392878323375,"lng":-8.4141517208464},{"lat":40.8394399628041,"lng":-8.41427594600306},{"lat":40.8396560680943,"lng":-8.41440027941389},{"lat":40.8398081093105,"lng":-8.41446983921988},{"lat":40.8399017319942,"lng":-8.41449301048005},{"lat":40.840044383639,"lng":-8.41450418510446},{"lat":40.8402280155144,"lng":-8.41451418425308},{"lat":40.8401196485358,"lng":-8.41463151429427},{"lat":40.839904152246,"lng":-8.41486483377142},{"lat":40.839844106988,"lng":-8.41500448806864},{"lat":40.8398599968563,"lng":-8.41518910780401},{"lat":40.8398789330072,"lng":-8.41530627442059},{"lat":40.8399003382708,"lng":-8.41542671343173},{"lat":40.8399352130996,"lng":-8.41556214829663},{"lat":40.8399789850432,"lng":-8.41574113740029},{"lat":40.8400108995498,"lng":-8.4158718172142},{"lat":40.8399475021615,"lng":-8.41628375634364},{"lat":40.8397596050248,"lng":-8.4175046005324},{"lat":40.8427698907548,"lng":-8.42279294553964},{"lat":40.8434460861942,"lng":-8.42252445151965},{"lat":40.8435738026875,"lng":-8.42247722562472},{"lat":40.8436690356871,"lng":-8.42245629963833},{"lat":40.8438640530668,"lng":-8.42243082968397},{"lat":40.8440006690187,"lng":-8.42242561707835},{"lat":40.8441138683981,"lng":-8.42242125124064},{"lat":40.8442571793438,"lng":-8.42241417076389},{"lat":40.844381278111,"lng":-8.42239846972844},{"lat":40.844595256896,"lng":-8.4223525677868},{"lat":40.8448403044637,"lng":-8.42230585435432},{"lat":40.8449738177657,"lng":-8.42228106245208},{"lat":40.8450921011177,"lng":-8.42226094860077},{"lat":40.8452775044161,"lng":-8.42222701545733},{"lat":40.8454359688942,"lng":-8.42219877665088},{"lat":40.845658309513,"lng":-8.42215789123301},{"lat":40.8458889359765,"lng":-8.42211644864092},{"lat":40.8461166687147,"lng":-8.42207985500749},{"lat":40.8463149502219,"lng":-8.42204538506802},{"lat":40.8464318628988,"lng":-8.4220332082156},{"lat":40.8466060823699,"lng":-8.42200634010578},{"lat":40.8467751735671,"lng":-8.42197767204577},{"lat":40.846905535092,"lng":-8.42195286510142},{"lat":40.8471014767832,"lng":-8.42191779131752},{"lat":40.8473164303728,"lng":-8.42187805773169},{"lat":40.8474823680754,"lng":-8.42185008567541},{"lat":40.847639747394,"lng":-8.42182397485852},{"lat":40.8477657749999,"lng":-8.42180341750492},{"lat":40.8480071951027,"lng":-8.42176676261574},{"lat":40.8481611528806,"lng":-8.42174051678657},{"lat":40.8487681702911,"lng":-8.42159434606985},{"lat":40.848884850688,"lng":-8.42156698802528},{"lat":40.8489914220273,"lng":-8.42154907292209},{"lat":40.8491404161486,"lng":-8.42152719119361},{"lat":40.8492354401168,"lng":-8.42148171386915},{"lat":40.8493555221305,"lng":-8.42146243406923},{"lat":40.8494788846201,"lng":-8.42146368392853},{"lat":40.8495920548964,"lng":-8.42143559863282},{"lat":40.8496914088108,"lng":-8.42138705637306},{"lat":40.8498340646186,"lng":-8.42134309052542},{"lat":40.8499640210334,"lng":-8.42130013675727},{"lat":40.8501649701567,"lng":-8.42124337958465},{"lat":40.8502945216174,"lng":-8.42121833047644},{"lat":40.8503745306186,"lng":-8.42116389159207},{"lat":40.8504714487198,"lng":-8.42115316755257},{"lat":40.8505672738025,"lng":-8.42114706252247},{"lat":40.8506767221298,"lng":-8.42113093760029},{"lat":40.850766804902,"lng":-8.42111686218983},{"lat":40.8508645511843,"lng":-8.42109902625467},{"lat":40.8509568741852,"lng":-8.42108934826925},{"lat":40.8510535535191,"lng":-8.42106640827411},{"lat":40.851194536214,"lng":-8.42104294964972},{"lat":40.8513191122081,"lng":-8.4210270084845},{"lat":40.8514129414384,"lng":-8.42102717984299},{"lat":40.8515375778635,"lng":-8.42102274213058},{"lat":40.8516541873399,"lng":-8.42102372482552},{"lat":40.851754166533,"lng":-8.42108464129765},{"lat":40.8518357129201,"lng":-8.42117073800047},{"lat":40.8537544981958,"lng":-8.42150648183335},{"lat":40.8540870451764,"lng":-8.42154079960013},{"lat":40.8542783995743,"lng":-8.42150602955569},{"lat":40.8550900099251,"lng":-8.42137240858219},{"lat":40.8557043483018,"lng":-8.42120700548956},{"lat":40.8558275885419,"lng":-8.42117782801576},{"lat":40.8559323818574,"lng":-8.42118634900767},{"lat":40.8560259870814,"lng":-8.42120407223731},{"lat":40.8561677478222,"lng":-8.42122924104023},{"lat":40.8562672055284,"lng":-8.42124722695052},{"lat":40.8563924031629,"lng":-8.42127042539646},{"lat":40.8564941141074,"lng":-8.42128723520289},{"lat":40.8566184082907,"lng":-8.42131161588518},{"lat":40.8567417181668,"lng":-8.42133350180001},{"lat":40.8568499231778,"lng":-8.42134607153268},{"lat":40.8570231729704,"lng":-8.42134670861813},{"lat":40.8572364790428,"lng":-8.42135309474696},{"lat":40.8573286730953,"lng":-8.42135895237773},{"lat":40.8574533170603,"lng":-8.42138748605941},{"lat":40.8576123418503,"lng":-8.42145839214253},{"lat":40.8577735946607,"lng":-8.42153891376931},{"lat":40.8578655030079,"lng":-8.42158663687152},{"lat":40.8579536355704,"lng":-8.42163185308441},{"lat":40.8580601919275,"lng":-8.42169114441148},{"lat":40.8581466209656,"lng":-8.42173338726146},{"lat":40.858252288374,"lng":-8.42178840656568},{"lat":40.8583405260396,"lng":-8.42182757385217},{"lat":40.8584438581033,"lng":-8.42188021127177},{"lat":40.8585540598892,"lng":-8.42193466485474},{"lat":40.8587275862822,"lng":-8.42198484529122},{"lat":40.8589411160218,"lng":-8.42199608670872},{"lat":40.8597815913249,"lng":-8.42249093898348},{"lat":40.8598627510195,"lng":-8.42258712762474},{"lat":40.8599641230578,"lng":-8.42270226160609},{"lat":40.8600421218877,"lng":-8.42276653091107},{"lat":40.860134133429,"lng":-8.42280880194121},{"lat":40.8602576511109,"lng":-8.4228553627746},{"lat":40.8603637478482,"lng":-8.42288263247185},{"lat":40.860462573083,"lng":-8.42290097627528},{"lat":40.8605794894974,"lng":-8.42292271878453},{"lat":40.8606868906639,"lng":-8.42275410113099},{"lat":40.8608370267366,"lng":-8.42267287958007},{"lat":40.8608611044456,"lng":-8.42283689818879},{"lat":40.8609466302612,"lng":-8.42295137217534},{"lat":40.8610530782563,"lng":-8.42308894871631},{"lat":40.8611543344202,"lng":-8.42314324053219},{"lat":40.8612781092189,"lng":-8.42323036651163},{"lat":40.86138377524,"lng":-8.42328527101429},{"lat":40.8615256235605,"lng":-8.42334697687995},{"lat":40.8616424256401,"lng":-8.42337832850243},{"lat":40.861736020564,"lng":-8.42339961459995},{"lat":40.861821074907,"lng":-8.42345110869957},{"lat":40.8636817126705,"lng":-8.42455465942079},{"lat":40.8637814410463,"lng":-8.4245008943638},{"lat":40.8638865971014,"lng":-8.42443766393606},{"lat":40.8640043422606,"lng":-8.42431043881798},{"lat":40.8641050358938,"lng":-8.42426699667982},{"lat":40.8642051634788,"lng":-8.42423339681202},{"lat":40.864308974013,"lng":-8.4242033713655},{"lat":40.8644263427734,"lng":-8.42418882492809},{"lat":40.8645453724862,"lng":-8.42415839148791},{"lat":40.8646606228602,"lng":-8.4241622207905},{"lat":40.8647750047413,"lng":-8.42418905742618},{"lat":40.8648667542634,"lng":-8.42419254548499},{"lat":40.8649722858508,"lng":-8.42415849432797},{"lat":40.8651031595942,"lng":-8.42414483709721},{"lat":40.8652734501297,"lng":-8.42410478499692},{"lat":40.8654191569212,"lng":-8.42408465706242},{"lat":40.8655432343936,"lng":-8.42408005304113},{"lat":40.8656435163962,"lng":-8.42408043873636},{"lat":40.8657441762473,"lng":-8.42408283868573},{"lat":40.8659098616023,"lng":-8.4240981587093},{"lat":40.8660071216343,"lng":-8.42411821884104},{"lat":40.8661920679436,"lng":-8.42415758357788},{"lat":40.8662899772266,"lng":-8.4241721625969},{"lat":40.866424426605,"lng":-8.42419922118973},{"lat":40.8665331953201,"lng":-8.42422582831948},{"lat":40.8666335944259,"lng":-8.42425430378745},{"lat":40.866753821831,"lng":-8.42428413244102},{"lat":40.8668570132235,"lng":-8.4243206466007},{"lat":40.8670057131068,"lng":-8.42437895110941},{"lat":40.8671192924282,"lng":-8.42443792991997},{"lat":40.8672550741789,"lng":-8.42449399334714},{"lat":40.8672759000439,"lng":-8.42343784511922},{"lat":40.8675152290129,"lng":-8.42159036283549},{"lat":40.8675171869454,"lng":-8.42081153171265},{"lat":40.8675182716025,"lng":-8.42037917124048},{"lat":40.8676053245343,"lng":-8.42020334755989},{"lat":40.8678017362125,"lng":-8.41980664472671},{"lat":40.8683671681962,"lng":-8.41907603638737},{"lat":40.8685796173417,"lng":-8.41863070712634},{"lat":40.8687931754765,"lng":-8.4177392549137},{"lat":40.8688567188073,"lng":-8.41706339023139},{"lat":40.8689872932535,"lng":-8.41567858111096},{"lat":40.869103787511,"lng":-8.41444294623821},{"lat":40.8693035646649,"lng":-8.41440748190062},{"lat":40.869421575994,"lng":-8.41437615768695},{"lat":40.8696459340996,"lng":-8.41430692865514},{"lat":40.8698318368846,"lng":-8.41430286034394},{"lat":40.8700582883571,"lng":-8.4143735840419},{"lat":40.8702953064587,"lng":-8.41446394745674},{"lat":40.8704641065062,"lng":-8.41451469639245},{"lat":40.8705610356417,"lng":-8.41455971218487},{"lat":40.8707928144689,"lng":-8.41455807874565},{"lat":40.8710186603779,"lng":-8.41457627751901},{"lat":40.8712474637334,"lng":-8.41459619098544},{"lat":40.8714671660696,"lng":-8.41463924439867},{"lat":40.871637196541,"lng":-8.41468482132694},{"lat":40.8718663703834,"lng":-8.41476026512292},{"lat":40.8720788115034,"lng":-8.41479416256877},{"lat":40.8722041380753,"lng":-8.41477402571951},{"lat":40.8725209239167,"lng":-8.41459942679643},{"lat":40.8731650477913,"lng":-8.414904382977},{"lat":40.8732876280916,"lng":-8.41485521564961},{"lat":40.8734857514736,"lng":-8.41473600597357},{"lat":40.8736103131878,"lng":-8.41465196024536},{"lat":40.8741013021204,"lng":-8.41411748788401},{"lat":40.8741873801046,"lng":-8.4139000539331},{"lat":40.8742733993465,"lng":-8.41359589991533},{"lat":40.8742424663096,"lng":-8.41324746892971},{"lat":40.8740304749834,"lng":-8.41259375255543},{"lat":40.8734562880711,"lng":-8.41150314562764},{"lat":40.8732071604952,"lng":-8.41102721010403},{"lat":40.872669209931,"lng":-8.40982185377533},{"lat":40.8726180689342,"lng":-8.40970656215974},{"lat":40.8725381476942,"lng":-8.40952614712443},{"lat":40.8719681869542,"lng":-8.40817831584199},{"lat":40.8717136076428,"lng":-8.40788341745156},{"lat":40.8715414519709,"lng":-8.40750274507262},{"lat":40.8716573095577,"lng":-8.40729041248511},{"lat":40.8723972458751,"lng":-8.40686797364694},{"lat":40.8729752992657,"lng":-8.40647510761164},{"lat":40.8726414499453,"lng":-8.40607857597967},{"lat":40.8721571984843,"lng":-8.40580337675513},{"lat":40.8717651267265,"lng":-8.40573177017792},{"lat":40.8713273242645,"lng":-8.40535000667233},{"lat":40.8711781571138,"lng":-8.40503029433396},{"lat":40.8710984283663,"lng":-8.40458915520876},{"lat":40.8707654163027,"lng":-8.40379763560732},{"lat":40.8711357023118,"lng":-8.40344956641468},{"lat":40.8714716380911,"lng":-8.40288878431617},{"lat":40.8719247291842,"lng":-8.40167484327062},{"lat":40.8723314706327,"lng":-8.40052143060049},{"lat":40.8729576980166,"lng":-8.39938419570216},{"lat":40.8733271548382,"lng":-8.39921921169894},{"lat":40.873417730321,"lng":-8.39919869794869},{"lat":40.8735079978627,"lng":-8.39916489876247},{"lat":40.8735988740338,"lng":-8.39913386955474},{"lat":40.8741333056519,"lng":-8.39883419552065},{"lat":40.8742471746656,"lng":-8.39879963213369},{"lat":40.8743445445237,"lng":-8.39871680640307},{"lat":40.8744290572435,"lng":-8.39860972590517},{"lat":40.8745111474825,"lng":-8.39849955104403},{"lat":40.8745975069102,"lng":-8.39839637560804},{"lat":40.8747106706897,"lng":-8.39836416919151},{"lat":40.8748040314353,"lng":-8.39835647129527},{"lat":40.874897519155,"lng":-8.39835072775603},{"lat":40.8749884659085,"lng":-8.39832257746985},{"lat":40.8750750360964,"lng":-8.39827572028776},{"lat":40.8751572667775,"lng":-8.3982167463341},{"lat":40.8752397489287,"lng":-8.39815843995397},{"lat":40.8753191348731,"lng":-8.39809439348805},{"lat":40.8753877983911,"lng":-8.39801050546337},{"lat":40.8754569716551,"lng":-8.39792734295898},{"lat":40.8755494033209,"lng":-8.39781675001088},{"lat":40.8756207633102,"lng":-8.39773690331057},{"lat":40.8757021544021,"lng":-8.39767834720211},{"lat":40.8757935019595,"lng":-8.397651259028},{"lat":40.8758856136408,"lng":-8.39762904730191},{"lat":40.8759784287359,"lng":-8.39761344860442},{"lat":40.8760707456611,"lng":-8.3976309219167},{"lat":40.8761706219041,"lng":-8.39763631991579},{"lat":40.8762645165044,"lng":-8.39763021859883},{"lat":40.8763567735235,"lng":-8.39760588133369},{"lat":40.876448073024,"lng":-8.39757578402567},{"lat":40.8765383675492,"lng":-8.39754098587452},{"lat":40.8766260900241,"lng":-8.39749608865727},{"lat":40.8767131628038,"lng":-8.39744902126942},{"lat":40.8767990694305,"lng":-8.39739842090033},{"lat":40.8768836457054,"lng":-8.39734402826023},{"lat":40.8770172684556,"lng":-8.39729082463556},{"lat":40.8771473312662,"lng":-8.39727746958015},{"lat":40.8772183534763,"lng":-8.3971963152344},{"lat":40.8772575090075,"lng":-8.3970858210987},{"lat":40.8773080645214,"lng":-8.39698123392617},{"lat":40.877359125941,"lng":-8.39687707094873},{"lat":40.877435926396,"lng":-8.39672100555719},{"lat":40.8774880201917,"lng":-8.39661773604906},{"lat":40.8775448123017,"lng":-8.39651893744407},{"lat":40.8775987945107,"lng":-8.39641751515041},{"lat":40.8776447606732,"lng":-8.39630932828917},{"lat":40.8776894043408,"lng":-8.39620017033613},{"lat":40.8777352273976,"lng":-8.39609186278716},{"lat":40.8777817119897,"lng":-8.39598404530507},{"lat":40.8778336969583,"lng":-8.39588082289336},{"lat":40.8778937944651,"lng":-8.39578542331146},{"lat":40.8779680783452,"lng":-8.39571086848468},{"lat":40.8780575479078,"lng":-8.39567824075717},{"lat":40.8781506388831,"lng":-8.39567897993446},{"lat":40.8782339976465,"lng":-8.39573540378221},{"lat":40.8783240211828,"lng":-8.39577151240459},{"lat":40.878416746179,"lng":-8.3957720224851},{"lat":40.8785073995719,"lng":-8.39573953217703},{"lat":40.878594374839,"lng":-8.39569225316397},{"lat":40.8786750862128,"lng":-8.39562880846002},{"lat":40.8787440765449,"lng":-8.39554493512648},{"lat":40.8788039895852,"lng":-8.39544935169165},{"lat":40.8788837457509,"lng":-8.39538773807256},{"lat":40.8789749864248,"lng":-8.39535749992257},{"lat":40.8790653430421,"lng":-8.39532280565828},{"lat":40.8791557344102,"lng":-8.3952882680418},{"lat":40.8792448000267,"lng":-8.39524828546257},{"lat":40.8793324304866,"lng":-8.3952030520391},{"lat":40.8794199268784,"lng":-8.39515736832179},{"lat":40.8795075009913,"lng":-8.39511194935925},{"lat":40.8795928106756,"lng":-8.39506005425574},{"lat":40.8796705494158,"lng":-8.394990169356},{"lat":40.8797476784333,"lng":-8.39491911802453},{"lat":40.8798145068508,"lng":-8.39483256941486},{"lat":40.8798689839238,"lng":-8.39473145626889},{"lat":40.8777093690179,"lng":-8.39245751349118},{"lat":40.874976914292,"lng":-8.38862028399364},{"lat":40.8723636986711,"lng":-8.38568845776781},{"lat":40.8696333478033,"lng":-8.38128408907332},{"lat":40.8678541714609,"lng":-8.37766819135556},{"lat":40.866788651044,"lng":-8.37452583881371},{"lat":40.8654844686604,"lng":-8.37153965012923},{"lat":40.8640601206769,"lng":-8.36902389867755},{"lat":40.8618008822563,"lng":-8.36697605389229},{"lat":40.8598977212026,"lng":-8.36555721154767},{"lat":40.8578747869112,"lng":-8.36445183575437},{"lat":40.8559703405041,"lng":-8.36366072994707},{"lat":40.8535881317435,"lng":-8.36349558117573},{"lat":40.8505377997013,"lng":-8.36393024875625},{"lat":40.8476751782564,"lng":-8.36570870815367},{"lat":40.8437371299439,"lng":-8.36908327244067},{"lat":40.837935310392,"lng":-8.37499176228939},{"lat":40.8367869362673,"lng":-8.3765821866671}]},{"zone_id":"vcb5","points":[{"lat":40.8449121207561,"lng":-8.2797637965397},{"lat":40.8311785754702,"lng":-8.28524810109897},{"lat":40.8373010020646,"lng":-8.2871922354957},{"lat":40.8426843239259,"lng":-8.29027212067606},{"lat":40.8465546416303,"lng":-8.29333127930942},{"lat":40.8497065890465,"lng":-8.29549378799852},{"lat":40.8526988054573,"lng":-8.29955508480488},{"lat":40.8559701407633,"lng":-8.30314168458192},{"lat":40.8588423995532,"lng":-8.30820511956128},{"lat":40.8605178263345,"lng":-8.3111587899659},{"lat":40.861874093166,"lng":-8.31506185442916},{"lat":40.8628314412674,"lng":-8.31875394243494},{"lat":40.8635294988675,"lng":-8.32215593781169},{"lat":40.8648490546131,"lng":-8.326925236189},{"lat":40.8654311479689,"lng":-8.3287136323494},{"lat":40.8672357839342,"lng":-8.3337223270969},{"lat":40.8702856677775,"lng":-8.34218746767164},{"lat":40.8713512191401,"lng":-8.34294042703706},{"lat":40.8726370932696,"lng":-8.34332113183672},{"lat":40.8752822022203,"lng":-8.34304700339876},{"lat":40.8771621399664,"lng":-8.34261604751042},{"lat":40.8790565380961,"lng":-8.34612791662026},{"lat":40.881045218864,"lng":-8.34523492269587},{"lat":40.8817564153201,"lng":-8.34467577578936},{"lat":40.8824471510241,"lng":-8.3433535920037},{"lat":40.8832477270087,"lng":-8.34170062632217},{"lat":40.8841678148168,"lng":-8.34040788918821},{"lat":40.8853739031025,"lng":-8.33802189912658},{"lat":40.8870602772952,"lng":-8.33701394001712},{"lat":40.8870581229019,"lng":-8.33665631286437},{"lat":40.8872132067459,"lng":-8.3362883762778},{"lat":40.8872998366251,"lng":-8.33618505955748},{"lat":40.8874169081271,"lng":-8.33602796522634},{"lat":40.8874874498653,"lng":-8.33590514410792},{"lat":40.8875801376837,"lng":-8.33568556319135},{"lat":40.8876101908456,"lng":-8.33554730084695},{"lat":40.8876372660347,"lng":-8.3352066146331},{"lat":40.8876106049652,"lng":-8.3350089766576},{"lat":40.8875948009691,"lng":-8.3348806567648},{"lat":40.8875767785772,"lng":-8.33458302159619},{"lat":40.8875882526311,"lng":-8.33425379032248},{"lat":40.8875863303488,"lng":-8.3339136116806},{"lat":40.8876331390432,"lng":-8.33371701852689},{"lat":40.8876544515086,"lng":-8.33358087170991},{"lat":40.8876589446821,"lng":-8.3332766609791},{"lat":40.887542980533,"lng":-8.33295902982096},{"lat":40.8874205384682,"lng":-8.33268765085026},{"lat":40.8873283799858,"lng":-8.3324036689066},{"lat":40.8872877058508,"lng":-8.33217964852225},{"lat":40.8872643608209,"lng":-8.33188365594264},{"lat":40.8872626182793,"lng":-8.33159247666039},{"lat":40.8872329322341,"lng":-8.33146994034717},{"lat":40.8873157115915,"lng":-8.331144007816},{"lat":40.8873524751446,"lng":-8.33087026768815},{"lat":40.8873590790491,"lng":-8.33048822472258},{"lat":40.8873255498835,"lng":-8.33019267919057},{"lat":40.8873051529037,"lng":-8.32975016128398},{"lat":40.887298779088,"lng":-8.32952991874348},{"lat":40.8873099927277,"lng":-8.32939718121286},{"lat":40.8873749870475,"lng":-8.3290930327953},{"lat":40.8873906341644,"lng":-8.32879158019329},{"lat":40.8873539209176,"lng":-8.32856626825902},{"lat":40.8873266088519,"lng":-8.32826610733945},{"lat":40.887320223945,"lng":-8.32768172398315},{"lat":40.8871743900638,"lng":-8.3270744983838},{"lat":40.8869902974587,"lng":-8.32661785442408},{"lat":40.8864031404351,"lng":-8.32568898425601},{"lat":40.8854222268308,"lng":-8.32436862918527},{"lat":40.8840262414806,"lng":-8.32322780679288},{"lat":40.8828287960967,"lng":-8.31916819520466},{"lat":40.8826804288266,"lng":-8.31866522416963},{"lat":40.8815704407001,"lng":-8.31603023058854},{"lat":40.8801998843561,"lng":-8.3134532772755},{"lat":40.8801584026436,"lng":-8.31337528318447},{"lat":40.8795592605433,"lng":-8.31267883230919},{"lat":40.8793542699609,"lng":-8.31244054993308},{"lat":40.878572006523,"lng":-8.3115815114907},{"lat":40.877850444097,"lng":-8.3106768088948},{"lat":40.877700713312,"lng":-8.31049798000351},{"lat":40.877608359698,"lng":-8.31038767894404},{"lat":40.8769774959995,"lng":-8.30958233572237},{"lat":40.8762268832378,"lng":-8.30862415358441},{"lat":40.8750332632183,"lng":-8.30546085855889},{"lat":40.8740811535778,"lng":-8.30341083597928},{"lat":40.8715952628937,"lng":-8.29796801460199},{"lat":40.8698327400608,"lng":-8.29591938092531},{"lat":40.8688698018463,"lng":-8.29474451271071},{"lat":40.866048808689,"lng":-8.29140841439602},{"lat":40.8640268179031,"lng":-8.29023250489627},{"lat":40.8625786485094,"lng":-8.28916153123169},{"lat":40.861691015444,"lng":-8.28850512135764},{"lat":40.8616015081637,"lng":-8.28843893133169},{"lat":40.8600884450516,"lng":-8.28738196115689},{"lat":40.8596304347532,"lng":-8.28706202292794},{"lat":40.8579918476139,"lng":-8.28563710825731},{"lat":40.8578631571456,"lng":-8.28552520272861},{"lat":40.8555650883173,"lng":-8.28502887931774},{"lat":40.8545953937047,"lng":-8.2846062306785},{"lat":40.8527408079256,"lng":-8.28379793215339},{"lat":40.8509765716939,"lng":-8.2830290559041},{"lat":40.8449121207561,"lng":-8.2797637965397}]},{"zone_id":"arc14","points":[{"lat":40.8998298118896,"lng":-8.3734949777164},{"lat":40.9015907096687,"lng":-8.3698701153352},{"lat":40.9150850426342,"lng":-8.34271639469538},{"lat":40.9158978054921,"lng":-8.34136719472603},{"lat":40.9061539090159,"lng":-8.32380593066879},{"lat":40.9053951204677,"lng":-8.32160556994951},{"lat":40.9050659517285,"lng":-8.31890402623316},{"lat":40.9051397482978,"lng":-8.31637601625597},{"lat":40.893172606933,"lng":-8.3184165750081},{"lat":40.8917729433435,"lng":-8.31837342465663},{"lat":40.8875735246312,"lng":-8.31852318991638},{"lat":40.884586257013,"lng":-8.31891832459348},{"lat":40.8828287960967,"lng":-8.31916819520466},{"lat":40.8840262414806,"lng":-8.32322780679288},{"lat":40.8854222268308,"lng":-8.32436862918527},{"lat":40.8864031404351,"lng":-8.32568898425601},{"lat":40.8869902974587,"lng":-8.32661785442408},{"lat":40.8871743900638,"lng":-8.3270744983838},{"lat":40.887320223945,"lng":-8.32768172398315},{"lat":40.8873266088519,"lng":-8.32826610733945},{"lat":40.8873539209176,"lng":-8.32856626825902},{"lat":40.8873906341644,"lng":-8.32879158019329},{"lat":40.8873749870475,"lng":-8.3290930327953},{"lat":40.8873099927277,"lng":-8.32939718121286},{"lat":40.887298779088,"lng":-8.32952991874348},{"lat":40.8873051529037,"lng":-8.32975016128398},{"lat":40.8873255498835,"lng":-8.33019267919057},{"lat":40.8873590790491,"lng":-8.33048822472258},{"lat":40.8873524751446,"lng":-8.33087026768815},{"lat":40.8873157115915,"lng":-8.331144007816},{"lat":40.8872329322341,"lng":-8.33146994034717},{"lat":40.8872626182793,"lng":-8.33159247666039},{"lat":40.8872643608209,"lng":-8.33188365594264},{"lat":40.8872877058508,"lng":-8.33217964852225},{"lat":40.8873283799858,"lng":-8.3324036689066},{"lat":40.8874205384682,"lng":-8.33268765085026},{"lat":40.887542980533,"lng":-8.33295902982096},{"lat":40.8876589446821,"lng":-8.3332766609791},{"lat":40.8876544515086,"lng":-8.33358087170991},{"lat":40.8876331390432,"lng":-8.33371701852689},{"lat":40.8875863303488,"lng":-8.3339136116806},{"lat":40.8875882526311,"lng":-8.33425379032248},{"lat":40.8875767785772,"lng":-8.33458302159619},{"lat":40.8875948009691,"lng":-8.3348806567648},{"lat":40.8876106049652,"lng":-8.3350089766576},{"lat":40.8876372660347,"lng":-8.3352066146331},{"lat":40.8876101908456,"lng":-8.33554730084695},{"lat":40.8875801376837,"lng":-8.33568556319135},{"lat":40.8874874498653,"lng":-8.33590514410792},{"lat":40.8874169081271,"lng":-8.33602796522634},{"lat":40.8872998366251,"lng":-8.33618505955748},{"lat":40.8872132067459,"lng":-8.3362883762778},{"lat":40.8870581229019,"lng":-8.33665631286437},{"lat":40.8870602772952,"lng":-8.33701394001712},{"lat":40.8870815858333,"lng":-8.33723410496234},{"lat":40.8871926018483,"lng":-8.33744244683957},{"lat":40.8873756855007,"lng":-8.33768234203774},{"lat":40.887417139854,"lng":-8.33776516955693},{"lat":40.8874923924589,"lng":-8.33803071448376},{"lat":40.8874981974735,"lng":-8.33900879124779},{"lat":40.8875010563188,"lng":-8.33932038499918},{"lat":40.887508865269,"lng":-8.33963543989951},{"lat":40.8875733345288,"lng":-8.33993428981485},{"lat":40.8876646145765,"lng":-8.34019917229578},{"lat":40.8877381882519,"lng":-8.34044430704133},{"lat":40.8877430426555,"lng":-8.34084620596114},{"lat":40.8877497925769,"lng":-8.3410466344326},{"lat":40.8878308696661,"lng":-8.3413233550673},{"lat":40.8879645874151,"lng":-8.34157259367312},{"lat":40.8881029232419,"lng":-8.34180488120308},{"lat":40.8882011631885,"lng":-8.34195398441964},{"lat":40.8883518269035,"lng":-8.34219212890308},{"lat":40.8883750219745,"lng":-8.3422613832618},{"lat":40.888400126245,"lng":-8.3423712158346},{"lat":40.8884031296076,"lng":-8.34245096234041},{"lat":40.8883970400152,"lng":-8.34267935297585},{"lat":40.8883915945508,"lng":-8.34275183835144},{"lat":40.8883924410275,"lng":-8.3430745814023},{"lat":40.8883937802745,"lng":-8.34371651146164},{"lat":40.8883942238958,"lng":-8.34384434500333},{"lat":40.8885666470103,"lng":-8.34388031695747},{"lat":40.8887393322784,"lng":-8.34399296478766},{"lat":40.8889475002217,"lng":-8.34450613823634},{"lat":40.8890941636067,"lng":-8.34491918028041},{"lat":40.8891414743067,"lng":-8.34523834913264},{"lat":40.889165129644,"lng":-8.34549493585698},{"lat":40.8891935160377,"lng":-8.34571397330459},{"lat":40.8892313645435,"lng":-8.34599559288008},{"lat":40.8891697357181,"lng":-8.3462892268478},{"lat":40.8891287186932,"lng":-8.34640101280533},{"lat":40.8892335810806,"lng":-8.3465944115361},{"lat":40.889400098938,"lng":-8.34682324730189},{"lat":40.8895553227127,"lng":-8.34707629677987},{"lat":40.8897189599774,"lng":-8.34733282319376},{"lat":40.8899492854527,"lng":-8.34762881929109},{"lat":40.8901256398304,"lng":-8.34786453138058},{"lat":40.8902373649644,"lng":-8.34812104957506},{"lat":40.8904315207827,"lng":-8.34839922603642},{"lat":40.8905153313566,"lng":-8.34851447797515},{"lat":40.8906786763431,"lng":-8.34872823826284},{"lat":40.8910100361497,"lng":-8.3492241327892},{"lat":40.8911664708918,"lng":-8.34943567883216},{"lat":40.8913164420292,"lng":-8.34965202984815},{"lat":40.8914568648762,"lng":-8.34989780595916},{"lat":40.8916057833755,"lng":-8.35017930639757},{"lat":40.8918052390517,"lng":-8.35043793278825},{"lat":40.8922118108807,"lng":-8.35076072418241},{"lat":40.8926075798299,"lng":-8.35109194495434},{"lat":40.8928032363781,"lng":-8.35126581351602},{"lat":40.8929987767157,"lng":-8.35142041940624},{"lat":40.8932088371348,"lng":-8.35158398623127},{"lat":40.8933911934974,"lng":-8.35174931790716},{"lat":40.8935587584565,"lng":-8.35195600844505},{"lat":40.8936414766852,"lng":-8.3520864757781},{"lat":40.8937003420969,"lng":-8.35219931552373},{"lat":40.8937441229851,"lng":-8.35233098039048},{"lat":40.8938233914471,"lng":-8.35262680208013},{"lat":40.8938567793145,"lng":-8.35275505371048},{"lat":40.8938784954224,"lng":-8.35288138887671},{"lat":40.8939020991987,"lng":-8.35300441775258},{"lat":40.8939639308883,"lng":-8.35332768988013},{"lat":40.8940033508699,"lng":-8.35347180281281},{"lat":40.8940912100865,"lng":-8.35362697125364},{"lat":40.8941804633955,"lng":-8.35370037842587},{"lat":40.8944022071708,"lng":-8.35383622002834},{"lat":40.8946908153471,"lng":-8.35403378841061},{"lat":40.8948922220731,"lng":-8.35417732926315},{"lat":40.8950457810006,"lng":-8.35438834752294},{"lat":40.8951961955848,"lng":-8.35462663347409},{"lat":40.8954949398609,"lng":-8.35513712411776},{"lat":40.8959626851734,"lng":-8.35589815429525},{"lat":40.8963661744502,"lng":-8.35667784422237},{"lat":40.8968220055832,"lng":-8.35765928070134},{"lat":40.8968743197343,"lng":-8.35777019676155},{"lat":40.8968185160239,"lng":-8.35788147701579},{"lat":40.8965223530772,"lng":-8.35856839734514},{"lat":40.8961715415403,"lng":-8.35954174977797},{"lat":40.8960022692129,"lng":-8.36001140646459},{"lat":40.8959428155119,"lng":-8.36079298312786},{"lat":40.896035427878,"lng":-8.3611776773102},{"lat":40.8960040081451,"lng":-8.36212026982798},{"lat":40.8957467782373,"lng":-8.36300107869763},{"lat":40.8957696656425,"lng":-8.36370061658262},{"lat":40.8957477021263,"lng":-8.3639087021992},{"lat":40.8957585486678,"lng":-8.36418149744799},{"lat":40.8958508700276,"lng":-8.36441706167891},{"lat":40.8959744006183,"lng":-8.36457686340743},{"lat":40.8961300257008,"lng":-8.36479094111374},{"lat":40.8963122156338,"lng":-8.36499701815204},{"lat":40.896493149244,"lng":-8.3652158597982},{"lat":40.8966671003303,"lng":-8.36543605173129},{"lat":40.8968390270705,"lng":-8.36562374204041},{"lat":40.8970490188173,"lng":-8.36578589048138},{"lat":40.8972519332101,"lng":-8.36593556422971},{"lat":40.8974567430641,"lng":-8.36607415324761},{"lat":40.897565289945,"lng":-8.36614408558833},{"lat":40.8976651252733,"lng":-8.36620757582624},{"lat":40.8978566536194,"lng":-8.36636453025537},{"lat":40.8979328267559,"lng":-8.36644059762276},{"lat":40.8980401567366,"lng":-8.36658637694352},{"lat":40.8980639656949,"lng":-8.36674030871847},{"lat":40.8980729329762,"lng":-8.36690974610961},{"lat":40.898076385651,"lng":-8.3670595253017},{"lat":40.8980519459292,"lng":-8.36745518466925},{"lat":40.8980287709262,"lng":-8.36778991833176},{"lat":40.8979959009358,"lng":-8.36809773733892},{"lat":40.8979753296471,"lng":-8.36840604827723},{"lat":40.897980434897,"lng":-8.3688229185183},{"lat":40.8980114308792,"lng":-8.36899106631901},{"lat":40.898120223905,"lng":-8.36931416533042},{"lat":40.8982590247741,"lng":-8.36968841618331},{"lat":40.8983593379899,"lng":-8.3699799063701},{"lat":40.8984784599336,"lng":-8.37037079594584},{"lat":40.8986255811345,"lng":-8.37086128618232},{"lat":40.8987061645921,"lng":-8.37123023752371},{"lat":40.8987855712969,"lng":-8.37152639172135},{"lat":40.8988567138005,"lng":-8.37182494016003},{"lat":40.8989388845643,"lng":-8.37212724280914},{"lat":40.8990274859209,"lng":-8.37242936140294},{"lat":40.8990743264741,"lng":-8.37260192560802},{"lat":40.8991190192021,"lng":-8.37272694732323},{"lat":40.8998298118896,"lng":-8.3734949777164}]},{"zone_id":"arc1","points":[{"lat":40.9148278970815,"lng":-8.28087985562267},{"lat":40.9185937347884,"lng":-8.29282597903297},{"lat":40.9234997846484,"lng":-8.28880906219787},{"lat":40.9256019171145,"lng":-8.28748655119905},{"lat":40.9315111556105,"lng":-8.28620974094388},{"lat":40.9353073692135,"lng":-8.285026727096},{"lat":40.9389901227832,"lng":-8.2836684597042},{"lat":40.9442244603512,"lng":-8.28185164552156},{"lat":40.9479424141662,"lng":-8.28087959854383},{"lat":40.9519452940998,"lng":-8.28066226563037},{"lat":40.9565186146699,"lng":-8.28150211632897},{"lat":40.9602911156922,"lng":-8.28256656608524},{"lat":40.9633978810915,"lng":-8.28368259639279},{"lat":40.9634747727877,"lng":-8.28195045886198},{"lat":40.9634993954525,"lng":-8.28139565985025},{"lat":40.9635820568814,"lng":-8.28104249470029},{"lat":40.9636145126829,"lng":-8.28090382806278},{"lat":40.9642566225697,"lng":-8.27816025438702},{"lat":40.9646922030047,"lng":-8.2762989398413},{"lat":40.9655702967835,"lng":-8.27254622018992},{"lat":40.9656352874068,"lng":-8.27176672262328},{"lat":40.9656495401102,"lng":-8.27159576703553},{"lat":40.9656979564541,"lng":-8.27101500872563},{"lat":40.9657595960539,"lng":-8.27078617235399},{"lat":40.9658908761754,"lng":-8.27029878922798},{"lat":40.9659927088696,"lng":-8.26992072434779},{"lat":40.9660309176215,"lng":-8.26977886873957},{"lat":40.966054841764,"lng":-8.26961887823692},{"lat":40.9659901620332,"lng":-8.2694432645297},{"lat":40.96589480857,"lng":-8.2692147477113},{"lat":40.9660275609689,"lng":-8.26874448506548},{"lat":40.9661533536802,"lng":-8.26832591343945},{"lat":40.9662796910245,"lng":-8.2679081331294},{"lat":40.9663409133279,"lng":-8.2677056769739},{"lat":40.9664790222382,"lng":-8.26724895865095},{"lat":40.9664701845529,"lng":-8.26696689649314},{"lat":40.9664582630914,"lng":-8.26658644803303},{"lat":40.9664505546966,"lng":-8.2663404721687},{"lat":40.9665673936201,"lng":-8.26578378767762},{"lat":40.9666254092905,"lng":-8.26550736382561},{"lat":40.9668378757142,"lng":-8.26423146186393},{"lat":40.9668744900446,"lng":-8.26401157633599},{"lat":40.9674947571958,"lng":-8.26028617324366},{"lat":40.9674052531786,"lng":-8.2597284596545},{"lat":40.9673805751013,"lng":-8.2596029047186},{"lat":40.9671309616155,"lng":-8.25833298774326},{"lat":40.967081396559,"lng":-8.25820328628515},{"lat":40.9670279839625,"lng":-8.25788873182504},{"lat":40.966924395649,"lng":-8.25767909651643},{"lat":40.9668597943868,"lng":-8.25755054973341},{"lat":40.9668211209019,"lng":-8.25742633448162},{"lat":40.9667619015205,"lng":-8.25731277492787},{"lat":40.9667441498409,"lng":-8.25707752911097},{"lat":40.9667461948903,"lng":-8.25677056249272},{"lat":40.9667872312454,"lng":-8.25646272046676},{"lat":40.9668531067077,"lng":-8.25616621583473},{"lat":40.9668810798502,"lng":-8.25577828440159},{"lat":40.9668363745003,"lng":-8.25548406198805},{"lat":40.9668305882784,"lng":-8.25509131394726},{"lat":40.9668558768382,"lng":-8.25491857670634},{"lat":40.9669047791382,"lng":-8.25467578616318},{"lat":40.9669803947334,"lng":-8.25444617147118},{"lat":40.9671013492499,"lng":-8.25425608577472},{"lat":40.9672156859787,"lng":-8.25404346196225},{"lat":40.9673832807794,"lng":-8.25375089123031},{"lat":40.9674700912111,"lng":-8.25366148165076},{"lat":40.9675591051058,"lng":-8.25353061412917},{"lat":40.9676892772814,"lng":-8.25313466793307},{"lat":40.9677731370151,"lng":-8.25285754666606},{"lat":40.9677781836763,"lng":-8.25259501712075},{"lat":40.9675847029186,"lng":-8.25203561401436},{"lat":40.9670524271463,"lng":-8.25049672404215},{"lat":40.9665358357055,"lng":-8.24716320950876},{"lat":40.9664287540205,"lng":-8.24693176449445},{"lat":40.9628327482784,"lng":-8.24659613782358},{"lat":40.9597361247228,"lng":-8.24596226689662},{"lat":40.9569376132215,"lng":-8.24501469024395},{"lat":40.9539285725511,"lng":-8.24352897185942},{"lat":40.9510713667694,"lng":-8.24163875756907},{"lat":40.9476786456657,"lng":-8.23911939244825},{"lat":40.943512116779,"lng":-8.23597066456277},{"lat":40.9415804723515,"lng":-8.23448925232643},{"lat":40.9355479208722,"lng":-8.22986343582352},{"lat":40.9336190742415,"lng":-8.22868252305134},{"lat":40.9315470712997,"lng":-8.22773708770907},{"lat":40.9261429788262,"lng":-8.22567733256258},{"lat":40.9231633138044,"lng":-8.2275578357843},{"lat":40.9208966464893,"lng":-8.23148085160518},{"lat":40.9186305389968,"lng":-8.23461837867762},{"lat":40.9166038705799,"lng":-8.23634271333629},{"lat":40.9150968464108,"lng":-8.23656107525625},{"lat":40.9137049297554,"lng":-8.24791234563159},{"lat":40.9134165636006,"lng":-8.25036154576368},{"lat":40.913139468909,"lng":-8.25321609627012},{"lat":40.9114580334285,"lng":-8.26569672714071},{"lat":40.9115142197504,"lng":-8.26860184754054},{"lat":40.9119625336645,"lng":-8.27240578311283},{"lat":40.9128760325009,"lng":-8.27535694716896},{"lat":40.9148278970815,"lng":-8.28087985562267}]},{"zone_id":"arc18","points":[{"lat":40.8974748155555,"lng":-8.12736948945749},{"lat":40.8965887273866,"lng":-8.12657551901313},{"lat":40.8960607499305,"lng":-8.12589522570644},{"lat":40.8951206379503,"lng":-8.12541956115238},{"lat":40.8950176162857,"lng":-8.12530564918842},{"lat":40.8943667190427,"lng":-8.12458576177782},{"lat":40.8938195921793,"lng":-8.12311021657155},{"lat":40.8934634905674,"lng":-8.12174180989239},{"lat":40.8931718327451,"lng":-8.12029772337965},{"lat":40.8919505651203,"lng":-8.1197494735222},{"lat":40.8912998525642,"lng":-8.1186008271626},{"lat":40.8909110875015,"lng":-8.1175265585673},{"lat":40.8907476940861,"lng":-8.11641119064477},{"lat":40.8906528381519,"lng":-8.11612002174848},{"lat":40.8905161064836,"lng":-8.11578721603264},{"lat":40.8903267096733,"lng":-8.11559313076209},{"lat":40.8900947494341,"lng":-8.11559318480959},{"lat":40.8894634436487,"lng":-8.11570439885827},{"lat":40.8891370285734,"lng":-8.11575988449579},{"lat":40.8888210501328,"lng":-8.1157321893762},{"lat":40.8884211297532,"lng":-8.11557980975757},{"lat":40.888157737525,"lng":-8.11555211019951},{"lat":40.8878315920148,"lng":-8.11558006736652},{"lat":40.8875578547528,"lng":-8.11562166288672},{"lat":40.8871577140752,"lng":-8.11582987345746},{"lat":40.8869692265438,"lng":-8.1158966465798},{"lat":40.8866734489401,"lng":-8.11588539391398},{"lat":40.88632611164,"lng":-8.11570524573752},{"lat":40.8861679709105,"lng":-8.11558045530943},{"lat":40.8857258941589,"lng":-8.11531715698596},{"lat":40.8853890881195,"lng":-8.11510924478157},{"lat":40.8850941554879,"lng":-8.11488744228453},{"lat":40.8847257911763,"lng":-8.11441601808657},{"lat":40.8842202571281,"lng":-8.1137230095472},{"lat":40.8835672696274,"lng":-8.11279428990099},{"lat":40.8834377771345,"lng":-8.1125858700658},{"lat":40.8831884350711,"lng":-8.11218455481155},{"lat":40.8828092752398,"lng":-8.11182409938876},{"lat":40.8820089085344,"lng":-8.11113109875904},{"lat":40.881608987771,"lng":-8.11102028438936},{"lat":40.8815146557101,"lng":-8.1110265305915},{"lat":40.8811979147937,"lng":-8.11104750347598},{"lat":40.8807772442758,"lng":-8.11115910721728},{"lat":40.8799138761813,"lng":-8.11118725057423},{"lat":40.8792611992547,"lng":-8.11160315352851},{"lat":40.8789924639814,"lng":-8.11177470768672},{"lat":40.8786197533005,"lng":-8.11192381492183},{"lat":40.8784297672273,"lng":-8.11199158088252},{"lat":40.8779139388923,"lng":-8.11178411815534},{"lat":40.8774611397873,"lng":-8.11152075431787},{"lat":40.875713177182,"lng":-8.1107032972215},{"lat":40.8749343430871,"lng":-8.1106202620694},{"lat":40.8736077708795,"lng":-8.11013559368204},{"lat":40.8734915945975,"lng":-8.10998294482007},{"lat":40.873369284285,"lng":-8.10980176851826},{"lat":40.8731337905689,"lng":-8.10931792093901},{"lat":40.8728598367374,"lng":-8.10869427061358},{"lat":40.8726914727018,"lng":-8.10836169375178},{"lat":40.8725124825687,"lng":-8.10803589031502},{"lat":40.8721963346768,"lng":-8.1076339715202},{"lat":40.8720733383724,"lng":-8.10754070220505},{"lat":40.8718490569356,"lng":-8.10737062841198},{"lat":40.8715118645146,"lng":-8.10709363734348},{"lat":40.8713987473383,"lng":-8.10700898260548},{"lat":40.8706942102596,"lng":-8.10612679743098},{"lat":40.8699070944957,"lng":-8.1055773547875},{"lat":40.8686252705419,"lng":-8.10447816078576},{"lat":40.8668710737311,"lng":-8.10304359951329},{"lat":40.8551653979639,"lng":-8.10642140821729},{"lat":40.8550043222778,"lng":-8.1073671484714},{"lat":40.8548793434274,"lng":-8.10829425576518},{"lat":40.8548369167439,"lng":-8.10868539817681},{"lat":40.8547573875195,"lng":-8.10955104693547},{"lat":40.854667411942,"lng":-8.10994291952675},{"lat":40.8542004949594,"lng":-8.11083715143183},{"lat":40.8539460136536,"lng":-8.11131207652794},{"lat":40.8532882163435,"lng":-8.11240201576725},{"lat":40.8530504274793,"lng":-8.11274758081205},{"lat":40.8526727071331,"lng":-8.11329649212489},{"lat":40.8526092072325,"lng":-8.11379933751793},{"lat":40.8526103077765,"lng":-8.11392978544666},{"lat":40.8525673107346,"lng":-8.11423184694748},{"lat":40.8525299856334,"lng":-8.11454861786198},{"lat":40.8525206635193,"lng":-8.11484592447244},{"lat":40.8525024293711,"lng":-8.11517762637682},{"lat":40.852494369727,"lng":-8.11547956703023},{"lat":40.8524935092797,"lng":-8.11577948136374},{"lat":40.8524865276659,"lng":-8.11609920691965},{"lat":40.8524725183502,"lng":-8.11641051056053},{"lat":40.8524286035325,"lng":-8.11663051016701},{"lat":40.8523816345541,"lng":-8.1169388507032},{"lat":40.8522432824964,"lng":-8.11733284187551},{"lat":40.8521235453124,"lng":-8.11758238480536},{"lat":40.852000302167,"lng":-8.11784686423677},{"lat":40.8519063254818,"lng":-8.11813482072345},{"lat":40.851795776097,"lng":-8.11839905709212},{"lat":40.8517072059252,"lng":-8.11872685848301},{"lat":40.8516965019281,"lng":-8.11885090571599},{"lat":40.8516545596694,"lng":-8.11902452696337},{"lat":40.8514214298589,"lng":-8.11994660151682},{"lat":40.8511295948798,"lng":-8.12071515344208},{"lat":40.8505298778089,"lng":-8.12212603528026},{"lat":40.8498510208277,"lng":-8.12357893479668},{"lat":40.8494518286493,"lng":-8.12487844374157},{"lat":40.8494428418225,"lng":-8.1252535304885},{"lat":40.8494428551112,"lng":-8.12550564268812},{"lat":40.8494419903181,"lng":-8.12630822945874},{"lat":40.8494386704269,"lng":-8.12656081870286},{"lat":40.8494031952996,"lng":-8.1266811854593},{"lat":40.8493038787251,"lng":-8.12684412956422},{"lat":40.8492518471507,"lng":-8.12719241490144},{"lat":40.84921691549,"lng":-8.12748117258216},{"lat":40.8491926977119,"lng":-8.12764825939332},{"lat":40.8490922992213,"lng":-8.12787120628233},{"lat":40.848938335838,"lng":-8.12844042411325},{"lat":40.8488689480231,"lng":-8.12942238093732},{"lat":40.8488534415351,"lng":-8.12964180793868},{"lat":40.8488335495714,"lng":-8.13117119604367},{"lat":40.8488689945194,"lng":-8.13141760104383},{"lat":40.8489989517006,"lng":-8.13232106657545},{"lat":40.84920271436,"lng":-8.13373773580055},{"lat":40.8496570877142,"lng":-8.13504607492668},{"lat":40.8497735701265,"lng":-8.13538148838805},{"lat":40.8519134026879,"lng":-8.13971135393787},{"lat":40.85217145532,"lng":-8.14004021605521},{"lat":40.8543027311388,"lng":-8.14275645650281},{"lat":40.8547174659207,"lng":-8.14440241831534},{"lat":40.8549564595805,"lng":-8.14535096640503},{"lat":40.8551613132033,"lng":-8.14923763745206},{"lat":40.8558931149026,"lng":-8.1504423074583},{"lat":40.8562818268434,"lng":-8.15111831334078},{"lat":40.856619077906,"lng":-8.15165813079282},{"lat":40.8567471522879,"lng":-8.15186313424041},{"lat":40.8572107975904,"lng":-8.15247994899444},{"lat":40.8583207195634,"lng":-8.15402154134922},{"lat":40.8589962686025,"lng":-8.15512517647369},{"lat":40.8599155486169,"lng":-8.15623677139431},{"lat":40.8609552682616,"lng":-8.15749405499488},{"lat":40.8612377671059,"lng":-8.15783567713233},{"lat":40.862290728125,"lng":-8.15920401352682},{"lat":40.8645603018806,"lng":-8.1632680507418},{"lat":40.865878185133,"lng":-8.16593764511948},{"lat":40.8660523911556,"lng":-8.16664480781423},{"lat":40.8661807319516,"lng":-8.16744019077869},{"lat":40.8659831684243,"lng":-8.16827851365994},{"lat":40.8673835805707,"lng":-8.16610495301658},{"lat":40.8684444018234,"lng":-8.16396344325584},{"lat":40.8697602547953,"lng":-8.16125232360274},{"lat":40.8709615646518,"lng":-8.15906834323987},{"lat":40.8726774757787,"lng":-8.15680913887174},{"lat":40.8746792180503,"lng":-8.15470057292364},{"lat":40.8769095539252,"lng":-8.15311928464598},{"lat":40.8793113364217,"lng":-8.15198995783962},{"lat":40.8820561444451,"lng":-8.15116200021597},{"lat":40.885209431337,"lng":-8.14945543526015},{"lat":40.8896483101803,"lng":-8.14567214169493},{"lat":40.8923899502929,"lng":-8.14283425367985},{"lat":40.8944135378135,"lng":-8.14008207185198},{"lat":40.8961107740743,"lng":-8.13457715303758},{"lat":40.8970245342616,"lng":-8.13019023867571},{"lat":40.8974748155555,"lng":-8.12736948945749}]},{"zone_id":"arc9","points":[{"lat":40.9504506359783,"lng":-8.1315727304949},{"lat":40.950359389701,"lng":-8.13160457928207},{"lat":40.9500903333485,"lng":-8.13167489057768},{"lat":40.9499929899684,"lng":-8.13169769349851},{"lat":40.9497431145025,"lng":-8.13176953809849},{"lat":40.9495178185556,"lng":-8.13186942109711},{"lat":40.9492907213642,"lng":-8.1319486311489},{"lat":40.9490628135393,"lng":-8.1320124047902},{"lat":40.9488354450462,"lng":-8.13207546777252},{"lat":40.9486106023355,"lng":-8.1321388818271},{"lat":40.9484737931377,"lng":-8.13224962294879},{"lat":40.9483706282521,"lng":-8.13235478470243},{"lat":40.9482231263396,"lng":-8.13248043454473},{"lat":40.9479980108012,"lng":-8.13260014388839},{"lat":40.9477093258759,"lng":-8.13260393937674},{"lat":40.9475905517364,"lng":-8.13257946811791},{"lat":40.9473609340059,"lng":-8.13255416653868},{"lat":40.947138070487,"lng":-8.13248789829478},{"lat":40.9470044437815,"lng":-8.13243777754125},{"lat":40.9468341654402,"lng":-8.13243693843613},{"lat":40.9465180115217,"lng":-8.1324333693256},{"lat":40.9447396003581,"lng":-8.13237336278968},{"lat":40.9444339647246,"lng":-8.13232116264787},{"lat":40.9440768627765,"lng":-8.13226017308606},{"lat":40.9435503642401,"lng":-8.13220304912929},{"lat":40.9429232803311,"lng":-8.13216847891573},{"lat":40.9425264490167,"lng":-8.13213700543159},{"lat":40.9423350331816,"lng":-8.13209553890412},{"lat":40.9422172297387,"lng":-8.13207001919477},{"lat":40.9420039092564,"lng":-8.13203605462559},{"lat":40.941901801113,"lng":-8.13194877346565},{"lat":40.9417827267413,"lng":-8.13158569570958},{"lat":40.941602130155,"lng":-8.13137841126921},{"lat":40.9414830395775,"lng":-8.1313090539867},{"lat":40.9413443243373,"lng":-8.13123329462001},{"lat":40.9412242022957,"lng":-8.1311330698735},{"lat":40.9411364088424,"lng":-8.13107797332697},{"lat":40.9410328552817,"lng":-8.13102263279589},{"lat":40.940915781023,"lng":-8.13088987587815},{"lat":40.9467756902433,"lng":-8.17022069650979},{"lat":40.9477331919919,"lng":-8.17665654125559},{"lat":40.9482012440066,"lng":-8.18164028413297},{"lat":40.9506401702424,"lng":-8.18164206993797},{"lat":40.9564425715199,"lng":-8.18144626278316},{"lat":40.9625891758814,"lng":-8.18201648177376},{"lat":40.9680208602347,"lng":-8.18296346047996},{"lat":40.9729989310967,"lng":-8.18480430980506},{"lat":40.9784542151116,"lng":-8.18730965328591},{"lat":40.9821701776528,"lng":-8.18882176716919},{"lat":40.9850287023523,"lng":-8.18976738345174},{"lat":40.9886021512386,"lng":-8.19033639911109},{"lat":40.9962329048741,"lng":-8.19111713496086},{"lat":40.9965033720119,"lng":-8.19114910007421},{"lat":40.9962521062864,"lng":-8.19069650326746},{"lat":40.9957108778942,"lng":-8.18975676206504},{"lat":40.9951768704564,"lng":-8.18875976368316},{"lat":40.9917852551812,"lng":-8.18339247064339},{"lat":40.9896937209991,"lng":-8.18023344625871},{"lat":40.9895544067373,"lng":-8.18006251230118},{"lat":40.9892117747027,"lng":-8.17964211861437},{"lat":40.9882025640792,"lng":-8.17807811850937},{"lat":40.9887034383827,"lng":-8.17730651886092},{"lat":40.9895206705618,"lng":-8.17622081007903},{"lat":40.989183513065,"lng":-8.17477555307997},{"lat":40.9886640067098,"lng":-8.17254882175941},{"lat":40.9885975862414,"lng":-8.17091302324789},{"lat":40.9886004602449,"lng":-8.1707258502001},{"lat":40.9886103893895,"lng":-8.17007904985998},{"lat":40.9883499853948,"lng":-8.1702932430745},{"lat":40.9881422015388,"lng":-8.17046415256678},{"lat":40.9872290858337,"lng":-8.16955418504151},{"lat":40.9868338087905,"lng":-8.16838655742783},{"lat":40.9860768994103,"lng":-8.16483072770143},{"lat":40.9861028329126,"lng":-8.16467675012767},{"lat":40.9862420556614,"lng":-8.16385010604454},{"lat":40.9864621935075,"lng":-8.16355094611909},{"lat":40.9870059372385,"lng":-8.16311277392194},{"lat":40.9882680404976,"lng":-8.1627099479514},{"lat":40.988999662852,"lng":-8.16203733567704},{"lat":40.9895120503797,"lng":-8.16090898246033},{"lat":40.9891630836679,"lng":-8.15999580861917},{"lat":40.9887929356319,"lng":-8.15914883380569},{"lat":40.9888631202814,"lng":-8.15861458038189},{"lat":40.9885435852589,"lng":-8.15807339446058},{"lat":40.9886343842729,"lng":-8.1575453266084},{"lat":40.9879959263611,"lng":-8.157686159566},{"lat":40.9874191943247,"lng":-8.1568652392858},{"lat":40.9867492056115,"lng":-8.15627224666808},{"lat":40.9869732530792,"lng":-8.1554843582345},{"lat":40.9870782976951,"lng":-8.15511494851405},{"lat":40.9866747175458,"lng":-8.15510791018081},{"lat":40.9865684846385,"lng":-8.15456576586691},{"lat":40.9859333730771,"lng":-8.15422699243553},{"lat":40.9855224846802,"lng":-8.1542834091461},{"lat":40.9845567511713,"lng":-8.15334600121968},{"lat":40.9848047784626,"lng":-8.1526291932609},{"lat":40.9839002223955,"lng":-8.15249294877624},{"lat":40.9836662752092,"lng":-8.15245771220061},{"lat":40.9836710145794,"lng":-8.15204793448522},{"lat":40.983534696592,"lng":-8.15159499494804},{"lat":40.9833612596647,"lng":-8.15113401986651},{"lat":40.9830781174901,"lng":-8.15030002224243},{"lat":40.9828992487279,"lng":-8.14997795833428},{"lat":40.9825017870386,"lng":-8.14977794203264},{"lat":40.9821239143173,"lng":-8.14967710811846},{"lat":40.9818901202455,"lng":-8.14933935231321},{"lat":40.9821749107748,"lng":-8.14914335927576},{"lat":40.9825697689161,"lng":-8.14887161444807},{"lat":40.9823410756797,"lng":-8.14812284481925},{"lat":40.9822320538038,"lng":-8.14794527284149},{"lat":40.9821456759787,"lng":-8.14752558730429},{"lat":40.9819146263844,"lng":-8.14744473200993},{"lat":40.9816362866363,"lng":-8.14708190613179},{"lat":40.970281151931,"lng":-8.14839997111646},{"lat":40.9622761581482,"lng":-8.14688950466129},{"lat":40.9573217754609,"lng":-8.14637252644747},{"lat":40.9543173719458,"lng":-8.14499080709898},{"lat":40.9528879589536,"lng":-8.14411064791726},{"lat":40.9519351230421,"lng":-8.14222501063955},{"lat":40.9504506359783,"lng":-8.1315727304949}]},{"zone_id":"arc8","points":[{"lat":40.963747079082,"lng":-8.24041211525196},{"lat":40.9638522572924,"lng":-8.24040970283394},{"lat":40.9639509068824,"lng":-8.24042192647049},{"lat":40.9641147278998,"lng":-8.24044499294378},{"lat":40.9643279959396,"lng":-8.24051787989673},{"lat":40.9644851769523,"lng":-8.24053170895975},{"lat":40.9648289065559,"lng":-8.24044927048119},{"lat":40.9650927179093,"lng":-8.24056129149491},{"lat":40.9652098304853,"lng":-8.24065215253966},{"lat":40.9653314310587,"lng":-8.24073088209543},{"lat":40.9655442482349,"lng":-8.24080400943512},{"lat":40.9657660836804,"lng":-8.24088068526778},{"lat":40.9659899062765,"lng":-8.24097181399005},{"lat":40.9663476913648,"lng":-8.24113446764023},{"lat":40.9676486130695,"lng":-8.24160203312641},{"lat":40.9679058515968,"lng":-8.24169641253396},{"lat":40.9680844688927,"lng":-8.2420139046743},{"lat":40.9681870082289,"lng":-8.24236546657755},{"lat":40.9682765735801,"lng":-8.24266995336493},{"lat":40.9683748947725,"lng":-8.24291428276499},{"lat":40.9684903776411,"lng":-8.24305264187798},{"lat":40.9686613244307,"lng":-8.24319786763043},{"lat":40.9688781503147,"lng":-8.24329743231652},{"lat":40.9691982039811,"lng":-8.24305475385657},{"lat":40.9693028734018,"lng":-8.24300487017777},{"lat":40.969553258828,"lng":-8.24299652777582},{"lat":40.9697372838322,"lng":-8.24308388270819},{"lat":40.9699563999418,"lng":-8.24317638262443},{"lat":40.9701307097381,"lng":-8.243189484746},{"lat":40.9703149524371,"lng":-8.24322325354396},{"lat":40.9704643899657,"lng":-8.2432796800061},{"lat":40.9706820894173,"lng":-8.24343368297044},{"lat":40.9711032079348,"lng":-8.24373119981532},{"lat":40.9713732668106,"lng":-8.24386424601882},{"lat":40.9715789697023,"lng":-8.24395107070068},{"lat":40.9720676910506,"lng":-8.24409836024279},{"lat":40.9722610699341,"lng":-8.24403900214384},{"lat":40.9725142026354,"lng":-8.24379756969445},{"lat":40.9728239327013,"lng":-8.24338982766214},{"lat":40.9730098448182,"lng":-8.24320810971319},{"lat":40.9732377427041,"lng":-8.24308705939885},{"lat":40.973479751367,"lng":-8.24297496098372},{"lat":40.9737607642419,"lng":-8.24279467611309},{"lat":40.9739144691982,"lng":-8.24271894543798},{"lat":40.9741185958761,"lng":-8.24262843977138},{"lat":40.9744143564409,"lng":-8.24249617048717},{"lat":40.9747293902142,"lng":-8.2422961204335},{"lat":40.9750485370723,"lng":-8.24241770252787},{"lat":40.9754960725742,"lng":-8.24244845138755},{"lat":40.9779880888671,"lng":-8.24296320386022},{"lat":40.9786725375461,"lng":-8.24310459095673},{"lat":40.9791337635267,"lng":-8.24319986845394},{"lat":40.9840168968505,"lng":-8.24420868231292},{"lat":40.9847349309421,"lng":-8.24435703490189},{"lat":40.9850150394711,"lng":-8.24441490884869},{"lat":40.9867769617454,"lng":-8.24437618251658},{"lat":40.9872688400183,"lng":-8.24436537088144},{"lat":40.9881056801487,"lng":-8.24434697652636},{"lat":40.9883390601423,"lng":-8.24442024159825},{"lat":40.9897413558934,"lng":-8.24470632699009},{"lat":40.9903686381989,"lng":-8.24483430422693},{"lat":40.9925935590004,"lng":-8.24450486029034},{"lat":40.9937212266105,"lng":-8.2443378780954},{"lat":40.9953447100137,"lng":-8.24409746670153},{"lat":40.9968419532869,"lng":-8.243875738908},{"lat":40.9969299840259,"lng":-8.24384908675535},{"lat":40.9974665240134,"lng":-8.24368664253904},{"lat":40.9978416612537,"lng":-8.24357306342148},{"lat":40.9998717112609,"lng":-8.24322482642235},{"lat":40.9999586349205,"lng":-8.24319169253173},{"lat":41.0000622268532,"lng":-8.24315121480341},{"lat":41.0001866731821,"lng":-8.24314786559032},{"lat":41.0004120527906,"lng":-8.24315169441591},{"lat":41.0005074333531,"lng":-8.24313105541813},{"lat":41.000552796712,"lng":-8.24296223700678},{"lat":41.0006106165098,"lng":-8.24285714420571},{"lat":41.0006425338539,"lng":-8.24272609974643},{"lat":41.0006379368905,"lng":-8.24254234445029},{"lat":41.0005291948375,"lng":-8.24232144071882},{"lat":41.0004227954682,"lng":-8.24210137951512},{"lat":41.0003462665485,"lng":-8.24181088715576},{"lat":41.0002719785263,"lng":-8.24152765297029},{"lat":41.000160236654,"lng":-8.24124173266503},{"lat":41.000027398434,"lng":-8.24098038577021},{"lat":40.99993933196,"lng":-8.24069641952837},{"lat":40.9998641837768,"lng":-8.24037467282159},{"lat":40.9998256504131,"lng":-8.2400828212594},{"lat":40.9998279220227,"lng":-8.23977700844493},{"lat":40.99986070457,"lng":-8.23957703249206},{"lat":40.9998798095664,"lng":-8.23941841415619},{"lat":40.9998856697242,"lng":-8.23907003846897},{"lat":40.9998808264596,"lng":-8.23885787377108},{"lat":40.9998118492565,"lng":-8.23856977788441},{"lat":40.9997403831264,"lng":-8.23825231649437},{"lat":40.999709954418,"lng":-8.23804902355217},{"lat":40.9997277454588,"lng":-8.23770259893745},{"lat":40.9998086793524,"lng":-8.2374250793416},{"lat":40.9999192257979,"lng":-8.23716686746063},{"lat":41.0000284267649,"lng":-8.23689521708296},{"lat":41.0001775273334,"lng":-8.23661876224234},{"lat":41.0002778478385,"lng":-8.23641009728368},{"lat":41.0003825217472,"lng":-8.2361681525734},{"lat":41.0005608605713,"lng":-8.23591776860105},{"lat":41.0007147143123,"lng":-8.23565629275732},{"lat":41.0007767532874,"lng":-8.23555833368348},{"lat":41.0008516622937,"lng":-8.23546669142418},{"lat":41.0009265665005,"lng":-8.23538646525865},{"lat":41.0010460670362,"lng":-8.23521007953039},{"lat":41.001266627595,"lng":-8.23492844641268},{"lat":41.0013725007925,"lng":-8.23485361540327},{"lat":41.0015702050542,"lng":-8.23469085184504},{"lat":41.0016591617386,"lng":-8.23459994577689},{"lat":41.0017216565418,"lng":-8.23449663990666},{"lat":41.0018427744454,"lng":-8.23438474326989},{"lat":41.0019464030299,"lng":-8.23429896626166},{"lat":41.0020193292224,"lng":-8.23421231955482},{"lat":41.002121497626,"lng":-8.23414960105963},{"lat":41.0022355188423,"lng":-8.23412279271522},{"lat":41.0023056694355,"lng":-8.23431474297437},{"lat":41.0023401352217,"lng":-8.2345405118179},{"lat":41.0025203867396,"lng":-8.23466274415009},{"lat":41.002647220136,"lng":-8.23470715753699},{"lat":41.0027622315275,"lng":-8.23478198379492},{"lat":41.002979167893,"lng":-8.2348651685266},{"lat":41.0031203483427,"lng":-8.23497474220767},{"lat":41.0033404175358,"lng":-8.23517667511351},{"lat":41.0039154251561,"lng":-8.23569605606745},{"lat":41.0044726894099,"lng":-8.23621851363219},{"lat":41.0047249257287,"lng":-8.23639696625128},{"lat":41.0053549350847,"lng":-8.23673328855861},{"lat":41.0056852985733,"lng":-8.23694242514112},{"lat":41.0060203063766,"lng":-8.23709653445216},{"lat":41.006210718901,"lng":-8.23712964477019},{"lat":41.0064169797548,"lng":-8.23716551773619},{"lat":41.0066891731013,"lng":-8.23717007714512},{"lat":41.0070819527417,"lng":-8.23708016859869},{"lat":41.0073036224511,"lng":-8.23690507525093},{"lat":41.0074033293492,"lng":-8.23678042574876},{"lat":41.0075912199629,"lng":-8.23652360928441},{"lat":41.0077642838378,"lng":-8.23622659484526},{"lat":41.0077645822636,"lng":-8.23589851695399},{"lat":41.0078005597735,"lng":-8.23564728218087},{"lat":41.0078533491792,"lng":-8.23542353814304},{"lat":41.0079892963161,"lng":-8.23506227197934},{"lat":41.0081047699364,"lng":-8.2343065574745},{"lat":41.008110352736,"lng":-8.23380279596322},{"lat":41.0080506594301,"lng":-8.2331874358912},{"lat":41.0080632350224,"lng":-8.23281574830373},{"lat":41.008195727484,"lng":-8.23256632597505},{"lat":41.00849539191,"lng":-8.23257403842907},{"lat":41.0087218659394,"lng":-8.23266615054444},{"lat":41.0090592292066,"lng":-8.2327060194307},{"lat":41.0094305986937,"lng":-8.23278302603357},{"lat":41.0096297663093,"lng":-8.23279569378708},{"lat":41.0098317360811,"lng":-8.2329000188682},{"lat":41.0101421650697,"lng":-8.23315535899802},{"lat":41.0103772180761,"lng":-8.23342259477784},{"lat":41.0105289006699,"lng":-8.23367496157823},{"lat":41.0106675174949,"lng":-8.23383304017536},{"lat":41.0108119561152,"lng":-8.23402548815943},{"lat":41.0110384002346,"lng":-8.2341502970037},{"lat":41.0113373616075,"lng":-8.23433502567769},{"lat":41.011502057941,"lng":-8.23453772525065},{"lat":41.0117929243157,"lng":-8.2347103130885},{"lat":41.0118238289259,"lng":-8.23459291072687},{"lat":41.0122610640285,"lng":-8.23360750837916},{"lat":41.0125373084504,"lng":-8.23322158381539},{"lat":41.0130365785419,"lng":-8.23274529899056},{"lat":41.0132934879399,"lng":-8.23224453774003},{"lat":41.0134260559592,"lng":-8.23198613634892},{"lat":41.0135385875458,"lng":-8.23170646907927},{"lat":41.0137699515732,"lng":-8.23093874015287},{"lat":41.0138974608489,"lng":-8.23051726517506},{"lat":41.0139887331437,"lng":-8.23003474448694},{"lat":41.0139324098847,"lng":-8.22966173946019},{"lat":41.0138921228773,"lng":-8.22928304495679},{"lat":41.0138856696582,"lng":-8.22914855061495},{"lat":41.0138746952708,"lng":-8.22891983621212},{"lat":41.0147566500744,"lng":-8.22767799902367},{"lat":41.015450340555,"lng":-8.22672985265878},{"lat":41.0157589924158,"lng":-8.22577269172643},{"lat":41.0158615736112,"lng":-8.22531145299416},{"lat":41.0159176265643,"lng":-8.22492206414599},{"lat":41.0159537764654,"lng":-8.22475696711529},{"lat":41.016060879699,"lng":-8.22426781796366},{"lat":41.0160568213391,"lng":-8.2238079969322},{"lat":41.0160061959711,"lng":-8.22317805142875},{"lat":41.0159679361458,"lng":-8.22282954496375},{"lat":41.015937041366,"lng":-8.22255839065411},{"lat":41.0158243322907,"lng":-8.22226499485693},{"lat":41.0156221953387,"lng":-8.22199248401978},{"lat":41.0154844320697,"lng":-8.22175738140107},{"lat":41.0153245049183,"lng":-8.22142524684552},{"lat":41.0153219991956,"lng":-8.22045286999289},{"lat":41.0152471377552,"lng":-8.22029437987711},{"lat":41.0147357238634,"lng":-8.21953184397425},{"lat":41.0143689521059,"lng":-8.21873366430032},{"lat":41.013956533768,"lng":-8.217913794949},{"lat":41.0137266241756,"lng":-8.2174686439023},{"lat":41.0136825672918,"lng":-8.2172592408014},{"lat":41.0136924434799,"lng":-8.21711070998415},{"lat":41.0137088870278,"lng":-8.21686340427881},{"lat":41.0137356739586,"lng":-8.21668226593264},{"lat":41.0137821313639,"lng":-8.21608412333523},{"lat":41.0134106903874,"lng":-8.21608757202488},{"lat":41.0130138073335,"lng":-8.2160749266207},{"lat":41.0126796324415,"lng":-8.21606427933225},{"lat":41.012333799174,"lng":-8.2160257972923},{"lat":41.0119708684839,"lng":-8.21584487502394},{"lat":41.0118323084698,"lng":-8.21569752767252},{"lat":41.0117920452346,"lng":-8.215588110084},{"lat":41.0117684114957,"lng":-8.21540322614808},{"lat":41.0117930484867,"lng":-8.21520699122091},{"lat":41.0123299710166,"lng":-8.21496154120081},{"lat":41.0123439457461,"lng":-8.21454615679985},{"lat":41.0122828210542,"lng":-8.21343587130647},{"lat":41.0123003385215,"lng":-8.21273130729278},{"lat":41.0123328938602,"lng":-8.21241121148681},{"lat":41.0123641748827,"lng":-8.21223317013219},{"lat":41.0124232024189,"lng":-8.21203150725762},{"lat":41.0124526238,"lng":-8.21180805398049},{"lat":41.0124785305502,"lng":-8.21146370259357},{"lat":41.012491916726,"lng":-8.21132931981756},{"lat":41.0125158635254,"lng":-8.21112541653494},{"lat":41.0126070464232,"lng":-8.21090905023886},{"lat":41.012857709541,"lng":-8.21062823038081},{"lat":41.0131717990254,"lng":-8.21027634936593},{"lat":41.013629136375,"lng":-8.20988090529058},{"lat":41.0135618850436,"lng":-8.20927854162994},{"lat":41.0135141274879,"lng":-8.20885079970072},{"lat":41.0135202724714,"lng":-8.20841763286904},{"lat":41.0135257522771,"lng":-8.20803125827835},{"lat":41.0131245978433,"lng":-8.20749286508884},{"lat":41.0129911194604,"lng":-8.20736779310241},{"lat":41.012865341317,"lng":-8.2072499369801},{"lat":41.0124989740134,"lng":-8.20720814217484},{"lat":41.0123495197774,"lng":-8.20668001595821},{"lat":41.0122496795799,"lng":-8.20632721690043},{"lat":41.0118101869738,"lng":-8.20629584738709},{"lat":41.0111899722218,"lng":-8.2062515792527},{"lat":41.0104595636786,"lng":-8.20496039067863},{"lat":41.0103666202641,"lng":-8.20456419599004},{"lat":41.0102888097809,"lng":-8.20444607161457},{"lat":41.0101807986705,"lng":-8.20423447241476},{"lat":41.0100935885145,"lng":-8.20416507363095},{"lat":41.0099811384275,"lng":-8.20413796836606},{"lat":41.0098273858764,"lng":-8.20406563960937},{"lat":41.0096299495615,"lng":-8.20387248995515},{"lat":41.0094738593696,"lng":-8.20377944743005},{"lat":41.0091714766939,"lng":-8.20424250933591},{"lat":41.0089541711091,"lng":-8.20484803994341},{"lat":41.008794006604,"lng":-8.20522645849498},{"lat":41.0086222955852,"lng":-8.2056413645274},{"lat":41.00854662369,"lng":-8.20576074183774},{"lat":41.008238085784,"lng":-8.20624747548582},{"lat":41.0080227009687,"lng":-8.20658725094854},{"lat":41.0075340009863,"lng":-8.20717568564791},{"lat":41.0072214596712,"lng":-8.2070233013653},{"lat":41.0068530632449,"lng":-8.20763013797428},{"lat":41.005863615661,"lng":-8.20860799431263},{"lat":41.0056309638753,"lng":-8.20908876895873},{"lat":41.0054617853863,"lng":-8.20931551720034},{"lat":41.0046268509019,"lng":-8.21043454574408},{"lat":41.004361142551,"lng":-8.21041544987015},{"lat":41.0040944561698,"lng":-8.21050522857248},{"lat":41.0038132103292,"lng":-8.21068841357625},{"lat":41.0036075033638,"lng":-8.21088191039522},{"lat":41.0032599713429,"lng":-8.21156857607483},{"lat":41.0031411459571,"lng":-8.21180335182867},{"lat":41.0025362969192,"lng":-8.21137894005893},{"lat":41.0020884014526,"lng":-8.21106466555109},{"lat":41.00021899989,"lng":-8.20878940785162},{"lat":40.9999174214943,"lng":-8.20894511035584},{"lat":40.9996661815618,"lng":-8.20883249897674},{"lat":40.9995065021216,"lng":-8.20874234028564},{"lat":40.9993158049584,"lng":-8.20860612089339},{"lat":40.9989887654334,"lng":-8.20843817741551},{"lat":40.9987588278936,"lng":-8.20851338323151},{"lat":40.9986318964118,"lng":-8.20864295186282},{"lat":40.9984096641404,"lng":-8.2088648949866},{"lat":40.9982922019968,"lng":-8.20892964962645},{"lat":40.9981684839675,"lng":-8.20892784391754},{"lat":40.9978261098679,"lng":-8.20868634543726},{"lat":40.9974324429672,"lng":-8.20840867039608},{"lat":40.997269263464,"lng":-8.20871633236398},{"lat":40.9967800392212,"lng":-8.20963870732932},{"lat":40.9964368327488,"lng":-8.21010526105003},{"lat":40.9962861386796,"lng":-8.21030260313139},{"lat":40.9961314454902,"lng":-8.21042721095141},{"lat":40.9959290575178,"lng":-8.21050587833437},{"lat":40.9957915675062,"lng":-8.2104977510925},{"lat":40.995639797528,"lng":-8.21055592583294},{"lat":40.995522243862,"lng":-8.2106267371114},{"lat":40.9953236532287,"lng":-8.21133125602384},{"lat":40.9953074124922,"lng":-8.21189053382484},{"lat":40.995211045068,"lng":-8.21243639126385},{"lat":40.9951247412469,"lng":-8.21287055086736},{"lat":40.9950455774963,"lng":-8.213020560934},{"lat":40.9949510814816,"lng":-8.2130704779514},{"lat":40.9948016570617,"lng":-8.21312567296287},{"lat":40.9946271204086,"lng":-8.21316538846157},{"lat":40.9944734052513,"lng":-8.21317517847256},{"lat":40.9943860434798,"lng":-8.21320704003613},{"lat":40.9943210515045,"lng":-8.21328912217443},{"lat":40.9941977280718,"lng":-8.21311113643093},{"lat":40.994090876504,"lng":-8.21286677062617},{"lat":40.9942889341771,"lng":-8.21250098400982},{"lat":40.9941917559279,"lng":-8.20993227259639},{"lat":40.9943511426482,"lng":-8.20899418488408},{"lat":40.9945041787992,"lng":-8.20851874400113},{"lat":40.9944847332332,"lng":-8.20811024670597},{"lat":40.9942792080246,"lng":-8.20695198738548},{"lat":40.9948095843893,"lng":-8.20490247231036},{"lat":40.9900857697032,"lng":-8.20412336486405},{"lat":40.9879414759115,"lng":-8.20421538950171},{"lat":40.98622591194,"lng":-8.20449651493161},{"lat":40.9846531785058,"lng":-8.20496642152924},{"lat":40.9814399255162,"lng":-8.20687022335596},{"lat":40.9789966535392,"lng":-8.20875375074108},{"lat":40.9766720497955,"lng":-8.21126600586502},{"lat":40.97452613221,"lng":-8.21369971396993},{"lat":40.972916166266,"lng":-8.21621252283094},{"lat":40.9713654736253,"lng":-8.2190396248295},{"lat":40.9699337148216,"lng":-8.22202391977954},{"lat":40.9685015646943,"lng":-8.2254009914919},{"lat":40.9672879152292,"lng":-8.22923239232421},{"lat":40.9659263612303,"lng":-8.23337934573623},{"lat":40.963747079082,"lng":-8.24041211525196}]},{"zone_id":"arc19","points":[{"lat":40.9497199974891,"lng":-8.41882062927747},{"lat":40.9491261525642,"lng":-8.41655856242714},{"lat":40.9473791945719,"lng":-8.40730685406982},{"lat":40.9471116383657,"lng":-8.40382704484533},{"lat":40.9469227745086,"lng":-8.40170686168011},{"lat":40.9461475065768,"lng":-8.39994679056564},{"lat":40.9452008810426,"lng":-8.39717789601564},{"lat":40.9397048870653,"lng":-8.37846243313495},{"lat":40.939129894942,"lng":-8.37669629089521},{"lat":40.9356732959254,"lng":-8.37762610071319},{"lat":40.9308474253404,"lng":-8.3782366010822},{"lat":40.9228164146771,"lng":-8.37822809558975},{"lat":40.9065714774975,"lng":-8.37658762604081},{"lat":40.9032232721474,"lng":-8.37624961587487},{"lat":40.9024570795975,"lng":-8.37606343048603},{"lat":40.9016125783681,"lng":-8.38103757486611},{"lat":40.9021218357163,"lng":-8.38238591359606},{"lat":40.903629531348,"lng":-8.38538661732374},{"lat":40.9039002371082,"lng":-8.38592542200049},{"lat":40.904196947553,"lng":-8.38569820675863},{"lat":40.9049520381516,"lng":-8.38519344473161},{"lat":40.9050148424643,"lng":-8.3853315057895},{"lat":40.9050652546412,"lng":-8.3854444386756},{"lat":40.9051285402323,"lng":-8.38558759397049},{"lat":40.9051783206814,"lng":-8.38569685832529},{"lat":40.9052695099526,"lng":-8.38585685630426},{"lat":40.9053636030354,"lng":-8.38600830616704},{"lat":40.9054781306088,"lng":-8.38619195325461},{"lat":40.9055810963113,"lng":-8.38635697054542},{"lat":40.9056855866919,"lng":-8.38644843662203},{"lat":40.9057772557959,"lng":-8.38652565547747},{"lat":40.9057990486116,"lng":-8.38669959071388},{"lat":40.9057726322472,"lng":-8.38685268899997},{"lat":40.9058048784236,"lng":-8.38703511485468},{"lat":40.9058441965063,"lng":-8.38728094704432},{"lat":40.9058622772211,"lng":-8.38740265786531},{"lat":40.9058737931591,"lng":-8.38752077228684},{"lat":40.9058567864803,"lng":-8.38766494559775},{"lat":40.9058321910882,"lng":-8.3877798570099},{"lat":40.9057865261977,"lng":-8.38797872878051},{"lat":40.9057379422551,"lng":-8.38821150987438},{"lat":40.9057182949279,"lng":-8.38833230423556},{"lat":40.9057011864052,"lng":-8.38854193139708},{"lat":40.905726576611,"lng":-8.38876015951313},{"lat":40.9057283737202,"lng":-8.3889568825662},{"lat":40.9057276509429,"lng":-8.38907711729655},{"lat":40.9057361002087,"lng":-8.38920189557274},{"lat":40.9057491215687,"lng":-8.38932235869172},{"lat":40.9057609076048,"lng":-8.38947737847929},{"lat":40.9057698157576,"lng":-8.38962926104351},{"lat":40.9057591829303,"lng":-8.38983176008948},{"lat":40.9057151473522,"lng":-8.39000974459709},{"lat":40.9056624022633,"lng":-8.39023645163607},{"lat":40.905624738562,"lng":-8.39034912700885},{"lat":40.9055836431802,"lng":-8.39046660187504},{"lat":40.9055450657391,"lng":-8.3905969941774},{"lat":40.9055125430475,"lng":-8.39071239945976},{"lat":40.9054473121123,"lng":-8.39081792837507},{"lat":40.9054227425463,"lng":-8.39097224198239},{"lat":40.9054217173987,"lng":-8.3911073022441},{"lat":40.9054164295909,"lng":-8.39145574098101},{"lat":40.9054184032612,"lng":-8.39189761554879},{"lat":40.9054090786913,"lng":-8.39204595201221},{"lat":40.9053913654955,"lng":-8.39242075649167},{"lat":40.905384938108,"lng":-8.39256731606093},{"lat":40.9053782402652,"lng":-8.3926937178245},{"lat":40.9053636925344,"lng":-8.39283513294967},{"lat":40.9053489621149,"lng":-8.39301678203368},{"lat":40.905339805315,"lng":-8.39320284427466},{"lat":40.905329443757,"lng":-8.39340350018012},{"lat":40.9053235553471,"lng":-8.39353250204658},{"lat":40.9053222803345,"lng":-8.39366315575684},{"lat":40.905367521866,"lng":-8.39380506666819},{"lat":40.9054571567135,"lng":-8.3938999996502},{"lat":40.9056257920149,"lng":-8.39407394069859},{"lat":40.9057484758647,"lng":-8.39419728123048},{"lat":40.9058225075565,"lng":-8.39428010181559},{"lat":40.9058990820051,"lng":-8.39437566684891},{"lat":40.9059458257287,"lng":-8.39448186701572},{"lat":40.9059717311401,"lng":-8.39463515382902},{"lat":40.9060044253752,"lng":-8.39482296106297},{"lat":40.9060331971043,"lng":-8.39494761016641},{"lat":40.906087353523,"lng":-8.39504769356805},{"lat":40.9061922240933,"lng":-8.39492813918884},{"lat":40.9063070892577,"lng":-8.39471523251276},{"lat":40.9063795802705,"lng":-8.39455657043658},{"lat":40.9064598000088,"lng":-8.39439937058344},{"lat":40.9066072151836,"lng":-8.39430996348693},{"lat":40.9066993774234,"lng":-8.39433576706513},{"lat":40.9068119318968,"lng":-8.39441441228634},{"lat":40.906964557604,"lng":-8.39457377360982},{"lat":40.9070558347156,"lng":-8.39472086448753},{"lat":40.9071249945314,"lng":-8.3948174999008},{"lat":40.9072066315157,"lng":-8.39495440351755},{"lat":40.907298102141,"lng":-8.39508226443901},{"lat":40.9073966301981,"lng":-8.39523209024063},{"lat":40.9074905250273,"lng":-8.39531403113901},{"lat":40.9075826042783,"lng":-8.39533101509033},{"lat":40.9076855303516,"lng":-8.39532952500261},{"lat":40.9077923034278,"lng":-8.39533944562145},{"lat":40.9078833467219,"lng":-8.39537600843361},{"lat":40.9079742219644,"lng":-8.3953797176489},{"lat":40.9080550248689,"lng":-8.39531521054377},{"lat":40.9081476756631,"lng":-8.39523961750325},{"lat":40.9082358279072,"lng":-8.39517065796542},{"lat":40.9083249639972,"lng":-8.39512756629015},{"lat":40.9084160716286,"lng":-8.39512350716703},{"lat":40.9085092921686,"lng":-8.39512267546131},{"lat":40.9085987800094,"lng":-8.39509506551678},{"lat":40.9087098737279,"lng":-8.39506607180221},{"lat":40.9087970417557,"lng":-8.39502558908062},{"lat":40.9088837877475,"lng":-8.39498648814478},{"lat":40.9089737709432,"lng":-8.3949610112096},{"lat":40.9090705557471,"lng":-8.3949674487453},{"lat":40.909164035089,"lng":-8.39498609630184},{"lat":40.9092916245362,"lng":-8.39516946939825},{"lat":40.9093656124888,"lng":-8.39527580629665},{"lat":40.909473354421,"lng":-8.39543052685414},{"lat":40.9095550819213,"lng":-8.39556378837528},{"lat":40.9096724255054,"lng":-8.39575452045553},{"lat":40.909731904308,"lng":-8.39585113682766},{"lat":40.9099657033149,"lng":-8.39614809067451},{"lat":40.9100104275701,"lng":-8.39600381982059},{"lat":40.9100543149944,"lng":-8.39587105768237},{"lat":40.9101166559625,"lng":-8.39570181149419},{"lat":40.9102002761047,"lng":-8.39559852496024},{"lat":40.9103215727475,"lng":-8.39552027397738},{"lat":40.9104443201565,"lng":-8.39555118926737},{"lat":40.910544478055,"lng":-8.39557900640363},{"lat":40.9106446365751,"lng":-8.3956069423231},{"lat":40.9108491902007,"lng":-8.39566010234671},{"lat":40.9109754405556,"lng":-8.39569728256724},{"lat":40.9111438045077,"lng":-8.39574626186443},{"lat":40.9113858713301,"lng":-8.39581547782736},{"lat":40.9115626091158,"lng":-8.39586508555906},{"lat":40.9117477113987,"lng":-8.39591864371135},{"lat":40.9119174279126,"lng":-8.39596715590101},{"lat":40.9120610478826,"lng":-8.39600808577162},{"lat":40.912192699543,"lng":-8.39604520529546},{"lat":40.9123349695929,"lng":-8.39608645100531},{"lat":40.9124786839723,"lng":-8.39612583986682},{"lat":40.9127234652875,"lng":-8.39619354528796},{"lat":40.9127915867045,"lng":-8.39628650090719},{"lat":40.9128638359329,"lng":-8.39637773734533},{"lat":40.9129409812178,"lng":-8.39647891507199},{"lat":40.9130030857595,"lng":-8.39656615587025},{"lat":40.9130655567635,"lng":-8.3966577201873},{"lat":40.9132017315739,"lng":-8.39672424035729},{"lat":40.9132951402688,"lng":-8.3967587388214},{"lat":40.913381825454,"lng":-8.39672401203532},{"lat":40.913525864238,"lng":-8.39666333381321},{"lat":40.9136223052174,"lng":-8.39662134930127},{"lat":40.9136850763901,"lng":-8.39670733251772},{"lat":40.9138016346455,"lng":-8.3970074803468},{"lat":40.913907249917,"lng":-8.3970502438142},{"lat":40.9138889865212,"lng":-8.39716995265374},{"lat":40.9140104119268,"lng":-8.39720888606972},{"lat":40.9141090460144,"lng":-8.39747858888554},{"lat":40.9142464018304,"lng":-8.39764369268769},{"lat":40.9143777458824,"lng":-8.39766889192839},{"lat":40.9144703716897,"lng":-8.39793004375398},{"lat":40.9146854974345,"lng":-8.3980421575884},{"lat":40.9147833778966,"lng":-8.39829372620604},{"lat":40.9148999363301,"lng":-8.39843626920533},{"lat":40.9150075812256,"lng":-8.39841815306803},{"lat":40.9150494400092,"lng":-8.39861985366657},{"lat":40.9151853472266,"lng":-8.39864151479305},{"lat":40.9152755549456,"lng":-8.39863590420869},{"lat":40.915358407588,"lng":-8.39868423336974},{"lat":40.9154397032787,"lng":-8.39875389923642},{"lat":40.9155261541221,"lng":-8.3987971009334},{"lat":40.9156268164331,"lng":-8.39880210641661},{"lat":40.915733526012,"lng":-8.39880729928283},{"lat":40.9158827086231,"lng":-8.39881787139796},{"lat":40.9159833623465,"lng":-8.39882551901647},{"lat":40.9161044775363,"lng":-8.39882458307137},{"lat":40.9162375516233,"lng":-8.39883081736927},{"lat":40.9163305253872,"lng":-8.39884971019668},{"lat":40.9164223464012,"lng":-8.39888202406569},{"lat":40.9165445456092,"lng":-8.39893616061412},{"lat":40.9166719715827,"lng":-8.39900932199185},{"lat":40.916767650174,"lng":-8.3990653813107},{"lat":40.9168612815632,"lng":-8.39911146118741},{"lat":40.9169696460814,"lng":-8.39917255738523},{"lat":40.9170683749428,"lng":-8.39923349643282},{"lat":40.9171787550357,"lng":-8.3993182239369},{"lat":40.917309443617,"lng":-8.39941944466806},{"lat":40.9173966195129,"lng":-8.39946467679248},{"lat":40.9174872685303,"lng":-8.39949803038342},{"lat":40.9175759311275,"lng":-8.39951985164546},{"lat":40.917629463099,"lng":-8.39961705140432},{"lat":40.9176978136373,"lng":-8.39973081254202},{"lat":40.917774076919,"lng":-8.39981053631044},{"lat":40.9178592810959,"lng":-8.39991783667408},{"lat":40.9179708209548,"lng":-8.40004554369222},{"lat":40.9180687324223,"lng":-8.40005767178642},{"lat":40.9183514646,"lng":-8.39992208661827},{"lat":40.9186156197287,"lng":-8.39982403403876},{"lat":40.9187224873609,"lng":-8.39987028820663},{"lat":40.9188087163513,"lng":-8.39994510695134},{"lat":40.9188826833392,"lng":-8.40002336038181},{"lat":40.9189587714012,"lng":-8.4001154990489},{"lat":40.9190237201369,"lng":-8.40020579532071},{"lat":40.9191745208212,"lng":-8.40037176950238},{"lat":40.9192674660003,"lng":-8.4004417111427},{"lat":40.9193589014931,"lng":-8.40046368741568},{"lat":40.9194516406955,"lng":-8.40049505357875},{"lat":40.919541869144,"lng":-8.40054361024402},{"lat":40.919637137548,"lng":-8.40058682714147},{"lat":40.9197650205781,"lng":-8.40065218766903},{"lat":40.9198442844145,"lng":-8.40072165840658},{"lat":40.9199226253454,"lng":-8.40079611163834},{"lat":40.920026730936,"lng":-8.40088631614138},{"lat":40.9201410610233,"lng":-8.40096774260023},{"lat":40.9202180450451,"lng":-8.40103687368561},{"lat":40.9204058928325,"lng":-8.40118614632535},{"lat":40.9204937699967,"lng":-8.40124454720039},{"lat":40.9206676967301,"lng":-8.40136160079947},{"lat":40.9207923196559,"lng":-8.4014390337958},{"lat":40.9208965958612,"lng":-8.40151472277315},{"lat":40.9223898859148,"lng":-8.40295599801327},{"lat":40.9224767414576,"lng":-8.40304191416064},{"lat":40.9225831461981,"lng":-8.40314990618636},{"lat":40.9226627617952,"lng":-8.4032390333004},{"lat":40.9227578249484,"lng":-8.40336466868096},{"lat":40.9228057539456,"lng":-8.4034686232315},{"lat":40.9228657337705,"lng":-8.40357915772862},{"lat":40.9229364432378,"lng":-8.40368368038173},{"lat":40.9230533901258,"lng":-8.40384495680872},{"lat":40.9231177284556,"lng":-8.40394109226553},{"lat":40.9231738763299,"lng":-8.40403475286042},{"lat":40.9232391202933,"lng":-8.40416537341975},{"lat":40.9233055747,"lng":-8.40427937738719},{"lat":40.9233777595152,"lng":-8.40436930629248},{"lat":40.9234551526459,"lng":-8.40446495639116},{"lat":40.9235321877479,"lng":-8.40455965430892},{"lat":40.9236094746493,"lng":-8.4046605987893},{"lat":40.9236835384192,"lng":-8.40475663770642},{"lat":40.9238302063881,"lng":-8.40495395897497},{"lat":40.9238938373073,"lng":-8.40504302204048},{"lat":40.9240117434553,"lng":-8.40521790724784},{"lat":40.9240691796009,"lng":-8.40530972819083},{"lat":40.9241546673809,"lng":-8.4054367738918},{"lat":40.9242055638694,"lng":-8.40554279931335},{"lat":40.9242583399467,"lng":-8.40565198241525},{"lat":40.9243066466863,"lng":-8.40577175420949},{"lat":40.924385192921,"lng":-8.40597291572358},{"lat":40.9244725222804,"lng":-8.40603795208789},{"lat":40.924627676424,"lng":-8.40609796242729},{"lat":40.9247214707012,"lng":-8.40607300974942},{"lat":40.9248971507191,"lng":-8.40605680559966},{"lat":40.9249972540391,"lng":-8.40605660858068},{"lat":40.9250957264515,"lng":-8.40605976274101},{"lat":40.925189368243,"lng":-8.40608542948108},{"lat":40.9252731714369,"lng":-8.40613623966161},{"lat":40.9254006984617,"lng":-8.40618889643565},{"lat":40.925562594484,"lng":-8.40630045935192},{"lat":40.9257047378943,"lng":-8.40641354356053},{"lat":40.925801667258,"lng":-8.40647174338787},{"lat":40.9258838712683,"lng":-8.40652183011155},{"lat":40.9259722375245,"lng":-8.40658218996879},{"lat":40.9260615216404,"lng":-8.40664933461268},{"lat":40.9261622512341,"lng":-8.40672008431634},{"lat":40.9262549464936,"lng":-8.40677982979147},{"lat":40.92635751972,"lng":-8.40685065872324},{"lat":40.9264393977693,"lng":-8.40691522879144},{"lat":40.9265150549089,"lng":-8.40698262153407},{"lat":40.9267346562304,"lng":-8.40721753946064},{"lat":40.9268873974841,"lng":-8.4073890511788},{"lat":40.9270021346317,"lng":-8.40751765833018},{"lat":40.927076447263,"lng":-8.40760362799247},{"lat":40.9271472471858,"lng":-8.40770817203738},{"lat":40.9272021251567,"lng":-8.40780265845943},{"lat":40.9272954983948,"lng":-8.40790860995239},{"lat":40.9273790687961,"lng":-8.40798908210546},{"lat":40.9274510934703,"lng":-8.40806803406211},{"lat":40.9275205913639,"lng":-8.40815161344386},{"lat":40.927591864628,"lng":-8.40824451667509},{"lat":40.9276692516734,"lng":-8.40834171962552},{"lat":40.9277189090946,"lng":-8.40844096021711},{"lat":40.9278091901966,"lng":-8.40860021634708},{"lat":40.9278699689949,"lng":-8.40870743079146},{"lat":40.9278793064308,"lng":-8.40883118174522},{"lat":40.9280048942251,"lng":-8.41029654615833},{"lat":40.9279998556461,"lng":-8.41062420301177},{"lat":40.9279969665355,"lng":-8.41081205344062},{"lat":40.9279825490847,"lng":-8.41174918093324},{"lat":40.9278278901064,"lng":-8.41273194989534},{"lat":40.9277590116668,"lng":-8.41316961421527},{"lat":40.9276046536437,"lng":-8.41358188434503},{"lat":40.9284112824421,"lng":-8.41512401397534},{"lat":40.9297821732361,"lng":-8.41827703276632},{"lat":40.9302255038092,"lng":-8.41869735519584},{"lat":40.9314253130417,"lng":-8.41971843311033},{"lat":40.9332557644854,"lng":-8.42127552974447},{"lat":40.9340903257275,"lng":-8.42205643111367},{"lat":40.9340215788415,"lng":-8.42270895576157},{"lat":40.9337996887288,"lng":-8.4248148213887},{"lat":40.9340629817877,"lng":-8.4256115659135},{"lat":40.9344966580907,"lng":-8.42692395317739},{"lat":40.9345923820948,"lng":-8.42800821563464},{"lat":40.9344800347524,"lng":-8.42820280358766},{"lat":40.9343348716649,"lng":-8.42845422728013},{"lat":40.9347936300796,"lng":-8.4295944262802},{"lat":40.935693330173,"lng":-8.43183067022664},{"lat":40.9357124353603,"lng":-8.43191795122648},{"lat":40.9358243189791,"lng":-8.43192199503684},{"lat":40.9360621874191,"lng":-8.43184327679757},{"lat":40.936324541323,"lng":-8.43169047080368},{"lat":40.9365134354886,"lng":-8.43151914287114},{"lat":40.9366358666037,"lng":-8.4313107710613},{"lat":40.9367687915575,"lng":-8.43103757246617},{"lat":40.9369262023411,"lng":-8.43076437387104},{"lat":40.9370661227227,"lng":-8.43050969721456},{"lat":40.9372060428079,"lng":-8.43029669492006},{"lat":40.9373669505395,"lng":-8.43011610601819},{"lat":40.9375733316216,"lng":-8.42988458178503},{"lat":40.9377937039295,"lng":-8.42960675270524},{"lat":40.9379651041052,"lng":-8.42941227234938},{"lat":40.9381260099879,"lng":-8.42923631393218},{"lat":40.9381680519302,"lng":-8.42904810343174},{"lat":40.9383049941047,"lng":-8.4290523969509},{"lat":40.9385388861928,"lng":-8.42903091473174},{"lat":40.9387134185164,"lng":-8.42901819796846},{"lat":40.9389564098666,"lng":-8.42899393002519},{"lat":40.9391791576302,"lng":-8.42892567141055},{"lat":40.9394165985922,"lng":-8.42886653149313},{"lat":40.9396061710121,"lng":-8.42886780420099},{"lat":40.9398691040849,"lng":-8.42895322222924},{"lat":40.9400824938965,"lng":-8.42913744067577},{"lat":40.9401578675359,"lng":-8.42924675239943},{"lat":40.9403295044514,"lng":-8.42961607242742},{"lat":40.9404435917299,"lng":-8.42989457588682},{"lat":40.940549370771,"lng":-8.43018557757841},{"lat":40.9405993991242,"lng":-8.43030452126453},{"lat":40.9406332573478,"lng":-8.43047066694342},{"lat":40.9406854714587,"lng":-8.43057072047366},{"lat":40.9407687316161,"lng":-8.4307302656176},{"lat":40.9408356837578,"lng":-8.43081319374209},{"lat":40.9409370290069,"lng":-8.43093872224924},{"lat":40.9411123659802,"lng":-8.43113913417785},{"lat":40.9412059959002,"lng":-8.4312257359009},{"lat":40.9413105314984,"lng":-8.43132242504594},{"lat":40.9413960915335,"lng":-8.43139896134432},{"lat":40.9415093393505,"lng":-8.43150026566161},{"lat":40.941622980055,"lng":-8.43160482376041},{"lat":40.9417080430722,"lng":-8.43168308850547},{"lat":40.941891538661,"lng":-8.4317935299641},{"lat":40.9420251090974,"lng":-8.43181717437041},{"lat":40.9422440633009,"lng":-8.43193525558345},{"lat":40.942346572333,"lng":-8.43200798344835},{"lat":40.9424355932953,"lng":-8.43207114204034},{"lat":40.9427395838045,"lng":-8.43224625975037},{"lat":40.9429790366562,"lng":-8.43234798336155},{"lat":40.9431424282252,"lng":-8.43239273180316},{"lat":40.9434162234703,"lng":-8.43245589592586},{"lat":40.9436120487576,"lng":-8.4325176563391},{"lat":40.9437148935046,"lng":-8.43255087110701},{"lat":40.9438548652311,"lng":-8.43237332408613},{"lat":40.944886312275,"lng":-8.43106495675323},{"lat":40.945136040731,"lng":-8.43074817256776},{"lat":40.9458987053457,"lng":-8.42971842371364},{"lat":40.9458116820235,"lng":-8.42954420361369},{"lat":40.9457986088013,"lng":-8.42944640486914},{"lat":40.9458380971008,"lng":-8.42932319613754},{"lat":40.9458667051602,"lng":-8.42927213273934},{"lat":40.9459262990961,"lng":-8.42916576297374},{"lat":40.9460497545849,"lng":-8.42903593322114},{"lat":40.9461504025198,"lng":-8.42893008564272},{"lat":40.9463823050104,"lng":-8.42867697267791},{"lat":40.9464746498438,"lng":-8.42858429118744},{"lat":40.9465868542971,"lng":-8.4284556369756},{"lat":40.946731933786,"lng":-8.42825152133831},{"lat":40.9468540590968,"lng":-8.42814729672057},{"lat":40.9469662675474,"lng":-8.42810789602193},{"lat":40.9471178857433,"lng":-8.42808054786465},{"lat":40.947277573138,"lng":-8.42804620286779},{"lat":40.9474092337873,"lng":-8.42801779956562},{"lat":40.947548814931,"lng":-8.4279523165553},{"lat":40.9477474739865,"lng":-8.42780929345917},{"lat":40.9478376770503,"lng":-8.42774435253547},{"lat":40.9479625906965,"lng":-8.42760609477419},{"lat":40.9480305928661,"lng":-8.42751592650412},{"lat":40.9481076815799,"lng":-8.42736513574921},{"lat":40.9481563083386,"lng":-8.42727001809662},{"lat":40.9482124542511,"lng":-8.42723373688834},{"lat":40.9490546072284,"lng":-8.42424647157619},{"lat":40.9487701642549,"lng":-8.42132077301543},{"lat":40.9497199974891,"lng":-8.41882062927747}]},{"zone_id":"arc2","points":[{"lat":40.9497199974891,"lng":-8.41882062927747},{"lat":40.9491261525642,"lng":-8.41655856242714},{"lat":40.9473791945719,"lng":-8.40730685406982},{"lat":40.9471116383657,"lng":-8.40382704484533},{"lat":40.9469227745086,"lng":-8.40170686168011},{"lat":40.9461475065768,"lng":-8.39994679056564},{"lat":40.9452008810426,"lng":-8.39717789601564},{"lat":40.9397048870653,"lng":-8.37846243313495},{"lat":40.939129894942,"lng":-8.37669629089521},{"lat":40.9356732959254,"lng":-8.37762610071319},{"lat":40.9308474253404,"lng":-8.3782366010822},{"lat":40.9228164146771,"lng":-8.37822809558975},{"lat":40.9065714774975,"lng":-8.37658762604081},{"lat":40.9032232721474,"lng":-8.37624961587487},{"lat":40.9024570795975,"lng":-8.37606343048603},{"lat":40.9016125783681,"lng":-8.38103757486611},{"lat":40.9021218357163,"lng":-8.38238591359606},{"lat":40.903629531348,"lng":-8.38538661732374},{"lat":40.9039002371082,"lng":-8.38592542200049},{"lat":40.904196947553,"lng":-8.38569820675863},{"lat":40.9049520381516,"lng":-8.38519344473161},{"lat":40.9050148424643,"lng":-8.3853315057895},{"lat":40.9050652546412,"lng":-8.3854444386756},{"lat":40.9051285402323,"lng":-8.38558759397049},{"lat":40.9051783206814,"lng":-8.38569685832529},{"lat":40.9052695099526,"lng":-8.38585685630426},{"lat":40.9053636030354,"lng":-8.38600830616704},{"lat":40.9054781306088,"lng":-8.38619195325461},{"lat":40.9055810963113,"lng":-8.38635697054542},{"lat":40.9056855866919,"lng":-8.38644843662203},{"lat":40.9057772557959,"lng":-8.38652565547747},{"lat":40.9057990486116,"lng":-8.38669959071388},{"lat":40.9057726322472,"lng":-8.38685268899997},{"lat":40.9058048784236,"lng":-8.38703511485468},{"lat":40.9058441965063,"lng":-8.38728094704432},{"lat":40.9058622772211,"lng":-8.38740265786531},{"lat":40.9058737931591,"lng":-8.38752077228684},{"lat":40.9058567864803,"lng":-8.38766494559775},{"lat":40.9058321910882,"lng":-8.3877798570099},{"lat":40.9057865261977,"lng":-8.38797872878051},{"lat":40.9057379422551,"lng":-8.38821150987438},{"lat":40.9057182949279,"lng":-8.38833230423556},{"lat":40.9057011864052,"lng":-8.38854193139708},{"lat":40.905726576611,"lng":-8.38876015951313},{"lat":40.9057283737202,"lng":-8.3889568825662},{"lat":40.9057276509429,"lng":-8.38907711729655},{"lat":40.9057361002087,"lng":-8.38920189557274},{"lat":40.9057491215687,"lng":-8.38932235869172},{"lat":40.9057609076048,"lng":-8.38947737847929},{"lat":40.9057698157576,"lng":-8.38962926104351},{"lat":40.9057591829303,"lng":-8.38983176008948},{"lat":40.9057151473522,"lng":-8.39000974459709},{"lat":40.9056624022633,"lng":-8.39023645163607},{"lat":40.905624738562,"lng":-8.39034912700885},{"lat":40.9055836431802,"lng":-8.39046660187504},{"lat":40.9055450657391,"lng":-8.3905969941774},{"lat":40.9055125430475,"lng":-8.39071239945976},{"lat":40.9054473121123,"lng":-8.39081792837507},{"lat":40.9054227425463,"lng":-8.39097224198239},{"lat":40.9054217173987,"lng":-8.3911073022441},{"lat":40.9054164295909,"lng":-8.39145574098101},{"lat":40.9054184032612,"lng":-8.39189761554879},{"lat":40.9054090786913,"lng":-8.39204595201221},{"lat":40.9053913654955,"lng":-8.39242075649167},{"lat":40.905384938108,"lng":-8.39256731606093},{"lat":40.9053782402652,"lng":-8.3926937178245},{"lat":40.9053636925344,"lng":-8.39283513294967},{"lat":40.9053489621149,"lng":-8.39301678203368},{"lat":40.905339805315,"lng":-8.39320284427466},{"lat":40.905329443757,"lng":-8.39340350018012},{"lat":40.9053235553471,"lng":-8.39353250204658},{"lat":40.9053222803345,"lng":-8.39366315575684},{"lat":40.905367521866,"lng":-8.39380506666819},{"lat":40.9054571567135,"lng":-8.3938999996502},{"lat":40.9056257920149,"lng":-8.39407394069859},{"lat":40.9057484758647,"lng":-8.39419728123048},{"lat":40.9058225075565,"lng":-8.39428010181559},{"lat":40.9058990820051,"lng":-8.39437566684891},{"lat":40.9059458257287,"lng":-8.39448186701572},{"lat":40.9059717311401,"lng":-8.39463515382902},{"lat":40.9060044253752,"lng":-8.39482296106297},{"lat":40.9060331971043,"lng":-8.39494761016641},{"lat":40.906087353523,"lng":-8.39504769356805},{"lat":40.9061922240933,"lng":-8.39492813918884},{"lat":40.9063070892577,"lng":-8.39471523251276},{"lat":40.9063795802705,"lng":-8.39455657043658},{"lat":40.9064598000088,"lng":-8.39439937058344},{"lat":40.9066072151836,"lng":-8.39430996348693},{"lat":40.9066993774234,"lng":-8.39433576706513},{"lat":40.9068119318968,"lng":-8.39441441228634},{"lat":40.906964557604,"lng":-8.39457377360982},{"lat":40.9070558347156,"lng":-8.39472086448753},{"lat":40.9071249945314,"lng":-8.3948174999008},{"lat":40.9072066315157,"lng":-8.39495440351755},{"lat":40.907298102141,"lng":-8.39508226443901},{"lat":40.9073966301981,"lng":-8.39523209024063},{"lat":40.9074905250273,"lng":-8.39531403113901},{"lat":40.9075826042783,"lng":-8.39533101509033},{"lat":40.9076855303516,"lng":-8.39532952500261},{"lat":40.9077923034278,"lng":-8.39533944562145},{"lat":40.9078833467219,"lng":-8.39537600843361},{"lat":40.9079742219644,"lng":-8.3953797176489},{"lat":40.9080550248689,"lng":-8.39531521054377},{"lat":40.9081476756631,"lng":-8.39523961750325},{"lat":40.9082358279072,"lng":-8.39517065796542},{"lat":40.9083249639972,"lng":-8.39512756629015},{"lat":40.9084160716286,"lng":-8.39512350716703},{"lat":40.9085092921686,"lng":-8.39512267546131},{"lat":40.9085987800094,"lng":-8.39509506551678},{"lat":40.9087098737279,"lng":-8.39506607180221},{"lat":40.9087970417557,"lng":-8.39502558908062},{"lat":40.9088837877475,"lng":-8.39498648814478},{"lat":40.9089737709432,"lng":-8.3949610112096},{"lat":40.9090705557471,"lng":-8.3949674487453},{"lat":40.909164035089,"lng":-8.39498609630184},{"lat":40.9092916245362,"lng":-8.39516946939825},{"lat":40.9093656124888,"lng":-8.39527580629665},{"lat":40.909473354421,"lng":-8.39543052685414},{"lat":40.9095550819213,"lng":-8.39556378837528},{"lat":40.9096724255054,"lng":-8.39575452045553},{"lat":40.909731904308,"lng":-8.39585113682766},{"lat":40.9099657033149,"lng":-8.39614809067451},{"lat":40.9100104275701,"lng":-8.39600381982059},{"lat":40.9100543149944,"lng":-8.39587105768237},{"lat":40.9101166559625,"lng":-8.39570181149419},{"lat":40.9102002761047,"lng":-8.39559852496024},{"lat":40.9103215727475,"lng":-8.39552027397738},{"lat":40.9104443201565,"lng":-8.39555118926737},{"lat":40.910544478055,"lng":-8.39557900640363},{"lat":40.9106446365751,"lng":-8.3956069423231},{"lat":40.9108491902007,"lng":-8.39566010234671},{"lat":40.9109754405556,"lng":-8.39569728256724},{"lat":40.9111438045077,"lng":-8.39574626186443},{"lat":40.9113858713301,"lng":-8.39581547782736},{"lat":40.9115626091158,"lng":-8.39586508555906},{"lat":40.9117477113987,"lng":-8.39591864371135},{"lat":40.9119174279126,"lng":-8.39596715590101},{"lat":40.9120610478826,"lng":-8.39600808577162},{"lat":40.912192699543,"lng":-8.39604520529546},{"lat":40.9123349695929,"lng":-8.39608645100531},{"lat":40.9124786839723,"lng":-8.39612583986682},{"lat":40.9127234652875,"lng":-8.39619354528796},{"lat":40.9127915867045,"lng":-8.39628650090719},{"lat":40.9128638359329,"lng":-8.39637773734533},{"lat":40.9129409812178,"lng":-8.39647891507199},{"lat":40.9130030857595,"lng":-8.39656615587025},{"lat":40.9130655567635,"lng":-8.3966577201873},{"lat":40.9132017315739,"lng":-8.39672424035729},{"lat":40.9132951402688,"lng":-8.3967587388214},{"lat":40.913381825454,"lng":-8.39672401203532},{"lat":40.913525864238,"lng":-8.39666333381321},{"lat":40.9136223052174,"lng":-8.39662134930127},{"lat":40.9136850763901,"lng":-8.39670733251772},{"lat":40.9138016346455,"lng":-8.3970074803468},{"lat":40.913907249917,"lng":-8.3970502438142},{"lat":40.9138889865212,"lng":-8.39716995265374},{"lat":40.9140104119268,"lng":-8.39720888606972},{"lat":40.9141090460144,"lng":-8.39747858888554},{"lat":40.9142464018304,"lng":-8.39764369268769},{"lat":40.9143777458824,"lng":-8.39766889192839},{"lat":40.9144703716897,"lng":-8.39793004375398},{"lat":40.9146854974345,"lng":-8.3980421575884},{"lat":40.9147833778966,"lng":-8.39829372620604},{"lat":40.9148999363301,"lng":-8.39843626920533},{"lat":40.9150075812256,"lng":-8.39841815306803},{"lat":40.9150494400092,"lng":-8.39861985366657},{"lat":40.9151853472266,"lng":-8.39864151479305},{"lat":40.9152755549456,"lng":-8.39863590420869},{"lat":40.915358407588,"lng":-8.39868423336974},{"lat":40.9154397032787,"lng":-8.39875389923642},{"lat":40.9155261541221,"lng":-8.3987971009334},{"lat":40.9156268164331,"lng":-8.39880210641661},{"lat":40.915733526012,"lng":-8.39880729928283},{"lat":40.9158827086231,"lng":-8.39881787139796},{"lat":40.9159833623465,"lng":-8.39882551901647},{"lat":40.9161044775363,"lng":-8.39882458307137},{"lat":40.9162375516233,"lng":-8.39883081736927},{"lat":40.9163305253872,"lng":-8.39884971019668},{"lat":40.9164223464012,"lng":-8.39888202406569},{"lat":40.9165445456092,"lng":-8.39893616061412},{"lat":40.9166719715827,"lng":-8.39900932199185},{"lat":40.916767650174,"lng":-8.3990653813107},{"lat":40.9168612815632,"lng":-8.39911146118741},{"lat":40.9169696460814,"lng":-8.39917255738523},{"lat":40.9170683749428,"lng":-8.39923349643282},{"lat":40.9171787550357,"lng":-8.3993182239369},{"lat":40.917309443617,"lng":-8.39941944466806},{"lat":40.9173966195129,"lng":-8.39946467679248},{"lat":40.9174872685303,"lng":-8.39949803038342},{"lat":40.9175759311275,"lng":-8.39951985164546},{"lat":40.917629463099,"lng":-8.39961705140432},{"lat":40.9176978136373,"lng":-8.39973081254202},{"lat":40.917774076919,"lng":-8.39981053631044},{"lat":40.9178592810959,"lng":-8.39991783667408},{"lat":40.9179708209548,"lng":-8.40004554369222},{"lat":40.9180687324223,"lng":-8.40005767178642},{"lat":40.9183514646,"lng":-8.39992208661827},{"lat":40.9186156197287,"lng":-8.39982403403876},{"lat":40.9187224873609,"lng":-8.39987028820663},{"lat":40.9188087163513,"lng":-8.39994510695134},{"lat":40.9188826833392,"lng":-8.40002336038181},{"lat":40.9189587714012,"lng":-8.4001154990489},{"lat":40.9190237201369,"lng":-8.40020579532071},{"lat":40.9191745208212,"lng":-8.40037176950238},{"lat":40.9192674660003,"lng":-8.4004417111427},{"lat":40.9193589014931,"lng":-8.40046368741568},{"lat":40.9194516406955,"lng":-8.40049505357875},{"lat":40.919541869144,"lng":-8.40054361024402},{"lat":40.919637137548,"lng":-8.40058682714147},{"lat":40.9197650205781,"lng":-8.40065218766903},{"lat":40.9198442844145,"lng":-8.40072165840658},{"lat":40.9199226253454,"lng":-8.40079611163834},{"lat":40.920026730936,"lng":-8.40088631614138},{"lat":40.9201410610233,"lng":-8.40096774260023},{"lat":40.9202180450451,"lng":-8.40103687368561},{"lat":40.9204058928325,"lng":-8.40118614632535},{"lat":40.9204937699967,"lng":-8.40124454720039},{"lat":40.9206676967301,"lng":-8.40136160079947},{"lat":40.9207923196559,"lng":-8.4014390337958},{"lat":40.9208965958612,"lng":-8.40151472277315},{"lat":40.9223898859148,"lng":-8.40295599801327},{"lat":40.9224767414576,"lng":-8.40304191416064},{"lat":40.9225831461981,"lng":-8.40314990618636},{"lat":40.9226627617952,"lng":-8.4032390333004},{"lat":40.9227578249484,"lng":-8.40336466868096},{"lat":40.9228057539456,"lng":-8.4034686232315},{"lat":40.9228657337705,"lng":-8.40357915772862},{"lat":40.9229364432378,"lng":-8.40368368038173},{"lat":40.9230533901258,"lng":-8.40384495680872},{"lat":40.9231177284556,"lng":-8.40394109226553},{"lat":40.9231738763299,"lng":-8.40403475286042},{"lat":40.9232391202933,"lng":-8.40416537341975},{"lat":40.9233055747,"lng":-8.40427937738719},{"lat":40.9233777595152,"lng":-8.40436930629248},{"lat":40.9234551526459,"lng":-8.40446495639116},{"lat":40.9235321877479,"lng":-8.40455965430892},{"lat":40.9236094746493,"lng":-8.4046605987893},{"lat":40.9236835384192,"lng":-8.40475663770642},{"lat":40.9238302063881,"lng":-8.40495395897497},{"lat":40.9238938373073,"lng":-8.40504302204048},{"lat":40.9240117434553,"lng":-8.40521790724784},{"lat":40.9240691796009,"lng":-8.40530972819083},{"lat":40.9241546673809,"lng":-8.4054367738918},{"lat":40.9242055638694,"lng":-8.40554279931335},{"lat":40.9242583399467,"lng":-8.40565198241525},{"lat":40.9243066466863,"lng":-8.40577175420949},{"lat":40.924385192921,"lng":-8.40597291572358},{"lat":40.9244725222804,"lng":-8.40603795208789},{"lat":40.924627676424,"lng":-8.40609796242729},{"lat":40.9247214707012,"lng":-8.40607300974942},{"lat":40.9248971507191,"lng":-8.40605680559966},{"lat":40.9249972540391,"lng":-8.40605660858068},{"lat":40.9250957264515,"lng":-8.40605976274101},{"lat":40.925189368243,"lng":-8.40608542948108},{"lat":40.9252731714369,"lng":-8.40613623966161},{"lat":40.9254006984617,"lng":-8.40618889643565},{"lat":40.925562594484,"lng":-8.40630045935192},{"lat":40.9257047378943,"lng":-8.40641354356053},{"lat":40.925801667258,"lng":-8.40647174338787},{"lat":40.9258838712683,"lng":-8.40652183011155},{"lat":40.9259722375245,"lng":-8.40658218996879},{"lat":40.9260615216404,"lng":-8.40664933461268},{"lat":40.9261622512341,"lng":-8.40672008431634},{"lat":40.9262549464936,"lng":-8.40677982979147},{"lat":40.92635751972,"lng":-8.40685065872324},{"lat":40.9264393977693,"lng":-8.40691522879144},{"lat":40.9265150549089,"lng":-8.40698262153407},{"lat":40.9267346562304,"lng":-8.40721753946064},{"lat":40.9268873974841,"lng":-8.4073890511788},{"lat":40.9270021346317,"lng":-8.40751765833018},{"lat":40.927076447263,"lng":-8.40760362799247},{"lat":40.9271472471858,"lng":-8.40770817203738},{"lat":40.9272021251567,"lng":-8.40780265845943},{"lat":40.9272954983948,"lng":-8.40790860995239},{"lat":40.9273790687961,"lng":-8.40798908210546},{"lat":40.9274510934703,"lng":-8.40806803406211},{"lat":40.9275205913639,"lng":-8.40815161344386},{"lat":40.927591864628,"lng":-8.40824451667509},{"lat":40.9276692516734,"lng":-8.40834171962552},{"lat":40.9277189090946,"lng":-8.40844096021711},{"lat":40.9278091901966,"lng":-8.40860021634708},{"lat":40.9278699689949,"lng":-8.40870743079146},{"lat":40.9278793064308,"lng":-8.40883118174522},{"lat":40.9280048942251,"lng":-8.41029654615833},{"lat":40.9279998556461,"lng":-8.41062420301177},{"lat":40.9279969665355,"lng":-8.41081205344062},{"lat":40.9279825490847,"lng":-8.41174918093324},{"lat":40.9278278901064,"lng":-8.41273194989534},{"lat":40.9277590116668,"lng":-8.41316961421527},{"lat":40.9276046536437,"lng":-8.41358188434503},{"lat":40.9284112824421,"lng":-8.41512401397534},{"lat":40.9297821732361,"lng":-8.41827703276632},{"lat":40.9302255038092,"lng":-8.41869735519584},{"lat":40.9314253130417,"lng":-8.41971843311033},{"lat":40.9332557644854,"lng":-8.42127552974447},{"lat":40.9340903257275,"lng":-8.42205643111367},{"lat":40.9340215788415,"lng":-8.42270895576157},{"lat":40.9337996887288,"lng":-8.4248148213887},{"lat":40.9340629817877,"lng":-8.4256115659135},{"lat":40.9344966580907,"lng":-8.42692395317739},{"lat":40.9345923820948,"lng":-8.42800821563464},{"lat":40.9344800347524,"lng":-8.42820280358766},{"lat":40.9343348716649,"lng":-8.42845422728013},{"lat":40.9347936300796,"lng":-8.4295944262802},{"lat":40.935693330173,"lng":-8.43183067022664},{"lat":40.9357124353603,"lng":-8.43191795122648},{"lat":40.9358243189791,"lng":-8.43192199503684},{"lat":40.9360621874191,"lng":-8.43184327679757},{"lat":40.936324541323,"lng":-8.43169047080368},{"lat":40.9365134354886,"lng":-8.43151914287114},{"lat":40.9366358666037,"lng":-8.4313107710613},{"lat":40.9367687915575,"lng":-8.43103757246617},{"lat":40.9369262023411,"lng":-8.43076437387104},{"lat":40.9370661227227,"lng":-8.43050969721456},{"lat":40.9372060428079,"lng":-8.43029669492006},{"lat":40.9373669505395,"lng":-8.43011610601819},{"lat":40.9375733316216,"lng":-8.42988458178503},{"lat":40.9377937039295,"lng":-8.42960675270524},{"lat":40.9379651041052,"lng":-8.42941227234938},{"lat":40.9381260099879,"lng":-8.42923631393218},{"lat":40.9381680519302,"lng":-8.42904810343174},{"lat":40.9383049941047,"lng":-8.4290523969509},{"lat":40.9385388861928,"lng":-8.42903091473174},{"lat":40.9387134185164,"lng":-8.42901819796846},{"lat":40.9389564098666,"lng":-8.42899393002519},{"lat":40.9391791576302,"lng":-8.42892567141055},{"lat":40.9394165985922,"lng":-8.42886653149313},{"lat":40.9396061710121,"lng":-8.42886780420099},{"lat":40.9398691040849,"lng":-8.42895322222924},{"lat":40.9400824938965,"lng":-8.42913744067577},{"lat":40.9401578675359,"lng":-8.42924675239943},{"lat":40.9403295044514,"lng":-8.42961607242742},{"lat":40.9404435917299,"lng":-8.42989457588682},{"lat":40.940549370771,"lng":-8.43018557757841},{"lat":40.9405993991242,"lng":-8.43030452126453},{"lat":40.9406332573478,"lng":-8.43047066694342},{"lat":40.9406854714587,"lng":-8.43057072047366},{"lat":40.9407687316161,"lng":-8.4307302656176},{"lat":40.9408356837578,"lng":-8.43081319374209},{"lat":40.9409370290069,"lng":-8.43093872224924},{"lat":40.9411123659802,"lng":-8.43113913417785},{"lat":40.9412059959002,"lng":-8.4312257359009},{"lat":40.9413105314984,"lng":-8.43132242504594},{"lat":40.9413960915335,"lng":-8.43139896134432},{"lat":40.9415093393505,"lng":-8.43150026566161},{"lat":40.941622980055,"lng":-8.43160482376041},{"lat":40.9417080430722,"lng":-8.43168308850547},{"lat":40.941891538661,"lng":-8.4317935299641},{"lat":40.9420251090974,"lng":-8.43181717437041},{"lat":40.9422440633009,"lng":-8.43193525558345},{"lat":40.942346572333,"lng":-8.43200798344835},{"lat":40.9424355932953,"lng":-8.43207114204034},{"lat":40.9427395838045,"lng":-8.43224625975037},{"lat":40.9429790366562,"lng":-8.43234798336155},{"lat":40.9431424282252,"lng":-8.43239273180316},{"lat":40.9434162234703,"lng":-8.43245589592586},{"lat":40.9436120487576,"lng":-8.4325176563391},{"lat":40.9437148935046,"lng":-8.43255087110701},{"lat":40.9438548652311,"lng":-8.43237332408613},{"lat":40.944886312275,"lng":-8.43106495675323},{"lat":40.945136040731,"lng":-8.43074817256776},{"lat":40.9458987053457,"lng":-8.42971842371364},{"lat":40.9458116820235,"lng":-8.42954420361369},{"lat":40.9457986088013,"lng":-8.42944640486914},{"lat":40.9458380971008,"lng":-8.42932319613754},{"lat":40.9458667051602,"lng":-8.42927213273934},{"lat":40.9459262990961,"lng":-8.42916576297374},{"lat":40.9460497545849,"lng":-8.42903593322114},{"lat":40.9461504025198,"lng":-8.42893008564272},{"lat":40.9463823050104,"lng":-8.42867697267791},{"lat":40.9464746498438,"lng":-8.42858429118744},{"lat":40.9465868542971,"lng":-8.4284556369756},{"lat":40.946731933786,"lng":-8.42825152133831},{"lat":40.9468540590968,"lng":-8.42814729672057},{"lat":40.9469662675474,"lng":-8.42810789602193},{"lat":40.9471178857433,"lng":-8.42808054786465},{"lat":40.947277573138,"lng":-8.42804620286779},{"lat":40.9474092337873,"lng":-8.42801779956562},{"lat":40.947548814931,"lng":-8.4279523165553},{"lat":40.9477474739865,"lng":-8.42780929345917},{"lat":40.9478376770503,"lng":-8.42774435253547},{"lat":40.9479625906965,"lng":-8.42760609477419},{"lat":40.9480305928661,"lng":-8.42751592650412},{"lat":40.9481076815799,"lng":-8.42736513574921},{"lat":40.9481563083386,"lng":-8.42727001809662},{"lat":40.9482124542511,"lng":-8.42723373688834},{"lat":40.9490546072284,"lng":-8.42424647157619},{"lat":40.9487701642549,"lng":-8.42132077301543},{"lat":40.9497199974891,"lng":-8.41882062927747}]}]
data_zones = pd.DataFrame(file_zones_json)
data_zones["zone_id"] = data_zones["zone_id"].str.upper()

points_coordinates_lat = [ [ p['lat'] for p in i] for i in list(data_zones['points']) ]
points_coordinates_lng = [ [ p['lng'] for p in i] for i in list(data_zones['points']) ]

zones_polygons = [] 
zones_centroids = []
for zone in range(len(points_coordinates_lat)):
    polygon_geom = Polygon(zip(points_coordinates_lng[zone], points_coordinates_lat[zone]))
    #crs=crs,crs = {'init':"EPSG:4326"}
    polygon = gpd.GeoDataFrame(index=[0], geometry=[polygon_geom])
    
    zones_polygons.append(polygon["geometry"][0])
    zones_centroids.append(polygon.centroid[0])
    
data_zones['geometry'] = zones_polygons
data_zones['centroid'] = zones_centroids

data_zones['coordinates'] = data_zones.apply(lambda x: [ [d["lat"], d["lng"] ] for d in x["points"] ], axis=1)

data_zones_gdf = gpd.GeoDataFrame(data_zones, geometry='geometry')
geojson_zones = gpd.GeoDataFrame(data_zones[['zone_id','geometry']], crs="EPSG:4326").to_json()

coordinates_Porto = [41.1492, -8.6064]

##### Routes/Lines per Zone

To understand how the different lines/routes where distributed among different zones we have plotted the number lines/routes the intersected eacch zone.

In [ ]:
lines_per_zone = database_Rodoviario_Ferrovia_merged.groupby(['line','zone_id']).size().reset_index().rename(columns={0:'count'}).groupby(['zone_id']).size().reset_index().rename(columns={0:'count'})
lines_per_zone_merged = pd.merge(lines_per_zone, data_zones, on=['zone_id'])

title_html = '''
             <h3 align="center" style="font-size:20px"><b> Number of lines/routes per zone </b></h3>
             '''
m = folium.Map(location = coordinates_Porto, zoom_start = 10, tiles="OpenStreetMap")
m.get_root().html.add_child(folium.Element(title_html))


choro_lines_per_zone = folium.Choropleth(geo_data = gpd.GeoDataFrame(lines_per_zone_merged[['zone_id','geometry']], crs="EPSG:4326").to_json(), #geojson_zones
    data = lines_per_zone,
    columns = ['zone_id','count'],
    key_on="feature.properties.zone_id",
    fill_opacity = 0.75,
    fill_color='YlGnBu',
    line_color = 'Black',
    line_opaciy = 0.05,
    legend_name = 'Number Lines',
    name='choropleth',
    bins = 8)

choro_lines_per_zone.add_to(m)
folium.GeoJsonTooltip(['zone_id']).add_to(choro_lines_per_zone.geojson)

m

From the analysis of the figure, we concluded that the central areas around "PRT1" were the ones that were intersected by the highest number of routes/lines.

##### Absolute number of Validations per Zone

In [ ]:
### Data
#All
zones_Rodoviario = database_Rodoviario.groupby(['zone_id']).sum()['Validações'].reset_index()
zones_Rodoviario_merged = pd.merge(zones_Rodoviario, data_zones, on=['zone_id'])
zones_Ferrovia = database_Ferrovia.groupby(['zone_id']).sum()['Validações'].reset_index()
zones_Ferrovia_merged = pd.merge(zones_Ferrovia, data_zones, on=['zone_id'])

coordinates_Porto = [41.1492, -8.6064]

######### All Data
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Absolute number of validations (All Dataset Rodoviário)</b></h3>
             '''
m_Rodoviario = folium.Map(location = coordinates_Porto, zoom_start = 10, tiles="OpenStreetMap")
m_Rodoviario.get_root().html.add_child(folium.Element(title_html))

choro_rod = folium.Choropleth(geo_data = gpd.GeoDataFrame(zones_Rodoviario_merged[['zone_id','geometry']], crs="EPSG:4326").to_json(), #geojson_zones
    data = zones_Rodoviario_merged,
    columns = ['zone_id','Validações'],
    key_on="feature.properties.zone_id",
    fill_opacity = 0.5,
    fill_color='YlGnBu',
    line_color = 'Black',
    line_opaciy = 0.05,
    legend_name = 'Number Validations',
    name='choropleth',
    bins = 5)
choro_rod.add_to(m_Rodoviario)
folium.GeoJsonTooltip(['zone_id']).add_to(choro_rod.geojson)
#m_Rodoviario


title_html = '''
             <h3 align="center" style="font-size:20px"><b>Absolute number of validations (All Dataset Ferroviário)</b></h3>
             '''
m_Ferroviario = folium.Map(location = coordinates_Porto, zoom_start = 10, tiles="OpenStreetMap")
m_Ferroviario.get_root().html.add_child(folium.Element(title_html))


choro_fer = folium.Choropleth(geo_data = gpd.GeoDataFrame(zones_Ferrovia_merged[['zone_id','geometry']], crs="EPSG:4326").to_json(), #geojson_zones
    data = zones_Ferrovia_merged,
    columns = ['zone_id','Validações'],
    key_on="feature.properties.zone_id",
    fill_opacity = 0.5,
    fill_color='YlGnBu',
    line_color = 'Black',
    line_opaciy = 0.05,
    legend_name = 'Number Validations',
    name='choropleth',
    bins = 5)

choro_fer.add_to(m_Ferroviario)
folium.GeoJsonTooltip(['zone_id']).add_to(choro_fer.geojson)
#m_Ferroviario

htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           .format(m_Rodoviario.get_root().render().replace('"', '&quot;'),300,400,
                   m_Ferroviario.get_root().render().replace('"', '&quot;'),300,400))
display(htmlmap)

##### Mean number of validations over time per stop

To understand the daily patterns of movimentation over time, we have represented the average number of validation in each stop over time.

In [ ]:
data_Rodoviario = database_Rodoviario_merged.groupby(['stop_name','line','zone_id','Ano','Hora']).mean()[['Validações','stop_lat','stop_lon']].reset_index()
data_Ferrovia = database_Ferrovia_merged.groupby(['stop_name','zone_id','Ano','Hora']).mean()[['Validações','stop_lat','stop_lon']].reset_index()

coordinates_Porto = [41.1492, -8.6064]

fig1 = px.density_mapbox(data_Rodoviario.sort_values(by=['Hora']), lat='stop_lat', lon='stop_lon', z='Validações', radius=17,
                        center=dict(lat=coordinates_Porto[0], lon=coordinates_Porto[1]), zoom=10,
                        animation_frame = 'Hora', animation_group = 'Validações', 
                        mapbox_style='open-street-map',
                        opacity = 0.7,
                        range_color = [0,data_Rodoviario['Validações'].max()],
                        width = 1000, height = 600,
                        color_continuous_scale= [[0, "blue"],[0.35, "green"],[0.5, "yellow"],[0.75, "orange"],[1, "red"]],
                        title = "Average number of validations per hour (Dataset Rodoviário)")
fig1.show()

fig2 = px.density_mapbox(data_Ferrovia.sort_values(by=['Hora']), lat='stop_lat', lon='stop_lon', z='Validações', radius=20,
                        center=dict(lat=coordinates_Porto[0], lon=coordinates_Porto[1]), zoom=10,
                        animation_frame = 'Hora', animation_group = 'Validações', 
                        mapbox_style='open-street-map',
                        opacity = 0.7,
                        range_color = [0, 200],#data_Ferrovia['Validações'].max()],
                        width = 1000, height = 600,
                        color_continuous_scale= [[0, "blue"],[0.35, "green"],[0.5, "yellow"],[0.75, "orange"],[1, "red"]],
                        title = "Average number of validations per hour (Dataset Ferroviário)")
fig2.show()

Regarding the "Dataset Ferroviário", we have limited the maximum number of validation in the plot to 200 since the number of validation per hour in São Bento's railway was much higher when comparing to the remaining stops, making harder to observer the activity of other stops.

Based on the analysis of the previous plots, we could conclude that the rush hours were the ones where the mean number of validations per hour was the highest. Moreover, the central areas around the city center were the ones with more validations per hour. We could also observe several areas were the average number of validations per hour was substantial, namely: Vila Nova de Gaia, Matosinhos, Maia, and Ermesinde


### Prepare data to develop forecasting models

In [ ]:
# Pre-COVID Dataset (Dataset only contains data from one year. Therefore, we can discard year column)
used_columns = ['Mês','Dia','Hora','Holiday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday',
               'Morning','Afternoon','Night','School']

# Leaving Porto
pre_covid_mod_set_enter_prt = pre_covid_mod_set[pre_covid_mod_set['ZonaSaida'].str.contains('PRT')==True]
post_covid_mod_set_enter_prt = post_covid_mod_set[post_covid_mod_set['ZonaSaida'].str.contains('PRT')==True]

# Entering Porto
pre_covid_mod_set_exit_prt = pre_covid_mod_set[pre_covid_mod_set['ZonaEntrada'].str.contains('PRT')==True]
post_covid_mod_set_exit_prt = post_covid_mod_set[post_covid_mod_set['ZonaEntrada'].str.contains('PRT')==True]

# Sum all validations for each hour
group_columns_enter = ['Mês','Dia','Hora','Holiday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday',
                       'Morning','Afternoon','Night','ZonaEntrada','School']
group_columns_exit = ['Mês','Dia','Hora','Holiday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday',
                      'Morning','Afternoon','Night','ZonaSaida','School']

# Pre-COVID dataset
pre_covid_mod_exit_prt_sum = pre_covid_mod_set_exit_prt.groupby(group_columns_exit).sum()['Validações'].reset_index()
pre_covid_mod_enter_prt_sum = pre_covid_mod_set_enter_prt.groupby(group_columns_enter).sum()['Validações'].reset_index()

# Post-COVID dataset
post_covid_mod_exit_prt_sum = post_covid_mod_set_exit_prt.groupby(group_columns_exit).sum()['Validações'].reset_index()
post_covid_mod_enter_prt_sum = post_covid_mod_set_enter_prt.groupby(group_columns_enter).sum()['Validações'].reset_index()

# Getting all zones out of Porto Metropolitan Area
all_zones_out_prt = ['PV_VC','PVZ2','PVZ3','PVZ4','PVZ5','PVZ6','VCD2','VCD3','VCD4','VCD5','VCD6','VCD7',
                      'VCD8','VCD9','VCD10','VCD11','VCD12','TRF1','TRF2','TRF3','STR1','STR2','STR3','STR4',
                      'STR5','STR6','STR7','STR8','MAI1','MAI2','MAI3','MAI4','MAI5','MTS1','MTS2','MTS3',
                      'VLG1','VLG2','VLG3','PRD1','PRD2','PRD3','PRD4','PRD5','PRD6','PRD7','PRD8','PRD9',
                      'GDM1','GDM2','GDM3','GDM4','GDM5','GDM6','GDM7','GDM8','GDM9','VNG1','VNG2','VNG3',
                      'VNG4','VNG5','VNG6','VNG7','VNG8','VNG9','VNG10','VNG11','VNG12','ESP1','SMF1','SMF2',
                      'SMF3','SMF4','SMF5','SMF6','SMF7','SMF8','SMF9','SMF10','SMF11','SMF12','SJM1','OAZ1',
                      'OAZ2','OAZ3','OAZ4','OAZ5','OAZ6','OAZ7','OAZ8','VCB1','VCB2','VCB3','VCB4','VCB5',
                      'VCB6','VCB7','VCB8','VCB9','ARC1','ARC2','ARC3','ARC4','ARC5','ARC6','ARC7','ARC8',
                      'ARC9','ARC10','ARC11','ARC12','ARC13','ARC14','ARC15','ARC16','ARC17','ARC18','ARC19',
                      'ARC20','ARC21']

# Pre-COVID dataset
pre_covid_mod_exit_prt_sum = pre_covid_mod_exit_prt_sum[pre_covid_mod_exit_prt_sum['ZonaSaida'].str.contains('PRT')==False].reset_index()
pre_covid_mod_enter_prt_sum = pre_covid_mod_enter_prt_sum[pre_covid_mod_enter_prt_sum['ZonaEntrada'].str.contains('PRT')==False].reset_index()

pre_covid_mod_exit_prt_zones = pre_covid_mod_exit_prt_sum['ZonaSaida'].values
pre_covid_mod_enter_prt_zones = pre_covid_mod_enter_prt_sum['ZonaEntrada'].values

pre_covid_mod_exit_prt_zones_df = one_hot_encoding_zones(pre_covid_mod_exit_prt_zones, all_zones_out_prt)
pre_covid_mod_enter_prt_zones_df = one_hot_encoding_zones(pre_covid_mod_enter_prt_zones, all_zones_out_prt)

pre_covid_exit_prt_dataset = pd.concat([pre_covid_mod_exit_prt_sum, pre_covid_mod_exit_prt_zones_df],axis=1).reset_index()
pre_covid_exit_prt_dataset = pre_covid_exit_prt_dataset.drop(columns=['index','level_0','ZonaSaida'])
pre_covid_enter_prt_dataset = pd.concat([pre_covid_mod_enter_prt_sum, pre_covid_mod_enter_prt_zones_df],axis=1).reset_index()
pre_covid_enter_prt_dataset = pre_covid_enter_prt_dataset.drop(columns=['index','level_0','ZonaEntrada'])

# Post-COVID dataset
post_covid_mod_exit_prt_sum = post_covid_mod_exit_prt_sum[post_covid_mod_exit_prt_sum['ZonaSaida'].str.contains('PRT')==False].reset_index()
post_covid_mod_enter_prt_sum = post_covid_mod_enter_prt_sum[post_covid_mod_enter_prt_sum['ZonaEntrada'].str.contains('PRT')==False].reset_index()

post_covid_mod_exit_prt_zones = post_covid_mod_exit_prt_sum['ZonaSaida'].values
post_covid_mod_enter_prt_zones = post_covid_mod_enter_prt_sum['ZonaEntrada'].values

post_covid_mod_exit_prt_zones_df = one_hot_encoding_zones(post_covid_mod_exit_prt_zones, all_zones_out_prt)
post_covid_mod_enter_prt_zones_df = one_hot_encoding_zones(post_covid_mod_enter_prt_zones, all_zones_out_prt)

post_covid_exit_prt_dataset = pd.concat([post_covid_mod_exit_prt_sum, post_covid_mod_exit_prt_zones_df],axis=1).reset_index()
post_covid_exit_prt_dataset = post_covid_exit_prt_dataset.drop(columns=['index','level_0','ZonaSaida'])
post_covid_enter_prt_dataset = pd.concat([post_covid_mod_enter_prt_sum, post_covid_mod_enter_prt_zones_df],axis=1).reset_index()
post_covid_enter_prt_dataset = post_covid_enter_prt_dataset.drop(columns=['index','level_0','ZonaEntrada'])

#### Split datasets in training and test sets

##### Pre-COVID

In [ ]:
# Short Term Data
split_datetime_pre_covid_short = datetime.datetime(2020,3,13)
train_pre_covid_exit_prt_short, test_pre_covid_exit_prt_short = split_dataset(pre_covid_exit_prt_dataset,
                                                                              split_datetime_pre_covid_short)
train_pre_covid_enter_prt_short, test_pre_covid_enter_prt_short = split_dataset(pre_covid_enter_prt_dataset,
                                                                               split_datetime_pre_covid_short)

# Long Term Data
split_datetime_pre_covid_long = datetime.datetime(2020,2,16)
train_pre_covid_exit_prt_long, test_pre_covid_exit_prt_long = split_dataset(pre_covid_exit_prt_dataset,
                                                                            split_datetime_pre_covid_long)
train_pre_covid_enter_prt_long, test_pre_covid_enter_prt_long = split_dataset(pre_covid_enter_prt_dataset,
                                                                              split_datetime_pre_covid_long)

##### Post-COVID

In [ ]:
# Short Term Data
split_datetime_post_covid_short = datetime.datetime(2020,12,29)
train_post_covid_exit_prt_short, test_post_covid_exit_prt_short = split_dataset(post_covid_exit_prt_dataset,
                                                                                split_datetime_post_covid_short)
train_post_covid_enter_prt_short, test_post_covid_enter_prt_short = split_dataset(post_covid_enter_prt_dataset,
                                                                                 split_datetime_post_covid_short)

# Long Term Data
split_datetime_post_covid_long = datetime.datetime(2020,10,1)
train_post_covid_exit_prt_long, test_post_covid_exit_prt_long = split_dataset(post_covid_exit_prt_dataset,
                                                                              split_datetime_post_covid_long)
train_post_covid_enter_prt_long, test_post_covid_enter_prt_long = split_dataset(post_covid_enter_prt_dataset,
                                                                                split_datetime_post_covid_long)

#### Get features and targets for each set

##### Pre-COVID

In [ ]:
# Sorting Data
train_pre_covid_exit_prt_short = train_pre_covid_exit_prt_short.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
train_pre_covid_enter_prt_short = train_pre_covid_enter_prt_short.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
train_pre_covid_exit_prt_long = train_pre_covid_exit_prt_long.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
train_pre_covid_enter_prt_long = train_pre_covid_enter_prt_long.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])

test_pre_covid_exit_prt_short = test_pre_covid_exit_prt_short.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
test_pre_covid_enter_prt_short = test_pre_covid_enter_prt_short.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
test_pre_covid_exit_prt_long = test_pre_covid_exit_prt_long.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
test_pre_covid_enter_prt_long = test_pre_covid_enter_prt_long.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])

# Features
train_pre_covid_exit_prt_short_X = train_pre_covid_exit_prt_short.loc[:, train_pre_covid_exit_prt_short.columns != 'Validações']
train_pre_covid_enter_prt_short_X = train_pre_covid_enter_prt_short.loc[:, train_pre_covid_enter_prt_short.columns != 'Validações']
train_pre_covid_exit_prt_long_X = train_pre_covid_exit_prt_long.loc[:, train_pre_covid_exit_prt_long.columns != 'Validações']
train_pre_covid_enter_prt_long_X = train_pre_covid_enter_prt_long.loc[:, train_pre_covid_enter_prt_long.columns != 'Validações']

test_pre_covid_exit_prt_short_X = test_pre_covid_exit_prt_short.loc[:, test_pre_covid_exit_prt_short.columns != 'Validações']
test_pre_covid_enter_prt_short_X = test_pre_covid_enter_prt_short.loc[:, test_pre_covid_enter_prt_short.columns != 'Validações']
test_pre_covid_exit_prt_long_X = test_pre_covid_exit_prt_long.loc[:, test_pre_covid_exit_prt_long.columns != 'Validações']
test_pre_covid_enter_prt_long_X = test_pre_covid_enter_prt_long.loc[:, test_pre_covid_enter_prt_long.columns != 'Validações']

pre_covid_enter_prt_X = pre_covid_enter_prt_dataset.loc[:, pre_covid_enter_prt_dataset.columns != 'Validações']
pre_covid_exit_prt_X = pre_covid_exit_prt_dataset.loc[:, pre_covid_exit_prt_dataset.columns != 'Validações']

# Targets
train_pre_covid_exit_prt_short_y = train_pre_covid_exit_prt_short['Validações']
train_pre_covid_enter_prt_short_y = train_pre_covid_enter_prt_short['Validações']
train_pre_covid_exit_prt_long_y = train_pre_covid_exit_prt_long['Validações']
train_pre_covid_enter_prt_long_y = train_pre_covid_enter_prt_long['Validações']

test_pre_covid_exit_prt_short_y = test_pre_covid_exit_prt_short['Validações']
test_pre_covid_enter_prt_short_y = test_pre_covid_enter_prt_short['Validações']
test_pre_covid_exit_prt_long_y = test_pre_covid_exit_prt_long['Validações']
test_pre_covid_enter_prt_long_y = test_pre_covid_enter_prt_long['Validações']

pre_covid_exit_prt_y = pre_covid_exit_prt_dataset['Validações']
pre_covid_enter_prt_y = pre_covid_enter_prt_dataset['Validações']

##### Post-COVID

In [ ]:
# Sorting Data
train_post_covid_exit_prt_short = train_post_covid_exit_prt_short.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
train_post_covid_enter_prt_short = train_post_covid_enter_prt_short.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
train_post_covid_exit_prt_long = train_post_covid_exit_prt_long.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
train_post_covid_enter_prt_long = train_post_covid_enter_prt_long.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])

test_post_covid_exit_prt_short = test_post_covid_exit_prt_short.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
test_post_covid_enter_prt_short = test_post_covid_enter_prt_short.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
test_post_covid_exit_prt_long = test_post_covid_exit_prt_long.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])
test_post_covid_enter_prt_long = test_post_covid_enter_prt_long.sort_values(['Mês', 'Dia', 'Hora'],
                                                                          ascending = [True, True, True])

# Features
train_post_covid_exit_prt_short_X = train_post_covid_exit_prt_short.loc[:, train_post_covid_exit_prt_short.columns != 'Validações']
train_post_covid_enter_prt_short_X = train_post_covid_enter_prt_short.loc[:, train_post_covid_enter_prt_short.columns != 'Validações']
train_post_covid_exit_prt_long_X = train_post_covid_exit_prt_long.loc[:, train_post_covid_exit_prt_long.columns != 'Validações']
train_post_covid_enter_prt_long_X = train_post_covid_enter_prt_long.loc[:, train_post_covid_enter_prt_long.columns != 'Validações']

test_post_covid_exit_prt_short_X = test_post_covid_exit_prt_short.loc[:, test_post_covid_exit_prt_short.columns != 'Validações']
test_post_covid_enter_prt_short_X = test_post_covid_enter_prt_short.loc[:, test_post_covid_enter_prt_short.columns != 'Validações']
test_post_covid_exit_prt_long_X = test_post_covid_exit_prt_long.loc[:, test_post_covid_exit_prt_long.columns != 'Validações']
test_post_covid_enter_prt_long_X = test_post_covid_enter_prt_long.loc[:, test_post_covid_enter_prt_long.columns != 'Validações']

post_covid_exit_prt_X = post_covid_exit_prt_dataset.loc[:, post_covid_exit_prt_dataset.columns != 'Validações']
post_covid_enter_prt_X = post_covid_enter_prt_dataset.loc[:, post_covid_enter_prt_dataset.columns != 'Validações']

# Targets
train_post_covid_exit_prt_short_y = train_post_covid_exit_prt_short['Validações']
train_post_covid_enter_prt_short_y = train_post_covid_enter_prt_short['Validações']
train_post_covid_exit_prt_long_y = train_post_covid_exit_prt_long['Validações']
train_post_covid_enter_prt_long_y = train_post_covid_enter_prt_long['Validações']

test_post_covid_exit_prt_short_y = test_post_covid_exit_prt_short['Validações']
test_post_covid_enter_prt_short_y = test_post_covid_enter_prt_short['Validações']
test_post_covid_exit_prt_long_y = test_post_covid_exit_prt_long['Validações']
test_post_covid_enter_prt_long_y = test_post_covid_enter_prt_long['Validações']

post_covid_exit_prt_y = post_covid_exit_prt_dataset['Validações']
post_covid_enter_prt_y = post_covid_enter_prt_dataset['Validações']

### Development of forecasting models

#### Pre-COVID

In [ ]:
pre_covid_exit_prt_short_model = DecisionTreeRegressor().fit(train_pre_covid_exit_prt_short_X,
                                                        train_pre_covid_exit_prt_short_y)
pre_covid_enter_prt_short_model = DecisionTreeRegressor().fit(train_pre_covid_enter_prt_short_X,
                                                         train_pre_covid_enter_prt_short_y)
pre_covid_exit_prt_long_model = DecisionTreeRegressor().fit(train_pre_covid_exit_prt_long_X,
                                                       train_pre_covid_exit_prt_long_y)
pre_covid_enter_prt_long_model = DecisionTreeRegressor().fit(train_pre_covid_enter_prt_long_X,
                                                        train_pre_covid_enter_prt_long_y)

pre_covid_exit_prt_model = DecisionTreeRegressor().fit(pre_covid_exit_prt_X,
                                                  pre_covid_exit_prt_y)
pre_covid_enter_prt_model = DecisionTreeRegressor().fit(pre_covid_enter_prt_X,
                                                   pre_covid_enter_prt_y)

In [ ]:
pred_pre_covid_exit_prt_short_y = pre_covid_exit_prt_short_model.predict(test_pre_covid_exit_prt_short_X)
pred_pre_covid_enter_prt_short_y = pre_covid_enter_prt_short_model.predict(test_pre_covid_enter_prt_short_X)
pred_pre_covid_exit_prt_long_y = pre_covid_exit_prt_long_model.predict(test_pre_covid_exit_prt_long_X)
pred_pre_covid_enter_prt_long_y = pre_covid_enter_prt_long_model.predict(test_pre_covid_enter_prt_long_X)

In the following plots, we present the validations of all zones. We did not divided the plots by zone because it would make the analysis more complex.

In [ ]:
plt.plot(pred_pre_covid_exit_prt_short_y)
plt.plot(test_pre_covid_exit_prt_short_y.values)
plt.title('Short-term Pre-COVID - Exit in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

In [ ]:
plt.plot(pred_pre_covid_enter_prt_short_y)
plt.plot(test_pre_covid_enter_prt_short_y.values)
plt.title('Short-term Pre-COVID - Enter in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

The plots shows that our models could follow the expected patterns (exit and enter in Porto). However, it is seen that they predicted more validations than were expected. These data corresponds to the three days before the COVID lockdown. Therefore, it is expected that the number of validations decreased as we got closer to the start of the lockdown as there were some establishments that started the lockdown earlier.

In [ ]:
plt.plot(pred_pre_covid_exit_prt_long_y)
plt.plot(test_pre_covid_exit_prt_long_y.values)
plt.title('Long-term Pre-COVID - Exit in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

In [ ]:
plt.plot(pred_pre_covid_enter_prt_long_y)
plt.plot(test_pre_covid_enter_prt_long_y.values)
plt.title('Long-term Pre-COVID - Enter in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

In general the long-term models were able to predict the patterns on the data. They could predict the decrease of validations that happened during Carnaval period. It could also predict the decrease of validations after the Carnaval. We found this strange since these were working days and, thus, higher validation peaks were expected. Therefore, we inspected the training data. We found that on the end of January there were also a decrease on the validations. This allowed our models to predict what happened on the end of February. We cannot explain this behavior. It would be great if there was no COVID which allowed us to see if the same behavior would happen in the following months.

The plots also present a difficulty in the prediction of the validations on the period before the lockdown. As explained previously, this could be explained by the fact that some companies decided to suspend their activity before the lockdown.

#### Post-COVID

In [ ]:
post_covid_exit_prt_short_model = DecisionTreeRegressor().fit(train_post_covid_exit_prt_short_X,
                                                        train_post_covid_exit_prt_short_y)
post_covid_enter_prt_short_model = DecisionTreeRegressor().fit(train_post_covid_enter_prt_short_X,
                                                         train_post_covid_enter_prt_short_y)
post_covid_exit_prt_long_model = DecisionTreeRegressor().fit(train_post_covid_exit_prt_long_X,
                                                       train_post_covid_exit_prt_long_y)
post_covid_enter_prt_long_model = DecisionTreeRegressor().fit(train_post_covid_enter_prt_long_X,
                                                        train_post_covid_enter_prt_long_y)

post_covid_exit_prt_model = DecisionTreeRegressor().fit(post_covid_exit_prt_X,
                                                   post_covid_exit_prt_y)
post_covid_enter_prt_model = DecisionTreeRegressor().fit(post_covid_enter_prt_X,
                                                    post_covid_enter_prt_y)

In [ ]:
pred_post_covid_exit_prt_short_y = post_covid_exit_prt_short_model.predict(test_post_covid_exit_prt_short_X)
pred_post_covid_enter_prt_short_y = post_covid_enter_prt_short_model.predict(test_post_covid_enter_prt_short_X)
pred_post_covid_exit_prt_long_y = post_covid_exit_prt_long_model.predict(test_post_covid_exit_prt_long_X)
pred_post_covid_enter_prt_long_y = post_covid_enter_prt_long_model.predict(test_post_covid_enter_prt_long_X)

In [ ]:
plt.plot(pred_post_covid_exit_prt_short_y)
plt.plot(test_post_covid_exit_prt_short_y.values)
plt.title('Short-term Post-COVID - Exit in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

In [ ]:
plt.plot(pred_post_covid_enter_prt_short_y)
plt.plot(test_post_covid_enter_prt_short_y.values)
plt.title('Short-term Post-COVID - Enter in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

The models were able to follow the pattern. The difference on the last data could be related to the fact that these data is related to the last day of the year when some workers usually do not work.

In [ ]:
plt.plot(pred_post_covid_exit_prt_long_y)
plt.plot(test_post_covid_exit_prt_long_y.values)
plt.title('Long-term Post-COVID - Exit in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

In [ ]:
plt.plot(pred_post_covid_enter_prt_long_y)
plt.plot(test_post_covid_enter_prt_long_y.values)
plt.title('Long-term Post-COVID - Enter in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

The models were able to follow the pattern. The last data corresponds to christmas time when there is usually a decrease on the usage of public transportation.

#### If there was no COVID

In [ ]:
pre_covid_exit_model = DecisionTreeRegressor().fit(pre_covid_exit_prt_X,
                                                   pre_covid_exit_prt_y)

pre_covid_enter_model = DecisionTreeRegressor().fit(pre_covid_enter_prt_X,
                                                    pre_covid_enter_prt_y)

pred_post_covid_exit_prt_y = pre_covid_exit_model.predict(post_covid_exit_prt_X)
pred_post_covid_enter_prt_y = pre_covid_enter_model.predict(post_covid_enter_prt_X)

In [ ]:
plt.plot(pred_post_covid_exit_prt_y)
plt.plot(post_covid_exit_prt_y.values)
plt.title('Post-COVID - Exit in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

In [ ]:
plt.plot(pred_post_covid_enter_prt_y)
plt.plot(post_covid_enter_prt_y.values)
plt.title('Post-COVID - Enter in Porto')
plt.ylabel('Validations')
plt.legend(['Predicted','Expected'])

As expected, the model predicts the flow of people in the post-COVID period taking into account the pre-COVID distribution. However, after the lockdown the flow of people decreased due to the restrictions imposed on the population and to the change from on-site work to remote work which happened in some jobs.

### Evaluating and understanding the predictions obtained from the forecasting models

#### Models Error Evaluation

##### Pre-COVID

In [ ]:
pre_covid_exit_short_rrmse = rrmse_metric(test_pre_covid_exit_prt_short_y.values,pred_pre_covid_exit_prt_short_y)
pre_covid_enter_short_rrmse = rrmse_metric(test_pre_covid_enter_prt_short_y.values,pred_pre_covid_enter_prt_short_y)
pre_covid_exit_long_rrmse = rrmse_metric(test_pre_covid_exit_prt_long_y.values,pred_pre_covid_exit_prt_long_y)
pre_covid_enter_long_rrmse = rrmse_metric(test_pre_covid_enter_prt_long_y.values,pred_pre_covid_enter_prt_long_y)

print(f'RRMSE (Short-term Pre-COVID - Exit in Porto): {pre_covid_exit_short_rrmse}%')
print(f'RRMSE (Short-term Pre-COVID - Enter in Porto): {pre_covid_enter_short_rrmse}%')
print(f'RRMSE (Long-term Pre-COVID - Exit in Porto): {pre_covid_exit_long_rrmse}%')
print(f'RRMSE (Long-term Pre-COVID - Enter in Porto): {pre_covid_enter_long_rrmse}%')

RRMSE for short-term pre-COVID is higher than for long-term. These could be related to the COVID effects that started some days before the lockdown.

##### Post-COVID

In [ ]:
post_covid_exit_short_rrmse = rrmse_metric(test_post_covid_exit_prt_short_y.values,pred_post_covid_exit_prt_short_y)
post_covid_enter_short_rrmse = rrmse_metric(test_post_covid_enter_prt_short_y.values,pred_post_covid_enter_prt_short_y)
post_covid_exit_long_rrmse = rrmse_metric(test_post_covid_exit_prt_long_y.values,pred_post_covid_exit_prt_long_y)
post_covid_enter_long_rrmse = rrmse_metric(test_post_covid_enter_prt_long_y.values,pred_post_covid_enter_prt_long_y)

print(f'RRMSE (Short-term Post-COVID - Exit in Porto): {post_covid_exit_short_rrmse}%')
print(f'RRMSE (Short-term Post-COVID - Enter in Porto): {post_covid_enter_short_rrmse}%')
print(f'RRMSE (Long-term Post-COVID - Exit in Porto): {post_covid_exit_long_rrmse}%')
print(f'RRMSE (Long-term Post-COVID - Enter in Porto): {post_covid_enter_long_rrmse}%')

RRMSE for short-term post-COVID is higher than for long-term. These could be related to the last day of the year when there are people that usually do not work despite being considered a working day (Thursday).

##### If there was no COVID

In [ ]:
post_covid_exit_all_rrmse = rrmse_metric(post_covid_exit_prt_y.values,pred_post_covid_exit_prt_y)
post_covid_enter_all_rrmse = rrmse_metric(post_covid_enter_prt_y.values,pred_post_covid_enter_prt_y)

print(f'RRMSE (Post-COVID - Exit in Porto): {post_covid_exit_all_rrmse}%')
print(f'RRMSE (Post-COVID - Enter in Porto): {post_covid_enter_all_rrmse}%')

RRMSE is high because as seen earlier there are clear differences in the pre-COVID and post-COVID data.

#### Shapley Values (Exit in Porto)

In [ ]:
display_features = 10
plot_feature_importances(pre_covid_exit_prt_short_model, test_pre_covid_exit_prt_short_X,display_features)

#### Shapley Values (Enter in Porto)

In [ ]:
plot_feature_importances(pre_covid_enter_prt_short_model, test_pre_covid_enter_prt_short_X,display_features)

Shapley values suggests that the hour is the most important feature to forecast the number of validations. It could be seen that the rush hours contribute positively and night hours contribute negatively. Furthermore, as expected, there is a clear decrease on the number of validations on saturdays and sundays (negative contribution). The most important zones that appear in the enter models are almost the same that appear in the exit models with the exception of the PV-VC (exit in Porto) and VCD8 (enter in Porto).

### Forecasting flow - Final model

As our final model, we propose our forescating model to predict the future flow of people in the city of Porto. We can also represent this flow in geographical terms, as the global number of validations betweens areas (as presented in the following figures for the pre-Covid period example).

In the map visualization we present the forecasted global number of validations per zones pairs. With this information we can easily see which zones are under most stress, and, consequently, allocate more resources to those areas, to improve the flow of people.

In [ ]:
# Flow representation obtained from the model prediction (Example) - Exit Porto
dataframe_plot = create_Dataframe(test_pre_covid_exit_prt_short, pred_pre_covid_exit_prt_short_y, data_zones)

title_html = '''
         <h3 align="center" style="font-size:20px"><b> Short-term Pre-COVID - Exit Porto (Flux between zones) </b></h3>
           '''
    
m = folium.Map(location = coordinates_Porto, zoom_start = 10, tiles="cartodbpositron", opacity=0.3) #tiles="OpenStreetMap"
geo = folium.GeoJson(data = geojson_zones)
geo.add_to(m) 
for i in range(dataframe_plot.shape[0]):
    plugins.PolyLineOffset([list(coordinates_Porto), list(dataframe_plot["centroid"][i].coords[0][::-1])],
                           color = "#f00", 
                           opacity = float(dataframe_plot["ValidaçõesNormalized"][i]), 
                           offset = 1,
                           weight = 2).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
folium.GeoJsonTooltip(['zone_id']).add_to(geo)
   
m


In [ ]:
# Flow representation obtained from the model prediction (Example) - Enter Porto
dataframe_plot = create_Dataframe(test_pre_covid_enter_prt_short, pred_pre_covid_enter_prt_short_y, data_zones)

title_html = '''
         <h3 align="center" style="font-size:20px"><b> Short-term Pre-COVID - Enter Porto (Flux between zones) </b></h3>
           '''
    
m = folium.Map(location = coordinates_Porto, zoom_start = 10, tiles="cartodbpositron", opacity=0.3) #tiles="OpenStreetMap"
geo = folium.GeoJson(data = geojson_zones)
geo.add_to(m) 
for i in range(dataframe_plot.shape[0]):
    plugins.PolyLineOffset([list(coordinates_Porto), list(dataframe_plot["centroid"][i].coords[0][::-1])],
                           color = "#f00", 
                           opacity = float(dataframe_plot["ValidaçõesNormalized"][i]), 
                           offset = 1,
                           weight = 2).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
folium.GeoJsonTooltip(['zone_id']).add_to(geo)
   
m

## 🖼️ Visualisations

We have already presented all the figures above throughout our data analysis.

## 👓 References
Datasets provided by the WDL 2022 Organisation:
- https://wdl-data.fra1.digitaloceanspaces.com/porto/InterMunicipality.zip

List all of the external links (even if they are already linked above), such as external datasets, papers, blog posts, code repositories and any other materials.

## ⏭️ Appendix
Add here any code, images or text that you still find relevant, but that was too long to include in the main report. This section is optional.